In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Data Processing**

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/kaggle/input/bus-stops/bus_stops_data.csv')

In [ ]:
print(df.columns)
df.shape

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates(subset='name', keep='first')

In [ ]:
df.info()

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
df['WARD'].value_counts()

In [ ]:
df.drop(['BEST_APP_1', 'BEST_BUS_S', 'BEST_HELP_',
       'OBJECTID', 'TWITTER_HA', 'WEBSITE',
       'altitude', 'BUS_DEPOT_','ADDRESS','WARD'],axis=1,inplace=True)

From the above observation, we can rely on the fact that each path will start and end on a bus depot

In [ ]:
df

### Label Encoding for styleUrl:
* 0 --> Bus Stop
* 1 --> Bus Depot

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Sample data (replace with your actual data)
data = ['#mumbai_best_stops_style', '#mumbai_bus_depots_style1']

# Create a LabelEncoder object
encoder = LabelEncoder()

# Encode the data
df['styleUrl'] = encoder.fit_transform(df['styleUrl'])

df


In [ ]:
df.info()

In [ ]:
df['name'].nunique()

# Parsing the KML file of the ward centres

In [ ]:
import xml.etree.ElementTree as ET

# Parse the KML file
tree = ET.parse('/kaggle/input/ward-info/mumbai-ward-offices.kml')
root = tree.getroot()

# Define the KML namespace
kml_namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

# Extract the Placemark elements
placemarks = root.findall('.//kml:Placemark', kml_namespace)

# Convert the Placemark elements to a DataFrame
ward_centers_data = []
for placemark in placemarks:
    name = placemark.find('kml:name', kml_namespace).text
    coordinates = placemark.find('.//kml:coordinates', kml_namespace).text.split(',')
    lon, lat = map(float, coordinates[:2])
    ward_centers_data.append({'Name': name, 'longitude': lon, 'latitude': lat})

ward_centers = pd.DataFrame(ward_centers_data)
ward_centers = ward_centers.replace(r'\n', '', regex=True)

In [ ]:
ward_centers['Name'] = ward_centers['Name'].str.strip()

In [ ]:
new_wards = pd.DataFrame({
    'Name': ['U/N', 'L/N'],
    'latitude': [19.126191, 19.046741],
    'longitude': [73.004432, 73.017578]
}, index=[0, 1])
# Lat:  Long:  Ward: U/N
ward_centers = pd.concat([ward_centers, new_wards], ignore_index=True)

# Assigning Bus Stops to Nearest Ward Centers

- The code calculates the great circle distance between two points on the earth using the Haversine formula. This is used to determine the distance between a bus stop and a ward center.
- It iterates over each bus stop and calculates its distance to every ward center. It keeps track of the minimum distance and the corresponding ward center.
- Finally, it assigns each bus stop to the ward center that is nearest to it. The result is a DataFrame where each bus stop is associated with the nearest ward center.

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

df['ward'] = ''
for i, bus_stop in df.iterrows():
    bus_stop_lon = bus_stop['longitude']
    bus_stop_lat = bus_stop['latitude']
    min_distance = float('inf')
    nearest_ward = None
    for j, ward_center in ward_centers.iterrows():
        ward_center_lon = ward_center['longitude']
        ward_center_lat = ward_center['latitude']
        distance = haversine(bus_stop_lon, bus_stop_lat, ward_center_lon, ward_center_lat)
        if distance < min_distance:
            min_distance = distance
            nearest_ward = ward_center['Name']
    df.at[i, 'ward'] = nearest_ward
    

In [ ]:
df['ward'] = df['ward'].str.strip()

In [ ]:
import pandas as pd

def update_ward(latitude, longitude, df, new_ward):
    """
    Updates the 'ward' column of the DataFrame `df` for rows where the 'latitude' and 'longitude'
    match the given values.

    Args:
        latitude (float): The latitude value to match.
        longitude (float): The longitude value to match.
        df (pd.DataFrame): The DataFrame containing the data.
        new_ward (str): The new value for the 'ward' column.

    Returns:
        pd.DataFrame: The updated DataFrame.
    """
    # Create a boolean mask to identify rows with matching latitude and longitude
    mask = (df['latitude'] == latitude) & (df['longitude'] == longitude)

    # Update the 'ward' column for the matching rows
    df.loc[mask, 'ward'] = new_ward

    return df

In [ ]:
def parse_and_update(input_str, df, new_ward):
    lines = input_str.split('\n')
#     print(lines)
    for line in lines:
        words = line.split()
        if len(words) >= 4:  # Ensure the line has at least 4 words
            latitude = float(words[1])
            longitude = float(words[3])
            df = update_ward(latitude, longitude, df, new_ward)
#         else:
#             print(words)
    return df

In [ ]:
input_str1 = """Lat 19.15238 Long 72.790378
Lat 19.151946 Long 72.790616
Lat 19.150321 Long 72.790988
Lat 19.150131 Long 72.79095
Lat 19.148224 Long 72.7908
Lat 19.147773 Long 72.790734
Lat 19.144592 Long 72.790224
Lat 19.144213 Long 72.790225
Lat: 19.144594 Long: 72.801859"""
input_str2 = """Lat 19.153734 Long 72.789369 
Lat 19.153896 Long 72.789178 
Lat 19.15572 Long 72.788036 
Lat 19.155828 Long 72.787893 
Lat 19.157399 Long 72.786618 
Lat 19.157598 Long 72.78657 
Lat 19.15898 Long 72.78615 
Lat 19.161654 Long 72.786375 
Lat 19.161726 Long 72.786547 
Lat 19.163184 Long 72.789549 
Lat 19.16341 Long 72.789872"""
df = parse_and_update(input_str1, df, 'K/W')
df = parse_and_update(input_str2, df, 'P/N')

# Visualizing Mumbai's Ward Centers and Bus Stops

- The code's primary purpose is to **visualize geographical data** on a map of Mumbai. It specifically targets the locations of ward centers and bus stops.
- A unique color is assigned to each ward center using a color palette generated with the seaborn library. This color coding helps in **distinguishing each ward visually**. All bus stops within a ward are represented with the same color as the ward center.
- The final result is an **interactive map** that provides a visual representation of the distribution of bus stops across different wards in Mumbai. This map can be used to explore and understand the geographical distribution of bus stops in the city.

In [ ]:
from sklearn.cluster import KMeans

# The number of current wards
n_wards = len(ward_centers)

# The maximum number of bus stops per ward
max_bus_stops = 128

while True:
    # Initialize the K-means model
    model = KMeans(n_clusters=n_wards)

    # Fit the model to the bus stop data
    model.fit(df[['longitude', 'latitude']])

    # Assign each bus stop to the nearest ward
    df['ward'] = model.predict(df[['longitude', 'latitude']])

    # Count the number of bus stops in each ward
    bus_stop_counts = df['ward'].value_counts()

    # If all wards have 128 or fewer bus stops, break the loop
    if all(count <= max_bus_stops for count in bus_stop_counts):
        break

    # Otherwise, increment the number of wards by 1
    n_wards += 1

# Create a mapping from cluster numbers to ward names
ward_names = {i: f'NW-{i+1}' for i in range(n_wards)}

# The old ward names
old_wards = set(ward_centers['Name'])

# Apply the nomenclature to new wards only
df['ward'] = df['ward'].apply(lambda x: x if x in old_wards else ward_names[x])

In [ ]:
# Create a DataFrame for the new ward centres
new_ward_centres_df = pd.DataFrame(model.cluster_centers_, columns=['longitude', 'latitude'])

# Add the ward names
new_ward_centres_df['name'] = pd.Series(ward_names)

# Reorder the columns
new_ward_centres_df = new_ward_centres_df[['name', 'longitude', 'latitude']]


In [ ]:
new_ward_centres_df

In [ ]:
import folium
import seaborn as sns
import matplotlib.colors as mcolors

# Assuming 'new_ward_centres_df' and 'df' are pandas DataFrames defined earlier in your code

# Create a map centered around Mumbai
mumbai_map = folium.Map(location=[19.0760, 72.8777], zoom_start=11)

# Create a color palette
palette = sns.color_palette("hls", len(new_ward_centres_df))

# Convert the color palette to hexadecimal format
hex_palette = [mcolors.rgb2hex(color) for color in palette]

# Plot the new ward centers
for _, ward_center in new_ward_centres_df.iterrows():
    folium.CircleMarker(
        location=[ward_center['latitude'], ward_center['longitude']],
        radius=10,  # Larger radius for ward centers
        color='black',  # Different color for ward centers
        fill=True,
        fill_color='black',
        fill_opacity=0.6,
        popup=folium.Popup('Ward Center: ' + ward_center['name'], parse_html=True)
    ).add_to(mumbai_map)

# Plot the bus stops
# for _, bus_stop in df.iterrows():
#     matching_wards = new_ward_centres_df[new_ward_centres_df['name'] == bus_stop['ward']]
#     if not matching_wards.empty:
#         ward_index = matching_wards.index[0]
#         s = 'Lat: ' + str(bus_stop['latitude']) + '\nLong: ' + str(bus_stop['longitude']) +"\nWard: " + bus_stop['ward']
#         folium.CircleMarker(
#             location=[bus_stop['latitude'], bus_stop['longitude']],
#             radius=3,
#             color=hex_palette[ward_index],
#             fill=True,
#             fill_color=hex_palette[ward_index],
#             fill_opacity=0.6,
#             popup=folium.Popup(s, parse_html=True)
#         ).add_to(mumbai_map)
#     else:
#         print(f"No matching ward found for bus stop in ward: {bus_stop['ward']}")

# Display the map
mumbai_map

# Implementation of the data into the algorithm 

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/kaggle/input/updated-dataset/updated_bus_stops.csv')

In [ ]:
df

In [ ]:
df['ward'].value_counts()

In [ ]:
# import requests
# import folium
# import pandas as pd


# # Reset the indices of 'filtered_df' so they are continuous and start from 0
# filtered_df.reset_index(drop=True, inplace=True)

# # Now, proceed with the rest of your code as before
# # Initialize a map centered around Mumbai
# mumbai_map = folium.Map(location=[19.0760, 72.8777], zoom_start=12)

# # Define multiple paths as outputs from the VRP function
# paths = [[6, 2, 10], [4, 3, 1, 7, 0, 9, 5, 8, 14, 16, 13, 15, 11, 12]]

# # Colors for different paths
# path_colors = ['blue', 'green']

# for path_index, path in enumerate(paths):
#     # Add markers for each location
#     for index, row in filtered_df.iterrows():
#         # Choose the marker color based on whether the location is in the current path
#         color = 'red'  # Default color for locations not in the current path
#         if index in path:
#             color = path_colors[path_index % len(path_colors)]  # Cycle through path_colors
            
#             # Create the marker
#             marker = folium.Marker(
#                 location=[row['latitude'], row['longitude']],
#                 icon=folium.Icon(color=color),
#                 popup=row['name']
#             )
            
#             # Add the marker to the map
#             marker.add_to(mumbai_map)

#     # Define the points for the OSRM API for the current path
#     points = [filtered_df.loc[i, ['longitude', 'latitude']].tolist() for i in path]

#     # Convert the points to a string with the format required by the OSRM API
#     points_str = ';'.join([f'{point[0]},{point[1]}' for point in points])

#     # Send a request to the OSRM API
#     response = requests.get(f'http://router.project-osrm.org/route/v1/driving/{points_str}?overview=full&geometries=geojson')

#     # Parse the JSON response
#     data = response.json()

#     # Get the coordinates of the route for the current path
#     route = data['routes'][0]['geometry']['coordinates']

#     # Convert the coordinates from (longitude, latitude) to (latitude, longitude)
#     route = [(lat, lon) for lon, lat in route]

#     # Draw the path with the color corresponding to the current path
#     folium.PolyLine(route, color=path_colors[path_index % len(path_colors)], weight=2.5, opacity=1).add_to(mumbai_map)

# # Display the map
# # mumbai_map

In [ ]:
ward_data = {    'NW-1': {'distance_matrix': [        [            0,            1118,            2733,            1156,            2995,            3625,            2274,            2620,            4898,            3071,            2748,            2236,            2134,            3487,            957,            667,            1081,            1809,            3241,            3151,            1678,            963,            1320,            2065,            2719,            1282,            1107,            1323,            1655,            2872,            1371,            2581,            1598,            1658,            2520,            553,            666,            1210,            2028,            3712,            935,            1306,            4012,            1319,            1120,            1367,            1992,            1806,            1533,            961,            1437,            2207,            2920,            3398,            ],        [            754,            0,            2166,            650,            1720,            3058,            1707,            1345,            4707,            4195,            3483,            2971,            2869,            4222,            465,            1421,            478,            1206,            3976,            3886,            2432,            1731,            2055,            2819,            3473,            2036,            1861,            2077,            2409,            3607,            2125,            3335,            995,            1091,            1953,            1307,            1401,            643,            2782,            4836,            1502,            1115,            4747,            752,            1874,            797,            2746,            1239,            2268,            1759,            2172,            2961,            3655,            4133,            ],        [            2444,            1690,            0,            2340,            4302,            1779,            404,            4575,            7406,            6894,            5173,            4661,            4559,            5912,            2155,            3111,            2168,            2197,            5666,            5576,            4122,            3407,            3745,            4509,            5163,            3726,            3551,            3767,            4099,            5297,            4154,            5025,            2308,            1169,            650,            2997,            3091,            1782,            4472,            7535,            3192,            2805,            6437,            1431,            3564,            2487,            4436,            1331,            3958,            3449,            3862,            4651,            5345,            5823,            ],        [            1104,            730,            2516,            0,            1134,            3408,            2057,            759,            3216,            2704,            2910,            2398,            2860,            3649,            456,            1241,            569,            1297,            3403,            3313,            2782,            1802,            2046,            3169,            3823,            2386,            2211,            2427,            2759,            3034,            2455,            3685,            1086,            1441,            2303,            1657,            1392,            1597,            3132,            3345,            1493,            1106,            4174,            1102,            2157,            211,            3096,            1589,            2259,            1806,            2163,            3311,            3082,            3560,            ],        [            2701,            2382,            3212,            1899,            0,            2625,            3616,            1016,            3847,            3335,            3541,            3029,            3350,            4280,            2051,            2468,            1928,            1957,            4034,            3944,            4324,            3104,            3273,            4711,            5365,            3928,            3753,            3969,            4301,            3665,            3682,            5227,            1746,            4381,            3862,            3254,            2619,            2257,            4674,            3976,            2720,            2333,            4805,            2627,            3813,            2312,            4638,            3114,            3486,            3444,            3390,            4853,            3713,            4191,            ],        [            3031,            2277,            587,            2927,            4020,            0,            991,            4293,            7124,            6612,            5760,            5248,            5146,            6499,            2742,            3698,            2755,            2784,            6253,            6163,            4709,            4008,            4332,            5096,            5750,            4313,            4138,            4354,            4686,            5884,            4741,            5612,            2895,            1756,            1237,            3584,            3678,            2369,            5059,            7253,            3779,            3392,            7024,            2018,            4151,            3074,            5023,            1918,            4545,            4036,            4449,            5238,            5932,            6410,            ],        [            2040,            1286,            1051,            1936,            4466,            1943,            0,            4739,            5993,            5481,            4769,            4257,            4155,            5508,            1751,            2707,            1764,            1793,            5262,            5172,            3718,            3017,            3341,            4105,            4759,            3322,            3147,            3363,            3695,            4893,            3750,            4621,            1904,            765,            246,            2593,            2687,            1378,            4068,            6122,            2788,            2401,            6033,            1027,            3160,            2083,            4032,            927,            3554,            3045,            3458,            4247,            4941,            5419,            ],        [            2353,            2034,            3587,            1551,            375,            3000,            3991,            0,            3499,            2987,            3193,            2681,            3002,            3932,            1703,            2120,            1580,            1609,            3686,            3596,            3976,            2756,            2925,            4363,            5017,            3580,            3405,            3621,            3953,            3317,            3334,            4879,            1398,            2618,            4237,            2862,            2271,            1909,            4326,            3628,            2372,            1985,            4457,            2279,            3465,            1964,            4290,            2766,            3138,            3096,            3042,            4505,            3365,            3843,            ],        [            3794,            4081,            5740,            3598,            3484,            6109,            5281,            3109,            0,            1931,            1605,            1490,            2018,            2364,            3750,            3416,            3627,            3656,            2098,            2008,            4573,            3175,            3333,            4786,            5614,            3731,            4002,            4218,            4376,            2126,            3742,            5476,            3445,            4665,            5527,            3280,            3226,            3956,            4923,            2572,            3684,            3483,            2889,            4326,            3873,            4011,            4713,            4813,            3246,            3519,            3450,            4928,            1777,            2255,            ],        [            4757,            4440,            6099,            3957,            3843,            6468,            5640,            3468,            2478,            0,            4083,            3968,            4496,            4842,            4109,            4526,            3986,            4015,            4576,            4486,            6382,            5162,            5331,            6769,            7423,            5986,            5811,            6027,            6359,            4376,            5740,            7285,            3804,            5024,            5886,            5310,            4677,            4315,            6732,            641,            4778,            4391,            5367,            4685,            5871,            4370,            6696,            5172,            5544,            5502,            5448,            6911,            4255,            4733,            ],        [            3767,            4325,            5984,            3919,            3728,            6353,            5525,            3353,            4653,            2175,            0,            1481,            1991,            2284,            4069,            3389,            3871,            3900,            647,            557,            4372,            3148,            3306,            4759,            5413,            3704,            3801,            4017,            4349,            1946,            3715,            5275,            3689,            4909,            5771,            3253,            3199,            4200,            4722,            2816,            3657,            4056,            1637,            4570,            3846,            4130,            4686,            5057,            3219,            3492,            3423,            4901,            172,            422,            ],        [            3387,            3070,            4729,            2587,            2473,            5098,            4270,            2098,            3398,            920,            512,            0,            1007,            1251,            2739,            2405,            2616,            2645,            1005,            915,            3562,            2164,            2322,            3775,            4603,            2720,            2991,            3207,            3365,            785,            2731,            4465,            2434,            3654,            4516,            2269,            2215,            2945,            3912,            1561,            2673,            2472,            1776,            3315,            2862,            3000,            3702,            3802,            2235,            2508,            2439,            3917,            684,            1162,            ],        [            2211,            3227,            4599,            2782,            2630,            5255,            4140,            2255,            3829,            1351,            943,            431,            0,            1682,            2583,            1833,            2773,            2802,            1436,            1346,            2816,            1592,            1750,            3203,            3857,            2148,            2245,            2461,            2793,            1067,            2159,            3719,            2591,            3524,            4386,            1697,            1643,            3102,            3166,            1992,            2101,            2500,            2207,            3185,            2290,            3157,            3130,            3672,            1663,            1936,            1867,            3345,            1115,            1593,            ],        [            3634,            4192,            5851,            3709,            3595,            6220,            5392,            3220,            4520,            2042,            2129,            1348,            1858,            0,            3861,            3256,            3738,            3767,            2230,            2184,            4413,            5143,            5301,            4626,            8641,            5699,            3842,            6186,            4216,            3630,            3582,            8821,            3556,            4776,            5638,            3120,            3066,            4067,            9225,            2683,            5652,            6051,            1497,            4437,            5577,            4122,            4553,            4924,            5214,            3359,            5418,            6360,            4109,            2387,            ],        [            650,            447,            2062,            582,            1652,            2954,            1603,            1277,            3734,            3222,            3398,            2886,            2784,            4137,            0,            1317,            410,            1138,            3891,            3801,            2328,            1613,            1970,            2715,            3369,            1932,            1757,            1973,            2305,            3522,            2021,            3231,            927,            987,            1849,            1203,            1316,            539,            2678,            4447,            1381,            1047,            4662,            648,            1770,            729,            2642,            1135,            2183,            1611,            2087,            2857,            3570,            4048,            ],        [            859,            1124,            2739,            530,            2356,            3631,            2280,            1981,            4259,            3347,            2939,            2427,            2325,            3678,            680,            0,            1099,            1827,            3432,            3342,            2320,            1060,            1511,            2707,            3361,            1924,            1749,            1965,            2297,            3063,            1920,            3223,            1616,            1664,            2526,            1201,            857,            1216,            2670,            3988,            958,            667,            4203,            1325,            1773,            741,            2634,            1812,            1724,            1440,            1628,            2849,            3111,            3589,            ],        [            949,            1153,            2812,            845,            1485,            3704,            2353,            1110,            3567,            3055,            3678,            3166,            3064,            4417,            660,            1616,            0,            728,            4171,            4081,            2627,            1912,            2250,            3014,            3668,            2231,            2056,            2272,            2604,            3802,            2320,            3530,            517,            1737,            2599,            1502,            1596,            1028,            2977,            3696,            1697,            1310,            4942,            1398,            2069,            562,            2941,            1885,            2463,            1910,            2367,            3156,            3850,            4328,            ],        [            1179,            425,            2084,            1075,            2205,            2976,            1625,            1830,            4287,            3065,            3908,            3396,            3294,            4647,            890,            1846,            898,            0,            4401,            4311,            2857,            2142,            2480,            3244,            3898,            2461,            2286,            2502,            2834,            4032,            2550,            3760,            404,            1009,            1871,            1732,            1826,            300,            3207,            4957,            1927,            1540,            5172,            670,            2299,            1282,            3171,            1157,            2693,            2140,            2597,            3386,            4080,            4558,            ],        [            3737,            4434,            6098,            3889,            3698,            6462,            5495,            3323,            4623,            2145,            579,            1451,            1961,            2254,            4039,            3359,            3841,            4009,            0,            90,            4342,            3118,            3276,            4729,            5383,            3674,            3771,            3987,            4319,            1916,            3685,            5245,            3798,            5023,            5885,            3223,            3169,            4309,            4692,            2786,            3627,            4026,            1607,            4684,            3816,            4100,            4656,            5171,            3189,            3462,            3393,            4871,            407,            157,            ],        [            3647,            4344,            6008,            3799,            3747,            6372,            5549,            3372,            4533,            2055,            669,            1361,            1871,            2164,            3949,            3269,            3890,            3919,            90,            0,            4252,            3028,            3186,            4639,            5293,            3584,            3681,            3897,            4229,            1826,            3595,            5155,            3708,            4933,            5795,            3133,            3079,            4219,            4602,            2696,            3537,            3936,            1517,            4594,            3726,            4010,            4566,            5081,            3099,            3372,            3303,            4781,            497,            247,            ],        [            1765,            2536,            4151,            2671,            3795,            5043,            3692,            3420,            5877,            4153,            3830,            3318,            3216,            4569,            2383,            2097,            2499,            3227,            4323,            4233,            0,            1516,            1394,            910,            1297,            776,            601,            426,            888,            3954,            1421,            1159,            3016,            3076,            3938,            1174,            1748,            2628,            606,            4794,            2365,            2743,            5094,            2737,            1205,            2818,            551,            3224,            1607,            1022,            1511,            634,            4002,            4480,            ],        [            1084,            1857,            3472,            1236,            3062,            4364,            3013,            2687,            4965,            2404,            2081,            1569,            1467,            2820,            1456,            706,            1820,            2548,            2574,            2484,            1689,            0,            817,            2076,            2730,            1021,            1118,            1334,            1666,            2205,            1013,            2592,            2337,            2397,            3259,            570,            516,            1949,            2039,            3045,            974,            1373,            3345,            2058,            1156,            1447,            2003,            2545,            1030,            809,            934,            2218,            2253,            2731,            ],        [            1655,            2377,            3992,            1768,            3594,            4884,            3533,            3219,            5497,            2872,            2549,            2037,            1935,            3288,            2027,            1238,            2340,            3068,            3042,            2952,            1566,            532,            0,            1551,            2607,            1051,            995,            1211,            1141,            2673,            409,            2177,            2857,            2917,            3779,            1095,            1053,            2469,            1916,            3513,            1506,            1905,            3813,            2578,            540,            1979,            1478,            3065,            347,            549,            117,            1693,            2721,            3199,            ],        [            2487,            3258,            4873,            3393,            5154,            5765,            4414,            4779,            7057,            4875,            4552,            4040,            3938,            5291,            3105,            2819,            3221,            3949,            5045,            4955,            722,            2238,            2116,            0,            683,            1498,            1323,            1148,            821,            4676,            1829,            545,            3738,            3798,            4660,            1896,            2470,            3350,            946,            5516,            3087,            3465,            5816,            3459,            1782,            3540,            484,            3946,            2334,            1744,            2104,            269,            4724,            5202,            ],        [            3221,            3992,            5607,            4127,            5888,            6499,            5148,            5513,            7791,            5609,            5286,            4774,            4672,            8694,            3839,            3553,            3955,            4683,            5779,            5689,            1456,            2972,            2850,            1340,            0,            2232,            2057,            1882,            1552,            8226,            2877,            659,            4472,            4532,            5394,            2630,            3204,            4084,            1060,            6250,            3821,            4199,            9148,            4193,            2661,            4274,            1215,            4680,            3063,            2478,            2967,            1000,            5458,            5936,            ],        [            1130,            1901,            3516,            1947,            3160,            4408,            3057,            2785,            5676,            3494,            3171,            2659,            2557,            3910,            1724,            1438,            1864,            2592,            3664,            3574,            859,            740,            618,            1246,            1900,            0,            288,            504,            836,            3295,            656,            1762,            2381,            2441,            3303,            619,            1089,            1993,            1209,            4135,            1706,            2084,            4435,            2102,            405,            2183,            1173,            2589,            831,            246,            735,            1042,            3343,            3821,            ],        [            1330,            2101,            3716,            2236,            3360,            4608,            3257,            2985,            5442,            3718,            3395,            2883,            2781,            4134,            1948,            1662,            2064,            2792,            3888,            3798,            571,            1161,            1039,            1030,            1612,            421,            0,            216,            794,            3519,            1066,            1474,            2581,            2641,            3503,            826,            1313,            2193,            921,            4359,            1930,            2380,            4659,            2302,            826,            2383,            671,            2789,            1252,            667,            1156,            754,            3567,            4045,            ],        [            1464,            2235,            3850,            2377,            3494,            4742,            3391,            3119,            6106,            3852,            3529,            3017,            2915,            4268,            2082,            1796,            2198,            2926,            4022,            3932,            379,            1302,            1180,            838,            1420,            562,            387,            0,            816,            3653,            1218,            1282,            2715,            2775,            3637,            960,            1447,            2327,            729,            4493,            2064,            2442,            4793,            2436,            967,            2517,            479,            2923,            1393,            808,            1297,            562,            3701,            4179,            ],        [            2027,            2798,            4413,            2933,            4766,            5305,            3954,            4391,            6684,            4038,            3715,            3203,            3101,            4454,            2728,            2382,            2761,            3489,            4208,            4118,            707,            1678,            1468,            410,            1174,            1483,            1399,            1133,            0,            3839,            1008,            1036,            3278,            3338,            4200,            1516,            2027,            2890,            1057,            4679,            2650,            3077,            4979,            2999,            961,            3080,            337,            3486,            1513,            1143,            1283,            552,            3887,            4365,            ],        [            3393,            3740,            5781,            3545,            3143,            5768,            5322,            2768,            3929,            1451,            1043,            757,            1267,            877,            3765,            3015,            3286,            3315,            1536,            1446,            4172,            2774,            2932,            4385,            5213,            3330,            3601,            3817,            3975,            0,            3341,            5075,            3104,            4706,            5568,            2879,            2825,            3615,            4522,            2190,            3283,            3682,            1402,            4367,            3472,            3756,            4312,            4854,            2845,            3118,            3049,            4527,            1215,            1693,            ],        [            1197,            1968,            3583,            2110,            3227,            4475,            3124,            2852,            5839,            3602,            3279,            2767,            2665,            4018,            1898,            1552,            1931,            2659,            3772,            3682,            1157,            848,            685,            1142,            2198,            642,            586,            802,            732,            3403,            0,            1768,            2448,            2508,            3370,            686,            1197,            2060,            1507,            4243,            1820,            2151,            4543,            2169,            131,            2250,            1069,            2656,            1218,            313,            988,            1284,            3451,            3929,            ],        [            2562,            3333,            4948,            3468,            5301,            5840,            4489,            4926,            7204,            4950,            4627,            4115,            4013,            5366,            3180,            2894,            3296,            4024,            5120,            5030,            797,            2401,            2756,            918,            404,            1573,            1398,            1223,            1468,            4751,            2229,            0,            3813,            3873,            4735,            1971,            2545,            3425,            401,            5591,            3162,            3612,            5891,            3534,            2013,            3615,            1131,            4021,            2969,            1819,            2844,            578,            4799,            5277,            ],        [            1390,            636,            2295,            1286,            2147,            3187,            1836,            1772,            4229,            4442,            4119,            3607,            3505,            4858,            1101,            2057,            840,            211,            4612,            4522,            3068,            2353,            2691,            3455,            4109,            2672,            2497,            2713,            3045,            4243,            2761,            3971,            0,            1220,            2082,            1943,            2037,            511,            3418,            5083,            2138,            1751,            5383,            881,            2510,            1224,            3382,            1368,            2904,            2351,            2808,            3597,            4291,            4769,            ],        [            1275,            521,            1446,            1171,            2241,            2338,            987,            1866,            5228,            4327,            4004,            3492,            3390,            4743,            986,            1942,            999,            1028,            4497,            4407,            2953,            2238,            2576,            3340,            3994,            2557,            2382,            2598,            2930,            4128,            2646,            3856,            1139,            0,            1233,            1828,            1922,            613,            3303,            4968,            2023,            1636,            5268,            262,            2395,            1318,            3267,            519,            2789,            2236,            2693,            3482,            4176,            4654,            ],        [            1794,            1040,            805,            1690,            4220,            1697,            346,            4493,            5747,            4846,            4523,            4011,            3909,            5262,            1505,            2461,            1518,            1547,            5016,            4926,            3472,            2757,            3095,            3859,            4513,            3076,            2901,            3117,            3449,            4647,            3165,            4375,            1658,            519,            0,            2347,            2441,            1132,            3822,            5487,            2542,            2155,            5787,            781,            2914,            1837,            3786,            681,            3308,            2755,            3212,            4001,            4695,            5173,            ],        [            511,            1282,            2897,            1417,            3250,            3789,            2438,            2875,            5153,            2899,            2576,            2064,            1962,            3315,            1212,            843,            1245,            1973,            3069,            2979,            1469,            797,            1148,            1856,            2510,            1073,            898,            1114,            1446,            2700,            1191,            2372,            1762,            1822,            2684,            0,            494,            1374,            1819,            3540,            1111,            1561,            3840,            1483,            940,            1564,            1783,            1970,            1361,            781,            1265,            1998,            2748,            3226,            ],        [            1128,            1883,            3498,            1699,            2864,            4390,            3039,            2489,            4767,            2405,            2082,            1570,            1468,            2821,            1500,            902,            1846,            2574,            2575,            2485,            1720,            496,            654,            2107,            2761,            1052,            1149,            1365,            1697,            2206,            1063,            2623,            2363,            2423,            3285,            601,            0,            1975,            2070,            3046,            943,            1121,            3346,            2084,            1194,            1910,            2034,            2571,            867,            840,            771,            2249,            2254,            2732,            ],        [            879,            125,            1784,            775,            1878,            2676,            1325,            1503,            3960,            3931,            3608,            3096,            2994,            4347,            590,            1210,            603,            676,            4101,            4011,            2557,            1842,            2180,            2944,            3598,            2161,            1986,            2202,            2534,            3732,            2250,            3460,            787,            709,            1571,            1432,            1526,            0,            2907,            4572,            1627,            1240,            4872,            370,            1999,            922,            2871,            857,            2393,            1840,            2297,            3086,            3780,            4258,            ],        [            2161,            2932,            4547,            3067,            4900,            5439,            4088,            4525,            6803,            4549,            4226,            3714,            3612,            4965,            2779,            2493,            2895,            3623,            4719,            4629,            396,            2000,            2355,            1205,            691,            1172,            997,            822,            1067,            4350,            1828,            553,            3412,            3472,            4334,            1570,            2144,            3024,            0,            5190,            2761,            3211,            5490,            3133,            1612,            3214,            730,            3620,            2568,            1418,            2443,            865,            4398,            4876,            ],        [            4116,            3799,            5458,            3316,            3202,            5827,            4999,            2827,            1837,            3768,            3442,            3270,            3780,            4201,            3468,            3885,            3345,            3374,            3935,            3845,            5741,            4521,            4690,            6128,            6782,            5345,            5170,            5386,            5718,            3735,            5099,            6644,            3163,            4383,            5245,            4669,            4036,            3674,            6091,            0,            4137,            3750,            4726,            4044,            5230,            3729,            6055,            4531,            4903,            4861,            4807,            6270,            3614,            4092,            ],        [            702,            1114,            2729,            520,            2346,            3621,            2270,            1971,            4249,            2995,            2672,            2160,            2058,            3411,            670,            439,            1077,            1805,            3165,            3075,            2295,            1075,            1244,            2682,            3336,            1899,            1724,            1940,            2272,            2796,            1653,            3198,            1594,            1654,            2516,            1181,            590,            1206,            2645,            3636,            0,            657,            3936,            1315,            1784,            731,            2609,            1802,            1457,            1415,            1361,            2824,            2844,            3322,            ],        [            1118,            1530,            3145,            936,            2250,            4037,            2686,            1875,            4153,            2935,            2612,            2100,            1998,            3351,            1086,            855,            1493,            2221,            3105,            3015,            2711,            1491,            1660,            3098,            3752,            2315,            2140,            2356,            2688,            2736,            2069,            3614,            2010,            2070,            2932,            1597,            1006,            1622,            3061,            3576,            416,            0,            3876,            1731,            2200,            1147,            3025,            2218,            1873,            1831,            1777,            3240,            2784,            3262,            ],        [            3409,            4106,            5626,            3561,            3370,            6662,            5167,            2995,            4295,            1817,            1409,            1123,            1633,            777,            3775,            3031,            3513,            3542,            2035,            1989,            4014,            2790,            2948,            4401,            5055,            3346,            3443,            3659,            3991,            1530,            3357,            9311,            3331,            4551,            5552,            2895,            2841,            3842,            4364,            2458,            3299,            3698,            0,            4351,            3488,            3772,            4328,            4699,            2861,            3134,            3065,            4543,            1581,            2192,            ],        [            1263,            509,            1414,            1159,            4829,            2306,            955,            5102,            4300,            4315,            3992,            3480,            3378,            4731,            974,            1594,            987,            1016,            4485,            4395,            2941,            2226,            2564,            3328,            3982,            2545,            2370,            2586,            2918,            4116,            2634,            3844,            1127,            339,            1201,            1816,            1910,            601,            3291,            4956,            2011,            1624,            5256,            0,            2383,            1306,            3255,            487,            2777,            2224,            2681,            3470,            4164,            4642,            ],        [            1066,            1837,            3452,            1972,            3805,            4344,            2993,            3430,            5708,            3471,            3148,            2636,            2534,            3887,            1767,            1421,            1800,            2528,            3641,            3551,            1026,            717,            554,            1413,            2067,            339,            455,            671,            1003,            3272,            639,            1929,            2317,            2377,            3239,            555,            1066,            1929,            1376,            4112,            1689,            2116,            4412,            2038,            0,            2119,            1340,            2525,            767,            182,            671,            1209,            3320,            3798,            ],        [            1290,            1087,            2702,            488,            923,            3594,            2243,            548,            3005,            2493,            2699,            2187,            2676,            3438,            640,            1057,            1050,            1778,            3192,            3102,            2913,            1693,            1862,            3300,            3954,            2517,            2342,            2558,            2890,            2823,            2271,            3816,            1567,            1627,            2489,            1799,            1208,            1179,            3263,            3134,            1309,            922,            3963,            1288,            2402,            0,            3227,            1775,            2075,            2033,            1979,            3442,            2871,            3349,            ],        [            2135,            2906,            4521,            3041,            4874,            5413,            4062,            4499,            6777,            4523,            4200,            3688,            3586,            4939,            2753,            2467,            2869,            3597,            4693,            4603,            370,            2015,            1805,            613,            837,            1146,            971,            796,            337,            4324,            1345,            699,            3386,            3446,            4308,            1544,            2118,            2998,            720,            5164,            2735,            3185,            5464,            3107,            1298,            3188,            0,            3594,            1850,            1480,            1620,            215,            4372,            4850,            ],        [            1934,            1180,            927,            1830,            4342,            1819,            468,            4615,            7446,            4986,            4663,            4151,            4049,            5402,            1645,            2265,            1658,            1687,            5156,            5066,            3612,            2897,            3235,            3999,            4653,            2938,            3041,            3257,            3589,            4787,            3305,            4515,            1798,            659,            714,            2487,            2581,            1272,            3962,            5627,            2682,            2295,            5927,            921,            3054,            1977,            3926,            0,            3448,            2895,            3352,            4141,            4835,            5313,            ],        [            2204,            2928,            4543,            2775,            3598,            5435,            4084,            3223,            5383,            2905,            2582,            2070,            1968,            3321,            2576,            1790,            2891,            3619,            3075,            2985,            2101,            1084,            564,            1804,            2568,            1619,            1547,            1763,            1394,            2706,            961,            2430,            3408,            3468,            4330,            1646,            1605,            3020,            2468,            3546,            2058,            1855,            3846,            3129,            1092,            2986,            1731,            3616,            0,            1101,            387,            1946,            2754,            3232,            ],        [            1283,            2007,            3622,            1854,            3693,            4514,            3163,            3318,            5596,            3020,            2697,            2185,            2083,            3436,            1655,            1219,            1970,            2698,            3190,            3100,            1479,            491,            372,            1599,            2520,            792,            908,            1124,            1189,            2821,            457,            2382,            2487,            2547,            3409,            725,            864,            2099,            1829,            3661,            1487,            2004,            3961,            2208,            588,            2065,            1526,            2695,            585,            0,            489,            1741,            2869,            3347,            ],        [            1617,            2341,            3956,            2188,            4027,            4848,            3497,            3652,            5233,            2755,            2432,            1920,            1818,            3171,            1989,            1203,            2304,            3032,            2925,            2835,            1531,            497,            414,            1516,            2572,            1016,            960,            1176,            1106,            2556,            374,            2142,            2821,            2881,            3743,            1059,            1018,            2433,            1881,            3396,            1471,            2338,            3696,            2542,            505,            2399,            1443,            3029,            230,            514,            0,            1658,            2604,            3082,            ],        [            2218,            2363,            4604,            3124,            4957,            5496,            4145,            4582,            6860,            4606,            4283,            3771,            3669,            5022,            2836,            2550,            2952,            3680,            4776,            4686,            453,            2230,            1847,            340,            622,            1229,            1054,            879,            552,            4407,            1560,            484,            3469,            3529,            4391,            1627,            2201,            3081,            885,            5247,            2818,            3268,            5547,            3190,            1513,            3271,            215,            3677,            2065,            1475,            1835,            0,            4455,            4933,            ],        [            3939,            4636,            6300,            4091,            4039,            6664,            5841,            3304,            4966,            2488,            172,            1653,            1803,            2456,            3881,            3561,            4182,            4211,            407,            729,            4544,            2960,            3118,            4571,            5585,            3516,            3973,            3829,            4521,            1758,            3887,            5447,            4000,            5225,            6087,            3425,            3371,            4511,            4894,            3129,            3829,            4228,            1809,            4886,            4018,            4302,            4858,            5373,            3391,            3664,            3595,            5073,            0,            250,            ],        [            3894,            4591,            6255,            4046,            3994,            6619,            5796,            3619,            4921,            2443,            422,            1608,            2118,            2411,            4196,            3516,            4137,            4166,            157,            247,            4499,            3275,            3073,            4886,            5180,            3831,            3928,            4144,            4476,            2073,            3842,            5402,            3595,            5180,            6042,            3380,            3326,            4466,            4489,            3084,            3784,            4183,            1764,            4841,            3973,            4257,            4453,            5328,            3346,            3619,            3550,            5028,            250,            0,            ],        ], 'starts': [12, 48, 37, 51], 'ends': [12, 48, 37, 51]},    'NW-2': {'distance_matrix': [        [            0,            960,            1200,            431,            233,            4740,            5415,            5776,            3562,            4580,            2945,            2560,            1981,            1303,            4009,            2079,            2410,            3291,            1731,            2511,            1603,            2489,            5475,            2603,            2110,            2465,            2936,            2607,            2923,            3472,            3792,            3580,            5270,            4922,            1843,            3321,            724,            1215,            2511,            6305,            3141,            2735,            1666,            971,            928,            4385,            1681,            3111,            2272,            1883,            2028,            2964,            3981,            5000,            2924,            2759,            2073,            2995,            3863,            5114,            1334,            239,            4856,            3622,            2014,            2881,            5541,            1842,            1568,            4271,            386,            1576,            2670,            2442,            2973,            2203,            5149,            1658,            5459,            1191,            2918,            ],        [            650,            0,            753,            1081,            883,            4293,            4968,            5329,            4212,            5230,            3595,            3210,            2158,            1953,            5391,            2575,            2906,            2844,            1908,            2688,            1780,            2666,            5028,            3099,            2606,            3240,            3460,            2784,            3100,            4190,            4288,            4076,            4823,            4475,            2339,            3498,            1220,            1711,            2688,            5858,            3318,            2912,            2162,            1467,            481,            5035,            1858,            3607,            2449,            2060,            2205,            3141,            3534,            4553,            2644,            2936,            2250,            3491,            4359,            4667,            1511,            534,            4409,            4118,            2191,            3058,            5094,            2492,            1745,            5096,            369,            1129,            2847,            2619,            3150,            2380,            4702,            2154,            5012,            1687,            3095,            ],        [            1151,            501,            0,            1582,            1384,            3742,            4417,            4778,            4713,            5731,            4096,            3711,            1918,            2454,            4840,            3076,            3407,            2293,            1668,            2448,            1540,            2426,            4477,            3600,            2406,            3000,            2909,            2544,            3016,            3639,            4433,            4577,            4272,            3924,            2840,            3414,            1721,            2212,            2448,            5307,            3078,            2672,            2663,            1968,            290,            5536,            1618,            4108,            2209,            1820,            1965,            2901,            2983,            4002,            2404,            2696,            2010,            3992,            4698,            4116,            1271,            1035,            3858,            4548,            1951,            2818,            4543,            2993,            1505,            4545,            870,            578,            2763,            2535,            2910,            2140,            4151,            2655,            4461,            2188,            3011,            ],        [            431,            1391,            1631,            0,            210,            5171,            5846,            6207,            3131,            4149,            2514,            2129,            2412,            872,            4440,            2510,            2841,            3722,            2162,            2942,            2034,            2920,            5906,            3034,            2541,            2896,            3367,            3038,            3354,            3903,            4223,            4011,            5701,            5353,            2274,            3752,            1155,            1646,            2942,            6736,            3572,            3166,            2097,            1402,            1359,            3954,            2112,            3542,            2703,            2314,            2459,            3395,            4412,            5431,            3355,            3190,            2504,            3426,            4294,            5545,            1765,            670,            5287,            4053,            2445,            3312,            5972,            1411,            1999,            4702,            817,            2007,            3101,            2873,            3404,            2634,            5580,            2089,            5890,            1622,            3349,            ],        [            233,            1193,            1433,            210,            0,            4973,            5648,            6009,            3341,            4359,            2724,            2339,            2214,            1082,            4242,            2312,            2643,            3524,            1964,            2744,            1836,            2722,            5708,            2836,            2343,            2698,            3169,            2840,            3156,            3705,            4025,            3813,            5503,            5155,            2076,            3554,            957,            1448,            2744,            6538,            3374,            2968,            1899,            1204,            1161,            4164,            1914,            3344,            2505,            2116,            2261,            3197,            4214,            5233,            3157,            2992,            2306,            3228,            4096,            5347,            1567,            472,            5089,            3855,            2247,            3114,            5774,            1621,            1801,            4504,            619,            1809,            2903,            2675,            3206,            2436,            5382,            1891,            5692,            1424,            3151,            ],        [            4618,            3968,            3467,            5049,            4851,            0,            903,            2962,            8823,            9321,            7125,            6740,            2297,            5921,            3024,            4416,            4212,            2413,            2640,            3093,            3189,            2805,            2256,            3887,            4055,            4386,            1313,            4193,            1379,            1772,            2617,            3798,            2456,            2817,            5036,            1639,            5188,            5476,            2991,            3595,            3596,            3317,            5936,            6103,            3757,            8674,            2492,            3305,            3488,            3099,            3244,            2910,            1194,            3284,            4845,            4345,            3659,            3506,            2980,            2456,            2920,            4502,            3104,            3172,            3600,            3463,            2883,            6460,            2784,            2827,            4337,            3441,            1632,            1860,            3770,            1944,            3061,            4851,            2095,            5538,            1384,            ],        [            4915,            4265,            3764,            5346,            5148,            297,            0,            2615,            8476,            8974,            9093,            7037,            2594,            6218,            2677,            4069,            3865,            2710,            2937,            3390,            3486,            3102,            2553,            3540,            4992,            4039,            1610,            4408,            1676,            2069,            2270,            3451,            2109,            2470,            4689,            1936,            5535,            5129,            3288,            3248,            3746,            3614,            5589,            5756,            4054,            8327,            2789,            2958,            3785,            3396,            3541,            3207,            1491,            3581,            4498,            4167,            3956,            3159,            2633,            2753,            3217,            4799,            2616,            2825,            3897,            3760,            2782,            6757,            3081,            2480,            4634,            3738,            1929,            2157,            4774,            2241,            2714,            4504,            2392,            5191,            1681,            ],        [            4755,            5197,            4696,            5186,            4988,            2949,            2652,            0,            5861,            6359,            6478,            6863,            2839,            6058,            1406,            2798,            2594,            2955,            3182,            3635,            3731,            3347,            1964,            2269,            3721,            2768,            3187,            3137,            3174,            2323,            999,            2180,            954,            1199,            3418,            2389,            4264,            3858,            3533,            1977,            2475,            2684,            4318,            4485,            4986,            5712,            3034,            1687,            4030,            3641,            3786,            2582,            3382,            2323,            3227,            2896,            3938,            1888,            1362,            1603,            3462,            5731,            1345,            1554,            3697,            2538,            1262,            7639,            3326,            1209,            5566,            4670,            2921,            2693,            3503,            2486,            1443,            3233,            1948,            3920,            3169,            ],        [            3586,            4546,            4786,            3155,            3365,            8666,            8369,            6167,            0,            1018,            617,            1002,            5567,            2259,            7123,            1427,            1758,            8667,            5317,            3219,            5189,            3438,            7681,            1951,            2350,            1813,            8899,            2182,            8886,            8040,            3140,            2928,            6671,            6916,            2047,            8106,            4310,            2487,            3321,            7689,            2716,            2995,            2947,            4557,            4514,            823,            5267,            2459,            3281,            5469,            3037,            3705,            9094,            8040,            2272,            1941,            2746,            2343,            7079,            7320,            4920,            3825,            7062,            2970,            2742,            2849,            6979,            1778,            5154,            6926,            3972,            5162,            8633,            8405,            2548,            8198,            7155,            1862,            7665,            4777,            8881,            ],        [            4604,            5564,            5804,            4173,            4383,            9164,            8867,            6665,            1018,            0,            1635,            2020,            9049,            3277,            7621,            2359,            2690,            9165,            9392,            4151,            6207,            4370,            8179,            2883,            3282,            2745,            9397,            3114,            9384,            8538,            4072,            3860,            7169,            7414,            2979,            8604,            5328,            3419,            4253,            8187,            3648,            3927,            3879,            4046,            5532,            647,            6285,            3391,            4213,            6487,            3969,            4637,            9592,            8538,            3204,            2873,            3678,            3275,            7577,            7818,            5938,            4843,            7560,            3902,            3674,            3781,            7477,            2796,            6172,            7424,            4990,            6180,            9131,            8903,            3480,            8696,            7653,            2794,            8163,            3481,            9379,            ],        [            2969,            3929,            4169,            2538,            2748,            9283,            8986,            6784,            617,            1635,            0,            385,            4950,            1642,            7740,            2044,            2375,            6260,            4700,            5480,            4572,            5458,            8298,            2568,            2967,            2430,            5905,            2799,            5892,            8657,            3757,            3545,            7288,            7533,            2664,            8723,            3693,            4184,            5480,            8306,            3333,            3612,            4635,            3940,            3897,            1440,            4650,            3076,            5241,            4852,            4997,            5933,            7052,            8657,            2889,            2558,            5042,            2960,            7696,            7937,            4303,            3208,            7679,            3587,            4983,            3466,            7596,            1161,            4537,            7543,            3355,            4545,            5639,            5411,            3165,            5172,            7772,            2479,            8282,            4160,            5887,            ],        [            2584,            3544,            3784,            2153,            2363,            9668,            9371,            7169,            1002,            2020,            385,            0,            4565,            1257,            8125,            2429,            2760,            5875,            4315,            5095,            4187,            5073,            8683,            2953,            4694,            2815,            5520,            5191,            5507,            9042,            4142,            3930,            7673,            7918,            3049,            9108,            3308,            3799,            5095,            8691,            3718,            5319,            4250,            3555,            3512,            1825,            4265,            3461,            4856,            4467,            4612,            5548,            6667,            6504,            3274,            2943,            4657,            3345,            8081,            8322,            3918,            2823,            8064,            3972,            4598,            3851,            7981,            776,            4152,            7928,            2970,            4160,            5254,            5026,            3550,            4787,            8157,            2864,            8667,            3775,            5502,            ],        [            2015,            2240,            1739,            2446,            2248,            2423,            4039,            2847,            5577,            6595,            4960,            4575,            0,            3318,            2909,            2939,            2716,            855,            475,            796,            1024,            508,            2510,            2909,            1890,            2484,            1087,            2028,            1074,            1623,            2502,            2042,            2341,            1957,            2699,            1472,            2585,            3076,            694,            1386,            1299,            1020,            3599,            2832,            2029,            6400,            327,            1573,            933,            934,            1079,            613,            1282,            2071,            1888,            2180,            1494,            1774,            1480,            2149,            755,            1899,            1891,            1334,            1435,            1166,            2576,            3857,            619,            1717,            2609,            1371,            821,            593,            1473,            354,            2184,            2514,            1946,            3052,            1069,            ],        [            1327,            2287,            2527,            896,            1106,            6220,            7205,            6013,            2259,            3277,            1642,            1257,            3308,            0,            5336,            3406,            3737,            4618,            3058,            3838,            2930,            3816,            5686,            3930,            3437,            3792,            4263,            3934,            4250,            4799,            5119,            4907,            5507,            5133,            3170,            4648,            2051,            2542,            3838,            7632,            4468,            4062,            2993,            2298,            2255,            3082,            3008,            4438,            3599,            3210,            3355,            4291,            5308,            5247,            4251,            4086,            3400,            4322,            5190,            6441,            2661,            1566,            5067,            4949,            3341,            4208,            6868,            539,            2895,            5598,            1713,            2903,            3997,            3769,            4300,            3530,            5996,            2985,            5122,            2518,            4245,            ],        [            3349,            4175,            3958,            3780,            3582,            3369,            3072,            1880,            2655,            3587,            3272,            5909,            1795,            4652,            0,            1392,            1188,            3375,            2270,            1542,            3304,            1284,            2231,            863,            2315,            1362,            3607,            1731,            3594,            2590,            986,            774,            1374,            1372,            2012,            2656,            2858,            2452,            1440,            1945,            1069,            1278,            2912,            3079,            4143,            2986,            3319,            281,            2830,            3584,            2586,            1551,            3802,            2590,            1821,            1490,            2532,            482,            1057,            1870,            3035,            3454,            1612,            816,            2291,            1132,            2047,            5191,            2992,            1465,            3601,            3590,            3341,            3113,            2097,            2906,            1400,            1827,            2215,            2514,            3589,            ],        [            2516,            3483,            2982,            2947,            2749,            4096,            3799,            2607,            1822,            2754,            2439,            5076,            2643,            3819,            1930,            0,            331,            4102,            2393,            1792,            2328,            2011,            3111,            524,            1482,            386,            4334,            755,            4321,            3470,            1713,            1501,            2101,            2346,            1179,            3536,            2025,            1619,            1894,            3124,            1289,            1568,            2079,            2246,            3272,            2153,            2343,            1032,            1854,            2608,            1610,            2278,            4529,            3470,            845,            514,            1556,            916,            1784,            2750,            2059,            2621,            2492,            1543,            1315,            1422,            2774,            4358,            2016,            2192,            2768,            2614,            4068,            3840,            1121,            3633,            2590,            994,            3095,            1681,            4316,            ],        [            2545,            3371,            3154,            2976,            2778,            3765,            3468,            2276,            1851,            2783,            2468,            5105,            2191,            3848,            1599,            588,            0,            3771,            2565,            1964,            2500,            1680,            2780,            193,            1511,            558,            4003,            927,            3990,            3139,            1382,            1170,            1770,            2015,            1208,            3205,            2054,            1648,            1836,            2793,            1461,            1740,            2108,            2275,            3339,            2182,            2515,            701,            2026,            2780,            1782,            1947,            4198,            3139,            1017,            686,            1728,            585,            1453,            2419,            2231,            2650,            2161,            1212,            1487,            1594,            2443,            4387,            2188,            1861,            2797,            2786,            3737,            3509,            1293,            3302,            2259,            1023,            2764,            1710,            3985,            ],        [            2930,            2280,            1779,            3361,            3163,            2903,            3578,            3939,            6492,            7510,            5875,            5490,            1982,            4233,            4001,            5393,            5189,            0,            2325,            2778,            2715,            2490,            3638,            4864,            3581,            4175,            2070,            3719,            2177,            2800,            3594,            4775,            3433,            3085,            6013,            2575,            3500,            3991,            2676,            4468,            3281,            3002,            4442,            3747,            2069,            7315,            2793,            4282,            3384,            2995,            3140,            4076,            2144,            3163,            3579,            3871,            3185,            4483,            3859,            3277,            2446,            2814,            3019,            3709,            3126,            3148,            3704,            4772,            2680,            3706,            2649,            1753,            1924,            1696,            4085,            1629,            3312,            5828,            3622,            3967,            2172,            ],        [            1540,            1765,            1264,            1971,            1773,            2804,            4420,            3228,            5102,            6120,            4485,            4100,            513,            2843,            3290,            2464,            2241,            1236,            0,            1087,            549,            1021,            2891,            2434,            1415,            2009,            1468,            1553,            1455,            2004,            3623,            3411,            2722,            2338,            2224,            1853,            2110,            2601,            1087,            1767,            1590,            1311,            3124,            2357,            1554,            5925,            213,            2942,            848,            459,            604,            1540,            1663,            2452,            1413,            1705,            1019,            2826,            1993,            2530,            280,            1424,            2272,            1847,            960,            1457,            2957,            3382,            144,            2995,            2134,            896,            1202,            974,            1919,            735,            2565,            2039,            2875,            2577,            1450,            ],        [            2545,            2770,            2269,            2976,            2778,            3222,            4050,            2858,            3223,            4155,            5490,            5105,            799,            3848,            2181,            1960,            1823,            1654,            1274,            0,            1554,            288,            2233,            2016,            1504,            1591,            1886,            1135,            1873,            2422,            1964,            1752,            2352,            1374,            2580,            2271,            3115,            3020,            102,            1947,            503,            224,            3480,            3647,            2559,            3554,            1282,            1283,            239,            1733,            483,            555,            2081,            2870,            1251,            1287,            1108,            1484,            1230,            1872,            1285,            2429,            1614,            1084,            833,            370,            2299,            4387,            955,            1467,            3139,            1901,            1620,            1392,            677,            1153,            1402,            2395,            2217,            3082,            1868,            ],        [            1670,            1895,            1394,            2101,            1903,            3409,            5160,            3968,            3092,            4024,            4615,            4230,            1118,            2973,            3291,            1915,            1692,            1841,            868,            1189,            0,            1375,            3496,            1885,            866,            1460,            2073,            1004,            2060,            2609,            3074,            2862,            3462,            2943,            1675,            2458,            2240,            2115,            1189,            4485,            1538,            1413,            2575,            2742,            1684,            3423,            818,            2393,            950,            1020,            706,            1642,            2268,            3057,            864,            1156,            470,            2277,            3145,            3135,            471,            1554,            2877,            2904,            411,            1671,            3562,            3512,            705,            3553,            2264,            1026,            1807,            1579,            1370,            1340,            3951,            1490,            3480,            2177,            2055,            ],        [            2526,            2751,            2250,            2957,            2759,            2934,            3832,            2252,            3511,            4443,            5471,            5086,            511,            3829,            1963,            2248,            2111,            1366,            986,            288,            1535,            0,            1975,            2304,            1690,            1879,            1598,            1423,            1585,            2134,            1746,            1534,            1746,            1116,            2868,            1983,            3096,            3308,            186,            1689,            791,            512,            3768,            3935,            2540,            3842,            838,            1065,            425,            1445,            669,            267,            1793,            2582,            1539,            1575,            1294,            1266,            972,            1614,            1266,            2410,            1356,            826,            1019,            658,            2041,            4368,            1130,            1209,            3120,            1882,            1332,            1104,            965,            865,            1144,            2683,            1959,            3370,            1580,            ],        [            5315,            4665,            4164,            5746,            5548,            3081,            2784,            1592,            7453,            7951,            8070,            8455,            2266,            6618,            1654,            3046,            2842,            2382,            2609,            2024,            3158,            1766,            0,            2517,            3969,            3016,            2614,            3385,            2601,            691,            1247,            2272,            1086,            650,            3666,            757,            4512,            4106,            1922,            1422,            2015,            2248,            4566,            4733,            4454,            7304,            2461,            1935,            3457,            3068,            3213,            2033,            1815,            1674,            3475,            3144,            3628,            2136,            1420,            200,            2889,            5199,            454,            1274,            3569,            2078,            627,            7157,            2753,            1492,            5034,            3505,            2348,            2120,            2189,            1913,            877,            3481,            316,            4168,            2596,            ],        [            3080,            3906,            3689,            3511,            3313,            3572,            3275,            2083,            2386,            3318,            3003,            5640,            1998,            4383,            1406,            1123,            919,            3578,            3805,            1745,            3035,            1487,            2587,            0,            2046,            1093,            3810,            1462,            3797,            2946,            1189,            977,            1577,            1822,            1743,            3012,            2589,            2183,            1643,            2600,            1272,            1481,            2643,            2810,            3874,            2717,            3050,            508,            2561,            3315,            2317,            1754,            4005,            2946,            1552,            1221,            2263,            392,            1260,            2226,            2766,            3185,            1968,            1019,            2022,            1335,            2250,            4922,            2723,            1668,            3332,            3321,            3544,            3316,            1828,            3109,            2066,            1558,            2571,            2245,            3792,            ],        [            2146,            2699,            2198,            2577,            2379,            5145,            4848,            3656,            2350,            3282,            5091,            4706,            1922,            3449,            2979,            1049,            1380,            2645,            1672,            1507,            1544,            1693,            4160,            1573,            0,            1435,            2877,            1322,            2864,            3413,            2762,            2550,            3150,            3395,            809,            3262,            1655,            1249,            1507,            4173,            1856,            1731,            1709,            1876,            2488,            2681,            1622,            2081,            1268,            1824,            1024,            1960,            3072,            3861,            1182,            1474,            396,            1965,            2833,            3799,            1275,            2251,            3541,            2592,            729,            1989,            3823,            3988,            1509,            3241,            2398,            1830,            2611,            2383,            1688,            2144,            3639,            624,            4144,            1311,            2859,            ],        [            2326,            3152,            3392,            2757,            2559,            3997,            3700,            2508,            1632,            2564,            2249,            4886,            2423,            3629,            1831,            369,            232,            4003,            2346,            1745,            2281,            1912,            3012,            425,            1292,            0,            4235,            708,            4222,            3371,            1614,            1402,            2002,            2247,            989,            3437,            1835,            1429,            2068,            3025,            1242,            1521,            1889,            2056,            3120,            1963,            2296,            933,            1807,            2561,            1563,            2179,            4430,            3371,            798,            467,            1688,            817,            1685,            2651,            2012,            2431,            2393,            1444,            1268,            1375,            2675,            4168,            1969,            2093,            2578,            2567,            3969,            3741,            1074,            3534,            2491,            804,            2996,            1491,            4217,            ],        [            3566,            2916,            2415,            3997,            3799,            1444,            2119,            2855,            8711,            9209,            5920,            5535,            1092,            4869,            2917,            4309,            4105,            1208,            1435,            1888,            1984,            1600,            1825,            3780,            2850,            3444,            0,            2988,            174,            1341,            2510,            3691,            2349,            1965,            4929,            1208,            4136,            4627,            1786,            2737,            2391,            2112,            5078,            4383,            2705,            8562,            1287,            3198,            2283,            1894,            2039,            2975,            303,            2079,            2848,            3140,            2454,            3399,            2775,            2157,            1715,            3450,            1899,            2589,            2395,            2258,            2584,            5408,            1579,            2622,            3285,            2389,            427,            655,            2565,            739,            2192,            4744,            1664,            4603,            179,            ],        [            2782,            2962,            2461,            3213,            3015,            4453,            4156,            2964,            2088,            3020,            2705,            5342,            2122,            4085,            2287,            825,            688,            2845,            1872,            1271,            1807,            1559,            3468,            881,            1748,            456,            3077,            0,            3064,            3827,            2070,            1858,            2458,            2703,            1445,            3893,            2291,            1885,            1373,            3481,            768,            1047,            2345,            2512,            2751,            2419,            1822,            1389,            1333,            2087,            1089,            1826,            3272,            3827,            324,            152,            1035,            1273,            2141,            3107,            1538,            2887,            2849,            1900,            794,            901,            3131,            4624,            1495,            2549,            3034,            2093,            2811,            2583,            600,            2344,            2947,            1260,            3452,            1947,            3059,            ],        [            2956,            3090,            2589,            3387,            3189,            1510,            2185,            2681,            8537,            9035,            5901,            5516,            1073,            4259,            2743,            4135,            3931,            1189,            1416,            1869,            1965,            1581,            1891,            3606,            2831,            3425,            174,            2969,            0,            1407,            2336,            3517,            2175,            1791,            4755,            907,            3526,            5195,            1767,            2563,            2372,            2093,            5655,            3773,            2879,            8388,            1268,            3024,            2264,            1875,            2020,            2956,            369,            1905,            2829,            3121,            2435,            3225,            2601,            1983,            1696,            2840,            1725,            2415,            2376,            2239,            2410,            4798,            1560,            2448,            3459,            2312,            253,            481,            2546,            720,            2018,            4570,            1730,            3993,            5,            ],        [            5799,            5149,            4648,            6230,            6032,            2135,            3268,            2076,            7937,            8435,            8554,            8939,            2750,            7102,            2138,            3530,            3326,            2866,            3093,            2508,            3642,            2250,            484,            3001,            4453,            3500,            1573,            3869,            1639,            0,            1731,            2756,            1570,            1134,            4150,            396,            4996,            4590,            2406,            1906,            2499,            2732,            5050,            5217,            4938,            7788,            2945,            2419,            3941,            3552,            3697,            2517,            1454,            2158,            3959,            3628,            4112,            2620,            1904,            684,            3373,            5683,            938,            1758,            4053,            2562,            1111,            7641,            3237,            1976,            5518,            4622,            2832,            2604,            2673,            2397,            1361,            3965,            323,            4652,            1644,            ],        [            4305,            4631,            4130,            4736,            4538,            2383,            2086,            894,            6755,            7253,            7372,            7757,            2273,            5608,            956,            2348,            2144,            2389,            2616,            3069,            3165,            2781,            1398,            1819,            3271,            2318,            2621,            2687,            2608,            1757,            0,            1730,            388,            633,            2968,            1823,            3814,            3408,            2967,            1411,            2025,            2234,            3868,            4035,            4420,            6606,            2468,            1237,            3464,            3075,            3220,            2016,            2816,            1757,            2777,            2446,            3488,            1438,            796,            1037,            2896,            5165,            779,            988,            3576,            2088,            1061,            6147,            2760,            643,            5000,            4104,            2355,            2127,            3053,            1920,            877,            2783,            1382,            3470,            2603,            ],        [            3778,            4843,            4342,            4209,            4011,            2595,            2298,            1106,            3084,            4016,            3701,            6338,            2485,            5081,            429,            1821,            1617,            2601,            2828,            1971,            3377,            1713,            1610,            1292,            2744,            1791,            2833,            2160,            2820,            1969,            212,            0,            600,            845,            2441,            2035,            3287,            2881,            1869,            1623,            1498,            1707,            3341,            3508,            4632,            3415,            2680,            710,            3259,            3287,            3015,            1980,            3028,            1969,            2250,            1919,            2961,            911,            1008,            1249,            3108,            3883,            991,            1200,            2720,            1561,            1273,            5620,            2972,            855,            4030,            4316,            2567,            2339,            2526,            2132,            1089,            2256,            1594,            2943,            2815,            ],        [            4244,            4686,            4185,            4675,            4477,            2438,            2141,            506,            6367,            6865,            6984,            7369,            2328,            5547,            895,            2287,            2083,            2444,            2671,            3124,            3220,            2836,            1453,            1758,            3210,            2257,            2676,            2626,            2663,            1812,            488,            1669,            0,            688,            2907,            1878,            3753,            3347,            3022,            1466,            1964,            2173,            3807,            3974,            4475,            6218,            2523,            1176,            3519,            3130,            3275,            2071,            2871,            1812,            2716,            2385,            3427,            1377,            851,            1092,            2951,            5220,            834,            1043,            3186,            2027,            673,            6086,            2815,            698,            5055,            4159,            2410,            2182,            2992,            1975,            932,            2722,            1437,            3409,            2658,            ],        [            4759,            4109,            3608,            5190,            4992,            2625,            2328,            1136,            6997,            7495,            7614,            6153,            1710,            6062,            1198,            2264,            2060,            1826,            2053,            1374,            2602,            1116,            859,            1735,            3187,            2234,            2058,            2603,            2045,            1218,            791,            1622,            630,            0,            2884,            1284,            3730,            3324,            1272,            866,            1365,            1598,            3784,            3951,            3898,            6848,            1905,            1153,            1511,            2512,            2657,            1383,            2253,            1218,            2693,            2362,            3072,            1354,            770,            498,            2333,            4643,            240,            624,            3013,            1428,            925,            6601,            2197,            936,            4478,            2949,            1792,            1564,            1539,            1357,            321,            2699,            843,            3386,            2040,            ],        [            1337,            2163,            2403,            1768,            1570,            5486,            5189,            3997,            4899,            5917,            4282,            3897,            3184,            2640,            3320,            1390,            1721,            4494,            2934,            3182,            2806,            3692,            4501,            1914,            1421,            1776,            4139,            2145,            4126,            4860,            3103,            2891,            3491,            3736,            0,            4926,            846,            440,            3284,            4514,            2679,            2958,            900,            1067,            2131,            5722,            2884,            2422,            3475,            3086,            3231,            3668,            5184,            6203,            2235,            1904,            1817,            2306,            3174,            4140,            2537,            1442,            3882,            2933,            2150,            2812,            4164,            3179,            2771,            3582,            1589,            2779,            3873,            3645,            2511,            3406,            3980,            969,            4485,            502,            4121,            ],        [            4482,            3832,            3331,            4913,            4715,            1739,            2414,            2910,            8771,            9269,            6989,            6604,            2161,            5785,            2972,            4364,            4160,            2277,            2504,            2957,            3053,            2669,            1318,            3835,            3919,            4513,            1177,            4057,            1243,            834,            2565,            3590,            2404,            1968,            4984,            0,            5052,            5424,            2855,            2740,            3460,            3181,            5884,            5299,            3621,            8622,            2356,            3253,            3352,            2963,            3108,            4044,            1058,            3148,            3917,            4209,            3523,            3454,            2738,            1518,            2784,            4366,            1772,            2592,            3464,            3327,            1945,            6324,            2648,            2810,            4201,            3305,            1496,            1724,            3634,            1808,            2195,            4799,            1157,            5519,            1248,            ],        [            936,            1762,            2002,            1367,            1169,            5451,            5154,            3962,            4498,            5516,            3881,            3496,            2783,            2239,            3285,            1355,            1686,            4093,            2533,            3147,            2405,            3291,            4466,            1879,            1386,            1741,            3738,            2110,            3725,            4825,            3068,            2856,            3456,            3701,            1119,            4891,            0,            491,            3249,            4479,            2644,            2923,            942,            666,            1730,            5321,            2483,            2387,            3074,            2685,            2830,            3633,            4783,            5802,            2200,            1869,            1782,            2271,            3139,            4105,            2136,            1041,            3847,            2898,            2115,            2777,            4129,            2778,            2370,            3547,            1188,            2378,            3472,            3244,            2476,            3005,            3945,            934,            4450,            467,            3720,            ],        [            3318,            4144,            4384,            3749,            3551,            7453,            7156,            5964,            6286,            5590,            5669,            5284,            5318,            4621,            5287,            3357,            3688,            7459,            5068,            5149,            4940,            5368,            6468,            3881,            3396,            3743,            6273,            4112,            6260,            6827,            5070,            4858,            5458,            5703,            3117,            6893,            2827,            0,            5251,            6481,            4646,            4925,            451,            3048,            4112,            4989,            5018,            4389,            5211,            5220,            4967,            5635,            7165,            6827,            4202,            3871,            3792,            4273,            5141,            6107,            4671,            3423,            5849,            4900,            4125,            4779,            6131,            5160,            5373,            5549,            3570,            4760,            6007,            5779,            4478,            5540,            5947,            2932,            6452,            2483,            6255,            ],        [            2545,            2770,            2269,            2976,            2778,            3120,            3988,            2408,            3325,            4257,            5490,            5105,            697,            3848,            2119,            2062,            1925,            1552,            1172,            102,            1554,            186,            2131,            2118,            1504,            1693,            1784,            1237,            1771,            2320,            1902,            1690,            1902,            1272,            2682,            2169,            3115,            3122,            0,            1845,            605,            326,            3582,            3749,            2559,            3656,            1024,            1221,            239,            1631,            483,            453,            1979,            2768,            1353,            1389,            1108,            1422,            1128,            1770,            1285,            2429,            1512,            982,            833,            472,            2197,            4387,            955,            1365,            3139,            1901,            1518,            1290,            779,            1051,            1300,            2497,            2115,            3184,            1766,            ],        [            2727,            3236,            2735,            3158,            2960,            2528,            4144,            2952,            6289,            9306,            5672,            5287,            844,            4030,            3014,            3651,            3428,            960,            1187,            1640,            1736,            1352,            2615,            3621,            2602,            3196,            1192,            2740,            1179,            1728,            2607,            3788,            2446,            2062,            3411,            1577,            3297,            3788,            1538,            0,            2143,            1864,            4311,            3544,            3025,            8659,            1039,            3295,            2035,            1646,            1791,            2727,            1387,            2176,            2600,            2892,            2206,            3496,            2872,            2254,            1467,            2611,            1996,            2686,            2147,            2010,            2681,            4569,            1331,            2719,            3605,            2083,            926,            698,            2317,            491,            2289,            3226,            2599,            3764,            1174,            ],        [            3414,            3386,            2885,            3845,            3647,            3883,            3586,            2394,            2720,            3652,            3337,            5974,            1302,            4717,            1717,            1457,            1320,            2157,            1777,            503,            2057,            791,            2193,            1513,            1855,            1088,            2389,            632,            2376,            2552,            1500,            1288,            1888,            1334,            2077,            2618,            2923,            2517,            605,            1907,            0,            279,            2977,            3144,            3175,            3051,            1785,            819,            742,            2236,            986,            1058,            2584,            2552,            748,            784,            1459,            1020,            1190,            1832,            1788,            3045,            1574,            1044,            1336,            133,            2259,            5256,            1458,            1427,            3755,            2517,            2123,            1895,            174,            1656,            1362,            1892,            2177,            2579,            2371,            ],        [            2769,            2994,            2493,            3200,            3002,            4123,            3826,            2634,            2999,            3931,            3616,            5329,            1023,            4072,            1957,            1736,            1599,            1878,            1498,            224,            1778,            512,            2433,            1792,            1728,            1367,            2110,            911,            2097,            2646,            1740,            1528,            2128,            1574,            2356,            2495,            3202,            2796,            326,            2147,            279,            0,            3256,            3423,            2783,            3330,            1506,            1059,            463,            1957,            707,            779,            2305,            3094,            1027,            1063,            1332,            1260,            1430,            2072,            1509,            2653,            1814,            1284,            1057,            146,            2499,            4611,            1179,            1667,            3363,            2125,            1844,            1616,            453,            1377,            1602,            2171,            2417,            2858,            2092,            ],        [            2867,            3693,            3933,            3298,            3100,            7002,            6705,            5513,            5835,            5139,            5218,            4833,            4867,            4170,            4836,            2906,            3237,            7008,            4617,            4698,            4489,            4917,            6017,            3430,            2945,            3292,            5822,            3661,            5809,            6376,            4619,            4407,            5007,            5252,            2666,            6442,            2376,            1818,            4800,            6030,            4195,            4474,            0,            2597,            3661,            4538,            4567,            3938,            4760,            4769,            4516,            5184,            6714,            6376,            3751,            3420,            3341,            3822,            4690,            5656,            4220,            2972,            5398,            4449,            3674,            4328,            5680,            4709,            4922,            5098,            3119,            4309,            5556,            5328,            4027,            5089,            5496,            2481,            6001,            2032,            5804,            ],        [            270,            1096,            1336,            701,            503,            4876,            5551,            5912,            3832,            4850,            3215,            2830,            2117,            1573,            4022,            2092,            2423,            3427,            1867,            2647,            1739,            2625,            5611,            2616,            2123,            2478,            3072,            2743,            3059,            4773,            3805,            3593,            5406,            5058,            1856,            3457,            737,            1228,            2647,            6441,            3277,            2871,            1679,            0,            1064,            4655,            1817,            3124,            2408,            2019,            2164,            3100,            4117,            5136,            2937,            2895,            2209,            3008,            3876,            5250,            1470,            375,            4992,            3635,            2150,            3017,            5677,            2112,            1704,            4284,            522,            1712,            2806,            2578,            3109,            2339,            5285,            1671,            5595,            1204,            3054,            ],        [            861,            211,            473,            1292,            1094,            4013,            4688,            5049,            4423,            5441,            3806,            3421,            2189,            2164,            5111,            2786,            3117,            2564,            1939,            2719,            1811,            2697,            4748,            3310,            2677,            3271,            3180,            2815,            3287,            3910,            4499,            4287,            4543,            4195,            2550,            3685,            1431,            1922,            2719,            5578,            3349,            2943,            2373,            1678,            0,            5246,            1889,            3818,            2480,            2091,            2236,            3172,            3254,            4273,            2675,            2967,            2281,            3702,            4969,            4387,            1542,            745,            4129,            4329,            2222,            3089,            4814,            2703,            1776,            4816,            580,            849,            3034,            2806,            3181,            2411,            4422,            2365,            4732,            1898,            3282,            ],        [            4409,            5369,            5609,            3978,            4188,            8517,            8220,            6018,            823,            647,            1440,            1825,            8402,            3082,            6974,            1758,            2089,            8518,            8745,            3550,            6012,            3769,            7532,            2282,            2681,            2144,            8750,            2513,            8737,            7891,            6567,            3259,            6522,            6767,            2378,            7957,            5133,            2818,            3652,            7540,            3047,            3326,            3278,            3445,            5337,            0,            6090,            2790,            3612,            6292,            3368,            4036,            8945,            7891,            2603,            2272,            3077,            2674,            6930,            7171,            5743,            4648,            6913,            3301,            3410,            3180,            6830,            2601,            5977,            6777,            4795,            5985,            8484,            8256,            2879,            8049,            7006,            2193,            7516,            2880,            8732,            ],        [            1859,            2084,            1583,            2290,            2092,            2591,            4207,            3015,            5421,            6439,            4804,            4419,            300,            3162,            3077,            2783,            2560,            1023,            319,            1096,            868,            808,            2678,            2753,            1734,            2328,            1255,            1872,            1242,            1791,            2670,            2342,            2509,            2125,            2543,            1640,            2429,            2920,            994,            1554,            1599,            1320,            3443,            2676,            1873,            6244,            0,            1873,            1167,            778,            923,            1859,            1450,            2239,            1732,            2024,            1338,            2074,            1780,            2317,            599,            1743,            2059,            1634,            1279,            1466,            2744,            3701,            463,            2782,            2453,            1215,            989,            761,            1773,            522,            2352,            2358,            2662,            2896,            1237,            ],        [            3068,            3894,            3677,            3499,            3301,            3405,            3108,            1916,            2374,            3306,            2991,            5628,            1831,            4371,            1239,            1111,            907,            3411,            3088,            2487,            3023,            1320,            2420,            582,            2034,            1081,            3643,            1450,            3630,            2779,            1022,            810,            1410,            1655,            1731,            2845,            2577,            2171,            1476,            2433,            1984,            2263,            2631,            2798,            3862,            2705,            3038,            0,            2549,            3303,            2305,            1587,            3838,            2779,            1540,            1209,            2251,            201,            1093,            2059,            2754,            3173,            1801,            852,            2010,            2117,            2083,            4910,            2711,            1501,            3320,            3309,            3377,            3149,            1816,            2942,            1899,            1546,            2404,            2233,            3625,            ],        [            2306,            2531,            2030,            2737,            2539,            3634,            5343,            4151,            3275,            4207,            5251,            4866,            936,            3609,            3474,            2012,            1875,            2066,            1093,            239,            1315,            425,            2370,            2068,            1265,            1643,            2298,            1187,            2285,            2834,            3257,            3045,            3645,            1511,            2632,            2683,            2876,            3072,            239,            2084,            742,            463,            3532,            3699,            2320,            3606,            1043,            1460,            0,            1309,            244,            692,            2493,            3282,            1047,            1339,            869,            2460,            1367,            2009,            1046,            2190,            1751,            1221,            594,            609,            2436,            4148,            716,            1604,            2900,            1662,            2032,            1804,            916,            1565,            1539,            2447,            2354,            3134,            2280,            ],        [            2157,            2382,            1881,            2588,            2390,            3485,            5194,            4002,            3126,            4058,            5102,            4717,            1194,            3460,            3325,            1863,            1726,            1917,            944,            628,            1166,            814,            3572,            1919,            1116,            1494,            2149,            1038,            2136,            2685,            3108,            2896,            3496,            1900,            2483,            2534,            2727,            2923,            628,            2473,            1131,            852,            3383,            3550,            2171,            3457,            894,            2427,            389,            0,            145,            1081,            2344,            3133,            898,            1190,            720,            2311,            1756,            3211,            897,            2041,            2953,            1610,            445,            998,            3638,            3999,            567,            1993,            2751,            1513,            1883,            1655,            1404,            1416,            1928,            2298,            3556,            2985,            2131,            ],        [            2062,            2287,            1786,            2493,            2295,            3390,            5099,            3907,            3031,            3963,            5007,            4622,            1099,            3365,            3230,            1768,            1631,            1822,            849,            483,            1071,            669,            3477,            1824,            1021,            1399,            2054,            943,            2041,            2590,            3013,            2801,            3401,            1755,            2388,            2439,            2632,            2828,            483,            2328,            986,            707,            3288,            3455,            2076,            3362,            799,            2332,            244,            1065,            0,            936,            2249,            3038,            803,            1095,            625,            2216,            1611,            3116,            802,            1946,            2858,            1465,            350,            853,            3543,            3904,            472,            1848,            2656,            1418,            1788,            1560,            1160,            1321,            1783,            2203,            3461,            2890,            2036,            ],        [            2259,            2484,            1983,            2690,            2492,            2667,            4283,            3091,            5821,            6839,            5204,            4819,            244,            3562,            3153,            3183,            2960,            1099,            719,            604,            1268,            316,            2754,            3153,            2134,            2728,            1331,            2272,            1318,            1867,            2062,            1850,            2585,            2201,            2943,            1716,            2829,            3320,            502,            1630,            1107,            828,            3843,            3076,            2273,            6644,            571,            1381,            741,            1178,            1323,            0,            1526,            2315,            2132,            2424,            1738,            1582,            1288,            2393,            999,            2143,            2135,            1142,            1679,            974,            2820,            4101,            863,            1525,            2853,            1615,            1065,            837,            1281,            598,            2428,            2758,            2738,            3296,            1313,            ],        [            3685,            3035,            2534,            4116,            3918,            2090,            2765,            2866,            8722,            9220,            5931,            5546,            1103,            4988,            2928,            4320,            4116,            1219,            1446,            1899,            1995,            1611,            2529,            3791,            2861,            3455,            119,            2999,            185,            1243,            2521,            3702,            2360,            1976,            4940,            1092,            4255,            4746,            1797,            2748,            2402,            2123,            5197,            4502,            2824,            8573,            1298,            3209,            2294,            1905,            2050,            2986,            0,            2090,            2859,            3151,            2465,            3410,            2786,            2168,            1726,            3569,            1910,            2600,            2406,            2269,            2595,            5527,            1590,            2633,            3404,            2508,            438,            666,            2576,            750,            2203,            4755,            1566,            4722,            190,            ],        [            4187,            4525,            4024,            4618,            4420,            2265,            1968,            776,            6637,            7135,            7254,            7639,            2167,            5490,            838,            2230,            2026,            2283,            2510,            2963,            3059,            2675,            1195,            1701,            3153,            2200,            2515,            2569,            2502,            1554,            431,            1612,            270,            530,            2850,            1620,            3696,            3290,            2861,            1305,            1907,            2116,            3750,            3917,            4314,            6488,            2362,            1119,            3358,            2969,            3114,            1913,            2710,            0,            2659,            2328,            3370,            1320,            696,            834,            2790,            5059,            576,            888,            3129,            1970,            943,            6029,            2654,            543,            4894,            3998,            2249,            2021,            2935,            1814,            771,            2665,            1179,            3352,            2497,            ],        [            2413,            2638,            2137,            2844,            2646,            4724,            4427,            3235,            2359,            3291,            2976,            4973,            1798,            3716,            2558,            1096,            959,            2521,            1548,            1248,            1483,            1434,            3739,            1152,            1107,            727,            2753,            271,            2740,            3289,            2341,            2129,            2729,            2974,            1716,            3138,            2562,            2156,            1248,            3752,            805,            1084,            2616,            2783,            2427,            2690,            1498,            1660,            1009,            1763,            765,            1701,            2948,            3737,            0,            423,            711,            1544,            2412,            3378,            1214,            2297,            3120,            2171,            470,            938,            3402,            4255,            1171,            2820,            3007,            1769,            2487,            2259,            637,            2020,            3218,            1531,            3723,            2218,            2735,            ],        [            2630,            3456,            2572,            3061,            2863,            4301,            4004,            2812,            1936,            2868,            2553,            5190,            2233,            3933,            2135,            673,            536,            2956,            1983,            1382,            1918,            1670,            3316,            729,            1596,            304,            3188,            345,            3175,            3675,            1918,            1706,            2306,            2551,            1293,            3741,            2139,            1733,            1484,            3329,            879,            1158,            2193,            2360,            2862,            2267,            1933,            1237,            1444,            2198,            1200,            1937,            3383,            3675,            435,            0,            1146,            1121,            1989,            2955,            1649,            2735,            2697,            1748,            905,            1012,            2979,            4472,            1606,            2397,            2882,            2204,            2922,            2694,            711,            2455,            2795,            1108,            3300,            1795,            3170,            ],        [            2078,            2303,            1802,            2509,            2311,            3817,            5082,            3890,            2746,            3678,            5023,            4638,            1526,            3381,            3213,            1445,            1614,            2249,            1276,            1111,            1148,            1297,            3904,            1807,            396,            1382,            2481,            926,            2468,            3017,            2996,            2784,            3384,            3629,            1205,            2866,            2051,            1645,            1111,            4407,            1460,            1335,            2105,            2272,            2092,            3077,            1226,            2315,            872,            1428,            628,            1564,            2676,            3465,            786,            1078,            0,            2199,            3067,            3543,            879,            1962,            3285,            2826,            333,            1593,            4057,            3920,            1113,            3475,            2672,            1434,            2215,            1987,            1292,            1748,            3873,            1020,            3888,            1707,            2463,            ],        [            2867,            3693,            3476,            3298,            3100,            3383,            3086,            1894,            2173,            3105,            2790,            5427,            1809,            4170,            1217,            910,            706,            3389,            2887,            2286,            2822,            1298,            2398,            381,            1833,            880,            3621,            1249,            3608,            2757,            1000,            788,            1388,            1633,            1530,            2823,            2376,            1970,            2388,            2411,            1783,            2062,            2430,            2597,            3661,            2504,            2837,            319,            2348,            3102,            2104,            1565,            3816,            2757,            1339,            1008,            2050,            0,            1071,            2037,            2553,            2972,            1779,            830,            1809,            1916,            2061,            4709,            2510,            1479,            3119,            3108,            3355,            3127,            1615,            2920,            1877,            1345,            2382,            2032,            3603,            ],        [            4077,            4530,            4029,            4508,            4310,            3046,            2749,            1557,            3383,            4315,            4000,            6637,            2131,            5380,            770,            2120,            1916,            2247,            2474,            1230,            3023,            972,            1280,            1591,            3043,            2090,            2479,            2459,            2466,            1639,            553,            341,            1051,            515,            2740,            1705,            3586,            3180,            1128,            1287,            1221,            1454,            3640,            3807,            4319,            3714,            2326,            1009,            1367,            2933,            1611,            1239,            2674,            1639,            2549,            2218,            3493,            1210,            0,            919,            2754,            5064,            661,            192,            3434,            1284,            1346,            5919,            2618,            243,            4899,            4003,            2213,            1985,            1395,            1778,            742,            2555,            1264,            3242,            2461,            ],        [            5159,            4509,            4008,            5590,            5392,            2925,            2628,            1436,            7297,            7795,            7914,            8299,            2110,            6462,            1498,            2890,            2686,            2226,            2453,            1868,            3002,            1610,            423,            2361,            3813,            2860,            2458,            3229,            2445,            782,            1091,            2116,            930,            494,            3510,            848,            4356,            3950,            1766,            1266,            1859,            2092,            4410,            4577,            4298,            7148,            2305,            1779,            3301,            2912,            3057,            1877,            1906,            1518,            3319,            2988,            3472,            1980,            1264,            0,            2733,            5043,            298,            1118,            3413,            1922,            427,            7001,            2597,            1336,            4878,            3349,            2192,            1964,            2033,            1757,            721,            3325,            407,            4012,            2440,            ],        [            1554,            1779,            1278,            1985,            1787,            3293,            5429,            4237,            5116,            4293,            4499,            4114,            1002,            2857,            3560,            2184,            1961,            1725,            752,            1458,            269,            1510,            3380,            2154,            1135,            1729,            1957,            1273,            1944,            2493,            3343,            3131,            3731,            2827,            1944,            2342,            2124,            2384,            1458,            2256,            1807,            1682,            2844,            3011,            1568,            3692,            702,            2662,            1219,            904,            975,            1911,            2152,            2941,            1133,            1425,            739,            2546,            3414,            3019,            0,            1438,            2761,            3173,            680,            1940,            3446,            3396,            589,            3822,            2148,            910,            1691,            1463,            1639,            1224,            3054,            1759,            3364,            2446,            1939,            ],        [            116,            942,            1182,            547,            349,            4722,            5840,            4648,            3678,            4696,            3061,            2676,            1963,            1419,            3971,            2041,            2372,            3273,            1713,            2493,            1585,            2471,            5457,            2565,            2072,            2427,            2918,            2796,            2905,            4619,            3754,            3542,            4142,            4387,            1805,            3303,            686,            1177,            2493,            5165,            3330,            2717,            1628,            933,            910,            4501,            1663,            3073,            2254,            1865,            2010,            2946,            3963,            4982,            2886,            2555,            2055,            2957,            3825,            5096,            1316,            0,            4838,            3584,            1996,            3463,            4815,            1958,            1550,            4233,            368,            1558,            2652,            2424,            3162,            2185,            4631,            1620,            5441,            1153,            2900,            ],        [            4957,            4307,            3806,            5388,            5190,            2723,            2426,            1234,            7095,            7593,            7712,            8097,            1908,            6260,            1296,            2688,            2484,            2024,            2251,            1666,            2800,            1408,            619,            2159,            3611,            2658,            2221,            3027,            2243,            978,            889,            1914,            728,            292,            3308,            1044,            4154,            3748,            1564,            1064,            1657,            1890,            4208,            4375,            4096,            6946,            2103,            1577,            3099,            2710,            2855,            1675,            2102,            1316,            3117,            2786,            3270,            1778,            1062,            258,            2531,            4841,            0,            916,            3211,            1720,            685,            6799,            2395,            1134,            4676,            3780,            1990,            1762,            1831,            1555,            519,            3123,            603,            3810,            2238,            ],        [            3931,            4440,            3939,            4362,            4164,            3135,            2838,            1760,            3237,            4169,            3854,            6491,            2041,            5234,            969,            1974,            1770,            2157,            2384,            1084,            2933,            826,            1483,            1445,            2897,            1944,            2389,            2313,            2376,            1842,            752,            540,            1254,            624,            2594,            1908,            3440,            3034,            982,            1197,            1075,            1308,            3494,            3661,            4229,            3568,            2236,            863,            1221,            2843,            1465,            1093,            2584,            1842,            2403,            2072,            3114,            1064,            192,            1122,            2664,            4036,            864,            0,            1815,            1138,            1549,            5773,            2528,            435,            4809,            3280,            2123,            1895,            1249,            1688,            652,            2409,            1467,            3096,            2371,            ],        [            3375,            2439,            1938,            3806,            3608,            5046,            4749,            3557,            2681,            3613,            3298,            5935,            1599,            4678,            2880,            1418,            1281,            2322,            1349,            1049,            1284,            1235,            4061,            1474,            908,            1049,            2554,            593,            2541,            4420,            2663,            2451,            3051,            3296,            2038,            2939,            2884,            2478,            1049,            4074,            1127,            1406,            2938,            3105,            2228,            3012,            1299,            1982,            810,            1564,            566,            1502,            2749,            3538,            453,            745,            512,            1866,            2734,            3700,            1015,            2098,            3442,            2493,            0,            1260,            3724,            5217,            972,            3142,            2808,            1570,            2288,            2060,            959,            1821,            3540,            1853,            4045,            2540,            2536,            ],        [            3547,            3140,            2639,            3978,            3780,            3977,            3680,            2488,            2853,            3785,            3470,            6107,            1169,            4850,            1811,            1590,            1453,            2024,            1644,            370,            1924,            658,            2287,            1646,            1988,            1221,            2256,            765,            2243,            2646,            1594,            1382,            1982,            1428,            2210,            2712,            3056,            2650,            472,            2001,            133,            146,            3110,            3277,            2929,            3184,            1652,            913,            609,            2103,            853,            925,            2451,            2646,            881,            917,            1592,            1114,            1284,            1926,            1655,            2799,            1668,            1138,            1203,            0,            2353,            5389,            1325,            1521,            3509,            2271,            1990,            1762,            307,            1523,            1456,            2025,            2271,            2712,            2238,            ],        [            4371,            4813,            4312,            4802,            4604,            2565,            2268,            960,            6821,            7319,            7438,            7823,            2455,            5674,            1022,            2414,            2210,            2571,            2798,            3251,            3347,            2963,            850,            1885,            3337,            2384,            2803,            2753,            2790,            1209,            615,            1796,            454,            815,            3034,            1275,            3880,            3474,            3149,            1593,            2091,            2300,            3934,            4101,            4602,            6672,            2650,            1303,            3646,            3257,            3402,            2198,            2998,            1939,            2843,            2512,            3554,            1504,            978,            427,            3078,            4476,            961,            1170,            3313,            2154,            0,            6213,            2942,            825,            5182,            4286,            2537,            2309,            3119,            2102,            1059,            2849,            834,            3536,            2785,            ],        [            1866,            2826,            3066,            1435,            1645,            6759,            7744,            7945,            1778,            2796,            1161,            776,            3847,            539,            5875,            3945,            4276,            5157,            3597,            4377,            3469,            4355,            6225,            4469,            3976,            4331,            4802,            4473,            4789,            5338,            5658,            5446,            6046,            5672,            3709,            5187,            2590,            3081,            4377,            9467,            5007,            4601,            3532,            2837,            2794,            2601,            3547,            4977,            4138,            3749,            3894,            4830,            5847,            5786,            4790,            4625,            3939,            4861,            5729,            6980,            3200,            2105,            5606,            5488,            3880,            4747,            7407,            0,            3434,            6137,            2252,            3442,            4536,            4308,            4839,            4069,            8933,            3524,            6209,            3057,            4784,            ],        [            1590,            1815,            1314,            2021,            1823,            2918,            4534,            3342,            5152,            6170,            4535,            4150,            627,            2893,            3404,            2178,            2041,            1350,            377,            943,            599,            1129,            3005,            2234,            1465,            1809,            1582,            1353,            1569,            2118,            3423,            3211,            2836,            2452,            2274,            1967,            2160,            2651,            943,            1881,            1446,            1167,            3174,            2407,            1604,            5975,            327,            2742,            704,            315,            460,            1396,            1777,            2566,            1213,            1505,            1069,            2626,            2071,            2644,            330,            1474,            2386,            1925,            760,            1313,            3071,            3432,            0,            2308,            2184,            946,            1316,            1088,            1719,            849,            2679,            2089,            2989,            2627,            1564,            ],        [            4314,            4377,            3876,            4745,            4547,            2893,            2596,            1404,            3620,            4552,            4237,            6874,            1978,            5617,            864,            2357,            2153,            2094,            2321,            1467,            2870,            1209,            1127,            1828,            3280,            2327,            2326,            2696,            2313,            1486,            427,            544,            898,            362,            2977,            1552,            3823,            3417,            1365,            1134,            1458,            1691,            3877,            4044,            4166,            3951,            2173,            1246,            1604,            2780,            2925,            1476,            2521,            1486,            2786,            2455,            3340,            1447,            243,            766,            2601,            4911,            508,            435,            3281,            1521,            1193,            6156,            2465,            0,            4746,            3217,            2060,            1832,            1632,            1625,            589,            2792,            1111,            3479,            2308,            ],        [            281,            788,            1090,            712,            514,            4568,            5243,            5604,            3843,            4861,            3226,            2841,            1789,            1584,            4136,            2206,            2537,            2512,            1539,            2319,            1411,            2297,            4167,            2730,            2237,            2871,            2744,            2415,            2731,            3280,            3919,            3707,            5098,            3614,            1970,            3129,            851,            1342,            2319,            3043,            2949,            2543,            1793,            1098,            756,            4666,            1489,            3238,            2080,            1691,            1836,            2772,            3809,            4828,            2275,            2567,            1881,            3122,            3990,            3806,            1142,            165,            3548,            3749,            1822,            2689,            4233,            2123,            1376,            4398,            0,            1123,            2478,            2250,            2781,            2011,            3841,            1785,            4151,            1318,            2726,            ],        [            1519,            869,            368,            1950,            1752,            3578,            4253,            4614,            5081,            6099,            4464,            4079,            1340,            2822,            4676,            2877,            2654,            2063,            1090,            1870,            962,            1848,            4313,            2847,            1828,            2422,            2295,            1966,            2282,            2831,            4036,            3824,            4108,            3760,            2637,            2680,            2089,            2580,            1870,            5143,            2500,            2094,            3031,            2336,            658,            5904,            1040,            3355,            1631,            1242,            1387,            2323,            2819,            3838,            1826,            2118,            1432,            3239,            4534,            3952,            693,            1403,            3694,            2674,            1373,            2240,            4379,            3361,            927,            4381,            1238,            0,            2029,            1801,            2332,            1562,            3987,            2452,            4297,            2556,            2277,            ],        [            2703,            2928,            2427,            3134,            2936,            1763,            2438,            2428,            8284,            8782,            5648,            5263,            820,            4006,            2490,            3882,            3678,            936,            1163,            1616,            1712,            1328,            2091,            3353,            2578,            3172,            427,            2716,            253,            1204,            2083,            3264,            1922,            1538,            4502,            1053,            3273,            4942,            1514,            2310,            2119,            1840,            5402,            3520,            2717,            8135,            1015,            2771,            2011,            1622,            1767,            2703,            622,            1652,            2576,            2868,            2182,            2972,            2348,            1730,            1443,            2587,            1472,            2162,            2123,            1986,            2157,            4545,            1307,            2195,            3297,            2059,            0,            228,            2293,            467,            1765,            4317,            2075,            3740,            248,            ],        [            2475,            2700,            2199,            2906,            2708,            1991,            3446,            2254,            8110,            8608,            5420,            5035,            592,            3778,            2316,            3708,            3504,            708,            935,            1388,            1484,            1100,            1917,            3179,            2350,            2944,            655,            2488,            481,            1030,            1909,            3090,            1748,            1364,            4328,            879,            3045,            4768,            1286,            2136,            1891,            1612,            5228,            3292,            2489,            7961,            787,            2597,            1783,            1394,            1539,            2475,            850,            1478,            2348,            2640,            1954,            2798,            2174,            1556,            1215,            2359,            1298,            1988,            1895,            1758,            1983,            4317,            1079,            2021,            3069,            1831,            228,            0,            2065,            239,            1591,            4143,            1901,            3512,            476,            ],        [            3246,            3218,            2717,            3677,            3479,            4917,            4620,            3428,            2552,            3484,            3169,            5806,            1476,            4549,            2751,            1289,            1152,            2331,            1951,            677,            2063,            965,            2343,            1345,            1687,            920,            2563,            464,            2550,            2702,            2534,            2322,            2922,            1484,            1909,            2768,            2755,            2349,            779,            2057,            174,            453,            2809,            2976,            3007,            2883,            2078,            969,            916,            2343,            1160,            1232,            2758,            2702,            580,            616,            1291,            1737,            1340,            1982,            1794,            2877,            1724,            1194,            1050,            307,            2409,            5088,            1751,            1577,            3587,            2349,            2297,            2069,            0,            1830,            1512,            1724,            2327,            2411,            2545,            ],        [            2236,            2461,            1960,            2667,            2469,            2069,            3685,            2493,            5798,            8847,            5181,            4796,            353,            3539,            2555,            3160,            2937,            501,            696,            1149,            1245,            861,            2156,            3130,            2111,            2705,            733,            2249,            720,            1269,            2148,            3329,            1987,            1603,            2920,            1118,            2806,            3297,            1047,            2375,            1652,            1373,            3820,            3053,            2250,            8200,            548,            2836,            1544,            1155,            1300,            2236,            928,            1717,            2109,            2401,            1715,            3037,            2413,            1795,            976,            2120,            1537,            2227,            1656,            1519,            2222,            4078,            840,            2260,            2830,            1592,            467,            239,            1826,            0,            1830,            2735,            2140,            3273,            715,            ],        [            3272,            3788,            3287,            3703,            3505,            3073,            4689,            3497,            6834,            9851,            6217,            5832,            1389,            4575,            3559,            4196,            3973,            1505,            1732,            2185,            2281,            1897,            3160,            4166,            3147,            3741,            1737,            3285,            1724,            2273,            3152,            4333,            2991,            2607,            3956,            2122,            3842,            4333,            2083,            545,            2688,            2409,            4856,            4089,            3577,            9204,            1584,            3840,            2580,            2191,            2336,            3272,            1932,            2721,            3145,            3437,            2751,            4041,            3417,            2799,            2012,            3156,            2541,            3231,            2692,            2555,            3226,            5114,            1876,            3264,            4157,            2628,            1471,            1243,            2862,            1036,            0,            3771,            3144,            4309,            1719,            ],        [            1522,            2348,            2588,            1953,            1755,            5671,            5374,            4182,            5084,            6102,            4467,            4082,            3369,            2825,            3505,            1575,            1906,            4679,            3119,            3367,            2991,            3877,            4686,            2099,            1606,            1961,            4324,            2330,            4311,            5045,            3288,            3076,            3676,            3921,            185,            5111,            1031,            625,            3469,            4699,            2864,            3143,            1085,            1252,            2316,            5907,            3069,            2607,            3660,            3271,            3416,            3853,            5369,            5045,            2420,            2089,            2002,            2491,            3359,            4325,            2722,            1627,            4067,            3118,            2335,            2997,            4349,            3364,            2956,            3767,            1774,            2964,            4058,            3830,            2696,            3591,            4165,            0,            4670,            687,            4306,            ],        [            5476,            4826,            4325,            5907,            5709,            3242,            2945,            1753,            7614,            8112,            8231,            8616,            2427,            6779,            1815,            3207,            3003,            2543,            2770,            2185,            3319,            1927,            161,            2678,            4130,            3177,            1779,            3546,            1845,            536,            1408,            2433,            1247,            811,            3827,            602,            4673,            4267,            2083,            1583,            2176,            2409,            4727,            4894,            4615,            7465,            2622,            2096,            3618,            3229,            3374,            2194,            1660,            1835,            3636,            3305,            3789,            2297,            1581,            361,            3050,            5360,            615,            1435,            3730,            2239,            788,            7318,            2914,            1653,            5195,            4299,            2509,            2281,            2350,            2074,            1038,            3642,            0,            4329,            1850,            ],        [            1989,            2815,            3055,            2420,            2222,            4984,            4687,            3495,            2189,            3121,            4934,            4549,            2841,            3292,            2818,            888,            1219,            3564,            2591,            2680,            2463,            2899,            3999,            1412,            919,            1274,            3796,            1643,            3783,            4358,            2601,            2389,            2989,            3234,            652,            4424,            1498,            1092,            2782,            4012,            2177,            2456,            1552,            1719,            2783,            2520,            2541,            1920,            2742,            2743,            2498,            3166,            3991,            4358,            1733,            1402,            1315,            1804,            2672,            3638,            2194,            2094,            3380,            2431,            1648,            2310,            3662,            3831,            2904,            3080,            2241,            2749,            3530,            3302,            2009,            3063,            3478,            467,            3983,            0,            3778,            ],        [            2951,            3095,            2594,            3382,            3184,            1515,            2190,            2676,            8532,            9030,            5896,            5511,            1068,            4254,            2738,            4130,            3926,            1184,            1411,            1864,            1960,            1576,            1896,            3601,            2826,            3420,            179,            2964,            5,            1412,            2331,            3512,            2170,            1786,            4750,            902,            3521,            5190,            1762,            2558,            2367,            2088,            5650,            3768,            2884,            8383,            1263,            3019,            2259,            1870,            2015,            2951,            374,            1900,            2824,            3116,            2430,            3220,            2596,            1978,            1691,            2835,            1720,            2410,            2371,            2234,            2405,            4793,            1555,            2443,            3464,            2307,            248,            476,            2541,            715,            2013,            4565,            1735,            3988,            0,            ],        ], 'starts': [11, 39, 13, 28], 'ends': [11, 39, 13, 28]},    'NW-3': {'distance_matrix': [        [            0,            1435,            2716,            3711,            1657,            4138,            6205,            1510,            3034,            1326,            4418,            1770,            7863,            1161,            5082,            1898,            3304,            1459,            1660,            2106,            583,            6974,            658,            2120,            2265,            2119,            1445,            8288,            1854,            ],        [            1926,            0,            959,            1954,            1873,            2381,            4448,            1726,            1277,            1542,            2661,            2342,            6106,            504,            3325,            463,            1547,            802,            225,            671,            1684,            5217,            840,            685,            830,            684,            156,            6531,            565,            ],        [            2786,            934,            0,            995,            2733,            1422,            3800,            2586,            318,            2402,            2013,            3202,            5458,            1364,            2677,            1397,            588,            1662,            1159,            1605,            3687,            4569,            1700,            1619,            1764,            1618,            1016,            5883,            1425,            ],        [            2935,            1929,            995,            0,            2299,            427,            4297,            2112,            677,            1968,            2510,            2768,            5955,            2359,            3174,            2392,            407,            2657,            2154,            2600,            2693,            5066,            2695,            2614,            2759,            2613,            2011,            6380,            2420,            ],        [            1559,            1957,            2770,            2396,            0,            2823,            6259,            736,            2765,            592,            4472,            469,            7917,            1683,            5136,            2420,            2495,            1678,            2182,            2628,            1317,            7028,            2137,            2642,            2787,            2641,            1967,            8342,            2376,            ],        [            3362,            2356,            1422,            427,            2726,            0,            4724,            2539,            1104,            2395,            2937,            3195,            6382,            2786,            3601,            2819,            834,            3084,            2581,            3027,            3120,            5493,            3122,            3041,            3186,            3040,            2438,            6807,            2847,            ],        [            3730,            1878,            1288,            2283,            3677,            2710,            0,            3530,            1606,            3346,            3001,            4146,            6446,            2308,            3665,            2341,            1876,            2606,            2103,            2549,            4631,            5557,            2644,            2563,            2708,            2562,            1960,            6871,            2369,            ],        [            823,            1221,            2034,            3029,            770,            3456,            5523,            0,            2352,            439,            3736,            1239,            7181,            947,            4400,            1684,            2622,            942,            1446,            1892,            581,            6292,            1401,            1906,            2051,            1905,            1231,            7606,            1640,            ],        [            3104,            1252,            318,            677,            2668,            1104,            3620,            2481,            0,            2337,            1833,            3137,            5278,            1682,            2497,            1715,            270,            1980,            1477,            1923,            4005,            4389,            2018,            1937,            2082,            1936,            1334,            5703,            1743,            ],        [            1307,            2288,            3101,            2727,            331,            3154,            6590,            1067,            3096,            0,            4803,            800,            8248,            2014,            5467,            2751,            2826,            2009,            2513,            2959,            1049,            7359,            2468,            2973,            3118,            2972,            2298,            8673,            2707,            ],        [            5517,            3665,            3075,            4070,            5464,            4497,            1787,            5317,            3393,            5133,            0,            5933,            3445,            4095,            664,            4128,            3663,            4393,            3890,            4336,            6418,            2556,            4431,            4350,            4495,            4349,            3747,            3870,            4156,            ],        [            1546,            2335,            3148,            2774,            1301,            3201,            6637,            1114,            3143,            970,            4850,            0,            8295,            2061,            5514,            2798,            2873,            2056,            2560,            3006,            1288,            7406,            2515,            3020,            3165,            3019,            2345,            8720,            2754,            ],        [            9517,            7665,            7075,            8070,            9464,            8497,            5787,            9317,            7393,            9133,            8788,            9933,            0,            8095,            4664,            8128,            7663,            8393,            7890,            8336,            9275,            5705,            8431,            8350,            8495,            8349,            7747,            425,            8156,            ],        [            2138,            286,            1099,            2094,            2085,            2521,            4588,            1938,            1417,            1754,            2801,            2554,            6246,            0,            3465,            749,            1687,            1014,            511,            957,            1896,            5357,            1052,            971,            1116,            970,            296,            6671,            705,            ],        [            4891,            3039,            2449,            3444,            4838,            3871,            1161,            4691,            2767,            4507,            4162,            5307,            7607,            3469,            0,            3502,            3037,            3767,            3264,            3710,            5792,            6718,            3805,            3724,            3869,            3723,            3121,            8032,            3530,            ],        [            2389,            463,            1422,            2417,            2336,            2844,            4911,            2189,            1740,            2005,            3124,            2805,            6569,            967,            3788,            0,            2010,            1265,            586,            484,            2147,            5680,            1303,            222,            325,            253,            619,            6994,            1028,            ],        [            3374,            1522,            588,            407,            2398,            834,            3890,            2211,            270,            2067,            2103,            2867,            5548,            1952,            2767,            1985,            0,            2250,            1747,            2193,            4275,            4659,            2288,            2207,            2352,            2206,            1604,            5973,            2013,            ],        [            1124,            819,            1632,            2596,            1071,            3023,            5121,            924,            1950,            740,            3334,            1540,            6779,            545,            3998,            1282,            2220,            0,            1044,            1490,            882,            5890,            999,            1504,            1649,            1503,            829,            7204,            1238,            ],        [            2151,            225,            1184,            2179,            2098,            2606,            4673,            1951,            1502,            1767,            2886,            2567,            6331,            729,            3550,            586,            1772,            1027,            0,            446,            1909,            5442,            1065,            708,            605,            807,            381,            6756,            790,            ],        [            2597,            671,            1630,            2625,            2544,            3052,            5119,            2397,            1948,            2213,            3332,            3013,            6777,            1175,            3996,            484,            2218,            1473,            446,            0,            2355,            5888,            1511,            262,            159,            737,            827,            7202,            1236,            ],        [            546,            1581,            2394,            2599,            1074,            3026,            5883,            927,            2712,            743,            4096,            1543,            7541,            1307,            4760,            2044,            2982,            883,            1806,            2252,            0,            6652,            1761,            2266,            2411,            2265,            1591,            7966,            2000,            ],        [            7036,            5184,            4594,            5589,            6983,            6016,            3306,            6836,            4912,            6652,            6307,            7452,            889,            5614,            2183,            5647,            5182,            5912,            5409,            5855,            7937,            0,            5950,            5869,            6014,            5868,            5266,            1314,            5675,            ],        [            1799,            2261,            3062,            4057,            3072,            4484,            6551,            2925,            3380,            2741,            4764,            3185,            8209,            1987,            5428,            2724,            3650,            2285,            2486,            2932,            1998,            7320,            0,            2946,            3091,            2945,            2271,            8634,            2680,            ],        [            2611,            685,            1644,            2639,            2558,            3066,            5133,            2411,            1962,            2227,            3346,            3027,            6791,            1189,            4010,            222,            2232,            1487,            708,            262,            2369,            5902,            1525,            0,            103,            475,            841,            7216,            1250,            ],        [            2756,            830,            1789,            2784,            2703,            3211,            5278,            2556,            2107,            2372,            3491,            3172,            6936,            1334,            4155,            325,            2377,            1632,            605,            159,            2514,            6047,            1670,            103,            0,            578,            986,            7361,            1395,            ],        [            2610,            684,            1643,            2638,            2557,            3065,            5132,            2410,            1961,            2226,            3345,            3026,            6790,            1188,            4009,            253,            2231,            1486,            807,            737,            2368,            5901,            1524,            475,            578,            0,            840,            7215,            1249,            ],        [            3285,            1433,            803,            1798,            3232,            2225,            4292,            3085,            1121,            2901,            2505,            3701,            5950,            1863,            3169,            1896,            1391,            2161,            1658,            2104,            4186,            5061,            2199,            2118,            2263,            2117,            0,            6375,            409,            ],        [            9092,            7240,            6650,            7645,            9039,            8072,            5362,            8892,            6968,            8708,            8363,            9508,            1259,            7670,            4239,            7703,            7238,            7968,            7465,            7911,            8850,            5280,            8006,            7925,            8070,            7924,            7322,            0,            7731,            ],        [            2861,            1009,            375,            1370,            2808,            1797,            3875,            2661,            693,            2477,            2088,            3277,            5533,            1439,            2752,            1472,            963,            1737,            1234,            1680,            3762,            4644,            1775,            1694,            1839,            1693,            1091,            5958,            0,            ],        ], 'starts': [23, 18, 9, 10, 28], 'ends': [23, 18, 9, 10, 28]},    'NW-4': {'distance_matrix': [        [            0,            3206,            3852,            1179,            729,            1315,            1029,            1448,            654,            1277,            5945,            1954,            2672,            4102,            5584,            4781,            5258,            3010,            3204,            5531,            5881,            3263,            4422,            3287,            5492,            6137,            3273,            2456,            6208,            4787,            4740,            1791,            4053,            6474,            3885,            3535,            1926,            3812,            3207,            5858,            5612,            2157,            3457,            5997,            5456,            5716,            5571,            3863,            4778,            4495,            3891,            3877,            2840,            2737,            2235,            1390,            3744,            4927,            2375,            4343,            3270,            3859,            2395,            3842,            9056,            2879,            1385,            2966,            4533,            1567,            5202,            2029,            4079,            3303,            3390,            1991,            2657,            843,            1354,            6138,            3388,            3297,            3923,            4957,            4463,            6264,            3851,            693,            5249,            5899,            6070,            652,            2711,            1750,            6189,            6289,            4690,            4426,            4042,            5162,            2321,            5792,            4876,            5429,            4045,            2246,            5557,            2527,            4090,            3484,            5384,            ],        [            998,            0,            3450,            2190,            1207,            1525,            1239,            1762,            864,            1487,            8183,            2268,            4964,            4580,            6062,            5259,            5736,            5302,            3682,            6092,            6359,            3741,            4900,            3765,            5090,            6615,            5682,            4748,            7964,            5265,            5218,            2105,            4531,            7619,            4363,            4013,            2136,            4290,            3685,            6336,            6173,            2471,            2809,            6475,            5054,            5314,            6049,            4341,            5733,            4973,            4369,            4355,            3401,            3215,            2713,            2669,            4718,            5488,            2853,            4904,            3831,            4420,            2956,            4320,            9534,            3357,            1863,            3444,            5094,            2045,            5763,            2507,            4640,            3781,            3868,            2552,            3135,            1404,            1564,            6699,            3866,            3775,            4401,            5435,            4941,            5862,            4329,            1254,            5810,            6460,            6548,            1213,            3189,            1960,            6750,            5780,            5168,            4904,            4520,            5640,            2799,            6353,            5354,            5907,            4523,            2560,            6035,            2737,            4568,            3962,            5945,            ],        [            2008,            1010,            0,            3200,            2217,            2535,            2249,            2772,            1874,            2497,            6403,            3661,            3184,            4754,            7072,            6269,            6746,            3522,            4692,            7102,            7369,            4751,            3831,            4775,            3310,            7625,            3902,            2968,            6184,            4469,            4422,            3498,            5541,            5839,            4663,            5023,            3146,            5300,            4695,            7346,            7183,            3864,            3924,            6956,            3274,            3534,            7059,            5351,            3953,            3904,            5379,            5365,            4411,            4225,            3723,            3679,            5728,            6498,            3863,            5914,            4841,            5430,            3966,            5330,            10544,            4367,            2873,            4454,            6104,            3055,            6773,            3517,            5650,            4791,            4878,            3562,            4145,            2414,            2574,            7709,            4713,            4785,            5411,            6445,            5951,            4082,            5339,            2264,            6820,            7470,            6978,            2223,            4199,            2970,            7760,            6790,            6178,            5914,            4135,            6650,            3809,            7363,            6364,            6917,            4572,            3953,            6834,            3661,            4183,            4972,            6955,            ],        [            690,            3241,            3887,            0,            738,            1466,            1145,            1569,            770,            1393,            6433,            2000,            5401,            3628,            5319,            4516,            4993,            4347,            3675,            5196,            6003,            3353,            3948,            3377,            4953,            5872,            4575,            5185,            5943,            4313,            4266,            1769,            3579,            6209,            3975,            3625,            2042,            3338,            2733,            5593,            5277,            1531,            3928,            5732,            5491,            5751,            5306,            3389,            4470,            4021,            3626,            3612,            2505,            2263,            1761,            745,            3409,            4592,            1901,            4008,            2935,            3524,            2060,            3507,            8791,            2405,            855,            2492,            4198,            1093,            4867,            1555,            3744,            2829,            3125,            1656,            1274,            263,            1470,            5803,            3478,            2823,            3449,            4692,            4198,            5065,            3586,            660,            4914,            5564,            5805,            905,            2237,            1855,            5854,            5954,            4425,            3952,            4132,            4897,            1847,            5457,            4611,            5164,            4135,            1620,            5292,            4278,            4180,            1733,            5049,            ],        [            187,            2905,            3551,            1366,            0,            1502,            1216,            1635,            841,            1464,            5644,            2141,            2859,            3801,            5404,            4601,            5078,            3197,            2903,            5281,            6088,            2962,            4121,            2986,            5191,            5957,            4184,            2643,            6028,            4486,            4439,            1978,            3752,            6294,            3584,            3234,            2113,            3511,            2906,            5678,            5362,            2773,            3156,            5817,            5155,            5415,            5391,            3562,            4477,            4194,            3711,            3697,            2590,            2436,            1934,            1140,            3494,            4677,            2074,            4093,            3020,            3609,            2145,            3592,            8876,            2578,            1084,            2665,            4283,            1266,            4952,            1728,            3829,            3002,            3210,            1741,            2356,            593,            1541,            5888,            3087,            2996,            3622,            4777,            4283,            5963,            3671,            443,            4999,            5649,            5890,            402,            2410,            1937,            5939,            6039,            4510,            4125,            3741,            4982,            2020,            5542,            4696,            5249,            3744,            2862,            5377,            2714,            3789,            3183,            5134,            ],        [            1449,            3358,            2348,            759,            1497,            0,            444,            331,            822,            56,            5534,            837,            1437,            3376,            6078,            5275,            5752,            1775,            1867,            5955,            6762,            2200,            2084,            2224,            2006,            6631,            2038,            1221,            5315,            3100,            3053,            674,            3099,            4970,            2822,            2472,            691,            2788,            1870,            6352,            6036,            1040,            1364,            6491,            1970,            2230,            6065,            2909,            2263,            2157,            4385,            4371,            3264,            3022,            2520,            1168,            4168,            5351,            2660,            4767,            3694,            4283,            2819,            4266,            9550,            2048,            1614,            2334,            4957,            1852,            5626,            2314,            4503,            2511,            3884,            2415,            1347,            1022,            197,            6562,            2325,            3582,            3131,            5451,            4957,            2778,            4345,            1381,            5673,            6323,            6564,            1664,            1623,            389,            6613,            6713,            5184,            3472,            2445,            5656,            2606,            6216,            5370,            5923,            2634,            1129,            6051,            1292,            2493,            1317,            5808,            ],        [            1609,            3415,            4061,            1223,            1818,            692,            0,            795,            378,            532,            6024,            1301,            1927,            5191,            6673,            5870,            6347,            2265,            2331,            6055,            6970,            4352,            2574,            4376,            2496,            7226,            2528,            1711,            5805,            3590,            3543,            1138,            5142,            5460,            4974,            4624,            1181,            4901,            2194,            6947,            6136,            1504,            1854,            7086,            2460,            2720,            6660,            4952,            2753,            2647,            4980,            4966,            3364,            3826,            3324,            1632,            5329,            5451,            3464,            4867,            3794,            4383,            2919,            4931,            10145,            4394,            2474,            4055,            5057,            2656,            5726,            3118,            4603,            4392,            4479,            2515,            1811,            1367,            609,            7126,            4477,            4386,            5012,            6046,            5552,            3268,            4940,            1865,            5773,            6423,            7159,            1176,            3800,            1081,            6713,            6391,            5779,            5515,            2935,            6251,            3410,            6316,            5965,            6518,            3124,            1593,            6646,            1782,            2983,            1781,            5908,            ],        [            1118,            3669,            2679,            428,            1166,            331,            681,            0,            1059,            275,            5865,            754,            1768,            4056,            5747,            4944,            5421,            1533,            1784,            5624,            6431,            3781,            1842,            2141,            2337,            6300,            1796,            1552,            5646,            2858,            2811,            591,            4007,            5301,            2739,            2389,            1022,            3766,            1787,            6021,            5705,            957,            1340,            6160,            2301,            2561,            5734,            3817,            2021,            1915,            4054,            4040,            2933,            2691,            2189,            837,            3837,            5020,            1298,            4436,            3363,            3952,            2488,            3935,            9219,            1965,            1283,            2920,            4626,            1521,            5295,            1983,            4172,            3257,            3553,            2084,            1016,            691,            528,            6231,            2242,            3251,            3877,            5120,            4626,            3109,            4014,            1050,            5342,            5992,            6233,            1333,            1540,            584,            6282,            6382,            4853,            4380,            2203,            5325,            2275,            5885,            5039,            5592,            2899,            1046,            5720,            1268,            2251,            1234,            5477,            ],        [            1657,            3463,            2929,            1326,            1866,            661,            375,            898,            0,            623,            6115,            1404,            2018,            3957,            6068,            5265,            5742,            2356,            2282,            5945,            6752,            2818,            2665,            2842,            2587,            6621,            2619,            1802,            5896,            3681,            3634,            1241,            3680,            5551,            3440,            3090,            1272,            3369,            2285,            6342,            6026,            1607,            1945,            6481,            2551,            2811,            6055,            3490,            2844,            2738,            4375,            4361,            3254,            3874,            3372,            1735,            4158,            5341,            3512,            4757,            3684,            4273,            2809,            4256,            9540,            2666,            2522,            4103,            4947,            2704,            5616,            3166,            4493,            4440,            3874,            2405,            1914,            1589,            700,            6552,            3319,            4434,            5060,            5441,            4947,            3359,            4335,            1107,            5663,            6313,            6554,            1066,            2190,            1096,            6603,            6703,            5174,            4053,            3026,            5646,            3458,            6206,            5360,            5913,            3215,            1696,            6041,            1873,            3074,            1884,            5798,            ],        [            1393,            3414,            2404,            703,            1441,            56,            406,            275,            784,            0,            5590,            781,            1493,            3432,            6022,            5219,            5696,            1831,            1811,            5899,            6706,            2144,            2140,            2168,            2062,            6575,            2094,            1277,            5371,            3156,            3109,            618,            3205,            5026,            2766,            2416,            747,            2844,            1814,            6296,            5980,            984,            1420,            6435,            2026,            2286,            6009,            3015,            2319,            2213,            4329,            4315,            3208,            2966,            2464,            1112,            4112,            5295,            1573,            4711,            3638,            4227,            2763,            4210,            9494,            1992,            1558,            2278,            4901,            1796,            5570,            2258,            4447,            2455,            3828,            2359,            1291,            966,            253,            6506,            2269,            3526,            3075,            5395,            4901,            2834,            4289,            1325,            5617,            6267,            6508,            1608,            1567,            445,            6557,            6657,            5128,            3578,            2501,            5600,            2550,            6160,            5314,            5867,            2926,            1073,            5995,            1348,            2549,            1261,            5752,            ],        [            4798,            5164,            3630,            3623,            4828,            3087,            3779,            3195,            5452,            3031,            0,            2328,            1931,            1085,            1928,            3070,            2290,            1719,            2445,            5573,            4170,            2304,            1593,            2379,            1708,            4426,            1330,            2147,            1355,            404,            357,            2550,            1622,            1281,            2091,            2028,            2724,            1569,            2448,            4147,            5654,            3248,            2698,            1689,            1448,            1189,            2227,            1265,            1225,            1779,            3060,            3356,            3398,            2981,            3261,            4837,            3719,            4969,            3401,            4385,            2857,            3901,            5705,            1755,            8877,            2666,            4031,            2489,            4575,            4213,            5244,            3368,            4121,            1788,            3582,            5301,            3683,            4731,            3110,            4326,            2141,            3123,            2076,            3198,            2670,            1457,            3289,            4804,            5291,            4674,            1982,            5013,            2854,            2659,            5016,            4044,            2979,            1995,            1563,            3451,            3371,            4619,            3165,            3670,            2000,            3337,            1838,            1858,            1611,            2890,            5426,            ],        [            3465,            3670,            2660,            1728,            3495,            1192,            1411,            1300,            1789,            1136,            3172,            0,            901,            2290,            3961,            3158,            3635,            689,            833,            4132,            4258,            1369,            998,            1393,            1602,            4514,            952,            653,            3036,            2014,            1967,            510,            2013,            3302,            1991,            1641,            435,            1702,            836,            4235,            4213,            645,            496,            3682,            1920,            1833,            3948,            1823,            1177,            1071,            2268,            2254,            2412,            1995,            1928,            1578,            2617,            3528,            1274,            2944,            1871,            2460,            4719,            1958,            7433,            1217,            2698,            1503,            3134,            2880,            3803,            2035,            2680,            1680,            1767,            4315,            992,            1991,            1215,            4414,            1494,            2137,            2039,            3334,            2840,            2041,            2228,            2350,            3850,            4500,            4447,            3680,            1405,            764,            4790,            3679,            3067,            2386,            1359,            3539,            2038,            4393,            3253,            3806,            1548,            734,            3934,            424,            1407,            547,            3985,            ],        [            4231,            3233,            2223,            2003,            4440,            1467,            1848,            1575,            2226,            1411,            2821,            621,            0,            1939,            4107,            3304,            3781,            338,            1605,            4278,            4404,            1464,            647,            1539,            1251,            4660,            601,            216,            2685,            1663,            1616,            930,            1662,            2951,            1251,            1188,            793,            1351,            1608,            4381,            4359,            1296,            923,            3331,            1382,            1482,            3869,            1472,            826,            720,            2414,            2400,            2558,            2141,            2421,            2229,            2763,            3674,            2561,            3090,            2017,            2606,            4865,            1607,            7579,            1826,            3191,            1649,            3280,            3373,            3949,            2528,            2826,            1640,            1913,            4461,            1643,            2266,            1523,            4560,            1301,            2283,            1688,            3480,            2986,            1690,            2374,            3964,            3996,            4646,            3624,            4446,            2014,            1039,            4936,            3825,            3213,            2035,            1008,            3685,            2531,            4539,            3399,            3952,            1197,            1385,            3480,            477,            1056,            981,            4131,            ],        [            3521,            4870,            3860,            4137,            3551,            3234,            4550,            2858,            4175,            3178,            2151,            2337,            2232,            0,            2377,            1574,            2051,            2020,            1511,            3288,            2674,            1189,            1094,            1213,            2009,            2930,            1631,            1853,            2015,            993,            946,            2552,            663,            2281,            1194,            1461,            2473,            484,            1514,            2651,            3369,            2314,            1764,            2661,            1749,            1490,            3199,            180,            1526,            1167,            1201,            1191,            2122,            1704,            1984,            3560,            1554,            2684,            2124,            2100,            1534,            1616,            4429,            670,            5849,            1425,            2754,            1212,            2290,            2936,            2959,            2091,            1836,            703,            1114,            4025,            2406,            3454,            3257,            2830,            1244,            1561,            751,            1738,            1242,            1758,            1243,            3527,            3006,            3656,            2954,            3736,            1613,            2806,            3946,            2416,            1483,            1036,            1086,            1955,            2094,            3549,            1669,            2210,            1089,            2403,            2810,            2159,            1134,            1956,            3141,            ],        [            5894,            7231,            6221,            6510,            5942,            5417,            6923,            5041,            6548,            5361,            1624,            4966,            3555,            2704,            0,            1099,            512,            3343,            3694,            4847,            2227,            3372,            3294,            3396,            3332,            2483,            2954,            3771,            1134,            2028,            1981,            4735,            2925,            1400,            3994,            3644,            4348,            2684,            3697,            2204,            4928,            4497,            3947,            403,            3072,            2813,            1501,            2735,            2849,            3367,            2334,            2630,            3867,            3887,            4167,            5933,            2993,            4243,            4307,            3659,            3326,            3175,            6174,            2816,            5402,            3608,            4937,            3395,            3849,            5309,            4518,            4274,            3395,            2819,            2856,            5770,            4589,            5827,            7248,            2383,            3497,            3744,            2858,            645,            1944,            3081,            2563,            5900,            4565,            2731,            620,            6109,            3796,            4989,            3073,            3318,            1036,            3298,            3187,            1508,            4277,            2676,            1222,            1151,            3289,            4586,            1386,            3482,            3235,            4139,            3154,            ],        [            6208,            8703,            7156,            6824,            6256,            3811,            7237,            3435,            6862,            3755,            2559,            2914,            2232,            1099,            1542,            0,            1216,            2513,            2088,            3241,            1839,            1766,            1689,            1790,            2502,            2095,            2124,            2448,            2069,            2963,            2916,            3129,            1320,            2335,            2388,            2038,            3050,            1079,            2091,            1816,            3322,            2891,            2341,            1858,            2357,            2098,            1432,            1130,            2045,            1762,            728,            1024,            2261,            2281,            2561,            6247,            1387,            2637,            2701,            2053,            1720,            1569,            4568,            1210,            5014,            2002,            3331,            1789,            2243,            5623,            2912,            2668,            1789,            1214,            1250,            4164,            2983,            6141,            7562,            1995,            1891,            2138,            1092,            818,            339,            2366,            957,            6214,            2959,            2343,            1931,            6423,            2190,            3383,            2685,            1712,            648,            1693,            1681,            1120,            2671,            2288,            834,            1290,            1684,            2980,            1418,            2691,            1729,            2533,            2766,            ],        [            6084,            8579,            6852,            6700,            6132,            7399,            7113,            4347,            6738,            7361,            2255,            4272,            4186,            2011,            651,            912,            0,            3974,            3000,            2756,            1715,            2678,            2601,            2702,            3963,            1971,            3585,            4402,            1765,            2659,            2612,            4041,            2232,            2031,            3300,            2950,            3962,            1991,            3003,            1692,            3372,            3803,            3253,            1034,            3703,            3444,            2132,            2042,            3480,            2674,            1640,            1936,            3173,            3193,            3473,            6123,            2299,            3099,            3613,            2965,            2632,            2481,            6366,            2122,            4890,            2914,            4243,            2701,            3155,            5499,            2824,            3580,            2701,            2126,            2162,            5962,            3895,            6017,            7438,            1871,            2803,            3050,            2004,            1166,            1251,            3712,            1869,            6090,            3141,            2219,            1251,            6299,            3102,            4295,            2561,            2609,            524,            2605,            2593,            996,            3583,            2164,            710,            1672,            2596,            3892,            2017,            4113,            2641,            3445,            2642,            ],        [            3620,            3941,            2931,            2384,            3650,            1848,            2556,            1956,            4274,            1792,            2920,            1089,            708,            1373,            3769,            2966,            3443,            0,            1267,            3940,            4066,            1126,            309,            1201,            1409,            4322,            759,            924,            2784,            1762,            1715,            1311,            1324,            3050,            913,            850,            1501,            1013,            1270,            4043,            4021,            1301,            735,            3430,            1727,            1571,            3756,            1134,            984,            382,            2076,            2062,            2220,            1803,            2083,            3659,            2153,            3336,            2223,            2752,            1679,            2268,            4527,            1269,            7241,            1488,            2853,            1311,            2942,            3035,            3611,            2190,            2488,            1302,            1575,            4123,            2505,            3553,            1871,            4547,            963,            1945,            1350,            3142,            2648,            1789,            2036,            3626,            3658,            4308,            4255,            3835,            1676,            1420,            4598,            4698,            2875,            1697,            856,            3347,            2193,            4201,            3061,            3614,            859,            1390,            3742,            619,            904,            943,            3793,            ],        [            3058,            4117,            3107,            1808,            3088,            1531,            1938,            1474,            2316,            1475,            3155,            826,            884,            1725,            3207,            2404,            2881,            672,            0,            3378,            3504,            615,            551,            639,            1585,            3760,            935,            1100,            3831,            1997,            1950,            1168,            1676,            4097,            1237,            887,            962,            1435,            218,            3481,            3459,            930,            253,            3620,            1903,            1816,            3194,            1486,            1160,            812,            1514,            1500,            1658,            1241,            1521,            1863,            1591,            2774,            1661,            2190,            1117,            1706,            3965,            1465,            6679,            545,            2291,            749,            2380,            2473,            3049,            1628,            1926,            926,            1013,            3561,            1277,            2991,            1554,            3985,            740,            1383,            1546,            2580,            2086,            2024,            1474,            3064,            3096,            3746,            3693,            3273,            733,            1103,            4036,            4136,            2313,            2049,            1394,            2785,            1631,            3639,            2499,            3052,            1397,            1019,            3180,            622,            1442,            572,            3231,            ],        [            5432,            7927,            8573,            6048,            5480,            5238,            6461,            4862,            6086,            5182,            4993,            4341,            4399,            3516,            2989,            2186,            2663,            4187,            3515,            0,            1021,            3193,            3899,            3217,            4793,            1559,            4415,            4615,            4503,            4018,            3971,            4556,            3530,            4769,            3815,            3465,            4477,            3289,            3518,            1280,            295,            4318,            3768,            3372,            4774,            4515,            3866,            3340,            4310,            3972,            2758,            2785,            3405,            3708,            3988,            5471,            2235,            819,            4128,            1403,            2817,            1887,            5712,            2837,            6461,            3429,            4758,            3216,            3091,            4847,            544,            4095,            1667,            3289,            3011,            5308,            4410,            5365,            6786,            821,            3318,            3565,            3400,            3252,            2756,            4905,            2718,            5438,            861,            1185,            3589,            5647,            3617,            4810,            872,            1575,            2139,            3903,            3972,            1631,            4098,            475,            2281,            3724,            3975,            4407,            3852,            4137,            4020,            3960,            670,            ],        [            6634,            9129,            9775,            7250,            6682,            7949,            7663,            8082,            7288,            7911,            3724,            4525,            3397,            2264,            1968,            1165,            1642,            3793,            3253,            1858,            0,            2931,            2854,            2955,            3782,            538,            3404,            3613,            3234,            2766,            2719,            4294,            2485,            3500,            3553,            3203,            4215,            2244,            3256,            259,            1939,            4056,            3506,            2351,            3522,            3263,            2597,            2295,            3210,            2927,            1893,            2189,            4252,            3446,            3726,            6673,            3082,            1666,            3866,            2250,            2885,            2734,            6916,            2375,            5440,            3167,            6909,            2954,            3408,            6049,            1391,            3833,            2514,            2379,            2415,            6512,            4148,            6567,            7988,            438,            3056,            3303,            2257,            1983,            1504,            3531,            2122,            6640,            1708,            786,            2568,            6849,            3355,            4548,            1128,            1176,            1118,            2858,            2846,            610,            3836,            731,            1260,            2455,            2849,            4145,            2583,            3875,            2894,            3698,            1209,            ],        [            2916,            4439,            3429,            2003,            2946,            2045,            2260,            1669,            3570,            1989,            3663,            1148,            1206,            1930,            3412,            2609,            3086,            994,            322,            3583,            3709,            0,            633,            844,            1907,            3965,            1257,            1422,            4036,            2505,            2458,            1363,            1881,            4302,            1442,            1092,            1284,            1640,            325,            3686,            3664,            1125,            575,            3825,            2573,            2314,            3399,            1691,            1482,            800,            1719,            1705,            1863,            1446,            1379,            2955,            1796,            2979,            1250,            2395,            1322,            1911,            4170,            1670,            6884,            668,            2149,            954,            2585,            2331,            3254,            1486,            2131,            1131,            1218,            3766,            1420,            2849,            2068,            4190,            945,            1588,            1751,            2785,            2291,            2532,            1679,            2922,            3301,            3951,            3898,            3131,            856,            1617,            4241,            4341,            2518,            2254,            1599,            2990,            1489,            3844,            2704,            3257,            1602,            1214,            3385,            944,            1647,            767,            3436,            ],        [            3405,            3890,            2880,            2733,            3435,            1813,            2505,            2399,            4059,            1757,            2465,            1054,            657,            1158,            2766,            1963,            2440,            906,            1052,            3725,            3063,            911,            0,            986,            895,            4107,            517,            873,            2329,            1307,            1260,            1276,            1109,            2595,            698,            635,            1450,            798,            327,            3040,            3806,            1855,            712,            2975,            1213,            1126,            3513,            919,            470,            167,            1861,            1847,            2005,            1588,            1868,            3444,            1938,            3121,            2008,            2537,            1464,            2053,            4312,            1054,            6238,            1273,            2638,            1096,            2727,            2820,            3396,            1975,            2273,            1087,            1360,            3908,            2290,            3338,            1836,            4332,            748,            1730,            1135,            2927,            2433,            1334,            1821,            3411,            3443,            4093,            3268,            3620,            1461,            1385,            4383,            4483,            1872,            1482,            652,            2344,            1978,            3986,            2058,            3399,            644,            1944,            3124,            584,            700,            1497,            3578,            ],        [            2758,            5108,            4022,            2086,            2788,            2128,            2343,            1752,            3412,            2072,            2819,            1231,            1799,            1272,            2907,            2104,            2581,            1587,            405,            3078,            3204,            264,            775,            0,            1576,            3460,            1198,            2015,            2683,            1661,            1614,            1446,            1223,            2949,            598,            248,            1367,            912,            408,            3181,            3159,            1208,            658,            3320,            1729,            1470,            2894,            1033,            1093,            848,            1214,            1200,            1358,            941,            1221,            2797,            1291,            2474,            1361,            1890,            817,            1406,            3665,            1168,            6379,            626,            1991,            449,            2080,            2173,            2749,            1328,            1626,            626,            713,            3261,            1643,            2691,            2151,            3685,            101,            1083,            1249,            2280,            1786,            1688,            1174,            2764,            2796,            3446,            3393,            2973,            814,            1700,            3736,            3836,            2013,            1596,            755,            2485,            1331,            3339,            2199,            2752,            758,            1297,            2880,            1027,            803,            850,            2931,            ],        [            4060,            4266,            3256,            2626,            4090,            2090,            2471,            2198,            5130,            2034,            2638,            1590,            680,            1756,            3387,            3406,            3749,            981,            1707,            4380,            4506,            1566,            855,            1641,            0,            4762,            592,            896,            2502,            1480,            1433,            1553,            1978,            2768,            1353,            1290,            1416,            1667,            1710,            4483,            4461,            2510,            1960,            3148,            318,            578,            3686,            1936,            643,            1041,            2516,            2502,            2660,            2243,            2523,            4099,            2593,            3776,            2663,            3192,            2119,            2708,            4967,            1923,            7681,            1928,            3293,            1751,            3382,            3475,            4051,            2630,            2928,            1928,            2015,            4563,            2945,            3993,            2113,            4987,            1403,            2385,            2004,            3582,            3088,            1126,            2476,            4066,            4098,            4748,            3441,            4275,            2116,            1662,            5038,            5138,            3315,            2351,            825,            3787,            2633,            4641,            3501,            4054,            1262,            2599,            3297,            1120,            873,            2152,            4233,            ],        [            7094,            9589,            10235,            7710,            7142,            8409,            8123,            8542,            7748,            8371,            4184,            5880,            3857,            2724,            2428,            1625,            2102,            4253,            4608,            2104,            726,            4286,            3314,            4310,            4242,            0,            3864,            4073,            3694,            3226,            3179,            5649,            2945,            3960,            4908,            4558,            5570,            2704,            4611,            279,            2185,            5411,            4861,            2811,            3982,            3723,            3057,            2755,            3670,            3387,            2353,            2649,            4498,            4801,            5081,            7133,            3328,            1912,            5221,            2496,            3910,            2980,            7376,            2835,            5900,            4522,            7369,            4309,            4184,            6509,            1637,            5188,            2760,            2839,            2875,            6972,            5503,            7027,            8448,            882,            4411,            4658,            2717,            2443,            1964,            3991,            2582,            7100,            1954,            1032,            3028,            7309,            4710,            5903,            1572,            1422,            1578,            3318,            3306,            1070,            5191,            1175,            1720,            2915,            3309,            5500,            3043,            5230,            3354,            5053,            1455,            ],        [            3642,            3834,            2824,            2293,            3672,            1757,            2449,            1865,            2827,            1701,            2220,            998,            601,            1338,            2969,            2988,            3331,            389,            1289,            3962,            4088,            1148,            437,            1223,            650,            4344,            0,            817,            2084,            1062,            1015,            1220,            1560,            2350,            935,            872,            1394,            1249,            1292,            4065,            4043,            1456,            937,            2730,            968,            881,            3268,            1518,            225,            771,            2098,            2084,            2242,            1825,            2105,            3681,            2175,            3358,            2245,            2774,            1701,            2290,            4549,            1505,            7263,            1510,            2875,            1333,            2964,            3057,            3633,            2212,            2510,            1510,            1597,            4145,            2527,            3575,            1780,            4569,            985,            1967,            1586,            3164,            2670,            1089,            2058,            3648,            3680,            4330,            3023,            3857,            1698,            1329,            4620,            4720,            2897,            1933,            407,            3369,            2215,            4223,            3083,            3636,            844,            1545,            2879,            528,            455,            1734,            3815,            ],        [            4015,            3017,            2007,            1787,            4224,            1251,            1632,            1359,            2010,            1195,            3037,            877,            216,            2155,            3535,            2732,            3209,            554,            1821,            4494,            3832,            1680,            863,            1755,            1467,            4088,            817,            0,            2901,            1879,            1832,            714,            1878,            3167,            1467,            1404,            577,            1567,            1190,            3809,            4575,            1080,            1001,            3547,            1166,            1426,            4085,            1688,            1042,            936,            2630,            2616,            2774,            2357,            2637,            2196,            2707,            3890,            2777,            3306,            2233,            2822,            5081,            1823,            7007,            2042,            3407,            1865,            3496,            3589,            4165,            2744,            3042,            1856,            2129,            4677,            1427,            2050,            1307,            3988,            1517,            2499,            1904,            2896,            2400,            1906,            2590,            4271,            4212,            4862,            3840,            4230,            2230,            823,            5152,            5252,            2641,            2251,            1224,            3113,            2747,            4755,            2827,            3368,            1413,            1169,            3696,            693,            1272,            1357,            4347,            ],        [            9412,            6481,            5471,            4113,            9460,            3577,            4269,            3685,            10066,            3521,            490,            2818,            2421,            1575,            1269,            2411,            1631,            2209,            2935,            4914,            3511,            2794,            2083,            3463,            2198,            3767,            1820,            2637,            0,            894,            847,            3040,            2238,            650,            2581,            2518,            3214,            2059,            2938,            3488,            4995,            3738,            3188,            1030,            1938,            1679,            1568,            1755,            1715,            2269,            2401,            2697,            3934,            3471,            4234,            9451,            3060,            4310,            3891,            3726,            3393,            3242,            6241,            2883,            8218,            3156,            4521,            2979,            3916,            8827,            4585,            3858,            3462,            2278,            2923,            5837,            4173,            9345,            3600,            3667,            2631,            3811,            2326,            2575,            2011,            1947,            2630,            9418,            4632,            4015,            1323,            9627,            3344,            3149,            4357,            3385,            2320,            2611,            2053,            2792,            3861,            3960,            2506,            3047,            2490,            3827,            1179,            2348,            2101,            3380,            4767,            ],        [            4394,            4760,            3750,            3219,            4424,            2683,            3375,            2791,            5048,            2627,            1368,            1924,            1527,            681,            2117,            1788,            2479,            1315,            2041,            4159,            2888,            1900,            1189,            1975,            1304,            3144,            926,            1743,            1232,            0,            163,            2146,            1344,            1498,            1687,            1624,            2320,            1165,            2044,            2865,            4240,            2844,            2294,            1878,            1044,            785,            2416,            861,            821,            1375,            1646,            1942,            2994,            2577,            2857,            4433,            2305,            3555,            2997,            2971,            2453,            2487,            5301,            1351,            6063,            2262,            3627,            2085,            3161,            3809,            3830,            2964,            2707,            1384,            1795,            4897,            3279,            4327,            2706,            3044,            1737,            2242,            1432,            1952,            1456,            1053,            1875,            4400,            3877,            3392,            2171,            4609,            2450,            2255,            3734,            2630,            1697,            1717,            1159,            2169,            2967,            3337,            1883,            2424,            1596,            2933,            2027,            1454,            1207,            2486,            3815,            ],        [            4441,            4807,            3797,            3266,            4471,            2730,            3422,            2838,            5095,            2674,            1415,            1971,            1574,            728,            2164,            1835,            2526,            1362,            2088,            4206,            2935,            1947,            1236,            2022,            1351,            3191,            973,            1790,            1279,            47,            0,            2193,            1391,            1545,            1734,            1671,            2367,            1212,            2091,            2912,            4287,            2891,            2341,            1925,            1091,            832,            2463,            908,            868,            1422,            1693,            1989,            3041,            2624,            2904,            4480,            2352,            3602,            3044,            3018,            2500,            2534,            5348,            1398,            6110,            2309,            3674,            2132,            3208,            3856,            3877,            3011,            2754,            1431,            1842,            4944,            3326,            4374,            2753,            3091,            1784,            2289,            1479,            1999,            1503,            1100,            1922,            4447,            3924,            3439,            2218,            4656,            2497,            2302,            3781,            2677,            1744,            1764,            1206,            2216,            3014,            3384,            1930,            2471,            1643,            2980,            2074,            1501,            1254,            2533,            3862,            ],        [            1908,            3455,            2445,            1218,            1956,            682,            1191,            790,            1569,            626,            3493,            231,            1222,            2636,            4118,            3315,            3792,            1010,            1193,            4289,            4415,            1526,            1319,            1550,            1923,            4671,            1273,            1028,            3357,            2335,            2288,            0,            2587,            3623,            2148,            1798,            497,            2346,            1196,            4392,            4370,            366,            817,            4531,            2010,            2154,            4105,            2397,            1498,            1392,            2425,            2411,            2569,            2152,            1135,            1299,            2502,            3685,            995,            3101,            2028,            2617,            3278,            2376,            7590,            1374,            2087,            1660,            3291,            2269,            3960,            1424,            2837,            1837,            1924,            2874,            713,            1481,            705,            4896,            1651,            2294,            2457,            3491,            2997,            2362,            2385,            1840,            4007,            4657,            4604,            2123,            949,            254,            4947,            5047,            3224,            2960,            1680,            3696,            1437,            4550,            3410,            3963,            2308,            455,            4091,            745,            1728,            643,            4142,            ],        [            3546,            5407,            4397,            4162,            3576,            3330,            4022,            2883,            4200,            3274,            2117,            2571,            2174,            553,            2866,            1889,            3228,            1962,            1536,            3313,            2989,            1214,            1119,            1238,            1951,            3245,            1573,            2390,            1981,            959,            912,            2577,            0,            2247,            1219,            1486,            2967,            509,            1539,            2966,            3394,            2339,            1789,            2627,            1691,            1432,            3165,            733,            1468,            1192,            1747,            1216,            2147,            1729,            2009,            3585,            1579,            2709,            2149,            2125,            1559,            1641,            4454,            695,            6164,            1450,            2779,            1237,            2315,            2961,            2984,            2116,            1861,            728,            1139,            4050,            2431,            3479,            3353,            3145,            1269,            1586,            776,            2053,            1557,            1700,            1268,            3552,            3031,            3493,            2700,            3761,            1638,            2902,            3835,            2731,            1798,            373,            1111,            2270,            2119,            3574,            1984,            2525,            1114,            2428,            2776,            2101,            1159,            1981,            3166,            ],        [            6254,            5831,            4821,            5650,            6302,            4891,            5221,            5222,            6695,            4947,            1041,            3369,            2972,            2126,            1562,            2704,            1924,            2760,            3486,            5207,            3804,            3732,            2634,            3756,            2749,            4060,            2371,            3188,            551,            1445,            1398,            3591,            2789,            0,            3132,            3069,            3765,            2610,            3489,            3781,            5288,            4289,            3739,            1323,            2576,            2317,            1861,            2306,            2266,            2820,            2694,            2990,            4227,            4247,            4527,            6293,            3353,            4603,            4667,            4019,            3686,            3535,            6534,            3176,            6979,            3968,            5297,            3755,            4209,            5669,            4878,            4634,            3755,            2829,            3216,            6130,            4949,            6187,            4896,            3960,            3182,            4104,            3218,            2057,            2304,            2940,            2923,            6260,            4925,            4308,            1396,            6469,            4156,            3700,            4650,            3678,            2613,            3162,            2604,            3085,            4637,            4253,            2799,            3340,            3041,            4378,            1472,            2899,            2652,            3931,            5060,            ],        [            3322,            4784,            3774,            2570,            3352,            2692,            3399,            2316,            3976,            2636,            2571,            1948,            1551,            1024,            2632,            1829,            2306,            1339,            969,            3642,            2929,            828,            527,            903,            1328,            3185,            950,            1767,            2435,            1413,            1366,            2010,            975,            2701,            0,            471,            2344,            664,            972,            2906,            3723,            1772,            1222,            3081,            1481,            1222,            2607,            785,            845,            600,            1451,            1441,            1922,            1505,            1785,            3361,            1804,            3038,            1925,            2454,            1381,            1970,            4229,            920,            6104,            1190,            2555,            1013,            2644,            2737,            3313,            1892,            2190,            953,            1364,            3825,            2207,            3255,            2715,            3085,            665,            1811,            1001,            1993,            1497,            1440,            1493,            3328,            3360,            4010,            3106,            3537,            1378,            2264,            3775,            2671,            1738,            1348,            507,            2210,            1895,            3903,            1924,            2465,            510,            1861,            2593,            1478,            555,            1414,            3495,            ],        [            3087,            4784,            3774,            2335,            3117,            2457,            2966,            2081,            3741,            2401,            2571,            1560,            1551,            1024,            2632,            1829,            2306,            1339,            734,            3407,            2929,            593,            527,            668,            1328,            3185,            950,            1767,            2435,            1413,            1366,            1775,            975,            2701,            350,            0,            1696,            664,            737,            2906,            3488,            1537,            987,            3081,            1481,            1222,            2607,            785,            845,            600,            1543,            1529,            1687,            1270,            1550,            3126,            1620,            2803,            1690,            2219,            1146,            1735,            3994,            920,            6104,            955,            2320,            778,            2409,            2502,            3078,            1657,            1955,            953,            1042,            3590,            1972,            3020,            2480,            4014,            430,            1412,            1001,            1993,            1497,            1440,            1493,            3093,            3125,            3775,            3106,            3302,            1143,            2029,            4065,            4165,            1738,            1348,            507,            2210,            1660,            3668,            1924,            2465,            510,            1626,            2593,            1478,            555,            1179,            3260,            ],        [            2010,            3364,            2354,            1320,            2058,            784,            1065,            892,            1443,            728,            3349,            264,            747,            2467,            4138,            3335,            3812,            866,            1010,            4309,            4435,            1546,            1175,            1570,            1779,            4691,            1129,            531,            3213,            2191,            2144,            247,            2190,            3479,            2168,            1818,            0,            1879,            1013,            4412,            4390,            613,            673,            3859,            1513,            1773,            4125,            2000,            1354,            1248,            2445,            2431,            2589,            2172,            1382,            1729,            2522,            3705,            1242,            3121,            2048,            2637,            3380,            2135,            7610,            1394,            2334,            1680,            3311,            2516,            3980,            1671,            2857,            1857,            1944,            2976,            960,            1583,            807,            4916,            1671,            2314,            2216,            3511,            3017,            2218,            2405,            1942,            4027,            4677,            4152,            2225,            1196,            356,            4967,            5067,            3244,            2563,            1536,            3716,            1684,            4570,            3430,            3983,            1725,            702,            4008,            601,            1584,            731,            4162,            ],        [            3305,            4584,            3574,            3921,            3335,            3018,            4334,            2642,            3959,            2962,            2401,            2121,            1351,            558,            2166,            1363,            1840,            1632,            1295,            3072,            2463,            973,            808,            997,            1621,            2719,            1243,            1567,            2265,            1243,            1196,            2336,            509,            2531,            908,            845,            2144,            0,            1298,            2440,            3153,            2098,            1548,            2567,            1999,            1740,            2141,            319,            1164,            881,            985,            975,            1906,            1488,            1768,            3344,            1338,            2468,            1908,            1884,            1318,            1400,            4213,            454,            5638,            1209,            2538,            996,            2074,            2720,            2743,            1875,            1620,            487,            898,            3809,            2190,            3238,            3041,            2619,            958,            1345,            535,            1527,            1031,            1733,            1027,            3311,            2790,            2967,            2640,            3520,            1397,            2590,            3309,            2205,            1272,            882,            800,            1744,            1878,            3333,            1458,            1999,            803,            2187,            2127,            1278,            848,            1740,            2925,            ],        [            2703,            4226,            3216,            1710,            2733,            1832,            2341,            1456,            3357,            1776,            2792,            1118,            993,            1717,            3199,            2396,            2873,            781,            109,            3370,            3496,            607,            327,            631,            1222,            3752,            844,            1209,            3823,            1634,            1587,            1150,            1668,            4089,            1229,            879,            1071,            1427,            0,            3473,            3451,            912,            362,            3612,            1540,            1453,            3186,            1478,            797,            494,            1506,            1492,            1650,            1233,            1166,            2742,            1583,            2766,            1037,            2182,            1109,            1698,            3957,            1457,            6671,            455,            1936,            741,            2372,            2118,            3041,            1273,            1918,            918,            1005,            3553,            1207,            2636,            1855,            3977,            606,            1375,            1538,            2572,            2078,            1661,            1466,            2709,            3088,            3738,            3685,            2918,            643,            1404,            4028,            4128,            2305,            2041,            1386,            2777,            1276,            3631,            2491,            3044,            1389,            1001,            3172,            731,            1434,            554,            3223,            ],        [            6815,            9366,            10012,            7431,            6863,            8130,            7844,            7811,            7469,            8092,            3905,            5101,            3578,            2445,            2149,            1346,            1823,            5001,            4329,            1825,            447,            4007,            3035,            4031,            3963,            279,            3585,            3794,            3415,            2947,            2900,            5370,            2666,            3681,            4629,            4279,            5291,            2425,            4332,            0,            1906,            5132,            4582,            2532,            3703,            3444,            2778,            2476,            3391,            3108,            2074,            2370,            4219,            4522,            4802,            6854,            3049,            1633,            4942,            2217,            3631,            2701,            7097,            2556,            5621,            4243,            5572,            4030,            3905,            6230,            1358,            4909,            2481,            2560,            2596,            6693,            5224,            6748,            8169,            603,            4132,            4379,            2438,            2164,            1685,            3712,            2303,            6821,            1675,            753,            2749,            7030,            4431,            5624,            1293,            1143,            1299,            3039,            3027,            791,            4912,            896,            1441,            2636,            3030,            5221,            2764,            4951,            3075,            4774,            1176,            ],        [            7360,            9911,            10557,            7976,            7408,            5177,            8389,            4801,            8014,            8637,            4450,            4280,            4338,            2990,            2694,            1891,            2368,            4126,            3454,            772,            726,            3132,            3580,            3156,            4732,            1264,            4354,            4554,            3960,            3492,            3445,            4495,            3211,            4226,            3754,            3404,            4416,            2970,            3457,            985,            0,            4257,            3707,            3077,            4248,            3989,            3323,            3021,            4249,            3653,            2619,            2915,            3344,            3647,            3927,            7399,            2174,            758,            4067,            1342,            2756,            1826,            5651,            2776,            6166,            3368,            4697,            3155,            3030,            6775,            483,            4034,            1606,            3228,            2950,            5247,            4349,            7293,            8714,            526,            3257,            3504,            2983,            2709,            2230,            4257,            2657,            7366,            800,            1124,            3294,            7575,            3556,            4749,            577,            1514,            1844,            3584,            3911,            1336,            4037,            180,            1986,            3181,            3914,            4346,            3309,            4076,            3959,            3899,            609,            ],        [            1488,            3852,            2842,            798,            1536,            1079,            1588,            544,            1966,            1023,            4087,            628,            1435,            2354,            3836,            3033,            3510,            1223,            911,            4007,            4133,            1244,            1241,            1268,            2844,            4389,            1486,            1425,            4460,            2929,            2882,            397,            2305,            4726,            1866,            1516,            894,            2064,            914,            4110,            4088,            0,            707,            4249,            2997,            2738,            3823,            2115,            1711,            1408,            2143,            2129,            2287,            1870,            769,            933,            2220,            3403,            629,            2819,            1746,            2335,            4594,            2094,            7308,            1092,            1721,            1378,            3009,            1903,            3678,            1058,            2555,            1555,            1642,            4190,            347,            1061,            1102,            4614,            1369,            2012,            2175,            3209,            2715,            2956,            2103,            1420,            3725,            4375,            4322,            1703,            667,            651,            4665,            4765,            2942,            2678,            2023,            3414,            1071,            4268,            3128,            3681,            2026,            89,            3809,            1111,            2071,            361,            3860,            ],        [            2969,            4255,            3245,            1638,            2999,            1760,            2269,            1384,            2647,            1704,            3716,            1046,            1022,            1983,            3465,            2662,            3139,            810,            337,            3636,            3762,            873,            689,            897,            1723,            4018,            1073,            1238,            4089,            2558,            2511,            1078,            1934,            4355,            1495,            1145,            1575,            1693,            340,            3739,            3717,            840,            0,            3878,            2041,            2367,            3452,            1744,            1298,            950,            1772,            1758,            1916,            1499,            1432,            3008,            1849,            3032,            1303,            2448,            1375,            1964,            4223,            1723,            6937,            721,            2202,            1007,            2638,            2384,            3307,            1539,            2184,            1184,            1271,            3819,            1187,            2902,            1783,            4243,            998,            1641,            1804,            2838,            2344,            2585,            1732,            2975,            3354,            4004,            3951,            3184,            909,            1332,            4294,            4394,            2571,            2307,            1652,            3043,            1542,            3897,            2757,            3310,            1655,            929,            3438,            741,            1700,            482,            3489,            ],        [            6685,            7107,            6097,            7301,            6733,            8000,            7714,            4917,            7339,            7962,            1500,            3828,            3431,            2287,            239,            1188,            601,            3219,            3570,            3357,            2316,            3248,            2877,            3272,            3208,            2572,            2830,            3647,            1010,            1904,            1857,            4611,            2508,            1276,            3870,            3520,            4224,            2267,            3573,            2293,            3973,            4373,            3823,            0,            2948,            2689,            1377,            2318,            2725,            2950,            2210,            2506,            3743,            3763,            4043,            6724,            2869,            4119,            4183,            3535,            3202,            3051,            6967,            2692,            5491,            3484,            4813,            3271,            3725,            6100,            3425,            4150,            3271,            2402,            2732,            6563,            4465,            6618,            8039,            2472,            3373,            3620,            2734,            734,            1527,            2957,            2439,            6691,            3742,            2820,            496,            6900,            3672,            4865,            3162,            3210,            1125,            2881,            3063,            1597,            4153,            2765,            1311,            1240,            2872,            4462,            1262,            3358,            3111,            4015,            3243,            ],        [            4343,            4114,            3104,            2756,            4373,            2220,            2601,            2328,            4978,            2164,            2373,            1700,            907,            1500,            3122,            2607,            3484,            1298,            1990,            4663,            3707,            1849,            1172,            1924,            317,            3963,            909,            1153,            2458,            1215,            1168,            1683,            2163,            2384,            1636,            1573,            1546,            1984,            1499,            3684,            4744,            2049,            2243,            2883,            0,            280,            3421,            1680,            960,            1609,            2465,            2785,            2943,            2526,            2806,            4382,            2876,            4059,            2946,            3475,            2402,            2991,            5250,            2170,            6882,            2211,            3576,            2034,            3665,            3758,            4334,            2913,            3211,            2211,            2298,            4846,            3228,            4276,            2243,            3863,            1686,            2668,            2251,            2771,            2275,            828,            2759,            4349,            4381,            5031,            3176,            4558,            2399,            1792,            5321,            5421,            2516,            2536,            1142,            2988,            2916,            4924,            2702,            3243,            1545,            2138,            3032,            1437,            1190,            2053,            4516,            ],        [            4083,            4373,            3363,            3015,            4113,            2479,            2860,            2587,            5237,            2423,            2113,            1788,            1391,            1240,            2862,            2347,            3224,            1179,            1730,            4403,            3447,            1589,            1053,            1664,            576,            3703,            790,            1412,            2198,            955,            908,            1942,            1903,            2124,            1376,            1313,            1805,            1724,            1380,            3424,            4484,            2533,            1983,            2623,            259,            0,            3161,            1420,            685,            1349,            2205,            2525,            2683,            2266,            2546,            4122,            2616,            3799,            2686,            3215,            2142,            2731,            4990,            1910,            6622,            1951,            3316,            1774,            3405,            3498,            4074,            2653,            2951,            1951,            2038,            4586,            2968,            4016,            2502,            3603,            1426,            2408,            1991,            2511,            2015,            568,            2499,            4089,            4121,            4771,            2916,            4298,            2139,            2051,            5061,            5161,            2256,            2276,            1023,            2728,            2656,            4664,            2442,            2983,            1285,            2622,            2772,            1318,            1071,            2175,            4256,            ],        [            6312,            8863,            9509,            6928,            6360,            3916,            7341,            3540,            6966,            3860,            2748,            3019,            2336,            1203,            1646,            843,            1320,            2865,            2193,            3346,            1943,            1871,            1793,            1895,            2721,            2199,            2343,            2552,            2258,            1705,            1658,            3234,            1424,            2524,            2493,            2143,            3155,            1183,            2196,            1920,            3427,            2996,            2446,            2047,            2461,            2202,            0,            1234,            2149,            1866,            833,            1129,            2366,            2386,            2666,            6351,            1492,            2742,            2806,            2158,            1825,            1674,            4673,            1315,            5118,            2107,            3436,            1894,            2348,            5727,            3017,            2773,            1894,            1318,            1355,            4269,            3088,            6245,            7666,            2099,            1996,            2243,            1357,            1007,            443,            2470,            1062,            6318,            3064,            2447,            2120,            6527,            2295,            3488,            2789,            1817,            752,            1797,            1785,            1224,            2776,            2392,            938,            1479,            1788,            3085,            1561,            2796,            1833,            2638,            3199,            ],        [            3371,            5777,            4392,            3987,            3401,            3084,            4400,            2708,            4025,            3028,            2088,            2187,            2169,            245,            2314,            1511,            1988,            1957,            1361,            3882,            2611,            1039,            944,            1063,            1946,            2867,            1568,            2385,            1952,            930,            883,            2402,            513,            2218,            1044,            1311,            2962,            334,            1271,            2588,            3963,            2164,            1614,            2598,            1686,            1427,            2289,            0,            1463,            1017,            1369,            1665,            2149,            1554,            1834,            3410,            2028,            3278,            1974,            2694,            1608,            2210,            4456,            520,            5786,            1275,            2604,            1062,            2884,            2786,            3553,            1941,            2430,            553,            927,            4052,            2256,            3304,            3107,            2767,            1094,            1411,            601,            1675,            1179,            1695,            1598,            3377,            3600,            3115,            2788,            3586,            1463,            2656,            3457,            2353,            1420,            886,            936,            1892,            1944,            3060,            1606,            2147,            939,            2253,            2275,            2096,            984,            1806,            3538,            ],        [            3618,            4441,            3431,            2866,            3648,            2364,            3056,            2472,            4272,            2308,            1995,            1605,            1208,            1113,            2744,            2220,            3106,            996,            1265,            3938,            3320,            1124,            811,            1199,            985,            3576,            607,            1424,            1859,            837,            790,            1827,            1536,            2125,            911,            848,            2001,            1225,            1138,            3297,            4019,            2068,            1518,            2505,            905,            646,            3043,            1293,            0,            884,            2074,            2060,            2218,            1801,            2081,            3657,            2151,            3334,            2221,            2750,            1677,            2266,            4525,            1481,            6495,            1486,            2851,            1309,            2940,            3033,            3609,            2188,            2486,            1486,            1573,            4121,            2503,            3551,            2387,            4545,            961,            1943,            1562,            2384,            1888,            864,            2034,            3624,            3656,            4306,            2798,            3833,            1674,            1936,            4596,            4696,            2129,            1909,            383,            2601,            2191,            4199,            2315,            2856,            820,            2157,            2654,            1135,            957,            1710,            3791,            ],        [            3238,            5644,            3741,            2486,            3268,            2608,            3117,            2232,            3892,            2552,            2538,            1711,            1518,            991,            2599,            1796,            2273,            1306,            885,            3558,            2896,            744,            482,            819,            1295,            3152,            917,            1734,            2402,            1380,            1333,            1926,            942,            2668,            531,            468,            1847,            631,            888,            2873,            3639,            1688,            1138,            3000,            1448,            1189,            2574,            752,            812,            0,            1694,            1680,            1838,            1421,            1701,            3277,            1771,            2954,            1841,            2370,            1297,            1886,            4145,            887,            6071,            1106,            2471,            929,            2560,            2653,            3229,            1808,            2106,            920,            1193,            3741,            2123,            3171,            2631,            3052,            581,            1563,            968,            1960,            1464,            1407,            1654,            3244,            3276,            3926,            3073,            3453,            1294,            2180,            4216,            4316,            1705,            1315,            474,            2177,            1811,            3819,            1891,            2432,            477,            1777,            2560,            1445,            522,            1330,            3411,            ],        [            3560,            6111,            6757,            4176,            3608,            3083,            4589,            2707,            4214,            3027,            3420,            2186,            2244,            1224,            2306,            1503,            1980,            2032,            1360,            2513,            3320,            1038,            1837,            1062,            2638,            2859,            2260,            2460,            2930,            1909,            1862,            2401,            1175,            3196,            1660,            1310,            2322,            934,            1363,            2580,            2594,            2163,            1613,            2719,            2665,            2532,            2293,            985,            2155,            1910,            0,            296,            1533,            1553,            1833,            3599,            659,            1909,            1973,            1325,            992,            841,            3840,            482,            5778,            1274,            2603,            1061,            1515,            2975,            2184,            1940,            1061,            1134,            522,            3436,            2255,            3493,            4914,            3120,            1163,            1410,            1046,            1679,            1185,            2750,            229,            3566,            2231,            2881,            2792,            3775,            1462,            2655,            3171,            3271,            1412,            1548,            1817,            1884,            1943,            2774,            1598,            2151,            1820,            2252,            2279,            1963,            1865,            1805,            2366,            ],        [            3560,            6111,            6757,            4176,            3608,            3366,            4589,            2990,            4214,            3310,            3531,            2469,            2527,            2216,            2417,            1614,            2091,            2315,            1643,            2217,            3024,            1321,            2120,            1345,            2921,            3543,            2543,            2743,            3041,            2901,            2854,            2684,            2167,            3307,            1943,            1593,            2605,            1926,            1646,            3264,            2298,            2446,            1896,            2830,            3074,            2815,            2404,            1977,            2438,            2193,            724,            0,            1533,            1836,            2116,            3599,            363,            1613,            2256,            1029,            945,            545,            3840,            611,            5889,            1557,            2886,            1344,            1219,            2975,            1888,            2223,            765,            1417,            651,            3436,            2538,            3493,            4914,            2824,            1446,            1693,            1175,            1790,            1296,            3033,            908,            3566,            1935,            2585,            2903,            3775,            1745,            2938,            2875,            2975,            1523,            2540,            2100,            1995,            2226,            2478,            1709,            2262,            2103,            2535,            2390,            2246,            2148,            2088,            2070,            ],        [            3087,            5638,            6284,            3703,            3135,            2893,            4116,            2517,            3741,            2837,            3922,            1996,            2054,            1743,            2808,            2005,            2482,            1842,            1170,            2691,            3498,            848,            1647,            872,            2448,            4017,            2070,            2270,            3432,            2428,            2381,            2211,            1694,            3698,            1470,            1120,            2132,            1453,            1173,            3738,            2772,            1973,            1423,            3221,            2601,            2342,            2795,            1504,            1965,            1720,            1115,            1101,            0,            1363,            1643,            3126,            904,            2087,            1783,            1503,            430,            1019,            3367,            1002,            6280,            1084,            2413,            871,            1693,            2502,            2362,            1750,            1239,            944,            620,            2963,            2065,            3020,            4441,            3298,            973,            1220,            1564,            2181,            1687,            2560,            1387,            3093,            2409,            3059,            3294,            3302,            1272,            2465,            3349,            3449,            1914,            2067,            1627,            2386,            1753,            2952,            2100,            2653,            1630,            2062,            2781,            1773,            1675,            1615,            2544,            ],        [            2504,            4910,            5556,            1911,            2534,            2033,            2542,            1657,            3158,            1977,            4149,            1319,            1359,            1553,            3035,            2232,            2709,            1147,            475,            3206,            4013,            658,            805,            832,            2408,            3588,            2030,            1575,            3659,            2238,            2191,            1351,            1504,            3925,            1430,            1080,            1848,            1263,            478,            3309,            3287,            1113,            728,            3448,            2561,            2302,            3022,            1314,            1925,            972,            1342,            1328,            1486,            0,            872,            2543,            1419,            2602,            732,            2018,            945,            1534,            3793,            1293,            6507,            150,            1737,            523,            2208,            1919,            2877,            1074,            1754,            754,            841,            3389,            902,            2437,            2056,            3813,            933,            829,            1374,            2408,            1914,            2520,            1302,            2510,            2924,            3574,            3521,            2719,            338,            1605,            3864,            3964,            2141,            1877,            1587,            2613,            1077,            3467,            2327,            2880,            1590,            1202,            3008,            1097,            1635,            755,            3059,            ],        [            1719,            4125,            4771,            2335,            1749,            1759,            2748,            1383,            2373,            1703,            4679,            1308,            1999,            2083,            3565,            2762,            3239,            1787,            1115,            3736,            4543,            1244,            2403,            1268,            2844,            4118,            2466,            2215,            4189,            2768,            2721,            1077,            2034,            4455,            1866,            1516,            1574,            1793,            1118,            3839,            3817,            839,            1155,            3978,            3524,            3265,            3552,            1844,            2759,            2476,            1872,            1858,            2016,            718,            0,            1758,            1949,            3132,            140,            2548,            1475,            2064,            2882,            1823,            7037,            664,            952,            947,            2738,            1134,            3407,            289,            2284,            1284,            1371,            2478,            422,            1652,            3073,            4343,            1369,            1278,            1904,            2938,            2444,            2956,            1832,            1725,            3454,            4104,            4051,            1934,            476,            1331,            4394,            4494,            2671,            2407,            2023,            3143,            302,            3997,            2857,            3410,            2026,            928,            3538,            1737,            2071,            809,            3589,            ],        [            2237,            4643,            5289,            1211,            2267,            1885,            3266,            1449,            2891,            1829,            4444,            1434,            2183,            2601,            4083,            3280,            3757,            1971,            1299,            4254,            5061,            1762,            1629,            1786,            3362,            4636,            2984,            2399,            4707,            3286,            3239,            1203,            2552,            4973,            2384,            2034,            1700,            2311,            1302,            4357,            4335,            965,            1339,            4496,            4042,            3783,            3933,            2362,            3277,            1796,            2390,            2376,            2534,            1236,            518,            0,            2467,            3650,            378,            3066,            1993,            2582,            3400,            2341,            7555,            848,            1470,            1465,            3256,            1652,            3925,            807,            2802,            1802,            1889,            2996,            548,            1474,            1908,            4861,            1887,            1796,            2422,            3458,            2962,            3474,            2350,            2243,            3972,            4622,            4569,            2452,            660,            1457,            4912,            5012,            3189,            2925,            2541,            3661,            820,            4515,            3375,            3928,            2544,            1054,            4056,            1921,            2589,            993,            4107,            ],        [            3197,            5748,            6394,            3813,            3245,            3003,            4226,            2627,            3851,            2947,            3522,            2106,            2164,            1853,            2408,            1605,            2082,            1952,            1280,            2336,            3143,            958,            1757,            982,            2558,            2961,            2180,            2380,            3032,            2538,            2491,            2321,            1804,            3298,            1580,            1230,            2363,            1563,            1283,            2682,            2417,            2083,            1533,            2821,            2711,            2452,            2395,            1614,            2075,            1830,            715,            701,            1170,            1473,            1753,            3236,            0,            1732,            1893,            1148,            582,            664,            3477,            602,            5880,            1194,            2523,            981,            1338,            2612,            2007,            1860,            884,            1054,            642,            3073,            2175,            3130,            4551,            2943,            1083,            1330,            1166,            1781,            1287,            2670,            1032,            3203,            2054,            2704,            2894,            3412,            1382,            2575,            2994,            3094,            1514,            2177,            1737,            1986,            1863,            2597,            1700,            2253,            1740,            2172,            2381,            1902,            1785,            1725,            2189,            ],        [            4613,            7164,            7810,            5229,            4661,            4419,            5642,            4043,            5267,            4363,            4746,            3522,            3580,            3271,            3379,            2576,            3053,            3368,            2696,            604,            1411,            2374,            3173,            2398,            3974,            1930,            3596,            3796,            4256,            3954,            3907,            3737,            3220,            4522,            2996,            2646,            3779,            2979,            2699,            1651,            685,            3499,            2949,            4045,            4127,            3868,            3619,            3030,            3491,            3246,            1939,            1966,            2586,            2889,            3169,            4652,            1416,            0,            3309,            584,            1998,            1068,            4893,            2018,            6851,            2610,            3939,            2397,            2272,            4028,            275,            3276,            848,            2470,            2192,            4489,            3591,            4546,            5967,            1211,            2499,            2746,            2582,            3005,            2511,            4086,            1899,            4619,            322,            972,            4118,            4828,            2798,            3991,            1262,            1362,            2738,            3593,            3153,            2021,            3279,            865,            2671,            3477,            3156,            3588,            3605,            3318,            3201,            3141,            457,            ],        [            1859,            4265,            4911,            2475,            1889,            1619,            2888,            1243,            2513,            1563,            4066,            1168,            1917,            2223,            3705,            2902,            3379,            1705,            1033,            3876,            4683,            1384,            1363,            1408,            2984,            4258,            2606,            2133,            4329,            2908,            2861,            937,            2174,            4595,            2006,            1656,            1434,            1933,            1036,            3979,            3957,            699,            1073,            4118,            3664,            3405,            3555,            1984,            2899,            2616,            2012,            1998,            2156,            858,            140,            1898,            2089,            3272,            0,            2688,            1615,            2204,            3022,            1963,            7177,            582,            1092,            1087,            2878,            1274,            3547,            429,            2424,            1424,            1511,            2618,            282,            1792,            3213,            4483,            1509,            1418,            2044,            3078,            2584,            3096,            1972,            1865,            3594,            4244,            4191,            2074,            394,            1191,            4534,            4634,            2811,            2547,            2163,            3283,            442,            4137,            2997,            3550,            2166,            788,            3678,            1655,            2211,            727,            3729,            ],        [            4029,            6580,            7226,            4645,            4077,            3835,            5058,            3459,            4683,            3779,            4162,            2938,            2996,            2687,            3048,            2245,            2722,            2784,            2112,            1188,            1995,            1790,            2589,            1814,            3390,            2514,            3012,            3212,            3672,            3370,            3323,            3153,            2636,            3938,            2412,            2062,            3195,            2395,            2115,            2235,            1269,            2915,            2365,            3461,            3543,            3284,            3035,            2446,            2907,            2662,            1355,            1382,            2002,            2305,            2585,            4068,            832,            584,            2725,            0,            1414,            484,            4309,            1434,            6520,            2026,            3355,            1813,            190,            3444,            859,            2692,            264,            1886,            1474,            3905,            3007,            3962,            5383,            1795,            1915,            2162,            1998,            2421,            1927,            3502,            1315,            4035,            906,            1556,            3534,            4244,            2214,            3407,            1846,            1946,            2154,            3009,            2569,            2626,            2695,            1449,            2340,            2893,            2572,            3004,            3021,            2734,            2617,            2557,            1041,            ],        [            3167,            5718,            6364,            3783,            3215,            2973,            4196,            2597,            3821,            2917,            3492,            2076,            2134,            2017,            2378,            1575,            2052,            1922,            1250,            2261,            3068,            928,            1727,            952,            2528,            2931,            2150,            2350,            3002,            1999,            1952,            2291,            1265,            3268,            1550,            1200,            2333,            1024,            1253,            2652,            2342,            2053,            1503,            2791,            2681,            2422,            2365,            1075,            2045,            1800,            685,            671,            1140,            1443,            1723,            3206,            474,            1657,            1863,            1073,            0,            589,            3447,            572,            5850,            1164,            2493,            951,            1263,            2582,            1932,            1830,            809,            1024,            612,            3043,            2145,            3100,            4521,            2868,            1053,            1300,            1136,            1751,            1257,            2640,            645,            3173,            1979,            2629,            2864,            3382,            1352,            2545,            2919,            3019,            1484,            2147,            1707,            1956,            1833,            2522,            1670,            2223,            1710,            2142,            2351,            1872,            1755,            1695,            2114,            ],        [            3545,            6096,            6742,            4161,            3593,            3351,            4574,            2975,            4199,            3295,            3678,            2454,            2512,            2201,            2564,            1761,            2238,            2300,            1628,            1672,            2479,            1306,            2105,            1330,            2906,            2998,            2528,            2728,            3188,            2886,            2839,            2669,            2152,            3454,            1928,            1578,            2711,            1911,            1631,            2719,            1753,            2431,            1881,            2977,            3059,            2800,            2551,            1962,            2423,            2178,            871,            898,            1518,            1821,            2101,            3584,            348,            1068,            2241,            484,            930,            0,            3825,            950,            6036,            1542,            2871,            1329,            674,            2960,            1343,            2208,            220,            1402,            990,            3421,            2523,            3478,            4899,            2279,            1431,            1678,            1514,            1937,            1443,            3018,            831,            3551,            1390,            2040,            3050,            3760,            1730,            2923,            2330,            2430,            1670,            2525,            2085,            2142,            2211,            1933,            1856,            2409,            2088,            2520,            2537,            2250,            2133,            2073,            1525,            ],        [            2502,            5053,            5699,            3118,            2550,            3817,            3531,            3498,            3156,            3779,            4367,            2441,            2499,            2188,            3253,            2450,            2927,            2287,            1615,            3136,            3943,            1293,            2092,            1317,            2893,            3806,            2515,            2715,            3877,            2873,            2826,            2656,            2139,            4143,            1915,            1565,            2698,            1898,            1618,            3527,            3217,            2418,            1868,            3666,            3046,            2787,            3240,            1949,            2410,            2165,            1560,            1546,            445,            1808,            2088,            2541,            1349,            2532,            2228,            1948,            875,            1464,            0,            1447,            6725,            1529,            2858,            1316,            2138,            1917,            2807,            2195,            1684,            1389,            1065,            2378,            2510,            2435,            3856,            3743,            1418,            1665,            2009,            2626,            2132,            3005,            1520,            2508,            2854,            3504,            3739,            2717,            1717,            2910,            3794,            3894,            2359,            2512,            2072,            2831,            2198,            3397,            2545,            3098,            2075,            2507,            3226,            2237,            2120,            2060,            2989,            ],        [            3479,            6030,            3828,            4095,            3527,            2989,            4508,            2613,            4133,            2933,            2585,            2092,            1605,            742,            2195,            1392,            1869,            1886,            1266,            2943,            2492,            944,            1062,            968,            1875,            2748,            1497,            1821,            2449,            1427,            1380,            2307,            693,            2715,            1162,            1216,            2349,            452,            1269,            2469,            3024,            2069,            1519,            2641,            2183,            1924,            2215,            503,            1418,            1135,            430,            726,            1452,            1459,            1739,            3518,            1089,            2339,            1879,            1755,            1002,            1271,            3759,            0,            5667,            1180,            2509,            967,            1945,            2894,            2614,            1846,            1491,            587,            532,            3355,            2161,            3412,            4833,            3550,            1212,            1316,            564,            1601,            1107,            1987,            659,            3485,            2661,            3311,            2714,            3694,            1368,            2561,            3601,            1946,            1301,            1066,            1054,            1773,            1849,            3204,            1487,            2073,            1057,            2158,            2201,            1532,            1102,            1711,            2796,            ],        [            1194,            3745,            4391,            1810,            1242,            2087,            2223,            2190,            1848,            2471,            7726,            2488,            3975,            3664,            6122,            3932,            5796,            3763,            3091,            4612,            5419,            2769,            3568,            2793,            4369,            5288,            3991,            4191,            7236,            4349,            4302,            2257,            3615,            7502,            3391,            3041,            3120,            3374,            3094,            5009,            4693,            2019,            3344,            6505,            4522,            4263,            4722,            3425,            3886,            3641,            3042,            3028,            1921,            3284,            2375,            1233,            2825,            4008,            2515,            3424,            2351,            2940,            1476,            2923,            0,            3005,            1469,            2792,            3614,            609,            4283,            2169,            3160,            2865,            2541,            1072,            1762,            1127,            2548,            5219,            2894,            3141,            3485,            4108,            3614,            4481,            3002,            1200,            4330,            4980,            6722,            1409,            3193,            2511,            5270,            5370,            3835,            3988,            3548,            4313,            2461,            4873,            4027,            4580,            3551,            2108,            4708,            3713,            3596,            3536,            4465,            ],        [            2374,            4780,            5426,            1887,            2404,            2009,            2518,            1633,            3028,            1953,            3372,            1295,            1335,            1529,            3011,            2208,            2685,            1123,            451,            3182,            3308,            634,            781,            808,            2384,            3564,            2006,            1551,            3236,            2214,            2167,            1327,            1480,            3502,            1406,            1056,            1824,            1239,            454,            3285,            3263,            1089,            704,            3424,            2537,            2278,            2998,            1290,            1901,            948,            1318,            1304,            1462,            483,            722,            2413,            1667,            2578,            582,            1994,            921,            1510,            3769,            1269,            6483,            0,            1607,            499,            2184,            1789,            2853,            944,            1730,            730,            817,            3365,            752,            2307,            2032,            3789,            909,            1162,            1350,            2384,            1890,            2496,            1278,            2380,            2900,            3550,            3497,            2589,            188,            1581,            3840,            2729,            2117,            1853,            1563,            2589,            947,            3443,            2303,            2856,            1566,            1178,            2984,            1073,            1611,            731,            3035,            ],        [            984,            3173,            3819,            1613,            797,            2299,            2013,            2432,            1638,            2261,            4790,            2388,            5333,            2947,            4429,            3626,            4103,            3102,            2430,            5066,            4726,            2108,            3267,            2132,            3708,            4982,            3330,            5117,            4654,            3632,            3585,            2157,            2898,            4920,            2730,            2380,            2910,            2657,            2478,            4703,            5147,            1919,            2683,            4842,            5423,            4129,            4416,            2708,            3623,            3340,            2736,            2722,            2375,            1582,            1080,            1036,            3085,            4462,            1220,            3878,            2805,            3394,            1930,            2687,            7901,            2150,            0,            1811,            4068,            412,            4737,            874,            3614,            2148,            2235,            1526,            1502,            930,            2338,            5673,            2233,            2142,            2768,            3802,            3308,            3820,            2696,            1003,            4784,            5434,            4915,            1199,            1556,            2411,            5724,            4147,            3535,            3271,            2887,            4007,            1166,            5327,            3721,            4274,            2890,            2008,            4402,            3052,            2935,            1889,            4919,            ],        [            2309,            4715,            5361,            2925,            2339,            2517,            3338,            2141,            2963,            2461,            3282,            1803,            1843,            1439,            2921,            2118,            2595,            1631,            959,            3092,            3218,            600,            1759,            624,            2200,            3474,            1822,            2059,            3146,            2124,            2077,            1835,            1390,            3412,            1222,            872,            2332,            1149,            962,            3195,            3173,            1597,            1212,            3334,            2880,            2621,            2908,            1200,            2115,            1832,            1228,            1214,            1372,            492,            772,            2348,            1577,            2488,            912,            1904,            831,            1420,            3679,            1179,            6393,            634,            1542,            0,            2094,            1724,            2763,            879,            1640,            640,            727,            3275,            1194,            2242,            3663,            3374,            725,            634,            1260,            2294,            1800,            2312,            1188,            2315,            2810,            3460,            3407,            2524,            822,            2089,            3750,            2639,            2027,            1763,            1379,            2499,            882,            3353,            2213,            2766,            1382,            1686,            2894,            1581,            1427,            1239,            2945,            ],        [            5245,            7796,            8442,            5861,            5293,            5051,            6274,            4675,            5899,            4995,            5198,            4154,            4212,            3721,            3797,            2994,            3471,            4000,            3328,            1022,            1829,            3006,            4311,            3030,            4606,            2348,            4228,            4428,            4708,            4223,            4176,            4369,            3942,            4974,            3628,            3278,            4411,            3682,            3331,            2069,            1103,            4131,            3581,            4180,            4979,            4720,            4071,            3752,            4123,            4365,            3278,            3574,            3218,            3521,            3801,            5284,            2048,            418,            3941,            1216,            2630,            1700,            5525,            2650,            7269,            3242,            4571,            3029,            0,            4660,            693,            3908,            1480,            3102,            2778,            5121,            4223,            5178,            6599,            1629,            3131,            3378,            3714,            3457,            2961,            4988,            2531,            5251,            740,            1390,            4397,            5460,            3430,            4623,            1680,            1780,            3188,            4296,            3785,            2439,            3911,            1283,            3089,            3929,            3788,            4220,            4057,            3950,            3833,            3773,            875,            ],        [            585,            3136,            3782,            1201,            633,            1478,            1614,            1581,            1239,            1862,            5476,            1879,            5296,            3633,            7014,            4521,            6688,            5634,            3680,            5201,            6008,            3358,            3991,            3382,            5422,            5877,            4580,            5080,            5948,            4318,            4271,            1648,            3584,            6214,            3980,            3630,            2511,            3343,            3683,            5598,            5282,            1410,            1958,            5737,            5386,            5646,            5311,            3394,            4475,            4026,            3631,            3617,            2510,            2268,            1142,            624,            3414,            4597,            1002,            4013,            2940,            3529,            2065,            3512,            8796,            1472,            860,            2497,            4203,            0,            4872,            1560,            3749,            2834,            3130,            1661,            1153,            518,            1939,            5808,            3483,            2828,            3454,            4697,            4203,            5070,            3591,            591,            4919,            5569,            5810,            800,            1284,            1902,            5859,            5959,            4430,            3957,            4137,            4902,            1852,            5462,            4616,            5169,            4140,            1499,            5297,            2165,            4185,            1612,            5054,            ],        [            4888,            7439,            8085,            5504,            4936,            4694,            5917,            4318,            5542,            4638,            4505,            3797,            3855,            3028,            3242,            2439,            2916,            3643,            2971,            467,            1274,            2649,            3282,            2673,            4249,            1655,            3871,            4071,            4015,            3530,            3483,            4012,            3495,            4281,            3271,            2921,            4054,            3254,            2974,            1376,            548,            3774,            3224,            3804,            4286,            4027,            3378,            3305,            3766,            3521,            2214,            2241,            2861,            3164,            3444,            4927,            1691,            275,            3584,            859,            2273,            1343,            5168,            2293,            6714,            2885,            4214,            2672,            2547,            4303,            0,            3551,            1123,            2745,            2421,            4764,            3866,            4821,            6242,            1074,            2774,            3021,            2857,            2764,            2268,            4361,            2174,            4894,            317,            697,            3877,            5103,            3073,            4266,            1125,            1087,            2495,            3868,            3428,            1884,            3554,            728,            2534,            3236,            3431,            3863,            3364,            3593,            3476,            3416,            182,            ],        [            1430,            3836,            4482,            2046,            1460,            2323,            2459,            2426,            2084,            2707,            5279,            2877,            3803,            3436,            4918,            4115,            4592,            3591,            2919,            5089,            5215,            2597,            3756,            2621,            4197,            5471,            3819,            5780,            5143,            4121,            4074,            2646,            3387,            5409,            3219,            2869,            3143,            3146,            2967,            5192,            5170,            2408,            3172,            5331,            4877,            4618,            4905,            3197,            4112,            3829,            3225,            3211,            3038,            2071,            1569,            1469,            3574,            4485,            1709,            3901,            2828,            3417,            2593,            3176,            8390,            2639,            663,            2300,            4091,            845,            4760,            0,            3637,            2637,            2724,            2189,            1991,            1363,            2784,            5696,            2722,            2631,            3257,            4291,            3797,            4309,            3185,            1436,            4807,            5457,            5404,            1645,            2045,            2900,            5747,            4636,            4024,            3760,            3376,            4496,            1655,            5350,            4210,            4763,            3379,            2497,            4891,            3541,            3424,            2378,            4942,            ],        [            3765,            6316,            6962,            4381,            3813,            3571,            4794,            3195,            4419,            3515,            3898,            2674,            2732,            2423,            2784,            1981,            2458,            2520,            1848,            1452,            2259,            1526,            2159,            1550,            3126,            2778,            2748,            2948,            3408,            3106,            3059,            2889,            2372,            3674,            2148,            1798,            2931,            2131,            1851,            2499,            1533,            2651,            2101,            3197,            3279,            3020,            2771,            2182,            2643,            2326,            1091,            1118,            1738,            2041,            2321,            3804,            568,            848,            2461,            264,            1150,            220,            4045,            1170,            6256,            1762,            3091,            1549,            454,            3180,            1123,            2428,            0,            1622,            1344,            3641,            2743,            3698,            5119,            2059,            1651,            1898,            1615,            2157,            1663,            3238,            1051,            3771,            1170,            1820,            3270,            3980,            1950,            3143,            2110,            2210,            1890,            2745,            2305,            2362,            2431,            1713,            2076,            2629,            2308,            2740,            2757,            2470,            2353,            2293,            1305,            ],        [            2818,            5224,            5870,            3434,            2848,            2531,            3847,            2155,            3472,            2475,            2642,            1634,            1662,            799,            2251,            1448,            1925,            1480,            808,            3316,            2548,            486,            1119,            510,            1932,            2804,            1554,            1878,            2506,            1484,            1437,            1849,            750,            2772,            1108,            758,            1891,            509,            811,            2525,            3397,            1611,            1061,            2652,            2240,            1981,            2226,            560,            1475,            1192,            1070,            1438,            1596,            1001,            1281,            2857,            1801,            2712,            1421,            2128,            1055,            1644,            3903,            539,            5723,            722,            2051,            509,            2318,            2233,            2987,            1388,            1864,            0,            951,            3499,            1703,            2751,            2554,            2704,            611,            858,            620,            1612,            1116,            2044,            1112,            2824,            3034,            3052,            2725,            3033,            910,            2103,            3394,            2290,            1357,            1123,            1111,            1829,            1391,            2997,            1543,            2084,            1114,            1700,            2212,            1430,            1159,            1253,            3169,            ],        [            3038,            5589,            6235,            3654,            3086,            2561,            4067,            2185,            3692,            2505,            3254,            1664,            1722,            1411,            2560,            1757,            2234,            1510,            838,            2731,            3538,            516,            1149,            540,            2116,            3113,            1738,            1938,            3184,            2096,            2049,            1879,            1362,            3450,            1138,            788,            1921,            1121,            841,            2834,            2812,            1641,            1091,            2973,            2269,            2010,            2547,            1172,            1633,            1316,            867,            853,            1011,            1031,            1311,            3077,            944,            2127,            1451,            1543,            470,            1059,            3318,            1042,            6032,            752,            2081,            539,            1733,            2453,            2402,            1418,            1279,            612,            0,            2914,            1733,            2971,            4392,            3338,            641,            888,            1232,            1933,            1439,            2228,            827,            3044,            2449,            3099,            3046,            3253,            940,            2133,            3389,            3489,            1666,            1735,            1295,            2138,            1421,            2992,            1852,            2405,            1298,            1730,            2533,            1460,            1343,            1283,            2584,            ],        [            2906,            5457,            6103,            3522,            2954,            4221,            3935,            3902,            3560,            4183,            4435,            2845,            2903,            2592,            3657,            2854,            3331,            2691,            2019,            3540,            4347,            1697,            2330,            1721,            3297,            4210,            2919,            3119,            4281,            3277,            3230,            3060,            2543,            4547,            2319,            1969,            3102,            2302,            2022,            3931,            3621,            2822,            2272,            4070,            3450,            3191,            3644,            2353,            2814,            2497,            1964,            1950,            849,            2212,            2492,            2945,            1753,            2936,            2632,            2352,            1279,            1868,            404,            1851,            7129,            1933,            3262,            1720,            2542,            2321,            3211,            2599,            2088,            1793,            1469,            0,            2914,            2839,            4260,            4147,            1822,            2069,            2413,            3030,            2536,            3409,            2236,            2912,            3258,            3908,            4143,            3121,            2121,            3314,            4198,            4298,            2763,            2916,            2476,            3235,            2602,            3801,            2949,            3502,            2479,            2911,            3630,            2641,            2524,            2464,            3393,            ],        [            2141,            4547,            5193,            1215,            2171,            1337,            1846,            961,            2795,            1281,            4348,            886,            1649,            2505,            3987,            3184,            3661,            1437,            1169,            4158,            4284,            1666,            1499,            1690,            2394,            4540,            1700,            1401,            4212,            3190,            3143,            655,            2456,            4478,            2288,            1938,            1152,            2215,            1172,            4261,            4239,            417,            965,            4400,            3946,            3687,            3974,            2266,            1969,            1666,            2294,            2280,            2438,            1140,            422,            2180,            2643,            3554,            282,            2970,            1897,            2486,            3304,            2245,            7459,            752,            1374,            1369,            3160,            1556,            3829,            711,            2706,            1706,            1793,            2900,            0,            2074,            1360,            4765,            1791,            1700,            2326,            3360,            2866,            3378,            2254,            2147,            3876,            4526,            4473,            2356,            564,            909,            4816,            3705,            3093,            2829,            2445,            3565,            724,            4419,            3279,            3832,            2448,            506,            3960,            1172,            2493,            619,            4011,            ],        [            427,            2978,            3624,            1369,            475,            1284,            1456,            1387,            1081,            1704,            5208,            1893,            5138,            3365,            6746,            4253,            6420,            4084,            3412,            4933,            5740,            3090,            3723,            3114,            5264,            5609,            4312,            4922,            5680,            4050,            4003,            1995,            3316,            5946,            3712,            3362,            2353,            3075,            3415,            5330,            5014,            1757,            3665,            5469,            5228,            5488,            5043,            3126,            4207,            3890,            3363,            3349,            2242,            2000,            1498,            792,            3146,            4329,            1638,            3745,            2672,            3261,            1797,            3244,            8528,            1640,            592,            2229,            3935,            830,            4604,            1292,            3481,            2566,            2862,            1393,            1340,            0,            1781,            5540,            3215,            2560,            3186,            4429,            3935,            4802,            3629,            397,            4651,            5301,            5542,            642,            1452,            1673,            5591,            5691,            4162,            3689,            3869,            4634,            1584,            5194,            4348,            4901,            3872,            1846,            5029,            4034,            3917,            1785,            4786,            ],        [            1799,            3239,            2229,            1109,            1847,            350,            975,            681,            1353,            406,            5415,            982,            1318,            3257,            5936,            3907,            4384,            1656,            1582,            6305,            7112,            2782,            1965,            2142,            1887,            5263,            1919,            1102,            5196,            2981,            2934,            819,            2980,            4851,            2569,            2390,            572,            2669,            1585,            4984,            6386,            1185,            1245,            5968,            1851,            2111,            6506,            2790,            2144,            2038,            3017,            3003,            3614,            2580,            2036,            1518,            4518,            5701,            1896,            5117,            4044,            4633,            3169,            2925,            8182,            1966,            1964,            2252,            5307,            2202,            5976,            2325,            4853,            2429,            2516,            2765,            1697,            1372,            0,            6912,            2619,            3932,            3006,            4083,            3589,            2659,            2977,            1731,            6023,            6673,            5990,            2014,            1768,            479,            6963,            7063,            3816,            3353,            2326,            4288,            2956,            6566,            4002,            4555,            2515,            1274,            5846,            1173,            2374,            1462,            6158,            ],        [            6834,            9385,            10031,            7450,            6882,            7727,            7863,            7830,            7488,            8111,            3772,            4279,            3578,            2464,            2168,            1365,            1842,            3859,            3453,            1159,            200,            3131,            3035,            3155,            3848,            738,            3470,            3794,            3282,            2966,            2919,            4494,            2666,            3548,            3753,            3403,            4536,            2425,            3456,            459,            1879,            4256,            3706,            2551,            3722,            3463,            2797,            2476,            3391,            3108,            2093,            2389,            4192,            3646,            3926,            6873,            3022,            1606,            4066,            2190,            3604,            2674,            7116,            2575,            5640,            3367,            7109,            3154,            3608,            6249,            1331,            4033,            2454,            2579,            2615,            6712,            7421,            6767,            8188,            0,            3256,            3503,            2457,            2183,            1704,            3731,            2322,            6840,            1648,            830,            2768,            7049,            3555,            4748,            964,            1220,            1318,            3039,            3027,            810,            4036,            567,            1460,            2655,            3030,            4345,            2783,            3505,            3075,            3898,            1149,            ],        [            2657,            5063,            5709,            3273,            2687,            2208,            3686,            1832,            3311,            2152,            3057,            1311,            1369,            1324,            2806,            2003,            2480,            1157,            485,            2977,            3103,            163,            796,            238,            1814,            3359,            1436,            1585,            2921,            1899,            1852,            1526,            1275,            3187,            836,            486,            1568,            1034,            488,            3080,            3058,            1288,            738,            3219,            1967,            1708,            2793,            1085,            1266,            963,            1113,            1099,            1257,            840,            1120,            2696,            1190,            2373,            1260,            1789,            716,            1305,            3564,            1064,            6278,            525,            1890,            348,            1979,            2072,            2648,            1227,            1525,            525,            612,            3160,            1542,            2590,            2231,            3584,            0,            982,            1145,            2179,            1685,            1926,            1073,            2663,            2695,            3345,            3292,            2872,            713,            1780,            3635,            3735,            1912,            1648,            993,            2384,            1230,            3238,            2098,            2651,            996,            1377,            2779,            1107,            1041,            930,            2830,            ],        [            2301,            4707,            5353,            2917,            2331,            2677,            3330,            2301,            2955,            2621,            3434,            1963,            1958,            1591,            3073,            2270,            2747,            1746,            1074,            3244,            3370,            752,            1385,            776,            2352,            3626,            1974,            2174,            3298,            2276,            2229,            1995,            1542,            3564,            1374,            1024,            2492,            1301,            1122,            3347,            3325,            1757,            1327,            3486,            3032,            2773,            3060,            1352,            2267,            1984,            1380,            1366,            1524,            484,            764,            2340,            1457,            2640,            904,            2056,            983,            1572,            3831,            1331,            6545,            794,            1534,            455,            2246,            1716,            2915,            871,            1792,            792,            879,            3427,            1186,            2234,            3655,            3851,            877,            0,            1412,            2446,            1952,            2464,            1340,            2307,            2962,            3612,            3559,            2516,            982,            2249,            3902,            4002,            2179,            1915,            1531,            2651,            874,            3505,            2365,            2918,            1534,            1846,            3046,            1696,            1579,            1399,            3097,            ],        [            6297,            8848,            9494,            6913,            6345,            3379,            7326,            3003,            6951,            3323,            2733,            2482,            1847,            880,            1631,            828,            1305,            2128,            1656,            3199,            1928,            1334,            1304,            1358,            2117,            2184,            1739,            2063,            2243,            1382,            1335,            2697,            935,            2509,            1404,            1606,            2739,            694,            1659,            1905,            3280,            2459,            1909,            2032,            2138,            1879,            1606,            745,            1660,            1377,            686,            982,            2219,            1849,            2129,            6336,            1345,            2595,            2269,            2011,            1678,            1527,            4526,            815,            5103,            1570,            2899,            1357,            2201,            5712,            2870,            2236,            1747,            848,            1208,            4122,            2551,            6230,            7651,            2084,            1454,            1706,            0,            992,            496,            2229,            915,            6303,            2917,            2432,            2105,            6512,            1758,            2951,            2774,            1670,            737,            1308,            1296,            1209,            2239,            2377,            923,            1464,            1299,            2548,            1592,            1774,            1344,            2101,            2855,            ],        [            5041,            7592,            6372,            5657,            5089,            4564,            6070,            4188,            5695,            4508,            1775,            3667,            2965,            1851,            599,            1491,            961,            3494,            2841,            3994,            2591,            2519,            2422,            2543,            3483,            2847,            3105,            3181,            1285,            2179,            2132,            3882,            2053,            1551,            3141,            2791,            3924,            1812,            2844,            2568,            4075,            3644,            3094,            1074,            3223,            2964,            648,            1863,            3000,            2495,            1481,            1777,            3014,            3034,            3314,            5080,            2140,            3390,            3454,            2806,            2473,            2322,            5321,            1963,            5766,            2755,            4084,            2542,            2996,            4456,            3665,            3421,            2542,            1966,            2003,            4917,            3736,            4974,            6395,            2747,            2644,            2891,            2005,            0,            1091,            3232,            1710,            5047,            3712,            3095,            1147,            5256,            2943,            4136,            3437,            2465,            1400,            2426,            2414,            1872,            3424,            3040,            1586,            506,            2417,            3733,            634,            3463,            2462,            3286,            3847,            ],        [            6105,            6099,            4097,            6721,            6153,            3406,            7134,            3030,            6759,            3350,            2420,            2509,            1874,            760,            1439,            636,            1113,            2155,            1683,            3007,            1736,            1361,            1331,            1385,            2144,            1992,            1766,            2090,            2051,            1262,            1215,            2724,            962,            2317,            1431,            1633,            2766,            721,            1686,            1713,            3088,            2486,            1936,            1840,            2018,            1759,            1414,            772,            1687,            1404,            494,            790,            2027,            1876,            2156,            6144,            1153,            2403,            2296,            1819,            1486,            1335,            4334,            842,            4911,            1597,            2926,            1384,            2009,            5520,            2678,            2263,            1555,            875,            1016,            3930,            2578,            6038,            7459,            1892,            1481,            1733,            753,            800,            0,            2027,            723,            6111,            2725,            2240,            1913,            6320,            1785,            2978,            2582,            1478,            545,            1335,            1323,            1017,            2266,            2185,            731,            1272,            1326,            2575,            1400,            1801,            1371,            2128,            2860,            ],        [            4090,            5056,            4046,            3090,            4120,            2554,            3246,            2662,            4744,            2498,            2467,            1795,            1398,            1585,            3495,            2692,            3169,            1186,            1737,            4410,            3792,            1596,            1060,            1671,            1175,            4048,            797,            1614,            2331,            1309,            1262,            2017,            2008,            2597,            1383,            1320,            2191,            1697,            1740,            3769,            4491,            2540,            1990,            2977,            1047,            788,            3515,            1765,            692,            1356,            2546,            2532,            2690,            2273,            2553,            4129,            2623,            3806,            2693,            3222,            2149,            2738,            4997,            1953,            6967,            1958,            3323,            1781,            3412,            3505,            4081,            2660,            2958,            1958,            2045,            4593,            2975,            4023,            2577,            5017,            1433,            2415,            2034,            2856,            2360,            0,            2506,            4096,            4128,            4778,            3270,            4305,            2146,            2126,            5068,            5168,            2601,            2381,            1030,            3073,            2663,            4671,            2787,            3328,            1292,            2629,            3126,            1325,            1078,            2182,            4263,            ],        [            3331,            5882,            6528,            3947,            3379,            2854,            4360,            2478,            3985,            2798,            3191,            1957,            2015,            995,            2077,            1274,            1751,            1803,            1131,            2587,            2374,            809,            1442,            833,            2409,            2630,            2031,            2231,            2701,            1680,            1633,            2172,            946,            2967,            1431,            1081,            2214,            705,            1134,            2351,            2668,            1934,            1384,            2490,            2436,            2303,            2064,            756,            1926,            1609,            384,            370,            1304,            1324,            1604,            3370,            733,            1983,            1744,            1399,            763,            915,            3611,            253,            5549,            1045,            2374,            832,            1589,            2746,            2258,            1711,            1135,            905,            293,            3207,            2026,            3264,            4685,            2530,            934,            1181,            817,            1450,            956,            2521,            0,            3337,            2305,            2955,            2563,            3546,            1233,            2426,            3245,            1795,            1183,            1319,            1588,            1655,            1714,            2848,            1369,            1922,            1591,            2023,            2050,            1753,            1636,            1576,            2440,            ],        [            670,            3426,            4072,            935,            949,            887,            799,            990,            424,            831,            5603,            1496,            2442,            3760,            7141,            4648,            6815,            2780,            3807,            5328,            6135,            3485,            4118,            3509,            3011,            6004,            3043,            2226,            6075,            4445,            4398,            1333,            3711,            6341,            4107,            3757,            1696,            3470,            3810,            5725,            5409,            1699,            1992,            5864,            2975,            5936,            5438,            3521,            3268,            3162,            3758,            3744,            2637,            2395,            1893,            1187,            3541,            4724,            2033,            4140,            3067,            3656,            2192,            3639,            8923,            1912,            987,            2624,            4330,            1225,            4999,            1687,            3876,            2961,            3257,            1788,            1735,            640,            1124,            5935,            3610,            2955,            3581,            4824,            4330,            5197,            4024,            0,            5046,            5696,            5937,            449,            1847,            1276,            5986,            6086,            4557,            4084,            4264,            5029,            1979,            5589,            4743,            5296,            4267,            1788,            5424,            1920,            4312,            1976,            5181,            ],        [            4935,            7486,            8132,            5551,            4983,            4741,            5964,            4365,            5589,            4685,            4822,            3844,            3902,            3345,            3421,            2618,            3095,            3690,            3018,            646,            1453,            2696,            3329,            2720,            4296,            1972,            3918,            4118,            4332,            3847,            3800,            4059,            3542,            4598,            3318,            2968,            4101,            3301,            3021,            1693,            727,            3821,            3271,            4121,            4603,            4190,            3695,            3352,            3813,            3496,            2261,            2288,            2908,            3211,            3491,            4974,            1738,            322,            3631,            906,            2320,            1390,            5215,            2340,            6893,            2932,            4261,            2719,            2594,            4350,            317,            3598,            1170,            2792,            2514,            4811,            3913,            4868,            6289,            1253,            2821,            3068,            2904,            3081,            2585,            4408,            2221,            4941,            0,            1014,            4194,            5150,            3120,            4313,            1304,            1404,            2812,            3915,            3475,            2063,            3601,            907,            2713,            3553,            3478,            3910,            3681,            3621,            3523,            3463,            499,            ],        [            5585,            8136,            8782,            6201,            5633,            5391,            6614,            5015,            6239,            5335,            3808,            4494,            3445,            2331,            2692,            2081,            2366,            4340,            3668,            1164,            916,            3346,            2902,            3370,            3715,            1076,            3337,            3661,            3318,            2833,            2786,            4709,            2533,            3584,            3968,            3618,            4751,            2292,            3671,            797,            1245,            4471,            3921,            3107,            3589,            3330,            2681,            2343,            3258,            2975,            1888,            2184,            3558,            3861,            4141,            5624,            2388,            972,            4281,            1556,            2970,            2040,            5865,            2370,            6164,            3582,            4911,            3369,            3244,            5000,            697,            4248,            1820,            2446,            2410,            5461,            4563,            5518,            6939,            716,            3471,            3718,            2324,            2067,            1571,            3598,            2117,            5591,            1014,            0,            3180,            5800,            3770,            4963,            1822,            390,            1798,            2906,            2894,            1526,            4251,            1425,            1984,            2539,            2897,            4560,            2667,            3372,            2942,            4113,            515,            ],        [            5560,            6623,            5613,            6176,            5608,            4103,            6589,            4707,            6214,            4047,            1016,            3344,            2947,            2101,            868,            2010,            1230,            2735,            3360,            4513,            3110,            3038,            2609,            3062,            2724,            3366,            2346,            3163,            526,            1420,            1373,            4401,            2572,            792,            3660,            3310,            3740,            2331,            3363,            3087,            4594,            4163,            3613,            629,            2464,            2205,            1167,            2281,            2241,            3014,            2000,            2296,            3533,            3553,            3833,            5599,            2659,            3909,            3973,            3325,            2992,            2841,            5840,            2482,            6285,            3274,            4603,            3061,            3515,            4975,            4184,            3940,            3061,            3134,            2522,            5436,            4255,            5493,            5688,            3266,            3163,            3410,            2524,            2174,            1610,            2473,            2229,            5566,            4231,            3614,            0,            5775,            3462,            3675,            3956,            2984,            1919,            2945,            2579,            2391,            3943,            3559,            2105,            2646,            2936,            4252,            778,            2874,            2627,            3805,            4366,            ],        [            373,            2924,            3570,            1315,            421,            1688,            1402,            1821,            1027,            1650,            5154,            2172,            5084,            3311,            6692,            4199,            6366,            4030,            3358,            4879,            5686,            3036,            3669,            3060,            5210,            5555,            4258,            4868,            5626,            3996,            3949,            1941,            3262,            5892,            3658,            3308,            2299,            3021,            3361,            5276,            4960,            1703,            3611,            5415,            5174,            5434,            4989,            3072,            4153,            3836,            3309,            3295,            2188,            1946,            1444,            738,            3092,            4275,            1584,            3691,            2618,            3207,            1743,            3190,            8474,            1586,            538,            2175,            3881,            776,            4550,            1238,            3427,            2512,            2808,            1339,            1286,            306,            1727,            5486,            3161,            2506,            3132,            4375,            3881,            4748,            3575,            379,            4597,            5247,            5488,            0,            1398,            2123,            5537,            5637,            4108,            3635,            3815,            4580,            1530,            5140,            4294,            4847,            3818,            1792,            4975,            3980,            3863,            1731,            4732,            ],        [            2562,            4968,            5614,            1498,            2592,            1620,            2129,            1244,            3216,            1564,            3560,            1169,            1679,            1717,            3199,            2396,            2873,            1467,            639,            3370,            3496,            822,            969,            996,            1864,            3752,            1486,            1895,            3424,            2402,            2355,            938,            1668,            3690,            1594,            1244,            1435,            1427,            642,            3473,            3451,            700,            884,            3612,            2182,            2466,            3186,            1478,            1439,            1136,            1506,            1492,            1650,            671,            476,            2601,            1583,            2766,            394,            2182,            1109,            1698,            3957,            1457,            6671,            188,            1795,            687,            2372,            1977,            3041,            1132,            1918,            918,            1005,            3553,            564,            2495,            1643,            3977,            1097,            1350,            1538,            2572,            2078,            2684,            1466,            2568,            3088,            3738,            3685,            2777,            0,            1192,            4028,            4128,            2305,            2041,            1751,            2777,            778,            3631,            2491,            3044,            1754,            789,            3172,            1155,            1799,            538,            3223,            ],        [            1666,            3400,            2390,            976,            1714,            440,            790,            548,            1168,            384,            3656,            484,            1222,            2774,            4439,            3636,            4113,            1173,            1514,            4610,            4736,            1847,            1482,            1871,            2086,            4992,            1436,            1006,            3520,            2498,            2451,            321,            2908,            3786,            2469,            2119,            475,            2186,            1517,            4713,            4691,            687,            980,            4166,            1988,            2248,            4289,            2718,            1661,            1555,            2746,            2732,            3481,            2473,            1456,            1385,            3095,            4006,            1316,            3422,            2349,            2938,            3036,            2697,            7911,            1458,            1831,            1981,            3612,            2069,            4281,            1745,            3158,            2158,            2245,            2632,            1034,            1239,            488,            5217,            1972,            2615,            2778,            3814,            3318,            2525,            2706,            1598,            4328,            4978,            4927,            1881,            1270,            0,            5268,            5368,            3545,            3281,            1843,            4017,            2823,            4871,            3731,            4286,            2629,            776,            4414,            908,            1891,            964,            4463,            ],        [            5627,            8178,            8824,            6243,            5675,            6942,            6656,            5056,            6281,            5377,            5188,            4536,            4594,            3711,            3184,            2381,            2858,            4382,            3710,            195,            1216,            3388,            4021,            3412,            4988,            1754,            4610,            4810,            4298,            4213,            4166,            4750,            3913,            4564,            4010,            3660,            4793,            3993,            3713,            1475,            490,            4513,            3963,            3567,            4969,            4710,            3922,            4044,            4505,            4188,            3268,            2980,            3600,            3903,            4183,            5666,            2430,            1014,            4323,            1598,            3012,            2082,            5907,            3032,            6656,            3624,            4953,            3411,            3286,            5042,            739,            4290,            1862,            3484,            3206,            5503,            4605,            5560,            6981,            1016,            3513,            3760,            3596,            3447,            2951,            5100,            2913,            5633,            1056,            1380,            3784,            5842,            3812,            5005,            0,            1770,            2334,            4286,            4167,            1826,            4293,            670,            2476,            3919,            4170,            4601,            4047,            4332,            4215,            4155,            865,            ],        [            5895,            8446,            9092,            6511,            5943,            7210,            6924,            5324,            6549,            7172,            4260,            4987,            3897,            2783,            3194,            2391,            2868,            4650,            3978,            1474,            1226,            3656,            4289,            3680,            4167,            1386,            3789,            4113,            3770,            3285,            3238,            5018,            2985,            4036,            4278,            3928,            5061,            2744,            3981,            1107,            1555,            4780,            4231,            3559,            4041,            3782,            2994,            2795,            3710,            3427,            2340,            2636,            3868,            4171,            4451,            5934,            2698,            1282,            4591,            1866,            3280,            2350,            6175,            2822,            6666,            3892,            5221,            3679,            3554,            5310,            1007,            4558,            2130,            3752,            2862,            5771,            4873,            5828,            7249,            1026,            3781,            4028,            2776,            2519,            2023,            4050,            2569,            5901,            1324,            310,            3632,            6110,            4080,            5081,            2132,            0,            2250,            3358,            3346,            1836,            4561,            1735,            2486,            2991,            3349,            4869,            3119,            4600,            3394,            4423,            825,            ],        [            5979,            8530,            9176,            6595,            6027,            7294,            7008,            6975,            6633,            7256,            2917,            3424,            2723,            1609,            1313,            510,            987,            3004,            2598,            2232,            1191,            2276,            2180,            2300,            2993,            1447,            2615,            2939,            2427,            2111,            2064,            3638,            1811,            2693,            2898,            2548,            3681,            1570,            2601,            1168,            2848,            3400,            2851,            1696,            2867,            2608,            1942,            1621,            2536,            2253,            1238,            1534,            2771,            2791,            3071,            6018,            1897,            2575,            3211,            3159,            2230,            2079,            6261,            1720,            4785,            2512,            6254,            2299,            2753,            5394,            2300,            3178,            2299,            1724,            1760,            5857,            3493,            5912,            7333,            1347,            2401,            2648,            1602,            1328,            849,            2876,            1467,            5985,            2617,            1695,            1913,            6194,            2700,            3701,            2037,            2085,            0,            2184,            2172,            472,            3181,            1640,            605,            1800,            2175,            3489,            1928,            2650,            2220,            3043,            2118,            ],        [            4311,            4962,            3952,            3421,            4341,            2885,            3577,            2993,            4965,            2829,            1672,            2126,            1729,            790,            2421,            1897,            2374,            1517,            2301,            4268,            2997,            1979,            1391,            2003,            1506,            3253,            1128,            1945,            1536,            514,            467,            2348,            373,            1802,            1889,            1826,            2522,            1274,            2304,            2974,            4349,            2584,            2065,            2182,            1246,            987,            2720,            970,            1023,            1862,            1755,            2051,            3288,            2494,            2774,            4350,            2414,            3664,            2914,            3080,            2747,            2596,            5595,            1460,            6172,            2215,            3544,            2002,            3270,            3726,            3939,            2881,            2816,            1493,            2277,            5191,            3196,            4244,            2908,            3153,            1939,            2351,            1541,            2061,            1565,            1255,            1984,            4317,            3986,            3501,            2475,            4526,            2403,            2457,            3843,            2739,            1806,            0,            1361,            2278,            2884,            3446,            1992,            2533,            1879,            2673,            2331,            1656,            1409,            2273,            3924,            ],        [            3582,            4277,            3267,            2736,            3612,            2200,            2892,            2308,            4236,            2144,            2064,            1441,            1044,            1182,            2813,            2289,            2766,            832,            1229,            3902,            3389,            1088,            775,            1163,            821,            3645,            443,            1260,            1928,            906,            859,            1663,            1500,            2194,            875,            812,            1837,            1189,            1232,            3366,            3983,            1899,            1482,            2574,            974,            715,            3112,            1362,            338,            848,            2038,            2024,            2182,            1765,            2045,            3621,            2115,            3298,            2185,            2714,            1641,            2230,            4489,            1445,            6564,            1450,            2815,            1273,            2904,            2997,            3573,            2152,            2450,            1450,            1537,            4085,            2467,            3515,            2223,            3545,            925,            1907,            1526,            2453,            1957,            933,            1998,            3588,            3620,            4270,            2867,            3797,            1638,            1772,            4560,            4660,            2198,            1873,            0,            2670,            2155,            4163,            2384,            2925,            784,            1988,            2723,            971,            724,            1674,            3755,            ],        [            6183,            8734,            9380,            6799,            6231,            7076,            7212,            7179,            6837,            7460,            3121,            3628,            2927,            1813,            1517,            714,            1191,            3208,            2802,            1760,            719,            2480,            2384,            2504,            3197,            975,            2819,            3143,            2631,            2315,            2268,            3842,            2015,            2897,            3102,            2752,            3885,            1774,            2805,            696,            2376,            3604,            3055,            1900,            3071,            2812,            2146,            1825,            2740,            2457,            1442,            1738,            2975,            2995,            3275,            6222,            2101,            2103,            3415,            2687,            2434,            3171,            6465,            1924,            4989,            2716,            4045,            2503,            2957,            5598,            1828,            3382,            2951,            1928,            1964,            6061,            3697,            6116,            7537,            875,            2605,            2852,            1806,            1532,            1053,            3080,            1671,            6189,            2145,            1223,            2117,            6398,            2904,            3905,            1565,            1613,            667,            2388,            2376,            0,            3385,            1168,            809,            2004,            2379,            3693,            2132,            2854,            2424,            3247,            1646,            ],        [            2217,            4623,            5269,            2833,            2247,            2439,            3246,            2063,            2871,            2383,            3624,            1988,            2644,            1781,            3263,            2460,            2937,            2137,            1309,            3434,            4241,            942,            1575,            966,            2542,            3816,            2164,            2860,            3488,            2466,            2419,            1757,            1732,            3754,            1564,            1214,            2254,            1491,            1312,            3537,            3515,            1519,            1517,            3676,            3222,            2963,            3250,            1542,            2457,            2174,            1570,            1556,            1714,            416,            680,            2256,            1647,            2830,            820,            2246,            1173,            1762,            4021,            1521,            6735,            984,            1450,            645,            2436,            1632,            3105,            787,            1982,            982,            1069,            3617,            1102,            2150,            3571,            4041,            1067,            976,            1602,            2636,            2142,            2654,            1530,            2223,            3152,            3802,            3749,            2432,            1172,            2011,            4092,            4192,            2369,            2105,            1721,            2841,            0,            3695,            2555,            3108,            1724,            1608,            3236,            1886,            1769,            1589,            3287,            ],        [            7180,            9731,            10377,            7796,            7228,            8495,            8209,            8176,            7834,            8457,            4118,            4434,            3924,            2810,            2514,            1711,            2188,            3924,            3608,            710,            546,            3286,            3381,            3310,            4194,            1084,            3816,            4140,            3628,            3312,            3265,            4648,            3012,            3894,            3908,            3558,            4691,            2771,            3611,            805,            1185,            4410,            3861,            2897,            4068,            3809,            3143,            2822,            3737,            3454,            2439,            2735,            3498,            3801,            4081,            7219,            2328,            912,            4221,            1496,            2910,            1980,            7462,            2921,            5986,            3522,            4851,            3309,            3184,            6595,            637,            4188,            1760,            3382,            2961,            7058,            4503,            7113,            8534,            346,            3411,            3658,            2803,            2529,            2050,            4077,            2668,            7186,            954,            1176,            3114,            7395,            3710,            4711,            515,            1566,            1664,            3385,            3373,            1156,            4191,            0,            1806,            3001,            3376,            4499,            3129,            3851,            3421,            4053,            763,            ],        [            5374,            7925,            8571,            5990,            5422,            6267,            6403,            6370,            6028,            6651,            2312,            6847,            3182,            2068,            708,            969,            382,            4031,            7271,            2731,            1690,            6949,            2639,            6973,            4020,            1946,            3642,            3398,            1822,            2716,            2669,            6616,            2270,            2088,            7571,            3007,            7300,            2029,            7274,            1667,            3347,            6378,            7524,            1091,            3760,            3501,            2189,            2080,            3537,            2712,            1697,            1993,            6101,            7464,            5931,            5413,            2356,            3074,            5791,            3022,            6531,            2538,            5656,            2179,            4180,            7185,            5649,            6972,            3212,            4789,            2799,            6349,            2758,            2183,            2219,            5252,            5961,            5307,            6728,            1846,            7074,            7321,            2061,            1223,            1308,            3769,            1926,            5380,            3116,            2194,            1308,            5589,            6073,            6870,            2536,            2584,            499,            2643,            2631,            971,            6641,            2139,            0,            1729,            2634,            6467,            2074,            3109,            2679,            6406,            2617,            ],        [            5299,            7850,            5866,            5915,            5347,            4821,            6328,            4445,            5953,            4765,            1269,            3597,            3200,            2109,            807,            1749,            1169,            2988,            3099,            4252,            2849,            2777,            2680,            2801,            2977,            3105,            2599,            3416,            779,            1673,            1626,            4139,            2311,            1045,            3399,            3049,            3993,            2070,            3102,            2826,            4333,            3901,            3352,            568,            2717,            2458,            906,            2121,            2494,            2753,            1739,            2035,            3272,            3292,            3572,            5338,            2398,            3648,            3712,            3064,            2731,            2580,            5579,            2221,            6024,            3013,            4342,            2800,            3254,            4714,            3923,            3679,            2800,            2224,            2261,            5175,            3994,            5232,            6653,            3005,            2902,            3149,            2263,            1913,            1349,            2726,            1968,            5305,            3970,            3353,            641,            5514,            3201,            4393,            3695,            2723,            1658,            2684,            2672,            2130,            3682,            3298,            1844,            0,            2675,            3990,            892,            3127,            2720,            3618,            4105,            ],        [            3561,            4384,            3374,            2843,            3591,            2307,            2999,            2415,            4215,            2251,            2346,            1548,            1151,            1464,            3710,            2907,            3384,            939,            1208,            3881,            4688,            1067,            754,            1142,            928,            4263,            550,            1367,            2210,            1188,            1141,            1770,            1479,            2476,            854,            791,            1944,            1168,            1211,            3984,            3962,            2006,            1487,            2856,            1246,            1159,            3394,            1289,            503,            827,            2017,            2003,            2161,            1744,            2024,            3600,            2094,            3277,            2164,            2693,            1620,            2209,            4468,            1424,            7182,            1429,            2794,            1252,            2883,            2976,            3552,            2131,            2429,            1429,            1516,            4064,            2446,            3494,            2330,            4488,            904,            1886,            1505,            3085,            2589,            1215,            1977,            3567,            3599,            4249,            3149,            3776,            1617,            1879,            4539,            4639,            2816,            1852,            282,            3288,            2134,            4142,            3002,            3557,            0,            2095,            3005,            1078,            137,            1653,            3734,            ],        [            1399,            3941,            2931,            709,            1447,            786,            1677,            455,            2055,            730,            4176,            717,            1618,            2443,            3925,            3122,            3599,            1406,            1000,            4096,            4903,            1333,            1330,            1357,            2225,            4478,            1669,            1514,            4040,            3018,            2971,            486,            2394,            4306,            1955,            1605,            983,            2153,            1003,            4199,            4177,            89,            796,            4340,            2543,            2827,            3775,            2204,            1800,            1497,            2232,            2218,            2376,            1959,            680,            844,            2309,            3492,            540,            2908,            1835,            2424,            2769,            2183,            7397,            944,            1632,            1467,            3098,            1814,            3767,            969,            2644,            1644,            1731,            2365,            258,            972,            983,            4703,            1458,            2101,            2264,            3300,            2804,            3045,            2192,            1331,            3814,            4464,            4413,            1614,            756,            740,            4754,            4854,            3031,            2767,            2112,            3503,            982,            4357,            3217,            3772,            2115,            0,            3900,            1141,            2160,            450,            3949,            ],        [            5253,            7804,            8450,            5869,            5301,            4775,            6282,            4399,            5907,            4719,            3608,            3879,            3177,            2063,            2506,            1703,            2180,            3458,            3053,            4206,            2803,            2731,            2634,            2755,            3447,            3059,            3069,            3393,            3118,            2565,            2518,            4093,            2265,            3384,            3353,            3003,            4136,            2024,            3056,            2780,            4287,            3855,            3306,            2907,            3321,            3062,            860,            2075,            2990,            2707,            1693,            1989,            3226,            3246,            3526,            5292,            2352,            3602,            3666,            3018,            2685,            2534,            5533,            2175,            5978,            2967,            4296,            2754,            3208,            4668,            3877,            3633,            2754,            2178,            2215,            5129,            3948,            5186,            6607,            2959,            2856,            3103,            2217,            1867,            1303,            3330,            1922,            5259,            3924,            3307,            2980,            5468,            3155,            4347,            3649,            2677,            1612,            2638,            2626,            2084,            3636,            3252,            1798,            2339,            2629,            3944,            0,            3104,            2674,            3572,            4059,            ],        [            3885,            3710,            2700,            1765,            3915,            1229,            1738,            1337,            2116,            1173,            2748,            470,            477,            1866,            3246,            2443,            2920,            265,            746,            4205,            3543,            1391,            574,            1466,            1178,            3799,            528,            693,            2612,            1590,            1543,            692,            1589,            2878,            1178,            1115,            660,            1278,            749,            3520,            4286,            928,            409,            3258,            1496,            1409,            3796,            1399,            753,            647,            2341,            2327,            2485,            2068,            2348,            1861,            2418,            3601,            2488,            3017,            1944,            2533,            4792,            1534,            6718,            1753,            3118,            1576,            3207,            3300,            3876,            2455,            2753,            1567,            1840,            4388,            1275,            2028,            1252,            3699,            1228,            2210,            1615,            2607,            2111,            1617,            2301,            2387,            3923,            4573,            3551,            4100,            1941,            801,            4863,            4963,            2352,            1962,            935,            2824,            2458,            4466,            2538,            3079,            1124,            1017,            3407,            0,            983,            617,            4058,            ],        [            3424,            4247,            3237,            2706,            3454,            2170,            2862,            2278,            4078,            2114,            2209,            1411,            1014,            1327,            2958,            2770,            3247,            802,            1071,            3744,            4551,            930,            617,            1005,            791,            4126,            413,            1230,            2073,            1051,            1004,            1633,            1342,            2339,            717,            654,            1807,            1031,            1074,            3847,            3825,            1869,            1288,            2719,            1109,            1022,            3257,            1152,            366,            690,            1880,            1866,            2024,            1607,            1887,            3463,            1957,            3140,            2027,            2556,            1483,            2072,            4331,            1287,            7045,            1292,            2657,            1115,            2746,            2839,            3415,            1994,            2292,            1292,            1379,            3927,            2309,            3357,            2193,            4351,            767,            1749,            1368,            2946,            2452,            1078,            1840,            3430,            3462,            4112,            3012,            3639,            1480,            1742,            4402,            4502,            2679,            1715,            145,            3151,            1997,            4005,            2865,            3418,            626,            1958,            2868,            941,            0,            1516,            3597,            ],        [            3326,            4051,            3041,            1156,            3356,            1278,            1787,            902,            2165,            1222,            3726,            564,            1074,            1993,            3475,            2672,            3149,            862,            550,            3646,            3772,            883,            880,            907,            1775,            4028,            1125,            1290,            3590,            2568,            2521,            596,            1944,            3856,            1505,            1155,            1093,            1703,            553,            3749,            3727,            358,            346,            3888,            2093,            2377,            3462,            1754,            1350,            1047,            1782,            1768,            1926,            1509,            1789,            1291,            1859,            3042,            727,            2458,            1385,            1974,            4233,            1733,            6947,            911,            2559,            1017,            2648,            2741,            3317,            1896,            2194,            1194,            1281,            3829,            705,            1419,            1301,            4253,            1008,            1651,            1814,            2848,            2354,            2595,            1742,            3332,            3364,            4014,            3961,            3541,            538,            850,            4304,            4404,            2581,            2317,            1662,            3053,            1899,            3907,            2767,            3322,            1665,            447,            3448,            617,            1710,            0,            3499,            ],        [            5070,            7621,            8267,            5686,            5118,            6385,            6099,            6066,            5724,            6347,            4323,            3979,            3960,            2846,            3203,            2400,            2877,            4442,            3153,            649,            1235,            2831,            3417,            2855,            4431,            1473,            4053,            4176,            3833,            3348,            3301,            4193,            3048,            4099,            3453,            3103,            4115,            2807,            3156,            1194,            730,            3955,            3406,            3622,            4104,            3845,            3196,            2858,            3948,            3490,            2403,            2699,            3043,            3346,            3626,            5109,            1873,            457,            3766,            1041,            2455,            1525,            5350,            2475,            6675,            3067,            4396,            2854,            2729,            4485,            182,            3733,            1305,            2927,            2515,            4946,            4048,            5003,            6424,            1035,            2956,            3203,            2839,            2582,            2086,            4113,            2356,            5076,            499,            515,            3695,            5285,            3255,            4522,            1307,            905,            2313,            3421,            3610,            1845,            3736,            910,            2495,            3054,            3613,            4119,            3182,            3887,            3658,            3672,            0,            ],        ], 'starts': [2, 88, 32, 61], 'ends': [2, 88, 32, 61]},    'NW-5': {'distance_matrix': [        [            0,            3971,            2510,            2285,            3686,            1479,            1781,            2067,            1995,            1263,            1253,            5364,            4704,            4931,            4638,            4707,            4835,            4791,            5005,            1688,            4030,            4856,            2466,            3438,            4370,            3440,            2052,            4458,            4552,            4830,            2440,            ],        [            3166,            0,            1818,            1593,            917,            2154,            1612,            1444,            1303,            2553,            2262,            6126,            5466,            5693,            5400,            5469,            5597,            5553,            5767,            2454,            4792,            5618,            2717,            4200,            5132,            4202,            2303,            5220,            1832,            5592,            3202,            ],        [            2728,            2910,            0,            392,            2625,            1716,            1174,            1006,            697,            2115,            1824,            7719,            7059,            7286,            6993,            7062,            7190,            7146,            7360,            4047,            6385,            7211,            4310,            5793,            6725,            5795,            3896,            6813,            2055,            7185,            4795,            ],        [            2953,            3135,            225,            0,            2850,            1941,            1399,            1231,            922,            2340,            2049,            7944,            7284,            7511,            7218,            7287,            7415,            7371,            7585,            4272,            6610,            7436,            4535,            6018,            6950,            6020,            4121,            7038,            2280,            7410,            5020,            ],        [            3811,            645,            2463,            2238,            0,            2799,            2257,            2089,            1948,            3198,            2907,            6771,            6111,            6338,            6045,            6114,            6242,            6198,            6412,            3099,            5437,            6263,            3362,            4845,            5777,            4847,            2948,            5865,            2477,            6237,            3847,            ],        [            1149,            3264,            1803,            1578,            2979,            0,            889,            1360,            1288,            495,            499,            6078,            5418,            5645,            5352,            5421,            5549,            5505,            5719,            2402,            4744,            5570,            3180,            4152,            5084,            4154,            2766,            5172,            3411,            5544,            3154,            ],        [            1998,            2641,            1180,            955,            2356,            542,            0,            737,            665,            1037,            657,            6214,            5554,            5781,            5488,            5557,            5685,            5641,            5855,            2538,            4880,            5706,            3316,            4288,            5220,            4290,            2902,            5308,            2767,            5680,            3290,            ],        [            2174,            1904,            1665,            1440,            1619,            1273,            1459,            0,            1150,            1768,            1308,            6390,            5730,            5957,            5664,            5733,            5861,            5817,            6031,            2714,            5056,            5882,            3492,            4464,            5396,            6304,            3078,            5484,            2485,            5856,            3466,            ],        [            2416,            2598,            515,            290,            2313,            1404,            862,            694,            0,            1803,            1512,            8234,            7574,            7801,            7508,            7577,            7705,            7661,            7875,            3408,            6900,            7726,            4825,            6308,            7240,            6998,            4411,            7328,            2570,            7700,            5310,            ],        [            654,            3649,            2188,            1963,            3364,            1157,            1459,            1745,            1673,            0,            1231,            5583,            4923,            5150,            4857,            4926,            5054,            5010,            5224,            1907,            4249,            5075,            2685,            3657,            4589,            3659,            2271,            4677,            4230,            5049,            2659,            ],        [            2012,            2820,            1359,            1134,            2535,            717,            698,            916,            844,            1082,            0,            6228,            5568,            5795,            5502,            5571,            5699,            5655,            5869,            2552,            4894,            5720,            3330,            4302,            5234,            4304,            2916,            5322,            2781,            5694,            3304,            ],        [            5070,            6911,            8729,            8504,            6626,            5870,            6399,            6866,            6826,            6365,            5926,            0,            554,            456,            1337,            784,            912,            985,            631,            4761,            1199,            1182,            5197,            2771,            2186,            2729,            4783,            1205,            7166,            1156,            4170,            ],        [            4528,            6369,            8187,            7962,            6084,            5328,            5857,            6324,            6284,            5823,            5384,            668,            0,            235,            795,            242,            370,            443,            962,            4219,            1016,            1583,            4655,            2229,            1644,            2187,            4241,            663,            6624,            1207,            3628,            ],        [            5166,            7007,            8825,            8600,            6722,            5966,            6495,            6962,            6922,            6461,            6022,            433,            650,            0,            1433,            880,            1008,            1081,            727,            4857,            1254,            1278,            5293,            2867,            2282,            2825,            4879,            1278,            7262,            1252,            4266,            ],        [            3733,            5574,            7392,            7167,            5289,            4533,            5062,            5529,            5489,            5028,            4589,            1680,            1020,            1247,            0,            1023,            1151,            1107,            1262,            3424,            318,            1113,            3860,            1434,            849,            1392,            3446,            774,            5829,            737,            2833,            ],        [            4286,            6127,            7945,            7720,            5842,            5086,            5615,            6082,            6042,            5581,            5142,            1299,            667,            866,            553,            0,            128,            754,            1102,            3977,            871,            1666,            4413,            1987,            1402,            1945,            3999,            421,            6382,            1290,            3386,            ],        [            4158,            5999,            7817,            7592,            5714,            4958,            5487,            5954,            5914,            5453,            5014,            1171,            539,            738,            425,            542,            0,            626,            974,            3849,            743,            1538,            4285,            1859,            1274,            1817,            3871,            293,            6254,            1162,            3258,            ],        [            4817,            6658,            8476,            8251,            6373,            5617,            6146,            6613,            6573,            6112,            5673,            1188,            528,            755,            1084,            531,            659,            0,            1482,            4508,            1536,            2103,            4944,            2518,            1933,            2476,            4530,            937,            6913,            1727,            3917,            ],        [            4863,            6704,            8522,            8297,            6419,            5663,            6192,            6659,            6619,            6158,            5719,            758,            953,            902,            1293,            1183,            1311,            1384,            0,            4554,            944,            551,            4990,            2564,            1979,            2522,            4576,            1161,            6959,            525,            3963,            ],        [            1700,            3542,            3971,            3746,            3257,            2500,            3029,            3496,            3456,            2995,            2556,            3676,            3016,            3243,            2950,            3019,            3147,            3103,            3317,            0,            2342,            3168,            1828,            1750,            2682,            1752,            1414,            2770,            3797,            3142,            752,            ],        [            3889,            5730,            7548,            7323,            5445,            4689,            5218,            5685,            5645,            5184,            4745,            1334,            999,            901,            1063,            1229,            1357,            1430,            944,            3580,            0,            795,            4016,            1590,            1005,            1548,            3602,            930,            5985,            419,            2989,            ],        [            4714,            6555,            8373,            8148,            6270,            5514,            6043,            6510,            6470,            6009,            5570,            1309,            1504,            1468,            1858,            1734,            1862,            1935,            551,            4405,            795,            0,            4841,            2415,            1830,            2373,            4427,            2001,            6810,            376,            3814,            ],        [            2878,            1714,            3532,            3307,            1429,            3678,            3326,            3158,            3017,            4173,            3734,            6241,            5581,            5808,            5515,            5584,            5712,            5668,            5882,            2569,            4907,            5733,            0,            4315,            5247,            4317,            2418,            5335,            1969,            5707,            3317,            ],        [            3816,            5657,            7475,            7250,            5372,            4616,            5145,            5612,            5572,            5111,            4672,            1926,            1266,            1493,            1200,            1269,            1397,            1353,            1567,            3507,            592,            1418,            3943,            0,            932,            1475,            3529,            1020,            5912,            1392,            2916,            ],        [            2884,            4725,            6543,            6318,            4440,            3684,            4213,            4680,            4640,            4179,            3740,            2511,            1851,            2078,            1785,            1854,            1982,            1938,            2152,            2575,            1177,            2003,            3011,            585,            0,            543,            2597,            1605,            4980,            1977,            1984,            ],        [            2341,            4182,            4612,            4387,            3897,            3141,            3670,            4137,            4097,            3636,            3197,            4365,            3705,            3932,            3639,            3708,            3836,            3792,            4006,            2032,            3031,            3857,            2468,            2439,            3371,            0,            2054,            3459,            4437,            3831,            1441,            ],        [            1743,            2128,            3946,            3721,            1843,            2543,            3740,            3572,            3431,            3038,            2599,            5110,            4450,            4677,            4384,            4453,            4581,            4537,            4751,            1434,            3776,            4602,            414,            3184,            4116,            3186,            0,            4204,            2383,            4576,            2186,            ],        [            4535,            6376,            8194,            7969,            6091,            5335,            5864,            6331,            6291,            5830,            5391,            906,            246,            473,            802,            249,            377,            333,            1200,            4226,            1254,            1821,            4662,            2236,            1651,            2194,            4248,            0,            6631,            1445,            3635,            ],        [            2301,            3969,            4572,            4347,            3684,            3101,            3630,            4097,            4057,            3596,            3157,            5664,            5004,            5231,            4938,            5007,            5135,            5091,            5305,            1992,            4330,            5156,            2255,            3738,            4670,            3740,            1841,            4758,            0,            5130,            2740,            ],        [            4688,            6529,            8347,            8122,            6244,            5488,            6017,            6484,            6444,            5983,            5544,            1525,            1190,            1092,            1482,            1420,            1548,            1621,            525,            4379,            419,            376,            4815,            2389,            1804,            2347,            4401,            1349,            6784,            0,            3788,            ],        [            3341,            5182,            5612,            5387,            4897,            4141,            4670,            5137,            5097,            4636,            4197,            2924,            2264,            2491,            2198,            2267,            2395,            2351,            2565,            3032,            1590,            2416,            3468,            998,            1930,            1000,            3054,            2018,            5437,            2390,            0,            ],        ], 'starts': [23, 25, 1, 16, 8], 'ends': [23, 25, 1, 16, 8]},    'NW-6': {'distance_matrix': [        [            0,            646,            3319,            4262,            5067,            4257,            3941,            1867,            2853,            5208,            4261,            2830,            3393,            3264,            5136,            2630,            2048,            1462,            3799,            4454,            1794,            3093,            3633,            4081,            1989,            4503,            2901,            4222,            2058,            4596,            4702,            2461,            3774,            3012,            3487,            3074,            1790,            2535,            2310,            2434,            3689,            3589,            1318,            2040,            2353,            1832,            3282,            4353,            2355,            2063,            1489,            3345,            3790,            2442,            2634,            3281,            2556,            2852,            2553,            3760,            1741,            4370,            176,            2507,            3554,            4187,            2452,            3666,            2682,            4555,            3053,            1155,            2342,            3624,            2295,            3772,            2435,            4152,            3147,            4483,            2867,            1870,            3389,            4013,            ],        [            667,            0,            2673,            3616,            4421,            3611,            4103,            1221,            2207,            4562,            3615,            2184,            2747,            2618,            4490,            1984,            1402,            1476,            3153,            3808,            1808,            2447,            2987,            3435,            1343,            3857,            2255,            3576,            1412,            3950,            4056,            1815,            3128,            2366,            2841,            2428,            1144,            1889,            1664,            1788,            3043,            2943,            672,            1394,            1707,            1186,            2636,            3707,            1709,            1417,            843,            2699,            3144,            1796,            1988,            2635,            1910,            2206,            1907,            3114,            1095,            3724,            843,            1861,            2908,            3541,            1806,            3020,            2036,            3909,            2407,            509,            1696,            2978,            1649,            3126,            1789,            3506,            2501,            3837,            2221,            1884,            2743,            3367,            ],        [            2674,            2667,            0,            1841,            2562,            4382,            4874,            1992,            2978,            2703,            1756,            2955,            3093,            3442,            3961,            2755,            2173,            3483,            3924,            1949,            3815,            2154,            3758,            4206,            2114,            4220,            2292,            3047,            2183,            2091,            2197,            2586,            1269,            3412,            982,            1824,            1319,            2317,            2247,            462,            3814,            1319,            1740,            1123,            1223,            2254,            292,            1618,            2777,            2485,            1911,            2095,            1285,            1463,            1913,            2031,            2338,            347,            1304,            1922,            2163,            4495,            2850,            1786,            1716,            2349,            2577,            3040,            2807,            3380,            521,            1826,            1621,            1119,            1796,            1502,            2560,            2977,            642,            4608,            2250,            3891,            987,            1528,            ],        [            4516,            4509,            1842,            0,            3396,            5209,            5701,            3834,            5356,            3537,            2590,            4700,            3927,            4276,            4795,            4872,            4015,            5325,            4751,            2783,            5657,            2988,            4585,            5033,            3956,            5054,            3126,            3881,            4025,            2925,            3031,            3625,            2103,            4246,            1820,            1865,            3161,            2358,            2489,            2304,            4641,            1360,            3582,            2965,            2707,            3184,            2134,            2072,            3549,            3940,            3753,            2625,            1180,            2542,            2361,            2072,            2379,            2189,            2383,            2190,            3361,            5187,            4692,            2488,            2307,            1906,            3616,            3874,            3846,            4214,            1730,            3668,            2700,            1957,            2740,            1057,            4402,            3811,            1670,            5435,            3084,            5733,            1536,            1300,            ],        [            4806,            4772,            3043,            3710,            0,            1813,            2305,            2805,            1960,            141,            1745,            1304,            531,            880,            1399,            1476,            1884,            3604,            1355,            1938,            3590,            1605,            1189,            1637,            2351,            1658,            1898,            485,            1828,            1365,            2631,            1400,            3287,            850,            2938,            4834,            3373,            4512,            4287,            3505,            1245,            4329,            3474,            3623,            4009,            3988,            3335,            4849,            3913,            4219,            3645,            5402,            3238,            3761,            3355,            5041,            3935,            3390,            3872,            4932,            3897,            1791,            4982,            3826,            4726,            4683,            1449,            478,            1214,            818,            3564,            3560,            3661,            3072,            3530,            4026,            1711,            415,            2871,            2039,            1864,            4012,            4030,            4035,            ],        [            3796,            4174,            3928,            4595,            2325,            0,            492,            2017,            1348,            2466,            2630,            1081,            1519,            2189,            757,            1253,            2026,            3006,            468,            2823,            2992,            2136,            734,            176,            2371,            498,            2341,            1373,            2036,            2250,            3516,            1843,            4172,            1838,            3823,            4439,            2761,            3900,            3675,            4390,            568,            5214,            2862,            3011,            3397,            3376,            4220,            5734,            3899,            3607,            3033,            6287,            4123,            3149,            3341,            5926,            3921,            4275,            3260,            5817,            3285,            365,            3972,            3214,            5611,            5568,            1892,            1787,            1657,            1706,            4449,            2948,            3049,            3957,            2918,            4911,            1488,            1303,            3756,            226,            2307,            3414,            4915,            4920,            ],        [            3779,            3745,            4420,            5087,            2817,            492,            0,            2509,            1840,            2958,            3122,            1573,            2011,            2681,            1249,            1745,            2518,            2577,            960,            3315,            2563,            2628,            1226,            668,            2863,            990,            2833,            1865,            2528,            2742,            4008,            2335,            4664,            2330,            4315,            4931,            3253,            4392,            4167,            4882,            1060,            5706,            3354,            3503,            3889,            3868,            4712,            6226,            4391,            4099,            3525,            6779,            4615,            3641,            3833,            6418,            4413,            4767,            3752,            6309,            3777,            857,            3955,            3706,            6103,            6060,            2384,            2279,            2149,            2198,            4941,            3440,            3541,            4449,            3410,            5403,            1980,            1795,            4248,            266,            2799,            2985,            5407,            5412,            ],        [            1813,            1806,            2307,            3250,            3539,            2622,            3114,            0,            1218,            3680,            2317,            963,            1526,            1397,            3501,            763,            181,            2622,            2164,            2510,            2954,            1226,            1392,            2446,            354,            2868,            1034,            2587,            191,            3464,            2758,            594,            2762,            1145,            2475,            2456,            778,            1917,            1692,            1422,            2054,            2487,            879,            1028,            1414,            1393,            2239,            3251,            1916,            1624,            1050,            2727,            2778,            1166,            1358,            2767,            1938,            1840,            1277,            2658,            1302,            2735,            1989,            1231,            2452,            3085,            585,            1799,            815,            2920,            2010,            965,            1066,            2612,            935,            2670,            568,            2517,            2135,            2848,            1000,            3030,            2443,            3001,            ],        [            2448,            2441,            2942,            3885,            3037,            2120,            2612,            845,            0,            3178,            3342,            1264,            2231,            1859,            2999,            1064,            1026,            2639,            1662,            3535,            2625,            2071,            1496,            1944,            1317,            2366,            1879,            2085,            1036,            2962,            4228,            1439,            3397,            1607,            3110,            3091,            1413,            2552,            2327,            2057,            1552,            3122,            1514,            1663,            2049,            2028,            2874,            3886,            2551,            2259,            1685,            3362,            3413,            1801,            1993,            3402,            2573,            2475,            1912,            3293,            1937,            2233,            2624,            1866,            3087,            3720,            1430,            2499,            1660,            2418,            2645,            1600,            1701,            3247,            1570,            3305,            869,            2015,            2770,            2346,            1845,            3047,            3078,            3636,            ],        [            4681,            4647,            2902,            3569,            1299,            1688,            2180,            1856,            1835,            0,            1604,            1179,            390,            739,            1258,            1351,            1743,            3479,            1230,            1797,            3465,            1464,            1064,            1512,            2210,            1517,            1757,            344,            1687,            1224,            2490,            1259,            3146,            709,            2797,            4693,            3248,            4387,            4162,            3364,            1120,            4188,            3349,            3498,            3884,            3863,            3194,            4708,            4386,            4094,            3520,            5261,            3097,            3636,            3828,            4900,            4408,            3249,            3747,            4791,            3772,            1650,            4857,            3701,            4585,            4542,            1308,            337,            1073,            677,            3423,            3435,            3536,            2931,            3405,            3885,            1586,            274,            2730,            1914,            1723,            3887,            3889,            3894,            ],        [            5539,            6254,            2865,            3532,            1482,            3295,            3787,            4287,            3442,            1623,            0,            2786,            2013,            2362,            2881,            2958,            3366,            5086,            2837,            193,            5072,            2645,            2671,            3119,            3833,            3140,            2783,            1967,            3310,            1187,            441,            2882,            3109,            2332,            2760,            4656,            4184,            5149,            5280,            3327,            2727,            4151,            4605,            3988,            4088,            4990,            3157,            4671,            5642,            5350,            4776,            5224,            3060,            4454,            4904,            4863,            5170,            3212,            4295,            4754,            5028,            3273,            5715,            4777,            4548,            4505,            2931,            1960,            2696,            2300,            3386,            4691,            4612,            2894,            4661,            3848,            3193,            1897,            2693,            3521,            2741,            5494,            3852,            3857,            ],        [            2777,            2770,            3651,            4318,            2048,            1081,            1573,            964,            602,            2189,            2353,            0,            1242,            1340,            2010,            200,            973,            3082,            623,            2546,            3068,            1627,            457,            905,            1318,            1327,            1826,            1096,            983,            1973,            3239,            1334,            3895,            1088,            3546,            3420,            1742,            2881,            2656,            2386,            513,            3451,            1843,            1992,            2378,            2357,            3943,            4215,            2880,            2588,            2014,            3691,            3846,            2130,            2322,            3731,            2902,            2804,            2241,            3622,            2266,            1194,            2953,            2195,            3416,            4049,            1377,            1510,            1148,            1429,            2974,            1929,            2030,            3680,            1899,            3634,            435,            1026,            3479,            1307,            1792,            3490,            3407,            3965,            ],        [            3279,            4486,            2907,            3574,            1304,            1527,            2019,            1466,            1674,            1445,            1609,            1018,            0,            875,            1263,            1190,            1353,            3318,            1069,            1802,            3304,            1162,            903,            1351,            1820,            1522,            1367,            349,            1297,            1229,            2495,            869,            3151,            319,            2802,            3922,            2244,            3383,            3158,            2888,            959,            4193,            2345,            2494,            2880,            2859,            3199,            4713,            3382,            3090,            2516,            5266,            3102,            2632,            2824,            4233,            3404,            3306,            2743,            4124,            2768,            1655,            3455,            2697,            3918,            4547,            918,            641,            683,            682,            3428,            2431,            2532,            2936,            2401,            3890,            1475,            279,            2735,            1753,            1333,            3726,            3894,            3899,            ],        [            3212,            3205,            2876,            3543,            1141,            1849,            2341,            1399,            2020,            1282,            1578,            1340,            1045,            0,            1902,            1512,            1286,            3664,            1391,            1771,            3650,            725,            1225,            1673,            1753,            2161,            843,            988,            1230,            1198,            2019,            802,            3120,            556,            2771,            3855,            2177,            3316,            3091,            2821,            1281,            4162,            2278,            2427,            2813,            2792,            3168,            4682,            3315,            3023,            2449,            4126,            3071,            2565,            2757,            4166,            3337,            3239,            2676,            4057,            2701,            2294,            3388,            2630,            3851,            4516,            851,            402,            616,            1321,            3397,            2364,            2465,            2905,            2334,            3859,            1408,            918,            2704,            2075,            809,            4072,            3863,            3868,            ],        [            5523,            5489,            3644,            4311,            2041,            757,            1249,            2698,            2677,            2182,            2346,            1398,            1232,            1902,            0,            1570,            2585,            4321,            785,            2539,            4307,            2394,            1051,            681,            3052,            259,            2599,            914,            2529,            1966,            3232,            2101,            3888,            1551,            3539,            5435,            4090,            5229,            5004,            4106,            885,            4930,            4191,            4340,            4726,            4705,            3936,            5450,            5228,            4936,            4362,            6003,            3839,            4478,            4670,            5642,            5250,            3991,            4589,            5533,            4614,            392,            5699,            4543,            5327,            5284,            2150,            1500,            1915,            581,            4165,            4277,            4378,            3673,            4247,            4627,            1805,            1016,            3472,            983,            2565,            4729,            4631,            4636,            ],        [            2577,            2570,            3071,            4014,            2220,            1253,            1745,            764,            636,            2361,            2525,            200,            1414,            1512,            2182,            0,            773,            3254,            795,            2718,            3240,            1818,            629,            1077,            1118,            1499,            1626,            1268,            783,            2145,            3411,            1186,            3526,            1112,            3239,            3220,            1542,            2681,            2456,            2186,            685,            3251,            1643,            1792,            2178,            2157,            3003,            4015,            2680,            2388,            1814,            3491,            3542,            1930,            2122,            3531,            2702,            2604,            2041,            3422,            2066,            1366,            2753,            1995,            3216,            3849,            1177,            1682,            1407,            1601,            2774,            1729,            1830,            3376,            1699,            3434,            235,            1198,            2899,            1479,            1592,            3662,            3207,            3765,            ],        [            1926,            1919,            2420,            3363,            2924,            1957,            2449,            113,            1331,            3065,            3362,            904,            1628,            1499,            2886,            704,            0,            2735,            1499,            3555,            3067,            1348,            1333,            1781,            467,            2203,            1156,            1972,            313,            2849,            3803,            716,            2875,            1247,            2588,            2569,            891,            2030,            1805,            1535,            1389,            2600,            992,            1141,            1527,            1506,            2352,            3364,            2029,            1737,            1163,            2840,            2891,            1279,            1471,            2880,            2051,            1953,            1390,            2771,            1415,            2070,            2102,            1344,            2565,            3198,            707,            1901,            937,            2305,            2123,            1078,            1179,            2725,            1048,            2783,            509,            1902,            2248,            2183,            1122,            3143,            2556,            3114,            ],        [            2161,            2127,            4800,            5743,            4576,            3659,            2815,            3439,            2594,            4717,            4881,            3382,            3770,            4440,            4538,            3554,            3620,            0,            3201,            5074,            332,            4437,            3035,            3483,            3470,            3905,            4473,            3624,            3630,            4501,            5767,            4639,            5255,            4089,            4968,            4555,            3271,            4016,            3791,            3915,            3091,            5070,            2799,            3521,            3834,            3313,            4763,            5834,            3836,            3536,            2970,            4826,            5271,            3923,            4115,            4762,            4037,            4333,            4034,            5241,            3222,            3772,            2337,            3988,            5035,            5668,            4024,            4038,            4453,            3957,            4534,            2636,            3823,            5105,            3776,            5253,            3463,            3554,            4628,            3885,            4439,            408,            4870,            5494,            ],        [            3372,            4301,            3470,            4137,            1867,            468,            960,            1559,            1489,            2008,            2172,            623,            1061,            1731,            973,            795,            1568,            3133,            0,            2365,            3119,            1678,            276,            292,            1913,            714,            1883,            915,            1578,            1792,            3058,            1385,            3714,            1380,            3365,            5261,            2337,            3476,            3251,            3932,            110,            4756,            2438,            2587,            2973,            2952,            3762,            5276,            3475,            3183,            2609,            5829,            3665,            2725,            2917,            5468,            3497,            3817,            2836,            5359,            2861,            581,            3548,            2790,            5153,            5110,            1434,            1329,            1199,            1248,            3991,            2524,            2625,            3499,            2494,            4453,            1030,            845,            3298,            694,            1849,            3541,            4457,            4462,            ],        [            4299,            4292,            1625,            2292,            1165,            2978,            3470,            3970,            3125,            1306,            214,            2469,            1696,            2045,            2564,            2641,            3049,            4769,            2520,            0,            4755,            1727,            2354,            2802,            3739,            2823,            1865,            1650,            2993,            694,            248,            2565,            1869,            2015,            1520,            3416,            2944,            3909,            4040,            2087,            2410,            2911,            3365,            2748,            2848,            3750,            1917,            3431,            4402,            4110,            3536,            3984,            1820,            3214,            3664,            3623,            3930,            1972,            3055,            3514,            3788,            2956,            4475,            3537,            3308,            3265,            2614,            1643,            2379,            1983,            2146,            3451,            3372,            1654,            3421,            2608,            2876,            1580,            1453,            3204,            1823,            5177,            2612,            2617,            ],        [            1955,            1921,            4594,            5537,            4370,            3453,            2609,            3233,            2388,            4511,            4675,            3176,            3564,            4234,            4332,            3348,            3414,            382,            2995,            4868,            0,            4231,            2829,            3277,            3264,            3699,            4267,            3418,            3424,            4295,            5561,            4433,            5049,            3883,            4762,            4349,            3065,            3810,            3585,            3709,            2885,            4864,            2593,            3315,            3628,            3107,            4557,            5628,            3630,            3330,            2764,            4620,            5065,            3717,            3909,            4556,            3831,            4127,            3828,            5035,            3016,            3566,            2131,            3782,            4829,            5462,            3818,            3832,            4247,            3751,            4328,            2430,            3617,            4899,            3570,            5047,            3257,            3348,            4422,            3679,            4233,            84,            4664,            5288,            ],        [            3177,            3170,            2405,            3072,            1866,            2136,            2628,            1364,            2307,            2007,            1269,            1627,            1224,            725,            2409,            1799,            1251,            3951,            1678,            1462,            3937,            0,            1512,            1960,            1718,            2668,            326,            1495,            1195,            1604,            1710,            825,            2649,            843,            2300,            4196,            2142,            3281,            3056,            2867,            1568,            3691,            2243,            2392,            2778,            2757,            2697,            4211,            3280,            2988,            2414,            4764,            2600,            2530,            2722,            4403,            3302,            2752,            2641,            4294,            2666,            2249,            3353,            2595,            4088,            4045,            816,            1127,            903,            1828,            2926,            2329,            2430,            2434,            2299,            3388,            1695,            1425,            2233,            2362,            284,            4359,            3392,            3397,            ],        [            3206,            4135,            3304,            3971,            1701,            734,            1226,            1393,            1323,            1842,            2006,            457,            895,            1565,            1663,            629,            1402,            2967,            276,            2199,            2953,            1512,            0,            558,            1747,            980,            1717,            749,            1412,            1626,            2892,            1219,            3548,            1214,            3199,            5095,            2171,            3310,            3085,            3766,            166,            4590,            2272,            2421,            2807,            2786,            3596,            5110,            3309,            3017,            2443,            5663,            3499,            2559,            2751,            5302,            3331,            3651,            2670,            5193,            2695,            847,            3382,            2624,            4987,            4944,            1268,            1163,            1033,            1082,            3825,            2358,            2459,            3333,            2328,            4287,            864,            679,            3132,            960,            1683,            3375,            4291,            4296,            ],        [            3620,            3998,            3752,            4419,            2149,            176,            668,            1841,            1172,            2290,            2454,            905,            1343,            2013,            681,            1077,            1850,            2830,            292,            2647,            2816,            1960,            558,            0,            2195,            422,            2165,            1197,            1860,            2074,            3340,            1667,            3996,            1662,            3647,            5543,            2585,            3724,            3499,            4214,            392,            5038,            2686,            2835,            3221,            3200,            4044,            5558,            3723,            3431,            2857,            6111,            3947,            2973,            3165,            5750,            3745,            4099,            3084,            5641,            3109,            289,            3796,            3038,            5435,            5392,            1716,            1611,            1481,            1530,            4273,            2772,            2873,            3781,            2742,            4735,            1312,            1127,            3580,            402,            2131,            3238,            4739,            4744,            ],        [            2485,            2478,            2979,            3922,            3185,            2268,            2760,            672,            864,            3326,            3490,            973,            2379,            1568,            3147,            773,            681,            2787,            1810,            3683,            2773,            1726,            1644,            2092,            0,            2514,            1534,            2233,            691,            3110,            4376,            1094,            3434,            1316,            3147,            3128,            1450,            2589,            2364,            2094,            1700,            3159,            1551,            1700,            2086,            2065,            2911,            3923,            2588,            2296,            1722,            3399,            3450,            1838,            2030,            3439,            2610,            2512,            1949,            3330,            1974,            2381,            2661,            1903,            3124,            3757,            1085,            2647,            1315,            2566,            2682,            1637,            1738,            3284,            1607,            3342,            578,            2163,            2807,            2494,            1500,            3195,            3115,            3673,            ],        [            4042,            4420,            3903,            4570,            2300,            498,            990,            2075,            1594,            2441,            2605,            1139,            1491,            2161,            259,            1311,            2844,            3252,            526,            2798,            3238,            2653,            792,            422,            2429,            0,            2858,            1173,            2094,            2225,            3491,            2360,            4147,            1810,            3798,            5694,            3007,            4146,            3921,            4365,            626,            5189,            3108,            3257,            3643,            3622,            4195,            5709,            4145,            3853,            3279,            6262,            4098,            3395,            3587,            5901,            4167,            4250,            3506,            5792,            3531,            133,            4218,            3460,            5586,            5543,            2409,            1759,            2174,            840,            4424,            3194,            3295,            3932,            3164,            4886,            1546,            1275,            3731,            724,            2824,            3660,            4890,            4895,            ],        [            3098,            3091,            2419,            3086,            2089,            2328,            2820,            1285,            2499,            2230,            1283,            1819,            1416,            917,            2601,            1991,            1172,            4143,            1870,            1476,            4129,            192,            1704,            2152,            1639,            2860,            0,            1687,            1116,            1618,            1724,            746,            2663,            1035,            2314,            4210,            2063,            3202,            2977,            2881,            1760,            3705,            2164,            2313,            2699,            2678,            2711,            4225,            3201,            2909,            2335,            4778,            2614,            2451,            2643,            4417,            3223,            2766,            2562,            4308,            2587,            2441,            3274,            2516,            4102,            4059,            737,            1319,            967,            2020,            2940,            2250,            2351,            2448,            2220,            3402,            1681,            1617,            2247,            2554,            205,            4551,            3406,            3411,            ],        [            4609,            4575,            2730,            3397,            1127,            1616,            2108,            1784,            1763,            1268,            1432,            1107,            318,            988,            914,            1279,            1671,            3407,            1158,            1625,            3393,            1480,            992,            1440,            2138,            1173,            1685,            0,            1615,            1052,            2318,            1187,            2974,            637,            2625,            4521,            3176,            4315,            4090,            3192,            1048,            4016,            3277,            3426,            3812,            3791,            3022,            4536,            4314,            4022,            3448,            5089,            2925,            3564,            3756,            4728,            4336,            3077,            3675,            4619,            3700,            1306,            4785,            3629,            4413,            4370,            1236,            586,            1001,            333,            3251,            3363,            3464,            2759,            3333,            3713,            1514,            102,            2558,            1842,            1651,            3815,            3717,            3722,            ],        [            2076,            2069,            2570,            3513,            2561,            2247,            2739,            263,            1481,            2702,            2126,            1054,            1335,            1206,            2520,            854,            150,            2885,            1789,            2319,            3217,            1035,            1623,            2071,            617,            2779,            843,            1606,            0,            2486,            2567,            403,            3025,            954,            2738,            2719,            1041,            2180,            1955,            1685,            1679,            2750,            1142,            1291,            1677,            1656,            2502,            3514,            2179,            1887,            1313,            2990,            3041,            1429,            1621,            3030,            2201,            2103,            1540,            2921,            1565,            2360,            2252,            1494,            2715,            3348,            394,            1608,            624,            1939,            2273,            1228,            1329,            2875,            1198,            2933,            659,            1536,            2398,            2473,            809,            3293,            2706,            3264,            ],        [            4352,            4345,            1678,            2345,            961,            2774,            3266,            3766,            2921,            1102,            380,            2265,            1492,            1841,            2360,            2437,            2845,            4565,            2316,            573,            4551,            1780,            2150,            2598,            3792,            2619,            1918,            1446,            2789,            0,            821,            2361,            1922,            1811,            1573,            3469,            2997,            3962,            4093,            2140,            2206,            2964,            3418,            2801,            2901,            3803,            1970,            3484,            4455,            4163,            3589,            4037,            1873,            3267,            3717,            3676,            3983,            2025,            3108,            3567,            3841,            2752,            4528,            3590,            3361,            3318,            2410,            1439,            2175,            1779,            2199,            3504,            3425,            1707,            3474,            2661,            2672,            1376,            1506,            3000,            1876,            4973,            2665,            2670,            ],        [            4547,            4540,            1873,            2540,            1413,            3226,            3718,            4218,            3373,            1554,            462,            2717,            1944,            2293,            2812,            2889,            3297,            5017,            2768,            248,            5003,            1975,            2602,            3050,            3987,            3071,            2113,            1898,            3241,            942,            0,            2813,            2117,            2263,            1768,            3664,            3192,            4157,            4288,            2335,            2658,            3159,            3613,            2996,            3096,            3998,            2165,            3679,            4650,            4358,            3784,            4232,            2068,            3462,            3912,            3871,            4178,            2220,            3303,            3762,            4036,            3204,            4723,            3785,            3556,            3513,            2862,            1891,            2627,            2231,            2394,            3699,            3620,            1902,            3669,            2856,            3124,            1828,            1701,            3452,            2071,            5425,            2860,            2865,            ],        [            2410,            2403,            2917,            3584,            2158,            1844,            2336,            597,            2015,            2299,            1781,            1335,            932,            803,            2117,            1188,            484,            3659,            1386,            1974,            3645,            690,            1220,            1668,            951,            2376,            498,            1203,            428,            2083,            2222,            0,            3161,            551,            2812,            3053,            1375,            2514,            2289,            2019,            1276,            3084,            1476,            1625,            2011,            1990,            2836,            3848,            2513,            2221,            1647,            3324,            3112,            1763,            1955,            3364,            2535,            2437,            1874,            3255,            1899,            1957,            2586,            1828,            3049,            3682,            49,            1205,            221,            1536,            2607,            1562,            1663,            2946,            1532,            3267,            993,            1133,            2745,            2070,            464,            4067,            3040,            3598,            ],        [            4042,            4035,            1368,            2035,            1293,            3106,            3598,            2061,            3253,            1434,            487,            2597,            1824,            2173,            2692,            2769,            1948,            4897,            2648,            680,            4883,            885,            2482,            2930,            2415,            2951,            1023,            1778,            1892,            822,            928,            1522,            0,            2143,            1263,            3159,            2687,            3652,            3783,            1830,            2538,            2654,            3108,            2491,            2591,            3493,            1660,            3174,            4145,            3853,            3279,            3727,            1563,            2957,            3407,            3366,            3673,            1715,            2798,            3257,            3531,            3084,            4218,            3280,            3051,            3008,            1513,            1771,            1743,            2111,            1889,            3194,            3115,            1397,            3164,            2351,            3004,            1708,            1196,            3332,            981,            5305,            2355,            2360,            ],        [            2960,            2953,            3210,            3877,            1607,            1597,            2089,            1147,            1977,            1748,            1912,            1088,            381,            556,            1566,            1118,            1034,            3621,            1139,            2105,            3607,            843,            973,            1421,            1501,            1825,            1048,            652,            978,            1532,            2798,            550,            3454,            0,            3105,            3603,            1925,            3064,            2839,            2569,            1029,            4496,            2026,            2175,            2561,            2540,            3502,            5016,            3063,            2771,            2197,            3874,            3405,            2313,            2505,            3914,            3085,            2987,            2424,            3805,            2449,            1958,            3136,            2378,            3599,            4850,            599,            944,            364,            985,            3731,            2112,            2213,            3239,            2082,            4193,            1156,            582,            3038,            1823,            1014,            4029,            4197,            4202,            ],        [            3202,            3195,            528,            1189,            2065,            3878,            4370,            2520,            4025,            2206,            1259,            3369,            2596,            2945,            3464,            3541,            2701,            4011,            3420,            1452,            4343,            1657,            3254,            3702,            2642,            3723,            1795,            2550,            2711,            1594,            1700,            2294,            772,            2915,            0,            2313,            1847,            2806,            2775,            990,            3310,            1808,            2268,            1651,            1751,            2782,            820,            2328,            3305,            3013,            2439,            2881,            717,            1991,            2441,            2520,            2827,            875,            1832,            2411,            2691,            3856,            3378,            2314,            2205,            2162,            2285,            2543,            2515,            2883,            1049,            2354,            2149,            551,            2324,            1505,            3088,            2480,            356,            4104,            1753,            4419,            1515,            1520,            ],        [            3015,            3562,            2036,            2103,            4712,            4990,            5482,            2600,            3586,            4853,            3906,            3563,            5243,            5592,            6111,            3363,            2781,            3824,            4532,            4099,            4156,            4304,            4366,            4814,            2722,            6370,            4442,            5197,            2791,            4241,            4347,            3194,            3419,            5562,            3136,            0,            2523,            750,            881,            2063,            4422,            796,            2081,            1920,            1689,            1576,            1641,            1277,            1941,            2332,            1971,            271,            2496,            1725,            1201,            464,            771,            1948,            1424,            824,            1753,            6503,            3191,            1328,            941,            1111,            3185,            5190,            3415,            5530,            1412,            2167,            1780,            3273,            1781,            979,            3168,            5127,            2986,            5216,            4400,            4232,            950,            1222,            ],        [            1490,            1483,            1984,            2927,            3732,            3198,            3690,            808,            1794,            3873,            2926,            1771,            2334,            2205,            4077,            1571,            989,            2299,            2740,            3119,            2631,            2034,            2574,            3022,            930,            3444,            1842,            3163,            999,            3261,            3367,            1402,            2439,            1953,            2152,            2133,            0,            1594,            1369,            1099,            2630,            2164,            556,            705,            1091,            1070,            1916,            2928,            1593,            1301,            727,            2404,            2455,            843,            1035,            2444,            1615,            1517,            954,            2335,            979,            3311,            1666,            908,            2129,            2762,            1393,            2607,            1623,            3496,            1687,            642,            743,            2289,            612,            2347,            1376,            3093,            1812,            3424,            1808,            2707,            2120,            2678,            ],        [            2359,            2352,            2418,            1917,            4508,            4334,            4826,            1944,            2930,            4649,            3702,            2907,            3470,            3341,            5213,            2707,            2125,            3168,            3876,            3895,            3500,            3170,            3710,            4158,            2066,            4580,            2978,            4299,            2135,            4037,            4143,            2538,            3215,            3089,            2932,            663,            1867,            0,            225,            2445,            3766,            1178,            1425,            2117,            1497,            920,            2023,            1942,            1285,            1676,            1315,            934,            2292,            1069,            545,            870,            115,            2330,            1180,            1349,            1097,            4447,            2535,            672,            1143,            1776,            2529,            4986,            2759,            4632,            1794,            1511,            1124,            3069,            1307,            1361,            2512,            4229,            2782,            4560,            2944,            3576,            1332,            1604,            ],        [            2134,            2127,            2519,            2018,            4609,            4109,            4601,            1719,            2705,            4750,            3803,            2682,            3245,            3116,            4988,            2482,            1900,            2943,            3651,            3996,            3275,            2945,            3485,            3933,            1841,            4355,            2753,            4074,            1910,            4138,            4244,            2313,            3316,            2864,            3033,            764,            1642,            225,            0,            2546,            3541,            1279,            1200,            1892,            1272,            695,            2124,            2043,            986,            1180,            1090,            1035,            2393,            844,            320,            971,            246,            2431,            955,            1450,            872,            4222,            2310,            447,            1244,            1877,            2304,            3518,            2534,            4407,            1895,            1286,            899,            3170,            1082,            1462,            2287,            4004,            2883,            4335,            2719,            3351,            1433,            1705,            ],        [            2212,            2205,            1404,            2347,            3152,            3920,            4412,            1530,            2516,            3293,            2346,            2493,            3683,            2927,            4551,            2293,            1711,            3021,            3462,            2539,            3353,            2756,            3296,            3744,            1652,            4166,            2564,            3637,            1721,            2681,            2787,            2124,            1859,            2675,            1572,            1780,            857,            2316,            2091,            0,            3352,            1757,            1278,            661,            761,            1792,            1318,            2471,            2315,            2023,            1449,            2051,            1875,            1102,            1757,            1987,            2337,            937,            943,            1878,            1701,            4033,            2388,            1630,            1672,            2305,            2115,            3630,            2345,            3970,            1089,            1364,            1465,            1709,            1334,            1890,            2098,            3815,            1232,            4146,            2530,            3429,            1425,            2098,            ],        [            3262,            4191,            3360,            4027,            1757,            568,            1060,            1449,            1379,            1898,            2062,            513,            951,            1621,            1073,            685,            1458,            3023,            110,            2255,            3007,            1568,            166,            392,            1803,            814,            1773,            805,            1468,            1682,            2948,            1275,            3604,            1270,            3255,            5151,            2227,            3366,            3141,            3822,            0,            4646,            2328,            2477,            2863,            2842,            3652,            5166,            3365,            3073,            2499,            5719,            3555,            2615,            2807,            5358,            3387,            3707,            2726,            5249,            2751,            681,            3438,            2680,            5043,            5000,            1324,            1219,            1089,            1138,            3881,            2414,            2515,            3389,            2384,            4343,            920,            735,            3188,            794,            1739,            3429,            4347,            4352,            ],        [            3419,            3966,            1418,            917,            3508,            5321,            5813,            2975,            3961,            3649,            2702,            4812,            4039,            4388,            4907,            3738,            3156,            4228,            4863,            2895,            4560,            3100,            4697,            5145,            3097,            5166,            3238,            3993,            3166,            3037,            3143,            3569,            2215,            4358,            1932,            661,            2302,            1154,            1285,            1445,            4753,            0,            2485,            1734,            1503,            1980,            1023,            299,            2345,            2736,            2375,            932,            1292,            1606,            1157,            868,            1175,            1330,            1447,            759,            2157,            5299,            3595,            1284,            553,            449,            3560,            3986,            3790,            4326,            794,            2571,            1764,            2069,            1804,            361,            3543,            3923,            1782,            5547,            3196,            4636,            332,            604,            ],        [            1443,            1436,            2671,            3227,            4282,            3472,            3964,            1082,            2068,            4423,            3476,            2045,            2608,            2479,            4351,            1845,            1263,            2252,            3014,            3669,            2584,            2308,            2848,            3296,            1204,            3718,            2116,            3437,            1273,            3811,            3917,            1676,            2989,            2227,            2702,            1973,            1005,            1434,            1209,            1649,            2904,            2488,            0,            1255,            1552,            514,            2247,            3252,            1037,            745,            171,            2244,            2686,            1124,            1316,            2180,            1455,            2067,            1235,            2659,            423,            3585,            1619,            1189,            2453,            3086,            1667,            2881,            1897,            3770,            2018,            595,            1024,            2839,            1362,            2671,            1650,            3367,            2362,            3698,            2082,            2660,            2354,            2914,            ],        [            1930,            1923,            1279,            2222,            3027,            3638,            4130,            1248,            2234,            3168,            2221,            2211,            3558,            3907,            4426,            2011,            1429,            2739,            3180,            2414,            3071,            2619,            3014,            3462,            1370,            4685,            2282,            3512,            1439,            2556,            2662,            1842,            1734,            2393,            1447,            1709,            575,            2034,            1809,            394,            3070,            1918,            996,            0,            479,            1510,            1479,            2400,            2033,            1741,            1167,            1980,            1750,            907,            1475,            1916,            2055,            812,            796,            1807,            1419,            3751,            2106,            1348,            1601,            2234,            1833,            3505,            2063,            3845,            1250,            1082,            1183,            1584,            1052,            1819,            1816,            3442,            1107,            3864,            2715,            3147,            1586,            1973,            ],        [            2063,            2056,            1594,            2222,            3342,            3771,            4263,            1381,            2367,            3483,            2536,            2344,            2907,            2778,            4650,            2144,            1562,            2872,            3313,            2729,            3204,            2607,            3147,            3595,            1503,            4017,            2415,            3736,            1572,            2871,            2977,            1975,            2049,            2526,            1762,            1578,            708,            1485,            1260,            709,            3203,            1505,            1129,            315,            0,            1012,            1261,            2269,            1789,            1497,            1300,            1849,            1700,            428,            926,            1785,            1506,            1127,            317,            1676,            1189,            3884,            2239,            799,            1470,            2103,            1966,            3820,            2196,            4069,            1032,            1215,            634,            1899,            553,            1688,            1949,            3666,            1422,            3997,            2381,            3280,            1368,            1909,            ],        [            1653,            1646,            2157,            2713,            4492,            3682,            4174,            1292,            2278,            4633,            3686,            2255,            2818,            2689,            4561,            2055,            1473,            2462,            3224,            3879,            2794,            2518,            3058,            3506,            1414,            3928,            2326,            3647,            1483,            4021,            4127,            1886,            3199,            2437,            2912,            1459,            1215,            920,            695,            1859,            3114,            1974,            811,            1465,            1038,            0,            1733,            2738,            777,            485,            395,            1730,            2172,            610,            802,            1666,            941,            2277,            721,            2145,            177,            3795,            1829,            675,            1939,            2572,            1877,            3091,            2107,            3980,            1504,            805,            510,            3049,            848,            2157,            1860,            3577,            2572,            3908,            2292,            2870,            1840,            2400,            ],        [            2634,            2627,            774,            1549,            2522,            4342,            4834,            1952,            2938,            2663,            1716,            2915,            3053,            3402,            3921,            2715,            2133,            3443,            3884,            1909,            3775,            2114,            3711,            4166,            2074,            4180,            2252,            3007,            2143,            2051,            2157,            2546,            1229,            3372,            942,            1532,            1279,            2025,            1955,            422,            3774,            1027,            1700,            1083,            1183,            1707,            0,            1326,            2484,            2192,            1871,            1803,            1027,            1171,            1621,            1739,            2046,            307,            1012,            1630,            1884,            4313,            2810,            1494,            1424,            2057,            2537,            3000,            2767,            3340,            229,            1786,            1329,            1079,            1369,            1210,            2520,            2937,            602,            4568,            2210,            3851,            695,            1236,            ],        [            3639,            4186,            1638,            1137,            3728,            5541,            6033,            3195,            4181,            3869,            2922,            5032,            4259,            4608,            5127,            3958,            3376,            4448,            5083,            3115,            4780,            3320,            4917,            5365,            3317,            5386,            3458,            4213,            3386,            3257,            3363,            3789,            2435,            4578,            2152,            881,            2522,            1374,            1505,            1665,            4973,            220,            2705,            1954,            1723,            2200,            1243,            0,            2565,            2956,            2595,            1152,            1512,            1826,            1377,            1088,            1395,            1550,            1667,            979,            2377,            5519,            3815,            1504,            773,            371,            3780,            4206,            4010,            4546,            1014,            2791,            1984,            2289,            2024,            581,            3763,            4143,            2002,            5767,            3416,            4856,            552,            824,            ],        [            1628,            1621,            2912,            3306,            4965,            4155,            4647,            1765,            2751,            5106,            4159,            2728,            3291,            3162,            5034,            2528,            1946,            2437,            3697,            4352,            2769,            2991,            3531,            3979,            1887,            4401,            2799,            4120,            1956,            4494,            4600,            2359,            3672,            2910,            3385,            2052,            1688,            1513,            1094,            2332,            3587,            2567,            1216,            1938,            1793,            755,            2488,            3331,            0,            391,            868,            2323,            2927,            1365,            1557,            2259,            1534,            2750,            1476,            2738,            650,            4268,            1804,            1430,            2532,            3165,            2350,            3564,            2580,            4453,            2259,            1278,            1265,            3522,            1603,            2750,            2333,            4050,            3045,            4381,            2765,            2845,            2721,            2993,            ],        [            1735,            1578,            2521,            3077,            4574,            3764,            4256,            1374,            2360,            4715,            3768,            2337,            2900,            2771,            4643,            2137,            1555,            2544,            3306,            3961,            2876,            2600,            3140,            3588,            1496,            4010,            2408,            3729,            1565,            4103,            4209,            1968,            3281,            2519,            2994,            1823,            1297,            1284,            1059,            1941,            3196,            2338,            825,            1547,            1402,            364,            2097,            3102,            643,            0,            477,            2094,            2536,            974,            1166,            2030,            1305,            2359,            1085,            2509,            259,            3877,            1911,            1039,            2303,            2936,            1959,            3173,            2189,            4062,            1868,            887,            874,            3131,            1212,            2521,            1942,            3659,            2654,            3990,            2374,            2952,            2204,            2764,            ],        [            1728,            1817,            2500,            3056,            4567,            3757,            4249,            1367,            2353,            4708,            3761,            2330,            2893,            2764,            4636,            2130,            1548,            2537,            3299,            3954,            2869,            2593,            3133,            3581,            1489,            4003,            2401,            3722,            1558,            4096,            4202,            1961,            3274,            2512,            2987,            1802,            1290,            1263,            1038,            1934,            3189,            2317,            1154,            1540,            1381,            343,            2076,            3081,            866,            574,            0,            2073,            2515,            953,            1145,            2009,            1284,            2352,            1064,            2488,            252,            3870,            1904,            1018,            2282,            2915,            1952,            3166,            2182,            4055,            1847,            880,            853,            3124,            1191,            2500,            1935,            3652,            2647,            3983,            2367,            2945,            2183,            2743,            ],        [            3046,            3593,            2067,            1566,            4157,            5021,            5513,            2631,            3617,            4298,            3351,            3594,            4688,            5037,            5556,            3394,            2812,            3855,            4563,            3544,            4187,            3749,            4397,            4845,            2753,            5815,            3887,            4642,            2822,            3686,            3792,            3225,            2864,            5007,            2581,            495,            2554,            781,            912,            2094,            4453,            827,            2112,            1951,            1720,            1607,            1672,            1161,            1972,            2363,            2002,            0,            1941,            1756,            1232,            495,            802,            1979,            1455,            708,            1784,            5134,            3222,            1359,            792,            995,            3216,            4635,            3446,            4975,            1443,            2198,            1811,            2718,            1812,            1010,            3199,            4572,            2431,            5247,            3845,            4263,            981,            1253,            ],        [            3736,            3729,            1062,            472,            2616,            4429,            4921,            3054,            4576,            2757,            1810,            3920,            3147,            3496,            4015,            4092,            3235,            4545,            3971,            2003,            4877,            2208,            3805,            4253,            3176,            4274,            2346,            3101,            3245,            2145,            2251,            2845,            1323,            3466,            1040,            1596,            2381,            2089,            2220,            1524,            3861,            1091,            2802,            2185,            1868,            2298,            1354,            1611,            3280,            3671,            2973,            2164,            0,            1762,            2092,            1803,            2110,            1409,            1603,            1694,            2475,            4407,            3912,            2219,            1488,            1445,            2836,            3094,            3066,            3434,            950,            2888,            1920,            1177,            1960,            788,            3622,            3031,            890,            4655,            2304,            4953,            1267,            1031,            ],        [            2049,            2042,            1547,            2084,            3778,            4024,            4516,            1634,            2620,            3919,            2972,            2597,            3160,            3031,            4903,            2397,            1815,            2858,            3566,            3165,            3190,            2860,            3400,            3848,            1756,            4270,            2668,            3989,            1825,            3307,            3413,            2228,            2485,            2779,            2065,            1622,            1136,            1083,            858,            1137,            3456,            1351,            1115,            743,            428,            610,            1123,            2115,            1387,            1095,            1005,            1893,            1562,            0,            524,            1631,            1104,            1430,            111,            1522,            787,            4137,            2225,            397,            1316,            1949,            2219,            4256,            2449,            4322,            894,            1201,            232,            2202,            374,            1534,            2202,            3919,            1725,            4250,            2634,            3266,            1230,            1771,            ],        [            2241,            2234,            2071,            1868,            4302,            4216,            4708,            1826,            2812,            4443,            3496,            2789,            3352,            3223,            5095,            2589,            2007,            3050,            3758,            3689,            3382,            3052,            3592,            4040,            1948,            4462,            2860,            4181,            2017,            3831,            3937,            2420,            3009,            2971,            2589,            1098,            1749,            559,            334,            1661,            3648,            1129,            1307,            1267,            952,            802,            1647,            1893,            1176,            1287,            1197,            1369,            2086,            524,            0,            1409,            580,            1954,            635,            1300,            979,            4329,            2417,            127,            1094,            1727,            2411,            3625,            2641,            4514,            1418,            1393,            722,            2726,            762,            1312,            2394,            4111,            2249,            4442,            2826,            3458,            1085,            1555,            ],        [            2551,            3098,            2610,            2109,            4700,            4526,            5018,            2136,            3122,            4841,            3894,            3099,            3662,            3533,            5405,            2899,            2317,            3360,            4068,            4087,            3692,            3362,            3902,            4350,            2258,            4772,            3170,            4491,            2327,            4229,            4335,            2730,            3407,            3281,            3124,            855,            2059,            286,            417,            2637,            3958,            1370,            1617,            2309,            1689,            1112,            2215,            2134,            1477,            1868,            1507,            1126,            2484,            1261,            737,            0,            307,            2522,            1372,            1541,            1289,            4639,            2727,            864,            1335,            1968,            2721,            5178,            2951,            4824,            1986,            1703,            1316,            3261,            1499,            1553,            2704,            4421,            2974,            4752,            3136,            3768,            1524,            1796,            ],        [            2798,            2791,            3265,            2764,            5355,            5325,            5817,            2935,            3921,            5496,            4549,            3898,            4461,            4332,            6204,            3698,            3116,            3607,            4867,            4742,            3939,            4161,            4701,            5149,            3057,            5571,            3969,            5290,            3126,            4884,            4990,            3529,            4062,            4080,            3779,            1510,            2858,            971,            1072,            3292,            4757,            2025,            2272,            3108,            2344,            1925,            2870,            2789,            1170,            1561,            2038,            1781,            3139,            1916,            1392,            1717,            0,            3177,            2027,            2196,            1820,            5438,            2974,            1519,            1990,            2623,            3520,            5833,            3750,            5623,            2641,            2448,            1971,            3916,            2154,            2208,            3503,            5220,            3629,            5551,            3935,            4015,            2179,            2451,            ],        [            3141,            3134,            467,            1410,            2215,            4028,            4520,            2459,            4175,            2356,            1409,            3519,            2746,            3095,            3614,            3691,            2640,            3950,            3570,            1602,            4282,            1807,            3404,            3852,            2581,            3873,            1945,            2700,            2650,            1744,            1850,            2444,            922,            3065,            635,            1913,            1786,            2406,            2380,            929,            3460,            1408,            2207,            1590,            1690,            2132,            759,            2549,            2909,            2617,            2378,            3102,            938,            1596,            2046,            2120,            2427,            0,            1437,            2011,            2309,            4006,            3317,            1919,            1805,            2383,            2435,            2693,            2665,            3033,            784,            2293,            1754,            772,            1794,            1591,            3027,            2630,            295,            4254,            1903,            4358,            1120,            1161,            ],        [            2160,            2153,            1436,            1973,            3667,            4088,            4580,            1698,            2684,            3808,            2861,            2661,            3224,            3095,            4967,            2461,            1879,            2969,            3630,            3054,            3301,            2924,            3464,            3912,            1820,            4334,            2732,            4053,            1889,            3196,            3302,            2292,            2374,            2843,            1954,            1313,            1025,            1236,            1011,            1026,            3520,            1240,            1226,            632,            317,            721,            1012,            2004,            1498,            1206,            1116,            1584,            1451,            111,            677,            1520,            1257,            1319,            0,            1411,            898,            4201,            2336,            550,            1205,            1838,            2283,            4145,            2513,            4386,            783,            1312,            385,            2091,            434,            1423,            2266,            3983,            1614,            4314,            2698,            3377,            1119,            1660,            ],        [            3665,            4212,            2474,            1419,            4028,            5841,            6333,            3250,            4236,            4169,            3222,            5332,            4559,            4908,            5427,            4013,            3431,            4474,            5383,            3415,            4806,            3620,            5217,            5665,            3372,            5686,            3758,            4513,            3441,            3557,            3663,            4257,            2735,            4878,            2452,            921,            2194,            1400,            1531,            2936,            5273,            813,            2731,            1994,            1763,            2226,            1715,            593,            2591,            2982,            2621,            619,            1812,            1657,            1417,            1114,            1421,            2022,            1498,            0,            2403,            5819,            3841,            1544,            301,            427,            4248,            4506,            4478,            4846,            1486,            2817,            1815,            2589,            1855,            1053,            3818,            4443,            2302,            6067,            3716,            4882,            1024,            1296,            ],        [            1476,            1469,            2567,            3274,            4315,            3505,            3997,            1115,            2101,            4456,            3509,            2078,            2641,            2512,            4384,            1878,            1296,            2285,            3047,            3702,            2617,            2341,            2881,            3329,            1237,            3751,            2149,            3470,            1306,            3844,            3950,            1709,            3022,            2260,            2735,            2020,            1038,            1481,            1256,            1682,            2937,            2535,            566,            1288,            1601,            561,            2294,            3299,            1084,            792,            218,            2291,            3038,            1171,            1363,            2227,            1502,            2100,            1282,            2706,            0,            3618,            1652,            1236,            2500,            3133,            1700,            2914,            1930,            3803,            2065,            628,            1071,            2872,            1543,            2718,            1683,            3400,            2395,            3731,            2115,            2693,            2401,            2961,            ],        [            3909,            4287,            4041,            4708,            2438,            365,            857,            2130,            1461,            2579,            2743,            1194,            1632,            2302,            392,            1366,            2139,            3119,            581,            2936,            3105,            2249,            847,            289,            2484,            133,            2454,            1306,            2149,            2363,            3629,            1956,            4285,            1951,            3936,            4552,            2874,            4013,            3788,            4503,            681,            5327,            2975,            3124,            3510,            3489,            4333,            5847,            4012,            3720,            3146,            6400,            4236,            3262,            3454,            6039,            4034,            4388,            3373,            5930,            3398,            0,            4085,            3327,            5724,            5681,            2005,            1900,            1770,            973,            4562,            3061,            3162,            4070,            3031,            5024,            1601,            1416,            3869,            591,            2420,            3527,            5028,            5033,            ],        [            811,            777,            3450,            4393,            5198,            4388,            4072,            1998,            2984,            5339,            4392,            2961,            3524,            3395,            5267,            2761,            2179,            1593,            3930,            4585,            1925,            3224,            3764,            4212,            2120,            4634,            3032,            4353,            2189,            4727,            4833,            2592,            3905,            3143,            3618,            3205,            1921,            2666,            2441,            2565,            3820,            3720,            1449,            2171,            2484,            1963,            3413,            4484,            2486,            2194,            1620,            3476,            3921,            2573,            2765,            3412,            2687,            2983,            2684,            3891,            1872,            4501,            0,            2638,            3685,            4318,            2583,            3797,            2813,            4686,            3184,            1286,            2473,            3755,            2426,            3903,            2566,            4283,            3278,            4614,            2998,            2001,            3520,            4144,            ],        [            2114,            2107,            1944,            1995,            4175,            4089,            4581,            1699,            2685,            4316,            3369,            2662,            3225,            3096,            4968,            2462,            1880,            2923,            3631,            3562,            3255,            2925,            3465,            3913,            1821,            4335,            2733,            4054,            1890,            3704,            3810,            2293,            2882,            2844,            2462,            1225,            1622,            686,            461,            1534,            3521,            1256,            1180,            1140,            825,            675,            1520,            2020,            1452,            1160,            1070,            1496,            1959,            397,            127,            1536,            707,            1827,            508,            1427,            852,            4202,            2290,            0,            1221,            1854,            2284,            3498,            2514,            4387,            1291,            1266,            595,            2599,            635,            1439,            2267,            3984,            2122,            4315,            2699,            3331,            1212,            1682,            ],        [            3466,            4013,            1897,            1625,            4234,            6047,            6539,            2654,            3640,            4375,            3428,            5538,            4765,            5114,            5633,            3417,            2835,            4275,            5589,            3621,            4607,            3826,            5423,            5871,            2776,            5892,            3964,            4719,            2845,            3763,            3869,            3248,            2941,            5084,            2658,            708,            1981,            1201,            1332,            1924,            5479,            657,            2532,            1781,            1550,            2027,            1502,            799,            2392,            2783,            2422,            825,            2018,            1444,            1204,            915,            1222,            1809,            1285,            206,            2204,            6025,            3642,            1331,            0,            633,            3239,            4712,            3469,            5052,            1273,            2618,            1602,            2795,            1642,            840,            3222,            4649,            2508,            6273,            3922,            4683,            811,            1083,            ],        [            3805,            4352,            1804,            1315,            3924,            5737,            6229,            3361,            5884,            4065,            3118,            5228,            4455,            4804,            5323,            4124,            3542,            4614,            5279,            3311,            4946,            3516,            5113,            5561,            3483,            5582,            3654,            4409,            3552,            3453,            3559,            4153,            2631,            4774,            2348,            1047,            2688,            1540,            1671,            1831,            5169,            386,            2871,            2120,            1889,            2366,            1409,            166,            2731,            3122,            2761,            1090,            1708,            1992,            1543,            1254,            1561,            1716,            1833,            655,            2543,            5715,            3981,            1670,            772,            0,            4144,            4402,            4374,            4742,            1180,            2957,            2150,            2485,            2190,            747,            3929,            4339,            2198,            5963,            3612,            5022,            718,            990,            ],        [            2746,            2739,            2868,            3535,            2113,            2238,            2730,            933,            2409,            2254,            1732,            1729,            1326,            972,            2511,            1524,            820,            4053,            1780,            1925,            4039,            641,            1614,            2062,            1287,            2770,            449,            1597,            764,            2067,            2173,            394,            3112,            945,            2763,            3389,            1711,            2850,            2625,            2355,            1670,            3420,            1812,            1961,            2347,            2326,            3160,            4674,            2849,            2557,            1983,            3660,            3063,            2099,            2291,            3700,            2871,            2773,            2210,            3591,            2235,            2351,            2922,            2164,            3385,            4508,            0,            1374,            615,            1930,            2943,            1898,            1999,            2897,            1868,            3851,            1329,            1527,            2696,            2464,            415,            4461,            3376,            3860,            ],        [            3614,            4900,            2474,            3141,            739,            1941,            2433,            1801,            2088,            880,            1176,            1432,            643,            402,            1500,            1604,            1688,            3732,            1483,            1369,            3718,            1127,            1317,            1765,            2155,            1759,            1245,            586,            1632,            796,            1617,            1204,            2718,            962,            2369,            4265,            2579,            3718,            3493,            2936,            1373,            3760,            2680,            2829,            3215,            3194,            2766,            4280,            3717,            3425,            2851,            4833,            2669,            2967,            3159,            4472,            3739,            2821,            3078,            4363,            3103,            1892,            3790,            3032,            4157,            4114,            1253,            0,            1018,            919,            2995,            2766,            2867,            2503,            2736,            3457,            1810,            516,            2302,            2167,            1211,            4140,            3461,            3466,            ],        [            2886,            2879,            3540,            4207,            1937,            1623,            2115,            1073,            1794,            2078,            2242,            1114,            711,            582,            1896,            884,            727,            3438,            1165,            2435,            3424,            869,            999,            1447,            1427,            2155,            741,            982,            671,            1862,            3128,            243,            3784,            330,            3435,            3529,            1851,            2990,            2765,            2495,            1055,            3560,            1952,            2101,            2487,            2466,            3832,            4324,            2989,            2697,            2123,            3800,            3735,            2239,            2431,            3840,            3011,            2913,            2350,            3731,            2375,            1736,            3062,            2304,            3525,            4158,            292,            984,            0,            1315,            3083,            2038,            2139,            3569,            2008,            3743,            922,            912,            3368,            1849,            707,            3846,            3516,            4074,            ],        [            4942,            4908,            3063,            3730,            1460,            1338,            1830,            2117,            2096,            1601,            1765,            1440,            651,            1321,            581,            1612,            2004,            3740,            1491,            1958,            3726,            1813,            1325,            1262,            2471,            840,            2018,            333,            1948,            1385,            2651,            1520,            3307,            970,            2958,            4854,            3509,            4648,            4423,            3525,            1381,            4349,            3610,            3759,            4145,            4124,            3355,            4869,            4647,            4355,            3781,            5422,            3258,            3897,            4089,            5061,            4669,            3410,            4008,            4952,            4033,            973,            5118,            3962,            4746,            4703,            1569,            919,            1334,            0,            3584,            3696,            3797,            3092,            3666,            4046,            1847,            435,            2891,            1564,            1984,            4148,            4050,            4055,            ],        [            2863,            2856,            783,            1320,            2751,            4571,            5063,            2181,            3167,            2892,            1945,            3144,            3282,            3631,            4150,            2944,            2362,            3672,            4113,            2138,            4004,            2343,            3940,            4395,            2303,            4409,            2481,            3236,            2372,            2280,            2386,            2775,            1458,            3601,            1171,            1303,            1508,            1796,            1726,            651,            4003,            798,            1929,            1312,            1048,            1478,            229,            1097,            2255,            1963,            2100,            1574,            798,            942,            1392,            1510,            1817,            536,            783,            1401,            1655,            4542,            3039,            1265,            1195,            1828,            2766,            3229,            2996,            3569,            0,            2015,            1100,            1308,            1140,            981,            2749,            3166,            831,            4797,            2439,            4080,            466,            1007,            ],        [            848,            841,            2392,            3335,            4140,            3330,            3822,            940,            1926,            4281,            3334,            1903,            2466,            2337,            4209,            1703,            1121,            1657,            2872,            3527,            1989,            2166,            2706,            3154,            1062,            3576,            1974,            3295,            1131,            3669,            3775,            1534,            2847,            2085,            2560,            2147,            863,            1608,            1383,            1507,            2762,            2662,            391,            1113,            1426,            905,            2355,            3426,            1428,            1136,            562,            2418,            2863,            1515,            1707,            2354,            1629,            1925,            1626,            2833,            814,            3443,            1024,            1580,            2627,            3260,            1525,            2739,            1755,            3628,            2126,            0,            1415,            2697,            1368,            2845,            1508,            3225,            2220,            3556,            1940,            2065,            2462,            3086,            ],        [            2642,            2635,            1836,            2186,            4067,            4617,            5109,            2227,            3213,            4208,            3261,            3190,            3753,            3624,            5496,            2990,            2408,            3451,            4159,            3454,            3783,            3453,            3993,            4441,            2349,            4863,            3261,            4582,            2418,            3596,            3702,            2821,            2774,            3372,            2354,            1416,            1425,            877,            652,            1426,            4049,            1447,            1708,            1032,            717,            809,            1412,            2211,            1494,            1294,            1204,            1687,            1851,            289,            318,            1727,            898,            1719,            400,            1618,            986,            4730,            2818,            191,            1412,            2045,            2812,            4026,            3042,            4915,            1183,            1794,            0,            2491,            573,            1630,            2795,            4512,            2014,            4843,            3227,            3859,            1403,            1873,            ],        [            3534,            3527,            860,            638,            2414,            4227,            4719,            2852,            4374,            2555,            1608,            3718,            2945,            3294,            3813,            3890,            3033,            4343,            3769,            1801,            4675,            2006,            3603,            4051,            2974,            4072,            2144,            2899,            3043,            1943,            2049,            2643,            1121,            3264,            838,            1762,            2179,            2255,            2386,            1322,            3659,            1257,            2600,            1897,            1666,            2096,            977,            1777,            2873,            2581,            2771,            2330,            166,            1560,            2010,            1969,            2276,            1284,            1401,            1860,            2273,            4205,            3710,            1883,            1654,            1611,            2634,            2892,            2864,            3232,            748,            2686,            1718,            0,            1758,            954,            3420,            2829,            688,            4453,            2102,            4751,            1084,            1125,            ],        [            1948,            1941,            1778,            2291,            4009,            3923,            4415,            1533,            2519,            4150,            3203,            2496,            3059,            2930,            4802,            2296,            1714,            2757,            3465,            3396,            3089,            2759,            3299,            3747,            1655,            4169,            2567,            3888,            1724,            3538,            3644,            2127,            2716,            2678,            2296,            1521,            1367,            982,            757,            1368,            3355,            1552,            1014,            974,            659,            509,            1354,            2316,            1286,            994,            904,            1792,            1793,            231,            423,            1832,            1003,            1661,            342,            1723,            686,            4036,            2124,            296,            1517,            2150,            2118,            3332,            2348,            4221,            1125,            1100,            131,            2433,            0,            1735,            2101,            3818,            1956,            4149,            2533,            3165,            1508,            1978,            ],        [            3731,            3724,            1057,            556,            3147,            4960,            5452,            3049,            5107,            3288,            2341,            4451,            3678,            4027,            4546,            3812,            3230,            4540,            4502,            2534,            4872,            2739,            4336,            4784,            3171,            4805,            2877,            3632,            3240,            2676,            2782,            3376,            1854,            3997,            1571,            1295,            2376,            1788,            1872,            1519,            4392,            790,            2797,            2180,            1822,            2252,            1177,            1695,            3029,            2737,            2968,            2248,            931,            1716,            1638,            1502,            1809,            1484,            1557,            1813,            2429,            4938,            3907,            2039,            1187,            1529,            3367,            3625,            3597,            3965,            948,            2883,            1874,            1708,            1914,            0,            3617,            3562,            1421,            5186,            2835,            4948,            542,            243,            ],        [            2942,            4617,            4018,            4685,            2415,            1448,            1940,            1129,            813,            2556,            2720,            395,            1609,            1347,            2377,            195,            1299,            3449,            990,            2913,            3435,            1783,            824,            1272,            1483,            1694,            1591,            1463,            1148,            2340,            3606,            1151,            4262,            1095,            3913,            3585,            1907,            3046,            2821,            2551,            880,            3616,            2008,            2157,            2543,            2522,            4310,            5824,            3045,            2753,            2179,            3856,            4213,            2295,            2487,            3896,            3067,            2969,            2406,            3787,            2431,            1561,            3118,            2360,            3581,            5658,            1142,            1877,            1127,            1796,            3139,            2094,            2195,            4047,            2064,            3799,            0,            1393,            3846,            1674,            1557,            3857,            3572,            5010,            ],        [            5831,            5797,            2628,            3295,            1025,            2838,            3330,            3830,            2985,            1166,            1330,            2329,            1556,            1905,            2424,            2501,            2909,            4629,            2380,            1523,            4615,            2630,            2214,            2662,            3376,            2683,            2923,            1510,            2853,            950,            2216,            2425,            2872,            1875,            2523,            4419,            3947,            4912,            5043,            3090,            2270,            3914,            4499,            3751,            3851,            4753,            2920,            4434,            5536,            5244,            4670,            4987,            2823,            4217,            4667,            4626,            4933,            2975,            4058,            4517,            4922,            2816,            6007,            4540,            4311,            4268,            2474,            1503,            2239,            1843,            3149,            4585,            4375,            2657,            4555,            3611,            2736,            0,            2456,            3064,            2889,            5037,            3615,            3620,            ],        [            2846,            2839,            172,            1477,            2734,            4554,            5046,            2164,            3150,            2875,            1928,            3127,            3265,            3614,            4133,            2927,            2345,            3655,            4096,            2121,            3987,            2326,            3930,            4378,            2286,            4392,            2464,            3219,            2355,            2263,            2369,            2758,            1441,            3584,            1154,            1996,            1491,            2489,            2419,            634,            3986,            1491,            1912,            1295,            1395,            2426,            464,            1790,            2949,            2657,            2083,            2267,            955,            1635,            2085,            2203,            2510,            519,            1476,            2094,            2335,            4667,            3022,            1958,            1888,            2450,            2749,            3212,            2979,            3552,            693,            1998,            1793,            1291,            1968,            1674,            2732,            3149,            0,            4780,            2422,            4063,            1159,            1164,            ],        [            4045,            4011,            4154,            4821,            2551,            226,            266,            2243,            1574,            2692,            2856,            1307,            1745,            2415,            983,            1479,            2252,            2843,            694,            3049,            2829,            2362,            960,            402,            2597,            724,            2567,            1599,            2262,            2476,            3742,            2069,            4398,            2064,            4049,            4665,            2987,            4126,            3901,            4616,            794,            5440,            3088,            3237,            3623,            3602,            4446,            5960,            4125,            3833,            3259,            6513,            4349,            3375,            3567,            6152,            4147,            4501,            3486,            6043,            3511,            591,            4221,            3440,            5837,            5794,            2118,            2013,            1883,            1932,            4675,            3174,            3275,            4183,            3144,            5137,            1714,            1529,            3982,            0,            2533,            3251,            5141,            5146,            ],        [            2893,            2886,            2527,            3194,            1902,            2172,            2664,            1080,            2343,            2043,            1391,            1663,            1260,            761,            2445,            1671,            967,            3987,            1714,            1584,            3973,            300,            1548,            1996,            1434,            2704,            108,            1531,            911,            1726,            1832,            541,            2771,            879,            2422,            3536,            1858,            2997,            2772,            2502,            1604,            3813,            1959,            2108,            2494,            2473,            2819,            4333,            2996,            2704,            2130,            3807,            2722,            2246,            2438,            3847,            3018,            2920,            2357,            3738,            2382,            2285,            3069,            2311,            3532,            4167,            532,            1163,            762,            1864,            3048,            2045,            2146,            2556,            2015,            3510,            1476,            1461,            2355,            2398,            0,            4395,            3523,            3519,            ],        [            1937,            1903,            4576,            5519,            4352,            3435,            2591,            3215,            2370,            4493,            4657,            3158,            3546,            4216,            4314,            3330,            3396,            735,            2977,            4850,            493,            4213,            2811,            3259,            3246,            3681,            4249,            3400,            3406,            4277,            5543,            4415,            5031,            3865,            4744,            4331,            3047,            3792,            3567,            3691,            2867,            4846,            2575,            3297,            3610,            3089,            4539,            5610,            3612,            3312,            2746,            4602,            5047,            3699,            3891,            4538,            3813,            4109,            3810,            5017,            2998,            3548,            2113,            3764,            4811,            5444,            3800,            3814,            4229,            3733,            4310,            2412,            3599,            4881,            3552,            5029,            3239,            3330,            4404,            3661,            4215,            0,            4646,            5270,            ],        [            3325,            3318,            1127,            958,            3213,            5033,            5525,            2643,            3629,            3354,            2407,            3606,            3744,            4093,            4612,            3406,            2824,            4134,            4575,            2600,            4466,            2805,            4402,            4857,            2765,            4871,            2943,            3698,            2834,            2742,            2848,            3237,            1920,            4063,            1633,            837,            1970,            1330,            1330,            1113,            4465,            332,            2391,            1611,            1380,            1810,            691,            631,            2521,            2295,            2562,            1108,            1130,            1274,            1096,            1044,            1351,            998,            1115,            935,            1987,            5004,            3501,            1223,            729,            1362,            3228,            3691,            3458,            4031,            462,            2477,            1432,            1770,            1472,            515,            3211,            3628,            1293,            5259,            2901,            4542,            0,            645,            ],        [            4024,            4017,            1350,            313,            2904,            4717,            5209,            3342,            4864,            3045,            2098,            4208,            3435,            3784,            4303,            4380,            3523,            4833,            4259,            2291,            5165,            2496,            4093,            4541,            3464,            4562,            2634,            3389,            3533,            2433,            2539,            3133,            1611,            3754,            1328,            1512,            2669,            2005,            2136,            1812,            4149,            1007,            3090,            2473,            2156,            2586,            1642,            1452,            3196,            3071,            3261,            2005,            688,            2050,            2500,            1719,            2026,            1697,            1891,            1570,            2763,            4695,            4200,            2373,            1687,            1286,            3124,            3382,            3354,            3722,            1238,            3176,            2208,            1465,            2248,            704,            3910,            3319,            1178,            4943,            2592,            5241,            1183,            0,            ],        ], 'starts': [78, 41, 3, 19, 15], 'ends': [78, 41, 3, 19, 15]},    'NW-7': {'distance_matrix': [        [            0,            2171,            2897,            4153,            3915,            3858,            1432,            3096,            10793,            2534,            2974,            2270,            2700,            3393,            3067,            3543,            3866,            3266,            4549,            3367,            2936,            3959,            4070,            4395,            3605,            5280,            4128,            4708,            4921,            3158,            3219,            4342,            5773,            5230,            4328,            3792,            2227,            4864,            2672,            3871,            2218,            3330,            3377,            4273,            4169,            3918,            1870,            4047,            3733,            3284,            4584,            4090,            4703,            2789,            3490,            5844,            1585,            3422,            3893,            4484,            4079,            1011,            4167,            ],        [            788,            0,            926,            2182,            2222,            2165,            2220,            1125,            11581,            563,            1281,            3058,            1007,            1422,            1374,            1572,            2173,            1573,            2578,            1674,            1243,            4747,            2377,            5183,            4393,            6068,            4916,            5496,            2950,            3946,            4007,            5130,            2750,            6018,            2635,            2099,            3015,            5652,            3460,            4659,            140,            4118,            4165,            5061,            4957,            4706,            2658,            4835,            4521,            4072,            5372,            4878,            5491,            3577,            4278,            2821,            2373,            1729,            4681,            3279,            2386,            1799,            2474,            ],        [            1710,            988,            0,            1256,            3144,            3087,            3142,            199,            12503,            544,            2203,            3980,            1929,            732,            2296,            646,            3095,            2495,            1652,            2596,            2165,            5669,            3299,            6105,            5315,            5727,            5838,            6418,            2024,            4868,            4929,            6052,            3672,            6940,            3557,            3021,            3937,            6574,            4382,            5581,            1000,            5040,            5087,            5983,            5879,            5628,            3580,            5757,            5443,            4994,            6294,            5800,            6413,            4499,            5200,            3743,            3295,            2651,            5603,            4201,            3308,            2721,            3396,            ],        [            2787,            2065,            1333,            0,            4221,            4164,            4219,            1178,            13580,            1621,            3280,            5057,            3006,            1809,            3373,            610,            4172,            3572,            396,            3673,            3242,            6746,            4376,            7182,            6392,            8067,            6915,            7495,            768,            5945,            6006,            7129,            4749,            8017,            4634,            4098,            5014,            7651,            5459,            6658,            2077,            6117,            6164,            7060,            6956,            6705,            4657,            6834,            6520,            6071,            7371,            6877,            7490,            5576,            6277,            4820,            4372,            3728,            6680,            5278,            4385,            3798,            4473,            ],        [            2144,            2200,            2926,            4182,            0,            537,            3576,            3125,            12937,            2563,            2716,            4414,            1289,            3422,            1328,            3572,            205,            1506,            4578,            1028,            1525,            2962,            1049,            1807,            2288,            3753,            2601,            3181,            4950,            2974,            2719,            2815,            798,            3703,            1307,            1219,            4371,            1702,            4816,            2065,            2247,            5474,            2380,            1911,            1845,            1849,            4014,            2520,            2206,            3100,            3587,            3668,            3176,            3084,            2990,            869,            3729,            1139,            2896,            1327,            434,            3155,            522,            ],        [            2117,            2173,            2899,            4155,            303,            0,            3549,            3098,            12910,            2536,            1578,            4387,            1158,            3395,            791,            3545,            508,            1870,            4551,            491,            922,            2829,            1303,            1674,            2155,            3620,            2468,            3048,            4923,            2841,            2586,            2682,            665,            3570,            1561,            1473,            4344,            1569,            4789,            1932,            2220,            2964,            2247,            1778,            1712,            1716,            3987,            2387,            2073,            2967,            3454,            3535,            3043,            2951,            2857,            736,            3702,            2026,            2763,            1194,            301,            3128,            389,            ],        [            15684,            13940,            16088,            17971,            18465,            17049,            0,            16287,            9362,            15725,            16165,            15461,            15891,            16584,            16258,            17361,            17057,            16457,            16318,            16558,            16127,            16288,            17261,            16724,            15934,            17609,            16457,            17037,            16690,            15487,            14126,            16671,            18102,            17559,            17519,            16983,            14556,            15771,            15001,            16200,            15409,            15659,            15706,            15180,            16498,            14825,            14199,            16376,            16062,            15613,            15648,            16419,            17032,            15118,            15819,            18173,            13914,            16613,            16222,            16813,            18265,            14285,            16805,            ],        [            1853,            1131,            399,            1057,            3287,            3230,            3285,            0,            12646,            687,            2346,            4123,            2072,            875,            2439,            447,            3238,            2638,            1453,            2739,            2308,            5812,            3442,            6248,            5458,            7133,            5981,            6561,            1825,            5011,            5072,            6195,            3815,            7083,            3700,            3164,            4080,            6717,            4525,            5724,            1143,            5183,            5230,            6126,            6022,            5771,            3723,            5900,            5586,            5137,            6437,            5943,            6556,            4642,            5343,            3886,            3438,            2794,            5746,            4344,            3451,            2864,            3539,            ],        [            4900,            4578,            5304,            6560,            7681,            6265,            2956,            5503,            0,            4941,            5381,            4677,            5107,            5800,            5474,            5950,            6273,            5673,            6956,            5774,            5343,            5504,            6477,            5940,            5150,            6825,            6470,            6253,            7328,            4703,            4764,            5887,            7318,            6775,            6735,            6199,            3772,            6409,            4217,            5416,            4625,            4875,            4922,            5818,            5714,            5463,            3415,            5592,            5278,            4829,            6129,            5635,            6248,            4334,            5035,            7389,            3130,            5829,            5438,            6029,            7481,            3501,            7443,            ],        [            1166,            444,            363,            1619,            2600,            2543,            2598,            562,            11959,            0,            1659,            3436,            1385,            188,            1752,            1009,            2551,            1951,            2015,            2052,            1621,            5125,            2755,            5561,            4771,            5183,            6091,            5874,            2387,            4324,            4385,            5508,            3128,            5133,            3013,            2477,            3393,            6030,            3838,            5037,            456,            4496,            4543,            5439,            5335,            5084,            3036,            5213,            4899,            4450,            5750,            5256,            5869,            3955,            4656,            3199,            2751,            2107,            5059,            3657,            2764,            2177,            2852,            ],        [            1485,            1541,            2267,            3523,            941,            1208,            2917,            2466,            12278,            1904,            0,            3755,            630,            2763,            997,            2913,            892,            292,            3919,            1297,            866,            5444,            1096,            2478,            2959,            5502,            6410,            6193,            4291,            4643,            4704,            5827,            1469,            5452,            1354,            818,            3712,            6349,            4157,            5356,            1588,            4815,            4862,            5758,            5654,            2520,            3355,            5532,            5218,            4769,            6069,            5575,            6188,            4274,            4975,            1540,            3070,            448,            5378,            1998,            1105,            2496,            1193,            ],        [            1285,            1341,            2067,            3323,            1645,            1588,            2717,            2266,            12078,            1704,            704,            0,            430,            2563,            797,            2713,            1596,            996,            3719,            1097,            666,            5244,            1800,            3182,            4890,            5302,            6210,            5993,            4091,            4443,            4504,            5627,            2173,            5252,            2058,            1522,            3512,            6149,            3957,            5156,            1388,            4615,            4662,            5558,            5454,            5203,            3155,            5332,            5018,            4569,            5869,            5375,            5988,            4074,            4775,            2244,            2870,            1152,            5178,            2702,            1809,            2296,            1897,            ],        [            959,            1015,            1741,            2997,            1461,            1158,            2391,            1940,            11752,            1378,            420,            3229,            0,            2237,            367,            2387,            1312,            712,            3393,            667,            236,            4918,            1516,            2832,            4564,            4976,            5884,            5667,            3765,            4117,            4178,            5301,            1823,            4926,            1774,            1238,            3186,            5823,            3631,            4830,            1062,            4289,            4336,            5232,            5128,            4877,            2829,            5006,            4692,            4243,            5543,            5049,            5662,            3748,            4449,            1894,            2544,            868,            4852,            2352,            1459,            1970,            1547,            ],        [            978,            256,            659,            1915,            2412,            2355,            2410,            858,            11771,            296,            1471,            3248,            1197,            0,            1564,            1305,            2363,            1763,            2311,            1864,            1433,            4937,            2567,            5373,            4583,            4995,            5903,            5686,            2683,            4136,            4197,            5320,            2940,            4945,            2825,            2289,            3205,            5842,            3650,            4849,            268,            4308,            4355,            5251,            5147,            4896,            2848,            5025,            4711,            4262,            5562,            5068,            5681,            3767,            4468,            3011,            2563,            1919,            4871,            3469,            2576,            1989,            2664,            ],        [            1326,            1382,            2108,            3364,            1094,            791,            2758,            2307,            12119,            1745,            787,            3596,            367,            2604,            0,            2754,            1299,            1079,            3760,            300,            131,            5285,            1883,            2465,            4931,            5343,            6251,            6034,            4132,            4484,            4545,            5668,            1456,            5293,            2141,            1605,            3553,            6190,            3998,            5197,            1429,            4656,            4703,            5599,            5495,            2507,            3196,            5373,            5059,            4610,            5910,            5416,            6029,            4115,            4816,            1527,            2911,            1235,            5219,            1985,            1092,            2337,            1180,            ],        [            2177,            1455,            723,            610,            3611,            3554,            3609,            568,            12970,            1011,            2670,            4447,            2396,            1199,            2763,            0,            3562,            2962,            1006,            3063,            2632,            6136,            3766,            6572,            5782,            7457,            7102,            6885,            1378,            5335,            5396,            6519,            4139,            7407,            4024,            3488,            4404,            7041,            4849,            6048,            1467,            5507,            5554,            6450,            6346,            6095,            4047,            6224,            5910,            5461,            6761,            6267,            6880,            4966,            5667,            4210,            3762,            3118,            6070,            4668,            3775,            3188,            3863,            ],        [            1939,            1995,            2721,            3977,            380,            647,            3371,            2920,            12732,            2358,            2511,            4209,            1084,            3217,            1451,            3367,            0,            1301,            4373,            1138,            1320,            3072,            844,            1917,            2398,            3863,            2711,            3291,            4745,            5097,            2829,            2925,            908,            3813,            1102,            1014,            4166,            1812,            4611,            2175,            2042,            5269,            2490,            2021,            1955,            1959,            3809,            2630,            2316,            5223,            3697,            6029,            3286,            4728,            3100,            979,            3524,            934,            3006,            1437,            544,            2950,            632,            ],        [            1573,            1629,            2355,            3611,            649,            916,            3005,            2554,            12366,            1992,            2145,            3843,            718,            2851,            1085,            3001,            600,            0,            4007,            1407,            954,            3341,            648,            2186,            2667,            4132,            2980,            3560,            4379,            4731,            3098,            3194,            1177,            4082,            906,            818,            3800,            2081,            4245,            2444,            1676,            4903,            2759,            2290,            2224,            2228,            3443,            2899,            2585,            4857,            3966,            5663,            3555,            4362,            3369,            1248,            3158,            568,            3275,            1706,            813,            2584,            901,            ],        [            3183,            2461,            1729,            396,            4617,            4560,            4615,            1574,            13976,            2017,            3676,            5453,            3402,            2205,            3769,            1006,            4568,            3968,            0,            4069,            3638,            7142,            4772,            7578,            6788,            8463,            8108,            7891,            372,            6341,            6402,            7525,            5145,            8413,            5030,            4494,            5410,            8047,            5855,            7054,            2473,            6513,            6560,            7456,            7352,            7101,            5053,            7230,            6916,            6467,            7767,            7273,            7886,            5972,            6673,            5216,            4768,            4124,            7076,            5674,            4781,            4194,            4869,            ],        [            1626,            1682,            2408,            3664,            794,            491,            3058,            2607,            12419,            2045,            1087,            3896,            667,            2904,            300,            3054,            999,            1379,            4060,            0,            431,            5585,            1794,            2165,            2646,            5643,            2959,            3539,            4432,            4784,            4845,            3173,            1156,            5593,            2052,            1964,            3853,            2060,            4298,            2423,            1729,            4956,            5003,            2269,            2203,            2207,            3496,            2878,            2564,            4910,            6210,            5716,            3534,            4415,            5116,            1227,            3211,            1535,            5519,            1685,            792,            2637,            880,            ],        [            1195,            1251,            1977,            3233,            1225,            922,            2627,            2176,            11988,            1614,            656,            3465,            236,            2473,            131,            2623,            1430,            948,            3629,            431,            0,            5154,            1752,            2596,            4800,            5212,            6120,            5903,            4001,            4353,            4414,            5537,            1587,            5162,            2010,            1474,            3422,            6059,            3867,            5066,            1298,            4525,            4572,            5468,            5364,            5113,            3065,            5242,            4928,            4479,            5779,            5285,            5898,            3984,            4685,            1658,            2780,            1104,            5088,            2116,            1223,            2206,            1311,            ],        [            4464,            4142,            4868,            6124,            2235,            2336,            2417,            5067,            11779,            4505,            4945,            4241,            3494,            5364,            3127,            5514,            2440,            3416,            6520,            2827,            3258,            0,            2539,            1230,            424,            1632,            1727,            2307,            6892,            773,            551,            1941,            1872,            1582,            2797,            2709,            1384,            1523,            1080,            690,            4189,            929,            212,            1863,            1759,            1122,            2979,            1646,            1332,            899,            1419,            1705,            2302,            916,            822,            1943,            2694,            3079,            728,            1303,            2035,            3065,            1997,            ],        [            2808,            2864,            3590,            4846,            1123,            1224,            4240,            3789,            13601,            3227,            3380,            5078,            1953,            4086,            2320,            4236,            1328,            1615,            5242,            1715,            2189,            2847,            0,            1692,            2173,            3638,            2486,            3066,            5614,            2859,            2604,            2700,            683,            3588,            258,            908,            5035,            1587,            3166,            1950,            2911,            2982,            2265,            1796,            1730,            1734,            4678,            2405,            2091,            2985,            3472,            3553,            3061,            2969,            2875,            754,            4393,            1278,            2781,            1212,            923,            3819,            885,            ],        [            3876,            3932,            4658,            5914,            1502,            1603,            3251,            4857,            12613,            4295,            3181,            5075,            2761,            5154,            2394,            5304,            1707,            2683,            6310,            2094,            2525,            834,            1806,            0,            1367,            2466,            1540,            3141,            6682,            1607,            1202,            2775,            1139,            2416,            2064,            1976,            2218,            1329,            1914,            1524,            3979,            1763,            1046,            1592,            1488,            928,            3813,            2480,            2166,            1733,            1514,            2539,            3136,            1750,            1656,            1210,            3528,            2346,            823,            570,            1302,            3899,            1264,            ],        [            4344,            4022,            4748,            6004,            2923,            3024,            2297,            4947,            11659,            4385,            4825,            4121,            4551,            5244,            3815,            5394,            3128,            4104,            6400,            3515,            3946,            674,            3227,            1110,            0,            1512,            1607,            2187,            6772,            653,            431,            1821,            2560,            1462,            3485,            3397,            1264,            1926,            960,            714,            4069,            809,            92,            1743,            1639,            1397,            2859,            1526,            1212,            779,            1299,            1585,            2182,            796,            702,            2631,            2574,            3767,            608,            1327,            2723,            2945,            2685,            ],        [            5822,            5878,            6604,            7860,            3448,            3549,            4077,            6803,            13439,            6241,            5127,            5901,            4707,            7100,            4340,            7250,            3653,            4629,            8256,            4040,            4471,            3351,            3752,            2707,            2643,            0,            1227,            882,            8628,            2433,            3108,            944,            3085,            177,            4010,            3922,            4996,            2451,            2740,            3263,            5925,            2722,            2769,            2268,            2164,            1922,            4639,            1239,            1553,            747,            3976,            306,            877,            3473,            3379,            3156,            4354,            4292,            3285,            2407,            3248,            4725,            3210,            ],        [            4597,            4275,            5001,            6257,            2291,            2392,            2550,            5200,            11912,            4638,            5078,            4374,            3550,            5497,            3183,            5647,            2496,            3472,            6653,            2883,            3314,            1824,            2595,            2260,            1470,            1227,            0,            1145,            7025,            906,            1581,            779,            1928,            1177,            2853,            2765,            3469,            1294,            1213,            1736,            4322,            1195,            1242,            1111,            1007,            765,            3112,            484,            396,            477,            2449,            1125,            1140,            1946,            1852,            1999,            2827,            3135,            1758,            1250,            2091,            3198,            2053,            ],        [            5311,            5367,            6093,            7349,            2937,            3038,            3566,            6292,            12928,            5730,            4616,            5390,            4196,            6589,            3829,            6739,            3142,            4118,            7745,            3529,            3960,            2840,            3241,            2196,            2132,            572,            823,            0,            8117,            1922,            2597,            433,            2574,            522,            3499,            3411,            4485,            1940,            2229,            2752,            5414,            2211,            2258,            1757,            1653,            1411,            4128,            728,            1042,            1242,            3465,            484,            366,            2962,            2868,            2645,            3843,            3781,            2774,            1896,            2737,            4214,            2699,            ],        [            3555,            2833,            2101,            768,            4989,            4932,            4987,            1946,            14348,            2389,            4048,            5825,            3774,            2577,            4141,            1378,            4940,            4340,            372,            4441,            4010,            7514,            5144,            7950,            7160,            8835,            8480,            8263,            0,            6713,            6774,            7897,            5517,            8785,            5402,            4866,            5782,            8419,            6227,            7426,            2845,            6885,            6932,            7828,            7724,            7473,            5425,            7602,            7288,            6839,            8139,            7645,            8258,            6344,            7045,            5588,            5140,            4496,            7448,            6046,            5153,            4566,            5241,            ],        [            3691,            3369,            4095,            5351,            3204,            3305,            1644,            4294,            11006,            3732,            4172,            3468,            3898,            4591,            4265,            4741,            3409,            4464,            5747,            3796,            4134,            1719,            3508,            2155,            1365,            1777,            1888,            2468,            6119,            0,            979,            2102,            2841,            1727,            3766,            3678,            2563,            2207,            307,            1631,            3416,            1090,            1137,            2024,            1920,            1678,            2206,            1807,            1493,            1044,            2344,            1850,            2463,            1344,            1250,            2912,            1921,            4620,            1653,            2244,            3004,            2292,            2966,            ],        [            4574,            4252,            4978,            6234,            2345,            2446,            2527,            5177,            11889,            4615,            5055,            4351,            3604,            5474,            3237,            5624,            2550,            3526,            6630,            2937,            3368,            755,            2649,            1191,            534,            1742,            1837,            2417,            7002,            883,            0,            2051,            1982,            1692,            2907,            2819,            1076,            1633,            1190,            800,            4299,            1039,            266,            1973,            1869,            1232,            3089,            1756,            1442,            1009,            994,            1815,            2412,            608,            514,            2053,            2804,            3189,            379,            1413,            2145,            3175,            2107,            ],        [            4878,            4934,            5660,            6916,            2504,            2605,            3133,            5859,            12495,            5297,            4183,            4957,            3763,            6156,            3396,            6306,            2709,            3685,            7312,            3096,            3527,            2407,            2808,            1763,            1699,            938,            779,            366,            7684,            1489,            2164,            0,            2141,            888,            3066,            2978,            4052,            1507,            1796,            2319,            4981,            1778,            1825,            1324,            1220,            978,            3695,            295,            609,            1256,            3032,            850,            361,            2529,            2435,            2212,            3410,            3348,            2341,            1463,            2304,            3781,            2266,            ],        [            2928,            2984,            3710,            4966,            565,            666,            4360,            3909,            13721,            3347,            2244,            5198,            2073,            4206,            1457,            4356,            770,            1735,            5362,            1157,            1588,            2262,            858,            1107,            1588,            3053,            1901,            2481,            5734,            2274,            2019,            2115,            0,            3003,            1116,            1028,            5155,            1002,            2581,            1365,            3031,            2563,            1680,            1211,            1145,            1149,            4798,            1820,            1506,            2400,            2887,            2968,            2476,            2384,            2290,            71,            4513,            1398,            2196,            627,            365,            3939,            327,            ],        [            5872,            5928,            6654,            7910,            3498,            3599,            4127,            6853,            13489,            6291,            5177,            5951,            4757,            7150,            4390,            7300,            3703,            4679,            8306,            4090,            4521,            3401,            3802,            2757,            2693,            50,            1277,            932,            8678,            2483,            3158,            994,            3135,            0,            4060,            3972,            5046,            2501,            2790,            3313,            5975,            2772,            2819,            2318,            2214,            1972,            4689,            1289,            1603,            797,            4026,            356,            927,            3523,            3429,            3206,            4404,            4342,            3335,            2457,            3298,            4775,            3260,            ],        [            2550,            2606,            3332,            4588,            865,            966,            3982,            3531,            13343,            2969,            3122,            4820,            1695,            3828,            2062,            3978,            1070,            1357,            4984,            1457,            1931,            2589,            480,            1434,            1915,            3380,            2228,            2808,            5356,            2601,            2346,            2442,            425,            3330,            0,            650,            4777,            1329,            2908,            1692,            2653,            2890,            2007,            1538,            1472,            1476,            4420,            2147,            1833,            2727,            3214,            3295,            2803,            2711,            2617,            496,            4135,            1020,            2523,            954,            665,            3561,            627,            ],        [            1900,            1956,            2682,            3938,            1356,            1623,            3332,            2881,            12693,            2319,            2472,            4170,            1045,            3178,            1412,            3328,            1307,            707,            4334,            1712,            1281,            5859,            575,            2267,            2748,            5917,            3061,            6608,            4706,            5058,            5119,            6242,            1258,            5867,            833,            0,            4127,            2162,            4572,            2525,            2003,            5230,            5277,            6173,            2305,            2309,            3770,            5947,            5633,            5184,            6484,            5990,            6603,            4689,            5390,            1329,            3485,            370,            5793,            1787,            1520,            2911,            1608,            ],        [            3829,            3507,            4233,            5489,            3217,            3318,            1782,            4432,            11144,            3870,            4310,            3606,            4036,            4729,            4109,            4879,            3422,            4398,            5885,            3809,            4272,            1732,            3521,            2168,            1378,            1790,            1901,            2481,            6257,            931,            992,            2115,            2854,            1740,            3779,            3691,            0,            2220,            445,            1644,            3554,            1103,            1150,            2037,            1933,            1691,            2344,            1820,            1506,            1057,            2357,            1863,            2476,            1357,            1263,            2925,            2059,            4061,            1666,            2257,            3017,            2430,            2979,            ],        [            3710,            3766,            4492,            5748,            1336,            1437,            3231,            4691,            12593,            4129,            3015,            5055,            2595,            4988,            2228,            5138,            1541,            2517,            6144,            1928,            2359,            1973,            1640,            818,            1299,            2366,            1214,            1794,            6516,            1587,            1730,            1428,            973,            2316,            1898,            1810,            4150,            0,            1894,            1076,            3813,            1876,            1391,            913,            809,            620,            3793,            1133,            819,            1713,            2598,            2281,            1789,            2095,            2001,            1044,            3508,            2180,            1907,            338,            1136,            4721,            1098,            ],        [            3384,            3062,            3788,            5044,            6165,            4749,            1337,            3987,            10699,            3425,            3865,            3161,            3591,            4284,            3958,            4434,            4757,            4157,            5440,            4258,            3827,            3988,            4961,            4424,            3634,            4046,            4157,            4737,            5812,            3187,            3248,            4371,            5802,            3996,            5219,            4683,            2256,            4893,            0,            3900,            3109,            3359,            3406,            4302,            4198,            3947,            1899,            4076,            3762,            3313,            4613,            4119,            4732,            2818,            3519,            5873,            1614,            4313,            3922,            4513,            5965,            1985,            5927,            ],        [            3919,            3975,            4701,            5957,            1545,            1646,            2816,            4900,            12178,            4338,            3224,            4640,            2804,            5197,            2437,            5347,            1750,            2726,            6353,            2137,            2568,            1193,            1849,            913,            519,            2031,            1044,            1624,            6725,            1172,            950,            1258,            1182,            1981,            2107,            2019,            1783,            833,            1479,            0,            4022,            1328,            611,            1096,            992,            432,            3378,            963,            649,            1298,            1818,            2104,            1619,            1315,            1221,            1253,            3093,            2389,            1127,            613,            1345,            3464,            1307,            ],        [            710,            90,            848,            2104,            2144,            2087,            2142,            1047,            11503,            485,            1203,            2980,            929,            1344,            1296,            1494,            2095,            1495,            2500,            1596,            1165,            4669,            2299,            5105,            4315,            4727,            4838,            5418,            2872,            3868,            3929,            5052,            2672,            4677,            2557,            2021,            2937,            5574,            3382,            4581,            0,            4040,            4087,            4983,            4879,            4628,            2580,            4757,            4443,            3994,            5294,            4800,            5413,            3499,            4200,            2743,            2295,            1651,            4603,            3201,            2308,            1721,            2396,            ],        [            4488,            4544,            5270,            6526,            2114,            2215,            2815,            5469,            12177,            4907,            3793,            4639,            3373,            5766,            3006,            5916,            2319,            3295,            6922,            2706,            3137,            2089,            2418,            1373,            1309,            1950,            798,            1378,            7294,            1171,            1846,            1012,            1751,            1900,            2676,            2588,            3734,            1117,            1478,            1086,            4591,            0,            1507,            934,            830,            588,            3377,            717,            403,            1297,            2714,            1862,            1373,            2211,            2117,            1822,            3092,            2958,            2023,            1073,            1914,            3463,            1876,            ],        [            4308,            3986,            4712,            5968,            2079,            2180,            2261,            4911,            11623,            4349,            4789,            4085,            3338,            5208,            2971,            5358,            2284,            3260,            6364,            2671,            3102,            622,            2383,            1058,            268,            1476,            1571,            2151,            6736,            617,            339,            1785,            1716,            1426,            2641,            2553,            1172,            1367,            924,            534,            4033,            773,            0,            1707,            1603,            966,            2823,            1490,            1176,            743,            1247,            1549,            2146,            704,            610,            1787,            2538,            2923,            556,            1147,            1879,            2909,            1841,            ],        [            4622,            4678,            5404,            6660,            2248,            2349,            3386,            5603,            12748,            5041,            3927,            5210,            3507,            5900,            3140,            6050,            2453,            3429,            7056,            2840,            3271,            2170,            2552,            1560,            1496,            2521,            1369,            1949,            7428,            1742,            1927,            1583,            1885,            2471,            2810,            2722,            4305,            591,            2049,            1273,            4725,            2031,            1588,            0,            964,            775,            3948,            1288,            974,            1868,            2795,            2436,            1944,            2292,            2198,            1956,            3663,            3092,            2104,            929,            2048,            4034,            2010,            ],        [            4436,            4492,            5218,            6474,            2062,            2163,            2773,            5417,            12135,            4855,            3741,            4597,            3321,            5714,            2954,            5864,            2267,            3243,            6870,            2654,            3085,            2047,            2366,            1321,            1257,            1908,            756,            1336,            7242,            1129,            1804,            970,            1699,            1858,            2624,            2536,            3692,            937,            1436,            1034,            4539,            1418,            1465,            882,            0,            536,            3335,            675,            361,            1255,            2672,            1823,            1331,            2169,            2075,            1770,            3050,            2906,            1981,            1021,            1862,            3421,            1824,            ],        [            4228,            4284,            5010,            6266,            1854,            1955,            2926,            5209,            12288,            4647,            3533,            4750,            3113,            5506,            2746,            5656,            2059,            3035,            6662,            2446,            2877,            1710,            2158,            1100,            1036,            2061,            909,            1489,            7034,            1282,            1467,            1123,            1491,            2011,            2416,            2328,            3845,            401,            1589,            813,            4331,            1571,            1128,            674,            570,            0,            3488,            828,            514,            1408,            2335,            1976,            1484,            1832,            1738,            1562,            3203,            2698,            1644,            739,            1654,            5239,            1616,            ],        [            4186,            3864,            4590,            5846,            3574,            3675,            2139,            4789,            11501,            4227,            4667,            3963,            4393,            5086,            4466,            5236,            3779,            4755,            6242,            4166,            4629,            2089,            3878,            2525,            1735,            2147,            2258,            2838,            6614,            1288,            1349,            2472,            3211,            2097,            4136,            4048,            357,            2577,            802,            2001,            3911,            1460,            1507,            2394,            2290,            2048,            0,            2177,            1863,            1414,            2714,            2220,            2833,            919,            1620,            3282,            2416,            4418,            2023,            2614,            3374,            2787,            3336,            ],        [            4583,            4639,            5365,            6621,            2209,            2310,            2838,            5564,            12200,            5002,            3888,            4662,            3468,            5861,            3101,            6011,            2414,            3390,            7017,            2801,            3232,            2112,            2513,            1468,            1404,            1233,            484,            661,            7389,            1194,            1869,            295,            1846,            1183,            2771,            2683,            3757,            1212,            1501,            2024,            4686,            1483,            1530,            1029,            925,            683,            3400,            0,            314,            961,            2737,            1145,            656,            2234,            2140,            1917,            3115,            3053,            2046,            1168,            2009,            3486,            1971,            ],        [            4269,            4325,            5051,            6307,            1895,            1996,            2524,            5250,            11886,            4688,            3574,            4348,            3154,            5547,            2787,            5697,            2100,            3076,            6703,            2487,            2918,            1798,            2199,            1154,            1090,            1547,            507,            975,            7075,            880,            1555,            609,            1532,            1497,            2457,            2369,            3443,            898,            1187,            1710,            4372,            1169,            1216,            715,            611,            369,            3086,            314,            0,            1006,            2423,            1459,            970,            1920,            1826,            1603,            2801,            2739,            1732,            854,            1695,            3172,            1657,            ],        [            4067,            3745,            4471,            5727,            2779,            2880,            2020,            4670,            11382,            4108,            4548,            3844,            4274,            4967,            3671,            5117,            2984,            3960,            6123,            3371,            3802,            1294,            3083,            1730,            940,            1098,            477,            1622,            6495,            376,            1051,            1256,            2416,            791,            3341,            3253,            2939,            1782,            683,            1206,            3792,            665,            712,            1599,            1495,            1253,            2582,            961,            1068,            0,            1919,            914,            1617,            1416,            1322,            2487,            2297,            3623,            1228,            1819,            2579,            2668,            2541,            ],        [            5368,            5046,            5772,            7028,            3139,            3240,            3321,            5971,            12683,            5409,            5849,            5145,            4398,            6268,            4031,            6418,            3344,            4320,            7424,            3731,            4162,            1682,            3443,            2118,            1328,            2536,            2631,            3211,            7796,            1677,            880,            2845,            2776,            2486,            3701,            3613,            989,            2427,            1984,            1594,            5093,            1833,            1060,            2767,            2663,            2026,            1939,            2550,            2236,            1803,            0,            2609,            3206,            521,            663,            2847,            1654,            3983,            926,            2207,            2939,            1640,            2901,            ],        [            5516,            5572,            6298,            7554,            3142,            3243,            3771,            6497,            13133,            5935,            4821,            5595,            4401,            6794,            4034,            6944,            3347,            4323,            7950,            3734,            4165,            3045,            3446,            2401,            2337,            334,            886,            576,            8322,            2127,            2802,            638,            2779,            284,            3704,            3616,            4690,            2145,            2434,            2957,            5619,            2416,            2463,            1962,            1858,            1616,            4333,            933,            1247,            990,            3670,            0,            571,            3167,            3073,            2850,            4048,            3986,            2979,            2101,            2942,            4419,            2904,            ],        [            5316,            5372,            6098,            7354,            2942,            3043,            3571,            6297,            12933,            5735,            4621,            5395,            4201,            6594,            3834,            6744,            3147,            4123,            7750,            3534,            3965,            2845,            3246,            2201,            2137,            577,            828,            5,            8122,            1927,            2602,            438,            2579,            527,            3504,            3416,            4490,            1945,            2234,            2757,            5419,            2216,            2263,            1762,            1658,            1416,            4133,            733,            1047,            1247,            3470,            489,            0,            2967,            2873,            2650,            3848,            3786,            2779,            1901,            2742,            4219,            2704,            ],        [            5177,            4710,            5436,            6692,            2803,            2904,            2985,            5635,            12347,            5073,            4482,            4809,            4062,            5932,            3695,            6082,            3008,            3984,            7088,            3395,            3826,            1346,            3107,            1782,            992,            2200,            2295,            2875,            7460,            1341,            544,            2509,            2440,            2150,            3365,            3277,            653,            2091,            1648,            1258,            4757,            1497,            724,            2431,            2327,            1690,            1603,            2214,            1900,            1467,            530,            2273,            2870,            0,            327,            2511,            1318,            3647,            923,            1871,            2603,            1304,            2565,            ],        [            4391,            4069,            4795,            6051,            2744,            2845,            2344,            4994,            11706,            4432,            4872,            4168,            4003,            5291,            3636,            5441,            2949,            3925,            6447,            3336,            3767,            1287,            3048,            1723,            933,            2141,            2236,            2816,            6819,            1282,            485,            2450,            2381,            2091,            3306,            3218,            562,            2032,            1007,            1199,            4116,            1438,            665,            2372,            2268,            1631,            1512,            2155,            1841,            1408,            624,            2214,            2811,            94,            0,            2452,            1227,            3588,            864,            1812,            2544,            1213,            2506,            ],        [            2999,            3055,            3781,            5037,            636,            737,            4431,            3980,            13792,            3418,            2315,            5269,            2144,            4277,            1528,            4427,            841,            1806,            5433,            1228,            1659,            2333,            929,            1178,            1659,            3124,            1972,            2552,            5805,            2345,            2090,            2186,            71,            3074,            1187,            1099,            5226,            1073,            2652,            1436,            3102,            2468,            1751,            1282,            1216,            1220,            4869,            1891,            1577,            2471,            2958,            3039,            2547,            2455,            2361,            0,            4584,            1469,            2267,            698,            436,            4010,            398,            ],        [            4471,            4149,            4875,            6131,            3859,            3960,            2424,            5074,            11786,            4512,            4952,            4248,            4678,            5371,            4751,            5521,            4064,            5040,            6527,            4451,            4882,            2374,            4163,            2810,            2020,            2432,            2543,            3123,            6899,            1573,            1634,            2757,            3496,            2382,            4421,            4333,            642,            2862,            1087,            2286,            4196,            1745,            1792,            2679,            2575,            2333,            285,            2462,            2148,            1699,            2999,            2505,            3118,            1204,            1905,            3567,            0,            4703,            2308,            2899,            3659,            969,            3621,            ],        [            2173,            2229,            2955,            4211,            1629,            1896,            3605,            3154,            12966,            2592,            2745,            4443,            1318,            3451,            1685,            3601,            1580,            980,            4607,            1985,            1554,            3495,            648,            2340,            2821,            4286,            3134,            3714,            4979,            5331,            3252,            3348,            1331,            4236,            906,            370,            4400,            2783,            4845,            2598,            2276,            5503,            2913,            2444,            2378,            2382,            4043,            3053,            2739,            5457,            4120,            6263,            3709,            4962,            3523,            1402,            3758,            0,            3429,            1860,            1571,            3184,            1533,            ],        [            4840,            4518,            5244,            6500,            2611,            2712,            2793,            5443,            12155,            4881,            5321,            4617,            3870,            5740,            3503,            5890,            2816,            3792,            6896,            3203,            3634,            376,            2915,            812,            800,            2008,            2103,            2683,            7268,            1149,            379,            2317,            2248,            1958,            3173,            3085,            1455,            1899,            1456,            1066,            4565,            1305,            588,            2239,            2135,            1498,            3355,            2022,            1708,            1275,            691,            2081,            2678,            987,            893,            2319,            3070,            3455,            0,            1679,            2411,            3441,            2373,            ],        [            3486,            3542,            4268,            5524,            1112,            1213,            3258,            4467,            12620,            3905,            2791,            5082,            2371,            4764,            2004,            4914,            1317,            2293,            5920,            1704,            2135,            1635,            1416,            480,            961,            2286,            1134,            1714,            6292,            1614,            1392,            1348,            749,            2236,            1674,            1586,            2225,            923,            1921,            738,            3589,            1770,            1053,            1186,            1082,            522,            3820,            1053,            739,            1740,            2260,            2546,            1709,            1757,            1663,            820,            3535,            1956,            1569,            0,            912,            4497,            874,            ],        [            2418,            2474,            3200,            4456,            200,            301,            3850,            3399,            13211,            2837,            1879,            4688,            1459,            3696,            1092,            3846,            405,            1879,            4852,            792,            1223,            2528,            1002,            1373,            1854,            3319,            2167,            2747,            5224,            2540,            2285,            2381,            364,            3269,            1260,            1172,            4645,            1268,            2847,            1631,            2521,            2663,            1946,            1477,            1411,            1415,            4288,            2086,            1772,            2666,            3153,            3234,            2742,            2650,            2556,            435,            4003,            1339,            2462,            893,            0,            3429,            286,            ],        [            2458,            2136,            2862,            4118,            5125,            3823,            421,            3061,            9783,            2499,            2939,            2235,            2665,            3358,            3032,            3508,            3831,            3231,            4514,            3332,            2901,            2948,            4035,            3384,            2594,            3006,            3117,            3697,            4886,            2147,            2208,            3331,            4762,            2956,            4293,            3757,            1216,            3853,            1661,            2860,            2183,            2319,            2366,            3262,            3158,            2907,            859,            3036,            2722,            2273,            3573,            3079,            3692,            1778,            2479,            4833,            574,            3387,            2882,            3473,            4925,            0,            4887,            ],        [            3034,            3090,            3816,            5072,            288,            389,            4466,            4015,            13827,            3453,            1967,            5304,            1547,            4312,            1180,            4462,            493,            1841,            5468,            880,            1311,            2490,            964,            1335,            1816,            3281,            2129,            2709,            5840,            2502,            2247,            2343,            326,            3231,            1222,            1134,            5261,            1230,            2809,            1593,            3137,            2625,            1908,            1439,            1373,            1377,            4904,            2048,            1734,            2628,            3115,            3196,            2704,            2612,            2518,            397,            4619,            1504,            2424,            855,            344,            4045,            0,            ],        ], 'starts': [33, 2, 7, 63], 'ends': [33, 2, 7, 63]},    'NW-8': {'distance_matrix': [        [            0,            1588,            2593,            1553,            1867,            818,            2603,            1416,            3280,            1545,            1188,            2536,            766,            1252,            315,            2470,            1417,            2698,            1369,            1863,            2476,            2154,            2632,            4183,            2516,            284,            1630,            2667,            2901,            1408,            2856,            2597,            2799,            2347,            3346,            3260,            2110,            3294,            3165,            3238,            3144,            2043,            2770,            4598,            2973,            2644,            3599,            2547,            5123,            2689,            2374,            2044,            1612,            2840,            1983,            2334,            1072,            1031,            515,            1735,            3179,            2556,            3482,            1354,            3111,            2911,            2042,            2264,            2020,            366,            1885,            2071,            1838,            2452,            3073,            2207,            1758,            2103,            2654,            1901,            2070,            2181,            1546,            4062,            2453,            2567,            2765,            2773,            1533,            1731,            2298,            4373,            2358,            2295,            53403,            1351,            1988,            1434,            2005,            3481,            2643,            655,            2713,            2773,            2471,            2524,            2546,            3566,            1538,            2534,            1352,            ],        [            1798,            0,            1512,            1602,            3213,            1695,            1341,            1465,            1712,            772,            400,            883,            1814,            2598,            1869,            817,            2048,            2382,            2715,            1547,            2160,            1286,            979,            2530,            863,            1401,            518,            1586,            1248,            2773,            3372,            3474,            3676,            3343,            4786,            4256,            3034,            4290,            4322,            3970,            4140,            3123,            2527,            4989,            2344,            3640,            5799,            3912,            5138,            4035,            3720,            3390,            2958,            4046,            330,            3680,            1121,            685,            1632,            1419,            4175,            2785,            4214,            2700,            2868,            4117,            3038,            3629,            3366,            1719,            1569,            3436,            3203,            3534,            2757,            1891,            3104,            3586,            3736,            3247,            3435,            1865,            1595,            5408,            2137,            3486,            1773,            2457,            2457,            3077,            3294,            4764,            3354,            3501,            53087,            2710,            3353,            2780,            352,            3662,            3989,            2008,            1060,            1120,            2155,            2208,            3892,            1998,            1222,            3880,            1035,            ],        [            2017,            1475,            0,            2649,            3432,            1914,            824,            2512,            2856,            2641,            1075,            1882,            3142,            2817,            2088,            1717,            3095,            1630,            2934,            795,            1408,            3250,            1978,            2494,            1862,            1628,            1739,            468,            1569,            2992,            4516,            4370,            4572,            4120,            5119,            5306,            3883,            5340,            6595,            5011,            6405,            3816,            3671,            6163,            3488,            4417,            6715,            4313,            6688,            4254,            3939,            3609,            3177,            7444,            1805,            3899,            2168,            912,            1851,            667,            6507,            3929,            5255,            2919,            4012,            4684,            3815,            3287,            3585,            1951,            817,            3655,            3099,            2782,            2005,            1139,            3323,            2834,            2984,            3466,            3654,            1113,            2642,            5627,            1385,            2734,            1120,            1705,            3211,            3296,            4071,            5938,            4131,            4645,            52335,            2929,            3011,            2999,            1827,            4806,            4208,            2240,            1381,            1583,            1403,            1456,            4111,            3142,            454,            4099,            1254,            ],        [            1479,            1895,            3489,            0,            1094,            1079,            3318,            481,            2147,            1565,            2295,            2420,            1232,            1411,            1059,            2416,            446,            5897,            2396,            5062,            5675,            1021,            2386,            3597,            2400,            1485,            1571,            3563,            2847,            2104,            2253,            1994,            2196,            1744,            2743,            2657,            1507,            2691,            2562,            2635,            2541,            1440,            2167,            2445,            2370,            2041,            2996,            3243,            3560,            2080,            1765,            1796,            1364,            2237,            1924,            1019,            825,            2232,            1716,            2936,            2623,            1953,            2879,            2190,            2508,            2308,            1439,            2960,            2127,            1496,            3086,            2767,            2534,            3148,            3787,            4525,            2120,            2799,            3350,            1653,            2766,            4920,            471,            3381,            5652,            3263,            3745,            4018,            562,            1002,            1695,            2670,            1755,            1692,            56602,            2381,            2684,            2110,            1946,            2878,            1863,            1785,            2659,            2719,            5670,            3725,            1937,            1114,            2739,            1925,            2551,            ],        [            2269,            2404,            3998,            1004,            0,            2163,            3827,            1772,            2656,            2074,            2804,            2929,            1550,            1225,            1094,            2925,            1037,            5449,            1948,            4614,            5227,            1530,            2895,            4106,            2909,            2337,            2080,            4072,            3356,            1656,            2476,            2217,            2419,            1967,            2966,            2880,            1730,            2914,            2785,            2858,            2764,            1663,            2390,            2668,            3593,            2264,            3219,            2795,            4127,            1765,            1450,            1348,            916,            2460,            2433,            1166,            1886,            3054,            2538,            3758,            2846,            2176,            3102,            1742,            2731,            2531,            1662,            2512,            1934,            2203,            3908,            2319,            2086,            2700,            3339,            4077,            1672,            2351,            2902,            1205,            2318,            4472,            997,            3066,            5204,            2815,            4254,            3570,            1153,            356,            1918,            3377,            1978,            1915,            56154,            1933,            2236,            1662,            2455,            3101,            1475,            1690,            3168,            3228,            5222,            4547,            1622,            2942,            3561,            1610,            3375,            ],        [            400,            1259,            2361,            735,            2218,            0,            2190,            598,            2462,            727,            1310,            1718,            947,            1126,            715,            1652,            1181,            2820,            1720,            1985,            2598,            1336,            1814,            3365,            1698,            406,            812,            2435,            2083,            1766,            2985,            2726,            2928,            2476,            3475,            3389,            2239,            3423,            3294,            3367,            3273,            2172,            2899,            3177,            3179,            2773,            3728,            2905,            5474,            3040,            2725,            2395,            1963,            2969,            1165,            1709,            254,            1153,            637,            1857,            3355,            2685,            3611,            1705,            3240,            3040,            2171,            2622,            2371,            724,            2007,            2429,            2196,            2810,            3195,            2329,            2109,            2461,            3012,            2252,            2428,            2303,            728,            4413,            2575,            2925,            2887,            2895,            1297,            1371,            2427,            4724,            2487,            2424,            53525,            1709,            2346,            1785,            1187,            3610,            2994,            1013,            1895,            1955,            2593,            2646,            2897,            2748,            1660,            2885,            1472,            ],        [            2188,            1646,            171,            2820,            3603,            2085,            0,            2683,            2233,            1785,            1246,            1259,            3313,            2988,            2259,            1094,            3266,            1801,            3105,            966,            1579,            2299,            1355,            1871,            1239,            1799,            1531,            245,            946,            3163,            3893,            3995,            4197,            4291,            5307,            5477,            3555,            5511,            4843,            4491,            5088,            3644,            3048,            6334,            2865,            4588,            6886,            4484,            6859,            4425,            4110,            3780,            3348,            7615,            1343,            4070,            2339,            1083,            2022,            838,            6678,            3306,            4735,            3090,            3389,            4855,            3986,            3458,            3756,            2122,            988,            3826,            3270,            2953,            2176,            1310,            3494,            3005,            3155,            3637,            3825,            1284,            2813,            5798,            1556,            2905,            497,            1876,            2978,            3467,            4242,            6109,            4302,            4022,            52506,            3100,            3182,            3170,            1365,            4183,            4379,            2411,            758,            960,            1574,            989,            4282,            2519,            625,            4270,            1425,            ],        [            998,            1455,            2557,            137,            1231,            598,            2386,            0,            1806,            1059,            1908,            2079,            1127,            1306,            946,            2075,            583,            3418,            2318,            2583,            3196,            680,            2045,            3256,            2059,            1004,            1008,            2631,            2506,            1963,            2390,            2131,            2333,            1881,            2880,            2794,            1644,            2828,            2699,            2772,            2678,            1577,            2304,            2582,            2743,            2178,            3133,            3102,            3697,            2217,            1902,            1933,            1501,            2374,            1361,            1156,            344,            1751,            1235,            2455,            2760,            2090,            3016,            2303,            2645,            2445,            1576,            2819,            2264,            1322,            2605,            2626,            2393,            3007,            3793,            2927,            1979,            2658,            3209,            1790,            2625,            2901,            130,            3518,            3173,            3122,            3485,            3493,            699,            1139,            1832,            2807,            1892,            1829,            54123,            2307,            2543,            2383,            1383,            3015,            2000,            1611,            2318,            2378,            3191,            3244,            2074,            2092,            2258,            2062,            2070,            ],        [            1507,            1421,            2540,            456,            1541,            1107,            2369,            509,            0,            1568,            1821,            1471,            2558,            2233,            1455,            1467,            902,            6457,            2956,            5622,            6235,            1049,            1629,            3625,            1513,            1513,            1517,            2614,            1898,            2664,            2709,            2450,            2652,            2200,            3199,            3113,            1963,            3147,            4796,            3091,            2997,            1896,            2623,            4364,            2512,            2497,            4916,            3803,            4889,            2527,            2212,            2356,            1924,            5645,            1327,            1522,            853,            2260,            1744,            2964,            3079,            2409,            3335,            2750,            2964,            2764,            1895,            3520,            2574,            1831,            3114,            3327,            3094,            3708,            4347,            5085,            2680,            3359,            3910,            2213,            3326,            5480,            499,            3828,            6212,            3823,            2801,            4578,            1018,            1897,            2151,            4139,            2211,            2148,            57162,            2941,            3244,            2670,            1349,            3334,            2310,            2698,            1710,            1770,            6230,            3753,            2384,            286,            2767,            2372,            2579,            ],        [            913,            777,            1879,            830,            1783,            513,            1708,            693,            1980,            0,            1177,            1236,            1042,            1221,            861,            1170,            1276,            3333,            2233,            2498,            3111,            854,            1332,            2883,            1216,            919,            330,            1953,            1601,            1878,            3725,            2821,            3023,            2571,            3570,            3484,            2334,            3518,            3389,            3462,            3368,            2267,            2880,            3272,            2697,            2868,            3823,            3017,            5187,            2825,            2510,            2180,            1748,            4399,            683,            1804,            349,            1462,            1150,            2370,            3450,            3138,            3706,            2218,            3221,            4470,            2266,            2734,            2156,            1237,            2520,            2541,            2308,            2922,            3708,            2842,            1894,            2573,            3124,            2037,            2540,            2816,            823,            4126,            3088,            3037,            2140,            3408,            1392,            1466,            2522,            3497,            2582,            3854,            54038,            2222,            2458,            1884,            705,            3705,            2559,            1526,            1413,            1473,            3106,            3159,            2682,            2266,            2173,            2670,            1985,            ],        [            1398,            400,            1912,            2030,            2813,            1295,            1741,            1893,            2112,            1172,            0,            1283,            2523,            2198,            1469,            1217,            2476,            1982,            2315,            1147,            1760,            1686,            1379,            2930,            1263,            1001,            664,            1986,            1648,            2373,            3772,            3874,            4076,            3501,            4500,            4414,            3434,            4448,            4722,            4370,            4298,            3197,            2927,            5544,            2744,            3798,            6096,            3512,            6069,            3635,            3320,            2990,            2558,            6825,            730,            3280,            1549,            285,            1232,            1019,            4380,            3185,            4614,            2300,            3268,            4065,            3196,            3229,            2966,            1319,            1169,            3036,            2803,            3134,            2357,            1491,            2704,            3186,            3336,            2847,            3035,            1465,            2023,            5008,            1737,            3086,            2049,            2057,            2592,            2677,            3452,            5319,            3512,            3901,            52687,            2310,            2953,            2380,            752,            4062,            3589,            1608,            1460,            1520,            1755,            1808,            3492,            2398,            822,            3480,            635,            ],        [            1819,            588,            1674,            1930,            3015,            1419,            1503,            1599,            1474,            906,            988,            0,            1948,            2127,            1767,            601,            2159,            2970,            3303,            2135,            2748,            1420,            763,            2314,            647,            1825,            652,            1748,            1032,            2784,            3134,            3236,            3438,            3477,            4548,            4436,            2796,            4424,            4084,            3732,            4274,            2885,            2289,            4751,            2106,            3774,            5561,            3923,            4900,            4001,            3686,            3086,            2654,            3808,            494,            2996,            1255,            1273,            2056,            2007,            4356,            2547,            3976,            3288,            2630,            3879,            3070,            3640,            3062,            2143,            2157,            3447,            3214,            4122,            3345,            2479,            2800,            3479,            4324,            2943,            3446,            2453,            1729,            4696,            2725,            4074,            1935,            3045,            2219,            2372,            3428,            4526,            3488,            3263,            53675,            3298,            3364,            3368,            516,            3424,            3784,            2432,            844,            904,            2743,            2796,            3858,            1760,            1810,            3846,            1623,            ],        [            619,            1603,            2705,            1079,            1292,            746,            2534,            942,            2806,            1071,            1787,            2062,            0,            510,            304,            1996,            1100,            3297,            1459,            2462,            3075,            1680,            2158,            3709,            2042,            913,            1156,            2779,            2427,            1167,            2539,            2280,            2482,            2030,            3029,            2943,            1793,            2977,            2848,            2921,            2827,            1726,            2453,            2731,            2656,            2327,            3282,            2306,            4476,            2114,            1799,            1469,            1037,            2523,            1509,            1263,            598,            1630,            1114,            2334,            2909,            2239,            3165,            1253,            2794,            2594,            1725,            2023,            1445,            741,            2484,            1830,            1597,            2211,            2850,            2806,            1183,            1862,            2413,            1326,            1829,            2780,            1060,            3415,            3052,            2326,            3364,            3081,            1216,            1156,            1981,            3726,            2041,            1978,            54002,            1444,            1747,            1173,            1531,            3164,            2068,            1030,            2239,            2299,            3070,            3123,            1971,            3092,            2137,            1959,            1951,            ],        [            1270,            2612,            3617,            1524,            782,            1618,            4609,            1527,            3438,            2345,            2212,            3711,            551,            0,            1341,            3707,            1764,            4450,            949,            3615,            4228,            2312,            3677,            4888,            3691,            1338,            2430,            3691,            4138,            657,            2996,            2737,            2939,            2487,            3486,            3400,            2250,            3434,            3873,            3378,            3284,            2183,            2910,            3441,            3113,            2784,            3993,            1796,            3966,            1604,            1289,            959,            527,            4722,            2783,            1249,            1872,            2055,            1539,            2759,            3366,            2696,            3622,            743,            3251,            4180,            2182,            1513,            935,            1204,            2909,            1320,            1087,            1701,            2340,            3078,            673,            1352,            1903,            816,            1319,            3473,            1517,            2905,            4205,            1816,            3789,            2571,            1880,            646,            2438,            3216,            2498,            2435,            55155,            934,            1237,            663,            2805,            3621,            1558,            691,            3950,            4010,            4223,            3548,            1461,            3724,            2562,            1449,            2376,            ],        [            315,            1883,            2888,            1624,            1938,            889,            2898,            1487,            3351,            1616,            1483,            2607,            451,            682,            0,            2541,            1102,            2993,            1440,            2158,            2771,            2225,            2703,            4254,            2587,            609,            1701,            2962,            2972,            1479,            2541,            2282,            2484,            2032,            3031,            2945,            1795,            2979,            2850,            2923,            2829,            1728,            2455,            2733,            2658,            2329,            3284,            2618,            5194,            2760,            2445,            2115,            1683,            2525,            2054,            1265,            1143,            1326,            810,            2030,            2911,            2241,            3167,            1425,            2796,            2596,            1727,            2335,            2091,            437,            2180,            2142,            1909,            2523,            3162,            2502,            1829,            2174,            2725,            1972,            2141,            2476,            1062,            4133,            2748,            2638,            3060,            3068,            1218,            927,            1983,            4444,            2043,            1980,            53698,            1422,            2059,            1505,            2076,            3166,            2714,            726,            2784,            2844,            2766,            2819,            2617,            3709,            1833,            2605,            1647,            ],        [            2159,            928,            1073,            1770,            2855,            1759,            902,            1939,            1314,            1246,            1328,            340,            2288,            2467,            2107,            0,            1999,            2703,            4007,            1868,            2481,            1760,            436,            1713,            320,            2165,            992,            1147,            431,            3124,            2974,            3076,            3278,            3514,            4388,            4276,            2636,            4461,            3924,            3572,            4833,            2725,            2129,            4591,            1946,            3811,            5401,            5386,            4740,            3841,            3526,            3670,            3238,            3648,            834,            2836,            1595,            1613,            2396,            1740,            4935,            2387,            3816,            3992,            2470,            3719,            2910,            4360,            3888,            2483,            1890,            3787,            3554,            3855,            3078,            2212,            3140,            3907,            4057,            3527,            3786,            2186,            1813,            4536,            2458,            3807,            1334,            2778,            2059,            3211,            3465,            4366,            3525,            3103,            52711,            4002,            4084,            4072,            856,            3264,            3624,            2772,            243,            303,            2476,            1826,            3698,            1600,            1527,            3686,            1963,            ],        [            1594,            2010,            2995,            482,            1317,            1194,            2824,            596,            1249,            1680,            2410,            1926,            1348,            1527,            1175,            1922,            0,            4014,            2512,            3179,            3792,            1136,            2501,            3712,            1968,            1600,            1686,            3069,            2353,            2220,            2159,            1900,            2102,            1650,            2649,            2563,            1413,            2597,            2468,            2541,            2447,            1346,            2073,            2351,            2276,            1947,            2902,            3359,            3466,            1545,            1988,            1912,            1480,            2143,            2039,            573,            940,            2347,            1831,            3051,            2529,            1859,            2785,            2306,            2414,            2214,            1345,            3076,            2350,            1918,            3201,            2883,            2650,            3264,            3903,            4641,            2236,            2915,            3466,            1769,            2882,            5036,            586,            2886,            3769,            3379,            3256,            4134,            116,            1118,            1601,            2576,            1661,            1598,            54885,            2497,            2800,            2226,            2061,            2784,            1388,            2207,            2165,            2225,            3787,            3840,            1684,            668,            2854,            1672,            2666,            ],        [            3215,            2673,            2154,            3847,            5118,            3112,            2164,            3710,            4385,            3839,            2273,            3222,            4828,            4503,            3286,            3057,            6100,            0,            4746,            1091,            222,            4448,            3318,            3871,            3202,            2826,            2937,            2228,            2909,            4454,            7332,            7073,            7275,            6823,            7649,            7736,            6586,            7770,            8117,            7714,            7927,            6519,            5200,            7685,            5017,            7120,            8237,            4609,            8210,            5848,            5533,            5295,            4863,            8966,            3003,            5585,            3366,            2110,            3049,            1333,            7702,            5458,            7958,            4885,            5541,            8424,            6518,            3583,            5077,            3149,            1483,            3845,            3395,            3078,            2301,            1435,            4815,            3130,            3280,            5152,            3866,            1409,            3840,            7149,            245,            3030,            2363,            2001,            6216,            4982,            6774,            7460,            6834,            6174,            51399,            4731,            3307,            4805,            3025,            7957,            5894,            3438,            2721,            2923,            227,            2122,            5705,            4671,            1652,            5693,            1735,            ],        [            819,            2161,            3166,            1902,            1133,            1167,            3176,            1765,            3789,            1894,            1761,            2885,            843,            518,            890,            2819,            2115,            3271,            0,            2436,            3049,            2663,            2981,            5239,            2865,            887,            1979,            3240,            3250,            687,            3347,            3088,            3290,            2838,            3837,            3751,            2601,            3785,            4224,            3729,            4034,            2534,            3261,            3792,            3464,            3135,            4344,            1826,            4317,            1955,            1640,            1310,            878,            5073,            2332,            1600,            1421,            1604,            1088,            2308,            3717,            3047,            3973,            611,            3602,            4531,            2533,            1543,            1286,            753,            2458,            1350,            1117,            1731,            2370,            2780,            1024,            1382,            1933,            1167,            1349,            2754,            1895,            3256,            3026,            1846,            3338,            2601,            2231,            997,            2789,            3567,            2849,            2786,            50826,            630,            1267,            691,            2354,            3972,            1909,            612,            3062,            3122,            3044,            3097,            1812,            4075,            2111,            1800,            1925,            ],        [            3370,            2828,            2309,            4002,            4302,            3267,            2319,            3865,            4540,            3994,            2428,            3377,            4012,            3687,            3441,            3212,            5284,            1025,            3930,            0,            803,            4603,            3473,            4202,            3357,            2981,            3092,            2383,            3064,            3638,            6516,            6257,            6459,            6007,            6833,            6920,            5770,            6954,            7301,            6898,            7111,            5703,            5355,            6869,            5172,            6304,            7421,            3793,            7394,            5032,            4717,            4479,            4047,            8150,            3158,            4769,            3521,            2265,            3204,            1488,            6886,            6216,            7142,            4069,            5696,            7608,            5702,            2767,            4261,            3304,            1638,            3029,            2579,            2262,            1485,            1590,            3999,            2314,            2464,            4336,            3050,            1564,            3995,            6333,            780,            2214,            2518,            2156,            5400,            4166,            5958,            6644,            6018,            6329,            51730,            3915,            2491,            3989,            3180,            7141,            5078,            3593,            2876,            3078,            798,            2277,            4889,            4251,            1807,            4877,            1890,            ],        [            2993,            2451,            1932,            3625,            4896,            2890,            1942,            3488,            3588,            3617,            2051,            3000,            4606,            4281,            3064,            2835,            5878,            222,            4524,            869,            0,            4226,            3096,            3649,            2980,            2604,            2715,            2006,            2687,            4232,            7110,            6851,            7053,            6601,            7427,            7514,            6364,            7548,            7895,            7492,            7705,            6297,            4978,            7463,            4795,            6898,            8015,            4387,            7988,            5626,            5311,            5073,            4641,            8744,            2781,            5363,            3144,            1888,            2827,            1111,            7480,            5236,            7736,            4663,            5319,            8202,            6296,            3361,            4855,            2927,            1261,            3623,            3173,            2856,            2079,            1213,            4593,            2908,            3058,            4930,            3644,            1187,            3618,            6927,            23,            2808,            2141,            1779,            5994,            4760,            6552,            7238,            6612,            5952,            51177,            4509,            3085,            4583,            2803,            7735,            5672,            3216,            2499,            2701,            5,            1900,            5483,            3874,            1430,            5471,            1513,            ],        [            2633,            1987,            2468,            1582,            2667,            2233,            2297,            1635,            1126,            2305,            2387,            1399,            3684,            3359,            2581,            1395,            2028,            4098,            4082,            3263,            3876,            0,            1365,            2576,            1379,            2639,            2051,            2542,            1826,            3790,            4033,            3576,            3778,            3326,            4325,            4239,            3089,            4273,            5922,            4217,            5732,            3022,            3188,            5490,            3005,            3623,            6042,            4929,            6015,            3653,            3338,            3482,            3050,            4707,            1893,            2648,            1979,            2672,            2870,            3135,            4158,            3446,            4461,            3876,            3529,            4778,            3021,            4646,            3700,            2957,            3285,            4453,            4220,            4834,            5473,            3607,            3806,            4485,            5036,            3339,            4452,            3581,            1625,            4954,            3853,            4949,            2724,            5704,            2144,            3023,            3277,            5265,            3337,            4162,            53749,            4067,            4370,            3796,            1915,            4460,            3436,            3824,            1638,            1698,            3871,            3216,            3510,            1412,            2922,            3498,            3022,            ],        [            2329,            1098,            1568,            2772,            3857,            1929,            1397,            2825,            2316,            1416,            1498,            510,            2458,            2637,            2277,            506,            3218,            3198,            3813,            2363,            2976,            1930,            0,            1562,            490,            2335,            1162,            1642,            926,            4980,            3144,            3246,            3448,            4516,            4558,            4446,            2806,            5463,            4094,            3742,            5003,            2895,            2299,            4761,            2116,            4813,            5571,            5881,            4910,            4843,            4528,            4672,            4240,            3818,            1004,            3838,            1765,            1783,            2566,            2235,            5348,            2557,            3986,            3798,            2640,            3889,            3080,            5836,            4890,            2653,            2385,            5643,            5410,            4350,            3573,            2707,            4996,            4402,            4552,            4529,            5642,            2681,            2815,            6144,            2953,            4302,            1508,            3273,            2229,            4213,            4467,            4536,            4527,            3273,            52885,            3808,            5560,            4986,            1026,            3434,            4626,            2942,            738,            798,            2971,            2000,            4700,            2602,            2022,            4688,            2133,            ],        [            3371,            1653,            1354,            2919,            4004,            2528,            1183,            2708,            2463,            2015,            2053,            1489,            3057,            3236,            3442,            1324,            3148,            2984,            4288,            2149,            2762,            2529,            1585,            0,            1469,            2982,            1761,            1428,            1176,            4346,            4123,            4225,            4427,            4586,            5537,            6040,            3785,            5610,            5073,            4721,            5383,            3874,            3278,            6827,            3095,            4883,            5915,            5667,            7352,            4990,            4675,            4963,            4531,            8108,            1573,            3985,            2364,            2266,            3205,            1627,            5542,            3536,            4965,            4273,            3619,            5150,            4059,            4641,            4939,            3305,            2171,            5009,            4453,            4136,            3359,            2493,            4677,            4188,            4338,            4820,            5008,            2467,            2962,            6291,            2739,            4088,            1113,            3059,            3208,            3481,            4537,            6602,            4597,            4252,            52148,            4283,            4365,            4353,            1595,            4413,            4773,            3594,            988,            1190,            2757,            1605,            4847,            2749,            1808,            4835,            2608,            ],        [            2121,            890,            1371,            1450,            2535,            1721,            1200,            1503,            994,            1208,            1290,            302,            2250,            3227,            2069,            298,            1679,            3001,            3605,            2166,            2779,            1722,            460,            2011,            0,            2127,            954,            1445,            729,            3658,            2654,            2756,            2958,            3194,            4068,            3956,            2316,            4141,            3604,            3252,            4513,            2405,            1809,            4271,            1626,            3491,            5081,            4797,            4420,            3521,            3206,            3350,            2918,            3328,            796,            2516,            1557,            1575,            2358,            2038,            4615,            2067,            3496,            3744,            2150,            3399,            2590,            4514,            3568,            2445,            2188,            4321,            4088,            4153,            3376,            2510,            3674,            4353,            4355,            3207,            4320,            2484,            1493,            4216,            2756,            4817,            1632,            3076,            1739,            2891,            3145,            4046,            3205,            2783,            53009,            3935,            4238,            3664,            818,            2944,            3304,            2734,            541,            601,            2774,            2124,            3378,            1280,            1825,            3366,            1925,            ],        [            384,            1764,            2769,            1505,            1806,            770,            2779,            1368,            3232,            1497,            1364,            2488,            1516,            1191,            455,            2422,            1557,            2874,            1321,            2039,            2652,            2106,            2584,            4135,            2468,            0,            1582,            2843,            2853,            1360,            2996,            2737,            2939,            2487,            3486,            3400,            2250,            3434,            4897,            3378,            4707,            2183,            2910,            4465,            3949,            2784,            5017,            2499,            4990,            2628,            2313,            1983,            1551,            5746,            1935,            2273,            1024,            1207,            691,            1911,            3319,            2696,            3622,            1284,            3251,            3051,            2182,            2216,            1959,            318,            2061,            2023,            1790,            2404,            3043,            2383,            1697,            2055,            2606,            1840,            2022,            2357,            1498,            3929,            2629,            2519,            2941,            2949,            1673,            1670,            2438,            4240,            2498,            2435,            53579,            1303,            1940,            1364,            1957,            3621,            2582,            607,            2665,            2725,            2647,            2700,            2485,            3518,            1714,            2473,            1528,            ],        [            1288,            447,            1549,            1205,            2701,            888,            1378,            1068,            1160,            375,            847,            906,            1417,            1596,            1236,            840,            1651,            2829,            2608,            1994,            2607,            889,            1002,            2553,            886,            1294,            0,            1623,            1271,            2253,            3395,            3497,            3699,            2946,            3945,            3859,            3057,            3893,            4345,            3993,            3743,            3146,            2550,            5012,            2367,            3243,            5822,            3392,            5161,            3687,            3372,            2555,            2123,            4069,            353,            2682,            724,            1132,            1525,            1866,            3778,            2808,            4237,            2593,            2891,            4140,            2641,            3109,            2531,            1612,            2016,            2916,            2683,            3981,            3204,            2338,            2269,            2948,            4183,            2412,            2915,            2312,            1198,            4988,            2584,            3412,            1810,            2904,            1923,            1841,            2897,            4787,            2957,            3524,            53534,            2597,            2833,            2259,            375,            3685,            3470,            1901,            1083,            1143,            2602,            2655,            3544,            1446,            1669,            3532,            1482,            ],        [            2483,            1698,            466,            3115,            3898,            2380,            476,            2978,            2508,            2060,            1541,            1534,            3608,            3283,            2554,            1369,            3193,            2096,            3400,            1261,            1874,            2574,            1630,            2146,            1514,            2094,            1806,            0,            1221,            3458,            4168,            4270,            4472,            4586,            5582,            5772,            3830,            5638,            5118,            4766,            5383,            3919,            3323,            6629,            3140,            4883,            7181,            4779,            7154,            4720,            4405,            4075,            3643,            7910,            1618,            4365,            2634,            1378,            2317,            1058,            6973,            3581,            5010,            3385,            3664,            5150,            4104,            3753,            4051,            2417,            1283,            4121,            3565,            3248,            2471,            1605,            3789,            3300,            3450,            3932,            4120,            1579,            3108,            6093,            1851,            3200,            772,            2171,            3253,            3762,            4537,            6404,            4597,            4297,            52801,            3395,            3477,            3465,            1640,            4458,            4674,            2706,            1033,            1235,            1869,            1264,            4577,            2794,            920,            4565,            1720,            ],        [            2659,            941,            642,            2231,            3316,            2556,            471,            1996,            1775,            1303,            1341,            801,            3784,            3459,            2730,            636,            2460,            2272,            3576,            1437,            2050,            1817,            897,            1413,            781,            2270,            1049,            716,            0,            3634,            3435,            3537,            3739,            3874,            4849,            4737,            3097,            4922,            4385,            4033,            5294,            3186,            2590,            5052,            2407,            4171,            5862,            4955,            5201,            4302,            3987,            4251,            3819,            4109,            861,            3297,            1652,            1554,            2493,            1309,            5396,            2848,            4277,            3561,            2931,            4180,            3371,            3929,            4227,            2593,            1459,            4297,            3741,            3424,            2647,            1781,            3965,            3476,            3626,            4108,            4296,            1755,            2126,            4997,            2027,            3376,            903,            2347,            2520,            3938,            3825,            4827,            3885,            3564,            52280,            3571,            3653,            3641,            883,            3725,            4085,            2882,            300,            502,            2045,            1395,            4159,            2061,            1096,            4147,            1896,            ],        [            2365,            3707,            4712,            2678,            1936,            2713,            5763,            2681,            4592,            3440,            3307,            4865,            1646,            1321,            2436,            4861,            2918,            3793,            1564,            2958,            3571,            3466,            4831,            6042,            4845,            2433,            3525,            4786,            5292,            0,            4150,            3891,            4093,            3641,            4640,            4554,            3404,            4588,            4935,            4532,            4745,            3337,            4064,            4503,            4267,            3938,            5055,            1139,            5028,            2666,            2351,            2113,            1681,            5784,            3878,            2403,            2967,            3150,            2634,            3854,            4847,            3850,            4776,            1703,            4405,            5242,            3336,            856,            1895,            2299,            4004,            663,            430,            1044,            1683,            2421,            1633,            695,            1246,            1970,            662,            2816,            2671,            3967,            3548,            1159,            4884,            1914,            3034,            1800,            3592,            4278,            3652,            3589,            50139,            1549,            580,            1623,            3900,            4775,            2712,            1786,            5104,            5164,            3566,            4643,            2523,            4878,            3657,            2511,            3471,            ],        [            2807,            3422,            4030,            2419,            2527,            3020,            3859,            2422,            3320,            3740,            3822,            2961,            2665,            2844,            2492,            2957,            2244,            7330,            3829,            6495,            7108,            2962,            3119,            4514,            3003,            3101,            3486,            4104,            3388,            3537,            0,            389,            591,            1337,            1533,            1509,            1055,            2023,            1777,            805,            2134,            852,            921,            2137,            1516,            1197,            2888,            4676,            3793,            3513,            3198,            3229,            2797,            2376,            3328,            2200,            2766,            4107,            3302,            4697,            2104,            884,            1049,            3623,            1032,            2654,            852,            4393,            3560,            2929,            4847,            4200,            3967,            4581,            5220,            5958,            3553,            4232,            4783,            3086,            4199,            6353,            2412,            2474,            7085,            4696,            4291,            5451,            2360,            2435,            1332,            2304,            1293,            2038,            56224,            3814,            4117,            3543,            3350,            497,            2960,            3218,            3200,            3260,            7103,            4783,            3370,            3606,            4484,            3358,            4457,            ],        [            2601,            3630,            4766,            2213,            2321,            2814,            4595,            2216,            3882,            3300,            3769,            3697,            2459,            2638,            2286,            3693,            2038,            7124,            3623,            6289,            6902,            2756,            4121,            5250,            3739,            2895,            3306,            4840,            4124,            3331,            1746,            0,            202,            948,            1144,            1351,            1027,            1634,            1388,            1123,            1745,            960,            1657,            1748,            1890,            808,            2389,            4470,            2665,            3307,            2992,            3023,            2591,            1987,            3659,            1994,            2560,            3612,            3096,            4316,            1644,            1620,            1367,            3417,            1768,            2058,            892,            4187,            3354,            2723,            4466,            3994,            3761,            4375,            5014,            5752,            3347,            4026,            4577,            2880,            3993,            6147,            2206,            2085,            6879,            4490,            5027,            5245,            2154,            2229,            943,            1915,            904,            2254,            57829,            3608,            3911,            3337,            3681,            1400,            2754,            3012,            3936,            3996,            6897,            5105,            3164,            2706,            4119,            3152,            3933,            ],        [            2399,            3428,            4564,            2011,            2119,            2612,            4393,            2014,            3680,            3098,            3567,            3495,            2257,            2436,            2084,            3491,            1836,            6922,            3421,            6087,            6700,            2554,            3919,            5048,            3537,            2693,            3104,            4638,            3922,            3129,            1544,            847,            0,            746,            942,            1149,            825,            1432,            1186,            921,            1543,            758,            1455,            1546,            1688,            606,            2187,            4268,            2463,            3105,            2790,            2821,            2389,            1785,            3457,            1792,            2358,            3410,            2894,            4114,            1442,            1418,            1165,            3215,            1566,            1856,            690,            3985,            3152,            2521,            4264,            3792,            3559,            4173,            4812,            5550,            3145,            3824,            4375,            2678,            3791,            5945,            2004,            1883,            6677,            4288,            4825,            5043,            1952,            2027,            741,            1713,            702,            2052,            57627,            3406,            3709,            3135,            3479,            1198,            2552,            2810,            3734,            3794,            6695,            4903,            2962,            2504,            3917,            2950,            3731,            ],        [            2836,            3865,            5459,            2448,            2556,            3049,            5288,            2451,            4117,            3535,            4004,            4390,            2694,            2873,            2521,            4386,            2273,            7359,            3858,            6524,            7137,            2991,            4356,            5567,            4370,            3130,            3541,            5533,            4817,            3566,            2172,            2101,            2303,            0,            999,            1422,            1759,            947,            818,            2134,            1042,            1692,            2404,            800,            2796,            900,            1577,            4705,            1717,            3542,            2890,            3258,            2826,            1054,            3894,            2229,            2795,            3847,            3331,            4551,            832,            2367,            2378,            3652,            3340,            1125,            1691,            4422,            3419,            2958,            4701,            4229,            3996,            4610,            5249,            5987,            3582,            4261,            4812,            3115,            4228,            6382,            2441,            1137,            7114,            4725,            5715,            5480,            2389,            2464,            798,            967,            652,            1366,            58064,            3843,            4146,            3572,            3916,            1826,            2989,            3247,            4629,            4689,            7132,            5340,            2920,            2941,            4354,            2824,            4168,            ],        [            1822,            2851,            4445,            1434,            1542,            2035,            4274,            1437,            3103,            2521,            2990,            3376,            1680,            1859,            1507,            3372,            1259,            6345,            2844,            5510,            6123,            1977,            3342,            4553,            3356,            2116,            2527,            4519,            3803,            2552,            1521,            1232,            1434,            982,            0,            1895,            745,            1929,            244,            1873,            1551,            678,            1405,            1309,            1608,            1279,            1861,            3691,            1935,            2528,            2213,            2244,            1812,            843,            2880,            1215,            1781,            2833,            2317,            3537,            1653,            1191,            2117,            2638,            1746,            914,            677,            3408,            2575,            1944,            3687,            3215,            2982,            3596,            4235,            4973,            2568,            3247,            3798,            2101,            3214,            5368,            1427,            1254,            6100,            3711,            4701,            4466,            1375,            1450,            933,            1084,            993,            1155,            57050,            2829,            3132,            2558,            2902,            2116,            1975,            2233,            3615,            3675,            6118,            4326,            2385,            1927,            3340,            2373,            3154,            ],        [            3110,            4139,            5733,            2722,            2830,            3323,            5562,            2725,            4391,            3809,            4278,            4664,            2968,            3147,            2795,            4660,            2547,            7633,            4132,            6798,            7411,            3265,            4630,            5841,            4644,            3404,            3815,            5807,            5091,            3840,            2177,            2106,            2308,            959,            1273,            0,            2033,            952,            1092,            2139,            1779,            1966,            2458,            1074,            3070,            954,            1379,            4979,            2284,            3816,            3501,            3532,            3100,            1328,            4168,            2503,            3069,            4121,            3605,            4825,            595,            2479,            2383,            3926,            3345,            1399,            1965,            4696,            3693,            3232,            4975,            4503,            4270,            4884,            5523,            6261,            3856,            4535,            5086,            3389,            4502,            6656,            2715,            1411,            7388,            4999,            5989,            5754,            2663,            2738,            1072,            1241,            706,            1640,            58338,            4117,            4420,            3846,            4190,            1831,            3263,            3521,            4903,            4963,            7406,            5614,            3673,            3215,            4628,            3098,            4442,            ],        [            2283,            3290,            3898,            1895,            2003,            2496,            3727,            1898,            3564,            2982,            3690,            2829,            2141,            2320,            1968,            2825,            1720,            6806,            3305,            5971,            6584,            2438,            2987,            5068,            2871,            2577,            2988,            3972,            3256,            3013,            1747,            883,            1085,            1138,            1553,            1659,            0,            2085,            1347,            1329,            2256,            175,            902,            2014,            1105,            1435,            2824,            4152,            2163,            2989,            2674,            2705,            2273,            1071,            3196,            1676,            2242,            3294,            2778,            3998,            1970,            688,            1573,            3099,            1243,            1142,            328,            3869,            3036,            2405,            4148,            3676,            3443,            4057,            4696,            5434,            3029,            3708,            4259,            2562,            3675,            5829,            1888,            1959,            6561,            4172,            4159,            4927,            1836,            1911,            1089,            1789,            1149,            526,            57511,            3290,            3593,            3019,            3218,            1483,            2436,            2694,            3068,            3128,            6579,            4787,            2846,            2388,            3801,            2834,            3615,            ],        [            2905,            4047,            5528,            2517,            2625,            3118,            5357,            2520,            4186,            3604,            4073,            3586,            2763,            2942,            2590,            3582,            2342,            7428,            3927,            6593,            7206,            3060,            3744,            5496,            3628,            3199,            3610,            4729,            4013,            3635,            1225,            1154,            1356,            887,            1083,            475,            1318,            0,            1327,            1187,            2254,            1251,            2146,            1425,            2897,            238,            1854,            4774,            2759,            3611,            3296,            3327,            2895,            1679,            3953,            2298,            2864,            3916,            3400,            4620,            1070,            2109,            1431,            3721,            2393,            1750,            1183,            4491,            3658,            3027,            4770,            4298,            4065,            4679,            5318,            6056,            3651,            4330,            4881,            3184,            4297,            6451,            2510,            1762,            7183,            4794,            6609,            5549,            2458,            2533,            882,            1592,            524,            1991,            58133,            3912,            4215,            3641,            3975,            879,            3058,            3316,            3825,            3885,            7201,            5409,            3468,            3010,            4423,            3456,            4237,            ],        [            2018,            3047,            4641,            1630,            1738,            2231,            4470,            1633,            3299,            2717,            3186,            3572,            1876,            2055,            1703,            3568,            1455,            6541,            3040,            5706,            6319,            2173,            3538,            4749,            3552,            2312,            2723,            4715,            3999,            2748,            1722,            1463,            1665,            1178,            772,            2091,            941,            2095,            0,            1559,            1307,            874,            1601,            1065,            2341,            1475,            1617,            3887,            1691,            2724,            2409,            2440,            2008,            599,            3076,            1411,            1977,            3029,            2513,            3733,            1409,            1387,            1803,            2834,            1942,            670,            873,            3604,            2771,            2140,            3883,            3411,            3178,            3792,            4431,            5169,            2764,            3443,            3994,            2297,            3410,            5564,            1623,            1010,            6296,            3907,            4897,            4662,            1571,            1646,            571,            840,            1189,            911,            57246,            3025,            3328,            2754,            3098,            1836,            2171,            2429,            3811,            3871,            6314,            4522,            2581,            2123,            3536,            2569,            3350,            ],        [            3136,            4111,            4719,            2748,            2856,            3349,            4548,            2751,            4009,            3835,            4511,            3650,            2994,            3173,            2821,            3646,            2573,            7659,            4158,            6824,            7437,            3291,            3808,            5560,            3692,            3430,            3841,            4793,            4077,            3866,            1289,            1218,            1420,            1118,            1314,            704,            1817,            1656,            1796,            0,            2483,            1750,            2210,            1778,            2961,            485,            2083,            5005,            2988,            3842,            3527,            3558,            3126,            2032,            4017,            2529,            3095,            4147,            3631,            4851,            1299,            2173,            244,            3952,            2457,            2103,            1682,            4722,            3889,            3258,            5356,            4529,            4296,            4910,            5549,            6287,            3882,            4561,            5112,            3415,            4528,            6682,            2741,            2115,            7414,            5025,            6673,            5780,            2689,            2764,            1113,            1945,            771,            2344,            58364,            4143,            4446,            3872,            4039,            943,            3289,            3547,            3889,            3949,            7432,            5995,            3898,            4295,            4654,            3802,            4468,            ],        [            2713,            3742,            5336,            2325,            2433,            2926,            5165,            2328,            3994,            3412,            3881,            4267,            2571,            2750,            2398,            4263,            2150,            7236,            3735,            6401,            7014,            2868,            4233,            5444,            4247,            3007,            3418,            5033,            4694,            3443,            1826,            1567,            1769,            1873,            876,            1605,            1636,            1130,            695,            1663,            0,            1569,            2175,            677,            2673,            1348,            1149,            4582,            2023,            3419,            3104,            3135,            2703,            931,            3771,            2106,            2672,            3724,            3208,            4428,            737,            2082,            1907,            3529,            2286,            1002,            1568,            4299,            3466,            2835,            4578,            4106,            3873,            4487,            5126,            5864,            3459,            4138,            4689,            2992,            4105,            6259,            2318,            1014,            6991,            4602,            5592,            5357,            2266,            2341,            675,            844,            1002,            1243,            57941,            3720,            4023,            3449,            3793,            1940,            2866,            3124,            4506,            4566,            7009,            5217,            2797,            2818,            4231,            2701,            4045,            ],        [            2168,            3115,            3723,            1780,            1888,            2381,            3552,            1783,            3013,            2867,            3515,            2654,            2026,            2205,            1853,            2650,            1605,            6691,            3190,            5856,            6469,            2323,            2812,            4893,            2696,            2462,            2873,            3797,            3081,            2898,            1572,            768,            970,            1023,            1438,            1544,            203,            1970,            2115,            1214,            1820,            0,            727,            1823,            930,            1320,            3592,            4037,            2931,            2874,            2559,            2590,            2158,            1839,            3021,            1561,            2127,            3179,            2663,            3883,            1855,            513,            1458,            2984,            1068,            1910,            213,            3754,            2921,            2290,            4033,            3561,            3328,            3942,            4581,            5319,            2914,            3593,            4144,            2447,            3560,            5714,            1773,            2727,            6446,            4057,            3984,            4812,            1721,            1796,            974,            2557,            1034,            1294,            57396,            3175,            3478,            2904,            3043,            1368,            2321,            2579,            2893,            2953,            6464,            4672,            2731,            2273,            3686,            2719,            3500,            ],        [            2895,            2453,            3061,            2507,            2615,            3284,            2890,            2510,            2351,            2771,            2853,            1992,            2753,            2932,            2580,            1988,            2332,            4691,            3917,            3856,            4469,            3050,            2150,            3902,            2034,            3189,            2517,            3135,            2419,            3625,            845,            947,            1149,            1750,            2091,            2147,            930,            2697,            2734,            1443,            2547,            727,            0,            2550,            904,            1755,            3526,            4764,            3550,            3601,            3286,            3317,            2885,            2458,            2359,            2288,            2854,            3138,            3390,            3728,            2582,            214,            1687,            3711,            341,            2529,            940,            4481,            3648,            3017,            3878,            4288,            4055,            4669,            5308,            4200,            3641,            4320,            4871,            3174,            4287,            4174,            2500,            3346,            4446,            4784,            3322,            5539,            2448,            2523,            1701,            3176,            1761,            1913,            54699,            3902,            4205,            3631,            2381,            1135,            3048,            3306,            2231,            2291,            4464,            3814,            3458,            2637,            3515,            3446,            3488,            ],        [            3109,            4138,            5732,            2721,            3229,            3322,            5561,            2724,            4390,            3808,            6261,            5148,            2967,            3571,            2794,            5144,            2546,            7795,            4500,            6960,            7573,            3264,            4629,            5840,            4643,            5387,            3814,            6291,            5575,            4002,            2255,            2184,            2386,            1261,            1118,            1505,            2032,            1030,            1091,            2217,            242,            1965,            3599,            0,            3613,            1268,            717,            5141,            1115,            2479,            2288,            2847,            3262,            1871,            4167,            2502,            3068,            6104,            5588,            6808,            344,            3385,            2461,            4100,            3423,            1398,            1964,            4858,            2817,            3231,            6958,            4665,            4432,            5046,            5685,            6423,            3155,            4697,            5248,            3086,            4664,            6818,            2714,            535,            7550,            5161,            7838,            5916,            2662,            2737,            917,            225,            1008,            2183,            58500,            4119,            4582,            4180,            4189,            1909,            2919,            5112,            5387,            5447,            7568,            7597,            2318,            3214,            6611,            2222,            6425,            ],        [            2919,            2685,            3293,            3039,            2639,            3516,            3122,            2534,            2583,            3003,            3085,            2224,            2777,            2956,            2604,            2220,            2356,            4923,            3941,            4088,            4701,            3074,            2382,            4463,            2266,            3213,            2749,            3367,            2651,            3649,            1500,            1602,            1804,            1774,            2442,            2802,            690,            2721,            1978,            2098,            2887,            779,            655,            2645,            0,            2071,            3455,            4788,            2794,            3625,            3310,            3341,            2909,            1702,            2591,            2312,            2878,            3370,            3414,            3960,            2989,            441,            2342,            3735,            996,            1773,            964,            4505,            3672,            3041,            4110,            4312,            4079,            4693,            5332,            4432,            3665,            4344,            4895,            3198,            4311,            4406,            2524,            2590,            4678,            4808,            3554,            4998,            2472,            2547,            1725,            2420,            1785,            1157,            54931,            3926,            4229,            3655,            2613,            1790,            3072,            3330,            2463,            2523,            4696,            4046,            3482,            2869,            3747,            4277,            3720,            ],        [            2651,            4005,            5274,            2263,            2371,            2864,            5103,            2266,            3932,            3350,            3819,            3544,            2509,            2688,            2336,            3540,            2088,            7174,            3673,            6339,            6952,            2806,            3702,            5454,            3586,            2945,            3356,            4687,            3971,            3381,            1183,            1112,            1314,            633,            829,            616,            1064,            1016,            1073,            1145,            1429,            997,            1694,            1187,            1927,            0,            1771,            4520,            2104,            3357,            3042,            3073,            2641,            1441,            3709,            2044,            2610,            3662,            3146,            4366,            1026,            1657,            1389,            3467,            2351,            1512,            929,            4237,            3404,            2773,            4516,            4044,            3811,            4425,            5064,            5802,            3397,            4076,            4627,            2930,            4043,            6197,            2256,            1524,            6929,            4540,            6567,            5295,            2204,            2279,            628,            1354,            286,            1753,            57879,            3658,            3961,            3387,            3731,            837,            2804,            3062,            3783,            3843,            6947,            5155,            3214,            2756,            4169,            3202,            3983,            ],        [            3113,            4142,            5736,            2725,            2833,            3326,            5188,            2728,            4394,            3812,            4281,            4774,            2971,            3150,            2798,            4770,            2550,            7636,            4804,            6801,            7414,            3268,            4633,            5844,            4647,            3407,            3818,            5917,            5094,            3843,            2413,            2342,            2544,            1419,            1276,            1663,            2036,            1188,            1095,            2375,            400,            1969,            2575,            304,            3073,            1426,            0,            4982,            1419,            2783,            2592,            3151,            3103,            1331,            4171,            2506,            3072,            4124,            3608,            7112,            502,            2482,            2619,            4404,            3581,            1402,            1968,            4699,            3121,            3235,            7262,            4506,            4273,            4887,            5526,            6264,            3459,            4538,            5089,            3390,            4505,            6659,            2718,            839,            7391,            5002,            5992,            5757,            2666,            2741,            1075,            529,            1166,            1643,            58341,            4423,            4423,            4484,            4193,            2067,            3223,            3524,            4906,            5073,            7409,            7901,            2622,            3218,            4631,            2526,            4445,            ],        [            2641,            3983,            4898,            2954,            2212,            2989,            4908,            2957,            4868,            3716,            3583,            5141,            1922,            1597,            2712,            5137,            3194,            3614,            1840,            2779,            3392,            3742,            5107,            6318,            5121,            2709,            3801,            4972,            5568,            1548,            4456,            4167,            4369,            3917,            4916,            4830,            3680,            4864,            5211,            4808,            5021,            3613,            4340,            4779,            4543,            4214,            5662,            0,            5304,            2942,            2627,            2389,            1957,            6060,            4154,            2679,            3243,            3426,            2910,            4077,            5123,            4126,            5052,            1979,            4681,            5518,            3612,            419,            2171,            2575,            4227,            939,            489,            865,            1504,            2242,            1909,            516,            1067,            2246,            1115,            2637,            2947,            4243,            3369,            980,            5107,            1735,            3310,            2076,            3868,            4554,            3928,            3865,            51401,            1825,            339,            1899,            4176,            5051,            2988,            2062,            5003,            5063,            3387,            4866,            2799,            5154,            3933,            2787,            3747,            ],        [            3070,            4494,            5316,            2682,            2790,            3283,            5522,            2685,            4392,            3769,            4894,            4033,            2928,            3107,            2755,            4029,            2507,            7593,            4458,            6758,            7371,            3225,            4191,            6272,            4075,            3364,            3775,            5176,            4460,            3800,            3329,            2207,            2409,            2230,            1516,            2053,            1762,            1578,            1052,            2765,            790,            2211,            2484,            548,            2498,            1816,            1358,            4939,            0,            2507,            2246,            2805,            3060,            756,            4400,            2463,            3029,            4214,            3565,            4918,            892,            2270,            3009,            4058,            2825,            847,            1925,            4656,            2775,            3325,            4935,            4463,            4230,            4844,            5483,            6221,            3113,            4495,            5046,            3349,            4462,            6616,            2675,            493,            7348,            4959,            5949,            5714,            2623,            2698,            1315,            323,            1556,            1068,            58298,            4077,            4380,            4138,            4422,            2457,            2877,            3614,            4272,            4332,            7366,            5707,            2276,            3175,            4721,            2180,            4535,            ],        [            2563,            3524,            5118,            1863,            1120,            2911,            4947,            2021,            3776,            3194,            3505,            4049,            1844,            1519,            2634,            4045,            1381,            5743,            2242,            4908,            5521,            2650,            4015,            5226,            4029,            2631,            3200,            5192,            4476,            1950,            3137,            2848,            3050,            2598,            3386,            3511,            2361,            3520,            2922,            3489,            2732,            2294,            3021,            2490,            3224,            2895,            3042,            3089,            3015,            0,            619,            1178,            1210,            3771,            3553,            866,            2365,            3348,            2832,            4052,            2834,            2807,            3733,            2036,            3362,            3229,            2293,            2806,            981,            2497,            4202,            2613,            2380,            2994,            3633,            4371,            1966,            2645,            3196,            1499,            2612,            4766,            2011,            1954,            5498,            3109,            5082,            3864,            1497,            1235,            2549,            2265,            2609,            2546,            56448,            2227,            2530,            1956,            3575,            3732,            451,            1984,            4288,            4348,            5516,            4841,            374,            2049,            3855,            362,            3669,            ],        [            2327,            3345,            4939,            1945,            941,            2675,            4768,            2713,            3597,            3015,            3269,            3870,            1608,            1283,            2398,            3866,            1923,            5507,            1707,            4672,            5285,            2471,            3836,            5047,            3850,            2395,            3021,            5013,            4297,            1714,            3447,            3158,            3360,            2908,            3205,            3814,            2671,            3339,            2741,            4526,            2551,            2604,            3331,            2309,            4534,            3205,            2861,            2853,            2834,            733,            0,            559,            974,            3590,            3374,            1408,            2827,            3112,            2596,            3816,            2653,            3117,            4770,            1133,            3672,            3048,            2603,            2570,            362,            2261,            3966,            2377,            2144,            2758,            3397,            4135,            700,            2409,            2960,            798,            2376,            4530,            1938,            1773,            5262,            2873,            4846,            3628,            2039,            1309,            2859,            2084,            2919,            3902,            56212,            1276,            2294,            1053,            3396,            4218,            1173,            1748,            4109,            4169,            5280,            4605,            590,            3883,            3619,            578,            3433,            ],        [            2118,            2786,            4380,            1386,            382,            2466,            4209,            2154,            3038,            2456,            3060,            3311,            1399,            1074,            2189,            3307,            1364,            5298,            1797,            4463,            5076,            1912,            3277,            4488,            3291,            2186,            2462,            4454,            3738,            1505,            2888,            2599,            2801,            2349,            3348,            3262,            2112,            3296,            3717,            3240,            3527,            2045,            2772,            3285,            3975,            2646,            3837,            2644,            3810,            1255,            1133,            0,            765,            4566,            2815,            849,            2268,            2903,            2387,            3607,            3629,            2558,            3484,            1591,            3113,            4024,            2044,            2361,            1495,            2052,            3757,            2168,            1935,            2549,            3188,            3926,            1521,            2200,            2751,            1054,            2167,            4321,            1379,            2749,            5053,            2664,            4636,            3419,            1480,            738,            2300,            3060,            2360,            2297,            56003,            1782,            2085,            1511,            2837,            3483,            1158,            1539,            3550,            3610,            5071,            4396,            1214,            3324,            3410,            1293,            3224,            ],        [            1353,            2695,            3700,            1359,            617,            1701,            4444,            1362,            3273,            2691,            2295,            3546,            634,            309,            1424,            3542,            1599,            4533,            1032,            3698,            4311,            2147,            3512,            4723,            3526,            1421,            2697,            3774,            3973,            740,            2861,            2572,            2774,            2322,            3321,            3235,            2085,            3269,            3708,            3213,            3518,            2018,            2745,            3276,            2948,            2619,            3828,            1879,            3801,            1439,            1124,            794,            0,            4557,            2866,            1084,            1955,            2138,            1622,            2842,            3154,            2531,            3457,            826,            3086,            4015,            2017,            1596,            1018,            1287,            2992,            1403,            1170,            1784,            2423,            3161,            756,            1435,            1986,            651,            1402,            3556,            1352,            2740,            4288,            1899,            3872,            2654,            1715,            481,            2273,            3051,            2333,            2270,            55238,            1017,            1320,            746,            2888,            3456,            1393,            774,            3785,            3845,            4306,            3631,            1296,            3559,            2645,            1284,            2459,            ],        [            3114,            3738,            5604,            2593,            2701,            3194,            5433,            2596,            3636,            4056,            4138,            3277,            2839,            3018,            2799,            3273,            2418,            7504,            4003,            6669,            7282,            3136,            3435,            5516,            3319,            3408,            3802,            4420,            3704,            3711,            2573,            2675,            2877,            2141,            1427,            3135,            1006,            2660,            963,            2214,            1872,            1455,            1728,            1630,            1742,            1899,            2440,            4850,            1779,            3589,            3328,            3403,            2971,            0,            3644,            2374,            2940,            4125,            3609,            4829,            1974,            1514,            2458,            3797,            2069,            758,            1640,            4567,            3857,            3236,            4979,            4374,            4141,            4755,            5394,            6132,            3727,            4406,            4957,            3260,            4373,            6527,            2586,            1575,            7259,            4870,            6798,            5625,            2534,            2609,            1226,            1405,            2152,            312,            58209,            3988,            4291,            3717,            3666,            2863,            3134,            3525,            3516,            3576,            7277,            5618,            3358,            3922,            4632,            3262,            4446,            ],        [            1727,            496,            1560,            1838,            2923,            1327,            1389,            1507,            1382,            814,            896,            553,            1856,            2035,            1675,            487,            2067,            2878,            3211,            2043,            2656,            1328,            649,            2200,            533,            1733,            560,            1634,            918,            2692,            3042,            3144,            3346,            3385,            4456,            4344,            2704,            4332,            3992,            3640,            4901,            2793,            2197,            4659,            2014,            3682,            5469,            3831,            4808,            3716,            3594,            2994,            2562,            3716,            0,            2904,            1163,            1181,            1964,            1915,            5003,            2455,            3884,            3196,            2538,            3787,            2978,            3548,            2970,            2051,            2065,            3355,            3122,            4030,            3253,            2387,            2708,            3387,            4232,            2851,            3354,            2361,            1637,            4604,            2633,            3982,            1821,            2953,            2127,            2280,            3336,            4434,            3396,            3171,            53198,            3206,            3272,            3276,            22,            3332,            3692,            2340,            730,            790,            2651,            2704,            3675,            1668,            1718,            3754,            1531,            ],        [            2951,            3912,            5506,            2227,            1508,            3299,            5335,            2385,            4164,            3582,            3893,            4437,            2232,            1907,            3022,            4433,            1745,            6131,            2630,            5296,            5909,            3038,            4403,            5614,            4417,            3019,            3588,            5580,            4864,            2338,            3501,            3212,            3414,            2962,            4123,            3875,            2725,            3909,            3659,            3853,            3469,            2658,            3385,            3227,            3588,            3259,            3779,            3477,            3752,            972,            1356,            1915,            1598,            4508,            3941,            0,            2729,            3736,            3220,            4440,            3571,            3171,            4097,            2424,            3726,            3966,            2657,            3194,            1718,            2885,            4590,            3001,            2768,            3382,            4021,            4759,            2354,            3033,            3584,            1887,            3000,            5154,            2375,            2691,            5886,            3497,            5470,            4252,            1861,            1623,            2913,            3002,            2973,            2910,            56836,            2615,            2918,            2344,            3963,            4096,            815,            2372,            4676,            4736,            5904,            5229,            1111,            2413,            4243,            1099,            4057,            ],        [            654,            1111,            2213,            481,            1524,            254,            2042,            344,            2150,            579,            1564,            1570,            783,            962,            602,            1504,            927,            3074,            1974,            2239,            2852,            1024,            1666,            3600,            1550,            660,            664,            2287,            1935,            1619,            2851,            2562,            2764,            2312,            3311,            3225,            2075,            3259,            3130,            3203,            3109,            2008,            2735,            3112,            3031,            2609,            3889,            2758,            4928,            2566,            2251,            1921,            1489,            2805,            1017,            1500,            0,            1407,            891,            2111,            3144,            2521,            3447,            1959,            3076,            2876,            2007,            2475,            1897,            978,            2261,            2282,            2049,            2663,            3449,            2583,            1635,            2314,            2865,            1778,            2281,            2557,            474,            3867,            2829,            2778,            3141,            3149,            1043,            1207,            2263,            4178,            2323,            2260,            53779,            1963,            2199,            2039,            1039,            3446,            2300,            1267,            1747,            1807,            2847,            2900,            2423,            2436,            1914,            2411,            1726,            ],        [            1100,            2020,            3025,            1731,            2522,            996,            3186,            1594,            3458,            1723,            1620,            2714,            1622,            1907,            1171,            2648,            2177,            3130,            2030,            2295,            2908,            2332,            2810,            4361,            2694,            716,            1808,            3099,            3079,            2076,            3742,            3453,            3655,            3203,            4202,            4116,            2966,            4150,            4021,            4094,            5501,            2899,            3626,            5259,            4175,            3500,            5811,            3215,            5784,            3344,            3029,            2699,            2267,            3696,            2161,            2989,            1250,            0,            947,            2167,            4035,            3412,            4338,            2000,            3967,            3767,            2898,            2932,            2675,            1034,            2317,            2739,            2506,            3120,            3505,            2639,            2413,            2771,            3322,            2556,            2738,            2613,            1724,            4723,            2885,            3235,            3197,            3205,            2293,            2386,            3154,            5034,            3214,            3151,            53835,            2019,            2656,            2080,            2183,            4337,            3298,            1323,            2891,            2951,            2903,            2956,            3201,            3744,            1970,            3189,            1782,            ],        [            1616,            1073,            2078,            2247,            3030,            1512,            2088,            2110,            2210,            1845,            673,            1956,            2138,            2415,            1687,            1890,            2693,            2183,            2532,            1348,            1961,            2359,            2052,            3603,            1936,            1232,            1337,            2152,            2321,            2592,            4445,            3969,            4171,            3719,            4718,            4632,            3482,            4666,            4537,            4610,            6003,            3415,            3600,            5761,            3417,            4016,            6313,            4866,            6286,            3852,            3537,            3207,            2775,            4212,            1403,            3497,            1766,            516,            0,            1220,            4551,            3858,            4854,            2517,            3941,            4283,            3414,            3448,            3183,            1550,            1370,            3255,            3022,            3335,            2558,            1692,            2921,            3387,            3537,            3064,            3254,            1666,            2240,            5225,            1938,            3287,            2250,            2258,            2809,            2894,            3670,            5536,            3730,            4574,            52888,            2535,            3172,            2597,            1425,            4853,            3806,            1839,            2133,            2193,            1956,            2009,            3709,            2496,            1023,            3697,            837,            ],        [            1882,            1340,            1390,            2514,            3297,            1779,            1400,            2377,            2477,            2506,            940,            2223,            2404,            2682,            1953,            2157,            3055,            1495,            2799,            660,            1273,            3115,            2319,            3269,            2203,            1493,            1604,            1464,            2588,            2857,            4524,            4235,            4437,            3985,            4984,            5171,            3748,            5205,            4803,            4876,            6270,            3681,            3867,            6028,            3684,            4282,            6580,            4178,            6553,            4119,            3804,            3474,            3042,            4478,            1670,            3764,            2033,            777,            1716,            0,            6372,            4194,            5120,            2784,            4208,            4549,            3680,            3152,            3450,            1816,            682,            3520,            3287,            2647,            1870,            1004,            3188,            2699,            2849,            3331,            3519,            978,            2507,            5492,            1250,            2599,            1562,            1570,            3171,            3161,            3936,            5803,            3996,            3933,            52200,            2794,            2876,            2864,            1692,            5392,            4073,            2105,            2400,            2460,            1268,            1321,            3976,            2763,            319,            3964,            1119,            ],        [            3370,            4266,            5860,            2849,            2957,            3450,            5689,            2852,            4518,            3936,            4538,            4272,            3095,            3274,            3055,            4268,            2674,            7760,            4259,            6925,            7538,            3392,            4757,            5968,            4314,            3664,            3942,            5415,            5218,            3967,            1911,            1840,            2042,            1573,            1400,            1161,            2004,            686,            1219,            1873,            1187,            1937,            2832,            1201,            3197,            924,            1213,            5106,            2124,            3750,            3291,            3659,            3227,            1455,            4295,            2630,            3196,            4381,            3865,            5307,            0,            2795,            2117,            4053,            3079,            1526,            1869,            4823,            3820,            3492,            5457,            4630,            4397,            5011,            5650,            6388,            3983,            4662,            5213,            3516,            4629,            6783,            2842,            1538,            7515,            5126,            6116,            5881,            2790,            2865,            1199,            1368,            964,            1767,            58465,            4244,            4547,            3973,            4317,            1565,            3390,            3781,            5030,            4571,            7533,            6096,            3321,            3342,            4888,            3225,            4702,            ],        [            2814,            2667,            3275,            2293,            2401,            2894,            3104,            2296,            2565,            2985,            3067,            2206,            2539,            2718,            2499,            2202,            2118,            4905,            3703,            4070,            4683,            2836,            2364,            4116,            2248,            3108,            2731,            3349,            2633,            3411,            753,            855,            1057,            1536,            1951,            2055,            716,            2483,            2395,            1351,            3304,            513,            214,            3062,            417,            1833,            3872,            4550,            3211,            3194,            3072,            3103,            2671,            2119,            2573,            2074,            2640,            3352,            3309,            3942,            2368,            0,            1595,            3497,            555,            2190,            726,            4267,            3434,            2936,            4092,            4074,            3841,            4455,            5094,            5832,            3427,            4106,            4657,            2960,            4073,            6227,            2286,            3007,            4660,            4570,            3536,            5325,            2234,            2309,            1487,            2837,            1547,            1574,            54913,            3688,            3991,            3417,            2595,            1043,            2834,            3225,            2445,            2505,            4678,            4028,            2961,            2851,            3729,            3232,            3702,            ],        [            3025,            3867,            5515,            2504,            2612,            3105,            5344,            2507,            4173,            3591,            4193,            3406,            2750,            2929,            2710,            3402,            2329,            7415,            3914,            6580,            7193,            3047,            3564,            5316,            3448,            3319,            3597,            4549,            3833,            3622,            1045,            974,            1176,            874,            1070,            460,            1305,            1412,            1552,            1007,            1520,            1238,            1966,            1534,            2561,            241,            1800,            4761,            2711,            3405,            3283,            3314,            2882,            1788,            3773,            2285,            2851,            4036,            3520,            4740,            1055,            1929,            0,            3708,            2213,            1859,            1170,            4478,            3645,            3147,            4890,            4285,            4052,            4666,            5305,            6043,            3638,            4317,            4868,            3171,            4284,            6438,            2497,            1871,            7170,            4781,            6429,            5536,            2445,            2520,            869,            1701,            527,            2100,            58120,            3899,            4202,            3628,            3795,            699,            3045,            3436,            3645,            3705,            7188,            5529,            3654,            2997,            4543,            3558,            4357,            ],        [            1375,            2717,            3722,            2458,            1394,            1723,            3732,            2321,            4050,            2450,            2317,            3441,            1104,            779,            1446,            3375,            2376,            4423,            556,            3588,            4201,            2924,            4289,            5500,            3421,            1443,            2535,            3796,            3806,            630,            3638,            3349,            3551,            3099,            4280,            4012,            2862,            4414,            3816,            3990,            3626,            2795,            3522,            3384,            3725,            3396,            3936,            1769,            3909,            1699,            1384,            1525,            1139,            4665,            2888,            1861,            1977,            2160,            1644,            2864,            3931,            3308,            4234,            0,            3863,            4123,            2794,            1486,            928,            1309,            3014,            1293,            1060,            1674,            2313,            3051,            666,            1325,            1876,            1428,            1292,            3446,            2129,            2848,            4178,            1789,            3894,            2544,            2492,            1258,            3050,            3159,            3110,            3047,            50769,            143,            1210,            80,            2910,            4233,            2170,            1168,            3618,            3678,            4196,            3653,            1556,            4336,            2667,            1544,            2481,            ],        [            3146,            2844,            3452,            2625,            2733,            3226,            3281,            2628,            2742,            3162,            3244,            2383,            2871,            3050,            2831,            2379,            2450,            5082,            4035,            4247,            4860,            3168,            2541,            4293,            2425,            3440,            2908,            3526,            2810,            3743,            504,            606,            808,            1868,            1942,            1806,            1048,            2758,            2852,            1102,            2910,            845,            810,            2668,            1295,            1587,            3146,            4882,            3668,            3719,            3404,            3435,            3003,            2576,            2750,            2406,            2972,            3529,            3641,            4119,            2401,            514,            1346,            3829,            0,            2647,            1058,            4599,            3766,            3268,            4269,            4406,            4173,            4787,            5426,            4591,            3759,            4438,            4989,            3292,            4405,            4565,            2618,            3464,            4837,            4902,            3713,            5657,            2566,            2641,            1819,            3294,            1879,            2031,            55090,            4020,            4323,            3749,            2772,            794,            3166,            3557,            2622,            2682,            4855,            4205,            3293,            3028,            3906,            3564,            3879,            ],        [            2999,            4290,            5489,            2478,            2586,            3079,            5318,            2481,            4147,            3565,            4690,            3829,            2724,            2903,            2684,            3825,            2303,            7389,            4782,            6554,            7167,            3021,            3987,            6068,            3871,            5669,            3571,            4972,            4256,            3596,            3125,            2003,            2205,            2026,            1312,            2377,            1558,            1902,            848,            2099,            1114,            2007,            2280,            872,            2294,            2140,            1682,            4735,            1021,            2831,            2570,            3129,            2856,            552,            4196,            2259,            2825,            6386,            5870,            7090,            1216,            2066,            2343,            4382,            2621,            0,            1721,            4452,            3099,            3121,            7240,            4259,            4026,            4640,            5279,            6017,            3437,            4291,            4842,            3368,            4258,            6412,            2471,            817,            7144,            4755,            5745,            5510,            2419,            2494,            1111,            647,            2037,            864,            58094,            4401,            4176,            4462,            4218,            2781,            3201,            3410,            4068,            4128,            7162,            7879,            2600,            2971,            6893,            2504,            6707,            ],        [            2088,            2984,            4578,            1567,            1675,            2168,            4407,            1570,            3236,            2654,            3256,            3509,            1813,            1992,            1773,            3505,            1392,            6478,            2977,            5643,            6256,            2110,            3475,            4686,            3489,            2382,            2660,            4652,            3936,            2685,            1288,            999,            1201,            810,            1299,            1723,            366,            1757,            1628,            1177,            1852,            299,            1026,            1610,            1229,            1107,            2387,            3824,            2188,            2468,            2346,            2377,            1945,            1096,            3013,            1348,            1914,            4083,            3567,            3670,            1642,            812,            1421,            2771,            1367,            1167,            0,            3541,            2708,            2210,            3953,            3348,            3115,            3729,            4368,            5106,            2701,            3380,            3931,            2234,            3347,            5501,            1560,            1987,            6233,            3844,            4834,            4599,            1508,            1583,            761,            1777,            821,            551,            57183,            2962,            3265,            2691,            3035,            1454,            2108,            2499,            3748,            3808,            6251,            4459,            2235,            2060,            4590,            2506,            4404,            ],        [            2578,            3920,            4835,            2891,            2149,            2926,            4845,            2894,            4805,            3653,            3520,            5078,            1859,            1534,            2649,            5074,            3131,            3551,            1777,            2716,            3329,            3679,            5044,            6255,            5058,            2646,            3738,            4909,            5505,            1485,            4393,            4104,            4306,            3854,            4853,            4767,            3617,            4801,            5148,            4745,            4958,            3550,            4277,            4716,            4480,            4151,            5268,            639,            5241,            2879,            2564,            2326,            1894,            5997,            4091,            2616,            3180,            3363,            2847,            4014,            4686,            4063,            4989,            1916,            4618,            5455,            3549,            0,            2108,            2512,            4164,            876,            426,            802,            1441,            2179,            1846,            453,            1004,            2183,            696,            2574,            2884,            4180,            3306,            917,            5044,            1672,            3247,            2013,            3805,            4491,            3865,            3802,            49639,            1762,            276,            1836,            4113,            4988,            2925,            1999,            5317,            5377,            3324,            4803,            2736,            5091,            3870,            2724,            3684,            ],        [            2164,            3506,            4511,            2065,            1214,            2512,            5041,            2068,            3870,            3288,            3106,            4143,            1157,            832,            2235,            4139,            2196,            4882,            1345,            4047,            4660,            2744,            4109,            5320,            4123,            2232,            3294,            4585,            4570,            1089,            3567,            3278,            3480,            3028,            3739,            3941,            2791,            3975,            3275,            3919,            3085,            2724,            3451,            2843,            3654,            3325,            3395,            2228,            3368,            1006,            691,            832,            852,            4124,            3647,            1681,            2766,            2949,            2433,            3653,            3860,            3237,            4163,            771,            3792,            3582,            2723,            1945,            0,            2098,            3803,            1752,            1519,            2133,            2772,            3510,            338,            1784,            2335,            603,            1751,            3905,            2058,            2307,            4637,            2248,            4683,            3003,            2312,            1187,            2979,            2618,            3039,            2976,            51228,            914,            1669,            691,            3669,            4162,            1446,            1297,            4382,            4442,            4655,            4442,            863,            4156,            3456,            851,            3270,            ],        [            526,            1868,            2873,            1609,            1488,            874,            2883,            1472,            4144,            1601,            1468,            2592,            1198,            873,            597,            2526,            2470,            2978,            1004,            2143,            2756,            3018,            2688,            5594,            2572,            594,            1686,            2947,            2957,            1042,            3168,            2879,            3081,            2629,            3628,            3542,            2392,            3576,            4579,            3520,            4389,            2325,            3052,            4147,            3255,            2926,            4699,            2181,            4672,            2310,            1995,            1665,            1233,            5428,            2039,            1955,            1128,            1311,            795,            2015,            3461,            2838,            3764,            966,            3393,            4886,            2324,            1898,            1641,            0,            2165,            1705,            1472,            2086,            2725,            2487,            1379,            1737,            2288,            1522,            1704,            2461,            1602,            3611,            2733,            2201,            3045,            2956,            2586,            1352,            2580,            3922,            2640,            2577,            53683,            985,            1622,            1046,            2061,            3763,            2264,            289,            2769,            2829,            2751,            2804,            2167,            4430,            1818,            2155,            1632,            ],        [            2214,            1672,            708,            2846,            3629,            2111,            718,            2709,            2809,            2838,            1272,            1776,            2736,            3014,            2285,            1611,            3387,            1827,            3131,            992,            1605,            3447,            1872,            2587,            1756,            1825,            1936,            782,            1463,            3189,            4856,            4567,            4769,            4317,            5316,            5503,            4080,            5369,            5135,            5208,            5114,            4013,            4199,            6360,            4016,            4614,            6912,            4510,            6885,            4451,            4136,            3806,            3374,            4810,            2002,            4096,            2365,            1109,            2048,            332,            6704,            4526,            5452,            3116,            4540,            4881,            4012,            3484,            3782,            2148,            0,            3852,            3619,            2979,            2202,            1336,            3520,            3031,            3181,            3663,            3851,            1310,            2839,            5824,            1582,            2931,            880,            1902,            3503,            3493,            4268,            6135,            4328,            4265,            52005,            3126,            3208,            3196,            2024,            5724,            4405,            2437,            1275,            1477,            1600,            639,            4308,            3095,            651,            4296,            1451,            ],        [            1702,            3044,            4049,            2015,            1273,            2050,            5100,            2018,            3929,            2777,            2644,            4202,            983,            658,            1773,            4198,            2255,            4402,            901,            3567,            4180,            2803,            4168,            5379,            4182,            1770,            2862,            4123,            4629,            609,            3517,            3228,            3430,            2978,            3977,            3891,            2741,            3925,            4272,            3869,            4082,            2674,            3401,            3840,            3604,            3275,            4392,            1748,            4365,            2003,            1688,            1450,            1018,            5121,            3215,            1740,            2304,            2487,            1971,            3191,            3810,            3187,            4113,            1040,            3742,            4579,            2673,            1465,            1232,            1636,            3341,            0,            1039,            1653,            2292,            3030,            970,            1304,            1855,            1307,            1271,            3425,            2008,            3304,            4157,            1768,            4221,            2523,            2371,            1137,            2929,            3615,            2989,            2926,            50748,            886,            1189,            960,            3237,            4112,            2049,            1123,            4441,            4501,            4175,            3980,            1860,            4215,            2994,            1848,            2808,            ],        [            2152,            3494,            4499,            2465,            1723,            2500,            5550,            2468,            4379,            3227,            3094,            4652,            1433,            1108,            2223,            4648,            2705,            3363,            1351,            2528,            3141,            3253,            4618,            5829,            4632,            2220,            3312,            4573,            5079,            1059,            3967,            3678,            3880,            3428,            4427,            4341,            3191,            4375,            4722,            4319,            4532,            3124,            3851,            4290,            4054,            3725,            4842,            709,            4815,            2453,            2138,            1900,            1468,            5571,            3665,            2190,            2754,            2937,            2421,            3641,            4260,            3637,            4563,            1490,            4192,            5029,            3123,            426,            1682,            2086,            3791,            450,            0,            614,            1253,            1991,            1420,            265,            816,            1757,            471,            2386,            2458,            3754,            3118,            729,            4671,            1484,            2821,            1587,            3379,            4065,            3439,            3376,            49709,            1336,            150,            1410,            3687,            4562,            2499,            1573,            4891,            4951,            3136,            4430,            2310,            4665,            3444,            2298,            3258,            ],        [            2744,            4086,            4341,            3057,            2315,            3092,            4351,            3060,            4971,            3819,            3686,            5244,            2025,            1700,            2815,            5240,            3297,            3057,            1943,            2222,            2835,            3845,            5210,            6234,            5224,            2812,            3904,            4415,            5671,            1651,            4559,            4270,            4472,            4020,            5019,            4933,            3783,            4799,            5314,            4911,            5124,            3716,            4443,            4882,            4646,            4317,            5434,            1063,            5407,            3045,            2730,            2492,            2060,            6163,            4257,            2782,            3346,            3529,            3013,            3520,            4852,            4229,            5155,            2082,            4784,            5621,            3715,            780,            2274,            2678,            3670,            1042,            592,            0,            947,            1685,            2012,            327,            202,            2349,            1063,            2080,            3050,            4346,            2812,            423,            4550,            1178,            3413,            2179,            3971,            4657,            4031,            3968,            50063,            1928,            504,            2002,            4279,            5154,            3091,            2165,            5483,            5543,            2830,            4309,            2902,            5257,            3839,            2890,            3922,            ],        [            3246,            4588,            4251,            3559,            2817,            3594,            4261,            3562,            5473,            4321,            4188,            5746,            2527,            2202,            3317,            5742,            3799,            2967,            2445,            2132,            2745,            4347,            5712,            6144,            5726,            3314,            4406,            4325,            5006,            2153,            5061,            4772,            4974,            4522,            5521,            5435,            4285,            5301,            5816,            5413,            5626,            4218,            4945,            5384,            5148,            4819,            5936,            1565,            5909,            3547,            3232,            2994,            2562,            6665,            4759,            3284,            3848,            4031,            3515,            3430,            5354,            4731,            5657,            2584,            5286,            6123,            4217,            1282,            2776,            3180,            3580,            1544,            1094,            777,            0,            1595,            2514,            829,            979,            2851,            1565,            1990,            3552,            4848,            2722,            729,            4460,            1088,            3915,            2681,            4473,            5159,            4533,            4470,            50754,            2430,            1006,            2504,            4781,            5656,            3593,            2667,            4818,            6045,            2740,            4219,            3404,            5759,            3749,            3392,            3832,            ],        [            4112,            3401,            2882,            4425,            3683,            4460,            2892,            4428,            6339,            5187,            3001,            3950,            3393,            3068,            4183,            3785,            4665,            1598,            3311,            763,            1376,            5213,            4046,            4775,            3930,            4180,            3665,            2956,            3637,            3019,            5927,            5638,            5840,            5388,            6387,            6301,            5151,            6167,            6682,            6279,            6492,            5084,            5811,            6250,            6014,            5685,            6802,            3174,            6775,            4413,            4098,            3860,            3428,            7531,            3731,            4150,            4714,            2838,            4381,            2061,            6220,            5597,            6523,            3450,            6152,            6989,            5083,            2148,            3642,            4046,            2211,            2410,            1960,            1643,            866,            0,            3380,            1695,            1845,            3717,            2431,            621,            4418,            5714,            1353,            1595,            3091,            566,            4781,            3547,            5339,            6025,            5399,            5336,            52303,            3296,            1872,            3370,            3753,            6522,            4459,            3533,            3449,            3651,            1371,            2850,            4270,            6625,            2380,            4258,            2463,            ],        [            2211,            3553,            4558,            2092,            1241,            2559,            5068,            2095,            3897,            3315,            3153,            4170,            1513,            1188,            2282,            4166,            2223,            4929,            1392,            4094,            4707,            2771,            4136,            5347,            4150,            2279,            3321,            4632,            4597,            1136,            3594,            3305,            3507,            3055,            3766,            3968,            2818,            4002,            3302,            3946,            3112,            2751,            3478,            2870,            4834,            3352,            3422,            2275,            3395,            1033,            718,            859,            879,            4151,            3674,            1708,            2813,            2996,            2480,            3700,            3887,            3264,            4190,            818,            3819,            3609,            2750,            1992,            262,            2145,            3850,            1799,            1566,            2180,            2819,            3557,            0,            1831,            2382,            630,            1798,            3952,            2085,            2334,            4684,            2295,            4730,            3050,            2339,            1214,            3006,            2645,            3066,            3003,            51275,            1011,            1716,            738,            3696,            4189,            1473,            2004,            4409,            4469,            4702,            4489,            890,            4183,            3503,            878,            3317,            ],        [            2417,            3759,            4382,            2730,            1988,            2765,            4392,            2733,            4644,            3492,            3359,            4917,            1698,            1373,            2488,            4913,            2970,            3098,            1616,            2263,            2876,            3518,            4883,            6275,            4897,            2485,            3577,            4456,            5344,            1324,            4232,            3943,            4145,            3693,            4692,            4606,            3456,            4640,            4987,            4584,            4797,            3389,            4116,            4555,            4319,            3990,            5107,            736,            5080,            2718,            2403,            2165,            1733,            5836,            3930,            2455,            3019,            3202,            2686,            3561,            4525,            3902,            4828,            1755,            4457,            5294,            3388,            453,            1947,            2351,            3711,            715,            265,            349,            988,            1726,            1685,            0,            551,            2022,            736,            2121,            2723,            4019,            2853,            464,            4591,            1219,            3086,            1852,            3644,            4330,            3704,            3641,            49736,            1601,            177,            1675,            3952,            4827,            2764,            1838,            5156,            5216,            2871,            4350,            2575,            4930,            3880,            2563,            3523,            ],        [            2946,            4288,            4543,            3259,            2517,            3294,            4553,            3262,            5173,            4021,            3888,            5446,            2227,            1902,            3017,            5442,            3499,            3259,            2145,            2424,            3037,            4047,            5412,            6436,            5426,            3014,            4106,            4617,            5873,            1853,            4761,            4472,            4674,            4222,            5221,            5135,            3985,            5001,            5516,            5113,            5326,            3918,            4645,            5084,            4848,            4519,            5636,            1265,            5609,            3247,            2932,            2694,            2262,            6365,            4459,            2984,            3548,            3731,            3215,            3722,            5054,            4431,            5357,            2284,            4986,            5823,            3917,            982,            2476,            2880,            3872,            1244,            794,            202,            1149,            1887,            2214,            529,            0,            2551,            1265,            2282,            3252,            4548,            3014,            625,            4752,            1380,            3615,            2381,            4173,            4859,            4233,            4170,            50265,            2130,            706,            2204,            4481,            5356,            3293,            2367,            5685,            5745,            3032,            4511,            3104,            5459,            4041,            3092,            4124,            ],        [            2732,            3400,            4994,            2000,            996,            3080,            4823,            2768,            3652,            3070,            3674,            3925,            2013,            1688,            2803,            3921,            1978,            5912,            2180,            5077,            5690,            2526,            3891,            5102,            3905,            2800,            3076,            5068,            4352,            2119,            3502,            3213,            3415,            2963,            3521,            3876,            2726,            3910,            3057,            3854,            2867,            2659,            3386,            2625,            4589,            3260,            3177,            3258,            3150,            788,            473,            614,            1379,            3906,            3429,            1463,            2882,            3517,            3001,            4221,            2969,            3172,            4098,            1606,            3727,            3364,            2658,            2975,            835,            2666,            4371,            2782,            2549,            3163,            3802,            4540,            1173,            2814,            3365,            0,            2781,            4935,            1993,            2089,            5667,            3278,            5250,            4033,            2094,            1352,            2914,            2400,            2974,            4218,            56617,            1749,            2699,            1526,            3451,            4097,            1228,            2153,            4164,            4224,            5685,            5010,            645,            3938,            4024,            633,            3838,            ],        [            2393,            3735,            4740,            2706,            1964,            2741,            5791,            2709,            4620,            3468,            3335,            4893,            1674,            1349,            2464,            4889,            2946,            4247,            1592,            3412,            4025,            3494,            4859,            6070,            4873,            2461,            3553,            4814,            5320,            1300,            4208,            3919,            4121,            3669,            4668,            4582,            3432,            4448,            4963,            4560,            4773,            3365,            4092,            4531,            4295,            3966,            5083,            1335,            5056,            2694,            2379,            2141,            1709,            5812,            3906,            2431,            2995,            3178,            2662,            3882,            4501,            3878,            4804,            1731,            4433,            5270,            3364,            696,            1923,            2327,            4032,            691,            458,            1498,            2137,            2875,            1661,            1149,            1700,            1998,            0,            3270,            2699,            3995,            4002,            1613,            4912,            2368,            3062,            1828,            3620,            4306,            3680,            3617,            50335,            1577,            972,            1651,            3928,            4803,            2740,            1814,            5132,            5192,            4020,            4671,            2551,            4906,            3685,            2539,            3499,            ],        [            3322,            2780,            2261,            3954,            4203,            3219,            2271,            3817,            3917,            3946,            2380,            3329,            3913,            3588,            3393,            3164,            5185,            977,            3831,            142,            755,            5733,            3425,            4154,            3309,            2933,            3044,            2335,            3016,            3539,            6447,            6158,            6360,            5908,            6907,            6821,            5671,            6687,            7202,            6799,            7012,            5604,            6331,            6770,            5124,            6205,            7322,            3694,            7295,            4933,            4618,            4380,            3948,            8051,            3110,            4670,            3473,            2217,            3156,            1440,            6740,            6117,            7043,            3970,            6672,            7509,            5603,            2668,            4162,            3256,            1590,            2930,            2480,            2163,            1386,            520,            3900,            2215,            2365,            4237,            2951,            0,            3947,            6234,            732,            2115,            2470,            1086,            5301,            4067,            5859,            6545,            5919,            5856,            51682,            3816,            2392,            3890,            3132,            7042,            4979,            3545,            2828,            3030,            750,            2229,            4790,            4203,            1759,            4778,            1842,            ],        [            1583,            1424,            3018,            929,            3217,            1183,            2847,            792,            1676,            1094,            1824,            1949,            1712,            1891,            1531,            1945,            1654,            3806,            2903,            2971,            3584,            550,            1915,            3126,            1929,            1589,            1100,            3092,            2376,            2548,            3641,            3743,            3945,            2673,            3672,            3586,            3303,            3620,            4591,            3564,            6282,            2369,            2796,            6040,            2613,            2970,            6592,            3687,            5407,            4203,            3888,            4032,            3600,            4315,            1453,            3198,            906,            2109,            1820,            2843,            3505,            3054,            3808,            2888,            3137,            4386,            2368,            3404,            4250,            1907,            2993,            3211,            2978,            3592,            4181,            3315,            2564,            3243,            3794,            3889,            3210,            3289,            0,            5504,            3561,            3707,            3274,            3881,            1714,            3573,            2624,            5815,            2684,            3770,            54299,            2892,            3128,            2554,            1475,            3931,            3986,            2196,            2188,            2248,            3579,            3632,            4060,            1962,            2646,            4048,            2459,            ],        [            4784,            5098,            6692,            2598,            2694,            5132,            6521,            2601,            5350,            4768,            5726,            5623,            3361,            3036,            4855,            5619,            2423,            7260,            3965,            6425,            7038,            4224,            5589,            6800,            5603,            4852,            4774,            6766,            6050,            3467,            2412,            2123,            2325,            2146,            1432,            2023,            1909,            1898,            968,            2219,            778,            1842,            2569,            536,            3559,            1904,            1088,            4606,            1061,            1944,            1753,            2312,            2727,            1817,            5127,            2379,            2945,            5569,            5053,            6273,            1212,            2355,            2463,            3565,            2910,            1275,            1841,            4323,            2282,            4718,            6423,            4130,            3897,            4511,            5150,            5888,            2620,            4162,            4713,            2551,            4129,            6283,            2591,            0,            7015,            4626,            7303,            5381,            2539,            2614,            1231,            311,            2157,            2129,            57965,            3584,            4047,            3645,            5149,            2496,            2384,            4577,            5862,            5922,            7033,            7062,            1783,            3091,            6076,            1687,            5890,            ],        [            2970,            2428,            1909,            3602,            4873,            2867,            1919,            3465,            3565,            3594,            2028,            2977,            4583,            4258,            3041,            2812,            5855,            245,            4501,            846,            23,            4203,            3073,            3626,            2957,            2581,            2692,            1983,            2664,            4209,            5612,            5323,            5525,            5073,            6072,            7491,            4836,            7357,            7872,            5964,            5870,            4769,            4955,            7440,            4772,            5370,            7992,            4364,            7965,            5603,            5288,            5050,            4618,            8721,            2758,            5340,            3121,            1865,            2804,            1088,            7410,            5282,            6208,            4640,            5296,            8179,            4768,            3338,            4832,            2904,            1238,            3600,            3150,            2833,            2056,            1190,            4570,            2885,            3035,            4907,            3621,            1164,            3595,            6904,            0,            2785,            2118,            1756,            5971,            4737,            5024,            7215,            5084,            5021,            51154,            4486,            3062,            4560,            2780,            7712,            5649,            3193,            2476,            2678,            18,            1877,            5460,            3851,            1407,            5448,            1490,            ],        [            2933,            4437,            3918,            3246,            2504,            3281,            3928,            3249,            5160,            4008,            4037,            5433,            2214,            1889,            3004,            5429,            3486,            2634,            2132,            1799,            2412,            4034,            5399,            5811,            5413,            3001,            4093,            3992,            4673,            1840,            4748,            4459,            4661,            4209,            5208,            5122,            3972,            4988,            5503,            5100,            5313,            3905,            4632,            5071,            4835,            4506,            5623,            2768,            5596,            3234,            2919,            2681,            2249,            6352,            4767,            2971,            3535,            3874,            3202,            3097,            5041,            4418,            5344,            2271,            4973,            5810,            3904,            969,            2463,            2867,            3247,            1231,            781,            464,            524,            1262,            2201,            516,            666,            2538,            1252,            1657,            3239,            4535,            2389,            0,            4127,            755,            3602,            2368,            4160,            4846,            4220,            4157,            50421,            2117,            693,            2191,            4789,            5343,            3280,            2354,            4485,            4687,            2407,            3886,            3091,            5446,            3416,            3079,            3499,            ],        [            2607,            1770,            590,            3239,            4022,            2504,            600,            3102,            3484,            2132,            1665,            1606,            3129,            3407,            2678,            1441,            3685,            2220,            3524,            1385,            1998,            2646,            1702,            1720,            1586,            2224,            1878,            664,            1293,            3582,            4740,            4960,            5162,            4710,            5709,            5896,            4473,            5762,            5690,            5601,            5507,            4406,            3895,            5411,            3712,            5007,            7305,            4903,            7278,            4844,            4529,            4199,            3767,            5414,            1690,            4489,            2758,            1502,            2441,            1087,            5815,            4153,            5845,            3509,            4236,            5274,            4405,            3877,            4175,            2541,            1257,            4245,            3689,            3372,            2595,            1729,            3913,            3424,            3574,            4056,            4244,            1703,            3232,            6217,            1975,            3324,            0,            2295,            3801,            3886,            4661,            6528,            4721,            4869,            51617,            3519,            3601,            3589,            1712,            5030,            4798,            2830,            1105,            1307,            1993,            492,            4701,            3770,            1044,            4689,            1844,            ],        [            3546,            3809,            3290,            3859,            3117,            3894,            3300,            3862,            5773,            4621,            3409,            4358,            2827,            2502,            3617,            4193,            4099,            2006,            2745,            1171,            1784,            4647,            6012,            5183,            4338,            3614,            4706,            3364,            4045,            2453,            5361,            5072,            5274,            4822,            5821,            5735,            4585,            5601,            6116,            5713,            5926,            4518,            5245,            5684,            5448,            5119,            6236,            2608,            6209,            3847,            3532,            3294,            2862,            6965,            4139,            3584,            4148,            3246,            3815,            2469,            5654,            5031,            5957,            2884,            5586,            6423,            4517,            1582,            3076,            3480,            2619,            1844,            1394,            1077,            300,            634,            2814,            1129,            1279,            3151,            1865,            1029,            3852,            5148,            1761,            1029,            3499,            0,            4215,            2981,            4773,            5459,            4833,            4770,            50261,            2730,            1306,            2804,            4161,            5956,            3893,            2967,            3857,            4059,            1779,            3258,            3704,            6059,            2788,            3692,            2871,            ],        [            1848,            3605,            3510,            997,            1201,            1682,            3339,            1084,            1764,            3275,            3879,            2441,            2218,            1893,            1533,            2437,            515,            6117,            2616,            5282,            5895,            2731,            4096,            5307,            4110,            3005,            3281,            3584,            2868,            2324,            2445,            2156,            2358,            1906,            2905,            2819,            1669,            2685,            2724,            2797,            2948,            1602,            2329,            2706,            2532,            2203,            3483,            3463,            4209,            1429,            1872,            2016,            1584,            2399,            3634,            457,            1428,            3722,            3206,            4426,            2738,            2115,            3041,            2410,            2670,            2470,            1601,            3180,            2234,            2871,            4576,            2987,            2754,            3368,            4007,            4745,            2340,            3019,            3570,            1873,            2986,            5140,            1074,            3148,            5872,            3483,            5455,            4238,            0,            1476,            1857,            3459,            1917,            1854,            56822,            2601,            2904,            2330,            3656,            3040,            1272,            2358,            2680,            2740,            5890,            5215,            1568,            1183,            4229,            1556,            4043,            ],        [            1843,            2428,            3645,            1004,            501,            2191,            3474,            1007,            2680,            2098,            2785,            2576,            1124,            799,            791,            2949,            1037,            5023,            1522,            4188,            4801,            1554,            2919,            4130,            2933,            1911,            2104,            4096,            3380,            1230,            2506,            2217,            2419,            1967,            2966,            2880,            1730,            2746,            3836,            2858,            3646,            1663,            2390,            3404,            3617,            2264,            3956,            2369,            3929,            1567,            1252,            922,            490,            2460,            2457,            968,            1910,            2628,            2112,            3332,            2799,            2176,            3102,            1316,            2731,            2531,            1662,            2086,            1508,            1777,            3482,            1893,            1660,            2274,            2913,            3651,            1246,            1925,            2476,            779,            1892,            4046,            997,            2868,            4778,            2389,            4362,            3144,            1153,            0,            1918,            3179,            1978,            1915,            55728,            1507,            1810,            1236,            2479,            3101,            1277,            1264,            2815,            2875,            4796,            4121,            1424,            2966,            3135,            1412,            2949,            ],        [            2444,            3644,            5067,            2056,            2164,            2657,            4896,            2059,            3725,            3143,            3612,            3183,            2302,            2481,            2129,            3179,            1881,            6967,            3466,            6132,            6745,            2599,            3341,            5093,            3225,            2738,            3149,            4326,            3610,            3174,            1151,            892,            1094,            691,            1110,            1216,            870,            1499,            1354,            988,            1733,            803,            1500,            1491,            1733,            673,            2254,            4313,            2408,            3150,            2835,            2866,            2434,            1745,            3550,            1837,            2403,            3455,            2939,            4159,            1509,            1463,            1232,            3260,            1611,            1816,            735,            4030,            3197,            2566,            4309,            3837,            3604,            4218,            4857,            5595,            3190,            3869,            4420,            2723,            3836,            5990,            2049,            1828,            6722,            4333,            6206,            5088,            1997,            2072,            0,            1658,            769,            2097,            57672,            3451,            3754,            3180,            3572,            1265,            2597,            2855,            3422,            3482,            6740,            4948,            3007,            2549,            3962,            2995,            3776,            ],        [            2884,            3913,            5507,            2496,            3004,            3097,            4959,            2499,            4165,            3583,            6036,            4923,            2742,            2921,            2569,            4919,            2321,            7570,            4275,            6735,            7348,            3039,            4404,            5615,            4418,            5162,            3589,            6066,            5350,            3777,            2280,            2021,            2223,            2044,            1330,            2062,            1807,            1587,            866,            2117,            467,            1740,            3374,            225,            3388,            1825,            917,            4916,            890,            2254,            2063,            2622,            3037,            1646,            3942,            2277,            2843,            5879,            5363,            6583,            901,            3160,            2361,            3875,            3715,            1173,            1739,            4633,            2592,            5028,            6733,            4440,            4207,            4821,            5460,            6198,            2930,            4472,            5023,            2861,            4439,            6593,            2489,            310,            7325,            4936,            7613,            5691,            2437,            2512,            1129,            0,            2055,            1958,            58275,            3894,            4357,            3955,            3964,            2466,            2694,            4887,            5162,            5222,            7343,            7372,            2093,            2989,            6386,            1997,            6200,            ],        [            2271,            3300,            4517,            1883,            1991,            2484,            4346,            1886,            3552,            2970,            3439,            3448,            2129,            2308,            1956,            3444,            1708,            6794,            3293,            5959,            6572,            2426,            3791,            5002,            3805,            2565,            2976,            4591,            3875,            3001,            1469,            1182,            1384,            253,            449,            902,            1194,            768,            693,            880,            1181,            1127,            1790,            939,            2057,            286,            1523,            4140,            1856,            2977,            2662,            2693,            2261,            1193,            3329,            1664,            2230,            3282,            2766,            3986,            778,            1753,            1124,            3087,            1901,            1264,            1025,            3857,            3024,            2393,            4136,            3664,            3431,            4045,            4684,            5422,            3017,            3696,            4247,            2550,            3663,            5817,            1876,            1276,            6549,            4160,            5150,            4915,            1824,            1899,            248,            1106,            0,            1505,            57499,            3278,            3581,            3007,            3351,            1123,            2424,            2682,            3687,            3747,            6567,            4775,            2834,            2376,            3789,            2822,            3603,            ],        [            2839,            3614,            4222,            2451,            2559,            3052,            4051,            2454,            3512,            3538,            4014,            3153,            2697,            2876,            2524,            3149,            2276,            7362,            3861,            6527,            7140,            2994,            3311,            5392,            3195,            3133,            3544,            4296,            3580,            3569,            2449,            1976,            2178,            1999,            1285,            2300,            882,            2778,            821,            2072,            1730,            1331,            1604,            1488,            1618,            1757,            2298,            4708,            1637,            3377,            3186,            3261,            2829,            545,            3520,            2232,            2798,            3850,            3334,            4554,            2164,            1390,            2316,            3655,            1945,            616,            1694,            4425,            3715,            2961,            4704,            4232,            3999,            4613,            5252,            5990,            3585,            4264,            4815,            3118,            4231,            6385,            2444,            1433,            7117,            4728,            4483,            5483,            2392,            2467,            1084,            1263,            2010,            0,            58067,            3846,            4149,            3575,            3542,            2349,            2992,            3250,            3392,            3452,            7135,            5343,            3216,            3798,            4357,            3120,            4171,            ],        [            6569,            14351,            12968,            13960,            13218,            13995,            13440,            13963,            15371,            13999,            13951,            13746,            5850,            5525,            6154,            13581,            14200,            12511,            5768,            6149,            12289,            14748,            13842,            12450,            7892,            6637,            14084,            8342,            13433,            5476,            15432,            15173,            15375,            14923,            15749,            15836,            7608,            15702,            9139,            15814,            16027,            14619,            15346,            15785,            15352,            8142,            16337,            3928,            16310,            13948,            13633,            13395,            5885,            17066,            13830,            13685,            13526,            7354,            6838,            13011,            15802,            15132,            16058,            5907,            15687,            16524,            14618,            4347,            13177,            6503,            13181,            4867,            4417,            5651,            4874,            6062,            5837,            4444,            5853,            6174,            5043,            6457,            13953,            15249,            12266,            5603,            12378,            5105,            14316,            6004,            14874,            15560,            7856,            14871,            0,            5753,            4267,            5827,            13852,            16057,            13994,            5990,            13245,            9037,            12284,            12325,            13805,            15657,            13330,            6715,            13376,            ],        [            1754,            3096,            4101,            2837,            1909,            2102,            4111,            2700,            4565,            2829,            2696,            4461,            1480,            1155,            1825,            3754,            2891,            4802,            935,            3967,            4580,            3439,            4804,            6015,            3800,            1822,            2914,            4175,            4185,            1009,            4262,            3973,            4175,            3723,            4117,            4636,            3486,            4502,            3653,            4614,            3463,            3419,            4146,            3221,            5502,            4020,            3773,            2148,            3746,            1701,            1386,            1527,            1547,            4502,            3267,            2376,            2356,            2539,            2023,            3243,            3897,            3932,            4858,            535,            4487,            3960,            3418,            1865,            930,            1688,            3393,            1672,            1439,            2053,            2692,            3430,            661,            1704,            2255,            1298,            1671,            3825,            2753,            2685,            4557,            2168,            4273,            2923,            3007,            1882,            3674,            2996,            3734,            4814,            51148,            0,            1589,            615,            3289,            4857,            2152,            1547,            3997,            4057,            4575,            4032,            1558,            4851,            3046,            1546,            2860,            ],        [            2302,            3644,            4559,            2615,            1873,            2650,            4569,            2618,            4529,            2654,            3244,            4425,            1583,            1258,            1887,            3579,            2855,            3275,            1501,            2440,            3053,            3403,            4768,            5979,            4782,            2370,            2739,            4633,            4010,            1209,            4117,            3828,            4030,            3578,            4577,            4491,            3341,            4357,            4872,            4469,            4682,            3274,            4001,            4440,            4204,            3875,            4992,            559,            4965,            2603,            2288,            2050,            1618,            5721,            3092,            2340,            2181,            3087,            2571,            3738,            4410,            3787,            4713,            1640,            4342,            5179,            3273,            276,            1832,            2236,            3888,            600,            150,            526,            1165,            1903,            1570,            177,            728,            1907,            621,            2298,            2608,            3904,            3030,            641,            4731,            1396,            2971,            1737,            3529,            4215,            3589,            3526,            49559,            1486,            0,            1560,            3114,            4712,            2649,            1723,            3822,            3882,            3048,            4490,            2460,            4815,            3594,            2448,            3408,            ],        [            1455,            2797,            3802,            2056,            1314,            1803,            3812,            2401,            3970,            2530,            2397,            3866,            1024,            699,            1526,            3455,            2296,            4749,            636,            3914,            4527,            2844,            4209,            5420,            3501,            1523,            2615,            3876,            3886,            956,            3558,            3269,            3471,            3019,            4360,            3932,            2782,            3798,            3896,            3910,            3706,            2715,            3442,            3464,            3645,            3316,            4016,            2095,            3989,            1619,            1304,            1445,            1059,            4745,            2968,            1781,            2057,            2240,            1724,            2944,            3851,            3228,            4154,            80,            3783,            4203,            2714,            1812,            848,            1389,            3094,            1619,            1386,            2000,            2639,            3377,            586,            1651,            2202,            1348,            1618,            3772,            2049,            2928,            4504,            2115,            3974,            2870,            2412,            1178,            2970,            3239,            3030,            2967,            51095,            223,            1536,            0,            2990,            4153,            2070,            1248,            3698,            3758,            4522,            3733,            1476,            4256,            2747,            1464,            2561,            ],        [            1705,            474,            1538,            1816,            2901,            1305,            1367,            1485,            1360,            792,            874,            531,            1834,            2013,            1653,            465,            2045,            2856,            3189,            2021,            2634,            1306,            627,            2178,            511,            1711,            538,            1612,            896,            2670,            3020,            3122,            3324,            3363,            4434,            4322,            2682,            4142,            3970,            3618,            4879,            2771,            2175,            4637,            1992,            3660,            5447,            3809,            4786,            3887,            3572,            2972,            2540,            3694,            380,            2882,            1141,            1159,            1942,            1893,            4195,            2433,            3862,            3174,            2516,            3765,            2956,            3526,            2948,            2029,            2043,            3333,            3100,            4008,            3231,            2365,            2686,            3365,            4210,            2829,            3332,            2339,            1615,            4582,            2611,            3960,            1799,            2931,            2105,            2258,            3314,            4412,            3374,            3149,            53176,            3184,            3250,            3254,            0,            3310,            3670,            2318,            708,            768,            2629,            2682,            3744,            1646,            1696,            3732,            1509,            ],        [            3005,            3725,            4333,            2617,            2725,            3218,            4162,            2620,            3623,            4043,            4125,            3264,            2863,            3042,            2690,            3260,            2442,            7528,            4027,            6693,            7306,            3160,            3422,            5174,            3306,            3299,            3789,            4407,            3691,            3735,            903,            832,            1034,            1182,            1378,            1012,            1431,            1890,            2104,            308,            2072,            1364,            1637,            2086,            2575,            793,            2352,            4874,            3003,            3711,            3396,            3427,            2995,            2340,            3631,            2398,            2964,            4410,            3500,            5000,            1607,            1600,            552,            3821,            2071,            2411,            1296,            4591,            3758,            3127,            5150,            4398,            4165,            4779,            5418,            6156,            3751,            4430,            4981,            3284,            4397,            6551,            2610,            2423,            7283,            4894,            4594,            5649,            2558,            2633,            1177,            2253,            1079,            2652,            55971,            4012,            4315,            3741,            3653,            0,            3158,            3416,            3503,            3563,            7301,            5086,            3568,            3909,            4787,            3556,            4760,            ],        [            2112,            3073,            4290,            1412,            669,            2460,            4119,            1570,            3325,            2743,            3054,            3221,            1393,            1068,            1697,            3217,            930,            5292,            1791,            4457,            5070,            2199,            3564,            4775,            3578,            2180,            2749,            4364,            3648,            1499,            2686,            2397,            2599,            2147,            3146,            3060,            1910,            2926,            3631,            3038,            3441,            1843,            2570,            3199,            2773,            2444,            3751,            2638,            3724,            884,            1047,            1191,            759,            4480,            3102,            415,            1991,            2897,            2381,            3601,            2979,            2356,            3282,            1585,            2911,            3938,            1842,            2355,            1409,            2046,            3751,            2162,            1929,            2543,            3182,            3920,            1515,            2194,            2745,            1048,            2161,            4315,            1560,            2663,            5047,            2658,            4631,            3413,            1046,            784,            2098,            2974,            2158,            2095,            55997,            1776,            2079,            1505,            3124,            3281,            0,            1533,            3460,            3520,            5065,            4390,            843,            1598,            3404,            831,            3218,            ],        [            1082,            2424,            3429,            1941,            1199,            1430,            3439,            1944,            3855,            1980,            2024,            3751,            909,            584,            1153,            2905,            2181,            4546,            715,            3711,            4324,            2729,            4094,            5305,            4108,            1150,            2065,            3503,            3336,            753,            3443,            3154,            3356,            2904,            3903,            3817,            2667,            3683,            4290,            3795,            4100,            2600,            3327,            3858,            3530,            3201,            4410,            1892,            4383,            2021,            1706,            1376,            944,            5139,            2418,            1666,            1684,            1867,            1351,            2571,            3736,            3113,            4039,            677,            3668,            4597,            2599,            1609,            1352,            1016,            2721,            1416,            1183,            1797,            2436,            3174,            1090,            1448,            1999,            1233,            1415,            3569,            1934,            3322,            4301,            1912,            3601,            2667,            2297,            1063,            2855,            3633,            2915,            2852,            50892,            696,            1333,            757,            2440,            4038,            1975,            0,            3148,            3208,            4319,            3360,            1878,            4141,            2374,            1866,            2188,            ],        [            2847,            1129,            830,            3192,            4277,            1938,            659,            2118,            2736,            1425,            1529,            519,            2467,            2646,            2918,            354,            3638,            2460,            3764,            1625,            2238,            1939,            615,            1630,            499,            2464,            1171,            904,            188,            3822,            3653,            3755,            3957,            3996,            5067,            4909,            3315,            4775,            4603,            4251,            4793,            3404,            2808,            5270,            2625,            4293,            6080,            5143,            5419,            5263,            4948,            4439,            4007,            4327,            1049,            4258,            1774,            1742,            2681,            1497,            5768,            3066,            4495,            3749,            3149,            4398,            3589,            4117,            4415,            2781,            1647,            4485,            3929,            3612,            2835,            1969,            4153,            3664,            3814,            4296,            4484,            1943,            2248,            6564,            2215,            3564,            1091,            2535,            2738,            4126,            3947,            5045,            4007,            3782,            52468,            3759,            3841,            3829,            1071,            3943,            5046,            3070,            0,            220,            2233,            1583,            5120,            3022,            1284,            5108,            2084,            ],        [            2628,            1397,            1313,            2972,            4057,            2228,            1142,            3025,            2516,            1715,            1797,            809,            2757,            2936,            3401,            644,            3418,            2943,            4247,            2108,            2721,            2229,            598,            1410,            789,            2947,            1461,            1387,            671,            4305,            4403,            4505,            4707,            4716,            5715,            5629,            3605,            5495,            4893,            5001,            7122,            3694,            3098,            6880,            2915,            5013,            6370,            5626,            7405,            5043,            4728,            4872,            4440,            4617,            1303,            4038,            2064,            2225,            3164,            1716,            5548,            3356,            5245,            4232,            3899,            4688,            4411,            4600,            5090,            3264,            2130,            4968,            4412,            4095,            3318,            2452,            5196,            4147,            4297,            4729,            4967,            2426,            3015,            6344,            2698,            4047,            1202,            3018,            3534,            4413,            4667,            6655,            4727,            4072,            52579,            4242,            4324,            4312,            1325,            4693,            4826,            3553,            483,            0,            2716,            1694,            4900,            2802,            1767,            4888,            2567,            ],        [            2988,            2446,            1927,            3620,            4891,            2885,            1937,            3483,            3583,            3612,            2046,            2995,            4601,            4276,            3059,            2830,            5873,            227,            4519,            864,            5,            4221,            3091,            3644,            2975,            2605,            2710,            2001,            2682,            4227,            5630,            5341,            5543,            5091,            5917,            7509,            4854,            7375,            7890,            5982,            5888,            4787,            4973,            7458,            4790,            5388,            8010,            4382,            7983,            5621,            5306,            5068,            4636,            8739,            2776,            5358,            3139,            1883,            2822,            1106,            7428,            5300,            6226,            4658,            5314,            8197,            4786,            3356,            4850,            2922,            1256,            3618,            3168,            2851,            2074,            1208,            4588,            2903,            3053,            4925,            3639,            1182,            3613,            6922,            18,            2803,            2136,            1774,            5989,            4755,            5042,            7233,            5102,            5039,            51172,            4504,            3080,            4578,            2798,            7730,            5667,            3211,            2494,            2696,            0,            1895,            5478,            3869,            1425,            5466,            1508,            ],        [            2568,            2026,            1062,            3200,            3983,            2465,            1072,            3063,            3163,            3192,            1626,            2130,            3090,            3368,            2639,            1965,            3646,            2181,            3485,            1346,            1959,            3801,            2226,            2176,            2110,            2185,            2290,            1136,            1817,            3543,            5210,            4921,            5123,            4671,            5670,            5857,            4434,            5723,            5489,            5562,            6956,            4367,            4553,            6714,            4370,            4968,            7266,            4864,            7239,            4805,            4490,            4160,            3728,            5725,            2356,            4450,            2719,            1463,            2402,            686,            5776,            4880,            5806,            3470,            4894,            5796,            4366,            3838,            4136,            2502,            856,            4206,            3973,            3333,            2556,            1690,            3874,            3385,            3535,            4017,            4205,            1664,            3193,            6178,            1936,            3285,            447,            2256,            3762,            3847,            4622,            6489,            4682,            4619,            51594,            3480,            3562,            3550,            2378,            6078,            4759,            2791,            1629,            1831,            1954,            0,            4662,            3449,            1005,            4650,            1805,            ],        [            2572,            3590,            4807,            2190,            1186,            2920,            4636,            2958,            3842,            3260,            3514,            3738,            1853,            1528,            2157,            3734,            2168,            5752,            1952,            4917,            5530,            2716,            4081,            5292,            4095,            2640,            3266,            4881,            4165,            1959,            3692,            3403,            3605,            3153,            3293,            4066,            2916,            3759,            2829,            4080,            2639,            2849,            3576,            2397,            4779,            3450,            2949,            3098,            2922,            978,            245,            804,            1219,            3678,            3619,            1653,            3072,            3357,            2841,            4061,            3073,            3362,            4324,            1378,            3917,            3136,            2848,            2815,            607,            2506,            4211,            2622,            2389,            3003,            3642,            4380,            945,            2654,            3205,            1043,            2621,            4775,            2183,            1861,            5507,            3118,            5091,            3873,            2284,            1554,            3092,            2172,            3164,            3990,            56457,            1521,            2539,            1298,            3641,            4287,            1429,            1993,            3977,            4037,            5525,            4850,            0,            4128,            3864,            453,            3678,            ],        [            1221,            1637,            2854,            170,            1264,            821,            2683,            223,            1889,            1307,            2131,            1785,            1402,            1581,            1536,            1781,            616,            3641,            2541,            2806,            3419,            763,            2128,            3339,            2142,            1227,            1231,            2928,            2212,            2274,            2453,            2164,            2366,            1914,            2913,            2827,            1677,            2693,            2732,            2805,            2956,            1610,            2337,            2714,            2826,            2211,            3491,            3413,            3631,            2250,            1935,            1966,            1534,            2968,            1584,            1189,            567,            1974,            1458,            2678,            2746,            2123,            3049,            2360,            2678,            3039,            1609,            3130,            2297,            1545,            2828,            2937,            2704,            3318,            3957,            3150,            2290,            2969,            3520,            1823,            2936,            3124,            213,            3551,            3396,            3433,            3487,            3716,            732,            1172,            1865,            2881,            1925,            1862,            54346,            2551,            2854,            2280,            1606,            3048,            2033,            1834,            2024,            2084,            3414,            3467,            2107,            0,            2481,            2095,            2293,            ],        [            1558,            1015,            2050,            2189,            2986,            1454,            2356,            2052,            2152,            1787,            615,            1898,            2080,            2371,            1629,            1832,            2635,            2155,            2488,            1320,            1933,            2790,            1994,            3545,            1878,            1174,            1279,            2124,            2263,            2534,            4200,            3911,            4113,            3661,            4660,            4574,            3424,            4440,            4479,            4552,            5959,            3357,            3542,            5717,            3359,            3958,            6269,            3673,            6242,            3808,            3493,            3163,            2731,            4715,            1345,            3453,            1708,            458,            1405,            1192,            4765,            3870,            4796,            2473,            3883,            4786,            3356,            3390,            3139,            1492,            1342,            3197,            2964,            3307,            2530,            1664,            2877,            3359,            3509,            3020,            3208,            1638,            2182,            5181,            1910,            3259,            2222,            2230,            2751,            2850,            3625,            5492,            3685,            3622,            52860,            2477,            3126,            2553,            1367,            4808,            3762,            1781,            2075,            2135,            1928,            1981,            3665,            2438,            0,            3653,            808,            ],        [            2668,            3686,            4903,            2286,            1282,            3016,            4732,            3054,            3938,            3356,            3610,            3834,            1949,            1624,            2253,            3830,            1633,            5848,            2048,            5013,            5626,            2812,            4177,            5388,            4191,            2736,            3362,            4977,            4261,            2055,            3974,            3715,            3917,            2850,            3024,            3965,            2613,            3490,            2560,            3811,            2370,            2546,            3273,            2128,            4875,            3147,            2680,            3194,            2653,            257,            341,            900,            1315,            3409,            3715,            1118,            3168,            3453,            2937,            4157,            2804,            3059,            4055,            1474,            3614,            2867,            2545,            2911,            703,            2602,            4307,            2718,            2485,            3099,            3738,            4476,            1041,            2750,            3301,            1139,            2717,            4871,            2279,            1592,            5603,            3214,            5187,            3969,            1749,            1650,            2823,            1903,            2861,            3804,            56553,            1617,            2635,            1394,            3737,            4088,            697,            2089,            4073,            4133,            5621,            4946,            96,            4224,            3960,            0,            3774,            ],        [            1758,            1216,            1250,            2390,            3173,            1655,            1260,            2253,            2353,            2382,            816,            2099,            2280,            2558,            1829,            2033,            2836,            1355,            2675,            520,            1133,            2991,            2195,            3129,            2079,            1375,            1480,            1324,            2005,            2733,            4400,            4111,            4313,            3861,            4860,            4774,            3624,            4640,            4679,            4752,            6146,            3557,            3743,            5904,            3560,            4158,            6456,            4038,            6429,            3995,            3680,            3350,            2918,            4915,            1546,            3640,            1909,            653,            1592,            392,            4966,            4070,            4996,            2660,            4084,            4986,            3556,            3012,            3326,            1692,            542,            3274,            2824,            2507,            1730,            864,            3064,            2559,            2709,            3207,            3395,            838,            2383,            5368,            1110,            2459,            1422,            1430,            2952,            3037,            3812,            5679,            3872,            3809,            52060,            2670,            2736,            2740,            1568,            4995,            3949,            1981,            1817,            2019,            1128,            1181,            3852,            2639,            195,            3840,            0,            ],        ], 'starts': [20, 3, 81, 58, 40], 'ends': [20, 3, 81, 58, 40]},    'NW-9': {'distance_matrix': [        [            0,            5567,            5880,            3380,            3070,            3847,            555,            484,            561,            5407,            2226,            4045,            793,            4870,            2025,            1896,            6010,            236,            5028,            322,            3872,            3439,            5075,            5408,            1802,            2450,            5638,            922,            4150,            6089,            3692,            6317,            4265,            1354,            5325,            ],        [            5462,            0,            313,            1338,            1354,            1811,            5663,            5203,            5860,            2270,            2298,            1217,            4736,            695,            2332,            2709,            2873,            5448,            2013,            5768,            1044,            3734,            5294,            5627,            3033,            1865,            5857,            4901,            1519,            522,            1393,            750,            1634,            3299,            205,            ],        [            5775,            313,            0,            1651,            1667,            1996,            5976,            5516,            6173,            2455,            2611,            1530,            5049,            1008,            2645,            3022,            3058,            5761,            2198,            6081,            1357,            3919,            5479,            5812,            3346,            2178,            6042,            5214,            1704,            209,            1502,            437,            1819,            3612,            518,            ],        [            3472,            1513,            1826,            0,            648,            2173,            4380,            3213,            4577,            2632,            2265,            474,            2746,            1028,            1626,            1935,            3235,            3458,            2375,            3778,            477,            4096,            5656,            5989,            2259,            1159,            6219,            2911,            1881,            2035,            2664,            2263,            1996,            2525,            1271,            ],        [            2892,            1556,            1869,            639,            0,            2812,            3264,            2633,            3437,            3271,            944,            1113,            2166,            1558,            978,            1355,            3874,            2878,            3014,            3198,            1116,            5346,            6982,            7315,            1679,            511,            7545,            2331,            2520,            2078,            3303,            2306,            2635,            1945,            1314,            ],        [            3623,            1600,            1709,            2465,            2917,            0,            3995,            3595,            4168,            1700,            3861,            2191,            3510,            1669,            3895,            4199,            2303,            3609,            1443,            3929,            2018,            1904,            3540,            3873,            4519,            3428,            4103,            3639,            949,            1918,            491,            2146,            1064,            4071,            2194,            ],        [            555,            5775,            6088,            3723,            3413,            6307,            0,            611,            390,            5750,            2569,            5014,            1136,            5823,            2368,            2239,            6353,            463,            5371,            233,            4841,            3782,            5418,            5751,            2145,            2793,            5981,            1265,            6015,            6297,            4035,            6525,            6130,            1697,            5533,            ],        [            483,            4966,            5279,            3280,            2970,            3747,            633,            0,            1002,            5307,            2126,            3945,            518,            4770,            1925,            1796,            5910,            247,            4928,            789,            3772,            3339,            4975,            5308,            1702,            2350,            5538,            822,            4050,            5488,            3592,            5716,            4165,            991,            4724,            ],        [            561,            5896,            6209,            4628,            4366,            6428,            390,            980,            0,            6887,            3522,            5135,            1338,            5944,            3321,            3192,            7490,            781,            5573,            239,            4962,            3984,            5355,            6516,            3098,            3746,            6746,            1467,            6136,            6418,            4237,            6646,            6251,            1899,            5654,            ],        [            5291,            1043,            1152,            2381,            2397,            1668,            5663,            5263,            5836,            0,            3341,            2737,            6256,            2215,            3375,            3752,            2011,            5277,            1151,            5597,            2564,            2872,            4432,            4765,            4076,            2908,            4995,            5307,            1495,            1361,            1680,            1589,            1491,            4342,            1248,            ],        [            2255,            2071,            2384,            1154,            844,            3327,            2627,            1996,            2800,            3786,            0,            1628,            1529,            2073,            341,            718,            4389,            2241,            3529,            2561,            1631,            4709,            6345,            6678,            1042,            224,            6908,            1694,            3035,            2593,            3818,            2821,            3150,            1308,            1829,            ],        [            3574,            1595,            1908,            274,            726,            2026,            4482,            3315,            4679,            2485,            1670,            0,            2848,            881,            1704,            2037,            3088,            3560,            2228,            3880,            330,            3949,            5509,            5842,            2361,            1237,            6072,            3013,            1734,            2117,            2517,            2345,            1849,            2627,            1353,            ],        [            856,            3435,            3748,            2518,            2208,            3718,            1228,            597,            1401,            5278,            1364,            3183,            0,            3437,            1163,            1034,            5881,            842,            4899,            1162,            3010,            3310,            4946,            5279,            665,            1588,            5509,            295,            4021,            3957,            3563,            4185,            4136,            473,            3193,            ],        [            4067,            1296,            1609,            796,            1248,            1702,            4975,            3808,            5172,            2161,            2192,            522,            3341,            0,            2226,            2530,            2764,            4053,            1904,            4373,            349,            3625,            5185,            5518,            2854,            1759,            5748,            3506,            1410,            1818,            2193,            2046,            1525,            3120,            1054,            ],        [            1996,            2510,            2823,            1593,            1283,            4937,            2368,            1737,            2541,            5396,            439,            2640,            1270,            2512,            0,            459,            5999,            1982,            5139,            2302,            2467,            4450,            6086,            6419,            783,            663,            6649,            1435,            4645,            3032,            5428,            3260,            4760,            1049,            2268,            ],        [            1537,            4807,            5120,            1946,            1636,            5339,            1909,            1278,            2082,            5798,            792,            2611,            811,            2865,            484,            0,            6401,            1523,            5541,            1843,            2438,            3991,            5627,            5960,            324,            1016,            6190,            976,            5047,            5329,            4244,            5557,            5162,            590,            4565,            ],        [            4051,            3872,            3981,            4482,            4934,            3139,            4423,            4023,            4563,            2829,            5371,            4208,            3938,            3686,            5170,            5041,            0,            4037,            2450,            4357,            4035,            861,            2421,            2754,            4947,            5595,            2984,            4067,            2966,            4190,            3630,            4418,            2962,            4499,            4211,            ],        [            236,            6057,            6370,            3360,            3050,            3827,            648,            248,            781,            5387,            2206,            4025,            773,            4850,            2005,            1876,            5990,            0,            5008,            542,            3852,            3419,            5055,            5388,            1782,            2430,            5618,            902,            4130,            6579,            3672,            6807,            4245,            1334,            5815,            ],        [            4312,            2003,            1595,            2032,            2484,            689,            4684,            4284,            4857,            580,            3428,            1758,            5277,            1236,            3462,            3766,            1183,            4298,            0,            4618,            1585,            2044,            3604,            3937,            4090,            2995,            4167,            4328,            516,            1804,            1180,            2032,            512,            4760,            1761,            ],        [            322,            6008,            6321,            3686,            3376,            4153,            233,            790,            239,            5713,            2532,            4351,            1099,            5176,            2331,            2202,            6316,            542,            5334,            0,            4178,            3745,            5381,            5714,            2108,            2756,            5944,            1228,            4456,            6530,            3998,            6758,            4571,            1660,            5766,            ],        [            3747,            1232,            1545,            447,            899,            1892,            4655,            3488,            4852,            2351,            1843,            173,            3021,            747,            1877,            2210,            2954,            3733,            2094,            4053,            0,            3815,            5375,            5708,            2534,            1410,            5938,            3186,            1600,            1754,            2383,            1982,            1715,            2800,            990,            ],        [            5901,            3011,            3120,            3621,            4073,            2278,            6273,            5873,            6446,            1968,            5017,            3347,            6866,            2825,            5051,            5355,            2571,            5887,            1589,            6207,            3174,            0,            4992,            5325,            5679,            4584,            5555,            5917,            2105,            3329,            2769,            3557,            2101,            6349,            3350,            ],        [            5483,            5364,            5473,            5974,            6426,            4631,            5855,            5455,            6028,            4321,            6803,            5700,            5370,            5178,            6602,            6473,            4924,            5469,            3942,            5789,            5527,            2368,            0,            333,            6379,            7027,            563,            5499,            4458,            5682,            5122,            5910,            4454,            5931,            5703,            ],        [            5150,            5031,            5140,            5641,            6093,            4298,            5522,            5122,            5695,            3988,            6470,            5367,            5037,            4845,            6269,            6140,            4591,            5136,            3609,            5456,            5194,            2035,            318,            0,            6046,            6694,            513,            5166,            4125,            5349,            4789,            5577,            4121,            5598,            5370,            ],        [            1146,            3228,            3541,            2311,            2001,            4008,            1518,            887,            1691,            5568,            1157,            2976,            456,            3230,            956,            827,            6171,            1132,            5189,            1452,            2803,            3600,            5236,            5569,            0,            1381,            5799,            585,            4311,            3750,            3853,            3978,            4426,            266,            2986,            ],        [            2827,            1847,            2160,            930,            620,            3103,            3199,            2568,            3372,            3562,            879,            1404,            2101,            1849,            913,            1290,            4165,            2813,            3305,            3133,            1407,            5281,            6917,            7250,            1614,            0,            7480,            2266,            2811,            2369,            3594,            2597,            2926,            1880,            1605,            ],        [            5617,            5498,            5607,            6108,            6560,            4765,            5989,            5589,            6129,            4455,            6937,            5834,            5504,            5312,            6736,            6607,            5058,            5603,            4076,            5923,            5661,            2502,            674,            467,            6513,            7161,            0,            5633,            4592,            5816,            5256,            6044,            4588,            6065,            5837,            ],        [            785,            4892,            5205,            3022,            2712,            4049,            935,            302,            1304,            5609,            1868,            3687,            687,            4940,            1667,            1538,            6212,            549,            5230,            1091,            3514,            3641,            5277,            5610,            1444,            2092,            5840,            0,            5132,            5414,            3894,            5642,            5247,            1160,            4650,            ],        [            4241,            1400,            1509,            1961,            2413,            618,            4613,            4213,            4786,            751,            3357,            1687,            5206,            1165,            3391,            3695,            1354,            4227,            494,            4547,            1514,            2215,            3775,            4108,            4019,            2924,            4338,            4257,            0,            1718,            1109,            1946,            441,            4689,            1690,            ],        [            5984,            522,            209,            1860,            1876,            2205,            6185,            5725,            6382,            2664,            2820,            1739,            5258,            1217,            2854,            3231,            3267,            5970,            2407,            6290,            1566,            4128,            5688,            6021,            3555,            2387,            6251,            5423,            1913,            0,            1711,            228,            2028,            3821,            727,            ],        [            3518,            1495,            1604,            2242,            2694,            1018,            3890,            3490,            4063,            1477,            3638,            1968,            3405,            1446,            3672,            3976,            2080,            3504,            1220,            3824,            1795,            1799,            4501,            4834,            4414,            3205,            5064,            3534,            726,            1813,            0,            2041,            841,            3966,            1971,            ],        [            6374,            912,            599,            2250,            2266,            2595,            6575,            6115,            6772,            3054,            3210,            2129,            5648,            1607,            3244,            3621,            3657,            6360,            2797,            6680,            1956,            4518,            6078,            6411,            3945,            2777,            6641,            5813,            2303,            390,            2101,            0,            2418,            4211,            1117,            ],        [            3800,            1777,            1886,            2642,            3094,            177,            4172,            3772,            4345,            1877,            4038,            2368,            3687,            1846,            4072,            4376,            2480,            3786,            1620,            4106,            2195,            1878,            3514,            3847,            4696,            3605,            4077,            3816,            1126,            2095,            668,            2323,            0,            4248,            2371,            ],        [            1317,            2962,            3275,            2045,            1735,            5438,            1689,            1058,            1862,            5897,            891,            2710,            591,            2964,            690,            561,            6500,            1303,            5640,            1623,            2537,            3771,            5407,            5740,            193,            1115,            5970,            756,            5146,            3484,            4024,            3712,            5261,            0,            2720,            ],        [            5257,            242,            555,            1133,            1149,            1789,            5458,            4998,            5655,            2248,            2093,            1012,            4531,            490,            2127,            2504,            2851,            5243,            1991,            5563,            839,            3712,            5272,            5605,            2828,            1660,            5835,            4696,            1497,            764,            2280,            992,            1612,            3094,            0,            ],        ], 'starts': [8, 31], 'ends': [8, 31]},    'NW-10': {'distance_matrix': [        [            0,            1111,            2949,            4866,            5819,            5490,            6121,            4343,            2192,            4493,            3689,            3992,            1816,            4148,            4342,            1675,            1509,            4983,            1603,            1399,            1231,            1709,            2429,            2505,            3663,            1242,            5333,            5110,            5105,            1338,            2565,            4108,            4338,            2574,            1995,            2277,            2288,            5692,            840,            9099,            3524,            4216,            4766,            1455,            3435,            2294,            2065,            5553,            1136,            4436,            857,            1945,            2807,            960,            5054,            4575,            2115,            2402,            925,            4017,            2699,            4082,            4729,            1412,            2299,            4139,            3511,            1262,            4840,            4830,            2300,            3922,            1795,            2658,            5047,            3570,            3692,            1726,            1143,            2718,            5356,            4525,            1560,            1724,            2065,            4960,            1313,            4122,            5303,            1818,            4240,            4438,            5636,            ],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [],        [            2097,            2798,            2666,            4583,            5087,            5207,            4478,            4060,            2198,            4210,            3406,            3998,            1533,            4154,            4059,            143,            1226,            4989,            1320,            2539,            2241,            2033,            2146,            2222,            3380,            2736,            4601,            5116,            6375,            1662,            2282,            4114,            5608,            1377,            177,            459,            2294,            4961,            2527,            10369,            3241,            4222,            4772,            3142,            3152,            1657,            1782,            4821,            2406,            4442,            2351,            1662,            2524,            1284,            4771,            4292,            1177,            2119,            2419,            3734,            1355,            3799,            4446,            1736,            2016,            3856,            3517,            979,            4846,            4836,            482,            3639,            1512,            2664,            4764,            3287,            3698,            1443,            764,            2435,            5073,            4242,            3054,            3218,            1782,            4966,            1191,            4128,            4347,            0,            4246,            4444,            5353,            ],        [            5297,            5587,            6035,            7279,            2807,            3739,            3110,            8159,            3743,            8100,            4998,            1873,            5384,            1422,            5651,            5509,            4429,            822,            5198,            4287,            4376,            3994,            4747,            5888,            4972,            4724,            2321,            1601,            5179,            4377,            4327,            1834,            4270,            5552,            5829,            6111,            3661,            2681,            5316,            9031,            5458,            1052,            526,            5757,            6563,            5272,            5043,            2541,            7037,            196,            5140,            4871,            5785,            4743,            4664,            4185,            5710,            5380,            4682,            7103,            6162,            5310,            7736,            4291,            4061,            7225,            2357,            4675,            600,            1307,            6134,            7529,            4155,            3637,            7974,            6656,            1573,            4264,            4977,            5696,            4966,            5834,            4406,            4242,            5043,            491,            5274,            646,            2067,            5652,            0,            198,            5246,            ],        [            5099,            5389,            5837,            7081,            2609,            3541,            2912,            7961,            3545,            7902,            4800,            2097,            5186,            1224,            5453,            5311,            4231,            624,            5000,            4089,            4178,            3796,            4549,            5690,            4774,            4526,            2123,            1403,            4981,            4179,            4129,            1636,            4072,            5354,            5631,            5913,            3463,            2483,            5118,            8833,            5260,            1276,            328,            5559,            6365,            5074,            4845,            2343,            6839,            1142,            4942,            4673,            5587,            4545,            4466,            3987,            5512,            5182,            4484,            6905,            5964,            5112,            7538,            4093,            3863,            7027,            2159,            4477,            402,            1109,            5936,            7331,            3957,            3439,            7776,            6458,            1797,            4066,            4779,            5498,            4768,            5636,            4208,            4044,            4845,            293,            5076,            870,            1869,            5454,            1271,            0,            5048,            ],        [            7778,            7183,            3468,            5899,            3737,            2359,            4102,            5383,            5627,            5533,            2326,            4744,            4467,            3932,            2746,            5334,            4482,            4579,            4272,            5120,            4822,            4440,            3517,            3321,            2372,            6973,            3251,            4706,            6111,            4823,            5216,            3526,            5202,            4635,            5654,            5936,            5545,            3673,            7305,            9963,            2891,            4380,            4607,            6689,            3954,            4355,            4126,            3471,            7969,            4437,            7301,            3954,            4868,            5192,            833,            1284,            5018,            4463,            5128,            4536,            5470,            1885,            6356,            4737,            4023,            4658,            4241,            4629,            4681,            4426,            5959,            4962,            4446,            5521,            6594,            4089,            4303,            4521,            4802,            4779,            454,            3162,            7291,            7455,            4126,            4556,            5099,            4286,            3283,            5477,            4404,            4439,            0,            ],        ], 'starts': [50, 92, 74, 80], 'ends': [50, 92, 74, 80]},    'NW-11': {'distance_matrix': [        [            0,            767,            595,            4141,            137,            4458,            873,            1677,            977,            2352,            3986,            1032,            962,            4630,            409,            3221,            3491,            890,            209,            3868,            512,            2661,            4661,            1763,            3769,            2139,            1559,            2741,            781,            670,            1628,            2467,            4149,            1068,            1869,            3566,            801,            2783,            2003,            598,            1210,            956,            1641,            2181,            1699,            1826,            4100,            3224,            1177,            707,            2513,            316,            872,            757,            1517,            1390,            722,            3479,            1480,            1126,            4694,            1974,            1404,            3245,            3407,            4427,            3467,            1531,            ],        [            694,            0,            920,            3530,            831,            3847,            508,            1066,            1013,            1741,            3375,            1268,            1339,            4019,            706,            2610,            2880,            279,            825,            3257,            603,            2050,            4050,            1152,            3158,            1528,            948,            2130,            334,            180,            1017,            1856,            3538,            1304,            1258,            2955,            618,            2172,            2034,            1917,            1265,            2105,            1672,            2212,            1730,            1857,            3489,            2613,            566,            1401,            1902,            1010,            261,            802,            1548,            1421,            1099,            2868,            1511,            1378,            4083,            1363,            1640,            2634,            2796,            3816,            2856,            920,            ],        [            698,            961,            0,            4335,            835,            4652,            412,            1871,            1381,            2546,            4180,            1846,            419,            4824,            252,            3415,            3685,            1084,            386,            4062,            355,            2855,            4855,            1957,            3963,            2333,            1753,            2935,            586,            740,            1822,            2661,            4343,            1882,            2063,            3760,            1423,            2977,            2738,            1296,            1969,            2809,            2376,            2916,            2434,            2561,            4294,            3418,            1371,            1405,            2707,            1014,            1066,            1607,            2252,            2125,            179,            3673,            2215,            1824,            4888,            2168,            2218,            3437,            3599,            4621,            3661,            1725,            ],        [            3916,            3335,            4142,            0,            4037,            709,            3730,            2969,            3284,            1452,            237,            3599,            4561,            952,            3928,            1527,            421,            3501,            4047,            798,            3825,            1143,            912,            3382,            2666,            3338,            3219,            3102,            3556,            3402,            3015,            2043,            400,            3575,            2488,            2463,            3547,            1659,            5256,            4362,            4487,            5327,            4894,            5434,            4952,            5079,            520,            1342,            2919,            4471,            1291,            4080,            3440,            3731,            4770,            4643,            4321,            409,            4733,            3873,            1016,            3173,            3434,            637,            775,            749,            1099,            2823,            ],        [            137,            966,            732,            4244,            0,            4561,            1010,            1782,            840,            2455,            4089,            895,            1099,            4733,            546,            3326,            3594,            1027,            346,            3971,            649,            2764,            4764,            1868,            3895,            2244,            1582,            2846,            918,            807,            1733,            2570,            4252,            931,            1974,            3692,            664,            2888,            1866,            461,            1073,            819,            1504,            2044,            1562,            1689,            4203,            3329,            1282,            570,            2616,            179,            1009,            620,            1380,            1253,            859,            3582,            1343,            989,            4797,            2079,            1267,            3348,            3510,            4530,            3572,            1636,            ],        [            4502,            3921,            4728,            586,            4623,            0,            4316,            3555,            3870,            2038,            449,            4185,            5147,            777,            4514,            2113,            1007,            4087,            4633,            1384,            4411,            1729,            203,            3641,            3252,            3924,            3805,            3688,            4142,            3988,            3601,            2629,            265,            4161,            3074,            3049,            4133,            2245,            5842,            4948,            5073,            5913,            5480,            6020,            5538,            5665,            1106,            1928,            3505,            5057,            1877,            4666,            4026,            4317,            5356,            5229,            4907,            995,            5319,            4459,            841,            3759,            4020,            1223,            1361,            860,            1685,            3409,            ],        [            986,            549,            412,            3923,            1123,            4238,            0,            1459,            969,            2132,            3766,            1434,            831,            4410,            464,            3003,            3273,            672,            664,            3650,            361,            2441,            4443,            1545,            3551,            1921,            1341,            2523,            174,            328,            1410,            2247,            3931,            1470,            1651,            3348,            1011,            2565,            2326,            2209,            1557,            2397,            1964,            2504,            2022,            2149,            3882,            3006,            959,            1693,            2295,            1302,            654,            1195,            1840,            1713,            591,            3261,            1803,            1544,            4474,            1756,            1806,            3027,            3187,            4209,            3249,            1313,            ],        [            2088,            1957,            2541,            3478,            1951,            3795,            2129,            0,            1594,            1689,            3323,            1054,            2960,            3967,            2497,            2558,            2828,            1900,            2297,            3205,            2224,            1998,            3998,            492,            3106,            462,            1618,            2078,            1955,            1801,            1414,            1804,            3486,            1254,            877,            2903,            1857,            2120,            3655,            2276,            2886,            2634,            3293,            3833,            3351,            3478,            3437,            2561,            1318,            2385,            1850,            1994,            1839,            2041,            3169,            3042,            2720,            2816,            3132,            1328,            4031,            297,            889,            2582,            2744,            3764,            2804,            146,            ],        [            890,            363,            1283,            3643,            753,            3960,            871,            1179,            0,            1854,            3488,            702,            1702,            4132,            1299,            2723,            2993,            642,            1099,            3370,            966,            2163,            4163,            1265,            3271,            1641,            979,            2243,            697,            543,            1130,            1969,            3651,            738,            1371,            3068,            263,            2285,            2397,            1078,            1628,            1436,            2035,            2575,            2093,            2220,            3602,            2726,            679,            1187,            2015,            796,            312,            447,            1911,            1784,            1462,            2981,            1874,            1196,            4196,            1476,            1074,            2747,            2909,            3929,            2969,            1033,            ],        [            2979,            2398,            3205,            1789,            3100,            2106,            2793,            2032,            2347,            0,            1634,            2662,            3624,            2278,            2991,            1332,            1139,            2564,            3110,            1516,            2888,            309,            2309,            2118,            1901,            2401,            2282,            881,            2619,            2465,            2078,            591,            1797,            2638,            1551,            1698,            2610,            894,            4319,            3425,            3550,            4390,            3957,            4497,            4015,            4142,            1748,            1335,            1982,            3534,            161,            3143,            2503,            2794,            3833,            3706,            3384,            1127,            3796,            2936,            2342,            2236,            2497,            893,            1055,            2075,            1144,            1886,            ],        [            4937,            4356,            5163,            506,            5058,            472,            4751,            3990,            4305,            1958,            0,            4620,            5582,            697,            4949,            2033,            927,            4522,            5068,            1304,            4846,            1649,            675,            4076,            3172,            4359,            4240,            2839,            4577,            4423,            4036,            2549,            163,            4596,            3509,            2969,            4568,            2165,            6277,            5383,            5508,            6348,            5915,            6455,            5973,            6100,            1026,            1848,            3940,            5492,            1797,            5101,            4461,            4752,            5791,            5664,            5342,            915,            5754,            4894,            761,            4194,            4455,            1143,            1281,            780,            1605,            3844,            ],        [            1034,            903,            1629,            3771,            897,            4088,            1618,            1307,            540,            1982,            3616,            0,            1996,            4260,            1443,            2851,            3121,            1182,            1243,            3498,            1546,            2291,            4291,            1393,            3399,            1442,            1107,            2371,            1444,            1290,            1258,            2097,            3779,            200,            1499,            3196,            803,            2413,            3144,            1222,            2375,            1580,            2782,            3322,            2840,            2967,            3730,            2854,            807,            1331,            2143,            940,            852,            987,            2658,            2531,            1756,            3109,            2621,            274,            4324,            1607,            372,            2875,            3037,            4057,            3097,            1161,            ],        [            1065,            1210,            419,            4584,            1202,            4901,            831,            2120,            1630,            2795,            4429,            2097,            0,            5073,            619,            3664,            3934,            1333,            753,            4311,            722,            3104,            5104,            2206,            4212,            2582,            2002,            3184,            1005,            1159,            2071,            2910,            4592,            2131,            2312,            4009,            1672,            3226,            3157,            1663,            2388,            2021,            2795,            3335,            2853,            2980,            4543,            3667,            1620,            1772,            2956,            1381,            1315,            1856,            2671,            2544,            240,            3922,            2634,            2191,            5137,            2417,            2469,            3688,            3850,            4870,            3910,            1974,            ],        [            5109,            4528,            5335,            875,            5230,            612,            4923,            4162,            4477,            2130,            738,            4792,            5754,            0,            5121,            1767,            1099,            4694,            5240,            762,            5018,            1821,            815,            4248,            3344,            4531,            4412,            3011,            4749,            4595,            4208,            2721,            554,            4768,            3681,            3141,            4740,            2337,            6449,            5555,            5680,            6520,            6087,            6627,            6145,            6272,            530,            1582,            4112,            5664,            1969,            5273,            4633,            4924,            5963,            5836,            5514,            1087,            5926,            5066,            64,            4366,            4627,            1315,            1453,            203,            1777,            4016,            ],        [            425,            657,            252,            4031,            562,            4348,            464,            1567,            1077,            2242,            3876,            1457,            619,            4520,            0,            3111,            3381,            780,            200,            3758,            103,            2551,            4551,            1653,            3659,            2029,            1449,            2631,            372,            526,            1518,            2357,            4039,            1493,            1759,            3456,            1119,            2673,            2524,            1023,            1755,            1381,            2162,            2702,            2220,            2347,            3990,            3114,            1067,            1132,            2403,            741,            762,            1303,            2038,            1911,            379,            3369,            2001,            1551,            4584,            1864,            1829,            3135,            3297,            4317,            3357,            1421,            ],        [            3944,            3363,            4170,            1814,            4065,            2131,            3758,            2997,            3312,            1503,            1659,            3627,            4589,            2303,            3956,            0,            1164,            3529,            4075,            1541,            3853,            1194,            2334,            3410,            1410,            3366,            3247,            1846,            3584,            3430,            3043,            2094,            1822,            3603,            2516,            1207,            3575,            403,            5284,            4390,            4515,            5355,            4922,            5462,            4980,            5107,            1773,            405,            2947,            4499,            1342,            4108,            3468,            3759,            4798,            4671,            4349,            1152,            4761,            3901,            2367,            3201,            3462,            918,            1080,            2100,            684,            2851,            ],        [            4010,            3429,            4236,            1112,            4131,            1429,            3824,            3063,            3378,            1031,            957,            3693,            4655,            1601,            4022,            1106,            0,            3595,            4141,            839,            3919,            722,            1632,            3476,            2245,            3432,            3313,            1912,            3650,            3496,            3109,            1622,            1120,            3669,            2582,            2042,            3641,            1238,            5350,            4456,            4581,            5421,            4988,            5528,            5046,            5173,            1071,            921,            3013,            4565,            870,            4174,            3534,            3825,            4864,            4737,            4415,            450,            4827,            3967,            1665,            3267,            3528,            216,            354,            1398,            678,            2917,            ],        [            1366,            785,            1592,            3251,            1487,            3568,            1180,            787,            734,            1462,            3096,            989,            2011,            3740,            1378,            2331,            2601,            0,            1497,            2978,            1275,            1771,            3771,            873,            2879,            1249,            669,            1851,            1006,            852,            738,            1575,            3259,            1025,            979,            2676,            997,            1893,            2706,            1812,            1937,            2777,            2344,            2884,            2402,            2529,            3210,            2334,            287,            1921,            1623,            1530,            890,            1181,            2220,            2093,            1771,            2589,            2183,            1099,            3804,            1084,            1361,            2355,            2517,            3537,            2577,            641,            ],        [            225,            791,            386,            4165,            362,            4482,            664,            1701,            1202,            2376,            4010,            1257,            753,            4654,            200,            3245,            3515,            914,            0,            3892,            303,            2685,            4685,            1787,            3793,            2163,            1583,            2765,            572,            726,            1652,            2491,            4173,            1293,            1893,            3590,            1026,            2807,            2724,            823,            1955,            1181,            2362,            2902,            2420,            2547,            4124,            3248,            1201,            932,            2537,            541,            896,            982,            2238,            2111,            513,            3503,            2201,            1351,            4718,            1998,            1629,            3269,            3431,            4451,            3491,            1555,            ],        [            4492,            3911,            4718,            1176,            4613,            1493,            4306,            3545,            3860,            1513,            1021,            4175,            5137,            762,            4504,            1005,            482,            4077,            4623,            0,            4401,            1204,            1696,            3631,            2727,            3914,            3795,            2394,            4132,            3978,            3591,            2104,            1184,            4151,            3064,            2524,            4123,            1720,            5832,            4938,            5063,            5903,            5470,            6010,            5528,            5655,            232,            820,            3495,            5047,            1352,            4656,            4016,            4307,            5346,            5219,            4897,            470,            5309,            4449,            826,            3749,            4010,            698,            836,            559,            651,            3399,            ],        [            528,            644,            370,            4018,            665,            4335,            361,            1554,            1064,            2229,            3863,            1529,            722,            4507,            103,            3098,            3368,            767,            303,            3745,            0,            2538,            4538,            1640,            3646,            2016,            1436,            2618,            269,            423,            1505,            2344,            4026,            1565,            1746,            3443,            1106,            2660,            2421,            1126,            1652,            2492,            2059,            2599,            2117,            2244,            3977,            3101,            1054,            1235,            2390,            844,            749,            1290,            1935,            1808,            482,            3356,            1898,            1654,            4571,            1851,            1901,            3122,            3284,            4304,            3344,            1408,            ],        [            3288,            2707,            3514,            1480,            2894,            1797,            3102,            2341,            2656,            309,            1325,            2971,            3418,            1969,            2785,            1263,            830,            2873,            3419,            1207,            2682,            0,            2000,            2754,            2210,            2803,            2591,            1190,            2928,            2259,            1872,            900,            1488,            2947,            1345,            2007,            2404,            1203,            4628,            3734,            3859,            4699,            4266,            4806,            4324,            4451,            1439,            1078,            1776,            3843,            148,            3452,            2297,            3103,            4142,            4015,            3693,            818,            4105,            3245,            2033,            2638,            2806,            584,            746,            1766,            835,            2195,            ],        [            4814,            4233,            5040,            383,            4935,            371,            4628,            3867,            4182,            1835,            246,            4497,            5459,            574,            4826,            1910,            804,            4399,            4945,            1181,            4723,            1526,            0,            4280,            3049,            4329,            4117,            2716,            4454,            4300,            3913,            2426,            62,            4473,            3386,            2846,            4445,            2042,            6154,            5260,            5385,            6225,            5792,            6332,            5850,            5977,            903,            1725,            3817,            5369,            1674,            4978,            4338,            4629,            5668,            5541,            5219,            792,            5631,            4771,            638,            4164,            4332,            1020,            1158,            657,            1482,            3721,            ],        [            1593,            1462,            2188,            3970,            1456,            4287,            2177,            492,            1099,            2181,            3815,            559,            2555,            4459,            2002,            3050,            3320,            1741,            1802,            3697,            2105,            2490,            4490,            0,            3598,            954,            1666,            2570,            2003,            1849,            1423,            2296,            3978,            759,            1369,            3395,            1362,            2612,            3703,            1781,            2934,            2139,            3341,            3881,            3399,            3526,            3929,            3053,            1366,            1890,            2342,            1499,            1411,            1546,            3217,            3090,            2315,            3308,            3180,            833,            4523,            789,            394,            3074,            3236,            4256,            3296,            346,            ],        [            2520,            1939,            2746,            2720,            2641,            3037,            2334,            1573,            1888,            931,            2565,            2203,            3165,            3209,            2532,            1800,            2070,            2105,            2651,            2447,            2429,            1240,            3240,            1659,            0,            1942,            1823,            1320,            2160,            2006,            1619,            1044,            2728,            2179,            1092,            2145,            2151,            1362,            3860,            2966,            3091,            3931,            3498,            4038,            3556,            3683,            2679,            1803,            1523,            3075,            1092,            2684,            2044,            2335,            3374,            3247,            2925,            2058,            3337,            2477,            3273,            1777,            2038,            1824,            1986,            3006,            2046,            1427,            ],        [            2550,            1840,            2647,            3584,            2413,            3901,            2235,            462,            1789,            1795,            3429,            1516,            3066,            4073,            2433,            2664,            2934,            2006,            2552,            3311,            2330,            2104,            4104,            954,            3212,            0,            1724,            2184,            2061,            1907,            1520,            1910,            3592,            1716,            983,            3009,            2052,            2226,            3761,            2738,            2992,            3096,            3399,            3939,            3457,            3584,            3543,            2667,            1424,            2847,            1956,            2456,            1945,            2236,            3275,            3148,            2826,            2922,            3238,            1790,            4137,            165,            1351,            2688,            2850,            3870,            2910,            608,            ],        [            697,            116,            923,            3533,            834,            3850,            511,            1069,            536,            1744,            3378,            1001,            1342,            4022,            709,            2613,            2883,            282,            828,            3260,            606,            2053,            4053,            1155,            3161,            1531,            0,            2133,            337,            183,            1020,            1859,            3541,            1037,            1261,            2958,            578,            2175,            2037,            1920,            1268,            2108,            1675,            2215,            1733,            1860,            3492,            2616,            569,            1404,            1905,            1013,            221,            762,            1551,            1424,            1102,            2871,            1514,            1111,            4086,            1366,            1373,            2637,            2799,            3819,            2859,            923,            ],        [            2098,            1517,            2324,            2296,            2219,            2613,            1912,            1151,            1466,            507,            2141,            1781,            2743,            2785,            2110,            1378,            1646,            1683,            2229,            2023,            2007,            816,            2816,            1237,            1947,            1520,            1401,            0,            1738,            1584,            1197,            622,            2304,            1757,            670,            1744,            1729,            940,            3438,            2544,            2669,            3509,            3076,            3616,            3134,            3261,            2255,            1381,            1101,            2653,            668,            2262,            1622,            1913,            2952,            2825,            2503,            1634,            2915,            1831,            2849,            1448,            1616,            1400,            1562,            2582,            1624,            1005,            ],        [            812,            375,            586,            3749,            949,            4066,            174,            1285,            795,            1960,            3594,            1487,            1005,            4238,            372,            2829,            3099,            498,            572,            3476,            269,            2269,            4269,            1371,            3377,            1747,            1167,            2349,            0,            154,            1236,            2073,            3757,            1523,            2162,            3174,            837,            2391,            2152,            2035,            1383,            2223,            1790,            2330,            1848,            1975,            3708,            2832,            785,            1519,            2121,            1128,            480,            1021,            1666,            1539,            765,            3087,            1629,            1597,            4302,            1582,            1754,            2853,            3015,            4035,            3075,            1139,            ],        [            658,            221,            740,            3595,            795,            3912,            328,            1131,            641,            1806,            3440,            1333,            1159,            4084,            526,            2675,            2945,            344,            726,            3322,            423,            2115,            4115,            1217,            3223,            1593,            1013,            2195,            154,            0,            1082,            1919,            3603,            1369,            2008,            3020,            683,            2237,            1998,            1881,            1229,            2069,            1636,            2176,            1694,            1821,            3554,            2678,            631,            1365,            1967,            974,            326,            867,            1512,            1385,            919,            2933,            1475,            1443,            4148,            1428,            1600,            2699,            2861,            3881,            2921,            985,            ],        [            1708,            1127,            1934,            2513,            1829,            2830,            1522,            761,            1076,            724,            2358,            1331,            2211,            3002,            1720,            1593,            1863,            1293,            1839,            2240,            1617,            1033,            3033,            847,            2141,            1223,            1011,            1113,            1348,            1194,            0,            837,            2521,            1367,            241,            1938,            1339,            1155,            3048,            2154,            2279,            3119,            2686,            3226,            2744,            2871,            2472,            1596,            711,            2263,            885,            1872,            1232,            1523,            2562,            2435,            1971,            1851,            2525,            1441,            3066,            1058,            1226,            1617,            1779,            2799,            1839,            615,            ],        [            3860,            3279,            4086,            2169,            3981,            2486,            3674,            2913,            3228,            1858,            2014,            3543,            4363,            2658,            3872,            757,            1519,            3445,            3991,            1896,            3769,            1549,            2689,            2999,            1326,            3375,            3163,            1762,            3500,            3346,            2959,            0,            2177,            3519,            2393,            1123,            3491,            319,            5200,            4306,            4431,            5271,            4838,            5378,            4896,            5023,            2128,            760,            2863,            4415,            1697,            4024,            3384,            3675,            4714,            4587,            4123,            1507,            4677,            3593,            2722,            3210,            3378,            1273,            1435,            2455,            1003,            2767,            ],        [            4279,            3698,            4505,            363,            4400,            329,            4093,            3332,            3647,            1815,            226,            3962,            4782,            554,            4291,            1890,            784,            3864,            4410,            1161,            4188,            1506,            532,            3418,            3029,            3794,            3582,            2696,            3919,            3765,            3378,            2406,            0,            3938,            2812,            2826,            3910,            2022,            5619,            4725,            4850,            5690,            5257,            5797,            5315,            5442,            883,            1705,            3282,            4834,            1654,            4443,            3803,            4094,            5133,            5006,            4542,            772,            5096,            4012,            618,            3629,            3797,            1000,            1138,            637,            1462,            3186,            ],        [            1543,            1412,            2138,            4280,            1406,            4594,            2127,            1816,            1049,            2491,            4125,            689,            2505,            4769,            1952,            3360,            3630,            1691,            1752,            4007,            2055,            2800,            4800,            1902,            3908,            2278,            1616,            2880,            1953,            1799,            1767,            2604,            4288,            0,            2693,            3705,            1312,            2922,            3653,            1731,            2884,            2089,            3291,            3831,            3349,            3476,            4239,            3363,            1316,            1840,            2652,            1449,            1361,            1496,            3167,            3040,            2265,            3618,            3130,            783,            4830,            2113,            1061,            3384,            3546,            4566,            3606,            1670,            ],        [            1893,            1312,            2119,            2734,            2014,            3051,            1707,            946,            1261,            945,            2579,            1516,            2396,            3223,            1905,            1814,            2084,            1478,            2024,            2461,            1802,            1254,            3254,            1032,            2383,            983,            1196,            1348,            1533,            1379,            992,            1058,            2742,            1552,            0,            2159,            1524,            1376,            3233,            2339,            2464,            3304,            2871,            3411,            2929,            3056,            2690,            1817,            896,            2448,            1103,            2057,            1417,            1708,            2747,            2620,            2156,            2069,            2710,            1626,            3287,            818,            1411,            1835,            2000,            3017,            2060,            800,            ],        [            2723,            2142,            2949,            2920,            2844,            3237,            2537,            1776,            2091,            1131,            2765,            2406,            3226,            3409,            2735,            1465,            2270,            2308,            2854,            2647,            2632,            1440,            3440,            1862,            203,            2238,            2026,            965,            2363,            2209,            1822,            1247,            2928,            2382,            1256,            0,            2354,            1027,            4063,            3169,            3294,            4134,            3701,            4241,            3759,            3886,            2879,            1468,            1726,            3278,            1292,            2887,            2247,            2538,            3577,            3450,            2986,            2258,            3540,            2456,            3473,            2073,            2241,            2024,            2186,            3206,            1711,            1630,            ],        [            627,            546,            1222,            3823,            490,            4140,            1054,            1362,            420,            2034,            3668,            839,            1589,            4312,            1036,            2906,            3173,            825,            836,            3550,            1139,            2343,            4343,            1448,            3475,            1824,            1162,            2440,            880,            726,            1313,            2150,            3831,            875,            2239,            3272,            0,            2468,            2580,            815,            1811,            1173,            2218,            2758,            2276,            2403,            3782,            2909,            862,            924,            2195,            533,            495,            184,            2094,            1967,            1349,            3161,            2057,            933,            4376,            1659,            1211,            2927,            3089,            4109,            3152,            1216,            ],        [            3541,            2960,            3767,            2804,            3662,            3121,            3355,            2594,            2909,            1949,            2649,            3224,            4044,            2603,            3553,            990,            2154,            3126,            3672,            1841,            3450,            2184,            3324,            2680,            1007,            3056,            2844,            1443,            3181,            3027,            2640,            2065,            2812,            3200,            2074,            804,            3172,            0,            4881,            3987,            4112,            4952,            4519,            5059,            4577,            4704,            2073,            1019,            2544,            4096,            2110,            3705,            3065,            3356,            4395,            4268,            3804,            2142,            4358,            3274,            2667,            2891,            3059,            1908,            2070,            2400,            1262,            2448,            ],        [            1534,            1815,            2458,            5186,            1397,            5506,            2046,            2725,            2235,            3397,            5031,            2927,            2877,            5678,            2244,            4269,            4536,            1938,            2444,            4913,            2141,            3706,            5706,            2811,            4838,            3187,            2607,            3789,            1872,            1718,            2676,            3513,            5194,            2963,            3602,            4635,            2277,            3831,            0,            1227,            614,            725,            847,            561,            905,            1032,            5145,            4272,            2225,            1364,            3558,            1217,            1920,            2461,            416,            596,            2637,            4524,            413,            3037,            5742,            3022,            3194,            4290,            4452,            5472,            4515,            2579,            ],        [            694,            1387,            1289,            5385,            557,            5702,            2242,            2921,            1261,            3596,            5230,            1316,            1656,            5874,            1103,            4465,            4732,            2134,            903,            5109,            1206,            3905,            5902,            3007,            5013,            3383,            2803,            3985,            2068,            1914,            2872,            3711,            5390,            1352,            3798,            4831,            1085,            4027,            1405,            0,            612,            358,            1043,            1583,            1101,            1228,            5344,            4468,            2421,            524,            3757,            377,            2116,            1041,            919,            792,            1416,            4720,            882,            1410,            5938,            3218,            1688,            4486,            4651,            5668,            4711,            2775,            ],        [            1173,            1399,            2042,            4770,            1036,            5090,            1630,            2309,            1819,            2984,            4618,            1795,            2461,            5262,            1828,            3853,            4123,            1522,            1382,            4500,            1725,            3293,            5293,            2395,            4401,            2771,            2191,            3373,            1456,            1302,            2260,            3099,            4781,            2547,            3186,            4198,            1861,            3415,            793,            652,            0,            864,            431,            971,            489,            616,            4732,            3856,            1809,            1003,            3145,            856,            1504,            2045,            307,            180,            2221,            4111,            270,            1889,            5326,            2606,            2778,            3877,            4039,            5059,            4099,            2163,            ],        [            809,            1502,            1404,            4782,            672,            5099,            2357,            2318,            1376,            2993,            4627,            1431,            1771,            5271,            1218,            3862,            4132,            2249,            1018,            4509,            1321,            3302,            5302,            2404,            4410,            2780,            2118,            3382,            2183,            2029,            2269,            3108,            4790,            1467,            3195,            4207,            1200,            3424,            1520,            316,            727,            0,            1158,            1698,            1216,            1343,            4741,            3865,            1818,            639,            3154,            492,            2231,            1156,            1034,            907,            1531,            4120,            997,            1525,            5335,            2615,            1803,            3886,            4048,            5068,            4108,            2172,            ],        [            1620,            1687,            2330,            5061,            1483,            5378,            1918,            2597,            2107,            3272,            4906,            2799,            2749,            5550,            2116,            4141,            4411,            1810,            2316,            4788,            2013,            3581,            5581,            2683,            4689,            3059,            2479,            3661,            1744,            1590,            2548,            3387,            5069,            2835,            3474,            4486,            2149,            3703,            1155,            1099,            486,            1226,            0,            1333,            58,            185,            5020,            4144,            2097,            1450,            3433,            1303,            1792,            2333,            418,            542,            2509,            4399,            632,            2909,            5614,            2894,            3066,            4165,            4327,            5347,            4387,            2451,            ],        [            1852,            2133,            2776,            5507,            1715,            5824,            2364,            3043,            2553,            3718,            5352,            3245,            3195,            5996,            2562,            4587,            4857,            2256,            2762,            5234,            2459,            4027,            6027,            3129,            5135,            3505,            2925,            4107,            2190,            2036,            2994,            3833,            5515,            3281,            3920,            4932,            2595,            4149,            318,            1545,            932,            1043,            1165,            0,            1223,            1350,            5466,            4590,            2543,            1682,            3879,            1535,            2238,            2779,            734,            914,            2955,            4845,            731,            3355,            6060,            3340,            3512,            4611,            4773,            5793,            4833,            2897,            ],        [            1678,            1745,            2388,            5119,            1541,            5436,            1976,            2655,            2165,            3330,            4964,            2857,            2807,            5608,            2174,            4199,            4469,            1868,            2374,            4846,            2071,            3639,            5639,            2741,            4747,            3117,            2537,            3719,            1802,            1648,            2606,            3445,            5127,            2893,            3532,            4544,            2207,            3761,            1213,            1157,            544,            1284,            58,            1391,            0,            127,            5078,            4202,            2155,            1508,            3491,            1361,            1850,            2391,            476,            600,            2567,            4457,            690,            2967,            5672,            2952,            3124,            4223,            4385,            5405,            4445,            2509,            ],        [            1805,            1872,            2515,            5246,            1668,            5563,            2103,            2782,            2292,            3457,            5091,            2984,            2934,            5735,            2301,            4326,            4596,            1995,            2501,            4973,            2198,            3766,            5766,            2868,            4874,            3244,            2664,            3846,            1929,            1775,            2733,            3572,            5254,            3020,            3659,            4671,            2334,            3888,            1340,            1284,            671,            1411,            185,            1518,            127,            0,            5205,            4329,            2282,            1635,            3618,            1488,            1977,            2518,            603,            727,            2694,            4584,            817,            3094,            5799,            3079,            3251,            4350,            4512,            5532,            4572,            2636,            ],        [            4162,            3581,            4388,            1285,            4283,            1142,            3976,            3215,            3530,            1698,            1253,            3849,            4665,            530,            4174,            1237,            667,            3747,            4293,            232,            4071,            1389,            1345,            3301,            2912,            3677,            3465,            2579,            3802,            3648,            3261,            2289,            964,            3821,            2734,            2709,            3793,            1905,            5502,            4608,            4733,            5573,            5140,            5680,            5198,            5325,            0,            1052,            3165,            4717,            1537,            4326,            3686,            3977,            5016,            4889,            4425,            655,            4979,            4119,            594,            3512,            3684,            883,            1021,            327,            883,            3069,            ],        [            3882,            3301,            4108,            1703,            4003,            2020,            3696,            2935,            3250,            1392,            1548,            3569,            4385,            1584,            3894,            185,            1156,            3467,            4013,            822,            3791,            1083,            2223,            3021,            1348,            3397,            3185,            1784,            3522,            3368,            2981,            1983,            1711,            3541,            2454,            1145,            3513,            588,            5222,            4328,            4453,            5293,            4860,            5400,            4918,            5045,            1054,            0,            2885,            4437,            1231,            4046,            3406,            3697,            4736,            4609,            4145,            1144,            4699,            3615,            1648,            3232,            3404,            807,            969,            1381,            243,            2789,            ],        [            1243,            662,            1469,            2964,            1364,            3281,            1057,            500,            611,            1175,            2809,            866,            1746,            3453,            1255,            2044,            2314,            828,            1374,            2691,            1152,            1484,            3484,            586,            2592,            962,            546,            1564,            883,            729,            451,            1290,            2972,            902,            1377,            2389,            874,            1606,            2583,            1689,            1814,            2654,            2221,            2761,            2279,            2406,            2923,            2047,            0,            1798,            1336,            1407,            767,            1058,            2097,            1970,            1506,            2302,            2060,            976,            3517,            797,            969,            2068,            2230,            3250,            2290,            354,            ],        [            904,            1597,            1499,            4877,            767,            5194,            1777,            2413,            1471,            3088,            4722,            1526,            1866,            5366,            1313,            3957,            4227,            1794,            1113,            4604,            1416,            3397,            5397,            2499,            4505,            2875,            2213,            3477,            1685,            1574,            2364,            3203,            4885,            1562,            3290,            4302,            1295,            3519,            2151,            746,            1358,            1104,            1789,            2329,            1847,            1974,            4836,            3960,            1913,            0,            3249,            587,            1546,            1251,            1665,            1538,            1626,            4215,            1628,            1620,            5430,            2710,            1898,            3981,            4143,            5163,            4203,            2267,            ],        [            2873,            2292,            3099,            1628,            2994,            1945,            2954,            1926,            2508,            161,            1473,            2556,            3376,            2117,            2885,            1493,            978,            2725,            3004,            1355,            2782,            148,            2148,            2012,            2062,            2388,            2176,            1042,            2513,            2359,            1972,            752,            1636,            2532,            1712,            1859,            2771,            1055,            4480,            3319,            3444,            4284,            3851,            4391,            4176,            4036,            1587,            1525,            1876,            3695,            0,            3037,            2397,            2955,            3727,            3600,            3136,            966,            3690,            2830,            2181,            2223,            2658,            732,            894,            1914,            983,            2047,            ],        [            317,            1010,            912,            4290,            180,            4607,            1190,            1826,            884,            2501,            4135,            939,            1279,            4779,            726,            3370,            3640,            1207,            526,            4017,            829,            2810,            4810,            1912,            3918,            2288,            1626,            2890,            1098,            987,            1777,            2614,            4298,            975,            2018,            3715,            708,            2932,            1687,            282,            894,            640,            1325,            1865,            1383,            1510,            4249,            3373,            1326,            391,            2662,            0,            959,            664,            1201,            1074,            1039,            3628,            1164,            1033,            4843,            2123,            1311,            3394,            3556,            4576,            3616,            1680,            ],        [            984,            201,            1121,            3721,            847,            4038,            709,            1257,            315,            1932,            3566,            780,            1540,            4210,            907,            2801,            3071,            480,            1193,            3448,            804,            2241,            4241,            1343,            3349,            1719,            1057,            2321,            535,            381,            1208,            2045,            3729,            816,            1449,            3146,            357,            2363,            2235,            1172,            1466,            1530,            1873,            2413,            1931,            2058,            3680,            2804,            757,            1281,            2093,            890,            0,            541,            1749,            1622,            1300,            3059,            1712,            890,            4274,            1554,            1152,            2825,            2987,            4007,            3047,            1111,            ],        [            443,            699,            1038,            3979,            306,            4296,            1316,            1515,            573,            2190,            3824,            655,            1405,            4468,            852,            3059,            3329,            978,            652,            3706,            955,            2499,            4499,            1601,            3607,            1977,            1315,            2579,            1224,            879,            1466,            2303,            3987,            691,            1707,            3404,            397,            2621,            2036,            631,            1243,            989,            1674,            2214,            1732,            1859,            3938,            3062,            1015,            740,            2351,            349,            648,            0,            1550,            1423,            1165,            3317,            1513,            749,            4532,            1812,            1027,            3083,            3245,            4265,            3305,            1369,            ],        [            1524,            1591,            2234,            4965,            1387,            5282,            1822,            2501,            2011,            3176,            4810,            2476,            2653,            5454,            2020,            4045,            4315,            1714,            2220,            4692,            1917,            3485,            5485,            2587,            4593,            2963,            2383,            3565,            1648,            1494,            2452,            3289,            4973,            2512,            2693,            4390,            2053,            3607,            712,            1003,            390,            783,            418,            890,            476,            603,            4924,            4048,            2001,            1354,            3335,            1207,            1696,            2237,            0,            372,            2413,            4301,            189,            2586,            5518,            2798,            2848,            4069,            4231,            5251,            4291,            2355,            ],        [            1417,            1484,            2127,            4858,            1280,            5175,            1715,            2394,            1904,            3069,            4703,            2369,            2546,            5347,            1913,            3938,            4206,            1607,            2113,            4585,            1810,            3378,            5376,            2480,            4486,            2856,            2276,            3458,            1541,            1387,            2345,            3182,            4866,            2405,            2586,            4283,            1946,            3500,            952,            896,            283,            1023,            251,            1130,            309,            436,            4817,            3941,            1894,            1247,            3230,            1100,            1589,            2130,            466,            0,            2306,            4196,            429,            2479,            5411,            2691,            2741,            3960,            4122,            5142,            4184,            2248,            ],        [            825,            1140,            179,            4514,            962,            4831,            591,            2050,            1560,            2723,            4357,            1857,            240,            5001,            379,            3594,            3862,            1263,            513,            4241,            482,            3032,            5034,            2136,            4142,            2512,            1932,            3114,            765,            919,            2001,            2838,            4520,            2061,            2242,            3960,            1602,            3156,            2917,            1423,            2148,            1781,            2555,            3095,            2613,            2740,            4471,            3597,            1550,            1532,            2886,            1141,            1245,            1786,            2431,            2304,            0,            3850,            2394,            1951,            5065,            2347,            2229,            3616,            3778,            4798,            3840,            1904,            ],        [            3507,            2926,            3733,            1124,            3628,            1441,            3321,            2560,            2875,            1043,            969,            3190,            4152,            1613,            3519,            1118,            12,            3092,            3638,            851,            3416,            734,            1644,            2973,            2257,            3022,            2810,            2693,            3147,            2993,            2606,            1634,            1132,            3166,            2040,            2054,            3138,            1250,            4847,            3953,            4078,            4918,            4485,            5025,            4543,            4670,            1083,            933,            2510,            4062,            882,            3671,            3031,            3322,            4361,            4234,            3912,            0,            4324,            3464,            1677,            2857,            3025,            228,            366,            1410,            690,            2414,            ],        [            1403,            1726,            2369,            5098,            1266,            5417,            1957,            2636,            2146,            3309,            4943,            2611,            2788,            5589,            2155,            4180,            4448,            1849,            2355,            4825,            2052,            3618,            5618,            2722,            4749,            3098,            2518,            3700,            1783,            1629,            2587,            3424,            5106,            2647,            2828,            4546,            2188,            3742,            523,            1138,            525,            594,            758,            701,            816,            943,            5057,            4183,            2136,            1233,            3470,            1086,            1831,            2372,            327,            507,            2548,            4436,            0,            2721,            5653,            2933,            2983,            4202,            4364,            5384,            4426,            2490,            ],        [            1156,            629,            1756,            3495,            1019,            3812,            1344,            1033,            266,            1706,            3340,            300,            2175,            3984,            1565,            2577,            2845,            908,            1365,            3222,            1439,            2015,            4015,            1119,            3146,            1495,            833,            2097,            1170,            1016,            984,            1821,            3503,            336,            1225,            2943,            529,            2139,            2870,            1344,            2101,            1702,            2508,            3048,            2566,            2693,            3454,            2580,            533,            1453,            1867,            1062,            578,            713,            2384,            2257,            1935,            2833,            2347,            0,            4048,            1330,            672,            2599,            2761,            3781,            2823,            887,            ],        [            4658,            4077,            4884,            939,            4779,            548,            4472,            3711,            4026,            2194,            802,            4341,            5303,            64,            4670,            1831,            1163,            4243,            4789,            826,            4567,            1885,            751,            4124,            3408,            4080,            3961,            3844,            4298,            4144,            4083,            2785,            618,            4317,            3230,            3205,            4289,            2401,            6513,            5104,            5744,            6584,            5636,            6691,            5694,            5821,            594,            1646,            3661,            5213,            2033,            4822,            4182,            4473,            5512,            5385,            5063,            1151,            5990,            4615,            0,            3915,            4176,            1379,            1517,            267,            1841,            3565,            ],        [            2385,            1675,            2482,            3417,            2248,            3734,            2070,            297,            1624,            1628,            3262,            1351,            2901,            3906,            2268,            2499,            2767,            1841,            2387,            3144,            2165,            1937,            3937,            789,            3068,            165,            1559,            2019,            1896,            1742,            1355,            1743,            3425,            1551,            818,            2865,            1887,            2061,            3596,            2573,            2827,            2931,            3234,            3774,            3292,            3419,            3376,            2502,            1259,            2682,            1789,            2291,            1780,            2071,            3110,            2983,            2661,            2755,            3073,            1625,            3970,            0,            1186,            2521,            2683,            3703,            2745,            443,            ],        [            1407,            1276,            2002,            3730,            1270,            4047,            1991,            1548,            913,            1941,            3575,            373,            2369,            4219,            1816,            2812,            3080,            1555,            1616,            3457,            1919,            2250,            4250,            1166,            3381,            1086,            1480,            2332,            1817,            1663,            1219,            2056,            3738,            573,            1460,            3178,            1176,            2374,            3517,            1595,            2748,            1953,            3155,            3695,            3213,            3340,            3689,            2815,            1180,            1704,            2102,            1313,            1225,            1360,            3031,            2904,            2129,            3068,            2994,            647,            4283,            1251,            0,            2834,            2996,            4016,            3058,            1694,            ],        [            3741,            3160,            3967,            896,            3862,            1213,            3555,            2794,            3109,            1277,            741,            3424,            4386,            1385,            3753,            1352,            246,            3326,            3872,            623,            3650,            968,            1416,            3207,            2491,            3163,            3044,            2927,            3381,            3227,            2840,            1868,            904,            3400,            2313,            2288,            3372,            1484,            5081,            4187,            4312,            5152,            4719,            5259,            4777,            4904,            855,            1167,            2744,            4296,            1116,            3905,            3265,            3556,            4595,            4468,            4146,            234,            4558,            3698,            1449,            2998,            3259,            0,            600,            1182,            924,            2648,            ],        [            4425,            3844,            4651,            1227,            4546,            1544,            4239,            3478,            3793,            1146,            1072,            4108,            5070,            1716,            4437,            752,            577,            4010,            4556,            954,            4334,            837,            1747,            3891,            1891,            3847,            3728,            2327,            4065,            3911,            3524,            1737,            1235,            4084,            2997,            1688,            4056,            884,            5765,            4871,            4996,            5836,            5403,            5943,            5461,            5588,            1186,            567,            3428,            4980,            985,            4589,            3949,            4240,            5279,            5152,            4830,            565,            5242,            4382,            1780,            3682,            3943,            331,            0,            1513,            324,            3332,            ],        [            4391,            3810,            4617,            958,            4512,            815,            4205,            3444,            3759,            1927,            821,            4074,            5036,            203,            4403,            1564,            896,            3976,            4522,            559,            4300,            1618,            1018,            3857,            3141,            3813,            3694,            3577,            4031,            3877,            3490,            2518,            637,            4050,            2963,            2938,            4022,            2134,            7015,            4837,            6246,            7086,            5369,            7193,            5427,            5554,            327,            1379,            3394,            4946,            1766,            4555,            3915,            4206,            5245,            5118,            4796,            884,            6492,            4348,            267,            3648,            3909,            1112,            1250,            0,            1574,            3298,            ],        [            4125,            3544,            4351,            1460,            4246,            1777,            3939,            3178,            3493,            1149,            1305,            3808,            4770,            1415,            4137,            428,            810,            3710,            4256,            653,            4034,            840,            1980,            3591,            1591,            3547,            3428,            2027,            3765,            3611,            3224,            1740,            1468,            3784,            2697,            1388,            3756,            831,            5465,            4571,            4696,            5536,            5103,            5643,            5161,            5288,            885,            243,            3128,            4680,            988,            4289,            3649,            3940,            4979,            4852,            4530,            798,            4942,            4082,            1479,            3382,            3643,            564,            726,            1212,            0,            3032,            ],        [            1942,            1811,            2537,            3622,            1805,            3939,            2275,            146,            1448,            1833,            3467,            908,            2904,            4111,            2351,            2704,            2972,            2090,            2151,            3349,            2454,            2142,            4142,            346,            3273,            608,            1764,            2224,            2101,            1947,            1754,            1948,            3630,            1108,            1023,            3070,            1711,            2266,            3801,            2130,            3032,            2488,            3439,            3979,            3497,            3624,            3581,            2707,            1715,            2239,            1994,            1848,            1985,            1895,            3315,            3188,            2664,            2960,            3278,            1182,            4175,            443,            743,            2726,            2888,            3908,            2950,            0,            ],        ], 'starts': [27, 29, 54, 52, 3], 'ends': [27, 29, 54, 52, 3]},    'NW-12': {'distance_matrix': [        [            0,            3545,            4519,            3534,            4339,            4957,            2114,            1917,            1756,            823,            3319,            1685,            2501,            1661,            2174,            694,            2344,            2665,            2711,            3604,            1431,            2784,            2976,            1236,            3767,            6029,            2747,            2694,            1272,            2525,            2084,            3811,            2387,            846,            4169,            4492,            2433,            2472,            1761,            769,            2861,            4067,            3666,            3032,            1947,            2002,            2436,            3016,            4098,            2560,            2496,            3640,            2933,            2763,            3253,            1322,            2320,            2713,            1584,            2300,            2746,            1971,            4164,            2403,            2299,            4790,            1251,            4004,            3572,            988,            2574,            3722,            2135,            3879,            3597,            ],        [            6271,            0,            974,            1128,            1432,            1723,            3258,            3133,            2972,            3809,            5526,            2590,            2566,            2180,            1791,            4384,            1806,            2164,            2498,            1757,            2667,            1615,            1248,            3050,            1920,            6094,            1700,            2198,            2722,            2024,            2847,            1131,            2013,            3436,            624,            1585,            3338,            2908,            2120,            3040,            2360,            1580,            1180,            1220,            3062,            2083,            2047,            1847,            4255,            1617,            2571,            2054,            1087,            3704,            1918,            3300,            3435,            1298,            2699,            1983,            2139,            2876,            1677,            1538,            2768,            1767,            3099,            1517,            1656,            3576,            1631,            847,            3040,            1392,            1468,            ],        [            8364,            1674,            0,            979,            1140,            749,            4403,            4278,            4117,            5902,            4552,            3735,            3711,            3325,            2936,            6477,            2951,            3309,            3643,            1608,            3812,            1764,            2393,            4195,            1771,            7239,            2845,            3343,            3867,            3169,            3992,            982,            3158,            5925,            489,            1293,            4483,            4053,            3265,            4185,            3505,            1648,            1761,            2365,            4207,            3228,            3192,            1547,            3281,            2762,            3716,            2036,            2232,            2730,            2423,            4445,            3026,            2443,            3844,            3128,            3284,            4021,            1473,            2683,            3913,            793,            4244,            1423,            2060,            5669,            2776,            1992,            4185,            1548,            1872,            ],        [            5786,            1227,            857,            0,            1997,            1606,            4184,            3253,            3092,            3929,            5409,            2373,            3190,            1963,            1569,            3998,            1170,            1528,            1862,            629,            2450,            785,            1348,            2833,            792,            5872,            2324,            1976,            2505,            1388,            2630,            277,            1377,            3556,            507,            2150,            3121,            2691,            1903,            2823,            1724,            2204,            1804,            1844,            3182,            2707,            2167,            568,            4138,            2241,            3195,            2893,            1711,            3587,            2542,            3420,            3555,            1922,            2819,            1766,            2763,            2659,            2330,            2162,            2845,            1650,            3219,            2141,            2280,            3696,            2255,            1471,            2823,            2016,            2092,            ],        [            6546,            804,            1778,            1932,            0,            2527,            3533,            3408,            3247,            4084,            5556,            2865,            2841,            2455,            2066,            4659,            2081,            2439,            2773,            2710,            2942,            1890,            1523,            3325,            2873,            6369,            1975,            2473,            2997,            2299,            3122,            1935,            2288,            3711,            1428,            423,            3613,            3183,            2395,            3315,            2635,            1856,            1455,            1495,            3337,            2358,            2322,            2122,            4285,            1892,            2846,            2329,            1362,            3734,            2193,            3575,            3710,            1573,            2974,            2258,            2414,            3151,            1953,            1813,            3043,            2571,            3374,            1793,            1931,            3851,            1906,            1122,            3315,            1668,            1743,            ],        [            6667,            925,            1899,            2053,            391,            0,            3654,            3529,            3368,            4205,            4627,            2986,            2962,            2576,            2187,            4780,            2202,            2560,            2894,            2831,            3063,            2011,            1644,            3446,            2994,            6490,            2096,            2594,            3118,            2420,            3243,            2056,            2409,            3832,            1549,            544,            3734,            3304,            2516,            3436,            2756,            899,            1012,            1616,            3458,            2479,            2443,            2243,            3356,            2013,            2967,            2111,            1483,            2805,            2314,            3696,            3101,            1694,            3095,            2379,            2535,            3272,            764,            1934,            3164,            882,            3495,            674,            1311,            3972,            2027,            1243,            3436,            799,            1123,            ],        [            5017,            3426,            4876,            3983,            4884,            4493,            0,            2053,            1892,            2555,            1205,            3510,            3575,            3100,            3419,            3130,            3589,            3910,            3956,            4053,            3587,            3233,            2857,            3672,            4216,            7258,            2628,            3923,            3708,            3770,            3767,            4260,            3632,            2578,            5073,            5037,            4258,            3828,            2869,            3205,            4106,            4123,            4236,            2913,            1828,            1883,            2317,            3465,            3062,            2441,            3580,            3651,            2814,            2952,            4038,            2220,            2201,            2594,            1465,            3545,            3266,            3796,            3738,            2284,            3777,            4326,            2723,            3898,            3453,            2322,            2455,            3603,            3960,            4023,            4347,            ],        [            4382,            2749,            3723,            3306,            3543,            4161,            837,            0,            413,            1447,            1934,            2840,            2377,            2430,            2749,            2495,            2919,            2957,            3286,            3376,            2917,            2556,            2180,            3300,            3539,            6588,            1951,            3253,            2972,            2817,            3097,            3583,            2962,            1336,            3373,            3696,            3588,            3158,            2199,            2570,            3153,            3271,            2870,            2236,            1151,            1206,            1640,            2788,            3734,            1764,            2382,            2844,            2137,            1967,            2457,            741,            1524,            1917,            788,            2875,            1950,            3126,            3368,            1607,            2579,            3994,            999,            3208,            2776,            1408,            1778,            2926,            3290,            3083,            2801,            ],        [            4119,            2868,            3842,            3425,            3662,            4280,            1437,            351,            0,            1034,            2285,            2959,            2496,            2549,            2868,            2232,            3038,            3076,            3405,            3495,            2969,            2675,            2299,            2774,            3658,            6707,            2070,            3372,            2810,            2936,            3216,            3702,            3081,            923,            3492,            3815,            3707,            3277,            2318,            2307,            3272,            3390,            2989,            2355,            1270,            1325,            1759,            2907,            3421,            1883,            2501,            2963,            2256,            2086,            2576,            328,            1643,            2036,            907,            2994,            2069,            3245,            3487,            1726,            2698,            4113,            586,            3327,            2895,            995,            1897,            3045,            3409,            3202,            2920,            ],        [            4313,            3345,            4319,            3902,            4139,            4757,            1914,            1717,            1044,            0,            3119,            3417,            2973,            3026,            3345,            2426,            3515,            3553,            3882,            3972,            3163,            3152,            2776,            2968,            4135,            7184,            2547,            3849,            3004,            3413,            3816,            4179,            3558,            111,            3969,            4292,            4165,            3754,            2795,            2501,            3749,            3867,            3466,            2832,            1747,            1802,            2236,            3384,            3898,            2360,            2978,            3440,            2733,            2563,            3053,            690,            2120,            2513,            1384,            3471,            2546,            3703,            3964,            2203,            3175,            4590,            532,            3804,            3372,            277,            2374,            3522,            3867,            3679,            3397,            ],        [            3986,            2221,            3195,            2778,            3015,            3633,            854,            848,            687,            1524,            0,            2312,            1849,            1902,            2221,            2099,            2391,            2429,            2758,            2848,            2389,            2028,            1652,            2772,            3011,            6060,            1423,            2725,            2444,            2289,            2569,            3055,            2434,            1151,            2845,            3168,            3060,            2630,            1671,            2174,            2625,            2743,            2342,            1708,            623,            678,            1112,            2260,            2774,            1236,            1854,            2316,            1609,            1439,            1929,            1015,            996,            1389,            260,            2347,            1422,            2598,            2840,            1079,            2051,            3466,            814,            2680,            2248,            1291,            1250,            2398,            2762,            2555,            2273,            ],        [            4111,            3115,            3844,            2987,            3635,            4253,            3637,            3517,            3356,            2465,            4842,            0,            1490,            693,            1627,            2323,            1797,            2118,            2164,            3057,            739,            2237,            1956,            1158,            3220,            4978,            3013,            2034,            568,            1978,            1073,            3264,            1840,            2488,            3739,            3788,            748,            2144,            710,            1148,            2314,            3637,            3236,            3276,            3446,            3501,            3935,            2469,            5597,            4059,            1485,            3934,            3143,            4262,            3547,            3684,            3819,            2720,            3083,            1753,            2689,            286,            3734,            3902,            1288,            4086,            3483,            3574,            3579,            3960,            4073,            2960,            450,            3449,            3524,            ],        [            5059,            2151,            3125,            2899,            2945,            3563,            2041,            1921,            1760,            2597,            3246,            1370,            0,            1192,            1705,            3172,            2340,            2550,            2242,            2969,            1447,            2149,            1773,            1830,            3132,            5985,            751,            2650,            1502,            2410,            417,            3176,            2547,            2224,            2775,            3098,            2118,            530,            1571,            1820,            2746,            2673,            2272,            1638,            1850,            799,            1233,            2381,            3861,            1058,            5,            1672,            1535,            2666,            1285,            2088,            2223,            1319,            1487,            1831,            427,            1656,            2770,            901,            202,            3396,            1887,            2610,            2178,            2364,            1180,            2328,            1820,            2485,            2203,            ],        [            3823,            2245,            2974,            2117,            2765,            3383,            2548,            2428,            2267,            2177,            3753,            410,            1084,            0,            757,            2035,            927,            1248,            1294,            2187,            487,            1367,            1086,            870,            2350,            4612,            1835,            1277,            542,            1108,            667,            2394,            970,            2200,            2624,            2918,            1158,            1055,            344,            860,            1444,            2767,            2366,            2406,            2357,            2412,            2846,            1599,            4508,            2142,            1079,            2756,            2273,            3173,            2369,            2595,            2730,            1850,            1994,            883,            1511,            696,            2864,            1985,            882,            3216,            2592,            2704,            2709,            2342,            2984,            2090,            860,            2579,            2654,            ],        [            8225,            2324,            2791,            1934,            2844,            3462,            3605,            3221,            3060,            3897,            4810,            1794,            2854,            1384,            0,            3419,            744,            641,            537,            2004,            1871,            1184,            1165,            2254,            2167,            4303,            2965,            407,            1926,            679,            2051,            2211,            546,            3524,            2441,            2997,            3360,            2112,            1324,            2244,            671,            2846,            2445,            2485,            3150,            2714,            2135,            1416,            5275,            2882,            2859,            3319,            2352,            3966,            3183,            3388,            3523,            1929,            2787,            962,            2427,            2080,            2943,            2464,            2266,            3295,            3187,            2783,            2788,            3664,            2262,            2169,            4601,            2658,            2733,            ],        [            3038,            2968,            3697,            2840,            3488,            4106,            3271,            2473,            2312,            1392,            4476,            991,            1807,            967,            1480,            0,            1650,            1971,            2017,            2910,            737,            2090,            1809,            542,            3073,            5335,            2653,            2000,            578,            1831,            1390,            3117,            1693,            1415,            3347,            3641,            1739,            1778,            1067,            75,            2167,            3490,            3089,            3129,            2503,            2558,            2992,            2322,            4654,            3116,            1802,            3574,            2996,            3319,            3187,            1878,            2876,            2573,            2140,            1606,            2329,            1277,            3587,            2959,            1605,            3939,            1807,            3427,            3432,            1557,            3130,            2813,            1441,            3302,            3377,            ],        [            4628,            1746,            2047,            1190,            2266,            2796,            3026,            2643,            2482,            3319,            4231,            1215,            2276,            805,            411,            2840,            0,            370,            704,            1260,            1292,            440,            738,            1675,            1423,            4714,            2015,            818,            1347,            230,            1472,            1467,            219,            2946,            1697,            2419,            1963,            1533,            745,            1665,            566,            2268,            1867,            1907,            2835,            2074,            1557,            672,            4697,            1981,            2281,            2741,            1774,            3651,            2605,            2810,            3208,            1351,            2472,            608,            1849,            1501,            2365,            1824,            1687,            2717,            2609,            2205,            2210,            3086,            1684,            1591,            1665,            2080,            2155,            ],        [            4936,            2115,            2416,            1559,            2635,            3165,            3334,            3214,            3053,            3890,            4539,            1523,            2197,            1113,            719,            3148,            369,            0,            330,            1316,            1600,            809,            1107,            1983,            1495,            5022,            2384,            1126,            1655,            140,            1780,            1836,            309,            3517,            2066,            2788,            2271,            1841,            1053,            1973,            196,            2637,            2236,            2276,            3143,            2443,            1926,            1041,            5066,            2350,            2192,            3110,            2143,            3959,            2974,            3381,            3516,            1720,            2780,            916,            2218,            1809,            2734,            2193,            1995,            3086,            3180,            2574,            2579,            3657,            2053,            1960,            1973,            2449,            2524,            ],        [            5134,            2417,            2718,            1861,            2937,            3467,            3532,            3412,            3251,            4088,            4737,            1721,            2395,            1311,            917,            3346,            671,            330,            0,            1599,            1798,            1111,            1409,            2181,            2094,            4774,            2686,            878,            1853,            470,            1978,            2138,            473,            3715,            2368,            3090,            2469,            2039,            1251,            2171,            134,            2939,            2538,            2578,            3341,            2745,            2228,            1343,            5368,            2652,            2390,            3412,            2445,            4157,            3276,            3579,            3714,            2022,            2978,            1114,            2520,            2007,            3036,            2495,            2193,            3388,            3378,            2876,            2881,            3855,            2355,            2262,            2171,            2751,            2826,            ],        [            5604,            1631,            1261,            404,            2401,            2010,            4002,            3071,            2910,            3747,            5813,            2191,            2704,            1781,            1387,            3816,            988,            1346,            1680,            0,            2268,            603,            1166,            2651,            461,            5690,            2443,            1794,            2323,            1206,            2448,            681,            1195,            3374,            911,            2554,            2939,            2509,            1721,            2641,            1542,            2608,            2295,            2335,            3000,            2502,            1985,            386,            4542,            2409,            2709,            3297,            2202,            3991,            3033,            3238,            3373,            1779,            2637,            1584,            2277,            2477,            2734,            2252,            2663,            2054,            3037,            2545,            2638,            3514,            2112,            1875,            2641,            2420,            2583,            ],        [            3712,            2597,            3326,            2469,            3117,            3735,            2900,            2780,            2619,            2066,            4105,            620,            1447,            596,            1109,            1924,            1279,            1600,            1646,            2539,            0,            1719,            1438,            759,            2702,            4964,            2024,            1629,            431,            1460,            1019,            2746,            1322,            2089,            2976,            3270,            1368,            1407,            696,            749,            1796,            3119,            2718,            2758,            2709,            2764,            3198,            1951,            4860,            2331,            1452,            2945,            2625,            3525,            2558,            2947,            3082,            2202,            2346,            1235,            1700,            906,            3216,            2174,            1234,            3568,            2481,            3056,            3061,            2231,            3336,            2442,            1070,            2931,            3006,            ],        [            5001,            1571,            1852,            995,            2091,            2601,            3399,            2468,            2307,            3144,            4604,            1588,            2101,            1178,            784,            3213,            385,            743,            1077,            1065,            1665,            0,            563,            2048,            1228,            5087,            1840,            1191,            1720,            603,            1845,            1272,            592,            2771,            1502,            2244,            2336,            1906,            1118,            2038,            939,            2093,            1692,            1732,            2397,            1899,            1382,            477,            4522,            1806,            2106,            2566,            1599,            3213,            2430,            2635,            2770,            1176,            2034,            981,            1674,            1874,            2190,            1649,            2060,            2542,            2434,            2030,            2035,            2911,            1509,            1416,            2038,            1905,            1980,            ],        [            4755,            2097,            2503,            1646,            2617,            3235,            3153,            3033,            2872,            3709,            4358,            1342,            2016,            932,            543,            2967,            456,            777,            1080,            1716,            1419,            896,            0,            1802,            1879,            4846,            2738,            950,            1474,            637,            1599,            1923,            626,            3336,            2153,            2770,            2090,            1660,            872,            1792,            973,            2619,            2218,            2258,            2962,            3017,            1908,            1128,            5048,            2655,            2011,            3092,            2125,            3778,            2956,            3200,            3335,            1702,            2599,            735,            2200,            1628,            2716,            2237,            1814,            3068,            2999,            2556,            2561,            3476,            2035,            1942,            1792,            2431,            2506,            ],        [            3490,            2993,            3722,            2865,            3513,            4131,            3296,            2925,            2764,            1844,            4501,            1016,            1832,            992,            1505,            1702,            1675,            1996,            2042,            2935,            762,            2115,            1834,            0,            3098,            5360,            2678,            2025,            603,            1856,            1415,            3142,            1718,            1867,            3372,            3666,            1764,            1803,            1092,            527,            2192,            3515,            3114,            3154,            3105,            3160,            3594,            2347,            5256,            2985,            1827,            3599,            3021,            3921,            3212,            2330,            3478,            2598,            2742,            1631,            2354,            1302,            3612,            2828,            1630,            3964,            2259,            3452,            3457,            2009,            3732,            2838,            1466,            3327,            3402,            ],        [            6181,            2208,            1838,            981,            2978,            2587,            3773,            3648,            3487,            4324,            9447,            2768,            3281,            2358,            1964,            4393,            1565,            1923,            2257,            577,            2845,            1180,            1743,            3228,            0,            6267,            3020,            2371,            2900,            1783,            3025,            1258,            1772,            3951,            1488,            3131,            3516,            3086,            2298,            3218,            2119,            3185,            2872,            2912,            3577,            3079,            2562,            963,            8176,            2986,            3286,            3874,            2779,            7625,            3610,            3815,            3950,            2356,            3214,            2161,            2854,            3054,            3311,            2829,            3240,            2631,            3614,            3122,            3215,            4091,            2689,            2452,            3218,            2997,            3160,            ],        [            4674,            2784,            3513,            2656,            3304,            3922,            3306,            3186,            3025,            3862,            4511,            1501,            2233,            1091,            1296,            2886,            1466,            1787,            1833,            2726,            1364,            1906,            1625,            1721,            2889,            0,            2682,            1703,            1131,            1647,            2040,            2933,            1509,            3489,            3408,            3457,            446,            1813,            1108,            1711,            1983,            3306,            2905,            2945,            3115,            3170,            3604,            2138,            6368,            3728,            2228,            3603,            2812,            3931,            3216,            3353,            3488,            2389,            2752,            1422,            2358,            462,            3403,            3571,            2031,            3755,            3152,            3243,            3248,            3629,            3742,            2629,            298,            3118,            3193,            ],        [            5156,            1545,            2519,            2580,            2339,            2957,            2143,            2018,            1857,            2694,            3348,            2228,            751,            1818,            1997,            3269,            2021,            2231,            2534,            2650,            2024,            1830,            1454,            2688,            2813,            6000,            0,            2665,            2360,            2091,            1168,            2857,            2228,            2321,            2169,            2492,            2976,            1281,            1586,            2678,            2427,            1322,            1666,            822,            1947,            480,            914,            2062,            3110,            739,            756,            921,            944,            2559,            534,            2185,            2320,            1000,            1584,            2189,            324,            2514,            2164,            388,            953,            2790,            1984,            2004,            1322,            2461,            861,            1846,            2678,            1879,            1134,            ],        [            7818,            6680,            9357,            6552,            7200,            9007,            7202,            7082,            6921,            7758,            8407,            5397,            6129,            4987,            5192,            6782,            5362,            5683,            5729,            6622,            5260,            5802,            5521,            5617,            6785,            3896,            6578,            0,            5027,            5543,            5936,            6829,            5405,            7385,            7304,            7353,            2953,            5709,            5004,            5607,            5879,            7202,            6801,            6841,            7011,            7066,            7500,            6034,            11475,            6885,            6124,            7499,            6708,            7827,            7112,            7249,            7384,            6285,            6648,            5318,            6254,            4358,            7299,            6728,            5927,            8807,            7048,            7139,            7144,            7525,            6618,            6525,            4194,            7014,            7089,            ],        [            3543,            2662,            3391,            2534,            3182,            3800,            2965,            2845,            2684,            1897,            4170,            685,            1501,            661,            1174,            1755,            1344,            1665,            1711,            2604,            431,            1784,            1503,            590,            2767,            5029,            2347,            1694,            0,            1525,            1084,            2811,            1387,            1920,            3041,            3335,            1153,            1472,            761,            580,            1861,            3184,            2783,            2823,            2774,            2829,            3263,            2016,            4925,            2654,            1496,            3268,            2690,            3590,            2881,            2383,            3147,            2267,            2411,            1300,            2023,            691,            3281,            2497,            1299,            3633,            2312,            3121,            3126,            2062,            3401,            2507,            855,            2996,            3071,            ],        [            4796,            1975,            2276,            1419,            2495,            3025,            3194,            3074,            2913,            3750,            4399,            1383,            2057,            973,            579,            3008,            229,            140,            432,            1489,            1460,            669,            967,            1843,            1652,            4882,            2244,            986,            1515,            0,            1640,            1696,            215,            3377,            1926,            2648,            2131,            1701,            913,            1833,            336,            2497,            2096,            2136,            3003,            2303,            1786,            901,            4926,            2210,            2052,            2970,            2003,            3819,            2834,            3241,            3376,            1580,            2640,            776,            2078,            1669,            2594,            2053,            1855,            2946,            3040,            2434,            2439,            3517,            1913,            1820,            1833,            2309,            2384,            ],        [            4366,            2568,            3542,            2648,            3362,            3980,            1848,            1728,            1567,            2404,            3053,            953,            417,            775,            1288,            2979,            1458,            1779,            1825,            2718,            1030,            1898,            1617,            1413,            2881,            5143,            1168,            1808,            1085,            1639,            0,            2925,            1501,            2031,            3192,            3515,            1701,            337,            875,            1403,            1975,            3090,            2689,            2055,            1657,            1216,            1650,            2130,            4278,            1475,            412,            2089,            1952,            2473,            1702,            1895,            2030,            1736,            1294,            1414,            844,            1239,            3187,            1318,            215,            3813,            1694,            3027,            2595,            2171,            1597,            2621,            1403,            2902,            2620,            ],        [            6618,            950,            580,            337,            1720,            1329,            3605,            3480,            3319,            4156,            5132,            2937,            2913,            2527,            2138,            4731,            1507,            1865,            2199,            966,            3014,            1122,            1595,            3397,            1129,            6441,            2047,            2545,            3069,            1725,            3194,            0,            1714,            3783,            230,            1873,            3685,            3255,            2467,            3387,            2061,            1927,            1527,            1567,            3409,            2430,            2394,            905,            3861,            1964,            2918,            2616,            1434,            3310,            2265,            3647,            3782,            1645,            3046,            2330,            2486,            3223,            2053,            1885,            3115,            1373,            3446,            1864,            2003,            3923,            1978,            1194,            3387,            1739,            1815,            ],        [            4661,            1969,            2270,            1413,            2489,            3019,            3059,            2939,            2778,            3615,            4264,            1248,            1922,            838,            444,            2873,            223,            301,            485,            1483,            1325,            663,            961,            1708,            1646,            4747,            2238,            851,            1380,            161,            1505,            1690,            0,            3242,            1920,            2642,            1996,            1566,            778,            1698,            497,            2491,            2090,            2130,            2868,            2923,            1780,            895,            4920,            2204,            1917,            2964,            1997,            3684,            2828,            3106,            3241,            1574,            2505,            641,            2072,            1534,            2588,            2047,            1720,            2940,            2905,            2428,            2433,            3382,            1907,            1814,            1698,            2303,            2378,            ],        [            4075,            3107,            4081,            3664,            3901,            4519,            1676,            1284,            933,            111,            2881,            2912,            2735,            2788,            3107,            1921,            3277,            3315,            3644,            3734,            2658,            2914,            2538,            2463,            3897,            6946,            2309,            3611,            2499,            3175,            3311,            3941,            3320,            0,            3731,            4054,            3660,            3516,            2557,            1996,            3511,            3629,            3228,            2594,            1509,            1564,            1998,            3146,            3660,            2122,            2740,            3202,            2495,            2325,            2815,            579,            1882,            2275,            1146,            3233,            2308,            3198,            3726,            1965,            2937,            4352,            421,            3566,            3134,            300,            2136,            3284,            3362,            3441,            3159,            ],        [            6615,            947,            360,            504,            1500,            1109,            3602,            3477,            3316,            4153,            4912,            2934,            2910,            2524,            2073,            4728,            1674,            2032,            2366,            1133,            3011,            1289,            1592,            3394,            1296,            6376,            2044,            2480,            3066,            1892,            3191,            507,            1881,            3780,            0,            1653,            3682,            3252,            2464,            3384,            2228,            1924,            1524,            1564,            3406,            2427,            2391,            1072,            3641,            1961,            2915,            2396,            1431,            3090,            2262,            3644,            3386,            1642,            3043,            2327,            2483,            3220,            1833,            1882,            3112,            1153,            3443,            1783,            2000,            3920,            1975,            1191,            3384,            1736,            1812,            ],        [            6123,            381,            1355,            1509,            1416,            2104,            3110,            2985,            2824,            3661,            5133,            2442,            2418,            2032,            1643,            4236,            1658,            2016,            2350,            2287,            2519,            1467,            1100,            2902,            2450,            5946,            1552,            2050,            2574,            1876,            2699,            1512,            1865,            3288,            1005,            0,            3190,            2760,            1972,            2892,            2212,            1433,            1032,            1072,            2914,            1935,            1899,            1699,            3862,            1469,            2423,            1906,            939,            3311,            1770,            3152,            3287,            1150,            2551,            1835,            1991,            2728,            1530,            1390,            2620,            2148,            2951,            1370,            1508,            3428,            1483,            699,            2892,            1245,            1320,            ],        [            4674,            2813,            3542,            2685,            3333,            3951,            3335,            3215,            3054,            3891,            4540,            1530,            2262,            1120,            1325,            2886,            1495,            1816,            1862,            2755,            1364,            1935,            1654,            1721,            2918,            3982,            2711,            1732,            1131,            1676,            2069,            2962,            1538,            3518,            3437,            3486,            0,            1842,            1137,            1711,            2012,            3335,            2934,            2974,            3144,            3199,            3633,            2167,            5295,            3757,            2257,            3632,            2841,            3960,            3245,            3382,            3517,            2418,            2781,            1451,            2387,            462,            3432,            3600,            2060,            3784,            3181,            3272,            3277,            3658,            3771,            2658,            298,            3147,            3222,            ],        [            5593,            2269,            3243,            3017,            3063,            3681,            2793,            2455,            2294,            3131,            3998,            2331,            752,            1921,            2434,            3706,            2458,            2668,            2971,            3087,            2025,            2267,            1891,            2545,            3250,            6103,            869,            2768,            2217,            2528,            1169,            3294,            2665,            2758,            2893,            3216,            3079,            0,            1689,            2535,            2864,            2791,            2390,            1756,            2384,            917,            1351,            2499,            3979,            1176,            757,            1790,            1653,            3200,            1403,            2622,            2757,            1437,            2021,            2117,            545,            2617,            2888,            1019,            954,            3514,            2421,            2728,            2296,            2898,            1298,            2446,            2781,            2603,            2321,            ],        [            4055,            2039,            2768,            1911,            2559,            3177,            2453,            2333,            2172,            3009,            3658,            642,            1571,            232,            551,            2267,            721,            1042,            1088,            1981,            719,            1161,            880,            1102,            2144,            4406,            1586,            1071,            774,            902,            899,            2188,            764,            2636,            2418,            2712,            1390,            960,            0,            1092,            1238,            2561,            2160,            2200,            2262,            1634,            2068,            1393,            4696,            1893,            1576,            2507,            2067,            3078,            2120,            2500,            2635,            1644,            1899,            677,            1262,            928,            2658,            1736,            1114,            3010,            2299,            2498,            2503,            2776,            2015,            1884,            1092,            2373,            2448,            ],        [            2963,            4026,            5000,            4015,            4820,            5438,            2595,            2398,            2237,            1317,            3800,            2166,            2982,            2142,            2655,            1175,            2825,            3146,            3192,            4085,            1912,            3265,            3457,            1717,            4248,            7252,            3228,            3175,            1753,            3006,            2565,            4292,            2868,            1340,            4650,            4973,            2914,            2953,            2242,            0,            3342,            4548,            4147,            3513,            2428,            2483,            2917,            3497,            4579,            3041,            2977,            4121,            3414,            3244,            3734,            1803,            2801,            3194,            2065,            2781,            3227,            2452,            4645,            2884,            2780,            5271,            1732,            4485,            4053,            1482,            3055,            4203,            2616,            4360,            4078,            ],        [            5132,            2311,            2612,            1755,            2831,            3361,            3530,            3410,            3249,            4086,            4735,            1719,            2393,            1309,            915,            3344,            565,            196,            134,            1476,            1796,            1005,            1303,            2179,            1655,            4921,            2580,            1025,            1851,            336,            1976,            2032,            469,            3713,            2262,            2984,            2467,            2037,            1249,            2169,            0,            2833,            2432,            2472,            3339,            2639,            2122,            1237,            5262,            2546,            2388,            3306,            2339,            4155,            3170,            3577,            3712,            1916,            2976,            1112,            2414,            2005,            2930,            2389,            2191,            3282,            3376,            2770,            2775,            3853,            2249,            2156,            2169,            2645,            2720,            ],        [            6115,            854,            1828,            1982,            938,            1534,            3102,            2977,            2816,            3653,            3988,            2898,            2161,            2488,            2099,            4228,            2114,            2472,            2806,            2743,            2975,            1923,            1556,            3358,            2906,            6402,            1410,            2506,            3030,            2332,            2578,            1985,            2321,            3280,            1478,            1091,            3646,            2691,            2428,            3348,            2668,            0,            401,            930,            2906,            1666,            1869,            2155,            2717,            1327,            2166,            1008,            797,            2166,            954,            3144,            2462,            1153,            2543,            2291,            1734,            3184,            385,            1248,            2363,            1367,            2943,            225,            412,            3420,            1486,            1155,            3348,            188,            224,            ],        [            6178,            453,            1427,            1581,            925,            1543,            3165,            3040,            2879,            3716,            4101,            2497,            2473,            2087,            1698,            4291,            1713,            2071,            2405,            2342,            2574,            1522,            1155,            2957,            2505,            6001,            1607,            2105,            2629,            1931,            2754,            1584,            1920,            3343,            1077,            1078,            3245,            2815,            2027,            2947,            2267,            401,            0,            1127,            2969,            1990,            1954,            1754,            2830,            1524,            2478,            1409,            994,            2279,            1355,            3207,            2575,            1205,            2606,            1890,            2046,            2783,            498,            1445,            2675,            1376,            3006,            338,            813,            3483,            1538,            754,            2947,            213,            625,            ],        [            5195,            899,            1873,            2027,            1693,            2311,            2182,            2057,            1896,            2733,            4561,            2784,            1247,            2374,            1985,            3308,            2000,            2358,            2692,            2629,            2520,            1809,            1442,            3244,            2792,            6496,            496,            2392,            2916,            2218,            1664,            2836,            2207,            2360,            1523,            1846,            3532,            1777,            2082,            3383,            2554,            1421,            1020,            0,            1986,            752,            953,            2041,            3290,            413,            1252,            1101,            298,            2739,            714,            2224,            2359,            687,            1623,            2177,            820,            3070,            1518,            334,            1449,            2144,            2023,            1358,            926,            2500,            781,            1200,            3234,            1233,            951,            ],        [            4122,            2462,            3436,            3019,            3256,            3874,            1267,            1142,            981,            1660,            3422,            2606,            2090,            2196,            2436,            2235,            2460,            2670,            2973,            3089,            2683,            2269,            1893,            2777,            3252,            6354,            1664,            3019,            2738,            2530,            2507,            3296,            2667,            1683,            3086,            3409,            3354,            2924,            1965,            2310,            2866,            2984,            2583,            1949,            0,            919,            1353,            2501,            2151,            1477,            2095,            2557,            1850,            816,            2170,            1309,            373,            1630,            363,            2641,            1663,            2892,            3081,            1320,            2292,            3707,            1108,            2921,            2489,            1427,            1491,            2639,            3056,            2796,            2514,            ],        [            4676,            1543,            2517,            2100,            2337,            2955,            1663,            1538,            1377,            2214,            2868,            1942,            1171,            1532,            1517,            2789,            1541,            1751,            2054,            2170,            2019,            1350,            974,            2402,            2333,            5714,            745,            1924,            2074,            1611,            1588,            2377,            1748,            1841,            2167,            2490,            2690,            1701,            1300,            2864,            1947,            2065,            1664,            1030,            1467,            0,            434,            1582,            3827,            558,            1176,            1638,            931,            2283,            1251,            1705,            1840,            711,            1104,            1709,            744,            2228,            2162,            401,            1373,            2788,            1504,            2002,            1570,            1981,            572,            1720,            2392,            1877,            1595,            ],        [            4946,            1296,            2270,            1666,            2090,            2708,            1933,            1808,            1647,            2484,            3138,            1882,            1441,            1472,            1083,            3059,            1107,            1317,            1620,            1736,            1959,            916,            540,            2342,            1899,            5386,            1180,            1490,            2014,            1177,            1858,            1943,            1314,            2111,            1920,            2243,            2630,            1971,            1412,            2332,            1513,            1818,            1417,            985,            1737,            1239,            0,            1148,            4247,            1142,            1446,            2101,            852,            2553,            1714,            1975,            2110,            464,            1374,            1275,            1014,            2168,            1915,            989,            1643,            2541,            1774,            1755,            1323,            2251,            797,            1473,            2332,            1630,            1348,            ],        [            5821,            1745,            1375,            518,            2515,            2124,            4219,            3288,            3127,            3964,            5927,            2408,            2921,            1998,            1604,            4033,            1205,            1563,            1897,            588,            2485,            820,            1383,            2868,            751,            5907,            2842,            2011,            2540,            1423,            2665,            795,            1412,            3591,            1025,            2668,            3156,            2726,            1938,            2858,            1759,            2722,            2322,            2362,            3217,            2719,            2202,            0,            4656,            2759,            2926,            3411,            2229,            4105,            3060,            3455,            3590,            1996,            2854,            1801,            2494,            2694,            2848,            2680,            2880,            2168,            3254,            2659,            2798,            3731,            2329,            1989,            2858,            2534,            2610,            ],        [            5083,            3492,            5064,            4049,            5072,            4681,            2125,            2119,            1958,            2621,            1271,            3576,            3641,            3166,            3485,            3196,            3655,            3976,            4022,            4119,            3653,            3299,            2923,            3738,            4282,            7324,            2694,            3989,            3774,            3836,            3833,            4326,            3698,            2644,            4116,            5225,            4324,            3894,            2935,            3271,            4172,            4311,            3613,            2979,            1894,            1949,            2383,            3531,            0,            2507,            3646,            3839,            2880,            3140,            4226,            2286,            2267,            2660,            1531,            3611,            3332,            3862,            3926,            2350,            3843,            4514,            2789,            4086,            3519,            2388,            2521,            3669,            4026,            4211,            3544,            ],        [            5211,            1887,            2861,            2635,            2681,            3299,            2198,            2073,            1912,            2749,            3403,            2283,            806,            1873,            2052,            3324,            2076,            2286,            2589,            2705,            2079,            1885,            1509,            2743,            2868,            6055,            1043,            2720,            2415,            2146,            1223,            2912,            2283,            2376,            2511,            2834,            3031,            1336,            1641,            2733,            2482,            2409,            2008,            1374,            2002,            535,            969,            2117,            4125,            0,            811,            1936,            1271,            2818,            1549,            2240,            2375,            1055,            1639,            2244,            379,            2569,            2506,            637,            1008,            3132,            2039,            2346,            1914,            2516,            916,            2064,            2733,            2221,            1939,            ],        [            5054,            2156,            3130,            2904,            2950,            3568,            2036,            1916,            1755,            2592,            3241,            1365,            5,            1187,            1700,            3167,            2345,            2555,            2237,            2974,            1442,            2154,            1778,            1825,            3137,            5990,            756,            2655,            1497,            2415,            412,            3181,            2552,            2219,            2780,            3103,            2113,            525,            1576,            1815,            2751,            2678,            2277,            1643,            1845,            804,            1238,            2386,            3866,            1063,            0,            1677,            1540,            2661,            1290,            2083,            2218,            1324,            1482,            1826,            432,            1651,            2775,            906,            197,            3401,            1882,            2615,            2183,            2359,            1185,            2333,            1815,            2490,            2208,            ],        [            7272,            1683,            2262,            2949,            2270,            1879,            3064,            2939,            2778,            4810,            3460,            3149,            1672,            2739,            2918,            5385,            2942,            3152,            3455,            3571,            2945,            2751,            2375,            3609,            3734,            6921,            921,            3586,            3281,            3012,            2089,            3778,            3149,            4833,            2307,            2423,            3897,            2202,            2507,            5460,            3348,            1008,            1409,            1098,            2307,            1401,            1835,            2983,            2189,            1461,            1677,            0,            1082,            1638,            387,            3106,            1934,            1735,            2505,            3110,            1245,            3435,            1124,            1382,            1874,            1712,            2905,            1233,            1008,            4577,            1782,            1984,            3599,            1196,            820,            ],        [            5491,            601,            1575,            1729,            1395,            2013,            2478,            2353,            2192,            3029,            4823,            2397,            1797,            1987,            1598,            3604,            1613,            1971,            2305,            2242,            2474,            1422,            1055,            2857,            2405,            5901,            786,            2005,            2529,            1831,            2214,            2449,            1820,            2656,            1225,            1548,            3145,            2327,            1927,            2847,            2167,            1123,            722,            306,            2282,            1314,            1245,            1654,            3552,            703,            1802,            1239,            0,            3001,            1004,            2520,            2655,            529,            1919,            1790,            1370,            2683,            1220,            624,            1999,            1846,            2319,            1060,            628,            2796,            862,            902,            2847,            935,            653,            ],        [            4791,            3131,            3739,            3688,            3747,            3356,            1936,            1811,            1650,            2329,            2789,            3275,            2759,            2865,            3105,            2904,            3129,            3339,            3642,            3758,            3352,            2938,            2562,            3446,            3921,            7023,            2333,            3688,            3407,            3199,            3176,            3965,            3336,            2352,            3936,            3900,            4023,            3593,            2634,            2979,            3535,            2986,            3099,            2618,            669,            1588,            2022,            3170,            1518,            2146,            2764,            2514,            2519,            0,            2901,            1978,            296,            2299,            1032,            3310,            2332,            3561,            2601,            1989,            2961,            3189,            1777,            2761,            3158,            2096,            2160,            3308,            3725,            2886,            3183,            ],        [            5690,            1499,            2473,            2627,            2293,            2266,            2677,            2552,            2391,            3228,            3847,            2762,            1285,            2352,            2531,            3803,            2555,            2765,            3068,            3184,            2558,            2364,            1988,            3222,            3347,            6534,            534,            3199,            2894,            2625,            1702,            3391,            2762,            2855,            2123,            2446,            3510,            1815,            2120,            3212,            2961,            954,            1620,            773,            2481,            1014,            1448,            2596,            2576,            1074,            1290,            387,            898,            2025,            0,            2719,            2321,            1348,            2118,            2723,            858,            3048,            1511,            995,            1487,            2099,            2518,            1179,            954,            2995,            1395,            1800,            3212,            1142,            766,            ],        [            3803,            2835,            3809,            3392,            3629,            4247,            1404,            705,            354,            706,            2639,            2907,            2463,            2516,            2835,            1916,            3005,            3043,            3372,            3462,            2653,            2642,            2266,            2458,            3625,            6674,            2037,            3339,            2494,            2903,            3306,            3669,            3048,            595,            3459,            3782,            3655,            3244,            2285,            1991,            3239,            3357,            2956,            2322,            1237,            1292,            1726,            2874,            3388,            1850,            2468,            2930,            2223,            2053,            2543,            0,            1610,            2003,            874,            2961,            2036,            3193,            3454,            1693,            2665,            4080,            258,            3294,            2862,            667,            1864,            3012,            3357,            3169,            2887,            ],        [            4495,            2835,            3809,            3392,            3629,            3616,            1640,            1515,            1354,            2033,            3049,            2979,            2463,            2569,            2809,            2608,            2833,            3043,            3346,            3462,            3056,            2642,            2266,            3150,            3625,            6727,            2037,            3392,            3111,            2903,            2880,            3669,            3040,            2056,            3459,            3782,            3727,            3297,            2338,            2683,            3239,            3246,            2956,            2322,            373,            1292,            1726,            2874,            1778,            1850,            2468,            2774,            2223,            443,            3161,            1682,            0,            2003,            736,            3014,            2036,            3265,            2861,            1693,            2665,            3449,            1481,            3021,            2862,            1800,            1864,            3012,            3429,            3169,            2887,            ],        [            5191,            832,            1806,            1960,            1626,            2244,            2178,            2053,            1892,            2729,            3383,            2097,            1497,            1687,            1298,            3304,            1313,            1671,            2005,            1942,            2174,            1122,            755,            2557,            2105,            5601,            1001,            1705,            2229,            1531,            1914,            2149,            1520,            2356,            1456,            1779,            2845,            2027,            1627,            2547,            1867,            1354,            953,            521,            1982,            1014,            945,            1354,            3783,            918,            1502,            1606,            388,            2798,            1219,            2220,            2355,            0,            1619,            1490,            1070,            2383,            1451,            839,            1699,            2077,            2019,            1291,            859,            2496,            562,            1009,            2547,            1166,            884,            ],        [            3759,            2099,            3073,            2656,            2893,            3511,            904,            779,            618,            1297,            2109,            2243,            1727,            1833,            2073,            1872,            2097,            2307,            2610,            2726,            2320,            1906,            1530,            2414,            2889,            5991,            1301,            2656,            2375,            2167,            2144,            2933,            2304,            1320,            2723,            3046,            2991,            2561,            1602,            1947,            2503,            2621,            2220,            1586,            363,            556,            990,            2138,            2514,            1114,            1732,            2194,            1487,            1179,            1807,            946,            736,            1267,            0,            2278,            1300,            2529,            2718,            957,            1929,            3344,            745,            2558,            2126,            1064,            1128,            2276,            2693,            2433,            2151,            ],        [            5397,            1362,            2336,            1568,            1882,            2500,            2384,            2259,            2098,            2935,            3589,            1545,            1892,            1135,            746,            3510,            659,            980,            1283,            1638,            1622,            818,            203,            2005,            1801,            5049,            2003,            1153,            1677,            840,            1802,            1845,            829,            2562,            1986,            2035,            2293,            1863,            1075,            1995,            1176,            1884,            1483,            1523,            2188,            1752,            1173,            1050,            4313,            1920,            1897,            2357,            1390,            3004,            2221,            2426,            2561,            967,            1825,            0,            1465,            1831,            1981,            1502,            2094,            2333,            2225,            1821,            1826,            2702,            1300,            1207,            1995,            1696,            1771,            ],        [            5048,            1724,            2698,            2472,            2518,            3136,            2468,            1910,            1749,            2586,            3673,            1904,            427,            1494,            1889,            3161,            1913,            2123,            2426,            2542,            1700,            1722,            1346,            2364,            2705,            5676,            324,            2341,            2036,            1983,            844,            2749,            2120,            2213,            2348,            2671,            2652,            957,            1262,            2354,            2319,            2246,            1845,            1211,            1839,            372,            806,            1954,            3434,            631,            432,            1245,            1108,            2655,            858,            2077,            2212,            892,            1476,            1690,            0,            2190,            2343,            474,            629,            2969,            1876,            2183,            1751,            2353,            753,            1901,            2354,            2058,            1776,            ],        [            4212,            2829,            3558,            2701,            3349,            3967,            3351,            3231,            3070,            2566,            4556,            1354,            2278,            845,            1341,            2424,            1511,            1832,            1878,            2771,            902,            1951,            1670,            1259,            2934,            4444,            2727,            1748,            669,            1692,            1236,            2978,            1554,            3534,            3453,            3502,            462,            1858,            913,            1249,            2028,            3351,            2950,            2990,            3160,            3215,            3649,            2183,            5311,            3773,            2273,            3648,            2857,            3976,            3261,            3398,            3533,            2434,            2797,            1467,            2403,            0,            3448,            3616,            2076,            3800,            3197,            3288,            3293,            3674,            3787,            2674,            164,            3163,            3238,            ],        [            7415,            951,            1925,            2079,            803,            1317,            3487,            3362,            3201,            4953,            3603,            2995,            2546,            2585,            2196,            5528,            2211,            2569,            2903,            2840,            3072,            2020,            1653,            3455,            3003,            6499,            1795,            2603,            3127,            2429,            2963,            2082,            2418,            4976,            1575,            956,            3743,            3313,            2525,            3445,            2765,            385,            498,            1315,            2450,            2051,            2254,            2252,            2332,            1712,            2551,            1087,            1182,            1781,            1474,            3529,            2077,            1538,            3863,            2388,            2119,            3281,            0,            1633,            2748,            1150,            3328,            160,            797,            4720,            1871,            1252,            3445,            285,            609,            ],        [            4999,            1235,            2209,            2363,            2029,            2647,            1986,            1861,            1700,            2537,            3191,            2378,            901,            1968,            1884,            3112,            1899,            2257,            2591,            2528,            2174,            1708,            1341,            2838,            2691,            6150,            406,            2291,            2510,            2117,            1318,            2735,            2106,            2164,            1859,            2182,            3126,            1431,            1736,            3187,            2453,            1757,            1356,            512,            1790,            418,            757,            1940,            3488,            157,            906,            1299,            634,            2606,            912,            2028,            2163,            586,            1427,            2076,            474,            2664,            1854,            0,            1103,            2480,            1827,            1694,            1262,            2304,            447,            1536,            2828,            1569,            1287,            ],        [            4857,            2353,            3327,            3101,            3147,            3765,            1839,            1719,            1558,            2395,            3044,            1168,            202,            990,            1503,            2970,            1673,            1994,            2040,            3171,            1245,            2351,            1975,            1628,            3334,            5358,            953,            2023,            1300,            1854,            215,            3378,            1716,            2022,            2977,            3300,            1916,            328,            1090,            1618,            2190,            2875,            2474,            1840,            1648,            1001,            1435,            2583,            4063,            1260,            197,            1874,            1737,            2464,            1487,            1886,            2021,            1521,            1285,            1629,            629,            1454,            2972,            1103,            0,            3598,            1685,            2812,            2380,            2162,            1382,            2530,            1618,            2687,            2405,            ],        [            8219,            1529,            550,            1237,            995,            604,            4335,            4210,            4049,            5757,            4407,            3667,            3643,            3257,            2806,            6332,            2407,            2765,            3099,            1866,            3744,            2022,            2325,            4127,            2029,            7109,            2777,            3213,            3799,            2625,            3924,            1240,            2614,            5780,            747,            1148,            4415,            3985,            3197,            4117,            2961,            1503,            1616,            2297,            4139,            3160,            3124,            1805,            3136,            2694,            3648,            1891,            2164,            2585,            2278,            4377,            2881,            2375,            3776,            3060,            3216,            3953,            1328,            2615,            3845,            0,            4176,            1278,            1915,            5524,            2708,            1924,            4117,            1403,            1727,            ],        [            3748,            2780,            3754,            3337,            3574,            4192,            1349,            957,            606,            448,            2554,            2852,            2408,            2461,            2780,            1861,            2950,            2988,            3317,            3407,            2598,            2587,            2211,            2403,            3570,            6619,            1982,            3284,            2439,            2848,            3251,            3614,            2993,            337,            3404,            3727,            3600,            3189,            2230,            1936,            3184,            3302,            2901,            2267,            1182,            1237,            1671,            2819,            3333,            1795,            2413,            2875,            2168,            1998,            2488,            252,            1555,            1948,            819,            2906,            1981,            3138,            3399,            1638,            2610,            4025,            0,            3239,            2807,            503,            1809,            2957,            3302,            3114,            2832,            ],        [            7575,            791,            1765,            1919,            713,            1471,            3327,            3202,            3041,            5113,            3763,            2835,            2386,            2425,            2036,            5688,            2051,            2409,            2743,            2680,            2912,            1860,            1493,            3295,            2843,            6339,            1635,            2443,            2967,            2269,            2803,            1922,            2258,            5136,            1415,            866,            3583,            3153,            2365,            3285,            2605,            225,            338,            1155,            3131,            1891,            2094,            2092,            2492,            1552,            2391,            1247,            1022,            1941,            1179,            3369,            2237,            1378,            2768,            2228,            1959,            3121,            160,            1473,            2588,            1304,            3168,            0,            637,            4880,            1711,            1092,            3285,            125,            449,            ],        [            5703,            700,            1674,            1828,            1494,            2112,            2690,            2565,            2404,            3241,            4670,            2609,            1749,            2199,            1810,            3816,            1825,            2183,            2517,            2454,            2686,            1634,            1267,            3069,            2617,            6113,            998,            2217,            2741,            2043,            2166,            1831,            2032,            2868,            1324,            1647,            3357,            2279,            2139,            3059,            2379,            812,            821,            518,            2494,            1254,            1457,            1866,            3399,            915,            1754,            1210,            385,            2848,            1156,            2732,            2867,            741,            2131,            2002,            1322,            2895,            1319,            836,            1951,            1945,            2531,            1159,            0,            3008,            1074,            1001,            3059,            1034,            624,            ],        [            4080,            3112,            4086,            3669,            3906,            4524,            1681,            1484,            1323,            277,            2886,            3184,            2740,            2793,            3112,            2193,            3282,            3320,            3649,            3739,            2930,            2919,            2543,            2735,            3902,            6951,            2314,            3616,            2771,            3180,            3583,            3946,            3325,            300,            3736,            4059,            3932,            3521,            2562,            2268,            3516,            3634,            3233,            2599,            1514,            1569,            2003,            3151,            3665,            2127,            2745,            3207,            2500,            2330,            2820,            697,            1887,            2280,            1151,            3238,            2313,            3470,            3731,            1970,            2942,            4357,            445,            3571,            3139,            0,            2141,            3289,            3634,            3446,            3164,            ],        [            4975,            1257,            2231,            2385,            2051,            2669,            1962,            1837,            1676,            2513,            3167,            2241,            1065,            1831,            1860,            3088,            1875,            2233,            2567,            2504,            2338,            1684,            1317,            2701,            2667,            6013,            428,            2267,            2373,            2093,            1482,            2711,            2082,            2140,            1881,            2204,            2989,            1595,            1599,            3163,            2429,            1779,            1378,            534,            1766,            582,            733,            1916,            3510,            345,            1070,            1321,            656,            2582,            934,            2004,            2139,            562,            1403,            2052,            638,            2527,            1876,            266,            1267,            2502,            1803,            1716,            1284,            2280,            0,            1558,            2691,            1591,            1309,            ],        [            5424,            846,            1820,            1518,            1366,            1984,            2411,            2286,            2125,            2962,            3616,            1743,            1719,            1333,            944,            3537,            959,            1317,            1651,            1588,            1820,            768,            401,            2203,            1751,            5247,            1212,            1351,            1875,            1177,            2000,            1795,            1166,            2589,            1470,            1519,            2491,            2061,            1273,            2193,            1513,            1368,            967,            972,            2215,            1236,            1200,            1000,            3797,            1129,            1724,            2057,            839,            3031,            1670,            2453,            2588,            451,            1852,            1136,            1292,            2029,            1465,            986,            1921,            1817,            2252,            1305,            1310,            2729,            784,            0,            2193,            1180,            1335,            ],        [            4376,            2665,            3394,            2537,            3185,            3803,            3187,            3067,            2906,            3743,            4392,            1382,            2114,            972,            1177,            2588,            1347,            1668,            1714,            2607,            1066,            1787,            1506,            1423,            2770,            4280,            2563,            1584,            833,            1528,            1921,            2814,            1390,            3370,            3289,            3338,            298,            1694,            989,            1413,            1864,            3187,            2786,            2826,            2996,            3051,            3485,            2019,            5147,            3609,            2109,            3484,            2693,            3812,            3097,            3234,            3369,            2270,            2633,            1303,            2239,            164,            3284,            3452,            1912,            3636,            3033,            3124,            3129,            3510,            3623,            2510,            0,            2999,            3074,            ],        [            7700,            666,            1640,            1794,            728,            1346,            3378,            3253,            3092,            5238,            3888,            2710,            2349,            2300,            1911,            5813,            1926,            2284,            2618,            2555,            2787,            1735,            1368,            3170,            2718,            6214,            1598,            2318,            2842,            2144,            2766,            1797,            2133,            3556,            1290,            881,            3458,            3028,            2240,            3160,            2480,            188,            213,            1118,            3182,            1854,            2167,            1967,            2617,            1515,            2354,            1196,            985,            2066,            1142,            3420,            2362,            1418,            2819,            2103,            1922,            2996,            285,            1436,            2551,            1179,            3219,            125,            600,            5005,            1751,            967,            3160,            0,            412,            ],        [            5891,            888,            1862,            2016,            1682,            2300,            2878,            2753,            2592,            3429,            4858,            2797,            1937,            2387,            1998,            4004,            2013,            2371,            2705,            2642,            2874,            1822,            1455,            3257,            2805,            6301,            1186,            2405,            2929,            2231,            2354,            2019,            2220,            3056,            1512,            1835,            3545,            2467,            2327,            3247,            2567,            1000,            1009,            706,            2682,            1442,            1645,            2054,            3587,            1103,            1942,            1398,            573,            3036,            1344,            2920,            3055,            929,            2319,            2190,            1510,            3083,            1507,            1024,            2139,            2133,            2719,            1347,            188,            3196,            1262,            1189,            3247,            1222,            0,            ],        [            4376,            2665,            3394,            2537,            3185,            3803,            3187,            3067,            2906,            3743,            4392,            1382,            2114,            972,            1177,            2588,            1347,            1668,            1714,            2607,            1066,            1787,            1506,            1423,            2770,            4280,            2563,            1584,            833,            1528,            1921,            2814,            1390,            3370,            3289,            3338,            298,            1694,            989,            1413,            1864,            3187,            2786,            2826,            2996,            3051,            3485,            2019,            5147,            3609,            2109,            3484,            2693,            3812,            3097,            3234,            3369,            2270,            2633,            1303,            2239,            164,            3284,            3452,            1912,            3636,            3033,            3124,            3129,            3510,            3623,            2510,            0,            2999,            3074,            ],        [            7700,            666,            1640,            1794,            728,            1346,            3378,            3253,            3092,            5238,            3888,            2710,            2349,            2300,            1911,            5813,            1926,            2284,            2618,            2555,            2787,            1735,            1368,            3170,            2718,            6214,            1598,            2318,            2842,            2144,            2766,            1797,            2133,            3556,            1290,            881,            3458,            3028,            2240,            3160,            2480,            188,            213,            1118,            3182,            1854,            2167,            1967,            2617,            1515,            2354,            1196,            985,            2066,            1142,            3420,            2362,            1418,            2819,            2103,            1922,            2996,            285,            1436,            2551,            1179,            3219,            125,            600,            5005,            1751,            967,            3160,            0,            412,            ],        [            5891,            888,            1862,            2016,            1682,            2300,            2878,            2753,            2592,            3429,            4858,            2797,            1937,            2387,            1998,            4004,            2013,            2371,            2705,            2642,            2874,            1822,            1455,            3257,            2805,            6301,            1186,            2405,            2929,            2231,            2354,            2019,            2220,            3056,            1512,            1835,            3545,            2467,            2327,            3247,            2567,            1000,            1009,            706,            2682,            1442,            1645,            2054,            3587,            1103,            1942,            1398,            573,            3036,            1344,            2920,            3055,            929,            2319,            2190,            1510,            3083,            1507,            1024,            2139,            2133,            2719,            1347,            188,            3196,            1262,            1189,            3247,            1222,            0,            ],        ], 'starts': [19, 23, 21, 24, 44], 'ends': [19, 23, 21, 24, 44]},    'NW-13': {'distance_matrix': [        [            0,            2227,            1472,            2510,            2123,            1837,            2460,            1824,            2332,            2689,            1913,            3528,            4115,            4392,            5839,            4939,            4682,            3665,            3815,            5308,            1588,            3015,            1326,            1701,            2213,            1817,            2830,            3078,            3673,            3809,            3886,            4009,            4717,            3136,            2535,            2938,            3967,            3990,            5540,            3058,            3600,            2082,            1546,            2367,            2633,            573,            2612,            966,            2275,            2552,            2219,            1506,            1951,            1287,            1002,            3706,            2561,            2784,            3323,            4466,            1640,            1376,            1503,            2451,            4508,            5160,            2543,            2597,            6334,            4792,            3642,            966,            3613,            5335,            4934,            2651,            1272,            4086,            2810,            3082,            588,            3469,            1224,            2261,            2724,            2785,            774,            2165,            879,            879,            3014,            3289,            1750,            3338,            924,            4100,            2404,            2694,            2264,            3773,            ],        [            2322,            0,            1738,            283,            2293,            1604,            1297,            1994,            1383,            2752,            3304,            3676,            4540,            3189,            4636,            3736,            3479,            3527,            3710,            4231,            3601,            5028,            4329,            1509,            4226,            3830,            1702,            2129,            3822,            3341,            4311,            3096,            4916,            4359,            3926,            4501,            3975,            4189,            4337,            3105,            2651,            4095,            2813,            4380,            4646,            2995,            4625,            1920,            1482,            4565,            2218,            4244,            2408,            4290,            3570,            5719,            4574,            4797,            5649,            6479,            2443,            2446,            1290,            2514,            4707,            3957,            4556,            987,            5131,            3589,            3785,            2398,            5939,            4132,            3731,            4664,            4275,            4305,            1861,            1729,            3010,            3525,            4394,            4274,            497,            4798,            1985,            4178,            2485,            4049,            787,            4206,            1942,            3385,            1802,            3632,            2467,            467,            1577,            2824,            ],        [            1379,            1253,            0,            1536,            2384,            1695,            988,            2085,            860,            1079,            1631,            2019,            3802,            2705,            4152,            3252,            2995,            2789,            2972,            3621,            1902,            3329,            2630,            229,            2527,            2131,            964,            1391,            3084,            2603,            3573,            2322,            4178,            3621,            2253,            2656,            3237,            3451,            3853,            1448,            1913,            2396,            1114,            2681,            2947,            1211,            2926,            799,            803,            2866,            609,            2545,            709,            2591,            1871,            4020,            2875,            3098,            3637,            4780,            2534,            708,            1381,            841,            3969,            3473,            2857,            1298,            4647,            3105,            3047,            906,            3927,            3648,            3247,            2965,            2576,            3567,            1123,            890,            1226,            1833,            2695,            2575,            1750,            3099,            1042,            2479,            993,            2350,            2040,            3468,            2033,            1728,            906,            2894,            794,            1720,            2525,            2086,            ],        [            2039,            424,            1455,            0,            2010,            1321,            1014,            1711,            1100,            2469,            3021,            3393,            4257,            2906,            4353,            3453,            3196,            3244,            3427,            3948,            3318,            4745,            4046,            1226,            3943,            3547,            1419,            1846,            3539,            3058,            4028,            2813,            4633,            4076,            3643,            4218,            3692,            3906,            4054,            2822,            2368,            3812,            2530,            4097,            4363,            2712,            4342,            1637,            1199,            4282,            1935,            3961,            2125,            4007,            3287,            5436,            4291,            4514,            5366,            6196,            2160,            2163,            1007,            2231,            4424,            3674,            4273,            704,            4848,            3306,            3502,            2115,            5656,            3849,            3448,            4381,            3992,            4022,            1578,            1446,            2727,            3242,            4111,            3991,            921,            4515,            1702,            3895,            2202,            3766,            1388,            3923,            1659,            3102,            1519,            3349,            2184,            184,            1655,            2541,            ],        [            2457,            1964,            2520,            2247,            0,            1739,            2508,            1135,            2165,            3534,            5512,            4458,            5322,            4097,            5544,            4644,            4387,            4309,            4492,            5139,            5187,            6614,            5360,            2291,            5812,            5416,            2484,            2911,            4604,            4123,            5093,            3842,            5698,            5141,            6134,            5283,            4757,            4971,            5245,            3887,            3433,            5681,            3595,            5966,            6232,            3130,            6211,            2055,            2323,            6151,            3000,            5540,            3190,            5321,            4601,            7305,            6160,            6383,            6922,            8065,            692,            2758,            1425,            3296,            5489,            4865,            6142,            2334,            6039,            4497,            4567,            2533,            7212,            5040,            4639,            6250,            5306,            5087,            2643,            2410,            3145,            4307,            4848,            5860,            2461,            6384,            2120,            5764,            2620,            4613,            1497,            4988,            1061,            4167,            1937,            4414,            3249,            2431,            707,            3606,            ],        [            718,            1624,            1115,            1907,            2122,            0,            2168,            1823,            1825,            2028,            2731,            2968,            4982,            3757,            5204,            4304,            4047,            3969,            4152,            4799,            2406,            3833,            2144,            1344,            3031,            2635,            2144,            2571,            4264,            3783,            4753,            3502,            5358,            4801,            3353,            3756,            4417,            4631,            4905,            2397,            3093,            2900,            2063,            3185,            3451,            1391,            3430,            316,            1983,            3370,            1558,            2324,            1658,            2105,            1820,            4524,            3379,            3602,            4141,            5284,            2272,            1019,            900,            1790,            5149,            4525,            3361,            1994,            5699,            4157,            4227,            794,            4431,            4700,            4299,            3469,            2090,            4747,            2303,            2070,            1406,            2782,            2042,            3079,            2121,            3603,            381,            2983,            881,            1697,            2411,            4648,            1771,            2677,            198,            4074,            1743,            2091,            2263,            3266,            ],        [            2377,            809,            982,            1092,            2348,            1659,            0,            2049,            824,            1576,            2128,            2500,            3364,            1898,            3345,            2445,            2188,            2351,            2534,            2940,            2425,            3852,            3153,            753,            3050,            2654,            469,            953,            2646,            2165,            3135,            1805,            3740,            3183,            2750,            3325,            2799,            3013,            3046,            1929,            1475,            2919,            1637,            3204,            3470,            2193,            3449,            1975,            185,            3389,            1042,            3068,            1232,            3114,            2394,            4543,            3398,            3621,            4473,            5303,            2498,            1690,            1345,            1338,            3531,            2666,            3380,            310,            3840,            2298,            2609,            1888,            4763,            2841,            2440,            3488,            3099,            3129,            685,            432,            2208,            2349,            3218,            3098,            1306,            3622,            2040,            3002,            1975,            2873,            1596,            3030,            1997,            2209,            1857,            2456,            1291,            1276,            2489,            1648,            ],        [            2214,            1721,            2277,            2004,            299,            1496,            2265,            0,            1922,            3291,            5811,            4215,            5079,            3854,            5301,            4401,            4144,            4066,            4249,            4896,            5486,            6913,            5659,            2048,            6111,            5715,            2241,            2668,            4361,            3880,            4850,            3599,            5455,            4898,            6433,            5040,            4514,            4728,            5002,            3644,            3190,            5980,            3352,            6265,            6531,            2887,            6510,            1812,            2080,            6450,            2757,            5839,            2947,            5620,            4900,            7604,            6459,            6682,            7221,            8364,            449,            2515,            1182,            3053,            5246,            4622,            6441,            2091,            5796,            4254,            4324,            2290,            7511,            4797,            4396,            6549,            5605,            4844,            2400,            2167,            2902,            4064,            5147,            6159,            2218,            6683,            1877,            6063,            2377,            3193,            1230,            4745,            818,            3924,            1694,            4171,            3006,            2188,            440,            3363,            ],        [            1553,            393,            969,            676,            1524,            835,            957,            1225,            0,            1983,            2535,            2907,            3771,            2526,            3973,            3073,            2816,            2758,            2941,            3568,            2832,            4259,            3560,            740,            3457,            3061,            933,            1360,            3053,            2572,            3542,            2433,            4147,            3590,            3157,            3732,            3206,            3420,            3674,            2336,            1882,            3326,            2044,            3611,            3877,            2226,            3856,            1151,            772,            3796,            1449,            3475,            1639,            3521,            2801,            4950,            3805,            4028,            4880,            5710,            1674,            1677,            521,            1745,            3938,            3294,            3787,            763,            4468,            2926,            3016,            1629,            5170,            3469,            3068,            3895,            3506,            3536,            1092,            859,            2241,            2756,            2877,            3505,            890,            4029,            1216,            3409,            1716,            2532,            1180,            3437,            1173,            2616,            1033,            2863,            1698,            860,            1665,            2055,            ],        [            2304,            2240,            1091,            2523,            3371,            2682,            1553,            3072,            1847,            0,            1246,            940,            2604,            1477,            2924,            2024,            1767,            870,            1214,            2393,            1605,            3402,            2703,            1320,            2600,            1849,            1203,            1339,            1886,            1405,            2375,            1094,            2980,            2423,            1868,            1844,            2039,            2253,            2625,            369,            715,            2094,            1187,            2379,            3020,            2365,            2999,            1724,            1368,            2939,            592,            2263,            782,            2664,            1944,            4093,            2329,            3171,            3421,            4853,            3521,            1633,            2324,            238,            2771,            2245,            2930,            1863,            3419,            1877,            1041,            1831,            3711,            2420,            2019,            2663,            2227,            2369,            1183,            1455,            2380,            754,            2768,            2273,            2737,            3172,            1967,            2552,            1918,            2423,            3027,            2270,            3020,            649,            1831,            1696,            285,            2707,            3512,            888,            ],        [            2241,            3020,            1871,            3303,            4453,            3565,            2333,            4154,            2627,            1474,            0,            1615,            2202,            3524,            4971,            4071,            3814,            1752,            1902,            3704,            709,            2194,            1442,            2100,            1337,            953,            1983,            2966,            1760,            1896,            1973,            3141,            2804,            1223,            622,            1025,            2054,            2077,            4672,            2073,            2364,            1198,            1300,            1483,            1702,            1808,            1947,            2631,            2148,            1783,            1372,            1367,            1705,            1403,            1387,            2499,            1083,            1306,            1845,            3259,            6261,            2540,            3231,            1236,            2595,            4292,            2072,            2643,            5466,            3924,            1729,            2130,            2135,            4467,            4066,            2523,            1331,            2173,            1963,            2235,            1823,            1556,            1612,            1377,            3517,            1578,            2507,            1289,            2043,            1866,            3807,            1376,            4102,            1906,            2986,            2187,            1189,            3487,            4292,            2537,            ],        [            2208,            3190,            2031,            3473,            4321,            3632,            2503,            4022,            2797,            940,            317,            0,            2938,            1811,            3258,            2358,            2101,            1204,            1548,            2727,            676,            2201,            1409,            2260,            1304,            920,            2153,            1673,            2220,            1739,            2709,            1428,            3314,            1230,            629,            1032,            2373,            2587,            2959,            571,            1049,            1165,            1267,            1450,            1724,            1775,            1703,            2664,            2318,            1643,            1532,            1334,            1722,            1370,            1354,            2506,            1090,            1313,            1852,            3266,            4471,            2573,            3318,            1178,            3105,            2579,            1634,            2813,            3753,            2211,            1375,            2097,            2142,            2754,            2353,            1734,            1298,            2703,            2153,            2405,            1790,            1088,            1579,            1344,            3687,            1585,            2474,            1256,            2010,            1833,            3977,            1383,            3970,            291,            3830,            2030,            1225,            3657,            4462,            1222,            ],        [            3643,            4053,            3818,            4336,            5184,            4495,            3366,            4885,            3660,            2328,            1760,            1633,            0,            2557,            4004,            3104,            2847,            1672,            1010,            2271,            2111,            2463,            2844,            3589,            2739,            2355,            3016,            2536,            859,            1486,            229,            2711,            1396,            702,            2072,            1043,            565,            669,            3705,            2388,            1934,            2600,            2702,            2568,            1971,            3210,            2216,            4033,            3181,            2052,            3375,            2769,            3107,            2805,            2789,            2768,            1871,            1611,            1109,            3528,            5334,            3942,            4181,            2566,            1187,            3325,            2341,            4026,            3942,            2957,            1406,            3532,            1399,            3500,            3099,            2792,            2733,            765,            3016,            3268,            3225,            1574,            3014,            2779,            4550,            1847,            3909,            2691,            3445,            3268,            4840,            826,            4833,            1924,            4693,            1253,            2613,            4520,            5325,            2107,            ],        [            4098,            2497,            2703,            2780,            5881,            3380,            1876,            5582,            2545,            1478,            2129,            1812,            2559,            0,            1447,            547,            290,            2085,            2268,            898,            2488,            4013,            3221,            2474,            4266,            2732,            1901,            1421,            1841,            1899,            2330,            335,            2935,            2378,            2441,            3059,            1994,            2208,            1148,            1241,            1209,            2977,            2853,            3262,            4686,            4031,            4665,            3696,            2066,            4605,            2258,            3146,            2448,            3182,            3610,            5759,            2902,            4170,            3668,            5078,            6031,            3299,            3066,            1716,            2726,            768,            4596,            2224,            1942,            400,            2343,            3497,            3958,            943,            542,            3546,            3110,            2324,            1901,            2153,            4046,            1626,            4434,            3156,            2994,            3397,            3761,            4218,            3584,            4089,            3284,            2225,            5530,            1521,            3578,            2190,            1763,            2964,            6022,            1382,            ],        [            5545,            3944,            4150,            4227,            7328,            4827,            3323,            7029,            3992,            2925,            3576,            3259,            4436,            1447,            0,            900,            1263,            3532,            3715,            1806,            5283,            5460,            6016,            3921,            5713,            5527,            3348,            2868,            3718,            3346,            4207,            1782,            3799,            4255,            3888,            4506,            3871,            4085,            327,            2688,            2656,            5772,            4300,            6057,            6133,            5478,            6112,            5143,            3513,            6052,            3705,            5941,            3895,            5977,            5057,            5765,            4349,            4572,            5545,            6525,            7478,            4746,            4513,            3163,            3727,            679,            6043,            3671,            495,            1047,            3790,            4944,            5835,            504,            905,            6341,            5905,            4201,            3348,            3600,            5493,            3073,            6186,            5951,            4441,            4844,            5208,            5665,            5031,            5536,            4731,            4102,            6977,            2968,            5025,            3637,            3210,            4411,            7469,            2829,            ],        [            4645,            3044,            3250,            3327,            6428,            3927,            2423,            6129,            3092,            2025,            2676,            2359,            3106,            547,            900,            0,            363,            2632,            2815,            1445,            4383,            4560,            5116,            3021,            4813,            4627,            2448,            1968,            2388,            2446,            2877,            882,            3482,            2925,            2988,            3606,            2541,            2755,            601,            1788,            1756,            4872,            3400,            5157,            5233,            4578,            5212,            4243,            2613,            5152,            2805,            5041,            2995,            5077,            4157,            4865,            3449,            4717,            4215,            5625,            6578,            3846,            3613,            2263,            3273,            221,            5143,            2771,            1395,            147,            2890,            4044,            4505,            396,            5,            5441,            5005,            2871,            2448,            2700,            4593,            2173,            5286,            5051,            3541,            3944,            4308,            4765,            4131,            4636,            3831,            2772,            6077,            2068,            4125,            2737,            2310,            3511,            6569,            1929,            ],        [            4486,            2885,            3091,            3168,            6269,            3768,            2264,            5970,            2933,            1866,            2517,            2200,            2947,            388,            1157,            257,            0,            2473,            2656,            2963,            4224,            4401,            4957,            2862,            4654,            4468,            2289,            1809,            2229,            2287,            2718,            723,            4956,            2766,            2829,            3447,            2382,            2596,            858,            1629,            1597,            4713,            3241,            4998,            5074,            4419,            5053,            4084,            2454,            4993,            2646,            4882,            2836,            4918,            3998,            4706,            3290,            3513,            4056,            5466,            6419,            3687,            3454,            2104,            4884,            478,            4984,            2612,            1652,            110,            2731,            3885,            4346,            653,            252,            5282,            4846,            2712,            2289,            2541,            4434,            2014,            5127,            4892,            3382,            3785,            4149,            4606,            3972,            4477,            3672,            2613,            5918,            1909,            3966,            2578,            2151,            3352,            6410,            1770,            ],        [            3283,            3238,            3003,            3521,            4369,            3680,            2551,            4070,            2845,            1065,            1400,            1273,            1850,            2279,            3726,            2826,            2569,            0,            657,            2459,            1751,            3181,            2484,            2774,            2379,            1995,            2201,            1721,            1132,            651,            1621,            1896,            2226,            1172,            1712,            974,            1285,            1499,            3427,            828,            1119,            2240,            2342,            2525,            2799,            2850,            2778,            2789,            2366,            2718,            1657,            2409,            1847,            2445,            2429,            3872,            2173,            3053,            2551,            4632,            4519,            2698,            3366,            1303,            2017,            3047,            2709,            2861,            4221,            2679,            484,            3172,            2841,            3222,            2821,            2809,            2373,            1615,            2201,            2453,            2865,            311,            2654,            2419,            3735,            2951,            3549,            2331,            3085,            2908,            4025,            1516,            4018,            1108,            3878,            942,            1350,            3705,            4510,            1292,            ],        [            4384,            3224,            2989,            3507,            4355,            3666,            2537,            4056,            2831,            1214,            1853,            1726,            1010,            2265,            3712,            2812,            2555,            462,            0,            1921,            2204,            3473,            2937,            2760,            2832,            2448,            2187,            1707,            215,            657,            781,            1882,            1386,            539,            2165,            898,            445,            659,            3413,            1559,            1105,            2693,            3139,            2978,            2981,            3303,            3226,            3982,            2352,            3171,            2544,            2862,            2734,            2898,            2882,            3778,            2626,            2621,            2119,            4538,            4505,            3585,            3352,            1452,            1215,            3033,            3162,            2847,            3592,            2665,            203,            3625,            2409,            3208,            2807,            3262,            2826,            775,            2187,            2439,            3318,            460,            3107,            2872,            3721,            2857,            4047,            2784,            3538,            3361,            4011,            599,            4004,            1839,            3864,            404,            1499,            3691,            4496,            1278,            ],        [            4992,            3525,            3597,            3808,            4963,            4274,            2904,            4664,            3439,            2372,            3023,            2706,            2270,            898,            1806,            1445,            1188,            2260,            1703,            0,            3754,            4733,            4487,            3368,            5160,            3998,            2795,            2315,            1552,            2793,            2041,            1153,            2382,            2089,            3335,            2686,            1705,            1919,            2046,            2135,            2103,            4243,            3747,            4528,            4241,            4925,            4486,            4590,            2960,            4322,            3152,            4412,            3342,            4448,            4504,            5038,            3796,            3881,            3379,            5798,            5113,            4193,            3960,            2610,            2173,            1666,            4611,            3252,            1311,            1298,            1994,            4391,            3669,            1841,            1440,            5062,            4376,            2035,            2795,            3047,            4940,            2520,            4657,            4422,            4022,            4117,            4655,            4334,            4478,            4983,            4312,            1936,            4612,            2415,            4472,            1841,            2657,            3992,            5104,            2276,            ],        [            1532,            2912,            1763,            3195,            3744,            2856,            2225,            3445,            2519,            1734,            1062,            2677,            3264,            3437,            4884,            3984,            3727,            2604,            3704,            4353,            0,            2164,            1437,            1992,            1362,            966,            1875,            2123,            2822,            3335,            3035,            3054,            3866,            2285,            1684,            2087,            3969,            3139,            4585,            2103,            2645,            1231,            591,            1516,            1782,            1099,            1761,            1922,            2040,            1701,            1264,            1380,            996,            1398,            678,            2855,            1710,            1933,            2472,            3615,            5552,            1831,            2522,            1496,            4437,            4205,            1692,            2535,            5379,            3837,            2775,            1421,            2762,            4380,            3979,            1800,            1383,            3235,            1855,            2127,            1114,            2488,            1502,            1410,            3409,            1934,            1798,            1314,            1334,            1157,            3699,            2438,            3393,            2383,            2277,            3626,            1449,            3379,            3885,            2818,            ],        [            3566,            5581,            3797,            5864,            5778,            4890,            4894,            5479,            5188,            3408,            2283,            2713,            2963,            4622,            6069,            5169,            4912,            2850,            3000,            4802,            2034,            0,            1681,            4026,            1490,            2222,            4544,            4064,            2858,            2994,            3071,            4239,            4305,            2321,            1720,            2123,            3152,            3175,            5770,            3171,            3462,            869,            2625,            944,            809,            3133,            1054,            3956,            4709,            890,            3298,            1215,            3030,            1662,            2712,            1342,            1326,            1463,            2002,            2102,            5928,            3865,            4556,            3530,            4096,            5390,            585,            5204,            6564,            5022,            2827,            3455,            2292,            5565,            5164,            576,            1449,            3271,            3889,            4796,            3148,            2654,            1871,            936,            6078,            685,            3832,            1277,            3368,            3191,            6368,            2474,            5427,            3004,            4311,            3285,            3483,            6048,            5919,            3635,            ],        [            1965,            3948,            2799,            4231,            4177,            3289,            3261,            3878,            3555,            2761,            1597,            3212,            3799,            4473,            5920,            5020,            4763,            3349,            3499,            5301,            1996,            1755,            0,            3028,            1884,            899,            2911,            3159,            3357,            3493,            3570,            4090,            4804,            2820,            2219,            2622,            3651,            3674,            5621,            3130,            3681,            1069,            1627,            1028,            2121,            1381,            1380,            2355,            3076,            2202,            2300,            814,            2032,            676,            1083,            2654,            2232,            2417,            2994,            3414,            4327,            2264,            2875,            2523,            4595,            5241,            1255,            3571,            6244,            4873,            3390,            1854,            3284,            5416,            5015,            1323,            580,            3770,            2891,            3163,            1162,            3153,            885,            824,            4445,            1997,            2231,            1633,            1767,            1112,            4735,            2973,            3746,            3410,            2710,            3848,            2476,            4066,            4318,            4198,            ],        [            1608,            1024,            229,            1307,            2155,            1466,            759,            1856,            631,            1308,            1860,            2709,            3573,            2476,            3923,            3023,            2766,            2560,            2743,            3392,            2131,            3558,            2859,            0,            2756,            2360,            735,            1162,            2855,            2374,            3344,            2093,            3685,            3392,            2482,            3534,            3008,            3222,            3624,            2138,            1684,            2625,            1343,            2910,            3176,            1440,            3155,            1028,            574,            3095,            838,            2774,            938,            2820,            2100,            4249,            3104,            3327,            3866,            5009,            2305,            937,            1152,            1070,            3476,            3244,            3086,            1069,            4418,            2876,            2818,            1135,            4156,            3419,            3018,            3194,            2805,            3338,            894,            661,            1455,            2558,            2924,            2804,            1521,            3328,            1271,            2708,            1222,            2579,            1811,            3239,            1804,            2418,            1664,            2665,            1023,            1491,            2296,            1857,            ],        [            2456,            3836,            2687,            4119,            4668,            3780,            3149,            4369,            3443,            2516,            1173,            1821,            2071,            3730,            5177,            4277,            4020,            1958,            2108,            3910,            924,            1459,            1601,            2916,            0,            1112,            2799,            3172,            1966,            2102,            2300,            3347,            3413,            1429,            828,            1231,            2260,            2283,            4878,            2279,            2570,            1703,            1515,            1564,            967,            2023,            1212,            2846,            2964,            1048,            2188,            1526,            1920,            1562,            1602,            1764,            348,            571,            1110,            2524,            4818,            2755,            3446,            2420,            3204,            4498,            1337,            3459,            5672,            4130,            1935,            2345,            1400,            4673,            4272,            1788,            1490,            2379,            2779,            3051,            2038,            1762,            1771,            1768,            4333,            843,            2722,            415,            2258,            2081,            4623,            1582,            4317,            2112,            3201,            2393,            2373,            4303,            4809,            2743,            ],        [            2139,            3408,            2259,            3691,            4351,            3463,            2721,            4052,            3015,            1862,            698,            2313,            2900,            3339,            4786,            3886,            3629,            2450,            2600,            4402,            607,            1787,            489,            2488,            985,            0,            2371,            2619,            2458,            2594,            2671,            2956,            3905,            1921,            1320,            1723,            2752,            2775,            4487,            2231,            3062,            846,            1198,            1131,            1405,            1706,            1384,            2529,            2536,            1324,            1760,            414,            1603,            450,            857,            2478,            1333,            1556,            2095,            3238,            4501,            2438,            3129,            1624,            3696,            4107,            1315,            3031,            5281,            3739,            2427,            2028,            2385,            4282,            3881,            1812,            378,            2871,            2351,            2623,            1154,            2254,            659,            1025,            3905,            1557,            2405,            937,            1941,            886,            4195,            2074,            4000,            2511,            2884,            2885,            1577,            3875,            4492,            3235,            ],        [            2197,            1037,            802,            1320,            2168,            1479,            350,            1869,            644,            1396,            1948,            2320,            3184,            2087,            3534,            2634,            2377,            2171,            2354,            3003,            2245,            3672,            2973,            573,            2870,            2474,            0,            773,            2466,            1985,            2955,            1704,            3296,            3003,            2570,            3145,            2619,            2833,            3235,            1749,            1295,            2739,            1457,            3024,            3290,            2013,            3269,            1795,            165,            3209,            862,            2888,            1052,            2934,            2214,            4363,            3218,            3441,            4293,            5123,            2318,            1510,            1165,            1158,            3087,            2855,            3200,            660,            4029,            2487,            2429,            1708,            4583,            3030,            2629,            3308,            2919,            2949,            505,            252,            2028,            2169,            3038,            2918,            1534,            3442,            1860,            2822,            1795,            2693,            1824,            2850,            1817,            2029,            1677,            2276,            1111,            1504,            2309,            1468,            ],        [            2955,            1795,            1560,            2078,            2926,            2237,            1108,            2627,            1402,            1213,            1572,            1547,            2411,            1314,            2761,            1861,            1604,            1398,            1581,            2230,            3149,            4579,            3882,            1331,            3777,            3393,            758,            0,            1693,            1212,            2182,            931,            2523,            2230,            2176,            2372,            1846,            2060,            2462,            976,            522,            3638,            1710,            3923,            4197,            2888,            4176,            2553,            923,            4116,            1115,            3807,            1305,            3843,            2467,            5270,            2637,            4022,            3520,            6030,            3076,            2156,            1923,            730,            2314,            2082,            4107,            1418,            3256,            1714,            1656,            2354,            3810,            2257,            1856,            4207,            3771,            2176,            758,            1010,            2903,            1396,            4052,            3817,            2292,            4349,            2618,            3729,            2441,            2946,            2582,            2077,            2575,            1256,            2435,            1503,            777,            2262,            3067,            695,            ],        [            3734,            3664,            3429,            3947,            4795,            4106,            2977,            4496,            3271,            1429,            1851,            1724,            859,            2168,            3615,            2715,            2458,            677,            215,            1882,            2202,            3322,            2935,            3200,            2830,            2446,            2627,            2147,            0,            1097,            630,            2322,            1317,            537,            2163,            1134,            294,            508,            3316,            1999,            1545,            2691,            2793,            2976,            3250,            3301,            3075,            4422,            2792,            3169,            2984,            2860,            3174,            2896,            2880,            3627,            2624,            2470,            1968,            4387,            4945,            4025,            3792,            1667,            1108,            2936,            3160,            3637,            3553,            2568,            418,            3623,            2258,            3111,            2710,            3260,            2824,            624,            2627,            2879,            3316,            675,            3105,            2870,            4161,            2706,            4000,            2782,            3536,            3359,            4451,            384,            4444,            2279,            4304,            619,            1714,            4131,            4936,            1718,            ],        [            4271,            3111,            2876,            3394,            4242,            3553,            2424,            3943,            2718,            1683,            2130,            2017,            1199,            2094,            3541,            2641,            2384,            730,            369,            1808,            2481,            3662,            3214,            2647,            3109,            2725,            2074,            1594,            481,            0,            970,            1769,            1311,            1018,            2442,            1139,            634,            848,            3242,            1446,            992,            2970,            3026,            3255,            3170,            3580,            3415,            3869,            2239,            3448,            2431,            3139,            2621,            3175,            3159,            3967,            3070,            2810,            2308,            4727,            4392,            3472,            3239,            1921,            1102,            2862,            3439,            2734,            3479,            2494,            444,            3902,            2598,            3037,            2636,            3991,            3103,            964,            2074,            2326,            3595,            728,            3384,            3149,            3608,            3046,            3934,            3061,            3815,            3638,            3898,            865,            3891,            1726,            3751,            291,            1968,            3578,            4383,            1165,            ],        [            3872,            3824,            3589,            4107,            4955,            4266,            3137,            4656,            3431,            2396,            1989,            1862,            229,            2328,            3775,            2875,            2618,            1443,            781,            2042,            2340,            2692,            3073,            3360,            2968,            2584,            2787,            2307,            630,            1257,            0,            2482,            1167,            750,            2301,            1272,            336,            440,            3476,            2159,            1705,            2829,            2931,            2797,            2200,            3439,            2445,            4582,            2952,            2281,            3144,            2998,            3336,            3034,            3018,            2997,            2100,            1840,            1338,            3757,            5105,            4171,            3952,            2634,            958,            3096,            2570,            3797,            3713,            2728,            984,            3761,            1628,            3271,            2870,            3021,            2962,            536,            2787,            3039,            3454,            1441,            3243,            3008,            4321,            2076,            4138,            2920,            3674,            3497,            4611,            597,            4604,            2439,            4464,            1024,            2681,            4291,            5096,            1878,            ],        [            3714,            2404,            2319,            2687,            3685,            2996,            1867,            3386,            2161,            1094,            1745,            1428,            2714,            335,            1782,            882,            625,            1701,            1884,            1167,            2104,            3629,            2837,            2090,            2732,            2348,            1517,            1037,            1996,            1515,            2485,            0,            2826,            2533,            2057,            2675,            2149,            2363,            1483,            857,            825,            2593,            2469,            2878,            3152,            3647,            3382,            3312,            1682,            4221,            1874,            4110,            2064,            4146,            3226,            3934,            2518,            2741,            3823,            4694,            3835,            2915,            2682,            1332,            2617,            1103,            4212,            2131,            2277,            735,            1959,            3113,            3570,            1278,            877,            4510,            4074,            2479,            1517,            1769,            3662,            1242,            3007,            4120,            2901,            3013,            3377,            2684,            3200,            3705,            3191,            2380,            3334,            1137,            3194,            1806,            1379,            2871,            3826,            998,            ],        [            5338,            4178,            3943,            4461,            5309,            4620,            3491,            5010,            3785,            2750,            3815,            3084,            1807,            2682,            4129,            3229,            2972,            1797,            1456,            2396,            3548,            4270,            4281,            3714,            4176,            3792,            3141,            2661,            1089,            1611,            1578,            2836,            0,            1626,            3509,            2223,            1242,            1456,            3830,            2513,            2059,            4037,            4093,            4322,            3778,            4647,            4023,            4936,            3306,            4515,            3498,            4206,            3688,            4242,            4226,            4575,            3678,            3418,            2916,            5335,            5459,            4539,            4306,            2988,            371,            3450,            4506,            3801,            4067,            3082,            1531,            4969,            3206,            3625,            3224,            4599,            4170,            1572,            3141,            3393,            4662,            1795,            4451,            4216,            4675,            3654,            5001,            4128,            4882,            4705,            4965,            1473,            4958,            2793,            4818,            1378,            3035,            4645,            5450,            2232,            ],        [            3197,            3763,            3428,            4046,            4894,            4205,            3076,            4595,            3370,            1882,            1314,            1187,            979,            2705,            4152,            3252,            2995,            1001,            539,            2419,            1665,            3021,            2398,            3299,            2293,            1909,            2726,            2246,            537,            1468,            750,            2421,            1581,            0,            1626,            597,            831,            854,            3853,            1645,            1644,            2154,            2256,            2439,            2713,            2764,            2692,            3587,            2891,            2632,            2929,            2323,            2661,            2359,            2343,            3326,            2641,            2169,            1667,            4086,            5044,            3496,            3891,            2120,            1372,            3473,            2623,            4174,            4090,            3105,            742,            3086,            1957,            3648,            3247,            2723,            2287,            950,            2726,            2978,            2779,            1128,            2568,            2333,            4260,            2405,            3463,            2245,            2999,            2822,            4550,            153,            4543,            1478,            3942,            943,            2167,            4230,            5035,            1817,            ],        [            3003,            3861,            3234,            4144,            4992,            4303,            3174,            4693,            3468,            1688,            1120,            993,            1580,            2902,            4349,            3449,            3192,            1130,            1280,            3082,            1471,            3334,            2204,            3397,            2099,            1715,            2824,            2344,            1138,            1274,            1351,            2519,            2585,            601,            0,            403,            1432,            1455,            4050,            1451,            1742,            1960,            2062,            2245,            2519,            2570,            2498,            3393,            2989,            2438,            2735,            2129,            2467,            2165,            2149,            3639,            2447,            2482,            1980,            4399,            5142,            3302,            3989,            1926,            2376,            3670,            2429,            3484,            4844,            3302,            1107,            2892,            2270,            3845,            3444,            2529,            2093,            1551,            2824,            3076,            2585,            934,            2374,            2139,            4358,            2718,            3269,            2051,            2805,            2628,            4648,            754,            4641,            1284,            3748,            1565,            1973,            4328,            5133,            1915,            ],        [            2919,            3458,            3223,            3741,            4589,            3900,            2771,            4290,            3065,            1285,            1036,            909,            2070,            2499,            3946,            3046,            2789,            727,            877,            2679,            1387,            2817,            2120,            2994,            2015,            1631,            2421,            1941,            1352,            871,            1841,            2116,            2182,            808,            1348,            0,            1505,            1719,            3647,            1048,            1339,            1876,            1978,            2161,            2435,            2486,            2414,            3009,            2586,            2354,            1877,            2045,            2383,            2081,            2065,            3508,            2363,            2689,            2187,            4268,            4739,            3218,            3586,            1523,            1973,            3267,            2345,            3081,            4441,            2899,            704,            2808,            2477,            3442,            3041,            2445,            2009,            1835,            2421,            2673,            2501,            531,            2290,            2055,            3955,            2587,            3185,            1967,            2721,            2544,            4245,            961,            4238,            1328,            4098,            1162,            1570,            3925,            4730,            1512,            ],        [            4648,            3488,            3253,            3771,            4619,            3930,            2801,            4320,            3095,            2060,            2943,            2394,            565,            1992,            3439,            2539,            2282,            1107,            445,            1706,            2676,            3028,            3409,            3024,            3304,            2920,            2451,            1971,            294,            921,            336,            2146,            1141,            750,            2637,            1608,            0,            214,            3140,            1823,            1369,            3165,            3267,            3133,            2536,            3775,            2781,            4246,            2616,            2617,            2808,            3334,            2998,            3370,            3354,            3333,            2436,            2176,            1674,            4093,            4769,            3849,            3616,            2298,            932,            2760,            2906,            3461,            3377,            2392,            648,            4097,            1964,            2935,            2534,            3357,            3298,            330,            2451,            2703,            3790,            1105,            3579,            3344,            3985,            2412,            4311,            3256,            4010,            3833,            4275,            597,            4268,            2103,            4128,            688,            2345,            3955,            4760,            1542,            ],        [            5124,            3964,            3729,            4247,            5095,            4406,            3277,            4796,            3571,            2536,            3601,            2870,            1593,            2468,            3915,            3015,            2758,            1583,            1242,            2182,            3334,            3570,            4067,            3500,            3962,            3578,            2927,            2447,            875,            1397,            1364,            2622,            727,            1412,            3499,            2009,            1028,            0,            3616,            2299,            1845,            3823,            3879,            4108,            3078,            4433,            3323,            4722,            3092,            4301,            3284,            3992,            3474,            4028,            4012,            3875,            2978,            2718,            2216,            4635,            5245,            4325,            4092,            2774,            518,            3236,            4292,            3587,            3853,            2868,            1317,            4755,            2506,            3411,            3010,            3899,            3956,            1358,            2927,            3179,            4448,            1581,            4237,            4002,            4461,            2954,            4787,            3914,            4668,            4491,            4751,            1259,            4744,            2579,            4604,            1164,            2821,            4431,            5236,            2018,            ],        [            5246,            3645,            3851,            3928,            7029,            4528,            3024,            6730,            3693,            2626,            3277,            2960,            3707,            1148,            327,            601,            964,            3233,            3416,            2133,            3636,            5161,            4369,            3622,            4264,            3880,            3049,            2569,            2989,            3047,            3478,            1483,            3819,            3526,            3589,            4207,            3142,            3356,            0,            2389,            2357,            4125,            4001,            4410,            4684,            5179,            4914,            4844,            3214,            4603,            3406,            4294,            3596,            4330,            4758,            5466,            4050,            4273,            4816,            6226,            7179,            4559,            4214,            2864,            3610,            380,            4594,            3372,            822,            748,            3491,            4757,            5106,            205,            606,            4694,            4258,            3472,            3049,            3301,            5194,            2774,            4539,            4304,            4142,            4545,            4909,            4216,            4844,            5237,            4432,            3373,            6678,            2669,            4726,            3338,            2911,            4112,            7170,            2530,            ],        [            2673,            2619,            1460,            2902,            3750,            3061,            1932,            3451,            2226,            369,            888,            571,            2367,            1240,            2687,            1787,            1530,            633,            977,            2156,            1247,            2772,            1980,            1689,            1875,            1491,            1582,            1102,            1649,            1168,            2138,            857,            2479,            2186,            1200,            1607,            1802,            2016,            2388,            0,            478,            1736,            1556,            2021,            2295,            2346,            2274,            2093,            1747,            2214,            961,            1905,            1151,            1941,            1925,            3077,            1661,            1884,            2423,            3837,            3900,            2002,            2747,            607,            2270,            2008,            2205,            2242,            3182,            1640,            804,            2200,            2713,            2183,            1782,            2305,            1869,            2132,            1582,            1834,            2361,            517,            2150,            1915,            3116,            2156,            2336,            1827,            2287,            2404,            3406,            2033,            3399,            280,            3259,            1459,            654,            3086,            3891,            651,            ],        [            3443,            2283,            2048,            2566,            3414,            2725,            1596,            3115,            1890,            855,            1506,            1189,            1889,            1324,            2771,            1871,            1614,            876,            1059,            2240,            2627,            4057,            3360,            1819,            3255,            2871,            1246,            766,            1171,            690,            1660,            941,            2001,            1708,            1818,            1850,            1324,            1538,            2472,            618,            0,            3116,            2198,            3401,            3675,            3726,            3654,            3041,            1411,            3594,            1603,            3285,            1793,            3321,            3305,            4748,            2279,            3500,            2998,            5508,            3564,            2644,            2411,            1093,            1792,            2092,            3585,            1906,            3266,            1724,            1134,            2842,            3288,            2267,            1866,            3685,            3249,            1654,            1246,            1498,            3741,            874,            3530,            3295,            2780,            3827,            3106,            3207,            2929,            3784,            3070,            1555,            3063,            898,            2923,            981,            1140,            2750,            3555,            173,            ],        [            2082,            4065,            2916,            4348,            4294,            3406,            3378,            3995,            3672,            2618,            1454,            3136,            3028,            5045,            6492,            5592,            5335,            3273,            3423,            5225,            1274,            756,            812,            3145,            957,            756,            3028,            3276,            3281,            3417,            3257,            4662,            4728,            2744,            2143,            2546,            3575,            3598,            6193,            2987,            3885,            0,            1744,            351,            1232,            1716,            491,            2472,            3193,            1313,            2417,            346,            2149,            793,            1200,            1765,            1305,            1528,            2067,            2525,            6102,            2381,            3072,            2380,            4519,            5813,            366,            3688,            6987,            5445,            3250,            1971,            2357,            5988,            5587,            665,            580,            3694,            3008,            3280,            1497,            3077,            1002,            245,            4562,            1108,            2348,            744,            1884,            1229,            4852,            2897,            3943,            3427,            2827,            3708,            2333,            4532,            4435,            4058,            ],        [            1611,            2321,            1172,            2604,            3452,            2763,            1634,            3153,            1928,            1143,            1113,            2083,            3943,            2846,            4293,            3393,            3136,            2013,            3113,            3762,            788,            2215,            1516,            1401,            1413,            1017,            1284,            1532,            3225,            2744,            3714,            2463,            4055,            2336,            1735,            2138,            3378,            3592,            3994,            1512,            2054,            1282,            0,            1567,            1833,            1178,            1812,            1805,            1449,            1752,            673,            1431,            405,            1477,            757,            2906,            1761,            1984,            2523,            3666,            3602,            1714,            2405,            905,            3846,            3614,            1743,            1944,            4788,            3246,            2184,            1500,            2813,            3789,            3388,            1851,            1395,            3708,            1264,            1536,            1193,            1897,            1581,            1461,            2818,            1985,            1877,            1365,            1413,            1236,            3108,            2489,            3101,            1792,            1912,            3035,            858,            2788,            3593,            2227,            ],        [            2894,            4274,            3125,            4557,            5106,            4218,            3587,            4807,            3881,            3096,            1611,            2997,            2889,            4906,            6353,            5453,            5196,            3134,            3284,            5086,            1362,            617,            1097,            3354,            818,            1041,            3237,            3485,            3142,            3278,            3118,            4523,            4589,            2605,            2004,            2407,            3436,            3459,            6054,            3455,            3746,            351,            1953,            0,            1093,            2461,            352,            3284,            3402,            1174,            2626,            631,            2358,            1078,            1485,            1626,            1166,            1389,            1928,            2386,            6914,            3193,            3884,            2858,            4380,            5674,            227,            3897,            6848,            5306,            3111,            2783,            2218,            5849,            5448,            496,            865,            3555,            3217,            3489,            1782,            2938,            1287,            204,            4771,            969,            3160,            605,            2696,            1514,            5061,            2758,            4755,            3288,            3639,            3569,            2811,            4741,            5247,            3919,            ],        [            2757,            4137,            2988,            4420,            4969,            4081,            3450,            4670,            3744,            2959,            1474,            2444,            2694,            4353,            5800,            4900,            4643,            2581,            2731,            4533,            1225,            768,            1902,            3217,            681,            1413,            3100,            3348,            2589,            2725,            2802,            3970,            4036,            2052,            1451,            1854,            2883,            2906,            5501,            2902,            3193,            1012,            1816,            873,            0,            2324,            521,            3147,            3265,            81,            2489,            1358,            2221,            1863,            1903,            1073,            1029,            1194,            1733,            1833,            6777,            3056,            3747,            2721,            3827,            5121,            646,            3760,            6295,            4753,            2558,            2646,            2023,            5296,            4895,            1097,            1592,            3002,            3080,            3352,            2339,            2385,            2072,            1077,            4634,            152,            3023,            468,            2559,            2382,            4924,            2205,            4618,            2735,            3502,            3016,            2674,            4604,            5110,            3366,            ],        [            729,            2443,            1124,            2726,            2941,            2053,            2607,            2642,            1984,            2116,            1340,            2955,            3542,            3819,            5266,            4366,            4109,            3092,            3242,            4735,            1015,            2442,            753,            1353,            1640,            1244,            2257,            2505,            3100,            3236,            3313,            3436,            4547,            2563,            1962,            2365,            3394,            3417,            4967,            2485,            3027,            1509,            973,            1794,            2060,            0,            2039,            1119,            2422,            1979,            1646,            933,            1378,            714,            429,            3133,            1988,            2211,            2750,            3893,            3091,            1028,            1719,            1878,            4338,            4587,            1970,            2917,            5761,            4219,            3069,            618,            3040,            4762,            4361,            2078,            699,            3513,            2237,            2509,            459,            2896,            651,            1688,            2940,            2212,            995,            1592,            531,            306,            3230,            2716,            2590,            2765,            1474,            3527,            1831,            2910,            3082,            3200,            ],        [            2736,            4116,            2967,            4399,            4948,            4060,            3429,            4649,            3723,            2938,            1453,            2645,            2895,            4554,            6001,            5101,            4844,            2782,            2932,            4734,            1204,            597,            1303,            3196,            660,            1392,            3079,            3327,            2790,            2926,            3003,            4171,            4237,            2253,            1652,            2055,            3084,            3107,            5702,            3103,            3394,            491,            1795,            352,            741,            2303,            0,            3126,            3244,            822,            2468,            837,            2200,            1284,            1882,            1274,            1008,            1395,            1934,            2034,            6756,            3035,            3726,            2700,            4028,            5322,            125,            3739,            6496,            4954,            2759,            2625,            2224,            5497,            5096,            848,            1071,            3203,            3059,            3331,            2318,            2586,            1493,            556,            4613,            617,            3002,            447,            2538,            1720,            4903,            2406,            4597,            2936,            3481,            3217,            2653,            4583,            5089,            3567,            ],        [            580,            1410,            799,            1693,            1908,            1020,            1954,            1609,            1611,            1712,            2415,            2652,            4768,            3543,            4990,            4090,            3833,            3755,            3938,            4585,            2090,            3517,            1828,            1028,            2715,            2319,            1930,            2357,            4050,            3569,            4539,            3288,            4880,            4587,            3037,            3440,            4203,            4417,            4691,            2081,            2879,            2584,            1747,            2869,            3135,            1075,            3114,            0,            1769,            3054,            1242,            2008,            1342,            1789,            1504,            4208,            3063,            3286,            3825,            4968,            2058,            703,            686,            1474,            4671,            4311,            3045,            1780,            5485,            3943,            4013,            478,            4115,            4486,            4085,            3153,            1774,            4533,            2089,            1856,            1090,            2466,            1726,            2763,            1907,            3287,            243,            2667,            565,            1381,            2197,            4434,            1557,            2361,            107,            3860,            1427,            1877,            2049,            3052,            ],        [            2192,            994,            797,            1277,            2163,            1474,            185,            1864,            639,            1391,            1943,            2315,            3179,            2082,            3529,            2629,            2372,            2166,            2349,            2998,            2240,            3667,            2968,            568,            2865,            2469,            284,            768,            2461,            1980,            2950,            1699,            3291,            2998,            2565,            3140,            2614,            2828,            3230,            1744,            1290,            2734,            1452,            3019,            3285,            2008,            3264,            1790,            0,            3204,            857,            2883,            1047,            2929,            2209,            4358,            3213,            3436,            4288,            5118,            2313,            1505,            1160,            1153,            3082,            2850,            3195,            495,            4024,            2482,            2424,            1703,            4578,            3025,            2624,            3303,            2914,            2944,            500,            247,            2023,            2164,            3033,            2913,            1491,            3437,            1855,            2817,            1790,            2688,            1781,            2845,            1812,            2024,            1672,            2271,            1106,            1461,            2304,            1463,            ],        [            2676,            4056,            2907,            4339,            4888,            4000,            3369,            4589,            3663,            2878,            1393,            2525,            2775,            4434,            5881,            4981,            4724,            2662,            2812,            4614,            1144,            849,            1821,            3136,            600,            1332,            3019,            3267,            2670,            2806,            2883,            4051,            4117,            2133,            1532,            1935,            2964,            2987,            5582,            2983,            3274,            1093,            1735,            954,            81,            2243,            602,            3066,            3184,            0,            2408,            1439,            2140,            1782,            1822,            1154,            948,            1275,            1814,            1914,            5038,            2975,            3666,            2640,            3908,            5202,            727,            3679,            6376,            4834,            2639,            2565,            2104,            5377,            4976,            1178,            1673,            3083,            2999,            3271,            2258,            2466,            1991,            1158,            4553,            233,            2942,            387,            2478,            2301,            4843,            2286,            4537,            2816,            3421,            3097,            2593,            4523,            5029,            3447,            ],        [            1712,            1752,            499,            2035,            2883,            2194,            1081,            2584,            1359,            932,            1484,            1872,            3390,            2293,            3740,            2840,            2583,            2377,            2560,            3209,            1781,            3208,            2509,            728,            2406,            2010,            731,            979,            2672,            2191,            3161,            1910,            3502,            3209,            2106,            2509,            2825,            3039,            3441,            1301,            1501,            2275,            993,            2560,            2826,            1544,            2805,            1132,            896,            2745,            0,            2424,            588,            2470,            1750,            3899,            2754,            2977,            3516,            4659,            3033,            1041,            1732,            694,            3293,            3061,            2736,            1391,            4235,            2693,            2635,            1239,            3806,            3236,            2835,            2844,            2455,            3155,            711,            983,            1559,            1686,            2574,            2454,            2249,            2978,            1375,            2358,            1326,            2229,            2539,            3056,            2532,            1581,            1239,            2482,            647,            2219,            3024,            1674,            ],        [            1736,            3719,            2570,            4002,            3948,            3060,            3032,            3649,            3326,            2272,            1108,            2723,            3310,            3749,            5196,            4296,            4039,            2860,            3010,            4812,            1017,            1102,            466,            2799,            1395,            410,            2682,            2930,            2868,            3004,            3081,            3366,            4315,            2331,            1730,            2133,            3162,            3185,            4897,            2641,            3472,            346,            1398,            631,            1578,            1370,            837,            2126,            2847,            1734,            2071,            0,            1803,            447,            854,            2111,            1743,            1966,            2505,            2871,            5756,            2035,            2726,            2034,            4106,            4517,            712,            3342,            5691,            4149,            2837,            1625,            2795,            4692,            4291,            915,            234,            3281,            2662,            2934,            1151,            2664,            656,            525,            4216,            1454,            2002,            1347,            1538,            883,            4506,            2484,            3597,            2921,            2481,            3295,            1987,            4186,            4089,            3645,            ],        [            2016,            1916,            767,            2199,            3047,            2358,            1229,            2748,            1523,            738,            1518,            1678,            3538,            2441,            3888,            2988,            2731,            1608,            2708,            3357,            1193,            2620,            1921,            996,            1818,            1422,            879,            1127,            2820,            2339,            3309,            2058,            3650,            3357,            2140,            2543,            2973,            3187,            3589,            1107,            1649,            1687,            405,            1972,            2238,            1583,            2217,            1400,            1044,            2157,            268,            1836,            0,            1882,            1162,            3311,            2166,            2389,            2928,            4071,            3197,            1309,            2000,            500,            3441,            3209,            2148,            1539,            4383,            2841,            1779,            1905,            3218,            3384,            2983,            2256,            1867,            3303,            859,            1131,            1598,            1492,            1986,            1866,            2413,            2390,            1643,            1770,            1818,            1641,            2703,            3204,            2696,            1387,            1507,            2630,            453,            2383,            3188,            1822,            ],        [            1289,            3272,            2123,            3555,            3501,            2613,            2585,            3202,            2879,            2094,            1318,            2933,            3520,            3797,            5244,            4344,            4087,            3070,            3220,            4713,            993,            2246,            491,            2352,            1618,            675,            2235,            2483,            3078,            3214,            3291,            3414,            4525,            2541,            1940,            2343,            3372,            3395,            4945,            2463,            3005,            1487,            951,            1519,            2038,            856,            2017,            1679,            2400,            1957,            1624,            671,            1356,            0,            407,            3111,            1966,            2189,            2728,            3871,            5309,            1588,            2279,            1856,            4316,            4565,            1948,            2895,            5739,            4197,            3047,            1178,            3018,            4740,            4339,            1814,            437,            3491,            2215,            2487,            871,            2874,            209,            1315,            3769,            2190,            1555,            1570,            1091,            436,            4059,            2694,            3150,            2743,            2034,            3505,            1809,            3739,            3642,            3178,            ],        [            2118,            2865,            1716,            3148,            3996,            3307,            2178,            3697,            2472,            1687,            911,            2526,            3113,            3390,            4837,            3937,            3680,            2663,            2813,            4306,            586,            2013,            759,            1945,            1211,            815,            1828,            2076,            2671,            2807,            2884,            3007,            4118,            2134,            1533,            1936,            2965,            2988,            4538,            2056,            2598,            1080,            544,            1365,            1631,            1685,            1610,            2349,            1993,            1550,            1217,            939,            949,            720,            0,            2704,            1559,            1782,            2321,            3464,            4146,            2417,            3108,            1449,            3909,            4158,            1541,            2488,            5332,            3790,            2640,            2007,            2611,            4333,            3932,            1649,            705,            3084,            1808,            2080,            1700,            2467,            929,            1259,            3362,            1783,            2384,            1163,            1920,            1743,            3652,            2287,            3645,            2336,            2863,            3098,            1402,            3332,            4137,            2771,            ],        [            4071,            6086,            4302,            6369,            6283,            5395,            5399,            5984,            5693,            3913,            2788,            3218,            3468,            5127,            6574,            5674,            5417,            3355,            3505,            5307,            2539,            1806,            2761,            4531,            1995,            2727,            5049,            4569,            3363,            3499,            3576,            4744,            4810,            2826,            2225,            2628,            3657,            3680,            6275,            3676,            3967,            2050,            3130,            1911,            1314,            3638,            1559,            4461,            5214,            1395,            3803,            2396,            3535,            2843,            3217,            0,            1831,            1968,            2507,            1958,            6433,            4370,            5061,            4035,            4601,            5895,            1684,            5709,            7069,            5527,            3332,            3960,            2797,            6070,            5669,            2135,            2630,            3776,            4394,            5301,            3304,            3159,            3052,            2115,            6583,            1190,            4337,            1782,            3873,            3696,            6873,            2979,            5932,            3509,            4816,            3790,            3988,            6553,            6424,            4140,            ],        [            2739,            4532,            2970,            4815,            5663,            4063,            3845,            5364,            4139,            2359,            1456,            1664,            1914,            3573,            5020,            4120,            3863,            1801,            1951,            3753,            1207,            1111,            1884,            3199,            511,            1395,            3495,            3015,            1809,            1945,            2022,            3190,            3256,            1272,            671,            1074,            2103,            2126,            4721,            2122,            2413,            1355,            1798,            1216,            619,            2306,            864,            3129,            3660,            700,            2471,            1701,            2203,            1845,            1885,            1416,            0,            414,            953,            2176,            5813,            3038,            3729,            2703,            3047,            4341,            989,            4155,            5515,            3973,            1778,            2628,            1243,            4516,            4115,            1440,            1935,            2222,            3062,            3747,            2321,            1605,            2054,            1420,            5029,            495,            3005,            1087,            2541,            2364,            5319,            1425,            5312,            1955,            3484,            2236,            2656,            4999,            5804,            2586,            ],        [            2761,            4554,            2992,            4837,            5685,            4085,            3867,            5386,            4161,            2381,            1478,            1686,            1688,            3595,            5042,            4142,            3885,            1823,            1973,            3775,            1229,            1371,            1906,            3221,            533,            1417,            3517,            3037,            1831,            1967,            1917,            3212,            3278,            1294,            693,            1096,            2125,            2148,            4743,            2144,            2435,            1615,            1820,            1476,            879,            2328,            1124,            3151,            3682,            960,            2493,            1831,            2225,            1867,            1907,            1676,            260,            0,            727,            2436,            6781,            3060,            3751,            2725,            3069,            4363,            1249,            4177,            5537,            3995,            1800,            2650,            1017,            4538,            4137,            1700,            1795,            2318,            3084,            3769,            2343,            1627,            2076,            1680,            5051,            755,            3027,            720,            2563,            2386,            5341,            1447,            5334,            1977,            3506,            2258,            2678,            5021,            5826,            2608,            ],        [            3619,            5523,            3850,            5806,            5831,            4943,            4836,            5532,            5130,            3350,            2336,            2655,            1168,            3725,            5172,            4272,            4015,            2792,            2263,            3439,            2087,            1354,            2410,            4079,            1543,            2275,            4486,            4006,            2261,            2936,            1397,            4181,            2564,            1724,            1773,            2065,            1733,            1837,            4873,            3113,            3404,            1598,            2678,            1459,            862,            3186,            1107,            4009,            4651,            943,            3351,            1944,            3083,            2391,            2765,            1659,            1379,            958,            0,            2419,            7639,            3918,            4609,            3583,            2355,            4493,            1232,            5146,            5110,            4125,            2769,            3508,            290,            4668,            4267,            1683,            2178,            1798,            3942,            4738,            3201,            2596,            2600,            1663,            6020,            738,            3885,            1330,            3421,            3244,            6310,            1877,            5480,            2946,            4364,            3227,            3536,            5990,            5972,            3577,            ],        [            3469,            5484,            3700,            5767,            5681,            4793,            4797,            5382,            5091,            3311,            2186,            2616,            2866,            4525,            5972,            5072,            4815,            2753,            2903,            4705,            1937,            1204,            2260,            3929,            1393,            2125,            4447,            3967,            2761,            2897,            2974,            4142,            4208,            2224,            1623,            2026,            3055,            3078,            5673,            3074,            3365,            1448,            2528,            1309,            712,            3036,            957,            3859,            4612,            793,            3201,            1794,            2933,            2241,            2615,            638,            1229,            1366,            1905,            0,            7489,            3768,            4459,            3433,            3999,            5293,            1082,            5107,            6467,            4925,            2730,            3358,            2195,            5468,            5067,            1533,            2028,            3174,            3792,            4699,            3051,            2557,            2450,            1513,            5981,            588,            3735,            1180,            3271,            3094,            6271,            2377,            5330,            2907,            4214,            3188,            3386,            5951,            5822,            3538,            ],        [            1765,            1272,            1828,            1555,            742,            1047,            1816,            443,            1473,            2842,            3394,            3766,            4630,            3405,            4852,            3952,            3695,            3617,            3800,            4447,            3691,            5118,            3191,            1599,            4316,            3920,            1792,            2219,            3912,            3431,            4401,            3150,            4742,            4449,            4016,            4591,            4065,            4279,            4553,            3195,            2741,            4185,            2903,            4470,            4736,            2438,            4715,            1363,            1631,            4655,            2308,            4334,            2498,            4380,            3660,            5809,            4664,            4887,            5739,            6569,            0,            2066,            733,            2604,            4533,            4173,            4646,            1642,            5347,            3805,            3875,            1841,            6029,            4348,            3947,            4754,            3137,            4395,            1951,            1718,            2453,            3615,            3143,            4364,            1769,            4888,            1428,            4268,            1928,            2744,            2059,            4296,            369,            3475,            1245,            3722,            2557,            1739,            883,            2914,            ],        [            966,            1415,            827,            1698,            1913,            1025,            1959,            1614,            1616,            1740,            2443,            3909,            4773,            3548,            4995,            4095,            3838,            3760,            3943,            4590,            2118,            3545,            1856,            1742,            2743,            2347,            1935,            2362,            4055,            3574,            4544,            3293,            4885,            4592,            3065,            3468,            4208,            4422,            4696,            3338,            2884,            2612,            1775,            2897,            3163,            1103,            3142,            564,            1774,            3082,            1270,            2036,            1370,            1817,            1532,            4236,            3091,            3314,            3853,            4996,            2063,            0,            691,            1502,            4676,            4316,            3073,            1785,            5490,            3948,            4018,            506,            4143,            4491,            4090,            3181,            1802,            4538,            2094,            1861,            1118,            3758,            1808,            2791,            1912,            3315,            629,            2695,            593,            1409,            2202,            4439,            1562,            3618,            446,            3865,            1455,            1882,            2054,            3057,            ],        [            1884,            724,            1280,            1007,            1222,            1166,            1268,            923,            925,            2294,            2846,            3218,            4082,            2857,            4304,            3404,            3147,            3069,            3252,            3899,            3143,            4570,            3871,            1051,            3768,            3372,            1244,            1671,            3364,            2883,            3853,            2602,            4194,            3901,            3468,            4043,            3517,            3731,            4005,            2647,            2193,            3637,            2355,            3922,            4188,            2491,            4167,            1482,            1083,            4107,            1760,            3786,            1950,            3832,            3112,            5261,            4116,            4339,            5191,            6021,            1372,            1988,            0,            2056,            3985,            3625,            4098,            1094,            4799,            3257,            3327,            1960,            5481,            3800,            3399,            4206,            3817,            3847,            1403,            1170,            2506,            3067,            3990,            3816,            1221,            4340,            1547,            3720,            2047,            3591,            1511,            3748,            871,            2927,            1364,            3174,            2009,            1191,            1363,            2366,            ],        [            2066,            2002,            853,            2285,            3123,            2444,            1315,            2824,            1609,            238,            1008,            1178,            2842,            1715,            3162,            2262,            2005,            1108,            1452,            2631,            1367,            3164,            2465,            1082,            2362,            1611,            965,            1213,            2124,            1643,            2613,            1332,            2954,            2661,            1630,            2082,            2277,            2491,            2863,            607,            953,            1856,            949,            2141,            2782,            2127,            2761,            1486,            1130,            2701,            354,            2025,            544,            2426,            1706,            3855,            2710,            2933,            3472,            4615,            3273,            1395,            2086,            0,            2745,            2483,            2692,            1625,            3657,            2115,            1279,            1593,            3762,            2658,            2257,            2425,            1989,            2607,            945,            1217,            2142,            992,            2584,            2035,            2499,            2934,            1729,            2314,            1680,            2185,            2789,            2508,            2772,            887,            1593,            1934,            47,            2469,            3264,            1126,            ],        [            4967,            3807,            3572,            4090,            4928,            4249,            3120,            4629,            3414,            2379,            3444,            2713,            1436,            2311,            3758,            2858,            2601,            1426,            1085,            2025,            3177,            3899,            3910,            3343,            3805,            3421,            2770,            2290,            718,            1240,            1207,            2465,            590,            1255,            3138,            1852,            871,            1085,            3459,            2142,            1688,            3666,            3722,            3951,            3407,            4276,            3652,            4565,            2935,            3488,            3127,            3835,            3317,            3871,            3855,            4204,            3307,            3047,            2545,            4964,            5078,            4168,            3935,            2617,            0,            3079,            4135,            3430,            3696,            2711,            1160,            4366,            2835,            3254,            2853,            4228,            3799,            1201,            2770,            3022,            4291,            1424,            4080,            3845,            4304,            3283,            4630,            3757,            4511,            4334,            4594,            1102,            4577,            2422,            4447,            1007,            2664,            4274,            5069,            1861,            ],        [            4866,            3265,            3471,            3548,            6649,            4148,            2644,            6350,            3313,            2246,            2897,            2580,            3327,            768,            679,            221,            584,            2853,            3036,            1666,            3256,            4781,            3989,            3242,            3884,            3500,            2669,            2189,            2609,            2667,            3098,            1103,            3439,            3146,            3209,            3827,            2762,            2976,            380,            2009,            1977,            3745,            3621,            4030,            4304,            4799,            4534,            4464,            2834,            4223,            3026,            3914,            3216,            3950,            4378,            5086,            3670,            3893,            4436,            5846,            6799,            4067,            3834,            2484,            3230,            0,            4214,            2992,            1174,            368,            3111,            4265,            4726,            175,            226,            4314,            3878,            3092,            2669,            2921,            4814,            2394,            4159,            3924,            3762,            4165,            4529,            3836,            4352,            4857,            4052,            2993,            6298,            2289,            4346,            2958,            2531,            3732,            6790,            2150,            ],        [            2667,            4047,            2898,            4330,            4879,            3991,            3360,            4580,            3654,            2869,            1384,            2770,            2662,            4679,            6126,            5226,            4969,            2907,            3057,            4859,            1135,            472,            1178,            3127,            591,            1122,            3010,            3258,            2915,            3051,            2891,            4296,            4362,            2378,            1777,            2180,            3209,            3232,            5827,            3228,            3519,            366,            1726,            227,            866,            2234,            125,            3057,            3175,            947,            2399,            712,            2131,            1159,            1813,            1399,            939,            1162,            1701,            2159,            6687,            2966,            3657,            2631,            4153,            5447,            0,            3670,            6621,            5079,            2884,            2556,            1991,            5622,            5221,            801,            946,            3328,            2990,            3262,            1863,            2711,            1368,            431,            4544,            742,            2933,            378,            2469,            1595,            4834,            2531,            4528,            3061,            3412,            3342,            2584,            4514,            5020,            3692,            ],        [            2204,            499,            1292,            782,            2165,            1486,            310,            1866,            1134,            1886,            2438,            2810,            3674,            2118,            3565,            2665,            2408,            2661,            2844,            3160,            2735,            4162,            3463,            1063,            3360,            2964,            779,            1263,            2956,            2475,            3445,            2025,            3786,            3493,            3060,            3635,            3109,            3323,            3266,            2239,            1785,            3229,            1947,            3514,            3780,            3125,            3759,            1802,            495,            3699,            1352,            3378,            1542,            3424,            2704,            4853,            3708,            3931,            4783,            5613,            2315,            2000,            1172,            1648,            3577,            2886,            3690,            0,            4060,            2518,            2919,            2280,            5073,            3061,            2660,            3798,            3409,            3439,            995,            742,            3140,            2659,            3582,            3408,            996,            3932,            1867,            3312,            2367,            3183,            1286,            3340,            1814,            2519,            1684,            2766,            1601,            966,            1820,            1958,            ],        [            6040,            4439,            4645,            4722,            7823,            5322,            3818,            7524,            4487,            3420,            4071,            3754,            3941,            1942,            495,            1395,            1758,            4027,            3374,            1311,            4430,            5955,            5163,            4416,            6208,            6022,            3843,            3363,            3223,            3841,            3712,            2277,            3304,            3760,            4383,            4357,            3376,            3590,            822,            3183,            3151,            6267,            4795,            6552,            5478,            5973,            6157,            5638,            4008,            5397,            4200,            5088,            4390,            5124,            5552,            6260,            4844,            5552,            5050,            7020,            7973,            5241,            5008,            3658,            3232,            1174,            6538,            4166,            0,            1542,            3665,            5439,            5340,            999,            1400,            5488,            6400,            3706,            3843,            4095,            5988,            3568,            6430,            5098,            4936,            5339,            5703,            5010,            5526,            6031,            5226,            3607,            7472,            3463,            5520,            4132,            3705,            4906,            7964,            3324,            ],        [            4498,            2897,            3103,            3180,            6281,            3780,            2276,            5982,            2945,            1878,            2529,            2212,            2959,            400,            1047,            147,            216,            2485,            2668,            1298,            2888,            4413,            3621,            2874,            4666,            3132,            2301,            1821,            2241,            2299,            2730,            735,            3071,            2778,            2841,            3459,            2394,            2608,            748,            1641,            1609,            3377,            3253,            3662,            5086,            4431,            5065,            4096,            2466,            5005,            2658,            4894,            2848,            3582,            4010,            4718,            3302,            4570,            4068,            5478,            6431,            3699,            3466,            2116,            2862,            368,            4996,            2624,            1542,            0,            2743,            3897,            4358,            543,            142,            5294,            4858,            2724,            2301,            2553,            4446,            2026,            4888,            3556,            3394,            3797,            4161,            3468,            3984,            4489,            3684,            2625,            5930,            1921,            3978,            2590,            2163,            3364,            6422,            1782,            ],        [            4459,            3299,            3064,            3582,            4420,            3741,            2612,            4121,            2906,            1041,            1689,            1375,            1213,            2340,            3787,            2887,            2630,            289,            203,            1996,            2040,            3470,            2773,            2835,            2668,            2284,            2262,            1782,            418,            555,            984,            1957,            1499,            742,            2001,            889,            648,            862,            3488,            804,            1180,            2529,            2631,            2814,            3088,            3139,            3067,            2765,            2427,            3007,            1633,            2698,            1823,            2734,            2718,            4161,            2462,            2824,            2322,            4921,            4570,            2674,            3427,            1279,            1290,            3108,            2998,            2922,            3667,            2740,            0,            3461,            2612,            3283,            2882,            3098,            2662,            978,            2262,            2514,            3154,            287,            3029,            2708,            3796,            3240,            4122,            2620,            3374,            3197,            4086,            802,            4069,            1084,            3939,            479,            1326,            3766,            4561,            1353,            ],        [            1376,            1825,            506,            2108,            2323,            1435,            1494,            2024,            1366,            1419,            1937,            2359,            4308,            3211,            4658,            3758,            3501,            2289,            3478,            4127,            1612,            3039,            1350,            735,            2237,            1841,            1470,            1897,            3590,            3109,            4079,            2828,            4420,            3160,            2559,            2962,            3743,            3957,            4359,            1788,            2419,            2106,            1454,            2391,            2657,            597,            2636,            501,            1309,            2576,            949,            1530,            1049,            1311,            1026,            3730,            2585,            2808,            3347,            4490,            2473,            410,            1101,            1181,            4211,            3979,            2567,            2195,            5153,            3611,            2460,            0,            3637,            4154,            3753,            2675,            1296,            4073,            1629,            1396,            612,            2173,            1302,            2285,            2322,            2809,            1039,            2189,            87,            903,            2612,            3313,            1972,            2068,            856,            3400,            1134,            2292,            2464,            2592,            ],        [            3329,            5344,            3560,            5627,            5541,            4653,            4657,            5242,            4951,            3171,            2046,            2476,            2726,            4385,            5832,            4932,            4675,            2613,            2763,            4565,            1797,            1064,            2120,            3789,            1253,            1985,            4307,            3827,            2621,            2757,            2834,            4002,            4068,            2084,            1483,            1886,            2915,            2938,            5533,            2934,            3225,            1308,            2388,            1169,            572,            2896,            817,            3719,            4472,            653,            3061,            1654,            2793,            2101,            2475,            1369,            1089,            1226,            1765,            2129,            7349,            3628,            4319,            3293,            3859,            5153,            942,            4967,            6327,            4785,            2590,            3218,            0,            5328,            4927,            1393,            1888,            3034,            3652,            4559,            2911,            2417,            2396,            1373,            5841,            448,            3595,            1040,            3131,            2954,            6131,            2237,            5190,            2767,            4074,            3048,            3246,            5811,            5682,            3398,            ],        [            4906,            3440,            3646,            3723,            4867,            4188,            2819,            4568,            3488,            2421,            3072,            2755,            3502,            943,            504,            396,            759,            3028,            3211,            1841,            4779,            4956,            5512,            3417,            5209,            5023,            2844,            2364,            2784,            2842,            3273,            1278,            3614,            3321,            3384,            4002,            2937,            3151,            205,            2184,            2152,            5268,            3796,            5553,            5629,            4974,            5608,            4504,            3009,            5548,            3201,            5437,            3391,            5473,            4553,            5261,            3845,            5113,            4611,            6021,            5017,            4242,            3874,            2659,            3405,            175,            5539,            3167,            999,            543,            3286,            4440,            4901,            0,            401,            5837,            5401,            3267,            2844,            3096,            4989,            2569,            5431,            5447,            3937,            4340,            4569,            5161,            4527,            5032,            4227,            3168,            4516,            2464,            4386,            3133,            2706,            3907,            4694,            2325,            ],        [            4505,            3039,            3245,            3322,            4466,            3787,            2418,            4167,            3087,            2020,            2671,            2354,            3101,            542,            905,            5,            358,            2627,            2810,            1440,            4378,            4555,            5111,            3016,            4808,            4622,            2443,            1963,            2383,            2441,            2872,            877,            3213,            2920,            2983,            3601,            2536,            2750,            606,            1783,            1751,            4867,            3395,            5152,            5228,            4573,            5207,            4103,            2608,            5147,            2800,            5036,            2990,            5072,            4152,            4860,            3444,            4712,            4210,            5620,            4616,            3841,            3473,            2258,            3004,            226,            5138,            2766,            1400,            142,            2885,            4039,            4500,            401,            0,            5436,            5000,            2866,            2443,            2695,            4588,            2168,            5030,            5046,            3536,            3939,            4168,            4760,            4126,            4631,            3826,            2767,            4115,            2063,            3985,            2732,            2305,            3506,            4293,            1924,            ],        [            3483,            6039,            3714,            6322,            5695,            4807,            5352,            5396,            5646,            3866,            2200,            3171,            3421,            5080,            6527,            5627,            5370,            3308,            3458,            5260,            1951,            458,            1398,            3943,            1407,            1342,            5002,            4522,            3316,            3452,            3529,            4697,            4763,            2779,            2178,            2581,            3610,            3633,            6228,            3629,            3920,            652,            2542,            513,            1267,            2302,            941,            3873,            5167,            1348,            3215,            932,            2947,            1379,            1937,            1800,            1784,            1921,            2460,            2560,            7503,            3782,            4473,            3447,            4554,            5848,            816,            5662,            7022,            5480,            3285,            3372,            2750,            6023,            5622,            0,            1166,            3729,            3806,            5254,            2083,            3112,            1674,            505,            6536,            1143,            3749,            1194,            3285,            1815,            6826,            2932,            5344,            3462,            4228,            3743,            3400,            6506,            5836,            4093,            ],        [            1653,            3636,            2487,            3919,            3852,            2964,            2949,            3553,            3243,            2298,            1134,            2749,            3336,            4161,            5608,            4708,            4451,            2886,            3036,            5077,            1043,            1336,            232,            2716,            1421,            436,            2599,            2847,            2894,            3030,            3107,            3778,            4341,            2357,            1756,            2159,            3188,            3211,            5309,            2667,            3369,            580,            1315,            865,            1841,            1136,            1071,            2030,            2764,            1760,            1988,            234,            1720,            213,            771,            2345,            1769,            1992,            2531,            3105,            5673,            1939,            2630,            2060,            4132,            4929,            946,            3259,            6103,            4561,            2863,            1529,            2821,            5104,            4703,            1555,            0,            3307,            2579,            2851,            917,            2690,            508,            759,            4133,            1993,            1919,            1373,            1442,            649,            4423,            2510,            3501,            2947,            2385,            3321,            2013,            4103,            3993,            3542,            ],        [            5048,            3888,            3653,            4171,            5019,            4330,            3201,            4720,            3495,            2460,            2595,            2468,            835,            2392,            3839,            2939,            2682,            1507,            845,            2106,            2946,            3083,            3679,            3424,            3574,            3190,            2851,            2371,            694,            1321,            606,            2546,            949,            1020,            2907,            1878,            400,            504,            3540,            2223,            1769,            3435,            3537,            3188,            2591,            4045,            2836,            4646,            3016,            2672,            3208,            3604,            3398,            3640,            3624,            3388,            2491,            2231,            1729,            4148,            5169,            4249,            4016,            2698,            1022,            3160,            2961,            3861,            3777,            2792,            1048,            4367,            2019,            3335,            2934,            3412,            3568,            0,            2851,            3103,            4060,            1505,            3935,            3614,            4385,            2467,            4711,            3526,            4280,            4103,            4675,            867,            4668,            2503,            4528,            1088,            2745,            4355,            5160,            1942,            ],        [            2709,            1549,            1314,            1832,            2670,            1991,            862,            2371,            1156,            1481,            1955,            1815,            2679,            1582,            3029,            2129,            1872,            1666,            1849,            2498,            2252,            3679,            2980,            1085,            2877,            2481,            512,            268,            1961,            1480,            2450,            1199,            2791,            2498,            2444,            2640,            2114,            2328,            2730,            1244,            790,            2746,            1464,            3031,            3297,            2642,            3276,            2307,            677,            3216,            869,            2895,            1059,            2941,            2221,            4370,            3225,            3448,            3788,            5130,            2820,            1910,            1677,            1165,            2582,            2350,            3207,            1172,            3524,            1982,            1924,            2108,            4078,            2525,            2124,            3315,            2926,            2444,            0,            764,            2657,            1664,            3099,            2925,            2046,            3449,            2372,            2829,            2195,            2700,            2336,            2345,            2319,            1524,            2189,            1771,            1118,            2016,            2811,            963,            ],        [            1945,            785,            550,            1068,            1906,            1227,            504,            1607,            392,            1629,            2777,            2488,            3352,            2255,            3702,            2802,            2545,            2339,            2522,            3171,            2452,            3879,            3180,            321,            3077,            2681,            514,            941,            2634,            2153,            3123,            1872,            3464,            3171,            3399,            3313,            2787,            3001,            3403,            1917,            1463,            2946,            1664,            3231,            3497,            1761,            3476,            1543,            335,            3416,            1159,            3095,            1259,            3141,            2421,            4570,            3425,            3648,            4187,            5330,            2056,            1258,            913,            1391,            3255,            3023,            3407,            1155,            4197,            2655,            2597,            1456,            4477,            3198,            2797,            3515,            3126,            3117,            673,            0,            1776,            2337,            3299,            3125,            1282,            3649,            1608,            3029,            1543,            2900,            1572,            3018,            1555,            2197,            1425,            2444,            1344,            1252,            2047,            1636,            ],        [            755,            2437,            1118,            2720,            2935,            2047,            2106,            2636,            1978,            2335,            1559,            3174,            3761,            4038,            5485,            4585,            4328,            3311,            3461,            4954,            1234,            2661,            971,            1347,            1859,            1155,            2476,            2724,            3319,            3455,            3532,            3655,            4766,            2782,            2181,            2584,            3613,            3636,            5186,            2704,            3246,            1728,            1192,            2013,            2279,            219,            2258,            1113,            1921,            2198,            1865,            1151,            1597,            932,            648,            3352,            2207,            2430,            2969,            4112,            3085,            1022,            1713,            2097,            4557,            4806,            2189,            2807,            5980,            4438,            3288,            612,            3259,            4981,            4580,            2294,            917,            3732,            2456,            2008,            0,            3115,            809,            1795,            2934,            2431,            1021,            1811,            525,            410,            3224,            2935,            2584,            2984,            1468,            3746,            2050,            2904,            3076,            3419,            ],        [            4201,            3041,            1845,            3324,            4162,            3483,            2354,            3863,            2648,            754,            1516,            1088,            1653,            1625,            3072,            2172,            1915,            116,            460,            2262,            1867,            3297,            2600,            2577,            2495,            2111,            2004,            1524,            935,            454,            1424,            1242,            1765,            1472,            1828,            1090,            1088,            1302,            2773,            517,            922,            2356,            2458,            2641,            2915,            2966,            2894,            2478,            2169,            2834,            1346,            2525,            1536,            2561,            2545,            3988,            2289,            3169,            2667,            4748,            4312,            2387,            3169,            992,            1556,            2393,            2825,            2664,            3567,            2025,            287,            3288,            2957,            2568,            2167,            2925,            2489,            1418,            2004,            2256,            2981,            0,            2856,            2535,            3538,            3067,            3864,            2447,            3201,            3024,            3828,            1319,            3811,            797,            3681,            745,            1039,            3508,            4303,            1095,            ],        [            1080,            3240,            2091,            3523,            3292,            2404,            2553,            2993,            2847,            2062,            1286,            2901,            3488,            3765,            5212,            4312,            4055,            3038,            3188,            4681,            961,            2388,            700,            2320,            1586,            1190,            2203,            2451,            3046,            3182,            3259,            3382,            4493,            2509,            1908,            2311,            3340,            3363,            4913,            2431,            2973,            1455,            919,            1740,            2006,            647,            1985,            1470,            2368,            1925,            1592,            880,            1324,            661,            375,            3079,            1934,            2157,            2696,            3839,            5100,            1379,            2070,            1824,            4284,            4533,            1916,            2863,            5707,            4165,            3015,            969,            2986,            4708,            4307,            2024,            646,            3459,            2183,            2455,            662,            2842,            0,            1634,            3737,            2158,            1346,            1538,            882,            253,            4027,            2662,            2941,            2711,            1825,            3473,            1777,            3707,            3433,            3146,            ],        [            3098,            4582,            3329,            4865,            4611,            3723,            3791,            4312,            4189,            2797,            1633,            3201,            3093,            5110,            6557,            5657,            5400,            3338,            3488,            5290,            1566,            821,            991,            3558,            1022,            935,            3441,            3689,            3346,            3482,            3322,            4727,            4793,            2809,            2208,            2611,            3640,            3663,            6258,            3659,            3950,            245,            2157,            204,            1297,            1895,            556,            2789,            3606,            1378,            2830,            525,            2562,            972,            1530,            1830,            1370,            1593,            2132,            2590,            7118,            2698,            3389,            2559,            4584,            5878,            431,            4101,            7052,            5510,            3315,            2288,            2422,            6053,            5652,            600,            759,            3759,            3421,            3693,            1676,            3142,            1267,            0,            5079,            1173,            3364,            809,            2201,            1408,            5369,            2962,            4260,            3492,            3144,            3773,            2512,            5049,            4752,            4123,            ],        [            2322,            707,            1738,            283,            2283,            1604,            1297,            1984,            1383,            2752,            3926,            3676,            4540,            3189,            4636,            3736,            3479,            3527,            3710,            4231,            3601,            5028,            4329,            1509,            4226,            3830,            1702,            2129,            3822,            3341,            4311,            3096,            4652,            4359,            4548,            4501,            3975,            4189,            4337,            3105,            2651,            4095,            2813,            4380,            4646,            2995,            4625,            1920,            1482,            4565,            2218,            4244,            2408,            4290,            3570,            5719,            4574,            4797,            5649,            6479,            2433,            2446,            1290,            2514,            4443,            3957,            4556,            987,            5131,            3589,            3785,            2398,            5939,            4132,            3731,            4664,            4275,            4305,            1861,            1729,            3010,            3525,            4448,            4274,            0,            4798,            1985,            4178,            2485,            4049,            787,            4206,            1932,            3385,            1802,            3632,            2467,            467,            1938,            2824,            ],        [            3145,            5160,            3376,            5443,            5357,            4469,            4473,            5058,            4767,            2987,            1862,            2292,            2542,            4201,            5648,            4748,            4491,            2429,            2579,            4381,            1613,            616,            1672,            3605,            1069,            1801,            4123,            3643,            2437,            2573,            2650,            3818,            3884,            1900,            1299,            1702,            2731,            2754,            5349,            2750,            3041,            860,            2204,            721,            388,            2712,            369,            3535,            4288,            469,            2877,            1206,            2609,            1653,            2291,            921,            905,            1042,            1581,            1681,            7165,            3444,            4135,            3109,            3675,            4969,            494,            4783,            6143,            4601,            2406,            3034,            1871,            5144,            4743,            945,            1440,            2850,            3468,            4375,            2727,            2233,            1948,            925,            5657,            0,            3411,            856,            2947,            2770,            5947,            2053,            5006,            2583,            3890,            2864,            3062,            5627,            5498,            3214,            ],        [            548,            1453,            991,            1736,            1951,            1063,            1997,            1652,            1654,            1904,            2607,            3947,            4811,            3586,            5033,            4133,            3876,            3798,            3981,            4628,            2282,            3709,            2020,            1780,            2907,            2511,            1973,            2400,            4093,            3612,            4582,            3331,            4923,            4630,            3229,            3632,            4246,            4460,            4734,            3376,            2922,            2776,            1939,            3061,            3327,            1267,            3306,            192,            1812,            3246,            1434,            2200,            1534,            1981,            1696,            4400,            3255,            3478,            4017,            5160,            2101,            895,            729,            1666,            4714,            4354,            3237,            1823,            5528,            3986,            4056,            670,            4307,            4529,            4128,            3345,            1966,            4576,            2132,            1899,            1282,            3796,            1972,            2955,            1950,            3479,            0,            2859,            757,            1573,            2240,            4477,            1600,            3656,            150,            3903,            1619,            1920,            2092,            3095,            ],        [            2289,            3669,            2520,            3952,            4501,            3613,            2982,            4202,            3276,            2491,            1006,            2108,            2358,            4017,            5464,            4564,            4307,            2245,            2395,            4197,            757,            850,            1434,            2749,            287,            945,            2632,            2880,            2253,            2389,            2587,            3634,            3700,            1716,            1115,            1518,            2547,            2570,            5165,            2860,            2857,            744,            1348,            605,            468,            1856,            447,            2679,            2797,            387,            2021,            1090,            1753,            1395,            1435,            1541,            635,            858,            1397,            2301,            6309,            2588,            3279,            2253,            3491,            4785,            378,            3292,            5959,            4417,            2222,            2178,            1687,            4960,            4559,            1565,            1323,            2666,            2612,            2884,            1871,            2049,            1690,            809,            4166,            620,            2555,            0,            2091,            1914,            4456,            1869,            4150,            2399,            3034,            2680,            2206,            4136,            4642,            3030,            ],        [            1463,            1912,            593,            2195,            2410,            1522,            1581,            2111,            1453,            1506,            1850,            2446,            4395,            3298,            4745,            3845,            3588,            2376,            3565,            4214,            1525,            2952,            1263,            822,            2150,            1754,            1557,            1984,            3677,            3196,            4166,            2915,            4507,            3073,            2472,            2875,            3830,            4044,            4446,            1875,            2506,            2019,            1483,            2304,            2570,            510,            2549,            588,            1396,            2489,            1036,            1443,            1136,            1224,            939,            3643,            2498,            2721,            3260,            4403,            2560,            497,            1188,            1268,            4298,            4066,            2480,            2282,            5240,            3698,            2547,            87,            3550,            4241,            3840,            2588,            1209,            4160,            1716,            1483,            525,            2260,            1215,            2198,            2409,            2722,            670,            2102,            0,            816,            2699,            3226,            2059,            2155,            943,            3487,            1221,            2379,            2551,            2679,            ],        [            1488,            3471,            2322,            3754,            3700,            2812,            2784,            3401,            3078,            2293,            1517,            3132,            3719,            3996,            5443,            4543,            4286,            3269,            3419,            4912,            1192,            2202,            447,            2551,            1817,            631,            2434,            2682,            3277,            3413,            3490,            3613,            4724,            2740,            2139,            2542,            3571,            3594,            5144,            2662,            3204,            973,            1150,            1475,            2237,            1055,            2216,            1878,            2599,            2156,            1823,            627,            1555,            408,            606,            3310,            2165,            2388,            2927,            4070,            5508,            1787,            2478,            2055,            4515,            4764,            2147,            3094,            5938,            4396,            3246,            1377,            3217,            4939,            4538,            1770,            393,            3690,            2414,            2686,            758,            3073,            399,            1271,            3968,            2389,            1754,            1769,            1290,            0,            4258,            2893,            3349,            2942,            2233,            3704,            2008,            3938,            3841,            3377,            ],        [            2625,            1170,            2041,            586,            1287,            1907,            1600,            2297,            1686,            3055,            4229,            3979,            4843,            3492,            4939,            4039,            3782,            3830,            4013,            4534,            3904,            5331,            4632,            1812,            4529,            4133,            2005,            2432,            4125,            3644,            4614,            3399,            4955,            4662,            4851,            4804,            4278,            4492,            4640,            3408,            2954,            4398,            3116,            4683,            4949,            3298,            4928,            2223,            1785,            4868,            2521,            4547,            2711,            4593,            3873,            6022,            4877,            5100,            5952,            6782,            1437,            2749,            1593,            2817,            4746,            4260,            4859,            1290,            5434,            3892,            4088,            2701,            6242,            4435,            4034,            4967,            4578,            4608,            2164,            2032,            3313,            3828,            4751,            4577,            816,            5101,            2288,            4481,            2788,            4352,            0,            4509,            1806,            3688,            2105,            3935,            2770,            770,            790,            3127,            ],        [            3350,            4048,            3813,            4331,            5179,            4490,            3361,            4880,            3655,            1813,            1467,            1340,            826,            2552,            3999,            3099,            2842,            1061,            599,            2266,            1818,            3174,            2551,            3584,            2446,            2062,            3011,            2531,            384,            1481,            597,            2706,            1428,            153,            1779,            750,            678,            701,            3700,            2383,            1929,            2307,            2409,            2592,            2866,            2917,            2845,            3740,            3176,            2785,            3082,            2476,            2814,            2512,            2496,            3479,            2240,            2322,            1820,            4239,            5329,            3649,            4176,            2051,            1219,            3320,            2776,            4021,            3937,            2952,            802,            3239,            2110,            3495,            3094,            2876,            2440,            797,            3011,            3263,            2932,            1059,            2807,            2486,            4545,            2558,            3616,            2398,            3152,            2975,            4835,            0,            4828,            1631,            4688,            1003,            2098,            4515,            5320,            2102,            ],        [            1396,            903,            1459,            1186,            1401,            678,            1447,            1102,            1104,            2473,            3025,            3397,            4261,            3036,            4483,            3583,            3326,            3248,            3431,            4078,            3322,            4749,            4050,            1230,            3947,            3551,            1423,            1850,            3543,            3062,            4032,            2781,            4373,            4080,            3647,            4222,            3696,            3910,            4184,            2826,            2372,            3816,            2534,            4101,            4367,            2069,            4346,            994,            1262,            4286,            1939,            3965,            2129,            4011,            3291,            5440,            4295,            4518,            5370,            6200,            1551,            1697,            364,            2235,            4164,            3804,            4277,            1273,            4978,            3436,            3506,            1472,            5660,            3979,            3578,            4385,            3996,            4026,            1582,            1349,            2084,            3246,            2774,            3995,            1400,            4519,            1059,            3899,            1559,            2375,            1690,            3927,            0,            3106,            876,            3353,            2188,            1370,            1542,            2545,            ],        [            2953,            2899,            1740,            3182,            4030,            3341,            2212,            3731,            2506,            649,            608,            291,            2647,            1520,            2967,            2067,            1810,            913,            1257,            2436,            967,            2492,            1700,            1969,            1595,            1211,            1862,            1382,            1929,            1448,            2418,            1137,            2759,            2466,            920,            1323,            2082,            2296,            2668,            280,            758,            1456,            1836,            1741,            2015,            2066,            1994,            2373,            2027,            1934,            1241,            1625,            1431,            1661,            1645,            2797,            1381,            1604,            2143,            3557,            4180,            2282,            3027,            887,            2550,            2288,            1925,            2522,            3462,            1920,            1084,            2388,            2433,            2463,            2062,            2025,            1589,            2412,            1862,            2114,            2081,            797,            1956,            1635,            3396,            1876,            2616,            1547,            2301,            2124,            3686,            2313,            3679,            0,            3539,            1739,            934,            3366,            4171,            931,            ],        [            520,            1303,            917,            1586,            1801,            913,            1847,            1502,            1504,            1830,            2533,            3797,            4661,            3436,            4883,            3983,            3726,            3648,            3831,            4478,            2208,            3635,            1946,            1630,            2833,            2437,            1823,            2250,            3943,            3462,            4432,            3181,            4773,            4480,            3155,            3558,            4096,            4310,            4584,            3226,            2772,            2702,            1865,            2987,            3253,            1193,            3232,            118,            1662,            3172,            1360,            2126,            1460,            1907,            1622,            4326,            3181,            3404,            3943,            5086,            1951,            821,            579,            1592,            4564,            4204,            3163,            1673,            5378,            3836,            3906,            596,            4233,            4379,            3978,            3271,            1892,            4426,            1982,            1749,            1208,            3646,            1898,            2881,            1800,            3405,            183,            2785,            683,            1499,            2090,            4327,            1450,            3506,            0,            3753,            1545,            1770,            1942,            2945,            ],        [            4459,            3545,            3064,            3828,            4430,            3741,            2612,            4131,            2906,            1871,            2041,            2205,            908,            1803,            3250,            2350,            2093,            918,            341,            1517,            2392,            3371,            3125,            2835,            3020,            2636,            2262,            1782,            190,            732,            679,            1710,            1020,            727,            2353,            1324,            343,            557,            2951,            1634,            1180,            2881,            2983,            3166,            2879,            3491,            3124,            4057,            2427,            2960,            2619,            3050,            2809,            3086,            3070,            3676,            2779,            2519,            2017,            4436,            4580,            3660,            3427,            2109,            811,            2571,            3249,            3272,            3188,            2203,            544,            3813,            2307,            2746,            2345,            3700,            3014,            673,            2262,            2514,            3506,            916,            3381,            3060,            4042,            2755,            4122,            2972,            3726,            3549,            4332,            574,            4079,            1914,            3939,            0,            2156,            4012,            4571,            1353,            ],        [            2019,            1955,            806,            2238,            3086,            2397,            1268,            2787,            1562,            285,            961,            1225,            2889,            1762,            3209,            2309,            2052,            1155,            1499,            2678,            1320,            3117,            2418,            1035,            2315,            1564,            918,            1166,            2171,            1690,            2660,            1379,            3001,            2708,            1583,            2129,            2324,            2538,            2910,            654,            1000,            1809,            902,            2094,            2735,            2080,            2714,            1439,            1083,            2654,            307,            1978,            497,            2379,            1659,            3808,            2663,            2886,            3425,            4568,            3236,            1348,            2039,            47,            2792,            2530,            2645,            1578,            3704,            2162,            1326,            1546,            3715,            2705,            2304,            2378,            1942,            2654,            898,            1170,            2095,            1039,            2537,            1988,            2452,            2887,            1682,            2267,            1633,            2138,            2742,            2555,            2735,            934,            1546,            1981,            0,            2422,            3227,            1173,            ],        [            1855,            240,            1271,            523,            1826,            1137,            830,            1527,            916,            2285,            2837,            3209,            4073,            2722,            4169,            3269,            3012,            3060,            3243,            3764,            3134,            4561,            3862,            1042,            3759,            3363,            1235,            1662,            3355,            2874,            3844,            2629,            4185,            3892,            3459,            4034,            3508,            3722,            3870,            2638,            2184,            3628,            2346,            3913,            4179,            2528,            4158,            1453,            1015,            4098,            1751,            3777,            1941,            3823,            3103,            5252,            4107,            4330,            5182,            6012,            1976,            1979,            823,            2047,            3976,            3490,            4089,            520,            4664,            3122,            3318,            1931,            5472,            3665,            3264,            4197,            3808,            3838,            1394,            1262,            2543,            3058,            3981,            3807,            737,            4331,            1518,            3711,            2018,            3582,            1204,            3739,            1475,            2918,            1335,            3165,            2000,            0,            1471,            2357,            ],        [            2412,            1450,            2475,            1376,            497,            1694,            2463,            1090,            2120,            3489,            4663,            4413,            5277,            3755,            5202,            4302,            4045,            4264,            4447,            4797,            4338,            5765,            5857,            2246,            4963,            4567,            2439,            2866,            4559,            4078,            5048,            3797,            5389,            5096,            5285,            5238,            4712,            4926,            4903,            3842,            3388,            4832,            3550,            5117,            5383,            3085,            5362,            2010,            2278,            5302,            2955,            4981,            3145,            5818,            5098,            6456,            5311,            5534,            6386,            7216,            647,            2713,            1380,            3251,            5180,            4523,            5293,            1820,            5697,            4155,            4522,            2488,            6676,            4698,            4297,            5401,            5803,            5042,            2598,            2365,            3100,            4262,            5399,            5011,            1606,            5535,            2075,            4915,            2575,            3391,            790,            4943,            1016,            4122,            1892,            4369,            3204,            1560,            0,            3561,            ],        [            3805,            2645,            2410,            2928,            3776,            3087,            1958,            3477,            2252,            760,            2103,            1551,            1716,            1686,            3133,            2233,            1976,            703,            886,            2602,            2454,            3884,            3187,            2181,            3082,            2698,            1608,            1128,            998,            517,            1487,            1303,            1828,            1535,            2415,            1677,            1151,            1365,            2834,            980,            526,            2943,            2560,            3228,            3502,            3553,            3481,            3403,            1773,            3421,            1965,            3112,            2155,            3148,            3132,            4575,            2876,            3327,            2825,            5335,            3926,            3006,            2773,            998,            1619,            2454,            3412,            2268,            3628,            2086,            961,            3204,            3115,            2629,            2228,            3512,            3076,            1481,            1608,            1860,            3568,            701,            3443,            3122,            3142,            3654,            3468,            3034,            3788,            3611,            3432,            1382,            3425,            1260,            3285,            808,            1045,            3112,            3917,            0,            ],        ], 'starts': [5, 23, 46, 19], 'ends': [5, 23, 46, 19]},    'NW-14': {'distance_matrix': [        [            0,            1716,            1546,            1957,            899,            1227,            683,            2390,            1623,            2316,            1422,            1406,            1792,            980,            767,            1347,            543,            2580,            2742,            1443,            2067,            1945,            1376,            725,            2612,            2553,            2848,            2468,            2649,            2112,            2764,            2706,            2023,            2861,            1655,            1688,            1713,            2276,            2685,            1461,            1670,            3006,            2488,            2238,            2679,            1602,            1685,            2421,            452,            2136,            913,            1929,            2282,            1363,            1369,            2538,            1462,            2627,            2629,            1910,            1836,            651,            2295,            2144,            2789,            2019,            865,            2563,            1958,            3331,            2146,            2209,            1487,            2295,            2176,            1923,            2456,            2254,            ],        [            1336,            0,            1361,            241,            714,            1042,            1709,            1663,            896,            2131,            695,            657,            520,            2006,            1346,            1926,            1122,            3159,            2867,            2022,            2743,            2524,            2402,            1769,            3191,            3579,            3657,            3494,            3675,            3138,            3343,            3285,            3049,            3440,            2681,            2714,            2739,            3302,            3264,            2487,            2696,            3585,            3067,            2817,            3705,            2181,            2711,            1694,            781,            974,            1939,            2508,            3308,            2532,            2395,            3117,            271,            3206,            3208,            2936,            1109,            706,            2874,            872,            3815,            3045,            658,            3142,            1773,            4357,            2725,            2788,            534,            2874,            1449,            538,            3035,            2833,            ],        [            1948,            1690,            0,            1931,            922,            705,            2321,            1786,            1597,            1203,            1860,            1380,            1791,            2618,            1958,            2538,            1734,            2896,            1877,            2634,            3355,            3136,            3014,            2381,            2961,            4191,            4269,            4106,            4287,            3750,            3113,            1743,            3661,            3210,            3293,            3326,            3351,            3914,            3876,            3099,            3308,            4197,            3679,            3429,            4317,            2793,            3323,            2166,            1393,            1881,            2551,            3120,            3920,            3144,            3007,            2254,            1436,            2783,            2785,            3548,            1417,            1318,            3486,            2143,            4427,            3657,            1172,            3754,            412,            4969,            3337,            3400,            1461,            3486,            1236,            1922,            2555,            2695,            ],        [            1577,            241,            1602,            0,            955,            1283,            1950,            1904,            1137,            2372,            936,            898,            592,            2247,            1587,            2167,            1363,            3400,            3108,            2263,            2984,            2765,            2643,            2010,            3432,            3820,            3898,            3735,            3916,            3379,            3584,            3526,            3290,            3681,            2922,            2955,            2980,            3543,            3505,            2728,            2937,            3826,            3308,            3058,            3946,            2422,            2952,            1003,            1022,            667,            2180,            2749,            3549,            2773,            2636,            3358,            512,            3447,            3449,            3177,            1350,            947,            3115,            944,            4056,            3286,            899,            3383,            2014,            4598,            2966,            3029,            409,            3115,            1690,            297,            3276,            3074,            ],        [            1617,            1246,            647,            1487,            0,            328,            1990,            2433,            1599,            1417,            1398,            1382,            1460,            2287,            1627,            2207,            1403,            3172,            2153,            2303,            3024,            2805,            2683,            2050,            3237,            3860,            3938,            3775,            3956,            3419,            3389,            1957,            3330,            3486,            2962,            2995,            3020,            3583,            3545,            2768,            2977,            3866,            3348,            3098,            3986,            2462,            2992,            2397,            1062,            2112,            2220,            2789,            3589,            2813,            2676,            2530,            1371,            3059,            3061,            3217,            1812,            987,            3155,            1812,            4096,            3326,            841,            3423,            1059,            4638,            3006,            3069,            1463,            3155,            1883,            1591,            2831,            2971,            ],        [            1894,            1523,            319,            1764,            868,            0,            2267,            2105,            1876,            1345,            2179,            1699,            1737,            2564,            1904,            2484,            1680,            3100,            2081,            2580,            3301,            3082,            2960,            2327,            3165,            4137,            4215,            4052,            4233,            3696,            3317,            1885,            3607,            3414,            3239,            3272,            3297,            3860,            3822,            3045,            3254,            4143,            3625,            3375,            4263,            2739,            3269,            2485,            1339,            2200,            2497,            3066,            3866,            3090,            2953,            2458,            1648,            2987,            2989,            3494,            1736,            1264,            3432,            2089,            4373,            3603,            1118,            3700,            731,            4915,            3283,            3346,            1780,            3432,            1555,            1868,            2759,            2899,            ],        [            236,            1952,            1782,            2193,            1135,            1463,            0,            2626,            1859,            2552,            1658,            1642,            2028,            642,            1003,            1583,            779,            2816,            2978,            1679,            1729,            1935,            1038,            612,            2848,            2215,            2510,            2130,            2311,            1774,            3000,            2942,            1685,            3097,            1317,            1350,            1375,            1938,            2921,            1123,            1332,            3242,            2724,            2474,            2341,            1838,            1347,            2657,            688,            2372,            575,            2165,            1944,            1025,            1031,            2774,            1698,            2863,            2865,            1572,            2072,            887,            1764,            2380,            2451,            1681,            1101,            2799,            2194,            2993,            2382,            2445,            1723,            1764,            2412,            2159,            2692,            2490,            ],        [            2419,            1546,            2048,            1787,            1730,            2491,            2792,            0,            868,            3013,            943,            1236,            2154,            3089,            2429,            3009,            2205,            4242,            3693,            3105,            3826,            3607,            3485,            2852,            4274,            4662,            4740,            4577,            4758,            4221,            4426,            4368,            4132,            4523,            3764,            3797,            3822,            4385,            4347,            3570,            3779,            4668,            4150,            3900,            4788,            3264,            3794,            583,            1864,            889,            3022,            3591,            4391,            3615,            3478,            4200,            1292,            4289,            4291,            4019,            701,            1789,            3957,            2506,            4898,            4128,            1712,            4225,            2222,            5440,            3808,            3871,            1356,            3957,            1722,            2285,            4118,            3916,            ],        [            1968,            1095,            1560,            1336,            1279,            2003,            2341,            767,            0,            2525,            492,            785,            1703,            2638,            1978,            2558,            1754,            4224,            3205,            2654,            3375,            3156,            3034,            2401,            3823,            4211,            4289,            4126,            4307,            3770,            3975,            3917,            3681,            4072,            3313,            3346,            3371,            3934,            3896,            3119,            3328,            4217,            3699,            3449,            4337,            2813,            3343,            798,            1413,            513,            2571,            3140,            3940,            3164,            3027,            3582,            841,            3838,            4113,            3568,            213,            1338,            3506,            2055,            4447,            3677,            1261,            3774,            1734,            4989,            3357,            3420,            980,            3506,            1234,            1560,            3667,            3465,            ],        [            2052,            1681,            719,            1922,            1026,            685,            2425,            2288,            2034,            0,            1833,            1817,            1895,            2722,            2062,            2642,            1838,            2371,            1352,            2738,            3459,            3240,            3118,            2485,            2436,            4295,            4373,            4210,            4391,            3854,            2588,            1156,            3765,            2685,            3397,            3430,            3455,            4018,            3152,            3203,            3412,            3473,            2955,            2705,            4421,            2897,            3427,            2832,            1497,            2547,            2655,            2396,            4024,            3248,            3111,            1729,            1806,            2258,            2260,            3652,            2247,            1422,            3590,            2247,            4531,            3761,            1276,            3858,            1131,            5073,            3441,            3058,            1898,            3590,            1738,            2026,            2030,            2170,            ],        [            1476,            603,            1329,            844,            787,            1115,            1849,            1299,            532,            2204,            0,            293,            1211,            2146,            1486,            2066,            1262,            3299,            3461,            2162,            2883,            2664,            2542,            1909,            3331,            3719,            3797,            3634,            3815,            3278,            3483,            3425,            3189,            3580,            2821,            2854,            2879,            3442,            3404,            2627,            2836,            3725,            3207,            2957,            3845,            2321,            2851,            1330,            921,            814,            2079,            2648,            3448,            2672,            2535,            3257,            349,            3346,            3348,            3076,            745,            846,            3014,            1563,            3955,            3185,            769,            3282,            1503,            4497,            2865,            2928,            374,            3014,            1085,            1342,            3175,            2973,            ],        [            1183,            785,            1141,            1026,            494,            822,            1556,            1526,            759,            1911,            558,            0,            999,            1853,            1193,            1773,            969,            3006,            3168,            1869,            2590,            2371,            2249,            1616,            3038,            3426,            3504,            3341,            3522,            2985,            3190,            3132,            2896,            3287,            2528,            2561,            2586,            3149,            3111,            2334,            2543,            3432,            2914,            2664,            3552,            2028,            2558,            1557,            628,            1272,            1786,            2355,            3155,            2379,            2242,            2964,            910,            3053,            3055,            2783,            972,            553,            2721,            1351,            3662,            2892,            476,            2989,            1553,            4204,            2572,            2635,            623,            2721,            1312,            1130,            2882,            2680,            ],        [            1550,            520,            1575,            592,            928,            1256,            1923,            2271,            1504,            2345,            1303,            1265,            0,            2220,            1560,            2140,            1336,            3373,            3081,            2236,            2957,            2738,            2616,            1983,            3405,            3793,            3871,            3708,            3889,            3352,            3557,            3499,            3263,            3654,            2895,            2928,            2953,            3516,            3478,            2701,            2910,            3799,            3281,            3031,            3919,            2395,            2925,            2302,            995,            1582,            2153,            2722,            3522,            2746,            2609,            3331,            879,            3420,            3422,            3150,            1717,            920,            3088,            352,            4029,            3259,            872,            3356,            1987,            4571,            2939,            3002,            1142,            3088,            2057,            295,            3249,            3047,            ],        [            590,            2306,            2136,            2547,            1489,            1817,            354,            2980,            2213,            2906,            2012,            1996,            2382,            0,            1357,            1548,            1133,            2878,            3040,            1644,            1738,            1944,            1047,            489,            2910,            2224,            2519,            2182,            2320,            1783,            3062,            3004,            1737,            3159,            1326,            1614,            1427,            1947,            2983,            1132,            1596,            3304,            2786,            2536,            2393,            1803,            1356,            3011,            1042,            2726,            627,            2227,            1953,            1034,            1083,            2836,            2052,            2925,            2927,            1581,            2426,            1241,            1773,            2734,            2460,            1690,            1455,            2614,            2548,            3002,            2548,            2889,            2077,            1773,            2766,            2513,            2754,            2552,            ],        [            1035,            1958,            1788,            2199,            1141,            1469,            1806,            2632,            1865,            2558,            1664,            1648,            2034,            1958,            0,            580,            398,            1813,            1975,            676,            1397,            1178,            1253,            1519,            1845,            2430,            2311,            3013,            2526,            1989,            1997,            1939,            2568,            2094,            1532,            1820,            2042,            2153,            1918,            1338,            1802,            2239,            1721,            1471,            3224,            835,            1562,            2663,            1067,            2378,            1638,            1162,            2159,            1186,            1947,            1771,            1704,            1860,            1862,            1787,            2078,            736,            1528,            2386,            2666,            1896,            1107,            1796,            2200,            3208,            1379,            1442,            1729,            1528,            2418,            2165,            1689,            1487,            ],        [            1331,            2254,            2084,            2495,            1437,            1765,            1704,            2928,            2161,            2854,            1960,            1944,            2330,            2001,            565,            0,            694,            2109,            2271,            468,            1189,            970,            1045,            1476,            2141,            2222,            2103,            2805,            2318,            1781,            2293,            2235,            2360,            2390,            1324,            1612,            1834,            1945,            2214,            1130,            1594,            2535,            2017,            1767,            3016,            627,            1354,            2959,            1363,            2674,            1934,            1458,            1951,            978,            1739,            2067,            2000,            2156,            2158,            1579,            2374,            1032,            1320,            2682,            2458,            1688,            1403,            1588,            2496,            3000,            1171,            1234,            2025,            1320,            2714,            2461,            1985,            1783,            ],        [            993,            1947,            1777,            2188,            1130,            1458,            1366,            2621,            1854,            2547,            1653,            1637,            2023,            1663,            224,            804,            0,            2037,            2199,            900,            1621,            1402,            1477,            1138,            2069,            2654,            2535,            3237,            2750,            2213,            2221,            2163,            2792,            2318,            1756,            2044,            2266,            2377,            2142,            1562,            2026,            2463,            1945,            1695,            3448,            1059,            1786,            2652,            1056,            2367,            1596,            1386,            2383,            1410,            2171,            1995,            1693,            2084,            2086,            2011,            2067,            725,            1752,            2375,            2890,            2120,            1096,            2020,            2189,            3432,            1603,            1666,            1718,            1752,            2407,            2154,            1913,            1711,            ],        [            2558,            3481,            3311,            3722,            2664,            2992,            2591,            4155,            3388,            2319,            3187,            3171,            3557,            2743,            1792,            1325,            1921,            0,            960,            1216,            2223,            2004,            2038,            2304,            296,            3215,            3137,            3798,            3311,            2774,            448,            924,            3353,            545,            2317,            2605,            2827,            2938,            1454,            2123,            2587,            1775,            1257,            1007,            4009,            1620,            2347,            4186,            2590,            3901,            2675,            698,            2944,            1971,            2732,            756,            3227,            311,            313,            2572,            3601,            2259,            2354,            3909,            3451,            2681,            2630,            2622,            2789,            3993,            2032,            1360,            3252,            2354,            3941,            3688,            674,            472,            ],        [            2904,            3094,            2132,            3335,            3010,            2098,            2937,            3698,            3734,            1413,            3533,            3517,            3308,            3089,            2138,            1671,            2267,            929,            0,            1562,            2569,            2350,            2384,            2650,            1232,            3561,            3483,            4144,            3657,            3120,            1384,            1270,            3699,            1481,            2663,            2951,            3173,            3284,            1800,            2469,            2933,            2121,            1603,            1353,            4355,            1966,            2693,            4532,            2936,            4247,            3021,            1044,            3290,            2317,            3078,            377,            3219,            1247,            818,            2918,            3947,            2605,            2700,            3660,            3797,            3027,            2976,            2968,            1883,            4339,            2378,            1706,            3598,            2700,            2773,            3439,            678,            818,            ],        [            1770,            2693,            2523,            2934,            1876,            2204,            1803,            3367,            2600,            3293,            2399,            2383,            2769,            1955,            1004,            537,            1133,            1234,            1396,            0,            1394,            1175,            1250,            1516,            1266,            2427,            2308,            3010,            2523,            1986,            1418,            1360,            2565,            1515,            1529,            1817,            2039,            2150,            1339,            1335,            1799,            1660,            1142,            892,            3221,            832,            1559,            3398,            1802,            3113,            1887,            583,            2156,            1183,            1944,            1192,            2439,            1281,            1283,            1784,            2813,            1471,            1525,            3121,            2663,            1893,            1842,            1793,            2935,            3205,            1376,            1245,            2464,            1525,            3153,            2900,            1110,            908,            ],        [            1926,            3151,            2981,            3392,            2334,            2662,            1690,            3825,            3058,            3751,            2857,            2841,            3227,            1842,            1462,            897,            1591,            2285,            2447,            993,            0,            206,            761,            1403,            2317,            1357,            1709,            2897,            1453,            916,            1879,            2411,            2452,            1922,            617,            1704,            1926,            1080,            1313,            1222,            1686,            1634,            1116,            1299,            3108,            742,            687,            3856,            2260,            3571,            1774,            1439,            1086,            1070,            1831,            2243,            2897,            2332,            2334,            714,            3271,            1929,            467,            3579,            1593,            823,            2300,            1194,            3393,            2135,            777,            840,            2922,            467,            3611,            3358,            2161,            1959,            ],        [            2193,            2945,            2775,            3186,            2128,            2456,            1957,            3619,            2852,            3545,            2651,            2635,            3021,            2109,            1256,            691,            1385,            2079,            2241,            787,            751,            0,            1404,            1670,            2111,            2916,            1503,            3164,            2019,            1848,            1673,            2205,            2719,            1716,            1711,            1971,            2193,            1646,            1107,            1489,            1953,            1428,            910,            1093,            3375,            536,            1585,            3650,            2054,            3365,            2041,            1233,            2018,            1337,            2098,            2037,            2691,            2126,            2128,            1646,            3065,            1723,            882,            3373,            2159,            1389,            2094,            988,            3187,            2701,            571,            634,            2716,            882,            3405,            3152,            1955,            1753,            ],        [            1588,            3500,            3330,            3741,            2683,            3011,            1352,            4174,            3407,            4100,            3206,            3190,            3576,            1504,            1803,            1246,            1939,            2576,            2738,            1342,            691,            897,            0,            1065,            2608,            1177,            1472,            2559,            1273,            736,            2760,            2702,            2114,            2857,            279,            1366,            1588,            900,            2004,            967,            1348,            2325,            1807,            1990,            2770,            1433,            309,            4205,            2040,            3920,            1436,            2130,            906,            732,            1493,            2534,            3246,            2623,            2625,            534,            3620,            2278,            726,            3928,            1413,            643,            2649,            1567,            3742,            1955,            1501,            1531,            3271,            726,            3960,            3707,            2452,            2250,            ],        [            523,            2239,            2069,            2480,            1422,            1750,            287,            2913,            2146,            2839,            1945,            1929,            2315,            627,            1290,            1870,            1066,            3103,            3265,            1966,            1714,            1920,            1023,            0,            3135,            2200,            2495,            2115,            2296,            1759,            3287,            3229,            1670,            3384,            1302,            1335,            1360,            1923,            3208,            1108,            1317,            3529,            3011,            2761,            2326,            2125,            1332,            2944,            975,            2659,            560,            2452,            1929,            1010,            1016,            3061,            1985,            3150,            3152,            1557,            2359,            1174,            1749,            2667,            2436,            1666,            1388,            2590,            2481,            2978,            2524,            2732,            2010,            1749,            2699,            2446,            2979,            2777,            ],        [            2788,            3711,            3017,            3952,            2894,            2983,            2821,            4583,            3618,            2298,            3417,            3401,            4193,            2973,            2022,            1555,            2151,            437,            1190,            1446,            2453,            2234,            2268,            2534,            0,            3445,            3367,            4028,            3541,            3004,            152,            1154,            3583,            249,            2547,            2835,            3057,            3168,            1684,            2353,            2817,            2005,            1487,            1237,            4239,            1850,            2577,            4416,            2820,            4131,            2905,            928,            3174,            2201,            2962,            986,            3457,            290,            176,            2802,            3831,            2489,            2584,            4545,            3681,            2911,            2860,            2852,            2768,            4223,            2262,            1590,            3482,            2584,            3658,            4324,            904,            702,            ],        [            2397,            4309,            4139,            4550,            3492,            3820,            2161,            4983,            4216,            4909,            4015,            3999,            4385,            2313,            2612,            2055,            2748,            3385,            3547,            2151,            1141,            1347,            1290,            1874,            3417,            0,            1735,            3368,            1536,            441,            3569,            3511,            2923,            3063,            1088,            2175,            2397,            1350,            2454,            1693,            2157,            2775,            2257,            2440,            3579,            1883,            962,            5014,            2849,            4729,            2245,            2580,            271,            1541,            2302,            3343,            4055,            3432,            3434,            769,            4429,            3087,            1290,            4737,            1676,            1093,            3458,            2334,            4551,            215,            1951,            1981,            4080,            1290,            4769,            4516,            3261,            3059,            ],        [            2654,            4213,            4043,            4454,            3396,            3724,            2418,            4887,            4120,            4813,            3919,            3903,            4289,            2570,            2524,            1959,            2653,            3289,            3451,            2055,            1398,            1604,            1547,            2131,            3321,            1755,            0,            3625,            858,            1482,            2941,            3415,            3180,            2984,            1345,            2432,            2654,            736,            1798,            1950,            2414,            1503,            1862,            2361,            3836,            1804,            1219,            4918,            3106,            4633,            2502,            2501,            1652,            1798,            2559,            3247,            3959,            3336,            3338,            1280,            4333,            2991,            1547,            4641,            998,            1023,            3362,            687,            4455,            1540,            1196,            1902,            3984,            1547,            4673,            4420,            3165,            2963,            ],        [            2391,            4107,            3937,            4348,            3290,            3618,            2155,            4781,            4014,            4707,            3813,            3797,            4183,            1801,            3076,            2519,            2934,            3849,            4011,            2615,            2709,            2915,            2018,            2338,            3881,            3195,            3490,            0,            3291,            2754,            4033,            3975,            343,            4130,            2190,            1290,            870,            2918,            3954,            1554,            1272,            4275,            3757,            3507,            505,            2774,            2327,            4812,            2843,            4527,            1838,            3198,            2924,            2005,            1082,            3807,            3853,            3896,            3898,            2552,            4227,            3042,            2744,            4535,            3431,            2661,            3256,            3585,            4349,            3973,            3519,            3860,            3878,            2744,            4567,            4314,            3725,            3523,            ],        [            2341,            4253,            4083,            4494,            3436,            3764,            2105,            4927,            4160,            4853,            3959,            3943,            4329,            2257,            2556,            1999,            2692,            3329,            3491,            2095,            1085,            1291,            1234,            1818,            3361,            1442,            506,            3312,            0,            1169,            3513,            3455,            2867,            3007,            1032,            2119,            2341,            423,            2146,            1637,            2101,            1921,            2201,            2384,            3523,            1827,            906,            4958,            2793,            4673,            2189,            2524,            1339,            1485,            2246,            3287,            3999,            3376,            3378,            967,            4373,            3031,            1234,            4681,            685,            710,            3402,            1105,            4495,            1227,            1544,            1925,            4024,            1234,            4713,            4460,            3205,            3003,            ],        [            1956,            3868,            3698,            4109,            3051,            3379,            1720,            4542,            3775,            4468,            3574,            3558,            3944,            1872,            2171,            1614,            2307,            2944,            3106,            1710,            700,            906,            849,            1433,            2976,            441,            1481,            2927,            1282,            0,            3128,            3070,            2482,            2622,            647,            1734,            1956,            909,            2013,            1252,            1716,            2334,            1816,            1999,            3138,            1442,            521,            4573,            2408,            4288,            1804,            2139,            170,            1100,            1861,            2902,            3614,            2991,            2993,            328,            3988,            2646,            849,            4296,            1422,            652,            3017,            1576,            4110,            656,            1510,            1540,            3639,            849,            4328,            4075,            2820,            2618,            ],        [            2940,            3863,            3169,            4104,            3046,            3135,            2973,            4735,            3770,            2450,            3569,            3553,            4345,            3125,            2174,            1707,            2303,            589,            1342,            1598,            2003,            1784,            2420,            2686,            152,            3597,            2917,            4180,            3693,            3156,            0,            1306,            3735,            97,            2699,            2987,            3209,            3320,            1234,            2505,            2969,            1555,            1037,            1389,            4391,            2002,            2729,            4568,            2972,            4283,            3057,            1080,            3326,            2353,            3114,            1138,            3609,            442,            328,            2954,            3983,            2641,            2134,            4697,            3833,            3063,            3012,            2402,            2920,            4375,            1812,            1140,            3634,            2134,            3810,            4476,            1056,            854,            ],        [            2606,            3529,            3359,            3770,            2712,            3040,            2639,            4203,            3436,            1600,            3235,            3219,            3605,            2791,            1840,            1373,            1969,            1397,            1559,            1264,            2271,            2052,            2086,            2352,            1429,            3263,            3185,            3846,            3359,            2822,            1581,            0,            3401,            1678,            2365,            2653,            2875,            2986,            1502,            2171,            2635,            1823,            1305,            1055,            4057,            1668,            2395,            4234,            2638,            3949,            2723,            746,            2992,            2019,            2780,            631,            3275,            1444,            1446,            2620,            3649,            2307,            2402,            3957,            3499,            2729,            2678,            2670,            3771,            4041,            2080,            1408,            3300,            2402,            3989,            3736,            330,            452,            ],        [            2048,            3764,            3594,            4005,            2947,            3275,            1812,            4438,            3671,            4364,            3470,            3454,            3840,            1458,            2733,            2176,            2591,            3506,            3668,            2272,            2366,            2572,            1675,            1995,            3538,            2852,            3147,            978,            2948,            2411,            3690,            3632,            0,            3787,            1847,            947,            527,            2575,            3611,            1211,            929,            3932,            3414,            3164,            1189,            2431,            1984,            4469,            2500,            4184,            1495,            2855,            2581,            1662,            739,            3464,            3510,            3553,            3555,            2209,            3884,            2699,            2401,            4192,            3088,            2318,            2913,            3242,            4006,            3630,            3176,            3517,            3535,            2401,            4224,            3971,            3382,            3180,            ],        [            3037,            3960,            3266,            4201,            3143,            3232,            3070,            4832,            3867,            2547,            3666,            3650,            4442,            3222,            2271,            1804,            2400,            686,            1439,            1695,            2046,            1827,            2517,            2783,            249,            3694,            2960,            4277,            3790,            3253,            97,            1403,            3832,            0,            2327,            3084,            3306,            3417,            1277,            2602,            3066,            1598,            1080,            1486,            4488,            2099,            2826,            4665,            3069,            4380,            3154,            1177,            3423,            2450,            3211,            1235,            3706,            539,            425,            3051,            4080,            2738,            2177,            4794,            3930,            3160,            3109,            2445,            3017,            4472,            1855,            1183,            3731,            2177,            3907,            4573,            1153,            951,            ],        [            1309,            3221,            3051,            3462,            2404,            2732,            1073,            3895,            3128,            3821,            2927,            2911,            3297,            1225,            1524,            967,            1660,            2297,            2459,            1063,            716,            849,            202,            786,            2329,            1379,            1674,            2280,            1475,            938,            2481,            2423,            1835,            2578,            0,            1087,            1309,            1102,            2402,            605,            1069,            2723,            2205,            1955,            2491,            1222,            511,            3926,            1761,            3641,            1157,            1646,            1108,            453,            1214,            2255,            2967,            2344,            2346,            736,            3341,            1999,            847,            3649,            1615,            845,            2370,            1769,            3463,            2157,            1703,            1483,            2992,            847,            3681,            3428,            2173,            1971,            ],        [            1712,            3428,            3258,            3669,            2611,            2939,            1476,            4102,            3335,            4028,            3134,            3118,            3504,            1122,            2014,            1457,            2150,            2787,            2949,            1553,            1647,            1853,            956,            1276,            2819,            2133,            2428,            1441,            2229,            1692,            2971,            2913,            996,            3068,            1128,            0,            470,            1856,            2892,            492,            210,            3213,            2695,            2445,            1652,            1712,            1265,            4133,            2164,            3848,            1054,            2136,            1862,            943,            668,            2745,            3174,            2834,            2836,            1490,            3548,            2489,            1682,            3856,            2369,            1599,            2577,            2523,            3670,            2911,            2457,            2798,            3199,            1682,            3888,            3635,            2663,            2461,            ],        [            1521,            3237,            3067,            3478,            2420,            2748,            1285,            3911,            3144,            3837,            2943,            2927,            3313,            931,            2206,            1649,            2064,            2979,            3141,            1745,            1839,            2045,            1148,            1468,            3011,            2325,            2620,            997,            2421,            1884,            3163,            3105,            552,            3260,            1320,            420,            0,            2048,            3084,            684,            402,            3405,            2887,            2637,            1208,            1904,            1457,            3942,            1973,            3657,            968,            2328,            2054,            1135,            354,            2937,            2983,            3026,            3028,            1682,            3357,            2172,            1874,            3665,            2561,            1791,            2386,            2715,            3479,            3103,            2649,            2990,            3008,            1874,            3697,            3444,            2855,            2653,            ],        [            2267,            4179,            4009,            4420,            3362,            3690,            2031,            4853,            4086,            4779,            3885,            3869,            4255,            2183,            2482,            1925,            2618,            3255,            3417,            2021,            1011,            1217,            1160,            1744,            3287,            1270,            572,            3238,            373,            1095,            3439,            3381,            2793,            2933,            958,            2045,            2267,            0,            2072,            1563,            2027,            1987,            2127,            2310,            3449,            1753,            832,            4884,            2719,            4599,            2115,            2450,            1265,            1411,            2172,            3213,            3925,            3302,            3304,            893,            4299,            2957,            1160,            4607,            513,            636,            3328,            1171,            4421,            1055,            1470,            1851,            3950,            1160,            4639,            4386,            3131,            2929,            ],        [            2911,            3834,            3664,            4075,            3017,            3345,            2944,            4508,            3741,            3257,            3540,            3524,            3910,            3096,            2145,            1581,            2274,            1682,            1844,            1569,            1497,            1278,            2391,            2657,            1714,            3140,            1665,            4151,            2243,            2077,            1276,            1808,            3706,            1319,            1778,            2958,            3180,            1870,            0,            2476,            2940,            321,            257,            696,            4362,            1426,            1782,            4539,            2943,            4254,            3028,            836,            2247,            2324,            3085,            1640,            3580,            1729,            1731,            1875,            3954,            2612,            1628,            4262,            2383,            1613,            2983,            1212,            4076,            2925,            777,            634,            3605,            1628,            4294,            4041,            1558,            1356,            ],        [            1220,            2936,            2766,            3177,            2119,            2447,            984,            3610,            2843,            3536,            2642,            2626,            3012,            630,            1544,            987,            1763,            2317,            2479,            1083,            1177,            1383,            486,            806,            2349,            1663,            1958,            1685,            1759,            1222,            2501,            2443,            1240,            2598,            765,            492,            714,            1386,            2422,            0,            474,            2743,            2225,            1975,            1896,            1242,            795,            3641,            1672,            3356,            562,            1666,            1392,            473,            619,            2275,            2682,            2364,            2366,            1020,            3056,            1871,            1212,            3364,            1899,            1129,            2085,            2053,            3178,            2441,            1987,            2328,            2707,            1212,            3396,            3143,            2193,            1991,            ],        [            1502,            3218,            3048,            3459,            2401,            2729,            1266,            3892,            3125,            3818,            2924,            2908,            3294,            912,            1804,            1247,            1940,            2577,            2739,            1343,            1437,            1643,            746,            1066,            2609,            1923,            2218,            1967,            2019,            1482,            2761,            2703,            1522,            2858,            918,            774,            996,            1646,            2682,            282,            0,            3003,            2485,            2235,            2178,            1502,            1055,            3923,            1954,            3638,            844,            1926,            1652,            733,            901,            2535,            2964,            2624,            2626,            1280,            3338,            2279,            1472,            3646,            2159,            1389,            2367,            2313,            3460,            2701,            2247,            2588,            2989,            1472,            3678,            3425,            2453,            2251,            ],        [            3229,            4152,            3982,            4393,            3335,            3663,            3262,            4826,            4059,            3575,            3858,            3842,            4228,            3414,            2463,            1899,            2592,            2000,            2162,            1887,            1815,            1596,            2709,            2975,            2032,            2983,            1344,            4469,            2086,            2395,            1594,            2126,            4024,            1637,            2096,            3276,            3498,            1964,            424,            2794,            3258,            0,            575,            1014,            4680,            1744,            2100,            4857,            3261,            4572,            3346,            1154,            2565,            2642,            3403,            1958,            3898,            2047,            2049,            2193,            4272,            2930,            1946,            4580,            2226,            1838,            3301,            1409,            4394,            2768,            1002,            952,            3923,            1946,            4612,            4359,            1876,            1674,            ],        [            2666,            3589,            3419,            3830,            2772,            3100,            2699,            4263,            3496,            3012,            3295,            3279,            3665,            2851,            1900,            1336,            2029,            1437,            1599,            1324,            1252,            1033,            2146,            2412,            1469,            2273,            2166,            3906,            2369,            1832,            1031,            1563,            3461,            1074,            1533,            2713,            2935,            1996,            203,            2231,            2695,            524,            0,            451,            4117,            1181,            1537,            4294,            2698,            4009,            2783,            591,            2002,            2079,            2840,            1395,            3335,            1484,            1486,            1630,            3709,            2367,            1383,            4017,            2509,            1739,            2738,            1651,            3831,            3051,            781,            389,            3360,            1383,            4049,            3796,            1313,            1111,            ],        [            2802,            3554,            3384,            3795,            2737,            3065,            2566,            4228,            3461,            2882,            3260,            3244,            3630,            2718,            1865,            1300,            1994,            1307,            1469,            793,            1216,            997,            1641,            2279,            1339,            2237,            2130,            3773,            2333,            1796,            901,            1433,            3328,            944,            1497,            2580,            2802,            1960,            447,            2098,            2562,            768,            250,            0,            3984,            1145,            1501,            4259,            2663,            3974,            2650,            461,            1966,            1946,            2707,            1265,            3300,            1354,            1356,            1594,            3674,            2332,            1347,            3982,            2473,            1703,            2703,            1615,            3796,            3015,            1025,            353,            3325,            1347,            4014,            3761,            1183,            981,            ],        [            2648,            4364,            4194,            4605,            3547,            3875,            2412,            5038,            4271,            4964,            4070,            4054,            4440,            2058,            3333,            2776,            3191,            4106,            4268,            2872,            2966,            3172,            2275,            2595,            4138,            3452,            3747,            257,            3548,            3011,            4290,            4232,            600,            4387,            2447,            1547,            1127,            3175,            4211,            1811,            1529,            4532,            4014,            3764,            0,            3031,            2584,            5069,            3100,            4784,            2095,            3455,            3181,            2262,            1339,            4064,            4110,            4153,            4155,            2809,            4484,            3299,            3001,            4792,            3688,            2918,            3513,            3842,            4606,            4230,            3776,            4117,            4135,            3001,            4824,            4571,            3982,            3780,            ],        [            2152,            2918,            2748,            3159,            2101,            2429,            1916,            3592,            2825,            3518,            2624,            2608,            2994,            2082,            1229,            664,            1358,            2052,            2214,            760,            562,            343,            987,            1629,            2084,            1583,            1476,            3137,            1992,            1142,            1646,            2178,            2692,            1689,            843,            1944,            2166,            1619,            1080,            1462,            1926,            1401,            883,            1066,            3348,            0,            847,            3623,            2027,            3338,            2014,            1206,            1312,            1296,            2071,            2010,            2664,            2099,            2101,            940,            3038,            1696,            693,            3346,            2132,            1362,            2067,            961,            3160,            2674,            544,            607,            2689,            693,            3378,            3125,            1928,            1726,            ],        [            1638,            3533,            3363,            3774,            2716,            3044,            1402,            4207,            3440,            4133,            3239,            3223,            3609,            1554,            1844,            1279,            1973,            2667,            2829,            1375,            382,            588,            531,            1115,            2699,            868,            1163,            2609,            964,            427,            2261,            2793,            2164,            2304,            329,            1416,            1638,            591,            1695,            934,            1398,            2016,            1498,            1681,            2820,            1124,            0,            4238,            2090,            3953,            1486,            1821,            597,            782,            1543,            2625,            3279,            2714,            2716,            225,            3653,            2311,            531,            3961,            1104,            334,            2682,            1258,            3775,            1646,            1192,            1222,            3304,            531,            3993,            3740,            2543,            2341,            ],        [            2401,            1237,            2030,            1116,            1805,            2133,            2774,            1237,            850,            2995,            925,            1218,            1845,            3071,            2411,            2991,            2187,            4224,            3675,            3087,            3808,            3589,            3467,            2834,            4256,            4644,            4722,            4559,            4740,            4203,            4408,            4350,            4114,            4505,            3746,            3779,            3804,            4367,            4329,            3552,            3761,            4650,            4132,            3882,            4770,            3246,            3776,            0,            1846,            306,            3004,            3573,            4373,            3597,            3460,            4052,            983,            4271,            4273,            4001,            683,            1771,            3939,            2197,            4880,            4110,            1749,            4207,            2204,            5422,            3790,            3853,            773,            3939,            1704,            1353,            4100,            3898,            ],        [            928,            1264,            1094,            1505,            447,            775,            1301,            1938,            1171,            1864,            970,            954,            1340,            1598,            839,            1419,            615,            2652,            2814,            1515,            2236,            2017,            2092,            1397,            2684,            3269,            3150,            3086,            3365,            2828,            2836,            2778,            2641,            2933,            2371,            2306,            2331,            2992,            2757,            2177,            2288,            3078,            2560,            2310,            3297,            1674,            2401,            1969,            0,            1684,            1531,            2001,            2998,            2025,            1987,            2610,            1010,            2699,            2701,            2626,            1384,            199,            2367,            1692,            3505,            2735,            413,            2635,            1506,            4047,            2218,            2281,            1035,            2367,            1724,            1471,            2528,            2326,            ],        [            2121,            931,            2146,            810,            1499,            1827,            2494,            1282,            1064,            2916,            970,            825,            1539,            2791,            2131,            2711,            1907,            3944,            3652,            2807,            3528,            3309,            3187,            2554,            3976,            4364,            4442,            4279,            4460,            3923,            4128,            4070,            3834,            4225,            3466,            3499,            3524,            4087,            4049,            3272,            3481,            4370,            3852,            3602,            4490,            2966,            3496,            691,            1566,            0,            2724,            3293,            4093,            3317,            3180,            3902,            677,            3991,            3993,            3721,            728,            1491,            3659,            1891,            4600,            3830,            1443,            3927,            2035,            5142,            3510,            3573,            467,            3659,            1617,            1047,            3820,            3618,            ],        [            1091,            2807,            2637,            3048,            1990,            2318,            855,            3481,            2714,            3407,            2513,            2497,            2883,            501,            1858,            2048,            1634,            3378,            3540,            2144,            2238,            2444,            1547,            990,            3410,            2724,            3019,            1555,            2820,            2283,            3562,            3504,            1110,            3659,            1719,            819,            800,            2447,            3483,            1083,            801,            3804,            3286,            3036,            1766,            2303,            1856,            3512,            1543,            3227,            0,            2727,            2453,            1534,            456,            3336,            2553,            3425,            3427,            2081,            2927,            1742,            2273,            3235,            2960,            2190,            1956,            3114,            3049,            3502,            3048,            3389,            2578,            2273,            3267,            3014,            3254,            3052,            ],        [            2075,            2998,            2828,            3239,            2181,            2509,            2108,            3672,            2905,            2421,            2704,            2688,            3074,            2260,            1309,            842,            1438,            846,            1008,            733,            1699,            1480,            1555,            1821,            878,            2732,            2613,            3315,            2828,            2291,            1030,            972,            2870,            1127,            1834,            2122,            2344,            2455,            1150,            1640,            2104,            1471,            953,            703,            3526,            1137,            1864,            3703,            2107,            3418,            2192,            0,            2461,            1488,            2249,            804,            2744,            893,            895,            2089,            3118,            1776,            1830,            3426,            2968,            2198,            2147,            2098,            3240,            3510,            1728,            1056,            2769,            1830,            3458,            3205,            722,            520,            ],        [            2126,            4038,            3868,            4279,            3221,            3549,            1890,            4712,            3945,            4638,            3744,            3728,            4114,            2042,            2341,            1784,            2477,            3114,            3276,            1880,            870,            1076,            1019,            1603,            3146,            271,            1651,            3097,            1452,            170,            3298,            3240,            2652,            2792,            817,            1904,            2126,            1079,            2183,            1422,            1886,            2504,            1986,            2169,            3308,            1612,            691,            4743,            2578,            4458,            1974,            2309,            0,            1270,            2031,            3072,            3784,            3161,            3163,            498,            4158,            2816,            1019,            4466,            1592,            822,            3187,            1746,            4280,            486,            1680,            1710,            3809,            1019,            4498,            4245,            2990,            2788,            ],        [            856,            2572,            2402,            2813,            1755,            2083,            620,            3246,            2479,            3172,            2278,            2262,            2648,            960,            1301,            1031,            1077,            2361,            2523,            1127,            1221,            1427,            530,            333,            2393,            1707,            2002,            2448,            1803,            1266,            2545,            2487,            2003,            2642,            809,            1097,            1319,            1430,            2466,            615,            1079,            2787,            2269,            2019,            2659,            1286,            839,            3277,            1308,            2992,            893,            1710,            1436,            0,            1349,            2319,            2318,            2408,            2410,            1064,            2692,            1507,            1256,            3000,            1943,            1173,            1721,            2097,            2814,            2485,            2031,            2372,            2343,            1256,            3032,            2779,            2237,            2035,            ],        [            1158,            2874,            2704,            3115,            2057,            2385,            922,            3548,            2781,            3474,            2580,            2564,            2950,            568,            1925,            2106,            1701,            3436,            3598,            2202,            2296,            2502,            1605,            1057,            3468,            2782,            3077,            1375,            2878,            2341,            3620,            3562,            930,            3717,            1777,            877,            605,            2505,            3541,            1141,            859,            3862,            3344,            3094,            1586,            2361,            1914,            3579,            1610,            3294,            531,            2785,            2511,            1592,            0,            3394,            2620,            3483,            3485,            2139,            2994,            1809,            2331,            3302,            3018,            2248,            2023,            3172,            3116,            3560,            3106,            3447,            2645,            2331,            3334,            3081,            3312,            3110,            ],        [            2635,            3558,            3388,            3799,            2741,            3069,            2668,            4232,            3465,            1978,            3264,            3248,            3634,            2820,            1869,            1402,            1998,            1017,            565,            1293,            2300,            2081,            2115,            2381,            963,            3292,            3214,            3875,            3388,            2851,            1115,            631,            3430,            1212,            2394,            2682,            2904,            3015,            1531,            2200,            2664,            1852,            1334,            1084,            4086,            1697,            2424,            4263,            2667,            3978,            2752,            775,            3021,            2048,            2809,            0,            3304,            978,            980,            2649,            3678,            2336,            2431,            3986,            3528,            2758,            2707,            2699,            2448,            4070,            2109,            1437,            3329,            2431,            4018,            3765,            301,            549,            ],        [            1569,            642,            1422,            606,            880,            1208,            1942,            1392,            625,            2387,            424,            386,            1385,            2239,            1579,            2159,            1355,            3392,            3061,            2255,            2976,            2757,            2635,            2002,            3424,            3812,            3890,            3727,            3908,            3371,            3576,            3518,            3282,            3673,            2914,            2947,            2972,            3535,            3497,            2720,            2929,            3818,            3300,            3050,            3938,            2414,            2944,            1423,            1014,            703,            2172,            2741,            3541,            2765,            2628,            3350,            0,            3439,            3441,            3169,            838,            939,            3107,            1737,            4048,            3278,            862,            3375,            1596,            4590,            2958,            3021,            263,            3107,            1178,            843,            3268,            3066,            ],        [            3141,            3689,            2727,            3930,            3247,            2693,            3174,            4293,            4042,            2008,            3841,            3825,            3903,            3326,            2375,            1908,            2504,            488,            705,            1799,            2806,            2587,            2621,            2887,            553,            3798,            3720,            4381,            3894,            3357,            705,            1507,            3936,            802,            2900,            3188,            3410,            3521,            2037,            2706,            3170,            2358,            1840,            1590,            4592,            2203,            2930,            4876,            3173,            4352,            3258,            1281,            3527,            2554,            3315,            829,            3814,            0,            377,            3155,            4255,            2842,            2937,            4255,            4034,            3264,            3284,            3205,            2478,            4576,            2615,            1943,            3906,            2937,            3368,            4034,            1257,            1055,            ],        [            2819,            3742,            2841,            3983,            2925,            2807,            2852,            4407,            3649,            2122,            3448,            3432,            4017,            3004,            2053,            1586,            2182,            261,            819,            1477,            2484,            2265,            2299,            2565,            176,            3476,            3398,            4059,            3572,            3035,            328,            1185,            3614,            425,            2578,            2866,            3088,            3199,            1715,            2384,            2848,            2036,            1518,            1268,            4270,            1881,            2608,            4447,            2851,            4162,            2936,            959,            3205,            2232,            2993,            943,            3488,            114,            0,            2833,            3862,            2520,            2615,            4369,            3712,            2942,            2891,            2883,            2592,            4254,            2293,            1621,            3513,            2615,            3482,            4148,            935,            733,            ],        [            1628,            3540,            3370,            3781,            2723,            3051,            1392,            4214,            3447,            4140,            3246,            3230,            3616,            1544,            1843,            1286,            1979,            2616,            2778,            1382,            372,            578,            521,            1105,            2648,            1851,            1153,            2599,            954,            456,            2800,            2742,            2154,            2294,            319,            1406,            1628,            581,            1685,            924,            1388,            2006,            1488,            1671,            2810,            1114,            193,            4245,            2080,            3960,            1476,            1811,            626,            772,            1533,            2574,            3286,            2663,            2665,            0,            3660,            2318,            521,            3968,            1094,            324,            2689,            1248,            3782,            1636,            1182,            1212,            3311,            521,            4000,            3747,            2492,            2290,            ],        [            2103,            1230,            1347,            1471,            2007,            1790,            2476,            706,            552,            2312,            627,            920,            1838,            2773,            2113,            2693,            1889,            4011,            2992,            2789,            3510,            3291,            3169,            2536,            4076,            4346,            4424,            4261,            4442,            3905,            4228,            4262,            3816,            4325,            3448,            3481,            3506,            4069,            4031,            3254,            3463,            4352,            3834,            3584,            4472,            2948,            3478,            933,            1548,            648,            2706,            3275,            4075,            3299,            3162,            3369,            976,            3898,            3900,            3703,            0,            1473,            3641,            2190,            4582,            3812,            2257,            3909,            1521,            5124,            3492,            3555,            1115,            3641,            1021,            1969,            3670,            3810,            ],        [            886,            1222,            1052,            1463,            405,            733,            1259,            1896,            1129,            1822,            928,            912,            1298,            1556,            656,            1236,            432,            2469,            2558,            1332,            2053,            1834,            1952,            1319,            2501,            3129,            2967,            3044,            3225,            2688,            2653,            2595,            2599,            2750,            2231,            2264,            2289,            2852,            2574,            1994,            2246,            2895,            2377,            2127,            3255,            1491,            2261,            1927,            331,            1642,            1489,            1818,            2858,            1842,            1945,            2427,            968,            2516,            2518,            2486,            1342,            0,            2184,            1650,            3365,            2595,            371,            2452,            1464,            3907,            2035,            2098,            993,            2184,            1682,            1429,            2345,            2143,            ],        [            1621,            3533,            3363,            3774,            2716,            3044,            1385,            4207,            3440,            4133,            3239,            3223,            3609,            1537,            1836,            1279,            1972,            2609,            2771,            1375,            404,            610,            514,            1098,            2641,            890,            1185,            2592,            986,            449,            2793,            2735,            2147,            2890,            312,            1399,            1621,            613,            1816,            917,            1381,            2137,            1520,            2267,            2803,            1534,            186,            4238,            2073,            3953,            1469,            1958,            619,            765,            1526,            2567,            3279,            2656,            2658,            247,            3653,            2311,            0,            3961,            1126,            356,            2682,            1280,            3775,            1668,            1214,            1244,            3304,            0,            3993,            3740,            2485,            2283,            ],        [            1902,            872,            1927,            944,            1280,            1608,            2275,            2623,            1856,            2697,            1655,            1617,            352,            2572,            1912,            2492,            1688,            3725,            3433,            2588,            3309,            3090,            2968,            2335,            3757,            4145,            4223,            4060,            4241,            3704,            3909,            3851,            3615,            4006,            3247,            3280,            3305,            3868,            3830,            3053,            3262,            4151,            3633,            3383,            4271,            2747,            3277,            2654,            1347,            1934,            2505,            3074,            3874,            3098,            2961,            3683,            1231,            3772,            3774,            3502,            2069,            1272,            3440,            0,            4381,            3611,            1224,            3708,            2339,            4923,            3291,            3354,            1494,            3440,            2409,            647,            3601,            3399,            ],        [            2631,            4543,            4373,            4784,            3726,            4054,            2395,            5217,            4450,            5143,            4249,            4233,            4619,            2547,            2846,            2289,            2982,            3619,            3781,            2385,            1375,            1581,            1524,            2108,            3651,            1067,            796,            3602,            290,            1459,            3803,            3745,            3157,            3297,            1322,            2409,            2631,            713,            2436,            1927,            2391,            2211,            2491,            2674,            3813,            2117,            1196,            5248,            3083,            4963,            2479,            2814,            1338,            1775,            2536,            3577,            4289,            3666,            3668,            1257,            4663,            3321,            1524,            4971,            0,            1000,            3692,            1395,            4785,            852,            1834,            2215,            4314,            1524,            5003,            4750,            3495,            3293,            ],        [            1932,            3844,            3674,            4085,            3027,            3355,            1696,            4518,            3751,            4444,            3550,            3534,            3920,            1848,            2147,            1590,            2283,            2920,            3082,            1686,            676,            882,            825,            1409,            2952,            1527,            829,            2903,            630,            760,            3104,            3046,            2458,            2598,            623,            1710,            1932,            257,            1737,            1228,            1692,            2058,            1792,            1975,            3114,            1418,            497,            4549,            2384,            4264,            1780,            2115,            930,            1076,            1837,            2878,            3590,            2967,            2969,            558,            3964,            2622,            825,            4272,            770,            0,            2993,            1428,            4086,            1312,            1135,            1516,            3615,            825,            4304,            4051,            2796,            2594,            ],        [            1210,            859,            1148,            1100,            501,            829,            1583,            1533,            766,            1918,            565,            549,            1026,            1880,            1220,            1800,            996,            3033,            2654,            1896,            2617,            2398,            2276,            1643,            3065,            3453,            3531,            3368,            3549,            3012,            3217,            3159,            2923,            3314,            2555,            2588,            2613,            3176,            3138,            2361,            2570,            3459,            2941,            2691,            3579,            2055,            2585,            1564,            655,            1279,            1813,            2382,            3182,            2406,            2269,            2991,            605,            3080,            3082,            2810,            979,            580,            2748,            1378,            3689,            2919,            0,            3016,            1737,            4231,            2599,            2662,            630,            2748,            1319,            1157,            2909,            2707,            ],        [            2628,            3504,            3334,            3745,            2687,            3015,            2392,            4178,            3411,            4104,            3210,            3194,            3580,            2544,            1815,            1250,            1944,            2580,            2742,            1346,            1372,            947,            1521,            2105,            2612,            2524,            1129,            3599,            1627,            1456,            2232,            2706,            3154,            2275,            1319,            2406,            2628,            1254,            1089,            1924,            2388,            1410,            1166,            1652,            3810,            1095,            1193,            4209,            2613,            3924,            2476,            1792,            1626,            1772,            2533,            2538,            3250,            2627,            2629,            1254,            3624,            2282,            1297,            3932,            1767,            997,            2653,            0,            3746,            2309,            487,            1193,            3275,            1297,            3964,            3711,            2456,            2254,            ],        [            2227,            1856,            894,            2097,            1201,            860,            2600,            2030,            2209,            791,            2008,            1992,            2070,            2897,            2237,            2817,            2013,            2546,            1527,            2913,            3634,            3415,            3293,            2660,            2611,            4470,            4548,            4385,            4566,            4029,            2763,            1331,            3940,            2860,            3572,            3605,            3630,            4193,            3327,            3378,            3587,            3648,            3130,            2880,            4596,            3072,            3602,            3007,            1672,            2125,            2830,            2571,            4199,            3423,            3286,            1904,            1981,            2433,            2435,            3827,            1661,            1597,            3765,            2422,            4706,            3936,            1451,            4033,            0,            5248,            3616,            3233,            2073,            3765,            1480,            2201,            2205,            2345,            ],        [            3215,            5127,            4957,            5368,            4310,            4638,            2979,            5801,            5034,            5727,            4833,            4817,            5203,            3131,            3430,            2873,            3566,            4203,            4365,            2969,            1959,            2165,            2108,            2692,            4235,            215,            1520,            4186,            1321,            656,            4387,            4329,            3741,            3881,            1906,            2993,            3215,            1297,            3020,            2511,            2975,            2935,            3075,            3258,            4397,            2701,            1780,            5832,            3667,            5547,            3063,            3398,            486,            2359,            3413,            4161,            4873,            4250,            4252,            984,            5247,            3905,            2234,            5555,            1461,            1584,            4276,            2119,            5369,            0,            2418,            2799,            4898,            2234,            5587,            5334,            4079,            3877,            ],        [            2265,            3017,            2847,            3258,            2200,            2528,            2029,            3691,            2924,            3617,            2723,            2707,            3093,            2181,            1328,            763,            1457,            2093,            2255,            859,            679,            460,            1476,            1742,            2125,            1700,            1593,            3236,            1796,            1259,            2277,            2219,            2791,            1788,            960,            2043,            2265,            1423,            1179,            1561,            2025,            1500,            982,            1165,            3447,            608,            964,            3722,            2126,            3437,            2113,            1305,            1429,            1409,            2463,            2051,            2763,            2140,            2142,            1057,            3137,            1795,            810,            3445,            1936,            1166,            2166,            1078,            3259,            2478,            0,            706,            2788,            810,            3477,            3224,            1969,            1767,            ],        [            2449,            3201,            3031,            3442,            2384,            2712,            2213,            3875,            3108,            3801,            2907,            2891,            3277,            2365,            1512,            947,            1641,            1658,            1820,            1043,            863,            644,            1660,            1926,            1690,            1884,            1777,            3420,            1980,            1443,            1842,            1784,            2975,            1295,            1144,            2227,            2449,            1607,            686,            1745,            2209,            1007,            489,            672,            3631,            792,            1148,            3906,            2310,            3621,            2297,            812,            1613,            1593,            2647,            1616,            2947,            1705,            1707,            1241,            3321,            1979,            994,            3629,            2120,            1350,            2350,            1262,            3443,            2662,            699,            0,            2972,            994,            3661,            3408,            1534,            1332,            ],        [            1654,            464,            1679,            705,            1032,            1360,            2027,            1364,            597,            2449,            396,            358,            1072,            2324,            1664,            2244,            1440,            3477,            3185,            2340,            3061,            2842,            2720,            2087,            3509,            3897,            3975,            3812,            3993,            3456,            3661,            3603,            3367,            3758,            2999,            3032,            3057,            3620,            3582,            2805,            3014,            3903,            3385,            3135,            4023,            2499,            3029,            1395,            1099,            537,            2257,            2826,            3626,            2850,            2713,            3435,            210,            3524,            3526,            3254,            810,            1024,            3192,            1424,            4133,            3363,            976,            3460,            1568,            4675,            3043,            3488,            0,            3192,            1150,            1203,            3353,            3151,            ],        [            1621,            3533,            3363,            3774,            2716,            3044,            1385,            4207,            3440,            4133,            3239,            3223,            3609,            1537,            1836,            1279,            1972,            2609,            2771,            1375,            404,            610,            514,            1098,            2641,            890,            1185,            2592,            986,            449,            2793,            2735,            2147,            2890,            312,            1399,            1621,            613,            1816,            917,            1381,            2137,            1520,            2267,            2803,            1534,            186,            4238,            2073,            3953,            1469,            1958,            619,            765,            1819,            2567,            3279,            2656,            2658,            247,            3653,            2311,            0,            3961,            1126,            356,            2682,            1280,            3775,            1668,            1214,            1244,            3304,            0,            3993,            3740,            2485,            2283,            ],        [            2012,            1454,            326,            1695,            986,            769,            2385,            1580,            1361,            1291,            1160,            1144,            1855,            2682,            2022,            2602,            1798,            2984,            1965,            2698,            3419,            3200,            3078,            2445,            3049,            4255,            4333,            4170,            4351,            3814,            3201,            1831,            3725,            3298,            3357,            3390,            3415,            3978,            3940,            3163,            3372,            4261,            3743,            3493,            4381,            2857,            3387,            1960,            1457,            1675,            2615,            3184,            3984,            3208,            3071,            2342,            1200,            2871,            2873,            3612,            1211,            1382,            3550,            2207,            4491,            3721,            1236,            3818,            500,            5033,            3401,            3846,            1225,            3550,            0,            1986,            2643,            2783,            ],        [            1681,            538,            1706,            297,            1059,            1387,            2054,            2402,            1635,            2476,            1434,            1396,            295,            2351,            1691,            2271,            1467,            3504,            3212,            2367,            3088,            2869,            2747,            2114,            3536,            3924,            4002,            3839,            4020,            3483,            3688,            3630,            3394,            3785,            3026,            3059,            3084,            3647,            3609,            2832,            3041,            3930,            3412,            3162,            4050,            2526,            3056,            1240,            1126,            904,            2284,            2853,            3653,            2877,            2740,            3462,            1010,            3551,            3553,            3281,            1848,            1051,            3219,            647,            4160,            3390,            1003,            3487,            2118,            4702,            3070,            3515,            646,            3219,            2188,            0,            3380,            3178,            ],        [            2625,            3279,            3109,            3520,            2462,            2790,            2389,            3953,            3186,            2279,            2985,            2969,            3355,            2541,            1590,            1123,            1719,            1318,            866,            1014,            2021,            1802,            1836,            2102,            1179,            3013,            2935,            3596,            3109,            2572,            1331,            330,            3151,            1428,            2115,            2403,            2625,            2736,            1252,            1921,            2385,            1573,            1055,            805,            3807,            1418,            2145,            3984,            2388,            3699,            2473,            496,            2742,            1769,            2823,            301,            3025,            1194,            1196,            2370,            3399,            2057,            2152,            3707,            3249,            2479,            2428,            2420,            3521,            3791,            1830,            1158,            3050,            2152,            3739,            3486,            0,            202,            ],        [            2423,            3077,            2907,            3318,            2260,            2588,            2187,            3751,            2984,            2520,            2783,            2767,            3153,            2339,            1388,            921,            1517,            945,            1107,            812,            1819,            1600,            1634,            1900,            977,            2811,            2733,            3394,            2907,            2370,            1129,            452,            2949,            1226,            1913,            2201,            2423,            2534,            1050,            1719,            2183,            1371,            853,            603,            3605,            1216,            1943,            3782,            2186,            3497,            2271,            294,            2540,            1567,            2621,            503,            2823,            992,            994,            2168,            3197,            1855,            1950,            3505,            3047,            2277,            2226,            2218,            3319,            3589,            1628,            956,            2848,            1950,            3537,            3284,            202,            0,            ],        ], 'starts': [51, 63], 'ends': [51, 63]},    'NW-15': {'distance_matrix': [        [            0,            6426,            2559,            5149,            594,            4745,            4998,            5429,            6719,            3287,            7942,            957,            4489,            7114,            7081,            5248,            5145,            6039,            4273,            2276,            8602,            691,            3783,            4823,            5184,            4119,            4914,            6461,            6382,            4637,            4683,            431,            7993,            3962,            5306,            4324,            8522,            4405,            4480,            4809,            1934,            8247,            491,            6016,            1502,            5437,            231,            1870,            6546,            934,            5795,            4918,            4642,            4093,            6903,            4341,            ],        [            7026,            0,            7361,            4708,            8281,            3640,            4260,            4324,            293,            6906,            2303,            6280,            5255,            2059,            2773,            4143,            2056,            640,            3190,            7320,            4619,            6538,            4742,            4382,            5966,            4679,            1825,            35,            3847,            5285,            1901,            8466,            3121,            4675,            3072,            5195,            2556,            4569,            5246,            5597,            7621,            4264,            6880,            3194,            6825,            1641,            8679,            7846,            2647,            8189,            959,            3813,            4201,            4257,            5342,            2411,            ],        [            2559,            5288,            0,            2590,            3153,            3930,            2448,            4614,            5581,            728,            5392,            1813,            1930,            4564,            6207,            4433,            3743,            4637,            2871,            469,            5013,            2071,            1224,            2264,            2625,            1560,            3512,            5323,            3832,            2078,            3281,            2990,            7178,            1403,            4491,            2012,            5972,            4565,            1921,            2250,            770,            4658,            2413,            3466,            1365,            4035,            2790,            995,            3996,            3493,            4393,            4103,            2083,            4253,            3314,            2939,            ],        [            4428,            3620,            2379,            0,            6288,            3211,            2474,            3895,            3913,            2100,            4069,            3682,            1724,            3372,            4539,            3714,            2075,            2969,            1203,            2338,            5784,            3940,            1732,            1411,            2419,            1726,            1844,            3655,            3858,            1225,            1613,            6473,            4887,            1722,            2492,            1159,            4359,            1587,            1715,            2070,            2639,            5429,            4282,            3260,            3234,            2367,            6686,            2864,            3960,            6196,            2725,            3384,            1230,            3257,            4085,            1271,            ],        [            1127,            7652,            3049,            5639,            0,            5971,            6224,            6655,            7945,            3777,            9168,            1447,            4979,            8340,            9638,            6474,            6371,            7265,            5499,            2766,            9828,            1181,            4273,            5313,            5674,            4609,            6140,            7687,            7608,            5127,            5909,            203,            9219,            4452,            6532,            4814,            9748,            5631,            4970,            5299,            2424,            9473,            981,            7242,            1992,            6663,            416,            2360,            7772,            340,            7021,            6144,            5132,            5319,            8129,            5567,            ],        [            4675,            5190,            4424,            3719,            5930,            0,            3672,            684,            4887,            3969,            4025,            3929,            3688,            4370,            3000,            503,            3073,            4798,            2201,            4383,            7863,            4187,            4465,            3393,            4383,            4801,            2842,            5225,            5056,            3189,            2611,            6115,            3328,            4644,            917,            3123,            5357,            3666,            3679,            4034,            4684,            7508,            4529,            5505,            5279,            3365,            6328,            4909,            4958,            5838,            4462,            173,            3212,            3354,            6164,            2269,            ],        [            6062,            4910,            4013,            3603,            6656,            4765,            0,            5449,            5203,            3138,            4169,            5316,            2035,            3341,            4639,            5268,            3828,            5550,            2757,            3972,            2781,            5574,            2899,            3277,            2677,            2838,            3597,            4945,            2609,            2698,            3366,            6493,            4987,            2681,            4046,            2608,            4749,            2955,            2026,            1736,            4273,            2426,            5916,            2243,            4868,            4409,            6293,            4498,            2773,            6996,            5186,            4938,            3096,            6319,            1082,            3024,            ],        [            5289,            4580,            5038,            4333,            6544,            943,            4286,            0,            4277,            4583,            3415,            4543,            4302,            4984,            3640,            385,            3687,            4188,            2815,            4997,            7570,            4801,            5079,            4007,            4997,            5415,            3456,            4615,            5670,            3803,            3225,            6729,            3968,            5258,            1531,            3737,            5971,            3373,            4293,            4648,            5298,            7215,            5143,            6119,            5893,            3979,            6942,            5523,            5572,            6452,            3852,            667,            3826,            3968,            5871,            2883,            ],        [            6733,            930,            7068,            4415,            7988,            3347,            4554,            4031,            0,            6613,            2010,            5987,            5549,            2353,            2480,            3850,            1763,            347,            2897,            7027,            4913,            6245,            4449,            4089,            6260,            4386,            1532,            965,            4141,            3885,            1608,            8173,            2828,            4382,            2779,            5489,            3340,            4276,            5540,            5891,            7328,            4558,            6587,            3488,            6532,            1348,            8386,            7553,            2941,            7896,            666,            3520,            3908,            3964,            6774,            2118,            ],        [            2924,            4963,            875,            1582,            3518,            3499,            2123,            4183,            5256,            0,            5412,            2178,            1605,            4239,            6419,            4002,            3418,            4312,            2546,            834,            4688,            2436,            899,            1939,            2300,            1235,            3187,            4998,            3507,            1753,            2956,            3355,            6747,            1078,            4060,            1687,            5647,            4523,            1596,            1925,            1135,            4333,            2778,            3141,            1730,            3710,            3155,            1360,            3671,            3858,            4068,            3672,            1758,            4211,            2989,            2614,            ],        [            5279,            2287,            5614,            2961,            6534,            1893,            3415,            2577,            1848,            5159,            0,            4533,            4404,            3081,            1004,            2396,            2491,            1730,            1443,            5573,            7019,            4791,            2995,            2635,            3625,            2932,            2260,            2322,            4869,            2431,            2336,            6719,            1352,            2928,            1325,            2365,            4068,            2822,            4395,            5921,            5874,            6664,            5133,            4216,            5078,            2076,            6932,            6099,            3669,            6442,            1394,            2066,            2454,            2510,            5320,            2846,            ],        [            1114,            6840,            1602,            4192,            1708,            5532,            4050,            6216,            6570,            2330,            7339,            0,            3532,            6166,            7809,            6035,            5295,            6189,            4473,            1319,            6615,            626,            2826,            3866,            4227,            3162,            5064,            6875,            5434,            3680,            4833,            1545,            8780,            3005,            6093,            3614,            7574,            4243,            3523,            3852,            977,            6260,            968,            5068,            545,            5587,            1345,            913,            5598,            2048,            5945,            5705,            3685,            3931,            4916,            4541,            ],        [            4870,            4168,            2821,            1568,            5464,            5445,            797,            6129,            3898,            1946,            3741,            4124,            0,            2913,            4211,            5948,            2623,            3517,            1751,            2780,            2953,            4382,            1578,            1242,            1003,            731,            2392,            4203,            2181,            663,            2161,            5301,            4559,            727,            3040,            573,            4321,            2449,            299,            654,            3081,            2598,            4724,            1815,            3676,            2915,            5101,            3306,            2345,            5804,            3273,            5618,            1061,            4284,            1886,            1819,            ],        [            7996,            1569,            5947,            4676,            9160,            3828,            2650,            4512,            1862,            5072,            3189,            7250,            3645,            0,            3659,            4331,            1934,            2089,            3142,            5906,            3009,            7508,            4704,            4350,            4356,            4248,            1703,            1604,            2237,            3675,            1779,            9345,            4007,            4244,            3260,            3585,            1408,            4273,            3636,            4386,            6207,            2654,            7850,            1584,            6802,            1068,            9558,            6432,            1037,            9068,            1845,            4001,            4169,            5143,            3732,            2289,            ],        [            6193,            2617,            6528,            3981,            7448,            2807,            4329,            3491,            2314,            6073,            1470,            5447,            5318,            3624,            0,            3310,            3034,            2273,            2300,            6487,            6184,            5705,            4015,            3655,            6029,            3952,            2803,            2652,            5412,            3451,            2879,            7633,            871,            3948,            2126,            3385,            4611,            3736,            5309,            6835,            6788,            5829,            6047,            4759,            5992,            2619,            7846,            7013,            4212,            7356,            1937,            2980,            3474,            3424,            6234,            3389,            ],        [            5286,            4577,            5035,            4493,            6541,            940,            4870,            181,            4274,            4580,            3412,            4540,            4462,            4981,            3637,            0,            3684,            4185,            2812,            4994,            8474,            4798,            5076,            4167,            5157,            5412,            3453,            4612,            6254,            3963,            3222,            6726,            3965,            5255,            1528,            3897,            5968,            4277,            4453,            4808,            5295,            8119,            5140,            6116,            5890,            3976,            6939,            5520,            5569,            6449,            3849,            664,            3986,            3965,            5868,            2880,            ],        [            6717,            1545,            7052,            5524,            7972,            3331,            3498,            4015,            1275,            5920,            1994,            5971,            4493,            1297,            2464,            3834,            0,            894,            1947,            7011,            3857,            6229,            5552,            5198,            5204,            5096,            508,            1580,            3085,            4523,            584,            8157,            2812,            5092,            2763,            4433,            2284,            4260,            4484,            5234,            7312,            3502,            6571,            2432,            6516,            292,            8370,            7537,            1885,            7880,            650,            3504,            5017,            3948,            4580,            1094,            ],        [            6386,            1212,            6721,            3967,            7641,            3000,            4207,            3684,            1036,            6266,            1663,            5640,            5202,            2006,            2133,            3503,            1416,            0,            2493,            6680,            4566,            5898,            4208,            3848,            5913,            4145,            1185,            1247,            3794,            5232,            1261,            7826,            2481,            4141,            2432,            5142,            2993,            3929,            5193,            7028,            6981,            4211,            6240,            3141,            6185,            1001,            8039,            7206,            2594,            7549,            319,            3173,            3667,            3617,            5289,            1771,            ],        [            4912,            2557,            3909,            2632,            6167,            1637,            2422,            2321,            2287,            3034,            1495,            4166,            2601,            2309,            1965,            2140,            1012,            1906,            0,            3868,            4869,            4424,            2666,            2306,            3296,            2603,            781,            2592,            4097,            2102,            550,            6352,            2313,            2599,            1069,            2036,            3296,            1509,            2592,            2947,            4169,            4514,            4766,            3444,            4764,            1304,            6565,            4394,            2897,            6075,            1662,            1810,            2125,            3449,            4007,            208,            ],        [            2090,            5713,            425,            3015,            2684,            4355,            2873,            5039,            5443,            1153,            5817,            1344,            2355,            4989,            6287,            4858,            4168,            5062,            3296,            0,            5438,            1602,            1649,            2689,            3050,            1985,            3937,            5748,            4257,            2503,            3706,            2521,            6635,            1828,            4916,            2437,            6397,            4990,            2346,            2675,            301,            5083,            1944,            3891,            896,            4460,            2321,            526,            4421,            3024,            4818,            4528,            2508,            4678,            3739,            3364,            ],        [            7554,            4439,            5376,            4966,            8148,            8129,            2916,            8684,            4732,            4501,            6059,            6679,            3398,            2870,            6529,            8503,            4804,            4959,            4360,            5335,            0,            7066,            4262,            4640,            4040,            4201,            4573,            4474,            1002,            4061,            4649,            9657,            6877,            4044,            5692,            3971,            4278,            4646,            3389,            3099,            5636,            585,            7408,            1772,            6231,            3938,            7785,            5861,            2250,            9380,            4715,            8173,            4459,            8013,            2327,            5159,            ],        [            488,            6789,            2227,            4817,            1082,            5108,            5361,            5792,            6519,            2955,            8305,            625,            4157,            7477,            8065,            5611,            5244,            6138,            4372,            1944,            8965,            0,            3451,            4491,            4852,            3787,            5013,            6824,            8771,            4305,            4782,            919,            8356,            3630,            5669,            4239,            7528,            4768,            4148,            4477,            1602,            8610,            342,            5693,            1170,            5536,            719,            1538,            6223,            1422,            5894,            5281,            4310,            4456,            7266,            4440,            ],        [            3292,            4498,            1208,            1800,            3886,            3867,            1708,            4551,            4228,            368,            4652,            2546,            1190,            3824,            5122,            4370,            2953,            3847,            2081,            1202,            4673,            2804,            0,            1474,            1885,            820,            2722,            4533,            3092,            1288,            2491,            6457,            5470,            663,            4428,            1222,            5232,            4502,            1181,            1910,            1503,            4318,            3146,            2726,            2098,            3245,            3523,            1728,            3256,            6180,            3603,            4040,            1293,            4190,            2974,            2149,            ],        [            4213,            3946,            2129,            326,            6587,            4182,            2027,            4866,            3676,            1885,            4395,            3467,            1509,            4143,            4865,            4685,            2401,            3295,            1529,            2123,            4183,            3725,            1517,            0,            2204,            1511,            2170,            3981,            3411,            1010,            1939,            6772,            5213,            1507,            2818,            944,            4685,            1913,            1500,            1855,            2424,            3828,            4067,            3045,            3019,            2693,            6985,            2649,            3575,            6495,            3051,            4355,            1015,            3042,            4411,            1597,            ],        [            4896,            5546,            2812,            2632,            5490,            5471,            1861,            6155,            4962,            1972,            4805,            4150,            1064,            3977,            5275,            5974,            3687,            4581,            2815,            2806,            3530,            4408,            1733,            2306,            0,            1632,            3456,            5581,            3336,            1727,            3225,            5327,            5623,            1475,            4104,            1637,            5385,            3513,            1055,            767,            3107,            3175,            4750,            2879,            3702,            3979,            5127,            3332,            3409,            7784,            4337,            5644,            2125,            5492,            1831,            2883,            ],        [            4961,            4573,            2877,            1659,            5555,            5536,            888,            6220,            3989,            2037,            3832,            4215,            370,            3004,            4302,            6039,            2714,            3608,            1842,            2871,            3044,            4473,            1669,            1333,            1065,            0,            2483,            4608,            2272,            754,            2252,            5392,            4650,            818,            3131,            664,            4412,            2540,            361,            716,            3172,            2689,            4815,            1906,            3767,            3006,            5192,            3397,            2436,            7849,            3364,            5709,            1152,            4375,            1977,            1910,            ],        [            6948,            1776,            7283,            5755,            8203,            3562,            3729,            4246,            1506,            6151,            2225,            6202,            4724,            1528,            2695,            4065,            231,            1125,            2178,            7242,            4088,            6460,            5783,            5429,            5435,            5327,            0,            1811,            3316,            4754,            815,            8388,            3043,            5323,            2994,            4664,            2515,            4491,            4715,            5465,            7543,            3733,            6802,            2663,            6747,            523,            8601,            7768,            2116,            8111,            881,            3735,            5248,            4179,            4811,            1325,            ],        [            6991,            561,            7326,            4572,            8246,            3605,            4225,            4289,            258,            6871,            2268,            6245,            5220,            2024,            2738,            4108,            2021,            605,            3098,            7285,            4584,            6503,            4813,            4453,            5931,            5823,            1790,            0,            3812,            5250,            1866,            8431,            3086,            5819,            3037,            5160,            2521,            4534,            5211,            5961,            7586,            4229,            6845,            3159,            6790,            1606,            8644,            7811,            2612,            8154,            924,            3778,            4272,            4222,            5307,            2376,            ],        [            7015,            3437,            5098,            3862,            8270,            6110,            1836,            6794,            3730,            4258,            5057,            6269,            2831,            1868,            5527,            6199,            3802,            3957,            3158,            5092,            772,            6527,            3890,            3536,            3542,            3434,            3571,            3472,            0,            2861,            3647,            8455,            5875,            3430,            4490,            2771,            3276,            3459,            2822,            2908,            5393,            417,            6869,            770,            5988,            2936,            8668,            5618,            1248,            8178,            3713,            5869,            3355,            7011,            2136,            4157,            ],        [            4462,            4702,            2378,            1788,            5056,            5037,            1017,            5721,            4995,            1538,            3961,            3716,            499,            3133,            4431,            5540,            3317,            4211,            1971,            2372,            3173,            3974,            1170,            1462,            1194,            922,            3086,            4737,            2401,            0,            2855,            4893,            4779,            918,            3260,            793,            4541,            2498,            490,            845,            2673,            2818,            4316,            2035,            3268,            4201,            4693,            2898,            2565,            7350,            3967,            5210,            1281,            5058,            2106,            2513,            ],        [            7179,            2007,            7514,            3377,            8434,            2382,            3960,            3066,            1737,            3779,            2456,            6433,            4955,            1759,            2926,            2885,            462,            1356,            1696,            7473,            4319,            6691,            3411,            3051,            5666,            3348,            231,            2042,            3547,            4985,            0,            8619,            3274,            3344,            1814,            4895,            2746,            4722,            4946,            5696,            7774,            3964,            7033,            2894,            6978,            754,            8832,            7999,            2347,            8342,            1112,            2555,            2870,            4410,            5042,            843,            ],        [            924,            7225,            2846,            5436,            596,            5544,            5294,            6228,            6955,            3574,            8031,            1244,            4776,            7410,            8501,            6047,            5680,            6574,            4808,            2563,            9401,            978,            4070,            5110,            5471,            4406,            5449,            7260,            6678,            4924,            5218,            0,            8792,            4249,            6105,            4858,            7964,            5204,            4767,            5096,            2221,            9046,            778,            6312,            1789,            5972,            213,            2157,            6842,            936,            6330,            5717,            4929,            4892,            7702,            4876,            ],        [            5358,            2100,            5658,            3146,            6613,            1972,            3494,            2656,            1661,            5238,            635,            4612,            6299,            2894,            817,            2475,            2304,            1543,            1465,            5652,            5454,            4870,            3180,            2820,            3810,            3117,            2073,            2135,            4682,            2616,            2149,            6798,            0,            3113,            1404,            2550,            3881,            2901,            3106,            6000,            5589,            5099,            5212,            4029,            5157,            1889,            7011,            5525,            3482,            6521,            1207,            2145,            2639,            2589,            5399,            2659,            ],        [            3544,            4316,            1460,            1618,            4138,            4119,            2123,            4803,            4046,            620,            5067,            2798,            1605,            4239,            5537,            4622,            2771,            3665,            1899,            1454,            4482,            3056,            252,            1292,            2300,            667,            2540,            4351,            3507,            1106,            2309,            3975,            5885,            0,            4680,            1040,            5647,            2606,            1596,            1719,            1755,            4127,            3398,            3141,            2350,            3063,            3775,            1980,            3671,            4478,            3421,            4292,            1111,            4442,            2783,            1967,            ],        [            4473,            3074,            4749,            2990,            5728,            719,            2475,            1403,            2771,            3392,            1909,            3727,            2959,            3478,            2379,            1222,            2181,            2682,            1309,            4743,            6064,            3985,            3024,            2664,            3654,            2961,            1950,            3109,            3859,            2460,            1719,            5913,            2727,            2957,            0,            2394,            4465,            1867,            2950,            3305,            5044,            5709,            4327,            4613,            5639,            2473,            6126,            5269,            4066,            5636,            2346,            892,            2483,            2485,            4365,            1377,            ],        [            4316,            3586,            2232,            995,            4910,            4891,            1482,            5575,            3316,            1392,            4035,            3570,            964,            3598,            4505,            5394,            2041,            2935,            1169,            2226,            3638,            3828,            1024,            669,            1659,            966,            1810,            3621,            2866,            465,            1579,            4747,            4853,            962,            2458,            0,            4325,            1876,            955,            1310,            2527,            3283,            4170,            2500,            3122,            2333,            4547,            2752,            3030,            5250,            2691,            5064,            488,            3711,            2600,            1237,            ],        [            7187,            161,            7522,            6285,            8442,            3801,            4259,            4485,            454,            7067,            2464,            6441,            5254,            2058,            2934,            4304,            2217,            801,            3294,            7481,            4618,            6699,            6313,            5959,            5965,            5857,            1986,            196,            3846,            5284,            2062,            8627,            3394,            5853,            3233,            5194,            0,            4730,            5245,            5995,            7782,            4263,            7041,            3193,            8377,            1802,            8840,            8007,            2646,            8350,            1120,            3974,            5778,            4418,            5341,            2572,            ],        [            5368,            3955,            3376,            2139,            6623,            3546,            593,            4230,            3685,            2536,            3537,            4622,            3398,            2709,            4007,            4049,            2410,            3304,            1538,            3370,            4197,            4880,            2168,            1813,            4040,            2110,            2179,            3990,            1977,            1609,            1948,            6808,            4355,            2106,            2827,            1543,            4117,            0,            3389,            3099,            3671,            3842,            5222,            1611,            4266,            2702,            7021,            3896,            2141,            6531,            3060,            3719,            1632,            4855,            2498,            1606,            ],        [            4879,            4677,            2795,            1577,            5473,            5454,            806,            6138,            4407,            1955,            3750,            4133,            9,            2922,            4220,            5957,            3132,            4026,            2260,            2789,            2962,            4391,            1587,            1251,            1012,            740,            2901,            4712,            2190,            672,            2670,            5310,            4568,            736,            3549,            582,            4330,            2458,            0,            663,            3090,            2607,            4733,            1824,            3685,            3424,            5110,            3315,            2354,            5813,            3782,            5627,            1070,            4293,            2099,            2328,            ],        [            5169,            4781,            3085,            1867,            5763,            5744,            1096,            6428,            4697,            2245,            4040,            4423,            299,            3212,            4510,            6247,            3422,            4316,            2550,            3079,            3252,            4681,            1877,            1541,            941,            1030,            3191,            4816,            2480,            962,            2960,            5600,            4858,            1026,            3839,            872,            4620,            2748,            290,            0,            3380,            2897,            5023,            2114,            3975,            3714,            5400,            3605,            2644,            6103,            4072,            5917,            1360,            4583,            1994,            2618,            ],        [            2008,            6422,            625,            3215,            2602,            4555,            3073,            5239,            6152,            1353,            7042,            1262,            2555,            5189,            7512,            5058,            4877,            5771,            4005,            342,            5638,            1520,            1849,            2889,            3250,            2185,            4646,            6457,            4457,            2703,            4415,            2439,            7803,            2028,            5116,            2637,            6597,            5190,            2546,            2875,            0,            5283,            1862,            4091,            814,            5169,            2239,            444,            4621,            2942,            5527,            4728,            2708,            4878,            3939,            4073,            ],        [            7215,            3854,            5002,            4279,            8687,            6113,            2253,            6797,            4147,            4162,            5474,            6686,            3059,            2285,            5944,            6616,            4219,            4374,            3575,            4996,            355,            6727,            3923,            3953,            3701,            3862,            3988,            3889,            417,            3278,            4064,            8872,            6292,            3705,            4907,            3188,            3693,            3876,            3050,            2760,            5297,            0,            7069,            1187,            5892,            3353,            9085,            5522,            1665,            8595,            4130,            6286,            3772,            7428,            1988,            4574,            ],        [            146,            6572,            2705,            5295,            740,            4891,            5144,            5575,            6302,            3433,            7378,            1103,            4635,            7260,            7848,            5394,            5027,            5921,            4155,            2422,            8748,            837,            3929,            4969,            5330,            4265,            4796,            6607,            8554,            4783,            4565,            577,            8139,            4108,            5452,            4470,            7311,            4551,            4626,            4955,            2080,            8393,            0,            6162,            1648,            5319,            377,            2016,            6692,            1080,            5677,            5064,            4788,            4239,            7049,            4223,            ],        [            6245,            3289,            4328,            3092,            7500,            4021,            1066,            4705,            3582,            3488,            3888,            5499,            2061,            1720,            4358,            4524,            3230,            3809,            2388,            4322,            1760,            5757,            3120,            2766,            2772,            2664,            2999,            3324,            988,            2091,            2768,            7685,            4706,            2660,            3720,            2001,            3128,            2689,            2052,            2802,            4623,            1405,            6099,            0,            5218,            2788,            7898,            4848,            1152,            7408,            3565,            4194,            2585,            5842,            2148,            2426,            ],        [            1194,            7151,            1354,            3944,            1788,            5284,            3802,            5968,            6881,            2082,            7771,            448,            3284,            5918,            8000,            5787,            5606,            6500,            4734,            1071,            6367,            706,            2578,            3618,            3979,            2914,            5375,            7186,            5186,            3432,            5144,            1625,            8532,            2757,            5845,            3366,            7326,            4323,            3275,            3604,            729,            6012,            1048,            4820,            0,            5898,            1425,            665,            5350,            2128,            6256,            5457,            3437,            4011,            4668,            4802,            ],        [            7064,            1253,            7399,            5232,            8319,            3139,            3206,            3823,            1546,            5628,            2341,            6318,            4201,            1005,            2811,            3642,            1245,            1241,            2453,            6462,            3565,            6576,            5260,            4906,            4912,            4804,            1014,            1288,            2793,            4231,            1090,            8504,            3159,            4800,            2571,            4141,            1992,            4607,            4192,            4942,            7659,            3210,            6918,            2140,            6863,            0,            8717,            7884,            1593,            8227,            997,            3312,            4725,            4295,            4288,            1600,            ],        [            711,            8380,            2633,            5223,            688,            5331,            5584,            6015,            8110,            3361,            7818,            1031,            4563,            7700,            8047,            5834,            7278,            7729,            6406,            2350,            9188,            765,            3857,            4897,            5258,            4193,            7047,            8415,            8994,            4711,            6816,            525,            8579,            4036,            5892,            4645,            7751,            4991,            4554,            4883,            2008,            8833,            565,            6602,            1576,            5759,            0,            1944,            7132,            1028,            8051,            5504,            4716,            4679,            7489,            4663,            ],        [            1564,            6815,            1018,            3608,            2158,            4948,            3466,            5632,            6545,            1746,            7194,            818,            2948,            5582,            7664,            5451,            5270,            6164,            4398,            735,            6031,            1076,            2242,            3282,            3643,            2578,            5039,            6850,            4850,            3096,            4808,            1995,            8196,            2421,            5509,            3030,            7554,            4498,            2939,            3268,            393,            5676,            1418,            4484,            370,            5562,            1795,            0,            5014,            2498,            5920,            5121,            3101,            4186,            4332,            4466,            ],        [            6792,            2665,            4875,            3639,            8047,            4924,            1613,            5608,            2958,            4035,            4285,            6046,            2608,            1096,            4755,            5427,            3030,            3185,            3038,            4869,            2307,            6304,            3667,            3313,            3319,            3211,            2799,            2700,            1535,            2638,            2875,            8232,            5103,            3207,            4267,            2548,            2504,            3236,            2599,            3349,            5170,            1952,            6646,            547,            5765,            2164,            8445,            5395,            0,            7955,            2941,            5097,            3132,            6239,            2695,            2973,            ],        [            1610,            9569,            3532,            5051,            501,            5631,            5884,            6315,            7042,            5792,            7877,            1930,            6994,            8000,            8347,            6134,            7578,            6925,            6706,            3249,            9488,            1664,            6288,            7328,            7689,            6624,            7347,            9604,            7268,            7142,            7116,            686,            8879,            6467,            6192,            6829,            9408,            5291,            6985,            8390,            2907,            9133,            1464,            6902,            2475,            7993,            899,            2843,            7432,            0,            8351,            5804,            7147,            4979,            7789,            6774,            ],        [            6661,            1935,            6996,            3986,            7916,            2991,            3888,            3675,            1219,            4388,            1938,            5915,            4883,            1687,            2408,            3494,            1097,            838,            2305,            6955,            4247,            6173,            4020,            3660,            5594,            3957,            866,            1970,            3475,            4913,            942,            8101,            2756,            3953,            2423,            4823,            2674,            4204,            4874,            5624,            7256,            3892,            6515,            2822,            6460,            682,            8314,            7481,            2275,            7824,            0,            3164,            3479,            3892,            4970,            1452,            ],        [            4622,            3913,            4336,            3829,            5877,            276,            3314,            511,            3610,            3916,            2748,            3876,            5118,            4317,            2973,            330,            3020,            3521,            2148,            4330,            5455,            4134,            4412,            3503,            5813,            4748,            2789,            3948,            4698,            3299,            2558,            6062,            3301,            4591,            864,            3233,            5304,            2706,            5109,            5438,            4631,            5100,            4476,            4317,            5226,            3312,            6275,            4856,            4905,            5785,            3185,            0,            3322,            3301,            5204,            2216,            ],        [            4394,            4127,            2310,            507,            6768,            4363,            2687,            5047,            3857,            2066,            4576,            3648,            1690,            3879,            5046,            4866,            2582,            3476,            1710,            2304,            6291,            3906,            1698,            181,            2385,            1692,            2351,            4162,            4071,            1191,            2120,            6953,            5394,            1688,            2999,            1125,            4866,            2094,            1681,            2036,            2605,            4488,            4248,            3705,            3200,            2874,            4625,            2830,            4235,            6676,            3232,            4536,            0,            3223,            4592,            1778,            ],        [            5680,            4267,            3688,            2451,            6935,            3858,            905,            4542,            3997,            2848,            3849,            4934,            1897,            3021,            4319,            4361,            2722,            3616,            1850,            3682,            3061,            5192,            2480,            2125,            2608,            2422,            2491,            4302,            2289,            1921,            2260,            7120,            4667,            2418,            3139,            1855,            4429,            312,            1888,            3411,            3983,            2706,            5534,            1923,            4578,            3014,            7333,            4208,            2453,            6843,            3372,            4031,            1944,            0,            2810,            1918,            ],        [            5607,            4505,            3394,            3019,            6201,            6182,            969,            6737,            4798,            2554,            3913,            4732,            1451,            2936,            4383,            6556,            3423,            5025,            2413,            3388,            2099,            5119,            2315,            2693,            2093,            2254,            3192,            4540,            1905,            2114,            2961,            7710,            4731,            2178,            4491,            2024,            4344,            2699,            1442,            1152,            3689,            1744,            5461,            1838,            4284,            4004,            5838,            3914,            2368,            7433,            4781,            6226,            2512,            5735,            0,            2619,            ],        [            7521,            2349,            4689,            3412,            8776,            2417,            5557,            3101,            2079,            3814,            2798,            6775,            6552,            2101,            3268,            2920,            804,            1698,            1731,            4648,            4661,            7033,            3446,            3086,            7263,            3383,            573,            2384,            3889,            2882,            342,            8961,            3616,            3379,            1849,            2816,            3088,            5064,            6543,            7293,            4949,            4306,            7375,            3236,            7320,            1096,            9174,            5174,            2689,            8684,            1454,            2590,            2905,            4752,            6639,            0,            ],        ], 'starts': [52, 23], 'ends': [52, 23]},    'NW-16': {'distance_matrix': [        [            0,            4177,            4577,            3094,            627,            2613,            1183,            1533,            3298,            3732,            2685,            2181,            2588,            2655,            6281,            4008,            3729,            4723,            4004,            3717,            4840,            2787,            2467,            3537,            3900,            4261,            4203,            5288,            6554,            2908,            2760,            1568,            2356,            385,            526,            3505,            3175,            3678,            4028,            4404,            3416,            1927,            3569,            3274,            3021,            4317,            6084,            4492,            1353,            5550,            5519,            5040,            4141,            1964,            4080,            4569,            2271,            2904,            5920,            ],        [            3383,            0,            894,            3384,            4010,            2903,            2929,            4916,            1021,            1455,            1722,            2572,            1595,            2981,            6587,            4314,            4035,            5029,            4310,            4023,            5146,            3093,            2773,            3843,            4206,            4567,            4509,            5594,            6860,            1230,            1940,            2748,            2747,            3768,            3909,            1274,            917,            461,            345,            721,            3722,            2277,            3875,            3580,            3311,            4623,            6390,            4798,            4736,            5856,            5825,            5346,            4447,            2355,            4386,            4875,            2682,            3194,            1945,            ],        [            4034,            927,            0,            4311,            4661,            3830,            3580,            5567,            1948,            2382,            2373,            3223,            2246,            3908,            7514,            5241,            4962,            5956,            5237,            4950,            6073,            4020,            3700,            4770,            5133,            5494,            5436,            6521,            7787,            1881,            2591,            3399,            3398,            4419,            4560,            2201,            1568,            1190,            612,            879,            4649,            2928,            4802,            4507,            4238,            5550,            7317,            5725,            5387,            6783,            6752,            6273,            5374,            3006,            5313,            5802,            3609,            4121,            2313,            ],        [            3270,            3468,            4281,            0,            2599,            1097,            3618,            1569,            2589,            3023,            5120,            4616,            5023,            1856,            5482,            3209,            2930,            3924,            3205,            2918,            4041,            1988,            1668,            2738,            3101,            3462,            3404,            4489,            5755,            5343,            5195,            4003,            4791,            2842,            2941,            2796,            3695,            3848,            3732,            4108,            2617,            4362,            2770,            2475,            689,            3518,            5285,            3693,            2054,            4751,            4720,            4241,            3342,            4399,            3281,            3770,            1562,            572,            5211,            ],        [            987,            3550,            4363,            2467,            0,            1986,            1335,            906,            2671,            3105,            2837,            2333,            2740,            2028,            5654,            3381,            3102,            4096,            3377,            3090,            4213,            2160,            1840,            2910,            3273,            3634,            3576,            4661,            5927,            3060,            2912,            1720,            2508,            559,            658,            2878,            3327,            3930,            3814,            4190,            2789,            2079,            2942,            2647,            2394,            3690,            5457,            3865,            726,            4923,            4892,            4413,            3514,            2116,            3453,            3942,            1644,            2277,            5293,            ],        [            2989,            3187,            4000,            481,            2318,            0,            3337,            1288,            2308,            2742,            4839,            4335,            4742,            1575,            5201,            2928,            2649,            3643,            2924,            2637,            3760,            1707,            1387,            2457,            2820,            3181,            3123,            4208,            5474,            5062,            4914,            3722,            4510,            2561,            2660,            2515,            3414,            3567,            3451,            3827,            2336,            4081,            2489,            2194,            408,            3237,            5004,            3412,            1773,            4470,            4439,            3960,            3061,            4118,            3000,            3489,            1281,            291,            4930,            ],        [            1071,            2778,            3394,            4165,            1698,            3684,            0,            2604,            3684,            4118,            1502,            998,            1405,            3726,            5079,            5079,            4800,            6732,            5075,            4788,            5911,            3858,            3538,            4608,            6895,            6534,            5274,            6081,            5352,            1725,            1577,            385,            1173,            1456,            1597,            3375,            1992,            2495,            2845,            3221,            4487,            744,            4640,            4345,            4092,            5388,            6871,            6963,            2424,            6421,            6312,            7129,            5212,            781,            5151,            7432,            3342,            3975,            4445,            ],        [            1867,            2644,            3457,            1561,            1196,            1080,            2215,            0,            1765,            2199,            3717,            3213,            3620,            1122,            4748,            2475,            2196,            3190,            2471,            2184,            3307,            1254,            934,            2004,            2367,            2728,            2670,            3755,            5021,            3940,            3792,            2600,            3388,            1439,            1538,            1972,            2871,            3024,            2908,            3284,            1883,            2959,            2036,            1741,            1488,            2784,            4551,            2959,            651,            4017,            3986,            3507,            2608,            2996,            2547,            3036,            738,            1371,            4387,            ],        [            3369,            3567,            4380,            2363,            2698,            1882,            3717,            1668,            0,            434,            5219,            4715,            5122,            1940,            5566,            3293,            3014,            4008,            3289,            3002,            4125,            2072,            1752,            2822,            3185,            3546,            3488,            4573,            5839,            5442,            5294,            4102,            4890,            2941,            3040,            2895,            3794,            3947,            3831,            4207,            2701,            4461,            2854,            2559,            2290,            3602,            5369,            3777,            2153,            4835,            4804,            4325,            3426,            4498,            3365,            3854,            1661,            2173,            5310,            ],        [            2969,            3167,            3980,            1963,            2298,            1482,            3317,            1268,            2288,            0,            4819,            4315,            4722,            1540,            5166,            2893,            2614,            3608,            2889,            2602,            3725,            1672,            1352,            2422,            2785,            3146,            3088,            4173,            5439,            5042,            4894,            3702,            4490,            2541,            2640,            2495,            3394,            3547,            3431,            3807,            2301,            4061,            2454,            2159,            1890,            3202,            4969,            3377,            1753,            4435,            4404,            3925,            3026,            4098,            2965,            3454,            1261,            1773,            4910,            ],        [            2127,            1276,            1892,            5221,            2754,            4740,            1673,            3660,            2182,            2616,            0,            1316,            339,            4782,            6186,            6135,            5856,            7839,            6131,            5844,            8036,            4914,            4594,            5664,            6027,            7641,            6330,            7188,            6459,            223,            684,            1492,            1491,            2512,            2653,            1873,            490,            993,            1343,            1719,            5543,            1021,            5696,            5401,            5148,            6444,            7978,            6619,            3480,            7528,            7419,            8236,            6268,            1099,            6207,            6696,            4398,            5031,            2943,            ],        [            1559,            2487,            3103,            4653,            2186,            4172,            1105,            3092,            3393,            3827,            1211,            0,            1114,            4214,            5220,            5567,            5288,            6873,            5563,            5276,            7070,            4346,            4026,            5096,            5459,            6675,            5762,            6222,            5493,            1434,            1286,            839,            175,            1944,            2085,            3084,            1701,            2204,            2554,            2930,            4975,            453,            5128,            4833,            4580,            5876,            7012,            7104,            2912,            6562,            6453,            7270,            5700,            217,            5639,            6128,            3830,            4463,            4154,            ],        [            1788,            1572,            2188,            4882,            2415,            4401,            1334,            3321,            2478,            2912,            296,            977,            0,            4443,            5847,            5796,            5517,            7500,            5792,            5505,            7697,            4575,            4255,            5325,            5688,            7302,            5991,            6849,            6120,            519,            345,            1153,            1152,            2173,            2314,            2169,            786,            1289,            1639,            2015,            5204,            682,            5357,            5062,            4809,            6105,            7639,            6280,            3141,            7189,            7080,            7897,            5929,            760,            5868,            6357,            4059,            4692,            3239,            ],        [            2146,            2344,            3157,            1261,            1475,            780,            2494,            445,            1465,            1899,            3996,            3492,            3899,            0,            4448,            2175,            1896,            2890,            2171,            1884,            3007,            954,            634,            1704,            2067,            2428,            2370,            3455,            4721,            4219,            4071,            2879,            3667,            1718,            1817,            1672,            2571,            2724,            2608,            2984,            1583,            3238,            1736,            1441,            1188,            2484,            4251,            2659,            930,            3717,            3686,            3207,            2308,            3275,            2247,            2736,            438,            1071,            4087,            ],        [            7649,            6381,            7346,            5470,            5684,            4989,            7195,            4654,            5502,            5936,            7413,            7214,            7286,            4209,            0,            2634,            4493,            1653,            2406,            4481,            1850,            3551,            3990,            3110,            1816,            1455,            2173,            1002,            273,            6921,            7631,            6781,            7389,            5927,            6026,            5861,            6608,            6761,            6645,            7021,            4180,            7322,            4333,            4038,            5397,            2587,            1792,            1884,            5139,            1342,            1233,            2050,            2767,            7359,            3464,            2353,            4647,            5280,            8124,            ],        [            8460,            7428,            8241,            6365,            6579,            5884,            8006,            5549,            6549,            6983,            8891,            8025,            8333,            5104,            2903,            0,            5388,            981,            228,            5376,            1048,            4446,            4885,            3299,            2705,            2344,            461,            1891,            3176,            7968,            8678,            7592,            8200,            8845,            6921,            6756,            7655,            7808,            7692,            8068,            5075,            8133,            5228,            4933,            6292,            293,            2687,            750,            6034,            1292,            2122,            848,            133,            8170,            4359,            545,            5542,            6175,            9171,            ],        [            3928,            4106,            4919,            3043,            3257,            2562,            4276,            2227,            3227,            3661,            5138,            5988,            5011,            1782,            4572,            2299,            0,            3014,            2295,            2008,            3131,            1078,            2549,            1828,            2191,            2552,            2494,            3579,            4845,            4646,            5356,            4661,            6163,            3500,            3599,            3434,            4333,            4486,            4370,            4746,            1707,            6232,            160,            455,            2970,            2608,            4375,            2783,            2712,            3841,            3810,            3331,            2432,            5771,            2371,            2860,            2220,            2853,            5849,            ],        [            7378,            6346,            7159,            5283,            5497,            4802,            6924,            4467,            5467,            5901,            7809,            6943,            7251,            4022,            1821,            981,            4306,            0,            753,            4294,            197,            3364,            3803,            2217,            1623,            1262,            520,            809,            2094,            6886,            7596,            6510,            7118,            7763,            5839,            5674,            6573,            6726,            6610,            6986,            3993,            7051,            4146,            3851,            5210,            934,            1605,            231,            4952,            311,            1040,            397,            1114,            7088,            3277,            700,            4460,            5093,            8089,            ],        [            8456,            7424,            8237,            6361,            9083,            5880,            8002,            5545,            6545,            6979,            8887,            8021,            8329,            5100,            2899,            228,            5384,            753,            0,            5372,            870,            4442,            4881,            3295,            2701,            2340,            233,            1887,            3172,            7964,            8962,            7588,            8196,            8841,            8982,            6752,            7651,            7804,            7688,            8064,            5071,            8129,            5224,            4929,            6288,            461,            2683,            522,            6030,            1064,            2118,            1070,            361,            8166,            4355,            713,            5538,            6171,            9167,            ],        [            8122,            7108,            7921,            6045,            8749,            5564,            7668,            5229,            6229,            6663,            8553,            7687,            8013,            4784,            2564,            3125,            5068,            2144,            2897,            0,            2341,            4126,            4565,            2979,            2385,            2024,            2664,            1571,            2837,            7648,            8628,            7254,            7862,            8507,            8648,            6436,            7335,            7488,            7372,            7748,            4755,            7795,            4908,            4613,            5972,            3078,            2367,            2375,            5714,            1833,            1802,            2541,            3258,            7832,            4039,            2844,            5222,            5855,            8851,            ],        [            7575,            6543,            7356,            5480,            8202,            4999,            7121,            4664,            5664,            6098,            8006,            7140,            7448,            4219,            2018,            1048,            4503,            197,            870,            4491,            0,            3561,            4000,            2414,            1820,            1459,            637,            1006,            2291,            7083,            7793,            6707,            7315,            7960,            8101,            5871,            6770,            6923,            6807,            7183,            4190,            7248,            4343,            4048,            5407,            755,            1802,            348,            5149,            508,            1237,            200,            1181,            7285,            3474,            503,            4657,            5290,            8286,            ],        [            4135,            4313,            5126,            3250,            3464,            2769,            4483,            2434,            3434,            3868,            5345,            6195,            5218,            1989,            3494,            1221,            2273,            1936,            1217,            930,            2053,            0,            2756,            3909,            3315,            2954,            1416,            2501,            3767,            4853,            5563,            8184,            6370,            3707,            3806,            3641,            4540,            4693,            4577,            4953,            629,            6439,            2113,            1818,            3177,            1530,            3297,            1705,            2919,            2763,            2732,            2253,            1354,            5978,            1293,            1782,            2427,            3060,            6056,            ],        [            4403,            4581,            5394,            3518,            3732,            3037,            4751,            2702,            3702,            4136,            5613,            6463,            5486,            2257,            5093,            2820,            2541,            3535,            2816,            2529,            3652,            1599,            0,            1234,            1700,            2061,            3015,            2514,            5366,            5121,            5831,            6639,            6638,            3975,            4074,            3909,            4808,            4961,            4845,            5221,            2228,            6168,            2381,            2086,            3445,            3129,            3468,            3304,            3187,            4362,            2819,            3852,            2953,            6246,            2792,            3381,            2695,            3328,            6324,            ],        [            3459,            3637,            4450,            2574,            2788,            2093,            3807,            1758,            2758,            3192,            4669,            5519,            4542,            1313,            4149,            1876,            1597,            2591,            1872,            1585,            2708,            655,            1094,            0,            756,            1117,            2071,            1570,            4422,            4177,            4887,            4192,            5694,            3031,            3130,            2965,            3864,            4017,            3901,            4277,            1284,            5119,            1437,            1142,            2501,            2185,            3952,            2360,            2243,            3418,            3387,            2908,            2009,            5302,            1848,            2437,            1751,            2384,            5380,            ],        [            4053,            4231,            5044,            3168,            3382,            2687,            4401,            2352,            3352,            3786,            5263,            6113,            5136,            1907,            2062,            2470,            2191,            1623,            2376,            2179,            1820,            1249,            1688,            594,            0,            361,            2143,            814,            2335,            4771,            5481,            6751,            6288,            3625,            3724,            3559,            4458,            4611,            4495,            4871,            1878,            5713,            2031,            1736,            3095,            2557,            1768,            1854,            2837,            1312,            1119,            2020,            2603,            5896,            2092,            2323,            2345,            2978,            5974,            ],        [            7258,            4592,            5405,            3529,            3743,            3048,            6804,            2713,            3713,            4147,            5624,            6823,            5497,            2268,            1701,            2243,            2552,            1262,            2015,            2540,            1459,            1610,            2049,            955,            361,            0,            1782,            453,            1974,            5132,            5842,            6390,            6998,            3986,            4085,            3920,            4819,            4972,            4856,            5232,            2239,            6179,            2392,            2097,            3456,            2196,            1407,            1493,            3198,            951,            758,            1659,            2376,            6257,            2109,            1962,            2706,            3339,            6335,            ],        [            7982,            6950,            7763,            5887,            8609,            5406,            7528,            5071,            6071,            6505,            8413,            7547,            8316,            4626,            2425,            461,            4910,            520,            233,            4898,            637,            3968,            4407,            2821,            2227,            1866,            0,            1413,            2698,            7490,            8488,            7114,            7722,            8367,            8508,            6278,            7177,            7330,            7214,            7590,            4597,            7655,            4750,            4455,            5814,            414,            2209,            289,            5556,            831,            1644,            837,            594,            7692,            3881,            531,            5064,            5697,            8693,            ],        [            6805,            5045,            5858,            3982,            7432,            3501,            6351,            3166,            4166,            4600,            7236,            6370,            7139,            2721,            1248,            1790,            3005,            809,            1562,            2993,            1006,            2063,            2502,            1408,            814,            453,            1329,            0,            1521,            5585,            7311,            5937,            6545,            7190,            7331,            4373,            5272,            5425,            5309,            5685,            2692,            6478,            2845,            2550,            3909,            1743,            954,            1040,            3651,            498,            305,            1206,            1923,            6515,            2626,            1509,            3159,            3792,            6788,            ],        [            7376,            5774,            7073,            5197,            8003,            4716,            6922,            4381,            4895,            5329,            7807,            6941,            7710,            3936,            1819,            2361,            4220,            1380,            2133,            4208,            1577,            3278,            3717,            2837,            1543,            1182,            1900,            729,            0,            8030,            7882,            6508,            7116,            7761,            5753,            5588,            6001,            6154,            6038,            6414,            3907,            7049,            4060,            3765,            5124,            2314,            1519,            1611,            4866,            1069,            960,            1777,            2494,            7086,            3191,            2080,            4374,            5007,            7517,            ],        [            2279,            1053,            1669,            5373,            2906,            4892,            1825,            3812,            1959,            2393,            618,            1468,            491,            4934,            6338,            6287,            6008,            7002,            6283,            5996,            7119,            5066,            4746,            5816,            6179,            6540,            6482,            7340,            6611,            0,            836,            1644,            1643,            2664,            2805,            1650,            267,            770,            1120,            1496,            5695,            1173,            5848,            5553,            5300,            6596,            8130,            6771,            3632,            7680,            7571,            7319,            6420,            1251,            6359,            6848,            4550,            5183,            2720,            ],        [            1443,            1982,            2598,            4537,            2070,            4056,            989,            2976,            2888,            3322,            706,            632,            609,            4098,            5502,            5451,            5172,            7155,            5447,            5160,            6283,            4230,            3910,            4980,            5343,            5704,            5646,            6504,            5775,            929,            0,            808,            807,            1828,            1969,            2579,            1196,            1699,            2049,            2425,            4859,            337,            5012,            4717,            4464,            5760,            7294,            5935,            2796,            6844,            6735,            6483,            5584,            415,            5523,            6012,            3714,            4347,            3649,            ],        [            868,            2575,            3191,            3962,            1495,            3481,            414,            2401,            4166,            4600,            1299,            795,            1202,            3523,            5151,            4876,            4597,            5591,            4872,            4585,            5708,            3655,            3335,            4405,            4768,            5129,            5071,            6156,            5424,            1522,            1374,            0,            970,            1253,            1394,            3172,            1789,            2292,            2642,            3018,            4284,            541,            4437,            4142,            3889,            5185,            6943,            5360,            2221,            6418,            6387,            5908,            5009,            578,            4948,            5437,            3139,            3772,            4242,            ],        [            1734,            2662,            3278,            4828,            2361,            4347,            1280,            3267,            3568,            4002,            1386,            175,            1289,            4389,            5395,            5742,            5463,            7048,            5738,            5451,            7245,            4521,            4201,            5271,            5634,            6850,            5937,            6397,            5668,            1609,            1461,            1014,            0,            2119,            2260,            3259,            1876,            2379,            2729,            3105,            5150,            628,            5303,            5008,            4755,            6051,            7187,            6226,            3087,            6737,            6628,            7445,            5875,            392,            5814,            6303,            4005,            4638,            4329,            ],        [            933,            3792,            4605,            2709,            242,            2228,            1281,            1148,            2913,            3347,            2783,            2279,            2686,            2270,            5896,            3623,            3344,            4338,            3619,            3332,            4455,            2402,            2082,            3152,            3515,            3876,            3818,            4903,            6169,            3006,            2858,            1666,            2454,            0,            604,            3120,            3273,            3776,            4056,            4432,            3031,            2025,            3184,            2889,            2636,            3932,            5699,            4107,            968,            5165,            5134,            4655,            3756,            2062,            3695,            4184,            1886,            2519,            5535,            ],        [            736,            3862,            4478,            3022,            555,            2541,            1084,            1461,            3226,            3660,            2586,            2082,            2489,            2583,            6163,            3936,            3657,            4651,            3932,            3645,            4768,            2715,            2395,            3465,            3828,            4189,            4131,            5216,            6436,            2809,            2661,            1469,            2257,            313,            0,            3433,            3076,            3579,            3929,            4305,            3344,            1828,            3497,            3202,            2949,            4245,            6012,            4420,            1281,            5478,            5447,            4968,            4069,            1865,            4008,            4497,            2199,            2832,            5529,            ],        [            3901,            669,            1482,            3036,            4528,            2555,            3447,            2341,            673,            1107,            2240,            3090,            2113,            2633,            6239,            3966,            3687,            4681,            3962,            3675,            4798,            2745,            2425,            3495,            3858,            4219,            4161,            5246,            6512,            1748,            2458,            3266,            3265,            4286,            4427,            0,            1435,            1049,            933,            1536,            3374,            2795,            3527,            3232,            2963,            4275,            6042,            4450,            2826,            5508,            5477,            4998,            4099,            2873,            4038,            4527,            2334,            2846,            2350,            ],        [            3030,            786,            1402,            6124,            3657,            5643,            2576,            4563,            1692,            2126,            1369,            2219,            1242,            3652,            7089,            4985,            4706,            5700,            4981,            4694,            5817,            3764,            3444,            4514,            4877,            5238,            5180,            6265,            7362,            877,            1587,            2395,            2394,            3415,            3556,            1383,            0,            503,            853,            1229,            4393,            1924,            4546,            4251,            6051,            5294,            7061,            5469,            4383,            6527,            6496,            6017,            5118,            2002,            5057,            5546,            5301,            5934,            2453,            ],        [            2844,            672,            1288,            5938,            3471,            5457,            2390,            4377,            1506,            1940,            1183,            2033,            1056,            3466,            6903,            4799,            4520,            5514,            4795,            4508,            5631,            3578,            3258,            4328,            4691,            5052,            4994,            6079,            7176,            691,            1401,            2209,            2208,            3229,            3370,            1197,            378,            0,            739,            1115,            4207,            1738,            4360,            4065,            5865,            5108,            6875,            5283,            4197,            6341,            6310,            5831,            4932,            1816,            4871,            5360,            5115,            5748,            2339,            ],        [            3422,            315,            563,            3699,            4049,            3218,            2968,            4955,            1336,            1770,            1761,            2611,            1634,            3296,            6902,            4629,            4350,            5344,            4625,            4338,            5461,            3408,            3088,            4158,            4521,            4882,            4824,            5909,            7175,            1269,            1979,            2787,            2786,            3807,            3948,            1589,            956,            578,            0,            390,            4037,            2316,            4190,            3895,            3626,            4938,            6705,            5113,            4775,            6171,            6140,            5661,            4762,            2394,            4701,            5190,            2997,            3509,            1751,            ],        [            3618,            511,            666,            3895,            4245,            3414,            3164,            5151,            1532,            1966,            1957,            2807,            1830,            3492,            7098,            4825,            4546,            5540,            4821,            4534,            5657,            3604,            3284,            4354,            4717,            5078,            5020,            6105,            7371,            1465,            2175,            2983,            2982,            4003,            4144,            1785,            1152,            774,            196,            0,            4233,            2512,            4386,            4091,            3822,            5134,            6901,            5309,            4971,            6367,            6336,            5857,            4958,            2590,            4897,            5386,            3193,            3705,            1780,            ],        [            16239,            11500,            10987,            9774,            10651,            9956,            15785,            9621,            10621,            11055,            12532,            12056,            12405,            9176,            10681,            8408,            7957,            9123,            8395,            8117,            9240,            8518,            8617,            11096,            10502,            10141,            8603,            9688,            10954,            12040,            12741,            15371,            12231,            10894,            9667,            9502,            11727,            11880,            11764,            12131,            0,            15912,            8126,            8421,            10364,            8708,            10475,            8892,            10097,            9950,            9919,            9440,            8541,            11839,            664,            8969,            9614,            8921,            13243,            ],        [            1106,            2613,            3229,            4200,            1733,            3719,            652,            2639,            3519,            3953,            1337,            833,            1240,            3761,            5165,            5114,            4835,            6818,            5110,            4823,            7015,            3893,            3573,            4643,            5006,            6620,            5309,            6167,            5438,            1560,            1412,            471,            1008,            1491,            1632,            3210,            1827,            2330,            2680,            3056,            4522,            0,            4675,            4380,            4127,            5423,            6957,            5598,            2459,            6507,            6398,            7215,            5247,            616,            5186,            5675,            3377,            4010,            4280,            ],        [            3768,            3946,            4759,            2883,            3097,            2402,            4116,            2067,            3067,            3501,            4978,            5828,            4851,            1622,            4412,            2139,            160,            2854,            2135,            1848,            2971,            918,            2389,            1668,            2031,            2392,            2334,            3419,            4685,            4486,            5196,            4501,            6003,            3340,            3439,            3274,            4173,            4326,            4210,            4586,            1547,            6072,            0,            295,            2810,            2448,            4215,            2623,            2552,            3681,            3650,            3171,            2272,            5611,            2211,            2700,            2060,            2693,            5689,            ],        [            3473,            3651,            4464,            2588,            2802,            2107,            3821,            1772,            2772,            3206,            4683,            5533,            4556,            1327,            4117,            1844,            455,            2559,            1840,            1553,            2676,            623,            2094,            1373,            1736,            2097,            2039,            3124,            4390,            4191,            4901,            4206,            5708,            3045,            3144,            2979,            3878,            4031,            3915,            4291,            1252,            5777,            295,            0,            2515,            2153,            3920,            2328,            2257,            3386,            3355,            2876,            1977,            5316,            1916,            2405,            1765,            2398,            5394,            ],        [            2581,            2779,            3592,            889,            1910,            408,            2929,            880,            1900,            2334,            4431,            3927,            4334,            1167,            4793,            2520,            2241,            3235,            2516,            2229,            3352,            1299,            979,            2049,            2412,            2773,            2715,            3800,            5066,            4654,            4506,            3314,            4102,            2153,            2252,            2107,            3006,            3159,            3043,            3419,            1928,            3673,            2081,            1786,            0,            2829,            4596,            3004,            1365,            4062,            4031,            3552,            2653,            3710,            2592,            3081,            873,            699,            4522,            ],        [            8330,            7298,            8111,            6235,            6449,            5754,            7876,            5419,            6419,            6853,            8761,            7895,            8203,            4974,            2773,            293,            5258,            952,            461,            5246,            755,            4316,            4755,            3169,            2575,            2214,            414,            1761,            3046,            7838,            8836,            7462,            8070,            8715,            6791,            6626,            7525,            7678,            7562,            7938,            4945,            8003,            5098,            4803,            6162,            0,            2557,            703,            5904,            1263,            1992,            555,            426,            8040,            4229,            252,            5412,            6045,            9041,            ],        [            4563,            4741,            5554,            3678,            3892,            3197,            4911,            2862,            3862,            4296,            5773,            6623,            5646,            2417,            3216,            2980,            2701,            2777,            2976,            2689,            2974,            1759,            2198,            1318,            1498,            1515,            3175,            1968,            3489,            5281,            5991,            5296,            6798,            4135,            4234,            4069,            4968,            5121,            5005,            5381,            2388,            6867,            2541,            2246,            3605,            3289,            0,            3008,            3347,            2466,            2273,            3174,            3113,            6406,            1672,            3477,            2855,            3488,            6484,            ],        [            7609,            6577,            7390,            5514,            5728,            5033,            6747,            4698,            5698,            6132,            8040,            7174,            7482,            4253,            2052,            750,            4537,            231,            522,            4525,            348,            3595,            4034,            2448,            1854,            1493,            289,            1040,            2325,            7117,            7827,            6741,            7349,            5971,            6070,            5905,            6804,            6957,            6841,            7217,            4224,            7282,            4377,            4082,            5441,            703,            1836,            0,            5183,            542,            1271,            548,            883,            7319,            3508,            756,            4691,            5324,            8320,            ],        [            1216,            3330,            4143,            2247,            545,            1766,            1564,            686,            2451,            2885,            3066,            2562,            2969,            1808,            5434,            3161,            2882,            3876,            3157,            2870,            3993,            1940,            1620,            2690,            3053,            3414,            3356,            4441,            5707,            3289,            3141,            1949,            2737,            788,            887,            2658,            3556,            4059,            3594,            3970,            2569,            2308,            2722,            2427,            2174,            3470,            5237,            3645,            0,            4703,            4672,            4193,            3294,            2345,            3233,            3722,            1424,            2057,            5073,            ],        [            7067,            6035,            6848,            4972,            5186,            4491,            6205,            4156,            5156,            5590,            7498,            6632,            6940,            3711,            1510,            1292,            3995,            311,            1064,            3983,            508,            3053,            3492,            1906,            1312,            951,            831,            498,            1783,            6575,            7285,            6199,            6807,            5429,            5528,            5363,            6262,            6415,            6299,            6675,            3682,            6740,            3835,            3540,            4899,            1245,            1294,            542,            4641,            0,            729,            708,            1425,            6777,            2966,            1011,            4149,            4782,            7778,            ],        [            5172,            5350,            6163,            4287,            4501,            3806,            5520,            3471,            4471,            4905,            6382,            6601,            6255,            3026,            1479,            2021,            3310,            1040,            1793,            3298,            1237,            2368,            3383,            1713,            1119,            758,            1560,            305,            1752,            5890,            6600,            6168,            6776,            4744,            4843,            4678,            5577,            5730,            5614,            5990,            2997,            6709,            3150,            2855,            4214,            1974,            1185,            1271,            3956,            729,            0,            1437,            2154,            6746,            2857,            1740,            3464,            4097,            7093,            ],        [            7775,            6743,            7556,            5680,            5894,            5199,            6913,            4864,            5864,            6298,            8206,            7340,            7648,            4419,            2218,            848,            4703,            397,            1070,            4691,            200,            3761,            4200,            2614,            2020,            1659,            837,            1206,            2491,            7283,            7993,            6907,            7515,            6137,            6236,            6071,            6970,            7123,            7007,            7383,            4390,            7448,            4543,            4248,            5607,            555,            2002,            548,            5349,            708,            1437,            0,            981,            7485,            3674,            303,            4857,            5490,            8486,            ],        [            8593,            7561,            8374,            6498,            6712,            6017,            8139,            5682,            6682,            7116,            9024,            8158,            8466,            5237,            3036,            133,            5521,            1114,            361,            5509,            1181,            4579,            5018,            3432,            2838,            2477,            594,            2024,            3309,            8101,            8811,            7725,            8333,            6955,            7054,            6889,            7788,            7941,            7825,            8201,            5208,            8266,            5361,            5066,            6425,            426,            2820,            883,            6167,            1425,            2255,            981,            0,            8303,            4492,            678,            5675,            6308,            9304,            ],        [            1342,            2270,            2886,            4436,            1969,            3955,            888,            2875,            3176,            3610,            994,            217,            897,            3997,            5401,            5350,            5071,            7054,            5346,            5059,            7251,            4129,            3809,            4879,            5242,            6856,            5545,            6403,            5674,            1217,            1069,            707,            392,            1727,            1868,            2867,            1484,            1987,            2337,            2713,            4758,            236,            4911,            4616,            4363,            5659,            7193,            5834,            2695,            6743,            6634,            7451,            5483,            0,            5422,            5911,            3613,            4246,            3937,            ],        [            10657,            10835,            10304,            9772,            9986,            9291,            11005,            8956,            9956,            10390,            11867,            11373,            11740,            8511,            10016,            7743,            7301,            8458,            7739,            7452,            8575,            7853,            7934,            8557,            9837,            9476,            7938,            9023,            10289,            11375,            12085,            14706,            11548,            10229,            8984,            8819,            11062,            11215,            11099,            11475,            7260,            12961,            7461,            8340,            9699,            8052,            9819,            8227,            9441,            9285,            9254,            8775,            7876,            11156,            0,            8304,            8949,            8238,            12578,            ],        [            8078,            7046,            7859,            5983,            6197,            5502,            7216,            5167,            6167,            6601,            8509,            7643,            7951,            4722,            2521,            545,            5006,            700,            713,            4994,            503,            4064,            4503,            2917,            2323,            1962,            666,            1509,            2794,            7586,            8296,            7210,            7818,            6440,            6539,            6374,            7273,            7426,            7310,            7686,            4693,            7751,            4846,            4551,            5910,            252,            2305,            684,            5652,            1011,            1740,            303,            678,            7788,            3977,            0,            5160,            5793,            8789,            ],        [            4599,            1906,            2719,            3398,            3733,            2917,            4145,            2703,            1027,            1461,            2938,            3788,            2811,            2987,            6593,            4320,            4041,            5035,            4316,            4029,            5152,            3099,            2779,            3849,            4212,            4573,            4515,            5600,            6866,            2446,            3156,            3964,            3963,            4984,            4075,            1234,            2133,            2286,            2170,            2546,            3728,            3493,            3881,            3586,            3325,            4629,            6396,            4804,            3188,            5862,            5831,            5352,            4453,            3571,            4392,            4881,            0,            3208,            3649,            ],        [            3112,            3310,            4123,            624,            2441,            939,            3460,            1411,            2431,            2865,            4962,            4458,            4865,            1698,            5324,            3051,            2772,            3766,            3047,            2760,            3883,            1830,            1510,            2580,            2943,            3304,            3246,            4331,            5597,            5185,            5037,            3845,            4633,            2684,            2783,            2638,            3537,            3690,            3574,            3950,            2459,            4204,            2612,            2317,            531,            3360,            5127,            3535,            1896,            4593,            4562,            4083,            3184,            4241,            3123,            3612,            1404,            0,            5053,            ],        [            4130,            898,            2037,            3219,            3554,            2738,            3676,            2524,            848,            1282,            2469,            3319,            2342,            2808,            6414,            4141,            3862,            4856,            4137,            3850,            4973,            2920,            2600,            3670,            4033,            4394,            4336,            5421,            6687,            1977,            2687,            3495,            3494,            4515,            3896,            1101,            1664,            1278,            1567,            1765,            3549,            3024,            3702,            3407,            3146,            4450,            6217,            4625,            3009,            5683,            5652,            5173,            4274,            3102,            4213,            4702,            2517,            3029,            0,            ],        ], 'starts': [9, 28, 4, 26], 'ends': [9, 28, 4, 26]},    'NW-17': {'distance_matrix': [        [            0,            766,            201,            4084,            1072,            585,            2658,            5783,            492,            4641,            6067,            1502,            2709,            364,            6646,            1352,            5588,            3059,            1486,            5247,            816,            2510,            1530,            4320,            6155,            5523,            ],        [            766,            0,            1069,            4437,            1940,            946,            3526,            6136,            1360,            4994,            6420,            2370,            1657,            520,            6999,            2220,            5941,            3412,            322,            5600,            1684,            3472,            478,            4673,            6508,            5876,            ],        [            567,            1592,            0,            4521,            871,            987,            2499,            6220,            291,            5078,            6504,            1565,            3065,            931,            7083,            1151,            6025,            3496,            1842,            5684,            615,            1569,            1886,            4757,            6592,            5960,            ],        [            1979,            2647,            1968,            0,            2297,            1598,            2613,            4774,            2285,            557,            5058,            2435,            5603,            2343,            5637,            2017,            4579,            2050,            3253,            4238,            1886,            2089,            3297,            3311,            5146,            4514,            ],        [            1256,            2442,            1187,            4177,            0,            1571,            1193,            5876,            844,            4734,            6160,            904,            3343,            1620,            6739,            280,            5681,            3152,            2120,            5340,            445,            698,            2164,            4413,            6248,            5616,            ],        [            2500,            3168,            2489,            3499,            2742,            0,            3058,            5198,            2730,            4056,            5482,            2880,            6124,            2864,            6061,            2462,            5003,            2474,            3774,            4662,            2331,            2534,            3818,            3735,            5570,            4938,            ],        [            2566,            3752,            2497,            4536,            1111,            2881,            0,            6235,            2242,            5093,            6519,            1228,            4653,            2930,            7098,            831,            6040,            3511,            3430,            5699,            1843,            413,            3474,            4772,            6607,            5975,            ],        [            6008,            6676,            5997,            4029,            6326,            5627,            6642,            0,            6314,            4586,            1362,            6464,            9632,            6372,            1941,            6046,            503,            3133,            7282,            1324,            5915,            6118,            7326,            2154,            1733,            694,            ],        [            636,            1402,            543,            4230,            580,            955,            2208,            5929,            0,            4787,            6213,            1274,            3409,            1000,            6792,            860,            5734,            3205,            2186,            5393,            324,            1278,            2230,            4466,            6301,            5669,            ],        [            1422,            2090,            1411,            4540,            2282,            1041,            3868,            6239,            1702,            0,            6523,            2712,            5046,            1786,            7102,            2562,            6044,            3515,            2696,            5703,            2026,            3720,            2740,            4776,            6611,            5979,            ],        [            5564,            6232,            5553,            3585,            5882,            5183,            6198,            1510,            5870,            4142,            0,            6020,            9188,            5928,            579,            5602,            1315,            2689,            6838,            880,            5471,            5674,            6882,            1710,            1008,            1250,            ],        [            1338,            2524,            1269,            5152,            1082,            1653,            1458,            6851,            1560,            5709,            7135,            0,            3425,            1702,            7714,            1362,            6656,            4127,            2202,            6315,            1078,            1310,            2246,            5388,            7223,            6591,            ],        [            2563,            1379,            2494,            5686,            3051,            2195,            4391,            7385,            2785,            6243,            7669,            3235,            0,            1873,            8248,            3331,            7190,            4661,            1057,            6849,            3047,            4243,            1011,            5922,            7757,            7125,            ],        [            364,            520,            565,            4931,            1436,            949,            3022,            6630,            856,            5488,            6914,            1866,            2122,            0,            7493,            1716,            6435,            3906,            816,            6094,            1180,            2874,            943,            5167,            7002,            6370,            ],        [            4985,            5653,            4974,            3006,            5303,            4604,            5619,            2603,            5291,            3563,            2887,            5441,            8609,            5349,            0,            5023,            2408,            2110,            6259,            301,            4892,            5095,            6303,            1131,            2975,            2343,            ],        [            1536,            2722,            1467,            3967,            280,            1851,            913,            5666,            1124,            4524,            5950,            624,            3623,            1900,            6529,            0,            5471,            2942,            2400,            5130,            725,            418,            2444,            4203,            6038,            5406,            ],        [            5505,            6173,            5494,            3526,            5823,            5124,            6139,            451,            5811,            4083,            1813,            5961,            9129,            5869,            1565,            5543,            0,            2630,            6779,            821,            5412,            5615,            6823,            1651,            1732,            191,            ],        [            5233,            5901,            5222,            3254,            5551,            4852,            5867,            2733,            5539,            3811,            3017,            5689,            8857,            5597,            3596,            5271,            2538,            0,            6507,            2197,            5140,            5343,            6551,            1270,            3105,            2473,            ],        [            1062,            322,            1663,            4629,            2220,            1138,            3560,            6328,            1954,            5186,            6612,            2404,            1335,            816,            7191,            2500,            6133,            3604,            0,            5792,            2216,            3412,            156,            4865,            6700,            6068,            ],        [            4684,            5352,            4673,            2705,            5002,            4303,            5318,            2302,            4990,            3262,            2586,            5140,            8308,            5048,            3165,            4722,            2107,            1809,            5958,            0,            4591,            4794,            6002,            830,            2674,            2042,            ],        [            753,            1519,            660,            4571,            729,            1072,            2357,            6270,            399,            5128,            6554,            2044,            3251,            1117,            7133,            1009,            6075,            3546,            2028,            5734,            0,            1427,            2072,            4807,            6642,            6010,            ],        [            2365,            3551,            2296,            4470,            698,            2680,            495,            6169,            1542,            5027,            6453,            1027,            4452,            2729,            7032,            418,            5974,            3445,            3229,            5633,            1143,            0,            3273,            4706,            6541,            5909,            ],        [            1724,            540,            1655,            4847,            2212,            1356,            3552,            6546,            1946,            5404,            6830,            2396,            1179,            1034,            7409,            2492,            6351,            3822,            218,            6010,            2208,            3404,            0,            5083,            6918,            6286,            ],        [            5620,            6288,            5609,            3641,            5938,            5239,            6254,            1472,            5926,            4198,            1756,            6076,            9244,            5984,            2335,            5658,            1277,            2745,            6894,            936,            5527,            5730,            6938,            0,            1844,            1212,            ],        [            6307,            6975,            6296,            4328,            6625,            5926,            6941,            566,            6613,            4885,            1746,            6763,            9931,            6671,            2325,            6345,            802,            3432,            7581,            1623,            6214,            6417,            7625,            2453,            0,            993,            ],        [            5607,            6275,            5596,            3628,            5925,            5226,            6241,            529,            5913,            4185,            1891,            6063,            9231,            5971,            2470,            5645,            334,            2732,            6881,            923,            5514,            5717,            6925,            1327,            2262,            0,            ],        ], 'starts': [20, 16, 2, 25], 'ends': [20, 16, 2, 25]},    'NW-18': {'distance_matrix': [        [            0,            1283,            2527,            1291,            4315,            1195,            778,            3011,            3245,            857,            1282,            3240,            3426,            2564,            3278,            193,            3554,            2346,            2240,            2132,            3254,            1508,            3167,            2170,            2880,            2979,            4072,            2897,            3502,            3715,            3320,            181,            408,            3825,            2895,            1779,            1076,            1304,            1959,            3819,            583,            2709,            1542,            1114,            ],        [            896,            0,            1809,            573,            3576,            1640,            1674,            2293,            2262,            1753,            564,            2522,            2708,            1846,            2560,            1089,            2815,            1628,            1522,            1414,            2271,            2404,            2449,            1452,            1897,            2261,            3333,            2179,            2682,            2997,            2337,            715,            1304,            3107,            2177,            1061,            342,            2200,            1241,            2836,            1479,            1991,            824,            2010,            ],        [            2704,            1808,            0,            1895,            1788,            3448,            3482,            765,            3849,            3561,            1886,            3844,            4030,            3168,            1032,            2897,            1027,            2950,            2844,            947,            3858,            4212,            3771,            2774,            3484,            733,            1545,            3501,            4106,            1209,            3924,            2523,            3112,            4429,            3499,            2383,            2150,            4008,            774,            4423,            3287,            463,            2146,            3818,            ],        [            1376,            758,            1515,            0,            3282,            2120,            2154,            1999,            1844,            2233,            836,            2794,            2316,            2118,            2266,            1569,            2521,            1900,            949,            1120,            1853,            2884,            2057,            1724,            1479,            1967,            3039,            2451,            2264,            2703,            1919,            1195,            1784,            2715,            2449,            488,            822,            2680,            947,            2418,            1959,            1697,            251,            2490,            ],        [            4210,            3314,            1807,            3401,            0,            4954,            4988,            1307,            5355,            5067,            3392,            5350,            5536,            4674,            1256,            4403,            761,            4456,            4350,            2453,            5364,            5718,            5277,            4280,            4990,            1554,            243,            5007,            5612,            579,            5430,            4029,            4618,            5935,            5005,            3889,            3656,            5514,            2390,            5929,            4793,            1585,            3652,            5324,            ],        [            1195,            2023,            3267,            2031,            5034,            0,            317,            3751,            3985,            1485,            2022,            3980,            4166,            3304,            4018,            1170,            4273,            3086,            2980,            2872,            3994,            834,            3907,            2910,            3620,            3719,            4791,            3637,            4242,            4455,            4060,            1527,            1122,            4565,            3635,            2519,            1816,            1038,            2699,            4559,            1560,            3449,            2282,            1228,            ],        [            778,            2041,            3285,            2049,            5052,            317,            0,            3769,            4003,            1014,            2040,            3998,            4184,            3322,            4036,            585,            4291,            3104,            2998,            2890,            4012,            624,            3925,            2928,            3638,            3737,            4809,            3655,            4260,            4473,            4078,            959,            394,            4596,            3653,            2537,            1834,            567,            2717,            4577,            901,            3467,            2300,            757,            ],        [            2927,            2031,            824,            2118,            1307,            3671,            3705,            0,            4072,            3784,            2109,            4067,            4253,            3391,            291,            3120,            546,            3173,            3067,            1170,            4081,            4435,            3994,            2997,            3707,            271,            1064,            3724,            4329,            728,            4147,            2746,            3335,            4652,            3722,            2606,            2373,            4231,            1055,            4646,            3510,            302,            2369,            4041,            ],        [            3130,            2234,            3271,            1756,            5038,            3874,            3908,            3755,            0,            3987,            2312,            1233,            1187,            1931,            4022,            3323,            4277,            1713,            683,            2876,            9,            4638,            928,            1493,            400,            3723,            4795,            1479,            605,            4459,            260,            2949,            3538,            1586,            1558,            1144,            2576,            4434,            2703,            759,            3713,            3453,            1717,            4244,            ],        [            857,            2242,            3486,            2250,            5253,            1485,            1175,            3970,            4204,            0,            2241,            4199,            4385,            3523,            4237,            664,            4492,            3305,            3199,            3091,            4213,            651,            4126,            3129,            3839,            3938,            5010,            3856,            4461,            4674,            4279,            1038,            805,            4784,            3854,            2738,            1933,            447,            2918,            4778,            292,            3668,            2501,            257,            ],        [            1740,            844,            2088,            852,            3855,            2484,            2518,            2572,            1850,            2597,            0,            1958,            2144,            1282,            2839,            1933,            3094,            1064,            1093,            1693,            1859,            3248,            1885,            888,            1485,            2540,            3612,            1615,            2220,            3276,            1925,            1559,            2148,            2543,            1613,            916,            1186,            3044,            1520,            2424,            2323,            2270,            1103,            2854,            ],        [            3047,            2151,            3395,            2412,            5162,            3791,            3825,            3879,            799,            3904,            2229,            0,            527,            1386,            4146,            3240,            4401,            1168,            911,            3000,            808,            4555,            268,            948,            1056,            3847,            4919,            819,            590,            4583,            609,            2866,            3455,            926,            898,            1372,            2493,            4351,            2827,            1373,            3630,            3577,            1609,            4161,            ],        [            3376,            2480,            3724,            2488,            5491,            4120,            4154,            4208,            1261,            4233,            2558,            462,            0,            1715,            4475,            3569,            4730,            1497,            1373,            3329,            1270,            4884,            259,            1277,            1518,            4176,            5248,            940,            1052,            4912,            1071,            3195,            3784,            457,            1019,            1834,            2822,            4680,            3156,            1541,            3959,            3906,            2071,            4490,            ],        [            1661,            765,            2009,            773,            3776,            2405,            2439,            2493,            1771,            2518,            843,            1911,            2097,            0,            2760,            1854,            3015,            1017,            1014,            1614,            1780,            3169,            1838,            841,            1406,            2461,            3533,            1568,            2173,            3197,            1846,            1480,            2069,            2496,            1566,            837,            1107,            2965,            1441,            2345,            2244,            2191,            1024,            2775,            ],        [            3194,            2298,            1091,            2385,            1256,            3938,            3972,            291,            4339,            4051,            2376,            4334,            4520,            3658,            0,            3387,            495,            3440,            3334,            1437,            4348,            4702,            4261,            3264,            3974,            538,            1013,            3991,            4596,            677,            4414,            3013,            3602,            4919,            3989,            2873,            2640,            4498,            1322,            4913,            3777,            569,            2636,            4308,            ],        [            193,            1476,            2720,            1484,            4487,            1170,            585,            3204,            3438,            664,            1475,            3433,            3619,            2757,            3471,            0,            3726,            2539,            2433,            2325,            3447,            1315,            3360,            2363,            3073,            3172,            4244,            3090,            3695,            3908,            3513,            374,            215,            4031,            3088,            1972,            1269,            1111,            2152,            4012,            390,            2902,            1735,            921,            ],        [            3449,            2553,            1086,            2640,            761,            4193,            4227,            546,            4594,            4306,            2631,            4589,            4775,            3913,            495,            3642,            0,            3695,            3589,            1692,            4603,            4957,            4516,            3519,            4229,            793,            518,            4246,            4851,            182,            4669,            3268,            3857,            5187,            4244,            3128,            2895,            4753,            1669,            5168,            4032,            824,            2891,            4563,            ],        [            1879,            983,            2227,            991,            3994,            2623,            2657,            2711,            1520,            2736,            1061,            2129,            2315,            218,            2978,            2072,            3233,            0,            1232,            1832,            1529,            3387,            2056,            1059,            1155,            2679,            3751,            1786,            1608,            3415,            1595,            1698,            2287,            2727,            1784,            1055,            1325,            3183,            1659,            2094,            2462,            2409,            1242,            2993,            ],        [            2359,            1463,            2464,            949,            4231,            3103,            3137,            2948,            757,            3216,            1541,            1413,            1367,            1009,            3215,            2552,            3470,            1299,            0,            2069,            766,            3867,            1108,            1079,            392,            2916,            3988,            1659,            1177,            3652,            832,            2178,            2767,            1766,            1804,            461,            1805,            3663,            1896,            1331,            2942,            2646,            698,            3473,            ],        [            2355,            1459,            395,            1546,            2183,            3099,            3133,            1160,            3500,            3212,            1537,            3495,            3681,            2819,            1427,            2548,            1422,            2601,            2495,            0,            3509,            3863,            3422,            2425,            3135,            1128,            1940,            3152,            3757,            1604,            3575,            2174,            2763,            4093,            3150,            2034,            1801,            3659,            425,            4074,            2938,            858,            1797,            3469,            ],        [            3121,            2225,            3262,            1747,            5029,            3865,            3899,            3746,            176,            3978,            2303,            1224,            1178,            1922,            4013,            3314,            4268,            1704,            674,            2867,            0,            4629,            919,            1484,            391,            3714,            4786,            1470,            596,            4450,            251,            2940,            3529,            1577,            1549,            1135,            2567,            4425,            2694,            750,            3704,            3444,            1708,            4235,            ],        [            1508,            2893,            4137,            2901,            5904,            834,            624,            4621,            4855,            651,            2892,            4850,            5036,            4174,            4888,            1315,            5143,            3956,            3850,            3742,            4864,            0,            4777,            3780,            4490,            4589,            5661,            4507,            5112,            5325,            4930,            1689,            1456,            5448,            4505,            3389,            2584,            204,            3569,            5429,            943,            4319,            3152,            394,            ],        [            3117,            2221,            3465,            2229,            5232,            3861,            3895,            3949,            1002,            3974,            2299,            203,            259,            1456,            4216,            3310,            4471,            1238,            1114,            3070,            1011,            4625,            0,            1018,            1259,            3917,            4989,            681,            793,            4653,            812,            2936,            3525,            658,            760,            1575,            2563,            4421,            2897,            1576,            3700,            3647,            1812,            4231,            ],        [            2655,            1759,            3003,            1767,            4770,            3399,            3433,            3487,            1869,            3512,            1837,            1070,            1256,            994,            3754,            2848,            4009,            776,            1545,            2608,            1878,            4163,            997,            0,            2126,            3455,            4527,            727,            1660,            4191,            1679,            2474,            3063,            1668,            725,            1831,            2101,            3959,            2435,            2443,            3238,            3185,            2018,            3769,            ],        [            2730,            1834,            2871,            1356,            4638,            3474,            3508,            3355,            584,            3587,            1912,            1632,            1586,            1138,            3622,            2923,            3877,            1428,            283,            2476,            593,            4238,            1327,            1208,            0,            3323,            4416,            1878,            1004,            4059,            659,            2549,            3138,            1985,            1957,            744,            2176,            4034,            2303,            1158,            3313,            3053,            1317,            3844,            ],        [            2895,            1999,            792,            2086,            1554,            3639,            3673,            271,            4040,            3752,            2077,            4035,            4221,            3359,            538,            3088,            793,            3141,            3035,            1138,            4049,            4403,            3962,            2965,            3675,            0,            1311,            3692,            4297,            975,            4115,            2714,            3303,            4633,            3690,            2574,            2341,            4199,            1023,            4614,            3478,            270,            2337,            4009,            ],        [            4077,            3181,            1564,            3158,            243,            4821,            4745,            1439,            5222,            4934,            3149,            5217,            5403,            4541,            1388,            4270,            518,            4213,            4217,            2320,            5231,            5585,            5144,            4147,            4857,            1968,            0,            4874,            5479,            336,            5297,            3786,            4485,            5692,            4872,            3756,            3523,            5381,            2147,            5796,            4660,            1698,            3409,            5191,            ],        [            2436,            1540,            2784,            1548,            4572,            3180,            3214,            3268,            1712,            3293,            1618,            913,            1099,            775,            3535,            2629,            3811,            557,            1326,            2389,            1721,            3944,            840,            337,            1969,            3236,            4308,            0,            1503,            3993,            1522,            2255,            2844,            1511,            568,            1612,            1882,            3740,            2216,            2286,            3019,            2966,            1799,            3550,            ],        [            3654,            2758,            3795,            2280,            5562,            4398,            4432,            4279,            667,            4511,            2836,            1315,            911,            1814,            4546,            3847,            4822,            1596,            1207,            3400,            676,            5162,            811,            1376,            924,            4247,            5340,            1362,            0,            4983,            477,            3473,            4062,            1310,            1441,            1668,            3100,            4958,            3227,            938,            4237,            3977,            2241,            4768,            ],        [            3741,            2735,            1228,            2822,            579,            4485,            4519,            728,            4886,            4598,            2813,            4771,            5067,            4095,            1052,            3824,            182,            3987,            3771,            1984,            4785,            5249,            4808,            3811,            4411,            1632,            336,            4428,            5033,            0,            4961,            3450,            4149,            5479,            4536,            3310,            3187,            5045,            1811,            5460,            4324,            1362,            3183,            4855,            ],        [            3332,            2436,            3680,            2167,            5447,            4076,            4110,            4164,            554,            4189,            2514,            973,            927,            1671,            4431,            3525,            4686,            1453,            1196,            3285,            563,            4840,            668,            1233,            811,            4132,            5204,            1219,            345,            4868,            0,            3151,            3740,            1326,            1298,            1657,            2778,            4636,            3112,            1283,            3915,            3862,            1894,            4446,            ],        [            181,            1102,            2346,            1110,            4113,            1531,            959,            2830,            3064,            1038,            1101,            3059,            3245,            2383,            3097,            374,            3352,            2165,            2059,            1951,            3073,            1689,            2986,            1989,            2699,            2798,            3870,            2716,            3321,            3534,            3139,            0,            589,            3657,            2714,            1598,            895,            1485,            1778,            3638,            764,            2528,            1361,            1295,            ],        [            408,            1691,            2935,            1699,            4702,            1122,            394,            3419,            3653,            805,            1690,            3648,            3834,            2972,            3686,            215,            3941,            2754,            2648,            2540,            3662,            834,            3575,            2578,            3288,            3387,            4459,            3305,            3910,            4123,            3728,            589,            0,            4246,            3303,            2187,            1484,            1252,            2367,            4227,            531,            3117,            1950,            1062,            ],        [            2726,            1830,            3074,            1838,            4841,            3470,            3504,            3558,            1432,            3583,            1908,            633,            819,            1065,            3825,            2919,            4080,            847,            1544,            2679,            1441,            4234,            560,            627,            1689,            3526,            4598,            290,            1223,            4262,            1242,            2545,            3134,            0,            369,            1902,            2172,            4030,            2506,            2006,            3309,            3256,            2089,            3840,            ],        [            3669,            2773,            4017,            2781,            5784,            4413,            4447,            4501,            2442,            4526,            2851,            1643,            1181,            2008,            4768,            3862,            5023,            1790,            2554,            3622,            2451,            5177,            1440,            1570,            2699,            4469,            5541,            1233,            2233,            5205,            2252,            3488,            4077,            943,            0,            2845,            3115,            4973,            3449,            3155,            4252,            4199,            3032,            4783,            ],        [            2187,            1291,            2003,            488,            3770,            2931,            2965,            2487,            1218,            3044,            1369,            2432,            1828,            1756,            2754,            2380,            3009,            1538,            461,            1608,            1227,            3695,            1569,            1362,            853,            2455,            3527,            2089,            1638,            3191,            1293,            2006,            2595,            2227,            2087,            0,            1633,            3491,            1435,            1792,            2770,            2185,            237,            3301,            ],        [            554,            1055,            2299,            1063,            4066,            1298,            1332,            2783,            3017,            1411,            1054,            3012,            3198,            2336,            3050,            747,            3305,            2118,            2012,            1904,            3026,            2062,            2939,            1942,            2652,            2751,            3823,            2669,            3274,            3487,            3092,            373,            962,            3610,            2667,            1551,            0,            1858,            1731,            3591,            1137,            2481,            1314,            1668,            ],        [            1304,            2689,            3933,            2697,            5700,            1038,            567,            4417,            4651,            447,            2688,            4646,            4832,            3970,            4684,            1111,            4939,            3752,            3646,            3538,            4660,            204,            4573,            3576,            4286,            4385,            5457,            4303,            4908,            5121,            4726,            1485,            1252,            5244,            4301,            3185,            2380,            0,            3365,            5225,            739,            4115,            2948,            190,            ],        [            1930,            1034,            568,            1121,            2335,            2674,            2708,            1052,            3075,            2787,            1112,            3070,            3256,            2394,            1319,            2123,            1574,            2176,            2070,            173,            3084,            3438,            2997,            2000,            2710,            1020,            2092,            2727,            3332,            1756,            3150,            1749,            2338,            3668,            2725,            1609,            1376,            3234,            0,            3649,            2513,            750,            1372,            3044,            ],        [            3485,            2833,            3870,            2355,            5393,            4473,            4507,            4110,            355,            4342,            2911,            1956,            1552,            2214,            4377,            3922,            4632,            1996,            1282,            3475,            364,            5237,            1753,            1776,            999,            4322,            5150,            1762,            949,            5058,            552,            3548,            4137,            1951,            1841,            1743,            3175,            4789,            3302,            0,            4312,            3808,            2316,            4843,            ],        [            583,            1968,            3212,            1976,            4979,            1777,            901,            3696,            3930,            292,            1967,            3925,            4111,            3249,            3963,            390,            4218,            3031,            2925,            2817,            3939,            943,            3852,            2855,            3565,            3664,            4736,            3582,            4187,            4400,            4005,            764,            531,            4523,            3580,            2464,            1659,            739,            2644,            4504,            0,            3394,            2227,            549,            ],        [            2625,            1729,            522,            1816,            1585,            3369,            3403,            302,            3770,            3482,            1807,            3765,            3951,            3089,            569,            2818,            824,            2871,            2765,            868,            3779,            4133,            3692,            2695,            3405,            270,            1342,            3422,            4027,            1006,            3845,            2444,            3033,            4363,            3420,            2304,            2071,            3929,            753,            4344,            3208,            0,            2067,            3739,            ],        [            1950,            1054,            1766,            251,            3533,            2694,            2728,            2250,            1455,            2807,            1132,            2669,            2065,            1993,            2517,            2143,            2772,            1775,            698,            1371,            1464,            3458,            1806,            1599,            1090,            2218,            3290,            2326,            1875,            2954,            1530,            1769,            2358,            2464,            2324,            237,            1396,            3254,            1198,            2029,            2533,            1948,            0,            3064,            ],        [            1114,            2499,            3743,            2507,            5510,            1228,            757,            4227,            4461,            257,            2498,            4456,            4642,            3780,            4494,            921,            4749,            3562,            3456,            3348,            4470,            394,            4383,            3386,            4096,            4195,            5267,            4113,            4718,            4931,            4536,            1295,            1062,            5054,            4111,            2995,            2190,            190,            3175,            5035,            549,            3925,            2758,            0,            ],        ], 'starts': [12, 14, 39, 2], 'ends': [12, 14, 39, 2]},    'NW-19': {'distance_matrix': [        [            0,            1489,            3110,            732,            2317,            2340,            1967,            2607,            2622,            1263,            2398,            2112,            718,            1734,            583,            2156,            2951,            3126,            2552,            2774,            2509,            1296,            3606,            2434,            2912,            2963,            1104,            3803,            2498,            1639,            2659,            1921,            1029,            1243,            3088,            2018,            3118,            1869,            2471,            3088,            1840,            3218,            2378,            2507,            2350,            1683,            3469,            2922,            1383,            2130,            1695,            2943,            2642,            1780,            1275,            2438,            5717,            2100,            907,            2184,            3076,            1654,            1220,            761,            1695,            3710,            2139,            1763,            383,            2307,            1934,            ],        [            1515,            0,            1621,            2448,            3761,            3784,            3854,            4051,            4118,            2707,            909,            3556,            1016,            3282,            1085,            3600,            1462,            1637,            4016,            1285,            4140,            409,            2117,            2845,            1423,            3805,            1515,            4482,            1415,            2050,            2913,            2763,            1871,            1152,            1599,            2429,            3529,            786,            3313,            3930,            2251,            3897,            3611,            2918,            1364,            583,            1980,            4366,            2232,            304,            2740,            1454,            3053,            2191,            2117,            1143,            7161,            611,            811,            2595,            1587,            3202,            1129,            2276,            3139,            2221,            3583,            2605,            1898,            481,            2606,            ],        [            3138,            1623,            0,            4071,            5384,            5407,            5138,            5674,            5741,            4330,            1025,            5179,            2639,            4905,            2708,            5223,            1187,            166,            5639,            1010,            5763,            2032,            1842,            4468,            1078,            5428,            3138,            6105,            2223,            3673,            4536,            4386,            3494,            2775,            1254,            4052,            5152,            1678,            4936,            5553,            3874,            5520,            5234,            4541,            2323,            2206,            509,            5989,            3855,            1529,            4160,            1179,            4676,            3814,            3740,            2102,            8784,            2137,            2434,            4218,            529,            4825,            2752,            3899,            4762,            1163,            5206,            4228,            3521,            1440,            4229,            ],        [            1613,            2958,            4274,            0,            1990,            2013,            1235,            2194,            1922,            1568,            3840,            1794,            2137,            1002,            2002,            1829,            4393,            4290,            1820,            4216,            1777,            2738,            5048,            2004,            4076,            2854,            1555,            3006,            3262,            1209,            1862,            1124,            1005,            2007,            4252,            1588,            2053,            2633,            2362,            2540,            1410,            2421,            1646,            2442,            3114,            2447,            4633,            2509,            1922,            2894,            1821,            4385,            2212,            1350,            1625,            3202,            5304,            3542,            2349,            2119,            4240,            922,            1984,            1798,            1918,            4874,            1812,            1654,            1420,            3071,            1825,            ],        [            2429,            3724,            5345,            1891,            0,            668,            1531,            442,            950,            1105,            4633,            732,            2953,            1247,            2818,            765,            5186,            5361,            899,            5009,            855,            3531,            5841,            3474,            5147,            3245,            2866,            3899,            4573,            2520,            2755,            2017,            2316,            3318,            5323,            2899,            2946,            3944,            2560,            2160,            2721,            3314,            2233,            3335,            4425,            3758,            5704,            1153,            2815,            4205,            2714,            5178,            3523,            2661,            2518,            4513,            3948,            4335,            3142,            3012,            5311,            1619,            3295,            2063,            1455,            5945,            357,            2547,            2387,            4382,            2718,            ],        [            2466,            4022,            5338,            2363,            974,            0,            863,            635,            282,            1142,            4904,            769,            2990,            1366,            2855,            238,            5457,            5354,            231,            5280,            187,            3802,            6112,            2806,            5140,            2577,            2606,            3231,            4326,            2260,            2087,            1349,            2069,            3071,            5316,            2639,            2278,            3697,            1892,            1492,            2461,            2646,            1565,            2667,            4178,            3511,            5697,            950,            2147,            3958,            2046,            5449,            3014,            2401,            1850,            4266,            3745,            4606,            3413,            2344,            5304,            951,            3048,            2100,            1492,            5938,            796,            1879,            2815,            4135,            2050,            ],        [            1663,            3159,            4475,            1500,            1491,            1514,            0,            1781,            1289,            1069,            4041,            1295,            2187,            503,            2052,            1330,            4594,            4491,            1187,            4417,            1311,            2939,            5249,            1943,            4277,            2216,            1743,            2368,            3463,            1397,            1224,            486,            1206,            2208,            4453,            1776,            1415,            2834,            1724,            1691,            1598,            1783,            797,            1804,            3315,            2648,            4834,            2096,            1284,            3095,            1183,            4586,            2151,            1538,            987,            3403,            4891,            3743,            2550,            1481,            4441,            227,            2185,            1297,            1419,            5075,            1313,            1016,            1952,            3272,            1187,            ],        [            2656,            3951,            5572,            2118,            538,            588,            1451,            0,            870,            1332,            4860,            959,            3180,            1474,            3045,            1025,            5413,            5588,            819,            5236,            775,            3758,            6068,            3542,            5374,            3165,            3093,            3819,            4800,            2747,            2675,            1937,            2543,            3545,            5550,            3126,            2866,            4171,            2480,            2080,            2948,            3234,            2153,            3255,            4652,            3985,            5931,            1073,            2735,            4432,            2634,            5405,            3750,            2888,            2438,            4740,            3868,            4562,            3369,            2932,            5538,            1539,            3522,            2290,            1682,            6172,            582,            2467,            2614,            4609,            2638,            ],        [            2244,            3740,            5056,            2081,            1289,            516,            581,            909,            0,            1179,            4622,            1084,            2768,            1084,            2633,            1128,            5175,            5072,            456,            4998,            549,            3520,            5830,            2524,            4858,            3048,            2324,            2949,            4044,            1978,            1805,            1067,            1787,            2789,            5034,            2357,            1996,            3415,            2305,            1963,            2179,            2364,            1069,            2385,            3896,            3229,            5415,            1224,            1865,            3676,            1764,            5167,            2732,            2119,            1568,            3984,            4019,            4324,            3131,            2062,            5022,            669,            2766,            1878,            1529,            5656,            1111,            1597,            2533,            3853,            1768,            ],        [            1324,            2619,            4240,            786,            1054,            1077,            1022,            1344,            1359,            0,            3528,            849,            1848,            344,            1713,            893,            4081,            4256,            1308,            3904,            1264,            2426,            4736,            2210,            4042,            3489,            1761,            3390,            3468,            1415,            2246,            1508,            1211,            2213,            4218,            1794,            2437,            2839,            2587,            2404,            1616,            2805,            1510,            2283,            3320,            2653,            4599,            1659,            1641,            3100,            2105,            4073,            2418,            1556,            1391,            3408,            4454,            3230,            2037,            1960,            4206,            1110,            2190,            958,            350,            4840,            876,            2038,            1282,            3277,            1971,            ],        [            2429,            914,            1028,            3090,            4675,            4698,            4325,            4965,            5032,            3621,            0,            4470,            1930,            4092,            1999,            4514,            553,            1044,            4930,            376,            5054,            1323,            1208,            3759,            669,            4719,            2429,            5396,            2329,            2964,            3827,            3677,            2785,            2066,            845,            3343,            4443,            1700,            4227,            4844,            3165,            4811,            4525,            3832,            1914,            1497,            1387,            5280,            3146,            1120,            3451,            545,            3967,            3105,            3031,            1693,            8075,            1428,            1725,            3509,            994,            4012,            2043,            3190,            4053,            1628,            4497,            3519,            2812,            1031,            3520,            ],        [            1697,            2992,            4613,            1159,            815,            838,            769,            1105,            1120,            373,            3901,            0,            2221,            515,            2086,            654,            4454,            4629,            1069,            4277,            1025,            2799,            5109,            2583,            4415,            3236,            2134,            3137,            3841,            1788,            1993,            1255,            1584,            2586,            4591,            2167,            2184,            3212,            2493,            2151,            1989,            2552,            1257,            2656,            3693,            3026,            4972,            1420,            2014,            3473,            1952,            4446,            2791,            1929,            1764,            3781,            4215,            3603,            2410,            2333,            4579,            857,            2563,            1331,            723,            5213,            637,            1785,            1655,            3650,            2344,            ],        [            736,            771,            2392,            1468,            3053,            3076,            2703,            3343,            3358,            1999,            1680,            2848,            0,            2470,            306,            2892,            2233,            2408,            3288,            2056,            3245,            578,            2888,            3166,            2194,            3699,            1836,            4803,            1736,            2371,            3234,            2657,            1765,            1473,            2370,            2750,            3850,            1107,            3207,            3824,            2572,            4218,            2960,            3239,            1588,            904,            2751,            3658,            2119,            1368,            2431,            2225,            3374,            2512,            2011,            1676,            6453,            1382,            189,            2916,            2358,            2390,            1450,            1497,            2431,            2992,            2875,            2499,            1119,            1545,            2927,            ],        [            1890,            3185,            4806,            1352,            988,            1011,            693,            1278,            1293,            566,            4094,            792,            2414,            0,            2279,            827,            4647,            4822,            1242,            4470,            1198,            2992,            5302,            2636,            4608,            3160,            2327,            3061,            4034,            1981,            1917,            1179,            1777,            2779,            4784,            2360,            2108,            3405,            2417,            2075,            2182,            2476,            1181,            2497,            3886,            3219,            5165,            1593,            1977,            3666,            1876,            4639,            2844,            2122,            1680,            3974,            4388,            3796,            2603,            2174,            4772,            781,            2756,            1524,            916,            5406,            810,            1709,            1848,            3843,            1880,            ],        [            544,            906,            2527,            1276,            2861,            2884,            2511,            3151,            3166,            1807,            1815,            2656,            135,            2278,            0,            2700,            2368,            2543,            3096,            2191,            3071,            713,            3023,            2604,            2329,            3507,            1971,            4241,            1871,            1809,            2672,            2465,            1573,            1608,            2505,            2188,            3288,            1242,            3015,            3632,            2010,            3656,            2768,            2677,            1723,            1039,            2886,            3466,            1927,            1503,            2239,            2360,            2812,            1950,            1819,            1811,            6261,            1517,            324,            2354,            2493,            2198,            1585,            1305,            2239,            3127,            2683,            2307,            927,            1680,            2365,            ],        [            2228,            3523,            5144,            1690,            736,            184,            1047,            577,            466,            904,            4432,            531,            2752,            1550,            2617,            0,            4985,            5160,            415,            4808,            371,            3330,            5640,            2990,            4946,            2761,            2790,            3415,            4510,            2444,            2271,            1533,            2253,            3255,            5122,            2823,            2462,            3881,            2076,            1676,            2645,            2830,            1535,            2851,            4362,            3695,            5503,            892,            2331,            4142,            2230,            4977,            3198,            2585,            2034,            4450,            3687,            4134,            2941,            2528,            5110,            1135,            3232,            1862,            1254,            5744,            558,            2063,            2186,            4319,            2234,            ],        [            2982,            1467,            1209,            3643,            5228,            5251,            4878,            5518,            5533,            4174,            553,            5023,            2483,            4645,            2552,            5067,            0,            1225,            5483,            177,            5438,            1876,            655,            4312,            1222,            5272,            2982,            5949,            2882,            3517,            4380,            4230,            3338,            2619,            1398,            3896,            4996,            2253,            4780,            5397,            3718,            5364,            5078,            4385,            2467,            2050,            1568,            5833,            3699,            1673,            4004,            8,            4520,            3658,            3584,            2246,            8628,            1981,            2278,            4062,            1547,            4994,            2596,            3743,            4606,            2181,            5050,            4072,            3365,            1584,            4073,            ],        [            3154,            1639,            166,            3815,            5400,            5423,            5368,            5690,            5705,            4346,            1041,            5195,            2655,            4614,            2724,            5239,            1203,            0,            5654,            1026,            5610,            2048,            1858,            4484,            1094,            5444,            3154,            6121,            2239,            3689,            4552,            4402,            3510,            2791,            1270,            4068,            5168,            1694,            4952,            5569,            3890,            5536,            5250,            4557,            2339,            2222,            459,            6005,            3871,            1545,            4176,            1195,            4692,            3830,            3756,            2118,            8800,            2153,            2450,            4234,            363,            5166,            2768,            3915,            4778,            997,            5222,            4244,            3537,            1456,            4245,            ],        [            2346,            3842,            5158,            1808,            1391,            618,            683,            1011,            102,            1281,            4724,            1186,            2870,            1186,            2735,            1230,            5277,            5174,            0,            5100,            651,            3622,            5932,            2626,            4960,            3150,            2426,            3051,            4146,            2080,            1907,            1169,            1889,            2891,            5136,            2459,            2098,            3517,            2407,            2065,            2281,            2466,            1171,            2487,            3998,            3331,            5517,            1326,            1967,            3778,            1866,            5269,            2834,            2221,            1670,            4086,            4121,            4426,            3233,            2164,            5124,            771,            2868,            1980,            1631,            5758,            1213,            1699,            2635,            3955,            1870,            ],        [            2805,            1290,            1032,            3466,            5051,            5074,            5019,            5341,            5356,            3997,            376,            4846,            2306,            4265,            2375,            4890,            177,            1048,            5305,            0,            5261,            1699,            832,            4135,            1045,            5095,            2805,            5772,            2705,            3340,            4203,            4053,            3161,            2442,            1221,            3719,            4819,            2076,            4603,            5220,            3541,            5187,            4901,            4208,            2290,            1873,            1391,            5656,            3522,            1496,            3827,            169,            4343,            3481,            3407,            2069,            8451,            1804,            2101,            3885,            1370,            4817,            2419,            3566,            4429,            2004,            4873,            3895,            3188,            1407,            3896,            ],        [            2619,            3914,            5399,            2081,            1127,            294,            924,            601,            343,            1295,            4823,            922,            3143,            1427,            3008,            391,            5376,            5415,            292,            5199,            0,            3721,            6031,            2867,            5201,            2638,            2667,            3292,            4387,            2321,            2148,            1410,            2130,            3132,            5377,            2700,            2339,            3758,            1953,            1553,            2522,            2707,            1412,            2728,            4239,            3572,            5758,            916,            2208,            4019,            2107,            5368,            3075,            2462,            1911,            4327,            3711,            4525,            3332,            2405,            5365,            1012,            3109,            2253,            1645,            5999,            949,            1940,            2876,            4196,            2111,            ],        [            1390,            325,            1946,            2051,            3636,            3659,            3604,            3926,            3941,            2582,            1234,            3431,            891,            2850,            960,            3475,            1787,            1962,            3890,            1610,            3846,            0,            2442,            2720,            1748,            3680,            1390,            4357,            1290,            1925,            2788,            2638,            1746,            1027,            1924,            2304,            3404,            661,            3188,            3805,            2126,            3772,            3486,            2793,            1142,            458,            2305,            4241,            2107,            922,            2412,            1779,            2928,            2066,            1992,            1230,            7036,            936,            686,            2470,            1912,            3402,            1004,            2151,            3014,            2546,            3458,            2480,            1773,            1099,            2481,            ],        [            3637,            2122,            1777,            4298,            5883,            5906,            5851,            6173,            6188,            4829,            1208,            5678,            3138,            5097,            3207,            5722,            655,            1793,            6137,            832,            6093,            2531,            0,            4967,            1877,            5927,            3637,            6604,            3537,            4172,            5035,            4885,            3993,            3274,            2053,            4551,            5651,            2908,            5435,            6052,            4373,            6019,            5733,            5040,            3122,            2705,            2136,            6488,            4354,            2328,            4659,            663,            5175,            4313,            4239,            2901,            9283,            2636,            2933,            4717,            2202,            5649,            3251,            4398,            5261,            2836,            5705,            4727,            4020,            2239,            4728,            ],        [            2024,            2811,            4127,            1379,            3041,            3064,            2529,            3232,            2793,            1987,            3693,            2836,            2548,            2098,            2413,            2880,            4246,            4143,            2691,            4069,            2815,            2591,            4901,            0,            3929,            2375,            1353,            2299,            3115,            655,            730,            1701,            1065,            1860,            4105,            1034,            1346,            2486,            1883,            2500,            1138,            1714,            2286,            502,            2967,            2300,            4486,            3646,            1163,            2747,            884,            4238,            208,            500,            1055,            3055,            6441,            3395,            2202,            504,            4093,            1859,            1837,            1814,            2419,            4727,            2863,            991,            1831,            2924,            750,            ],        [            2008,            577,            1544,            2485,            4070,            4093,            4038,            4360,            4375,            3016,            1250,            3865,            1509,            3284,            1578,            3909,            1803,            1560,            4324,            1626,            4280,            1871,            2458,            3154,            0,            4114,            1824,            4791,            1145,            2359,            3222,            3072,            2180,            1493,            176,            2738,            3838,            600,            3622,            4239,            2560,            4206,            3920,            3227,            1245,            1076,            1903,            4675,            2541,            451,            2846,            1795,            3362,            2500,            2426,            1024,            7470,            1976,            1304,            2904,            1510,            3836,            1438,            2713,            3448,            2144,            3892,            2914,            2335,            362,            2915,            ],        [            1889,            3184,            5214,            1351,            1717,            1740,            1160,            1863,            1424,            1295,            4093,            1521,            2413,            729,            2278,            1556,            4646,            5230,            1322,            4469,            1446,            2991,            5301,            2682,            5016,            0,            2482,            2793,            4202,            2136,            1963,            1225,            1945,            2947,            5192,            2515,            2154,            3573,            2211,            1811,            2337,            2522,            917,            2543,            4054,            3387,            5573,            2178,            2023,            3834,            1922,            4638,            2890,            2277,            1726,            4142,            4973,            3795,            2602,            2220,            5180,            490,            2924,            1523,            1645,            5814,            1539,            1755,            1900,            4011,            1926,            ],        [            1001,            1458,            2774,            958,            2543,            2566,            2511,            2833,            2848,            1489,            2340,            2338,            1054,            1757,            1123,            2382,            2893,            2790,            2797,            2716,            2753,            1238,            3548,            1844,            2576,            2587,            0,            3427,            1762,            1049,            2283,            1545,            653,            507,            2752,            1428,            2474,            1133,            2095,            2712,            1250,            2842,            2386,            1917,            1614,            947,            3133,            3148,            1007,            1394,            1319,            2885,            2052,            1190,            899,            1702,            5943,            2042,            849,            1594,            2740,            2302,            484,            1186,            1921,            3374,            2365,            1387,            808,            1571,            1558,            ],        [            2911,            3698,            5014,            2266,            2792,            2815,            2235,            2938,            2499,            2370,            4580,            2596,            3435,            1804,            3300,            2631,            5133,            5030,            2397,            4956,            2521,            3478,            5788,            1633,            4816,            2081,            2240,            0,            4002,            1850,            914,            1188,            1952,            2747,            4992,            2229,            305,            3373,            1589,            2206,            2342,            701,            1992,            1494,            3854,            3187,            5373,            3253,            1960,            3634,            1579,            5125,            1841,            1704,            1942,            3942,            6048,            4282,            3089,            1171,            4980,            1565,            2724,            2611,            2720,            5614,            2614,            1480,            2718,            3811,            1445,            ],        [            2305,            921,            1929,            2801,            4386,            4409,            4354,            4676,            4691,            3332,            1635,            4181,            1806,            3600,            1875,            4225,            2188,            1945,            4640,            2011,            4596,            1990,            2843,            3470,            1731,            4430,            2140,            5107,            0,            2675,            3538,            3388,            2496,            1809,            1907,            3054,            4154,            944,            3938,            4555,            2876,            4522,            4236,            3543,            671,            1373,            2288,            4991,            2857,            795,            3162,            2180,            3678,            2816,            2742,            450,            7786,            2361,            1601,            3220,            1895,            4152,            1754,            3029,            3764,            2529,            4208,            3230,            2651,            458,            3231,            ],        [            1413,            2200,            3516,            768,            2430,            2453,            2174,            2720,            2735,            1376,            3082,            2225,            1937,            1644,            1802,            2269,            3635,            3532,            2336,            3458,            2640,            1980,            4290,            809,            3318,            2132,            742,            2758,            2504,            0,            1189,            1090,            454,            1249,            3494,            379,            1805,            1875,            1640,            2257,            527,            2173,            1931,            913,            2356,            1689,            3875,            3035,            552,            2136,            1016,            3627,            1017,            155,            444,            2444,            5830,            2784,            1591,            871,            3482,            1847,            1226,            1203,            1808,            4116,            2252,            932,            1220,            2313,            882,            ],        [            1997,            2784,            4100,            1352,            3014,            3037,            2362,            3304,            2626,            1960,            3666,            2809,            2521,            1931,            2386,            2853,            4219,            4116,            2524,            4042,            2648,            2564,            4874,            719,            3902,            2716,            1326,            2132,            3088,            936,            0,            1674,            1038,            1833,            4078,            1315,            1179,            2459,            2224,            2841,            1428,            1547,            2119,            580,            2940,            2273,            4459,            3619,            1136,            2720,            665,            4211,            927,            790,            1028,            3028,            6414,            3368,            2175,            257,            4066,            1692,            1810,            1787,            2392,            4700,            2836,            772,            1804,            2897,            531,            ],        [            2562,            3522,            4838,            1917,            2533,            2556,            1976,            2679,            2240,            2111,            4404,            2337,            3086,            1545,            2951,            2372,            4957,            4854,            2138,            4780,            2262,            3302,            5612,            1457,            4640,            1822,            2064,            1882,            3826,            1674,            738,            0,            1623,            2571,            4816,            2053,            929,            3197,            1330,            1947,            2015,            1297,            1733,            1318,            3678,            3011,            5197,            2994,            1701,            3458,            1403,            4949,            1665,            1528,            1766,            3766,            5789,            4106,            2913,            995,            4804,            1306,            2548,            2352,            2461,            5438,            2355,            1221,            2369,            3635,            1269,            ],        [            1215,            2033,            3349,            570,            2232,            2255,            1976,            2522,            2537,            1178,            2915,            2027,            1739,            1446,            1604,            2071,            3468,            3365,            2138,            3291,            2442,            1813,            4123,            1191,            3151,            1934,            742,            2774,            2337,            396,            1630,            892,            0,            1082,            3327,            775,            1821,            1708,            1442,            2059,            597,            2189,            1733,            1264,            2189,            1522,            3708,            2837,            354,            1969,            666,            3460,            1399,            537,            246,            2277,            5632,            2617,            1424,            941,            3315,            1649,            1059,            1005,            1610,            3949,            2054,            734,            1022,            2146,            905,            ],        [            1372,            951,            2267,            1668,            3253,            3276,            3221,            3543,            3558,            2199,            1973,            3048,            873,            2467,            942,            3092,            2526,            2283,            3507,            2349,            3463,            1057,            3181,            2337,            2069,            3297,            1007,            3974,            1255,            1542,            2405,            2255,            1363,            0,            2245,            1921,            3021,            626,            2805,            3422,            1743,            3389,            3103,            2410,            1107,            440,            2626,            3858,            1724,            887,            2029,            2518,            2545,            1683,            1609,            1195,            6653,            1861,            668,            2087,            2233,            3019,            621,            1896,            2631,            2867,            3075,            2097,            1518,            1064,            2098,            ],        [            1832,            401,            1368,            2309,            3894,            3917,            3862,            4184,            4199,            2840,            1074,            3689,            1333,            3108,            1402,            3733,            1627,            1384,            4148,            1450,            4104,            1695,            2282,            2978,            1170,            3938,            1648,            4615,            969,            2183,            3046,            2896,            2004,            1317,            0,            2562,            3662,            424,            3446,            4063,            2384,            4030,            3744,            3051,            1069,            900,            1727,            4499,            2365,            275,            2670,            1619,            3186,            2324,            2250,            848,            7294,            1800,            1128,            2728,            1334,            3660,            1262,            2537,            3272,            1968,            3716,            2738,            2159,            186,            2739,            ],        [            1792,            2579,            3895,            1147,            2809,            2832,            2553,            3099,            3114,            1755,            3461,            2604,            2316,            2023,            2181,            2648,            4014,            3911,            2715,            3837,            3019,            2359,            4669,            1188,            3697,            2511,            1121,            3137,            2883,            379,            1568,            1469,            833,            1628,            3873,            0,            2184,            2254,            2019,            2636,            906,            2552,            2310,            1292,            2735,            2068,            4254,            3414,            931,            2515,            1243,            4006,            1396,            534,            823,            2823,            6209,            3163,            1970,            1250,            3861,            2226,            1605,            1582,            2187,            4495,            2631,            1311,            1599,            2692,            1261,            ],        [            2516,            3393,            4709,            1871,            2487,            2510,            1930,            2633,            2194,            2065,            4275,            2291,            3040,            1499,            2905,            2326,            4828,            4725,            2092,            4651,            2216,            3173,            5483,            1328,            4511,            1776,            1935,            2210,            3697,            1545,            609,            883,            1647,            2442,            4687,            1924,            0,            3068,            1284,            1901,            2037,            1625,            1687,            1189,            3549,            2882,            5068,            2948,            1655,            3329,            1274,            4820,            1536,            1399,            1637,            3637,            5743,            3977,            2784,            866,            4675,            1260,            2419,            2306,            2415,            5309,            2309,            1175,            2323,            3506,            1140,            ],        [            1558,            325,            1641,            2035,            3620,            3643,            3588,            3910,            3925,            2566,            1347,            3415,            1059,            2834,            1128,            3459,            1900,            1657,            3874,            1723,            3830,            1968,            2555,            2704,            1443,            3664,            1374,            4341,            967,            1909,            2772,            2622,            1730,            1043,            1619,            2288,            3388,            0,            3172,            3789,            2110,            3756,            3470,            2777,            1321,            626,            2000,            4225,            2091,            261,            2396,            1892,            2912,            2050,            1976,            1100,            7020,            2073,            854,            2454,            1607,            3386,            988,            2263,            2998,            2241,            3442,            2464,            1885,            438,            2465,            ],        [            2555,            3569,            4885,            1910,            2625,            2010,            2068,            2317,            1808,            2203,            4451,            2638,            3079,            1637,            2944,            2107,            5004,            4901,            1706,            4827,            1900,            3349,            5659,            2602,            4687,            908,            2153,            2205,            3873,            1807,            1660,            922,            1616,            2618,            4863,            2186,            1851,            3244,            0,            617,            2008,            2219,            1438,            2240,            3725,            3058,            5244,            2632,            1694,            3505,            1451,            4996,            2810,            1948,            1397,            3813,            5427,            4153,            2960,            1917,            4851,            1398,            2595,            2345,            2553,            5485,            2447,            1214,            2362,            3682,            1317,            ],        [            2732,            3746,            5062,            2087,            2691,            2207,            2134,            2514,            2005,            2269,            4628,            2495,            3256,            1703,            3121,            2304,            5181,            5078,            1903,            5004,            2097,            3526,            5836,            2779,            5088,            1085,            2330,            2402,            4050,            1984,            1837,            1099,            1793,            2795,            5264,            2363,            2028,            3421,            400,            0,            2185,            2396,            1635,            2417,            3902,            3235,            5421,            2829,            1871,            3682,            1628,            5173,            2987,            2125,            1574,            3990,            5624,            4330,            3137,            2529,            5028,            1464,            2772,            2522,            2619,            5662,            2513,            1391,            2539,            3859,            1494,            ],        [            1821,            2608,            3924,            1176,            2838,            2861,            2582,            3128,            3143,            1784,            3490,            2633,            2345,            2030,            2210,            2677,            4043,            3940,            2744,            3866,            2868,            2388,            4698,            806,            3726,            1952,            1150,            2231,            2912,            760,            662,            910,            862,            1657,            3902,            1139,            1278,            2283,            1460,            2077,            0,            1646,            2339,            667,            2764,            2097,            4283,            3443,            960,            2544,            489,            4035,            1014,            877,            852,            2852,            6238,            3192,            1999,            344,            3890,            1791,            1634,            1611,            2216,            4524,            2660,            596,            1628,            2721,            355,            ],        [            3406,            4283,            5599,            2761,            3377,            3400,            2820,            3523,            3084,            2955,            5165,            3181,            3930,            2389,            3795,            3216,            5718,            5615,            2982,            5541,            3106,            4063,            6373,            2218,            5401,            2666,            2825,            585,            4587,            2435,            1499,            1773,            2537,            3332,            5577,            2814,            890,            3958,            2174,            2791,            2927,            0,            2577,            2079,            4439,            3772,            5958,            3838,            2545,            4219,            2164,            5710,            2426,            2289,            2527,            4527,            6633,            4867,            3674,            1756,            5565,            2150,            3309,            3196,            3305,            6199,            3199,            2065,            3213,            4396,            2030,            ],        [            2212,            3708,            5024,            1674,            1616,            1304,            908,            1611,            1339,            1618,            4590,            1411,            2736,            1052,            2601,            1401,            5143,            5040,            1237,            4966,            1194,            3488,            5798,            2492,            4826,            2810,            2292,            2707,            4012,            1946,            1773,            1035,            1755,            2757,            5002,            2325,            1964,            3383,            2125,            1725,            2147,            2332,            0,            2353,            3864,            3197,            5383,            1926,            1833,            3644,            1732,            5135,            2700,            2087,            1536,            3952,            4721,            4292,            3099,            2030,            4990,            643,            2734,            1846,            1968,            5624,            1438,            1565,            2501,            3821,            1736,            ],        [            2163,            2950,            4266,            1518,            3180,            3203,            2924,            3470,            3485,            2126,            3832,            2975,            2687,            2237,            2552,            3019,            4385,            4282,            3086,            4208,            3210,            2730,            5040,            139,            4068,            2882,            1492,            2438,            3254,            794,            869,            1840,            1204,            1999,            4244,            1173,            1485,            2625,            2390,            3007,            1277,            1853,            2681,            0,            3106,            2439,            4625,            3785,            1302,            2886,            1023,            4377,            347,            639,            1194,            3194,            6580,            3534,            2341,            643,            4232,            2597,            1976,            1953,            2558,            4866,            3002,            1130,            1970,            3063,            889,            ],        [            1634,            803,            2119,            2130,            3715,            3738,            3683,            4005,            4020,            2661,            1825,            3510,            1135,            2929,            1204,            3554,            2378,            2135,            3969,            2201,            3925,            1319,            3033,            2799,            1921,            3759,            1469,            4436,            801,            2004,            2867,            2717,            1825,            1138,            2097,            2383,            3483,            478,            3267,            3884,            2205,            3851,            3565,            2872,            0,            702,            2478,            4320,            2186,            739,            2491,            2370,            3007,            2145,            2071,            741,            7115,            2123,            930,            2549,            2085,            3481,            1083,            2358,            3093,            2719,            3537,            2559,            1980,            916,            2560,            ],        [            932,            810,            2160,            1593,            3178,            3201,            3146,            3468,            3483,            2124,            1719,            2973,            433,            2392,            502,            3017,            2272,            2176,            3432,            2095,            3388,            617,            2927,            2262,            2233,            3222,            932,            3899,            1155,            1467,            2330,            2180,            1288,            569,            2409,            1846,            2946,            526,            2730,            3347,            1668,            3314,            3028,            2335,            1007,            0,            2519,            3783,            1649,            787,            1954,            2264,            2470,            1608,            1534,            1095,            6578,            1421,            228,            2012,            2126,            2944,            546,            1693,            2556,            2760,            3000,            2022,            1315,            964,            2023,            ],        [            3497,            1982,            509,            4158,            5743,            5766,            5711,            6033,            6048,            4689,            1384,            5538,            2998,            4957,            3067,            5582,            1546,            459,            5997,            1369,            5953,            2391,            2201,            4827,            1437,            5787,            3497,            6464,            2582,            4032,            4895,            4745,            3853,            3134,            1613,            4411,            5511,            2037,            5295,            5912,            4233,            5879,            5593,            4900,            2682,            2565,            0,            6348,            4214,            1888,            4519,            1538,            5035,            4173,            4099,            2461,            9143,            2496,            2793,            4577,            822,            5509,            3111,            4258,            5121,            1456,            5565,            4587,            3880,            1799,            4588,            ],        [            3123,            4418,            6039,            2585,            1631,            917,            1780,            1046,            1199,            1799,            5327,            1426,            3647,            2283,            3512,            1014,            5880,            6055,            1148,            5703,            1104,            4225,            6535,            3723,            5841,            3494,            3560,            4148,            5243,            3177,            3004,            2266,            2986,            3988,            6017,            3556,            3195,            4614,            2809,            2409,            3378,            3563,            2482,            3584,            5095,            4428,            6398,            0,            3064,            4899,            2963,            5872,            3931,            3318,            2767,            5183,            2795,            5029,            3836,            3261,            6005,            1868,            3965,            2757,            2149,            6639,            1453,            2796,            3081,            5052,            2967,            ],        [            1838,            3530,            4846,            1871,            2330,            2018,            1622,            2325,            1886,            1645,            4412,            2125,            2362,            1704,            2227,            2115,            4965,            4862,            1784,            4788,            1908,            3310,            5620,            2314,            4648,            2587,            2114,            2739,            3834,            1768,            1595,            857,            1577,            2579,            4824,            2147,            1786,            3205,            2095,            2273,            1969,            2154,            1379,            2175,            3686,            3019,            5205,            2640,            0,            3466,            1554,            4957,            2522,            1909,            1358,            3774,            5435,            4114,            2921,            1852,            4812,            1295,            2556,            1472,            1208,            5446,            2152,            1387,            2323,            3643,            1558,            ],        [            2813,            1298,            1380,            2300,            3885,            3908,            3853,            4175,            4190,            2831,            1086,            3680,            2314,            3099,            2383,            3724,            1639,            1396,            4140,            1462,            4095,            1707,            2294,            2969,            1182,            3929,            1639,            4606,            960,            2174,            3037,            2887,            1995,            1308,            1358,            2553,            3653,            415,            3437,            4054,            2375,            4021,            3735,            3042,            1060,            1881,            1739,            4490,            2356,            0,            2661,            1631,            3177,            2315,            2241,            839,            7285,            1812,            2109,            2719,            1346,            3651,            1253,            2528,            3263,            1980,            3707,            2729,            2150,            177,            2730,            ],        [            1478,            2265,            3581,            833,            2495,            2518,            2239,            2785,            2503,            1441,            3147,            2290,            2002,            1709,            1867,            2334,            3700,            3597,            2401,            3523,            2525,            2045,            4355,            1212,            3383,            2197,            807,            3037,            2569,            417,            1893,            1155,            519,            1314,            3559,            796,            2084,            1940,            1705,            2322,            784,            2452,            1996,            1316,            2421,            1754,            3940,            3100,            617,            2201,            0,            3692,            1420,            558,            509,            2509,            5895,            2849,            1656,            1128,            3547,            1912,            1291,            1268,            1873,            4181,            2317,            997,            1285,            2378,            1168,            ],        [            2974,            1459,            1201,            3635,            5220,            5243,            5188,            5510,            5525,            4166,            545,            5015,            2475,            4434,            2544,            5059,            8,            1217,            5475,            169,            5430,            1868,            663,            4304,            1214,            5264,            2974,            5941,            2874,            3509,            4372,            4222,            3330,            2611,            1390,            3888,            4988,            2245,            4772,            5389,            3710,            5356,            5070,            4377,            2459,            2042,            1560,            5825,            3691,            1665,            3996,            0,            4512,            3650,            3576,            2238,            8620,            1973,            2270,            4054,            1539,            4986,            2588,            3735,            4598,            2173,            5042,            4064,            3357,            1576,            4065,            ],        [            1807,            2594,            3910,            1162,            2824,            2847,            2568,            3114,            2832,            1770,            3476,            2619,            2331,            2038,            2196,            2663,            4029,            3926,            2730,            3852,            2854,            2374,            4684,            587,            3712,            2526,            1136,            2924,            2898,            438,            1355,            1484,            848,            1643,            3888,            817,            1971,            2269,            2034,            2651,            921,            2339,            2325,            691,            2750,            2083,            4269,            3429,            946,            2530,            1258,            4021,            0,            283,            838,            2838,            6224,            3178,            1985,            1265,            3876,            2241,            1620,            1597,            2202,            4510,            2646,            1326,            1614,            2707,            1375,            ],        [            1524,            2311,            3627,            879,            2541,            2564,            2285,            2831,            2549,            1487,            3193,            2336,            2048,            1755,            1913,            2380,            3746,            3643,            2447,            3569,            2571,            2091,            4401,            654,            3429,            2243,            853,            2869,            2615,            155,            1300,            1201,            565,            1360,            3605,            534,            1916,            1986,            1751,            2368,            638,            2284,            2042,            758,            2467,            1800,            3986,            3146,            663,            2247,            1127,            3738,            862,            0,            555,            2555,            5941,            2895,            1702,            982,            3593,            1958,            1337,            1314,            1919,            4227,            2363,            1043,            1331,            2424,            993,            ],        [            1400,            2414,            3730,            755,            2417,            2440,            2161,            2707,            3004,            1363,            3296,            2212,            1924,            1631,            1789,            2256,            3849,            3746,            2902,            3672,            3096,            2194,            4504,            1447,            3532,            1688,            998,            2528,            2718,            652,            1384,            646,            461,            1463,            3708,            1031,            1575,            2089,            1196,            1813,            853,            1943,            2634,            1520,            2570,            1903,            4089,            3022,            539,            2350,            420,            3841,            1655,            793,            0,            2658,            5817,            2998,            1805,            1197,            3696,            2178,            1440,            1190,            1795,            4330,            2239,            488,            1207,            2527,            659,            ],        [            1855,            1024,            2340,            2351,            3936,            3959,            3904,            4226,            4241,            2882,            2046,            3731,            1356,            3150,            1425,            3775,            2599,            2356,            4191,            2422,            4146,            1540,            3254,            3020,            2142,            3980,            1690,            4657,            369,            2225,            3088,            2938,            2046,            1359,            2318,            2604,            3704,            699,            3488,            4105,            2426,            4072,            3786,            3093,            221,            923,            2699,            4541,            2407,            960,            2712,            2591,            3228,            2366,            2292,            0,            7336,            2344,            1151,            2770,            2306,            3702,            1304,            2579,            3314,            2940,            3758,            2780,            2201,            1137,            2781,            ],        [            5727,            7022,            8643,            5189,            4235,            3521,            4384,            3650,            3803,            4403,            7931,            4030,            6251,            4887,            6116,            3618,            8484,            8659,            3752,            8307,            3708,            6829,            9139,            6327,            8445,            6098,            6164,            6752,            7871,            5818,            5608,            4870,            5614,            6616,            8621,            6197,            5799,            7242,            5413,            5013,            6019,            6167,            5086,            6188,            7723,            7056,            9002,            2966,            5668,            7503,            5567,            8476,            6535,            5959,            5371,            7811,            0,            7633,            6440,            5865,            8609,            4472,            6593,            5361,            4753,            9243,            4057,            5400,            5685,            7680,            5571,            ],        [            2872,            1357,            1010,            3533,            5118,            5141,            5086,            5408,            5423,            4064,            298,            4913,            2373,            4332,            2442,            4957,            851,            1026,            5373,            674,            5328,            1766,            1506,            4202,            812,            5162,            2872,            5839,            1957,            3407,            4270,            4120,            3228,            2509,            988,            3786,            4886,            1412,            4670,            5287,            3608,            5254,            4968,            4275,            2057,            1940,            1369,            5723,            3589,            1263,            3894,            843,            4410,            3548,            3474,            1836,            8518,            0,            2168,            3952,            976,            4884,            2486,            3633,            4496,            1610,            4940,            3962,            3255,            1174,            3963,            ],        [            1647,            582,            2203,            2308,            3893,            3916,            3861,            4183,            4198,            2839,            1491,            3688,            1148,            3107,            1217,            3732,            2044,            2219,            4148,            1867,            4103,            389,            2699,            2977,            2005,            3937,            1647,            4614,            1547,            2182,            3045,            2895,            2003,            1284,            2181,            2561,            3661,            918,            3445,            4062,            2383,            4029,            3743,            3050,            1399,            715,            2562,            4498,            2364,            1179,            2669,            2036,            3185,            2323,            2249,            1487,            7293,            1193,            0,            2727,            2169,            3659,            1261,            2408,            3271,            2803,            3715,            2737,            2030,            1356,            2738,            ],        [            2057,            2844,            4160,            1412,            3074,            3097,            2818,            3364,            3082,            2020,            3726,            2869,            2581,            2288,            2446,            2913,            4279,            4176,            2980,            4102,            3174,            2624,            4934,            837,            3962,            2776,            1386,            2424,            3148,            688,            855,            1734,            1098,            1893,            4138,            1067,            1471,            2519,            2284,            2901,            1171,            1839,            2575,            860,            3000,            2333,            4519,            3679,            1196,            2780,            1660,            4271,            1045,            533,            1088,            3088,            6474,            3428,            2235,            0,            4126,            2491,            1870,            1847,            2452,            4760,            2896,            1576,            1864,            2957,            1526,            ],        [            3106,            1591,            529,            3767,            5352,            5375,            5320,            5642,            5657,            4298,            993,            5147,            2607,            4566,            2676,            5191,            1546,            363,            5607,            1369,            5562,            2000,            2201,            4436,            1046,            5396,            3106,            6073,            2191,            3641,            4504,            4354,            3462,            2743,            1222,            4020,            5120,            1646,            4904,            5521,            3842,            5488,            5202,            4509,            2291,            2174,            822,            5957,            3823,            1497,            4128,            1538,            4644,            3782,            3708,            2070,            8752,            2105,            2402,            4186,            0,            5118,            2720,            3867,            4730,            644,            5174,            4196,            3489,            1408,            4197,            ],        [            1976,            3472,            4788,            1438,            1021,            1044,            313,            1272,            1000,            911,            4354,            816,            2500,            816,            2365,            1062,            4907,            4804,            898,            4730,            855,            3252,            5562,            2256,            4590,            2758,            2056,            2655,            3776,            1710,            1537,            799,            1519,            2521,            4766,            2089,            1728,            3147,            2073,            1673,            1911,            2096,            779,            2117,            3628,            2961,            5147,            1587,            1597,            3408,            1496,            4899,            2464,            1851,            1300,            3716,            4382,            4056,            2863,            1794,            4754,            0,            2498,            1610,            1261,            5388,            843,            1329,            2265,            3585,            1500,            ],        [            1090,            1519,            2835,            1047,            2632,            2655,            2600,            2922,            2937,            1578,            2401,            2427,            1115,            1846,            1479,            2471,            2954,            2851,            2887,            2777,            2842,            1299,            3609,            1716,            2637,            2676,            386,            3353,            1823,            921,            1784,            1634,            742,            568,            2813,            1300,            2400,            1194,            2184,            2801,            1122,            2768,            2482,            1789,            1675,            1008,            3194,            3237,            1103,            1455,            1611,            2946,            1924,            1062,            988,            1763,            6032,            2103,            910,            1466,            2801,            2398,            0,            1275,            2010,            3435,            2454,            1476,            897,            1632,            1477,            ],        [            963,            2258,            3879,            425,            2010,            2033,            1978,            2300,            2315,            956,            3167,            1805,            1487,            1224,            1352,            1849,            3720,            3895,            2264,            3543,            2220,            2065,            4375,            1849,            3921,            2718,            1400,            3486,            3107,            1054,            1917,            1549,            850,            1852,            3857,            1433,            2533,            2478,            2226,            2965,            1255,            2901,            2071,            1922,            2959,            2292,            4478,            2615,            1280,            2739,            1744,            3712,            2057,            1195,            1030,            3047,            5410,            2869,            1676,            1599,            4085,            2066,            1829,            0,            1388,            4719,            1832,            1518,            921,            2916,            1610,            ],        [            974,            2269,            4130,            436,            1483,            1506,            1451,            1773,            1788,            429,            3178,            1278,            1498,            697,            1363,            1322,            3731,            4146,            1737,            3554,            1693,            2076,            4386,            1860,            3932,            2729,            1411,            3497,            3118,            1065,            1928,            1560,            861,            1863,            4108,            1444,            2544,            2489,            2237,            2833,            1266,            2912,            1939,            1933,            2970,            2303,            4489,            2088,            1291,            2750,            1755,            3723,            2068,            1206,            1041,            3058,            4883,            2880,            1687,            1610,            4096,            1539,            1840,            608,            0,            4730,            1305,            1529,            932,            2927,            1621,            ],        [            3740,            2225,            1163,            4401,            5986,            6009,            5954,            6276,            6343,            4932,            1627,            5781,            3241,            5200,            3310,            5825,            2180,            997,            6241,            2003,            6196,            2634,            2835,            5070,            1680,            6030,            3740,            6707,            2825,            4275,            5138,            4988,            4096,            3377,            1856,            4654,            5754,            2280,            5538,            6155,            4476,            6122,            5836,            5143,            2925,            2808,            1077,            6591,            4457,            2131,            4762,            2172,            5278,            4416,            4342,            2704,            9386,            2739,            3036,            4820,            644,            5752,            3354,            4501,            5364,            0,            5808,            4830,            4123,            2042,            4831,            ],        [            2363,            3658,            5519,            1825,            178,            916,            1779,            778,            1198,            1039,            4567,            666,            2887,            1181,            2752,            732,            5120,            5535,            1147,            4943,            1103,            3465,            5775,            3249,            5321,            3493,            2800,            4147,            4507,            2454,            3003,            2265,            2250,            3252,            5497,            2833,            3194,            3878,            2808,            2408,            2655,            3562,            2481,            3322,            4359,            3692,            5878,            1331,            2680,            4139,            3144,            5112,            3457,            2595,            2430,            4447,            4126,            4269,            3076,            2999,            5485,            1867,            3229,            1997,            1389,            6119,            0,            2795,            2321,            4316,            3010,            ],        [            1695,            2739,            4055,            1050,            2712,            2735,            2319,            3002,            2832,            1658,            3621,            2507,            2219,            2245,            2084,            2551,            4174,            4071,            2730,            3997,            2924,            2519,            4829,            1686,            3857,            1516,            1281,            2356,            3043,            891,            1212,            474,            756,            1788,            4033,            1270,            1403,            2414,            1024,            1641,            1148,            1771,            2462,            1815,            2895,            2228,            4414,            3317,            834,            2675,            397,            4166,            1894,            1032,            537,            2983,            6112,            3323,            2130,            1492,            4021,            2006,            1765,            1485,            2090,            4655,            2534,            0,            1502,            2852,            263,            ],        [            1073,            2368,            4043,            349,            1934,            1957,            1902,            2224,            2239,            880,            3277,            1729,            1597,            1148,            1462,            1773,            3830,            4059,            2188,            3653,            2144,            2175,            4485,            1985,            3845,            2642,            1324,            3410,            3031,            978,            1841,            1473,            774,            1776,            4021,            1357,            2457,            2402,            2150,            2889,            1179,            2825,            1995,            1846,            2883,            2216,            4402,            2539,            1204,            2663,            1668,            3822,            1981,            1119,            954,            2971,            5334,            2979,            1786,            1523,            4009,            1990,            1753,            707,            1312,            4643,            1756,            1442,            0,            2840,            1534,            ],        [            1894,            463,            1471,            2371,            3956,            3979,            3924,            4246,            4324,            2902,            1177,            3751,            1395,            3170,            1464,            3795,            1730,            1487,            4222,            1553,            4166,            1798,            2385,            3040,            1273,            4000,            1710,            4677,            783,            2245,            3108,            2958,            2066,            1379,            1449,            2624,            3724,            486,            3508,            4125,            2446,            4092,            3806,            3113,            883,            962,            1830,            4561,            2427,            337,            2732,            1722,            3248,            2386,            2312,            662,            7356,            1903,            1190,            2790,            1437,            3733,            1324,            2599,            3334,            2071,            3778,            2800,            2221,            0,            2801,            ],        [            1689,            2476,            3792,            1044,            2706,            2729,            2461,            2996,            2725,            1652,            3358,            2501,            2213,            1920,            2078,            2545,            3911,            3808,            2623,            3734,            2817,            2256,            4566,            1423,            3594,            1726,            1018,            2566,            2780,            628,            1422,            684,            730,            1525,            3770,            1007,            1613,            2151,            1234,            1851,            995,            1981,            2207,            1662,            2632,            1965,            4151,            3311,            828,            2412,            134,            3903,            1631,            769,            720,            2720,            6106,            3060,            1867,            1339,            3758,            2134,            1502,            1479,            2084,            4392,            2528,            370,            1496,            2589,            0,            ],        ], 'starts': [16, 12, 51, 69, 50], 'ends': [16, 12, 51, 69, 50]},    'NW-20': {'distance_matrix': [        [            0,            218,            2720,            3094,            3380,            3578,            2852,            3091,            3575,            1257,            3099,            2989,            2664,            2765,            2547,            2483,            1548,            2783,            2895,            2055,            1229,            2563,            2430,            2194,            2630,            1727,            1872,            3360,            2630,            1290,            3245,            1225,            3611,            5532,            5984,            5450,            5305,            4617,            4219,            2198,            3318,            3761,            4868,            5057,            4076,            2722,            1829,            2760,            4590,            4121,            5722,            2449,            1497,            3209,            3002,            2552,            3958,            3113,            960,            2653,            5483,            2838,            1844,            766,            3327,            597,            570,            1300,            1172,            5002,            2588,            1328,            1891,            5430,            622,            1815,            6025,            3525,            3767,            2477,            742,            3330,            2794,            2268,            ],        [            218,            0,            2938,            3312,            3598,            3796,            3070,            3309,            3793,            1475,            3317,            3207,            2882,            2983,            2765,            2701,            1766,            3001,            3113,            2273,            1447,            2781,            2648,            2412,            2848,            1945,            2090,            3578,            2848,            1508,            3463,            1443,            3829,            5750,            6202,            5668,            5523,            4835,            4437,            2416,            3536,            3979,            5086,            5275,            4294,            2940,            2047,            2978,            4808,            4339,            5940,            2667,            1715,            3427,            3220,            2770,            4176,            3331,            742,            2871,            5701,            3056,            2062,            984,            3545,            815,            788,            1518,            954,            5220,            2806,            1110,            2109,            5648,            404,            2033,            6243,            3743,            3985,            2695,            960,            3548,            3012,            2486,            ],        [            2713,            2931,            0,            628,            3498,            1151,            3693,            842,            4416,            2098,            3940,            947,            3505,            967,            749,            261,            1919,            3624,            1234,            737,            1996,            3404,            3271,            3035,            433,            2568,            2639,            353,            1193,            1431,            4086,            1596,            4452,            2358,            2734,            2200,            2201,            1515,            3124,            2132,            3252,            2826,            3773,            3962,            2981,            22,            1946,            954,            3495,            3186,            2472,            1541,            2264,            697,            3843,            3393,            1709,            3954,            2948,            3494,            4388,            2772,            2685,            1947,            1078,            2116,            2143,            2067,            2954,            1898,            3132,            2697,            2435,            2180,            3335,            906,            2775,            1276,            4608,            2411,            1971,            383,            3635,            2812,            ],        [            3081,            3299,            628,            0,            3418,            916,            3254,            429,            4784,            2466,            3137,            105,            3873,            1595,            1377,            889,            1960,            2847,            233,            1038,            2364,            3772,            3639,            3403,            221,            2936,            3007,            275,            573,            2009,            4454,            1964,            4820,            1945,            2321,            1787,            1788,            1102,            2711,            1512,            2632,            2413,            3360,            3549,            2568,            650,            1326,            334,            3082,            2773,            2059,            921,            2632,            1325,            4211,            3761,            1296,            4322,            3316,            3055,            3975,            2152,            3053,            2315,            665,            2484,            2511,            2435,            3322,            1485,            2512,            3065,            1815,            1767,            3703,            1534,            2362,            863,            4976,            1791,            2339,            289,            4003,            2192,            ],        [            3385,            3603,            4159,            4533,            0,            4024,            1004,            3537,            3782,            2173,            281,            4428,            1616,            4204,            3986,            3922,            2987,            597,            4334,            3494,            2668,            2770,            1791,            1962,            3978,            2246,            3311,            3914,            4069,            2729,            3452,            2664,            1788,            3751,            4203,            3669,            3524,            2836,            2438,            3637,            1228,            778,            3231,            3420,            2439,            4161,            3268,            4199,            2953,            1138,            3941,            3888,            2936,            4648,            3209,            2759,            3992,            1290,            3620,            805,            3846,            920,            1843,            2619,            3773,            2788,            2815,            2739,            3626,            3221,            906,            3369,            1603,            3649,            4007,            3254,            4244,            3971,            1944,            1471,            2643,            3884,            1848,            1226,            ],        [            3477,            3695,            825,            828,            3166,            0,            3650,            398,            5180,            2862,            3533,            723,            4269,            1792,            1574,            1086,            2356,            3243,            1010,            1562,            2760,            4168,            4035,            3799,            536,            3332,            3403,            472,            969,            2405,            4850,            2360,            5216,            2026,            1950,            1416,            1869,            1183,            2792,            1908,            3028,            2494,            3441,            3630,            2649,            847,            1722,            730,            3163,            2854,            1688,            1317,            3028,            1113,            4607,            4157,            925,            4718,            3712,            3451,            4056,            2548,            3449,            2711,            294,            2880,            2907,            2831,            3718,            1566,            2908,            3461,            2211,            1396,            4099,            1731,            1991,            492,            5372,            2187,            2735,            857,            4399,            2588,            ],        [            3177,            3395,            3951,            4325,            1121,            4809,            0,            4322,            2683,            1965,            840,            4220,            612,            3996,            3778,            3714,            2779,            524,            4126,            3286,            2460,            1671,            787,            958,            3861,            1719,            3103,            4591,            3861,            2521,            2353,            2456,            784,            6346,            6798,            6264,            6119,            5431,            5033,            3429,            4549,            1899,            5046,            5235,            5034,            3953,            3060,            3991,            4768,            3815,            6536,            3680,            2728,            4440,            2110,            1660,            5189,            286,            3412,            316,            5661,            4069,            1374,            2411,            4558,            2580,            2607,            2531,            3418,            5816,            859,            3161,            1556,            6244,            3799,            3046,            6839,            4756,            940,            3708,            2435,            4561,            824,            1179,            ],        [            3079,            3297,            842,            430,            2768,            487,            3252,            0,            4782,            2464,            3135,            325,            3871,            1809,            1591,            1103,            1958,            2845,            612,            1579,            2362,            3770,            3637,            3401,            553,            2934,            3005,            489,            571,            2007,            4452,            1962,            4818,            1628,            1892,            1358,            1471,            785,            2394,            1510,            2630,            2096,            3043,            3232,            2251,            864,            1324,            332,            2765,            2456,            1630,            919,            2630,            1539,            4209,            3759,            867,            4320,            3314,            3053,            3658,            2150,            3051,            2313,            236,            2482,            2509,            2433,            3320,            1168,            2510,            3063,            1813,            1338,            3701,            1748,            1933,            434,            4974,            1789,            2337,            459,            4001,            2190,            ],        [            3700,            3918,            4474,            4848,            3748,            5332,            2835,            4845,            0,            2488,            3467,            4743,            2251,            4519,            4301,            4237,            3302,            3151,            4649,            3809,            2983,            1688,            2017,            1781,            4384,            2242,            3626,            5114,            4384,            3044,            1669,            2979,            3198,            7286,            7738,            7204,            7059,            6371,            5973,            3952,            5072,            5515,            6622,            6811,            5830,            4476,            3583,            4514,            6344,            5875,            7476,            4203,            3251,            4963,            1426,            1322,            5712,            2700,            3935,            2943,            7237,            4592,            2015,            2934,            5081,            3103,            3130,            3054,            3941,            6756,            3486,            3684,            3645,            7184,            4322,            3569,            7779,            5279,            3354,            4231,            2958,            5084,            2381,            3806,            ],        [            1212,            1430,            1986,            2360,            2720,            2844,            2192,            2357,            2915,            0,            2439,            2255,            2004,            2031,            1813,            1749,            814,            2123,            2161,            1321,            495,            1903,            1770,            1534,            1896,            1067,            1138,            2626,            1896,            556,            2585,            491,            2951,            4798,            5250,            4716,            4571,            3883,            3485,            1464,            2584,            3027,            4134,            4323,            3342,            1988,            1095,            2026,            3856,            3387,            4988,            1715,            763,            2475,            2342,            1892,            3224,            2453,            1447,            1993,            4749,            2104,            1184,            446,            2593,            615,            642,            566,            1453,            4268,            1854,            1196,            1157,            4696,            1834,            1081,            5291,            2791,            3107,            1743,            470,            2596,            2134,            1534,            ],        [            3104,            3322,            3878,            4252,            281,            4736,            723,            4249,            3501,            1892,            0,            4147,            1335,            3923,            3705,            3641,            2706,            316,            4053,            3213,            2387,            2489,            1510,            1681,            3788,            1965,            3030,            4518,            3788,            2448,            3171,            2383,            1507,            4032,            4484,            3950,            3805,            3117,            2719,            3356,            1509,            1059,            3512,            3701,            2720,            3880,            2987,            3918,            3234,            1419,            4222,            3607,            2655,            4367,            2928,            2478,            4273,            1009,            3339,            524,            4127,            1201,            1562,            2338,            4485,            2507,            2534,            2458,            3345,            3502,            625,            3088,            1322,            3930,            3726,            2973,            4525,            4683,            1663,            3635,            2362,            4488,            1567,            945,            ],        [            2976,            3194,            946,            105,            3313,            811,            3149,            324,            4679,            2361,            3032,            0,            3768,            1913,            1695,            1207,            1855,            2742,            128,            1143,            2259,            3667,            3534,            3298,            326,            2831,            2902,            593,            468,            1904,            4349,            1859,            4715,            1840,            2216,            1682,            1683,            997,            2606,            1407,            2527,            2308,            3255,            3444,            2463,            968,            1221,            229,            2977,            2668,            1954,            816,            2527,            1643,            4106,            3656,            1191,            4217,            3211,            2950,            3870,            2047,            2948,            2210,            560,            2379,            2406,            2330,            3217,            1380,            2407,            2960,            1710,            1662,            3598,            1852,            2257,            758,            4871,            1686,            2234,            563,            3898,            2087,            ],        [            2405,            2623,            3179,            3553,            1586,            4037,            1058,            3550,            2218,            1193,            1305,            3448,            0,            3224,            3006,            2942,            2007,            989,            3354,            2514,            1688,            1206,            322,            493,            3089,            1266,            2331,            3819,            3089,            1749,            1888,            1684,            1116,            5991,            6443,            5909,            5764,            5076,            4678,            2657,            3777,            4220,            5327,            5516,            4535,            3181,            2288,            3219,            5049,            4147,            6181,            2908,            1956,            3668,            1645,            1195,            4417,            618,            2640,            859,            5942,            3297,            863,            1639,            3786,            1808,            1835,            1759,            2646,            5461,            1324,            2389,            2021,            5889,            3027,            2274,            6484,            3984,            1272,            2936,            1663,            3789,            299,            1644,            ],        [            2758,            2976,            967,            1595,            4266,            1320,            3738,            1809,            4461,            2143,            3985,            1914,            3550,            0,            232,            730,            1964,            3669,            2201,            782,            2041,            3449,            3316,            3080,            1400,            2613,            1941,            1320,            2160,            1476,            4131,            1641,            4497,            3325,            3701,            3167,            3168,            2482,            4091,            2614,            3734,            3793,            4740,            4929,            3948,            969,            2245,            1921,            4462,            4153,            3439,            2508,            2122,            866,            3888,            3438,            2676,            3999,            2993,            3539,            5355,            3254,            2730,            1992,            2045,            2161,            2188,            2112,            2999,            2865,            3004,            2742,            2307,            3147,            3380,            951,            3742,            2243,            4653,            2893,            2016,            1350,            3680,            2684,            ],        [            2540,            2758,            749,            1377,            4048,            1088,            3520,            1591,            4243,            1925,            3767,            1696,            3332,            232,            0,            512,            1746,            3451,            1983,            564,            1823,            3231,            3098,            2862,            1182,            2395,            2173,            1102,            1942,            1258,            3913,            1423,            4279,            3107,            3483,            2949,            2950,            2264,            3873,            2396,            3516,            3575,            4522,            4711,            3730,            751,            2027,            1703,            4244,            3935,            3221,            2290,            2091,            634,            3670,            3220,            2458,            3781,            2775,            3321,            5137,            3036,            2512,            1774,            1827,            1943,            1970,            1894,            2781,            2647,            2786,            2524,            2089,            2929,            3162,            733,            3524,            2025,            4435,            2675,            1798,            1132,            3462,            2466,            ],        [            2476,            2694,            261,            889,            3984,            1204,            3456,            1103,            4179,            1861,            3703,            1208,            3268,            730,            512,            0,            1682,            3387,            1495,            500,            1759,            3167,            3034,            2798,            694,            2331,            2402,            614,            1454,            1194,            3849,            1359,            4215,            2619,            2995,            2461,            2462,            1776,            3385,            2393,            3513,            3087,            4034,            4223,            3242,            263,            1963,            1215,            3756,            3447,            2733,            1802,            2027,            750,            3606,            3156,            1970,            3717,            2711,            3257,            4649,            3033,            2448,            1710,            1339,            1879,            1906,            1830,            2717,            2159,            2722,            2460,            2025,            2441,            3098,            669,            3036,            1537,            4371,            2672,            1734,            644,            3398,            2402,            ],        [            1629,            1847,            2165,            1546,            1946,            2030,            1782,            1543,            3332,            1014,            1665,            1441,            2421,            2032,            1814,            1750,            0,            1375,            1347,            1322,            912,            2320,            2187,            1951,            1876,            1484,            1555,            1812,            1082,            557,            3002,            512,            2566,            3984,            4436,            3902,            3757,            3069,            2671,            650,            1770,            2213,            3320,            3509,            2528,            2187,            281,            1212,            3042,            2573,            4174,            901,            1180,            2862,            2759,            2309,            2410,            2068,            1864,            1583,            3935,            1290,            1601,            863,            1779,            1032,            1059,            983,            1870,            3454,            1040,            1613,            343,            3882,            2251,            1082,            4477,            1977,            2722,            929,            887,            1782,            2551,            720,            ],        [            2788,            3006,            3562,            3936,            597,            4420,            407,            3933,            3185,            1576,            316,            3831,            1019,            3607,            3389,            3325,            2390,            0,            3737,            2897,            2071,            2173,            1194,            1365,            3472,            1649,            2714,            4202,            3472,            2132,            2855,            2067,            1191,            6374,            6826,            6292,            6147,            5459,            5061,            3040,            4160,            1375,            5453,            5642,            4918,            3564,            2671,            3602,            5175,            1735,            6564,            3291,            2339,            4051,            2612,            2162,            4800,            693,            3023,            208,            6068,            3680,            1246,            2022,            4169,            2191,            2218,            2142,            3029,            5844,            335,            2772,            1032,            6272,            3410,            2657,            6867,            4367,            1347,            3319,            2046,            4172,            1251,            655,            ],        [            3531,            3749,            818,            724,            2852,            683,            4511,            196,            5234,            2916,            3133,            829,            4323,            1785,            1567,            1079,            2737,            3614,            0,            1555,            2814,            4222,            4089,            3853,            529,            3386,            3457,            465,            1340,            2249,            4904,            2414,            5876,            1712,            2088,            1554,            1555,            869,            2478,            2279,            2173,            2180,            3127,            3316,            2335,            840,            2093,            1101,            2849,            2540,            1826,            1688,            3082,            1515,            4661,            4211,            1063,            4772,            3766,            4312,            3742,            2532,            3503,            2765,            432,            2934,            2961,            2885,            3772,            1252,            3279,            3515,            2582,            1534,            4153,            1724,            2129,            630,            6032,            2558,            2789,            435,            4453,            2959,            ],        [            2098,            2316,            665,            1293,            3606,            1608,            3078,            1507,            3801,            1483,            3325,            1612,            2890,            710,            492,            428,            1304,            3009,            1899,            0,            1381,            2789,            2656,            2420,            1098,            1953,            2024,            1018,            1858,            816,            3471,            981,            3837,            3023,            3399,            2865,            2866,            2180,            3789,            1954,            3074,            3491,            4438,            4627,            3646,            667,            1585,            1619,            4160,            3851,            3137,            2206,            1649,            1154,            3228,            2778,            2374,            3339,            2333,            2879,            5053,            2594,            2070,            1332,            1743,            1501,            1528,            1452,            2339,            2563,            2344,            2082,            1647,            2845,            2720,            291,            3440,            1941,            3993,            2233,            1356,            1048,            3020,            2024,            ],        [            1580,            1798,            2428,            2802,            3088,            3286,            2560,            2799,            3283,            965,            2807,            2697,            2372,            2230,            2462,            2191,            1256,            2491,            2603,            1763,            0,            2271,            2138,            1902,            2338,            1435,            643,            3068,            2338,            998,            2953,            933,            3319,            5240,            5692,            5158,            5013,            4325,            3927,            1906,            3026,            3469,            4576,            4765,            3784,            2430,            1537,            2468,            4298,            3829,            5430,            2157,            268,            3096,            2710,            2260,            3666,            2821,            1356,            2361,            5191,            2546,            1552,            814,            3035,            983,            1010,            475,            1362,            4710,            2296,            1105,            1599,            5138,            1564,            1523,            5733,            3233,            3475,            2185,            838,            3038,            2502,            1976,            ],        [            2382,            2600,            3156,            3530,            2430,            4014,            1517,            3527,            1481,            1170,            2149,            3425,            933,            3201,            2983,            2919,            1984,            1833,            3331,            2491,            1665,            0,            699,            463,            3066,            924,            2308,            3796,            3066,            1726,            1151,            1661,            1880,            5968,            6420,            5886,            5741,            5053,            4655,            2634,            3754,            4197,            5304,            5493,            4512,            3158,            2265,            3196,            5026,            4557,            6158,            2885,            1933,            3645,            908,            458,            4394,            1382,            2617,            1625,            5919,            3274,            697,            1616,            3763,            1785,            1812,            1736,            2623,            5438,            2168,            2366,            2327,            5866,            3004,            2251,            6461,            3961,            2036,            2913,            1640,            3766,            1063,            2488,            ],        [            2639,            2857,            3413,            3787,            1731,            4271,            818,            3784,            2001,            1427,            1450,            3682,            234,            3458,            3240,            3176,            2241,            1134,            3588,            2748,            1922,            855,            0,            276,            3323,            1037,            2565,            4053,            3323,            1983,            1671,            1918,            1181,            6225,            6677,            6143,            5998,            5310,            4912,            2891,            4011,            4454,            5443,            5632,            4769,            3415,            2522,            3453,            5165,            4212,            6415,            3142,            2190,            3902,            1428,            978,            4651,            683,            2874,            926,            6058,            3531,            1097,            1873,            4020,            2042,            2069,            1993,            2880,            5695,            1469,            2623,            2166,            6123,            3261,            2508,            6718,            4218,            1337,            3170,            1897,            4023,            364,            1789,            ],        [            2219,            2437,            2993,            3367,            1967,            3851,            1054,            3364,            1725,            1007,            1686,            3262,            470,            3038,            2820,            2756,            1821,            1370,            3168,            2328,            1502,            713,            236,            0,            2903,            761,            2145,            3633,            2903,            1563,            1395,            1498,            1417,            5805,            6257,            5723,            5578,            4890,            4492,            2471,            3591,            4034,            5141,            5330,            4349,            2995,            2102,            3033,            4863,            4394,            5995,            2722,            1770,            3482,            1152,            702,            4231,            919,            2454,            1162,            5756,            3111,            534,            1453,            3600,            1622,            1649,            1573,            2460,            5275,            1705,            2203,            2402,            5703,            2841,            2088,            6298,            3798,            1573,            2750,            1477,            3603,            600,            2025,            ],        [            3412,            3630,            433,            221,            3209,            1040,            3585,            553,            5115,            2797,            3468,            326,            4204,            1400,            1182,            694,            1828,            3178,            454,            843,            2695,            4103,            3970,            3734,            0,            3267,            3338,            80,            904,            1340,            4785,            1505,            5151,            2069,            2445,            1911,            1912,            1226,            2835,            1843,            2963,            2537,            3484,            3673,            2692,            455,            1657,            665,            3206,            2897,            2183,            1252,            2963,            1130,            4542,            4092,            1420,            4653,            3647,            3386,            4099,            2483,            3384,            2646,            789,            2815,            2842,            2766,            3653,            1609,            2843,            3396,            2146,            1891,            4034,            815,            2486,            987,            5307,            2122,            2670,            94,            4334,            2523,            ],        [            1458,            1676,            2232,            2606,            2007,            3090,            1479,            2603,            2236,            246,            1726,            2501,            1097,            2277,            2059,            1995,            1060,            1410,            2407,            1567,            741,            1224,            1091,            855,            2142,            0,            1384,            2872,            2142,            802,            1906,            737,            2106,            5044,            5496,            4962,            4817,            4129,            3731,            1710,            2830,            3273,            4380,            4569,            3588,            2234,            1341,            2272,            4102,            3633,            5234,            1961,            1009,            2721,            1663,            1213,            3470,            1608,            1693,            1280,            4995,            2350,            505,            692,            2839,            861,            888,            812,            1699,            4514,            1745,            1442,            1403,            4942,            2080,            1327,            5537,            3037,            2262,            1989,            716,            2842,            1289,            2065,            ],        [            1999,            2217,            2554,            3182,            3507,            2907,            2979,            3218,            3702,            1384,            3226,            3116,            2791,            1587,            1819,            2317,            1675,            2910,            3022,            2369,            882,            2690,            2557,            2321,            2987,            1854,            0,            2907,            2757,            1417,            3372,            1352,            3738,            5659,            6111,            5577,            5432,            4744,            4346,            2325,            3445,            3888,            4995,            5184,            4203,            2556,            1956,            2887,            4717,            4248,            5849,            2576,            590,            2453,            3129,            2679,            4085,            3240,            1775,            2780,            5610,            2965,            1971,            1233,            3454,            1402,            1429,            894,            1781,            5129,            2715,            1524,            2018,            5557,            1983,            1942,            6152,            3652,            3894,            2604,            1257,            2937,            2921,            2395,            ],        [            3348,            3566,            353,            275,            3145,            976,            3521,            489,            5051,            2733,            3404,            594,            4140,            1320,            1102,            614,            2227,            3114,            881,            1090,            2631,            4039,            3906,            3670,            80,            3203,            3274,            0,            840,            1784,            4721,            1949,            5087,            2005,            2381,            1847,            1848,            1162,            2771,            1779,            2899,            2473,            3420,            3609,            2628,            375,            1593,            601,            3142,            2833,            2119,            1188,            2899,            1050,            4478,            4028,            1356,            4589,            3583,            3322,            4035,            2419,            3320,            2582,            725,            2751,            2778,            2702,            3589,            1545,            2779,            3332,            2082,            1827,            3970,            1259,            2422,            923,            5243,            2058,            2606,            30,            4270,            2459,            ],        [            2508,            2726,            1214,            595,            2845,            1079,            2681,            592,            4211,            1893,            2564,            490,            3300,            2181,            1963,            1475,            1387,            2274,            396,            2201,            1791,            3199,            3066,            2830,            925,            2363,            2434,            861,            0,            1436,            3881,            1391,            4247,            2108,            2484,            1950,            1951,            1265,            2960,            939,            2059,            2502,            3609,            3798,            2817,            1236,            753,            261,            3331,            2862,            2222,            348,            2059,            1911,            3638,            3188,            1459,            3749,            2743,            2482,            4224,            1579,            2480,            1742,            828,            1911,            1938,            1862,            2749,            1648,            1939,            2492,            1242,            1930,            3130,            1961,            2525,            1026,            4403,            1218,            1766,            831,            3430,            1619,            ],        [            1282,            1500,            1430,            2034,            2790,            2518,            2262,            2031,            2985,            667,            2509,            1929,            2074,            1475,            1257,            1193,            488,            2193,            1835,            765,            565,            1973,            1840,            1604,            1340,            1137,            1208,            1783,            1570,            0,            2655,            165,            3021,            4472,            4924,            4390,            4245,            3557,            3159,            1138,            2258,            2701,            3808,            3997,            3016,            1432,            769,            1700,            3530,            3061,            4662,            1389,            833,            1919,            2412,            1962,            2898,            2523,            1517,            2063,            4423,            1778,            1254,            516,            2267,            685,            712,            636,            1523,            3942,            1528,            1266,            831,            4370,            1904,            525,            4965,            2465,            3177,            1417,            540,            1813,            2204,            1208,            ],        [            4030,            4248,            4804,            5178,            4078,            5662,            3165,            5175,            330,            2818,            3797,            5073,            2581,            4849,            4631,            4567,            3632,            3481,            4979,            4139,            3313,            2018,            2347,            2111,            4714,            2572,            3956,            5444,            4714,            3374,            0,            3309,            3528,            7616,            8068,            7534,            7389,            6701,            6303,            4282,            5402,            5845,            6952,            7141,            6160,            4806,            3913,            4844,            6674,            6205,            7806,            4533,            3581,            5293,            1756,            1652,            6042,            3030,            4265,            3273,            7567,            4922,            2345,            3264,            5411,            3433,            3460,            3384,            4271,            7086,            3816,            4014,            3975,            7514,            4652,            3899,            8109,            5609,            3684,            4561,            3288,            5414,            2711,            4136,            ],        [            1315,            1533,            1673,            1869,            2269,            2353,            2105,            1866,            3018,            700,            1988,            1764,            2107,            1718,            1500,            1436,            323,            1698,            1670,            1008,            598,            2006,            1873,            1637,            2199,            1170,            1241,            2135,            1405,            243,            2688,            0,            3054,            4307,            4759,            4225,            4080,            3392,            2994,            973,            2093,            2536,            3643,            3832,            2851,            1675,            604,            1535,            3365,            2896,            4497,            1224,            866,            2162,            2445,            1995,            2733,            2556,            1550,            1906,            4258,            1613,            1287,            549,            2102,            718,            745,            669,            1556,            3777,            1363,            1299,            666,            4205,            1937,            768,            4800,            2300,            3210,            1252,            573,            2105,            2237,            1043,            ],        [            3501,            3719,            4275,            4649,            1658,            5133,            745,            4646,            3007,            2289,            1377,            4544,            936,            4320,            4102,            4038,            3103,            1061,            4450,            3610,            2784,            1995,            1111,            1282,            4185,            2043,            3427,            4915,            4185,            2845,            2677,            2780,            0,            5940,            6392,            5858,            5713,            5025,            4627,            3753,            4229,            4402,            4640,            4829,            4628,            4277,            3384,            4315,            4362,            3409,            6130,            4004,            3052,            4764,            2434,            1984,            6181,            841,            3736,            853,            5255,            4588,            1698,            2735,            4882,            2904,            2931,            2855,            3742,            5410,            1396,            3485,            2093,            5838,            4123,            3370,            6433,            5080,            534,            4032,            2759,            4885,            1148,            1716,            ],        [            4920,            5138,            3957,            3545,            1140,            3714,            4561,            3227,            6623,            4305,            1421,            3440,            4752,            4924,            4706,            4218,            3799,            1737,            3979,            4613,            4203,            5611,            5478,            5242,            3668,            4775,            4846,            3604,            3686,            3848,            6293,            3803,            4164,            0,            3893,            3359,            3214,            2526,            2128,            2732,            1730,            468,            2921,            3110,            2129,            3979,            3165,            3447,            2643,            828,            3631,            3533,            4471,            4654,            6050,            5600,            3682,            4657,            5155,            4669,            3536,            2089,            4892,            4154,            3463,            4323,            4350,            4274,            5161,            2911,            2046,            4904,            3654,            3339,            5542,            4373,            3934,            3661,            4320,            2640,            4178,            3574,            4964,            4031,            ],        [            6248,            6466,            3008,            2596,            3737,            2313,            7158,            2166,            7951,            5633,            4018,            2491,            7349,            3975,            3757,            3269,            5127,            6014,            2778,            3745,            5531,            6939,            6806,            6570,            2719,            6103,            6174,            2655,            2737,            5176,            7621,            5131,            6761,            2597,            0,            827,            1343,            1779,            3363,            4060,            3058,            3065,            4012,            4201,            3220,            3030,            4493,            2498,            3734,            3425,            992,            3085,            5799,            3094,            7378,            6928,            1299,            7254,            6483,            7266,            4627,            3417,            6220,            5482,            1930,            5651,            5678,            5602,            6489,            2162,            5679,            6232,            4982,            934,            6870,            3914,            227,            1732,            6917,            3968,            5506,            2625,            7170,            5359,            ],        [            5528,            5746,            2288,            1876,            3017,            1593,            6438,            1446,            7231,            4913,            3298,            1771,            6629,            3255,            3037,            2549,            4407,            5294,            2058,            3025,            4811,            6219,            6086,            5850,            1999,            5383,            5454,            1935,            2017,            4456,            6901,            4411,            6041,            1877,            534,            0,            623,            1059,            2643,            3340,            2338,            2345,            3292,            3481,            2500,            2310,            3773,            1778,            3014,            2705,            272,            2365,            5079,            2374,            6658,            6208,            579,            6534,            5763,            6546,            3907,            2697,            5500,            4762,            1210,            4931,            4958,            4882,            5769,            1442,            4959,            5512,            4262,            214,            6150,            3194,            575,            1012,            6197,            3248,            4786,            1905,            6450,            4639,            ],        [            7042,            7260,            3932,            3520,            4531,            3689,            7952,            3202,            8745,            6427,            4812,            3415,            8143,            4899,            4681,            4193,            5921,            6808,            3702,            4669,            6325,            7733,            7600,            7364,            3643,            6897,            6968,            3579,            3661,            5970,            8415,            5925,            7555,            3391,            2490,            2254,            0,            2573,            4157,            4854,            3852,            3859,            4806,            4995,            4014,            3954,            5287,            3422,            4528,            4219,            2228,            4009,            6593,            4629,            8172,            7722,            2577,            8048,            7277,            8060,            5421,            4211,            7014,            6276,            3438,            6445,            6472,            6396,            7283,            2956,            6473,            7026,            5776,            2234,            7664,            4838,            2100,            3010,            7711,            4762,            6300,            3549,            7964,            6153,            ],        [            5619,            5837,            2509,            2097,            3108,            2266,            6529,            1779,            7322,            5004,            3389,            1992,            6720,            3476,            3258,            2770,            4498,            5385,            2279,            3246,            4902,            6310,            6177,            5941,            2220,            5474,            5545,            2156,            2238,            4547,            6992,            4502,            6132,            1968,            1365,            831,            686,            0,            2734,            3431,            2429,            2436,            3383,            3572,            2591,            2531,            3864,            1999,            3105,            2796,            1103,            2586,            5170,            3206,            6749,            6299,            1154,            6625,            5854,            6637,            3998,            2788,            5591,            4853,            1785,            5022,            5049,            4973,            5860,            383,            5050,            5603,            4353,            811,            6241,            3415,            1406,            1587,            6288,            3339,            4877,            2126,            6541,            4730,            ],        [            4964,            5182,            1829,            1417,            2453,            1586,            5874,            1099,            6667,            4349,            2734,            1312,            6065,            2796,            2578,            2090,            3843,            4730,            1599,            2566,            4247,            5655,            5522,            5286,            1540,            4819,            4890,            1476,            1558,            3892,            6337,            3847,            5477,            1313,            1765,            1231,            1086,            398,            0,            2776,            1774,            1781,            2728,            2917,            1936,            1851,            3209,            1319,            2450,            2141,            1503,            1906,            4515,            2526,            6094,            5644,            1554,            5970,            5199,            5982,            3343,            2133,            4936,            4198,            1335,            4367,            4394,            4318,            5205,            783,            4395,            4948,            3698,            1211,            5586,            2735,            1806,            1533,            5633,            2684,            4222,            1446,            5886,            4075,            ],        [            2343,            2561,            2220,            1601,            2235,            2085,            2516,            1598,            4046,            1728,            2399,            1496,            3135,            2746,            2528,            2481,            1222,            2109,            1402,            2036,            1626,            3034,            2901,            2665,            1931,            2198,            2269,            1867,            1137,            1271,            3716,            1226,            4082,            3334,            3786,            3252,            3107,            2419,            2021,            0,            1120,            1563,            2670,            2859,            1878,            2242,            588,            1267,            2392,            1923,            3524,            956,            1894,            2917,            3473,            3023,            2465,            3584,            2578,            2317,            3285,            640,            2315,            1577,            1834,            1746,            1773,            1697,            2584,            2804,            1774,            2327,            1077,            3232,            2965,            1796,            3827,            2032,            4238,            279,            1601,            1837,            3265,            1454,            ],        [            3484,            3702,            2730,            2318,            1115,            2487,            4536,            2000,            5187,            2869,            1396,            2213,            4276,            3697,            3479,            2991,            2363,            3250,            2543,            3177,            2767,            4175,            4042,            3806,            2441,            3339,            3410,            2377,            2278,            2412,            4857,            2367,            4139,            2214,            2666,            2132,            1987,            1299,            901,            1296,            0,            443,            1550,            1739,            758,            2752,            1729,            2408,            1272,            803,            2404,            2097,            3035,            3427,            4614,            4164,            2455,            4725,            3719,            3458,            2165,            653,            3456,            2718,            2236,            2887,            2914,            2838,            3725,            1684,            2915,            3468,            2218,            2112,            4106,            2937,            2707,            2434,            4295,            1204,            2742,            2347,            4406,            2595,            ],        [            4452,            4670,            3489,            3077,            2107,            3246,            4093,            2759,            6155,            3837,            2388,            2972,            4284,            4456,            4238,            3750,            3331,            4409,            3511,            4145,            3735,            5893,            4459,            4630,            3200,            4307,            4378,            3136,            3218,            3380,            5825,            3335,            3696,            2973,            3425,            2891,            2746,            2058,            1660,            2264,            1262,            0,            2453,            2642,            1661,            3511,            2697,            2979,            2175,            360,            3163,            3065,            4003,            4186,            5582,            5132,            3214,            4189,            4687,            4201,            3068,            1621,            4424,            3686,            2995,            3855,            3882,            3806,            4693,            2443,            3883,            4436,            3186,            2871,            5074,            3905,            3466,            3193,            3852,            2172,            3710,            3106,            4496,            3563,            ],        [            4835,            5053,            3713,            3301,            2490,            3470,            5140,            2983,            6538,            4220,            2771,            3196,            5331,            4680,            4462,            3974,            3714,            4601,            3894,            4528,            4118,            5526,            5393,            5157,            3424,            4690,            4761,            3360,            3442,            3763,            6208,            3718,            4395,            3197,            3649,            3115,            2970,            2282,            1884,            2647,            1645,            1818,            0,            189,            869,            3735,            3080,            3203,            278,            2178,            3387,            3448,            4386,            4410,            5965,            5515,            3438,            5236,            5070,            5248,            615,            2004,            4807,            4069,            3219,            4238,            4265,            4189,            5076,            2667,            4266,            4819,            3569,            3095,            5457,            4288,            3690,            3417,            4117,            2555,            4093,            3330,            5757,            3946,            ],        [            5024,            5242,            3902,            3490,            2679,            3659,            5329,            3172,            6727,            4409,            2960,            3385,            5520,            4869,            4651,            4163,            3903,            4790,            4083,            4717,            4307,            5715,            5582,            5346,            3613,            4879,            4950,            3549,            3631,            3952,            6397,            3907,            4584,            3386,            3838,            3304,            3159,            2471,            2073,            2836,            1834,            2007,            189,            0,            1058,            3924,            3269,            3392,            467,            2367,            3576,            3637,            4575,            4599,            6154,            5704,            3627,            5425,            5259,            5437,            426,            2193,            4996,            4258,            3408,            4427,            4454,            4378,            5265,            2856,            4455,            5008,            3758,            3284,            5646,            4477,            3879,            3606,            4306,            2744,            4282,            3519,            5946,            4135,            ],        [            4213,            4431,            3091,            2679,            1868,            2848,            4386,            2361,            5916,            3598,            2149,            2574,            5005,            4058,            3840,            3352,            3092,            3979,            3272,            3906,            3496,            4904,            4771,            4535,            2802,            4068,            4139,            2738,            2820,            3141,            5586,            3096,            4457,            2575,            3027,            2493,            2348,            1660,            1262,            2025,            1023,            1196,            792,            981,            0,            3113,            2458,            2581,            514,            1556,            2765,            2826,            3764,            3788,            5343,            4893,            2816,            5454,            4448,            4187,            1407,            1382,            4185,            3447,            2597,            3616,            3643,            3567,            4454,            2045,            3644,            4197,            2947,            2473,            4835,            3666,            3068,            2795,            4179,            1933,            3471,            2708,            5135,            3324,            ],        [            2715,            2933,            22,            650,            3520,            935,            3695,            864,            4418,            2100,            3942,            969,            3507,            969,            751,            263,            1921,            3626,            1256,            739,            1998,            3406,            3273,            3037,            455,            2570,            2641,            375,            1215,            1433,            4088,            1598,            4454,            2380,            2756,            2222,            2223,            1537,            3146,            2154,            3274,            2848,            3795,            3984,            3003,            0,            2202,            976,            3517,            3208,            2494,            1563,            2266,            481,            3845,            3395,            1731,            3956,            2950,            3496,            4410,            2794,            2687,            1949,            1100,            2118,            2145,            2069,            2956,            1920,            2961,            2699,            2264,            2202,            3337,            908,            2797,            1298,            4610,            2433,            1973,            405,            3637,            2641,            ],        [            1978,            2196,            1884,            1265,            2315,            1749,            2151,            1262,            3681,            1363,            2034,            1160,            2770,            2381,            2163,            2145,            857,            1744,            1066,            1671,            1261,            2669,            2536,            2300,            1595,            1833,            1904,            1531,            801,            906,            3351,            861,            3717,            3703,            4155,            3621,            3476,            2788,            2390,            369,            1489,            1932,            3039,            3228,            2247,            1906,            0,            931,            2761,            2292,            3893,            620,            1529,            2581,            3108,            2658,            2129,            3219,            2213,            1952,            3654,            1009,            1950,            1212,            1498,            1381,            1408,            1332,            2219,            3173,            1409,            1962,            712,            3601,            2600,            1431,            4196,            1696,            3873,            648,            1236,            1501,            2900,            1089,            ],        [            2747,            2965,            953,            334,            3084,            818,            2920,            331,            4450,            2132,            2803,            229,            3539,            1920,            1702,            1214,            1626,            2513,            135,            1690,            2030,            3438,            3305,            3069,            664,            2602,            2673,            600,            239,            1675,            4120,            1630,            4486,            1847,            2223,            1689,            1690,            1004,            2613,            1178,            2298,            2315,            3262,            3451,            2470,            975,            992,            0,            2984,            2675,            1961,            587,            2298,            1650,            3877,            3427,            1198,            3988,            2982,            2721,            3877,            1818,            2719,            1981,            567,            2150,            2177,            2101,            2988,            1387,            2178,            2731,            1481,            1669,            3369,            1859,            2264,            765,            4642,            1457,            2005,            570,            3669,            1858,            ],        [            4557,            4775,            3435,            3023,            2212,            3192,            4862,            2705,            6260,            3942,            2493,            2918,            5053,            4402,            4184,            3696,            3436,            4323,            3616,            4250,            3840,            5248,            5115,            4879,            3146,            4412,            4483,            3082,            3164,            3485,            5930,            3440,            4117,            2919,            3371,            2837,            2692,            2004,            1606,            2369,            1367,            1540,            278,            467,            591,            3457,            2802,            2925,            0,            1900,            3109,            3170,            4108,            4132,            5687,            5237,            3160,            4958,            4792,            4970,            893,            1726,            4529,            3791,            2941,            3960,            3987,            3911,            4798,            2389,            3988,            4541,            3291,            2817,            5179,            4010,            3412,            3139,            3839,            2277,            3815,            3052,            5479,            3668,            ],        [            6279,            6497,            5316,            4904,            4646,            5073,            3733,            4586,            5995,            5277,            4365,            4799,            3924,            6283,            6065,            5577,            5158,            4049,            5338,            5972,            5562,            5533,            4099,            4270,            5027,            5031,            6205,            4963,            5045,            5207,            5665,            5162,            3336,            4800,            5252,            4718,            4573,            3885,            3487,            4091,            3089,            3262,            4280,            4469,            3488,            5338,            4524,            4806,            4002,            0,            4990,            4892,            5830,            6013,            5422,            4972,            5041,            3829,            6514,            3841,            4895,            3448,            4686,            5513,            4822,            5682,            5709,            5633,            6520,            4270,            4384,            6263,            5081,            4698,            6901,            5732,            5293,            5020,            3492,            3999,            5537,            4933,            4136,            4704,            ],        [            6258,            6476,            3018,            2606,            3747,            2323,            7168,            2176,            7961,            5643,            4028,            2501,            7359,            3985,            3767,            3279,            5137,            6024,            2788,            3755,            5541,            6949,            6816,            6580,            2729,            6113,            6184,            2665,            2747,            5186,            7631,            5141,            6771,            2607,            262,            837,            1353,            1789,            3373,            4070,            3068,            3075,            4022,            4211,            3230,            3040,            4503,            2508,            3744,            3435,            0,            3095,            5809,            3104,            7388,            6938,            1309,            7264,            6493,            7276,            4637,            3427,            6230,            5492,            1940,            5661,            5688,            5612,            6499,            2172,            5689,            6242,            4992,            944,            6880,            3924,            489,            1742,            6927,            3978,            5516,            2635,            7180,            5369,            ],        [            2160,            2378,            1635,            1016,            2497,            1500,            2333,            1013,            3863,            1545,            2216,            911,            2952,            2563,            2345,            1896,            1039,            1926,            817,            1853,            1443,            2851,            2718,            2482,            1346,            2015,            2086,            1282,            552,            1088,            3533,            1043,            3899,            2529,            2905,            2371,            3698,            3010,            2612,            591,            1711,            2154,            3261,            3450,            2469,            1657,            405,            682,            2983,            2514,            2643,            0,            1711,            2332,            3290,            2840,            1880,            3401,            2395,            2134,            3876,            1231,            2132,            1394,            1249,            1563,            1590,            1514,            2401,            3395,            1591,            2144,            894,            2351,            2782,            1613,            2946,            1447,            4055,            870,            1418,            1252,            3082,            1271,            ],        [            1409,            1627,            2257,            2631,            2917,            3115,            2389,            2628,            3112,            794,            2636,            2526,            2201,            2089,            2084,            2020,            1085,            2320,            2432,            1592,            292,            2100,            1967,            1731,            2167,            1264,            502,            2897,            2167,            827,            2782,            762,            3148,            5069,            5521,            4987,            4842,            4154,            3756,            1735,            2855,            3298,            4405,            4594,            3613,            2259,            1366,            2297,            4127,            3658,            5259,            1986,            0,            2746,            2539,            2089,            3495,            2650,            1185,            2190,            5020,            2375,            1381,            643,            2864,            812,            839,            304,            1191,            4539,            2125,            934,            1428,            4967,            1393,            1352,            5562,            3062,            3304,            2014,            667,            2867,            2331,            1805,            ],        [            3925,            4143,            1273,            1276,            3614,            454,            4098,            846,            5628,            3310,            3981,            1171,            4717,            1316,            1084,            1207,            2804,            3691,            1458,            1683,            3208,            4616,            4483,            4247,            984,            3780,            3851,            920,            1417,            2853,            5298,            2808,            5664,            2474,            2398,            1864,            2317,            1631,            3240,            2356,            3476,            2942,            3889,            4078,            3097,            1295,            2170,            1178,            3611,            3302,            2136,            1765,            3476,            0,            5055,            4605,            1373,            5166,            4160,            3899,            4504,            2996,            3897,            3159,            742,            3328,            3355,            3279,            4166,            2014,            3356,            3909,            2659,            1844,            4547,            1852,            2439,            940,            5820,            2635,            3183,            1305,            4847,            3036,            ],        [            1649,            1867,            3557,            3931,            3380,            4415,            2467,            3928,            1062,            2120,            3099,            3826,            1883,            3602,            3384,            3320,            2385,            2783,            3732,            2892,            2066,            1320,            1649,            1413,            3467,            1874,            2709,            4197,            3467,            2127,            732,            2062,            2830,            6369,            6821,            6287,            6142,            5454,            5056,            3035,            4155,            4598,            5705,            5894,            4913,            3559,            2666,            3597,            5427,            4958,            6559,            3286,            2334,            4046,            0,            954,            4795,            2332,            2609,            2575,            6320,            3675,            1647,            1603,            4164,            1434,            1407,            2137,            2821,            5839,            3118,            2977,            2728,            6267,            2271,            2652,            6862,            4362,            2986,            3314,            1579,            4167,            2013,            3105,            ],        [            2744,            2962,            3518,            3892,            2792,            4376,            1879,            3889,            1023,            1532,            2511,            3787,            1295,            3563,            3345,            3281,            2346,            2195,            3693,            2853,            2027,            732,            1061,            825,            3428,            1286,            2670,            4158,            3428,            2088,            693,            2023,            2242,            6330,            6782,            6248,            6103,            5415,            5017,            2996,            4116,            4559,            5666,            5855,            4874,            3520,            2627,            3558,            5388,            4919,            6520,            3247,            2295,            4007,            450,            0,            4756,            1744,            2979,            1987,            6281,            3636,            1059,            1978,            4125,            2147,            2174,            2098,            2985,            5800,            2530,            2728,            2689,            6228,            3366,            2613,            6823,            4323,            2398,            3275,            2002,            4128,            1425,            2850,            ],        [            3946,            4164,            1709,            1297,            2962,            1014,            4119,            867,            5649,            3331,            3243,            1192,            4738,            2676,            2458,            1970,            2825,            3712,            1479,            2446,            3229,            4637,            4504,            4268,            1420,            3801,            3872,            1356,            1438,            2874,            5319,            2829,            5986,            1822,            1025,            491,            828,            979,            2588,            2377,            2283,            2290,            3237,            3426,            2445,            1731,            2191,            1199,            2959,            2650,            763,            1786,            3497,            1795,            5076,            4626,            0,            5187,            4181,            3920,            3852,            2642,            3918,            3180,            631,            3349,            3376,            3300,            4187,            1362,            3377,            3930,            2680,            471,            4568,            2615,            1066,            433,            6142,            2656,            3204,            1326,            4868,            3057,            ],        [            3651,            3869,            4425,            4799,            1808,            5283,            895,            4796,            3157,            2439,            1527,            4694,            1086,            4470,            4252,            4188,            3253,            1211,            4600,            3760,            2934,            2145,            1261,            1432,            4335,            2193,            3577,            5065,            4335,            2995,            2827,            2930,            498,            6060,            6512,            5978,            5833,            5145,            4747,            3903,            4349,            4522,            4760,            4949,            4748,            4427,            3534,            4465,            4482,            3529,            6250,            4154,            3202,            4914,            2584,            2134,            6301,            0,            3886,            1003,            5375,            4708,            1848,            2885,            5032,            3054,            3081,            3005,            3892,            5530,            1546,            3635,            2243,            5958,            4273,            3520,            6553,            5230,            654,            4182,            2909,            5035,            1298,            1866,            ],        [            1064,            846,            2830,            3204,            3490,            3688,            2962,            3201,            3685,            1367,            3209,            3099,            2774,            3095,            2657,            2593,            1658,            2893,            3005,            2165,            865,            2673,            2540,            2304,            2740,            1837,            1508,            3470,            2740,            1400,            3355,            1335,            3721,            5642,            6094,            5560,            5415,            4727,            4329,            2308,            3428,            3871,            4978,            5167,            4186,            2832,            1939,            2870,            4700,            4231,            5832,            2559,            1133,            3319,            3112,            2662,            4068,            3223,            0,            2763,            5593,            2948,            1954,            1216,            3437,            1385,            1412,            936,            212,            5112,            2698,            368,            2001,            5540,            434,            1925,            6135,            3635,            3877,            2587,            1240,            3440,            2904,            2378,            ],        [            2658,            2876,            3432,            3806,            805,            4290,            199,            3803,            3055,            1446,            524,            3701,            811,            3477,            3259,            3195,            2260,            208,            3607,            2767,            1941,            2043,            986,            1157,            3342,            1519,            2584,            4072,            3342,            2002,            2725,            1937,            983,            6244,            6696,            6162,            6017,            5329,            4931,            2910,            4030,            1583,            5245,            5434,            4788,            3434,            2541,            3472,            4967,            1943,            6434,            3161,            2209,            3921,            2482,            2032,            4670,            485,            2893,            0,            5860,            3550,            1116,            1892,            4039,            2061,            2088,            2012,            2899,            5714,            543,            2642,            1240,            6142,            3280,            2527,            6737,            4237,            1139,            3189,            1916,            4042,            1121,            863,            ],        [            5684,            5902,            4562,            4150,            3339,            4319,            5989,            3832,            7387,            5069,            3620,            4045,            6180,            5529,            5311,            4823,            4563,            5450,            4743,            5377,            4967,            6375,            6242,            6006,            4273,            5539,            5610,            4209,            4291,            4612,            7057,            4567,            5244,            4046,            4498,            3964,            3819,            3131,            2733,            3496,            2494,            2667,            849,            660,            1718,            4584,            3929,            4052,            1127,            3027,            4236,            4297,            5235,            5259,            6814,            6364,            4287,            6085,            5919,            6097,            0,            2853,            5656,            4918,            4068,            5087,            5114,            5038,            5925,            3516,            5115,            5668,            4418,            3944,            6306,            5137,            4539,            4266,            4966,            3404,            4942,            4179,            6606,            4795,            ],        [            2831,            3049,            2708,            2089,            1619,            2573,            3004,            2086,            4534,            2216,            2887,            1984,            3623,            3234,            3016,            2969,            1710,            2597,            1890,            2524,            2114,            3522,            3389,            3153,            2419,            2686,            2757,            2355,            1625,            1759,            4204,            1714,            4643,            2718,            3170,            2636,            2491,            1803,            1405,            643,            504,            947,            2054,            2243,            1262,            2730,            1076,            1755,            1776,            1307,            2908,            1444,            2382,            3405,            3961,            3511,            2959,            4072,            3066,            2805,            2669,            0,            2803,            2065,            2322,            2234,            2261,            2185,            3072,            2188,            2262,            2815,            1565,            2616,            3453,            2284,            3211,            2520,            4799,            551,            2089,            2325,            3753,            1942,            ],        [            1861,            2079,            2635,            3009,            1830,            3493,            1302,            3006,            1939,            649,            1549,            2904,            920,            2680,            2462,            2398,            1463,            1233,            2810,            1970,            1144,            927,            662,            594,            2545,            403,            1787,            3275,            2545,            1205,            1609,            1140,            1929,            5447,            5899,            5365,            5220,            4532,            4134,            2113,            3233,            3676,            4783,            4972,            3991,            2637,            1744,            2675,            4505,            4036,            5637,            2364,            1412,            3124,            1366,            916,            3873,            1431,            2096,            1103,            5398,            2753,            0,            1095,            3242,            1264,            1291,            1215,            2102,            4917,            1568,            1845,            1806,            5345,            2483,            1730,            5940,            3440,            2085,            2392,            1119,            3245,            1112,            1888,            ],        [            766,            984,            1954,            2328,            2614,            2812,            2086,            2325,            2809,            491,            2333,            2223,            1898,            1999,            1781,            1717,            782,            2017,            2129,            1289,            463,            1797,            1664,            1428,            1864,            961,            1106,            2594,            1864,            524,            2479,            459,            2845,            4766,            5218,            4684,            4539,            3851,            3453,            1432,            2552,            2995,            4102,            4291,            3310,            1956,            1063,            1994,            3824,            3355,            4956,            1683,            731,            2443,            2236,            1786,            3192,            2347,            1726,            1887,            4717,            2072,            1078,            0,            2561,            169,            196,            534,            1938,            4236,            1822,            1164,            1125,            4664,            1388,            1049,            5259,            2759,            3001,            1711,            24,            2564,            2028,            1502,            ],        [            3315,            3533,            1078,            666,            3004,            383,            3488,            236,            5018,            2700,            3371,            561,            4107,            2045,            1827,            1339,            2194,            3081,            848,            1815,            2598,            4006,            3873,            3637,            789,            3170,            3241,            725,            807,            2243,            4688,            2198,            5054,            1864,            1656,            1122,            1707,            1021,            2630,            1746,            2866,            2332,            3279,            3468,            2487,            1100,            1560,            568,            3001,            2692,            1394,            1155,            2866,            1164,            4445,            3995,            631,            4556,            3550,            3289,            3894,            2386,            3287,            2549,            0,            2718,            2745,            2669,            3556,            1404,            2746,            3299,            2049,            1102,            3937,            1984,            1697,            198,            5210,            2025,            2573,            695,            4237,            2426,            ],        [            597,            815,            2123,            2497,            2783,            2981,            2255,            2494,            2978,            660,            2502,            2392,            2067,            2168,            1950,            1886,            951,            2186,            2298,            1458,            632,            1966,            1833,            1597,            2033,            1130,            1275,            2763,            2033,            693,            2648,            628,            3014,            4935,            5387,            4853,            4708,            4020,            3622,            1601,            2721,            3164,            4271,            4460,            3479,            2125,            1232,            2163,            3993,            3524,            5125,            1852,            900,            2612,            2405,            1955,            3361,            2516,            1557,            2056,            4886,            2241,            1247,            169,            2730,            0,            27,            703,            1769,            4405,            1991,            1333,            1294,            4833,            1219,            1218,            5428,            2928,            3170,            1880,            145,            2733,            2197,            1671,            ],        [            570,            788,            2150,            2524,            2810,            3008,            2282,            2521,            3005,            687,            2529,            2419,            2094,            2195,            1977,            1913,            978,            2213,            2325,            1485,            659,            1993,            1860,            1624,            2060,            1157,            1302,            2790,            2060,            720,            2675,            655,            3041,            4962,            5414,            4880,            4735,            4047,            3649,            1628,            2748,            3191,            4298,            4487,            3506,            2152,            1259,            2190,            4020,            3551,            5152,            1879,            927,            2639,            2432,            1982,            3388,            2543,            1530,            2083,            4913,            2268,            1274,            196,            2757,            27,            0,            730,            1742,            4432,            2018,            1360,            1321,            4860,            1192,            1245,            5455,            2955,            3197,            1907,            172,            2760,            2224,            1698,            ],        [            1483,            1501,            2331,            2705,            2991,            3189,            2463,            2702,            3186,            868,            2710,            2600,            2275,            2596,            2158,            2094,            1159,            2394,            2506,            1666,            366,            2174,            2041,            1805,            2241,            1338,            1009,            2971,            2241,            901,            2856,            836,            3222,            5143,            5595,            5061,            4916,            4228,            3830,            1809,            2929,            3372,            4479,            4668,            3687,            2333,            1440,            2371,            4201,            3732,            5333,            2060,            634,            2820,            2613,            2163,            3569,            2724,            881,            2264,            5094,            2449,            1455,            717,            2938,            886,            913,            0,            887,            4613,            2199,            630,            1502,            5041,            1089,            1426,            5636,            3136,            3378,            2088,            741,            2941,            2405,            1879,            ],        [            1265,            1047,            2618,            2992,            3278,            3476,            2750,            2989,            3473,            1155,            2997,            2887,            2562,            2883,            2445,            2381,            1446,            2681,            2793,            1953,            653,            2461,            2328,            2092,            2528,            1625,            1296,            3258,            2528,            1188,            3143,            1123,            3509,            5430,            5882,            5348,            5203,            4515,            4117,            2096,            3216,            3659,            4766,            4955,            3974,            2620,            1727,            2658,            4488,            4019,            5620,            2347,            921,            3107,            2900,            2450,            3856,            3011,            427,            2551,            5381,            2736,            1742,            1004,            3225,            1173,            1200,            724,            0,            4900,            2486,            156,            1789,            5328,            635,            1713,            5923,            3423,            3665,            2375,            1028,            3228,            2692,            2166,            ],        [            5236,            5454,            2126,            1714,            2725,            1883,            6146,            1396,            6939,            4621,            3006,            1609,            6337,            3093,            2875,            2387,            4115,            5002,            1896,            2863,            4519,            5927,            5794,            5558,            1837,            5091,            5162,            1773,            1855,            4164,            6609,            4119,            5749,            1585,            982,            448,            303,            767,            2351,            3048,            2046,            2053,            3000,            3189,            2208,            2148,            3481,            1616,            2722,            2413,            720,            2203,            4787,            2823,            6366,            5916,            771,            6242,            5471,            6254,            3615,            2405,            5208,            4470,            1402,            4639,            4666,            4590,            5477,            0,            4667,            5220,            3970,            428,            5858,            3032,            1023,            1204,            5905,            2956,            4494,            1743,            6158,            4347,            ],        [            3123,            3341,            3897,            4271,            906,            4755,            742,            4268,            3520,            1911,            625,            4166,            1354,            3942,            3724,            3660,            2725,            335,            4072,            3232,            2406,            2508,            1529,            1700,            3807,            1984,            3049,            4537,            3807,            2467,            3190,            2402,            1526,            6709,            7161,            6627,            6482,            5794,            5396,            3375,            4495,            1684,            5788,            5977,            5253,            3899,            3006,            3937,            5510,            2044,            6899,            3626,            2674,            4386,            2947,            2497,            5135,            1028,            3358,            543,            6403,            4015,            1581,            2357,            4504,            2526,            2553,            2477,            3364,            6179,            0,            3107,            697,            6607,            3745,            2992,            7202,            4702,            1682,            3654,            2381,            4507,            1586,            320,            ],        [            1614,            1199,            2462,            2836,            3122,            3320,            2594,            2833,            3317,            999,            2841,            2731,            2406,            2727,            2289,            2225,            1290,            2525,            2637,            1797,            497,            2305,            2172,            1936,            2372,            1469,            1140,            3102,            2372,            1032,            2987,            967,            3353,            5274,            5726,            5192,            5047,            4359,            3961,            1940,            3060,            3503,            4610,            4799,            3818,            2464,            1571,            2502,            4332,            3863,            5464,            2191,            765,            2951,            2744,            2294,            3700,            2855,            579,            2395,            5225,            2580,            1586,            848,            3069,            1017,            1044,            568,            585,            4744,            2330,            0,            1633,            5172,            787,            1557,            5767,            3267,            3509,            2219,            872,            3072,            2536,            2010,            ],        [            3820,            4038,            4594,            4968,            1603,            5452,            1439,            4965,            4217,            2608,            1322,            4863,            2051,            4639,            4421,            4357,            3422,            1032,            4769,            3929,            3103,            3205,            2226,            2397,            4504,            2681,            3746,            5234,            4504,            3164,            3887,            3099,            2223,            7406,            7858,            7324,            7179,            6491,            6093,            4072,            5192,            2381,            6485,            6674,            5950,            4596,            3703,            4634,            6207,            2741,            7596,            4323,            3371,            5083,            3644,            3194,            5832,            1725,            4055,            1240,            7100,            4712,            2278,            3054,            5201,            3223,            3250,            3174,            4061,            6876,            697,            3804,            0,            7304,            4442,            3689,            7899,            5399,            2379,            4351,            3078,            5204,            2283,            377,            ],        [            5459,            5677,            2219,            1807,            2948,            1524,            6369,            1377,            7162,            4844,            3229,            1702,            6560,            3186,            2968,            2480,            4338,            5225,            1989,            2956,            4742,            6150,            6017,            5781,            1930,            5314,            5385,            1866,            1948,            4387,            6832,            4342,            5972,            1808,            554,            20,            554,            990,            2574,            3271,            2269,            2276,            3223,            3412,            2431,            2241,            3704,            1709,            2945,            2636,            292,            2296,            5010,            2305,            6589,            6139,            510,            6465,            5694,            6477,            3838,            2628,            5431,            4693,            1141,            4862,            4889,            4813,            5700,            1373,            4890,            5443,            4193,            0,            6081,            3125,            595,            943,            6128,            3179,            4717,            1836,            6381,            4570,            ],        [            630,            412,            3350,            3724,            4010,            4208,            3482,            3721,            4205,            1887,            3729,            3619,            3294,            3395,            3177,            3113,            2178,            3413,            3525,            2685,            1203,            3193,            3060,            2824,            3260,            2357,            1846,            3990,            3260,            1920,            3875,            1855,            4241,            6162,            6614,            6080,            5935,            5247,            4849,            2828,            3948,            4391,            5498,            5687,            4706,            3352,            2459,            3390,            5220,            4751,            6352,            3079,            1471,            3839,            3632,            3182,            4588,            3743,            338,            3283,            6113,            3468,            2474,            1396,            3957,            1227,            1200,            1274,            550,            5632,            3218,            706,            2521,            6060,            0,            2445,            6655,            4155,            4397,            3107,            1372,            3960,            3424,            2898,            ],        [            1807,            2025,            905,            1533,            3315,            1848,            2787,            1747,            3510,            1192,            3034,            1852,            2599,            950,            732,            668,            1013,            2718,            2360,            240,            1090,            2498,            2365,            2129,            815,            1662,            1733,            1258,            2098,            525,            3180,            690,            3546,            3263,            3639,            3105,            3106,            2420,            3684,            1663,            2783,            3226,            4333,            4522,            3541,            907,            1294,            1859,            4055,            3586,            3377,            1914,            1358,            1394,            2937,            2487,            2614,            3048,            2042,            2588,            4948,            2303,            1779,            1041,            1983,            1210,            1237,            1161,            2048,            2803,            2053,            1791,            1356,            3085,            2429,            0,            3680,            2181,            3702,            1942,            1065,            1288,            2729,            1733,            ],        [            6021,            6239,            2781,            2369,            3510,            2086,            6931,            1939,            7724,            5406,            3791,            2264,            7122,            3748,            3530,            3042,            4900,            5787,            2551,            3518,            5304,            6712,            6579,            6343,            2492,            5876,            5947,            2428,            2510,            4949,            7394,            4904,            6534,            2370,            285,            600,            1116,            1552,            3136,            3833,            2831,            2838,            3785,            3974,            2993,            2803,            4266,            2271,            3507,            3198,            765,            2858,            5572,            2867,            7151,            6701,            1072,            7027,            6256,            7039,            4400,            3190,            5993,            5255,            1703,            5424,            5451,            5375,            6262,            1935,            5452,            6005,            4755,            707,            6643,            3687,            0,            1505,            6690,            3741,            5279,            2398,            6943,            5132,            ],        [            3513,            3731,            1276,            864,            3202,            581,            3686,            434,            5216,            2898,            3569,            759,            4305,            2243,            2025,            1537,            2392,            3279,            1046,            2013,            2796,            4204,            4071,            3835,            987,            3368,            3439,            923,            1005,            2441,            4886,            2396,            5252,            2062,            1458,            924,            1261,            1219,            2828,            1944,            3064,            2530,            3477,            3666,            2685,            1298,            1758,            766,            3199,            2890,            1196,            1353,            3064,            1362,            4643,            4193,            433,            4754,            3748,            3487,            4092,            2584,            3485,            2747,            198,            2916,            2943,            2867,            3754,            1602,            2944,            3497,            2247,            904,            4135,            2182,            1499,            0,            5408,            2223,            2771,            893,            4435,            2624,            ],        [            3793,            4011,            4567,            4941,            1950,            5425,            1037,            4938,            3299,            2581,            1669,            4836,            1228,            4612,            4394,            4330,            3395,            1353,            4742,            3902,            3076,            2287,            1403,            1574,            4477,            2335,            3719,            5207,            4477,            3137,            2969,            3072,            292,            6141,            6593,            6059,            5914,            5226,            4828,            4045,            4430,            4603,            4114,            4303,            4253,            4569,            3676,            4607,            3836,            3610,            6331,            4296,            3344,            5056,            2726,            2276,            6382,            1133,            4028,            1145,            4729,            4789,            1990,            3027,            5174,            3196,            3223,            3147,            4034,            5611,            1688,            3777,            2385,            6039,            4415,            3662,            6634,            5372,            0,            4324,            3051,            5177,            1440,            2008,            ],        [            2617,            2835,            2494,            1875,            1956,            2359,            2790,            1872,            4320,            2002,            2673,            1770,            3409,            3020,            2802,            2755,            1496,            2383,            1676,            2310,            1900,            3308,            3175,            2939,            2205,            2472,            2543,            2141,            1411,            1545,            3990,            1500,            4980,            3055,            3507,            2973,            2828,            2140,            1742,            429,            841,            1284,            2391,            2580,            1599,            2516,            862,            1541,            2113,            1644,            3245,            1230,            2168,            3191,            3747,            3297,            3296,            3858,            2852,            2591,            3006,            361,            2589,            1851,            2108,            2020,            2047,            1971,            2858,            2525,            2048,            2601,            1351,            2953,            3239,            2070,            3548,            2306,            5136,            0,            1875,            2111,            3539,            1728,            ],        [            742,            960,            1978,            2352,            2638,            2836,            2110,            2349,            2833,            515,            2357,            2247,            1922,            2023,            1805,            1741,            806,            2041,            2153,            1313,            487,            1821,            1688,            1452,            1888,            985,            1130,            2618,            1888,            548,            2503,            483,            2869,            4790,            5242,            4708,            4563,            3875,            3477,            1456,            2576,            3019,            4126,            4315,            3334,            1980,            1087,            2018,            3848,            3379,            4980,            1707,            755,            2467,            2260,            1810,            3216,            2371,            1702,            1911,            4741,            2096,            1102,            24,            2585,            145,            172,            558,            1914,            4260,            1846,            1188,            1149,            4688,            1364,            1073,            5283,            2783,            3025,            1735,            0,            2588,            2052,            1526,            ],        [            3318,            3536,            383,            289,            3115,            946,            3491,            459,            5021,            2703,            3374,            564,            4110,            1350,            1132,            644,            2197,            3084,            851,            1120,            2601,            4009,            3876,            3640,            94,            3173,            3244,            30,            810,            1814,            4691,            1979,            5057,            1975,            2351,            1817,            1818,            1132,            2741,            1749,            2869,            2443,            3390,            3579,            2598,            405,            1563,            571,            3112,            2803,            2089,            1158,            2869,            1080,            4448,            3998,            1326,            4559,            3553,            3292,            4005,            2389,            3290,            2552,            695,            2721,            2748,            2672,            3559,            1515,            2749,            3302,            2052,            1797,            3940,            1289,            2392,            893,            5213,            2028,            2576,            0,            4240,            2429,            ],        [            2873,            3091,            3647,            4021,            1367,            4505,            454,            4018,            2379,            1661,            1086,            3916,            308,            3692,            3474,            3410,            2475,            770,            3822,            2982,            2156,            1367,            483,            654,            3557,            1415,            2799,            4287,            3557,            2217,            2049,            2152,            817,            6379,            6831,            6377,            6232,            5544,            5146,            3125,            4245,            2145,            5079,            5268,            5067,            3649,            2756,            3687,            4801,            3848,            6649,            3376,            2424,            4136,            1806,            1356,            4885,            319,            3108,            562,            5694,            3765,            1070,            2107,            4254,            2276,            2303,            2227,            3114,            5929,            1105,            2857,            1802,            6357,            3495,            2742,            6872,            4452,            973,            3404,            2131,            4257,            0,            1425,            ],        [            3443,            3661,            4217,            4591,            1226,            5075,            1062,            4588,            3840,            2231,            945,            4486,            1674,            4262,            4044,            3980,            3045,            655,            4392,            3552,            2726,            2828,            1849,            2020,            4127,            2304,            3369,            4857,            4127,            2787,            3510,            2722,            1846,            7029,            7481,            6947,            6802,            6114,            5716,            3695,            4815,            2004,            6108,            6297,            5573,            4219,            3326,            4257,            5830,            2364,            7219,            3946,            2994,            4706,            3267,            2817,            5455,            1348,            3678,            863,            6723,            4335,            1901,            2677,            4824,            2846,            2873,            2797,            3684,            6499,            320,            3427,            377,            6927,            4065,            3312,            7522,            5022,            2002,            3974,            2701,            4827,            1906,            0,            ],        ], 'starts': [8, 33, 70, 80, 54], 'ends': [8, 33, 70, 80, 54]},    'NW-21': {'distance_matrix': [        [            0,            582,            2171,            4683,            4413,            6971,            5577,            5123,            4758,            6391,            5609,            1600,            3192,            2693,            3395,            3065,            3872,            6314,            2832,            3799,            5530,            2967,            3270,            2719,            2161,            3523,            5818,            8444,            3379,            4102,            5028,            2254,            2562,            5054,            5863,            3621,            4062,            5306,            2965,            4232,            5668,            5565,            5766,            5338,            4596,            5013,            5991,            6283,            4037,            4469,            5240,            4371,            4841,            4012,            5225,            8542,            2539,            4403,            5490,            4393,            6375,            5262,            4827,            4546,            4518,            5647,            2846,            2016,            3657,            5920,            3900,            2704,            3855,            ],        [            510,            0,            2681,            4101,            4049,            6389,            4995,            4541,            4176,            5809,            5027,            1018,            2610,            3203,            2813,            2483,            3290,            6824,            2250,            3217,            4706,            2385,            2688,            2137,            2671,            4033,            4994,            7862,            3889,            3520,            4204,            1672,            1980,            4472,            5281,            3039,            3480,            4724,            3475,            3650,            5086,            4983,            5184,            4756,            4014,            4431,            5409,            5669,            3455,            3887,            4658,            3789,            4259,            3430,            4643,            7960,            2421,            4913,            4908,            3811,            5793,            4680,            4245,            3964,            3936,            5065,            2264,            1434,            3075,            5306,            4410,            2122,            3273,            ],        [            2090,            2543,            0,            3690,            3066,            5978,            4584,            4130,            3765,            5398,            4616,            3561,            1871,            1302,            2118,            1788,            2879,            4967,            2836,            2806,            4183,            3033,            3336,            2723,            814,            2176,            4471,            5939,            2032,            2825,            3681,            2381,            2073,            4061,            5448,            2628,            3069,            4313,            1618,            3239,            4675,            4572,            4773,            4345,            3603,            4020,            4998,            5836,            3044,            3476,            4247,            3378,            3848,            3019,            4232,            7549,            1148,            3056,            4497,            3400,            5382,            4269,            3834,            3553,            3525,            4654,            1569,            2756,            2380,            5473,            2553,            2708,            2578,            ],        [            3286,            3739,            1196,            0,            396,            2848,            4060,            3606,            3241,            4874,            2504,            2468,            1867,            2498,            628,            958,            2128,            5425,            2832,            2076,            2160,            3029,            3332,            2719,            1234,            668,            2448,            3916,            1416,            263,            1658,            1769,            1461,            3800,            4529,            2294,            2545,            4052,            1002,            3235,            4414,            4311,            4512,            4084,            2598,            3759,            4474,            4839,            2520,            2952,            3723,            2854,            3324,            2142,            3708,            5437,            2344,            1436,            4236,            2876,            4858,            4008,            3310,            3029,            3264,            4130,            1177,            2144,            366,            4365,            888,            2704,            168,            ],        [            4340,            4793,            2250,            1376,            0,            2452,            3664,            3210,            2845,            4478,            2108,            3901,            2692,            3552,            2004,            2334,            1732,            6020,            3236,            1680,            1764,            2965,            3268,            3123,            2288,            1722,            2052,            3520,            860,            1639,            1262,            3202,            2894,            3404,            4133,            1898,            2149,            3656,            2027,            3171,            4018,            3915,            4116,            3688,            2202,            3363,            4078,            4443,            2124,            2556,            3327,            2458,            2928,            1746,            3312,            5041,            3171,            1040,            3840,            2480,            4462,            3612,            2914,            2633,            2868,            3734,            3014,            3577,            1742,            3969,            492,            3108,            1544,            ],        [            6688,            6059,            3777,            2581,            2529,            0,            3663,            4146,            4176,            3276,            3107,            5077,            3868,            4519,            3209,            4409,            2908,            4803,            4412,            3999,            885,            4141,            4444,            4299,            3815,            3249,            400,            1514,            2844,            2844,            1231,            4378,            4070,            4622,            5351,            3821,            3367,            4874,            3583,            4347,            5236,            5133,            5334,            4906,            3420,            4581,            3586,            5661,            3342,            3774,            3742,            3676,            4146,            3317,            4204,            2054,            4347,            1919,            5058,            3698,            2911,            4830,            4132,            3851,            4086,            3242,            4190,            4753,            2947,            5187,            2476,            4284,            2749,            ],        [            8013,            7384,            5102,            3906,            3854,            2835,            0,            2033,            2063,            1163,            4432,            6402,            5193,            5844,            4534,            5734,            4233,            4773,            5737,            5324,            4185,            5466,            5769,            5624,            6841,            4574,            3235,            2839,            4714,            4169,            3683,            5703,            5395,            3422,            4383,            5146,            2863,            3674,            4908,            5672,            4036,            3933,            4134,            3706,            2916,            3381,            1473,            4771,            2640,            2574,            1629,            2306,            2166,            3334,            2091,            1978,            5672,            4894,            3858,            2624,            798,            3630,            1868,            2149,            2886,            1129,            5515,            6078,            4272,            3987,            4346,            5609,            4074,            ],        [            5548,            4919,            5207,            4011,            3959,            2940,            1711,            0,            704,            1268,            4537,            3937,            2728,            3379,            3599,            3269,            2010,            4878,            3272,            2859,            4290,            2997,            3300,            3159,            6946,            4679,            3340,            2944,            4819,            4274,            3788,            3238,            2930,            2063,            2792,            2681,            1504,            2315,            5013,            3203,            2677,            2574,            2775,            2347,            1557,            2022,            868,            3102,            1281,            1215,            1024,            947,            807,            1975,            732,            2083,            3207,            4999,            2499,            1265,            959,            2271,            1263,            1544,            1527,            524,            3050,            3613,            4377,            2628,            4451,            3144,            4179,            ],        [            4844,            4215,            3625,            2429,            2377,            3465,            979,            525,            0,            1793,            3965,            3233,            2024,            2675,            2895,            2565,            1306,            5403,            2568,            2155,            3034,            2293,            2596,            2455,            3748,            3097,            3865,            3469,            3237,            2692,            2532,            2534,            2226,            1359,            2088,            1977,            800,            1611,            3431,            2499,            1973,            1870,            2071,            1643,            853,            1318,            1393,            2398,            577,            511,            1549,            243,            243,            1271,            627,            2608,            2503,            3417,            1795,            561,            1777,            1567,            1145,            864,            823,            1049,            2346,            2909,            2795,            1924,            2869,            2440,            2597,            ],        [            6850,            6221,            3939,            2743,            2691,            1672,            2424,            2907,            2937,            0,            3269,            5239,            4030,            4681,            3371,            4571,            3070,            3610,            4574,            4161,            3022,            4303,            4606,            4461,            5678,            3411,            2072,            1676,            3551,            3006,            2520,            4540,            4232,            4784,            5712,            3983,            3529,            5036,            3745,            4509,            5398,            5295,            5496,            5068,            3790,            4743,            2347,            6100,            3504,            3936,            2503,            3180,            3040,            3479,            2965,            815,            4509,            3731,            5220,            3860,            1672,            4992,            2742,            4013,            4248,            2003,            4352,            4915,            3109,            5349,            3183,            4446,            2911,            ],        [            8009,            7380,            5098,            3902,            3850,            1362,            4542,            5025,            5055,            4155,            0,            6398,            5189,            5840,            4530,            5730,            4229,            5717,            5733,            5320,            2247,            5462,            5765,            5620,            7785,            4570,            1762,            2835,            4206,            4165,            2593,            5699,            5391,            5943,            7830,            5142,            4688,            6195,            4904,            5668,            6557,            6454,            6655,            6227,            5908,            5902,            4465,            8218,            4663,            5095,            4621,            5298,            5158,            4638,            5083,            2933,            5668,            3281,            6379,            5019,            3790,            6151,            4860,            5172,            5407,            4121,            5511,            6074,            4268,            6508,            3838,            5605,            4070,            ],        [            2351,            1722,            2658,            3083,            3031,            5371,            3977,            3523,            3158,            4791,            4009,            0,            1592,            1575,            1795,            1465,            2272,            6801,            1232,            2199,            3688,            1367,            1670,            1119,            2648,            2907,            3976,            6844,            3866,            2502,            3186,            654,            962,            3454,            4263,            2021,            2462,            3706,            3452,            2632,            4068,            3965,            4166,            3738,            2996,            3413,            4391,            4651,            2437,            2869,            3640,            2771,            3241,            2412,            3625,            6942,            1403,            4071,            3890,            2793,            4775,            3662,            3227,            2946,            2918,            4047,            1246,            416,            2057,            4288,            3523,            1104,            2255,            ],        [            2820,            2191,            1734,            2443,            1711,            4731,            3337,            2883,            2518,            4151,            3369,            1209,            0,            651,            871,            541,            1632,            5877,            1589,            1559,            3048,            1786,            2089,            1476,            1724,            1983,            3336,            6204,            2942,            1578,            2546,            510,            202,            2814,            4201,            1381,            1822,            3066,            2528,            1992,            3428,            3325,            3526,            3098,            2356,            2773,            3751,            4589,            1797,            2229,            3000,            2131,            2601,            1772,            2985,            6302,            479,            2751,            3250,            2153,            4135,            3022,            2587,            2306,            2278,            3407,            322,            885,            1133,            4226,            2203,            1461,            1331,            ],        [            2837,            3290,            1571,            2388,            2336,            4676,            3282,            2828,            2463,            4096,            3314,            1778,            569,            0,            816,            486,            1577,            5714,            1534,            1504,            2993,            1731,            2034,            1421,            1561,            1928,            3281,            6149,            2779,            1523,            2491,            1079,            771,            2759,            4146,            1326,            1767,            3011,            2365,            1937,            3373,            3270,            3471,            3043,            2301,            2718,            3696,            4534,            1742,            2174,            2945,            2076,            2546,            1717,            2930,            6247,            316,            3376,            3195,            2098,            4080,            2967,            2532,            2251,            2223,            3352,            267,            1454,            1078,            4171,            2828,            1406,            1276,            ],        [            3451,            2822,            1640,            2216,            840,            3292,            3952,            3498,            3133,            4766,            3984,            1840,            1239,            1282,            0,            330,            2247,            5869,            2204,            2174,            2604,            2401,            2704,            2091,            1678,            1112,            2892,            4360,            1860,            707,            2102,            1141,            833,            3429,            4816,            1996,            2437,            3681,            1446,            2607,            4043,            3940,            4141,            3713,            2971,            3388,            4366,            5204,            2412,            2844,            3615,            2746,            3216,            2387,            3600,            6917,            1110,            1880,            3865,            2768,            4750,            3637,            3202,            2921,            2893,            4022,            549,            1516,            262,            4841,            1332,            2076,            460,            ],        [            3121,            2492,            1970,            2728,            1170,            5016,            3622,            3168,            2803,            4436,            3654,            1510,            909,            952,            330,            0,            1917,            6178,            1874,            1844,            3333,            2071,            2374,            1761,            2025,            1442,            3621,            6489,            2190,            1037,            2831,            811,            503,            3099,            4486,            1666,            2107,            3351,            1776,            2277,            3713,            3610,            3811,            3383,            2641,            3058,            4036,            4874,            2082,            2514,            3285,            2416,            2886,            2057,            3270,            6587,            780,            2210,            3535,            2438,            4420,            3307,            2872,            2591,            2563,            3692,            219,            1186,            592,            4511,            1662,            1746,            790,            ],        [            3780,            3151,            3171,            1975,            1923,            4263,            2780,            2326,            1961,            3594,            2901,            2169,            960,            1611,            1831,            1501,            0,            6837,            2221,            1091,            2580,            1233,            1536,            2108,            2684,            2643,            2868,            5736,            2783,            2238,            2078,            1470,            1162,            2261,            3648,            913,            1265,            2513,            2977,            1439,            2875,            2772,            2973,            2545,            1803,            2220,            3194,            4036,            1240,            1672,            2443,            1574,            2044,            1215,            2428,            5834,            1439,            2963,            2697,            1596,            3578,            2469,            2030,            1749,            1725,            2850,            1282,            1845,            2093,            3673,            2415,            1376,            2143,            ],        [            8868,            8239,            5957,            4761,            4709,            2221,            5624,            6107,            6137,            5014,            859,            7257,            6048,            6699,            5389,            6589,            5088,            0,            7309,            6179,            3106,            6321,            6624,            7196,            8867,            5429,            2621,            3694,            5065,            5024,            3452,            6558,            6250,            6802,            8912,            6001,            5547,            7054,            5763,            6527,            7416,            7313,            7514,            7086,            6990,            6761,            5324,            9077,            5522,            5954,            5703,            6380,            6240,            5497,            5942,            3792,            6527,            4140,            7238,            5878,            4649,            7010,            5942,            6031,            6266,            5203,            6370,            6933,            5127,            7367,            4697,            6464,            4929,            ],        [            2469,            1840,            2677,            2434,            2382,            4722,            3328,            2874,            2509,            4142,            3360,            858,            943,            1594,            1814,            1484,            1623,            6820,            0,            1550,            3039,            718,            1021,            470,            2667,            3102,            3327,            6195,            3242,            2697,            2537,            381,            689,            2805,            3614,            1372,            1813,            3057,            3471,            1983,            3419,            3316,            3517,            3089,            2347,            2764,            3742,            4002,            1788,            2220,            2991,            2122,            2592,            1763,            2976,            6293,            1422,            3422,            3241,            2144,            4126,            3013,            2578,            2297,            2269,            3398,            1265,            534,            2076,            3639,            2874,            455,            2602,            ],        [            4170,            4128,            2080,            884,            832,            2177,            2909,            2455,            2090,            3723,            1833,            3146,            1937,            2588,            1512,            2478,            977,            6309,            3198,            0,            1489,            2210,            2513,            3085,            2118,            1552,            1777,            3245,            1692,            1147,            987,            2447,            2139,            2649,            3378,            853,            1394,            2901,            1886,            2416,            3263,            3160,            3361,            2933,            1447,            2608,            3323,            3688,            1369,            1801,            2572,            1703,            2173,            991,            2557,            4766,            2416,            1872,            3085,            1725,            3707,            2857,            2159,            1878,            2113,            2979,            2259,            2822,            1250,            3214,            1324,            2353,            1052,            ],        [            4650,            5103,            2560,            1364,            1312,            994,            3652,            3198,            2833,            3824,            1793,            3889,            2680,            3331,            1992,            2322,            1720,            5351,            3941,            1668,            0,            2953,            3256,            3828,            2598,            2032,            594,            2062,            2172,            1627,            346,            3190,            2882,            3392,            4121,            1886,            2137,            3644,            2366,            3159,            4006,            3903,            4104,            3676,            2190,            3351,            4066,            4431,            2112,            2544,            3315,            2446,            2916,            1734,            3300,            2602,            3159,            2352,            3828,            2468,            3459,            3600,            2902,            2621,            2856,            3722,            3002,            3565,            1730,            3957,            1804,            3096,            1532,            ],        [            3962,            3333,            3356,            2160,            2108,            4448,            2934,            2480,            2115,            3748,            3086,            2351,            1142,            1793,            2013,            1683,            1229,            9026,            1686,            1276,            2765,            0,            303,            1573,            2866,            2828,            3053,            5921,            2968,            2423,            2263,            1652,            1344,            2411,            3798,            1098,            1419,            2663,            3162,            1589,            3025,            2922,            3123,            2695,            1953,            2370,            3348,            4186,            1394,            1826,            2597,            1728,            2198,            1369,            2582,            6019,            1621,            3148,            2847,            1750,            3732,            2619,            2184,            1903,            1875,            3004,            1464,            2027,            2526,            3823,            2600,            1558,            2328,            ],        [            5143,            5101,            3053,            1857,            1805,            4145,            3882,            3428,            3063,            4696,            2783,            4119,            2910,            3561,            2485,            3451,            1950,            8723,            4171,            973,            2462,            3183,            0,            4058,            3091,            2525,            2750,            5618,            2665,            2120,            1960,            3420,            3112,            3622,            4351,            795,            2367,            3874,            2859,            3389,            4236,            4133,            4334,            3906,            2420,            3581,            4296,            4661,            2342,            2774,            3545,            2676,            3146,            1964,            3530,            5716,            3389,            2845,            4058,            2698,            4680,            3830,            3132,            2851,            3086,            3952,            3232,            3795,            2223,            4187,            2297,            3326,            2025,            ],        [            2582,            1953,            2790,            2547,            2495,            4835,            3441,            2987,            2622,            4255,            3473,            971,            1056,            1707,            1927,            1597,            1736,            6933,            113,            1663,            3152,            831,            1134,            0,            2780,            3215,            3440,            6308,            3355,            2810,            2650,            494,            802,            2918,            3727,            1485,            1926,            3170,            3584,            2096,            3532,            3429,            3630,            3202,            2460,            2877,            3855,            4115,            1901,            2333,            3104,            2235,            2705,            1876,            3089,            6406,            1535,            3535,            3354,            2257,            4239,            3126,            2691,            2410,            2382,            3511,            1378,            647,            2189,            3752,            2987,            342,            2715,            ],        [            3980,            4433,            1890,            2981,            2252,            4057,            5269,            4815,            4450,            6083,            3713,            5451,            4297,            3192,            3609,            3678,            3337,            6119,            5558,            3285,            3369,            4570,            4873,            4728,            0,            1362,            3657,            5125,            1218,            3244,            2867,            4807,            4499,            5009,            5738,            3503,            3754,            5261,            804,            4776,            5623,            5520,            5721,            5293,            3807,            4968,            5683,            6048,            3729,            4161,            4932,            4063,            4533,            3351,            4917,            6646,            3038,            2242,            5445,            4085,            6067,            5217,            4519,            4238,            4473,            5339,            3459,            5182,            3347,            5574,            1739,            4713,            3149,            ],        [            2618,            3071,            528,            3547,            2818,            6978,            5112,            4658,            4293,            5926,            5616,            4089,            2399,            1830,            2646,            2316,            3407,            4757,            3364,            3334,            3935,            3561,            3864,            3251,            566,            0,            4223,            5691,            1784,            1767,            3433,            2909,            2601,            4589,            5976,            3156,            3597,            4841,            1370,            3767,            5203,            5100,            5301,            4873,            4131,            4548,            5526,            6364,            3572,            4004,            4775,            3906,            4376,            3547,            4760,            8549,            1676,            2808,            5025,            3928,            5910,            4797,            4362,            4081,            4053,            5182,            2097,            3284,            2908,            6001,            2305,            3236,            1672,            ],        [            5135,            6013,            3045,            1849,            1797,            400,            3617,            4100,            4130,            3230,            2278,            5031,            3822,            4473,            2477,            4363,            2862,            4757,            4366,            2153,            485,            4095,            4398,            4253,            3083,            2517,            0,            1468,            2444,            2112,            831,            4332,            4024,            4576,            5305,            3775,            3321,            4828,            2851,            4301,            5190,            5087,            5288,            4860,            3374,            4535,            3540,            5615,            3296,            3728,            3696,            3630,            4100,            3271,            4158,            2008,            4301,            1519,            5012,            3652,            2865,            4784,            4086,            3805,            4040,            3196,            4144,            4707,            2215,            5141,            2076,            4238,            2017,            ],        [            4353,            4806,            2263,            1067,            1015,            2955,            3355,            2901,            2536,            4169,            1593,            3592,            2383,            3034,            1695,            2025,            1423,            7533,            2927,            1371,            1346,            2656,            2959,            2814,            2301,            1735,            1634,            0,            1875,            1330,            844,            2893,            2585,            3095,            3824,            1589,            1840,            3347,            2069,            2862,            3709,            3606,            3807,            3379,            1893,            3054,            3769,            4134,            1815,            2247,            3018,            2149,            2619,            1437,            3003,            4526,            2862,            2055,            3531,            2171,            4153,            3303,            2605,            2324,            2559,            3425,            2705,            3268,            1433,            3660,            1507,            2799,            1235,            ],        [            3885,            4338,            1795,            1763,            1034,            2839,            4051,            3597,            3232,            4865,            2495,            4288,            3079,            3097,            2391,            2721,            2119,            6024,            4340,            2067,            2151,            3352,            3655,            4227,            1833,            1267,            2439,            3907,            0,            2026,            1649,            3589,            3281,            3791,            4520,            2285,            2536,            4043,            1572,            3558,            4405,            4302,            4503,            4075,            2589,            3750,            4465,            4830,            2511,            2943,            3714,            2845,            3315,            2133,            3699,            5428,            2943,            1024,            4227,            2867,            4849,            3999,            3301,            3020,            3255,            4121,            3401,            3964,            2129,            4356,            521,            3495,            1931,            ],        [            3023,            3476,            933,            2199,            823,            3275,            4487,            4033,            3668,            5301,            2931,            4494,            2804,            2235,            1075,            1405,            2555,            5162,            3769,            2503,            2587,            3966,            4269,            3656,            971,            405,            2875,            4343,            1153,            0,            2085,            2216,            1908,            4227,            4956,            3561,            2972,            4479,            739,            4172,            4841,            4738,            4939,            4511,            3025,            4186,            4901,            5266,            2947,            3379,            4150,            3281,            3751,            2569,            4135,            5864,            2081,            1863,            4663,            3303,            5285,            4435,            3737,            3456,            3691,            4557,            1624,            2591,            813,            4792,            1315,            3641,            615,            ],        [            4473,            4926,            2383,            1187,            1135,            1190,            3475,            3021,            2656,            4020,            1616,            3712,            2503,            3154,            1815,            2145,            1543,            5547,            3764,            1491,            502,            2776,            3079,            3651,            2421,            1855,            790,            2258,            1995,            1450,            0,            3013,            2705,            3215,            3944,            1709,            1960,            3467,            2189,            2982,            3829,            3726,            3927,            3499,            2013,            3174,            3889,            4254,            1935,            2367,            3138,            2269,            2739,            1557,            3123,            2798,            2982,            2190,            3651,            2291,            3655,            3423,            2725,            2444,            2679,            3545,            2825,            3388,            1553,            3780,            1627,            2919,            1355,            ],        [            2310,            1681,            2004,            2651,            2599,            4939,            3545,            3091,            2726,            4359,            3577,            699,            1160,            921,            1141,            811,            1840,            6147,            800,            1767,            3256,            935,            1238,            687,            1994,            2253,            3544,            6412,            3212,            1848,            2754,            0,            308,            3022,            3831,            1589,            2030,            3274,            2798,            2200,            3636,            3533,            3734,            3306,            2564,            2981,            3959,            4219,            2005,            2437,            3208,            2339,            2809,            1980,            3193,            6510,            749,            3639,            3458,            2361,            4343,            3230,            2795,            2514,            2486,            3615,            592,            375,            1403,            3856,            3091,            672,            1601,            ],        [            2618,            1989,            1696,            2405,            2353,            4693,            3299,            2845,            2480,            4113,            3331,            1007,            586,            613,            833,            503,            1594,            5839,            1108,            1521,            3010,            1243,            1546,            995,            1686,            1945,            3298,            6166,            2904,            1540,            2508,            308,            0,            2776,            4163,            1343,            1784,            3028,            2490,            1954,            3390,            3287,            3488,            3060,            2318,            2735,            3713,            4551,            1759,            2191,            2962,            2093,            2563,            1734,            2947,            6264,            441,            3393,            3212,            2115,            4097,            2984,            2549,            2268,            2240,            3369,            284,            683,            1095,            4188,            2845,            980,            1293,            ],        [            5921,            5292,            4702,            3506,            3454,            5182,            2696,            2242,            1877,            3510,            5042,            4310,            3101,            3752,            3972,            3642,            2383,            7120,            3645,            3232,            4111,            2535,            2838,            3532,            4825,            4174,            4399,            5186,            4314,            3769,            3609,            3611,            3303,            0,            1478,            3054,            1877,            1001,            4508,            2615,            1363,            1260,            1461,            257,            1930,            468,            3110,            1788,            1654,            1250,            3266,            1490,            1960,            2348,            2344,            4325,            3580,            4494,            1185,            1824,            3494,            957,            2408,            2127,            1095,            2766,            3423,            3986,            3872,            1314,            3946,            3517,            3674,            ],        [            5790,            5161,            4947,            3861,            3809,            6237,            3402,            2597,            2232,            4565,            6339,            4179,            3456,            4107,            4327,            3997,            2738,            8175,            3321,            4529,            4466,            3253,            3556,            3208,            6119,            4529,            4754,            6241,            4669,            4124,            3964,            3702,            3658,            1645,            0,            4351,            2232,            602,            4863,            3586,            911,            808,            1062,            1346,            2285,            1021,            3465,            388,            2009,            1605,            3621,            1845,            2315,            2703,            2699,            5380,            3935,            4849,            786,            2179,            4200,            540,            2763,            2482,            1450,            3121,            3778,            3855,            4227,            583,            4301,            3116,            4029,            ],        [            4348,            4306,            2258,            1062,            1010,            3350,            3087,            2633,            2268,            3901,            1988,            3324,            2115,            2766,            1690,            2656,            1155,            7928,            2659,            178,            1667,            2388,            2691,            2546,            2296,            1730,            1955,            4823,            1870,            1325,            1165,            2625,            2317,            2827,            3556,            0,            1572,            3079,            2064,            2594,            3441,            3338,            3539,            3111,            1625,            2786,            3501,            3866,            1547,            1979,            2750,            1881,            2351,            1169,            2735,            4921,            2594,            2050,            3263,            1903,            3885,            3035,            2337,            2056,            2291,            3157,            2437,            3000,            1428,            3392,            1502,            2531,            1230,            ],        [            4292,            3663,            3073,            1877,            1825,            4775,            1763,            1309,            944,            2577,            3413,            2681,            1472,            2123,            2343,            2013,            754,            6187,            2016,            1603,            2482,            1741,            2044,            1903,            3196,            2545,            2770,            4253,            2685,            2140,            1980,            1982,            1674,            1503,            2232,            1425,            0,            1755,            2879,            1947,            2117,            2014,            2215,            1787,            301,            1462,            1612,            2542,            223,            655,            1178,            557,            1027,            719,            1411,            3392,            1951,            2865,            1939,            331,            2561,            1711,            765,            484,            967,            1833,            1794,            2357,            2243,            2068,            2317,            1888,            2045,            ],        [            5810,            5181,            4591,            3395,            3343,            5071,            2585,            2131,            1766,            3399,            4931,            4199,            2990,            3641,            3861,            3531,            2272,            7009,            3916,            3121,            4000,            2424,            2727,            3803,            4714,            4063,            4288,            5075,            4203,            3658,            3498,            3500,            3192,            1179,            595,            2943,            1766,            0,            4397,            2504,            343,            240,            460,            880,            1819,            555,            2999,            905,            1543,            1139,            3155,            1379,            1849,            2237,            2233,            4214,            3469,            4383,            184,            1713,            3383,            74,            2297,            2016,            984,            2655,            3312,            3875,            3761,            431,            3835,            3711,            3563,            ],        [            3176,            3629,            1086,            2177,            1448,            3253,            4465,            4011,            3646,            5279,            2909,            4647,            3493,            2388,            2805,            2874,            2533,            5315,            4037,            2481,            2565,            3766,            4069,            3924,            1124,            558,            2853,            4321,            414,            2440,            2063,            4003,            3695,            4205,            4934,            2699,            2950,            4457,            0,            3972,            4819,            4716,            4917,            4489,            3003,            4164,            4879,            5244,            2925,            3357,            4128,            3259,            3729,            2547,            4113,            5842,            2234,            1438,            4641,            3281,            5263,            4413,            3715,            3434,            3669,            4535,            2655,            4378,            2543,            4770,            935,            3909,            2345,            ],        [            3856,            3227,            4283,            3087,            3035,            5375,            3861,            3407,            3042,            4675,            4013,            2245,            2069,            2720,            2940,            2610,            2156,            7946,            1387,            2203,            3692,            927,            1230,            1274,            3793,            3755,            3980,            6848,            3895,            3350,            3190,            1768,            2271,            3338,            2218,            2025,            2346,            2681,            4089,            0,            2990,            2887,            3141,            3425,            2880,            3100,            4275,            2606,            2321,            2753,            3524,            2655,            3125,            2296,            3509,            6946,            2548,            4075,            2865,            2677,            4659,            2619,            3111,            2830,            2802,            3931,            2391,            1921,            3453,            2243,            3527,            1182,            3255,            ],        [            6161,            5532,            4942,            3746,            3694,            5422,            2936,            2482,            2117,            3750,            5282,            4550,            3341,            3992,            4212,            3882,            2623,            7360,            4200,            3472,            4351,            2775,            3078,            4087,            5065,            4414,            4639,            5426,            4554,            4009,            3849,            3851,            3543,            1530,            879,            3294,            2117,            343,            4748,            2855,            0,            103,            441,            1231,            2170,            906,            3350,            1189,            1894,            1490,            3506,            1730,            2200,            2588,            2584,            4565,            3820,            4734,            165,            2064,            3734,            425,            2648,            2367,            1335,            3006,            3663,            4226,            4112,            747,            4186,            3995,            3914,            ],        [            6058,            5429,            4839,            3643,            3591,            5319,            2833,            2379,            2014,            3647,            5179,            4447,            3238,            3889,            4109,            3779,            2520,            7257,            4097,            3369,            4248,            2672,            2975,            3984,            4962,            4311,            4536,            5323,            4451,            3906,            3746,            3748,            3440,            1427,            776,            3191,            2014,            240,            4645,            2752,            103,            0,            338,            1128,            2067,            803,            3247,            1086,            1791,            1387,            3403,            1627,            2097,            2485,            2481,            4462,            3717,            4631,            62,            1961,            3631,            322,            2545,            2264,            1232,            2903,            3560,            4123,            4009,            644,            4083,            3892,            3811,            ],        [            6298,            5669,            5079,            3883,            3831,            4890,            2055,            2882,            2912,            3218,            5419,            4687,            3478,            4129,            4349,            4019,            2760,            6828,            4404,            3609,            4488,            2912,            3215,            4291,            5202,            4551,            5290,            4894,            4691,            4146,            3986,            3988,            3680,            1667,            1083,            3431,            2254,            488,            4885,            2992,            469,            366,            0,            1368,            2307,            1043,            3528,            538,            2031,            1627,            3684,            1867,            2337,            2725,            2864,            4033,            3957,            4871,            304,            2201,            2853,            562,            2785,            2504,            1472,            3184,            3800,            4363,            4249,            919,            4323,            4199,            4051,            ],        [            5697,            5068,            4478,            3282,            3230,            4958,            2472,            2018,            1653,            3286,            4818,            4086,            2877,            3528,            3748,            3418,            2159,            6896,            3421,            3008,            3887,            2311,            2614,            3308,            4601,            3950,            4175,            4962,            4090,            3545,            3385,            3387,            3079,            274,            1811,            2830,            1653,            1334,            4284,            2391,            1696,            1593,            1794,            0,            1706,            742,            2886,            2121,            1430,            1026,            3042,            1266,            1736,            2124,            2120,            4101,            3356,            4270,            1518,            1600,            3270,            1290,            2184,            1903,            871,            2542,            3199,            3762,            3648,            1647,            3722,            3293,            3450,            ],        [            3991,            3362,            2772,            1576,            1524,            4474,            1888,            1434,            1069,            2702,            3112,            2380,            1171,            1822,            2042,            1712,            453,            6312,            1715,            1302,            2181,            1440,            1743,            1602,            2895,            2244,            2469,            4378,            2384,            1839,            1679,            1681,            1373,            1628,            2357,            1124,            373,            1880,            2578,            1646,            2242,            2139,            2340,            1912,            0,            1587,            2302,            2667,            348,            780,            1551,            682,            1152,            418,            1536,            3517,            1650,            2564,            2064,            704,            2686,            1836,            1138,            857,            1092,            1958,            1493,            2056,            1942,            2193,            2016,            1587,            1744,            ],        [            5453,            4824,            4234,            3038,            2986,            4714,            2228,            1774,            1409,            3042,            4574,            3842,            2633,            3284,            3504,            3174,            1915,            6652,            3177,            2764,            3643,            2067,            2370,            3064,            4357,            3706,            3931,            4718,            3846,            3301,            3141,            3143,            2835,            822,            1010,            2586,            1409,            533,            4040,            2147,            895,            792,            993,            565,            1462,            0,            2642,            1320,            1186,            782,            2798,            1022,            1492,            1880,            1876,            3857,            3112,            4026,            717,            1356,            3026,            489,            1940,            1659,            627,            2298,            2955,            3518,            3404,            846,            3478,            3049,            3206,            ],        [            6244,            5615,            5121,            3925,            3873,            2854,            1625,            1370,            1400,            1182,            4451,            4633,            3424,            4075,            4553,            3965,            4252,            4792,            3968,            3555,            4204,            3693,            3996,            3855,            6860,            4593,            3254,            2858,            4733,            4188,            3702,            3934,            3626,            2759,            3488,            3377,            1597,            3011,            4927,            3899,            3373,            3270,            3471,            3043,            2253,            2718,            0,            3798,            1977,            1911,            269,            1643,            1503,            2671,            1428,            1997,            3903,            4913,            3195,            1429,            873,            2967,            847,            1128,            2223,            438,            3746,            4309,            4291,            3324,            4365,            3840,            4093,            ],        [            6178,            5549,            5257,            4171,            4119,            6625,            3790,            2907,            2542,            4953,            6727,            4567,            3766,            4417,            4637,            4307,            3048,            8563,            3709,            4917,            4776,            3641,            3944,            3596,            6507,            4839,            5064,            6629,            4979,            4434,            4274,            4090,            3968,            1955,            388,            4739,            2542,            912,            5173,            3974,            1221,            1118,            1372,            1656,            2595,            1331,            3775,            0,            2319,            1915,            3931,            2155,            2625,            3013,            3009,            5768,            4245,            5159,            1096,            2489,            4588,            850,            3073,            2792,            1760,            3431,            4088,            4243,            4537,            893,            4611,            3504,            4339,            ],        [            4267,            3638,            3048,            1852,            1800,            4750,            1540,            1086,            721,            2354,            3388,            2656,            1447,            2098,            2318,            1988,            729,            5964,            1991,            1578,            2457,            1716,            2019,            1878,            3171,            2520,            2745,            4030,            2660,            2115,            1955,            1957,            1649,            1280,            2009,            1400,            223,            1532,            2854,            1922,            1894,            1791,            1992,            1564,            276,            1239,            1954,            2319,            0,            432,            2110,            334,            804,            694,            1188,            3169,            1926,            2840,            1716,            554,            2338,            1488,            988,            707,            744,            1610,            1769,            2332,            2218,            1845,            2292,            1863,            2020,            ],        [            4914,            4285,            3695,            2499,            2447,            4175,            1689,            1235,            870,            2503,            4035,            3303,            2094,            2745,            2965,            2635,            1376,            6113,            2638,            2225,            3104,            2363,            2666,            2525,            3818,            3167,            3392,            4179,            3307,            2762,            2602,            2604,            2296,            848,            1577,            2047,            870,            1100,            3501,            1848,            1462,            1359,            1560,            1132,            923,            807,            2103,            1887,            647,            0,            2259,            483,            953,            1341,            1337,            3318,            2573,            3487,            1284,            817,            2487,            1056,            1401,            1120,            312,            1759,            2416,            2979,            2865,            1413,            2939,            2510,            2667,            ],        [            6400,            5771,            5277,            4081,            4029,            3010,            1781,            1526,            1556,            1338,            4607,            4789,            3580,            4231,            4709,            4121,            4408,            4948,            4124,            3711,            4360,            3849,            4152,            4011,            7016,            4749,            3410,            3014,            4889,            4344,            3858,            4090,            3782,            2915,            3644,            3533,            1290,            3167,            5083,            4055,            3529,            3426,            3627,            3199,            2409,            2874,            269,            3954,            2133,            2067,            0,            1799,            1659,            2827,            1584,            2153,            4059,            5069,            3351,            1122,            1029,            3123,            540,            694,            2379,            594,            3902,            4465,            4447,            3480,            4521,            3996,            4249,            ],        [            4601,            3972,            3382,            2186,            2134,            3692,            1206,            752,            387,            2020,            3722,            2990,            1781,            2432,            2652,            2322,            1063,            5630,            2325,            1912,            2791,            2050,            2353,            2212,            3505,            2854,            3079,            3696,            2994,            2449,            2289,            2291,            1983,            1116,            1845,            1734,            557,            1368,            3188,            2256,            1730,            1627,            1828,            1400,            610,            1075,            1620,            2155,            334,            268,            1776,            0,            470,            1028,            854,            2835,            2260,            3174,            1552,            334,            2004,            1324,            918,            637,            580,            1276,            2103,            2666,            2552,            1681,            2626,            2197,            2354,            ],        [            5200,            4571,            3981,            2785,            2733,            3222,            736,            282,            356,            1550,            4819,            3589,            2380,            3031,            3251,            2921,            1662,            5160,            2924,            2511,            4572,            2649,            2952,            2811,            4104,            3453,            3622,            3226,            3593,            3048,            4070,            2890,            2582,            1715,            2444,            2333,            1156,            1967,            3787,            2855,            2329,            2226,            2427,            1999,            1209,            1674,            1150,            2754,            933,            867,            1306,            599,            0,            1627,            384,            2365,            2859,            3773,            2151,            917,            1534,            1923,            1545,            1220,            1179,            806,            2702,            3265,            3151,            2280,            3225,            2796,            2953,            ],        [            4741,            4112,            2354,            1158,            1106,            2451,            2223,            1769,            1404,            3037,            2107,            3130,            1921,            2572,            1786,            2462,            961,            6647,            2465,            909,            1763,            2194,            2497,            2352,            2392,            1826,            2051,            4713,            1966,            1421,            1261,            2431,            2123,            1963,            2692,            1127,            708,            2215,            2160,            2400,            2577,            2474,            2675,            2247,            761,            1922,            2637,            3002,            683,            1115,            1886,            1017,            1487,            0,            1871,            3852,            2400,            2146,            2399,            1039,            3021,            2171,            1473,            1192,            1427,            2293,            2243,            2806,            1524,            2528,            1598,            2337,            1326,            ],        [            5389,            4760,            4170,            2974,            2922,            3455,            889,            515,            545,            1783,            4510,            3778,            2569,            3220,            3440,            3110,            1851,            5393,            3113,            2700,            4805,            2838,            3141,            3000,            4293,            3642,            3855,            3459,            3782,            3237,            4303,            3079,            2771,            1904,            2633,            2522,            1345,            2156,            3976,            3044,            2518,            2415,            2616,            2188,            1398,            1863,            1383,            2943,            1122,            1056,            1539,            788,            648,            1816,            0,            2598,            3048,            3962,            2340,            1106,            1687,            2112,            1778,            1409,            1368,            1039,            2891,            3454,            3340,            2469,            3414,            2985,            3142,            ],        [            6966,            6337,            5161,            3965,            3913,            2894,            1609,            2092,            2122,            1222,            4491,            5355,            4146,            4797,            4593,            4687,            4292,            4832,            4690,            4277,            4244,            5525,            5828,            4577,            6900,            4633,            3294,            2898,            4773,            4228,            3742,            4656,            4348,            3481,            4897,            4099,            2922,            3733,            4967,            5731,            4095,            3992,            4193,            3765,            2975,            3440,            1532,            5285,            2699,            2633,            1688,            2365,            2225,            3393,            2150,            0,            4625,            4953,            3917,            2683,            857,            3689,            1927,            2208,            2945,            1188,            4468,            5031,            4331,            4046,            4405,            4562,            4133,            ],        [            2521,            2974,            1255,            2968,            3497,            5256,            3862,            3408,            3043,            4676,            3894,            1512,            1149,            1273,            1396,            1066,            2157,            5398,            1613,            2084,            4614,            1748,            2051,            1500,            1245,            2607,            4902,            6729,            2463,            2103,            4112,            813,            563,            3339,            4726,            1906,            2347,            3591,            2049,            2517,            3953,            3850,            4051,            3623,            2881,            3298,            4276,            5114,            2322,            2754,            3525,            2656,            3126,            2297,            3510,            6827,            0,            3487,            3775,            2678,            4660,            3547,            3112,            2831,            2803,            3932,            847,            1188,            1658,            4751,            2984,            1485,            1856,            ],        [            4405,            4858,            2315,            1799,            1070,            2875,            4087,            3633,            3268,            4901,            2531,            4324,            3115,            3617,            2427,            2757,            2155,            5191,            4376,            2103,            2187,            3388,            3691,            4263,            2353,            1787,            2475,            3943,            925,            2062,            1685,            3625,            3317,            3827,            4556,            2321,            2572,            4079,            2092,            3594,            4441,            4338,            4539,            4111,            2625,            3786,            4501,            4866,            2547,            2979,            3750,            2881,            3351,            2169,            3735,            5464,            3463,            0,            4263,            2903,            4885,            4035,            3337,            3056,            3291,            4157,            3437,            4000,            2165,            4392,            557,            3531,            1967,            ],        [            5994,            5365,            4775,            3579,            3527,            5255,            2769,            2315,            1950,            3583,            5115,            4383,            3174,            3825,            4045,            3715,            2456,            7193,            4100,            3305,            4184,            2608,            2911,            3987,            4898,            4247,            4472,            5259,            4387,            3842,            3682,            3684,            3376,            1363,            779,            3127,            1950,            184,            4581,            2688,            165,            62,            276,            1064,            2003,            739,            3183,            1089,            1727,            1323,            3339,            1563,            2033,            2421,            2417,            4398,            3653,            4567,            0,            1897,            3567,            258,            2481,            2200,            1168,            2839,            3496,            4059,            3945,            615,            4019,            3895,            3747,            ],        [            4610,            3981,            3391,            2195,            2143,            4010,            1524,            1070,            705,            2338,            3731,            2999,            1790,            2441,            2661,            2331,            1072,            5948,            2334,            1921,            2800,            2059,            2362,            2221,            3514,            2863,            3088,            4014,            3003,            2458,            2298,            2300,            1992,            1450,            2179,            1743,            318,            1702,            3197,            2265,            2064,            1961,            2162,            1734,            619,            1409,            1431,            2489,            668,            602,            997,            334,            788,            1037,            1172,            3153,            2269,            3183,            1886,            0,            2322,            1658,            584,            303,            914,            1594,            2112,            2675,            2561,            2015,            2635,            2206,            2363,            ],        [            6109,            5480,            4564,            3368,            3316,            2297,            1068,            1235,            1265,            625,            3894,            4498,            3289,            3940,            3996,            3830,            3695,            4235,            3930,            3420,            3647,            3558,            3861,            3817,            6303,            4036,            2697,            2301,            4176,            3631,            3145,            3799,            3491,            2624,            3353,            3242,            2065,            2876,            4370,            3764,            3238,            3135,            3336,            2908,            2118,            2583,            675,            3663,            1842,            1776,            831,            1508,            1368,            2536,            1293,            1440,            3768,            4356,            3060,            1826,            0,            2832,            1070,            1351,            2088,            331,            3611,            4174,            3734,            3189,            3808,            3705,            3536,            ],        [            5736,            5107,            4517,            3321,            3269,            4997,            2511,            2057,            1692,            3325,            4857,            4125,            2916,            3567,            3787,            3457,            2198,            6935,            3557,            3047,            3926,            2350,            2653,            3444,            4640,            3989,            4214,            5001,            4129,            3584,            3424,            3426,            3118,            1105,            826,            2869,            1692,            349,            4323,            2430,            711,            608,            809,            806,            1745,            481,            2925,            1136,            1469,            1065,            3081,            1305,            1775,            2163,            2159,            4140,            3395,            4309,            533,            1639,            3309,            0,            2223,            1942,            910,            2581,            3238,            3801,            3687,            662,            3761,            3332,            3489,            ],        [            5042,            4413,            3823,            2627,            2575,            3249,            2020,            1765,            1287,            1577,            4163,            3431,            2222,            2873,            3093,            2763,            1504,            5187,            2863,            2353,            3232,            2491,            2794,            2750,            3946,            3295,            3649,            3253,            3435,            2890,            2730,            2732,            2424,            2032,            2761,            2175,            750,            2284,            3629,            2697,            2646,            2543,            2744,            2316,            1051,            1991,            847,            3071,            973,            1184,            540,            916,            1370,            1469,            1823,            2392,            2701,            3615,            2468,            582,            1268,            2240,            0,            281,            1496,            833,            2544,            3107,            2993,            2597,            3067,            2638,            2795,            ],        [            4763,            4134,            3544,            2348,            2296,            3530,            1827,            1373,            1008,            1858,            3884,            3152,            1943,            2594,            2814,            2484,            1225,            5468,            2584,            2074,            2953,            2212,            2515,            2471,            3667,            3016,            3241,            3534,            3156,            2611,            2451,            2453,            2145,            1753,            2482,            1896,            471,            2005,            3350,            2418,            2367,            2264,            2465,            2037,            772,            1712,            1128,            2792,            694,            905,            694,            637,            1091,            1190,            1475,            2673,            2422,            3336,            2189,            303,            1549,            1961,            281,            0,            1217,            1114,            2265,            2828,            2714,            2318,            2788,            2359,            2516,            ],        [            5167,            4538,            3948,            2752,            2700,            4428,            1942,            1488,            1123,            2756,            4288,            3556,            2347,            2998,            3218,            2888,            1629,            6366,            2988,            2478,            3357,            1781,            2084,            2875,            4071,            3420,            3645,            4432,            3560,            3015,            2855,            2857,            2549,            536,            1265,            2300,            1123,            788,            3754,            1861,            1150,            1047,            1248,            820,            1176,            495,            2356,            1575,            900,            496,            2512,            736,            1206,            1594,            1590,            3571,            2826,            3740,            972,            1070,            2740,            744,            1654,            1373,            0,            2012,            2669,            3232,            3118,            1101,            3192,            2763,            2920,            ],        [            8577,            5625,            4683,            3487,            3435,            2416,            1187,            1380,            1410,            744,            4013,            4643,            3434,            4085,            4115,            3975,            3814,            4354,            4075,            3565,            3766,            5047,            5350,            3962,            6422,            4155,            2816,            2420,            4295,            3750,            3264,            3944,            3636,            2769,            4475,            3387,            2210,            3021,            4489,            5253,            3383,            3280,            3481,            3053,            2263,            2728,            820,            4863,            1987,            1921,            976,            1653,            1513,            2681,            1438,            1559,            3913,            4475,            3205,            1971,            435,            2977,            1215,            1496,            2233,            0,            3756,            4319,            3853,            3334,            3927,            3850,            3655,            ],        [            2902,            2273,            1816,            2509,            1389,            4797,            3403,            2949,            2584,            4217,            3435,            1291,            690,            733,            549,            219,            1698,            5959,            1752,            1625,            3114,            1852,            2155,            1639,            1806,            1661,            3402,            6270,            3024,            1256,            2612,            592,            284,            2880,            4267,            1447,            1888,            3132,            2610,            2058,            3494,            3391,            3592,            3164,            2422,            2839,            3817,            4655,            1863,            2295,            3066,            2197,            2667,            1838,            3051,            6368,            561,            2429,            3316,            2219,            4201,            3088,            2653,            2372,            2344,            3473,            0,            967,            811,            4292,            1881,            1527,            1009,            ],        [            1935,            1306,            2982,            3407,            3355,            5695,            4301,            3847,            3482,            5115,            4333,            324,            1916,            1899,            2119,            1789,            2596,            7765,            1653,            2523,            4012,            1691,            1994,            1540,            2972,            3231,            4300,            7168,            4190,            2826,            3510,            978,            1286,            3778,            4587,            2345,            2786,            4030,            3776,            2956,            4392,            4289,            4490,            4062,            3320,            3737,            4715,            4975,            2761,            3193,            3964,            3095,            3565,            2736,            3949,            7266,            1727,            4395,            4214,            3117,            5099,            3986,            3551,            3270,            3242,            4371,            1570,            0,            2381,            4612,            3847,            1428,            2579,            ],        [            3468,            3921,            1378,            1954,            578,            3030,            4242,            3788,            3423,            5056,            2686,            2102,            1501,            1544,            262,            592,            2310,            5607,            2563,            2436,            2342,            2663,            2966,            2450,            1416,            850,            2630,            4098,            1598,            445,            1840,            1403,            1095,            3691,            5078,            2258,            2727,            3943,            1184,            2869,            4305,            4202,            4403,            3975,            3233,            3650,            4656,            5466,            2702,            3134,            3905,            3036,            3506,            2324,            3890,            5619,            1372,            1618,            4127,            3058,            5040,            3899,            3492,            3211,            3155,            4312,            811,            1778,            0,            5103,            1070,            2338,            198,            ],        [            5824,            5195,            4893,            3697,            3645,            5373,            2887,            2433,            2068,            3701,            5233,            4213,            3292,            3943,            4163,            3833,            2574,            7311,            3355,            3423,            4302,            3287,            3590,            3242,            5016,            4365,            4590,            5377,            4505,            3960,            3800,            3736,            3494,            1481,            583,            3245,            2068,            438,            4699,            3620,            747,            644,            898,            1182,            2121,            857,            3301,            893,            1845,            1441,            3457,            1681,            2151,            2539,            2535,            4516,            3771,            4685,            622,            2015,            3685,            376,            2599,            2318,            1286,            2957,            3614,            3889,            4063,            0,            4137,            3150,            3865,            ],        [            3848,            4301,            1758,            1740,            1011,            2816,            4028,            3574,            3209,            4842,            2472,            5319,            3056,            3060,            2368,            3546,            2096,            5987,            3697,            2044,            2128,            3329,            3632,            3584,            1796,            1230,            2416,            3884,            368,            2003,            1626,            3566,            3258,            3768,            4497,            2262,            2513,            4020,            1535,            3535,            4382,            4279,            4480,            4052,            2566,            3727,            4442,            4807,            2488,            2920,            3691,            2822,            3292,            2110,            3676,            5405,            2906,            1001,            4204,            2844,            4826,            3976,            3278,            2997,            3232,            4098,            3327,            3941,            2106,            4333,            0,            3472,            1908,            ],        [            2674,            2045,            2882,            2639,            2587,            4927,            3356,            2902,            2537,            4170,            3565,            1063,            1148,            1799,            2019,            1689,            1651,            7025,            205,            1755,            3244,            422,            725,            92,            2872,            3307,            3532,            6400,            3447,            2902,            2742,            586,            1350,            3010,            3177,            1577,            1841,            3262,            3676,            2188,            3949,            3846,            3722,            3294,            2552,            2969,            3770,            3565,            1816,            2248,            3019,            2150,            2620,            1791,            3004,            6498,            1627,            3627,            3446,            2172,            4154,            3578,            2606,            2325,            2474,            3426,            1470,            739,            2281,            3202,            3079,            0,            2479,            ],        [            3270,            3723,            1180,            1756,            380,            2832,            4044,            3590,            3225,            4858,            2488,            2300,            1699,            2482,            460,            790,            2112,            5409,            2761,            2060,            2144,            2861,            3164,            2648,            1218,            652,            2432,            3900,            1400,            247,            1642,            1601,            1293,            3784,            4513,            2456,            2529,            4036,            986,            3067,            4398,            4295,            4496,            4068,            2582,            3743,            4458,            4823,            2504,            2936,            3707,            2838,            3308,            2126,            3692,            5421,            1570,            1420,            4220,            2860,            4842,            3992,            3294,            3013,            3248,            4114,            1009,            1976,            198,            4349,            872,            2536,            0,            ],        ], 'starts': [11, 42, 47, 20], 'ends': [11, 42, 47, 20]},    'NW-22': {'distance_matrix': [        [            0,            4735,            3956,            3135,            5532,            4349,            1686,            3162,            3765,            4047,            2923,            4605,            2418,            4474,            3435,            4891,            4731,            4636,            3864,            4723,            2665,            3532,            3913,            3517,            4295,            4950,            3088,            4106,            4796,            5324,            6021,            9260,            3889,            4015,            3848,            4615,            4275,            3721,            3524,            4546,            4297,            9242,            4470,            6366,            9013,            4202,            3899,            4692,            3975,            3748,            4560,            4407,            4335,            2897,            2553,            4048,            3612,            4593,            4569,            2552,            3038,            3743,            4959,            3611,            2535,            4495,            6856,            3052,            3132,            4321,            4900,            2810,            ],        [            2296,            0,            900,            2181,            798,            327,            2283,            1621,            998,            749,            1969,            1790,            3035,            1776,            4052,            1667,            2152,            2037,            4481,            1634,            3282,            4149,            4530,            4134,            4912,            1300,            3705,            4723,            1927,            5941,            6638,            9877,            4506,            4632,            2576,            1726,            2066,            1077,            4141,            2398,            2009,            9859,            1871,            6983,            9630,            1146,            933,            1558,            4592,            4365,            1781,            328,            4952,            3514,            3170,            750,            4229,            5210,            5186,            3169,            3655,            4360,            5576,            4228,            2927,            389,            7473,            2098,            2442,            355,            229,            3427,            ],        [            2207,            991,            0,            2092,            1822,            1267,            2194,            1532,            639,            965,            1880,            2814,            2946,            5002,            3963,            2691,            5259,            5164,            4392,            5251,            3193,            4060,            4441,            4045,            4823,            2324,            3616,            4634,            5324,            5852,            6549,            9788,            4417,            4543,            4376,            2750,            4803,            988,            4052,            5074,            4825,            9770,            4998,            6894,            9541,            246,            574,            2582,            4503,            4276,            2805,            663,            4863,            3425,            3081,            966,            4140,            5121,            5097,            3080,            3566,            4271,            5487,            4139,            2838,            1413,            7384,            2009,            2083,            1239,            1156,            3338,            ],        [            565,            2246,            1255,            0,            2203,            1648,            552,            709,            1894,            1346,            222,            3491,            1304,            3360,            2321,            3777,            3617,            3522,            2750,            3609,            1551,            2418,            2799,            2403,            3181,            3836,            1974,            2992,            3682,            4210,            4907,            8146,            2775,            2901,            2734,            3501,            3161,            1020,            2410,            3432,            3183,            8128,            3356,            5252,            7899,            1501,            1829,            3578,            2861,            2634,            3446,            1918,            3221,            1783,            1439,            1347,            2498,            3479,            3455,            1438,            1924,            2629,            3845,            2497,            1196,            1794,            5742,            351,            1819,            1620,            2411,            1696,            ],        [            2524,            798,            1128,            2409,            0,            555,            2511,            1849,            1767,            977,            2197,            1006,            1529,            978,            2292,            883,            1368,            1253,            1864,            850,            1850,            2203,            1913,            2004,            3481,            502,            2273,            1386,            1143,            4510,            5207,            8446,            2746,            2872,            1778,            928,            1268,            1305,            2381,            2250,            1211,            8428,            1073,            5552,            8199,            1374,            1702,            760,            1563,            2605,            983,            1126,            3521,            2082,            1738,            978,            2043,            3779,            3755,            1737,            2223,            1761,            4145,            2044,            3155,            409,            6042,            2326,            3211,            583,            1027,            1995,            ],        [            1969,            327,            573,            1854,            555,            0,            1956,            1294,            1212,            422,            1642,            1547,            2708,            1533,            3725,            1424,            1909,            1794,            4154,            1391,            2955,            3822,            4203,            3807,            4585,            1057,            3378,            4396,            1684,            5614,            6311,            9550,            4179,            4305,            4138,            1483,            1823,            750,            3814,            4836,            1766,            9532,            1628,            6656,            9303,            819,            1147,            1315,            4265,            4038,            1538,            655,            4625,            3187,            2843,            423,            3902,            4883,            4859,            2842,            3328,            4033,            5249,            3901,            2600,            146,            7146,            1771,            2656,            28,            556,            3100,            ],        [            1216,            3411,            2420,            1165,            4892,            2813,            0,            1874,            3059,            2511,            1387,            3965,            1778,            3834,            2795,            4251,            4091,            3996,            3224,            4083,            2025,            2892,            3273,            2877,            3655,            4310,            2448,            3466,            4156,            4684,            5381,            8620,            3249,            3375,            3208,            3975,            3635,            2185,            2884,            3906,            3657,            8602,            3830,            5726,            8373,            2666,            2994,            4052,            3335,            3108,            3920,            3083,            3695,            2257,            1913,            2512,            2972,            3953,            3929,            1912,            2398,            3103,            4319,            2971,            644,            2959,            6216,            1516,            4348,            2785,            3576,            2170,            ],        [            675,            2244,            1253,            560,            2201,            1646,            662,            0,            1892,            1344,            348,            3601,            1414,            3470,            2431,            3887,            3727,            3632,            2860,            3719,            1661,            2528,            2909,            2513,            3291,            3946,            2084,            3102,            3792,            4320,            5017,            8256,            2885,            3011,            2844,            3611,            3271,            1018,            2520,            3542,            3293,            8238,            3466,            5362,            8009,            1499,            1827,            3688,            2971,            2744,            3556,            1916,            3331,            1893,            1549,            1345,            2608,            3589,            3565,            1548,            2034,            2739,            3955,            2607,            1306,            1792,            5852,            477,            1929,            1618,            2409,            1806,            ],        [            2240,            1079,            907,            2125,            1855,            1300,            2227,            1565,            0,            998,            1913,            2847,            2979,            5035,            3996,            2724,            5292,            5197,            4425,            5284,            3226,            4093,            4474,            4078,            4856,            2357,            3649,            4667,            5357,            5885,            6582,            9821,            4450,            4576,            4409,            2783,            4836,            1021,            4085,            5107,            4858,            9803,            5031,            6927,            9574,            1153,            219,            2615,            4536,            4309,            2838,            751,            4896,            3458,            3114,            999,            4173,            5154,            5130,            3113,            3599,            4304,            5520,            4172,            2871,            1446,            7417,            2042,            1444,            1272,            1244,            3371,            ],        [            1708,            629,            312,            1593,            857,            302,            1695,            1033,            951,            0,            1381,            1849,            2447,            1835,            3464,            1726,            2211,            2096,            3893,            1693,            2694,            3561,            3942,            3546,            4324,            1359,            3117,            4135,            1986,            5353,            6050,            9289,            3918,            4044,            3877,            1785,            2125,            489,            3553,            4575,            2068,            9271,            1930,            6395,            9042,            558,            886,            1617,            4004,            3777,            1840,            576,            4364,            2926,            2582,            1,            3641,            4622,            4598,            2581,            3067,            3772,            4988,            3640,            2339,            448,            6885,            1510,            2395,            274,            858,            2839,            ],        [            914,            2024,            1033,            799,            1981,            1426,            901,            239,            1672,            1124,            0,            3840,            1653,            3709,            2670,            4126,            3966,            3871,            3099,            3958,            1900,            2767,            3148,            2752,            3530,            2483,            2323,            3341,            4031,            4559,            5256,            8495,            3124,            3250,            3083,            3850,            3510,            798,            2759,            3781,            3532,            8477,            3705,            5601,            8248,            1279,            1607,            3927,            3210,            2983,            3795,            1696,            3570,            2132,            1788,            1125,            2847,            3828,            3804,            1787,            2273,            2978,            4194,            2846,            1545,            1572,            6091,            716,            2168,            1398,            2189,            2045,            ],        [            2243,            1898,            2228,            2192,            1114,            1655,            2412,            2949,            2867,            2077,            2082,            0,            1346,            477,            1353,            473,            1496,            843,            925,            305,            1667,            1264,            974,            1065,            2215,            953,            1450,            531,            733,            3244,            3941,            7180,            1807,            1933,            839,            933,            1130,            2405,            1442,            1311,            300,            7162,            1078,            4286,            6933,            2474,            2802,            695,            1036,            1666,            988,            2226,            2255,            1899,            1555,            2078,            1104,            2513,            2489,            1554,            1953,            822,            2879,            1105,            3056,            1509,            4776,            1769,            3679,            1683,            2127,            1812,            ],        [            892,            2755,            1764,            841,            3346,            2157,            1061,            1218,            2403,            1855,            731,            2419,            0,            2288,            1249,            2705,            2545,            2450,            1678,            2537,            478,            1346,            1727,            1331,            2109,            2764,            901,            1920,            2610,            3138,            3835,            7074,            1703,            1829,            1662,            2429,            2089,            1529,            1338,            2360,            2111,            7056,            2284,            4180,            6827,            2010,            2338,            2506,            1789,            1562,            2374,            2427,            2149,            710,            366,            1856,            1426,            2407,            2383,            365,            851,            1557,            2773,            1425,            1705,            2303,            4670,            418,            2328,            2129,            2920,            623,            ],        [            2204,            1957,            2287,            2153,            1173,            1714,            2373,            3008,            2926,            2136,            2043,            541,            1307,            0,            1314,            827,            1457,            1197,            886,            659,            1628,            1225,            935,            1026,            2176,            476,            1411,            408,            1087,            3205,            3902,            7141,            1768,            1894,            800,            456,            759,            2464,            1403,            1272,            233,            7123,            601,            4247,            6894,            2533,            2861,            218,            585,            1627,            511,            2285,            2216,            1860,            1516,            2137,            1065,            2474,            2450,            1515,            1914,            783,            2840,            1066,            3017,            1568,            4737,            1730,            3640,            1742,            2186,            1773,            ],        [            2287,            3125,            3455,            2236,            2341,            2882,            2456,            2613,            4094,            3304,            2126,            1414,            1105,            1283,            0,            1700,            1540,            1445,            673,            1532,            1154,            341,            722,            326,            1106,            1759,            247,            915,            1605,            2135,            2832,            6071,            604,            730,            657,            1424,            1084,            2924,            307,            1355,            1106,            6053,            1279,            3177,            5824,            3701,            4029,            1501,            784,            463,            1369,            3453,            1146,            981,            1042,            3305,            421,            1404,            1380,            783,            750,            552,            1770,            420,            3100,            2736,            3667,            1813,            3723,            2910,            3354,            525,            ],        [            1543,            2103,            2433,            1492,            1319,            1860,            1712,            1869,            3072,            2282,            1382,            801,            646,            1278,            1738,            0,            485,            370,            2167,            645,            967,            1835,            2216,            1354,            2598,            1754,            1390,            1332,            260,            3627,            4324,            7563,            2192,            2318,            1640,            1734,            2578,            2180,            1827,            731,            1101,            7545,            1879,            4669,            7316,            2679,            3007,            1496,            2278,            2051,            1789,            2431,            2638,            1199,            855,            2283,            1915,            2896,            2872,            854,            1340,            1111,            3262,            1914,            2356,            1714,            5159,            1069,            2979,            1888,            2332,            1112,            ],        [            1370,            1908,            2238,            1319,            1124,            1665,            1539,            1696,            2877,            2087,            1209,            606,            473,            1083,            1565,            348,            0,            197,            1531,            450,            794,            1662,            1580,            1159,            2425,            1559,            1217,            1137,            65,            3454,            4151,            7390,            2019,            2145,            1445,            1539,            1736,            2415,            1654,            536,            906,            7372,            1684,            4496,            7143,            2484,            2812,            1301,            1642,            1878,            1594,            2236,            2465,            1026,            682,            2088,            1438,            2723,            2699,            681,            1167,            916,            3089,            1437,            2183,            1519,            4986,            896,            2806,            1693,            2137,            939,            ],        [            1173,            3036,            2045,            1292,            3465,            2438,            1342,            1499,            2684,            2136,            1012,            2538,            276,            2407,            1368,            2824,            2664,            0,            1797,            2656,            597,            1465,            1846,            1450,            2228,            2883,            1020,            2039,            2729,            3257,            3954,            7193,            1822,            1948,            1781,            2548,            2208,            1810,            1457,            2479,            2230,            7175,            2403,            4299,            6946,            2291,            2619,            2625,            1908,            1681,            2493,            2708,            2268,            829,            485,            2137,            1545,            2526,            2502,            484,            970,            1676,            2892,            1544,            1986,            2584,            4789,            699,            2609,            2410,            3201,            742,            ],        [            2479,            2692,            3351,            2598,            1908,            2449,            2648,            2557,            3990,            2871,            2318,            981,            1582,            850,            669,            1267,            1732,            1637,            0,            1099,            1673,            533,            289,            518,            1531,            1326,            766,            482,            1527,            2560,            3257,            6496,            1123,            1249,            224,            991,            651,            3116,            758,            1547,            673,            6478,            846,            3602,            6249,            3597,            3925,            1068,            351,            982,            936,            3020,            1571,            1500,            1561,            2872,            179,            1829,            1805,            1302,            1269,            744,            2195,            180,            3292,            2303,            4092,            2005,            3915,            2477,            2921,            1044,            ],        [            1711,            1593,            1923,            1830,            809,            1350,            1880,            1789,            2562,            1772,            1550,            442,            814,            919,            1906,            168,            653,            538,            1367,            0,            1135,            2003,            1416,            1507,            2766,            799,            1558,            973,            428,            3795,            4492,            7731,            2360,            2486,            1281,            1375,            1572,            2348,            1995,            899,            742,            7713,            1520,            4837,            7484,            2169,            2497,            1137,            1478,            2219,            1430,            1921,            2806,            1367,            1023,            1773,            2083,            3064,            3040,            1022,            1508,            1264,            3430,            2082,            2524,            1204,            5327,            1237,            3147,            1378,            1822,            1280,            ],        [            1324,            3187,            2196,            1391,            3399,            2589,            1493,            1402,            2835,            2287,            1163,            2472,            435,            2341,            1017,            2758,            2598,            2503,            1731,            2590,            0,            1399,            1780,            1384,            2313,            2817,            907,            1973,            2663,            3342,            4039,            7278,            775,            901,            1715,            2482,            2142,            1961,            1365,            2413,            2164,            7260,            2337,            4384,            7031,            2442,            2770,            2559,            1842,            1019,            2427,            2859,            2353,            232,            112,            2288,            1479,            2611,            2587,            371,            373,            1610,            2977,            1478,            2137,            2735,            4874,            850,            2760,            2561,            3352,            629,            ],        [            2305,            2784,            3177,            2372,            2000,            2541,            2474,            2383,            3816,            3268,            2144,            1073,            1408,            942,            483,            1359,            1558,            1729,            332,            1191,            1487,            0,            381,            344,            1345,            1418,            580,            574,            1623,            2374,            3071,            6310,            937,            1063,            316,            1083,            743,            2942,            572,            1373,            765,            6292,            938,            3416,            6063,            3423,            3751,            1160,            443,            796,            1028,            3112,            1385,            1314,            1375,            3269,            511,            1643,            1619,            1116,            1083,            570,            2009,            512,            3118,            2395,            3906,            1831,            3741,            2569,            3013,            858,            ],        [            2536,            2403,            2733,            2422,            1619,            2160,            2524,            2433,            3372,            2582,            2194,            692,            1458,            561,            1031,            978,            1608,            1348,            603,            810,            1779,            956,            0,            1177,            1893,            1037,            1128,            193,            1238,            2922,            3619,            6858,            1485,            1611,            517,            1017,            854,            2992,            1120,            1423,            384,            6840,            1162,            3964,            6611,            2979,            3307,            779,            680,            1344,            1072,            2731,            1933,            1862,            1667,            2583,            782,            2191,            2167,            1666,            1631,            934,            2557,            783,            3168,            2014,            4454,            1881,            3791,            2188,            2632,            1406,            ],        [            2349,            2983,            3040,            2235,            2199,            2740,            2337,            2246,            3679,            3131,            2007,            1272,            1271,            1141,            297,            1558,            1421,            1326,            458,            1390,            1301,            199,            580,            0,            1159,            1617,            394,            773,            1486,            2188,            2885,            6124,            751,            877,            515,            1282,            942,            2805,            386,            1236,            964,            6106,            1137,            3230,            5877,            3286,            3614,            1359,            642,            610,            1227,            3311,            1199,            1128,            1189,            3132,            279,            1457,            1433,            930,            897,            433,            1823,            278,            2981,            2594,            3720,            1694,            3604,            2768,            3212,            672,            ],        [            2679,            3720,            3551,            2798,            2936,            3477,            2848,            2757,            4190,            3642,            2518,            2009,            1985,            1878,            932,            2295,            2135,            2040,            1003,            2127,            1936,            936,            1317,            921,            0,            2354,            1029,            1510,            2200,            1832,            2529,            5768,            1386,            1512,            1227,            2019,            1679,            3316,            1021,            1950,            1701,            5750,            1874,            2874,            5521,            3797,            4125,            2096,            1379,            1245,            1964,            4214,            1834,            1763,            1824,            3643,            824,            1101,            1077,            1565,            1532,            1147,            1467,            823,            3492,            3331,            3364,            2205,            4115,            3505,            3949,            1307,            ],        [            2861,            1514,            1844,            2747,            730,            1271,            2849,            2565,            2483,            1693,            2519,            1017,            1783,            476,            1790,            1303,            1933,            1673,            1362,            1135,            2104,            1701,            1411,            1502,            2652,            0,            1887,            884,            1563,            3681,            4378,            7617,            2244,            2370,            1276,            426,            766,            2021,            1879,            1748,            709,            7599,            571,            4723,            7370,            2090,            2418,            258,            1061,            2103,            481,            1842,            2692,            2336,            1992,            1694,            1541,            2950,            2926,            1991,            2390,            1259,            3316,            1542,            3493,            1125,            5213,            2206,            4116,            1299,            1743,            2249,            ],        [            1928,            3276,            2619,            1814,            2492,            3033,            1916,            1825,            3258,            2710,            1586,            1565,            858,            1434,            279,            1851,            1691,            1596,            824,            1683,            907,            492,            873,            477,            1257,            1910,            0,            1066,            1756,            2286,            2983,            6222,            733,            859,            808,            1575,            1235,            2384,            458,            1506,            1257,            6204,            1430,            3328,            5975,            2865,            3193,            1652,            935,            592,            1520,            3282,            1297,            1110,            795,            2711,            572,            1555,            1531,            536,            879,            703,            1921,            571,            2560,            2887,            3818,            1273,            3183,            3061,            3505,            278,            ],        [            2343,            2210,            2540,            2229,            1426,            1967,            2331,            2240,            3179,            2389,            2001,            499,            1265,            368,            1281,            785,            1415,            1155,            844,            617,            1586,            1183,            893,            984,            2143,            844,            1378,            0,            1045,            3172,            3869,            7108,            1735,            1861,            758,            824,            1021,            2799,            1370,            1230,            191,            7090,            969,            4214,            6861,            2786,            3114,            586,            847,            1594,            879,            2538,            2183,            1818,            1474,            2390,            1263,            2441,            2417,            1473,            1881,            741,            2807,            1262,            2975,            1821,            4704,            1688,            3598,            1995,            2439,            1731,            ],        [            1558,            1915,            2249,            1444,            1131,            1672,            1546,            1455,            2888,            2340,            1216,            613,            480,            1090,            1572,            355,            319,            204,            1538,            457,            801,            1669,            1587,            1094,            2432,            1566,            1224,            1144,            0,            3461,            4158,            7397,            2026,            2152,            1452,            1546,            1743,            2014,            1661,            471,            913,            7379,            1691,            4503,            7150,            2495,            2823,            1308,            1649,            1885,            1601,            2243,            2472,            1033,            689,            2341,            1373,            2730,            2706,            688,            1174,            851,            3096,            1372,            2190,            1526,            4993,            903,            2813,            1700,            2144,            946,            ],        [            3907,            4948,            4779,            4026,            4164,            4705,            4076,            3985,            5418,            4870,            3746,            3237,            3213,            3106,            2160,            3523,            3363,            3268,            2231,            3355,            3164,            2164,            2545,            2149,            1228,            3582,            2257,            2738,            3428,            0,            697,            3936,            2614,            2740,            2455,            3247,            2907,            4544,            2249,            3178,            2929,            3918,            3102,            1042,            3689,            5025,            5353,            3324,            2607,            2473,            3192,            5442,            1385,            2991,            3052,            4871,            2052,            814,            790,            2793,            2760,            2375,            365,            2051,            4720,            4559,            1532,            3433,            5343,            4733,            5177,            2535,            ],        [            4604,            5645,            5476,            4723,            4861,            5402,            4773,            4682,            6115,            5567,            4443,            3934,            3910,            3803,            2857,            4220,            4060,            3965,            2928,            4052,            3861,            2861,            3242,            2846,            1925,            4279,            2954,            3435,            4125,            697,            0,            3239,            3311,            3437,            3152,            3944,            3604,            5241,            2946,            3875,            3626,            3221,            3799,            345,            2992,            5722,            6050,            4021,            3304,            3170,            3889,            6139,            2082,            3688,            3749,            5568,            2749,            1511,            1487,            3490,            3457,            3072,            1062,            2748,            5417,            5256,            835,            4130,            6040,            5430,            5874,            3232,            ],        [            7838,            9706,            8715,            7992,            8100,            9108,            8094,            7921,            9354,            8806,            7682,            7173,            7149,            7042,            6096,            7459,            7299,            7204,            6167,            7291,            7100,            6100,            6481,            6085,            5164,            7518,            6193,            6674,            7364,            3936,            3239,            0,            6550,            6676,            6391,            7183,            6843,            8480,            6185,            7114,            6865,            18,            7038,            2894,            247,            8961,            9289,            7260,            6543,            6409,            7128,            9378,            5321,            6927,            6988,            8807,            5988,            4750,            4726,            6729,            6696,            6311,            4301,            5987,            8738,            9254,            2404,            7369,            9279,            9080,            9871,            6471,            ],        [            2640,            3674,            3512,            2759,            2890,            3431,            2809,            2718,            4151,            3603,            2479,            1963,            1273,            1832,            549,            2249,            2089,            1994,            1222,            2081,            838,            890,            1271,            875,            1655,            2308,            796,            1464,            2154,            2684,            3381,            6620,            0,            126,            1206,            1973,            1633,            3277,            856,            1904,            1655,            6602,            1828,            3726,            6373,            3758,            4086,            2050,            1333,            551,            1918,            4175,            1695,            696,            950,            3604,            970,            1953,            1929,            1332,            465,            1101,            2319,            969,            4634,            3285,            4216,            2166,            4076,            3459,            3903,            1074,            ],        [            2793,            3827,            3665,            2912,            3043,            3584,            2962,            2871,            4304,            3756,            2632,            2116,            1426,            1985,            702,            2402,            2242,            2147,            1375,            2234,            991,            1043,            1424,            1028,            1808,            2461,            949,            1617,            2307,            2837,            3534,            6773,            472,            0,            1359,            2126,            1786,            3430,            1009,            2057,            1808,            6755,            1981,            3879,            6526,            3911,            4239,            2203,            1486,            704,            2071,            4328,            1848,            849,            1103,            3757,            1123,            2106,            2082,            1485,            618,            1254,            2472,            1122,            4508,            3438,            4369,            2319,            4229,            3612,            4056,            1227,            ],        [            2517,            2841,            3140,            2217,            2057,            2598,            2437,            2414,            3847,            3231,            2175,            1130,            1439,            999,            514,            1416,            1589,            1494,            389,            1248,            1518,            439,            438,            375,            1376,            1475,            611,            631,            1654,            2405,            3102,            6341,            968,            1094,            0,            1140,            800,            2905,            603,            1404,            822,            6323,            995,            3447,            6094,            3454,            3714,            1217,            500,            827,            1085,            3803,            1416,            1345,            1406,            3232,            496,            1674,            1650,            1147,            1114,            601,            2040,            495,            3081,            2452,            3937,            1862,            3704,            2626,            3070,            889,            ],        [            2655,            1940,            2270,            2609,            1156,            1697,            2829,            2991,            2909,            2119,            2499,            997,            1763,            456,            1455,            1283,            1913,            1653,            1027,            1115,            2084,            1681,            1006,            1482,            2317,            426,            1552,            864,            1543,            3346,            4043,            7282,            1909,            2035,            941,            0,            340,            2447,            1544,            1728,            689,            7264,            145,            4388,            7035,            2516,            2844,            238,            640,            1768,            55,            2268,            2357,            2316,            1972,            2120,            1206,            2615,            2591,            1971,            2055,            1239,            2981,            1207,            3473,            1551,            4878,            2186,            4096,            1725,            2169,            2229,            ],        [            2869,            2280,            2610,            2818,            1496,            2037,            3038,            2947,            3249,            2459,            2708,            1358,            2040,            759,            1115,            1644,            2190,            2014,            687,            1476,            2119,            1040,            666,            976,            1977,            766,            1212,            859,            2255,            3006,            3703,            6942,            1569,            1695,            601,            340,            0,            3506,            1204,            2005,            1050,            6924,            195,            4048,            6695,            2856,            3184,            578,            300,            1428,            285,            2608,            2017,            1946,            2007,            2460,            866,            2275,            2251,            1748,            1715,            1202,            2641,            867,            3682,            1891,            4538,            2395,            4305,            2065,            2509,            1490,            ],        [            1679,            1226,            235,            1564,            1183,            628,            1666,            1004,            874,            326,            1352,            2175,            2418,            2161,            3435,            2052,            4731,            2422,            3864,            2019,            2665,            3532,            3913,            3517,            4295,            1685,            3532,            4106,            2312,            5324,            6021,            9260,            3889,            4015,            3848,            2111,            4275,            0,            3524,            4546,            4297,            9242,            2256,            6366,            9013,            481,            809,            1943,            3975,            3748,            2166,            898,            4335,            2897,            2553,            327,            3612,            4593,            4569,            2552,            3038,            3743,            4959,            3611,            2310,            774,            6856,            1481,            2318,            600,            1391,            2810,            ],        [            2216,            3250,            3088,            2165,            2466,            3007,            2385,            2294,            3727,            3179,            2055,            1539,            1515,            1408,            462,            1825,            1665,            1570,            798,            1657,            1466,            466,            847,            451,            873,            1884,            559,            1040,            1730,            1902,            2599,            5838,            916,            1042,            782,            1549,            1209,            2853,            0,            1480,            1231,            5820,            1404,            2944,            5591,            3334,            3662,            1626,            909,            775,            1494,            3751,            913,            1293,            1354,            3180,            546,            1171,            1147,            1095,            1062,            677,            1537,            545,            3029,            2861,            3434,            1742,            3652,            3035,            3479,            837,            ],        [            1258,            2093,            2135,            1212,            1309,            1850,            1432,            1341,            2774,            2226,            1102,            791,            366,            1268,            1458,            533,            185,            90,            1716,            635,            687,            1555,            1765,            1344,            2318,            1744,            1110,            1322,            250,            3347,            4044,            7283,            1912,            2038,            1630,            1724,            1921,            1900,            1547,            0,            1091,            7265,            1869,            4389,            7036,            2381,            2709,            1486,            1827,            1771,            1779,            2421,            2358,            919,            575,            2227,            1635,            2616,            2592,            574,            1060,            1101,            2982,            1634,            2076,            1704,            4879,            789,            2699,            1878,            2322,            832,            ],        [            2132,            2019,            2349,            2086,            1235,            1776,            2306,            2215,            2988,            2198,            1976,            308,            1240,            495,            1502,            594,            1079,            964,            1074,            426,            1561,            1311,            1123,            1112,            3192,            971,            1984,            680,            854,            4221,            4918,            8157,            1956,            2082,            988,            951,            1148,            2774,            1591,            1358,            0,            8139,            1096,            5263,            7910,            2595,            2923,            713,            974,            1815,            1006,            2347,            3232,            1793,            1449,            2199,            1253,            3490,            3466,            1448,            1934,            869,            3856,            1254,            2950,            1630,            5753,            1663,            3573,            1804,            2248,            1706,            ],        [            7820,            9688,            8697,            7974,            8082,            9090,            8076,            7903,            9336,            8788,            7664,            7155,            7131,            7024,            6078,            7441,            7281,            7186,            6149,            7273,            7082,            6082,            6463,            6067,            5146,            7500,            6175,            6656,            7346,            3918,            3221,            18,            6532,            6658,            6373,            7165,            6825,            8462,            6167,            7096,            6847,            0,            7020,            2876,            229,            8943,            9271,            7242,            6525,            6391,            7110,            9360,            5303,            6909,            6970,            8789,            5970,            4732,            4708,            6711,            6678,            6293,            4283,            5969,            8720,            9236,            2386,            7351,            9261,            9062,            9853,            6453,            ],        [            2800,            2085,            2415,            2754,            1301,            1842,            2974,            3136,            3054,            2264,            2644,            1142,            1908,            601,            1310,            1428,            2058,            1798,            882,            1260,            2229,            1235,            861,            1171,            2172,            571,            1407,            1054,            1688,            3201,            3898,            7137,            1764,            1890,            796,            145,            195,            2592,            1399,            1873,            834,            7119,            0,            4243,            6890,            2661,            2989,            383,            495,            1623,            90,            2413,            2212,            2141,            2117,            2265,            1061,            2470,            2446,            2116,            1910,            1397,            2836,            1062,            3618,            1696,            4733,            2331,            4241,            1870,            2314,            1685,            ],        [            4944,            5990,            5821,            5098,            5206,            6214,            5200,            5027,            6460,            5912,            4788,            4279,            4255,            4148,            3202,            4565,            4405,            4310,            3273,            4397,            4206,            3206,            3587,            3191,            2270,            4624,            3299,            3780,            4470,            1042,            345,            2894,            3656,            3782,            3497,            4289,            3949,            5586,            3291,            4220,            3971,            2876,            4144,            0,            2647,            6067,            6395,            4366,            3649,            3515,            4234,            6484,            2427,            4033,            4094,            5913,            3094,            1856,            1832,            3835,            3802,            3417,            1407,            3093,            5844,            5601,            490,            4475,            6385,            6186,            6219,            3577,            ],        [            7591,            9459,            8468,            7745,            7853,            8861,            7847,            7674,            9107,            8559,            7435,            6926,            6902,            6795,            5849,            7212,            7052,            6957,            5920,            7044,            6853,            5853,            6234,            5838,            4917,            7271,            5946,            6427,            7117,            3689,            2992,            247,            6303,            6429,            6144,            6936,            6596,            8233,            5938,            6867,            6618,            229,            6791,            2647,            0,            8714,            9042,            7013,            6296,            6162,            6881,            9131,            5074,            6680,            6741,            8560,            5741,            4503,            4479,            6482,            6449,            6064,            4054,            5740,            8491,            9007,            2157,            7122,            9032,            8833,            9624,            6224,            ],        [            1961,            745,            628,            1846,            1576,            1021,            1948,            1286,            393,            719,            1634,            2568,            2700,            4756,            3717,            2445,            5013,            4918,            4146,            5005,            2947,            3814,            4195,            3799,            4577,            2078,            3814,            4388,            5078,            5606,            6303,            9542,            4171,            4297,            4130,            2504,            4557,            742,            3806,            4828,            4579,            9524,            4752,            6648,            9295,            0,            328,            2336,            4257,            4030,            2559,            417,            4617,            3179,            2835,            720,            3894,            4875,            4851,            2834,            3320,            4025,            5241,            3893,            2592,            1167,            7138,            1763,            1837,            993,            910,            3092,            ],        [            2021,            860,            688,            1906,            1636,            1081,            2008,            1346,            305,            779,            1694,            2628,            2760,            4816,            3777,            2505,            5073,            4978,            4206,            5065,            3007,            3874,            4255,            3859,            4637,            2138,            3874,            4448,            5138,            5666,            6363,            9602,            4231,            4357,            4190,            2564,            4617,            802,            3866,            4888,            4639,            9584,            4812,            6708,            9355,            934,            0,            2396,            4317,            4090,            2619,            532,            4677,            3239,            2895,            780,            3954,            4935,            4911,            2894,            3380,            4085,            5301,            3953,            2652,            1227,            7198,            1823,            1897,            1053,            1025,            3152,            ],        [            2417,            1772,            2102,            2371,            988,            1529,            2591,            2500,            2741,            1951,            2261,            759,            1525,            218,            1532,            1045,            1675,            1415,            1104,            877,            1846,            1443,            1153,            1244,            2394,            258,            1629,            626,            1305,            3423,            4120,            7359,            1986,            2112,            1018,            238,            578,            2279,            1621,            1490,            451,            7341,            383,            4465,            7112,            2348,            2676,            0,            803,            1845,            293,            2100,            2434,            2078,            1734,            1952,            1283,            2692,            2668,            1733,            2132,            1001,            3058,            1284,            3235,            1383,            4955,            1948,            3858,            1557,            2001,            1991,            ],        [            2569,            2436,            3441,            2518,            1652,            2193,            2738,            2647,            4080,            3532,            2408,            1058,            1740,            585,            815,            1344,            1890,            1714,            387,            1176,            1819,            740,            366,            676,            1677,            1061,            912,            559,            1955,            2706,            3403,            6642,            1269,            1395,            301,            640,            300,            3206,            904,            1705,            750,            6624,            495,            3748,            6395,            3687,            4015,            803,            0,            1128,            585,            2764,            1717,            1646,            1707,            3533,            566,            1975,            1951,            1448,            1415,            902,            2341,            567,            3382,            2047,            4238,            2095,            4005,            2221,            2665,            1190,            ],        [            2474,            3508,            3346,            2423,            2724,            3265,            2643,            2552,            3985,            3437,            2313,            1797,            1095,            1666,            383,            2083,            1923,            1828,            1056,            1915,            660,            724,            1105,            709,            1489,            2142,            630,            1298,            1988,            2518,            3215,            6454,            141,            267,            1040,            1807,            1467,            3111,            690,            1738,            1489,            6436,            1662,            3560,            6207,            3592,            3920,            1884,            1167,            0,            1752,            4009,            1529,            518,            772,            3438,            804,            1787,            1763,            1031,            287,            935,            2153,            803,            3287,            3119,            4050,            2000,            3910,            3293,            3737,            908,            ],        [            2710,            1995,            2325,            2664,            1211,            1752,            2884,            3046,            2964,            2174,            2554,            1052,            1818,            511,            1400,            1338,            1968,            1708,            972,            1170,            2139,            1325,            951,            1537,            2262,            481,            1497,            919,            1598,            3291,            3988,            7227,            1854,            1980,            886,            55,            285,            2502,            1489,            1783,            744,            7209,            90,            4333,            6980,            2571,            2899,            293,            585,            1713,            0,            2323,            2302,            2231,            2027,            2175,            1151,            2560,            2536,            2026,            2000,            1294,            2926,            1152,            3528,            1606,            4823,            2241,            4151,            1780,            2224,            1775,            ],        [            2238,            328,            905,            2123,            1126,            655,            2225,            1563,            670,            996,            1911,            2118,            2977,            2104,            3994,            1995,            2480,            2365,            4423,            1962,            3224,            4091,            4472,            4076,            4854,            1628,            4091,            4665,            2255,            5883,            6580,            9819,            4448,            4574,            4407,            2054,            4834,            1019,            4083,            5105,            2337,            9801,            2199,            6925,            9572,            1151,            605,            1886,            4534,            4307,            2109,            0,            4894,            3456,            3112,            997,            4171,            5152,            5128,            3111,            3597,            4302,            5518,            4170,            2869,            717,            7415,            2040,            2114,            683,            493,            3369,            ],        [            3113,            4154,            3985,            3062,            3370,            3911,            3282,            3191,            4624,            4076,            2952,            2443,            2419,            2312,            1366,            2729,            2569,            2474,            1437,            2561,            2370,            1370,            1751,            1355,            434,            2788,            1463,            1944,            2634,            1463,            2160,            5399,            1820,            1946,            1661,            2453,            2113,            3750,            1455,            2384,            2135,            5381,            2308,            2505,            5152,            4231,            4559,            2530,            1813,            1679,            2398,            4482,            0,            2197,            2258,            4077,            1258,            732,            708,            1999,            1966,            1581,            1098,            1257,            5355,            3765,            2995,            2639,            4549,            3939,            4383,            1741,            ],        [            1738,            4000,            2428,            1623,            3216,            3757,            1725,            1634,            3067,            2519,            1395,            2289,            667,            2158,            875,            2575,            2415,            2320,            1548,            2407,            232,            1216,            1597,            1201,            1981,            2634,            1139,            1790,            2480,            3010,            3707,            6946,            633,            759,            1532,            2299,            1959,            2193,            1182,            2230,            1981,            6928,            2154,            4052,            6699,            2674,            3002,            2376,            1659,            877,            2244,            3091,            2021,            0,            344,            2520,            1296,            2279,            2255,            603,            231,            1427,            2645,            1295,            2369,            3611,            4542,            1082,            2992,            2793,            4229,            861,            ],        [            1394,            3075,            2084,            1279,            3287,            2477,            1381,            1290,            2723,            2175,            1051,            2360,            323,            2229,            1129,            2646,            2486,            2391,            1619,            2478,            112,            1287,            1668,            1272,            2201,            2705,            795,            1861,            2551,            3230,            3927,            7166,            887,            1013,            1603,            2370,            2030,            1849,            1253,            2301,            2052,            7148,            2225,            4272,            6919,            2330,            2658,            2447,            1730,            1387,            2315,            2747,            2241,            344,            0,            2176,            1367,            2499,            2475,            259,            485,            1498,            2865,            1366,            2025,            2623,            4762,            738,            2648,            2449,            3240,            517,            ],        [            1707,            628,            311,            1592,            856,            301,            1694,            1032,            950,            160,            1380,            1848,            2446,            1834,            3463,            1725,            2210,            2095,            3892,            1692,            2693,            3560,            3941,            3545,            4323,            1358,            3560,            4134,            1985,            5352,            6049,            9288,            3917,            4043,            3876,            1784,            4303,            488,            3552,            4574,            2067,            9270,            1929,            6394,            9041,            557,            885,            1616,            4003,            3776,            1839,            575,            4363,            2925,            2581,            0,            3640,            4621,            4597,            2580,            3066,            3771,            4987,            3639,            2338,            447,            6884,            1509,            2394,            273,            857,            2838,            ],        [            2244,            2805,            3116,            2193,            2007,            2562,            2413,            2378,            3755,            3263,            2083,            1160,            1403,            1029,            490,            1446,            1553,            1458,            179,            1278,            1494,            354,            468,            339,            1352,            1505,            587,            661,            1618,            2381,            3078,            6317,            944,            1070,            403,            1170,            830,            2881,            579,            1368,            852,            6299,            1025,            3423,            6070,            3362,            3690,            1247,            530,            803,            1115,            3133,            1392,            1321,            1382,            3208,            0,            1650,            1626,            1123,            1090,            565,            2016,            1,            3057,            2416,            3913,            1770,            3680,            2656,            3034,            865,            ],        [            3093,            4068,            3965,            3042,            3270,            3825,            3262,            3171,            4604,            4056,            2932,            2423,            2399,            2292,            1346,            2709,            2549,            2454,            1417,            2541,            2350,            1350,            1731,            1335,            414,            2768,            1443,            1924,            2614,            890,            1587,            4826,            1800,            1926,            1641,            2433,            2093,            3730,            1435,            2364,            2115,            4808,            2288,            1932,            4579,            4211,            4539,            2510,            1793,            1659,            2378,            4628,            571,            2177,            2238,            4057,            1238,            0,            135,            1979,            1946,            1561,            525,            1237,            3906,            3679,            2422,            2619,            4529,            3919,            4297,            1721,            ],        [            3117,            4092,            3989,            3066,            3294,            3915,            3286,            3195,            4628,            4080,            2956,            2447,            2423,            2316,            1370,            2733,            2573,            2478,            1441,            2565,            2374,            1374,            1755,            1359,            438,            2792,            1467,            1948,            2638,            914,            1611,            4850,            1824,            1950,            1665,            2457,            2117,            3754,            1459,            2388,            2139,            4832,            2312,            1956,            4603,            4235,            4563,            2534,            1817,            1683,            2402,            4486,            595,            2201,            2262,            4081,            1262,            24,            0,            2003,            1970,            1585,            549,            1261,            3930,            3703,            2446,            2643,            4553,            4354,            4387,            1745,            ],        [            1393,            3074,            2083,            1278,            2948,            2476,            1380,            1289,            2722,            2174,            1050,            2101,            322,            1970,            815,            2387,            2227,            2132,            1360,            2219,            371,            1028,            1409,            1013,            2200,            2446,            536,            1602,            2292,            3229,            3926,            7165,            1269,            1395,            1344,            2111,            1771,            1848,            994,            2042,            1793,            7147,            1966,            4271,            6918,            2329,            2657,            2188,            1471,            1128,            2056,            2746,            2240,            603,            259,            2175,            1108,            2498,            2474,            0,            744,            1239,            2864,            1107,            2024,            3357,            4761,            737,            2647,            2448,            3239,            258,            ],        [            1692,            3703,            2569,            1764,            2905,            3460,            1866,            1775,            3208,            2660,            1536,            2058,            808,            1927,            644,            2344,            2184,            2089,            1317,            2176,            373,            985,            1366,            970,            1750,            2403,            891,            1559,            2249,            2779,            3476,            6715,            402,            528,            1301,            2068,            1728,            2334,            951,            1999,            1750,            6697,            1923,            3821,            6468,            2815,            3143,            2145,            1428,            646,            2013,            3232,            1790,            231,            485,            2661,            1065,            2048,            2024,            744,            0,            1196,            2414,            1064,            2510,            3314,            4311,            1223,            3133,            3488,            3932,            1002,            ],        [            1735,            2515,            2607,            1684,            1717,            2272,            1904,            1813,            3246,            2698,            1574,            870,            838,            739,            637,            1156,            988,            893,            798,            988,            1159,            539,            920,            340,            1499,            1215,            734,            793,            1053,            2528,            3225,            6464,            1091,            1217,            1101,            1195,            1282,            2372,            726,            803,            562,            6446,            1340,            3570,            6217,            2853,            3181,            957,            982,            950,            1250,            2843,            1539,            1391,            1047,            2699,            619,            1797,            1773,            1046,            1237,            0,            2163,            618,            2548,            2126,            4060,            1261,            3171,            2300,            2744,            1304,            ],        [            3537,            4583,            4414,            3491,            3799,            4340,            3711,            3868,            5053,            4505,            3381,            2872,            2848,            2741,            1795,            3158,            2998,            2903,            1866,            2990,            2799,            1799,            2180,            1784,            863,            3217,            1892,            2373,            3063,            365,            1062,            4301,            2249,            2375,            2090,            2882,            2542,            4179,            1884,            2813,            2564,            4283,            2737,            1407,            4054,            4660,            4988,            2959,            2242,            2108,            2827,            4845,            1020,            2626,            2687,            4506,            1687,            449,            425,            2428,            2395,            2010,            0,            1686,            4355,            4128,            1897,            3068,            4978,            4302,            4746,            2170,            ],        [            2299,            2806,            3171,            2248,            2008,            2563,            2468,            2625,            3810,            3262,            2138,            1161,            1402,            1030,            489,            1447,            1552,            1457,            180,            1279,            1493,            353,            469,            338,            1351,            1506,            586,            662,            1617,            2380,            3077,            6316,            943,            1069,            404,            1171,            831,            2936,            578,            1367,            853,            6298,            1026,            3422,            6069,            3417,            3745,            1248,            531,            802,            1116,            3134,            1391,            1320,            1381,            3263,            1,            1649,            1625,            1122,            1089,            564,            2015,            0,            3112,            2417,            3912,            1825,            3735,            2591,            3035,            864,            ],        [            806,            3001,            2010,            755,            2958,            2403,            1027,            1216,            2649,            2101,            977,            3555,            1368,            3424,            2385,            3841,            3681,            3586,            2814,            3673,            1615,            2482,            2863,            2467,            3245,            3900,            2482,            3056,            3746,            4274,            4971,            8210,            2839,            2965,            2798,            3565,            3225,            1775,            2474,            3496,            3247,            8192,            3420,            5316,            7963,            2256,            2584,            3642,            2925,            2698,            3510,            2673,            3285,            1847,            1503,            2102,            2562,            3543,            3519,            1502,            1988,            2693,            3909,            2561,            0,            2549,            5806,            1106,            2574,            2375,            3166,            1760,            ],        [            2115,            389,            719,            2000,            409,            146,            2102,            1440,            1358,            568,            1788,            1401,            2854,            1387,            3871,            1278,            1763,            1648,            2273,            1245,            3101,            3968,            4349,            3953,            4731,            911,            3968,            1795,            1538,            5760,            6457,            9696,            4325,            4451,            2187,            1337,            1677,            896,            3960,            2009,            1620,            9678,            1482,            6802,            9449,            965,            1293,            1169,            1972,            4184,            1392,            717,            4771,            3333,            2989,            569,            4048,            5029,            5005,            2988,            3474,            4179,            5395,            4047,            2746,            0,            7292,            1917,            2802,            174,            618,            3246,            ],        [            5434,            7302,            6311,            5588,            5696,            6704,            5690,            5517,            6950,            6402,            5278,            4769,            4745,            4638,            3692,            5055,            4895,            4800,            3763,            4887,            4696,            3696,            4077,            3681,            2760,            5114,            3789,            4270,            4960,            1532,            835,            2404,            4146,            4272,            3987,            4779,            4439,            6076,            3781,            4710,            4461,            2386,            4634,            490,            2157,            6557,            6885,            4856,            4139,            4005,            4724,            6974,            2917,            4523,            4584,            6403,            3584,            2346,            2322,            4325,            4292,            3907,            1897,            3583,            6334,            6091,            0,            4965,            6875,            6676,            7467,            4067,            ],        [            474,            2669,            1678,            423,            2626,            2071,            695,            884,            2317,            1769,            645,            3634,            1447,            3503,            2464,            3920,            3760,            3665,            2893,            3752,            1694,            2561,            2942,            2546,            3324,            3979,            2561,            3135,            3825,            4353,            5050,            8289,            2918,            3044,            2877,            3644,            3304,            1443,            2553,            3575,            3326,            8271,            3499,            5395,            8042,            1924,            2252,            3721,            3004,            2777,            3589,            2341,            3364,            1926,            1582,            1770,            2641,            3622,            3598,            1581,            2067,            2772,            3988,            2640,            1339,            2217,            5885,            0,            2242,            2043,            2834,            1839,            ],        [            2132,            2443,            2271,            2017,            3219,            2664,            2119,            1978,            1473,            2362,            1805,            5058,            2871,            4927,            3888,            5344,            5184,            5089,            4317,            5176,            3118,            3985,            4366,            3970,            4748,            5403,            3985,            4559,            5249,            5777,            6474,            9713,            4342,            4468,            4301,            5068,            4728,            2385,            3977,            4999,            4750,            9695,            4923,            6819,            9466,            2517,            1607,            5145,            4428,            4201,            5013,            2115,            4788,            3350,            3006,            2363,            4065,            5046,            5022,            3005,            3491,            4196,            5412,            4064,            2763,            2810,            7309,            1934,            0,            2636,            2608,            3263,            ],        [            1941,            355,            545,            1826,            583,            28,            1928,            1266,            1184,            394,            1614,            1575,            2680,            1561,            3697,            1452,            1937,            1822,            4126,            1419,            2927,            3794,            4175,            3779,            4557,            1085,            3794,            4368,            1712,            5586,            6283,            9522,            4151,            4277,            4110,            1511,            1851,            722,            3786,            4808,            1794,            9504,            1656,            6628,            9275,            791,            1119,            1343,            4237,            4010,            1566,            683,            4597,            3159,            2815,            395,            3874,            4855,            4831,            2814,            3300,            4005,            5221,            3873,            2572,            174,            7118,            1743,            2628,            0,            584,            3072,            ],        [            2525,            229,            1129,            2410,            1027,            556,            2512,            1850,            1163,            978,            2198,            2019,            3264,            2005,            4281,            1896,            2381,            2266,            4710,            1863,            3511,            4378,            4759,            4363,            5141,            1529,            4378,            4952,            2156,            6170,            6867,            10106,            4735,            4861,            2805,            1955,            2295,            1306,            4370,            2627,            2238,            10088,            2100,            7212,            9859,            1375,            1098,            1787,            4821,            4594,            2010,            493,            5181,            3743,            3399,            979,            4458,            5439,            5415,            3398,            3884,            4589,            5805,            4457,            3156,            618,            7702,            2327,            2607,            584,            0,            3656,            ],        [            1654,            3488,            2344,            1421,            2690,            2737,            1641,            1798,            2983,            2435,            1311,            1843,            580,            1712,            557,            2129,            1969,            1874,            1102,            1961,            629,            770,            1151,            755,            1535,            2188,            278,            1344,            2034,            2564,            3261,            6500,            1011,            1137,            1086,            1853,            1513,            2109,            736,            1784,            1535,            6482,            1708,            3606,            6253,            2590,            2918,            1930,            1213,            870,            1798,            3007,            1575,            861,            517,            2436,            850,            1833,            1809,            258,            1002,            981,            2199,            849,            2285,            3099,            4096,            998,            2908,            3273,            3717,            0,            ],        ], 'starts': [12, 62, 37, 60, 20], 'ends': [12, 62, 37, 60, 20]},    'NW-23': {'distance_matrix': [        [            0,            1250,            1697,            871,            3716,            2198,            1946,            3068,            3123,            3342,            4595,            4191,            2515,            2026,            3395,            1666,            1180,            3694,            3912,            1916,            4571,            2691,            2764,            4186,            4104,            2220,            1570,            3369,            3868,            5350,            2103,            1203,            5886,            1533,            2652,            2649,            3000,            3817,            3321,            3125,            3455,            3107,            1371,            416,            5027,            3298,            618,            4026,            4828,            1374,            2581,            3582,            4434,            842,            4326,            1494,            2467,            3605,            ],        [            951,            0,            2648,            227,            2879,            1361,            1109,            2829,            2884,            3103,            4356,            3952,            2276,            1787,            3156,            1427,            941,            3455,            3673,            1677,            4332,            2452,            1800,            3947,            3865,            1981,            1331,            2532,            3629,            5111,            1864,            741,            5647,            1294,            1688,            2410,            2070,            2980,            2391,            2195,            2787,            2868,            552,            579,            4788,            2461,            527,            3787,            3898,            1135,            1744,            3343,            4195,            206,            3489,            1258,            1630,            2675,            ],        [            2146,            2799,            0,            2420,            5392,            3874,            3622,            4617,            4672,            4891,            6144,            5740,            4064,            3575,            4944,            3215,            2729,            5243,            5461,            3465,            6120,            4240,            4313,            5735,            5653,            3769,            3119,            5045,            5417,            6899,            3652,            2752,            7435,            3082,            4201,            4198,            4549,            5493,            4870,            4674,            5004,            4656,            2920,            1965,            6576,            4974,            2167,            5575,            6377,            2923,            4257,            5131,            5983,            2391,            6308,            3043,            4143,            5154,            ],        [            724,            509,            2421,            0,            2975,            1457,            1205,            2602,            2657,            2876,            4129,            3725,            2049,            1560,            2929,            1200,            714,            3228,            3446,            1450,            4105,            2225,            1896,            3720,            3638,            1754,            1104,            2628,            3402,            4884,            1637,            514,            5420,            1067,            1784,            2183,            2166,            3076,            2487,            2291,            2883,            2641,            892,            352,            4561,            2557,            300,            3560,            3994,            908,            1840,            3116,            3968,            101,            3585,            884,            1726,            2771,            ],        [            3740,            2653,            5437,            2880,            0,            1631,            2140,            3262,            3317,            571,            4789,            4385,            2709,            1985,            3589,            2193,            2463,            923,            1141,            2065,            4765,            2885,            2865,            1415,            1333,            1283,            2391,            771,            1097,            5544,            2297,            2650,            6080,            2639,            2727,            1619,            2374,            3284,            2695,            2499,            2829,            902,            2512,            3232,            5221,            418,            3097,            1255,            4202,            2372,            2048,            811,            1663,            2859,            3793,            2289,            1295,            2979,            ],        [            2109,            1022,            3806,            1249,            1809,            0,            509,            2174,            2229,            2403,            3701,            3297,            1621,            897,            2501,            1105,            1219,            2755,            2973,            977,            3677,            1797,            1491,            3247,            3165,            1281,            1303,            1462,            2929,            4456,            1209,            1019,            4992,            1572,            1353,            1710,            1000,            1910,            1321,            1125,            1717,            2168,            881,            1601,            4133,            1391,            1466,            3087,            2828,            1413,            674,            2643,            3495,            1228,            2419,            658,            560,            1605,            ],        [            1491,            912,            3188,            1139,            2183,            665,            0,            2598,            2653,            2872,            4125,            3721,            2045,            1556,            2925,            1196,            710,            3224,            3442,            1446,            4101,            2221,            1865,            3716,            3634,            1750,            1100,            1836,            3398,            4880,            1633,            510,            5416,            1063,            1727,            2179,            1374,            2284,            1695,            1499,            1829,            2637,            771,            1491,            4557,            1765,            1356,            3556,            3202,            904,            1048,            3112,            3964,            1118,            2793,            149,            934,            1979,            ],        [            2798,            2839,            4495,            3066,            3105,            1817,            2326,            0,            415,            2804,            1887,            1483,            591,            1488,            687,            1316,            1713,            3156,            3374,            1378,            1863,            643,            3051,            3648,            3566,            1682,            1641,            2758,            3330,            2642,            1064,            1979,            3178,            1889,            2913,            2111,            2560,            3470,            2881,            2685,            3015,            2569,            2698,            2753,            2319,            2687,            2667,            3488,            4388,            1622,            2234,            3044,            3896,            3045,            3979,            2475,            1856,            3165,            ],        [            3032,            3073,            4729,            3300,            3339,            2051,            2560,            234,            0,            3038,            1472,            1068,            825,            1722,            272,            1550,            1947,            3390,            3608,            1612,            1448,            877,            3285,            3882,            3800,            1916,            1875,            2992,            3564,            2227,            1298,            2213,            2763,            2123,            3147,            2345,            2794,            3704,            3115,            2919,            3249,            2803,            2932,            2987,            1904,            2921,            2901,            3722,            4622,            1856,            2468,            3278,            4130,            3279,            4213,            2709,            2090,            3399,            ],        [            3329,            3325,            5026,            3552,            647,            2303,            2812,            3043,            3098,            0,            4570,            4166,            2490,            1974,            3370,            1974,            2244,            352,            570,            1609,            4546,            2666,            3537,            844,            762,            1236,            2172,            1665,            526,            5325,            2078,            2510,            5861,            2420,            3399,            873,            3046,            3956,            3367,            3171,            3501,            429,            3184,            3284,            5002,            1043,            3769,            684,            4874,            2153,            2720,            240,            1092,            3531,            4465,            2961,            2342,            3651,            ],        [            4505,            4546,            6202,            4773,            4812,            3524,            4033,            1707,            1473,            4511,            0,            566,            2298,            3195,            1291,            3023,            3420,            4863,            5081,            3085,            946,            2350,            4758,            5355,            5273,            3389,            3348,            4465,            5037,            1725,            2771,            3686,            2261,            3596,            4620,            3818,            4267,            5177,            4588,            4392,            4722,            4276,            4405,            4460,            1402,            4394,            4374,            5195,            6095,            3329,            3941,            4751,            5603,            4752,            5686,            4182,            3563,            4872,            ],        [            4101,            4142,            5798,            4369,            4408,            3120,            3629,            1303,            1069,            4107,            566,            0,            1894,            2791,            887,            2619,            3016,            4459,            4677,            2681,            380,            1946,            4354,            4951,            4869,            2985,            2944,            4061,            4633,            1159,            2367,            3282,            1695,            3192,            4216,            3414,            3863,            4773,            4184,            3988,            4318,            3872,            4001,            4056,            836,            3990,            3970,            4791,            5691,            2925,            3537,            4347,            5199,            4348,            5282,            3778,            3159,            4468,            ],        [            2207,            2248,            3904,            2475,            2514,            1226,            1735,            809,            864,            2213,            2336,            1932,            0,            897,            1136,            725,            1122,            2565,            2783,            787,            2312,            432,            2460,            3057,            2975,            1091,            1050,            2167,            2739,            3091,            473,            1388,            3627,            1298,            2322,            1520,            1969,            2879,            2290,            2094,            2424,            1978,            2107,            2162,            2768,            2096,            2076,            2897,            3797,            1031,            1643,            2453,            3305,            2454,            3388,            1884,            1265,            2574,            ],        [            2438,            1351,            4135,            1578,            1617,            329,            838,            1982,            2037,            2211,            3509,            3105,            1429,            0,            2309,            913,            1183,            2563,            2781,            785,            3485,            1605,            1563,            3055,            2973,            1089,            1111,            1270,            2737,            4264,            1017,            1348,            4800,            1359,            1425,            1518,            1072,            1982,            1393,            1197,            1527,            1976,            1210,            1930,            3941,            1199,            1795,            2895,            2900,            1092,            746,            2451,            3303,            1557,            2491,            987,            368,            1677,            ],        [            3333,            3374,            5030,            3601,            3640,            2352,            2861,            535,            301,            3339,            1200,            796,            1126,            2023,            0,            1851,            2248,            3691,            3909,            1913,            1176,            1178,            3586,            4183,            4101,            2217,            2176,            3293,            3865,            1955,            1599,            2514,            2491,            2424,            3448,            2646,            3095,            4005,            3416,            3220,            3550,            3104,            3233,            3288,            1632,            3222,            3202,            4023,            4923,            2157,            2769,            3579,            4431,            3580,            4514,            3010,            2391,            3700,            ],        [            1571,            1943,            3268,            1270,            2209,            921,            1277,            1489,            1544,            1908,            3016,            2612,            936,            592,            1816,            0,            486,            2260,            2478,            482,            2992,            1112,            2155,            2752,            2670,            786,            178,            1862,            2434,            3771,            524,            752,            4307,            426,            2017,            1215,            1664,            2574,            1985,            1789,            2119,            1673,            1802,            1526,            3448,            1791,            1440,            2592,            3492,            395,            1338,            2148,            3000,            1241,            3083,            1122,            960,            2269,            ],        [            1091,            1611,            2788,            790,            2475,            1187,            797,            1900,            1955,            2174,            3427,            3023,            1347,            858,            2227,            498,            0,            2526,            2744,            748,            3403,            1523,            2421,            3018,            2936,            1052,            402,            2128,            2700,            4182,            935,            272,            4718,            508,            2283,            1481,            1930,            2840,            2251,            2055,            2385,            1939,            1470,            1046,            3859,            2057,            1248,            2858,            3758,            206,            1604,            2414,            3266,            761,            3349,            642,            1226,            2535,            ],        [            3605,            3601,            5302,            3828,            923,            2579,            3088,            3319,            3374,            382,            4846,            4442,            2766,            2250,            3646,            2250,            2520,            0,            218,            1885,            4822,            2942,            3813,            492,            410,            1512,            2448,            1941,            174,            5601,            2354,            2786,            6137,            2696,            3675,            1149,            3322,            4232,            3643,            3447,            3777,            705,            3460,            3560,            5278,            1319,            4045,            332,            5150,            2429,            2996,            244,            740,            3807,            4741,            3237,            2618,            3927,            ],        [            3823,            3819,            5520,            4046,            1141,            2797,            3306,            3537,            3592,            600,            5064,            4660,            2984,            2468,            3864,            2468,            2738,            218,            0,            2103,            5040,            3160,            4031,            452,            192,            1730,            2666,            2159,            134,            5819,            2572,            3004,            6355,            2914,            3893,            1367,            3540,            4450,            3861,            3665,            3995,            923,            3678,            3778,            5496,            1537,            4263,            292,            5368,            2647,            3214,            462,            522,            4025,            4959,            3455,            2836,            4145,            ],        [            1720,            1716,            3417,            1943,            1982,            694,            1203,            1434,            1489,            1726,            2961,            2557,            881,            365,            1761,            365,            635,            2078,            2296,            0,            2937,            1057,            1928,            2570,            2488,            604,            563,            1635,            2252,            3716,            469,            901,            4252,            811,            1790,            1033,            1437,            2347,            1758,            1562,            1892,            1491,            1575,            1675,            3393,            1564,            2160,            2410,            3265,            544,            1111,            1966,            2818,            1922,            2856,            1352,            733,            2042,            ],        [            4481,            4522,            6178,            4749,            4788,            3500,            4009,            1683,            1449,            4487,            946,            380,            2274,            3171,            1267,            2999,            3396,            4839,            5057,            3061,            0,            2326,            4734,            5331,            5249,            3365,            3324,            4441,            5013,            779,            2747,            3662,            1315,            3572,            4596,            3794,            4243,            5153,            4564,            4368,            4698,            4252,            4381,            4436,            456,            4370,            4966,            5171,            6071,            3305,            3917,            4727,            5579,            4728,            6002,            4158,            3539,            4848,            ],        [            2806,            2847,            4503,            3074,            3113,            1825,            2334,            377,            432,            2812,            1904,            1500,            599,            1496,            704,            1324,            1721,            3164,            3382,            1386,            1880,            0,            3059,            3656,            3574,            1690,            1649,            2766,            3338,            2659,            1072,            1987,            3195,            1897,            2921,            2119,            2568,            3478,            2889,            2693,            3023,            2577,            2706,            2761,            2336,            2695,            3291,            3496,            4396,            1630,            2242,            3052,            3904,            3053,            4327,            2483,            1864,            3173,            ],        [            2952,            1865,            4649,            2092,            2361,            843,            1352,            2726,            2781,            2955,            4253,            3849,            2173,            1449,            3053,            1657,            1927,            3307,            3525,            1529,            4229,            2349,            0,            3799,            3717,            1833,            1855,            2014,            3481,            5008,            1761,            1862,            5544,            2103,            630,            2262,            236,            1620,            557,            361,            691,            2720,            1724,            2444,            4685,            1943,            2309,            3639,            2064,            1836,            375,            3195,            4047,            2071,            1995,            1501,            1112,            841,            ],        [            4097,            4093,            5794,            4320,            1415,            3071,            3580,            3811,            3866,            874,            5338,            4934,            3258,            2742,            4138,            2742,            3012,            492,            452,            2377,            5314,            3434,            4305,            0,            753,            2004,            2940,            2433,            318,            6093,            2846,            3278,            6629,            3188,            4167,            1641,            3814,            4724,            4135,            3939,            4269,            1197,            3952,            4052,            5770,            1811,            4537,            160,            5642,            2921,            3488,            736,            423,            4299,            5573,            3729,            3110,            4419,            ],        [            4015,            4011,            5712,            4238,            1333,            2989,            3498,            3729,            3784,            792,            5256,            4852,            3176,            2660,            4056,            2660,            2930,            410,            192,            2295,            5232,            3352,            4223,            644,            0,            1922,            2858,            2351,            326,            6011,            2764,            3196,            6547,            3106,            4085,            1559,            3732,            4642,            4053,            3857,            4187,            1115,            3870,            3970,            5688,            1729,            4455,            484,            5560,            2839,            3406,            654,            330,            4217,            5491,            3647,            3028,            4337,            ],        [            2093,            2089,            3790,            2316,            1323,            1067,            1576,            1807,            1862,            1178,            3334,            2930,            1254,            738,            2134,            738,            1008,            1530,            1748,            373,            3310,            1430,            2301,            2022,            1940,            0,            936,            993,            1704,            4089,            842,            1274,            4625,            1184,            2163,            485,            1810,            2720,            2131,            1935,            2265,            943,            1948,            2048,            3766,            905,            2533,            1862,            3638,            917,            1484,            1418,            2270,            2295,            3229,            1725,            1106,            2415,            ],        [            1475,            1995,            3172,            1174,            2405,            1117,            1181,            1830,            1885,            2104,            3357,            2953,            1277,            788,            2157,            178,            390,            2456,            2674,            678,            3333,            1453,            2351,            2948,            2866,            982,            0,            2058,            2630,            4112,            865,            656,            4648,            248,            2213,            1411,            1860,            2770,            2181,            1985,            2315,            1869,            1854,            1430,            3789,            1987,            1344,            2788,            3688,            299,            1534,            2344,            3196,            1145,            3619,            1026,            1156,            2465,            ],        [            2969,            1882,            4666,            2109,            814,            860,            1369,            2491,            2546,            1541,            4018,            3614,            1938,            1214,            2818,            1422,            1692,            1893,            2111,            1294,            3994,            2114,            2094,            2385,            2303,            512,            1620,            0,            2067,            4773,            1526,            1879,            5309,            1868,            1956,            848,            1603,            2513,            1924,            1728,            2058,            1306,            1741,            2461,            4450,            396,            2326,            2225,            3431,            1601,            1277,            1781,            2633,            2088,            3362,            1518,            524,            2208,            ],        [            3779,            3775,            5476,            4002,            1097,            2753,            3262,            3493,            3548,            556,            5020,            4616,            2940,            2424,            3820,            2424,            2694,            174,            134,            2059,            4996,            3116,            3987,            318,            326,            1686,            2622,            2115,            0,            5775,            2528,            2960,            6311,            2870,            3849,            1323,            3496,            4406,            3817,            3621,            3951,            879,            3634,            3734,            5452,            1493,            4219,            158,            5324,            2603,            3170,            418,            741,            3981,            5255,            3411,            2792,            4101,            ],        [            5260,            5301,            6957,            5528,            5567,            4279,            4788,            2462,            2228,            5266,            1725,            1159,            3053,            3950,            2046,            3778,            4175,            5618,            5836,            3840,            779,            3105,            5513,            6110,            6028,            4144,            4103,            5220,            5792,            0,            3526,            4441,            536,            4351,            5375,            4573,            5022,            5932,            5343,            5147,            5477,            5031,            5160,            5215,            323,            5149,            5129,            5950,            6850,            4084,            4696,            5506,            6358,            5507,            6781,            4937,            4318,            5627,            ],        [            2102,            2143,            3799,            2370,            2409,            1121,            1630,            965,            1020,            2108,            2492,            2088,            412,            792,            1292,            620,            1017,            2460,            2678,            682,            2468,            588,            2355,            2952,            2870,            986,            945,            2062,            2634,            3247,            0,            1283,            3783,            1193,            2217,            1415,            1864,            2774,            2185,            1989,            2319,            1873,            2002,            2057,            2924,            1991,            2587,            2792,            3692,            926,            1538,            2348,            3200,            2349,            3623,            1779,            1160,            2469,            ],        [            981,            1501,            2678,            680,            2663,            1375,            687,            2088,            2143,            2362,            3615,            3211,            1535,            1046,            2415,            686,            200,            2714,            2932,            936,            3591,            1711,            2609,            3206,            3124,            1240,            590,            2316,            2888,            4370,            1123,            0,            4906,            553,            2471,            1669,            2118,            3028,            2439,            2243,            2573,            2127,            1360,            936,            4047,            2245,            1138,            3046,            3946,            394,            1792,            2602,            3454,            651,            3877,            532,            1414,            2723,            ],        [            5796,            5837,            7493,            6064,            6103,            4815,            5324,            2998,            2764,            5802,            2261,            1695,            3589,            4486,            2582,            4314,            4711,            6154,            6372,            4376,            1315,            3641,            6049,            6646,            6564,            4680,            4639,            5756,            6328,            536,            4062,            4977,            0,            4887,            5911,            5109,            5558,            6468,            5879,            5683,            6013,            5567,            5696,            5751,            859,            5685,            6281,            6486,            7386,            4620,            5232,            6042,            6894,            6043,            7317,            5473,            4854,            6163,            ],        [            1437,            1957,            3134,            1136,            2653,            1365,            1143,            2078,            2133,            2352,            3605,            3201,            1525,            1036,            2405,            426,            638,            2704,            2922,            926,            3581,            1701,            2599,            3196,            3114,            1230,            248,            2306,            2878,            4360,            1113,            618,            4896,            0,            2461,            1659,            2108,            3018,            2429,            2233,            2563,            2117,            1816,            1392,            4037,            2235,            1306,            3036,            3936,            547,            1782,            2592,            3444,            1107,            3867,            988,            1404,            2713,            ],        [            2279,            1209,            3976,            1436,            3074,            1556,            1166,            3439,            3494,            3668,            4966,            4562,            2886,            2162,            3766,            2370,            2640,            4020,            4238,            2242,            4942,            3062,            713,            4512,            4430,            2546,            2568,            2727,            4194,            5721,            2474,            1676,            6257,            2816,            0,            2975,            949,            2333,            1270,            1074,            1404,            3433,            1068,            1788,            5398,            2656,            1956,            4352,            2777,            2549,            1088,            3908,            4760,            1415,            2708,            1315,            1825,            1554,            ],        [            2661,            2657,            4358,            2884,            1231,            1635,            2144,            2375,            2430,            693,            3902,            3498,            1822,            1306,            2702,            1306,            1576,            1045,            1263,            941,            3878,            1998,            2869,            1537,            1455,            568,            1504,            987,            1219,            4657,            1410,            1842,            5193,            1752,            2731,            0,            2378,            3288,            2699,            2503,            2833,            458,            2516,            2616,            4334,            956,            3101,            1377,            4206,            1485,            2052,            933,            1785,            2863,            4137,            2293,            1674,            2983,            ],        [            3361,            2274,            5058,            2501,            2770,            1252,            1761,            3135,            3190,            3364,            4662,            4258,            2582,            1858,            3462,            2066,            2336,            3716,            3934,            1938,            4638,            2758,            1177,            4208,            4126,            2242,            2264,            2423,            3890,            5417,            2170,            2271,            5953,            2512,            1039,            2671,            0,            1384,            321,            125,            455,            3129,            2133,            2853,            5094,            2352,            2718,            4048,            1828,            2245,            784,            3604,            4456,            2480,            1759,            1910,            1521,            605,            ],        [            3385,            2298,            5082,            2525,            2794,            1276,            1785,            3159,            3214,            3388,            4686,            4282,            2606,            1882,            3486,            2090,            2360,            3740,            3958,            1962,            4662,            2782,            1201,            4232,            4150,            2266,            2288,            2447,            3914,            5441,            2194,            2295,            5977,            2536,            1063,            2695,            636,            0,            193,            761,            327,            3153,            2157,            2877,            5118,            2376,            2742,            4072,            1950,            2269,            808,            3628,            4480,            2504,            375,            1934,            1545,            477,            ],        [            4078,            2991,            5775,            3218,            3487,            1969,            2478,            3852,            3907,            4081,            5379,            4975,            3299,            2575,            4179,            2783,            3053,            4433,            4651,            2655,            5355,            3475,            1928,            4925,            4843,            2959,            2981,            3140,            4607,            6134,            2887,            2988,            6670,            3229,            1174,            3388,            1363,            193,            0,            1488,            134,            3846,            2850,            3570,            5811,            3069,            3435,            4765,            1715,            2962,            1618,            4321,            5173,            3197,            568,            2627,            2238,            284,            ],        [            3160,            2073,            4857,            2300,            2569,            1051,            1560,            2934,            2989,            3163,            4461,            4057,            2381,            1657,            3261,            1865,            2135,            3515,            3733,            1737,            4437,            2557,            976,            4007,            3925,            2041,            2063,            2222,            3689,            5216,            1969,            2070,            5752,            2311,            838,            2470,            411,            1259,            196,            0,            330,            2928,            1932,            2652,            4893,            2151,            2517,            3847,            1703,            2044,            583,            3403,            4255,            2279,            1634,            1709,            1320,            480,            ],        [            4212,            2891,            5909,            3352,            3621,            2103,            2378,            3986,            3807,            4215,            5513,            5109,            3199,            2475,            4313,            2683,            3187,            4567,            4551,            2789,            5255,            3609,            1794,            5059,            4977,            2859,            2881,            3274,            4507,            6034,            2787,            2888,            6804,            3129,            1656,            3288,            1229,            327,            134,            1548,            0,            3980,            2984,            3470,            5711,            2969,            3335,            4899,            1581,            2862,            1401,            4455,            5307,            3097,            702,            2761,            2372,            150,            ],        [            2900,            2896,            4597,            3123,            1369,            1874,            2383,            2614,            2669,            831,            4141,            3737,            2061,            1545,            2941,            1545,            1815,            1183,            1401,            1180,            4117,            2237,            3108,            1675,            1593,            807,            1743,            1236,            1357,            4896,            1649,            2081,            5432,            1991,            2970,            444,            2617,            3527,            2938,            2742,            3072,            0,            2755,            2855,            4573,            1205,            3340,            1515,            4445,            1724,            2291,            1071,            1923,            3102,            4376,            2532,            1913,            3222,            ],        [            1271,            320,            2968,            547,            2472,            954,            702,            2837,            2892,            3066,            4364,            3960,            2284,            1560,            3164,            1768,            1412,            3418,            3636,            1640,            4340,            2460,            1393,            3910,            3828,            1944,            1802,            2125,            3592,            5119,            1872,            1212,            5655,            1765,            1281,            2373,            1629,            3013,            1950,            1754,            2084,            2831,            0,            899,            4796,            2054,            847,            3750,            3457,            1606,            1337,            3306,            4158,            526,            3388,            851,            1223,            2234,            ],        [            327,            779,            2024,            400,            3245,            1727,            1475,            2934,            2989,            3208,            4461,            4057,            2381,            1892,            3261,            1532,            1046,            3560,            3778,            1782,            4437,            2557,            2567,            4052,            3970,            2086,            1436,            2898,            3734,            5216,            1969,            802,            5752,            1399,            2455,            2515,            2803,            4187,            3124,            2928,            3258,            2973,            840,            0,            4893,            2827,            202,            3892,            4631,            1240,            2110,            3448,            4300,            371,            4562,            1624,            1996,            3408,            ],        [            4937,            4978,            6634,            5205,            5244,            3956,            4465,            2139,            1905,            4943,            1402,            836,            2730,            3627,            1723,            3455,            3852,            5295,            5513,            3517,            456,            2782,            5190,            5787,            5705,            3821,            3780,            4897,            5469,            323,            3203,            4118,            859,            4028,            5052,            4250,            4699,            5609,            5020,            4824,            5154,            4708,            4837,            4892,            0,            4826,            4806,            5627,            6527,            3761,            4373,            5183,            6035,            5184,            6458,            4614,            3995,            5304,            ],        [            3322,            2235,            5019,            2462,            418,            1213,            1722,            2844,            2899,            968,            4371,            3967,            2291,            1567,            3171,            1775,            2045,            1320,            1538,            1647,            4347,            2467,            2447,            1812,            1730,            865,            1973,            353,            1494,            5126,            1879,            2232,            5662,            2221,            2309,            1201,            1956,            2866,            2277,            2081,            2411,            1160,            2094,            2814,            4803,            0,            2679,            1652,            3784,            1954,            1630,            1208,            2060,            2441,            3375,            1871,            877,            2561,            ],        [            703,            727,            2400,            348,            3110,            1592,            1340,            2838,            2893,            3112,            4365,            3961,            2285,            1796,            3165,            1436,            950,            3464,            3682,            1686,            4341,            2461,            2365,            3956,            3874,            1990,            1340,            2763,            3638,            5120,            1873,            750,            5656,            1303,            2253,            2419,            2601,            3985,            2922,            2726,            3056,            2877,            638,            202,            4797,            2692,            0,            3796,            4429,            1144,            1975,            3352,            4204,            319,            4360,            1489,            1861,            3206,            ],        [            3937,            3933,            5634,            4160,            1255,            2911,            3420,            3651,            3706,            714,            5178,            4774,            3098,            2582,            3978,            2582,            2852,            332,            292,            2217,            5154,            3274,            4145,            160,            484,            1844,            2780,            2273,            158,            5933,            2686,            3118,            6469,            3028,            4007,            1481,            3654,            4564,            3975,            3779,            4109,            1037,            3792,            3892,            5610,            1651,            4377,            0,            5482,            2761,            3328,            576,            583,            4139,            5413,            3569,            2950,            4259,            ],        [            5044,            3957,            6741,            4184,            4453,            2935,            3444,            4818,            4873,            5047,            6345,            5941,            4265,            3541,            5145,            3749,            4019,            5399,            5617,            3621,            6321,            4441,            2860,            5891,            5809,            3925,            3947,            4106,            5573,            7100,            3853,            3954,            7636,            4195,            2722,            4354,            2295,            2173,            1980,            3163,            1846,            4812,            3816,            4536,            6777,            4035,            4401,            5731,            0,            3928,            2467,            5287,            6139,            4163,            2548,            3593,            3204,            1696,            ],        [            1325,            2003,            3022,            1024,            2269,            981,            1490,            1694,            1749,            1968,            3221,            2817,            1141,            652,            2021,            292,            240,            2320,            2538,            542,            3197,            1317,            2215,            2812,            2730,            846,            196,            1922,            2494,            3976,            729,            506,            4512,            444,            2077,            1275,            1724,            2634,            2045,            1849,            2179,            1733,            1862,            1280,            3653,            1851,            1194,            2652,            3552,            0,            1398,            2208,            3060,            995,            3143,            876,            1020,            2329,            ],        [            2766,            1679,            4463,            1906,            2175,            657,            1166,            2540,            2595,            2769,            4067,            3663,            1987,            1263,            2867,            1471,            1741,            3121,            3339,            1343,            4043,            2163,            817,            3613,            3531,            1647,            1669,            1828,            3295,            4822,            1575,            1676,            5358,            1917,            679,            2076,            326,            1710,            647,            451,            781,            2534,            1538,            2258,            4499,            1757,            2123,            3453,            2154,            1650,            0,            3009,            3861,            1885,            2085,            1315,            926,            931,            ],        [            3505,            3501,            5202,            3728,            823,            2479,            2988,            3219,            3274,            282,            4746,            4342,            2666,            2150,            3546,            2150,            2420,            244,            462,            1785,            4722,            2842,            3713,            736,            654,            1412,            2348,            1841,            418,            5501,            2254,            2686,            6037,            2596,            3575,            1049,            3222,            4132,            3543,            3347,            3677,            605,            3360,            3460,            5178,            1219,            3945,            576,            5050,            2329,            2896,            0,            984,            3707,            4981,            3137,            2518,            3827,            ],        [            4345,            4341,            6042,            4568,            1663,            3319,            3828,            4059,            4114,            1122,            5586,            5182,            3506,            2990,            4386,            2990,            3260,            740,            522,            2625,            5562,            3682,            4553,            423,            330,            2252,            3188,            2681,            741,            6341,            3094,            3526,            6877,            3436,            4415,            1889,            4062,            4972,            4383,            4187,            4517,            1445,            4200,            4300,            6018,            2059,            4785,            583,            5890,            3169,            3736,            984,            0,            4547,            5821,            3977,            3358,            4667,            ],        [            753,            418,            2450,            29,            2884,            1366,            1114,            2631,            2686,            3478,            4158,            3754,            2078,            1972,            2958,            1229,            743,            3830,            4048,            2052,            4134,            2254,            1805,            4322,            4240,            2356,            1133,            2537,            4004,            4913,            1666,            543,            5449,            1096,            1693,            2785,            2041,            3425,            2362,            2166,            2496,            3243,            557,            381,            4590,            2466,            329,            4162,            3869,            937,            1749,            3718,            4570,            0,            3800,            1263,            1635,            2646,            ],        [            3510,            2423,            5207,            2650,            2919,            1401,            1910,            3284,            3339,            3513,            4811,            4407,            2731,            2007,            3611,            2215,            2485,            3865,            4083,            2087,            4787,            2907,            1646,            4357,            4275,            2391,            2413,            2572,            4039,            5566,            2319,            2420,            6102,            2661,            1508,            2820,            1081,            445,            707,            1206,            573,            3278,            2282,            3002,            5243,            2501,            2867,            4197,            2395,            2394,            1050,            3753,            4605,            2629,            0,            2059,            1670,            922,            ],        [            1342,            1296,            3039,            1523,            2567,            1049,            482,            2449,            2504,            2723,            3976,            3572,            1896,            1407,            2776,            1047,            561,            3075,            3293,            1297,            3952,            2072,            2403,            3567,            3485,            1601,            951,            2220,            3249,            4731,            1484,            361,            5267,            914,            2111,            2030,            1758,            2668,            2079,            1883,            2213,            2488,            1155,            1297,            4408,            2149,            1069,            3407,            3586,            755,            1432,            2963,            3815,            870,            3517,            0,            1318,            2363,            ],        [            2445,            1358,            4142,            1585,            2307,            336,            845,            1967,            2022,            2196,            3494,            3090,            1414,            690,            2294,            898,            1168,            2548,            2766,            770,            3470,            1590,            1570,            3040,            2958,            1074,            1096,            1960,            2722,            4249,            1002,            1355,            4785,            1344,            1432,            1503,            1079,            1989,            1400,            1204,            1534,            1961,            1217,            1937,            3926,            1889,            1802,            2880,            2907,            1077,            753,            2436,            3288,            1564,            2498,            994,            0,            1684,            ],        [            3649,            2562,            5346,            2789,            3058,            1540,            2049,            3423,            3478,            3652,            4950,            4546,            2870,            2146,            3750,            2354,            2624,            4004,            4222,            2226,            4926,            3046,            1644,            4496,            4414,            2530,            2552,            2711,            4178,            5705,            2458,            2559,            6241,            2800,            1506,            2959,            1079,            477,            284,            1398,            150,            3417,            2421,            3141,            5382,            2640,            3006,            4336,            1431,            2533,            1189,            3892,            4744,            2768,            852,            2198,            1809,            0,            ],        ], 'starts': [2, 26, 36, 28, 38], 'ends': [2, 26, 36, 28, 38]},    'NW-24': {'distance_matrix': [        [            0,            1921,            6067,            6060,            6360,            4565,            1796,            5590,            5520,            2466,            6188,            6387,            585,            4542,            4176,            2463,            4781,            6058,            1383,            5849,            6664,            4184,            4594,            5717,            2723,            6031,            6262,            3233,            5193,            4962,            5414,            4387,            ],        [            2043,            0,            8063,            8056,            8356,            6561,            3792,            7586,            7516,            4462,            8176,            8383,            2819,            6538,            6172,            4459,            6777,            8054,            3379,            7845,            8660,            6180,            6590,            7713,            4719,            8027,            8258,            1486,            7189,            6958,            7410,            6383,            ],        [            6427,            8276,            0,            2833,            1368,            1469,            4870,            1928,            861,            3735,            2930,            1283,            6014,            1883,            2914,            3999,            1687,            954,            5179,            2663,            1987,            1567,            2155,            2531,            4259,            927,            1158,            9012,            1531,            2523,            469,            2299,            ],        [            6300,            8149,            866,            0,            300,            1918,            4743,            2794,            1727,            4184,            1547,            1288,            5887,            2749,            3410,            3872,            2553,            638,            5052,            1280,            604,            2316,            2651,            1148,            4132,            802,            1033,            9461,            2397,            3019,            1335,            3152,            ],        [            6460,            8309,            566,            1610,            0,            2078,            4903,            2494,            1427,            4344,            1707,            988,            6047,            2449,            3570,            4032,            2253,            338,            5212,            1440,            764,            2476,            2811,            1308,            4292,            502,            733,            9621,            2097,            3179,            1035,            2865,            ],        [            4382,            6231,            2283,            2276,            2576,            0,            2825,            2437,            1736,            2266,            2404,            2603,            3969,            1389,            1437,            1954,            1628,            2274,            3134,            2065,            2880,            398,            733,            1933,            2214,            2247,            2478,            7543,            2040,            1101,            1630,            1234,            ],        [            1851,            3700,            4271,            4264,            4564,            2769,            0,            3794,            3724,            670,            4392,            4591,            1438,            2746,            2380,            667,            2985,            4262,            636,            4053,            4868,            2388,            2798,            3921,            927,            4235,            4466,            5012,            3397,            3166,            3618,            2591,            ],        [            5555,            7404,            2023,            2753,            2428,            2547,            3998,            0,            1070,            3439,            2881,            1570,            5142,            1011,            3335,            3127,            815,            2014,            4307,            3723,            3047,            2140,            2576,            2410,            3387,            1914,            1683,            8716,            564,            2944,            1454,            1427,            ],        [            5502,            7351,            1037,            1683,            1442,            1120,            3945,            1303,            0,            3686,            1811,            1357,            5089,            1258,            2612,            3074,            1062,            1028,            4254,            1472,            2061,            1071,            1853,            1340,            3334,            1001,            1232,            8663,            906,            2221,            384,            1674,            ],        [            2116,            3965,            4075,            4068,            4368,            2573,            559,            3598,            3528,            0,            4196,            4395,            1703,            2550,            2184,            471,            2789,            4066,            868,            3857,            4672,            2192,            2602,            3725,            731,            4039,            4270,            5277,            3201,            2970,            3422,            2395,            ],        [            6218,            8067,            2299,            1433,            1733,            1836,            4661,            2995,            1928,            4102,            0,            2795,            5805,            3225,            3328,            3790,            2754,            2071,            4970,            1198,            2037,            2234,            2569,            470,            4050,            2235,            2466,            9379,            2598,            2937,            1822,            3070,            ],        [            6210,            8059,            728,            2557,            1092,            3025,            4653,            1711,            1725,            4094,            2654,            0,            5797,            1666,            4517,            3782,            1470,            678,            4962,            2387,            1711,            2795,            3758,            2212,            4042,            486,            255,            9371,            1219,            4126,            1197,            2082,            ],        [            413,            2262,            6041,            6034,            6334,            4539,            1770,            5564,            5494,            2440,            6162,            6361,            0,            4516,            4150,            2437,            4755,            6032,            1357,            5823,            6638,            4158,            4568,            5691,            2697,            6005,            6236,            3574,            5167,            4936,            5388,            4361,            ],        [            4544,            6393,            2110,            3033,            2515,            1536,            2987,            1048,            1157,            2428,            3161,            1657,            4131,            0,            2324,            2116,            239,            2101,            3296,            2822,            3134,            1129,            1565,            2690,            2376,            2001,            1770,            7705,            651,            1933,            1541,            416,            ],        [            2980,            4829,            3462,            3455,            3755,            1960,            1423,            2933,            2915,            864,            3583,            3782,            2567,            1885,            0,            552,            2124,            3453,            1732,            3244,            4059,            1579,            1989,            3112,            812,            3426,            3657,            6141,            2536,            552,            2809,            1730,            ],        [            3051,            4900,            3604,            3597,            3897,            2102,            1494,            3127,            3057,            935,            3725,            3924,            2638,            2079,            1713,            0,            2318,            3595,            1803,            3386,            4201,            1721,            2131,            3254,            260,            3568,            3799,            6212,            2730,            2499,            2951,            1924,            ],        [            4740,            6589,            2417,            3229,            2822,            1732,            3183,            1355,            1464,            2624,            3357,            1964,            4327,            196,            2520,            2312,            0,            2408,            3492,            3018,            3441,            1325,            1761,            2886,            2572,            2308,            2077,            7901,            958,            2129,            1848,            612,            ],        [            6757,            8606,            256,            1907,            442,            2375,            5200,            2184,            1117,            4641,            2004,            678,            6344,            2139,            3867,            4329,            1943,            0,            5509,            1737,            1061,            2773,            3108,            1605,            4589,            192,            423,            9918,            1787,            3476,            725,            2555,            ],        [            1365,            3214,            4828,            4821,            5121,            3326,            557,            4351,            4281,            1227,            4949,            5148,            952,            3303,            2937,            1224,            3542,            4819,            0,            4610,            5425,            2945,            3355,            4478,            1484,            4792,            5023,            4526,            3954,            3723,            4175,            3148,            ],        [            5019,            6868,            1277,            1276,            1576,            637,            3462,            1797,            730,            2903,            1404,            1597,            4606,            1752,            2129,            2591,            1556,            1268,            3771,            0,            1880,            1035,            1370,            933,            2851,            1241,            1472,            8180,            1400,            1738,            624,            1871,            ],        [            5696,            7545,            1712,            846,            1146,            1314,            4139,            2473,            1406,            3580,            943,            2134,            5283,            2703,            2806,            3268,            2232,            1484,            4448,            676,            0,            1712,            2047,            544,            3528,            1648,            1879,            8857,            2076,            2415,            1300,            2548,            ],        [            4237,            6086,            1916,            1909,            2209,            559,            2680,            2203,            1369,            2121,            2037,            2236,            3824,            1155,            1347,            1809,            1394,            1907,            2989,            1698,            2513,            0,            588,            1566,            2069,            1880,            2111,            7398,            1806,            956,            1263,            1000,            ],        [            3655,            5504,            2769,            2762,            3062,            818,            2098,            2923,            2222,            1539,            2890,            3089,            3242,            1875,            765,            1227,            2114,            2760,            2407,            2551,            3366,            884,            0,            2419,            1487,            2733,            2964,            6816,            2526,            374,            2116,            1720,            ],        [            5755,            7604,            1771,            905,            1205,            1373,            4198,            2532,            1465,            3639,            483,            2193,            5342,            2762,            2865,            3327,            2291,            1543,            4507,            735,            1509,            1771,            2106,            0,            3587,            1707,            1938,            8916,            2135,            2474,            1359,            2607,            ],        [            2791,            4640,            3344,            3337,            3637,            1842,            1234,            2867,            2797,            675,            3465,            3664,            2378,            1819,            1453,            363,            2058,            3335,            1543,            3126,            3941,            1461,            1871,            2994,            0,            3308,            3539,            5952,            2470,            2239,            2691,            1664,            ],        [            6921,            8770,            242,            2071,            606,            2539,            5364,            2055,            1103,            4805,            2168,            486,            6508,            2010,            4031,            4493,            1814,            192,            5673,            1901,            1225,            2937,            3272,            1769,            4753,            0,            231,            10082,            1563,            3640,            711,            2426,            ],        [            7152,            9001,            473,            2302,            837,            2770,            5595,            1824,            1334,            5036,            2399,            255,            6739,            1779,            4262,            4724,            1583,            423,            5904,            2132,            1456,            3168,            3503,            2000,            4984,            231,            0,            10313,            1332,            3871,            942,            2195,            ],        [            557,            1966,            6556,            6549,            6849,            5054,            2285,            6079,            6009,            2955,            6677,            6876,            1333,            5031,            4665,            2952,            5270,            6547,            1892,            6338,            7153,            4673,            5083,            6206,            3212,            6520,            6751,            0,            5682,            5451,            5903,            4876,            ],        [            5362,            7211,            1543,            2189,            1948,            1626,            3805,            863,            506,            3246,            2317,            1472,            4949,            818,            3142,            2934,            622,            1534,            4114,            1978,            2567,            1947,            2383,            1846,            3194,            1507,            1738,            8523,            0,            2751,            890,            1234,            ],        [            3281,            5130,            3852,            3845,            4145,            2350,            1724,            3234,            3305,            1165,            3973,            4172,            2868,            2186,            336,            853,            2425,            3843,            2033,            3634,            4449,            1969,            2379,            3502,            1113,            3816,            4047,            6442,            2837,            0,            3199,            2031,            ],        [            5382,            7231,            1103,            1563,            1508,            1000,            3825,            1459,            392,            3266,            1691,            1423,            4969,            1414,            2492,            2954,            1218,            1094,            4134,            1352,            2127,            1098,            1733,            1220,            3214,            1067,            1298,            8543,            1062,            2101,            0,            1830,            ],        [            4661,            6510,            2265,            3150,            2670,            1653,            3104,            1203,            1312,            2545,            3278,            1812,            4248,            155,            2441,            2233,            394,            2256,            3413,            2939,            3289,            1246,            1682,            2807,            2493,            2156,            1925,            7822,            806,            2050,            1696,            0,            ],        ], 'starts': [12, 11], 'ends': [12, 11]},    'NW-25': {'distance_matrix': [        [            0,            1846,            1334,            1703,            2356,            2372,            5781,            1289,            1548,            2000,            4634,            1998,            2026,            2865,            4898,            481,            5051,            2272,            4723,            4092,            1103,            14263,            2236,            1725,            2682,            624,            1683,            4460,            1433,            1691,            6708,            2334,            3000,            455,            4136,            124,            1852,            1236,            3083,            2626,            196,            ],        [            1812,            0,            2905,            2430,            1455,            2410,            5770,            1928,            2275,            1099,            4652,            2313,            2753,            2487,            4953,            2293,            5781,            859,            4538,            3718,            2674,            16582,            4022,            4048,            1120,            1351,            3990,            5190,            2163,            2421,            6558,            531,            3727,            1494,            4866,            1953,            2132,            2305,            3813,            2455,            2008,            ],        [            1858,            3348,            0,            3106,            2940,            2481,            5007,            3586,            2951,            2584,            3860,            2107,            3429,            1531,            4124,            1916,            4277,            2856,            5065,            2758,            490,            12929,            902,            391,            3266,            2027,            349,            3686,            1507,            1185,            6107,            2918,            4403,            3147,            3362,            1527,            1926,            861,            2309,            1712,            1631,            ],        [            1553,            1222,            2887,            0,            2580,            3535,            6909,            632,            355,            2321,            5777,            3315,            861,            3612,            6092,            2034,            5254,            2081,            5663,            4843,            2656,            15816,            3789,            3278,            1764,            1092,            3236,            6033,            3019,            3277,            7683,            1753,            1835,            1235,            6088,            1677,            3354,            2789,            4824,            3580,            1749,            ],        [            3129,            1599,            1955,            3762,            0,            781,            4329,            3250,            3607,            1867,            3211,            735,            4085,            1032,            3512,            2329,            2674,            1138,            3097,            2263,            1724,            14903,            2559,            2369,            1051,            2668,            2311,            2094,            1213,            1471,            5117,            1200,            5059,            2811,            1759,            2403,            574,            1355,            2863,            1000,            2475,            ],        [            3098,            2106,            1924,            3716,            1232,            0,            3820,            3219,            3561,            1836,            2702,            704,            4039,            1001,            3003,            2298,            2643,            2108,            2588,            2232,            1693,            14872,            2528,            2338,            1558,            2637,            2280,            2958,            1182,            1440,            4608,            2170,            5013,            2780,            1728,            2372,            543,            1324,            2832,            969,            2444,            ],        [            6250,            5234,            4402,            6883,            4289,            3129,            0,            6371,            6728,            4965,            718,            3833,            7206,            2005,            447,            5299,            2154,            4773,            2930,            3236,            4670,            15772,            3084,            3952,            4686,            5789,            3894,            938,            4292,            3970,            3028,            4835,            8180,            5932,            1239,            5373,            3672,            4301,            2276,            2186,            5445,            ],        [            921,            1007,            2255,            831,            2365,            3320,            6711,            0,            676,            2106,            5578,            2716,            1154,            3341,            5828,            1402,            5995,            1866,            5448,            5013,            2024,            15184,            3157,            2646,            1549,            460,            2604,            5404,            2377,            2635,            7468,            1538,            2128,            603,            5080,            1045,            2535,            2157,            4027,            3309,            1117,            ],        [            1198,            867,            2532,            789,            2225,            3180,            6554,            277,            0,            1966,            5422,            2960,            1112,            3257,            5723,            1679,            6282,            1726,            5308,            4488,            2301,            15461,            3434,            2923,            1409,            737,            2881,            5678,            2664,            2922,            7328,            1398,            2086,            880,            5367,            1322,            2999,            2434,            4314,            3225,            1394,            ],        [            2311,            1230,            2318,            2929,            356,            1311,            4671,            2432,            2774,            0,            3553,            1091,            3252,            1388,            3854,            2792,            3030,            840,            3439,            2619,            2087,            15259,            2915,            2725,            682,            1850,            2667,            3809,            1576,            1834,            5459,            902,            4226,            1993,            2115,            2452,            1545,            1718,            3226,            1356,            2507,            ],        [            5725,            4732,            3921,            6402,            3787,            2627,            1147,            5846,            6203,            4463,            0,            3331,            6725,            1524,            301,            4818,            1673,            4271,            2428,            2755,            4189,            16790,            2603,            3471,            4184,            5264,            3413,            457,            3811,            3489,            3622,            4333,            7699,            5407,            758,            4892,            3170,            3820,            1795,            1705,            4964,            ],        [            2394,            2014,            1220,            3012,            1488,            1048,            4826,            2515,            2857,            1132,            3679,            0,            3335,            1299,            3943,            1594,            4096,            1404,            3364,            2530,            989,            14897,            2337,            2363,            1814,            1933,            2305,            3505,            478,            736,            5346,            1466,            4309,            2076,            3181,            1668,            493,            620,            2128,            1267,            1740,            ],        [            2198,            1867,            3532,            645,            3225,            4180,            7883,            1277,            1000,            2966,            6422,            3960,            0,            4257,            7000,            2679,            5899,            2726,            6308,            5488,            3301,            16461,            4434,            3923,            2409,            1737,            3881,            6678,            3535,            3793,            8328,            2398,            974,            1880,            4984,            2322,            3999,            3434,            5185,            4225,            2394,            ],        [            4100,            3108,            2439,            4733,            2234,            1477,            3075,            4221,            4578,            2838,            1928,            1706,            5056,            0,            2192,            3966,            2345,            2647,            3538,            1231,            3337,            14574,            2230,            2040,            2560,            3639,            1982,            1765,            2184,            2637,            4172,            2709,            6030,            3782,            1430,            4040,            1545,            2968,            2490,            988,            4112,            ],        [            5803,            4787,            3955,            6436,            3842,            2682,            1197,            5924,            6281,            4518,            271,            3386,            6759,            1558,            0,            4852,            1707,            4326,            2483,            2789,            4223,            15325,            2637,            3505,            4239,            5342,            3447,            491,            3845,            3523,            3677,            4388,            7733,            5485,            792,            4926,            3225,            3854,            1829,            1739,            4998,            ],        [            1288,            2482,            853,            2536,            2074,            2114,            5488,            2122,            2381,            1718,            4341,            1740,            2859,            2384,            4605,            0,            4758,            1990,            4430,            3611,            622,            13782,            1755,            1244,            2400,            1457,            1202,            4167,            1140,            1398,            6552,            2052,            3833,            1288,            3843,            957,            1559,            755,            2790,            2333,            1522,            ],        [            4096,            3979,            2248,            4729,            3571,            3112,            2820,            4217,            4574,            3215,            1673,            2738,            5052,            1888,            1937,            3145,            0,            3487,            3885,            3119,            2516,            14336,            1785,            1798,            3897,            3635,            1740,            1499,            2138,            1816,            5919,            3549,            6011,            3778,            1175,            3219,            2557,            2147,            122,            2069,            3291,            ],        [            2039,            1470,            2046,            2657,            596,            1551,            4911,            2160,            2502,            240,            3793,            1454,            2980,            1628,            4094,            1915,            3270,            0,            3679,            2859,            1815,            15723,            3163,            3189,            922,            1578,            3131,            4331,            1304,            1562,            5661,            630,            3954,            1721,            4007,            2180,            1273,            1446,            2954,            1596,            2235,            ],        [            4137,            2897,            2227,            5323,            2023,            1586,            2864,            4826,            5168,            3824,            1717,            1495,            5661,            498,            1981,            3754,            2134,            4096,            0,            1729,            3125,            14362,            2018,            1828,            2349,            4244,            1770,            1554,            2747,            2425,            4670,            4158,            6620,            4387,            1219,            3828,            1334,            2756,            2278,            777,            3900,            ],        [            5638,            4645,            4534,            4158,            3700,            2540,            3539,            5759,            6101,            4376,            2421,            3244,            4509,            2805,            2722,            4838,            3893,            4184,            2307,            0,            5432,            16669,            4325,            4135,            4097,            5177,            4077,            2677,            3722,            4732,            4289,            4246,            5483,            5320,            2978,            4912,            3083,            5063,            4585,            3084,            4984,            ],        [            1738,            2967,            231,            2986,            2559,            2100,            4134,            2572,            2831,            2203,            2987,            1726,            3309,            1762,            3251,            1796,            3404,            2475,            5296,            2989,            0,            13160,            1133,            622,            2885,            1907,            580,            2824,            1126,            1416,            5930,            2537,            4283,            1738,            2489,            1407,            1545,            741,            2540,            1943,            1511,            ],        [            2970,            4175,            1487,            4218,            3783,            3829,            4728,            4413,            4063,            3411,            3581,            3255,            4541,            2356,            3845,            3028,            3998,            3683,            5890,            3583,            1602,            0,            1713,            1202,            4109,            3139,            1160,            3418,            2334,            2012,            6524,            3745,            5515,            3974,            3083,            2639,            2753,            1973,            3134,            2537,            2743,            ],        [            2636,            3823,            1153,            4558,            3083,            2646,            4790,            4061,            4403,            3059,            2528,            2555,            4881,            1656,            2792,            2989,            2945,            3331,            5540,            3233,            1268,            13277,            0,            739,            3409,            3479,            808,            2365,            1982,            1660,            6174,            3393,            5855,            3622,            2030,            2305,            2401,            1639,            2092,            1837,            2409,            ],        [            3855,            5060,            2372,            5103,            4668,            4714,            5613,            5298,            4948,            4296,            4466,            4140,            5426,            3241,            4730,            3913,            5987,            4568,            6775,            4468,            2487,            15910,            2598,            0,            4994,            4024,            2045,            4303,            3219,            2897,            7409,            4630,            6400,            4859,            3968,            3524,            3638,            2858,            4019,            3422,            3628,            ],        [            2360,            548,            3453,            1803,            1158,            2113,            5473,            1291,            1648,            1647,            4355,            1893,            2126,            2190,            4656,            2841,            3832,            1407,            4241,            3421,            3222,            17130,            4570,            4596,            0,            1899,            4538,            4611,            2711,            2969,            6223,            1079,            3100,            2042,            5414,            2501,            2680,            2853,            4361,            2158,            2556,            ],        [            765,            1255,            2099,            1079,            2613,            3568,            6442,            665,            924,            1813,            5295,            2560,            1402,            3185,            5559,            1246,            5712,            2114,            5250,            4857,            1868,            15028,            3001,            2490,            1797,            0,            2448,            5121,            2094,            2352,            7232,            1786,            2376,            447,            4797,            889,            2379,            2012,            3744,            3153,            961,            ],        [            2266,            3453,            783,            4188,            3045,            2586,            4887,            3691,            4033,            2689,            3740,            2212,            4511,            1636,            4004,            2619,            4157,            2961,            5170,            2863,            898,            16279,            553,            369,            3371,            3109,            0,            2462,            1612,            1290,            5804,            3023,            5485,            3252,            3242,            1935,            2031,            1269,            2189,            1817,            2039,            ],        [            4744,            3368,            3464,            5930,            2494,            3263,            1999,            5433,            5775,            3098,            852,            1966,            6253,            1067,            1063,            4361,            1216,            2907,            3064,            2298,            3732,            14834,            2146,            3014,            2820,            4851,            2956,            0,            3354,            3032,            4258,            2969,            7227,            4994,            301,            4435,            1805,            3363,            1338,            1248,            4507,            ],        [            2028,            2479,            748,            3214,            2071,            1612,            4348,            2717,            3059,            1715,            3201,            1238,            3537,            1903,            3465,            1645,            3618,            1987,            3928,            3134,            1016,            17791,            1859,            1885,            2397,            2135,            1827,            3027,            0,            258,            7230,            2049,            4511,            2278,            2703,            1719,            1057,            647,            1650,            2084,            1791,            ],        [            3936,            4387,            2026,            5122,            3979,            3520,            4090,            4625,            4967,            3623,            2943,            3146,            5445,            1645,            3207,            3553,            3360,            3895,            5155,            2876,            2924,            17533,            1601,            1627,            4305,            4043,            1569,            2769,            2546,            0,            6972,            3957,            6419,            4186,            2445,            3627,            2965,            2555,            1392,            1826,            3699,            ],        [            5208,            3968,            3298,            6394,            3094,            2657,            3935,            5897,            6239,            4895,            2788,            2566,            6717,            1569,            3052,            4825,            3205,            5167,            1071,            2800,            4196,            18805,            3089,            2899,            3420,            5315,            2841,            2625,            3818,            3496,            0,            5229,            7691,            5458,            2290,            4899,            2405,            3827,            3349,            1848,            4971,            ],        [            1281,            548,            2431,            1899,            1906,            2861,            6221,            1397,            1744,            1097,            5103,            2311,            2222,            2938,            5404,            1762,            5779,            857,            4989,            4169,            2200,            16580,            4020,            4046,            1090,            820,            3988,            5359,            2161,            2419,            6971,            0,            3196,            963,            4864,            1422,            2130,            1805,            3811,            2906,            1477,            ],        [            2383,            2052,            3717,            830,            3410,            4365,            8068,            1462,            1185,            3151,            6607,            4186,            726,            4442,            7185,            2864,            7338,            2911,            6493,            5673,            3486,            16646,            4619,            4108,            2594,            1922,            4066,            6747,            3720,            3978,            8475,            2583,            0,            2065,            6423,            2507,            4005,            3630,            5370,            4410,            2579,            ],        [            318,            1739,            1652,            1590,            2674,            2725,            6099,            1176,            1435,            2318,            4952,            2351,            1913,            3183,            5216,            799,            5369,            1624,            5041,            4410,            1421,            14581,            2554,            2043,            2281,            511,            2001,            4778,            1751,            2009,            7351,            1686,            2887,            0,            4454,            442,            2170,            1565,            3401,            2944,            514,            ],        [            4443,            4894,            3163,            5629,            4486,            4027,            3735,            5132,            5474,            4130,            2588,            3653,            5952,            2803,            2852,            4060,            915,            4402,            4800,            4034,            3431,            15251,            2700,            2713,            4812,            4550,            2655,            2414,            3053,            2731,            6834,            4464,            6926,            4693,            0,            4134,            3472,            3062,            1037,            2984,            4206,            ],        [            331,            1728,            1665,            1579,            2387,            3152,            6115,            1165,            1424,            2031,            4968,            2778,            1902,            3196,            5232,            812,            5385,            1613,            5468,            4423,            1434,            14594,            2567,            2056,            2270,            500,            2014,            4794,            1767,            2025,            7364,            1675,            2876,            331,            4470,            0,            2597,            1578,            3417,            3371,            527,            ],        [            2903,            1866,            1729,            3521,            992,            555,            4103,            3019,            3366,            1641,            2985,            509,            3844,            806,            3286,            2103,            2448,            1405,            2871,            2037,            1498,            14677,            2333,            2143,            1318,            2442,            2085,            1868,            987,            1245,            4853,            1467,            4818,            2585,            1533,            2177,            0,            1129,            2593,            774,            2249,            ],        [            1599,            2226,            1104,            2961,            1818,            1359,            4733,            2459,            2806,            1462,            3586,            985,            3284,            1610,            3850,            1216,            4003,            1734,            3675,            2841,            873,            18176,            2244,            2270,            2144,            1882,            2212,            3412,            385,            643,            5657,            1796,            4258,            2025,            3088,            1290,            804,            0,            2035,            1578,            1362,            ],        [            4074,            4067,            2794,            5260,            3193,            3962,            2698,            4758,            5105,            3797,            1551,            2665,            5583,            1766,            1815,            3691,            1968,            3606,            3763,            2997,            3062,            15533,            2331,            2344,            3519,            4181,            2286,            1377,            2684,            2362,            4957,            3668,            6557,            4324,            1053,            3765,            2504,            2693,            0,            1947,            3837,            ],        [            3112,            2120,            1938,            3730,            1246,            809,            2782,            3228,            3575,            1850,            1635,            718,            4053,            410,            1899,            2312,            2052,            1659,            3125,            1641,            1707,            17653,            1937,            1747,            1572,            2651,            1689,            1472,            1196,            1454,            4582,            1721,            5027,            2794,            1137,            2386,            557,            1338,            2197,            0,            2458,            ],        [            853,            2107,            1138,            2101,            2191,            2242,            5616,            1687,            1946,            1835,            4469,            1868,            2424,            2669,            4733,            285,            4886,            2107,            4558,            3896,            907,            14067,            2040,            1529,            2517,            1022,            1487,            4295,            1268,            1526,            6837,            2169,            3398,            853,            3971,            522,            1687,            1051,            2918,            2461,            0,            ],        ], 'starts': [1, 17, 31, 30], 'ends': [1, 17, 31, 30]},    'NW-26': {'distance_matrix': [        [            0,            549,            4130,            1871,            2202,            3909,            5739,            5643,            4698,            4325,            3315,            4322,            2504,            981,            1917,            2403,            4161,            3875,            3468,            4447,            4228,            4688,            4894,            4995,            5005,            5198,            2344,            3942,            2787,            1868,            4251,            4261,            1373,            5303,            3077,            1062,            1985,            4915,            2185,            963,            4460,            2467,            3897,            4390,            5111,            3654,            5562,            4260,            4718,            4752,            1580,            3077,            3897,            1634,            1218,            4308,            4520,            113,            3913,            5379,            5596,            290,            2059,            875,            5914,            3655,            3082,            2059,            1450,            2034,            ],        [            549,            0,            3667,            1627,            1653,            3446,            5276,            5180,            4235,            3862,            2852,            3859,            2041,            432,            1454,            1940,            3698,            3412,            3005,            3984,            3765,            4225,            4431,            4532,            4542,            4735,            1795,            3479,            2324,            1319,            3788,            3798,            910,            4840,            2614,            818,            1522,            4452,            1722,            500,            3997,            2472,            3434,            3927,            4648,            3191,            5099,            3797,            4255,            4289,            1336,            2614,            3434,            1085,            669,            3845,            4057,            662,            3450,            5509,            5133,            259,            1815,            631,            5451,            3192,            2619,            1510,            806,            1790,            ],        [            4662,            4199,            0,            5199,            5043,            2429,            2306,            1794,            568,            2328,            2046,            2821,            2158,            3912,            3909,            2816,            2164,            843,            1893,            2700,            2727,            2691,            764,            1146,            1156,            1068,            5185,            1728,            1875,            4709,            2308,            2264,            3289,            1454,            2532,            4390,            2677,            2701,            2477,            3699,            2463,            3348,            1900,            260,            1262,            1707,            1432,            2263,            3217,            2755,            4908,            2532,            1464,            4475,            4059,            2311,            2523,            4775,            1916,            1842,            1747,            4372,            3756,            4465,            1784,            1481,            2285,            4900,            3538,            3781,            ],        [            1871,            1627,            4667,            0,            3492,            4446,            6276,            6180,            5235,            4862,            3852,            4859,            3041,            2059,            2454,            1128,            4698,            4412,            4005,            4984,            4765,            5225,            5431,            5532,            5542,            5735,            3634,            4479,            3324,            3158,            4788,            4798,            1910,            5840,            3614,            921,            2522,            5452,            2722,            1500,            4997,            596,            4434,            4927,            5648,            4191,            6099,            4797,            5255,            5289,            291,            3614,            4434,            2924,            2508,            4845,            5057,            1984,            4450,            5916,            6133,            1581,            188,            996,            6451,            4192,            3619,            3349,            1987,            163,            ],        [            2202,            1653,            4511,            3280,            0,            4290,            6120,            6024,            5079,            4706,            3696,            4703,            2885,            1221,            2298,            2784,            4542,            4256,            3849,            4828,            4609,            5069,            5275,            5376,            5386,            5579,            480,            4323,            3168,            1292,            4632,            4642,            1754,            5684,            3458,            2683,            2366,            5542,            2566,            1992,            4841,            3316,            4278,            4771,            5492,            4035,            5943,            4641,            5099,            5133,            2989,            3458,            4278,            1058,            984,            4689,            4901,            2315,            4294,            6353,            5977,            1912,            3680,            2284,            6295,            4036,            3463,            143,            1505,            3655,            ],        [            4395,            3932,            2998,            4932,            4776,            0,            2446,            2220,            1830,            846,            1080,            473,            2495,            3645,            3642,            4716,            1159,            2743,            927,            598,            379,            916,            1634,            1985,            1995,            1330,            4918,            3628,            3775,            4442,            342,            306,            3022,            1700,            1566,            4123,            2410,            4601,            2210,            3432,            808,            5248,            1368,            2138,            2101,            741,            1062,            608,            869,            980,            4641,            1566,            934,            4208,            3792,            656,            1518,            4508,            594,            1472,            2173,            4105,            5120,            4198,            1414,            3381,            1319,            4633,            3271,            5095,            ],        [            5629,            5166,            2102,            6166,            6010,            2463,            0,            1512,            2670,            1585,            2139,            2855,            3125,            4879,            4876,            3783,            1487,            1847,            1986,            2409,            2761,            1948,            2866,            1277,            1287,            3170,            6152,            2695,            2842,            5676,            2342,            2298,            4256,            1172,            2625,            5357,            3644,            6654,            3444,            4666,            2172,            4315,            1675,            2362,            2085,            1800,            1648,            1972,            3251,            2012,            5875,            2625,            1993,            5442,            5026,            1934,            1134,            5742,            2305,            1248,            1465,            5339,            4723,            5432,            2473,            2448,            2378,            5867,            4505,            4748,            ],        [            4161,            3698,            2168,            4698,            4542,            1229,            2251,            0,            1000,            1128,            846,            1621,            3228,            3411,            3408,            3886,            964,            1913,            693,            1500,            1527,            1491,            804,            1790,            1800,            500,            4684,            2798,            2945,            4208,            1108,            1064,            2788,            1685,            1332,            3889,            2176,            3771,            1976,            3198,            1263,            4418,            700,            1308,            1906,            507,            136,            1063,            2017,            1555,            4407,            1332,            264,            3974,            3558,            1111,            1323,            4274,            716,            1370,            1978,            3871,            4826,            3964,            1312,            2551,            1085,            4399,            3037,            4851,            ],        [            4732,            4269,            1168,            5269,            5113,            1861,            2610,            1864,            0,            1760,            1478,            2253,            2228,            3982,            3979,            2886,            1596,            913,            1325,            2132,            2159,            2123,            196,            1216,            1226,            500,            5255,            1798,            1945,            4779,            1740,            1696,            3359,            1502,            1964,            4460,            2747,            2771,            2547,            3769,            1895,            3418,            1332,            308,            1332,            1139,            864,            1695,            2649,            2187,            4978,            1964,            896,            4545,            4129,            1743,            1955,            4845,            1348,            1274,            1817,            4442,            3826,            4535,            1216,            1551,            1717,            4970,            3608,            3851,            ],        [            4301,            3838,            2576,            4838,            4682,            878,            1600,            1374,            1736,            0,            986,            1270,            2401,            3551,            3548,            4257,            313,            2321,            833,            824,            1176,            363,            1540,            1139,            1149,            1236,            4824,            3169,            3316,            4348,            757,            713,            2928,            1034,            1472,            4029,            2316,            7128,            2116,            3338,            587,            4789,            522,            2044,            1255,            647,            968,            387,            1666,            427,            4547,            1472,            840,            4114,            3698,            349,            672,            4414,            720,            1110,            1327,            4011,            5026,            4104,            1320,            2922,            1225,            4539,            3177,            5001,            ],        [            3315,            2852,            2646,            3852,            3696,            1066,            2133,            1907,            1478,            1010,            0,            1503,            1415,            2565,            2562,            3048,            846,            2391,            153,            1382,            1409,            1373,            1282,            1672,            1682,            978,            3838,            3276,            1698,            3362,            990,            946,            1942,            1348,            486,            3043,            1330,            4249,            1130,            2352,            1145,            3580,            582,            1786,            1788,            339,            710,            945,            1899,            1437,            3561,            486,            582,            3128,            2712,            993,            1205,            3428,            598,            1120,            1860,            3025,            4040,            3118,            1062,            3029,            239,            3553,            2191,            4015,            ],        [            4405,            3942,            3514,            4942,            4786,            516,            2962,            2736,            2346,            1362,            1596,            0,            2505,            3655,            3652,            4138,            1675,            3259,            1443,            791,            572,            996,            2150,            2501,            2511,            1846,            4928,            4144,            2788,            4452,            858,            822,            3032,            2216,            1576,            4133,            2420,            5117,            2220,            3442,            828,            4670,            1884,            2654,            2617,            1257,            1578,            996,            396,            988,            4651,            1576,            1450,            4218,            3802,            1172,            2034,            4518,            1110,            1988,            2689,            4115,            5130,            4208,            1930,            3897,            1329,            4643,            3281,            5105,            ],        [            2504,            2041,            1626,            3041,            2885,            2009,            3235,            3139,            2194,            2425,            1415,            2422,            0,            1754,            1751,            1890,            2261,            1371,            1568,            2547,            2328,            2788,            2390,            2491,            2501,            2694,            3027,            1438,            283,            2551,            2351,            2361,            1131,            2799,            1429,            2232,            519,            2411,            319,            1541,            2560,            2422,            1997,            1886,            2607,            1754,            3058,            2360,            2818,            2852,            2750,            1429,            3090,            2317,            1901,            2408,            3104,            2617,            2013,            2875,            3092,            2214,            3229,            2307,            3410,            1151,            1182,            2742,            1380,            3204,            ],        [            981,            432,            3380,            2059,            1221,            3159,            4989,            4893,            3948,            3575,            2565,            3572,            1754,            0,            1167,            1653,            3411,            3125,            2718,            3697,            3478,            3938,            4144,            4245,            4255,            4448,            1363,            3192,            2037,            887,            3501,            3511,            623,            4553,            2327,            1250,            1235,            4165,            1435,            861,            3710,            2185,            3147,            3640,            4361,            2904,            4812,            3510,            3968,            4002,            1768,            2327,            3147,            653,            237,            3558,            3770,            1094,            3163,            4629,            4846,            691,            2247,            1063,            5164,            2905,            2332,            1078,            374,            2222,            ],        [            1917,            1454,            3377,            2454,            2298,            3156,            4986,            4890,            3945,            3572,            2562,            3569,            1751,            1167,            0,            486,            3408,            3122,            2715,            3694,            3475,            3935,            4141,            4242,            4252,            4445,            2440,            3189,            2034,            1964,            3498,            3508,            620,            4550,            2324,            1645,            1232,            4162,            1432,            954,            3707,            1018,            3144,            3637,            4358,            2901,            4809,            3507,            3965,            3999,            2163,            2324,            3144,            1730,            1314,            3555,            3767,            2030,            3160,            5219,            4843,            1627,            1426,            1720,            5161,            2902,            2329,            2155,            793,            1451,            ],        [            2403,            1940,            2284,            1128,            2784,            3642,            3893,            3797,            2852,            4213,            3048,            5105,            1890,            1653,            486,            0,            4448,            2029,            3201,            4984,            5011,            4576,            3048,            3149,            3159,            3352,            2926,            2096,            1607,            2450,            4592,            4548,            1106,            3457,            2810,            2131,            1718,            3069,            1918,            1440,            4747,            532,            4184,            2544,            3265,            3991,            3716,            4547,            5501,            4640,            1419,            2810,            3748,            2216,            1800,            4595,            3762,            2516,            4200,            4126,            3750,            2113,            940,            2206,            4068,            1809,            2815,            2641,            1279,            965,            ],        [            4265,            3802,            2263,            4802,            4646,            1274,            1287,            1061,            1700,            396,            950,            1666,            3286,            3515,            3512,            3944,            0,            2008,            797,            1220,            1572,            759,            1504,            826,            836,            1200,            4788,            2856,            3003,            4312,            1153,            1109,            2892,            721,            1436,            3993,            2280,            6815,            2080,            3302,            983,            4476,            486,            1644,            942,            611,            932,            783,            2062,            823,            4511,            1436,            804,            4078,            3662,            745,            359,            4378,            1116,            797,            1014,            3975,            4884,            4068,            1284,            2609,            1189,            4503,            3141,            4909,            ],        [            4917,            4454,            255,            5454,            5298,            2684,            2561,            2049,            823,            2583,            2301,            3076,            2413,            4167,            4164,            3071,            2419,            0,            2148,            2955,            2982,            2946,            1019,            1401,            1411,            1323,            5440,            1983,            2130,            4964,            2563,            2519,            3544,            1709,            2787,            4645,            2932,            2956,            2732,            3954,            2718,            3603,            2155,            515,            1517,            1962,            1687,            2518,            3472,            3010,            5163,            2787,            1719,            4730,            4314,            2566,            2778,            5030,            2171,            2097,            2002,            4627,            4011,            4720,            2039,            1736,            2540,            5155,            3793,            4036,            ],        [            3468,            3005,            2493,            4005,            3849,            958,            1980,            1754,            1325,            857,            153,            1350,            1568,            2718,            2715,            3201,            693,            2238,            0,            1229,            1256,            1220,            1129,            1519,            1529,            825,            3991,            3123,            1851,            3515,            837,            793,            2095,            1195,            639,            3196,            1483,            4096,            1283,            2505,            992,            3733,            429,            1633,            1635,            186,            557,            792,            1746,            1284,            3714,            639,            429,            3281,            2865,            840,            1052,            3581,            445,            967,            1707,            3178,            4193,            3271,            909,            2876,            392,            3706,            2344,            4168,            ],        [            4755,            4292,            3143,            5292,            5136,            651,            2167,            1941,            2190,            567,            1440,            759,            2855,            4005,            4002,            4824,            880,            2888,            1287,            0,            432,            205,            1994,            1706,            1716,            1690,            5278,            3736,            3883,            4802,            991,            947,            3382,            1601,            1926,            4483,            2770,            7695,            2570,            3792,            305,            5356,            1089,            2498,            1822,            1101,            1422,            621,            1155,            197,            5001,            1926,            1294,            4568,            4152,            583,            1239,            4868,            954,            1677,            1894,            4465,            5480,            4558,            1774,            3489,            1679,            4993,            3631,            5455,            ],        [            4682,            4219,            3285,            5219,            5063,            870,            2386,            2160,            2117,            786,            1367,            978,            2782,            3932,            3929,            5003,            1099,            3030,            1214,            219,            0,            424,            1921,            1925,            1935,            1617,            5205,            3915,            4062,            4729,            918,            874,            3309,            1820,            1853,            4410,            2697,            4888,            2497,            3719,            256,            5535,            1308,            2425,            2041,            1028,            1349,            424,            1374,            416,            4928,            1853,            1221,            4495,            4079,            618,            1458,            4795,            881,            1759,            2113,            4392,            5407,            4485,            1701,            3668,            1606,            4920,            3558,            5382,            ],        [            4550,            4087,            2938,            5087,            4931,            790,            1962,            1736,            1985,            362,            1235,            898,            2650,            3800,            3797,            4619,            675,            2683,            1082,            441,            571,            0,            1789,            1501,            1511,            1485,            5073,            3531,            3678,            4597,            786,            742,            3177,            1396,            1721,            4278,            2565,            7490,            2365,            3587,            172,            5151,            884,            2293,            1617,            896,            1217,            416,            1294,            64,            4796,            1721,            1089,            4363,            3947,            378,            1034,            4663,            749,            1472,            1689,            4260,            5275,            4353,            1569,            3284,            1474,            4788,            3426,            5250,            ],        [            4928,            4465,            1364,            5465,            5309,            1665,            2327,            2060,            196,            1564,            1282,            2057,            2424,            4178,            4175,            3082,            1400,            1109,            1129,            1936,            1963,            1927,            0,            1412,            1422,            304,            5451,            1994,            2141,            4975,            1544,            1500,            3555,            1306,            1768,            4656,            2943,            2967,            2743,            3965,            1699,            3614,            1136,            504,            1528,            943,            668,            1499,            2453,            1991,            5174,            1768,            700,            4741,            4325,            1547,            1759,            5041,            1152,            1078,            2013,            4638,            4022,            4731,            1020,            1747,            1521,            5166,            3804,            4047,            ],        [            5054,            4591,            1527,            5591,            5435,            2002,            1255,            708,            1216,            1124,            1678,            2394,            2550,            4304,            4301,            3208,            1026,            1272,            1525,            1948,            2300,            1487,            1412,            0,            10,            1208,            5577,            2120,            2267,            5101,            1881,            1837,            3681,            368,            2164,            4782,            3069,            6079,            2869,            4091,            1711,            3740,            1214,            908,            176,            1339,            844,            1511,            2790,            1551,            5300,            2164,            972,            4867,            4451,            1473,            673,            5167,            1844,            444,            661,            4764,            4148,            4857,            2012,            1873,            1917,            5292,            3930,            4173,            ],        [            5064,            4601,            1537,            5601,            5445,            2012,            1265,            718,            1226,            1134,            1688,            2404,            2560,            4314,            4311,            3218,            1036,            1282,            1535,            1958,            2310,            1497,            1422,            10,            0,            1218,            5587,            2130,            2277,            5111,            1891,            1847,            3691,            378,            2174,            4792,            3079,            6089,            2879,            4101,            1721,            3750,            1224,            918,            186,            1349,            854,            1521,            2800,            1561,            5310,            2174,            982,            4877,            4461,            1483,            683,            5177,            1854,            454,            671,            4774,            4158,            4867,            2022,            1883,            1927,            5302,            3940,            4183,            ],        [            5232,            4769,            1668,            5769,            5613,            1361,            2023,            2157,            500,            1260,            978,            1753,            2728,            4482,            4479,            3386,            1096,            1413,            825,            1632,            1659,            1623,            304,            1221,            1231,            0,            5755,            2298,            2445,            5279,            1240,            1196,            3859,            1002,            1464,            4960,            3247,            3271,            3047,            4269,            1395,            3918,            832,            808,            1337,            639,            364,            1195,            2149,            1687,            5478,            1464,            396,            5045,            4629,            1243,            1455,            5345,            848,            774,            2110,            4942,            4326,            5035,            716,            2051,            1217,            5470,            4108,            4351,            ],        [            2344,            1795,            4653,            3422,            480,            4432,            6262,            6166,            5221,            4848,            3838,            4845,            3027,            1363,            2440,            2926,            4684,            4398,            3991,            4970,            4751,            5211,            5417,            5518,            5528,            5721,            0,            4465,            3310,            1466,            4774,            4784,            1896,            5826,            3600,            2825,            2508,            5684,            2708,            2134,            4983,            3458,            4420,            4913,            5634,            4177,            6085,            4783,            5241,            5275,            3131,            3600,            4420,            1232,            1126,            4831,            5043,            2457,            4436,            6495,            6119,            2054,            3610,            2426,            6437,            4178,            3605,            337,            1647,            3585,            ],        [            3694,            3231,            988,            4231,            4075,            3417,            2448,            2700,            1556,            3116,            3034,            3809,            1190,            2944,            2941,            1848,            3152,            733,            2881,            3688,            3715,            3479,            1752,            2052,            2062,            2056,            4217,            0,            907,            3741,            3296,            3252,            2321,            2360,            2619,            3422,            1709,            1268,            1509,            2731,            3451,            2380,            2888,            1248,            2168,            2695,            2420,            3251,            4205,            3543,            3940,            2619,            2452,            3507,            3091,            3299,            2665,            3807,            2904,            2436,            2653,            3404,            2788,            3497,            2772,            513,            2372,            3932,            2570,            2813,            ],        [            2787,            2324,            1343,            3324,            3168,            2292,            2952,            2856,            1911,            3272,            1698,            2705,            283,            2037,            2034,            1607,            3507,            1088,            1851,            4043,            2611,            3635,            2107,            2208,            2218,            2411,            3310,            1155,            0,            2834,            3651,            3607,            1414,            2516,            1712,            2515,            802,            2128,            602,            1824,            3806,            2139,            3243,            1603,            2324,            2037,            2775,            3606,            3101,            3699,            3033,            1712,            2807,            2600,            2184,            3654,            2821,            2900,            3259,            3185,            2809,            2497,            3512,            2590,            3127,            868,            1465,            3025,            1663,            3487,            ],        [            1868,            1319,            4177,            2946,            1292,            3956,            5786,            5690,            4745,            4372,            3362,            4369,            2551,            887,            1964,            2450,            4208,            3922,            3515,            4494,            4275,            4735,            4941,            5042,            5052,            5245,            1434,            3989,            2834,            0,            4298,            4308,            1420,            5350,            3124,            2349,            2032,            4962,            2232,            1658,            4507,            2982,            3944,            4437,            5158,            3701,            5609,            4307,            4765,            4799,            2655,            3124,            3944,            234,            650,            4355,            4567,            1981,            3960,            6019,            5643,            1578,            3134,            1950,            5961,            3702,            3129,            1149,            1171,            3109,            ],        [            4053,            3590,            2656,            4590,            4434,            830,            2104,            1878,            1488,            504,            738,            824,            2153,            3303,            3300,            4374,            817,            2401,            585,            703,            730,            574,            1292,            1643,            1653,            988,            4576,            3286,            3433,            4100,            0,            267,            2680,            1358,            1224,            3781,            2068,            4259,            1868,            3090,            466,            4906,            1026,            1796,            1759,            399,            720,            266,            1220,            638,            4299,            1224,            592,            3866,            3450,            314,            1176,            4166,            252,            1130,            1831,            3763,            4778,            3856,            1072,            3039,            977,            4291,            2929,            4753,            ],        [            4054,            3591,            3090,            4591,            4435,            165,            2538,            2312,            1922,            938,            1172,            557,            2154,            3304,            3301,            3787,            1251,            2835,            1019,            682,            463,            887,            1726,            2077,            2087,            1422,            4577,            3720,            2437,            4101,            434,            0,            2681,            1792,            1225,            3782,            2069,            4693,            1869,            3091,            719,            4319,            1460,            2230,            2193,            833,            1154,            700,            953,            879,            4300,            1225,            1026,            3867,            3451,            748,            1610,            4167,            686,            1564,            2265,            3764,            4779,            3857,            1506,            3473,            978,            4292,            2930,            4754,            ],        [            1373,            910,            2757,            1910,            1754,            2536,            4366,            4270,            3325,            2952,            1942,            2949,            1131,            623,            620,            1106,            2788,            2502,            2095,            3074,            2855,            3315,            3521,            3622,            3632,            3825,            1896,            2569,            1414,            1420,            2878,            2888,            0,            3930,            1704,            1101,            612,            3542,            812,            410,            3087,            1638,            2524,            3017,            3738,            2281,            4189,            2887,            3345,            3379,            1619,            1704,            2524,            1186,            770,            2935,            3147,            1486,            2540,            4599,            4223,            1083,            2098,            1176,            4541,            2282,            1709,            1611,            249,            2073,            ],        [            5335,            4872,            1808,            5872,            5716,            1721,            1173,            492,            1492,            1042,            1596,            2113,            2831,            4585,            4582,            3489,            944,            1553,            1443,            1866,            2019,            1405,            1296,            371,            381,            992,            5858,            2401,            2548,            5382,            1600,            1556,            3962,            0,            2082,            5063,            3350,            6360,            3150,            4372,            1629,            4021,            1132,            1189,            487,            1257,            628,            1429,            2509,            1469,            5581,            2082,            756,            5148,            4732,            1391,            591,            5448,            1208,            228,            445,            5045,            4429,            5138,            1930,            2154,            1835,            5573,            4211,            4454,            ],        [            3077,            2614,            3132,            3614,            3458,            1080,            2619,            2393,            1964,            1496,            486,            1493,            1429,            2327,            2324,            2810,            1332,            2877,            639,            1618,            1399,            1859,            1768,            2158,            2168,            1464,            3600,            2867,            1712,            3124,            1422,            1432,            1704,            1834,            0,            2805,            1092,            3840,            1144,            2114,            1631,            3342,            1068,            2272,            2274,            825,            1196,            1431,            1889,            1923,            3323,            0,            1068,            2890,            2474,            1479,            1691,            3190,            1084,            1606,            2346,            2787,            3802,            2880,            1548,            2580,            247,            3315,            1953,            3777,            ],        [            1062,            818,            3858,            921,            2683,            3637,            5467,            5371,            4426,            4053,            3043,            4050,            2232,            1250,            1645,            2131,            3889,            3603,            3196,            4175,            3956,            4416,            4622,            4723,            4733,            4926,            2825,            3670,            2515,            2349,            3979,            3989,            1101,            5031,            2805,            0,            1713,            4643,            1913,            691,            4188,            1517,            3625,            4118,            4839,            3382,            5290,            3988,            4446,            4480,            630,            2805,            3625,            2115,            1699,            4036,            4248,            1175,            3641,            5700,            5324,            772,            1109,            187,            5642,            3383,            2810,            2540,            1178,            1084,            ],        [            1985,            1522,            2145,            2522,            2366,            1924,            3754,            3658,            2713,            2340,            1330,            2337,            519,            1235,            1232,            1718,            2176,            1890,            1483,            2462,            2243,            2703,            2909,            3010,            3020,            3213,            2508,            1957,            802,            2032,            2266,            2276,            612,            3318,            1092,            1713,            0,            2930,            200,            1022,            2475,            2250,            1912,            2405,            3126,            1669,            3577,            2275,            2733,            2767,            2231,            1092,            1912,            1798,            1382,            2323,            2535,            2098,            1928,            3394,            3611,            1695,            2710,            1788,            3929,            1670,            1097,            2223,            861,            2685,            ],        [            4187,            3724,            1481,            4724,            4568,            3910,            2966,            3218,            2049,            3634,            3527,            4302,            1683,            3437,            3434,            2341,            3645,            1226,            3374,            4181,            4208,            3997,            2245,            2570,            2580,            2549,            4710,            1169,            1400,            4234,            3789,            3745,            2814,            2878,            3112,            3915,            2202,            0,            2002,            3224,            3944,            2873,            3381,            1741,            2686,            3188,            2913,            3744,            4698,            4061,            4433,            3112,            2945,            4000,            3584,            3792,            3183,            4300,            3397,            2954,            3171,            3897,            3281,            3990,            3265,            1006,            2865,            4425,            3063,            3306,            ],        [            2185,            1722,            1945,            2722,            2566,            1724,            3554,            3458,            2513,            2140,            1130,            2137,            319,            1435,            1432,            1918,            1976,            1690,            1283,            2262,            2043,            2503,            2709,            2810,            2820,            3013,            2708,            1757,            602,            2232,            2066,            2076,            812,            3118,            1144,            1913,            200,            2730,            0,            1222,            2275,            2450,            1712,            2205,            2926,            1469,            3377,            2075,            2533,            2567,            2431,            1144,            1712,            1998,            1582,            2123,            2335,            2298,            1728,            3194,            3411,            1895,            2910,            1988,            3729,            1470,            897,            2423,            1061,            2885,            ],        [            963,            500,            3167,            1500,            1992,            2946,            4776,            4680,            3735,            3362,            2352,            3359,            1541,            861,            954,            1440,            3198,            2912,            2505,            3484,            3265,            3725,            3931,            4032,            4042,            4235,            2134,            2979,            1824,            1658,            3288,            3298,            410,            4340,            2114,            691,            1022,            3952,            1222,            0,            3497,            1972,            2934,            3427,            4148,            2691,            4599,            3297,            3755,            3789,            1209,            2114,            2934,            1424,            1008,            3345,            3557,            1076,            2950,            5009,            4633,            673,            1688,            766,            4951,            2692,            2119,            1849,            487,            1663,            ],        [            4458,            3995,            3061,            4995,            4839,            815,            2134,            1908,            1893,            534,            1143,            998,            2558,            3708,            3705,            4779,            847,            2806,            990,            269,            671,            172,            1697,            1673,            1683,            1393,            4981,            3691,            3838,            4505,            694,            650,            3085,            1568,            1629,            4186,            2473,            4664,            2273,            3495,            0,            5311,            1056,            2201,            1789,            804,            1125,            200,            1394,            108,            4704,            1629,            997,            4271,            3855,            394,            1206,            4571,            657,            1535,            1861,            4168,            5183,            4261,            1477,            3444,            1382,            4696,            3334,            5158,            ],        [            2467,            2472,            2816,            596,            3316,            4174,            4425,            4329,            3384,            4745,            3580,            5637,            2422,            2185,            1018,            532,            4980,            2561,            3733,            5516,            5543,            5108,            3580,            3681,            3691,            3884,            3458,            2628,            2139,            2982,            5124,            4526,            1638,            3989,            3342,            1517,            2250,            3601,            2450,            1972,            5279,            0,            4716,            3076,            3797,            4523,            4248,            5079,            6033,            5172,            887,            3342,            4280,            2748,            2332,            4573,            4294,            2580,            4732,            4658,            4282,            2177,            408,            1592,            4600,            2341,            3347,            3173,            1811,            433,            ],        [            3779,            3316,            2382,            4316,            4160,            847,            1755,            1529,            1214,            632,            464,            1239,            1879,            3029,            3026,            4100,            468,            2127,            311,            1118,            1145,            995,            1018,            1303,            1313,            714,            4302,            3012,            3159,            3826,            726,            682,            2406,            1084,            950,            3507,            1794,            3985,            1594,            2816,            881,            4632,            0,            1522,            1419,            125,            446,            681,            1635,            1059,            4025,            950,            318,            3592,            3176,            729,            827,            3892,            334,            856,            1482,            3489,            4504,            3582,            798,            2765,            703,            4017,            2655,            4479,            ],        [            4424,            3961,            860,            4961,            4805,            2169,            2302,            1556,            308,            2068,            1786,            2561,            1920,            3674,            3671,            2578,            1904,            605,            1633,            2440,            2467,            2431,            504,            908,            918,            808,            4947,            1490,            1637,            4471,            2048,            2004,            3051,            1216,            2272,            4152,            2439,            2463,            2239,            3461,            2203,            3110,            1640,            0,            1024,            1447,            1172,            2003,            2957,            2495,            4670,            2272,            1204,            4237,            3821,            2051,            2263,            4537,            1656,            1582,            1509,            4134,            3518,            4227,            1524,            1243,            2025,            4662,            3300,            3543,            ],        [            5261,            4798,            1734,            5798,            5642,            1826,            1079,            532,            1532,            948,            1502,            2218,            2757,            4511,            4508,            3415,            850,            1479,            1349,            1772,            2124,            1311,            1336,            297,            307,            1032,            5784,            2327,            2474,            5308,            1705,            1661,            3888,            192,            1988,            4989,            3276,            3300,            3076,            4298,            1535,            3947,            1038,            1115,            0,            1163,            668,            1335,            2614,            1375,            5507,            1988,            796,            5074,            4658,            1297,            497,            5374,            1668,            268,            485,            4971,            4355,            5064,            1836,            2080,            1741,            5499,            4137,            4380,            ],        [            3654,            3191,            2307,            4191,            4035,            772,            1794,            1568,            1139,            671,            339,            1164,            1754,            2904,            2901,            4025,            507,            2052,            186,            1043,            1070,            1034,            943,            1333,            1343,            639,            4177,            2937,            3084,            3701,            651,            607,            2281,            1009,            825,            3382,            1669,            3910,            1469,            2691,            806,            4557,            243,            1447,            1449,            0,            371,            606,            1560,            1098,            3900,            825,            243,            3467,            3051,            654,            866,            3767,            259,            781,            1521,            3364,            4379,            3457,            723,            2690,            578,            3892,            2530,            4354,            ],        [            4025,            3562,            2032,            4562,            4406,            1093,            1659,            1889,            864,            992,            710,            1485,            3092,            3275,            3272,            3750,            828,            1777,            557,            1364,            1391,            1355,            668,            857,            867,            364,            4548,            2662,            2809,            4072,            972,            928,            2652,            638,            1196,            3753,            2040,            3635,            1840,            3062,            1127,            4282,            564,            1172,            973,            371,            0,            927,            1881,            1419,            4271,            1196,            128,            3838,            3422,            975,            1187,            4138,            580,            410,            1842,            3735,            4690,            3828,            352,            2415,            949,            4263,            2901,            4715,            ],        [            4258,            3795,            2861,            4795,            4639,            615,            1984,            1758,            1693,            384,            943,            1007,            2358,            3508,            3505,            4579,            697,            2606,            790,            437,            913,            372,            1497,            1523,            1533,            1193,            4781,            3491,            3638,            4305,            494,            450,            2885,            1418,            1429,            3986,            2273,            4464,            2073,            3295,            200,            5111,            906,            2001,            1639,            604,            925,            0,            1403,            308,            4504,            1429,            797,            4071,            3655,            194,            1056,            4371,            457,            1335,            1711,            3968,            4983,            4061,            1277,            3244,            1182,            4496,            3134,            4958,            ],        [            4546,            4083,            3655,            5083,            4927,            657,            3103,            2877,            2487,            1503,            1737,            390,            2646,            3796,            3793,            4279,            1816,            3400,            1584,            932,            713,            1137,            2291,            2642,            2652,            1987,            5069,            4285,            2929,            4593,            999,            963,            3173,            2357,            1717,            4274,            2561,            5258,            2361,            3583,            969,            4811,            2025,            2795,            2758,            1398,            1719,            1137,            0,            1129,            4792,            1717,            1591,            4359,            3943,            1313,            2175,            4659,            1251,            2129,            2830,            4256,            5271,            4349,            2071,            4038,            1470,            4784,            3422,            5246,            ],        [            4614,            4151,            3002,            5151,            4995,            782,            2026,            1800,            2049,            426,            1299,            890,            2714,            3864,            3861,            4683,            739,            2747,            1146,            377,            563,            64,            1853,            1565,            1575,            1549,            5137,            3595,            3742,            4661,            850,            806,            3241,            1460,            1785,            4342,            2629,            4568,            2429,            3651,            108,            5215,            948,            2357,            1681,            960,            1281,            308,            1286,            0,            4860,            1785,            1153,            4427,            4011,            442,            1098,            4727,            813,            1536,            1753,            4324,            5339,            4417,            1633,            3348,            1538,            4852,            3490,            5314,            ],        [            1580,            1336,            4376,            291,            3201,            4155,            5985,            5889,            4944,            4571,            3561,            4568,            2750,            1768,            2163,            1419,            4407,            4121,            3714,            4693,            4474,            4934,            5140,            5241,            5251,            5444,            3343,            4188,            3033,            2867,            4497,            4507,            1619,            5549,            3323,            630,            2231,            5161,            2431,            1209,            4706,            887,            4143,            4636,            5357,            3900,            5808,            4506,            4964,            4998,            0,            3323,            4143,            2633,            2217,            4554,            4766,            1693,            4159,            6218,            5842,            1290,            479,            705,            6160,            3901,            3328,            3058,            1696,            454,            ],        [            3077,            2614,            3132,            3614,            3458,            1080,            2619,            2393,            1964,            1496,            486,            1493,            1429,            2327,            2324,            2810,            1332,            2877,            639,            1618,            1399,            1859,            1768,            2158,            2168,            1464,            3600,            2867,            1712,            3124,            1422,            1432,            1704,            1834,            0,            2805,            1092,            3840,            1144,            2114,            1631,            3342,            1068,            2272,            2274,            825,            1196,            1431,            1889,            1923,            3323,            0,            1068,            2890,            2474,            1479,            1691,            3190,            1084,            1606,            2346,            2787,            3802,            2880,            1548,            2580,            247,            3315,            1953,            3777,            ],        [            3897,            3434,            2064,            4434,            4278,            965,            1787,            1761,            896,            864,            582,            1357,            3124,            3147,            3144,            3782,            700,            1809,            429,            1236,            1263,            1227,            700,            985,            995,            396,            4420,            2694,            2841,            3944,            844,            800,            2524,            766,            1068,            3625,            1912,            3667,            1712,            2934,            999,            4314,            436,            1204,            1101,            243,            128,            799,            1753,            1291,            4143,            1068,            0,            3710,            3294,            847,            1059,            4010,            452,            538,            1714,            3607,            4722,            3700,            480,            2447,            821,            4135,            2773,            4747,            ],        [            1634,            1085,            3943,            2712,            1058,            3722,            5552,            5456,            4511,            4138,            3128,            4135,            2317,            653,            1730,            2216,            3974,            3688,            3281,            4260,            4041,            4501,            4707,            4808,            4818,            5011,            1200,            3755,            2600,            234,            4064,            4074,            1186,            5116,            2890,            2115,            1798,            4974,            1998,            1424,            4273,            2748,            3710,            4203,            4924,            3467,            5375,            4073,            4531,            4565,            2421,            2890,            3710,            0,            416,            4121,            4333,            1747,            3726,            5785,            5409,            1344,            2900,            1716,            5727,            3468,            2895,            915,            937,            2875,            ],        [            1218,            669,            3527,            2296,            984,            3306,            5136,            5040,            4095,            3722,            2712,            3719,            1901,            237,            1314,            1800,            3558,            3272,            2865,            3844,            3625,            4085,            4291,            4392,            4402,            4595,            1126,            3339,            2184,            650,            3648,            3658,            770,            4700,            2474,            1699,            1382,            4312,            1582,            1008,            3857,            2332,            3294,            3787,            4508,            3051,            4959,            3657,            4115,            4149,            2005,            2474,            3294,            416,            0,            3705,            3917,            1331,            3310,            5369,            4993,            928,            2484,            1300,            5311,            3052,            2479,            841,            521,            2459,            ],        [            4491,            4028,            2766,            5028,            4872,            775,            1790,            1564,            1926,            190,            1176,            1158,            2591,            3741,            3738,            4447,            503,            2511,            1023,            701,            831,            260,            1730,            1329,            1339,            1426,            5014,            3467,            3506,            4538,            654,            610,            3118,            1224,            1662,            4219,            2506,            4332,            2306,            3528,            432,            4979,            712,            2234,            1445,            837,            1158,            284,            1554,            324,            4737,            1662,            1030,            4304,            3888,            0,            862,            4604,            617,            1300,            1517,            4201,            5216,            4294,            1510,            3112,            1415,            4729,            3367,            5191,            ],        [            5431,            4968,            1904,            5968,            5812,            1675,            928,            702,            1593,            797,            1351,            2067,            2927,            4681,            4678,            3585,            699,            1649,            1198,            1621,            1973,            1160,            1506,            467,            477,            1202,            5954,            2605,            2644,            5478,            1554,            1510,            4058,            362,            1837,            5159,            3446,            6456,            3246,            4468,            1384,            4117,            887,            1285,            583,            1012,            838,            1184,            2463,            1224,            5677,            1837,            966,            5244,            4828,            1146,            0,            5544,            1517,            438,            655,            5141,            4525,            5234,            1685,            2250,            1590,            5669,            4307,            4550,            ],        [            113,            662,            4243,            1984,            2315,            4022,            5843,            5880,            4811,            4438,            3428,            4435,            2617,            1094,            2030,            2516,            4274,            3988,            3581,            4560,            4341,            4801,            5007,            5232,            5242,            5311,            2457,            4055,            2900,            1981,            4364,            4374,            1486,            5540,            3190,            1175,            2098,            5028,            2298,            1076,            4573,            2580,            4010,            4503,            5224,            3767,            5675,            4373,            4831,            4865,            1693,            3190,            4010,            1747,            1331,            4421,            4633,            0,            4026,            6085,            5709,            403,            2172,            988,            6027,            3768,            3195,            2172,            1563,            2147,            ],        [            3801,            3338,            2404,            4338,            4182,            671,            1891,            1665,            1236,            768,            486,            1063,            1901,            3051,            3048,            4122,            604,            2149,            333,            942,            969,            1131,            1040,            1325,            1335,            736,            4324,            3034,            3181,            3848,            550,            506,            2428,            1106,            972,            3529,            1816,            4007,            1616,            2838,            705,            4654,            340,            1544,            1441,            147,            468,            505,            1459,            1195,            4047,            972,            340,            3614,            3198,            553,            963,            3914,            0,            878,            1618,            3511,            4526,            3604,            820,            2787,            725,            4039,            2677,            4501,            ],        [            5411,            4948,            1884,            5948,            5792,            1493,            1249,            264,            1264,            1118,            1110,            1885,            2907,            4661,            4658,            3565,            1228,            1629,            957,            1764,            1791,            1481,            1068,            447,            457,            764,            5934,            2477,            2624,            5458,            1372,            1328,            4038,            228,            1596,            5139,            3426,            6436,            3226,            4448,            1527,            4097,            1208,            1572,            563,            771,            400,            1327,            2281,            1545,            5657,            1596,            528,            5224,            4808,            1375,            667,            5524,            980,            0,            217,            5121,            4505,            5214,            1576,            2230,            1349,            5649,            4287,            4530,            ],        [            4208,            3745,            2215,            4745,            4589,            1276,            2298,            47,            1047,            1175,            893,            1668,            3275,            3458,            3455,            3933,            1011,            1960,            740,            1547,            1574,            1538,            851,            1837,            1847,            547,            4731,            2845,            2992,            4255,            1155,            1111,            2835,            1732,            1379,            3936,            2223,            3818,            2023,            3245,            1310,            4465,            747,            1355,            1953,            554,            183,            1110,            2064,            1602,            4454,            1379,            311,            4021,            3605,            1158,            1370,            4321,            763,            1417,            0,            3918,            4873,            4011,            1359,            2598,            1132,            4446,            3084,            4898,            ],        [            290,            259,            3840,            1581,            1912,            3619,            5664,            5353,            4408,            4035,            3025,            4032,            2214,            691,            1627,            2113,            3871,            3585,            3178,            4157,            3938,            4398,            4604,            4705,            4715,            4908,            2054,            3652,            2497,            1578,            3961,            3971,            1083,            5013,            2787,            772,            1695,            4625,            1895,            673,            4170,            2177,            3607,            4100,            4821,            3364,            5272,            3970,            4428,            4462,            1290,            2787,            3607,            1344,            928,            4018,            4230,            403,            3623,            5682,            5306,            0,            1769,            585,            5624,            3365,            2792,            1769,            1160,            1744,            ],        [            2059,            1815,            3224,            188,            3680,            4634,            5048,            4737,            3792,            5050,            4040,            5047,            3229,            2247,            1426,            940,            4886,            2969,            4193,            5172,            4953,            5413,            3988,            4089,            4099,            4292,            3822,            3036,            3512,            3346,            4976,            4986,            2098,            4397,            3802,            1109,            2710,            4009,            2910,            1688,            5185,            408,            4622,            3484,            4205,            4379,            4656,            4985,            5443,            5477,            479,            3802,            4622,            3112,            2696,            5033,            5245,            2172,            4638,            5066,            4690,            1769,            0,            1184,            5008,            2749,            3807,            3537,            2175,            25,            ],        [            875,            631,            3933,            996,            2284,            3712,            5375,            5446,            4501,            4128,            3118,            4125,            2307,            1063,            1720,            2206,            3964,            3678,            3271,            4250,            4031,            4491,            4697,            4798,            4808,            5001,            2426,            3745,            2590,            1950,            4054,            4064,            1176,            5106,            2880,            187,            1788,            4718,            1988,            766,            4263,            1592,            3700,            4193,            4914,            3457,            5365,            4063,            4521,            4555,            705,            2880,            3700,            1716,            1300,            4111,            4323,            988,            3716,            5775,            5399,            585,            1184,            0,            5717,            3458,            2885,            2141,            1253,            1159,            ],        [            5469,            5006,            1942,            6006,            5850,            2054,            1307,            1103,            1631,            1176,            1730,            2446,            2965,            4719,            4716,            3623,            1078,            1687,            1577,            2000,            2352,            1539,            1827,            505,            515,            1980,            5992,            2535,            2682,            5516,            1933,            1889,            4096,            286,            2216,            5197,            3484,            6494,            3284,            4506,            1763,            4155,            1266,            1323,            621,            1391,            1712,            1563,            2842,            1603,            5715,            2216,            1584,            5282,            4866,            1525,            725,            5582,            1896,            58,            1056,            5179,            4563,            5272,            0,            2288,            1969,            5707,            4345,            4588,            ],        [            4039,            3576,            475,            4576,            4420,            2904,            1917,            1988,            1043,            2404,            2521,            3296,            1535,            3289,            3286,            2193,            2639,            220,            2368,            3175,            3202,            2767,            1239,            1340,            1350,            1543,            4562,            1105,            1252,            4086,            2783,            2739,            2666,            1648,            3007,            3767,            2054,            2078,            1854,            3076,            2938,            2725,            2375,            735,            1456,            2182,            1907,            2738,            3692,            2831,            4285,            3007,            1939,            3852,            3436,            2786,            1953,            4152,            2391,            1724,            1941,            3749,            3133,            3842,            2259,            0,            2760,            4277,            2915,            3158,            ],        [            3082,            2619,            2885,            3619,            3463,            833,            2372,            2146,            1717,            1249,            239,            1246,            1182,            2332,            2329,            2815,            1085,            2630,            392,            1371,            1152,            1612,            1521,            1911,            1921,            1217,            3605,            2620,            1465,            3129,            1175,            1185,            1709,            1587,            247,            2810,            1097,            3593,            897,            2119,            1384,            3347,            821,            2025,            2027,            578,            949,            1184,            1642,            1676,            3328,            247,            821,            2895,            2479,            1232,            1444,            3195,            837,            1359,            2099,            2792,            3807,            2885,            1301,            2333,            0,            3320,            1958,            3782,            ],        [            2059,            1510,            4368,            3137,            143,            4147,            5810,            5881,            4936,            4563,            3553,            4560,            2742,            1078,            2155,            2641,            4399,            4113,            3706,            4685,            4466,            4926,            5132,            5233,            5243,            5436,            337,            4180,            3025,            1149,            4489,            4499,            1611,            5541,            3315,            2540,            2223,            5153,            2423,            1849,            4698,            3173,            4135,            4628,            5349,            3892,            5800,            4498,            4956,            4990,            2846,            3315,            4135,            915,            841,            4546,            4758,            2172,            4151,            6210,            5834,            1769,            3325,            2141,            6152,            3893,            3320,            0,            1362,            3300,            ],        [            1450,            806,            3006,            1987,            1505,            2785,            4615,            4519,            3574,            3201,            2191,            3198,            1380,            374,            793,            1279,            3037,            2751,            2344,            3323,            3104,            3564,            3770,            3871,            3881,            4074,            1647,            2818,            1663,            1171,            3127,            3137,            249,            4179,            1953,            1178,            861,            3791,            1061,            487,            3336,            1811,            2773,            3266,            3987,            2530,            4438,            3136,            3594,            3628,            1696,            1953,            2773,            937,            521,            3184,            3396,            1563,            2789,            4255,            4472,            1160,            2175,            1253,            4790,            2531,            1958,            1362,            0,            2150,            ],        [            2034,            1790,            4830,            163,            3655,            4609,            6439,            6343,            5398,            5025,            4015,            5022,            3204,            2222,            1451,            965,            4861,            4575,            4168,            5147,            4928,            5388,            5594,            5695,            5705,            5898,            3797,            4642,            3487,            3321,            4951,            4961,            2073,            6003,            3777,            1084,            2685,            5615,            2885,            1663,            5160,            433,            4597,            5090,            5811,            4354,            6262,            4960,            5418,            5452,            454,            3777,            4597,            3087,            2671,            5008,            5220,            2147,            4613,            6672,            6296,            1744,            25,            1159,            6614,            4355,            3782,            3512,            2150,            0,            ],        ], 'starts': [35, 32, 12, 0, 47], 'ends': [35, 32, 12, 0, 47]},    'NW-27': {'distance_matrix': [        [            0,            1368,            1167,            1398,            1042,            3040,            1131,            3524,            3111,            3877,            4647,            4522,            3539,            5152,            2913,            2678,            2522,            2497,            5319,            4482,            4744,            2859,            3433,            4098,            4499,            4559,            2059,            4440,            3788,            4678,            3895,            4072,            3371,            3113,            2750,            5147,            3918,            2663,            3096,            5432,            4223,            4866,            4733,            3385,            3381,            2662,            4683,            2462,            2506,            1727,            3396,            3579,            5691,            3904,            2012,            3538,            5036,            2973,            4952,            2232,            4082,            5427,            2936,            2937,            5241,            2126,            2694,            5679,            3404,            2476,            3810,            2874,            3845,            5755,            3443,            ],        [            1446,            0,            840,            587,            966,            1672,            637,            2713,            2784,            3550,            3836,            3711,            3212,            4341,            3383,            1867,            2195,            2170,            4992,            4940,            4417,            2532,            2660,            3771,            4172,            4232,            1732,            3629,            3461,            3867,            3568,            3745,            3616,            2302,            1939,            4336,            3591,            2336,            2773,            4621,            3896,            4539,            4406,            3630,            3626,            2619,            3872,            2135,            2179,            916,            3069,            2768,            5364,            3093,            1685,            2727,            4225,            2590,            4141,            1905,            3271,            5100,            2609,            2126,            4914,            1799,            2292,            5352,            2593,            2149,            2999,            2547,            2501,            4944,            2632,            ],        [            606,            1269,            0,            663,            943,            2526,            1032,            2789,            2376,            3142,            3912,            3787,            2804,            4417,            3519,            1943,            1787,            1762,            4584,            3747,            4009,            2124,            2252,            3363,            3764,            3824,            1324,            3705,            3053,            3943,            3160,            3337,            3977,            2378,            2015,            4412,            3183,            1928,            2365,            4697,            3488,            4131,            3998,            3991,            3987,            1927,            3948,            1727,            1771,            992,            2661,            2844,            4956,            3169,            1277,            2803,            4301,            2182,            4217,            1497,            3347,            4692,            2201,            2202,            4506,            1391,            1884,            4944,            2669,            1741,            3075,            2139,            2093,            5020,            2708,            ],        [            1325,            923,            719,            0,            1575,            1863,            1246,            2126,            2666,            3432,            3249,            3124,            3094,            3754,            3262,            1280,            2074,            2049,            4874,            4353,            4299,            2411,            2539,            3653,            4054,            4114,            1611,            3042,            3387,            3280,            3450,            3627,            3495,            1715,            1352,            3749,            3473,            2319,            2652,            4034,            3778,            4421,            4288,            3509,            3505,            2032,            3285,            2014,            2058,            329,            2995,            2181,            5246,            2506,            1564,            2140,            3638,            2469,            3554,            1784,            2684,            4812,            2592,            1539,            4796,            1678,            2171,            5064,            2006,            2028,            2412,            2426,            2380,            4357,            2045,            ],        [            305,            968,            691,            922,            0,            2785,            731,            3048,            2635,            3401,            4171,            4046,            3063,            4676,            3218,            2202,            2046,            2021,            4843,            4006,            4268,            2383,            2511,            3622,            4023,            4083,            1583,            3964,            3312,            4202,            3419,            3596,            3676,            2637,            2274,            4671,            3442,            2187,            2624,            4956,            3747,            4390,            4257,            3690,            3686,            2186,            4207,            1986,            2030,            1251,            2920,            3103,            5215,            3428,            1536,            3062,            4560,            2441,            4476,            1756,            3606,            4951,            2460,            2461,            4765,            1650,            2143,            5203,            2928,            2000,            3334,            2398,            2352,            5279,            2967,            ],        [            1856,            1454,            1250,            1039,            2106,            0,            1777,            1838,            2378,            3144,            2961,            2836,            2806,            3466,            4110,            992,            2605,            2580,            4586,            4065,            4011,            2942,            3070,            3365,            3766,            3826,            2142,            2754,            3099,            2992,            3162,            3339,            4343,            1427,            1064,            3461,            3185,            2031,            3183,            3746,            3490,            4133,            4000,            4357,            4353,            1744,            2997,            2545,            2589,            577,            2707,            1893,            4958,            2218,            2095,            1852,            3350,            3000,            3266,            2315,            2396,            4524,            2304,            1251,            4508,            2209,            2702,            4776,            1718,            2559,            2124,            2957,            2911,            4069,            1757,            ],        [            634,            706,            1020,            1251,            329,            2378,            0,            3377,            2964,            3730,            4500,            4375,            3392,            5005,            3547,            2531,            2375,            2350,            5172,            4335,            4597,            2712,            2840,            3951,            4352,            4412,            1912,            4293,            3641,            4531,            3748,            3925,            4005,            2966,            2603,            5000,            3771,            2516,            2953,            5285,            4076,            4719,            4586,            4019,            4015,            2515,            4536,            2315,            2359,            1580,            3249,            3432,            5544,            3757,            1865,            3391,            4889,            2770,            4805,            2085,            3935,            5280,            2789,            2790,            5094,            1979,            2472,            5532,            3257,            2329,            3663,            2727,            2681,            5608,            3296,            ],        [            3019,            2535,            2413,            2664,            3356,            1037,            2858,            0,            1432,            2198,            2205,            2080,            1860,            2710,            3120,            1351,            1945,            1632,            3640,            3309,            3065,            2180,            2308,            2419,            2820,            2880,            1960,            1998,            2109,            2236,            2216,            2393,            3353,            380,            1678,            2705,            2239,            984,            2635,            2990,            2544,            3187,            3054,            3367,            3363,            697,            2241,            2363,            1407,            1614,            1717,            1137,            4012,            735,            1553,            1096,            2594,            2818,            2510,            1227,            1640,            3748,            1257,            495,            3562,            2027,            2520,            4000,            366,            1899,            1368,            2121,            1963,            3313,            1001,            ],        [            3799,            3024,            3193,            3444,            3676,            1526,            3347,            1718,            0,            1696,            1492,            1367,            1358,            1997,            2618,            1840,            1862,            1549,            3138,            2573,            2563,            2097,            2225,            1917,            2318,            2378,            2740,            1262,            1607,            1500,            1714,            1891,            2851,            1160,            1912,            1969,            1737,            1764,            2472,            2254,            2042,            2685,            2552,            2865,            2861,            1477,            1505,            2624,            1324,            2103,            1215,            450,            3510,            1660,            2333,            1165,            1858,            2655,            1797,            2007,            904,            3032,            1202,            794,            3060,            2288,            3272,            3284,            1451,            1816,            870,            2038,            1880,            2577,            506,            ],        [            3573,            3633,            2967,            3218,            3910,            2665,            3956,            2485,            1139,            0,            1684,            2134,            340,            3136,            2739,            2979,            1966,            1653,            2018,            1181,            1443,            2201,            2329,            221,            1198,            1258,            2514,            1363,            1728,            1601,            912,            1089,            2972,            2299,            3051,            2070,            41,            1253,            2593,            2355,            722,            1565,            1432,            2986,            2982,            1574,            1606,            2728,            1428,            3547,            1336,            1589,            2390,            2799,            2107,            2304,            1959,            2776,            2453,            1781,            967,            2126,            1306,            1933,            1940,            2392,            3393,            2378,            2365,            1920,            2009,            2142,            1984,            1453,            1645,            ],        [            4267,            3881,            3661,            3912,            4533,            2383,            4204,            1849,            1721,            1452,            0,            450,            1114,            2128,            3321,            2697,            2660,            2347,            2894,            1942,            2319,            2895,            3023,            1673,            2074,            2134,            3208,            1101,            2310,            1339,            1470,            2156,            3554,            2017,            2734,            1338,            1493,            1947,            3175,            1623,            1416,            2441,            2308,            3568,            3564,            2334,            1345,            3422,            2122,            2960,            1918,            1307,            3266,            1791,            2801,            1296,            1357,            3358,            769,            2475,            743,            2401,            2000,            1651,            2497,            3086,            3975,            2653,            1769,            2614,            1001,            2836,            2678,            1946,            1363,            ],        [            3817,            3431,            3211,            3462,            4083,            1933,            3754,            2106,            1271,            1002,            972,            0,            664,            2385,            2871,            2247,            2210,            1897,            2444,            1962,            1869,            2445,            2573,            1223,            1624,            1684,            2758,            651,            1860,            889,            1020,            1197,            3104,            1567,            2991,            1358,            1043,            1497,            2725,            1643,            966,            1991,            1858,            3118,            3114,            1884,            894,            2972,            1672,            2510,            1468,            857,            2816,            2048,            2351,            1553,            1247,            2908,            1741,            2025,            293,            2421,            1550,            1201,            2366,            2636,            3525,            2673,            1858,            2164,            1258,            2386,            2228,            1966,            913,            ],        [            3233,            3293,            2627,            2878,            3570,            2325,            3616,            2145,            799,            338,            1344,            1794,            0,            2796,            2399,            2639,            1626,            1313,            1780,            943,            1205,            1861,            1989,            559,            960,            1020,            2174,            1023,            1388,            1261,            356,            533,            2632,            1959,            3402,            1730,            379,            913,            2253,            2015,            382,            1327,            1194,            2646,            2642,            1234,            1266,            2388,            1088,            3207,            996,            1249,            2152,            2459,            1767,            1964,            1619,            2436,            2113,            1441,            627,            1888,            966,            1593,            1702,            2052,            3053,            2140,            2025,            1580,            1669,            1802,            1644,            1215,            1305,            ],        [            4034,            3259,            3428,            3679,            3911,            1761,            3582,            1175,            1113,            1791,            857,            732,            1453,            0,            2713,            2075,            2097,            1784,            3233,            2277,            2658,            2332,            2904,            2012,            2413,            2473,            2975,            1339,            1702,            1577,            1809,            2491,            2946,            1395,            2060,            1673,            1832,            1999,            2567,            1958,            1698,            2780,            2647,            2960,            2956,            1712,            1680,            3524,            1559,            2338,            1310,            685,            3605,            1117,            2568,            622,            1692,            2750,            1162,            2242,            1025,            2736,            1437,            1029,            2832,            2523,            3367,            2988,            1095,            2051,            327,            2273,            3189,            2281,            741,            ],        [            5578,            5923,            5722,            5953,            5597,            7589,            5686,            7409,            6063,            6829,            7555,            7430,            6491,            8060,            0,            7903,            5683,            5590,            8271,            7434,            7696,            5083,            4929,            7050,            7451,            7511,            4801,            7325,            6740,            7563,            6847,            7024,            4867,            7223,            8666,            8032,            6870,            6177,            4592,            8317,            7175,            7818,            7685,            4881,            4877,            6498,            7568,            4327,            6352,            6282,            6348,            6513,            8643,            7723,            7031,            7228,            7921,            4775,            7860,            6705,            6967,            8379,            6230,            6857,            8193,            6089,            4170,            8631,            7289,            5617,            6933,            5451,            5341,            7706,            6569,            ],        [            3074,            2299,            2468,            1840,            2951,            801,            2622,            846,            1386,            2152,            1969,            1844,            1814,            2474,            3118,            0,            2121,            1808,            3594,            3073,            3019,            2356,            2484,            2373,            2774,            2834,            2015,            1762,            2107,            2000,            2170,            2347,            3351,            435,            1187,            2469,            2193,            1039,            2972,            2754,            2498,            3141,            3008,            3365,            3361,            752,            2005,            2418,            1583,            1378,            1715,            901,            3966,            1226,            1608,            860,            2358,            3155,            2274,            1282,            1404,            3532,            1312,            259,            3516,            2082,            2575,            3784,            726,            2075,            1132,            2297,            2139,            3077,            765,            ],        [            2436,            2496,            1830,            2081,            2773,            2611,            2819,            2135,            1085,            1851,            2577,            2452,            1513,            3082,            1396,            2925,            0,            227,            3293,            2456,            2718,            459,            587,            2072,            2473,            2533,            901,            2347,            1762,            2585,            1869,            2046,            1629,            1628,            3688,            3054,            1892,            903,            914,            3339,            2197,            2840,            2707,            1643,            1639,            1224,            2590,            1082,            452,            2410,            1370,            1535,            3665,            2745,            970,            2250,            2943,            1097,            2882,            1033,            1989,            3401,            956,            1879,            3215,            746,            1239,            3653,            2015,            274,            1955,            352,            242,            2728,            1296,            ],        [            2209,            2269,            1603,            1854,            2546,            2281,            2592,            1908,            904,            1670,            2396,            2271,            1332,            2901,            1485,            3134,            529,            0,            3112,            2275,            2537,            548,            676,            1891,            2292,            2352,            901,            2166,            1581,            2404,            1688,            1865,            1718,            1401,            3206,            2873,            1711,            676,            1003,            3158,            2016,            2659,            2526,            1732,            1728,            997,            2409,            1171,            225,            2183,            1189,            1205,            3484,            2157,            743,            2061,            2762,            1186,            2701,            806,            1808,            3220,            729,            1549,            3034,            835,            1328,            3472,            1788,            363,            1774,            489,            365,            2547,            1069,            ],        [            5758,            5818,            5152,            5403,            6095,            4717,            6141,            4481,            3324,            2761,            2632,            3082,            2525,            4760,            4303,            5031,            4151,            3838,            0,            2473,            1895,            4386,            4494,            2982,            1980,            1710,            4699,            2583,            3913,            2821,            2777,            2227,            4536,            4351,            5366,            2387,            2802,            3438,            4157,            1549,            2907,            2017,            1884,            4550,            4546,            3759,            2324,            5114,            3613,            5732,            3521,            3641,            1179,            4423,            4292,            3928,            2336,            4340,            3386,            3966,            3077,            1862,            3491,            3985,            1676,            4577,            4957,            2114,            4401,            4105,            3633,            4327,            4779,            2477,            3697,            ],        [            5523,            4798,            4917,            5168,            5450,            3300,            5121,            3064,            2638,            2526,            1215,            1665,            2290,            3343,            4238,            3614,            3916,            3603,            1475,            0,            1660,            4151,            4279,            2747,            1745,            1475,            4464,            1166,            3227,            1404,            2387,            214,            4471,            2934,            3949,            970,            2567,            3203,            4092,            1255,            2672,            1782,            1649,            4485,            4481,            3251,            907,            4678,            3378,            3877,            2835,            2224,            1847,            3006,            4057,            2511,            919,            4275,            1969,            3731,            1660,            459,            3256,            2568,            555,            4342,            4892,            711,            2984,            3870,            2216,            4092,            3934,            272,            2280,            ],        [            4172,            4232,            3566,            3817,            4509,            3264,            4555,            3084,            1738,            1175,            2500,            2950,            939,            3735,            3338,            3578,            2565,            2252,            924,            1406,            0,            2800,            2928,            1396,            394,            124,            3113,            1962,            2327,            2200,            2116,            1397,            3571,            2898,            4341,            2255,            1216,            1852,            3192,            1668,            1321,            471,            338,            3585,            3581,            2173,            2205,            3327,            2027,            4146,            1935,            2188,            1296,            3398,            2706,            2903,            2204,            3375,            3254,            2380,            1566,            1032,            1905,            2532,            846,            2991,            3992,            1284,            2964,            2519,            2608,            2741,            2583,            1535,            2244,            ],        [            2680,            2740,            2074,            2325,            3017,            3682,            3063,            2590,            2156,            2922,            3648,            3523,            2584,            4153,            937,            3996,            775,            682,            4364,            3527,            3789,            0,            128,            3143,            3544,            3604,            1011,            3418,            2833,            3656,            2940,            3117,            1170,            3316,            3677,            4125,            2963,            1358,            455,            4410,            3268,            3911,            3778,            1184,            1180,            1679,            3661,            1412,            907,            2654,            2441,            2606,            4736,            3816,            1425,            3321,            4014,            638,            3953,            1488,            3060,            4472,            1411,            2950,            4286,            1040,            1255,            4724,            2470,            709,            3026,            427,            619,            3799,            1751,            ],        [            2684,            2744,            2078,            2329,            3021,            3686,            3067,            2594,            2160,            2926,            3652,            3527,            2588,            4157,            941,            3609,            779,            686,            4368,            3531,            3793,            334,            0,            3147,            3548,            3608,            1015,            3422,            2837,            3660,            2944,            3121,            1174,            2087,            3681,            4129,            2967,            1362,            459,            4414,            3272,            3915,            3782,            1188,            1184,            1683,            3665,            1416,            911,            2658,            2445,            2610,            4740,            3820,            1429,            3325,            4018,            642,            3957,            1492,            3064,            4476,            1415,            2954,            4290,            1044,            1259,            4728,            2474,            713,            3030,            299,            491,            3803,            1755,            ],        [            3794,            3854,            3188,            3439,            4131,            2886,            4177,            2706,            1360,            221,            1905,            2355,            561,            3357,            2960,            3200,            2187,            1874,            2239,            1402,            1664,            2422,            2550,            0,            1419,            1479,            2735,            1584,            1949,            1822,            1133,            1310,            3193,            2520,            3963,            2291,            180,            1474,            2814,            2576,            943,            1786,            1653,            3207,            1354,            1795,            1827,            2949,            1649,            3768,            1557,            1810,            2611,            3020,            2328,            2525,            2180,            2997,            2674,            2002,            1188,            2347,            1527,            2154,            2161,            2613,            3614,            2599,            2586,            2141,            2230,            2363,            2205,            1674,            1866,            ],        [            3778,            3838,            3172,            3423,            4115,            2870,            4161,            2690,            1344,            781,            1889,            2339,            545,            3341,            2944,            3184,            2171,            1858,            1363,            1845,            788,            2406,            2534,            1002,            0,            603,            2719,            1568,            1933,            1806,            1722,            1836,            3177,            2504,            3947,            2275,            822,            1458,            2798,            2107,            927,            910,            777,            3191,            3187,            1779,            1811,            2933,            1633,            3752,            1541,            1794,            1735,            3004,            2312,            2509,            2164,            2981,            2658,            1986,            1172,            1471,            1511,            2138,            1285,            2597,            3598,            1723,            2570,            2125,            2214,            2347,            2189,            1974,            1850,            ],        [            4048,            4108,            3442,            3693,            4385,            3140,            4431,            2960,            1614,            1051,            2159,            2609,            815,            3611,            3214,            3454,            2441,            2128,            1108,            1590,            533,            2676,            2804,            1272,            270,            0,            2989,            1838,            2203,            2076,            1992,            1581,            3447,            2774,            4217,            2545,            1092,            1728,            3068,            1852,            1197,            655,            522,            3461,            3457,            2049,            2081,            3203,            1903,            4022,            1811,            2064,            1480,            3274,            2582,            2779,            2434,            3251,            2928,            2256,            1442,            1216,            1781,            2408,            1030,            2867,            3868,            1468,            2840,            2395,            2484,            2617,            2459,            1719,            2120,            ],        [            1999,            2059,            1393,            1644,            2336,            3074,            2382,            2509,            1548,            2314,            3040,            2915,            1976,            3545,            1859,            2924,            463,            690,            3756,            2919,            3181,            922,            1050,            2535,            2936,            2996,            0,            2810,            2225,            3048,            2332,            2509,            2092,            2002,            2996,            3517,            2355,            1277,            1377,            3802,            2660,            3303,            3170,            2106,            2102,            1598,            3053,            562,            826,            1973,            1833,            1998,            4128,            2758,            684,            2713,            3406,            1017,            3345,            939,            2452,            3864,            1330,            2342,            3678,            226,            719,            4116,            2389,            417,            2418,            815,            705,            3191,            1670,            ],        [            4018,            3632,            3412,            3663,            4284,            2134,            3955,            2307,            1472,            1203,            2081,            1956,            865,            2586,            3072,            2448,            2411,            2098,            2645,            1808,            2070,            2646,            3263,            1424,            1825,            1885,            2959,            0,            2061,            238,            1221,            1398,            3305,            1768,            3192,            2558,            1244,            1698,            2926,            2843,            1167,            2192,            2059,            3319,            3315,            2085,            2094,            3173,            1873,            2711,            1669,            1058,            3017,            2249,            2552,            1754,            2447,            3109,            2386,            2226,            494,            2753,            1751,            1402,            2567,            2837,            3726,            3005,            2059,            2365,            1459,            2587,            3548,            2080,            1114,            ],        [            3875,            3935,            3269,            3520,            4212,            3729,            4258,            3549,            2203,            2969,            3695,            3570,            2631,            4200,            1011,            4043,            1829,            1736,            4411,            3574,            3836,            1356,            1202,            3190,            3591,            3651,            2067,            3465,            0,            3703,            2987,            3164,            1244,            3363,            4806,            4172,            3010,            2317,            865,            4457,            3315,            3958,            3825,            1258,            1254,            2638,            3708,            1822,            2492,            3849,            2488,            2653,            4783,            3863,            2479,            3368,            4061,            1048,            4000,            2845,            3107,            4519,            2370,            2997,            4333,            2235,            1665,            4771,            3429,            1763,            3073,            1597,            1487,            3846,            2709,            ],        [            3780,            3394,            3174,            3425,            4046,            1896,            3717,            2069,            1234,            965,            1843,            1718,            627,            2348,            2834,            2210,            2173,            1860,            2407,            1570,            1832,            2408,            2536,            1186,            1587,            1647,            2721,            1613,            1823,            0,            983,            1160,            3067,            1530,            2954,            2320,            1006,            1460,            2688,            2605,            929,            1954,            1821,            3081,            3077,            1847,            1856,            2935,            1635,            2473,            1431,            820,            2779,            2011,            2314,            1516,            2209,            2871,            2148,            1988,            256,            2515,            1513,            1164,            2329,            2599,            3488,            2767,            1821,            2127,            1221,            2349,            2191,            1842,            876,            ],        [            3589,            3649,            2983,            3234,            3926,            2468,            3972,            2501,            1155,            869,            1545,            1995,            356,            2920,            2755,            2782,            1982,            1669,            2034,            833,            1459,            2217,            2345,            1090,            1214,            1274,            2530,            1224,            1744,            1462,            0,            177,            2988,            2102,            2854,            1931,            910,            1269,            2609,            2216,            738,            1581,            1448,            3002,            2998,            1590,            1467,            2744,            1444,            3563,            1352,            1392,            2406,            2583,            2123,            2088,            1820,            2792,            2314,            1797,            828,            1102,            1322,            1736,            1198,            2408,            2901,            1354,            2381,            1936,            1793,            2158,            2000,            1105,            1448,            ],        [            3766,            3826,            3160,            3411,            4103,            2645,            4149,            3261,            1332,            1046,            1412,            1862,            533,            3540,            2932,            2959,            2159,            1846,            1482,            197,            1667,            2394,            2522,            1267,            1752,            1482,            2707,            1363,            1921,            1601,            177,            0,            3165,            2279,            4146,            1167,            1087,            1446,            2786,            1452,            915,            1789,            1656,            3179,            3175,            1767,            1104,            2921,            1621,            3740,            1529,            1569,            1854,            3203,            2300,            2708,            1116,            2969,            2166,            1974,            1005,            466,            1499,            1913,            562,            2585,            3586,            718,            2558,            2113,            2413,            2335,            2177,            469,            1625,            ],        [            3910,            4671,            3304,            3555,            4345,            3764,            4293,            3584,            2238,            3004,            3730,            3605,            2666,            4235,            685,            4078,            1864,            1771,            4446,            3609,            3871,            1264,            1110,            1639,            3626,            3686,            1975,            3500,            2915,            3738,            3022,            3199,            0,            3398,            4150,            4207,            3045,            2352,            773,            4492,            3350,            3993,            3860,            14,            517,            2673,            3743,            1730,            2527,            3884,            2523,            2688,            4818,            3898,            2514,            3403,            4096,            956,            4035,            2880,            3142,            4554,            2405,            3032,            4368,            2270,            1573,            4806,            3464,            1798,            3108,            1632,            1522,            3881,            2744,            ],        [            2639,            2699,            2033,            2284,            2976,            1445,            3022,            953,            1052,            1818,            2544,            2419,            1480,            3049,            2740,            1759,            1565,            1252,            3260,            2423,            2685,            1800,            1928,            2039,            2440,            2500,            1580,            2314,            1729,            2552,            1836,            2013,            2973,            0,            2145,            3021,            1859,            604,            2255,            3306,            2164,            2807,            2674,            2987,            2983,            317,            2557,            1983,            1027,            2613,            1337,            1353,            3632,            1202,            1173,            1247,            2910,            2438,            2849,            847,            1956,            3368,            877,            903,            3182,            1647,            2140,            3620,            833,            1519,            1922,            1741,            1583,            2695,            1217,            ],        [            2461,            2059,            1855,            1644,            2711,            605,            2382,            1684,            2224,            2990,            2807,            2682,            2652,            3312,            3956,            838,            2959,            2646,            4432,            3911,            3857,            3194,            3322,            3211,            3612,            3672,            2747,            2600,            2945,            2838,            3008,            3185,            4189,            1273,            0,            3307,            3031,            1877,            3810,            3592,            3336,            3979,            3846,            4203,            4199,            1590,            2843,            3150,            2421,            1182,            2553,            1739,            4804,            2064,            2446,            1698,            3196,            3605,            3112,            2120,            2242,            4370,            2150,            1097,            4354,            2814,            3307,            4622,            1564,            2913,            1970,            3135,            2977,            3915,            1603,            ],        [            4866,            4480,            4260,            4511,            5132,            2982,            4803,            2746,            2320,            2051,            897,            1347,            1713,            3025,            3920,            3296,            3259,            2946,            2428,            1186,            2613,            3494,            3622,            2272,            2698,            2428,            3807,            848,            2909,            1086,            2069,            1400,            4153,            2616,            3631,            0,            2092,            2546,            3774,            867,            2015,            2735,            2602,            4167,            4163,            2933,            589,            4021,            2721,            3559,            2517,            1906,            2875,            2688,            3400,            2193,            601,            3957,            999,            3074,            1342,            1645,            2599,            2250,            1741,            3685,            4574,            1897,            2666,            3213,            1898,            3435,            4396,            1190,            1962,            ],        [            3614,            3674,            3008,            3259,            3951,            2706,            3997,            2526,            1180,            41,            1725,            2175,            381,            3177,            2780,            3020,            2007,            1694,            2059,            1222,            1484,            2242,            2370,            180,            1239,            1299,            2555,            1404,            1769,            1642,            953,            1130,            3013,            2340,            3092,            2111,            0,            1294,            2634,            2396,            763,            1606,            1473,            3027,            3023,            1615,            1647,            2769,            1469,            3588,            1377,            1630,            2431,            2840,            2148,            2345,            2000,            2817,            2494,            1822,            1008,            2167,            1347,            1974,            1981,            2433,            2926,            2419,            2406,            1961,            2050,            2183,            2025,            1494,            1686,            ],        [            2419,            2479,            1813,            2064,            2756,            1825,            2802,            1331,            448,            1214,            1940,            1815,            876,            2445,            2136,            2139,            961,            648,            2656,            1819,            2081,            1196,            1324,            1435,            1836,            1896,            1333,            1710,            1125,            1948,            1232,            1409,            2369,            824,            2211,            2417,            1255,            0,            1651,            2702,            1560,            2203,            2070,            2383,            2379,            420,            1953,            1723,            423,            2393,            733,            749,            3028,            2108,            953,            1605,            2306,            1834,            2245,            627,            1352,            2764,            273,            1093,            2578,            1387,            1880,            3016,            1211,            915,            1318,            1137,            979,            2091,            613,            ],        [            2876,            2936,            2270,            2521,            3213,            3355,            3259,            3175,            1829,            2595,            3321,            3196,            2257,            3826,            610,            3669,            1094,            1001,            4037,            3200,            3462,            619,            465,            2816,            3217,            3277,            1207,            3091,            2506,            3329,            2613,            2790,            843,            2989,            3741,            3798,            2636,            1943,            0,            4083,            2941,            3584,            3451,            857,            853,            2264,            3334,            957,            1226,            2850,            2114,            2279,            4409,            3489,            1744,            2994,            3687,            183,            3626,            2471,            2733,            4145,            1996,            2623,            3959,            1236,            800,            4397,            3055,            1028,            2699,            782,            1113,            3472,            2335,            ],        [            5249,            4863,            4643,            4894,            5515,            3365,            5186,            3129,            2703,            2434,            1280,            1730,            2096,            3408,            4303,            3679,            3642,            3329,            1682,            1121,            1917,            3877,            4005,            2655,            2002,            1732,            4190,            1231,            3292,            1469,            2452,            1335,            4536,            2999,            4014,            1035,            2475,            2929,            4157,            0,            2398,            2039,            1906,            4550,            4546,            3316,            972,            4404,            3104,            3942,            2900,            2289,            2008,            3071,            3783,            2576,            984,            4340,            2034,            3457,            1725,            1074,            2982,            2633,            888,            4068,            4957,            1326,            3049,            3596,            2281,            3818,            4779,            1125,            2345,            ],        [            2851,            2911,            2245,            2496,            3188,            1943,            3234,            1763,            417,            777,            1909,            1784,            439,            2414,            2017,            2257,            1244,            931,            2219,            1382,            1644,            1479,            1607,            998,            1399,            1459,            1792,            1679,            1006,            1917,            795,            972,            2250,            1577,            2329,            2386,            818,            531,            1871,            2671,            0,            1766,            1633,            2264,            2260,            852,            1922,            2006,            706,            2825,            614,            867,            2591,            2077,            1385,            1582,            2275,            2054,            2214,            1059,            986,            2327,            584,            1211,            2141,            1670,            2163,            2579,            1643,            1198,            1287,            1420,            1262,            1654,            923,            ],        [            4560,            4620,            3954,            4205,            4897,            3652,            4943,            3472,            2126,            1563,            2888,            3338,            1327,            4123,            3726,            3966,            2953,            2640,            1312,            1794,            388,            3188,            3316,            1784,            782,            512,            3501,            2350,            2715,            2588,            2504,            1785,            3959,            3286,            4038,            2643,            1604,            2240,            3580,            2056,            1709,            0,            133,            3973,            3969,            2561,            2593,            3715,            2415,            4534,            2323,            2576,            1684,            3786,            3094,            3291,            2592,            3763,            3642,            2768,            1954,            1420,            2293,            2920,            1234,            3379,            3872,            1672,            3352,            2907,            2996,            3129,            2971,            1923,            2632,            ],        [            4427,            4487,            3821,            4072,            4764,            3519,            4810,            3339,            1993,            1430,            2755,            3205,            1194,            3990,            3593,            3833,            2820,            2507,            1179,            1661,            255,            3055,            3183,            1651,            649,            379,            3368,            2217,            2582,            2455,            2371,            1652,            3826,            3153,            3905,            2510,            1471,            2107,            3447,            1923,            1576,            133,            0,            3840,            3836,            2428,            2460,            3582,            2282,            4401,            2190,            2443,            1551,            3653,            2961,            3158,            2459,            3630,            3509,            2635,            1821,            1287,            2160,            2787,            1101,            3246,            3739,            1539,            3219,            2774,            2863,            2996,            2838,            1790,            2499,            ],        [            3896,            4657,            3290,            3541,            4331,            3750,            4279,            3570,            2224,            2990,            3716,            3591,            2652,            4221,            671,            4064,            1850,            1757,            4432,            3595,            3857,            1250,            1096,            1625,            3612,            3672,            1961,            3486,            2901,            3724,            3008,            3185,            493,            3384,            4136,            4193,            3031,            2338,            759,            4478,            3336,            3979,            3846,            0,            503,            2659,            3729,            1716,            2513,            3870,            2509,            2674,            4804,            3884,            2500,            3389,            4082,            942,            4021,            2866,            3128,            4540,            2391,            3018,            4354,            2256,            1559,            4792,            3450,            1784,            3094,            1618,            1508,            3867,            2730,            ],        [            3223,            4250,            2617,            2868,            3924,            3343,            3606,            3163,            1817,            2583,            3309,            3184,            2245,            3814,            498,            3657,            1443,            1225,            4025,            3188,            3450,            843,            689,            2804,            3205,            3265,            1554,            3079,            2494,            3317,            2601,            2778,            731,            2977,            3729,            3786,            2624,            1931,            352,            4071,            2929,            3572,            3439,            745,            0,            2252,            3322,            1309,            1450,            3197,            2102,            2267,            4397,            3477,            1968,            2982,            3675,            535,            3614,            2459,            2721,            4133,            1984,            2611,            3947,            1849,            1152,            4385,            3043,            1377,            2687,            1211,            1101,            3460,            2323,            ],        [            2322,            2382,            1716,            1967,            2659,            2112,            2705,            1234,            735,            1501,            2227,            2102,            1163,            2732,            2423,            3247,            1248,            935,            2943,            2106,            2368,            1483,            1611,            1722,            2123,            2183,            1263,            1997,            1412,            2235,            1519,            1696,            2656,            727,            2426,            2704,            1542,            287,            1938,            2989,            1847,            2490,            2357,            2670,            2666,            0,            2240,            1666,            710,            2296,            1020,            1036,            3315,            1483,            856,            1892,            2593,            2121,            2532,            530,            1639,            3051,            560,            1380,            2865,            1330,            1823,            3303,            1114,            1202,            1605,            1424,            1266,            2378,            900,            ],        [            4277,            3891,            3671,            3922,            4543,            2393,            4214,            2566,            1731,            1462,            610,            1060,            1124,            2845,            3331,            2707,            2670,            2357,            2904,            1600,            2329,            2905,            3033,            1683,            2084,            2144,            3218,            259,            2320,            497,            1480,            1814,            3564,            2027,            2779,            996,            1503,            1957,            3185,            1281,            1426,            2451,            2318,            3578,            3574,            2344,            0,            3432,            2132,            2970,            1928,            1317,            3276,            2508,            2811,            2013,            885,            3368,            1379,            2485,            753,            2059,            2010,            1661,            2155,            3096,            3985,            2311,            2318,            2624,            1718,            2846,            2688,            1604,            1373,            ],        [            2143,            2203,            1537,            1788,            2480,            3651,            2526,            2612,            2022,            2788,            3514,            3389,            2450,            4019,            1567,            3068,            937,            1164,            4230,            3393,            3655,            1274,            1402,            3009,            3410,            3470,            474,            3284,            2699,            3522,            2806,            2983,            1800,            2105,            3140,            3991,            2829,            1702,            957,            4276,            3134,            3777,            3644,            1814,            1810,            1701,            3527,            0,            1300,            2117,            2307,            2472,            4602,            2861,            1051,            3187,            3880,            774,            3819,            1271,            2926,            4338,            1804,            2816,            4152,            715,            157,            4590,            2492,            891,            2892,            1289,            1179,            3665,            2144,            ],        [            2218,            2278,            1612,            1863,            2555,            2056,            2601,            1683,            679,            1445,            2171,            2046,            1107,            2676,            1710,            2370,            538,            225,            2887,            2050,            2312,            773,            901,            1666,            2067,            2127,            910,            1941,            1356,            2179,            1463,            1640,            1943,            1176,            2442,            2648,            1486,            451,            1228,            2933,            1791,            2434,            2301,            1957,            1953,            772,            2184,            1300,            0,            2192,            964,            980,            3259,            1932,            752,            1836,            2537,            1411,            2476,            581,            1583,            2995,            504,            1324,            2809,            964,            1457,            3247,            1563,            492,            1549,            714,            556,            2322,            844,            ],        [            1560,            1158,            954,            743,            1810,            1534,            1481,            1797,            2337,            3103,            2920,            2795,            2765,            3425,            4069,            951,            2309,            2284,            4545,            4024,            3970,            2646,            2774,            3324,            3725,            3785,            1846,            2713,            3058,            2951,            3121,            3298,            4302,            1386,            1023,            3420,            3144,            1990,            2887,            3705,            3449,            4092,            3959,            4316,            4312,            1703,            2956,            2249,            2534,            0,            2666,            1852,            4917,            2177,            1799,            1811,            3309,            2704,            3225,            2019,            2355,            4483,            2263,            1210,            4467,            1913,            2406,            4735,            1677,            2263,            2083,            2661,            2615,            4028,            1716,            ],        [            4267,            4327,            3661,            3912,            4604,            4121,            4650,            3941,            2595,            3361,            4087,            3962,            3023,            4592,            1403,            4435,            2221,            2128,            4803,            3966,            4228,            1748,            1594,            3582,            3983,            4043,            2459,            3857,            392,            4095,            3379,            3556,            1636,            3755,            4507,            4564,            3402,            2709,            1257,            4849,            3707,            4350,            4217,            1650,            1646,            3030,            4100,            2214,            2884,            4241,            0,            3045,            3945,            4255,            2871,            3760,            4453,            1440,            4392,            3237,            3499,            4911,            2762,            3389,            4725,            2627,            2057,            5163,            3821,            2155,            3465,            1989,            1879,            4238,            3101,            ],        [            2932,            2574,            2326,            2115,            3226,            1076,            2897,            1121,            962,            1728,            1545,            1420,            1390,            2050,            2694,            1390,            1697,            1384,            3170,            2649,            2595,            1932,            2060,            1949,            2350,            2410,            2290,            1338,            1683,            1576,            1746,            1923,            2927,            710,            1462,            2045,            1769,            1314,            2548,            2330,            2074,            2717,            2584,            2941,            2937,            1027,            1581,            2693,            1159,            1653,            1291,            0,            3542,            1222,            1883,            856,            1934,            2731,            1850,            1557,            980,            3108,            1587,            344,            3092,            2357,            2850,            3360,            1001,            1651,            1128,            1873,            1715,            2653,            341,            ],        [            4579,            5169,            3973,            4224,            4916,            3671,            5492,            3491,            2145,            1582,            2151,            2601,            1346,            4142,            3745,            3985,            2972,            2659,            575,            1057,            716,            3207,            3335,            1803,            801,            531,            3520,            2102,            2734,            2340,            1598,            1048,            3978,            3305,            4057,            1906,            1623,            2259,            3599,            1319,            1728,            838,            705,            3992,            3988,            2580,            1843,            3734,            2434,            4248,            2342,            2595,            0,            3805,            3113,            3310,            1855,            3782,            2905,            2787,            1973,            683,            2312,            2939,            497,            3398,            3891,            935,            3371,            2926,            3015,            3148,            2990,            1186,            2651,            ],        [            3310,            2908,            2704,            2493,            3560,            1454,            3231,            604,            2036,            2731,            1854,            1729,            2393,            2359,            3710,            1687,            2549,            2236,            4173,            3274,            3598,            2784,            2912,            2952,            3353,            3413,            2564,            2380,            2699,            2618,            2749,            3488,            3943,            984,            943,            2670,            2772,            1588,            3239,            2955,            2695,            3720,            3587,            3957,            3953,            1301,            2677,            2967,            2011,            2031,            2307,            1682,            4963,            0,            2157,            648,            2689,            3422,            2159,            1831,            2022,            3733,            1861,            1046,            3829,            2631,            3124,            3985,            524,            2503,            920,            2725,            2567,            3278,            1406,            ],        [            1466,            1526,            860,            1111,            1803,            2974,            1849,            1935,            1473,            2239,            2965,            2840,            1901,            3470,            2058,            2391,            870,            975,            3681,            2844,            3106,            1207,            1335,            2460,            2861,            2921,            407,            2735,            2150,            2973,            2257,            2434,            2291,            1428,            2463,            3442,            2280,            1025,            1448,            3727,            2585,            3228,            3095,            2305,            2301,            1024,            2978,            810,            984,            1440,            1758,            1774,            4053,            2184,            0,            2229,            3331,            1265,            3270,            594,            2377,            3789,            1298,            2118,            3603,            474,            967,            4041,            1815,            824,            2343,            1222,            1112,            3116,            1638,            ],        [            3413,            3055,            2807,            2596,            3707,            1557,            3378,            877,            1462,            2083,            1206,            1081,            1745,            1711,            3062,            1871,            2541,            2228,            3525,            2626,            2950,            2776,            2904,            2304,            2705,            2765,            2837,            1732,            2051,            1970,            2101,            2840,            3295,            1257,            1762,            2022,            2124,            1861,            2916,            2307,            2047,            3072,            2939,            3309,            3305,            1574,            2029,            3240,            2003,            2134,            1659,            1034,            4315,            819,            2430,            0,            2041,            3099,            1511,            2104,            1374,            3085,            2134,            825,            3181,            2904,            3397,            3337,            797,            2495,            272,            2717,            2559,            2630,            1185,            ],        [            4535,            4177,            3929,            3718,            4829,            2679,            4500,            2145,            2017,            1748,            296,            746,            1410,            2424,            3617,            2993,            2956,            2643,            3190,            1742,            2615,            3191,            3319,            1969,            2370,            2430,            3504,            723,            2606,            961,            1766,            1956,            3850,            2313,            3030,            1138,            1789,            2243,            3471,            1423,            1712,            2737,            2604,            3864,            3860,            2630,            1145,            3718,            2418,            3256,            2214,            1603,            3431,            2087,            3097,            1592,            0,            3654,            1065,            2771,            1039,            2201,            2296,            1947,            2297,            3382,            4271,            2453,            2065,            2910,            1297,            3132,            2974,            1746,            1659,            ],        [            2693,            2753,            2087,            2338,            3030,            3538,            3076,            3358,            2012,            2778,            3504,            3379,            2440,            4009,            793,            3852,            1277,            1184,            4220,            3383,            3645,            802,            648,            2999,            3400,            3460,            1024,            3274,            2689,            3512,            2796,            2973,            1026,            3172,            3924,            3981,            2819,            2126,            183,            4266,            3124,            3767,            3634,            1040,            1036,            2447,            3517,            774,            1409,            2667,            2297,            2462,            4592,            3672,            1601,            3177,            3870,            0,            3809,            1821,            2916,            4328,            2179,            2806,            4142,            1053,            617,            4580,            3238,            1211,            2882,            965,            1296,            3655,            2518,            ],        [            4457,            4099,            3851,            3640,            4751,            2601,            4422,            2067,            1939,            1670,            611,            668,            1332,            2346,            3539,            2915,            2878,            2565,            3112,            2031,            2537,            3113,            3241,            1891,            2292,            2352,            3426,            1693,            2528,            1931,            1688,            2245,            3772,            2235,            2952,            1427,            1711,            2165,            3393,            1712,            1634,            2659,            2526,            3786,            3782,            2552,            1434,            3640,            2340,            3178,            2136,            1525,            3720,            2009,            3019,            1514,            1446,            3576,            0,            2693,            961,            2490,            2218,            1869,            2586,            3304,            4193,            2742,            1987,            2832,            1219,            3054,            2896,            2035,            1581,            ],        [            1792,            1852,            1186,            1437,            2129,            2429,            2175,            1514,            1052,            1818,            2544,            2419,            1480,            3049,            2384,            2717,            1196,            811,            3260,            2423,            2685,            1359,            1487,            2039,            2440,            2500,            733,            2314,            1729,            2552,            1836,            2013,            2617,            1007,            2789,            3021,            1859,            604,            1774,            3306,            2164,            2807,            2674,            2631,            2627,            603,            2557,            1136,            820,            1766,            1337,            1353,            3632,            1763,            326,            1808,            2910,            1591,            2849,            0,            1956,            3368,            877,            1697,            3182,            800,            1293,            3620,            1394,            909,            1922,            1300,            1142,            2695,            1217,            ],        [            3496,            3138,            2890,            2679,            3790,            1640,            3461,            1813,            978,            1656,            1587,            1462,            1318,            2092,            2578,            1954,            1976,            1663,            3098,            2668,            2523,            2211,            2339,            1877,            2278,            2338,            2854,            1357,            1567,            1595,            1674,            1851,            2811,            1274,            2026,            2064,            1697,            1878,            2432,            2349,            2002,            2645,            2512,            2825,            2821,            1591,            1600,            2738,            1438,            2217,            1175,            564,            3470,            1755,            2447,            1260,            1953,            2615,            1892,            2121,            0,            3206,            1316,            908,            3020,            2402,            3232,            3458,            1565,            1930,            965,            2152,            1994,            2672,            620,            ],        [            5154,            5744,            4548,            4799,            5491,            4246,            6067,            3528,            2720,            2157,            1679,            2129,            1921,            3807,            4320,            4560,            3547,            3234,            1106,            464,            1291,            3782,            3910,            2378,            1376,            1106,            4095,            1630,            3309,            1868,            1005,            455,            4553,            3880,            4413,            1434,            2198,            2834,            4174,            1084,            2303,            1413,            1280,            4567,            4563,            3155,            1371,            4309,            3009,            4823,            2917,            3170,            1478,            3470,            3688,            2975,            1383,            4357,            2433,            3362,            2124,            0,            2887,            3514,            186,            3973,            4466,            252,            3448,            3501,            2680,            3723,            3565,            503,            3226,            ],        [            2740,            3050,            2134,            2385,            3702,            1552,            3373,            1597,            345,            1111,            1782,            1657,            773,            2287,            2033,            1866,            1133,            820,            2553,            1716,            1978,            1368,            1496,            1332,            1733,            1793,            1681,            1575,            1022,            1813,            1129,            1306,            2266,            1186,            1938,            2282,            1152,            420,            1887,            2567,            1457,            2100,            1967,            2280,            2276,            741,            1818,            1895,            595,            2129,            630,            476,            2925,            1698,            1274,            1332,            2171,            2070,            2087,            948,            1217,            2661,            0,            820,            2475,            1559,            2052,            2913,            1477,            1087,            1160,            1309,            1151,            1988,            340,            ],        [            2588,            2230,            1982,            1771,            2882,            732,            2553,            777,            1337,            2103,            1920,            1795,            1765,            2425,            3069,            1046,            1931,            1618,            3545,            3024,            2970,            2166,            2294,            2324,            2725,            2785,            1946,            1713,            2058,            1951,            2121,            2298,            3302,            366,            1118,            2420,            2144,            970,            2621,            2705,            2449,            3092,            2959,            3316,            3312,            683,            1956,            2349,            1393,            1309,            1666,            852,            3917,            1026,            1539,            811,            2309,            2804,            2225,            1213,            1355,            3483,            1243,            0,            3467,            2013,            2506,            3735,            657,            1885,            1083,            2107,            1949,            3028,            716,            ],        [            4968,            5558,            4362,            4613,            5305,            4060,            5881,            3579,            2534,            1971,            1730,            2180,            1735,            3858,            4134,            4374,            3361,            3048,            920,            560,            1105,            3596,            3724,            2192,            1190,            920,            3909,            1681,            3123,            1919,            1101,            551,            4367,            3694,            4464,            1485,            2012,            2648,            3988,            898,            2117,            1227,            1094,            4381,            4377,            2969,            1422,            4123,            2823,            4637,            2731,            2984,            1292,            3521,            3502,            3026,            1434,            4171,            2484,            3176,            2175,            186,            2701,            3328,            0,            3787,            4280,            438,            3760,            3315,            2731,            3537,            3379,            689,            3040,            ],        [            1984,            2044,            1378,            1629,            2321,            3368,            2367,            2453,            1991,            2757,            3483,            3358,            2419,            3988,            1903,            2909,            1027,            1132,            4199,            3362,            3624,            1233,            1361,            2978,            3379,            3439,            564,            3253,            2668,            3491,            2775,            2952,            2136,            1946,            2981,            3960,            2798,            1543,            1293,            4245,            3103,            3746,            3613,            2150,            2146,            1542,            3496,            336,            1141,            1958,            2276,            2292,            4571,            2702,            892,            2747,            3849,            1110,            3788,            1112,            2895,            4307,            1816,            2636,            4121,            0,            493,            4559,            2333,            981,            2861,            1379,            1269,            3634,            2156,            ],        [            2300,            2360,            1694,            1945,            2637,            4155,            2683,            2769,            2629,            3395,            4121,            3996,            3057,            4626,            1410,            3225,            1094,            1321,            4837,            4000,            4262,            1044,            1172,            3616,            4017,            4077,            631,            3891,            3306,            4129,            3413,            3590,            1643,            2262,            3297,            4598,            3436,            1859,            800,            4883,            3741,            4384,            4251,            1657,            1653,            1858,            4134,            157,            1457,            2274,            2914,            3079,            5209,            3018,            1208,            3794,            4487,            617,            4426,            1428,            3533,            4945,            1961,            3423,            4759,            823,            0,            5197,            2649,            1048,            3499,            1207,            1336,            4272,            2301,            ],        [            5406,            5053,            4800,            5051,            5705,            3555,            5376,            3319,            2972,            2409,            1470,            1920,            2173,            3598,            4572,            3869,            3799,            3486,            1358,            716,            1543,            4034,            4162,            2630,            1628,            1358,            4347,            1421,            3561,            1659,            1257,            707,            4805,            3189,            4204,            1225,            2450,            3086,            4426,            1336,            2555,            1665,            1532,            4819,            4815,            3407,            1162,            4561,            3261,            4132,            3169,            2479,            1730,            3261,            3940,            2766,            1174,            4609,            2224,            3614,            1915,            252,            3139,            2823,            438,            4225,            5226,            0,            3239,            3753,            2471,            3975,            3817,            251,            2535,            ],        [            3465,            2981,            2859,            2522,            3633,            1483,            3304,            446,            1878,            2644,            1925,            1800,            2306,            2430,            3566,            2056,            2391,            2078,            4086,            3345,            3511,            2626,            2754,            2865,            3266,            3326,            2406,            2451,            2555,            2689,            2662,            3559,            3799,            826,            1312,            2741,            2685,            1430,            3081,            3026,            2766,            3633,            3500,            3813,            3809,            1143,            2748,            2809,            1853,            2060,            2163,            1583,            4458,            369,            1999,            719,            2760,            3264,            2230,            1673,            2093,            3804,            1703,            941,            3900,            2473,            2966,            4056,            0,            2345,            991,            2567,            2409,            3349,            1447,            ],        [            2112,            2172,            1506,            1757,            2449,            3340,            2495,            2581,            1814,            2580,            3306,            3181,            2242,            3811,            1550,            3037,            729,            718,            4022,            3185,            3447,            613,            741,            2801,            3202,            3262,            591,            3076,            2491,            3314,            2598,            2775,            1783,            2074,            3109,            3783,            2621,            1671,            1068,            4068,            2926,            3569,            3436,            1797,            1793,            1670,            3319,            808,            1092,            2086,            2099,            2264,            4394,            2830,            1020,            2979,            3672,            1263,            3611,            1240,            2718,            4130,            1596,            2608,            3944,            472,            965,            4382,            2461,            0,            2684,            757,            971,            3457,            1936,            ],        [            3694,            3336,            3088,            2877,            3988,            1838,            3659,            1779,            1190,            1811,            934,            809,            1473,            1439,            2790,            2152,            2174,            1861,            3253,            2354,            2678,            2409,            2537,            2032,            2433,            2493,            3052,            1460,            1779,            1698,            1829,            2568,            3023,            1472,            2224,            1750,            1852,            2076,            2644,            2035,            1775,            2800,            2667,            3037,            3033,            1789,            1757,            2936,            1636,            2415,            1387,            762,            3625,            1721,            2645,            1226,            1769,            2827,            1239,            2319,            1102,            2813,            1514,            1106,            2909,            2600,            3444,            3065,            1763,            2128,            0,            2350,            2192,            2358,            818,            ],        [            2580,            2640,            1974,            2225,            2917,            2581,            2963,            2401,            1055,            1821,            2547,            2422,            1483,            3052,            1252,            2895,            534,            441,            3263,            2426,            2688,            315,            299,            2042,            2443,            2503,            1044,            2317,            1732,            2555,            1839,            2016,            1485,            2215,            2967,            3024,            1862,            1169,            770,            3309,            2167,            2810,            2677,            1499,            1495,            1490,            2560,            1276,            666,            2554,            1340,            1505,            3635,            2715,            1184,            2220,            2913,            953,            2852,            1247,            1959,            3371,            1170,            1849,            3185,            940,            1203,            3623,            2281,            468,            1925,            0,            192,            2698,            1561,            ],        [            2388,            2448,            1782,            2033,            2725,            2633,            2771,            2157,            1107,            1873,            2599,            2474,            1535,            3104,            1398,            2947,            342,            249,            3315,            2478,            2740,            461,            589,            2094,            2495,            2555,            903,            2369,            1784,            2607,            1891,            2068,            1631,            1650,            3019,            3076,            1914,            925,            916,            3361,            2219,            2862,            2729,            1645,            1641,            1246,            2612,            1084,            474,            2362,            1392,            1557,            3687,            2406,            992,            2272,            2965,            1099,            2904,            1055,            2011,            3423,            978,            1901,            3237,            748,            1241,            3675,            2037,            276,            1977,            402,            0,            2750,            1318,            ],        [            5160,            4802,            4554,            4343,            5454,            3304,            5125,            3068,            2642,            2373,            1219,            1669,            2035,            3347,            4242,            3618,            3581,            3268,            1747,            272,            1932,            3816,            3944,            2594,            2017,            1747,            4129,            1170,            3231,            1408,            1508,            486,            4475,            2938,            3953,            974,            2414,            2868,            4096,            1259,            2337,            2054,            1921,            4489,            4485,            3255,            911,            4343,            3043,            3881,            2839,            2228,            2119,            3010,            3722,            2515,            923,            4279,            1973,            3396,            1664,            503,            2921,            2572,            827,            4007,            4896,            251,            2988,            3535,            2220,            3757,            4718,            0,            2284,            ],        [            3068,            2710,            2462,            2251,            3362,            1212,            3033,            1257,            991,            1669,            1442,            1317,            1331,            1947,            2591,            1526,            1833,            1520,            3111,            2546,            2536,            2068,            2196,            1890,            2291,            2351,            2426,            1235,            1580,            1473,            1687,            1864,            2824,            846,            1598,            1942,            1710,            1450,            2445,            2227,            2283,            2658,            2525,            2838,            2834,            1163,            1478,            2829,            1295,            1789,            1188,            136,            3483,            1358,            2019,            992,            1831,            2628,            1747,            1693,            877,            3005,            1723,            480,            3033,            2493,            2986,            3257,            1137,            1787,            820,            2009,            1851,            2550,            0,            ],        ], 'starts': [34, 41, 57, 12], 'ends': [34, 41, 57, 12]},    'NW-28': {'distance_matrix': [        [            0,            416,            772,            705,            1693,            1269,            1230,            476,            2996,            3244,            2933,            3187,            3514,            3923,            1783,            1449,            2499,            2623,            3991,            2884,            3898,            2952,            4482,            5625,            4412,            3994,            3039,            7125,            3307,            3736,            3627,            5579,            2996,            3888,            4178,            3186,            3848,            4638,            4478,            5936,            3074,            2979,            3467,            3062,            2151,            3823,            3766,            4033,            6646,            801,            4011,            2871,            2289,            3395,            3727,            3208,            1140,            4174,            5564,            3263,            4014,            5346,            3537,            2843,            2904,            690,            1027,            2868,            3495,            4430,            6262,            2062,            2656,            3221,            3072,            2021,            3747,            3372,            3586,            2894,            1322,            ],        [            299,            0,            356,            1004,            1277,            1107,            814,            775,            2834,            3082,            2771,            3025,            3352,            3761,            1621,            1287,            2337,            2461,            3829,            2722,            3736,            2790,            4320,            4213,            4250,            3832,            2877,            6963,            3145,            3574,            3465,            5417,            2834,            3726,            4016,            3024,            3686,            4476,            4316,            5778,            2912,            2817,            3305,            2900,            1989,            3661,            3604,            3871,            4953,            661,            3849,            2709,            2127,            3233,            3565,            3046,            322,            4012,            5326,            3101,            3852,            4137,            3375,            2681,            2742,            528,            865,            2706,            3333,            4268,            6024,            1900,            2494,            3059,            2910,            1859,            3585,            3210,            3424,            2732,            906,            ],        [            751,            452,            0,            1350,            921,            1559,            458,            1227,            3286,            3534,            3223,            3477,            3643,            4052,            2073,            1739,            2454,            2770,            4120,            3013,            4027,            2276,            3806,            3699,            3736,            4123,            2363,            7254,            2631,            3784,            3551,            5708,            3125,            3212,            3502,            2510,            3172,            3962,            3802,            4198,            3221,            3108,            3596,            3352,            2142,            3952,            3090,            3357,            5244,            681,            4140,            3018,            2579,            2719,            3051,            3337,            342,            3498,            5284,            3392,            3338,            3623,            3666,            3133,            3033,            980,            1317,            2997,            2819,            3754,            5982,            2209,            2946,            3350,            3362,            2311,            3876,            2696,            2910,            3184,            550,            ],        [            663,            868,            1224,            0,            2145,            1453,            1682,            1139,            3180,            3428,            3117,            3371,            3698,            4107,            1967,            1633,            2683,            2807,            4175,            3068,            4082,            3136,            4666,            4559,            4596,            4178,            3223,            7309,            3491,            3920,            3811,            5763,            3180,            4072,            4362,            3370,            4032,            4822,            4662,            6124,            3258,            3163,            3651,            3246,            2335,            4007,            3950,            4217,            5299,            1529,            4195,            3055,            2473,            3579,            3911,            3392,            1190,            4358,            6658,            3447,            4198,            4483,            3721,            3027,            3088,            874,            1211,            3052,            3679,            4614,            7356,            2246,            2840,            3405,            3256,            2205,            3931,            3556,            3770,            3078,            1774,            ],        [            1578,            1279,            1236,            2177,            0,            2386,            521,            2054,            3207,            3455,            3144,            3195,            3359,            3768,            2900,            2566,            2170,            2486,            3836,            2729,            3743,            1992,            3522,            3415,            3452,            3839,            2079,            6970,            2347,            3500,            3267,            5424,            2841,            2928,            3218,            2226,            2888,            3678,            3518,            5865,            2937,            2824,            3312,            3273,            1858,            3668,            2806,            3073,            6575,            1508,            3856,            2734,            2500,            2435,            2767,            3053,            1169,            3214,            5493,            3108,            3054,            3339,            3382,            3054,            2749,            1807,            2144,            2713,            2535,            3470,            6191,            1925,            2867,            3066,            3283,            2232,            3592,            2412,            2626,            3105,            633,            ],        [            892,            1097,            1453,            1597,            2374,            0,            1911,            1368,            1727,            1975,            1664,            1918,            2245,            2654,            514,            180,            1230,            1354,            2722,            1615,            2629,            1683,            3213,            3106,            3143,            2725,            1770,            5856,            2038,            2467,            2358,            4310,            1727,            2619,            2909,            1917,            2579,            3369,            3209,            4671,            1805,            1710,            2198,            1793,            882,            2554,            2497,            2764,            3846,            1693,            2742,            1602,            1020,            2126,            2458,            1939,            1419,            2905,            5205,            1994,            2745,            3030,            2268,            1574,            1635,            824,            341,            1599,            2226,            3161,            5903,            793,            1387,            1952,            1803,            752,            2478,            2103,            2317,            1625,            2003,            ],        [            1057,            758,            715,            1656,            767,            1865,            0,            1533,            3592,            3840,            3529,            3783,            3489,            3898,            2379,            2045,            2300,            3219,            3966,            2859,            3873,            2122,            3652,            3545,            3582,            3969,            2209,            7100,            2477,            3630,            3397,            5554,            2971,            3058,            3348,            2356,            3018,            3808,            3648,            4044,            3670,            2954,            3442,            3658,            1988,            3798,            2936,            3203,            5090,            987,            3986,            3467,            2885,            2565,            2897,            3183,            648,            3344,            5590,            3238,            3184,            3469,            3512,            3439,            2879,            1286,            1623,            2843,            2665,            3600,            6288,            2658,            3252,            3196,            3668,            2617,            3722,            2542,            2756,            3490,            396,            ],        [            796,            972,            929,            548,            1850,            1586,            1387,            0,            3313,            3561,            3250,            3504,            3831,            4240,            2100,            1766,            2816,            2940,            4308,            3201,            4215,            3269,            4799,            5149,            5389,            4311,            3356,            7442,            3624,            4053,            3944,            6688,            3313,            4205,            4495,            3503,            4165,            5615,            4795,            5460,            3391,            3296,            3784,            3379,            2468,            4140,            4083,            4350,            6170,            325,            4328,            3188,            2606,            3712,            4971,            3525,            664,            4491,            5088,            3580,            4331,            4870,            3854,            3160,            3221,            1007,            1344,            3185,            3812,            5847,            5786,            2379,            2973,            3538,            3389,            2338,            4064,            3689,            3903,            3211,            1479,            ],        [            2869,            3074,            3430,            3574,            3316,            1977,            3220,            3345,            0,            740,            429,            683,            2183,            2592,            1506,            1797,            1543,            687,            2660,            1321,            2567,            2356,            3886,            3779,            3816,            2663,            2443,            5794,            2711,            2405,            2296,            4248,            1665,            2635,            3582,            2590,            3252,            4042,            3882,            4609,            941,            744,            1904,            558,            1555,            2492,            3170,            3129,            3784,            3670,            2448,            780,            942,            2799,            3131,            1668,            3396,            3578,            5143,            1700,            3418,            3703,            1974,            339,            1754,            2801,            2318,            1933,            2899,            3834,            5841,            1784,            176,            1658,            592,            1210,            2416,            2776,            2990,            390,            3332,            ],        [            3429,            3634,            3990,            4134,            3876,            2537,            3780,            3905,            560,            0,            311,            1127,            2743,            3152,            2066,            2357,            2103,            1131,            3220,            1881,            3127,            2916,            4446,            4339,            4376,            3223,            3003,            6354,            3271,            2965,            2856,            4808,            2225,            3195,            4142,            3150,            3812,            4602,            4442,            5169,            1385,            1304,            2464,            182,            2115,            3052,            3730,            3689,            4344,            4230,            3008,            1224,            1502,            3359,            3691,            2228,            3956,            4138,            5703,            2260,            3978,            4263,            2534,            783,            2314,            3361,            2878,            2493,            3459,            4394,            6401,            2344,            736,            2218,            1152,            1770,            2976,            3336,            3550,            834,            3892,            ],        [            3118,            3323,            3679,            3823,            3565,            2226,            3469,            3594,            249,            311,            0,            816,            2432,            2841,            1755,            2046,            1792,            820,            2909,            1570,            2816,            2605,            4135,            4028,            4065,            2912,            2692,            6043,            2960,            2654,            2545,            4497,            1914,            2884,            3831,            2839,            3501,            4291,            4131,            4858,            1074,            993,            2153,            129,            1804,            2741,            3419,            3378,            4033,            3919,            2697,            913,            1191,            3048,            3380,            1917,            3645,            3827,            5392,            1949,            3667,            3952,            2223,            472,            2003,            3050,            2567,            2182,            3148,            4083,            6090,            2033,            425,            1907,            841,            1459,            2665,            3025,            3239,            523,            3581,            ],        [            3103,            3308,            3664,            3808,            3279,            2211,            3183,            3579,            771,            1127,            816,            0,            2684,            3093,            2007,            2031,            2044,            709,            3161,            1822,            3068,            2319,            3849,            3742,            3779,            3164,            2406,            6295,            2674,            2906,            2797,            4749,            2166,            3136,            3545,            2553,            3215,            4005,            3845,            5110,            258,            1245,            2405,            945,            1518,            2993,            3133,            3630,            4285,            3904,            2949,            461,            1443,            2762,            3094,            2169,            3630,            3541,            5644,            2201,            3381,            3666,            2475,            344,            2255,            3035,            2552,            2434,            2862,            3797,            6342,            1695,            677,            2159,            1093,            1711,            2917,            2739,            2953,            293,            3295,            ],        [            2968,            3173,            3529,            3673,            3092,            2076,            2996,            3444,            2341,            2589,            2278,            2532,            0,            776,            1629,            1896,            1319,            2121,            844,            1221,            751,            2132,            3334,            3939,            4103,            1570,            2219,            4701,            2487,            1312,            1203,            3155,            565,            1542,            3122,            2366,            2792,            4329,            3658,            3516,            2572,            1445,            1762,            2407,            1331,            1399,            2618,            2036,            2691,            3769,            1157,            2369,            2135,            2575,            2907,            852,            3495,            3118,            4050,            1600,            2958,            3479,            1620,            2188,            548,            2900,            2417,            859,            2675,            3374,            4748,            1560,            2001,            1558,            2417,            1867,            1323,            2552,            2766,            2239,            3108,            ],        [            3126,            3331,            3687,            3831,            3250,            2234,            3154,            3602,            2499,            2747,            2436,            2690,            158,            0,            1787,            2054,            1477,            2279,            655,            1379,            562,            2290,            3492,            4097,            4261,            1728,            2377,            4859,            2645,            1470,            1361,            3313,            723,            1700,            3280,            2524,            2950,            4487,            3816,            3674,            2730,            1603,            1573,            2565,            1489,            1557,            2776,            2194,            2849,            3927,            968,            2527,            2293,            2733,            3065,            1010,            3653,            3276,            4208,            1758,            3116,            3637,            1431,            2346,            706,            3058,            2575,            1017,            2833,            3532,            4906,            1718,            2159,            1716,            2575,            2025,            1481,            2710,            2924,            2397,            3266,            ],        [            2147,            2352,            2748,            2852,            2129,            1255,            2033,            2623,            2092,            2340,            2029,            1549,            1731,            2140,            0,            1075,            716,            840,            2208,            1101,            2115,            1169,            2699,            2592,            2629,            2211,            1256,            5342,            1524,            1953,            1844,            3796,            1213,            2105,            2395,            1403,            2065,            2855,            2695,            4157,            1291,            1196,            1684,            2158,            368,            2040,            1983,            2250,            3332,            3020,            2228,            1088,            1314,            1612,            1944,            1425,            2681,            2391,            4691,            1480,            2231,            2516,            1754,            1939,            1121,            2079,            1596,            1085,            1712,            2647,            5389,            279,            1752,            1438,            2168,            1046,            1964,            1589,            1803,            1990,            2145,            ],        [            1072,            1277,            1633,            1777,            2554,            180,            2091,            1548,            1547,            1795,            1484,            1738,            2065,            2474,            334,            0,            1050,            1174,            2542,            1435,            2449,            1503,            3033,            2926,            2963,            2545,            1590,            5676,            1858,            2287,            2178,            4130,            1547,            2439,            2729,            1737,            2399,            3189,            3029,            4491,            1625,            1530,            2018,            1613,            702,            2374,            2317,            2584,            3666,            1873,            2562,            1422,            840,            1946,            2278,            1759,            1599,            2725,            5025,            1814,            2565,            2850,            2088,            1394,            1455,            1004,            521,            1419,            2046,            2981,            5723,            613,            1207,            1772,            1623,            572,            2298,            1923,            2137,            1445,            2183,            ],        [            2628,            2833,            2574,            3333,            1955,            1736,            1859,            3104,            2054,            2302,            1991,            2245,            1622,            2031,            1289,            1556,            0,            1781,            2099,            1063,            2006,            995,            2525,            2418,            2455,            2102,            1082,            5233,            1350,            1844,            1735,            3687,            1104,            1931,            2221,            1229,            1891,            2681,            2521,            4048,            2232,            1158,            1646,            2120,            749,            1931,            1809,            2076,            3223,            2846,            2412,            2029,            1795,            1438,            1770,            1316,            2507,            2217,            4582,            1442,            2057,            2342,            1716,            1901,            1012,            2560,            2077,            976,            1538,            2473,            5280,            1220,            1714,            1400,            2130,            1527,            1855,            1415,            1629,            1952,            1971,            ],        [            2394,            2599,            2955,            3099,            2570,            1502,            2474,            2870,            988,            1344,            1033,            709,            2172,            2581,            1055,            1322,            1157,            0,            2649,            1542,            2556,            1610,            3140,            3033,            3070,            2652,            1697,            5783,            1965,            2394,            2285,            4237,            1654,            2546,            2836,            1844,            2506,            3296,            3136,            4598,            451,            1637,            2125,            1162,            809,            2481,            2424,            2691,            3773,            3195,            2669,            248,            1561,            2053,            2385,            1866,            2921,            2832,            5132,            1921,            2672,            2957,            2195,            561,            1562,            2326,            1843,            1526,            2153,            3088,            5830,            986,            894,            1879,            1310,            1293,            2405,            2030,            2244,            538,            2586,            ],        [            3626,            3831,            4187,            4331,            3750,            2734,            3654,            4102,            2999,            3247,            2936,            3190,            658,            500,            2287,            2554,            1977,            2779,            0,            1424,            475,            2790,            3992,            4597,            4761,            2228,            2877,            5359,            3145,            1970,            1861,            3813,            1223,            2200,            3780,            3024,            3450,            4987,            4316,            4174,            3230,            2103,            918,            3065,            1989,            2057,            3276,            2694,            3349,            4427,            313,            3027,            2793,            3233,            3565,            1510,            4153,            3776,            4708,            1053,            3616,            4137,            776,            2846,            1206,            3558,            3075,            1517,            3333,            4032,            5406,            2218,            2659,            1148,            3075,            2525,            1981,            3210,            3424,            2897,            3766,            ],        [            2525,            2730,            3086,            3230,            2649,            1633,            2553,            3001,            1645,            1893,            1582,            1836,            1318,            1940,            1186,            1453,            876,            1678,            1440,            0,            1702,            1689,            3219,            3112,            3149,            1798,            1776,            4929,            2044,            1540,            1431,            3383,            800,            1770,            2915,            1923,            2585,            3375,            3215,            3744,            2094,            749,            583,            1711,            888,            1627,            2503,            2264,            2919,            3326,            1127,            1926,            1482,            2132,            2464,            803,            3052,            2911,            4278,            379,            2751,            3036,            653,            1492,            1087,            2457,            1974,            1068,            2232,            3167,            4976,            1117,            1305,            337,            1721,            1424,            1551,            2109,            2323,            1543,            2665,            ],        [            3813,            4018,            4374,            4518,            3937,            2921,            3841,            4289,            3186,            3434,            3123,            3377,            845,            687,            2474,            2741,            2164,            2966,            775,            2066,            0,            2977,            4179,            4784,            4948,            2415,            3064,            5546,            3332,            2157,            2048,            4000,            1410,            2387,            3967,            3211,            3637,            5174,            4503,            4361,            3417,            2290,            1693,            3252,            2176,            2244,            3463,            2881,            3536,            4614,            1088,            3214,            2980,            3420,            3752,            1697,            4340,            3963,            4895,            1828,            3803,            4324,            1551,            3033,            1393,            3745,            3262,            1704,            3520,            4219,            5593,            2405,            2846,            1923,            3262,            2712,            2168,            3397,            3611,            3084,            3953,            ],        [            2583,            2468,            2425,            3288,            1806,            1691,            1710,            3059,            2457,            2705,            2394,            2445,            2309,            2718,            1244,            1511,            1120,            1736,            2786,            1679,            2693,            0,            1229,            1423,            1460,            2683,            784,            5814,            692,            2395,            2162,            3162,            1791,            1823,            1124,            234,            794,            1686,            1526,            1922,            2187,            1774,            2262,            2523,            808,            2512,            1429,            1623,            2644,            2697,            2806,            1984,            1750,            836,            775,            2003,            2358,            1120,            2917,            2058,            960,            1347,            2332,            2304,            1699,            2515,            2032,            1663,            543,            1376,            3615,            1175,            2117,            2016,            2533,            1482,            2436,            1035,            1176,            2355,            1822,            ],        [            3703,            3588,            3545,            4408,            2926,            2811,            2830,            4179,            3577,            3825,            3514,            3565,            3429,            3838,            2364,            2631,            2240,            2856,            3906,            2799,            3813,            1186,            0,            519,            452,            2711,            1507,            5152,            1092,            2423,            2190,            2154,            2834,            1851,            1152,            1003,            822,            678,            548,            914,            3307,            2894,            3382,            3643,            1928,            2722,            1384,            1651,            1636,            3817,            3926,            3104,            2870,            1236,            434,            3123,            3478,            1148,            765,            3178,            988,            953,            3452,            3424,            2819,            3635,            3152,            2783,            782,            910,            1463,            2295,            3237,            3136,            3653,            2602,            2464,            1236,            1204,            3475,            2942,            ],        [            3968,            3853,            3810,            4673,            3191,            3076,            3095,            4444,            3842,            4090,            3779,            3830,            3947,            4356,            2629,            2896,            2505,            3121,            4424,            3064,            4331,            1451,            461,            0,            164,            2262,            1772,            4864,            1357,            2773,            2664,            1739,            3184,            2152,            874,            1268,            1123,            390,            849,            499,            3572,            3159,            3647,            3908,            2193,            2434,            1685,            1952,            1221,            4082,            4737,            3369,            3135,            1501,            699,            3641,            3743,            878,            350,            3443,            1038,            1218,            3717,            3689,            3337,            3900,            3417,            3648,            1047,            622,            1048,            2560,            3502,            3401,            3918,            2867,            2491,            1537,            1505,            3740,            3207,            ],        [            4058,            3943,            3900,            4763,            3281,            3166,            3185,            4534,            5024,            5272,            4961,            3920,            3841,            4250,            2719,            2986,            2595,            3211,            4318,            3904,            4225,            1541,            355,            164,            0,            2156,            1739,            4758,            1324,            2667,            2558,            1903,            3078,            1979,            710,            1358,            950,            226,            676,            663,            3662,            4128,            4487,            5090,            2283,            2328,            1512,            1779,            1385,            4172,            4631,            3459,            3225,            1468,            789,            3535,            3833,            714,            514,            4283,            874,            1308,            4557,            4871,            3231,            3990,            3507,            3542,            1137,            458,            1212,            2650,            4684,            4241,            5100,            2957,            2385,            1364,            1332,            4922,            3297,            ],        [            4106,            4311,            4667,            4811,            4230,            3214,            4134,            4582,            3479,            3727,            3416,            3670,            2296,            2705,            2767,            3034,            2457,            3259,            2773,            2359,            2680,            2895,            3144,            2369,            2533,            0,            2196,            3131,            2531,            1122,            1013,            1585,            1533,            1352,            2932,            3129,            2602,            2759,            3218,            1946,            3710,            2583,            2942,            3545,            2469,            783,            2428,            1846,            1121,            4907,            3086,            3507,            3273,            2619,            3062,            1990,            4633,            2928,            2480,            2738,            2768,            4427,            3012,            3326,            1686,            4038,            3555,            1997,            3410,            2991,            3178,            2698,            3139,            2696,            3555,            3005,            840,            2298,            2248,            3377,            4246,            ],        [            2783,            2668,            2625,            3488,            2006,            1891,            1910,            3259,            2657,            2905,            2594,            2645,            2590,            2999,            1444,            1711,            1320,            1936,            3067,            1879,            2974,            779,            1493,            1773,            1738,            1704,            0,            4993,            415,            1416,            1183,            3289,            1827,            844,            1281,            1013,            951,            1964,            1876,            3650,            2387,            1974,            2462,            2723,            1008,            1533,            874,            802,            2825,            2897,            3380,            2184,            1950,            503,            1927,            2284,            2558,            1277,            2123,            2258,            1117,            2147,            2532,            2504,            1980,            2715,            2232,            2291,            1724,            1533,            2821,            1375,            2317,            2216,            2733,            1682,            1457,            480,            694,            2555,            2022,            ],        [            5019,            5224,            5580,            5724,            5143,            4127,            5047,            5495,            4392,            4640,            4329,            4583,            3209,            3618,            3680,            3947,            3370,            4172,            3686,            3272,            3593,            3808,            4248,            3279,            3443,            1524,            3109,            0,            3444,            2035,            1926,            2495,            2446,            2265,            4036,            4042,            3706,            3669,            4128,            2856,            4623,            3496,            3855,            4458,            3382,            1696,            3532,            2950,            2031,            5820,            3999,            4420,            4186,            3532,            3972,            2903,            5546,            4032,            3390,            3651,            3872,            5337,            3925,            4239,            2599,            4951,            4468,            2910,            4320,            3901,            4088,            3611,            4052,            3609,            4468,            3918,            1753,            3402,            3161,            4290,            5159,            ],        [            2813,            2698,            2655,            3518,            2036,            1921,            1940,            3289,            2687,            2935,            2624,            2675,            2539,            2948,            1474,            1741,            1350,            1966,            3016,            1909,            2923,            663,            1078,            1358,            1323,            2064,            415,            5195,            0,            1776,            1543,            3649,            2021,            1204,            866,            897,            536,            1549,            1461,            1857,            2417,            2004,            2492,            2753,            1038,            1893,            737,            1004,            3185,            2927,            3036,            2214,            1980,            144,            1512,            2233,            2588,            862,            1708,            2288,            702,            1732,            2562,            2534,            1929,            2745,            2262,            1893,            1309,            1118,            2406,            1405,            2347,            2246,            2763,            1712,            1817,            343,            557,            2585,            2052,            ],        [            2984,            3189,            3545,            3689,            3108,            2092,            3012,            3460,            2357,            2605,            2294,            2548,            1174,            1583,            1645,            1912,            1335,            2137,            1651,            1237,            1558,            2148,            2548,            3153,            3317,            784,            2235,            3915,            1935,            0,            417,            2369,            411,            756,            2336,            2382,            2006,            3543,            2869,            2730,            2588,            1461,            1820,            2423,            1347,            613,            1832,            1250,            1905,            3785,            1964,            2385,            2151,            2023,            2923,            868,            3511,            2332,            3264,            1616,            2172,            3495,            1890,            2204,            564,            2916,            2433,            875,            2691,            2588,            3962,            1576,            2017,            1574,            2433,            1883,            537,            1702,            1652,            2255,            3124,            ],        [            3217,            3422,            3778,            3922,            3341,            2325,            3245,            3693,            2590,            2838,            2527,            2781,            1407,            1816,            1878,            2145,            1568,            2370,            1884,            1470,            1791,            1882,            2131,            2890,            2376,            521,            1183,            3652,            1518,            233,            0,            2106,            644,            339,            1919,            2116,            1589,            2602,            2452,            2467,            2821,            1694,            2053,            2656,            1580,            350,            1415,            833,            1642,            4018,            2197,            2618,            2384,            1606,            2565,            1101,            3744,            1915,            3001,            1849,            1755,            3084,            2123,            2437,            797,            3149,            2666,            1108,            2913,            2171,            3699,            1809,            2250,            1807,            2666,            2116,            274,            1285,            1235,            2488,            3357,            ],        [            5958,            5659,            5521,            6255,            5823,            3854,            3873,            6044,            6090,            6338,            6027,            6281,            4907,            5316,            3407,            3674,            5068,            3899,            5384,            4970,            5291,            2229,            1245,            784,            948,            3222,            2550,            5824,            2135,            3733,            3624,            0,            4144,            3348,            1658,            2046,            1907,            1174,            1633,            361,            4350,            5194,            5553,            6156,            2971,            3394,            3578,            2996,            2077,            5590,            5697,            4147,            3913,            2279,            1477,            4601,            5929,            1662,            895,            5349,            1822,            2842,            5623,            5937,            4297,            6714,            4195,            4608,            1825,            1406,            1593,            3338,            5750,            5307,            6166,            3645,            3451,            3448,            2289,            5988,            6071,            ],        [            2765,            2970,            3326,            3470,            2889,            1873,            2793,            3241,            2138,            2386,            2075,            2329,            955,            1364,            1426,            1693,            1116,            1918,            1432,            1018,            1339,            1929,            2866,            3471,            3635,            1102,            1918,            4233,            2253,            844,            735,            2687,            0,            1074,            2654,            2163,            2324,            3861,            3187,            3048,            2369,            1242,            1601,            2204,            1128,            931,            2150,            1568,            2223,            3566,            1745,            2166,            1932,            2341,            3300,            649,            3292,            2650,            3582,            1397,            2490,            3276,            1671,            1985,            345,            2697,            2214,            656,            2472,            2906,            4280,            1357,            1798,            1355,            2214,            1664,            855,            2020,            1970,            2036,            2905,            ],        [            3556,            3761,            3389,            4261,            2770,            2664,            2674,            4032,            2929,            3177,            2866,            3120,            1746,            2155,            2217,            2484,            1907,            2709,            2223,            1809,            2130,            1543,            1792,            2311,            2037,            860,            844,            3991,            1179,            572,            339,            2445,            983,            0,            1580,            1777,            1250,            2263,            2113,            2806,            3160,            2033,            2392,            2995,            1919,            689,            1076,            494,            1981,            3661,            2536,            2957,            2723,            1267,            2226,            1440,            3322,            1576,            3340,            2188,            1416,            2745,            2462,            2776,            1136,            3488,            3005,            1447,            2574,            1832,            4038,            2148,            2589,            2146,            3005,            2455,            613,            946,            896,            2827,            2786,            ],        [            3679,            3564,            3521,            4384,            2902,            2787,            2806,            4155,            3553,            3801,            3490,            3541,            3267,            3676,            2340,            2607,            2216,            2832,            3744,            2775,            3651,            1529,            1008,            874,            710,            2381,            1281,            5468,            866,            2093,            1860,            2613,            2504,            1521,            0,            1240,            330,            484,            1329,            1373,            3283,            2870,            3358,            3619,            1904,            2210,            769,            1321,            2095,            3793,            4057,            3080,            2846,            1010,            1442,            2961,            3454,            4,            1224,            3154,            164,            1961,            3428,            3400,            2657,            3611,            3128,            2968,            1790,            252,            1922,            2271,            3213,            3112,            3629,            2578,            2134,            906,            874,            3451,            2918,            ],        [            2930,            2815,            2772,            3635,            2153,            2038,            2057,            3406,            2804,            3052,            2741,            2792,            2656,            3065,            1591,            1858,            1467,            2083,            3133,            2026,            3040,            413,            995,            1189,            1226,            3455,            1275,            5940,            860,            3167,            2934,            2928,            2138,            2595,            1292,            0,            962,            1452,            1292,            1688,            2534,            2121,            2609,            2870,            1155,            3284,            1597,            2395,            2410,            3044,            3153,            2331,            2097,            1004,            541,            2350,            2705,            1288,            2683,            2405,            1128,            1113,            2679,            2651,            2046,            2862,            2379,            2010,            309,            1684,            3381,            1522,            2464,            2363,            2880,            1829,            3208,            1203,            1948,            2702,            2169,            ],        [            3349,            3234,            3191,            4054,            2572,            2457,            2476,            3825,            3223,            3471,            3160,            3211,            2937,            3346,            2010,            2277,            1886,            2502,            3414,            2445,            3321,            1199,            678,            1197,            923,            2051,            951,            5182,            536,            1763,            1530,            3636,            2174,            1191,            330,            910,            0,            814,            999,            1592,            2953,            2540,            3028,            3289,            1574,            1880,            439,            991,            2314,            3463,            3727,            2750,            2516,            680,            1112,            2631,            3124,            326,            1443,            2824,            166,            1631,            3098,            3070,            2327,            3281,            2798,            2638,            1460,            582,            2141,            1941,            2883,            2782,            3299,            2248,            1804,            576,            544,            3121,            2588,            ],        [            4284,            4169,            4126,            4989,            3507,            3392,            3411,            4760,            5250,            5498,            5187,            4146,            4067,            4476,            2945,            3212,            2821,            3437,            4544,            4130,            4451,            1767,            581,            390,            226,            2382,            1965,            4984,            1550,            2893,            2784,            2129,            3304,            2205,            484,            1584,            814,            0,            902,            889,            3888,            4354,            4713,            5316,            2509,            2554,            1738,            2005,            1611,            4398,            4857,            3685,            3451,            1694,            1015,            3761,            4059,            488,            740,            4509,            648,            1534,            4783,            5097,            3457,            4216,            3733,            3768,            1363,            232,            1438,            2876,            4910,            4467,            5326,            3183,            2611,            1590,            1558,            5148,            3523,            ],        [            3461,            3346,            3303,            4166,            2684,            2569,            2588,            3937,            3335,            3583,            3272,            3323,            3049,            3458,            2122,            2389,            1998,            2614,            3526,            2557,            3433,            1311,            566,            1085,            811,            2163,            1063,            5294,            648,            1875,            1642,            3748,            2286,            1303,            604,            1022,            274,            1037,            0,            1480,            3065,            2652,            3140,            3401,            1686,            1992,            836,            1103,            2202,            3575,            3839,            2862,            2628,            792,            1000,            2743,            3236,            600,            1331,            2936,            440,            1519,            3210,            3182,            2439,            3393,            2910,            2750,            1348,            856,            2029,            2053,            2995,            2894,            3411,            2360,            1916,            688,            656,            3233,            2700,            ],        [            4461,            5412,            5274,            6008,            3684,            3569,            3588,            5797,            5815,            6063,            5752,            6006,            4632,            5041,            3122,            3389,            4793,            3614,            5109,            4695,            5016,            1944,            960,            499,            663,            2947,            2265,            5549,            1850,            3458,            3349,            2320,            3869,            3073,            1373,            1761,            1622,            889,            1348,            0,            4065,            4919,            5278,            5881,            2686,            3119,            3303,            2721,            1802,            5343,            5422,            3862,            3628,            1994,            1192,            4326,            5682,            1377,            563,            5074,            1537,            2595,            5348,            5662,            4022,            4393,            3910,            4333,            1540,            1121,            1261,            3053,            5475,            5032,            5891,            3360,            3176,            2036,            2004,            5713,            3700,            ],        [            2845,            3050,            3406,            3550,            3021,            1953,            2925,            3321,            1029,            1385,            1074,            258,            2623,            3032,            1506,            1773,            1608,            451,            3100,            2080,            3007,            2061,            3591,            3484,            3521,            3103,            2148,            6234,            2416,            2845,            2736,            4688,            2105,            2997,            3287,            2295,            2957,            3747,            3587,            5049,            0,            1503,            2663,            1203,            1260,            2932,            2875,            3142,            4224,            3646,            3207,            203,            1701,            2504,            2836,            2317,            3372,            3283,            5583,            2459,            3123,            3408,            2733,            602,            2013,            2777,            2294,            1977,            2604,            3539,            6281,            1437,            935,            2417,            1351,            1744,            2856,            2481,            2695,            551,            3037,            ],        [            2660,            2865,            3221,            3365,            3145,            1768,            3049,            3136,            896,            1144,            833,            1087,            2012,            2421,            1297,            1588,            1372,            1091,            2489,            1150,            2396,            2185,            3715,            3608,            3645,            2492,            2272,            5623,            2540,            2234,            2125,            4077,            1494,            2464,            3411,            2419,            3081,            3871,            3711,            4438,            1345,            0,            1733,            962,            1384,            2321,            2999,            2958,            3613,            3461,            2277,            1184,            733,            2628,            2960,            1497,            3187,            3407,            4972,            1529,            3247,            3532,            1803,            743,            1583,            2592,            2109,            1762,            2728,            3663,            5670,            1576,            556,            1487,            972,            1001,            2245,            2605,            2819,            794,            3771,            ],        [            3031,            3236,            3592,            3736,            3155,            2139,            3059,            3507,            2151,            2399,            2088,            2342,            1742,            1584,            1692,            1959,            1382,            2184,            1084,            506,            1559,            2195,            3725,            3618,            3655,            2304,            2282,            5435,            2550,            2046,            1937,            3889,            1306,            2276,            3421,            2429,            3091,            3881,            3721,            4250,            2600,            1255,            0,            2217,            1394,            2133,            3009,            2770,            3425,            3832,            771,            2432,            1988,            2638,            2970,            1309,            3558,            3417,            4784,            135,            3257,            3542,            297,            1998,            1593,            2963,            2480,            1574,            2738,            3673,            5482,            1623,            1811,            230,            2227,            1930,            2057,            2615,            2829,            2049,            3171,            ],        [            3247,            3452,            3808,            3952,            3694,            2355,            3598,            3723,            378,            182,            129,            945,            2561,            2970,            1884,            2175,            1921,            949,            3038,            1699,            2945,            2734,            4264,            4157,            4194,            3041,            2821,            6172,            3089,            2783,            2674,            4626,            2043,            3013,            3960,            2968,            3630,            4420,            4260,            4987,            1203,            1122,            2282,            0,            1933,            2870,            3548,            3507,            4162,            4048,            2826,            1042,            1320,            3177,            3509,            2046,            3774,            3956,            5521,            2078,            3796,            4081,            2352,            601,            2132,            3179,            2696,            2311,            3277,            4212,            6219,            2162,            554,            2036,            970,            1588,            2794,            3154,            3368,            652,            3710,            ],        [            1775,            1980,            2336,            2480,            1761,            883,            1665,            2251,            1649,            1897,            1586,            1637,            1501,            1910,            436,            703,            348,            928,            1978,            871,            1885,            801,            2331,            2224,            2261,            1981,            888,            5112,            1156,            1723,            1614,            3566,            983,            1737,            2027,            1035,            1697,            2487,            2327,            3927,            1379,            966,            1454,            1715,            0,            1810,            1615,            1882,            3102,            2576,            1998,            1176,            942,            1244,            1576,            1195,            2302,            2023,            4461,            1250,            1863,            2148,            1524,            1496,            891,            1707,            1224,            855,            1344,            2279,            5159,            367,            1309,            1208,            1725,            674,            1734,            1221,            1435,            1547,            1777,            ],        [            3323,            3528,            3884,            4028,            3447,            2431,            3351,            3799,            2696,            2944,            2633,            2887,            1513,            1922,            1984,            2251,            1674,            2476,            1990,            1576,            1897,            2112,            2361,            2776,            2940,            407,            1413,            3538,            1748,            339,            230,            1992,            750,            569,            2149,            2346,            1819,            3166,            2682,            2353,            2927,            1800,            2159,            2762,            1686,            0,            1645,            1063,            1528,            4124,            2303,            2724,            2490,            1836,            2795,            1207,            3850,            2145,            2887,            1955,            1985,            3314,            2229,            2543,            903,            3255,            2772,            1214,            3143,            2401,            3585,            1915,            2356,            1913,            2772,            2222,            140,            1515,            1465,            2594,            3463,            ],        [            3387,            3272,            3229,            4092,            2610,            2495,            2514,            3863,            3261,            3509,            3198,            3249,            2975,            3384,            2048,            2315,            1924,            2540,            3452,            2483,            3359,            1237,            716,            1235,            961,            2089,            989,            5220,            574,            1801,            1568,            2870,            2212,            1229,            504,            948,            174,            1187,            1037,            1630,            2991,            2578,            3066,            3327,            1612,            1918,            0,            1029,            2352,            3501,            3765,            2788,            2554,            718,            1150,            2669,            3162,            500,            1481,            2862,            340,            1669,            3136,            3108,            2365,            3319,            2836,            2676,            1498,            756,            2179,            1979,            2921,            2820,            3337,            2286,            1842,            614,            582,            3159,            2626,            ],        [            3371,            3256,            3213,            4076,            2594,            2479,            2498,            3847,            3245,            3493,            3182,            3233,            2366,            2775,            2032,            2299,            1908,            2524,            2843,            2467,            2750,            1367,            1298,            1817,            1543,            1480,            932,            4611,            795,            1192,            959,            3065,            1603,            620,            1086,            1601,            756,            1769,            1619,            2212,            2975,            2562,            3050,            3311,            1596,            1309,            582,            0,            2601,            3485,            3156,            2772,            2538,            883,            1732,            2060,            3146,            1082,            2063,            2846,            922,            2251,            3120,            3092,            1756,            3303,            2820,            2067,            2080,            1338,            2761,            1963,            2905,            2804,            3321,            2270,            1233,            452,            420,            3143,            2610,            ],        [            4643,            4848,            4132,            5348,            3513,            3751,            3417,            5119,            4016,            4264,            3953,            4207,            2833,            3242,            3304,            3571,            2994,            3796,            3310,            2896,            3217,            2286,            2217,            2992,            3156,            1148,            1579,            3750,            1714,            1659,            1550,            2208,            2070,            1271,            2005,            2520,            1675,            3382,            2538,            2569,            4247,            3120,            3479,            4082,            3006,            1320,            1501,            919,            0,            4404,            3623,            4044,            3810,            1802,            2651,            2527,            4065,            2001,            3103,            3275,            1841,            3170,            3549,            3863,            2223,            4575,            4092,            2534,            2999,            2257,            3801,            3235,            3676,            3233,            4092,            3542,            1377,            1371,            1339,            3914,            3529,            ],        [            946,            647,            604,            669,            1525,            1754,            1062,            536,            3481,            3729,            3418,            3672,            3999,            4408,            2268,            1934,            2984,            3108,            4476,            3369,            4383,            2880,            4410,            5330,            4340,            4479,            2967,            7610,            3235,            4221,            4112,            6064,            3481,            3816,            4106,            3114,            3776,            4566,            4406,            5641,            3559,            3464,            3952,            3547,            2636,            4308,            3694,            3961,            6351,            0,            4496,            3356,            2774,            3323,            3655,            3693,            339,            4102,            5269,            3748,            3942,            4227,            4022,            3328,            3389,            1175,            1512,            3353,            3423,            4358,            5967,            2547,            3141,            3706,            3557,            2506,            4232,            3300,            3514,            3379,            1154,            ],        [            3636,            3841,            4197,            4341,            3760,            2744,            3664,            4112,            2756,            3004,            2693,            2947,            971,            813,            2297,            2564,            1987,            2789,            313,            1111,            788,            2800,            4305,            4910,            5074,            2541,            2887,            5672,            3155,            2283,            2174,            4126,            1536,            2513,            4093,            3034,            3763,            5300,            4326,            4487,            3205,            1860,            605,            2822,            1999,            2370,            3589,            3007,            3662,            4437,            0,            3037,            2593,            3243,            3575,            1823,            4163,            4089,            5021,            740,            3929,            4147,            463,            2603,            1519,            3568,            3085,            1830,            3343,            4345,            5719,            2228,            2416,            835,            2832,            2535,            2294,            3220,            3434,            2654,            3776,            ],        [            2642,            2847,            3203,            3347,            2818,            1750,            2722,            3118,            868,            1224,            913,            461,            2420,            2829,            1303,            1570,            1405,            248,            2897,            1790,            2804,            1858,            3388,            3281,            3318,            2900,            1945,            6031,            2213,            2642,            2533,            4485,            1902,            2794,            3084,            2092,            2754,            3544,            3384,            4846,            203,            1342,            2373,            1042,            1057,            2729,            2672,            2939,            4021,            3443,            2917,            0,            1540,            2301,            2633,            2114,            3169,            3080,            5380,            2169,            2920,            3205,            2443,            441,            1810,            2574,            2091,            1774,            2401,            3336,            6078,            1234,            774,            2127,            1190,            1541,            2653,            2278,            2492,            418,            2834,            ],        [            1927,            2132,            2488,            2632,            2693,            1035,            2597,            2403,            707,            955,            644,            898,            2295,            2704,            564,            855,            1280,            1404,            2772,            1536,            2679,            1733,            3263,            3156,            3193,            2775,            1820,            5906,            2088,            2517,            2408,            4360,            1777,            2669,            2959,            1967,            2629,            3419,            3259,            4721,            1156,            959,            2119,            773,            932,            2604,            2547,            2814,            3896,            2728,            2663,            995,            0,            2176,            2508,            1989,            2454,            2955,            5255,            1915,            2795,            3080,            2189,            554,            1685,            1859,            1376,            1649,            2276,            3211,            5953,            843,            367,            1873,            783,            268,            2528,            2153,            2367,            605,            3038,            ],        [            2669,            2554,            2511,            3374,            1892,            1777,            1796,            3145,            2543,            2791,            2480,            2531,            2395,            2804,            1330,            1597,            1206,            1822,            2872,            1765,            2779,            519,            1411,            1446,            1483,            2617,            870,            5748,            553,            2329,            2096,            4202,            1877,            1757,            1199,            753,            869,            1709,            1549,            1945,            2273,            1860,            2348,            2609,            894,            2446,            1290,            1557,            2667,            2783,            2892,            2070,            1836,            0,            1294,            2089,            2444,            1195,            1796,            2144,            1035,            1866,            2418,            2390,            1785,            2601,            2118,            1749,            1062,            1451,            2494,            1261,            2203,            2102,            2619,            1568,            2370,            896,            1110,            2441,            1908,            ],        [            3269,            3154,            3111,            3974,            2492,            2377,            2396,            3745,            3143,            3391,            3080,            3131,            2995,            3404,            1930,            2197,            1806,            2422,            3472,            2365,            3379,            752,            671,            865,            902,            3131,            1470,            5616,            1097,            2843,            2610,            2604,            2477,            2271,            1572,            569,            1242,            1128,            968,            1364,            2873,            2460,            2948,            3209,            1494,            2960,            1804,            2071,            2086,            3383,            3492,            2670,            2436,            1241,            0,            2689,            3044,            1568,            1215,            2744,            1408,            918,            3018,            2990,            2385,            3201,            2718,            2349,            348,            1360,            1913,            1861,            2803,            2702,            3219,            2168,            2884,            1656,            1624,            3041,            2508,            ],        [            2369,            2574,            2930,            3074,            2493,            1477,            2397,            2845,            1489,            1737,            1426,            1680,            1247,            1656,            1030,            1297,            720,            1522,            1724,            369,            1631,            1533,            3063,            2956,            2993,            1727,            1620,            4858,            1888,            1469,            1360,            3312,            729,            1699,            2759,            1767,            2429,            3219,            3059,            3673,            1938,            593,            952,            1555,            732,            1556,            2347,            2193,            2848,            3170,            1496,            1770,            1326,            1976,            2308,            0,            2896,            2755,            4207,            748,            2595,            2880,            1022,            1336,            931,            2301,            1818,            895,            2076,            3011,            4905,            961,            1149,            706,            1565,            1268,            1480,            1953,            2167,            1387,            2509,            ],        [            607,            308,            265,            1008,            1186,            1415,            723,            875,            3142,            3390,            3079,            3333,            3660,            4069,            1929,            1595,            2645,            2769,            4137,            3030,            4044,            2541,            4071,            3964,            4001,            4140,            2628,            7271,            2896,            3882,            3773,            5725,            3142,            3477,            3767,            2775,            3437,            4227,            4067,            6086,            3220,            3125,            3613,            3208,            2297,            3969,            3355,            3622,            5261,            339,            4157,            3017,            2435,            2984,            3316,            3354,            0,            3763,            5608,            3409,            3603,            3888,            3683,            2989,            3050,            836,            1173,            3014,            3084,            4019,            6306,            2208,            2802,            3367,            3218,            2167,            3893,            2961,            3175,            3040,            815,            ],        [            3675,            3560,            3517,            4380,            2898,            2783,            2802,            4151,            3549,            3797,            3486,            3537,            3263,            3672,            2336,            2603,            2212,            2828,            3740,            2771,            3647,            1525,            1004,            878,            714,            2377,            1277,            5472,            862,            2089,            1856,            2617,            2500,            1517,            4,            1236,            326,            488,            1325,            1377,            3279,            2866,            3354,            3615,            1900,            2206,            765,            1317,            2099,            3789,            4053,            3076,            2842,            1006,            1438,            2957,            3450,            0,            1228,            3150,            160,            1957,            3424,            3396,            2653,            3607,            3124,            2964,            1786,            256,            1926,            2267,            3209,            3108,            3625,            2574,            2130,            902,            870,            3447,            2914,            ],        [            8972,            7915,            7777,            8511,            6496,            8080,            6400,            8300,            8345,            8593,            8282,            8536,            7162,            7571,            7633,            7900,            7323,            8125,            7639,            7225,            7546,            4756,            3848,            3311,            3551,            5477,            5911,            8079,            4662,            5988,            5879,            4850,            6399,            5603,            4261,            4573,            4443,            3777,            4169,            3622,            8576,            7449,            7808,            8411,            5498,            5649,            5833,            5251,            4332,            7846,            7952,            8373,            8139,            4806,            4004,            6856,            8185,            4265,            0,            7604,            4425,            5098,            7878,            8192,            6552,            8904,            8421,            6863,            4352,            4009,            698,            7564,            8005,            7562,            8421,            7871,            5706,            5703,            4825,            8243,            6512,            ],        [            2896,            3101,            3457,            3601,            3020,            2004,            2924,            3372,            2016,            2264,            1953,            2207,            1735,            1577,            1557,            1824,            1247,            2049,            1077,            371,            1552,            2060,            3590,            3483,            3520,            2169,            2147,            5300,            2415,            1911,            1802,            3754,            1171,            2141,            3286,            2294,            2956,            3746,            3586,            4115,            2465,            1120,            220,            2082,            1259,            1998,            2874,            2635,            3290,            3697,            764,            2297,            1853,            2503,            2835,            1174,            3423,            3282,            4649,            0,            3122,            3407,            290,            1863,            1458,            2828,            2345,            1439,            2603,            3538,            5347,            1488,            1676,            95,            2092,            1795,            1922,            2480,            2694,            1914,            3036,            ],        [            3515,            3400,            3357,            4220,            2738,            2623,            2642,            3991,            3389,            3637,            3326,            3377,            3103,            3512,            2176,            2443,            2052,            2668,            3580,            2611,            3487,            1365,            844,            1038,            874,            2217,            1117,            5348,            702,            1929,            1696,            3802,            2340,            1357,            164,            1076,            166,            648,            1165,            1537,            3119,            2706,            3194,            3455,            1740,            2046,            605,            1157,            2259,            3629,            3893,            2916,            2682,            846,            1278,            2797,            3290,            160,            1388,            2990,            0,            1797,            3264,            3236,            2493,            3447,            2964,            2804,            1626,            416,            2086,            2107,            3049,            2948,            3465,            2414,            1970,            742,            710,            3287,            2754,            ],        [            3370,            3255,            3212,            4075,            2593,            2478,            2497,            3846,            3244,            3492,            3181,            3232,            3096,            3505,            2031,            2298,            1907,            2523,            3573,            2466,            3480,            853,            558,            752,            789,            3018,            1571,            5503,            1208,            2730,            2497,            2491,            2578,            2158,            1459,            670,            1129,            1015,            855,            1251,            2974,            2561,            3049,            3310,            1595,            2847,            1691,            1958,            1973,            3484,            3593,            2771,            2537,            1352,            101,            2790,            3145,            1455,            1102,            2845,            1295,            0,            3119,            3091,            2486,            3302,            2819,            2450,            449,            1247,            1800,            1962,            2904,            2803,            3320,            2269,            2771,            1543,            1511,            3142,            2609,            ],        [            3173,            3378,            3734,            3878,            3297,            2281,            3201,            3649,            2293,            2541,            2230,            2484,            1445,            1287,            1834,            2101,            1524,            2326,            787,            648,            1262,            2337,            3867,            3760,            3797,            2446,            2424,            5577,            2692,            2188,            2079,            4031,            1448,            2418,            3563,            2571,            3233,            4023,            3863,            4392,            2742,            1397,            142,            2359,            1536,            2275,            3151,            2912,            3567,            3974,            474,            2574,            2130,            2780,            3112,            1451,            3700,            3559,            4926,            277,            3399,            3684,            0,            2140,            1735,            3105,            2622,            1716,            2880,            3815,            5624,            1765,            1953,            372,            2369,            2072,            2199,            2757,            2971,            2191,            3313,            ],        [            3026,            3231,            3587,            3731,            3473,            2134,            3377,            3502,            427,            783,            472,            344,            2340,            2749,            1663,            1954,            1700,            561,            2817,            1478,            2724,            2513,            4043,            3936,            3973,            2820,            2600,            5951,            2868,            2562,            2453,            4405,            1822,            2792,            3739,            2747,            3409,            4199,            4039,            4766,            602,            901,            2061,            601,            1712,            2649,            3327,            3286,            3941,            3827,            2605,            441,            1099,            2956,            3288,            1825,            3553,            3735,            5300,            1857,            3575,            3860,            2131,            0,            1911,            2958,            2475,            2090,            3056,            3991,            5998,            1941,            333,            1815,            749,            1367,            2573,            2933,            3147,            51,            3489,            ],        [            2673,            2878,            3234,            3378,            2797,            1781,            2701,            3149,            1793,            2041,            1730,            1984,            841,            1250,            1334,            1601,            1024,            1826,            1318,            673,            1225,            1837,            3085,            3690,            3854,            1321,            1924,            4452,            2192,            1063,            954,            2906,            323,            1293,            3063,            2071,            2733,            4080,            3363,            3267,            2242,            897,            1256,            1859,            1036,            1150,            2369,            1787,            2442,            3474,            1631,            2074,            1630,            2280,            2612,            304,            3200,            3059,            3801,            1052,            2899,            3184,            1326,            1640,            0,            2605,            2122,            591,            2380,            3315,            4499,            1265,            1453,            1010,            1869,            1572,            1074,            2257,            2471,            1691,            2813,            ],        [            455,            660,            1016,            1160,            1937,            579,            1474,            931,            2306,            2554,            2243,            2497,            2824,            3233,            1093,            759,            1809,            1933,            3301,            2194,            3208,            2262,            3792,            3685,            3722,            3304,            2349,            6435,            2617,            3046,            2937,            4889,            2306,            3198,            3488,            2496,            3158,            3948,            3788,            5250,            2384,            2289,            2777,            2372,            1461,            3133,            3076,            3343,            4425,            1256,            3321,            2181,            1599,            2705,            3037,            2518,            982,            3484,            5784,            2573,            3324,            3609,            2847,            2153,            2214,            0,            337,            2178,            2805,            3740,            6482,            1372,            1966,            2531,            2382,            1331,            3057,            2682,            2896,            2204,            1566,            ],        [            551,            756,            1112,            1256,            2033,            473,            1570,            1027,            2200,            2448,            2137,            2391,            2718,            3127,            987,            653,            1703,            1827,            3195,            2088,            3102,            2156,            3686,            3579,            3616,            3198,            2243,            6329,            2511,            2940,            2831,            4783,            2200,            3092,            3382,            2390,            3052,            3842,            3682,            5144,            2278,            2183,            2671,            2266,            1355,            3027,            2970,            3237,            4319,            1352,            3215,            2075,            1493,            2599,            2931,            2412,            1078,            3378,            5678,            2467,            3218,            3503,            2741,            2047,            2108,            483,            0,            2072,            2699,            3634,            6376,            1266,            1860,            2425,            1593,            1225,            2951,            2576,            2790,            2098,            1662,            ],        [            2109,            2314,            2670,            2814,            2233,            1217,            2137,            2585,            1983,            2231,            1920,            1971,            1103,            1512,            770,            1037,            460,            1262,            1580,            935,            1487,            1273,            2803,            2696,            2733,            1583,            1360,            4714,            1628,            1325,            1216,            3168,            585,            1555,            2499,            1507,            2169,            2959,            2799,            3529,            1713,            1030,            1518,            2049,            472,            1412,            2087,            2354,            2704,            2910,            1893,            1510,            1276,            1716,            2048,            797,            2636,            2495,            4063,            1314,            2335,            2620,            1588,            1830,            493,            2041,            1558,            0,            1816,            2751,            4761,            701,            1643,            1272,            2059,            1008,            1336,            1693,            1907,            1881,            2249,            ],        [            2921,            2806,            2763,            3626,            2144,            2029,            2048,            3397,            2795,            3043,            2732,            2783,            2647,            3056,            1582,            1849,            1458,            2074,            3124,            2017,            3031,            404,            932,            1126,            1163,            2831,            1122,            5877,            887,            2543,            2310,            2865,            2129,            1971,            1319,            221,            989,            1389,            1229,            1625,            2525,            2112,            2600,            2861,            1146,            2660,            1624,            1818,            2347,            3035,            3144,            2322,            2088,            1031,            434,            2341,            2696,            1315,            1476,            2396,            1155,            1006,            2670,            2642,            2037,            2853,            2370,            2001,            0,            1621,            2174,            1513,            2455,            2354,            2871,            1820,            2584,            1230,            1371,            2693,            2160,            ],        [            3931,            3816,            3773,            4636,            3154,            3039,            3058,            4407,            3805,            4053,            3742,            3793,            4299,            4708,            2592,            2859,            2468,            3084,            4776,            3027,            4683,            1781,            813,            622,            458,            2614,            1533,            5216,            1118,            3125,            2112,            2361,            3536,            1773,            252,            1492,            582,            232,            1134,            1121,            3535,            3122,            3610,            3871,            2156,            2786,            1021,            1573,            1843,            4045,            5089,            3332,            3098,            1262,            1247,            3993,            3706,            256,            972,            3406,            416,            1766,            3680,            3652,            3689,            3863,            3380,            4000,            1595,            0,            1670,            2523,            3465,            3364,            3881,            2830,            2843,            1158,            1126,            3703,            3170,            ],        [            9082,            8025,            7887,            8621,            6606,            8190,            6510,            8410,            8455,            8703,            8392,            8646,            7272,            7681,            7743,            8010,            7433,            8235,            7749,            7335,            7656,            4866,            3958,            3421,            3661,            5587,            6021,            8189,            4772,            6098,            5989,            4960,            6509,            5713,            4371,            4683,            4553,            3887,            4279,            3732,            8686,            7559,            7918,            8521,            5608,            5759,            5943,            5361,            4442,            7956,            8062,            8483,            8249,            4916,            4114,            6966,            8295,            4375,            3360,            7714,            4535,            5208,            7988,            8302,            6662,            9014,            8531,            6973,            4462,            4119,            0,            7674,            8115,            7672,            8531,            7981,            5816,            5813,            4935,            8353,            6622,            ],        [            2173,            2378,            2734,            2878,            2321,            1281,            2225,            2649,            2284,            1610,            1299,            1270,            1923,            2332,            834,            1101,            908,            561,            2400,            1293,            2307,            1361,            2891,            2784,            2821,            2403,            1448,            5534,            1716,            2145,            2036,            3988,            1405,            2297,            2587,            1595,            2257,            3047,            2887,            4349,            1012,            1388,            1876,            1428,            560,            2232,            2175,            2442,            3524,            2974,            2420,            809,            1340,            1804,            2136,            1617,            2700,            2583,            4883,            1672,            2423,            2708,            1946,            1122,            1313,            2105,            1622,            1277,            1904,            2839,            5581,            0,            1944,            1630,            2360,            1072,            2156,            1781,            1995,            1099,            2337,            ],        [            2220,            2425,            2781,            2925,            3656,            2317,            3560,            2696,            340,            802,            491,            1023,            2523,            2932,            1846,            2137,            1883,            1027,            3000,            1661,            2907,            2696,            4226,            4119,            4156,            3003,            2783,            6134,            3051,            2745,            2636,            4588,            2005,            2975,            3922,            2930,            3592,            4382,            4222,            4949,            1281,            1084,            2244,            620,            1895,            2832,            3510,            3469,            4124,            3021,            2788,            1120,            1282,            3139,            3471,            2008,            2747,            3918,            5483,            2040,            3758,            4043,            2314,            679,            2094,            2152,            1884,            2273,            3239,            4174,            6181,            2124,            0,            1998,            416,            1550,            2756,            3116,            3330,            730,            3331,            ],        [            2801,            3006,            3362,            3506,            2925,            1909,            2829,            3277,            1921,            2169,            1858,            2112,            1594,            2003,            1462,            1729,            1152,            1954,            1535,            276,            1978,            1965,            3495,            3388,            3425,            2074,            2052,            5205,            2320,            1816,            1707,            3659,            1076,            2046,            3191,            2199,            2861,            3651,            3491,            4020,            2370,            1025,            678,            1987,            1164,            1903,            2779,            2540,            3195,            3602,            1222,            2202,            1758,            2408,            2740,            1079,            3328,            3187,            4554,            474,            3027,            3312,            748,            1768,            1363,            2733,            2250,            1344,            2508,            3443,            5252,            1393,            1581,            0,            1997,            1700,            1827,            2385,            2599,            1819,            2941,            ],        [            2018,            2223,            2579,            2723,            3927,            1924,            3831,            2494,            611,            1073,            762,            1294,            2794,            3203,            2117,            2408,            2154,            1298,            3271,            1932,            3178,            2967,            4497,            4390,            4427,            3274,            3054,            6405,            3322,            3016,            2907,            4859,            2276,            3246,            4193,            3201,            3863,            4653,            4493,            5220,            1552,            1355,            2515,            891,            2166,            3103,            3781,            3740,            4395,            2819,            3059,            1391,            1553,            3410,            3742,            2279,            2545,            4189,            5754,            2311,            4029,            4314,            2585,            950,            2365,            1950,            1682,            2544,            3510,            4445,            6452,            2395,            787,            2269,            0,            1821,            3027,            3387,            3601,            1001,            3129,            ],        [            1659,            1864,            2220,            2364,            2425,            767,            2329,            2135,            975,            1223,            912,            1166,            2027,            2436,            296,            587,            1012,            1136,            2504,            1397,            2411,            1465,            2995,            2888,            2925,            2507,            1552,            5638,            1820,            2249,            2140,            4092,            1509,            2401,            2691,            1699,            2361,            3151,            2991,            4453,            1587,            1227,            1980,            1041,            664,            2336,            2279,            2546,            3628,            2460,            2524,            1384,            268,            1908,            2240,            1721,            2186,            2687,            4987,            1776,            2527,            2812,            2050,            822,            1417,            1591,            1108,            1381,            2008,            2943,            5685,            575,            635,            1734,            1051,            0,            2260,            1885,            2099,            873,            2770,            ],        [            3343,            3548,            3904,            4048,            3467,            2451,            3371,            3819,            2716,            2964,            2653,            2907,            1533,            1942,            2004,            2271,            1694,            2496,            2010,            1596,            1917,            2132,            2381,            2796,            2960,            427,            1433,            3558,            1768,            359,            250,            2012,            770,            589,            2169,            2366,            1839,            3186,            2702,            2373,            2947,            1820,            2179,            2782,            1706,            143,            1665,            1083,            1548,            4144,            2323,            2744,            2510,            1856,            2815,            1227,            3870,            2165,            2907,            1975,            2005,            4854,            2249,            2563,            923,            3275,            2792,            1234,            3163,            2421,            3605,            1935,            2376,            1933,            2792,            2242,            0,            1535,            1485,            2614,            3483,            ],        [            2919,            2804,            2761,            3624,            2142,            2027,            2046,            3395,            2793,            3041,            2730,            2781,            2607,            3016,            1580,            1847,            1456,            2072,            3084,            2015,            2991,            915,            1110,            1629,            1355,            1721,            480,            4852,            343,            1433,            1200,            3306,            1844,            861,            898,            1149,            568,            1581,            1431,            2024,            2523,            2110,            2598,            2859,            1144,            1550,            394,            661,            2842,            3033,            3397,            2320,            2086,            431,            1544,            2301,            2694,            894,            1875,            2394,            734,            2063,            2668,            2640,            1997,            2851,            2368,            2308,            1652,            1150,            2573,            1511,            2453,            2352,            2869,            1818,            1474,            0,            214,            2691,            2158,            ],        [            3818,            3703,            3660,            4523,            3041,            2926,            2945,            4294,            3576,            3824,            3513,            3767,            2393,            2802,            2479,            2746,            2554,            2971,            2870,            2456,            2777,            1814,            1745,            2264,            1990,            1507,            955,            4638,            1242,            1219,            986,            3092,            1630,            647,            1533,            2048,            1203,            2216,            2066,            3453,            3422,            2680,            3039,            3642,            2043,            1336,            1029,            447,            2628,            3932,            3183,            3219,            2985,            1330,            2179,            2087,            3593,            1529,            3987,            2835,            1369,            2698,            3109,            3423,            1783,            3750,            3267,            2094,            2527,            1785,            4685,            2410,            3236,            2793,            3652,            2717,            1260,            899,            0,            3474,            3057,            ],        [            3077,            3282,            3638,            3782,            3524,            2185,            3428,            3553,            478,            834,            523,            293,            2391,            2800,            1714,            2005,            1751,            538,            2868,            1529,            2775,            2564,            4094,            3987,            4024,            2871,            2651,            6002,            2919,            2613,            2504,            4456,            1873,            2843,            3790,            2798,            3460,            4250,            4090,            4817,            551,            952,            2112,            652,            1763,            2700,            3378,            3337,            3992,            3878,            2656,            418,            1150,            3007,            3339,            1876,            3604,            3786,            5351,            1908,            3626,            3911,            2182,            51,            1962,            3009,            2526,            2141,            3107,            4042,            6049,            1992,            384,            1866,            800,            1418,            2624,            2984,            3198,            0,            3540,            ],        [            1332,            1033,            990,            1931,            371,            2175,            275,            1808,            2941,            3189,            2878,            2929,            3093,            3502,            1728,            1995,            1904,            2220,            3570,            2463,            3477,            1726,            3256,            3149,            3186,            3573,            1813,            6704,            2081,            3234,            3001,            5158,            2575,            2662,            2952,            1960,            2622,            3412,            3252,            3648,            2671,            2558,            3046,            3007,            1592,            3402,            2540,            2807,            4694,            1262,            3590,            2468,            2234,            2169,            2501,            2787,            923,            2948,            4643,            2842,            2788,            3073,            3116,            2788,            2483,            1561,            1898,            2447,            2269,            3204,            5341,            1659,            2601,            2800,            3017,            1966,            3326,            2146,            2360,            2839,            0,            ],        ], 'starts': [16, 67, 9, 71], 'ends': [16, 67, 9, 71]},    'NW-29': {'distance_matrix': [        [            0,            2561,            2546,            2628,            1332,            1671,            1682,            1472,            2163,            1176,            3075,            3303,            2773,            2944,            2640,            2750,            1717,            2894,            753,            3007,            3532,            2645,            501,            1728,            2567,            926,            245,            3635,            3578,            3012,            3715,            3881,            3959,            2937,            3032,            6647,            5220,            3632,            4245,            4268,            3836,            3896,            3117,            2037,            2222,            3486,            2842,            2289,            1183,            2259,            1611,            6304,            1311,            3278,            2721,            3134,            4330,            3734,            2850,            2078,            994,            4949,            3244,            4317,            1835,            5330,            2130,            1105,            638,            3763,            1612,            2665,            4056,            3142,            4271,            1466,            3259,            4468,            4177,            ],        [            3164,            0,            570,            619,            2477,            1546,            956,            1347,            3549,            2637,            2514,            2047,            2212,            2383,            2079,            2189,            1181,            877,            2982,            1004,            1529,            1282,            3665,            3114,            2006,            3155,            3409,            1632,            2769,            2203,            2906,            3072,            3150,            2128,            2223,            5838,            4411,            2823,            3436,            3459,            3027,            3087,            2308,            3423,            614,            2925,            1479,            1833,            3412,            1565,            2445,            5495,            2593,            2717,            2160,            2678,            3521,            2925,            2289,            1055,            2714,            4140,            2435,            3508,            1954,            4521,            1763,            3334,            3802,            2954,            2446,            628,            3247,            2333,            3462,            2927,            2803,            1607,            3368,            ],        [            3344,            265,            0,            332,            1507,            846,            386,            647,            2849,            1937,            2694,            1370,            2392,            2563,            2259,            2369,            1059,            307,            2282,            711,            1236,            712,            1751,            2414,            2186,            2455,            3589,            1301,            2949,            2383,            3086,            3252,            3330,            2308,            2403,            6018,            4591,            3003,            3616,            3639,            3207,            3267,            2488,            2723,            879,            3105,            909,            1263,            2712,            995,            1745,            5675,            1893,            2897,            2340,            2108,            3701,            3105,            2469,            661,            1169,            4320,            2615,            3688,            1254,            4701,            1193,            2634,            1888,            3134,            1746,            369,            3427,            2513,            3642,            2227,            2233,            1669,            3548,            ],        [            3283,            190,            244,            0,            2596,            1090,            630,            891,            3093,            2181,            2633,            1721,            2331,            2502,            2198,            2308,            1111,            551,            2526,            678,            1203,            956,            1995,            2658,            2125,            2699,            3528,            1306,            2888,            2322,            3025,            3191,            3269,            2247,            2342,            5957,            4530,            2942,            3555,            3578,            3146,            3206,            2427,            2967,            804,            3044,            1153,            1507,            2956,            1239,            1989,            5614,            2137,            2836,            2279,            2352,            3640,            3044,            2408,            713,            1413,            4259,            2554,            3627,            1498,            4640,            1437,            2878,            2132,            3073,            1990,            308,            3366,            2452,            3581,            2471,            2477,            1630,            3487,            ],        [            2393,            1443,            1428,            1510,            0,            1451,            1297,            1252,            2561,            1574,            1743,            2932,            1441,            1612,            1308,            1418,            599,            1735,            1151,            1889,            2414,            2140,            1080,            2126,            1235,            1324,            1088,            2517,            2054,            1488,            2191,            2357,            2435,            1413,            1508,            5123,            3696,            2108,            2721,            2744,            2312,            2372,            1593,            2435,            1292,            2154,            2337,            2691,            1581,            2423,            2009,            4780,            1709,            1946,            1389,            3536,            2806,            2210,            1518,            960,            337,            3425,            1720,            2793,            1859,            3806,            1662,            1503,            710,            2239,            2010,            1547,            2532,            1618,            2747,            1864,            3661,            2944,            2653,            ],        [            1346,            1547,            1282,            1614,            1079,            0,            586,            392,            2003,            1091,            2831,            1384,            2529,            2700,            2396,            2506,            1210,            975,            1436,            1993,            2518,            726,            905,            1568,            2323,            1609,            1129,            2233,            3086,            2520,            3223,            3389,            3467,            2445,            2540,            6155,            4728,            3140,            3753,            3776,            3344,            3404,            2625,            1877,            1126,            3242,            923,            1277,            1866,            1009,            899,            5812,            1047,            3034,            2477,            2122,            3838,            3242,            2606,            812,            741,            4457,            2752,            3825,            408,            4838,            211,            1788,            1042,            3271,            900,            1651,            3564,            2650,            3779,            1381,            2247,            2951,            3685,            ],        [            3185,            651,            386,            718,            1121,            460,            0,            261,            2463,            1551,            2535,            1464,            2233,            2404,            2100,            2210,            914,            401,            1896,            1097,            1622,            806,            1365,            2028,            2027,            2069,            1589,            1431,            2790,            2224,            2927,            3093,            3171,            2149,            2244,            5859,            4432,            2844,            3457,            3480,            3048,            3108,            2329,            2337,            830,            2946,            1003,            1357,            2326,            1089,            1359,            5516,            1507,            2738,            2181,            2202,            3542,            2946,            2310,            516,            783,            4161,            2456,            3529,            868,            4542,            671,            2248,            1502,            2975,            1360,            755,            3268,            2354,            3483,            1841,            2327,            2055,            3389,            ],        [            1545,            1163,            774,            1230,            1278,            199,            388,            0,            2202,            1290,            2633,            1583,            2331,            2502,            2198,            2308,            1012,            1174,            1635,            1609,            2134,            925,            1104,            1767,            2125,            1808,            1328,            1819,            2888,            2322,            3025,            3191,            3269,            2247,            2342,            5957,            4530,            2942,            3555,            3578,            3146,            3206,            2427,            2076,            928,            3044,            1122,            1476,            2065,            1208,            1098,            5614,            1246,            2836,            2279,            2321,            3640,            3044,            2408,            614,            940,            4259,            2554,            3627,            607,            4640,            410,            1987,            1241,            3073,            1099,            1143,            3366,            2452,            3581,            1580,            2446,            2567,            3487,            ],        [            2707,            3217,            2980,            3284,            3097,            2032,            2442,            2248,            0,            1647,            4840,            3082,            4538,            4709,            4405,            4515,            3482,            2673,            1992,            3663,            4188,            2424,            2266,            855,            4332,            2165,            2490,            3931,            5801,            5235,            5938,            6104,            6182,            5160,            5255,            8870,            7443,            5855,            6468,            6491,            6059,            6119,            5340,            126,            2982,            5251,            2621,            2068,            2422,            2038,            1098,            8527,            1292,            5043,            4486,            2913,            6553,            5957,            4615,            2668,            2759,            7172,            5467,            6540,            1614,            7553,            1909,            2344,            2403,            5986,            1391,            3349,            6279,            5365,            6494,            1937,            3038,            6691,            6400,            ],        [            1263,            2262,            2025,            2329,            1653,            1077,            1487,            1293,            987,            0,            3396,            2127,            3094,            3265,            2961,            3071,            2038,            1718,            548,            2708,            3233,            1469,            822,            552,            2888,            721,            1046,            2976,            4357,            3791,            4494,            4660,            4738,            3716,            3811,            7426,            5999,            4411,            5024,            5047,            4615,            4675,            3896,            861,            2027,            3807,            1666,            1113,            978,            1083,            435,            7083,            135,            3599,            3042,            1958,            5109,            4513,            3171,            1713,            1315,            5728,            4023,            5096,            659,            6109,            954,            900,            959,            4542,            436,            2394,            4835,            3921,            5050,            290,            2083,            5247,            4956,            ],        [            2732,            2273,            2258,            2340,            1288,            2281,            2127,            2082,            4895,            3908,            0,            3762,            1131,            1467,            418,            528,            1429,            2565,            3485,            2719,            3244,            2970,            3233,            4460,            1570,            3658,            2977,            3347,            2868,            2302,            3005,            3171,            3249,            2227,            2322,            5937,            4510,            2922,            3535,            3558,            3126,            3186,            2407,            4769,            2122,            2113,            3167,            3521,            3915,            3253,            3180,            5594,            4043,            921,            1244,            4366,            3620,            3024,            602,            1790,            1625,            4239,            2534,            3607,            2689,            4620,            2492,            3837,            1998,            3053,            4344,            2377,            3346,            2432,            3561,            4198,            4491,            3758,            3467,            ],        [            3251,            1744,            1370,            1630,            2337,            1613,            1503,            1634,            3103,            2191,            4173,            0,            3871,            4042,            3738,            3848,            2452,            1063,            2536,            1541,            1215,            658,            2810,            2668,            3665,            2709,            3034,            849,            2952,            3862,            4565,            4731,            4809,            3787,            3882,            5600,            6070,            4482,            5095,            5118,            4686,            4746,            3967,            2977,            2468,            4584,            461,            1365,            2966,            1097,            1999,            5257,            2147,            4376,            3819,            2210,            5180,            4584,            3948,            2054,            1999,            5799,            4094,            5167,            1508,            4647,            1295,            2888,            2947,            4613,            2000,            1848,            4906,            3992,            5121,            2481,            2243,            2792,            5027,            ],        [            4079,            3620,            3605,            3687,            2635,            3628,            3474,            3429,            6242,            5255,            2108,            5109,            0,            233,            1361,            1648,            2776,            3912,            4832,            4066,            4591,            4317,            4580,            5807,            1201,            5005,            4324,            4694,            3729,            3163,            3659,            3921,            4040,            2902,            2997,            7284,            5164,            3576,            4326,            4905,            3780,            3977,            3268,            6116,            3469,            1744,            4514,            4868,            5262,            4600,            4527,            6941,            5390,            2377,            875,            5713,            4411,            3815,            1949,            3137,            2972,            4893,            3209,            4468,            4036,            5481,            3839,            5184,            3345,            3803,            5691,            3724,            4096,            3293,            4132,            5545,            5838,            3932,            4258,            ],        [            3846,            3387,            3372,            3454,            2402,            3395,            3241,            3196,            6009,            5022,            1875,            4876,            529,            0,            1128,            1415,            2543,            3679,            4599,            3833,            4358,            4084,            4347,            5574,            968,            4772,            4091,            4461,            3496,            2930,            3426,            3688,            3807,            2669,            2764,            7051,            4931,            3343,            4093,            4672,            3547,            3744,            3035,            5883,            3236,            1511,            4281,            4635,            5029,            4367,            4294,            6708,            5157,            2144,            642,            5480,            4178,            3582,            1716,            2904,            2739,            4660,            2976,            4235,            3803,            5248,            3606,            4951,            3112,            3570,            5458,            3491,            3863,            3060,            3899,            5312,            5605,            3699,            4025,            ],        [            2718,            2259,            2244,            2326,            1274,            2267,            2113,            2068,            4881,            3894,            813,            3748,            713,            1049,            0,            488,            1415,            2551,            3471,            2705,            3230,            2956,            3219,            4446,            1152,            3644,            2963,            3333,            2854,            2288,            2991,            3157,            3235,            2213,            2308,            5923,            4496,            2908,            3521,            3544,            3112,            3172,            2393,            4755,            2108,            1695,            3153,            3507,            3901,            3239,            3166,            5580,            4029,            1016,            826,            4352,            3606,            3010,            588,            1776,            1611,            4225,            2520,            3593,            2675,            4606,            2478,            3823,            1984,            3039,            4330,            2363,            3332,            2418,            3547,            4184,            4477,            3744,            3453,            ],        [            3154,            2695,            2680,            2762,            1710,            2703,            2549,            2504,            5317,            4330,            861,            4184,            1449,            1785,            550,            0,            1851,            2987,            3907,            3141,            3666,            3392,            3655,            4882,            1888,            4080,            3399,            3769,            3290,            2724,            3427,            3593,            3671,            2649,            2744,            6359,            4932,            3344,            3957,            3980,            3548,            3608,            2829,            5191,            2544,            2431,            3589,            3943,            4337,            3675,            3602,            6016,            4465,            920,            1562,            4788,            4042,            3446,            1024,            2212,            2047,            4661,            2956,            4029,            3111,            5042,            2914,            4259,            2420,            3475,            4766,            2799,            3768,            2854,            3983,            4620,            4913,            4180,            3889,            ],        [            3231,            844,            829,            911,            1513,            852,            698,            653,            2855,            1943,            2581,            2333,            2279,            2450,            2146,            2256,            0,            1136,            2288,            1290,            1815,            1541,            1757,            2420,            2073,            2461,            1981,            1918,            2836,            2270,            2973,            3139,            3217,            2195,            2290,            5905,            4478,            2890,            3503,            3526,            3094,            3154,            2375,            2729,            693,            2992,            1738,            2092,            2718,            1824,            1751,            5562,            1899,            2784,            2227,            2937,            3588,            2992,            2356,            361,            1175,            4207,            2502,            3575,            1260,            4588,            1063,            2640,            1894,            3021,            1752,            948,            3314,            2400,            3529,            2233,            3062,            2248,            3435,            ],        [            1966,            572,            307,            639,            1459,            620,            605,            599,            2623,            1711,            3001,            1063,            2699,            2870,            2566,            2676,            1417,            0,            2056,            1018,            1543,            405,            1525,            2188,            2493,            2229,            1749,            1352,            3256,            2690,            3393,            3559,            3637,            2615,            2710,            6325,            4898,            3310,            3923,            3946,            3514,            3574,            2795,            2497,            1186,            3412,            602,            956,            2486,            688,            1519,            5982,            1667,            3204,            2647,            1801,            4008,            3412,            2776,            1019,            1121,            4627,            2922,            3995,            1028,            5008,            886,            2408,            1662,            3441,            1520,            676,            3734,            2820,            3949,            2001,            1926,            1976,            3855,            ],        [            1270,            2685,            2448,            2752,            1660,            1500,            1910,            1716,            1410,            423,            3403,            2550,            3101,            3272,            2968,            3078,            2045,            2141,            0,            3131,            3656,            1892,            829,            975,            2895,            173,            1053,            3399,            4364,            3798,            4501,            4667,            4745,            3723,            3818,            7433,            6006,            4418,            5031,            5054,            4622,            4682,            3903,            1284,            2450,            3814,            2089,            1536,            430,            1506,            858,            7090,            558,            3606,            3049,            2381,            5116,            4520,            3178,            2136,            1322,            5735,            4030,            5103,            1082,            6116,            1377,            352,            966,            4549,            859,            2817,            4842,            3928,            5057,            713,            2506,            5254,            4963,            ],        [            3608,            515,            569,            325,            2921,            1415,            955,            1216,            3418,            2506,            2958,            1683,            2656,            2827,            2523,            2633,            1436,            876,            2851,            0,            1249,            1281,            2320,            2983,            2450,            3024,            3853,            1268,            1647,            2647,            3350,            3516,            3594,            2572,            2667,            6282,            4855,            3267,            3880,            3903,            3471,            3531,            2752,            3292,            1129,            3369,            1129,            1832,            3281,            1564,            2314,            5939,            2462,            3161,            2604,            2677,            3965,            3369,            2733,            1038,            1738,            4584,            2879,            3952,            1823,            4965,            1762,            3203,            2457,            3398,            2315,            633,            3691,            2777,            3906,            2796,            2802,            1487,            3812,            ],        [            3934,            841,            895,            651,            3247,            1741,            1281,            1542,            3744,            2832,            3284,            1211,            2982,            3153,            2849,            2959,            1762,            1202,            3177,            326,            0,            1607,            2646,            3309,            2776,            3350,            4179,            776,            1973,            2973,            3676,            3842,            3920,            2898,            2993,            6608,            5181,            3593,            4206,            4229,            3797,            3857,            3078,            3618,            1455,            3695,            1455,            2158,            3607,            1890,            2640,            6265,            2788,            3487,            2930,            3003,            4291,            3695,            3059,            1364,            2064,            4910,            3205,            4278,            2149,            5291,            2088,            3529,            2783,            3724,            2641,            959,            4017,            3103,            4232,            3122,            3128,            1813,            4138,            ],        [            2593,            977,            712,            1044,            1679,            955,            1170,            976,            2445,            1533,            3406,            658,            3104,            3275,            2971,            3081,            1794,            405,            1878,            1423,            1873,            0,            2152,            2010,            2898,            2051,            2376,            1507,            3661,            3095,            3798,            3964,            4042,            3020,            3115,            6730,            5303,            3715,            4328,            4351,            3919,            3979,            3200,            2319,            1710,            3817,            197,            707,            2308,            439,            1341,            6387,            1489,            3609,            3052,            1552,            4413,            3817,            3181,            1396,            1341,            5032,            3327,            4400,            850,            5413,            637,            2230,            2289,            3846,            1342,            1081,            4139,            3225,            4354,            1823,            1677,            2381,            4260,            ],        [            754,            2060,            2045,            2127,            831,            1170,            1181,            971,            1988,            1001,            2574,            2396,            2272,            2443,            2139,            2249,            1216,            2352,            578,            2506,            3031,            1738,            0,            1553,            2066,            751,            537,            3134,            2885,            2319,            3022,            3188,            3266,            2244,            2339,            5954,            4527,            2939,            3552,            3575,            3143,            3203,            2424,            1862,            1721,            2985,            1935,            2114,            1008,            2084,            1436,            5611,            1136,            2777,            2220,            2959,            3637,            3041,            2349,            1577,            493,            4256,            2551,            3624,            1660,            4637,            1955,            930,            137,            3070,            1437,            2164,            3363,            2449,            3578,            1291,            3084,            3775,            3484,            ],        [            2809,            3319,            3082,            3386,            3199,            2134,            2544,            2350,            435,            1749,            4942,            3184,            4640,            4811,            4507,            4617,            3584,            2775,            2094,            3765,            4290,            2526,            2368,            0,            4434,            2267,            2592,            4033,            5903,            5337,            6040,            6206,            6284,            5262,            5357,            8972,            7545,            5957,            6570,            6593,            6161,            6221,            5442,            309,            3084,            5353,            2723,            2170,            2524,            2140,            1200,            8629,            1527,            5145,            4588,            3015,            6655,            6059,            4717,            2770,            2861,            7274,            5569,            6642,            1716,            7655,            2011,            2446,            2505,            6088,            1493,            3451,            6381,            5467,            6596,            2039,            3140,            6793,            6502,            ],        [            2805,            2346,            2331,            2413,            1361,            2354,            2200,            2155,            4968,            3981,            900,            3835,            590,            761,            465,            575,            1502,            2638,            3558,            2792,            3317,            3043,            3306,            4533,            0,            3731,            3050,            3420,            2941,            2375,            3078,            3244,            3322,            2300,            2395,            6010,            4583,            2995,            3608,            3631,            3199,            3259,            2480,            4842,            2195,            1303,            3240,            3594,            3988,            3326,            3253,            5667,            4116,            1103,            538,            4439,            3693,            3097,            675,            1863,            1698,            4312,            2607,            3680,            2762,            4693,            2565,            3910,            2071,            3126,            4417,            2450,            3419,            2505,            3634,            4271,            4564,            3831,            3540,            ],        [            1233,            2648,            2411,            2715,            1623,            1463,            1873,            1679,            1373,            386,            3366,            2513,            3064,            3235,            2931,            3041,            2008,            2104,            518,            3094,            3619,            1855,            792,            938,            2858,            0,            1016,            3362,            4327,            3761,            4464,            4630,            4708,            3686,            3781,            7396,            5969,            4381,            4994,            5017,            4585,            4645,            3866,            1247,            2413,            3777,            2052,            1499,            257,            1469,            821,            7053,            521,            3569,            3012,            2344,            5079,            4483,            3141,            2099,            1285,            5698,            3993,            5066,            1045,            6079,            1340,            179,            929,            4512,            822,            2780,            4805,            3891,            5020,            676,            2469,            5217,            4926,            ],        [            697,            2316,            2301,            2383,            1087,            1426,            1437,            1227,            1918,            931,            2830,            3058,            2528,            2699,            2395,            2505,            1472,            2649,            508,            2762,            3287,            2400,            256,            1483,            2322,            681,            0,            3390,            3141,            2575,            3278,            3444,            3522,            2500,            2595,            6210,            4783,            3195,            3808,            3831,            3399,            3459,            2680,            1792,            1977,            3241,            2597,            2044,            938,            2014,            1366,            5867,            1066,            3033,            2476,            2889,            3893,            3297,            2605,            1833,            749,            4512,            2807,            3880,            1590,            4893,            1885,            860,            393,            3326,            1367,            2420,            3619,            2705,            3834,            1221,            3014,            4031,            3740,            ],        [            3974,            895,            1025,            781,            3287,            1519,            1059,            1320,            3522,            2610,            3324,            1118,            3022,            3193,            2889,            2999,            1802,            980,            2955,            587,            713,            1252,            2424,            3087,            2816,            3128,            4219,            0,            2103,            3013,            3716,            3882,            3960,            2938,            3033,            6648,            5221,            3633,            4246,            4269,            3837,            3897,            3118,            3396,            1619,            3735,            1147,            1936,            3385,            1668,            2418,            6305,            2566,            3527,            2970,            2781,            4331,            3735,            3099,            1404,            1842,            4950,            3245,            4318,            1927,            4165,            1866,            3307,            2561,            3764,            2419,            999,            4057,            3143,            4272,            2900,            2906,            1943,            4178,            ],        [            3498,            3907,            3892,            3974,            2811,            3915,            3761,            3716,            5661,            4674,            3138,            5396,            2836,            3007,            2703,            2813,            3063,            4199,            4251,            4353,            4878,            4604,            3999,            5226,            2630,            4424,            3743,            4981,            0,            510,            1323,            1489,            1567,            829,            924,            4255,            2390,            1717,            1853,            1876,            1921,            1504,            385,            5535,            3756,            2324,            4801,            5155,            4681,            4887,            5109,            3912,            4809,            3341,            2784,            6000,            1938,            1342,            2913,            3424,            3048,            3034,            1136,            1034,            5333,            2047,            4126,            4603,            4136,            1371,            5110,            4011,            1664,            464,            1879,            4964,            6125,            979,            1785,            ],        [            3146,            3555,            3540,            3622,            2459,            3563,            3409,            3364,            5309,            4322,            2786,            5044,            2484,            2655,            2351,            2461,            2711,            3847,            3899,            4001,            4526,            4252,            3647,            4874,            2278,            4072,            3391,            4629,            566,            0,            971,            1137,            1215,            477,            572,            3903,            2038,            1365,            1501,            1524,            1569,            1152,            105,            5183,            3404,            1972,            4449,            4803,            4329,            4535,            4757,            3560,            4457,            2989,            2432,            5648,            1586,            990,            2561,            3072,            2696,            2682,            784,            1305,            4981,            2318,            3774,            4251,            3784,            1019,            4758,            3659,            1312,            130,            1527,            4612,            5773,            750,            1433,            ],        [            3578,            3987,            3972,            4054,            2891,            3995,            3841,            3796,            5741,            4754,            3218,            5476,            2916,            3087,            2783,            2893,            3143,            4279,            4331,            4433,            4958,            4684,            4079,            5306,            2710,            4504,            3823,            5061,            1564,            998,            0,            262,            381,            861,            659,            3317,            1178,            1640,            667,            938,            1844,            318,            1103,            5615,            3836,            2356,            4881,            5235,            4761,            4967,            5189,            1341,            4889,            3421,            2864,            6080,            752,            156,            2993,            3504,            3128,            2442,            450,            2303,            5413,            2201,            4206,            4683,            4216,            144,            5190,            4091,            437,            1128,            652,            5044,            6205,            1131,            599,            ],        [            3920,            4329,            4314,            4396,            3233,            4337,            4183,            4138,            6083,            5096,            3560,            5818,            3258,            3429,            3125,            3235,            3485,            4621,            4673,            4775,            5300,            5026,            4421,            5648,            3052,            4846,            4165,            5403,            1840,            1274,            342,            0,            657,            1203,            1001,            3593,            1045,            1982,            943,            1214,            2186,            594,            1379,            5957,            4178,            2698,            5223,            5577,            5103,            5309,            5531,            1617,            5231,            3763,            3206,            6422,            745,            432,            3335,            3846,            3470,            2180,            792,            2579,            5755,            2477,            4548,            5025,            4558,            420,            5532,            4433,            175,            1404,            390,            5386,            6547,            1407,            676,            ],        [            3959,            4368,            4353,            4435,            3272,            4376,            4222,            4177,            6122,            5135,            3599,            5857,            3297,            3468,            3164,            3274,            3524,            4660,            4712,            4814,            5339,            5065,            4460,            5687,            3091,            4885,            4204,            5442,            1879,            1313,            381,            547,            0,            1242,            1040,            2843,            797,            2021,            286,            1253,            2225,            241,            1418,            5996,            4217,            2737,            5262,            5616,            5142,            5348,            5570,            960,            5270,            3802,            3245,            6461,            371,            225,            3374,            3885,            3509,            2727,            831,            2618,            5794,            2516,            4587,            5064,            4597,            429,            5571,            4472,            722,            1443,            937,            5425,            6586,            1446,            218,            ],        [            3010,            3419,            3404,            3486,            2323,            3427,            3273,            3228,            5173,            4186,            2650,            4908,            2190,            2361,            2215,            2325,            2575,            3711,            3763,            3865,            4390,            4116,            3511,            4738,            2142,            3936,            3255,            4493,            1150,            584,            1080,            1342,            1461,            0,            418,            4213,            2476,            888,            1747,            1834,            1092,            1398,            689,            5047,            3268,            1495,            4313,            4667,            4193,            4399,            4621,            2421,            4321,            2853,            2138,            5512,            1832,            1236,            2425,            2936,            2560,            2205,            630,            1889,            4845,            2902,            3638,            4115,            3648,            1224,            4622,            3523,            1517,            714,            1968,            4476,            5637,            1353,            1679,            ],        [            2919,            3328,            3313,            3395,            2232,            3336,            3182,            3137,            5082,            4095,            2559,            4817,            2257,            2428,            2124,            2234,            2484,            3620,            3672,            3774,            4299,            4025,            3420,            4647,            2051,            3845,            3164,            4402,            1059,            493,            662,            924,            1043,            202,            0,            4122,            1604,            981,            1329,            1743,            1185,            980,            598,            4956,            3177,            1697,            4222,            4576,            4102,            4308,            4530,            2003,            4230,            2762,            2205,            5421,            1414,            818,            2334,            2845,            2469,            2298,            212,            1798,            4754,            2811,            3547,            4024,            3557,            806,            4531,            3432,            1099,            623,            1135,            4385,            5546,            1793,            1261,            ],        [            3452,            3861,            3846,            3928,            2765,            3869,            3715,            3670,            5615,            4628,            3092,            5350,            2790,            2961,            2657,            2767,            3017,            4153,            4205,            4307,            4832,            4558,            3953,            5180,            2584,            4378,            3697,            4935,            4819,            4253,            4956,            5122,            5200,            4178,            4273,            0,            6461,            4873,            5486,            5509,            5077,            5137,            4358,            5489,            3710,            3503,            4755,            5109,            4635,            4841,            5063,            7545,            4763,            3295,            2738,            5954,            5571,            4975,            2867,            3378,            3002,            6190,            4485,            5558,            5287,            6571,            4080,            4557,            4090,            5004,            5064,            3965,            5297,            4383,            5512,            4918,            6079,            5709,            5418,            ],        [            4494,            4903,            4888,            4970,            3807,            4911,            4757,            4712,            6657,            5670,            4134,            6392,            3832,            4003,            3699,            3809,            4059,            5195,            5247,            5349,            4687,            5600,            4995,            6222,            3626,            5420,            4739,            5977,            2515,            1949,            916,            675,            535,            1777,            1575,            3105,            0,            2556,            498,            1889,            2760,            593,            2054,            6531,            4752,            3272,            5797,            6151,            5677,            5883,            6105,            1222,            5805,            4337,            3780,            6996,            196,            760,            3909,            4420,            4044,            2855,            1366,            3254,            6329,            3152,            5122,            5599,            5132,            964,            6106,            5007,            850,            2079,            653,            5960,            7121,            2082,            317,            ],        [            4990,            5399,            5384,            5466,            4303,            5407,            5253,            5208,            7153,            6166,            4630,            6888,            4328,            4499,            4195,            4305,            4555,            5691,            5743,            5845,            6370,            6096,            5491,            6718,            4122,            5916,            5235,            6473,            2693,            2127,            1352,            1280,            1667,            1836,            1634,            4603,            1588,            0,            1953,            2224,            204,            1604,            2232,            7027,            5248,            3609,            6293,            6647,            6173,            6379,            6601,            2627,            6301,            4833,            4276,            7492,            1569,            1442,            4405,            4916,            4540,            1643,            1551,            3432,            6825,            3487,            5618,            6095,            5628,            1430,            6602,            5503,            964,            2257,            1080,            6456,            7617,            2417,            1500,            ],        [            4245,            4654,            4639,            4721,            3558,            4662,            4508,            4463,            6408,            5421,            3885,            6143,            3583,            3754,            3450,            3560,            3810,            4946,            4998,            5100,            4189,            5351,            4746,            5973,            3377,            5171,            4490,            4602,            2165,            1599,            667,            833,            286,            1528,            1326,            2607,            617,            2307,            0,            1539,            2511,            527,            1704,            6282,            4503,            3023,            5548,            5902,            5428,            5634,            5856,            724,            5556,            4088,            3531,            6747,            306,            511,            3660,            4171,            3795,            3287,            1117,            2904,            6080,            2802,            4873,            5350,            4883,            715,            5857,            4758,            1008,            1729,            704,            5711,            6872,            1732,            346,            ],        [            5831,            6240,            6225,            4518,            5144,            6248,            6094,            6049,            7994,            7007,            5471,            4732,            5169,            5340,            5036,            5146,            5396,            6532,            6584,            4193,            3867,            6937,            6332,            7559,            4963,            6757,            6076,            4280,            3727,            3161,            2478,            2644,            2722,            3179,            3233,            2379,            2472,            4067,            1855,            0,            4271,            2659,            3266,            7868,            6089,            5882,            7134,            7488,            7014,            7220,            7442,            2036,            7142,            5674,            5117,            8333,            2161,            2497,            5246,            5757,            5381,            4824,            2928,            4466,            7666,            4364,            6459,            6936,            6469,            2526,            7443,            4826,            2819,            3291,            3034,            7297,            6075,            3294,            2151,            ],        [            4786,            5195,            5180,            5262,            4099,            5203,            5049,            5004,            6949,            5962,            4426,            6684,            4124,            4295,            3991,            4101,            4351,            5487,            5539,            5641,            6166,            5892,            5287,            6514,            3918,            5712,            5031,            6269,            2489,            1923,            1148,            1076,            1463,            1632,            1430,            4399,            1384,            1878,            1749,            2020,            0,            1400,            2028,            6823,            5044,            3405,            6089,            6443,            5969,            6175,            6397,            2423,            6097,            4629,            4072,            7288,            1365,            1238,            4201,            4712,            4336,            1439,            1347,            3228,            6621,            3283,            5414,            5891,            5424,            1226,            6398,            5299,            760,            2053,            876,            6252,            7413,            2213,            1296,            ],        [            3896,            4305,            4290,            4372,            3209,            4313,            4159,            4114,            6059,            5072,            3536,            5794,            3234,            3405,            3101,            3211,            3461,            4597,            4649,            4751,            5276,            5002,            4397,            5624,            3028,            4822,            4141,            5379,            1816,            1250,            318,            484,            241,            1179,            977,            3569,            1038,            1958,            527,            1190,            2162,            0,            1355,            5933,            4154,            2674,            5199,            5553,            5079,            5285,            5507,            1201,            5207,            3739,            3182,            6398,            612,            162,            3311,            3822,            3446,            2664,            768,            2555,            5731,            2453,            4524,            5001,            4534,            366,            5508,            4409,            659,            1380,            874,            5362,            6523,            1383,            459,            ],        [            3151,            3560,            3545,            3627,            2464,            3568,            3414,            3369,            5314,            4327,            2791,            5049,            2489,            2660,            2356,            2466,            2716,            3852,            3904,            4006,            4531,            4257,            3652,            4879,            2283,            4077,            3396,            4634,            587,            163,            976,            1142,            1220,            482,            577,            3908,            2043,            1370,            1506,            1529,            1574,            1157,            0,            5188,            3409,            1977,            4454,            4808,            4334,            4540,            4762,            3565,            4462,            2994,            2437,            5653,            1591,            995,            2566,            3077,            2701,            2687,            789,            1326,            4986,            2339,            3779,            4256,            3789,            1024,            4763,            3664,            1317,            79,            1532,            4617,            5778,            1026,            1438,            ],        [            2581,            3091,            2854,            3158,            2971,            1906,            2316,            2122,            126,            1521,            4714,            2956,            4412,            4583,            4279,            4389,            3356,            2547,            1866,            3537,            4062,            2298,            2140,            729,            4206,            2039,            2364,            3805,            5675,            5109,            5812,            5978,            6056,            5034,            5129,            8744,            7317,            5729,            6342,            6365,            5933,            5993,            5214,            0,            2856,            5125,            2495,            1942,            2296,            1912,            972,            8401,            1218,            4917,            4360,            2787,            6427,            5831,            4489,            2542,            2633,            7046,            5341,            6414,            1488,            7427,            1783,            2218,            2277,            5860,            1265,            3223,            6153,            5239,            6368,            1811,            2912,            6565,            6274,            ],        [            2774,            847,            866,            921,            2087,            990,            836,            791,            2993,            2081,            2124,            2374,            1822,            1993,            1689,            1799,            625,            1237,            2426,            1300,            1825,            1716,            3275,            2558,            1616,            2599,            3019,            1928,            2379,            1813,            2516,            2682,            2760,            1738,            1833,            5448,            4021,            2433,            3046,            3069,            2637,            2697,            1918,            2867,            0,            2535,            1913,            2267,            2856,            1999,            1889,            5105,            2037,            2327,            1770,            3112,            3131,            2535,            1899,            499,            2324,            3750,            2045,            3118,            1398,            4131,            1201,            2778,            3412,            2564,            1890,            924,            2857,            1943,            3072,            2371,            3237,            3269,            2978,            ],        [            3845,            4254,            4239,            4321,            3158,            4262,            4108,            4063,            6008,            5021,            3485,            5743,            1519,            1690,            3050,            3160,            3410,            4546,            4598,            4700,            5225,            4951,            4346,            5573,            1639,            4771,            4090,            5328,            1985,            1419,            1915,            2177,            2296,            1158,            1253,            5048,            3420,            1832,            2582,            2669,            2036,            2233,            1524,            5882,            4103,            0,            5148,            5502,            5028,            5234,            5456,            3256,            5156,            3688,            1467,            6347,            2667,            2071,            3260,            3771,            3395,            3149,            1465,            2724,            5680,            3737,            4473,            4950,            4483,            2059,            5457,            4358,            2352,            1549,            2388,            5311,            6472,            2188,            2514,            ],        [            2790,            1174,            909,            1010,            1876,            1152,            1120,            1173,            2642,            1730,            3603,            461,            3301,            3472,            3168,            3278,            1991,            602,            2075,            1214,            1676,            197,            2349,            2207,            3095,            2248,            2573,            1310,            2332,            3292,            3995,            4161,            4239,            3217,            3312,            6927,            5500,            3912,            4525,            4548,            4116,            4176,            3397,            2516,            1907,            4014,            0,            904,            2505,            636,            1538,            6584,            1686,            3806,            3249,            1749,            4610,            4014,            3378,            1593,            1538,            5229,            3524,            4597,            1047,            5610,            834,            2427,            2486,            4043,            1539,            1278,            4336,            3422,            4551,            2020,            1874,            2172,            4457,            ],        [            2520,            1528,            1263,            1595,            2910,            1506,            1491,            1485,            2372,            1460,            3957,            1365,            3655,            3826,            3522,            3632,            2303,            956,            1805,            1974,            2499,            707,            2079,            1937,            3449,            1978,            2303,            2308,            4212,            3646,            4349,            4515,            4593,            3571,            3666,            7281,            5854,            4266,            4879,            4902,            4470,            4530,            3751,            2246,            2142,            4368,            904,            0,            2235,            268,            1268,            6938,            1416,            4160,            3603,            845,            4964,            4368,            3732,            1905,            2572,            5583,            3878,            4951,            959,            5964,            1188,            2157,            2216,            4397,            1269,            1632,            4690,            3776,            4905,            1750,            970,            2932,            4811,            ],        [            1490,            2905,            2668,            2972,            1880,            1720,            2130,            1936,            1630,            643,            3623,            2770,            3321,            3492,            3188,            3298,            2265,            2361,            775,            3351,            3876,            2112,            1049,            1195,            3115,            257,            1273,            3619,            4584,            4018,            4721,            4887,            4965,            3943,            4038,            7653,            6226,            4638,            5251,            5274,            4842,            4902,            4123,            1504,            2670,            4034,            2309,            1756,            0,            1726,            1078,            7310,            778,            3826,            3269,            2601,            5336,            4740,            3398,            2356,            1542,            5955,            4250,            5323,            1302,            6336,            1597,            78,            1186,            4769,            1079,            3037,            5062,            4148,            5277,            933,            2726,            5474,            5183,            ],        [            2252,            1260,            995,            1327,            2642,            1238,            1223,            1217,            2104,            1192,            3689,            1097,            3387,            3558,            3254,            3364,            2035,            688,            1537,            1706,            2231,            439,            1811,            1669,            3181,            1710,            2035,            2040,            3944,            3378,            4081,            4247,            4325,            3303,            3398,            7013,            5586,            3998,            4611,            4634,            4202,            4262,            3483,            1978,            1874,            4100,            636,            268,            1967,            0,            1000,            6670,            1148,            3892,            3335,            1113,            4696,            4100,            3464,            1637,            2304,            5315,            3610,            4683,            691,            5696,            920,            1889,            1948,            4129,            1001,            1364,            4422,            3508,            4637,            1482,            1238,            2664,            4543,            ],        [            1609,            2119,            1882,            2186,            1999,            934,            1344,            1150,            1104,            549,            3742,            1984,            3440,            3611,            3307,            3417,            2384,            1575,            894,            2565,            3090,            1326,            1168,            669,            3234,            1067,            1392,            2833,            4703,            4137,            4840,            5006,            5084,            4062,            4157,            7772,            6345,            4757,            5370,            5393,            4961,            5021,            4242,            978,            1884,            4153,            1523,            970,            1324,            940,            0,            7429,            505,            3945,            3388,            1815,            5455,            4859,            3517,            1570,            1661,            6074,            4369,            5442,            516,            6455,            811,            1246,            1305,            4888,            293,            2251,            5181,            4267,            5396,            839,            1940,            5593,            5302,            ],        [            3836,            4245,            4230,            4312,            3149,            4253,            4099,            4054,            5999,            5012,            3476,            5734,            3174,            3345,            3041,            3151,            3401,            4537,            4589,            4691,            5216,            4942,            4337,            5564,            2968,            4762,            4081,            5319,            1934,            1368,            442,            608,            686,            1167,            1197,            4201,            1509,            2082,            972,            1822,            2286,            623,            1473,            5873,            4094,            2662,            5139,            5493,            5019,            5225,            5447,            0,            5147,            3679,            3122,            6338,            1057,            461,            3251,            3762,            3386,            2788,            892,            2673,            5671,            3085,            4464,            4941,            4474,            490,            5448,            4349,            783,            1498,            998,            5302,            6463,            2015,            904,            ],        [            1617,            2127,            1890,            2194,            2007,            942,            1352,            1158,            1404,            557,            3597,            1992,            3295,            3466,            3162,            3272,            2392,            1583,            902,            2573,            3098,            1334,            1176,            969,            3089,            1075,            1400,            2841,            4711,            4145,            4848,            5014,            5092,            4070,            4165,            7780,            6353,            4765,            5378,            5401,            4969,            5029,            4250,            1278,            1892,            4008,            1531,            978,            1332,            948,            300,            7437,            0,            3800,            3243,            1823,            5463,            4867,            3372,            1578,            1669,            6082,            4377,            5450,            524,            6463,            819,            1254,            1313,            4896,            301,            2259,            5189,            4275,            5404,            847,            1948,            3531,            5310,            ],        [            3686,            3227,            3212,            3294,            2242,            3235,            3081,            3036,            5849,            4862,            954,            4716,            2199,            2535,            1372,            1269,            2383,            3519,            4439,            3673,            4198,            3924,            4187,            5414,            2638,            4612,            3931,            4301,            3822,            3256,            3959,            4125,            4203,            3181,            3276,            6891,            5464,            3876,            4489,            4512,            4080,            4140,            3361,            5723,            3076,            3181,            4121,            4475,            4869,            4207,            4134,            6548,            4997,            0,            2312,            5320,            4574,            3978,            1556,            2744,            2579,            5193,            3488,            4561,            3643,            5574,            3446,            4791,            2952,            4007,            5298,            3331,            4300,            3386,            4515,            5152,            5445,            4712,            4421,            ],        [            4069,            3610,            3595,            3677,            2625,            3618,            3464,            3419,            6232,            5245,            2098,            5099,            52,            223,            1351,            1638,            2766,            3902,            4822,            4056,            4581,            4307,            4570,            5797,            1191,            4995,            4314,            4684,            3719,            3153,            3649,            3911,            4030,            2892,            2987,            7274,            5154,            3566,            4316,            4895,            3770,            3967,            3258,            6106,            3459,            1734,            4504,            4858,            5252,            4590,            4517,            6931,            5380,            2367,            0,            5703,            4401,            3805,            1939,            3127,            2962,            4883,            3199,            4458,            4026,            5471,            3829,            5174,            3335,            3793,            5681,            3714,            4086,            3283,            4122,            5535,            5828,            3922,            4248,            ],        [            3512,            2520,            2255,            2587,            3902,            2498,            2483,            2477,            3364,            2452,            4949,            1668,            4647,            4818,            4514,            4624,            3295,            1948,            2797,            2966,            3276,            1699,            3071,            2929,            4441,            2970,            3295,            2910,            5204,            4638,            5341,            5507,            5585,            4563,            4658,            6651,            6846,            5258,            5871,            5894,            5462,            5522,            4743,            3238,            3134,            5360,            1896,            992,            3227,            1260,            2260,            6308,            2408,            5152,            4595,            0,            5956,            5360,            4724,            2897,            3564,            6575,            4870,            5943,            1951,            5698,            2180,            3149,            3208,            5389,            2261,            2624,            5682,            4768,            5897,            2742,            495,            3924,            5803,            ],        [            4330,            4739,            4724,            4806,            3643,            4747,            4593,            4548,            6493,            5506,            3970,            6228,            3668,            3839,            3535,            3645,            3895,            5031,            5083,            5185,            4491,            5436,            4831,            6058,            3462,            5256,            4575,            5813,            2351,            1785,            752,            511,            371,            1613,            1411,            2909,            373,            2392,            302,            1725,            2596,            429,            1890,            6367,            4588,            3108,            5633,            5987,            5513,            5719,            5941,            1026,            5641,            4173,            3616,            6832,            0,            596,            3745,            4256,            3880,            3247,            1202,            3090,            6165,            2988,            4958,            5435,            4968,            800,            5942,            4843,            686,            1915,            489,            5796,            6957,            1918,            153,            ],        [            3734,            4143,            4128,            4210,            3047,            4151,            3997,            3952,            5897,            4910,            3374,            5632,            3072,            3243,            2939,            3049,            3299,            4435,            4487,            4589,            5114,            4840,            4235,            5462,            2866,            4660,            3979,            5217,            1654,            1088,            156,            322,            225,            1017,            815,            3407,            1022,            1796,            511,            1028,            2000,            162,            1193,            5771,            3992,            2512,            5037,            5391,            4917,            5123,            5345,            1185,            5045,            3577,            3020,            6236,            596,            0,            3149,            3660,            3284,            2502,            606,            2393,            5569,            2291,            4362,            4839,            4372,            204,            5346,            4247,            497,            1218,            712,            5200,            6361,            1221,            443,            ],        [            2130,            1671,            1656,            1738,            686,            1679,            1525,            1480,            4293,            3306,            1955,            3160,            1653,            1824,            1520,            1630,            827,            1963,            2883,            2117,            2642,            2368,            2631,            3858,            1447,            3056,            2375,            2745,            2266,            1700,            2403,            2569,            2647,            1625,            1720,            5335,            3908,            2320,            2933,            2956,            2524,            2584,            1805,            4167,            1520,            2366,            2565,            2919,            3313,            2651,            2578,            4992,            3441,            2158,            1601,            3764,            3018,            2422,            0,            1188,            1023,            3637,            1932,            3005,            2087,            4018,            1890,            3235,            1396,            2451,            3742,            1775,            2744,            1830,            2959,            3596,            3889,            3156,            2865,            ],        [            1837,            713,            485,            780,            1152,            491,            337,            292,            2494,            1582,            2343,            1875,            2041,            2212,            1908,            2018,            722,            738,            1927,            1159,            1684,            1217,            1396,            2059,            1835,            2100,            1620,            1768,            2598,            2032,            2735,            2901,            2979,            1957,            2052,            5667,            4240,            2652,            3265,            3288,            2856,            2916,            2137,            2368,            638,            2754,            1414,            1768,            2357,            1500,            1390,            5324,            1538,            2546,            1989,            2613,            3350,            2754,            2118,            0,            814,            3969,            2264,            3337,            899,            4350,            702,            2279,            1533,            2783,            1391,            854,            3076,            2162,            3291,            1872,            2738,            2117,            3197,            ],        [            1625,            1781,            1766,            1848,            338,            1789,            1635,            1590,            2881,            1894,            2081,            3270,            1779,            1950,            1646,            1756,            937,            2073,            1471,            2227,            2752,            2478,            1400,            2446,            1573,            1644,            1408,            2855,            2392,            1826,            2529,            2695,            2773,            1751,            1846,            5461,            4034,            2446,            3059,            3082,            2650,            2710,            1931,            2755,            1630,            2492,            2675,            3029,            1901,            2761,            2329,            5118,            2029,            2284,            1727,            3874,            3144,            2548,            1856,            1298,            0,            3763,            2058,            3131,            2197,            4144,            2000,            1823,            1030,            2577,            2330,            1885,            2870,            1956,            3085,            2184,            3999,            3282,            2991,            ],        [            3347,            3756,            3741,            3823,            2660,            3764,            3610,            3565,            5510,            4523,            2987,            5245,            2685,            2856,            2552,            2662,            2912,            4048,            4100,            4202,            4727,            4453,            3848,            5075,            2479,            4273,            3592,            4830,            1621,            1055,            1551,            1719,            1932,            794,            889,            4684,            2027,            439,            2218,            2305,            643,            1869,            1160,            5384,            3605,            1966,            4650,            5004,            4530,            4736,            4958,            2892,            4658,            3190,            2633,            5849,            2303,            1707,            2762,            3273,            2897,            0,            1101,            2360,            5182,            3373,            3975,            4452,            3985,            1695,            4959,            3860,            1403,            1185,            1519,            4813,            5974,            1824,            2150,            ],        [            3128,            3537,            3522,            3604,            2441,            3545,            3391,            3346,            5291,            4304,            2768,            5026,            2466,            2637,            2333,            2443,            2693,            3829,            3881,            3983,            4508,            4234,            3629,            4856,            2260,            4054,            3373,            4611,            1268,            702,            450,            712,            831,            411,            209,            3767,            1628,            1190,            1117,            1388,            1394,            768,            807,            5165,            3386,            1906,            4431,            4785,            4311,            4517,            4739,            1791,            4439,            2971,            2414,            5630,            1202,            606,            2543,            3054,            2678,            2507,            0,            2007,            4963,            3020,            3756,            4233,            3766,            594,            4740,            3641,            887,            832,            1102,            4594,            5755,            1581,            1049,            ],        [            4778,            2708,            2762,            2518,            4091,            5195,            3148,            4996,            6941,            5954,            4418,            4048,            4116,            4287,            3983,            4093,            4343,            3069,            5531,            3509,            3183,            3474,            5279,            6506,            3910,            5704,            5023,            3596,            1280,            1790,            1606,            1772,            1850,            2109,            2265,            3505,            2673,            2997,            2136,            2166,            3201,            1787,            1665,            6815,            3213,            3604,            3322,            4025,            5961,            3757,            6389,            3162,            6089,            4621,            4064,            5118,            2221,            1625,            4193,            4704,            4328,            3952,            2056,            0,            6613,            1013,            5406,            5883,            5416,            1654,            6390,            2826,            1947,            1744,            2162,            6244,            5391,            1120,            2068,            ],        [            1913,            1603,            1366,            1670,            1337,            418,            828,            634,            1765,            853,            3073,            1468,            2771,            2942,            2638,            2748,            1452,            1059,            1198,            2049,            2574,            810,            1472,            1330,            2565,            1371,            1696,            2317,            3328,            2762,            3465,            3631,            3709,            2687,            2782,            6397,            4970,            3382,            3995,            4018,            3586,            3646,            2867,            1639,            1368,            3484,            1007,            1102,            1628,            1093,            661,            6054,            809,            3276,            2719,            1947,            4080,            3484,            2848,            1054,            999,            4699,            2994,            4067,            0,            5080,            295,            1550,            1609,            3513,            662,            1735,            3806,            2892,            4021,            1143,            2072,            3007,            3927,            ],        [            3765,            1695,            1749,            1505,            3078,            4182,            2135,            3983,            5928,            4941,            3405,            2863,            3103,            3274,            2970,            3080,            3330,            2056,            4518,            1729,            2254,            2461,            4266,            5493,            2897,            4691,            4010,            2448,            267,            777,            1590,            1756,            1834,            1096,            1191,            4522,            2657,            1984,            2120,            2143,            2188,            1771,            652,            5802,            2200,            2591,            2309,            3012,            4948,            2744,            5376,            4179,            5076,            3608,            3051,            3857,            2205,            1609,            3180,            3691,            3315,            3301,            1403,            929,            5600,            0,            4393,            4870,            4403,            1638,            5377,            1813,            1931,            731,            2146,            5231,            3982,            107,            2052,            ],        [            1309,            1308,            919,            1375,            1042,            123,            533,            339,            1966,            1054,            2778,            1347,            2476,            2647,            2343,            2453,            1157,            934,            714,            1754,            2279,            689,            868,            1531,            2270,            887,            1092,            1964,            3033,            2467,            3170,            3336,            3414,            2392,            2487,            6102,            4675,            3087,            3700,            3723,            3291,            3351,            2572,            1840,            1073,            3189,            886,            1240,            1144,            972,            862,            5759,            1010,            2981,            2424,            2085,            3785,            3189,            2553,            759,            704,            4404,            2699,            3772,            371,            4785,            0,            1066,            1005,            3218,            863,            1288,            3511,            2597,            3726,            1344,            2210,            2712,            3632,            ],        [            1412,            2827,            2590,            2894,            1802,            1642,            2052,            1858,            1552,            565,            3545,            2692,            3243,            3414,            3110,            3220,            2187,            2283,            697,            3273,            3798,            2034,            971,            1117,            3037,            179,            1195,            3541,            4506,            3940,            4643,            4809,            4887,            3865,            3960,            7575,            6148,            4560,            5173,            5196,            4764,            4824,            4045,            1426,            2592,            3956,            2231,            1678,            78,            1648,            1000,            7232,            700,            3748,            3191,            2523,            5258,            4662,            3320,            2278,            1464,            5877,            4172,            5245,            1224,            6258,            1519,            0,            1108,            4691,            1001,            2959,            4984,            4070,            5199,            855,            2648,            5396,            5105,            ],        [            759,            1923,            1908,            1990,            694,            1033,            1044,            834,            2240,            1253,            2437,            2259,            2135,            2306,            2002,            2112,            1079,            2215,            830,            2369,            2894,            1601,            759,            1805,            1929,            1003,            542,            2997,            2748,            2182,            2885,            3051,            3129,            2107,            2202,            5817,            4390,            2802,            3415,            3438,            3006,            3066,            2287,            2114,            1584,            2848,            1798,            2366,            1260,            1884,            1688,            5474,            1388,            2640,            2083,            3211,            3500,            2904,            2212,            1440,            356,            4119,            2414,            3487,            1912,            4500,            1086,            1182,            0,            2933,            1689,            2027,            3226,            2312,            3441,            1543,            3336,            3638,            3347,            ],        [            4038,            4447,            4432,            4514,            3351,            4455,            4301,            4256,            6201,            5214,            3678,            5936,            3376,            3547,            3243,            3353,            3603,            4739,            4791,            4893,            5418,            5144,            4539,            5766,            3170,            4964,            4283,            5521,            1958,            1392,            460,            118,            775,            1321,            1119,            3711,            1019,            2100,            1061,            1332,            2304,            712,            1497,            6075,            4296,            2816,            5341,            5695,            5221,            5427,            5649,            1735,            5349,            3881,            3324,            6540,            593,            550,            3453,            3964,            3588,            2298,            910,            2697,            5873,            2595,            4666,            5143,            4676,            0,            5650,            4551,            293,            1522,            508,            5504,            6665,            1525,            524,            ],        [            1784,            1826,            1589,            1893,            2174,            641,            1051,            857,            1636,            724,            3296,            1691,            2994,            3165,            2861,            2971,            1675,            1282,            1069,            2272,            2797,            1033,            1343,            1201,            2788,            1242,            1567,            2540,            3551,            2985,            3688,            3854,            3932,            2910,            3005,            6620,            5193,            3605,            4218,            4241,            3809,            3869,            3090,            1510,            1591,            3707,            1230,            973,            1499,            943,            532,            6277,            680,            3499,            2942,            1818,            4303,            3707,            3071,            1277,            1836,            4922,            3217,            4290,            223,            5303,            518,            1421,            1480,            3736,            0,            1958,            4029,            3115,            4244,            1014,            1943,            3230,            4150,            ],        [            2975,            612,            597,            679,            2288,            896,            983,            697,            2899,            1987,            2325,            2101,            2023,            2194,            1890,            2000,            803,            904,            2332,            1058,            1583,            1309,            3476,            2464,            1817,            2505,            3220,            1686,            2580,            2014,            2717,            2883,            2961,            1939,            2034,            5649,            4222,            2634,            3247,            3270,            2838,            2898,            2119,            2773,            620,            2736,            1506,            1860,            2762,            1592,            1795,            5306,            1943,            2528,            1971,            2705,            3332,            2736,            2100,            405,            2525,            3951,            2246,            3319,            1304,            4332,            1107,            2684,            3613,            2765,            1796,            0,            3058,            2144,            3273,            2277,            2830,            2016,            3179,            ],        [            3900,            4309,            4294,            4376,            3213,            4317,            4163,            4118,            6063,            5076,            3540,            5798,            3238,            3409,            3105,            3215,            3465,            4601,            4653,            4755,            5280,            5006,            4401,            5628,            3032,            4826,            4145,            5383,            1961,            1395,            463,            391,            778,            1183,            981,            3714,            870,            1962,            915,            1335,            1828,            715,            1500,            5937,            4158,            2678,            5203,            5557,            5083,            5289,            5511,            1738,            5211,            3743,            3186,            6402,            704,            553,            3315,            3826,            3450,            2005,            898,            2700,            5735,            2598,            4528,            5005,            4538,            541,            5512,            4413,            0,            1525,            215,            5366,            6527,            1528,            635,            ],        [            3120,            3529,            3514,            3596,            2433,            3537,            3383,            3338,            5283,            4296,            2760,            5018,            2458,            2629,            2325,            2435,            2685,            3821,            3873,            3975,            4500,            4226,            3621,            4848,            2252,            4046,            3365,            4603,            698,            132,            945,            1111,            1189,            451,            546,            3877,            2012,            1339,            1475,            1498,            1543,            1126,            79,            5157,            3378,            1946,            4423,            4777,            4303,            4509,            4731,            3534,            4431,            2963,            2406,            5622,            1560,            964,            2535,            3046,            2670,            2656,            758,            1437,            4955,            2450,            3748,            4225,            3758,            993,            4732,            3633,            1286,            0,            1501,            4586,            5747,            995,            1407,            ],        [            4049,            4458,            4443,            4525,            3362,            4466,            4312,            4267,            6212,            5225,            3689,            5947,            3387,            3558,            3254,            3364,            3614,            4750,            4802,            4904,            5429,            5155,            4550,            5777,            3181,            4975,            4294,            5532,            2152,            1586,            654,            322,            638,            1332,            1130,            3905,            915,            2111,            700,            1526,            1977,            406,            1691,            6086,            4307,            2827,            5352,            5706,            5232,            5438,            5660,            1440,            5360,            3892,            3335,            6551,            489,            744,            3464,            3975,            3599,            2154,            1047,            2891,            5884,            2789,            4677,            5154,            4687,            732,            5661,            4562,            266,            1716,            0,            5515,            6676,            1719,            420,            ],        [            2074,            2584,            2347,            2651,            2464,            1399,            1809,            1615,            697,            1014,            4207,            2449,            3905,            4076,            3772,            3882,            2849,            2040,            1359,            3030,            3555,            1791,            1633,            262,            3699,            1532,            1857,            3298,            5168,            4602,            5305,            5471,            5549,            4527,            4622,            8237,            6810,            5222,            5835,            5858,            5426,            5486,            4707,            571,            2349,            4618,            1988,            1435,            1789,            1405,            465,            7894,            970,            4410,            3853,            2280,            5920,            5324,            3982,            2035,            2126,            6539,            4834,            5907,            981,            6920,            1276,            1711,            1770,            5353,            758,            2716,            5646,            4732,            5861,            0,            2405,            6058,            5767,            ],        [            3017,            2025,            1760,            2092,            3407,            2003,            1988,            1982,            2869,            1957,            4454,            1173,            4152,            4323,            4019,            4129,            2800,            1453,            2302,            2471,            2996,            1204,            2576,            2434,            3946,            2475,            2800,            2022,            4709,            4143,            4846,            5012,            5090,            4068,            4163,            7778,            6351,            4763,            5376,            5399,            4967,            5027,            4248,            2743,            2639,            4865,            1401,            497,            2732,            765,            1765,            7435,            1913,            4657,            4100,            515,            5461,            4865,            4229,            2402,            3069,            6080,            4375,            5448,            1456,            6461,            1685,            2654,            2713,            4894,            1766,            2129,            5187,            4273,            5402,            2247,            0,            3429,            5308,            ],        [            3658,            4067,            4052,            4134,            2971,            4075,            3921,            3876,            5821,            4834,            3298,            5556,            2996,            3167,            2863,            2973,            3223,            4359,            4411,            4513,            5038,            4764,            4159,            5386,            2790,            4584,            3903,            5141,            160,            670,            1483,            1649,            1727,            989,            1084,            4415,            2550,            1877,            2013,            2036,            2081,            1664,            545,            5695,            3916,            2484,            4961,            5315,            4841,            5047,            5269,            4072,            4969,            3501,            2944,            6160,            2098,            1502,            3073,            3584,            3208,            3194,            1296,            822,            5493,            1835,            4286,            4763,            4296,            1531,            5270,            4171,            1824,            624,            2039,            5124,            6285,            0,            1945,            ],        [            4177,            4586,            4571,            4653,            3490,            4594,            4440,            4395,            6340,            5353,            3817,            6075,            3515,            3686,            3382,            3492,            3742,            4878,            4930,            5032,            4485,            5283,            4678,            5905,            3309,            5103,            4422,            5660,            2097,            1531,            599,            442,            218,            1460,            1258,            2903,            579,            2239,            364,            1471,            2443,            459,            1636,            6214,            4435,            2955,            5480,            5834,            5360,            5566,            5788,            1020,            5488,            4020,            3463,            6679,            153,            443,            3592,            4103,            3727,            2622,            1049,            2836,            6012,            2734,            4805,            5282,            4815,            647,            5789,            4690,            617,            1661,            420,            5643,            6804,            1664,            0,            ],        ], 'starts': [4, 78, 56, 69], 'ends': [4, 78, 56, 69]},    'NW-30': {'distance_matrix': [        [            0,            1610,            2351,            1582,            1770,            1351,            1318,            1420,            3331,            1969,            2612,            797,            1385,            3652,            2720,            3308,            664,            766,            1189,            2781,            1260,            3382,            2262,            3517,            1993,            2302,            3480,            2538,            1037,            780,            2110,            3742,            1938,            3566,            2549,            1478,            1417,            1568,            2145,            3097,            1547,            1981,            3934,            565,            974,            2214,            1655,            1822,            600,            2898,            2937,            2401,            2858,            3147,            2618,            4174,            2035,            2915,            1263,            4187,            2828,            1470,            2982,            2624,            2517,            2323,            4549,            3049,            1009,            1946,            3744,            3410,            1405,            1676,            1544,            449,            4419,            1837,            2337,            2471,            3112,            3894,            808,            970,            3401,            3948,            2115,            1726,            3450,            2503,            1278,            3136,            ],        [            1377,            0,            2568,            2450,            400,            2918,            1535,            251,            3548,            2837,            2829,            1014,            1602,            3869,            2937,            3525,            1600,            681,            301,            3649,            2299,            4250,            3130,            3734,            2497,            2519,            4348,            2755,            643,            997,            2327,            4115,            2155,            3783,            2766,            1695,            2435,            198,            775,            3314,            2415,            611,            4151,            2023,            1191,            2431,            1872,            2861,            1047,            3115,            3154,            2618,            3075,            3364,            2835,            4391,            2252,            3132,            2131,            4404,            3045,            1687,            3199,            2841,            2734,            2540,            4766,            3266,            494,            2163,            3961,            4278,            1622,            2544,            2341,            1826,            4636,            2705,            2554,            2688,            3329,            4762,            878,            1187,            3618,            4165,            2332,            2765,            3667,            2720,            2296,            4004,            ],        [            2421,            2726,            0,            1950,            2886,            1224,            1326,            2536,            980,            2337,            259,            1913,            1555,            1514,            1705,            1419,            2247,            2477,            3027,            3149,            1756,            3368,            2630,            1263,            1324,            698,            3466,            1523,            2153,            1896,            1154,            2883,            932,            2336,            198,            1172,            1262,            2684,            3261,            975,            1981,            3097,            1752,            2148,            2090,            924,            1133,            1439,            2843,            547,            3073,            352,            507,            1991,            1264,            1358,            1937,            2216,            1908,            1371,            874,            2532,            3349,            2526,            2631,            1567,            1665,            2756,            2310,            858,            1353,            3165,            1276,            2110,            1168,            2493,            1603,            2271,            1381,            704,            1980,            2905,            3044,            1859,            1050,            1826,            706,            1597,            1328,            2405,            1123,            3674,            ],        [            2012,            2960,            1719,            0,            3125,            900,            1565,            2775,            2699,            612,            1980,            2152,            2503,            2980,            1474,            2636,            1462,            2499,            2659,            1424,            806,            2025,            905,            2845,            634,            1670,            2123,            1292,            2392,            2135,            922,            2385,            1019,            2872,            1917,            1411,            980,            2923,            3500,            1781,            1031,            3336,            2688,            1722,            2329,            1319,            2081,            463,            2490,            2266,            4280,            1769,            2226,            3710,            2618,            3077,            3378,            3935,            958,            3090,            2228,            2813,            4325,            3967,            4350,            2921,            3384,            4475,            2549,            1051,            3072,            2053,            1652,            1160,            781,            2067,            3322,            1321,            1091,            1484,            3699,            2537,            2247,            2098,            2769,            3276,            1791,            367,            2778,            3846,            779,            1779,            ],        [            1467,            400,            2708,            2850,            0,            2239,            1675,            391,            3688,            3237,            2969,            1154,            1742,            4009,            3077,            3665,            2000,            821,            701,            4049,            2699,            4650,            3530,            3874,            2637,            2659,            4748,            2895,            783,            1137,            2467,            4255,            2295,            3923,            2906,            1835,            2575,            202,            935,            3454,            2815,            211,            4291,            2032,            1331,            2571,            2012,            2808,            1187,            3255,            3294,            2758,            3215,            3504,            2975,            4531,            2392,            3272,            2531,            4544,            3185,            1827,            3339,            2981,            2874,            2680,            4906,            3406,            634,            2303,            4101,            4537,            1762,            2944,            2481,            1916,            4776,            3105,            2694,            2828,            3469,            4277,            1278,            1327,            3758,            4305,            2472,            3165,            3807,            2860,            2436,            4404,            ],        [            1760,            2065,            819,            1024,            2225,            0,            665,            1875,            1799,            1411,            1080,            1252,            1603,            2120,            1188,            1776,            1586,            1816,            2366,            2223,            830,            2851,            1704,            1985,            748,            770,            2949,            1006,            1492,            1235,            578,            2366,            406,            2034,            1017,            511,            336,            2023,            2600,            1565,            1055,            2436,            2402,            1487,            1429,            682,            1181,            513,            2182,            1366,            3380,            869,            1326,            2810,            1718,            2177,            2478,            3035,            982,            2190,            1328,            1913,            3425,            3067,            3450,            2021,            2484,            3575,            1649,            414,            2172,            2648,            752,            1184,            463,            1832,            2422,            1345,            805,            939,            2799,            2388,            2383,            1198,            1869,            2416,            891,            671,            1918,            2946,            197,            2578,            ],        [            1429,            1734,            1033,            1588,            1894,            862,            0,            1544,            2013,            1975,            1294,            921,            1272,            2334,            1402,            1990,            1255,            1485,            2035,            2787,            1394,            3065,            2268,            2199,            962,            984,            3163,            1220,            1161,            904,            792,            2580,            620,            2248,            1231,            160,            900,            1692,            2269,            1779,            1619,            2105,            2616,            1156,            1098,            896,            850,            1077,            1851,            1580,            3049,            1083,            1540,            3024,            1900,            2391,            2147,            3027,            1546,            2404,            1510,            1582,            3094,            2736,            2629,            2203,            2698,            3161,            1318,            628,            2386,            2862,            421,            1748,            806,            1501,            2636,            1909,            1019,            1153,            3013,            2602,            2052,            867,            2083,            2630,            1073,            1235,            2132,            2615,            761,            3142,            ],        [            1076,            344,            2317,            2421,            504,            1848,            1284,            0,            3297,            2808,            2578,            763,            1351,            3618,            2686,            3274,            1571,            430,            645,            3620,            2270,            4221,            3101,            3483,            2246,            2268,            4319,            2504,            392,            746,            2076,            3864,            1904,            3532,            2515,            1444,            2184,            302,            879,            3063,            2386,            715,            3900,            1641,            940,            2180,            1621,            2417,            796,            2864,            2903,            2367,            2824,            3113,            2584,            4140,            2001,            2881,            2102,            4153,            2794,            1436,            2948,            2590,            2483,            2289,            4515,            3015,            243,            1912,            3710,            4146,            1371,            2515,            2090,            1525,            4385,            2676,            2303,            2437,            3078,            3886,            1222,            936,            3367,            3914,            2081,            2736,            3416,            2469,            2045,            3975,            ],        [            3327,            3632,            1208,            2856,            3792,            2130,            2232,            3442,            0,            3243,            802,            2819,            2484,            534,            2007,            439,            3153,            3383,            3933,            3680,            2662,            3091,            3164,            283,            2106,            1604,            3189,            1825,            3059,            2802,            1936,            2606,            2377,            1975,            1142,            2078,            2168,            3590,            4167,            1277,            2887,            4003,            1401,            3054,            2996,            1587,            2062,            2277,            3749,            703,            3171,            1258,            731,            2089,            1488,            1030,            2161,            2314,            2814,            1469,            1098,            2756,            3447,            2750,            2729,            1791,            1129,            2854,            3216,            1764,            1451,            2888,            2205,            3016,            2246,            3399,            1701,            3177,            2110,            1330,            2078,            2628,            3950,            2765,            462,            1465,            1635,            2435,            581,            2629,            2029,            3397,            ],        [            1778,            2726,            2167,            360,            3126,            1348,            2013,            2977,            3147,            0,            2428,            2248,            2836,            3468,            1919,            3124,            1228,            2265,            2425,            1061,            729,            1662,            542,            3333,            994,            2118,            1760,            1737,            2657,            2231,            1282,            2022,            1379,            2509,            2365,            1859,            1414,            2924,            3501,            2226,            792,            3337,            3214,            1651,            2425,            2030,            2529,            823,            2256,            2714,            4388,            2217,            2674,            4158,            3066,            3525,            3486,            4383,            724,            3538,            2676,            2921,            4433,            4075,            3968,            3369,            3832,            4500,            2508,            1762,            3520,            1690,            2100,            921,            1541,            1900,            3770,            1082,            1536,            1844,            4147,            2174,            2013,            2421,            3217,            3662,            2239,            727,            3266,            3954,            1275,            1416,            ],        [            2520,            2825,            401,            2049,            2985,            1323,            1425,            2635,            706,            2436,            0,            2012,            1654,            1408,            1583,            1064,            2346,            2576,            3126,            2523,            1855,            3246,            2729,            1273,            1253,            797,            3344,            1401,            2252,            1995,            1083,            2761,            1031,            2214,            335,            1271,            1361,            2783,            3360,            853,            2080,            3196,            1630,            2247,            2189,            734,            1232,            1538,            2942,            461,            3055,            451,            489,            1973,            1246,            1348,            1919,            2198,            2007,            1353,            856,            1964,            3331,            2508,            2613,            1549,            1356,            2738,            2409,            957,            1335,            3043,            1375,            2209,            1267,            2592,            1585,            2370,            1686,            477,            1962,            2783,            3143,            1958,            780,            1704,            805,            1696,            1206,            2387,            1222,            3552,            ],        [            905,            1039,            1735,            1998,            1199,            1273,            709,            849,            2722,            2385,            1996,            0,            588,            3036,            2111,            2692,            1166,            790,            1340,            3197,            1847,            3774,            2678,            2901,            1671,            1693,            3872,            1929,            466,            179,            1501,            3289,            1329,            2957,            1665,            869,            1609,            997,            1574,            2481,            1963,            1410,            3258,            1067,            177,            1605,            858,            1842,            1156,            2289,            2140,            1579,            2249,            2350,            1821,            3377,            1238,            2118,            1679,            3390,            2008,            673,            2185,            1827,            1720,            1526,            3752,            2252,            623,            1337,            2947,            3571,            1205,            2092,            1515,            977,            3622,            2253,            1728,            2105,            2315,            3311,            1528,            361,            2792,            3332,            1343,            2000,            2834,            1706,            1470,            3552,            ],        [            1503,            1637,            1147,            2408,            1797,            1427,            820,            1447,            2134,            2795,            1408,            1427,            0,            2448,            2222,            2104,            1761,            1388,            1938,            3607,            1926,            3885,            3088,            2313,            1782,            1804,            3983,            2040,            1064,            777,            1612,            3400,            1440,            3068,            1077,            980,            1720,            1595,            2172,            1893,            2439,            2008,            2670,            1662,            525,            1716,            270,            1953,            1754,            1701,            2049,            991,            1661,            2259,            1074,            2512,            1147,            2027,            2366,            2525,            1420,            582,            2094,            1736,            1629,            1435,            2819,            2161,            1221,            1448,            2856,            3682,            617,            2568,            1626,            2007,            2757,            2729,            1839,            1517,            2224,            3422,            2126,            967,            2204,            2744,            755,            2111,            2246,            1615,            1581,            3962,            ],        [            3678,            3983,            1559,            3207,            4143,            2481,            2583,            3793,            958,            3792,            1350,            3170,            2812,            0,            1920,            551,            3504,            3734,            4284,            3163,            3013,            2574,            3276,            776,            2218,            1955,            2672,            1937,            3410,            3153,            2048,            2089,            2489,            1458,            1690,            2429,            2519,            3941,            4518,            1389,            3238,            4354,            874,            3405,            3347,            1699,            2390,            2389,            4100,            1251,            3792,            1609,            1279,            2710,            2036,            1651,            2709,            2935,            3165,            2090,            1646,            3304,            4068,            3298,            3350,            2339,            1402,            3475,            3567,            2115,            2072,            2371,            2533,            3367,            2358,            3750,            2322,            3528,            2222,            1442,            2699,            2111,            4301,            3116,            1010,            948,            1963,            2547,            337,            3177,            2380,            2880,            ],        [            2629,            2934,            1618,            1979,            3094,            1507,            1534,            2744,            2327,            1996,            1584,            2121,            2472,            2023,            0,            1679,            2455,            2685,            3235,            2252,            1785,            1663,            1633,            1888,            895,            1186,            1761,            335,            2361,            2104,            749,            1178,            846,            846,            1816,            1380,            1545,            2892,            3469,            824,            2010,            3305,            1214,            2356,            2298,            1146,            2050,            1066,            3051,            2165,            4691,            1668,            2125,            3609,            2882,            2925,            3347,            3834,            1937,            2989,            2492,            2782,            4294,            3936,            4249,            3185,            2734,            4374,            2518,            878,            2971,            1460,            1621,            2139,            1032,            2701,            3221,            2300,            579,            1147,            3598,            1200,            3226,            2067,            2357,            1662,            2022,            1224,            1821,            3815,            1344,            1969,            ],        [            3127,            3432,            1008,            2656,            3592,            1930,            2032,            3242,            873,            3088,            974,            2619,            2261,            569,            1568,            0,            2953,            3183,            3733,            3241,            2462,            2652,            2725,            434,            1667,            1404,            2750,            1386,            2859,            2602,            1497,            2167,            1938,            1536,            1206,            1878,            1968,            3390,            3967,            838,            2687,            3803,            962,            2854,            2796,            1148,            1839,            1838,            3549,            1144,            4081,            1058,            1515,            2999,            1929,            1471,            2602,            3224,            2614,            2379,            1539,            2571,            4357,            3191,            3639,            2232,            1280,            3764,            3016,            1416,            2361,            2449,            1982,            2816,            1807,            3199,            2611,            2977,            1671,            891,            2988,            2189,            3750,            2565,            903,            1026,            1412,            1996,            367,            3070,            1829,            2958,            ],        [            696,            1936,            2070,            1133,            2096,            1239,            1037,            1746,            3050,            1520,            2331,            1116,            1704,            3371,            2439,            3027,            0,            1277,            1378,            2332,            982,            2933,            1813,            3236,            1715,            2021,            3031,            2257,            1669,            1099,            1829,            3293,            1657,            3285,            2268,            1197,            1305,            1894,            2471,            2816,            1098,            2307,            3653,            423,            1293,            1933,            1974,            1544,            1111,            2617,            3256,            2120,            2577,            3466,            2153,            3428,            2354,            3234,            814,            3441,            2547,            1789,            3301,            2943,            2836,            2642,            3735,            3368,            1520,            1665,            4063,            2961,            1124,            1227,            1432,            768,            3673,            1388,            2056,            2190,            3431,            3445,            966,            1289,            3120,            3667,            1834,            1448,            3169,            2822,            1166,            2687,            ],        [            646,            659,            2317,            1991,            819,            2459,            1284,            469,            3297,            2378,            2578,            763,            1351,            3618,            2686,            3274,            1141,            0,            960,            3190,            1840,            3791,            2671,            3483,            2246,            2268,            3889,            2504,            392,            746,            2076,            3864,            1904,            3532,            2515,            1444,            2184,            617,            1194,            3063,            1956,            1030,            3900,            1211,            940,            2180,            1621,            2402,            366,            2864,            2903,            2367,            2824,            3113,            2584,            4140,            2001,            2881,            1672,            4153,            2794,            1436,            2948,            2590,            2483,            2289,            4515,            3015,            243,            1912,            3710,            3819,            1371,            2085,            2090,            1095,            4385,            2246,            2303,            2437,            3078,            4303,            1010,            936,            3367,            3914,            2081,            2306,            3416,            2469,            1924,            3545,            ],        [            1076,            301,            2869,            2149,            701,            2617,            1836,            552,            3849,            2536,            3130,            1315,            1903,            4170,            3238,            3826,            1299,            749,            0,            3348,            1998,            3949,            2829,            4035,            2731,            2820,            4047,            3056,            944,            1298,            2628,            4309,            2456,            4796,            3067,            1996,            2683,            499,            1076,            3615,            2114,            912,            4452,            1722,            1492,            2732,            2173,            2560,            820,            3416,            3455,            2919,            3376,            3665,            3136,            4692,            2553,            3433,            1830,            4705,            3346,            1988,            3500,            3142,            3035,            2841,            5067,            3567,            795,            2464,            4262,            3977,            1923,            2243,            2810,            1525,            4937,            2404,            2855,            2989,            3630,            4461,            577,            1488,            3919,            4466,            2633,            2464,            3968,            3021,            2544,            3703,            ],        [            2407,            3355,            2796,            989,            3755,            1977,            2642,            3606,            3776,            629,            3057,            2877,            3465,            2643,            1137,            2299,            1857,            2894,            3054,            0,            1358,            1053,            288,            2508,            1280,            2747,            1151,            955,            3286,            2860,            1568,            1413,            1665,            1900,            2994,            2488,            1626,            3553,            4130,            1444,            1421,            3966,            2605,            2280,            3054,            1965,            3158,            1161,            2885,            3343,            5113,            2846,            3303,            4787,            3695,            4154,            4211,            5012,            1353,            4167,            3305,            3550,            5158,            4800,            4693,            3998,            3354,            5225,            3137,            1697,            4149,            1081,            2729,            1550,            1427,            2529,            4399,            1711,            1485,            1767,            4776,            1565,            2642,            3050,            2977,            3053,            2868,            1319,            2441,            4679,            1425,            807,            ],        [            1759,            2684,            1438,            747,            2844,            619,            1284,            2494,            2418,            1134,            1699,            1871,            2222,            2739,            1631,            2395,            1209,            2435,            2406,            1946,            0,            2547,            1427,            2604,            733,            1389,            2645,            1449,            2111,            1854,            1021,            2809,            1118,            2477,            1636,            1130,            685,            2642,            3219,            1938,            778,            3055,            2845,            1109,            2048,            1301,            1800,            562,            1797,            1985,            3999,            1488,            1945,            3429,            2337,            2796,            3097,            3654,            705,            2809,            1947,            2532,            4044,            3686,            4069,            2640,            3103,            4194,            2268,            1033,            2791,            2575,            1371,            907,            812,            1454,            3041,            1068,            1248,            1583,            3418,            3059,            1994,            1817,            2488,            3035,            1510,            466,            2537,            3565,            546,            2301,            ],        [            3831,            4779,            3541,            2413,            5023,            3401,            3463,            4673,            3629,            2053,            3507,            4050,            4401,            2868,            1929,            2981,            3281,            4318,            4478,            1424,            2782,            0,            1712,            3190,            2824,            3937,            933,            2264,            4290,            4033,            2678,            980,            2775,            1467,            3739,            3309,            3474,            4821,            5398,            2753,            2845,            5234,            2172,            3704,            4227,            3075,            3979,            2995,            4309,            3900,            6614,            3591,            4048,            5532,            4805,            4227,            5276,            5757,            2777,            4912,            4415,            4711,            6890,            5865,            6172,            5108,            4036,            6297,            4561,            2807,            4894,            632,            3550,            2974,            2961,            3953,            5144,            3135,            2508,            3076,            5521,            1132,            4066,            3996,            3659,            2620,            3945,            3153,            3123,            5744,            3273,            1141,            ],        [            2488,            3436,            2305,            1070,            3836,            2058,            2723,            3687,            3014,            710,            2271,            2958,            3546,            3368,            1204,            2366,            1938,            2975,            3135,            519,            1439,            1120,            0,            2575,            1361,            2828,            1218,            1022,            3367,            2941,            1649,            1480,            1533,            1967,            2503,            2569,            1707,            3634,            4211,            1511,            1502,            4047,            2672,            2361,            3135,            1833,            3239,            1242,            2966,            2852,            5098,            2355,            2812,            4296,            3776,            3612,            4196,            4521,            1434,            3676,            3386,            3631,            5143,            4785,            4936,            4079,            3421,            5061,            3218,            1565,            3658,            1148,            2810,            1631,            1508,            2610,            3908,            1792,            1266,            1834,            4285,            1632,            2723,            3131,            3044,            3120,            2949,            1400,            2508,            4664,            1506,            874,            ],        [            3684,            3989,            1565,            3213,            4149,            2487,            2589,            3799,            822,            3645,            1214,            3176,            2818,            251,            2125,            557,            3510,            3740,            4290,            3414,            3019,            2825,            3282,            0,            2224,            1961,            2923,            1943,            3416,            3159,            2054,            2340,            2495,            1709,            1554,            2435,            2525,            3947,            4524,            1395,            3244,            4360,            1125,            3411,            3353,            1705,            2396,            2395,            4106,            1115,            3595,            1615,            1143,            2513,            1900,            1454,            2573,            2738,            3171,            1893,            1510,            3168,            3871,            3162,            3153,            2203,            1205,            3278,            3573,            2121,            1875,            2622,            2539,            3373,            2364,            3756,            2125,            3534,            2228,            1448,            2502,            2362,            4307,            3122,            874,            1199,            1969,            2553,            343,            3041,            2386,            3131,            ],        [            1796,            2473,            1198,            1084,            2633,            598,            1073,            2283,            2178,            1015,            1228,            1660,            2011,            2108,            898,            1764,            1546,            2224,            2743,            1827,            890,            2428,            1308,            1973,            0,            725,            2659,            716,            1900,            1643,            288,            2076,            385,            1744,            1396,            919,            636,            2431,            3008,            1205,            1115,            2844,            2112,            1523,            1837,            685,            1589,            171,            2211,            1745,            3788,            1248,            1705,            3189,            2097,            2556,            2886,            3414,            1042,            2569,            1707,            2321,            3833,            3475,            3829,            2400,            2819,            3954,            2057,            417,            2551,            2358,            1160,            1244,            200,            1868,            2801,            1405,            515,            850,            3178,            2098,            2331,            1606,            2248,            2404,            1270,            329,            1906,            3354,            435,            2182,            ],        [            1723,            2028,            753,            1252,            2188,            526,            628,            1838,            1733,            1639,            1014,            1215,            1566,            2054,            1066,            1710,            1549,            1779,            2329,            2451,            1058,            2729,            1932,            1919,            626,            0,            2827,            884,            1455,            1198,            456,            2244,            284,            1912,            951,            474,            564,            1986,            2563,            1373,            1283,            2399,            2280,            1450,            1392,            560,            1144,            741,            2145,            1300,            3343,            803,            1260,            2744,            1744,            2111,            2441,            3321,            1210,            2124,            1262,            1876,            3388,            3030,            2923,            2047,            2418,            3455,            1612,            292,            2106,            2526,            715,            1412,            470,            1795,            2356,            1573,            683,            817,            2733,            2266,            2346,            1161,            1803,            2350,            825,            899,            1852,            2909,            425,            2806,            ],        [            2898,            3846,            3287,            1480,            4246,            2468,            3133,            4097,            3477,            1120,            3355,            3368,            3956,            2716,            1777,            2829,            2348,            3385,            3545,            491,            1849,            468,            779,            3038,            1771,            3238,            0,            1446,            3777,            3351,            2059,            828,            2156,            1315,            3485,            2979,            2117,            4044,            4621,            2601,            1912,            4457,            2020,            2771,            3545,            2456,            3649,            1652,            3376,            3834,            5604,            3439,            3794,            5278,            4186,            4075,            4702,            5503,            1844,            4658,            3796,            4041,            5649,            5291,            5184,            4489,            3884,            5716,            3628,            2188,            4640,            496,            3220,            2041,            1918,            3020,            4890,            2202,            1976,            2924,            5267,            980,            3133,            3541,            3507,            2468,            3359,            1810,            2971,            5170,            1916,            208,            ],        [            2661,            2966,            1650,            2011,            3126,            1539,            1566,            2776,            2359,            2028,            1616,            2153,            2504,            2055,            182,            1711,            2487,            2717,            3267,            2434,            1817,            1845,            1665,            1920,            927,            1218,            1943,            0,            2393,            2136,            781,            1360,            878,            1028,            1848,            1412,            1577,            2924,            3501,            856,            2042,            3337,            1396,            2388,            2330,            1178,            2082,            1098,            3083,            2197,            4723,            1700,            2157,            3641,            2914,            2957,            3379,            3866,            1969,            3021,            2524,            2814,            4326,            3968,            4281,            3217,            2766,            4406,            2550,            910,            3003,            1642,            1653,            2171,            1064,            2733,            3253,            2332,            611,            1179,            3630,            1382,            3258,            2099,            2389,            1844,            2054,            1256,            1853,            3847,            1376,            2151,            ],        [            1139,            742,            1925,            2173,            902,            1456,            892,            552,            2905,            2560,            2186,            371,            959,            3226,            2294,            2882,            1341,            493,            1043,            3372,            2022,            3957,            2853,            3091,            1854,            1876,            4055,            2112,            0,            354,            1684,            3472,            1512,            3140,            2123,            1052,            1792,            700,            1277,            2671,            2138,            1113,            3508,            1242,            548,            1788,            1229,            2025,            859,            2472,            2511,            1975,            2432,            2721,            2192,            3748,            1609,            2489,            1854,            3761,            2402,            1044,            2556,            2198,            2091,            1897,            4123,            2623,            326,            1520,            3318,            3754,            979,            2267,            1698,            1152,            3993,            2428,            1911,            2045,            2686,            3494,            1620,            544,            2975,            3522,            1689,            2183,            3024,            2077,            1653,            3727,            ],        [            943,            1077,            1571,            1823,            1237,            1102,            538,            887,            2551,            2210,            1832,            264,            852,            2872,            1940,            2528,            1436,            828,            1378,            3022,            1601,            3603,            2503,            2737,            1500,            1522,            3701,            1758,            504,            0,            1330,            3118,            1158,            2786,            1769,            698,            1438,            1035,            1612,            2317,            2157,            1448,            3154,            1337,            441,            1434,            1054,            1671,            1194,            2118,            2404,            1621,            2078,            2614,            1654,            2929,            1502,            2382,            2084,            2942,            2048,            937,            2449,            2091,            1984,            1790,            3236,            2516,            661,            1166,            2924,            3400,            625,            2286,            1344,            1015,            3174,            2447,            1557,            1691,            2579,            3140,            1566,            190,            2621,            3168,            1335,            1829,            2670,            1970,            1299,            3377,            ],        [            2539,            2844,            1711,            1889,            3004,            1417,            1444,            2654,            2420,            1906,            1677,            2031,            2382,            2116,            610,            1772,            2365,            2595,            3145,            1550,            1695,            2273,            1543,            1981,            805,            1770,            2371,            428,            2271,            2014,            0,            1788,            756,            1456,            1909,            1290,            1455,            2802,            3379,            917,            1920,            3215,            1824,            2266,            2208,            1497,            1960,            976,            2961,            2258,            4784,            1761,            2218,            3702,            2975,            3018,            3257,            3927,            1847,            3082,            2585,            2692,            5060,            3846,            4342,            3278,            2827,            4467,            2428,            788,            3064,            2070,            1531,            2049,            942,            2611,            3314,            2210,            227,            1240,            3691,            1810,            3136,            1977,            2450,            2412,            2115,            1134,            1914,            3725,            1254,            2579,            ],        [            3481,            4429,            3191,            2063,            4829,            3051,            3113,            4680,            3279,            1703,            3157,            3700,            4051,            2518,            1579,            2631,            2931,            3968,            4128,            1074,            2432,            485,            1362,            2840,            2474,            3587,            583,            1914,            4360,            3934,            2328,            0,            2425,            1117,            3389,            2959,            2700,            4627,            5204,            2403,            2495,            5040,            1822,            3354,            3877,            2725,            3629,            2235,            3959,            3550,            6264,            3241,            3698,            5182,            4455,            3877,            4926,            5407,            2427,            4562,            4065,            4361,            6540,            5515,            5822,            4758,            3686,            5947,            4211,            2457,            4544,            282,            3200,            2624,            2611,            3603,            4794,            2785,            2158,            2726,            5171,            782,            3716,            3646,            3309,            2270,            3595,            2393,            2773,            5394,            2499,            791,            ],        [            1783,            2088,            813,            1312,            2248,            586,            688,            1898,            1793,            1699,            1074,            1275,            1626,            2114,            1182,            1770,            1609,            1839,            2389,            2511,            1118,            2845,            1992,            1979,            742,            764,            2943,            1000,            1515,            1258,            572,            2360,            0,            2028,            1011,            534,            624,            2046,            2623,            1559,            1343,            2459,            2396,            1510,            1452,            676,            1204,            801,            2205,            1360,            3403,            863,            1320,            2804,            1712,            2171,            2501,            3029,            1270,            2184,            1322,            1936,            3448,            3090,            3444,            2015,            2478,            3569,            1672,            408,            2166,            2642,            775,            1472,            751,            1855,            2416,            1633,            799,            933,            2793,            2382,            2406,            1221,            1863,            2410,            885,            959,            1912,            2969,            485,            2866,            ],        [            3091,            3396,            2080,            2441,            3556,            1969,            1996,            3206,            2789,            2458,            2046,            2583,            2934,            2485,            462,            2141,            2917,            3147,            3697,            1878,            2247,            2299,            2095,            2350,            1357,            1648,            2397,            797,            2823,            2566,            1211,            1814,            1308,            0,            2278,            1842,            2007,            3354,            3931,            1286,            2472,            3767,            1850,            2818,            2760,            1608,            2512,            1528,            3513,            2627,            5153,            2130,            2587,            4071,            3344,            3387,            3809,            4296,            2399,            3451,            2954,            3244,            4756,            4398,            4711,            3647,            3196,            4836,            2980,            1340,            3433,            2096,            2083,            2601,            1494,            3163,            3683,            2762,            1041,            1609,            4060,            1836,            3688,            2529,            2819,            2781,            2484,            1686,            2283,            4277,            1806,            2605,            ],        [            2497,            2802,            366,            2026,            2962,            1300,            1241,            2612,            1056,            2413,            335,            1989,            1317,            1590,            1842,            1495,            2323,            2553,            3103,            3225,            1832,            3505,            2706,            1339,            1400,            774,            3603,            1660,            2229,            1822,            1230,            3020,            1008,            2473,            0,            1248,            1338,            2760,            3337,            1112,            2057,            3173,            1889,            2224,            1570,            1000,            895,            1515,            2919,            623,            3149,            210,            583,            2067,            1340,            1434,            2013,            2292,            1984,            1447,            950,            1627,            3425,            2602,            2707,            1643,            1741,            2832,            2386,            934,            1429,            3302,            1038,            2186,            1244,            2569,            1679,            2347,            1457,            736,            2056,            3042,            3120,            1935,            1126,            1963,            468,            1673,            1465,            2481,            1199,            3811,            ],        [            1579,            1884,            879,            1434,            2044,            708,            484,            1694,            1859,            1821,            1140,            1071,            1422,            2180,            1248,            1836,            1405,            1635,            2185,            2633,            1240,            2911,            2114,            2045,            808,            830,            3009,            1066,            1311,            1054,            638,            2426,            466,            2094,            1077,            0,            746,            1842,            2419,            1625,            1465,            2255,            2462,            1306,            1248,            742,            1000,            979,            2001,            1426,            3199,            929,            1386,            2870,            1746,            2237,            2297,            3095,            1392,            2250,            1356,            1732,            3244,            2886,            3510,            2049,            2544,            3635,            1468,            474,            2232,            2708,            571,            1594,            652,            1651,            2482,            1755,            865,            999,            2859,            2448,            2202,            1017,            1929,            2476,            919,            1137,            1978,            2765,            607,            2988,            ],        [            1700,            2648,            1407,            688,            3048,            588,            1253,            2899,            2387,            1075,            1668,            1840,            2191,            2708,            1776,            2364,            1150,            2187,            2347,            1887,            494,            2488,            1368,            2573,            950,            1358,            2586,            1594,            2080,            2086,            1166,            2848,            994,            3335,            1605,            1099,            0,            2846,            3188,            2153,            719,            3259,            2990,            1410,            2017,            1270,            1769,            831,            2178,            1954,            3968,            1457,            1914,            3398,            2306,            2765,            3066,            3623,            646,            2778,            1916,            2501,            4013,            3655,            4038,            2609,            3072,            4163,            2430,            1002,            2760,            2516,            1340,            848,            781,            1755,            3010,            1009,            1393,            1527,            3387,            3000,            1935,            1786,            2457,            3004,            1479,            960,            2506,            3534,            515,            2242,            ],        [            1265,            198,            2506,            2648,            202,            2037,            1473,            189,            3486,            3035,            2767,            952,            1540,            3807,            2875,            3463,            1798,            619,            499,            3847,            2497,            4448,            3328,            3672,            2435,            2457,            4546,            2693,            581,            935,            2265,            4053,            2093,            3721,            2704,            1633,            2373,            0,            733,            3252,            2613,            413,            4089,            1830,            1129,            2369,            1810,            2606,            985,            3053,            3092,            2556,            3013,            3302,            2773,            4329,            2190,            3070,            2329,            4342,            2983,            1625,            3137,            2779,            2672,            2478,            4704,            3204,            432,            2101,            3899,            4335,            1560,            2742,            2279,            1714,            4574,            2903,            2492,            2626,            3267,            4075,            1076,            1125,            3556,            4103,            2270,            2963,            3605,            2658,            2234,            4202,            ],        [            1707,            640,            2948,            3090,            780,            2479,            1915,            631,            3928,            3477,            3209,            1394,            1982,            4249,            3317,            3905,            2240,            1061,            941,            4289,            2939,            4890,            3770,            4114,            2877,            2899,            4988,            3135,            1023,            1377,            2707,            4495,            2535,            4163,            3146,            2075,            2815,            578,            0,            3694,            3055,            991,            4531,            2272,            1571,            2811,            2252,            3048,            1427,            3495,            3534,            2998,            3455,            3744,            3215,            4771,            2632,            3512,            2771,            4784,            3425,            2067,            3579,            3221,            3114,            2920,            5146,            3646,            874,            2543,            4341,            4777,            2002,            3184,            2721,            2156,            5016,            3345,            2934,            3068,            3709,            4517,            1518,            1567,            3998,            4545,            2712,            3405,            4047,            3100,            2676,            4644,            ],        [            2819,            3124,            1037,            2348,            3284,            1988,            1724,            2934,            1562,            2477,            1003,            2311,            2290,            1258,            957,            914,            2645,            2875,            3425,            1897,            2154,            2203,            2114,            1123,            1376,            1096,            2301,            775,            2551,            2294,            1230,            1718,            1327,            1087,            1235,            1570,            1660,            3082,            3659,            0,            2379,            3495,            1009,            2546,            2488,            823,            1868,            1547,            3241,            1584,            4110,            1087,            1544,            3028,            2301,            2160,            2974,            3253,            2306,            2408,            1911,            2600,            4386,            3563,            3668,            2604,            1969,            3793,            2708,            1091,            2390,            2000,            2011,            2508,            1513,            2891,            2640,            2669,            1060,            566,            3017,            1740,            3442,            2257,            1592,            1082,            1441,            1705,            1056,            3442,            1825,            2509,            ],        [            1298,            2246,            1687,            396,            2646,            868,            1533,            2497,            2667,            783,            1948,            1768,            2471,            2988,            1880,            2644,            748,            1785,            1945,            1595,            249,            2196,            1076,            2853,            982,            1638,            2294,            1698,            2177,            1751,            1270,            2556,            1367,            3043,            1885,            1379,            934,            2444,            3021,            2187,            0,            2857,            3094,            1171,            1945,            1550,            2049,            811,            1776,            2234,            3908,            1737,            2194,            3678,            2586,            3045,            3006,            3903,            244,            3058,            2196,            2441,            3953,            3595,            4318,            2889,            3352,            4443,            2028,            1282,            3040,            2224,            1620,            490,            1061,            1420,            3290,            651,            1497,            1832,            3667,            2708,            1533,            2066,            2737,            3284,            1759,            715,            2786,            3474,            795,            1950,            ],        [            1678,            611,            2919,            3061,            211,            2450,            1886,            602,            3899,            3448,            3180,            1365,            1953,            4220,            3288,            3876,            2211,            1032,            912,            4260,            2910,            4861,            3741,            4085,            2848,            2870,            4959,            3106,            994,            1348,            2678,            4466,            2506,            4134,            3117,            2046,            2786,            413,            1146,            3665,            3026,            0,            4502,            2243,            1542,            2782,            2223,            3019,            1398,            3466,            3505,            2969,            3426,            3715,            3186,            4742,            2603,            3483,            2742,            4755,            3396,            2038,            3550,            3192,            3085,            2891,            5117,            3617,            845,            2514,            4312,            4748,            1973,            3155,            2692,            2127,            4987,            3316,            2905,            3039,            3680,            4488,            1489,            1538,            3969,            4516,            2683,            3376,            4018,            3071,            2647,            4615,            ],        [            3488,            3793,            1369,            3017,            3953,            2291,            2393,            3603,            1457,            3292,            1335,            2980,            2622,            696,            1420,            809,            3314,            3544,            4094,            2663,            2823,            2074,            3086,            1018,            2028,            1765,            2172,            1747,            3220,            2963,            1858,            1589,            2299,            958,            1567,            2239,            2329,            3751,            4328,            1199,            3048,            4164,            0,            3215,            3157,            1509,            2200,            2199,            3910,            1728,            4442,            1419,            1876,            3360,            2633,            2055,            3306,            3585,            2975,            2740,            2243,            2932,            4718,            3895,            4000,            2936,            1864,            4125,            3377,            1925,            2722,            1871,            2343,            3177,            2168,            3560,            2972,            3338,            2032,            1252,            3349,            1611,            4111,            2926,            1487,            448,            1773,            2357,            951,            3774,            2190,            2380,            ],        [            273,            1513,            2247,            1366,            1673,            1834,            1214,            1323,            3227,            1753,            2508,            693,            1281,            3548,            2616,            3204,            516,            854,            1277,            2565,            1215,            3166,            2046,            3413,            1948,            2198,            3264,            2434,            1246,            676,            2006,            3526,            1834,            3462,            2445,            1374,            1240,            1471,            2048,            2993,            1331,            1884,            3830,            0,            870,            2110,            1551,            1777,            688,            2794,            2833,            2297,            2754,            3043,            2514,            4070,            1931,            2811,            1047,            4083,            2724,            1366,            2878,            2520,            2413,            2219,            4445,            2945,            1097,            1842,            3640,            3194,            1301,            1460,            2027,            345,            4315,            1621,            2233,            2367,            3008,            3678,            896,            866,            3297,            3844,            2011,            1681,            3346,            2399,            1761,            2920,            ],        [            1157,            1291,            1558,            2250,            1451,            1838,            1231,            1101,            2545,            2637,            1819,            478,            411,            2859,            2633,            2515,            1418,            1042,            1592,            3449,            2099,            4296,            2930,            2724,            2193,            2215,            4394,            2451,            686,            431,            2023,            3811,            1851,            3479,            1488,            1391,            2131,            1249,            1826,            2304,            2215,            1662,            3081,            1319,            0,            2127,            681,            2364,            1408,            2112,            1963,            1402,            2072,            2173,            1644,            3200,            1061,            1941,            1931,            3213,            1831,            496,            2008,            1650,            1543,            1349,            3575,            2075,            875,            1859,            2770,            4093,            1028,            2344,            2037,            1229,            3445,            2505,            2250,            1928,            2138,            3833,            1780,            621,            2615,            3155,            1166,            2522,            2657,            1529,            1992,            3804,            ],        [            2097,            2402,            743,            1626,            2562,            900,            1002,            2212,            1723,            1699,            709,            1589,            1940,            1589,            1124,            1245,            1923,            2153,            2703,            2511,            1432,            2787,            1992,            1454,            684,            374,            2885,            942,            1829,            1572,            514,            2302,            342,            1970,            941,            848,            938,            2360,            2937,            657,            1657,            2773,            1811,            1824,            1766,            0,            1518,            855,            2519,            1290,            3816,            793,            1250,            2734,            2007,            2101,            2815,            2959,            1584,            2114,            1617,            2250,            3762,            3404,            3374,            2310,            2300,            3499,            1986,            268,            2096,            2584,            1089,            1786,            528,            2169,            2346,            1947,            741,            257,            2723,            2324,            2720,            1535,            1923,            1885,            1147,            1013,            1387,            3283,            799,            3093,            ],        [            1665,            1970,            877,            2138,            2130,            1157,            550,            1780,            1864,            2525,            1138,            1157,            1049,            2178,            1952,            1834,            1491,            1721,            2271,            3337,            1656,            3615,            2818,            2043,            1512,            1534,            3713,            1770,            1397,            1140,            1342,            3130,            1170,            2798,            807,            710,            1450,            1928,            2505,            1623,            2169,            2341,            2400,            1392,            1334,            1446,            0,            1683,            2087,            1431,            2660,            721,            1391,            2875,            804,            2242,            1477,            2357,            2096,            2255,            1150,            1359,            2424,            2066,            1959,            1107,            2549,            2491,            1554,            1178,            2237,            3412,            347,            2298,            1356,            1737,            2487,            2459,            1569,            1247,            2554,            3152,            2288,            1103,            1934,            2474,            485,            1841,            1976,            1945,            1311,            3692,            ],        [            1677,            2544,            1298,            913,            2704,            479,            1144,            2354,            2278,            844,            1559,            1731,            2082,            2279,            1069,            1935,            1375,            2295,            2572,            1656,            719,            2257,            1137,            2144,            171,            896,            2355,            887,            1971,            1714,            459,            2247,            556,            1915,            1496,            990,            517,            2502,            3079,            1376,            944,            2915,            2283,            1404,            1908,            856,            1660,            0,            2092,            1845,            3859,            1348,            1805,            3289,            2197,            2656,            2957,            3514,            871,            2669,            1807,            2392,            3904,            3546,            3929,            2500,            2963,            4054,            2128,            588,            2651,            2285,            1231,            1073,            318,            1749,            2901,            1234,            686,            1021,            3278,            2269,            2160,            1677,            2348,            2575,            1370,            158,            2077,            3425,            316,            2011,            ],        [            278,            1025,            2629,            1623,            1185,            2091,            1596,            835,            3609,            2010,            2890,            1075,            1663,            3930,            2998,            3586,            773,            366,            583,            2822,            1472,            3423,            2303,            3795,            2205,            2580,            3521,            2816,            758,            1058,            2388,            3783,            2216,            4270,            2827,            1756,            1695,            983,            1560,            3375,            1588,            1396,            4212,            843,            1252,            2492,            1933,            2034,            0,            3176,            3215,            2679,            3136,            3425,            2896,            4452,            2313,            3193,            1304,            4465,            3106,            1748,            3260,            2902,            2795,            2601,            4827,            3327,            609,            2224,            4022,            3451,            1683,            1717,            1822,            727,            4697,            1878,            2615,            2749,            3390,            3935,            642,            1248,            3679,            4226,            2393,            1938,            3728,            2781,            1556,            3177,            ],        [            2624,            2929,            505,            2153,            3089,            1427,            1529,            2739,            1221,            2540,            462,            2116,            1758,            1733,            1908,            1389,            2450,            2680,            3230,            3352,            1959,            3571,            2833,            1598,            1527,            901,            3669,            1726,            2356,            2099,            1357,            3086,            1135,            2539,            439,            1375,            1465,            2887,            3464,            1178,            2184,            3300,            1955,            2351,            2293,            1127,            1336,            1642,            3046,            0,            3314,            555,            748,            2232,            1505,            1599,            2178,            2457,            2111,            1612,            1115,            2068,            3590,            2767,            2872,            1808,            1906,            2997,            2513,            1061,            1594,            3368,            1479,            2313,            1371,            2696,            1844,            2474,            1584,            907,            2221,            3108,            3247,            2062,            1291,            2029,            909,            1800,            1531,            2646,            1326,            3877,            ],        [            2937,            3071,            3158,            4030,            3231,            3338,            2731,            2881,            3086,            4417,            3115,            2258,            1911,            3620,            4561,            3525,            3198,            2822,            3372,            5229,            3879,            6224,            4710,            3369,            3693,            3554,            6322,            4379,            2498,            2211,            3523,            5739,            3351,            5192,            3092,            2891,            3631,            3029,            3606,            3831,            3995,            3442,            4608,            3099,            1959,            3780,            2181,            3864,            3188,            2653,            0,            3208,            2681,            1098,            1481,            2125,            908,            866,            3711,            2138,            3048,            1503,            487,            846,            468,            1186,            2500,            311,            2655,            3359,            1695,            6021,            2528,            4124,            3537,            3009,            2370,            4285,            3750,            3560,            1063,            5761,            3560,            2401,            3103,            4682,            2666,            4022,            3667,            933,            3492,            5670,            ],        [            2242,            2547,            270,            1991,            2707,            1265,            1127,            2357,            1250,            2378,            531,            1734,            1203,            1571,            1746,            1227,            2068,            2298,            2848,            3190,            1797,            3409,            2671,            1436,            1365,            739,            3507,            1564,            1974,            1708,            1195,            2924,            973,            2377,            200,            1287,            1303,            2505,            3082,            1016,            2022,            2918,            1793,            1969,            1456,            965,            781,            1480,            2664,            817,            2980,            0,            777,            2261,            1181,            1628,            2078,            2486,            1949,            1641,            791,            1513,            3025,            2667,            2901,            1484,            1935,            3026,            2131,            899,            1623,            3206,            924,            2151,            1209,            2314,            1873,            2312,            1422,            640,            2250,            2946,            2865,            1680,            1320,            1867,            354,            1638,            1369,            2546,            1164,            3715,            ],        [            3396,            3701,            1277,            2925,            3861,            2199,            2301,            3511,            1205,            3312,            1234,            2888,            3567,            1739,            2680,            1644,            3222,            3452,            4002,            4124,            2731,            4343,            3605,            1488,            2299,            1673,            4441,            2498,            3128,            3867,            2129,            3858,            1907,            3311,            1211,            2147,            2237,            3659,            4236,            1950,            2956,            4072,            2727,            3123,            3615,            1899,            2131,            2414,            3818,            772,            2566,            1327,            0,            1484,            1557,            851,            2564,            1709,            2883,            864,            1167,            3159,            2842,            3153,            2124,            1860,            1226,            2249,            3285,            1833,            846,            4140,            2274,            3085,            2143,            3468,            1096,            3246,            2356,            1679,            1473,            3880,            4019,            2834,            1275,            2801,            1704,            2572,            1786,            3288,            2098,            4649,            ],        [            3109,            3243,            2517,            4165,            3403,            3439,            2903,            3053,            2445,            4552,            2474,            2430,            2083,            2979,            3920,            2884,            3370,            2994,            3544,            5364,            3971,            5583,            4845,            2728,            3539,            2913,            5681,            3738,            2670,            2383,            3369,            5098,            3147,            4551,            2451,            3063,            3477,            3201,            3778,            3190,            4196,            3614,            3967,            3271,            2131,            3139,            2353,            3654,            3360,            2012,            1082,            2567,            2040,            0,            1653,            1484,            1080,            225,            3883,            1497,            2407,            1675,            1358,            1669,            640,            1358,            1859,            765,            2827,            3073,            1054,            5380,            2700,            4325,            3383,            3181,            1729,            4486,            3596,            2919,            422,            5120,            3732,            2573,            2462,            4041,            2838,            3812,            3026,            1804,            3338,            5889,            ],        [            2702,            2836,            1345,            2821,            2996,            1840,            1233,            2646,            1381,            3208,            1410,            1840,            1676,            1915,            2856,            1820,            2174,            2587,            3137,            4020,            2339,            4519,            3501,            1664,            2195,            1849,            4617,            2674,            2263,            1976,            2025,            4034,            1853,            3487,            1074,            1393,            2133,            2794,            3371,            2126,            2852,            3207,            2903,            2075,            1724,            2075,            887,            2366,            2953,            948,            1856,            1189,            908,            1785,            0,            1759,            673,            1553,            2779,            1772,            681,            1268,            1620,            1262,            1155,            303,            2066,            1687,            2420,            1861,            1754,            4316,            1030,            2981,            2039,            2420,            2004,            3142,            2252,            1855,            1750,            4056,            3325,            2166,            1451,            2977,            1218,            2524,            1962,            1141,            1994,            4375,            ],        [            4168,            4302,            1402,            3050,            4462,            2324,            2426,            4112,            1108,            3437,            1359,            3013,            3142,            1642,            2805,            1547,            3347,            4053,            4603,            4249,            2856,            4468,            3730,            1391,            2424,            1798,            4566,            2623,            3729,            3442,            2254,            3983,            2032,            3436,            1336,            2272,            2362,            4260,            4837,            2075,            3081,            4673,            2852,            3248,            3190,            2024,            3412,            2539,            4419,            897,            2141,            1452,            925,            1059,            1682,            0,            2139,            1284,            3008,            439,            1292,            2734,            2417,            2728,            1699,            2417,            566,            1824,            3886,            1958,            421,            4265,            2399,            3210,            2268,            3593,            671,            3371,            2481,            1804,            1048,            4005,            4791,            2959,            1169,            2926,            1829,            2697,            1700,            2863,            2223,            4774,            ],        [            2207,            2341,            2096,            3300,            2501,            2608,            2001,            2151,            2132,            3687,            2161,            1528,            1181,            2666,            3403,            2571,            2468,            2092,            2642,            4499,            3149,            5066,            3980,            2415,            2963,            2600,            5164,            3221,            1768,            1481,            2793,            4581,            2621,            4249,            1825,            2161,            2901,            2299,            2876,            2877,            3265,            2712,            3654,            2369,            1229,            2897,            1451,            3134,            2458,            1699,            1183,            1940,            1659,            1280,            751,            2307,            0,            1048,            2981,            2320,            1432,            773,            947,            589,            650,            456,            2682,            1494,            1925,            2629,            1877,            4863,            1798,            3394,            2807,            2279,            2552,            3555,            3020,            2606,            1245,            4603,            2830,            1671,            2202,            3728,            1936,            3292,            2713,            468,            2762,            4940,            ],        [            3341,            3475,            2292,            3940,            3635,            3214,            3135,            3285,            2220,            4327,            2249,            2662,            2315,            2754,            3695,            2659,            3602,            3226,            3776,            5139,            3746,            5358,            4620,            2503,            3314,            2688,            5456,            3513,            2902,            2615,            3144,            4873,            2922,            4326,            2226,            3162,            3252,            3433,            4010,            2965,            3971,            3846,            3742,            3503,            2363,            2914,            2585,            3429,            3592,            1787,            1314,            2342,            1815,            232,            1885,            1259,            1312,            0,            3898,            1272,            2182,            1907,            1590,            1901,            872,            1590,            1634,            997,            3059,            2848,            829,            5155,            2932,            4100,            3158,            3413,            1504,            4261,            3371,            2694,            197,            4895,            3964,            2805,            2237,            3816,            2719,            3587,            2801,            2036,            3113,            5664,            ],        [            1054,            2002,            1915,            628,            2402,            1096,            1541,            2253,            2895,            1015,            2176,            1524,            2112,            3216,            2108,            2872,            504,            1541,            1701,            1827,            477,            2428,            1308,            3081,            1210,            1866,            2526,            1926,            1933,            1507,            1498,            2788,            1595,            3275,            2113,            1607,            1162,            2200,            2777,            2415,            593,            2613,            3322,            927,            1701,            1778,            2382,            1039,            1532,            2462,            3664,            1965,            2422,            3874,            2814,            3273,            2762,            3642,            0,            3286,            2424,            2197,            3709,            3351,            3244,            3050,            3580,            3776,            1784,            1510,            3268,            2456,            1628,            722,            1289,            1176,            3518,            883,            1725,            2060,            3839,            2940,            1289,            1697,            2965,            3512,            1987,            943,            3014,            3230,            1023,            2182,            ],        [            4482,            4140,            1716,            3364,            4300,            2638,            2740,            3950,            1644,            3751,            1673,            3327,            3456,            2178,            3119,            2083,            3661,            3891,            4441,            4563,            3170,            4782,            4044,            1927,            2738,            2112,            4880,            2937,            3567,            3756,            2568,            4297,            2346,            3750,            1650,            2586,            2676,            4098,            4675,            2389,            3395,            4511,            3166,            3562,            3504,            2338,            3726,            2853,            4257,            1211,            2455,            1766,            1239,            1373,            1996,            683,            2453,            1598,            3322,            0,            1606,            3048,            2731,            3042,            2013,            2731,            1058,            2138,            3724,            2272,            735,            4579,            2713,            3524,            2582,            3907,            232,            3685,            2795,            2118,            1362,            4319,            5105,            3273,            1661,            3240,            2143,            3011,            2225,            3177,            2537,            5088,            ],        [            2460,            2765,            1548,            2933,            2925,            1952,            1345,            2575,            1307,            3320,            1809,            1952,            1421,            1841,            2747,            1746,            2286,            2516,            3066,            4132,            2451,            4410,            3613,            1590,            2307,            2017,            4508,            2565,            2192,            1926,            2137,            3925,            1965,            3655,            1478,            1505,            2245,            2723,            3300,            2294,            2964,            3136,            3071,            2187,            1674,            2241,            999,            2422,            2882,            874,            2246,            1392,            834,            2175,            390,            1685,            1063,            1943,            2891,            1698,            0,            1658,            2010,            1652,            1545,            693,            1992,            2077,            2349,            1973,            2772,            4207,            1142,            3093,            2151,            2532,            1930,            3254,            2364,            1918,            2140,            3947,            3083,            1898,            1377,            3145,            1156,            2580,            1888,            1531,            2106,            4487,            ],        [            1434,            1568,            1555,            2527,            1728,            1835,            1228,            1378,            2542,            2914,            1816,            755,            408,            2856,            2630,            2512,            1695,            1319,            1869,            3726,            2376,            4293,            3207,            2721,            2190,            2024,            4391,            2448,            995,            708,            2020,            3808,            1848,            3476,            1485,            1388,            2128,            1526,            2103,            2301,            2492,            1939,            3078,            1596,            456,            2124,            678,            2361,            1685,            2109,            1980,            1399,            2069,            2190,            1661,            3217,            1078,            1958,            2208,            3230,            1828,            0,            2025,            1667,            1560,            1366,            3592,            2092,            1152,            1856,            2787,            4090,            1025,            2621,            2034,            1506,            3462,            2782,            2247,            1925,            2155,            3830,            2057,            898,            2612,            3152,            1163,            2519,            2654,            1546,            1989,            4081,            ],        [            3173,            3307,            3394,            4266,            3467,            3574,            2967,            3117,            3322,            4653,            3351,            2494,            2147,            3856,            4797,            3761,            3434,            3058,            3608,            5465,            4115,            6460,            4946,            3605,            3929,            3790,            6558,            4615,            2734,            2447,            3759,            5975,            3587,            5428,            3328,            3127,            3867,            3265,            3842,            4067,            4231,            3678,            4844,            3335,            2195,            4016,            2417,            4100,            3424,            2889,            236,            3444,            2917,            1334,            1717,            2361,            1144,            1102,            3947,            2374,            3284,            1739,            0,            660,            704,            1422,            2736,            547,            2891,            3595,            1931,            6257,            2764,            4360,            3773,            3245,            2606,            4521,            3986,            3796,            1299,            5997,            3796,            2637,            3339,            4918,            2902,            4258,            3903,            747,            3728,            5906,            ],        [            2783,            2917,            3752,            3876,            3077,            3184,            2577,            2727,            3680,            4263,            3709,            2104,            1757,            4214,            3979,            4119,            3044,            2668,            3218,            5075,            3725,            5642,            4556,            3963,            3539,            4148,            5740,            3797,            2344,            2057,            3369,            5157,            3197,            5786,            2401,            2737,            3477,            2875,            3452,            4425,            3841,            3288,            5202,            2945,            1805,            3473,            2027,            3710,            3034,            3247,            594,            3802,            2235,            1692,            1327,            2719,            1502,            1460,            3557,            2732,            2008,            1349,            358,            0,            1062,            1032,            3094,            905,            2501,            3205,            2289,            5439,            2374,            3970,            3383,            2855,            2964,            4131,            3596,            4154,            1657,            5179,            3406,            2247,            3697,            5276,            2512,            3868,            4261,            308,            3338,            5516,            ],        [            2469,            2603,            2872,            3562,            2763,            2870,            2263,            2413,            2800,            3949,            2829,            1790,            1443,            3334,            3665,            3239,            2730,            2354,            2904,            4761,            3411,            5328,            4242,            3083,            3225,            3268,            5426,            3483,            2030,            1743,            3055,            4843,            2883,            4511,            2087,            2423,            3163,            2561,            3138,            3545,            3527,            2974,            4322,            2631,            1491,            3159,            1713,            3396,            2720,            2367,            602,            2434,            1921,            812,            1013,            1839,            440,            580,            3243,            1852,            1694,            1035,            878,            1029,            0,            718,            2214,            714,            2187,            2891,            1409,            5125,            2060,            3656,            3069,            2541,            2084,            3817,            3282,            3274,            777,            4865,            3092,            1933,            2817,            4396,            2198,            3554,            3381,            908,            3024,            5202,            ],        [            2407,            2541,            1648,            3124,            2701,            2143,            1536,            2351,            1684,            3511,            1713,            1728,            1381,            2218,            3159,            2123,            2668,            2292,            2842,            4323,            2642,            4822,            3804,            1967,            2498,            2152,            4920,            2977,            1968,            1681,            2328,            4337,            2156,            3790,            1377,            1696,            2436,            2499,            3076,            2429,            3465,            2912,            3206,            2569,            1429,            2378,            1651,            2669,            2658,            1251,            1561,            1492,            1211,            1490,            303,            2517,            378,            1258,            3181,            2530,            984,            973,            1325,            967,            860,            0,            2892,            1392,            2125,            2164,            2087,            4619,            1333,            3594,            2342,            2479,            2762,            3755,            2555,            2158,            1455,            4359,            3030,            1871,            1754,            3280,            1521,            2827,            2265,            846,            2297,            5128,            ],        [            4417,            4551,            1651,            3299,            4711,            2573,            2675,            4361,            696,            3686,            1172,            3262,            3391,            1230,            2703,            1135,            3596,            4302,            4852,            4498,            3105,            3787,            3979,            979,            2673,            2047,            3885,            2521,            3978,            3691,            2503,            3302,            2281,            2671,            1585,            2521,            2611,            4509,            5086,            1973,            3330,            4922,            2097,            3497,            3439,            2273,            3661,            2788,            4668,            1146,            2390,            1701,            1174,            1308,            1931,            249,            2388,            1533,            3257,            688,            1541,            2983,            2666,            2977,            1948,            2666,            0,            2073,            4135,            2207,            670,            3584,            2648,            3459,            2517,            3842,            920,            3620,            2730,            2053,            1297,            3324,            5040,            3208,            832,            2161,            2078,            2946,            1363,            3112,            2472,            4093,            ],        [            3063,            3197,            2854,            4502,            3357,            3776,            2857,            3007,            2782,            4889,            2811,            2384,            2037,            3316,            4257,            3221,            3324,            2948,            3498,            5701,            4005,            5920,            5182,            3065,            3876,            3250,            6018,            4075,            2624,            2337,            3706,            5435,            3484,            4888,            2788,            3017,            3814,            3155,            3732,            3527,            4533,            3568,            4304,            3225,            2085,            3476,            2307,            3991,            3314,            2349,            620,            2904,            2377,            794,            1607,            1821,            1034,            562,            3837,            1834,            2744,            1629,            896,            1623,            594,            1312,            2196,            0,            2781,            3410,            1391,            5717,            2654,            4662,            3720,            3135,            2066,            4823,            3933,            3256,            759,            5457,            3686,            2527,            2799,            4378,            2792,            4149,            3363,            1342,            3675,            6226,            ],        [            921,            524,            2074,            2266,            684,            1605,            1041,            334,            3054,            2653,            2335,            520,            1108,            3375,            2443,            3031,            1416,            275,            825,            3465,            2115,            4106,            2946,            3240,            2003,            2025,            4204,            2261,            149,            503,            1833,            3621,            1661,            3289,            2272,            1201,            1941,            482,            1059,            2820,            2231,            895,            3657,            1391,            697,            1937,            1378,            2118,            641,            2621,            2660,            2124,            2581,            2870,            2341,            3897,            1758,            2638,            1947,            3910,            2551,            1193,            2705,            2347,            2240,            2046,            4272,            2772,            0,            1669,            3467,            3903,            1128,            2360,            1847,            1370,            4142,            2521,            2060,            2194,            2835,            3643,            1402,            693,            3124,            3671,            1838,            2276,            3173,            2226,            1802,            3820,            ],        [            1857,            2162,            887,            1386,            2322,            1014,            762,            1972,            1867,            1431,            1148,            1349,            1700,            1857,            856,            1513,            1683,            1913,            2463,            2243,            1192,            2519,            1724,            1722,            416,            308,            2617,            674,            1589,            1332,            246,            2034,            74,            1702,            1085,            608,            698,            2120,            2697,            1163,            1417,            2533,            2070,            1584,            1526,            268,            1278,            587,            2279,            1434,            3477,            937,            1394,            2878,            1786,            2245,            2575,            3103,            1344,            2258,            1396,            2010,            3522,            3164,            3518,            2089,            2568,            3643,            1746,            0,            2240,            2316,            849,            1546,            260,            1929,            2490,            1707,            473,            525,            2867,            2056,            2480,            1295,            1937,            2153,            959,            745,            1655,            3043,            851,            2825,            ],        [            3747,            3881,            1725,            3373,            4041,            2647,            3541,            3691,            1653,            3760,            1682,            3068,            2721,            2187,            3128,            2092,            4008,            3632,            4182,            4572,            3179,            4791,            4053,            1936,            2747,            2121,            4889,            2946,            3308,            3021,            2577,            4306,            2355,            3759,            1659,            2595,            2685,            3839,            4416,            2398,            3404,            4252,            3175,            3909,            2769,            2347,            2991,            2862,            3998,            1220,            1720,            1775,            1248,            638,            2291,            692,            1718,            863,            3331,            705,            1615,            2313,            1996,            2307,            1278,            1996,            1067,            1403,            3465,            2281,            0,            4588,            3338,            3533,            2591,            3819,            937,            3694,            2804,            2127,            627,            4328,            4370,            3211,            1670,            3249,            2152,            3020,            2234,            2442,            2546,            5097,            ],        [            3199,            4147,            3453,            1781,            4547,            2769,            3434,            4398,            3541,            1421,            3419,            3669,            4257,            2780,            1841,            2893,            2649,            3686,            3846,            792,            2150,            203,            1080,            3102,            2072,            3539,            301,            2176,            4078,            3652,            2360,            892,            2687,            1379,            3651,            3280,            2418,            4345,            4922,            2665,            2213,            4758,            2084,            3072,            3846,            2987,            3950,            1953,            3677,            3812,            5905,            3503,            3960,            5444,            4487,            4139,            5003,            5669,            2145,            4824,            4097,            4342,            5950,            5592,            6084,            4790,            3948,            6209,            3929,            2719,            4806,            0,            3521,            2342,            2219,            3321,            5056,            2503,            2420,            2988,            5433,            1044,            3434,            3842,            3571,            2532,            3660,            2111,            3035,            5471,            2217,            509,            ],        [            1318,            1623,            1236,            1791,            1783,            810,            203,            1433,            2089,            2178,            1363,            810,            851,            2403,            1605,            2059,            1144,            1374,            1924,            2990,            1309,            3268,            2471,            2268,            1165,            1187,            3366,            1423,            1050,            793,            995,            2783,            823,            2451,            1032,            363,            1103,            1581,            2158,            1848,            1822,            1994,            2819,            1045,            987,            1099,            429,            1336,            1740,            1656,            2628,            946,            1616,            2838,            1029,            2467,            1726,            2606,            1749,            2480,            1375,            1161,            2673,            2315,            2208,            1332,            2774,            2740,            1207,            831,            2462,            3065,            0,            1951,            1009,            1390,            2712,            2112,            1222,            1356,            2803,            2805,            1941,            756,            2159,            2699,            710,            1494,            2201,            2194,            964,            3345,            ],        [            1769,            2717,            2158,            867,            3117,            1339,            2004,            2968,            3138,            1254,            2419,            2239,            2942,            3459,            2351,            3115,            1236,            2256,            2416,            2066,            720,            2667,            1547,            3324,            1453,            2109,            2765,            2169,            2648,            2222,            1741,            3027,            1838,            3514,            2356,            1850,            1405,            2915,            3492,            2658,            471,            3328,            3565,            1659,            2416,            2021,            2520,            1282,            2128,            2705,            4379,            2208,            2665,            4149,            3057,            3516,            3477,            4374,            715,            3529,            2667,            2912,            4424,            4066,            4789,            3360,            3823,            4914,            2499,            1753,            3511,            2695,            2091,            0,            1532,            1891,            3761,            161,            1968,            2303,            4138,            3179,            1885,            2537,            3208,            3755,            2230,            1186,            3257,            3945,            1266,            2421,            ],        [            1627,            2316,            1041,            1155,            2476,            429,            916,            2126,            2021,            1162,            1302,            1503,            1854,            2342,            1038,            1998,            1617,            2067,            2814,            1728,            961,            2329,            1209,            2207,            200,            568,            2799,            856,            1743,            1486,            428,            2216,            228,            1884,            1239,            762,            467,            2274,            2851,            1345,            1186,            2687,            2252,            1354,            1680,            528,            1432,            318,            2042,            1588,            3631,            1091,            1548,            3032,            1940,            2399,            2729,            3257,            1113,            2412,            1550,            2164,            3676,            3318,            3672,            2243,            2706,            3797,            1900,            260,            2394,            2498,            1003,            1315,            0,            1699,            2644,            1476,            655,            990,            3021,            2238,            2402,            1449,            2091,            2638,            1113,            476,            2140,            3197,            266,            2083,            ],        [            1558,            1161,            1902,            2154,            1321,            1433,            869,            971,            2882,            2541,            2163,            348,            936,            3203,            2271,            2859,            1318,            912,            1462,            3353,            1932,            3934,            2834,            3068,            1831,            1853,            4032,            2089,            588,            331,            1661,            3449,            1489,            3117,            2100,            1029,            1769,            1119,            1696,            2648,            2488,            1532,            3485,            1668,            525,            1765,            1206,            1946,            1278,            2449,            2488,            1952,            2409,            2698,            2169,            3725,            1586,            2466,            1831,            3738,            2379,            1021,            2533,            2175,            2068,            1874,            4100,            2600,            745,            1497,            3295,            3731,            956,            2617,            1675,            0,            3970,            2778,            1888,            2022,            2663,            3471,            2039,            521,            2952,            3499,            1666,            2104,            3001,            2054,            1630,            3708,            ],        [            4696,            4830,            1930,            3578,            4990,            2852,            2954,            4640,            1858,            3965,            1887,            3541,            3670,            2392,            3333,            2297,            3875,            4581,            5131,            4777,            3384,            4996,            4258,            2141,            2952,            2326,            5094,            3151,            4257,            3970,            2782,            4511,            2560,            3964,            1864,            2800,            2890,            4788,            5365,            2603,            3609,            5201,            3380,            3776,            3718,            2552,            3940,            3067,            4947,            1425,            2669,            1980,            1453,            1587,            2210,            897,            2667,            1812,            3536,            967,            1820,            3262,            2945,            3256,            2227,            2945,            1272,            2352,            4414,            2486,            949,            4793,            2927,            3738,            2796,            4121,            0,            3899,            3009,            2332,            1576,            4533,            5319,            3487,            1875,            3454,            2357,            3225,            2439,            3391,            2751,            5302,            ],        [            1969,            2917,            2358,            1067,            3317,            1539,            2204,            3168,            3338,            1454,            2619,            2439,            3142,            3659,            2551,            3315,            1419,            2456,            2616,            2266,            920,            2867,            1747,            3524,            1653,            2309,            2965,            2369,            2848,            2422,            1941,            3227,            2038,            3714,            2556,            2050,            1605,            3115,            3692,            2858,            671,            3528,            3765,            1842,            2616,            2221,            2720,            1482,            2447,            2905,            4579,            2408,            2865,            4349,            3257,            3716,            3677,            4574,            915,            3729,            2867,            3112,            4624,            4266,            4989,            3560,            4023,            5114,            2699,            1953,            3711,            2895,            2291,            826,            1732,            2091,            3961,            0,            2168,            2503,            4338,            3379,            2204,            2737,            3408,            3955,            2430,            1386,            3457,            4145,            1466,            2621,            ],        [            2312,            2617,            1484,            1662,            2777,            1190,            1217,            2427,            2193,            1679,            1450,            1804,            2155,            1889,            383,            1545,            2138,            2368,            2918,            1323,            1468,            2046,            1316,            1754,            578,            1543,            2144,            201,            2044,            1787,            432,            1561,            529,            1229,            1682,            1063,            1228,            2575,            3152,            690,            1693,            2988,            1597,            2039,            1981,            1270,            1733,            749,            2734,            2031,            4557,            1534,            1991,            3475,            2748,            2791,            3030,            3700,            1620,            2855,            2358,            2465,            4833,            3619,            4115,            3051,            2600,            4240,            2201,            561,            2837,            1843,            1304,            1822,            715,            2384,            3087,            1983,            0,            1013,            3464,            1583,            2909,            1750,            2223,            2185,            1888,            907,            1687,            3498,            1027,            2352,            ],        [            2253,            2558,            599,            1782,            2718,            1056,            1158,            2368,            1636,            2169,            452,            1745,            2096,            1332,            1130,            988,            2079,            2309,            2859,            2981,            1588,            2793,            2462,            1197,            850,            530,            2891,            948,            1985,            1728,            680,            2308,            599,            1976,            797,            1004,            1094,            2516,            3093,            400,            1813,            2929,            1554,            1980,            1922,            257,            1674,            1021,            2675,            1146,            3672,            977,            1106,            2590,            1863,            2234,            2971,            2815,            1740,            1970,            1473,            2406,            3948,            3560,            3230,            2166,            2043,            3355,            2142,            525,            1952,            2590,            1245,            1942,            785,            2325,            2202,            2103,            907,            0,            2579,            2330,            2876,            1691,            1666,            1628,            1331,            1179,            1130,            3439,            955,            3099,            ],        [            3706,            3840,            2095,            3743,            4000,            3017,            3500,            3650,            2023,            4130,            2052,            3027,            2680,            2557,            3498,            2462,            3967,            3591,            4141,            4942,            3549,            5161,            4423,            2306,            3117,            2491,            5259,            3316,            3267,            2980,            2947,            4676,            2725,            4129,            2029,            2965,            3055,            3798,            4375,            2768,            3774,            4211,            3545,            3868,            2728,            2717,            2950,            3232,            3957,            1590,            1679,            2145,            1618,            597,            2250,            1062,            1677,            822,            3701,            1075,            1985,            2272,            1955,            2266,            1237,            1955,            1437,            1362,            3424,            2651,            632,            4958,            3297,            3903,            2961,            3778,            1307,            4064,            3174,            2497,            0,            4698,            4329,            3170,            2040,            3619,            2522,            3390,            2604,            2401,            2916,            5467,            ],        [            3426,            3731,            2409,            2606,            3891,            2304,            2331,            3541,            2497,            2246,            2375,            2918,            3269,            1736,            797,            1849,            3252,            3482,            4032,            1617,            2582,            1028,            1905,            2058,            1692,            2805,            1126,            1132,            3158,            2901,            1546,            543,            1643,            335,            2607,            2177,            2342,            3689,            4266,            1621,            2807,            4102,            1040,            3153,            3095,            1943,            2847,            1863,            3848,            2768,            5482,            2459,            2916,            4400,            3673,            3095,            4144,            4625,            2734,            3780,            3283,            3579,            5758,            4733,            5040,            3976,            2904,            5165,            3315,            1675,            3762,            825,            2418,            2936,            1829,            3498,            4012,            3097,            1376,            1944,            4389,            0,            4023,            2864,            2527,            1488,            2813,            2021,            1991,            4612,            2141,            1334,            ],        [            499,            1200,            2850,            1844,            1360,            2312,            1817,            1010,            3830,            2231,            3111,            1296,            1884,            4151,            3219,            3807,            994,            541,            708,            3043,            1693,            3644,            2524,            4016,            2426,            2801,            3742,            3037,            933,            1279,            2609,            4004,            2437,            4491,            3048,            1977,            1916,            1158,            1735,            3596,            1809,            1571,            4433,            1064,            1473,            2713,            2154,            2255,            243,            3397,            3436,            2900,            3357,            3646,            3117,            4673,            2534,            3414,            1525,            4686,            3327,            1969,            3481,            3123,            3016,            2822,            5048,            3548,            784,            2445,            4243,            3672,            1904,            1938,            2043,            948,            4918,            2099,            2836,            2970,            3611,            4156,            0,            1469,            3900,            4447,            2614,            2159,            3949,            3002,            1777,            3398,            ],        [            1200,            1334,            1381,            1633,            1494,            912,            348,            1144,            2361,            2020,            1642,            521,            1109,            2682,            1750,            2338,            1246,            1085,            1635,            2832,            1411,            3413,            2313,            2547,            1310,            1332,            3511,            1568,            761,            504,            1140,            2928,            968,            2596,            1579,            508,            1248,            1292,            1869,            2127,            1967,            1705,            2964,            1147,            698,            1244,            864,            1481,            1451,            1928,            2661,            1431,            1888,            2871,            1464,            2739,            1759,            2639,            1894,            2752,            1858,            1194,            2706,            2348,            2241,            1767,            3046,            2773,            918,            976,            2734,            3210,            435,            2096,            1154,            1492,            2984,            2257,            1367,            1501,            2836,            2950,            1823,            0,            2431,            2978,            1145,            1639,            2480,            2227,            1109,            3187,            ],        [            2865,            3170,            746,            2394,            3330,            1668,            1770,            2980,            432,            2781,            340,            2357,            2022,            966,            1923,            871,            2691,            2921,            3471,            3593,            2200,            3586,            3074,            715,            1768,            1142,            3684,            1741,            2597,            2340,            1598,            3101,            1376,            2554,            680,            1616,            1706,            3128,            3705,            1193,            2425,            3541,            1970,            2592,            2275,            1368,            1600,            1883,            3287,            241,            2835,            796,            269,            1753,            1026,            1109,            1699,            1978,            2352,            1133,            636,            2294,            3111,            2288,            2393,            1329,            1117,            2518,            2754,            1302,            1115,            3383,            1743,            2554,            1612,            2937,            1365,            2715,            1825,            817,            1742,            3123,            3488,            2303,            0,            2044,            1173,            2041,            1013,            2167,            1567,            3892,            ],        [            3601,            3906,            1817,            3204,            4066,            2479,            2506,            3716,            1905,            2844,            1783,            3093,            3070,            1144,            972,            1257,            3427,            3657,            4207,            2215,            2757,            1626,            2503,            1466,            1867,            2213,            1724,            1307,            3333,            3076,            1721,            1141,            1818,            510,            2015,            2352,            2517,            3864,            4441,            1796,            2982,            4277,            448,            3328,            3270,            1413,            2648,            2038,            4023,            2176,            4890,            1867,            2324,            3808,            3081,            2503,            3754,            4033,            2909,            3188,            2691,            3380,            5166,            4343,            4448,            3384,            2312,            4573,            3490,            1850,            3170,            1423,            2593,            3111,            2004,            3673,            3420,            3272,            1551,            1156,            3797,            1163,            4198,            3039,            1935,            0,            2221,            2196,            1399,            4222,            2316,            1932,            ],        [            1888,            2193,            693,            2361,            2353,            1380,            773,            2003,            1680,            2748,            954,            1380,            849,            1994,            2175,            1650,            1714,            1944,            2494,            3560,            1879,            3838,            3041,            1859,            1735,            1162,            3936,            1993,            1620,            1354,            1565,            3353,            1393,            2800,            623,            933,            1673,            2151,            2728,            1439,            2392,            2564,            2216,            1615,            1102,            1669,            427,            1906,            2310,            1247,            2626,            537,            1207,            2836,            903,            2058,            1724,            2604,            2319,            2071,            966,            1159,            2671,            2313,            2206,            1206,            2365,            2738,            1777,            1401,            2053,            3635,            570,            2521,            1579,            1960,            2303,            2682,            1792,            1063,            2801,            3375,            2511,            1326,            1750,            2290,            0,            2064,            1792,            2192,            1534,            3915,            ],        [            1960,            2908,            1667,            811,            3073,            848,            1513,            2723,            2647,            686,            1928,            2100,            2451,            3054,            1548,            2710,            1410,            2447,            2607,            1498,            754,            2099,            979,            2919,            582,            1618,            2197,            1366,            2340,            2083,            870,            2459,            967,            2946,            1865,            1359,            928,            2871,            3448,            1855,            979,            3284,            2762,            1670,            2277,            1267,            2029,            411,            2438,            2214,            4228,            1717,            2174,            3658,            2566,            3025,            3326,            3883,            906,            3038,            2176,            2761,            4273,            3915,            4298,            2869,            3332,            4423,            2497,            999,            3020,            2127,            1600,            1108,            729,            2015,            3270,            1269,            1165,            1432,            3647,            2611,            2195,            2046,            2717,            3350,            1739,            0,            2852,            3794,            727,            1853,            ],        [            3365,            3670,            1246,            2894,            3830,            2168,            2270,            3480,            902,            3326,            1061,            2857,            2499,            226,            1806,            238,            3191,            3421,            3971,            3479,            2700,            2890,            2963,            463,            1905,            1642,            2988,            1624,            3097,            2840,            1735,            2405,            1856,            1774,            1444,            2116,            2206,            3628,            4205,            1076,            2925,            4041,            1200,            3092,            3034,            1386,            2077,            2076,            3787,            1126,            3594,            1296,            1154,            2512,            1911,            1453,            2584,            2737,            2852,            1892,            1521,            3179,            3870,            3173,            3152,            2214,            1232,            3277,            3254,            1802,            1874,            2687,            2220,            3054,            2045,            3437,            2124,            3215,            1909,            1129,            2501,            2427,            3988,            2803,            885,            1264,            1650,            2234,            0,            3052,            2067,            3196,            ],        [            2475,            2609,            2364,            3568,            2769,            2876,            2269,            2419,            2400,            3955,            2429,            1796,            1449,            2934,            3671,            2839,            2736,            2360,            2910,            4767,            3417,            5334,            4248,            2683,            3231,            2868,            5432,            3489,            2036,            1749,            3061,            4849,            2889,            4517,            2093,            2429,            3169,            2567,            3144,            3145,            3533,            2980,            3922,            2637,            1497,            3165,            1719,            3402,            2726,            1967,            952,            2208,            1927,            1548,            1019,            2575,            684,            1316,            3249,            2588,            1700,            1041,            716,            358,            918,            724,            2950,            1450,            2193,            2897,            2145,            5131,            2066,            3662,            3075,            2547,            2820,            3823,            3288,            2874,            1513,            4871,            3098,            1939,            2470,            3996,            2204,            3560,            2981,            0,            3030,            5208,            ],        [            1361,            2228,            982,            889,            2388,            163,            828,            2038,            1962,            1276,            1243,            1415,            1766,            2283,            1331,            1939,            1351,            1979,            2548,            2088,            695,            2689,            1569,            2148,            435,            933,            2787,            1149,            1655,            1398,            723,            2509,            494,            2177,            1180,            674,            201,            2186,            2763,            1638,            920,            2599,            2545,            1088,            1592,            845,            1344,            316,            1776,            1529,            3543,            1032,            1489,            2973,            1881,            2340,            2641,            3198,            847,            2353,            1491,            2076,            3588,            3230,            3613,            2184,            2647,            3738,            1812,            577,            2335,            2717,            915,            1049,            266,            1433,            2585,            1210,            948,            1285,            2962,            2531,            2136,            1361,            2032,            2579,            1054,            474,            2081,            3109,            0,            2443,            ],        [            2690,            3638,            3079,            1272,            4038,            2260,            2925,            3889,            3724,            912,            3340,            3160,            3748,            2963,            2024,            3076,            2140,            3177,            3337,            283,            1641,            715,            571,            3285,            1563,            3030,            938,            1238,            3569,            3143,            1851,            1075,            1948,            1562,            3277,            2771,            1909,            3836,            4413,            1727,            1704,            4249,            2267,            2563,            3337,            2248,            3441,            1444,            3168,            3626,            5396,            3129,            3586,            5070,            3978,            4322,            4494,            5295,            1636,            4450,            3588,            3833,            5441,            5083,            5710,            4281,            4131,            5508,            3420,            1980,            4432,            743,            3012,            1833,            1710,            2812,            4682,            1994,            1768,            2050,            5059,            1227,            2925,            3333,            3754,            2715,            3151,            1602,            3218,            4962,            1708,            0,            ],        ], 'starts': [24, 50, 29, 33, 10], 'ends': [24, 50, 29, 33, 10]},    'NW-31': {'distance_matrix': [        [            0,            3476,            3746,            1551,            2020,            2741,            2700,            2018,            2698,            1859,            2972,            2073,            1065,            811,            2700,            3182,            383,            857,            2686,            1870,            1256,            1232,            2730,            1528,            2597,            4187,            4554,            3886,            4440,            3015,            2089,            1664,            2080,            2513,            2439,            2284,            2738,            3037,            1426,            1690,            2379,            2381,            1758,            3306,            2250,            1941,            1365,            1108,            2250,            1732,            392,            696,            1011,            3777,            1972,            2109,            972,            1344,            2801,            3435,            1041,            3255,            2597,            2941,            2258,            3694,            2769,            1184,            1897,            1613,            3516,            3338,            3451,            ],        [            3199,            0,            270,            2578,            1855,            1307,            1345,            2601,            2345,            1708,            771,            2370,            4264,            4010,            984,            491,            3007,            2983,            933,            1915,            3526,            3502,            1298,            2157,            4376,            4125,            6116,            4542,            4378,            5527,            3279,            2854,            2610,            3358,            2969,            2814,            3688,            3987,            3655,            2880,            1540,            3331,            3590,            1105,            1897,            2006,            2429,            2754,            1458,            2296,            3469,            3157,            2939,            4662,            3162,            2639,            3305,            2120,            4580,            4726,            2820,            586,            3547,            2588,            4037,            4439,            3719,            2373,            2427,            2061,            847,            669,            782,            ],        [            2929,            537,            0,            2308,            1585,            1037,            1075,            2331,            2075,            1438,            501,            2100,            3994,            3740,            714,            221,            2737,            2713,            663,            1645,            3256,            3232,            1028,            1887,            4106,            3855,            5846,            4272,            4108,            5257,            3009,            2584,            2340,            3088,            2699,            2544,            3418,            3717,            3385,            2610,            1270,            3061,            3320,            835,            1627,            1736,            2159,            2484,            1188,            2026,            3199,            2887,            2669,            4392,            2892,            2369,            3035,            1850,            4310,            4456,            2550,            316,            3277,            2318,            3767,            4169,            3449,            2103,            2157,            1791,            577,            399,            512,            ],        [            1593,            2816,            3086,            0,            1816,            1938,            2928,            2812,            3492,            1199,            2312,            2867,            1741,            1487,            2567,            2522,            1478,            1651,            2026,            2546,            2541,            2517,            2881,            2322,            3391,            4981,            5348,            4680,            5234,            3691,            982,            557,            1420,            1406,            1779,            1279,            1631,            1930,            1077,            583,            3123,            1274,            1293,            2646,            3044,            2637,            2159,            1902,            2089,            2257,            891,            579,            361,            4571,            865,            1449,            2087,            2081,            3595,            4229,            1835,            2595,            1490,            3735,            3052,            4488,            1662,            1978,            1237,            953,            2856,            2678,            2791,            ],        [            1344,            2334,            2604,            1806,            0,            1434,            1470,            1759,            2439,            936,            1802,            1814,            2409,            2155,            1393,            1792,            1152,            781,            1544,            1114,            1671,            1647,            1423,            1052,            2521,            4111,            4478,            3810,            4364,            3672,            2507,            2082,            1838,            2586,            2197,            2042,            2916,            3215,            2770,            2108,            1665,            2559,            2818,            2136,            2022,            1465,            1106,            849,            943,            441,            1736,            2040,            2167,            3701,            2390,            1867,            1450,            265,            2725,            3359,            965,            2113,            2775,            2682,            2182,            3618,            2947,            708,            1655,            1289,            2374,            2196,            2309,            ],        [            1892,            1587,            1857,            1271,            548,            0,            2018,            2307,            2987,            401,            1083,            2362,            2957,            2703,            1941,            1293,            1700,            1329,            797,            1662,            2219,            2195,            1971,            1600,            3069,            4659,            5026,            4358,            4912,            4220,            1972,            1547,            1303,            2051,            1662,            1507,            2381,            2680,            2348,            1573,            2213,            2024,            2283,            1417,            2570,            2013,            1654,            1397,            860,            989,            2162,            1850,            1632,            4249,            1855,            1332,            1998,            813,            3273,            3907,            1513,            1366,            2240,            3230,            2730,            4166,            2412,            1256,            1120,            754,            1627,            1449,            1562,            ],        [            2391,            1235,            1505,            2538,            1815,            1267,            0,            1567,            1237,            1668,            731,            1622,            3456,            3202,            322,            721,            2199,            1949,            893,            881,            2718,            2694,            190,            1123,            3568,            3664,            5655,            4081,            3917,            4719,            3239,            2814,            2570,            3318,            2929,            2774,            3648,            3947,            3615,            2840,            432,            3291,            3550,            1065,            789,            972,            1395,            1720,            760,            1519,            2783,            3087,            2899,            4201,            3122,            2599,            2497,            1343,            3772,            4406,            2012,            1014,            3507,            1480,            3229,            3978,            3679,            1339,            2387,            2021,            1275,            1097,            1210,            ],        [            2031,            1917,            2187,            3220,            2497,            1949,            682,            0,            680,            2350,            1413,            705,            3096,            2842,            1004,            1403,            1839,            1372,            1575,            1018,            2262,            2238,            595,            1050,            3112,            3107,            5098,            3524,            3360,            4263,            3921,            3496,            3252,            4000,            3611,            3456,            4330,            4629,            3457,            3522,            361,            3973,            4232,            1747,            232,            1168,            818,            1143,            1442,            2201,            2423,            2727,            3042,            3644,            3804,            3281,            2041,            2025,            3316,            3708,            1556,            1696,            4189,            923,            2773,            3421,            4361,            1266,            3069,            2703,            1957,            1779,            1892,            ],        [            2279,            2506,            2776,            3069,            2764,            2216,            1271,            672,            0,            2617,            2002,            430,            3344,            3090,            1593,            1992,            2087,            1620,            2164,            1345,            2510,            2486,            1184,            1298,            3360,            3696,            5317,            4649,            3949,            4511,            3770,            3345,            3519,            4267,            3878,            3723,            4419,            4718,            3705,            3371,            950,            4062,            4081,            2336,            398,            1416,            1066,            1391,            1725,            2484,            2671,            2975,            3290,            4233,            3653,            3548,            2289,            2308,            3564,            4198,            1804,            2285,            4278,            1512,            3021,            4010,            4450,            1514,            3336,            2970,            2546,            2368,            2481,            ],        [            1550,            2250,            2520,            870,            1250,            1372,            2362,            2769,            3449,            0,            1746,            2824,            2615,            2361,            2001,            1956,            1358,            1608,            1460,            1980,            2498,            2474,            2315,            2279,            3348,            4938,            5305,            4637,            5191,            4499,            1571,            1146,            902,            1650,            1261,            1106,            1980,            2279,            1947,            1172,            2557,            1623,            1882,            2080,            3001,            2071,            2116,            1859,            1523,            1691,            1761,            1449,            1231,            4528,            1454,            931,            2277,            1515,            3552,            4186,            1792,            2029,            1839,            3692,            3009,            4445,            2011,            1935,            719,            353,            2290,            2112,            2225,            ],        [            2880,            914,            1184,            2259,            1536,            988,            1026,            1764,            2026,            1389,            0,            2051,            3945,            3691,            665,            630,            2688,            2615,            614,            1547,            3207,            3183,            979,            1789,            4057,            4232,            6223,            4649,            4485,            5208,            2874,            2535,            2291,            3039,            2650,            2495,            3369,            3668,            3336,            2561,            1221,            3012,            3271,            334,            1578,            1638,            2061,            2386,            1139,            1977,            3150,            2838,            2620,            4769,            2843,            2320,            2986,            1801,            4261,            4895,            2501,            481,            3228,            2269,            3718,            4546,            3400,            2005,            2108,            1742,            727,            472,            631,            ],        [            1849,            2686,            2956,            2639,            2334,            1786,            1459,            777,            1457,            2187,            2190,            0,            2914,            2660,            1781,            2180,            1657,            1190,            1896,            915,            2080,            2056,            1372,            868,            2930,            3884,            4887,            4219,            4137,            4081,            3254,            2915,            3089,            3837,            3448,            3293,            3989,            4288,            3275,            2941,            1138,            3632,            3651,            2524,            756,            986,            636,            961,            1295,            2054,            2241,            2545,            2860,            4421,            3223,            3118,            1859,            1878,            3134,            3768,            1374,            2465,            3848,            1700,            2591,            4198,            4020,            1084,            2906,            2540,            2726,            2548,            2661,            ],        [            1635,            4516,            4786,            2030,            3060,            3781,            3740,            3058,            3738,            2899,            4012,            3113,            0,            610,            3740,            4222,            1423,            1897,            3726,            2910,            2783,            2759,            3770,            2568,            3637,            5080,            5447,            4772,            5333,            2614,            2602,            2263,            3120,            3112,            3479,            3324,            3337,            3636,            1389,            2289,            3419,            2980,            2357,            4346,            3290,            2981,            2405,            2148,            3290,            2772,            991,            1295,            1610,            4663,            2571,            3149,            2499,            2384,            3687,            4321,            2081,            4295,            3196,            3981,            3144,            4580,            3368,            2224,            2937,            2653,            4556,            4378,            4491,            ],        [            1025,            3906,            4176,            1420,            2450,            3171,            3130,            2448,            3128,            2289,            3402,            2503,            623,            0,            3130,            3612,            813,            1287,            3116,            2300,            1706,            1682,            3160,            1958,            3027,            4617,            4984,            4316,            4870,            2573,            1992,            1653,            2510,            2502,            2869,            2714,            2727,            3026,            1415,            1679,            2809,            2370,            1747,            3736,            2680,            2371,            1795,            1538,            2680,            2162,            381,            685,            1000,            4207,            1961,            2539,            1422,            1774,            3231,            3865,            1471,            3685,            2586,            3371,            2688,            4124,            2758,            1614,            2327,            2043,            3946,            3768,            3881,            ],        [            2593,            913,            1183,            2216,            1493,            945,            562,            1300,            1562,            1346,            409,            1587,            3658,            3404,            0,            399,            2401,            2151,            571,            1083,            2920,            2896,            515,            1325,            3770,            4231,            6222,            4648,            4484,            4921,            2831,            2492,            2248,            2996,            2607,            2452,            3326,            3625,            3293,            2518,            757,            2969,            3228,            743,            1114,            1174,            1597,            1922,            1096,            1721,            3107,            2795,            2577,            4768,            2800,            2277,            2699,            1758,            3974,            4608,            2214,            692,            3185,            1805,            3431,            4545,            3357,            1541,            2065,            1699,            953,            775,            888,            ],        [            2708,            784,            577,            2087,            1364,            816,            854,            2110,            1854,            1217,            280,            1879,            3773,            3519,            493,            0,            2516,            2492,            442,            1424,            3035,            3011,            807,            1666,            3885,            4281,            5842,            4698,            4534,            5036,            2702,            2363,            2119,            2867,            2478,            2323,            3197,            3496,            3164,            2389,            1049,            2840,            3099,            614,            1406,            1515,            1938,            2263,            967,            1805,            2978,            2666,            2448,            4818,            2671,            2148,            2814,            1629,            4089,            4723,            2329,            573,            3056,            2097,            3546,            4595,            3228,            1882,            1936,            1570,            824,            656,            759,            ],        [            416,            3093,            3363,            1168,            1637,            2358,            2317,            1635,            2315,            1476,            2589,            1690,            1481,            1227,            2317,            2799,            0,            474,            2303,            1487,            1364,            1340,            2347,            1145,            2214,            3804,            4229,            3503,            4057,            3365,            1783,            1444,            1697,            2293,            2056,            1901,            2518,            2817,            1842,            1470,            1996,            2161,            2180,            2923,            1867,            1558,            982,            725,            1867,            1349,            808,            1112,            1529,            3394,            1752,            1726,            1143,            961,            2418,            3052,            658,            2872,            2377,            2558,            1875,            3311,            2549,            801,            1514,            1230,            3133,            2955,            3068,            ],        [            659,            2691,            2961,            1449,            1279,            2000,            1959,            1277,            1957,            1486,            2187,            1332,            1724,            1470,            1959,            2397,            467,            0,            1901,            1129,            1006,            982,            1989,            787,            1856,            3504,            3871,            3145,            3757,            3007,            2064,            1725,            1978,            2574,            2337,            2182,            2799,            3098,            2085,            1751,            1638,            2442,            2461,            2521,            1509,            1200,            624,            367,            1509,            991,            1051,            1355,            1670,            3036,            2033,            2007,            785,            603,            2060,            2694,            300,            2470,            2658,            2200,            1517,            2953,            2830,            443,            1795,            1839,            2731,            2553,            2666,            ],        [            2266,            1125,            1395,            1645,            922,            374,            1338,            1668,            2348,            775,            621,            1723,            3331,            3077,            1261,            831,            2074,            2050,            0,            982,            2593,            2569,            1291,            1224,            3443,            4443,            5458,            4732,            4696,            4594,            2260,            1921,            1677,            2425,            2036,            1881,            2755,            3054,            2722,            1947,            1533,            2398,            2657,            955,            1890,            1073,            1496,            1821,            525,            1363,            2536,            2224,            2006,            4623,            2229,            1706,            2372,            1187,            3647,            4281,            1887,            904,            2614,            2591,            3104,            4540,            2786,            1440,            1494,            1128,            1165,            987,            1100,            ],        [            1510,            2060,            2330,            2807,            1730,            1160,            1368,            686,            1366,            1937,            1528,            741,            2575,            2321,            1119,            1518,            1318,            1068,            1270,            0,            1837,            1813,            1149,            242,            2687,            3793,            4702,            3976,            4046,            3838,            3422,            3083,            2839,            3587,            3198,            3043,            3917,            4216,            2936,            3109,            1047,            3560,            3819,            1862,            918,            360,            514,            839,            669,            1442,            1902,            2206,            2521,            4330,            3391,            2868,            1616,            1054,            2891,            3525,            1131,            1839,            3776,            1609,            2348,            3784,            3948,            458,            2656,            2290,            2100,            1922,            2035,            ],        [            963,            3710,            3980,            1753,            1958,            2810,            2554,            1872,            2552,            2165,            3178,            1927,            2028,            1774,            2769,            3168,            771,            679,            2920,            1939,            0,            295,            2799,            1466,            2135,            3783,            4150,            2047,            2243,            2002,            2368,            2029,            2282,            2878,            2641,            2486,            3103,            3402,            2389,            2055,            2233,            2746,            2765,            3512,            2104,            2010,            1219,            962,            2319,            1670,            1355,            1659,            1974,            1938,            2337,            2311,            78,            1282,            2339,            2973,            455,            3489,            2962,            2795,            1796,            2138,            3134,            1122,            2099,            2518,            3750,            3572,            3685,            ],        [            987,            3734,            4004,            1777,            1982,            2834,            2578,            1896,            2576,            2189,            3202,            1951,            2052,            1798,            2793,            3192,            795,            703,            2944,            1963,            24,            0,            2823,            1490,            2159,            3807,            4174,            2071,            2267,            2026,            2392,            2053,            2306,            2902,            2665,            2510,            3127,            3426,            2413,            2079,            2257,            2770,            2789,            3536,            2128,            2034,            1243,            986,            2343,            1694,            1379,            1683,            1998,            1962,            2361,            2335,            102,            1306,            2363,            2997,            479,            3513,            2986,            2819,            1820,            2162,            3158,            1146,            2123,            2542,            3774,            3596,            3709,            ],        [            2398,            1471,            1741,            2774,            2051,            1503,            236,            785,            1047,            1904,            967,            1072,            3463,            3209,            558,            957,            2206,            1739,            1129,            633,            2629,            2605,            0,            875,            3479,            3474,            5465,            3891,            3727,            4630,            3389,            3050,            2806,            3554,            3165,            3010,            3884,            4183,            3824,            3076,            242,            3527,            3786,            1301,            599,            724,            1185,            1510,            996,            1755,            2790,            3094,            3135,            4011,            3358,            2835,            2408,            1579,            3683,            4317,            1923,            1250,            3743,            1290,            3140,            3788,            3915,            1091,            2623,            2257,            1511,            1333,            1446,            ],        [            1473,            2113,            2383,            2484,            1761,            1213,            910,            757,            1437,            1614,            1581,            812,            2538,            2284,            1172,            1571,            1281,            1139,            1323,            342,            2029,            2005,            1202,            0,            2879,            3864,            4894,            4168,            4117,            4030,            3099,            2760,            2516,            3264,            2875,            2720,            3594,            3893,            2899,            2786,            1444,            3237,            3496,            1915,            989,            413,            585,            910,            722,            1481,            1865,            2169,            2845,            4401,            3068,            2545,            1808,            1305,            3083,            3717,            1323,            1892,            3453,            1680,            2540,            3976,            3625,            421,            2333,            1967,            2153,            1975,            2088,            ],        [            3496,            5868,            6138,            4286,            4116,            4968,            4712,            4030,            4710,            4323,            5336,            4085,            2419,            2471,            4927,            5326,            3304,            2837,            5078,            4097,            2288,            2264,            4957,            3624,            0,            3668,            4035,            3309,            3921,            1151,            4901,            4562,            4815,            5411,            5174,            5019,            5636,            5935,            4922,            4588,            4391,            5279,            5298,            5670,            4262,            4168,            3377,            3120,            4477,            3828,            3888,            4192,            4507,            3200,            4870,            4844,            2067,            3440,            2224,            2858,            2737,            5647,            5495,            4953,            1681,            3117,            5667,            3280,            4632,            4676,            5908,            5730,            5843,            ],        [            4081,            3355,            3625,            5933,            5210,            4662,            3596,            3312,            3330,            5063,            4126,            3599,            4945,            4997,            3918,            3846,            3889,            3422,            4288,            3991,            2652,            2628,            3509,            3944,            2533,            0,            2034,            417,            253,            3677,            6548,            6209,            5965,            6713,            6324,            6169,            7043,            7342,            5507,            6235,            3275,            6686,            6945,            4460,            3126,            4062,            3712,            4037,            4371,            5130,            4473,            4777,            5092,            537,            6517,            5994,            2431,            4954,            1481,            655,            3322,            3941,            6902,            2413,            1989,            368,            7074,            4160,            5782,            5416,            3553,            4024,            3650,            ],        [            3983,            6355,            6625,            4773,            4603,            5455,            5199,            4517,            5197,            4810,            5823,            4572,            2906,            2958,            5414,            5813,            3791,            3324,            5565,            4584,            2775,            2751,            5444,            4111,            487,            4155,            0,            3796,            4408,            1638,            5388,            5049,            5302,            5898,            5661,            5506,            6123,            6422,            5409,            5075,            4878,            5766,            5785,            6157,            4749,            4655,            3864,            3607,            4964,            4315,            4375,            4679,            4994,            3687,            5357,            5331,            2554,            3927,            2711,            3345,            3224,            6134,            5982,            5440,            2168,            3604,            6154,            3767,            5119,            5163,            6395,            6217,            6330,            ],        [            3774,            3779,            4049,            4564,            4394,            5086,            4020,            3736,            3754,            4601,            4550,            4023,            5369,            4585,            4342,            4270,            3582,            3115,            4712,            4415,            2345,            2321,            3933,            3902,            2957,            424,            2458,            0,            333,            4101,            5179,            4840,            5093,            5689,            5452,            5297,            5914,            6213,            5200,            4866,            3699,            5557,            5576,            4884,            3550,            4486,            3655,            3398,            4795,            4106,            4166,            4470,            4785,            230,            5148,            5122,            2124,            3718,            1440,            614,            3015,            4365,            5773,            2837,            1682,            344,            5945,            3558,            4910,            4954,            3977,            4448,            4074,            ],        [            3828,            3745,            4015,            4618,            4448,            5052,            3986,            3702,            3720,            4655,            4516,            3989,            5335,            4639,            4308,            4236,            3636,            3169,            4678,            4429,            2399,            2375,            3899,            3956,            2923,            390,            2424,            164,            0,            4067,            5233,            4894,            5147,            5743,            5506,            5351,            5968,            6267,            5254,            4920,            3665,            5611,            5630,            4850,            3516,            4500,            3709,            3452,            4809,            4160,            4220,            4524,            4839,            284,            5202,            5176,            2178,            3772,            1494,            668,            3069,            4331,            5827,            2803,            1736,            398,            5999,            3612,            4964,            5008,            3943,            4414,            4040,            ],        [            2745,            5117,            5387,            3535,            3365,            4217,            3961,            3279,            3959,            3572,            4585,            3334,            2455,            2507,            4176,            4575,            2553,            2086,            4327,            3346,            2692,            2668,            4206,            2873,            2386,            2917,            3233,            2558,            3119,            0,            4150,            3811,            4064,            4660,            4423,            4268,            4885,            5184,            4171,            3837,            3640,            4528,            4547,            4919,            3511,            3417,            2626,            2369,            3726,            3077,            3137,            3441,            3756,            2449,            4119,            4093,            2471,            2689,            1473,            2107,            1986,            4896,            4744,            4202,            930,            2366,            4916,            2529,            3881,            3925,            5157,            4979,            5092,            ],        [            1976,            3199,            3469,            716,            2199,            2321,            3311,            3195,            3875,            1582,            2695,            3250,            2457,            2203,            2950,            2905,            1784,            2034,            2409,            2929,            2924,            2900,            3264,            2705,            3774,            5364,            5731,            5063,            5617,            4925,            0,            687,            897,            661,            1256,            534,            886,            1185,            1413,            713,            3506,            529,            951,            3029,            3427,            3020,            2542,            2285,            2472,            2640,            1607,            1295,            1077,            4954,            158,            926,            2703,            2464,            3978,            4612,            2218,            2978,            745,            4118,            3435,            4871,            917,            2361,            833,            1281,            3239,            3061,            3174,            ],        [            1793,            3016,            3286,            533,            2016,            2138,            3128,            3012,            3692,            1399,            2512,            3067,            2274,            2020,            2767,            2722,            1601,            1851,            2226,            2746,            2741,            2717,            3081,            2522,            3591,            5181,            5548,            4880,            5434,            4742,            339,            0,            1085,            849,            1238,            722,            1074,            1373,            1563,            26,            3323,            717,            736,            2846,            3244,            2837,            2359,            2102,            2289,            2457,            1424,            1112,            894,            4771,            308,            1114,            2520,            2281,            3795,            4429,            2035,            2795,            933,            3935,            3252,            4688,            1105,            2178,            793,            1469,            3056,            2878,            2991,            ],        [            2053,            2747,            3017,            1373,            1747,            1869,            2859,            3163,            3859,            1130,            2243,            3218,            3118,            2864,            2498,            2453,            1861,            2111,            1957,            2477,            3001,            2977,            2812,            2799,            3851,            5441,            5808,            5140,            5694,            5002,            1134,            1649,            0,            748,            359,            772,            1267,            1489,            2358,            1675,            3054,            910,            1577,            2577,            3411,            2568,            2619,            2596,            2020,            2188,            2264,            1952,            1734,            5031,            1103,            29,            2780,            2012,            4055,            4689,            2295,            2526,            1126,            4086,            3512,            4948,            1298,            2455,            351,            856,            2787,            2609,            2722,            ],        [            2724,            3418,            3688,            1252,            2418,            2540,            3530,            3834,            4530,            1801,            2914,            3889,            2993,            2739,            3169,            3124,            2532,            2782,            2628,            3148,            3672,            3648,            3483,            3470,            4522,            6112,            6479,            5811,            6365,            5673,            725,            1223,            880,            0,            389,            821,            780,            1079,            1949,            1249,            3725,            423,            1147,            3248,            4082,            3239,            3290,            3267,            2691,            2859,            2143,            1831,            1613,            5702,            694,            750,            3451,            2683,            4726,            5360,            2966,            3197,            639,            4757,            4183,            5619,            811,            3126,            1022,            1527,            3458,            3280,            3393,            ],        [            2335,            3029,            3299,            1655,            2029,            2151,            3141,            3445,            4141,            1412,            2525,            3500,            3400,            3146,            2780,            2735,            2143,            2393,            2239,            2759,            3283,            3259,            3094,            3081,            4133,            5723,            6090,            5422,            5976,            5284,            1114,            1931,            491,            389,            0,            1020,            1169,            1352,            2338,            1957,            3336,            812,            1536,            2859,            3693,            2850,            2901,            2878,            2302,            2470,            2546,            2234,            2016,            5313,            1083,            361,            3062,            2294,            4337,            4971,            2577,            2808,            1028,            4368,            3794,            5230,            1200,            2737,            633,            1138,            3069,            2891,            3004,            ],        [            2149,            2935,            3205,            889,            1935,            2057,            3047,            3368,            4048,            1318,            2431,            3423,            2630,            2376,            2686,            2641,            1957,            2207,            2145,            2665,            3097,            3073,            3000,            2987,            3947,            5537,            5904,            5236,            5790,            5098,            362,            860,            660,            645,            1019,            0,            870,            1169,            1586,            886,            3242,            513,            759,            2765,            3599,            2756,            2715,            2458,            2208,            2376,            1780,            1468,            1250,            5127,            331,            689,            2876,            2200,            4151,            4785,            2391,            2714,            729,            4291,            3608,            5044,            901,            2643,            596,            1044,            2975,            2797,            2910,            ],        [            3300,            3994,            4264,            1658,            2994,            3116,            4106,            4410,            5106,            2377,            3490,            4465,            3399,            3145,            3745,            3700,            3108,            3358,            3204,            3724,            4248,            4224,            4059,            4046,            5098,            6688,            7055,            6387,            6941,            6249,            1131,            1629,            1456,            1009,            1199,            1227,            0,            299,            2434,            1655,            4301,            829,            1559,            3824,            4658,            3815,            3866,            3843,            3267,            3435,            2549,            2237,            2019,            6278,            1100,            1326,            4027,            3259,            5302,            5936,            3542,            3773,            427,            5333,            4759,            6195,            579,            3702,            1598,            2103,            4034,            3856,            3969,            ],        [            3266,            4213,            4483,            2006,            3213,            3335,            4325,            4629,            5325,            2596,            3709,            4684,            3747,            3493,            3964,            3919,            3074,            3324,            3423,            3943,            4214,            4190,            4278,            4265,            5064,            6654,            7021,            6353,            6907,            6215,            1594,            1977,            1938,            1252,            1547,            1575,            569,            0,            2135,            2003,            4520,            1177,            1260,            4043,            4877,            4034,            3832,            3575,            3486,            3654,            2897,            2585,            2040,            6244,            1448,            1674,            3993,            3478,            5268,            5902,            3508,            3992,            775,            5552,            4725,            6161,            927,            3921,            1946,            2322,            4253,            4075,            4188,            ],        [            2112,            3335,            3605,            843,            2335,            2457,            3447,            3331,            4011,            1718,            2831,            3386,            1827,            1573,            3086,            3041,            1564,            2170,            2545,            3065,            3060,            3036,            3400,            2841,            3910,            5500,            5867,            5199,            5753,            3777,            1184,            1076,            1939,            1925,            2298,            1798,            2150,            2449,            0,            1102,            3642,            1451,            850,            3165,            3563,            3156,            2678,            2421,            2608,            2776,            977,            665,            423,            5090,            1384,            1968,            2173,            2600,            4114,            4748,            2354,            3114,            2009,            4254,            3571,            5007,            2181,            2497,            1756,            1472,            3375,            3197,            3310,            ],        [            1767,            2990,            3260,            507,            1990,            2112,            3102,            2986,            3666,            1373,            2486,            3041,            2248,            1994,            2741,            2696,            1575,            1825,            2200,            2720,            2715,            2691,            3055,            2496,            3565,            5155,            5522,            4854,            5408,            4716,            313,            478,            1059,            823,            1212,            696,            1048,            1347,            1537,            0,            3297,            691,            710,            2820,            3218,            2811,            2333,            2076,            2263,            2431,            1398,            1086,            868,            4745,            282,            1088,            2494,            2255,            3769,            4403,            2009,            2769,            907,            3909,            3226,            4662,            1079,            2152,            767,            1443,            3030,            2852,            2965,            ],        [            2156,            1681,            1951,            2984,            2261,            1713,            446,            543,            805,            2114,            1177,            830,            3221,            2967,            768,            1167,            1964,            1497,            1339,            782,            2387,            2363,            359,            1175,            3237,            3232,            5223,            3649,            3485,            4388,            3599,            3260,            3016,            3764,            3375,            3220,            4094,            4393,            3582,            3286,            0,            3737,            3996,            1511,            357,            1293,            943,            1268,            1206,            1965,            2548,            2852,            3167,            3769,            3568,            3045,            2166,            1789,            3441,            4075,            1681,            1460,            3953,            1048,            2898,            3546,            4125,            1391,            2833,            2467,            1721,            1543,            1656,            ],        [            2436,            3383,            3653,            1176,            2383,            2505,            3495,            3799,            4495,            1766,            2879,            3854,            2917,            2663,            3134,            3089,            2244,            2494,            2593,            3113,            3384,            3360,            3448,            3435,            4234,            5824,            6191,            5523,            6077,            5385,            721,            1147,            1108,            422,            811,            745,            647,            946,            1569,            1173,            3690,            0,            732,            3213,            4047,            3204,            3002,            2745,            2656,            2824,            2067,            1755,            1537,            5414,            618,            932,            3163,            2648,            4438,            5072,            2678,            3162,            506,            4722,            3895,            5331,            678,            3091,            1044,            1492,            3423,            3245,            3358,            ],        [            2310,            3533,            3803,            1050,            2533,            2655,            3645,            3529,            4209,            1916,            3029,            3584,            2791,            2537,            3284,            3239,            2118,            2368,            2743,            3263,            3258,            3234,            3598,            3039,            4108,            5698,            6065,            5397,            5951,            5259,            334,            1021,            1231,            1005,            1394,            868,            1230,            1529,            1182,            1047,            3840,            601,            0,            3363,            3761,            3354,            2876,            2619,            2806,            2974,            1941,            1629,            1411,            5288,            323,            1260,            3037,            2798,            4312,            4946,            2552,            3312,            1089,            4452,            3769,            5205,            1261,            2695,            1167,            1615,            3573,            3395,            3508,            ],        [            3218,            804,            1074,            2597,            1874,            1326,            1364,            2620,            2364,            1727,            468,            2389,            4283,            4029,            1003,            510,            3026,            3002,            952,            1934,            3545,            3521,            1317,            2176,            4395,            4122,            6113,            4539,            4375,            5546,            3212,            2873,            2629,            3377,            2988,            2833,            3707,            4006,            3674,            2899,            1559,            3350,            3609,            0,            1916,            2025,            2448,            2773,            1477,            2315,            3488,            3176,            2958,            4659,            3181,            2658,            3324,            2139,            4599,            4723,            2839,            147,            3566,            2784,            4056,            4436,            3738,            2392,            2446,            2080,            534,            278,            437,            ],        [            1799,            2103,            2373,            2589,            2284,            1736,            868,            186,            881,            2137,            1599,            473,            2864,            2610,            1190,            1589,            1607,            1140,            1846,            865,            2030,            2006,            781,            818,            2880,            3293,            4837,            4169,            3546,            4031,            3204,            2865,            3039,            3787,            3398,            3243,            3939,            4238,            3225,            2891,            547,            3582,            3601,            1933,            0,            936,            586,            911,            1245,            2004,            2191,            2495,            2810,            3830,            3173,            3068,            1809,            1828,            3084,            3718,            1324,            1882,            3798,            1109,            2541,            3607,            3970,            1034,            2856,            2490,            2143,            1965,            2078,            ],        [            1990,            1700,            1970,            2071,            1348,            800,            836,            1166,            1846,            1201,            1168,            1221,            3055,            2801,            759,            1158,            1798,            1548,            910,            480,            2317,            2293,            789,            722,            3167,            4273,            5124,            4456,            4526,            4318,            2686,            2347,            2103,            2851,            2462,            2307,            3181,            3480,            3148,            2373,            1031,            2824,            3083,            1502,            1388,            0,            994,            1319,            309,            1068,            2382,            2686,            2432,            4810,            2655,            2132,            2096,            892,            3371,            4005,            1611,            1479,            3040,            2089,            2828,            4264,            3212,            938,            1920,            1554,            1740,            1562,            1675,            ],        [            1233,            2491,            2761,            2023,            2139,            1591,            1335,            653,            1333,            1992,            1959,            708,            2298,            2044,            1550,            1949,            1041,            574,            1701,            720,            1464,            1440,            1580,            673,            2314,            3760,            4271,            3603,            4013,            3465,            2638,            2299,            2552,            3148,            2911,            2756,            3373,            3672,            2659,            2325,            1014,            3016,            3035,            2293,            885,            791,            0,            345,            1100,            1565,            1625,            1929,            2244,            3494,            2607,            2581,            1243,            1177,            2518,            3152,            758,            2270,            3232,            1576,            1975,            3411,            3404,            889,            2369,            2345,            2531,            2353,            2466,            ],        [            888,            2920,            3190,            1678,            1508,            2229,            2104,            1422,            2102,            1715,            2416,            1477,            1953,            1699,            2188,            2626,            696,            229,            2130,            1358,            1119,            1095,            2218,            1016,            1969,            3559,            3926,            3258,            3812,            3120,            2293,            1954,            2207,            2803,            2566,            2411,            3028,            3327,            2314,            1980,            1783,            2671,            2690,            2750,            1654,            1429,            769,            0,            1738,            1220,            1280,            1584,            1899,            3149,            2262,            2236,            898,            832,            2173,            2807,            413,            2699,            2887,            2345,            1630,            3066,            3059,            672,            2024,            2068,            2960,            2782,            2895,            ],        [            1967,            1727,            1997,            2098,            1375,            827,            813,            1143,            1823,            1228,            1145,            1198,            3032,            2778,            736,            1135,            1775,            1525,            937,            457,            2294,            2270,            766,            699,            3144,            4250,            5101,            4433,            4503,            4295,            2713,            2374,            2130,            2878,            2489,            2334,            3208,            3507,            3393,            2400,            1008,            2851,            3110,            1479,            1365,            548,            971,            1296,            0,            1095,            2359,            2663,            2459,            4787,            2682,            2159,            2073,            919,            3348,            3982,            1588,            1506,            3067,            2066,            2805,            4241,            3239,            915,            1947,            1581,            1767,            1589,            1702,            ],        [            1632,            1700,            1970,            1365,            288,            822,            1812,            2047,            2727,            495,            1196,            2102,            2697,            2443,            1681,            1406,            1440,            1069,            910,            1402,            1959,            1935,            1765,            1340,            2809,            4399,            4766,            4098,            4652,            3960,            1980,            1641,            1397,            2145,            1756,            1601,            2475,            2774,            2442,            1667,            2007,            2118,            2377,            1530,            2364,            1753,            1394,            1137,            973,            0,            2256,            1944,            1726,            3989,            1949,            1426,            1738,            553,            3013,            3647,            1253,            1479,            2334,            2970,            2470,            3906,            2506,            996,            1214,            848,            1740,            1562,            1675,            ],        [            799,            3680,            3950,            1755,            2224,            2945,            2904,            2222,            2902,            2063,            3176,            2277,            850,            596,            2904,            3386,            587,            1061,            2890,            2074,            1480,            1456,            2934,            1732,            2801,            4391,            4758,            4090,            4644,            2800,            1955,            1616,            2284,            2465,            2643,            2488,            2690,            2989,            1378,            1642,            2583,            2333,            1710,            3510,            2454,            2145,            1569,            1312,            2454,            1936,            0,            648,            963,            3981,            1924,            2313,            1196,            1548,            3005,            3639,            1245,            3459,            2549,            3145,            2462,            3898,            2721,            1388,            2101,            1817,            3720,            3542,            3655,            ],        [            1197,            3227,            3497,            735,            2227,            2349,            3302,            2620,            3300,            1610,            2723,            2675,            1248,            994,            2978,            2933,            985,            1459,            2437,            2957,            1878,            1854,            3292,            2130,            3199,            4789,            5156,            4488,            5042,            3198,            1307,            968,            1831,            1817,            2190,            1690,            2042,            2341,            730,            994,            3534,            1685,            1062,            3057,            2852,            2543,            1967,            1710,            2500,            2334,            398,            0,            315,            4379,            1276,            1860,            1594,            2492,            3403,            4037,            1643,            3006,            1901,            3543,            2860,            4296,            2073,            1786,            1648,            1364,            3267,            3089,            3202,            ],        [            1689,            2912,            3182,            420,            1912,            2034,            3024,            2908,            3588,            1295,            2408,            2963,            1853,            1599,            2663,            2618,            1497,            1747,            2122,            2642,            2637,            2613,            2977,            2418,            3487,            5077,            5444,            4776,            5330,            3803,            992,            653,            1516,            1502,            1875,            1375,            1727,            2026,            1189,            679,            3219,            1370,            1389,            2742,            3140,            2733,            2255,            1998,            2185,            2353,            1003,            691,            0,            4667,            961,            1545,            2416,            2177,            3691,            4325,            1931,            2691,            1586,            3831,            3148,            4584,            1758,            2074,            1333,            1049,            2952,            2774,            2887,            ],        [            3544,            4015,            4285,            4334,            4164,            5016,            4760,            4078,            3990,            4371,            4786,            4133,            5236,            5288,            4975,            4506,            3352,            2885,            4948,            4145,            2115,            2091,            5005,            3672,            2824,            660,            2325,            236,            432,            3968,            4949,            4610,            4863,            5459,            5222,            5067,            5684,            5983,            4970,            4636,            4439,            5327,            5346,            5120,            3786,            4216,            3425,            3168,            4525,            3876,            3936,            4240,            4555,            0,            4918,            4892,            1894,            3488,            1210,            384,            2785,            4601,            5543,            3073,            1452,            515,            5715,            3328,            4680,            4724,            4213,            4684,            4310,            ],        [            2093,            3316,            3586,            833,            2316,            2438,            3428,            3312,            3992,            1699,            2812,            3367,            2574,            2320,            3067,            3022,            1901,            2151,            2526,            3046,            3041,            3017,            3381,            2822,            3891,            5481,            5848,            5180,            5734,            5042,            117,            804,            1014,            1096,            1485,            651,            1321,            1620,            1255,            830,            3623,            692,            428,            3146,            3544,            3137,            2659,            2402,            2589,            2757,            1724,            1412,            1160,            5071,            0,            1043,            2820,            2581,            4095,            4729,            2335,            3095,            1180,            4235,            3552,            4988,            1352,            2478,            950,            1398,            3356,            3178,            3291,            ],        [            1984,            2678,            2948,            1304,            1678,            1800,            2790,            3094,            3790,            1061,            2174,            3149,            3049,            2795,            2429,            2384,            1792,            2042,            1888,            2408,            2932,            2908,            2743,            2730,            3782,            5372,            5739,            5071,            5625,            4933,            1031,            1580,            140,            749,            360,            669,            1268,            1490,            2255,            1606,            2985,            911,            1428,            2508,            3342,            2499,            2550,            2527,            1951,            2119,            2195,            1883,            1665,            4962,            1000,            0,            2711,            1943,            3986,            4728,            2226,            2457,            1127,            4017,            3443,            4987,            1299,            2386,            282,            787,            2718,            2540,            2653,            ],        [            885,            3667,            3937,            1675,            1915,            2767,            2511,            1829,            2509,            2122,            3135,            1884,            1950,            1696,            2726,            3125,            693,            636,            2877,            1896,            492,            468,            2756,            1423,            2092,            3682,            4049,            2364,            3935,            3243,            2290,            1951,            2204,            2800,            2563,            2408,            3025,            3324,            2311,            1977,            2190,            2668,            2687,            3469,            2061,            1967,            1176,            919,            2276,            1627,            1277,            1581,            1896,            2255,            2259,            2233,            0,            1239,            2296,            3038,            412,            3446,            2884,            2752,            1753,            3297,            3056,            1079,            2021,            2475,            3707,            3529,            3642,            ],        [            1079,            2315,            2585,            1980,            903,            1637,            1697,            1494,            2174,            1110,            1811,            1549,            2144,            1890,            1596,            2021,            887,            516,            1525,            1129,            1406,            1382,            1626,            787,            2256,            3846,            4213,            3545,            4099,            3407,            2595,            2256,            2012,            2760,            2371,            2216,            3090,            3389,            2505,            2282,            1868,            2733,            2992,            2145,            1726,            1200,            841,            584,            1146,            615,            1471,            1775,            2341,            3436,            2564,            2041,            1185,            0,            2460,            3202,            700,            2094,            2949,            2417,            1917,            3461,            3121,            443,            1829,            1463,            2355,            2177,            2290,            ],        [            2441,            4813,            5083,            3231,            3061,            3913,            3657,            2975,            3655,            3268,            4281,            3030,            3506,            3252,            3872,            4271,            2249,            1782,            4023,            3042,            2388,            2364,            3902,            2569,            2082,            2151,            2518,            1487,            2404,            3233,            3846,            3507,            3760,            4356,            4119,            3964,            4581,            4880,            3867,            3533,            3336,            4224,            4243,            4615,            3207,            3113,            2322,            2065,            3422,            2773,            2833,            3137,            3452,            1378,            3815,            3789,            2167,            2385,            0,            907,            1682,            4592,            4440,            3898,            326,            1166,            4612,            2225,            3577,            3621,            4853,            4675,            4788,            ],        [            3295,            4705,            4975,            4085,            3915,            4767,            4511,            3829,            4680,            4122,            5135,            3884,            4880,            4932,            4726,            5196,            3103,            2636,            4877,            3896,            2253,            2229,            4756,            3423,            2468,            1006,            1969,            582,            641,            3612,            4700,            4361,            4614,            5210,            4973,            4818,            5435,            5734,            4721,            4387,            4190,            5078,            5097,            5469,            4061,            3967,            3176,            2919,            4276,            3627,            3687,            3991,            4306,            587,            4669,            4643,            2032,            3239,            854,            0,            2536,            5291,            5294,            3763,            1180,            287,            5466,            3079,            4431,            4475,            4903,            5374,            5000,            ],        [            883,            3255,            3525,            1673,            1503,            2355,            2099,            1417,            2097,            1710,            2723,            1472,            1948,            1694,            2314,            2713,            691,            224,            2465,            1484,            706,            682,            2344,            1011,            1680,            3270,            3637,            2969,            3523,            2831,            2288,            1949,            2202,            2798,            2561,            2406,            3023,            3322,            2309,            1975,            1778,            2666,            2685,            3057,            1649,            1555,            764,            507,            1864,            1215,            1275,            1579,            1894,            2860,            2257,            2231,            485,            827,            1884,            2626,            0,            3034,            2882,            2340,            1341,            2885,            3054,            667,            2019,            2063,            3295,            3117,            3230,            ],        [            3071,            657,            927,            2450,            1727,            1179,            1217,            2473,            2217,            1580,            321,            2242,            4136,            3882,            856,            363,            2879,            2855,            805,            1787,            3398,            3374,            1170,            2029,            4248,            3975,            5966,            4392,            4228,            5399,            3065,            2726,            2482,            3230,            2841,            2686,            3560,            3859,            3527,            2752,            1412,            3203,            3462,            292,            1769,            1878,            2301,            2626,            1330,            2168,            3341,            3029,            2811,            4512,            3034,            2511,            3177,            1992,            4452,            4576,            2692,            0,            3419,            2637,            3909,            4289,            3591,            2245,            2299,            1933,            387,            131,            290,            ],        [            2858,            4135,            4405,            1598,            3135,            3257,            4247,            4551,            5247,            2518,            3631,            4606,            3339,            3085,            3886,            3841,            2666,            2916,            3345,            3865,            3806,            3782,            4200,            4187,            4656,            6246,            6613,            5945,            6499,            5807,            1071,            1569,            1597,            844,            1340,            1167,            141,            440,            2295,            1595,            4442,            769,            1493,            3965,            4799,            3956,            3424,            3167,            3408,            3576,            2489,            2177,            1959,            5836,            1040,            1467,            3585,            3400,            4860,            5494,            3100,            3914,            0,            5474,            4317,            5861,            519,            3843,            1739,            2244,            4175,            3997,            4110,            ],        [            2512,            2106,            2376,            3302,            2997,            2449,            1183,            899,            917,            2850,            1914,            1186,            3577,            3323,            1505,            1904,            2320,            1853,            2076,            1578,            2743,            2719,            1096,            1531,            3593,            2574,            4565,            2991,            2827,            4744,            3917,            3578,            3752,            4500,            4111,            3956,            4652,            4951,            3938,            3604,            862,            4295,            4314,            2248,            713,            1649,            1299,            1624,            1958,            2717,            2904,            3208,            3523,            3111,            3886,            3781,            2522,            2541,            3797,            3175,            2037,            2197,            4511,            0,            3254,            2888,            4683,            1747,            3569,            3203,            2458,            2280,            2393,            ],        [            2115,            4487,            4757,            2905,            2735,            3587,            3331,            2649,            3329,            2942,            3955,            2704,            3180,            2926,            3546,            3945,            1923,            1456,            3697,            2716,            2062,            2038,            3576,            2243,            1756,            1927,            2294,            1628,            2180,            2907,            3520,            3181,            3434,            4030,            3793,            3638,            4255,            4554,            3541,            3207,            3010,            3898,            3917,            4289,            2881,            2787,            1996,            1739,            3096,            2447,            2507,            2811,            3126,            1519,            3489,            3463,            1841,            2059,            543,            1177,            1356,            4266,            4114,            3572,            0,            1436,            4286,            1899,            3251,            3295,            4527,            4349,            4462,            ],        [            3554,            3842,            4112,            4344,            4174,            5026,            4083,            4088,            3817,            4381,            4613,            4143,            5139,            5191,            4405,            4333,            3362,            2895,            4775,            4155,            2415,            2391,            3996,            3682,            2727,            737,            2228,            313,            354,            3871,            4959,            4620,            4873,            5469,            5232,            5077,            5694,            5993,            4980,            4646,            3762,            5337,            5356,            4947,            3613,            4226,            3435,            3178,            4535,            3886,            3946,            4250,            4565,            300,            4928,            4902,            2194,            3498,            1113,            287,            2795,            4428,            5553,            2900,            1439,            0,            5725,            3338,            4690,            4734,            4040,            4511,            4137,            ],        [            2339,            3286,            3556,            1079,            2286,            2408,            3398,            3702,            4398,            1669,            2782,            3757,            2820,            2566,            3037,            2992,            2147,            2397,            2496,            3016,            3287,            3263,            3351,            3338,            4137,            5727,            6094,            5426,            5980,            5288,            552,            1050,            1011,            325,            714,            648,            607,            906,            1776,            1076,            3593,            250,            974,            3116,            3950,            3107,            2905,            2648,            2559,            2727,            1970,            1658,            1440,            5317,            521,            835,            3066,            2551,            4341,            5083,            2581,            3065,            466,            4625,            3798,            5342,            0,            2994,            947,            1395,            3326,            3148,            3261,            ],        [            1052,            2457,            2727,            2349,            1272,            1557,            1254,            1101,            1781,            1479,            1925,            1156,            2117,            1863,            1516,            1915,            860,            548,            1667,            686,            1379,            1355,            1546,            344,            2229,            3819,            4186,            3518,            4072,            3380,            2964,            2625,            2381,            3129,            2740,            2585,            3459,            3758,            2478,            2651,            1788,            3102,            3361,            2259,            1333,            757,            576,            319,            1066,            984,            1444,            1748,            2063,            3409,            2933,            2410,            1158,            596,            2433,            3175,            673,            2236,            3318,            2024,            1890,            3434,            3490,            0,            2198,            1832,            2497,            2319,            2432,            ],        [            2037,            2731,            3001,            1276,            1731,            1853,            2843,            3147,            3843,            1114,            2227,            3202,            3102,            2848,            2482,            2437,            1845,            2095,            1941,            2461,            2985,            2961,            2796,            2783,            3835,            5425,            5792,            5124,            5678,            4986,            749,            1247,            456,            1036,            898,            387,            1261,            1560,            1973,            1273,            3038,            904,            1146,            2561,            3395,            2552,            2603,            2580,            2004,            2172,            2167,            1855,            1637,            5015,            718,            485,            2764,            1996,            4039,            4781,            2279,            2510,            1120,            4070,            3496,            5040,            1292,            2439,            0,            840,            2771,            2593,            2706,            ],        [            1197,            2442,            2712,            517,            1442,            1564,            2554,            2416,            3096,            825,            1938,            2471,            2262,            2008,            2193,            2148,            1005,            1255,            1652,            2172,            2145,            2121,            2507,            1926,            2995,            4585,            4952,            4284,            4838,            4146,            1132,            793,            1046,            1642,            1405,            1250,            1867,            2166,            1594,            819,            2749,            1510,            1529,            2272,            2648,            2339,            1763,            1506,            1715,            1883,            1408,            1096,            878,            4175,            1101,            1075,            1924,            1707,            3199,            3941,            1439,            2221,            1726,            3339,            2656,            4200,            1898,            1582,            863,            0,            2482,            2304,            2417,            ],        [            3331,            270,            540,            2710,            1987,            1439,            1477,            2733,            2477,            1840,            659,            2502,            4396,            4142,            1116,            623,            3139,            3115,            1065,            2047,            3658,            3634,            1430,            2289,            4508,            3601,            5592,            4018,            3854,            5659,            3325,            2986,            2742,            3490,            3101,            2946,            3820,            4119,            3787,            3012,            1672,            3463,            3722,            679,            2029,            2138,            2561,            2886,            1590,            2428,            3601,            3289,            3071,            4138,            3294,            2771,            3437,            2252,            4712,            4202,            2952,            387,            3679,            2263,            4169,            3915,            3851,            2505,            2559,            2193,            0,            256,            97,            ],        [            3154,            522,            792,            2533,            1810,            1262,            1300,            2556,            2300,            1663,            404,            2325,            4219,            3965,            939,            446,            2962,            2938,            888,            1870,            3481,            3457,            1253,            2112,            4331,            3840,            5831,            4257,            4093,            5482,            3148,            2809,            2565,            3313,            2924,            2769,            3643,            3942,            3610,            2835,            1495,            3286,            3545,            423,            1852,            1961,            2384,            2709,            1413,            2251,            3424,            3112,            2894,            4377,            3117,            2594,            3260,            2075,            4535,            4441,            2775,            131,            3502,            2502,            3992,            4154,            3674,            2328,            2382,            2016,            256,            0,            159,            ],        [            3266,            363,            633,            2645,            1922,            1374,            1412,            2668,            2412,            1775,            563,            2437,            4331,            4077,            1051,            558,            3074,            3050,            1000,            1982,            3593,            3569,            1365,            2224,            4443,            3681,            5672,            4098,            3934,            5594,            3260,            2921,            2677,            3425,            3036,            2881,            3755,            4054,            3722,            2947,            1607,            3398,            3657,            582,            1964,            2073,            2496,            2821,            1525,            2363,            3536,            3224,            3006,            4218,            3229,            2706,            3372,            2187,            4647,            4282,            2887,            290,            3614,            2343,            4104,            3995,            3786,            2440,            2494,            2128,            97,            159,            0,            ],        ], 'starts': [24, 50, 29, 33, 10], 'ends': [24, 50, 29, 33, 10]},    'NW-32': {'distance_matrix': [        [            0,            5976,            7688,            6962,            7936,            7291,            7696,            4494,            5555,            6159,            6281,            7194,            7087,            8078,            5767,            4876,            4787,            4850,            7289,            5541,            5733,            7377,            8041,            8002,            4066,            7154,            5098,            7185,            6232,            6250,            6198,            5081,            5596,            5729,            5263,            8092,            6350,            9656,            9437,            9568,            7031,            6054,            6395,            7385,            5821,            6084,            5818,            6086,            5185,            7751,            5411,            1785,            6032,            4924,            6539,            6486,            4886,            6275,            8227,            8258,            1109,            7382,            6722,            4678,            7800,            3706,            7904,            ],        [            2526,            0,            2985,            2113,            3233,            2588,            2993,            2970,            1448,            1456,            965,            1895,            2238,            2779,            209,            2443,            1477,            3326,            2588,            435,            1008,            2528,            2742,            2867,            2542,            2451,            878,            1791,            5041,            3843,            3457,            2791,            2624,            3217,            2973,            3389,            2806,            4955,            4734,            4867,            4169,            4863,            2965,            2157,            505,            1977,            1115,            1979,            1875,            3048,            565,            4311,            3520,            1614,            4065,            1783,            2052,            3789,            3524,            3555,            3635,            1363,            3979,            1434,            2501,            3480,            2605,            ],        [            1710,            3413,            0,            1183,            1907,            370,            1667,            3996,            4281,            1977,            2661,            1598,            1341,            2471,            3204,            5276,            3497,            4290,            1524,            4267,            2536,            1541,            2434,            725,            4503,            1125,            3824,            1955,            4367,            3169,            2783,            3244,            3624,            2543,            3426,            2691,            2108,            3891,            3408,            3803,            3471,            4189,            2535,            1471,            4547,            4810,            2697,            4812,            3491,            2350,            4137,            3495,            2846,            3230,            3391,            2304,            3668,            3115,            2826,            2229,            2819,            2263,            3281,            3404,            2193,            5441,            2297,            ],        [            1194,            2573,            1555,            0,            2426,            1409,            2186,            3480,            3765,            1461,            1613,            1027,            125,            1423,            2364,            4323,            2981,            3774,            341,            2348,            2020,            1196,            1386,            1054,            3987,            1644,            2791,            907,            4737,            3539,            3153,            3614,            3994,            2913,            3796,            3061,            2478,            2708,            3927,            2620,            3841,            4559,            2905,            423,            2068,            3857,            2181,            3859,            2975,            2720,            2478,            2979,            3216,            2714,            3761,            1788,            3152,            3485,            3196,            2748,            2303,            1215,            3651,            2888,            1145,            4925,            1249,            ],        [            3109,            3471,            1439,            2959,            0,            1740,            485,            4054,            4339,            2154,            2893,            2307,            3084,            3191,            3262,            5334,            3555,            4348,            3286,            4325,            2594,            2024,            3154,            2095,            4561,            1004,            3882,            3182,            5621,            3652,            3266,            3727,            4107,            3026,            3909,            3174,            2591,            3876,            1501,            3788,            3954,            5443,            3018,            2477,            4605,            4868,            2755,            4870,            3549,            2833,            4195,            4894,            3329,            3288,            3874,            1769,            3726,            3598,            3309,            322,            4218,            2495,            3764,            3462,            2913,            7010,            3017,            ],        [            1340,            3043,            823,            813,            1651,            0,            1411,            3626,            3911,            1607,            2291,            1228,            971,            2101,            2834,            4906,            3127,            3920,            1154,            3897,            2166,            1285,            2064,            355,            4133,            869,            3454,            1585,            4111,            2913,            2527,            2988,            3368,            2287,            3170,            2435,            1852,            3521,            3152,            3433,            3215,            3933,            2279,            1101,            4177,            4440,            2327,            4442,            3121,            2094,            3767,            3125,            2590,            2860,            3135,            1934,            3298,            2859,            2570,            1973,            2449,            1893,            3025,            3034,            1823,            5071,            1927,            ],        [            2624,            2986,            954,            2474,            678,            1255,            0,            3569,            3854,            1669,            2408,            1822,            2599,            2706,            2777,            4849,            3070,            3863,            2801,            3840,            2109,            1539,            2669,            1610,            4076,            519,            3397,            2697,            4365,            3167,            2781,            3242,            3075,            2541,            3424,            2689,            2106,            5168,            2179,            5080,            3469,            4187,            2533,            1992,            4120,            4383,            2270,            4385,            3064,            2348,            3710,            4409,            2844,            2803,            3389,            1284,            3241,            3113,            2824,            1000,            3733,            2010,            3279,            2977,            2428,            5014,            2532,            ],        [            4085,            2231,            3654,            3935,            3902,            3257,            3662,            0,            1810,            3132,            2536,            4167,            4060,            5051,            2022,            1849,            1760,            356,            4262,            1796,            1612,            4350,            5014,            3536,            1132,            3120,            1353,            4158,            1389,            2013,            1882,            971,            1280,            1413,            1153,            2652,            1743,            6629,            5403,            6541,            2727,            3033,            1868,            4358,            2076,            2339,            2791,            2341,            1293,            2311,            1666,            5870,            1716,            633,            2235,            2452,            392,            1959,            2787,            4224,            5194,            2934,            2406,            933,            4773,            1945,            4877,            ],        [            3456,            1448,            4030,            3306,            4278,            3633,            4038,            2600,            0,            2501,            2031,            2961,            3431,            3845,            1239,            995,            1107,            2956,            3633,            1013,            2053,            3721,            3808,            3912,            2172,            3496,            570,            2857,            5195,            4888,            4502,            3679,            3669,            4262,            3861,            4434,            3851,            6000,            5779,            5912,            5214,            5017,            4010,            3223,            1293,            529,            2160,            531,            1505,            4093,            883,            5241,            4565,            1244,            5110,            2828,            1682,            4834,            4569,            4600,            4565,            2429,            5024,            1064,            3567,            2575,            3671,            ],        [            1844,            2206,            1529,            2476,            1777,            1132,            1537,            2845,            3074,            0,            3294,            2708,            2601,            3592,            1997,            4069,            2290,            3071,            2021,            3060,            1329,            1391,            3555,            1411,            3296,            995,            2617,            2760,            3585,            2387,            2001,            2462,            2295,            1761,            2644,            1933,            1350,            4388,            3278,            4300,            2713,            3407,            1509,            1763,            3340,            3603,            1490,            3605,            2284,            1592,            2930,            3629,            2064,            2023,            2609,            327,            2461,            2333,            2068,            2099,            2953,            2896,            2523,            2197,            3314,            4540,            3418,            ],        [            1561,            960,            2545,            1148,            2793,            2148,            2553,            3270,            2011,            1828,            0,            930,            1273,            1814,            751,            2710,            1777,            3626,            1489,            735,            2387,            1563,            1777,            2202,            2842,            2011,            1178,            826,            5253,            4055,            3669,            4130,            4510,            3429,            4312,            3577,            2994,            3856,            4294,            3768,            4357,            5075,            3421,            1192,            455,            2244,            2830,            2246,            2175,            3236,            865,            3346,            3732,            1914,            4277,            2155,            2352,            4001,            3712,            3115,            2670,            398,            4167,            1734,            1536,            4290,            1640,            ],        [            1184,            2115,            2168,            771,            2416,            1771,            2176,            3470,            3166,            1451,            1155,            0,            896,            884,            1906,            3865,            2932,            3764,            1112,            1890,            2010,            1186,            847,            1825,            3977,            1634,            2333,            449,            4876,            3678,            3292,            3753,            4133,            3052,            3935,            3200,            2617,            3479,            3917,            3391,            3980,            4698,            3044,            262,            1610,            3399,            2171,            3401,            2965,            2859,            2020,            2969,            3355,            2704,            3900,            1778,            3142,            3624,            3335,            2738,            2293,            757,            3790,            2878,            606,            5445,            710,            ],        [            1069,            2448,            1430,            656,            2301,            1372,            2061,            3355,            3640,            1336,            1488,            902,            0,            1298,            2239,            4198,            2856,            3649,            304,            2223,            1895,            1071,            1261,            1017,            3862,            1519,            2666,            782,            4761,            3563,            3177,            3638,            4018,            2937,            3820,            3085,            2502,            2671,            3802,            2583,            3865,            4583,            2929,            298,            1943,            3732,            2056,            3734,            2850,            2744,            2353,            2854,            3240,            2589,            3785,            1663,            3027,            3509,            3220,            2623,            2178,            1090,            3675,            2763,            1020,            4800,            1124,            ],        [            1423,            2354,            2407,            1010,            2655,            2010,            2415,            3709,            3405,            1690,            1394,            808,            1135,            0,            2145,            4104,            3171,            4003,            1351,            2129,            2249,            1425,            827,            2064,            4216,            1873,            2572,            688,            5115,            3917,            3531,            3992,            4372,            3291,            4174,            3439,            2856,            3718,            4156,            3630,            4219,            4937,            3283,            566,            1849,            3638,            2410,            3640,            3204,            3098,            2259,            3208,            3594,            2943,            4139,            2017,            3381,            3863,            3574,            2977,            2532,            996,            4029,            3117,            586,            5684,            965,            ],        [            2317,            209,            2921,            1904,            3169,            2524,            2929,            2761,            1239,            1392,            756,            1686,            2029,            2570,            0,            2234,            1268,            3117,            2245,            226,            944,            2319,            2533,            2803,            2333,            2387,            669,            1582,            4977,            3779,            3393,            2727,            2560,            3153,            2909,            3325,            2742,            4612,            4670,            4524,            4105,            4799,            2901,            1948,            296,            1768,            1051,            1770,            1666,            2984,            356,            4102,            3456,            1405,            4001,            1719,            1843,            3725,            3460,            3491,            3426,            1154,            3915,            1225,            2292,            3271,            2396,            ],        [            4500,            2118,            4700,            3529,            4948,            4303,            4708,            3270,            670,            3171,            2381,            3311,            3654,            4195,            1909,            0,            1777,            3626,            4677,            1683,            2723,            3944,            4158,            4582,            2842,            4166,            1240,            3207,            4654,            4672,            5172,            3138,            3653,            4932,            3320,            5104,            4521,            7044,            6449,            6956,            5453,            4476,            4680,            3573,            1963,            331,            2830,            461,            2175,            4763,            1553,            6285,            5235,            1914,            4961,            3498,            2352,            4722,            5239,            5270,            5609,            2779,            4779,            1734,            3917,            2034,            4021,            ],        [            2349,            1609,            2923,            2199,            3171,            2526,            2931,            903,            1188,            1394,            1914,            2431,            2324,            3315,            1400,            2183,            0,            1197,            2526,            1174,            946,            2614,            3278,            2805,            1410,            2389,            731,            2422,            4979,            3781,            3395,            2729,            2562,            3155,            2911,            3327,            2744,            4893,            4672,            4805,            4107,            4801,            2903,            2622,            1454,            1717,            1053,            1719,            398,            2986,            1044,            4134,            3458,            137,            4003,            1721,            575,            3727,            3462,            3493,            3458,            2312,            3917,            311,            3037,            2348,            3141,            ],        [            2659,            1875,            3233,            2509,            3481,            2836,            3241,            356,            1454,            1704,            2180,            2741,            2634,            3625,            1666,            2449,            670,            0,            2836,            1440,            1256,            2924,            3588,            3115,            1676,            2699,            997,            3006,            1683,            2307,            2176,            1265,            1574,            1707,            1447,            2946,            2037,            5203,            4982,            5115,            3021,            3327,            2162,            2932,            1720,            1983,            1363,            1985,            478,            2605,            1310,            4444,            2010,            277,            2529,            2031,            686,            2253,            3081,            3803,            3768,            2578,            2700,            577,            3347,            2301,            3451,            ],        [            3213,            4916,            3526,            2686,            4320,            2703,            4080,            5499,            5784,            3480,            4164,            4636,            2844,            4213,            4707,            6779,            5000,            5793,            0,            5770,            4039,            3208,            4432,            2348,            6006,            3538,            5327,            3458,            9272,            5867,            5481,            5942,            6322,            5241,            6124,            5413,            4830,            2367,            4765,            2279,            6193,            6887,            4989,            2974,            6050,            6313,            4200,            6315,            4994,            5072,            5640,            4998,            5544,            4733,            6089,            3807,            5171,            5813,            5548,            4642,            4322,            3766,            6003,            4907,            3696,            6944,            3800,            ],        [            2301,            435,            3285,            1888,            3533,            2888,            3293,            2535,            1013,            2436,            740,            1670,            2013,            2554,            226,            2008,            1042,            2891,            2229,            0,            1988,            2303,            2517,            2942,            2107,            2751,            443,            1566,            6021,            4823,            4437,            3771,            3604,            4197,            3953,            4369,            3786,            4596,            5034,            4508,            5149,            5843,            3945,            1932,            280,            1542,            2095,            1544,            1440,            4028,            130,            4086,            4500,            1179,            5045,            2763,            1617,            4769,            4504,            3855,            3410,            1138,            4959,            999,            2276,            3045,            2380,            ],        [            2950,            2166,            3524,            2800,            3772,            3127,            3532,            1460,            1745,            1995,            2471,            3032,            2925,            3916,            1957,            2740,            961,            1754,            3127,            1731,            0,            3215,            3879,            3406,            1967,            2990,            1288,            3297,            2201,            1967,            1836,            1783,            1616,            1367,            1965,            2606,            1988,            5494,            5273,            5406,            2681,            2987,            1822,            3223,            2011,            2274,            1654,            2276,            955,            2265,            1601,            4735,            1670,            694,            2189,            2322,            1132,            1913,            2741,            4094,            4059,            2869,            2360,            868,            3638,            2816,            3742,            ],        [            1849,            2311,            982,            1799,            1230,            585,            990,            2894,            3179,            994,            2617,            2031,            1086,            2216,            2102,            4174,            2395,            3188,            2126,            3165,            1434,            0,            2179,            864,            3401,            448,            2722,            1700,            3690,            2492,            2106,            2567,            2947,            1866,            2749,            2014,            1431,            4493,            2731,            4405,            2794,            3512,            1858,            1216,            3445,            3708,            1595,            3710,            2389,            1673,            3035,            3634,            2169,            2128,            2714,            609,            2566,            2438,            2149,            1552,            2958,            2219,            2604,            2302,            1938,            4250,            2042,            ],        [            2057,            2988,            3041,            1644,            3289,            3011,            3049,            4343,            4039,            2324,            2028,            1442,            1769,            634,            2779,            4738,            3805,            4637,            1985,            2763,            2883,            2059,            0,            2656,            4850,            2507,            3206,            1322,            5749,            4551,            4165,            4626,            5006,            3925,            4808,            4073,            3490,            4352,            4790,            4264,            4853,            5571,            3917,            1200,            2483,            4272,            3044,            4274,            3838,            3732,            2893,            3842,            4228,            3577,            4773,            2651,            4015,            4497,            4208,            3611,            3166,            1630,            4663,            3751,            721,            5699,            604,            ],        [            985,            2688,            1178,            458,            2006,            355,            1766,            3271,            3556,            1252,            1936,            873,            616,            1746,            2479,            4551,            2772,            3565,            799,            3542,            1811,            980,            1709,            0,            3778,            1224,            3099,            1230,            4837,            3639,            3253,            3714,            4094,            3013,            3896,            3185,            2602,            3166,            3507,            3078,            3965,            4659,            2761,            746,            3822,            4085,            1972,            4087,            2766,            2844,            3412,            2770,            3316,            2505,            3861,            1579,            2943,            3585,            3320,            2532,            2094,            1538,            3775,            2679,            1468,            4627,            1572,            ],        [            3731,            2306,            4307,            3581,            4555,            3910,            4315,            428,            1885,            2778,            2611,            3813,            3706,            4697,            2097,            1495,            1406,            784,            3908,            1871,            1687,            3996,            4660,            4621,            0,            3773,            1428,            3804,            1817,            2441,            2310,            1399,            1708,            1841,            1581,            3080,            2462,            6275,            6056,            6187,            3155,            3461,            2296,            4004,            2151,            2414,            2437,            2416,            909,            2739,            1741,            5516,            2144,            708,            2663,            3105,            820,            2387,            3215,            4877,            4840,            4001,            2834,            1008,            4419,            1502,            4523,            ],        [            2549,            2911,            859,            2399,            782,            1160,            542,            3494,            3779,            1594,            2333,            1747,            1767,            2631,            2702,            4774,            2995,            3788,            2726,            3765,            2034,            1464,            2594,            1515,            4001,            0,            3322,            2622,            4290,            3092,            2706,            3167,            3547,            2466,            3349,            2614,            2031,            5093,            2283,            5005,            3394,            4112,            2458,            1897,            4045,            4308,            2195,            4310,            2989,            2273,            3635,            4334,            2769,            2728,            3314,            1209,            3166,            3038,            2749,            1104,            3658,            1935,            3204,            2902,            2353,            5245,            2457,            ],        [            2948,            878,            3522,            2798,            3770,            3125,            3530,            2092,            570,            1993,            1183,            2113,            2923,            2997,            669,            1565,            599,            2448,            3125,            443,            1545,            3213,            2960,            3404,            1664,            2988,            0,            2009,            5578,            4380,            3994,            3328,            3161,            3754,            3510,            3926,            3343,            5492,            5271,            5404,            4706,            5400,            3502,            2375,            723,            1099,            1652,            1101,            997,            3585,            313,            4733,            4057,            736,            4602,            2320,            1174,            4326,            4061,            4092,            4057,            1581,            4516,            556,            2719,            3145,            2823,            ],        [            735,            2118,            1719,            322,            1967,            1322,            1727,            3021,            3306,            1002,            1158,            572,            447,            1456,            1909,            4301,            2522,            3315,            663,            1893,            1561,            737,            1419,            1376,            3528,            1185,            2849,            0,            4427,            3229,            2843,            3304,            3684,            2603,            3486,            2751,            2168,            3030,            3468,            2942,            3531,            4249,            2595,            398,            1613,            3835,            1722,            3837,            2516,            2410,            2023,            2520,            2906,            2255,            3451,            1329,            2693,            3175,            2886,            2289,            1844,            760,            3341,            2429,            1178,            4377,            1282,            ],        [            3824,            3679,            3449,            3411,            3697,            3052,            3457,            1239,            3258,            2713,            4229,            3643,            3536,            4527,            3470,            4253,            2474,            1465,            3752,            3244,            3153,            3826,            4490,            3331,            3480,            2915,            2801,            3695,            0,            421,            1198,            418,            876,            1208,            244,            1640,            1766,            5466,            5198,            5378,            1202,            225,            1663,            3834,            4684,            3787,            3314,            3789,            2468,            2106,            3114,            5609,            1448,            1610,            710,            2247,            1237,            930,            1952,            4019,            4933,            3831,            1674,            2381,            4249,            2978,            4353,            ],        [            3403,            3258,            3028,            2990,            3276,            2631,            3036,            1765,            2837,            2292,            3808,            3222,            3115,            4106,            3049,            3832,            2053,            1991,            3331,            2823,            2732,            3405,            4069,            2910,            3059,            2494,            2380,            3274,            1743,            0,            777,            1382,            455,            787,            1564,            1219,            1345,            5045,            4777,            4957,            1259,            1565,            1242,            3413,            4263,            3366,            2893,            3368,            2047,            1685,            2693,            5188,            1027,            1786,            767,            1826,            1763,            509,            1531,            3598,            4512,            3410,            1253,            1960,            3828,            3460,            3932,            ],        [            3657,            3758,            3157,            3244,            3405,            2760,            3165,            2612,            4626,            2441,            4062,            3476,            3369,            4360,            3549,            5621,            3842,            2838,            3585,            4612,            2881,            3659,            4323,            3039,            4848,            2623,            4169,            3528,            2161,            1312,            0,            2229,            1302,            1182,            2405,            442,            1262,            4268,            4562,            4180,            730,            1983,            1689,            3667,            4517,            5155,            3042,            5157,            3836,            717,            4482,            5442,            1180,            3575,            837,            1955,            2610,            1224,            754,            3727,            4766,            3664,            540,            3749,            4082,            4307,            4186,            ],        [            3766,            3404,            3391,            3353,            3639,            2994,            3399,            821,            2983,            2655,            4171,            3585,            3478,            4469,            3195,            3022,            2199,            1047,            3694,            2969,            3095,            3768,            4432,            3273,            2920,            2857,            2526,            3637,            418,            839,            1616,            0,            1294,            1150,            182,            2058,            1771,            5884,            5140,            5796,            1620,            643,            1605,            3776,            3249,            3512,            3256,            3514,            1393,            2048,            2839,            5551,            1453,            1192,            1128,            2189,            819,            1348,            2370,            3961,            4875,            3773,            2092,            2106,            4191,            2560,            4295,            ],        [            2948,            2803,            2573,            2535,            2821,            2176,            2581,            1310,            2382,            1837,            3353,            2767,            2660,            3651,            2594,            3377,            1598,            1536,            2876,            2368,            2277,            2950,            3614,            2455,            2604,            2039,            1925,            2819,            1345,            932,            801,            927,            0,            332,            1109,            1571,            662,            5069,            4322,            4981,            1646,            1952,            787,            2958,            3808,            2911,            2438,            2913,            1592,            1230,            2238,            4733,            635,            1331,            1154,            1371,            1308,            878,            1706,            3143,            4057,            2955,            1325,            1505,            3373,            3005,            3477,            ],        [            3209,            3393,            2792,            2796,            3040,            2395,            2800,            1868,            4261,            2076,            3614,            3028,            2921,            3912,            3184,            5256,            3477,            2094,            3137,            4247,            2516,            3211,            3875,            2674,            4483,            2258,            3804,            3080,            2614,            1394,            469,            1485,            1755,            0,            1667,            911,            621,            4737,            4541,            4649,            1183,            2436,            1048,            3219,            4069,            4790,            2677,            4792,            3471,            1159,            4117,            4994,            303,            3210,            1290,            1590,            1866,            1340,            1223,            3362,            4318,            3216,            993,            3384,            3634,            3563,            3738,            ],        [            4068,            3923,            3693,            3655,            3941,            3296,            3701,            1003,            3502,            2957,            4473,            3887,            3780,            4771,            3714,            3204,            2718,            1229,            3996,            3488,            3397,            4070,            4734,            3575,            3102,            3159,            3045,            3939,            244,            665,            1442,            182,            1120,            1452,            0,            1884,            2010,            5710,            5442,            5622,            1446,            469,            1907,            4078,            4928,            4031,            3558,            4033,            1575,            2350,            3358,            5853,            1692,            1374,            954,            2491,            1001,            1174,            2196,            4263,            5177,            4075,            1918,            2625,            4493,            2742,            4597,            ],        [            3520,            3621,            3020,            3107,            3268,            2623,            3028,            2662,            4489,            2304,            3925,            3339,            3232,            4223,            3412,            5484,            3705,            2888,            3448,            4475,            2744,            3522,            4186,            2902,            4711,            2486,            4032,            3391,            2211,            1362,            980,            2279,            1352,            1232,            2455,            0,            1125,            4303,            4597,            4215,            780,            2033,            1552,            3530,            4380,            5018,            2905,            5020,            3699,            580,            4345,            5305,            1230,            3438,            887,            1818,            2660,            1274,            724,            3590,            4629,            3527,            590,            3612,            3945,            4357,            4049,            ],        [            2588,            2794,            2213,            2175,            2461,            1816,            2221,            1763,            3662,            1477,            2993,            2407,            2300,            3291,            2585,            4657,            2878,            1989,            2516,            3648,            1917,            2590,            3254,            2095,            3884,            1679,            3205,            2459,            2503,            1305,            1148,            1380,            1213,            679,            1562,            1211,            0,            4883,            3962,            4795,            2019,            2325,            427,            2598,            3448,            4191,            2078,            4193,            2872,            870,            3518,            4373,            982,            2611,            1527,            1011,            1761,            1251,            1346,            2783,            3697,            2595,            1672,            2785,            3013,            3458,            3117,            ],        [            3349,            5052,            3662,            2822,            4574,            2839,            4334,            5635,            5920,            3616,            3823,            3237,            2947,            4349,            4843,            6915,            5136,            5929,            2711,            5906,            4175,            3344,            4568,            2484,            6142,            3792,            5463,            3594,            6467,            5637,            5255,            6554,            5627,            5507,            6711,            5697,            4966,            0,            3317,            289,            5055,            6289,            5125,            3110,            4278,            6449,            4336,            6451,            5130,            5208,            5776,            5134,            5505,            4869,            5143,            3943,            5307,            5530,            6009,            4896,            4458,            3425,            4846,            5043,            3832,            6991,            3936,            ],        [            4631,            4993,            2961,            4481,            1864,            3262,            2007,            5576,            5861,            3676,            4415,            3829,            4606,            4713,            4784,            6856,            5077,            5858,            4808,            5847,            4116,            3546,            4676,            3617,            6083,            2526,            5404,            4704,            6594,            5174,            4788,            5249,            5629,            4548,            5431,            4696,            4113,            3274,            0,            3186,            5476,            6416,            4540,            3999,            6127,            6390,            4277,            6392,            5071,            4355,            5717,            6416,            4851,            4810,            5396,            3291,            5248,            5120,            4831,            1295,            5740,            4017,            5286,            4984,            4435,            6932,            4539,            ],        [            3261,            4964,            3574,            2734,            4486,            2751,            4246,            5547,            5832,            3528,            3735,            3149,            2859,            4261,            4755,            6827,            5048,            5841,            2623,            5818,            4087,            3256,            4480,            2396,            6054,            3704,            5375,            3506,            6379,            5549,            5167,            6466,            5539,            5419,            6623,            5609,            4878,            289,            3229,            0,            4967,            6201,            5037,            3022,            4190,            6361,            4248,            6363,            5042,            5120,            5688,            5046,            5417,            4781,            5055,            3855,            5219,            5442,            5921,            4808,            4370,            3337,            4758,            4955,            3744,            6903,            3848,            ],        [            3520,            3375,            3145,            3107,            3393,            2748,            3153,            1882,            2954,            2409,            3925,            3339,            3232,            4223,            3166,            3949,            2170,            2108,            3448,            2940,            2849,            3522,            4186,            3027,            3176,            2611,            2497,            3391,            1780,            582,            608,            1499,            572,            590,            1681,            1050,            1176,            4876,            4894,            4788,            0,            1602,            1359,            3530,            4380,            3483,            3010,            3485,            2164,            1802,            2810,            5305,            858,            1903,            804,            1943,            1880,            340,            1362,            3715,            4629,            3527,            1084,            2077,            3945,            3577,            4049,            ],        [            3599,            3454,            3224,            3186,            3472,            2827,            3232,            1961,            3033,            2488,            4004,            3418,            3311,            4302,            3245,            4028,            2249,            2187,            3527,            3019,            2928,            3601,            4265,            3106,            3255,            2690,            2576,            3470,            391,            196,            973,            809,            651,            983,            635,            1415,            1541,            5241,            4973,            5153,            977,            0,            1438,            3609,            4459,            3562,            3089,            3564,            2243,            1881,            2889,            5384,            1223,            1982,            485,            2022,            1959,            705,            1727,            3794,            4708,            3606,            1449,            2156,            4024,            3656,            4128,            ],        [            2161,            2367,            1786,            1748,            2034,            1389,            1794,            2950,            3235,            1050,            2566,            1980,            1873,            2864,            2158,            4230,            2451,            2397,            2089,            3221,            1490,            2163,            2827,            1668,            3457,            1252,            2778,            2032,            2911,            1713,            1437,            1788,            2168,            1087,            1970,            1390,            807,            4456,            3535,            4368,            2170,            2733,            0,            2171,            3021,            3764,            1651,            3766,            2445,            1049,            3091,            3946,            1390,            2184,            1935,            584,            2622,            1659,            1525,            2356,            3270,            2168,            1980,            2358,            2586,            3866,            2690,            ],        [            1219,            2150,            1560,            806,            2451,            1039,            2211,            3505,            3201,            1486,            1190,            604,            298,            1000,            1941,            3900,            2967,            3799,            602,            1925,            2045,            1221,            963,            1222,            4012,            1669,            2368,            484,            4911,            3713,            3327,            3788,            4168,            3087,            3970,            3235,            2652,            2969,            3952,            2881,            4015,            4733,            3079,            0,            1645,            3434,            2206,            3436,            3000,            2894,            2055,            3004,            3390,            2739,            3935,            1813,            3177,            3659,            3370,            2773,            2328,            792,            3825,            2913,            722,            4861,            826,            ],        [            2021,            505,            3005,            1608,            3253,            2608,            3013,            2815,            1293,            2716,            460,            1390,            1733,            2274,            296,            2288,            1322,            3171,            1949,            280,            2268,            2023,            2237,            2662,            2387,            2471,            723,            1286,            5713,            4515,            4129,            4051,            4970,            3889,            4233,            4037,            3454,            4316,            4754,            4228,            4817,            5535,            3881,            1652,            0,            1822,            2375,            1824,            1720,            3696,            410,            3806,            4192,            1459,            4737,            3043,            1897,            4461,            4172,            3575,            3130,            858,            4627,            1279,            1996,            3236,            2100,            ],        [            4512,            2150,            5088,            4362,            5336,            4691,            5096,            1987,            702,            3559,            2413,            3343,            4487,            4227,            1941,            466,            1809,            2343,            4689,            1715,            2755,            4777,            4190,            5402,            1559,            4554,            1272,            3239,            4666,            4684,            4267,            3150,            3665,            3798,            3332,            5492,            4909,            7056,            6837,            6968,            5465,            4488,            4253,            3605,            1995,            0,            3218,            493,            2207,            5151,            1585,            6297,            4101,            1946,            4973,            3886,            2384,            4344,            5627,            5658,            5621,            2811,            4791,            2078,            3949,            2046,            4053,            ],        [            1296,            1757,            1870,            1146,            2118,            1473,            1878,            2340,            2625,            341,            1779,            1378,            1271,            2262,            1548,            3620,            1841,            2634,            1473,            2611,            880,            1561,            2225,            1752,            2847,            1336,            2168,            1369,            3926,            2728,            2342,            2803,            3183,            2102,            2985,            2274,            1691,            3840,            3619,            3752,            3054,            3748,            1850,            1569,            2891,            3154,            0,            3156,            1835,            1933,            2481,            3081,            2405,            1574,            2950,            668,            2012,            2674,            2409,            2440,            2405,            1381,            2864,            1748,            1984,            3696,            2088,            ],        [            3665,            1657,            4239,            3068,            4487,            3842,            4247,            2809,            209,            2710,            1920,            2850,            3193,            3734,            1448,            908,            1316,            3165,            3842,            1222,            2262,            3483,            3697,            4121,            2381,            3705,            779,            2746,            5108,            5097,            4711,            3592,            3878,            4471,            3774,            4643,            4060,            6209,            5988,            6121,            5423,            4930,            4219,            3112,            1502,            442,            2369,            0,            1714,            4302,            1092,            5450,            4774,            1453,            5319,            3037,            1891,            5043,            4778,            4809,            4774,            2318,            5233,            1273,            3456,            2488,            3560,            ],        [            4140,            2674,            3765,            3727,            4013,            3368,            3773,            505,            2253,            3029,            2979,            3959,            3852,            4843,            2465,            2354,            2265,            799,            4068,            2239,            2055,            4142,            4806,            3647,            1637,            3231,            1796,            4011,            1500,            2124,            1993,            1082,            1391,            1524,            1264,            2763,            2145,            6261,            5514,            6173,            2838,            3144,            1979,            4150,            2519,            2782,            3296,            2784,            0,            2422,            2109,            5925,            1827,            1076,            2346,            2563,            177,            2070,            2898,            4335,            5249,            4147,            2517,            1376,            4565,            2361,            4669,            ],        [            3626,            3553,            2952,            3213,            3200,            2555,            2960,            3003,            4421,            2236,            4031,            3445,            3338,            4329,            3344,            5416,            3637,            3229,            3554,            4407,            2676,            3628,            4292,            2834,            4643,            2418,            3964,            3497,            2552,            1703,            1321,            2620,            1693,            1573,            2796,            341,            1187,            4644,            4701,            4556,            1121,            2374,            1614,            3636,            4486,            4950,            2837,            4952,            3631,            0,            4277,            5411,            1571,            3370,            1228,            1750,            3001,            1615,            476,            3522,            4735,            3633,            931,            3544,            4051,            4698,            4155,            ],        [            2431,            565,            3835,            2018,            4083,            3438,            3843,            2405,            883,            2306,            870,            1800,            2143,            2684,            356,            1878,            912,            2761,            2359,            130,            1858,            2433,            2647,            3717,            1977,            3301,            313,            1696,            5891,            4693,            4307,            3641,            3474,            4067,            3823,            4239,            3656,            4726,            5584,            4638,            5019,            5713,            3815,            2062,            410,            1412,            1965,            1414,            1310,            3898,            0,            4216,            4370,            1049,            4915,            2633,            1487,            4639,            4374,            4405,            3540,            1268,            4829,            869,            2406,            2826,            2510,            ],        [            5238,            4102,            5814,            5088,            6062,            5417,            5822,            2620,            3681,            4285,            4407,            5320,            5213,            6204,            3893,            3002,            2913,            2976,            5415,            3667,            3859,            5503,            6167,            6128,            2192,            5280,            3224,            5311,            4447,            4465,            4413,            3296,            3811,            3944,            3478,            6218,            4565,            7782,            7563,            7694,            5246,            4269,            4399,            5511,            3947,            4210,            3944,            4212,            3311,            5877,            3537,            0,            4247,            3050,            4754,            4612,            3012,            4490,            6353,            6384,            6347,            5508,            4937,            2804,            5926,            1832,            6030,            ],        [            2906,            3090,            2489,            2493,            2737,            2092,            2497,            2081,            3958,            1773,            3311,            2725,            2618,            3609,            2881,            4953,            3174,            2307,            2834,            3944,            2213,            2908,            3572,            2371,            4180,            1955,            3501,            2777,            2821,            1623,            989,            1698,            2078,            997,            1880,            1197,            318,            5500,            4238,            5412,            1977,            2643,            745,            2916,            3766,            4487,            2374,            4489,            3168,            856,            3814,            4691,            0,            2907,            1845,            1287,            2079,            1569,            1332,            3059,            4015,            2913,            1787,            3081,            3331,            3776,            3435,            ],        [            2382,            1598,            2956,            2232,            3204,            2559,            2964,            766,            1177,            1427,            1903,            2464,            2357,            3348,            1389,            2172,            393,            1060,            2559,            1163,            979,            2647,            3311,            2838,            1399,            2422,            720,            2729,            1761,            2385,            2254,            1343,            1652,            1785,            1525,            3360,            2406,            4926,            4705,            4838,            3099,            3405,            2240,            2655,            1443,            1706,            1086,            1708,            261,            3019,            1033,            4167,            2088,            0,            2607,            1754,            438,            2331,            3495,            3526,            3491,            2301,            2778,            300,            3070,            2248,            3174,            ],        [            3567,            3422,            3192,            3154,            3440,            2795,            3200,            1929,            3001,            2456,            3972,            3386,            3279,            4270,            3213,            3996,            2217,            2155,            3495,            2987,            2896,            3569,            4233,            3074,            3223,            2658,            2544,            3438,            1827,            629,            655,            1546,            619,            637,            1728,            1097,            1223,            4923,            4941,            4835,            1343,            1649,            1406,            3577,            4427,            3530,            3057,            3532,            2211,            1761,            2857,            5352,            905,            1950,            0,            1990,            1927,            387,            1409,            3762,            4676,            3574,            1131,            2124,            3992,            3624,            4096,            ],        [            2069,            2809,            1202,            1314,            1450,            805,            1210,            3392,            3677,            1492,            2626,            2529,            1306,            2436,            2600,            4672,            2893,            3686,            1610,            3663,            1932,            1064,            2399,            1084,            3899,            668,            3220,            1920,            4188,            2990,            2604,            3065,            3445,            2364,            3247,            2512,            1929,            3977,            2951,            3889,            3292,            4010,            2356,            1436,            3943,            4206,            2093,            4208,            2887,            2171,            3533,            3854,            2667,            2626,            3212,            0,            3064,            2936,            2647,            1772,            3178,            2228,            3102,            2800,            2158,            5143,            2262,            ],        [            3963,            2497,            3588,            3550,            3836,            3191,            3596,            328,            2076,            2852,            2802,            3782,            3675,            4666,            2288,            2177,            2088,            622,            3891,            2062,            1878,            3965,            4629,            3470,            1460,            3054,            1619,            3834,            1323,            1947,            1816,            905,            1214,            1347,            1087,            2586,            1968,            6084,            5337,            5996,            2661,            2967,            1802,            3973,            2342,            2605,            3119,            2607,            1621,            2245,            1932,            5748,            1650,            899,            2169,            2386,            0,            1893,            2721,            4158,            5072,            3970,            2340,            1199,            4388,            2983,            4492,            ],        [            3424,            3630,            3007,            3011,            3255,            2610,            3015,            2039,            4498,            2313,            3829,            3243,            3136,            4127,            3421,            5493,            3714,            2265,            3352,            4484,            2753,            3426,            4090,            2889,            4720,            2473,            4041,            3295,            2365,            1565,            268,            1656,            1506,            520,            1838,            710,            836,            4536,            4756,            4448,            934,            2187,            1263,            3434,            4284,            5027,            2914,            5029,            3708,            1374,            4354,            5209,            518,            3447,            1041,            1805,            2037,            0,            1022,            3577,            4533,            3431,            744,            3621,            3849,            3734,            3953,            ],        [            3150,            3077,            2476,            2737,            2724,            2079,            2484,            2595,            3945,            1760,            3555,            2969,            2862,            3853,            2868,            4940,            3161,            2821,            3078,            3931,            2200,            3152,            3816,            2358,            4167,            1942,            3488,            3021,            2791,            1942,            1462,            2212,            1932,            1511,            3035,            580,            832,            4883,            4225,            4795,            1360,            2613,            1259,            3160,            4010,            4474,            2361,            4476,            3155,            239,            3801,            4935,            1340,            2894,            1467,            1274,            3332,            1854,            0,            3046,            4259,            3157,            1170,            3068,            3575,            4290,            3679,            ],        [            3471,            3833,            1801,            3321,            704,            2102,            847,            4416,            4701,            2516,            4139,            3553,            3446,            4437,            3624,            6539,            3917,            4698,            3648,            4687,            2956,            2386,            4400,            2457,            4923,            1366,            4244,            3544,            5299,            4014,            3628,            4089,            4469,            3388,            5543,            3536,            2953,            5490,            1769,            5402,            4316,            5121,            3380,            2839,            4967,            5230,            3117,            5232,            3911,            3195,            4557,            5256,            3691,            3650,            4236,            2131,            4088,            3960,            3671,            0,            4580,            3741,            4126,            3824,            4159,            6167,            4263,            ],        [            5914,            4778,            6490,            5764,            6738,            6093,            6498,            3296,            4357,            4961,            5083,            5996,            5889,            6880,            4569,            3678,            3589,            3652,            6091,            4343,            4535,            6179,            6843,            6804,            2868,            5956,            3900,            5987,            5123,            5141,            5089,            3972,            4487,            4620,            4154,            6894,            5241,            8458,            8239,            8370,            5922,            4945,            5286,            6187,            4623,            4886,            4620,            4888,            3987,            6553,            4213,            676,            4923,            3726,            5430,            5288,            3688,            5166,            7029,            7060,            0,            6184,            5613,            3480,            6602,            2508,            6706,            ],        [            1163,            1358,            2147,            750,            2395,            1750,            2155,            3668,            2409,            1430,            398,            532,            875,            1416,            1149,            3108,            2175,            4024,            1091,            1133,            1989,            1165,            1379,            1804,            3240,            1613,            1576,            428,            4855,            3657,            3271,            3732,            4112,            3031,            3914,            3179,            2596,            3458,            3896,            3370,            3959,            4677,            3023,            794,            853,            2642,            2150,            2644,            2573,            2838,            1263,            2948,            3334,            2312,            3879,            1757,            2750,            3603,            3314,            2717,            2272,            0,            3769,            2132,            1138,            4688,            1242,            ],        [            3864,            3719,            3489,            3451,            3737,            3092,            3497,            2226,            3298,            2753,            4269,            3683,            3576,            4567,            3510,            4293,            2514,            2452,            3792,            3284,            3193,            3866,            4530,            3371,            3520,            2955,            2841,            3735,            1621,            926,            952,            1843,            916,            934,            1865,            1394,            1520,            5220,            5238,            5132,            789,            1443,            1703,            3874,            4724,            3827,            3354,            3829,            2508,            2058,            3154,            5649,            1202,            2247,            297,            2287,            2224,            684,            1706,            4059,            4973,            3871,            0,            2421,            4289,            3921,            4393,            ],        [            2754,            1298,            3328,            2604,            3576,            2931,            3336,            1898,            877,            1799,            1603,            2533,            2729,            3417,            1089,            1872,            405,            2254,            2931,            863,            1351,            3019,            3380,            3210,            1470,            2794,            420,            2429,            5384,            4186,            3800,            3134,            2967,            3560,            3316,            3732,            3149,            5298,            5077,            5210,            4512,            5206,            3308,            2795,            1143,            1406,            1458,            1408,            803,            3391,            733,            4539,            3863,            542,            4408,            2126,            980,            4132,            3867,            3898,            3863,            2001,            4322,            0,            3139,            2319,            3243,            ],        [            2098,            3029,            2801,            1685,            3330,            2280,            3090,            4384,            4080,            2365,            2069,            1483,            1539,            675,            2820,            4779,            3846,            4678,            1843,            2804,            2924,            2100,            241,            2697,            4891,            2548,            3247,            1363,            5790,            4592,            4206,            4667,            5047,            3966,            4849,            4114,            3531,            4210,            4831,            4122,            4894,            5612,            3958,            1241,            2524,            4313,            3085,            4315,            3879,            3773,            2934,            3883,            4269,            3618,            4814,            2692,            4056,            4538,            4249,            3652,            3207,            1671,            4704,            3792,            0,            6359,            645,            ],        [            3406,            2270,            3982,            3256,            4230,            3585,            3990,            788,            1849,            2453,            2575,            3488,            3381,            4372,            2061,            1170,            1081,            1144,            3583,            1835,            2027,            3671,            4335,            4296,            360,            3448,            1392,            3479,            4022,            4840,            4454,            2900,            5295,            4214,            3082,            4386,            3803,            5950,            5731,            5862,            5166,            3844,            3962,            3679,            2115,            2378,            2112,            2380,            1479,            4045,            1705,            5191,            4517,            1218,            5062,            2780,            1180,            4786,            4521,            4552,            4515,            3676,            6275,            972,            4094,            0,            4198,            ],        [            2208,            3139,            2911,            1795,            3440,            2558,            3200,            4494,            4190,            2475,            2179,            1593,            1649,            785,            2930,            4889,            3956,            4788,            2090,            2914,            3034,            2210,            745,            2203,            5001,            2658,            3357,            1473,            5900,            4702,            4316,            4777,            5157,            4076,            4959,            4224,            3641,            4457,            4941,            4369,            5004,            5722,            4068,            1351,            2634,            4423,            3195,            4425,            3989,            3883,            3044,            3993,            4379,            3728,            4924,            2802,            4166,            4648,            4359,            3762,            3317,            1781,            4814,            3902,            990,            6469,            0,            ],        ], 'starts': [44, 7, 64, 26, 34], 'ends': [44, 7, 64, 26, 34]},    'NW-33': {'distance_matrix': [        [            0,            554,            2112,            514,            15929,            4383,            2658,            4102,            2815,            2999,            4784,            3002,            4112,            3282,            3032,            2887,            2414,            2067,            3312,            5355,            5111,            4174,            4244,            2402,            4446,            3880,            4789,            4422,            4069,            825,            2358,            3287,            1591,            2694,            4213,            2224,            3099,            1991,            2975,            868,            1394,            1156,            4199,            4649,            5139,            3054,            2316,            3733,            3354,            2126,            3220,            2400,            4159,            2704,            3464,            1622,            3921,            4481,            1903,            2927,            3823,            2701,            4777,            3017,            762,            2838,            947,            264,            750,            3053,            5084,            4563,            1666,            3927,            513,            2803,            4924,            3066,            ],        [            522,            0,            1714,            870,            17189,            4905,            3180,            3739,            3337,            3521,            5306,            3036,            3749,            3638,            2680,            3409,            2448,            2589,            2949,            5877,            5633,            4089,            3881,            2318,            4083,            3788,            4426,            4944,            3553,            1347,            2274,            3809,            2113,            2389,            4735,            2746,            4285,            2513,            3009,            1390,            1916,            1678,            4721,            5171,            4776,            3576,            3244,            3767,            4540,            2648,            3742,            2922,            4193,            3226,            3986,            2144,            3836,            4118,            2425,            2537,            4345,            3223,            5299,            3373,            351,            2533,            393,            786,            913,            3575,            4721,            5085,            2188,            3564,            696,            2719,            4960,            3588,            ],        [            880,            1228,            0,            1105,            15468,            4946,            3221,            4665,            3698,            4636,            5362,            3565,            4675,            2821,            3327,            3450,            2977,            2630,            3875,            5933,            6384,            4737,            4807,            2965,            5009,            4443,            5352,            5000,            4632,            1745,            2921,            3850,            2154,            3036,            4791,            2787,            3468,            2554,            3538,            1431,            1957,            1394,            5472,            5922,            5702,            3617,            2879,            4296,            3723,            2689,            4857,            3283,            4722,            3267,            4027,            2185,            4484,            5044,            2466,            3184,            4386,            3264,            5355,            2556,            1436,            3180,            1621,            827,            1313,            3616,            5647,            5141,            2229,            4490,            1187,            3366,            5502,            4703,            ],        [            885,            1233,            844,            0,            15415,            5637,            4216,            4670,            3645,            4583,            5309,            3570,            4680,            2768,            3332,            4445,            2982,            2635,            3880,            5880,            6331,            4742,            4812,            2970,            5014,            4448,            5357,            4947,            4637,            1692,            2926,            3855,            2159,            3041,            4738,            2792,            3415,            2559,            3543,            1436,            1962,            1341,            5419,            5869,            5707,            3622,            2884,            4301,            3670,            2694,            4804,            3230,            4727,            3272,            4032,            2190,            4489,            5049,            2471,            3189,            5077,            3269,            5302,            2503,            1441,            3185,            1626,            832,            1318,            3621,            5688,            5088,            2234,            4495,            1192,            3371,            5449,            4650,            ],        [            5106,            5454,            4226,            5331,            0,            2698,            4095,            5886,            3524,            3917,            4551,            5989,            5048,            2647,            6571,            4324,            5401,            3861,            6299,            3823,            3392,            5329,            5861,            6178,            5659,            5035,            5498,            4281,            5224,            4583,            5566,            4468,            3994,            5681,            4072,            3704,            3294,            4504,            6534,            4845,            5143,            4232,            2480,            2930,            5990,            4953,            5236,            6550,            3549,            5113,            4138,            3109,            6288,            5275,            4053,            4249,            5076,            5636,            3968,            6721,            2138,            4962,            4636,            2382,            5662,            5825,            5847,            5071,            5539,            5624,            5022,            4422,            4181,            5711,            5413,            6011,            4783,            3984,            ],        [            5117,            5465,            4237,            5342,            15212,            0,            4106,            5897,            3535,            3928,            4562,            6000,            5059,            2658,            6582,            4335,            5412,            3872,            6310,            3834,            3403,            5340,            5872,            6189,            5670,            5046,            5509,            4292,            5235,            4594,            5577,            4479,            4005,            5692,            4083,            3715,            3305,            4515,            6545,            4856,            5154,            4243,            2491,            2941,            6001,            4964,            5247,            6561,            3560,            5124,            4149,            3120,            6299,            5286,            4064,            4260,            5087,            5647,            3979,            6732,            2149,            4973,            4647,            2393,            5673,            5836,            5858,            5082,            5550,            5635,            5033,            4433,            4192,            5722,            5424,            6022,            4794,            3995,            ],        [            3442,            3790,            4151,            3667,            16937,            1725,            0,            3387,            3358,            1418,            2144,            3999,            2549,            4383,            4380,            229,            3762,            1529,            4309,            2884,            2453,            2830,            3529,            4142,            3252,            2536,            3091,            1782,            2725,            2864,            3927,            1969,            2051,            4042,            1573,            1686,            2382,            2563,            4141,            3126,            2779,            3195,            1541,            1991,            3440,            1788,            3701,            4051,            1919,            3474,            1639,            2943,            3789,            2776,            1554,            1750,            2577,            3137,            1469,            4275,            1165,            2463,            2137,            4118,            3998,            4186,            4183,            3109,            3875,            3125,            2523,            1923,            2238,            3212,            3749,            4372,            2284,            1485,            ],        [            4498,            4145,            5275,            4435,            19619,            4407,            2682,            0,            4400,            2292,            3018,            1566,            1006,            3791,            2139,            2911,            2463,            3075,            1876,            3203,            5135,            687,            142,            2375,            344,            1032,            992,            2028,            786,            3988,            2369,            1521,            3129,            2632,            2447,            3232,            3230,            3641,            1708,            4250,            3196,            4319,            4223,            4673,            1342,            2006,            3339,            1618,            3141,            3026,            2513,            3985,            1356,            2328,            2416,            3296,            940,            684,            3015,            2508,            3847,            2015,            1673,            5773,            4327,            2776,            3700,            4233,            4421,            2677,            1287,            1887,            3316,            779,            4672,            2814,            1526,            2359,            ],        [            1861,            2209,            2570,            2086,            16837,            4103,            2378,            3567,            0,            2745,            3471,            2467,            3876,            1128,            3049,            2607,            1879,            1058,            2777,            4042,            4831,            2899,            3709,            2656,            4579,            2605,            4418,            3109,            2794,            1283,            2044,            2012,            470,            2159,            2900,            1215,            1577,            982,            3012,            1545,            1198,            1614,            3919,            4369,            3864,            1779,            1714,            3691,            1832,            1591,            2966,            878,            4117,            1695,            1748,            613,            2646,            3206,            894,            3199,            3543,            1426,            3464,            3110,            2417,            2303,            2602,            1528,            2294,            2044,            3850,            3250,            657,            3281,            2168,            2489,            3611,            2812,            ],        [            3550,            3898,            4259,            3775,            18649,            3437,            1712,            1969,            3430,            0,            2048,            2581,            1131,            2821,            2962,            1941,            2344,            2105,            2891,            2619,            4165,            1412,            2111,            2724,            2188,            1118,            2027,            1686,            1307,            2972,            2509,            551,            2159,            2624,            1477,            2262,            2260,            2671,            2723,            3234,            3188,            3303,            3253,            3703,            2377,            1036,            3467,            2633,            2171,            2056,            221,            3015,            2371,            1358,            1446,            2326,            1159,            1719,            2045,            2857,            2877,            1045,            2041,            4803,            4106,            2768,            4049,            3217,            3983,            1707,            2322,            1827,            2346,            1794,            3857,            2954,            2561,            67,            ],        [            5554,            5552,            6263,            5779,            20111,            4899,            3248,            1452,            4966,            2782,            0,            2973,            1943,            4357,            3546,            3477,            4251,            3641,            3283,            3300,            5627,            1371,            1310,            3782,            1108,            1843,            947,            2125,            1699,            4976,            3776,            2458,            4163,            4039,            2538,            3798,            3796,            4675,            3115,            5238,            4603,            5307,            4715,            5165,            1439,            2943,            4746,            3025,            3707,            3963,            3003,            4551,            2763,            3265,            3353,            3862,            1624,            1368,            3581,            3915,            4339,            2952,            1770,            7292,            5734,            4183,            5107,            5221,            5828,            3614,            1384,            1984,            4350,            2186,            5861,            4221,            1623,            2849,            ],        [            3181,            3529,            3890,            3406,            18899,            4511,            2786,            1100,            4504,            2396,            3122,            0,            1110,            3895,            1475,            3015,            857,            3179,            310,            3693,            5239,            1450,            1242,            1347,            1444,            1149,            1787,            2760,            914,            2603,            1132,            1625,            3233,            1247,            2551,            3336,            3334,            3745,            1219,            2865,            1811,            2934,            4327,            4777,            2137,            2110,            2090,            1129,            3245,            3130,            2617,            4089,            867,            2432,            2520,            3400,            1197,            1479,            3119,            1559,            3951,            2119,            2468,            5877,            3378,            1391,            2751,            2848,            3614,            2781,            2082,            2901,            3420,            925,            3488,            1577,            2321,            2463,            ],        [            3522,            3870,            4231,            3747,            18917,            3409,            1684,            1967,            3402,            1294,            2020,            2579,            0,            2793,            2934,            1913,            2316,            2077,            2889,            2591,            4137,            871,            2109,            2696,            1647,            1116,            1486,            1658,            1305,            2944,            2481,            523,            2131,            2596,            1449,            2234,            2232,            2643,            2877,            3206,            3160,            3275,            3225,            3675,            1836,            1008,            3439,            2631,            2143,            2028,            1515,            2987,            2369,            1330,            1418,            2298,            618,            1178,            2017,            2829,            2849,            1017,            2013,            4775,            4078,            2740,            4021,            3189,            3955,            1679,            1781,            1799,            2318,            1792,            3829,            2926,            2160,            1361,            ],        [            2738,            3086,            3162,            2963,            15736,            3173,            1448,            3784,            877,            1815,            2541,            3342,            2946,            0,            3924,            1677,            2754,            1214,            3652,            3112,            3901,            3227,            3926,            3531,            3649,            2933,            3488,            2179,            3122,            2160,            2919,            2366,            1347,            3034,            1970,            1057,            647,            1857,            3887,            2422,            2496,            2491,            2989,            3439,            3837,            2185,            2589,            4448,            902,            2466,            2036,            462,            4186,            3173,            1951,            1490,            2974,            3534,            1399,            4074,            2613,            2860,            2534,            2231,            3294,            3178,            3479,            2405,            3171,            3522,            2920,            2320,            1534,            3609,            3045,            3364,            2681,            1882,            ],        [            2758,            2405,            3535,            2695,            18108,            5111,            3386,            1700,            3140,            2996,            3722,            600,            1710,            2975,            0,            3615,            723,            2862,            910,            4293,            5839,            2050,            1842,            635,            2044,            1749,            2387,            3360,            1514,            2248,            629,            2225,            1763,            892,            3151,            3019,            3424,            2203,            573,            2510,            1456,            2579,            4927,            5377,            2737,            2710,            1599,            1331,            3679,            1277,            3217,            2725,            1467,            3032,            3120,            2417,            1797,            2079,            2698,            768,            4551,            1852,            3068,            4737,            2587,            1036,            1960,            2493,            2681,            3381,            2682,            3501,            1878,            1525,            3133,            1074,            2921,            3063,            ],        [            3213,            3561,            3922,            3438,            17004,            1496,            882,            3158,            3129,            1189,            1915,            3770,            2320,            4154,            4151,            0,            3533,            1300,            4080,            2655,            2224,            2601,            3300,            3913,            3023,            2307,            2862,            1553,            2496,            2635,            3698,            1740,            1822,            3813,            1344,            1457,            2153,            2334,            3912,            2897,            2550,            2966,            1312,            1762,            3211,            1559,            3472,            3822,            1690,            3245,            1410,            2714,            3560,            2547,            1325,            1521,            2348,            2908,            1240,            4046,            936,            2234,            1908,            3889,            3769,            3957,            3954,            2880,            3646,            2896,            2294,            1694,            2009,            2983,            3520,            4143,            2055,            1256,            ],        [            2768,            2415,            3409,            2705,            18118,            5099,            3374,            1688,            3150,            2984,            3710,            588,            1698,            2985,            618,            3603,            0,            2405,            898,            4281,            5827,            2038,            1830,            645,            2032,            1737,            2375,            3348,            1502,            2122,            639,            2213,            1773,            902,            3139,            2562,            3434,            2213,            561,            2384,            1330,            2453,            4915,            5365,            2725,            2698,            1609,            1319,            3689,            1287,            3205,            2735,            1745,            3020,            3108,            1960,            1785,            2067,            2241,            778,            4539,            1862,            3056,            4967,            2597,            1046,            1970,            2367,            2691,            3369,            2670,            3489,            1888,            1513,            2942,            1084,            2909,            3051,            ],        [            2158,            2506,            2867,            2383,            16929,            2523,            798,            3108,            1829,            1139,            1865,            3103,            2270,            1220,            3346,            1027,            2515,            0,            3413,            2436,            3251,            2551,            3250,            2953,            2973,            2257,            2812,            1503,            2446,            1580,            2680,            1943,            767,            2795,            1294,            157,            853,            1279,            3076,            1842,            1495,            1911,            2339,            2789,            3161,            1008,            2417,            3772,            1108,            2227,            1360,            1414,            3510,            1481,            897,            466,            2298,            2858,            185,            3028,            1963,            1212,            1858,            3202,            2714,            2939,            2899,            1825,            2591,            1830,            2244,            1644,            954,            2933,            2465,            3125,            2005,            1206,            ],        [            4334,            3981,            5111,            4271,            19684,            4201,            2476,            790,            4194,            2086,            2812,            1402,            800,            3585,            1975,            2705,            1098,            2869,            0,            3383,            4929,            1140,            932,            2211,            1134,            839,            1477,            2450,            604,            3824,            2205,            1315,            2923,            2468,            2241,            3026,            3024,            3435,            1544,            4086,            3032,            4155,            4017,            4467,            1827,            1800,            3175,            1454,            2935,            2820,            2307,            3779,            1192,            2122,            2210,            3090,            887,            1169,            2809,            2344,            3641,            1809,            2158,            5567,            4163,            2612,            3536,            4069,            4257,            2471,            1772,            2591,            3110,            615,            4508,            2650,            2011,            2153,            ],        [            3664,            4012,            4373,            3889,            18504,            2996,            1358,            2180,            3076,            892,            728,            3473,            2023,            2467,            3854,            1587,            3236,            1751,            3783,            0,            3724,            2099,            2038,            3616,            1836,            2010,            1675,            874,            2199,            3086,            3401,            1443,            2273,            3516,            536,            1908,            1906,            2785,            3615,            3348,            4080,            3417,            2812,            3262,            2167,            1928,            4359,            3525,            1817,            2948,            1113,            2661,            3263,            2250,            2338,            1972,            2051,            2096,            1691,            3749,            2436,            1937,            1229,            4449,            4220,            3660,            4941,            3331,            4097,            2599,            1615,            1015,            2460,            2686,            3971,            3846,            1376,            959,            ],        [            4095,            4443,            4804,            4320,            17560,            2052,            1789,            2611,            3507,            1323,            1159,            3904,            2454,            2898,            4285,            2018,            3667,            2182,            4214,            431,            0,            2530,            2469,            4047,            2267,            2441,            2106,            1305,            2630,            3517,            3832,            1874,            2704,            3947,            967,            2339,            2337,            3216,            4046,            3779,            4511,            3848,            1868,            2318,            2598,            2359,            4790,            3956,            2248,            3379,            1544,            3092,            3694,            2681,            2769,            2403,            2482,            2527,            2122,            4180,            1492,            2368,            1660,            4445,            4651,            4091,            5372,            3762,            4528,            3030,            2046,            1446,            2891,            3117,            4402,            4277,            1807,            1390,            ],        [            3982,            4330,            4691,            4207,            19933,            4425,            2144,            1120,            3862,            2308,            2222,            1935,            468,            3253,            2508,            2373,            2776,            2537,            2245,            2826,            5153,            0,            978,            2744,            776,            472,            615,            1651,            661,            3404,            2941,            983,            2591,            3056,            1909,            2694,            2692,            3103,            2077,            3666,            3620,            3735,            4241,            4691,            965,            1468,            3899,            1987,            2603,            2488,            2529,            3447,            1725,            1790,            1878,            2758,            253,            307,            2477,            2877,            3865,            1477,            1296,            5235,            4696,            3200,            4069,            3649,            4415,            2139,            910,            1510,            2778,            1148,            4289,            3386,            1149,            2375,            ],        [            4640,            4287,            5417,            4577,            19730,            4518,            2793,            142,            4511,            2403,            3129,            1708,            1117,            3902,            2281,            3022,            2605,            3186,            2018,            3061,            5246,            545,            0,            2517,            202,            1017,            850,            1886,            873,            4130,            2511,            1632,            3240,            2774,            2558,            3343,            3341,            3752,            1850,            4392,            3338,            4461,            4334,            4784,            1200,            2117,            3481,            1760,            3252,            3137,            2624,            4096,            1498,            2439,            2527,            3407,            798,            542,            3126,            2650,            3958,            2126,            1531,            5884,            4469,            2918,            3842,            4375,            4563,            2788,            1145,            1745,            3427,            921,            4814,            2956,            1384,            2470,            ],        [            3026,            2741,            3735,            3031,            18444,            6287,            4562,            2427,            3477,            4929,            4449,            1327,            2437,            3312,            954,            4791,            1139,            2732,            1637,            5020,            7015,            2777,            2569,            0,            2771,            2476,            3114,            4087,            2241,            2448,            965,            2952,            2100,            1080,            3878,            2889,            3761,            2540,            917,            2710,            1656,            2779,            6103,            6553,            3464,            2541,            1936,            1596,            4016,            1613,            5150,            3062,            2022,            3369,            2951,            2287,            2524,            2806,            2568,            1104,            5727,            2188,            3795,            5294,            2923,            1224,            2296,            2693,            3017,            3718,            3409,            4228,            2215,            2252,            3333,            1410,            3648,            4996,            ],        [            4429,            4524,            5138,            4814,            19528,            4316,            2591,            344,            4309,            2201,            2927,            1945,            915,            3700,            2518,            2820,            3223,            2984,            2255,            2859,            5186,            343,            202,            2754,            0,            815,            648,            1684,            671,            3851,            2748,            1430,            3038,            3011,            2356,            3141,            3139,            3550,            2087,            4113,            3575,            4182,            4132,            4582,            998,            1915,            3718,            1997,            3050,            2935,            2422,            3894,            1735,            2237,            2325,            3205,            596,            340,            2924,            2887,            3898,            1924,            1329,            5682,            4706,            3155,            4079,            4096,            4800,            2586,            943,            1543,            3225,            1158,            4736,            3193,            1182,            2268,            ],        [            4395,            4042,            5172,            4332,            19046,            3834,            2109,            851,            3827,            1719,            2445,            1463,            433,            3218,            2036,            2338,            2360,            2502,            1773,            3235,            4562,            683,            993,            2272,            1185,            0,            1024,            2060,            189,            3885,            2266,            948,            2556,            2529,            1874,            2659,            2657,            3068,            1605,            4147,            3093,            4216,            3650,            4100,            1374,            1433,            3236,            1515,            2568,            2453,            1940,            3412,            1253,            1755,            1843,            2723,            430,            716,            2442,            2405,            3274,            1442,            1705,            5200,            4224,            2673,            3597,            4130,            4318,            2104,            1319,            1919,            2743,            676,            4569,            2711,            1558,            1786,            ],        [            4510,            4605,            5219,            4895,            19609,            4397,            2672,            505,            4390,            2282,            3008,            2026,            996,            3781,            2599,            2901,            3304,            3065,            2336,            2940,            5125,            424,            363,            2835,            161,            896,            0,            1765,            752,            3932,            2829,            1511,            3119,            3092,            2437,            3222,            3220,            3631,            2168,            4194,            3656,            4263,            4213,            4663,            1079,            1996,            3799,            2078,            3131,            3016,            2503,            3975,            1816,            2318,            2406,            3286,            677,            421,            3005,            2968,            3837,            2005,            1410,            5763,            4787,            3236,            4160,            4177,            4881,            2667,            1024,            1624,            3306,            1239,            4817,            3274,            1263,            2349,            ],        [            3398,            3746,            4107,            3623,            18366,            3154,            1092,            1813,            2810,            626,            984,            3207,            1757,            2201,            3588,            1321,            2970,            1485,            3517,            1555,            3882,            1732,            1671,            3350,            1469,            1744,            1308,            0,            1933,            2820,            3135,            1177,            2007,            3250,            413,            1642,            1640,            2519,            3349,            3082,            3814,            3151,            2970,            3420,            1658,            1662,            4093,            3259,            1551,            2682,            847,            2395,            2997,            1984,            2072,            1706,            1785,            1729,            1425,            3483,            2594,            1671,            355,            4183,            3954,            3394,            4675,            3065,            3831,            2333,            741,            141,            2194,            2420,            3705,            3580,            502,            693,            ],        [            4206,            3853,            4983,            4143,            19363,            4151,            2426,            662,            4144,            2036,            2762,            1274,            750,            3535,            1847,            2655,            2171,            2819,            1584,            3333,            4879,            1090,            804,            2083,            1006,            848,            1349,            2400,            0,            3696,            2077,            1265,            2873,            2340,            2191,            2976,            2974,            3385,            1416,            3958,            2904,            4027,            3967,            4417,            1699,            1750,            3047,            1326,            2885,            2770,            2257,            3729,            1064,            2072,            2160,            3040,            837,            1041,            2759,            2216,            3591,            1759,            2030,            5517,            4035,            2484,            3408,            3941,            4129,            2421,            1644,            2244,            3060,            487,            4380,            2522,            1883,            2103,            ],        [            821,            1169,            1530,            1046,            16461,            3777,            2052,            3496,            2209,            2393,            4178,            2396,            3506,            2044,            2426,            2281,            1808,            1461,            2706,            4749,            4505,            3568,            3638,            2188,            3840,            3274,            4183,            3816,            3463,            0,            1973,            2681,            985,            2088,            3607,            1618,            2493,            1385,            2369,            262,            788,            574,            3593,            4043,            4533,            2448,            1710,            3127,            2748,            1520,            2614,            1794,            3553,            2098,            2858,            1016,            3315,            3875,            1297,            2321,            3217,            2095,            4171,            2732,            1377,            2232,            1562,            488,            1254,            2447,            4478,            3957,            1060,            3321,            1128,            2418,            4318,            2460,            ],        [            2061,            2409,            2770,            2286,            17701,            4695,            2970,            2624,            2511,            2580,            3306,            1524,            2634,            2346,            1554,            3199,            936,            1766,            1834,            3877,            5423,            2696,            2766,            1316,            2968,            2402,            3311,            2944,            2591,            1483,            0,            1809,            1134,            794,            2735,            1923,            2795,            1574,            1497,            1745,            691,            1814,            4511,            4961,            3661,            1576,            970,            2255,            3050,            648,            2801,            2096,            2681,            1492,            1986,            1321,            2443,            3003,            1602,            1449,            4135,            1223,            3299,            4328,            2617,            650,            2802,            1728,            2494,            1841,            3606,            3085,            1249,            2449,            2368,            445,            3446,            2647,            ],        [            3467,            3815,            4176,            3692,            18098,            2886,            1161,            2300,            2879,            771,            1497,            2912,            1902,            2270,            3485,            1390,            3115,            1554,            3222,            2068,            3614,            1743,            2807,            3495,            2605,            1449,            2444,            1135,            1638,            2889,            3280,            0,            2076,            3395,            926,            1711,            1709,            2588,            3054,            3151,            2804,            3220,            2702,            3152,            2793,            945,            3726,            2964,            1620,            2827,            992,            2464,            2702,            2129,            1501,            1775,            1490,            2050,            1494,            3628,            2326,            1816,            1490,            4252,            4023,            3539,            4208,            3134,            3900,            2478,            1876,            1276,            2263,            2125,            3774,            3725,            1637,            838,            ],        [            1391,            1739,            2100,            1616,            17031,            3719,            1994,            3436,            3191,            3392,            4118,            2336,            3446,            3026,            2366,            2223,            1748,            1403,            2646,            4689,            4447,            3508,            3578,            2128,            3780,            3214,            4123,            3756,            3403,            813,            1913,            2621,            0,            2028,            3547,            1560,            3475,            1327,            2309,            1075,            728,            1144,            3535,            3985,            4473,            2388,            1650,            3067,            3730,            1460,            3613,            2776,            3493,            2040,            2798,            958,            3255,            3815,            1239,            2261,            3159,            2035,            4111,            3302,            1947,            2172,            2132,            1058,            1824,            2389,            4418,            3897,            1002,            3261,            1698,            2358,            4258,            3459,            ],        [            2570,            2918,            3279,            2795,            18225,            4922,            3453,            2126,            2994,            3063,            3789,            1423,            2136,            2829,            1067,            3682,            835,            2606,            1336,            4360,            5906,            2476,            2268,            705,            2470,            2885,            2813,            3427,            1940,            1992,            483,            2292,            1617,            0,            3218,            2763,            3278,            2057,            1396,            2254,            1200,            2323,            4994,            5444,            3163,            2059,            1453,            2154,            3533,            1131,            3284,            2579,            2580,            1975,            2469,            2161,            2223,            2505,            2442,            885,            4618,            1706,            3782,            4811,            3126,            144,            2077,            2237,            3003,            2324,            3108,            3568,            1732,            1951,            2877,            349,            3347,            3130,            ],        [            3507,            3855,            4216,            3732,            17953,            2741,            1201,            2023,            2919,            735,            571,            3316,            1866,            2310,            3697,            1430,            3079,            1594,            3626,            1142,            3469,            1942,            1881,            3459,            1679,            1853,            1518,            717,            2042,            2929,            3244,            1286,            2116,            3359,            0,            1751,            1749,            2628,            3458,            3191,            3923,            3260,            2557,            3007,            2010,            1771,            4202,            3368,            1660,            2791,            956,            2504,            3106,            2093,            2181,            1815,            1894,            1939,            1534,            3592,            2181,            1780,            1072,            5134,            4063,            3503,            4784,            3174,            3940,            2442,            1458,            858,            2303,            2529,            3814,            3689,            1219,            802,            ],        [            2315,            2663,            3024,            2540,            16772,            2680,            955,            3265,            1672,            1296,            2022,            3260,            2427,            1063,            3503,            1184,            2672,            157,            3570,            2593,            3408,            2708,            3407,            3110,            3130,            2414,            2969,            1660,            2603,            1737,            2837,            2100,            924,            2952,            1451,            0,            696,            1436,            3233,            1999,            1652,            2068,            2496,            2946,            3461,            1165,            2574,            3929,            951,            2384,            1517,            1257,            3667,            1638,            1054,            623,            2455,            3015,            342,            3185,            2120,            1369,            2015,            3045,            2871,            3096,            3056,            1982,            2748,            1987,            2401,            1801,            1111,            3090,            2622,            3282,            2162,            1363,            ],        [            3192,            3540,            3901,            3417,            17738,            2526,            801,            3137,            3108,            1168,            1894,            3749,            2299,            2499,            4130,            1030,            3512,            1279,            4059,            2465,            3254,            2580,            3279,            3892,            3002,            2286,            2841,            1532,            2475,            2614,            3677,            1719,            1801,            3792,            1323,            1436,            0,            2313,            3891,            2876,            2529,            2945,            2342,            2792,            3190,            1538,            3451,            3801,            255,            3224,            1389,            2693,            3539,            2526,            1304,            1500,            2327,            2887,            1219,            4025,            1966,            2213,            1887,            4919,            3748,            3936,            3933,            2859,            3625,            2875,            2273,            1673,            1988,            2962,            3499,            4122,            2034,            1235,            ],        [            2009,            2357,            2718,            2234,            17649,            4656,            2931,            2585,            2273,            2541,            3267,            1485,            2595,            2108,            1515,            3160,            897,            1528,            1795,            3838,            5384,            2657,            2727,            1277,            2929,            2363,            3272,            2905,            2552,            1431,            1062,            1770,            896,            1177,            2696,            1685,            2557,            0,            1458,            1693,            639,            1762,            4472,            4922,            3622,            1537,            732,            2216,            2812,            609,            2762,            1858,            2642,            1453,            1947,            1083,            2404,            2964,            1364,            1410,            4096,            1184,            3260,            4090,            2565,            1321,            2602,            1676,            2442,            1802,            3567,            3046,            1011,            2410,            2316,            1507,            3407,            2608,            ],        [            3189,            2836,            3966,            3126,            18539,            4921,            3196,            1510,            3571,            2806,            3532,            410,            1520,            3406,            431,            3425,            1154,            3589,            720,            4103,            5649,            1860,            1652,            564,            1854,            1559,            2197,            3170,            1324,            2679,            1060,            2035,            2194,            1323,            2961,            3746,            3744,            2634,            0,            2941,            1887,            3010,            4737,            5187,            2547,            2520,            2030,            963,            3655,            1708,            3027,            3156,            1389,            2842,            2930,            2848,            1607,            1889,            3529,            1199,            4361,            2529,            2878,            5168,            3018,            1467,            2391,            2924,            3112,            3191,            2492,            3311,            2309,            1335,            3564,            1505,            2731,            2873,            ],        [            1189,            1537,            1898,            1414,            16829,            3515,            1790,            3234,            1947,            2131,            3916,            2134,            3244,            1782,            2164,            2019,            1546,            1199,            2444,            4487,            4243,            3306,            3376,            1926,            3578,            3012,            3921,            3554,            3201,            611,            1711,            2419,            723,            1826,            3345,            1356,            2231,            1123,            2107,            0,            526,            942,            3331,            3781,            4271,            2186,            1448,            2865,            2486,            1258,            2352,            1532,            3291,            1836,            2596,            754,            3053,            3613,            1035,            2059,            2955,            1833,            3909,            3100,            1745,            1970,            1930,            856,            1622,            2185,            4216,            3695,            798,            3059,            1496,            2156,            4056,            2198,            ],        [            1370,            1718,            2079,            1595,            17010,            3722,            1997,            3441,            2154,            2338,            4123,            2341,            3451,            1989,            2371,            2226,            1753,            1406,            2651,            4694,            4450,            3513,            3583,            2133,            3785,            3219,            4128,            3761,            3408,            792,            1918,            2626,            930,            2033,            3552,            1563,            2438,            1330,            2314,            1054,            0,            1123,            3538,            3988,            4478,            2393,            1655,            3072,            2693,            1465,            2559,            1739,            3498,            2043,            2803,            961,            3260,            3820,            1242,            2266,            3162,            2040,            4116,            3281,            1926,            2177,            2111,            1037,            1803,            2392,            4423,            3902,            1005,            3266,            1677,            2363,            4263,            2405,            ],        [            1172,            1520,            1248,            1397,            15599,            4128,            2403,            3847,            2560,            2744,            4529,            2747,            3857,            2715,            2777,            2632,            2159,            1812,            3057,            5100,            4856,            3919,            3989,            2539,            4191,            3625,            4534,            4167,            3814,            351,            2324,            3032,            1336,            2439,            3958,            1969,            2844,            1736,            2720,            613,            1139,            0,            3944,            4394,            4884,            2799,            2061,            3478,            3099,            1871,            2965,            2145,            3904,            2449,            3209,            1367,            3666,            4226,            1648,            2672,            3568,            2446,            4522,            2450,            1728,            2583,            1913,            839,            1605,            2798,            4829,            4308,            1411,            3672,            1479,            2769,            4669,            2811,            ],        [            3623,            3971,            4332,            3848,            17037,            1825,            1292,            3568,            3539,            1599,            2233,            4180,            2730,            2930,            4561,            1521,            3943,            1710,            4490,            1505,            1074,            3011,            3543,            4323,            3341,            2717,            3180,            1963,            2906,            3045,            4108,            2150,            2232,            4223,            1754,            1867,            2563,            2744,            4322,            3307,            2960,            3376,            0,            612,            3672,            2635,            3882,            4232,            2100,            3655,            1820,            3124,            3970,            2957,            1735,            1931,            2758,            3318,            1650,            4456,            1265,            2644,            2318,            4218,            4179,            4367,            4364,            3290,            4056,            3306,            2704,            2104,            2419,            3393,            3930,            4553,            2465,            1666,            ],        [            4557,            4905,            5266,            4782,            17726,            2514,            2251,            3073,            3969,            1785,            1621,            4366,            2916,            3360,            4747,            2480,            4129,            2644,            4676,            893,            462,            2992,            2931,            4509,            2729,            2903,            2568,            1767,            3092,            3979,            4294,            2336,            3166,            4409,            1429,            2801,            2799,            3678,            4508,            4241,            3894,            4310,            2330,            0,            3060,            2821,            5252,            4418,            2710,            3841,            2006,            3554,            4156,            3143,            3231,            2865,            2944,            2989,            2584,            4642,            1954,            2830,            2122,            4907,            5113,            4553,            5298,            4224,            4990,            3492,            2508,            1908,            3353,            3579,            4864,            4739,            2269,            1852,            ],        [            4115,            4463,            4824,            4340,            18672,            3460,            1809,            3312,            3527,            1343,            1257,            3924,            2474,            2918,            4305,            2038,            3687,            2202,            4234,            1861,            4188,            2755,            2567,            4067,            2365,            2461,            2204,            1325,            2650,            3537,            3852,            1894,            2724,            3967,            987,            2359,            2357,            3236,            4066,            3799,            4531,            3868,            3276,            3726,            0,            2379,            4810,            3976,            2268,            3399,            1564,            3112,            3714,            2701,            2789,            2423,            2502,            3062,            2142,            4200,            2900,            2388,            1680,            5853,            4671,            4111,            5392,            3782,            4548,            3050,            2066,            1466,            2911,            3137,            4422,            4297,            1827,            1410,            ],        [            3296,            3644,            4005,            3521,            17947,            2735,            1010,            2346,            2728,            1351,            2077,            2565,            1482,            2119,            2595,            1239,            1977,            1383,            2875,            2648,            3463,            1789,            2488,            2357,            2565,            1495,            2404,            1715,            1684,            2718,            2142,            902,            1905,            2257,            1506,            1540,            1558,            2417,            2538,            2980,            2633,            3049,            2551,            3001,            2754,            0,            3100,            3010,            1469,            1689,            1572,            2313,            2748,            991,            410,            1604,            1536,            2096,            1323,            2490,            2175,            678,            2070,            4101,            3852,            2401,            4037,            2963,            3729,            1340,            2699,            1856,            2092,            2171,            3603,            2587,            2217,            1418,            ],        [            3402,            3750,            4289,            3627,            17085,            4351,            2626,            4962,            1541,            2993,            3719,            4006,            4124,            1376,            4588,            2855,            3418,            2392,            4316,            4290,            5079,            4405,            5104,            4195,            4827,            4111,            4666,            3357,            4300,            2824,            3583,            3544,            2011,            3698,            3148,            2235,            1825,            2521,            4551,            3086,            3160,            4295,            4167,            4617,            5158,            4029,            0,            5626,            2080,            3130,            3214,            1126,            5364,            4351,            3129,            2154,            4152,            4712,            2577,            4738,            3791,            3705,            3712,            3358,            3958,            3842,            4143,            3069,            3835,            4700,            4098,            3498,            2198,            4787,            3709,            4028,            3859,            3060,            ],        [            3736,            3383,            4513,            3673,            19086,            5315,            3590,            1904,            4118,            3200,            3926,            804,            1914,            3953,            1377,            3819,            1701,            3983,            1114,            4497,            6043,            2254,            2046,            1613,            2248,            1953,            2591,            3564,            1718,            3226,            1607,            2429,            2741,            1870,            3355,            4140,            4138,            3181,            946,            3488,            2434,            3557,            5131,            5581,            2941,            2914,            2577,            0,            4049,            2255,            3421,            3703,            594,            3236,            3324,            3395,            2001,            2283,            3923,            1746,            4755,            2923,            3272,            5715,            3565,            2014,            2938,            3471,            3659,            3585,            2886,            3705,            2856,            1729,            3910,            2052,            3125,            3267,            ],        [            2937,            3285,            3646,            3162,            17483,            2271,            546,            2882,            2853,            913,            1639,            3494,            2044,            2244,            3875,            775,            3257,            1024,            3804,            2210,            2999,            2325,            3024,            3637,            2747,            2031,            2586,            1277,            2220,            2359,            3422,            1464,            1546,            3537,            1068,            1181,            1877,            2058,            3636,            2621,            2274,            2690,            2087,            2537,            3078,            1949,            3196,            3546,            0,            2969,            1134,            2438,            3284,            2271,            1049,            1245,            2072,            2632,            964,            3770,            1711,            1958,            1632,            4664,            3493,            3681,            3678,            2604,            3370,            2620,            2018,            1418,            1733,            2707,            3244,            3867,            1779,            980,            ],        [            2502,            2850,            3211,            2727,            18141,            5387,            3662,            1976,            2952,            3272,            3998,            876,            1986,            2787,            906,            3891,            288,            2538,            1186,            4569,            6115,            2326,            2118,            668,            2320,            2025,            2663,            3636,            1790,            1924,            453,            2501,            1575,            568,            3427,            2695,            3236,            2015,            849,            2186,            1132,            2255,            5203,            5653,            3013,            2029,            1411,            1607,            3491,            0,            3493,            2537,            2033,            3308,            2439,            2093,            2073,            2355,            2374,            801,            4827,            1676,            3344,            4413,            2620,            712,            1993,            2169,            2935,            3657,            2958,            3777,            1690,            1801,            2809,            898,            3197,            3339,            ],        [            3329,            3677,            4038,            3554,            18428,            3216,            1491,            1748,            3209,            1101,            1827,            2360,            910,            2600,            2741,            1720,            2123,            1884,            2670,            2398,            3944,            1191,            1890,            2503,            1967,            897,            1806,            1465,            1086,            2751,            2288,            330,            1938,            2403,            1256,            2041,            2039,            2450,            2502,            3013,            2967,            3082,            3032,            3482,            2156,            815,            3246,            2412,            1950,            1835,            0,            2794,            2150,            1137,            1225,            2105,            938,            1498,            1824,            2636,            2656,            824,            1820,            4582,            3885,            2547,            3828,            2996,            3762,            1486,            2101,            1606,            2125,            1573,            3636,            2733,            1967,            1168,            ],        [            2276,            2624,            2985,            2501,            17252,            4518,            2793,            3980,            415,            3160,            3886,            2880,            3990,            1543,            3462,            3022,            2292,            1473,            3190,            4457,            5246,            4052,            4122,            3069,            4324,            3758,            4667,            3524,            3947,            1698,            2457,            3165,            885,            2572,            3315,            1630,            1992,            1395,            3425,            1960,            2034,            2029,            4334,            4784,            5017,            2932,            2127,            4104,            2247,            2004,            3381,            0,            4530,            2110,            3342,            1028,            3799,            4359,            1309,            3612,            3958,            2579,            3879,            3525,            2832,            2716,            3017,            1943,            2709,            2459,            4265,            3665,            1072,            3805,            2583,            2902,            4026,            3227,            ],        [            3142,            2789,            3919,            3079,            18492,            4721,            2996,            1310,            3524,            2606,            3332,            210,            1320,            3359,            783,            3225,            1107,            3389,            520,            3903,            5449,            1660,            1452,            1019,            1654,            1359,            1997,            2970,            1124,            2632,            1013,            1835,            2147,            1276,            2761,            3546,            3544,            2587,            352,            2894,            1840,            2963,            4537,            4987,            2347,            2320,            1983,            1189,            3455,            1661,            2827,            3109,            0,            2642,            2730,            2801,            1407,            1689,            3329,            1152,            4161,            2329,            2678,            5341,            2971,            1420,            2344,            2877,            3065,            2991,            2292,            3111,            2262,            1135,            3316,            1458,            2531,            2673,            ],        [            3200,            3548,            3909,            3425,            18839,            6085,            4360,            2674,            3650,            3970,            4696,            1574,            2684,            3485,            1604,            4589,            986,            3236,            1884,            5267,            6813,            3024,            2816,            1366,            3018,            2723,            3361,            4334,            2488,            2622,            1151,            3199,            2273,            1266,            4125,            3393,            3934,            2713,            1547,            2884,            1830,            2953,            5901,            6351,            3711,            2727,            2109,            2305,            4189,            698,            4191,            3235,            2731,            0,            3137,            2791,            2771,            3053,            3072,            1499,            5525,            2374,            4042,            5111,            3318,            1410,            2691,            2867,            3633,            349,            3656,            4475,            2388,            2499,            3507,            1596,            3895,            4037,            ],        [            2886,            3234,            3595,            3111,            17537,            2325,            600,            2580,            2318,            941,            1667,            2799,            1716,            1709,            2829,            829,            2211,            973,            3109,            2238,            3053,            2023,            2722,            2591,            2799,            1729,            2638,            1305,            1918,            2308,            2376,            1136,            1495,            2491,            1096,            1130,            1148,            2007,            2772,            2570,            2223,            2639,            2141,            2591,            2988,            623,            3145,            3244,            1059,            1923,            1162,            1903,            2982,            1225,            0,            1194,            1770,            2330,            913,            2724,            1765,            912,            1660,            3691,            3442,            2635,            3627,            2553,            3319,            1574,            2046,            1446,            1682,            2405,            3193,            2821,            1807,            1008,            ],        [            1692,            2040,            2401,            1917,            17332,            2761,            1036,            3235,            1681,            1377,            2103,            2637,            2508,            1516,            2880,            1265,            2049,            445,            2947,            2674,            3489,            2678,            3377,            2487,            3211,            2384,            3293,            1741,            2573,            1114,            2214,            1791,            301,            2329,            1532,            602,            1298,            813,            2610,            1376,            1029,            1445,            2577,            3027,            3643,            1246,            1951,            3368,            1495,            1761,            1598,            1266,            3794,            1474,            1135,            0,            2425,            2985,            281,            2562,            2201,            1205,            2096,            3498,            2248,            2473,            2433,            1359,            2125,            1823,            2482,            1882,            488,            3060,            1999,            2659,            2243,            1444,            ],        [            3729,            4077,            4438,            3954,            18828,            3616,            1891,            1070,            3609,            1501,            2227,            1682,            215,            3000,            2255,            2120,            2523,            2284,            1992,            3079,            4344,            253,            1231,            2491,            1029,            219,            868,            1904,            408,            3151,            2688,            730,            2338,            2803,            1656,            2441,            2439,            2850,            1824,            3413,            3367,            3482,            3432,            3882,            1218,            1215,            3646,            1734,            2350,            2235,            1722,            3194,            1472,            1537,            1625,            2505,            0,            560,            2224,            2624,            3056,            1224,            1549,            4982,            4443,            2947,            3816,            3396,            4162,            1886,            1163,            1763,            2525,            895,            4036,            3133,            1402,            1568,            ],        [            4818,            5121,            5527,            5203,            19330,            4118,            2467,            813,            4185,            2001,            1915,            2334,            1304,            3576,            2907,            2696,            3612,            2860,            2644,            2519,            4846,            732,            671,            3143,            469,            1204,            308,            1344,            1060,            4195,            3137,            1819,            3382,            3400,            1645,            3017,            3015,            3894,            2476,            4457,            3964,            4526,            3934,            4384,            658,            2304,            4107,            2386,            2926,            3324,            2222,            3770,            2124,            2626,            2714,            3081,            985,            0,            2800,            3276,            3558,            2313,            989,            6511,            5095,            3544,            4468,            4440,            5189,            2975,            603,            1203,            3569,            1547,            5125,            3582,            842,            2068,            ],        [            1973,            2321,            2682,            2198,            17613,            2480,            755,            3065,            1993,            1096,            1822,            2918,            2227,            1384,            3161,            984,            2330,            164,            3228,            2393,            3208,            2508,            3207,            2768,            2930,            2214,            2769,            1460,            2403,            1395,            2495,            1900,            582,            2610,            1251,            321,            1017,            1094,            2891,            1657,            1310,            1726,            2296,            2746,            3118,            965,            2232,            3729,            1214,            2042,            1317,            1578,            3467,            1755,            854,            281,            2255,            2815,            0,            2843,            1920,            1486,            1815,            3366,            2529,            2754,            2714,            1640,            2406,            2104,            2201,            1601,            769,            2890,            2280,            2940,            1962,            1163,            ],        [            1990,            1637,            2771,            1927,            17340,            6154,            4429,            2743,            3789,            4039,            4765,            2040,            2753,            3624,            1684,            4658,            1452,            3044,            1953,            5336,            6882,            3093,            2885,            1322,            3087,            2792,            3430,            4403,            2557,            2238,            1278,            3268,            2412,            1393,            4194,            3201,            4073,            2852,            2013,            2281,            1969,            3268,            5970,            6420,            3780,            2854,            2248,            2771,            4328,            1926,            4260,            3374,            3197,            4075,            3264,            2599,            2840,            3122,            2880,            0,            5594,            2501,            4111,            4430,            1819,            1537,            1192,            1677,            1913,            4424,            3725,            4544,            2527,            2568,            2164,            1723,            3964,            4106,            ],        [            5677,            6025,            4797,            5902,            15772,            560,            4666,            6457,            4095,            4488,            5122,            6560,            5619,            3218,            7142,            4895,            5972,            4432,            6870,            4394,            3963,            5900,            6432,            6749,            6230,            5606,            6069,            4852,            5795,            5154,            6137,            5039,            4565,            6252,            4643,            4275,            3865,            5075,            7105,            5416,            5714,            4803,            3051,            3501,            6561,            5524,            5807,            7121,            4120,            5684,            4709,            3680,            6859,            5846,            4624,            4820,            5647,            6207,            4539,            7292,            0,            5533,            5207,            2953,            6233,            6396,            6418,            5642,            6110,            6195,            5593,            4993,            4752,            6282,            5984,            6582,            5354,            4555,            ],        [            2505,            2853,            3214,            2730,            19152,            3472,            1747,            2030,            3465,            1357,            2083,            1887,            1166,            2856,            1917,            1976,            1299,            1150,            2197,            2654,            4200,            1473,            2172,            1679,            2249,            1179,            2088,            1721,            1368,            1927,            1464,            586,            1114,            1579,            1512,            1307,            2295,            1626,            1860,            2189,            2143,            2258,            3288,            3738,            2438,            353,            2422,            2694,            2206,            1011,            1578,            3050,            2432,            313,            763,            1205,            1220,            1780,            1090,            1812,            2912,            0,            2076,            4838,            3061,            1723,            3004,            2172,            2938,            662,            2383,            1862,            1301,            1855,            2812,            1909,            2223,            1424,            ],        [            3753,            4101,            4462,            3978,            18384,            3172,            1447,            1458,            3165,            981,            1339,            2979,            1949,            2556,            3552,            1676,            3325,            1840,            3289,            1910,            3900,            1377,            1316,            3705,            1114,            2099,            953,            355,            1705,            3175,            3490,            1532,            2362,            3605,            768,            1997,            1995,            2874,            3121,            3437,            3090,            3506,            2988,            3438,            1303,            2017,            4448,            3031,            1906,            3037,            1202,            2750,            2769,            2339,            2427,            2061,            1630,            1374,            1780,            3921,            2612,            2026,            0,            4538,            4309,            3749,            4494,            3420,            4186,            2688,            386,            214,            2549,            2192,            4060,            3935,            147,            1048,            ],        [            3003,            3351,            3200,            3228,            15774,            3438,            1713,            4049,            1142,            2080,            2806,            3607,            3211,            265,            4189,            1942,            3019,            1479,            3917,            3377,            4166,            3492,            4191,            3796,            3914,            3198,            3753,            2444,            3387,            2425,            3184,            2631,            1612,            3299,            2235,            1322,            912,            2122,            4152,            2687,            2761,            3206,            3254,            3704,            4102,            3116,            2854,            4713,            1167,            2731,            2301,            727,            4451,            3438,            2216,            1755,            3239,            3799,            1664,            4339,            2878,            3125,            2799,            0,            3559,            3443,            3744,            2670,            3436,            3787,            3185,            2585,            1799,            3874,            3310,            3629,            2946,            2147,            ],        [            621,            268,            1813,            969,            16384,            5004,            3279,            4007,            3436,            3620,            5405,            3304,            4017,            3737,            2948,            3508,            2716,            2688,            3217,            5976,            5732,            4795,            4149,            2586,            4351,            4501,            5410,            5043,            4690,            1446,            2542,            3908,            2212,            2657,            4834,            2845,            4384,            2612,            3277,            1489,            2015,            1777,            4820,            5270,            5760,            3675,            2937,            4035,            4639,            2747,            3841,            3021,            4461,            3325,            4085,            2243,            4542,            5102,            2524,            2805,            4444,            3322,            5398,            3472,            0,            2801,            661,            885,            1181,            3674,            5705,            5184,            2287,            3832,            795,            2987,            5545,            3687,            ],        [            2223,            2571,            2932,            2448,            17863,            5483,            3132,            2786,            2673,            2742,            3468,            1686,            2796,            2508,            1211,            3361,            1098,            1928,            1996,            4039,            5585,            2858,            2928,            849,            3130,            2564,            3473,            3106,            2753,            1645,            627,            1971,            1296,            144,            2897,            2085,            2957,            1736,            1659,            1907,            853,            1976,            4673,            5123,            3823,            1738,            1132,            2417,            3212,            810,            2963,            2258,            2843,            1654,            2148,            1483,            2605,            3165,            1764,            1029,            4297,            1385,            3461,            4490,            2779,            0,            2221,            1890,            2656,            2003,            3768,            3247,            1411,            2611,            2530,            205,            3608,            2809,            ],        [            827,            474,            1862,            1018,            16776,            4887,            3162,            3350,            4396,            3503,            5372,            2647,            3360,            3786,            2291,            3391,            2059,            2571,            2560,            5943,            5615,            3700,            3492,            1929,            3694,            3399,            4037,            5010,            3164,            1329,            1885,            3875,            3019,            2000,            4801,            2728,            4433,            2495,            2620,            1372,            1898,            2359,            4703,            5153,            4387,            3461,            2855,            3378,            4688,            2533,            3724,            3981,            3804,            3208,            3871,            2126,            3447,            3729,            2407,            2148,            4327,            3108,            4718,            3521,            656,            2144,            0,            768,            739,            3557,            4332,            5151,            2170,            3175,            1001,            2330,            4571,            3570,            ],        [            333,            681,            1848,            558,            15973,            4119,            2394,            3838,            2551,            2735,            4520,            2738,            3848,            2386,            2768,            2623,            2150,            1803,            3048,            5091,            4847,            3910,            3980,            2530,            4182,            3616,            4525,            4158,            3805,            561,            2315,            3023,            1327,            2430,            3949,            1960,            2835,            1727,            2711,            604,            1130,            892,            3935,            4385,            4875,            2790,            2052,            3469,            3090,            1862,            2956,            2136,            3895,            2440,            3200,            1358,            3657,            4217,            1639,            2663,            3559,            2437,            4513,            3050,            889,            2574,            1074,            0,            766,            2789,            4820,            4299,            1402,            3663,            640,            2760,            4660,            2802,            ],        [            2173,            2521,            3242,            2398,            17811,            6484,            4759,            3073,            4119,            4369,            5095,            2370,            3083,            3954,            2014,            4988,            1782,            3374,            2283,            5666,            7212,            3423,            3215,            1652,            3417,            3122,            3760,            4733,            2887,            2709,            1608,            3598,            2742,            1723,            4524,            3531,            4403,            3182,            2343,            2752,            2299,            3739,            6300,            6750,            4110,            3184,            2578,            3101,            4658,            2256,            4590,            3704,            3527,            4405,            3594,            2929,            3170,            3452,            3210,            1871,            5924,            2831,            4441,            4901,            2027,            1867,            1699,            2148,            0,            4754,            4055,            4874,            2857,            2898,            2480,            2053,            4294,            4436,            ],        [            2851,            3199,            3560,            3076,            18490,            5736,            4011,            2325,            3301,            3621,            4347,            1225,            2335,            3136,            1255,            4240,            637,            2887,            1535,            4918,            6464,            2675,            2467,            1017,            2669,            2374,            3012,            3985,            2139,            2273,            802,            2850,            1924,            917,            3776,            3044,            3585,            2364,            1198,            2535,            1481,            2604,            5552,            6002,            3362,            2378,            1760,            1956,            3840,            349,            3842,            2886,            2382,            3657,            2788,            2442,            2422,            2704,            2723,            1150,            5176,            2025,            3693,            4762,            2969,            1061,            2342,            2518,            3284,            0,            3307,            4126,            2039,            2150,            3158,            1247,            3546,            3688,            ],        [            4139,            4487,            4848,            4364,            19589,            4377,            1833,            1072,            3551,            1367,            1725,            2593,            1563,            2942,            3166,            2062,            3871,            2226,            2903,            2778,            5105,            991,            930,            3402,            728,            1463,            567,            741,            1319,            3561,            3396,            2078,            2748,            3659,            1154,            2383,            2381,            3260,            2735,            3823,            3476,            3892,            4193,            4643,            917,            2563,            4366,            2645,            2292,            3583,            1588,            3136,            2383,            2885,            2973,            2447,            1244,            988,            2166,            3535,            3817,            2572,            386,            4924,            5354,            3803,            4727,            3806,            4572,            3234,            0,            600,            2935,            1806,            4446,            3841,            239,            1434,            ],        [            3539,            3887,            4248,            3764,            18170,            2958,            1233,            1672,            2951,            767,            1125,            3348,            1898,            2342,            3729,            1462,            3111,            1626,            3658,            1696,            3686,            1591,            1530,            3491,            1328,            1885,            1167,            141,            2074,            2961,            3276,            1318,            2148,            3391,            554,            1783,            1781,            2660,            3490,            3223,            2876,            3292,            2774,            3224,            1517,            1803,            4234,            3400,            1692,            2823,            988,            2536,            3138,            2125,            2213,            1847,            1926,            1588,            1566,            3624,            2398,            1812,            214,            4324,            4095,            3535,            4280,            3206,            3972,            2474,            600,            0,            2335,            2561,            3846,            3721,            361,            834,            ],        [            2334,            2682,            3043,            2559,            17974,            4981,            3256,            2910,            2598,            2866,            3592,            1810,            2920,            2433,            2392,            3485,            1222,            1853,            2120,            4163,            5709,            2982,            3052,            1999,            3254,            2688,            3597,            3230,            2877,            1756,            1387,            2095,            1221,            1502,            3021,            2010,            2882,            325,            2355,            2018,            964,            2087,            4797,            5247,            3947,            1862,            1057,            3034,            3137,            934,            3087,            2183,            3460,            1778,            2272,            1408,            2729,            3289,            1689,            2542,            4421,            1509,            3585,            4415,            2890,            1646,            3734,            2001,            2767,            2127,            3892,            3371,            0,            2735,            2641,            1832,            3732,            2933,            ],        [            3719,            3366,            4496,            3656,            19069,            4475,            2750,            904,            4101,            2360,            3086,            787,            1074,            3936,            1360,            2979,            1684,            3143,            1097,            3657,            5203,            1414,            1046,            1596,            1248,            1113,            1945,            2724,            1198,            3209,            1590,            1589,            2724,            1853,            2515,            3300,            3298,            3164,            929,            3471,            2417,            3540,            4291,            4741,            2295,            2074,            2560,            839,            3209,            2238,            2581,            3686,            577,            2396,            2484,            3378,            1161,            1637,            3083,            1729,            3915,            2083,            2626,            5918,            3548,            1997,            2921,            3454,            3642,            2745,            2240,            2865,            2839,            0,            3893,            2035,            2479,            2427,            ],        [            711,            948,            1903,            1059,            16474,            5094,            3369,            4514,            3526,            3710,            5495,            3811,            4524,            3827,            3455,            3598,            3223,            2778,            3724,            6066,            5822,            4885,            4656,            3093,            4858,            4591,            5500,            5133,            4780,            1536,            3049,            3998,            2302,            3164,            4924,            2935,            3810,            2702,            3784,            1579,            2105,            1867,            4910,            5360,            5850,            3765,            3027,            4542,            4065,            2837,            3931,            3111,            4968,            3415,            4175,            2333,            4632,            5192,            2614,            3312,            4534,            3412,            5488,            3562,            1156,            3308,            1341,            975,            1461,            3764,            5795,            5274,            2377,            4339,            0,            3494,            5635,            3777,            ],        [            2018,            2366,            2727,            2243,            17658,            4652,            2927,            2581,            2468,            2537,            3263,            1481,            2591,            2303,            1511,            3156,            893,            1723,            1791,            3834,            5380,            2653,            2723,            1273,            2925,            2359,            3268,            2901,            2548,            1440,            832,            1766,            1091,            349,            2692,            1880,            2752,            1531,            1454,            1702,            648,            1771,            4468,            4918,            3618,            1533,            927,            2212,            3007,            605,            2758,            2053,            2638,            1449,            1943,            1278,            2400,            2960,            1559,            1406,            4092,            1180,            3256,            4285,            2574,            205,            2759,            1685,            2451,            1798,            3563,            3042,            1206,            2406,            2325,            0,            3403,            2604,            ],        [            3900,            4248,            4609,            4125,            18531,            3319,            1594,            1311,            3312,            1128,            1486,            2832,            1802,            2703,            3405,            1823,            3472,            1987,            3142,            2057,            4047,            1230,            1169,            3852,            967,            1702,            806,            502,            1558,            3322,            3637,            1679,            2509,            3752,            915,            2144,            2142,            3021,            2974,            3584,            3237,            3653,            3135,            3585,            1156,            2164,            4595,            2884,            2053,            3184,            1349,            2897,            2622,            2486,            2574,            2208,            1483,            1227,            1927,            3774,            2759,            2173,            147,            4685,            4456,            3896,            4966,            3567,            4333,            2835,            239,            361,            2696,            2045,            4207,            4082,            0,            1195,            ],        [            3483,            3831,            4192,            3708,            18582,            3370,            1645,            1902,            3363,            679,            1981,            2514,            1064,            2754,            2895,            1874,            2277,            2038,            2824,            2552,            4098,            1345,            2044,            2657,            2121,            1051,            1960,            1619,            1240,            2905,            2442,            484,            2092,            2557,            1410,            2195,            2193,            2604,            2656,            3167,            3121,            3236,            3186,            3636,            2310,            969,            3400,            2566,            2104,            1989,            154,            2948,            2304,            1291,            1379,            2259,            1092,            1652,            1978,            2790,            2810,            978,            1974,            4736,            4039,            2701,            3982,            3150,            3916,            1640,            2255,            1760,            2279,            1727,            3790,            2887,            2494,            0,            ],        ], 'starts': [14, 26, 37, 56, 69], 'ends': [14, 26, 37, 56, 69]},    'NW-34': {'distance_matrix': [        [            0,            6205,            4135,            2764,            6542,            6891,            4481,            4795,            4926,            2285,            7200,            5604,            6275,            7771,            7624,            8349,            4683,            7370,            6343,            6909,            1304,            5796,            2019,            4560,            6657,            4833,            5472,            1297,            8774,            6334,            5145,            6631,            6577,            7436,            6908,            7202,            6874,            7512,            3968,            4565,            ],        [            3339,            0,            2059,            3511,            367,            1817,            2521,            2835,            1268,            3848,            2126,            1692,            1201,            2697,            2550,            3275,            2723,            2296,            1922,            1274,            3407,            592,            4346,            2600,            1583,            2873,            722,            3400,            3700,            576,            1049,            838,            1503,            910,            1273,            2128,            2946,            665,            7284,            2605,            ],        [            2666,            2070,            0,            2838,            2407,            2756,            462,            776,            791,            3175,            3065,            1019,            2140,            3636,            3489,            4214,            664,            3235,            1860,            2426,            2734,            1661,            3673,            541,            2522,            814,            1337,            2727,            4639,            2199,            1010,            2496,            2442,            2953,            2425,            3067,            2273,            3029,            6611,            546,            ],        [            1688,            3728,            3044,            0,            4065,            4414,            3506,            3820,            2449,            479,            4723,            2180,            3798,            5294,            5147,            5872,            3708,            4893,            2919,            3485,            1571,            3319,            977,            3585,            4180,            3858,            2995,            1564,            6297,            3857,            2668,            4154,            4100,            4012,            3484,            4725,            565,            4088,            5656,            3590,            ],        [            3979,            411,            2458,            4151,            0,            2216,            2920,            3234,            1667,            4488,            2525,            2316,            1600,            3096,            2949,            3674,            3122,            2695,            1555,            907,            4047,            991,            4986,            2999,            1982,            3272,            1121,            4040,            4099,            975,            1448,            1237,            1902,            543,            906,            2527,            3586,            298,            7924,            3004,            ],        [            4081,            1873,            2801,            4253,            2210,            0,            3263,            3577,            2010,            4590,            1426,            2434,            661,            880,            1850,            2575,            3465,            479,            3275,            3117,            4149,            1464,            5088,            3342,            883,            3615,            1464,            4142,            3000,            2002,            1791,            2299,            803,            2753,            3116,            311,            3688,            2508,            8026,            3347,            ],        [            3128,            2532,            462,            3300,            2869,            3218,            0,            340,            1253,            3637,            3527,            1481,            2602,            4098,            3951,            4676,            202,            3697,            2322,            2888,            3196,            2123,            4135,            105,            2984,            352,            1799,            3189,            5101,            2661,            1472,            2958,            2904,            3415,            2887,            3529,            2735,            3491,            7073,            110,            ],        [            3442,            2846,            776,            3614,            3183,            3532,            340,            0,            1567,            3951,            3841,            1795,            2916,            4412,            4265,            4990,            542,            4011,            2636,            3202,            3510,            2437,            4449,            235,            3298,            692,            2113,            3503,            5415,            2975,            1786,            3272,            3218,            3729,            3201,            3843,            3049,            3805,            7387,            230,            ],        [            2071,            1279,            791,            2243,            1616,            1965,            1253,            1567,            0,            2580,            2274,            424,            1349,            2845,            2698,            3423,            1455,            2444,            1265,            1831,            2139,            870,            3078,            1332,            1731,            1605,            546,            2132,            3848,            1408,            219,            1705,            1651,            2358,            1830,            2276,            1678,            2434,            6016,            1337,            ],        [            1590,            4065,            3381,            479,            4402,            4751,            3843,            4157,            2786,            0,            5060,            2517,            4135,            5631,            5484,            6209,            4045,            5230,            3256,            3822,            1092,            3656,            498,            3922,            4517,            4195,            3332,            1085,            6634,            4194,            3005,            4491,            4437,            4349,            3821,            5062,            902,            4425,            5558,            3927,            ],        [            4345,            2137,            3065,            4517,            2474,            1381,            3527,            3841,            2274,            4854,            0,            2698,            925,            2261,            424,            1149,            3729,            1860,            3539,            3381,            4413,            1728,            5352,            3606,            1147,            3879,            1728,            4406,            1574,            2266,            2055,            2563,            623,            3017,            3380,            1692,            3952,            2772,            8290,            3611,            ],        [            2918,            1548,            864,            3090,            1885,            2234,            1326,            1640,            269,            3427,            2543,            0,            1618,            3114,            2967,            3692,            1528,            2713,            1649,            2215,            2986,            1139,            3925,            1405,            2000,            1678,            815,            2979,            4117,            1677,            488,            1974,            1920,            2428,            2214,            2545,            2525,            2183,            6863,            1410,            ],        [            3420,            1212,            2140,            3592,            1549,            616,            2602,            2916,            1349,            3929,            925,            1773,            0,            1496,            1349,            2074,            2804,            1095,            2614,            2456,            3488,            803,            4427,            2681,            382,            2954,            803,            3481,            2499,            1341,            1130,            1638,            302,            2092,            2455,            927,            3027,            1847,            7365,            2686,            ],        [            4827,            2619,            3547,            4999,            2956,            1863,            4009,            4323,            2756,            5336,            2172,            3180,            1407,            0,            2596,            3321,            4211,            1134,            4021,            3863,            4895,            2210,            5834,            4088,            1629,            4361,            2210,            4888,            3746,            2748,            2537,            3045,            1549,            3499,            3862,            966,            4434,            3254,            8772,            4093,            ],        [            4769,            2561,            3489,            4941,            2898,            1805,            3951,            4265,            2698,            5278,            424,            3122,            1349,            2685,            0,            725,            4153,            2284,            3963,            3805,            4837,            2152,            5776,            4030,            1571,            4303,            2152,            4830,            1150,            2690,            2479,            2987,            1047,            3441,            3804,            2116,            4376,            3196,            8714,            4035,            ],        [            5494,            3286,            4214,            5666,            3623,            2530,            4676,            4990,            3423,            6003,            1149,            3847,            2074,            3410,            725,            0,            4878,            3009,            4688,            4530,            5562,            2877,            6501,            4755,            2296,            5028,            2877,            5555,            425,            3415,            3204,            3712,            1772,            4166,            4529,            2841,            5101,            3921,            9439,            4760,            ],        [            3330,            2734,            664,            3502,            3071,            3420,            202,            542,            1455,            3839,            3729,            1683,            2804,            4300,            4153,            4878,            0,            3899,            2524,            3090,            3398,            2325,            4337,            307,            3186,            150,            2001,            3391,            5303,            2863,            1674,            3160,            3106,            3617,            3089,            3731,            2937,            3693,            7275,            312,            ],        [            4535,            2327,            3255,            4707,            2664,            1571,            3717,            4031,            2464,            5044,            1880,            2888,            1115,            401,            2304,            3029,            3919,            0,            3729,            3571,            4603,            1918,            5542,            3796,            1337,            4069,            1918,            4596,            3454,            2456,            2245,            2753,            1257,            3207,            3570,            674,            4142,            2962,            8480,            3801,            ],        [            2424,            2309,            1625,            2596,            1927,            2995,            2087,            2401,            1030,            2933,            3304,            761,            2379,            3875,            3728,            4453,            2289,            3474,            0,            1014,            2492,            1900,            3431,            2166,            2761,            2439,            1576,            2485,            4878,            2438,            1249,            2735,            2681,            1541,            1013,            3306,            2031,            1617,            6369,            2171,            ],        [            3220,            1160,            2421,            3392,            913,            2965,            2883,            3197,            1826,            3729,            3274,            1557,            2349,            3845,            3698,            4423,            3085,            3444,            796,            0,            3288,            1740,            4227,            2962,            2731,            3235,            2372,            3281,            4848,            1724,            2045,            1986,            2651,            527,            147,            3276,            2827,            603,            7165,            2967,            ],        [            498,            3129,            2445,            1031,            3466,            3815,            2907,            3221,            1850,            1368,            4124,            1581,            3199,            4695,            4548,            5273,            3109,            4294,            2320,            2886,            0,            2720,            1278,            2986,            3581,            3259,            2396,            556,            5698,            3258,            2069,            3555,            3501,            3413,            2885,            4126,            466,            3489,            4466,            2991,            ],        [            3006,            409,            1726,            3178,            746,            1484,            2188,            2502,            935,            3515,            1793,            1359,            868,            2364,            2217,            2942,            2390,            1963,            2200,            1653,            3074,            0,            4013,            2267,            1250,            2540,            389,            3067,            3367,            538,            716,            835,            1170,            1289,            1652,            1795,            2613,            1044,            6951,            2272,            ],        [            1092,            3723,            3039,            1988,            4060,            4409,            3501,            3815,            2444,            1509,            4718,            2175,            3793,            5289,            5142,            5867,            3703,            4888,            2914,            3480,            594,            3314,            0,            3580,            4175,            3853,            2990,            587,            6292,            3852,            2663,            4149,            4095,            4007,            3479,            4720,            1060,            4083,            5060,            3585,            ],        [            3207,            2611,            541,            3379,            2948,            3297,            105,            235,            1332,            3716,            3606,            1560,            2681,            4177,            4030,            4755,            307,            3776,            2401,            2967,            3275,            2202,            4214,            0,            3063,            457,            1878,            3268,            5180,            2740,            1551,            3037,            2983,            3494,            2966,            3608,            2814,            3570,            7152,            5,            ],        [            4315,            2107,            3035,            4487,            2444,            234,            3497,            3811,            2244,            4824,            1660,            2668,            895,            1114,            2084,            2809,            3699,            713,            3509,            3351,            4383,            1698,            5322,            3576,            0,            3849,            1698,            4376,            3234,            2236,            2025,            2533,            1037,            2987,            3350,            545,            3922,            2742,            8260,            3581,            ],        [            3480,            2884,            814,            3652,            3221,            3570,            352,            692,            1605,            3989,            3879,            1833,            2954,            4450,            4303,            5028,            150,            4049,            2674,            3240,            3548,            2475,            4487,            457,            3336,            0,            2151,            3541,            5453,            3013,            1824,            3310,            3256,            3767,            3239,            3881,            3087,            3843,            7425,            462,            ],        [            2617,            733,            1337,            2789,            1070,            1419,            1799,            2113,            546,            3126,            1728,            970,            803,            2299,            2152,            2877,            2001,            1898,            1811,            2377,            2685,            324,            3624,            1878,            1185,            2151,            0,            2678,            3302,            862,            327,            1159,            1105,            1613,            2376,            1730,            2224,            1368,            6562,            1883,            ],        [            505,            3136,            2452,            1038,            3473,            3822,            2914,            3228,            1857,            1375,            4131,            1588,            3206,            4702,            4555,            5280,            3116,            4301,            2327,            2893,            7,            2727,            1285,            2993,            3588,            3266,            2403,            0,            5705,            3265,            2076,            3562,            3508,            3420,            2892,            4133,            473,            3496,            4473,            2998,            ],        [            5919,            3711,            4639,            6091,            4048,            2955,            5101,            5415,            3848,            6428,            1574,            4272,            2499,            3835,            1150,            425,            5303,            3434,            5113,            4955,            5987,            3302,            6926,            5180,            2721,            5453,            3302,            5980,            0,            3840,            3629,            4137,            2197,            4591,            4954,            3266,            5526,            4346,            9864,            5185,            ],        [            3465,            597,            2185,            3637,            934,            1943,            2647,            2961,            1394,            3974,            2252,            1818,            1327,            2823,            2676,            3401,            2849,            2422,            2659,            1841,            3533,            718,            4472,            2726,            1709,            2999,            848,            3526,            3826,            0,            1175,            362,            1629,            1477,            1840,            2254,            3072,            1232,            7410,            2731,            ],        [            2290,            1060,            1010,            2462,            1397,            1746,            1472,            1786,            219,            2799,            2055,            643,            1130,            2626,            2479,            3204,            1674,            2225,            1484,            2050,            2358,            651,            3297,            1551,            1512,            1824,            327,            2351,            3629,            1189,            0,            1486,            1432,            2577,            2049,            2057,            1897,            1695,            6235,            1556,            ],        [            3827,            959,            2547,            3999,            1296,            2305,            3009,            3323,            1756,            4336,            2614,            2180,            1689,            3185,            3038,            3763,            3211,            2784,            3021,            2203,            3895,            1080,            4834,            3088,            2071,            3361,            1210,            3888,            4188,            362,            1537,            0,            1991,            1839,            2202,            2616,            3434,            1594,            7772,            3093,            ],        [            3722,            1514,            2442,            3894,            1851,            758,            2904,            3218,            1651,            4231,            623,            2075,            302,            1638,            1047,            1772,            3106,            1237,            2916,            2758,            3790,            1105,            4729,            2983,            524,            3256,            1105,            3783,            2197,            1643,            1432,            1940,            0,            2394,            2757,            1069,            3329,            2149,            7667,            2988,            ],        [            3714,            911,            2915,            3886,            664,            2716,            3377,            3691,            2320,            4223,            3025,            2051,            2100,            3596,            3449,            4174,            3579,            3195,            1290,            642,            3782,            1491,            4721,            3456,            2482,            3729,            1621,            3775,            4599,            1475,            2539,            1737,            2402,            0,            641,            3027,            3321,            354,            7659,            3461,            ],        [            3221,            1161,            2422,            3393,            914,            2966,            2884,            3198,            1827,            3730,            3275,            1558,            2350,            3846,            3699,            4424,            3086,            3445,            797,            1,            3289,            1741,            4228,            2963,            2732,            3236,            2373,            3282,            4849,            1725,            2046,            1987,            2652,            528,            0,            3277,            2828,            604,            7166,            2968,            ],        [            4703,            2495,            3423,            4875,            2832,            1739,            3885,            4199,            2632,            5212,            2048,            3056,            1283,            569,            2472,            3197,            4087,            168,            3897,            3739,            4771,            2086,            5710,            3964,            1505,            4237,            2086,            4764,            3622,            2624,            2413,            2921,            1425,            3375,            3738,            0,            4310,            3130,            8648,            3969,            ],        [            1123,            3163,            2479,            565,            3500,            3849,            2941,            3255,            1884,            902,            4158,            1615,            3233,            4729,            4582,            5307,            3143,            4328,            2354,            2920,            1994,            2754,            1400,            3020,            3615,            3293,            2430,            1987,            5732,            3292,            2103,            3589,            3535,            3447,            2919,            4160,            0,            3523,            5091,            3025,            ],        [            3858,            557,            2604,            4030,            310,            2362,            3066,            3380,            1813,            4367,            2671,            2195,            1746,            3242,            3095,            3820,            3268,            2841,            1434,            786,            3926,            1137,            4865,            3145,            2128,            3418,            1267,            3919,            4245,            1121,            1594,            1383,            2048,            422,            785,            2673,            3465,            0,            7803,            3150,            ],        [            3299,            3184,            2500,            3471,            3521,            3870,            2962,            3276,            1905,            3808,            4179,            1636,            3254,            4750,            4603,            5328,            3164,            4349,            2375,            2941,            3367,            2775,            4306,            3041,            3636,            3314,            2451,            3360,            5753,            3313,            2124,            3610,            3556,            3468,            2940,            4181,            2906,            3544,            0,            3046,            ],        [            3212,            2616,            546,            3384,            2953,            3302,            110,            230,            1337,            3721,            3611,            1565,            2686,            4182,            4035,            4760,            312,            3781,            2406,            2972,            3280,            2207,            4219,            5,            3068,            462,            1883,            3273,            5185,            2745,            1556,            3042,            2988,            3499,            2971,            3613,            2819,            3575,            7157,            0,            ],        ], 'starts': [2, 23, 30, 38], 'ends': [2, 23, 30, 38]},    'NW-35': {'distance_matrix': [        [            0,            4589,            1950,            2796,            4864,            5379,            4430,            4694,            3134,            5510,            3575,            3274,            5443,            4440,            4337,            3870,            5517,            3906,            3210,            4667,            5339,            3026,            3515,            677,            5603,            3879,            1511,            3622,            5042,            3454,            3648,            4936,            1102,            1300,            890,            2726,            3729,            2942,            1501,            3175,            2437,            2377,            1857,            2224,            3824,            4263,            2162,            244,            1120,            578,            2198,            4665,            3584,            3133,            1156,            3282,            1487,            1983,            1783,            1285,            3066,            2516,            4308,            2757,            1846,            2689,            1729,            2000,            1033,            3922,            3340,            3407,            2362,            2162,            5315,            5809,            2442,            4053,            4422,            3543,            3388,            3126,            1553,            2953,            3552,            3432,            1499,            3335,            5358,            3447,            3206,            2631,            3679,            2987,            4154,            4378,            2631,            5443,            2807,            1677,            ],        [            4730,            0,            2653,            1876,            301,            816,            1119,            563,            1385,            1545,            1078,            1904,            1551,            3328,            464,            1413,            1150,            2794,            1840,            772,            972,            1938,            2427,            4603,            1040,            1382,            2764,            963,            479,            1160,            1046,            373,            5069,            4159,            4390,            4340,            5283,            4496,            4946,            4729,            3971,            3911,            4172,            3805,            5108,            1732,            3696,            4737,            4418,            4773,            4466,            5122,            5138,            4687,            5123,            5122,            3987,            3597,            3857,            2990,            2243,            4130,            5592,            4291,            4694,            5596,            4300,            4069,            4634,            4728,            4894,            4961,            3916,            4932,            752,            1049,            4443,            5694,            4855,            2173,            3170,            4917,            4987,            1146,            3153,            2925,            4049,            2329,            795,            2519,            4740,            4185,            2673,            4541,            335,            1847,            1381,            880,            1846,            4064,            ],        [            3758,            2602,            0,            972,            2877,            3392,            2438,            1967,            1142,            3523,            2251,            1282,            3456,            2453,            2350,            1878,            3530,            1919,            1218,            1959,            3352,            1034,            1523,            2960,            3616,            2555,            964,            1630,            3055,            1462,            1656,            2949,            3426,            2516,            2747,            2697,            3640,            2853,            3303,            3086,            2328,            2268,            2529,            2162,            3465,            2271,            2053,            3841,            2775,            3130,            2823,            4306,            3495,            3044,            3480,            3479,            2344,            1954,            2214,            1190,            747,            2487,            3949,            2648,            3051,            3953,            2657,            2426,            2991,            3833,            3251,            3318,            2273,            3289,            3328,            3822,            2800,            3694,            4063,            1551,            3359,            3274,            3344,            1629,            3443,            3403,            2406,            395,            3371,            1623,            3097,            2542,            518,            2898,            2162,            2386,            1307,            3456,            976,            2421,            ],        [            2862,            3754,            2641,            0,            4029,            4544,            3595,            3859,            2299,            4675,            4138,            2439,            4608,            3605,            3502,            3035,            4682,            3071,            2375,            3832,            4504,            2191,            2680,            3302,            4768,            4442,            2752,            2787,            4207,            2619,            2813,            4101,            3853,            4441,            4672,            4622,            5565,            4778,            5228,            5011,            4253,            4193,            4454,            4087,            5390,            3428,            3978,            2869,            4700,            3203,            4748,            6231,            5420,            4969,            3907,            5404,            4269,            3879,            4139,            2146,            2231,            4412,            5874,            4573,            4976,            5878,            4582,            4351,            3784,            5758,            5176,            5243,            4198,            5214,            4480,            4974,            4725,            5619,            5988,            2708,            5284,            5199,            5269,            4206,            5368,            5328,            4331,            2500,            4523,            651,            5022,            4467,            2844,            4823,            3319,            3543,            3633,            4608,            3302,            4346,            ],        [            5001,            281,            2924,            2147,            0,            664,            2008,            834,            1425,            1816,            1349,            2175,            1822,            3599,            735,            1522,            998,            3065,            1606,            1043,            820,            2209,            2698,            4874,            888,            1653,            3035,            1003,            588,            1200,            1155,            482,            5340,            4430,            4661,            4611,            5554,            4767,            5217,            5000,            4242,            4182,            4443,            4076,            5379,            1841,            3967,            5008,            4689,            5044,            4737,            5393,            5409,            4958,            5394,            5393,            4258,            3868,            4128,            3261,            2514,            4401,            5863,            4562,            4965,            5867,            4571,            4340,            4905,            4999,            5165,            5232,            4187,            5203,            600,            897,            4714,            5965,            5126,            2444,            3441,            5188,            5258,            1417,            3424,            3196,            4320,            2600,            643,            2790,            5011,            4456,            2944,            4812,            606,            1956,            1652,            728,            2117,            4335,            ],        [            5297,            577,            3220,            2443,            296,            0,            2304,            1130,            1721,            2112,            1645,            2471,            1198,            3895,            1031,            1818,            646,            3361,            1902,            1339,            468,            2505,            2994,            5170,            523,            1949,            3331,            1299,            884,            1496,            1451,            778,            5636,            4726,            4957,            4907,            5850,            5063,            5513,            5296,            4538,            4478,            4739,            4372,            5675,            2137,            4263,            5304,            4985,            5340,            5033,            5689,            5705,            5254,            5690,            5689,            4554,            4164,            4424,            3557,            2810,            4697,            6159,            4858,            5261,            6163,            4867,            4636,            5201,            5295,            5461,            5528,            4483,            5499,            248,            545,            5010,            6261,            5422,            2740,            2894,            5484,            5554,            1713,            2877,            2649,            4616,            2896,            291,            3086,            3292,            4752,            3240,            5108,            902,            2252,            1948,            376,            2413,            4631,            ],        [            4596,            1119,            2511,            1734,            1284,            1799,            0,            2321,            1496,            2654,            2187,            1156,            4189,            3186,            3083,            560,            2133,            2652,            1425,            1881,            1955,            1796,            2285,            4461,            2023,            2491,            2622,            1984,            680,            1816,            1201,            910,            4927,            4017,            4248,            4198,            5141,            4354,            4802,            4587,            3829,            3769,            4030,            3663,            4966,            167,            3554,            4603,            4276,            4631,            4324,            5807,            4996,            4545,            4981,            4980,            3845,            3455,            3715,            2848,            2101,            3988,            5450,            4149,            4552,            5454,            4158,            3927,            4492,            5334,            4752,            4819,            3774,            4790,            1735,            2032,            4301,            5195,            5564,            887,            4860,            4775,            4845,            2255,            4944,            4904,            3907,            2187,            1778,            2385,            4598,            4043,            2531,            4399,            2516,            52,            3320,            1863,            2989,            3922,            ],        [            4073,            663,            1988,            1211,            938,            1453,            1783,            0,            822,            1732,            1265,            1113,            1738,            2663,            651,            1297,            1812,            2129,            1003,            959,            1634,            1273,            1762,            3938,            1677,            1569,            2099,            400,            1116,            597,            930,            1010,            4404,            3494,            3725,            3675,            4618,            3831,            4281,            4064,            3306,            3246,            3507,            3140,            4443,            1616,            3031,            4080,            3753,            4108,            3801,            5284,            4473,            4022,            4458,            4457,            3322,            2932,            3192,            2325,            1578,            3465,            4927,            3626,            4029,            4931,            3635,            3404,            3969,            4915,            4229,            4296,            3251,            4267,            1389,            1686,            3778,            4672,            5042,            1382,            4337,            4252,            4322,            1333,            3340,            4381,            3384,            1664,            1432,            1862,            4075,            3520,            2008,            3876,            320,            1731,            1568,            1517,            2033,            3399,            ],        [            3251,            1990,            1166,            389,            2265,            2780,            1447,            944,            0,            2911,            2374,            291,            2844,            1841,            1738,            887,            2918,            1307,            181,            2068,            2740,            451,            940,            3116,            3004,            2678,            1277,            607,            2443,            439,            606,            2337,            3582,            2672,            2903,            2853,            3796,            3009,            3459,            3242,            2484,            2424,            2685,            2318,            3621,            1280,            2209,            3258,            2931,            3286,            2979,            4462,            3651,            3200,            3636,            3635,            2500,            2110,            2370,            1503,            756,            2643,            4105,            2804,            3207,            4109,            2813,            2582,            3147,            3989,            3407,            3474,            2429,            3445,            2716,            3210,            2956,            3850,            4219,            560,            3515,            3430,            3500,            2297,            3599,            3559,            2562,            842,            2759,            1040,            3253,            2698,            1186,            3054,            1139,            1395,            1975,            2844,            1644,            2577,            ],        [            3193,            1932,            1108,            331,            2207,            2722,            1515,            1044,            219,            0,            2316,            359,            2786,            1783,            1680,            955,            2860,            1249,            295,            2010,            2682,            393,            882,            3058,            2946,            2620,            1219,            707,            2385,            539,            733,            2279,            3524,            2614,            2845,            2795,            3738,            2951,            3401,            3184,            2426,            2366,            2627,            2260,            3563,            1348,            2151,            3200,            2873,            3228,            2921,            4404,            3593,            3142,            3578,            3577,            2442,            2052,            2312,            1445,            698,            2585,            4047,            2746,            3149,            4051,            2755,            2524,            3089,            3931,            3349,            3416,            2371,            3387,            2658,            3152,            2898,            3792,            4161,            628,            3457,            3372,            3442,            2239,            3541,            3501,            2504,            784,            2701,            982,            3195,            2640,            1128,            2996,            1239,            1463,            1917,            2786,            1586,            2519,            ],        [            3846,            1156,            3185,            2408,            1431,            2077,            3592,            1261,            2296,            2077,            0,            2436,            1226,            3860,            904,            3032,            1300,            3326,            2372,            422,            1122,            2470,            2959,            3048,            2170,            304,            3296,            1661,            1609,            1858,            2191,            1503,            3514,            2604,            2835,            2785,            3728,            2941,            3391,            3174,            2416,            2356,            2617,            2250,            3553,            3425,            2141,            3480,            2863,            3218,            2911,            4009,            3206,            3132,            3568,            3017,            2432,            2042,            2302,            3522,            2775,            2575,            4037,            2736,            3139,            4041,            2745,            2514,            3079,            3615,            2962,            3406,            2361,            3377,            1882,            1592,            2888,            3782,            3742,            2705,            2205,            3362,            3432,            796,            2188,            1960,            2494,            2267,            1925,            3051,            2603,            2630,            1898,            2986,            1033,            3540,            1031,            2010,            1496,            2509,            ],        [            3440,            1703,            1355,            578,            1771,            2286,            1156,            1165,            340,            2772,            2563,            0,            3033,            2030,            1927,            596,            3107,            1496,            269,            2257,            2929,            640,            1129,            3305,            2510,            2867,            1466,            828,            2156,            660,            680,            2050,            3771,            2861,            3092,            3042,            3985,            3198,            3648,            3431,            2673,            2613,            2874,            2507,            3810,            989,            2398,            3447,            3120,            3475,            3168,            4651,            3840,            3389,            3825,            3824,            2689,            2299,            2559,            1692,            945,            2832,            4294,            2993,            3396,            4298,            3002,            2771,            3336,            4178,            3596,            3663,            2618,            3634,            2222,            2519,            3145,            4039,            4408,            269,            3704,            3619,            3689,            2486,            3788,            3748,            2751,            1031,            2265,            1229,            3442,            2887,            1375,            3243,            1360,            1104,            2164,            2350,            1833,            2766,            ],        [            4671,            565,            2594,            1817,            840,            1355,            3001,            670,            1705,            1486,            1019,            1845,            0,            3269,            313,            2441,            1588,            2735,            1781,            713,            1410,            1879,            2368,            4544,            1579,            1323,            2705,            1070,            1018,            1267,            1600,            912,            5010,            4100,            4331,            4281,            5224,            4437,            4887,            4670,            3912,            3852,            4113,            3746,            5049,            2834,            3637,            4678,            4359,            4714,            4407,            5085,            5079,            4628,            5064,            5063,            3928,            3538,            3798,            2931,            2184,            4071,            5533,            4232,            4635,            5537,            4241,            4010,            4575,            4691,            4835,            4902,            3857,            4873,            1291,            1588,            4384,            5278,            4818,            2114,            3133,            4858,            4928,            1087,            3116,            2888,            3990,            2270,            1334,            2460,            4681,            4126,            2614,            4482,            442,            2949,            1322,            1419,            1787,            4005,            ],        [            5670,            1568,            3593,            2816,            1843,            1854,            4000,            1673,            2704,            2485,            1286,            2844,            1003,            0,            1316,            3440,            1077,            3734,            2780,            1571,            899,            2878,            3367,            5543,            1747,            1430,            3704,            3192,            2021,            2270,            3218,            1915,            6009,            5099,            5330,            3276,            5235,            5436,            5884,            5209,            4911,            4851,            5112,            4745,            4941,            3833,            4636,            5677,            5358,            5713,            5406,            4574,            4518,            5627,            6063,            3434,            4927,            4537,            4797,            3930,            3183,            3392,            4836,            3334,            5634,            6275,            5240,            5009,            5574,            4180,            3379,            4977,            4856,            5872,            1459,            1369,            5383,            5146,            4307,            3113,            2622,            5596,            5927,            1945,            2605,            2377,            4989,            3269,            1502,            3459,            3020,            5125,            3613,            5481,            1445,            3948,            2180,            1587,            2645,            5004,            ],        [            4524,            252,            2447,            1670,            527,            1042,            2140,            357,            1179,            1339,            872,            1470,            1345,            3122,            0,            1654,            1419,            2588,            1360,            566,            1241,            1732,            2221,            4397,            1266,            1176,            2558,            757,            705,            954,            1287,            599,            4863,            3953,            4184,            4134,            5077,            4290,            4740,            4523,            3765,            3705,            3966,            3599,            4902,            1973,            3490,            4531,            4212,            4567,            4260,            4916,            4932,            4481,            4917,            4916,            3781,            3391,            3651,            2784,            2037,            3924,            5386,            4085,            4488,            5390,            4094,            3863,            4428,            4522,            4688,            4755,            3710,            4726,            978,            1275,            4237,            5488,            4649,            1739,            2964,            4711,            4781,            940,            2947,            2719,            3843,            2123,            1021,            2313,            4534,            3979,            2467,            4335,            129,            2088,            1175,            1106,            1640,            3858,            ],        [            4036,            1404,            1951,            1174,            1569,            2084,            560,            1761,            936,            3368,            3159,            596,            3629,            2626,            2523,            0,            3703,            2092,            865,            2853,            3525,            1236,            1725,            3901,            2308,            3463,            2062,            1424,            1240,            1256,            715,            1641,            4367,            3457,            3688,            3638,            4581,            3794,            4244,            4027,            3269,            3209,            3470,            3103,            4406,            393,            2994,            4043,            3716,            4071,            3764,            5247,            4436,            3985,            4421,            4420,            3285,            2895,            3155,            2288,            1541,            3428,            4890,            3589,            3992,            4894,            3598,            3367,            3932,            4774,            4192,            4259,            3214,            4230,            2020,            2317,            3741,            4635,            5004,            327,            4300,            4215,            4285,            3082,            4384,            4344,            3347,            1627,            2063,            1825,            4038,            3483,            1971,            3839,            1956,            508,            2760,            2148,            2429,            3362,            ],        [            5663,            1171,            3586,            2809,            890,            777,            3993,            1666,            2697,            2478,            1356,            2837,            996,            4261,            1309,            3433,            0,            3727,            2773,            1709,            178,            2871,            3360,            5536,            1001,            1500,            3697,            2066,            1478,            2263,            2045,            1372,            6002,            5092,            5323,            5273,            5310,            5429,            5877,            5662,            4904,            4844,            5105,            4738,            5016,            3826,            4629,            5670,            5351,            5706,            4198,            4649,            4593,            5620,            6056,            3504,            4920,            4530,            4790,            3923,            3176,            3462,            4911,            3404,            5627,            6350,            5233,            5002,            5567,            4255,            3449,            5052,            4849,            4664,            713,            292,            4175,            5221,            4382,            3106,            2692,            5671,            5920,            2083,            2675,            2447,            4982,            3262,            756,            3452,            3090,            5118,            3606,            5474,            1438,            3941,            2318,            841,            2783,            4997,            ],        [            4789,            683,            2712,            1935,            958,            1473,            3119,            788,            1823,            1604,            1067,            1963,            1537,            534,            431,            2559,            1611,            0,            1899,            761,            1433,            1997,            2486,            4662,            1697,            1371,            2823,            1188,            1136,            1385,            1718,            1030,            5128,            4218,            4449,            4399,            5769,            4555,            5005,            4788,            4030,            3970,            4231,            3864,            5475,            2952,            3755,            4796,            4477,            4832,            4525,            5108,            5052,            4746,            5182,            3968,            4046,            3656,            3916,            3049,            2302,            4189,            5370,            4350,            4753,            5655,            4359,            4128,            4693,            4714,            3913,            5511,            3975,            4991,            1409,            1903,            4502,            5680,            4841,            2232,            3156,            4976,            5046,            1135,            3139,            2911,            4108,            2388,            1452,            2578,            3554,            4244,            2732,            4600,            560,            3067,            1370,            1537,            1835,            4123,            ],        [            3372,            1603,            1287,            510,            1671,            2186,            1425,            1065,            240,            3032,            2495,            269,            2965,            1962,            1859,            865,            3039,            1428,            0,            2189,            2861,            572,            1061,            3237,            2410,            2799,            1398,            728,            2056,            560,            425,            1950,            3703,            2793,            3024,            2974,            3917,            3130,            3580,            3363,            2605,            2545,            2806,            2439,            3742,            1258,            2330,            3379,            3052,            3407,            3100,            4583,            3772,            3321,            3757,            3756,            2621,            2231,            2491,            1624,            877,            2764,            4226,            2925,            3328,            4230,            2934,            2703,            3268,            4110,            3528,            3595,            2550,            3566,            2122,            2419,            3077,            3971,            4340,            538,            3636,            3551,            3621,            2418,            3720,            3680,            2683,            963,            2165,            1161,            3374,            2819,            1307,            3175,            1260,            1373,            2096,            2250,            1765,            2698,            ],        [            3424,            1438,            2303,            2690,            1713,            2228,            3874,            1543,            2578,            2359,            996,            2718,            2295,            3515,            1186,            3314,            2369,            2981,            2654,            0,            2191,            2428,            2917,            2626,            2452,            1300,            2414,            1943,            1891,            2140,            2473,            1785,            3092,            2182,            2413,            2363,            3306,            2519,            2969,            2752,            1994,            1934,            2195,            1828,            3131,            3707,            1719,            3058,            2441,            2796,            2489,            3972,            3161,            2710,            3146,            3145,            2010,            1620,            1880,            2640,            2197,            2153,            3615,            2314,            2717,            3619,            2323,            2092,            2657,            3499,            2917,            2984,            1939,            2955,            2164,            2461,            2466,            3360,            3729,            2987,            3025,            2940,            3010,            374,            3109,            3069,            2072,            1845,            2207,            3333,            2763,            2208,            1476,            2564,            1315,            3822,            609,            2292,            1074,            2087,            ],        [            5485,            993,            3408,            2631,            712,            955,            3815,            1488,            2519,            2300,            1178,            2659,            818,            4083,            1131,            3255,            178,            3549,            2595,            1531,            0,            2693,            3182,            5358,            936,            1322,            3519,            3007,            1300,            2085,            1867,            1194,            5824,            4914,            5145,            5095,            5132,            5251,            5699,            5484,            4726,            4666,            4927,            4560,            4838,            3648,            4451,            5492,            5173,            5528,            4020,            4471,            4415,            5442,            5878,            3326,            4742,            4352,            4612,            3745,            2998,            3284,            4733,            3226,            5449,            6172,            5055,            4824,            5389,            4077,            3271,            4874,            4671,            4486,            648,            470,            3997,            5043,            4204,            2928,            2514,            5493,            5742,            1905,            2497,            2269,            4804,            3084,            691,            3274,            2912,            4940,            3428,            5296,            1260,            3763,            2140,            776,            2605,            4819,            ],        [            3196,            2046,            1111,            334,            2321,            2836,            1847,            1376,            551,            2967,            2430,            691,            2900,            1897,            1794,            1287,            2974,            1363,            627,            2124,            2796,            0,            489,            3172,            3060,            2734,            1222,            1039,            2499,            871,            1065,            2393,            3638,            2728,            2959,            2909,            3852,            3065,            3515,            3298,            2540,            2480,            2741,            2374,            3677,            1680,            2265,            3203,            2987,            3342,            3035,            4518,            3707,            3256,            3692,            3691,            2556,            2166,            2426,            1448,            701,            2699,            4161,            2860,            3263,            4165,            2869,            2638,            3203,            4045,            3463,            3530,            2485,            3501,            2772,            3266,            3012,            3906,            4275,            960,            3571,            3486,            3556,            2242,            3655,            3615,            2618,            787,            2815,            985,            3309,            2754,            1131,            3110,            1571,            1795,            1920,            2900,            1589,            2633,            ],        [            3685,            2535,            1600,            823,            2810,            3325,            2336,            1865,            1040,            3456,            2919,            1180,            3389,            2386,            2283,            1776,            3463,            1852,            1116,            2613,            3285,            489,            0,            3661,            3549,            3223,            1711,            1528,            2988,            1360,            1554,            2882,            4127,            3217,            3448,            3398,            4341,            3554,            4004,            3787,            3029,            2969,            3230,            2863,            4166,            2169,            2754,            3692,            3476,            3831,            3524,            5007,            4196,            3745,            4181,            4180,            3045,            2655,            2915,            1937,            1190,            3188,            4650,            3349,            3752,            4654,            3358,            3127,            3692,            4534,            3952,            4019,            2974,            3990,            3261,            3755,            3501,            4395,            4764,            1449,            4060,            3975,            4045,            2731,            4144,            4104,            3107,            1276,            3304,            1474,            3798,            3243,            1620,            3599,            2060,            2284,            2409,            3389,            2078,            3122,            ],        [            695,            5029,            2390,            3236,            5304,            5819,            4870,            5134,            3574,            5950,            3183,            3714,            5883,            4880,            4777,            4310,            5957,            4346,            3650,            2891,            5779,            3466,            3955,            0,            6043,            3487,            1951,            4062,            5482,            3894,            4088,            5376,            1123,            908,            498,            2334,            3689,            2902,            1608,            3135,            2045,            1985,            1817,            2184,            3514,            4703,            1770,            432,            728,            170,            2305,            4355,            3544,            3093,            1177,            3242,            1095,            1591,            1391,            1725,            3506,            2124,            3998,            2365,            1953,            2738,            1689,            1960,            993,            3882,            3300,            3367,            2322,            2269,            5755,            6249,            2549,            3743,            4112,            3983,            2996,            3175,            1660,            2561,            3160,            3040,            1459,            3775,            5798,            3887,            2814,            2591,            2817,            2947,            4594,            4818,            2239,            5883,            2415,            1285,            ],        [            5773,            1053,            3696,            2919,            772,            476,            2780,            1606,            2197,            2588,            2121,            2947,            1674,            4371,            1507,            2294,            1001,            3837,            2378,            1815,            944,            2981,            3470,            5646,            0,            2425,            3807,            1775,            1360,            1972,            1927,            1254,            6112,            5202,            5433,            5383,            6326,            5539,            5989,            5772,            5014,            4954,            5215,            4848,            6151,            2613,            4739,            5780,            5461,            5816,            5509,            6165,            6181,            5730,            6166,            6165,            5030,            4640,            4900,            4033,            3286,            5173,            6635,            5334,            5737,            6639,            5343,            5112,            5677,            5771,            5937,            6004,            4959,            5975,            288,            709,            5486,            6737,            5898,            3216,            3370,            5960,            6030,            2189,            3353,            3125,            5092,            3372,            455,            3562,            3768,            5228,            3716,            5584,            1378,            2728,            2424,            160,            2889,            5107,            ],        [            4083,            1393,            3422,            2645,            1668,            2273,            3829,            1498,            2533,            2314,            330,            2673,            1422,            4097,            1141,            3269,            1496,            3563,            2609,            659,            1318,            2707,            3196,            3285,            2407,            0,            3533,            1898,            1846,            2095,            2428,            1740,            3751,            2841,            3072,            2596,            3104,            3178,            3626,            3411,            2653,            2593,            2854,            2487,            3790,            3662,            2378,            3717,            3100,            3455,            3148,            4174,            2943,            3369,            3805,            2754,            2669,            2279,            2539,            3759,            3012,            2712,            4274,            2654,            3376,            4278,            2982,            2751,            3316,            2060,            2699,            3643,            2598,            3614,            2119,            1788,            3125,            4746,            3907,            2942,            1942,            3599,            3669,            1033,            1925,            1697,            2731,            2504,            2162,            3288,            2340,            2867,            2135,            3223,            1270,            3777,            1268,            2247,            1733,            2746,            ],        [            1824,            2740,            439,            1110,            3015,            3530,            2576,            2105,            1280,            3661,            2696,            1420,            3594,            2591,            2488,            2016,            3668,            2057,            1356,            2818,            3490,            1172,            1661,            2264,            3754,            3000,            0,            1768,            3193,            1600,            1794,            3087,            2815,            3422,            2477,            3603,            4546,            3759,            3289,            3992,            3234,            3174,            3435,            3068,            4371,            2409,            2959,            1831,            2707,            2165,            3729,            5212,            4401,            3950,            2869,            4385,            3250,            2860,            3120,            226,            521,            3393,            4855,            3554,            3957,            4607,            3563,            3332,            2746,            4739,            4157,            4224,            3179,            4195,            3466,            3960,            3706,            4600,            4969,            1689,            4265,            4180,            3341,            2074,            4349,            4309,            3312,            664,            3509,            1761,            4003,            3448,            963,            3804,            2300,            2524,            1752,            3594,            1421,            3327,            ],        [            3858,            875,            1773,            996,            943,            1458,            1568,            337,            607,            1944,            1477,            898,            1950,            2448,            863,            1082,            2024,            1914,            788,            1171,            1846,            1058,            1547,            3723,            1682,            1781,            1884,            0,            1328,            382,            715,            1222,            4189,            3279,            3510,            3460,            4403,            3616,            4064,            3849,            3091,            3031,            3292,            2925,            4228,            1401,            2816,            3865,            3538,            3893,            3586,            5069,            4258,            3807,            4243,            4242,            3107,            2717,            2977,            2110,            1363,            3250,            4712,            3411,            3814,            4716,            3420,            3189,            3754,            5127,            4014,            4081,            3036,            4052,            1394,            1691,            3563,            4457,            5254,            1167,            3569,            4037,            4107,            1545,            3552,            3324,            3169,            1449,            1437,            1647,            3860,            3305,            1793,            3661,            532,            1516,            1780,            1522,            2245,            3184,            ],        [            5159,            439,            3082,            2305,            604,            1119,            739,            992,            1814,            1974,            1507,            2333,            1980,            3757,            893,            1680,            1453,            3223,            2269,            1201,            1275,            2367,            2856,            5032,            1343,            1811,            3193,            1392,            0,            1589,            1313,            230,            5498,            4588,            4819,            4769,            5712,            4925,            5373,            5158,            4400,            4340,            4601,            4234,            5537,            906,            4125,            5166,            4847,            5202,            4895,            5551,            5567,            5116,            5552,            5551,            4416,            4026,            4286,            3419,            2672,            4559,            6021,            4720,            5123,            6025,            4729,            4498,            5063,            5157,            5323,            5390,            4345,            5361,            1055,            1352,            4872,            6123,            5284,            2602,            3599,            5346,            5416,            1575,            3582,            3354,            4478,            2758,            1098,            2948,            5169,            4614,            3102,            4970,            764,            791,            1810,            1183,            2275,            4493,            ],        [            3482,            1220,            1397,            620,            1288,            1803,            1678,            682,            231,            2289,            1822,            522,            3075,            2072,            1208,            1118,            3149,            1538,            412,            1516,            2971,            682,            1171,            3347,            2027,            2126,            1508,            345,            1673,            0,            611,            1567,            3813,            2903,            3134,            3084,            4027,            3240,            3690,            3473,            2715,            2655,            2916,            2549,            3852,            1511,            2440,            3489,            3162,            3517,            3210,            4693,            3882,            3431,            3867,            3866,            2731,            2341,            2601,            1734,            987,            2874,            4336,            3035,            3438,            4340,            3044,            2813,            3378,            4220,            3638,            3705,            2660,            3676,            1739,            2036,            3187,            4081,            4450,            791,            3746,            3661,            3731,            1890,            3830,            3790,            2793,            1073,            1782,            1271,            3484,            2929,            1417,            3285,            877,            1626,            2125,            1867,            1875,            2808,            ],        [            3615,            1178,            1530,            753,            1343,            1858,            1342,            1130,            425,            2737,            2270,            498,            3208,            2205,            2102,            856,            3282,            1671,            243,            1964,            3104,            815,            1304,            3480,            2082,            2574,            1641,            793,            1262,            625,            0,            2015,            3946,            3036,            3267,            3217,            4160,            3373,            3821,            3606,            2848,            2788,            3049,            2682,            3985,            1175,            2573,            3622,            3295,            3650,            3343,            4826,            4015,            3564,            4000,            3999,            2864,            2474,            2734,            1867,            1120,            3007,            4469,            3168,            3571,            4473,            3177,            2946,            3511,            4353,            3771,            3838,            2793,            3809,            1794,            2091,            3320,            4214,            4583,            767,            3879,            3794,            3864,            2661,            3963,            3923,            2926,            1206,            1837,            1404,            3617,            3062,            1550,            3418,            1325,            1290,            2339,            1922,            2008,            2941,            ],        [            5024,            304,            2947,            2170,            469,            984,            1021,            857,            1679,            1839,            1372,            2198,            1845,            3622,            758,            1545,            1318,            3088,            2134,            1066,            1140,            2232,            2721,            4897,            1208,            1676,            3058,            1257,            381,            1454,            1178,            0,            5363,            4453,            4684,            4634,            5577,            4790,            5240,            5023,            4265,            4205,            4466,            4099,            5402,            1188,            3990,            5031,            4712,            5067,            4760,            5416,            5432,            4981,            5417,            5416,            4281,            3891,            4151,            3284,            2537,            4424,            5886,            4585,            4988,            5890,            4594,            4363,            4928,            5022,            5188,            5255,            4210,            5226,            920,            1217,            4737,            5988,            5149,            2467,            3464,            5211,            5281,            1440,            3447,            3219,            4343,            2623,            963,            2813,            5034,            4479,            2967,            4835,            629,            1073,            1675,            1048,            2140,            4358,            ],        [            1079,            4067,            3029,            3875,            4342,            4857,            5509,            4172,            4213,            4988,            3535,            4353,            4921,            3918,            3815,            4949,            4995,            3384,            4289,            3243,            4817,            4105,            4594,            1281,            5081,            3839,            2590,            4572,            4520,            4769,            5102,            4414,            0,            1847,            1071,            2686,            2420,            1633,            485,            1866,            2397,            2337,            1087,            1454,            2750,            5342,            2122,            1323,            1388,            1182,            1182,            3591,            3522,            2081,            431,            3485,            1447,            1943,            1743,            2364,            4145,            2476,            3234,            2717,            830,            1615,            959,            1230,            745,            3860,            3306,            2098,            1681,            1146,            4793,            5287,            1426,            2979,            3348,            4622,            3348,            2052,            537,            2913,            3272,            3392,            1144,            3538,            4836,            4526,            2926,            2118,            3169,            2481,            3944,            5457,            2591,            4921,            2767,            1637,            ],        [            1551,            3246,            3175,            4498,            3521,            4036,            5682,            3351,            4386,            4167,            2714,            4526,            4100,            3097,            2994,            5122,            4174,            2563,            4462,            2422,            3996,            3300,            3789,            444,            4260,            3018,            2395,            3751,            3699,            3948,            4281,            3593,            1219,            0,            231,            1865,            2996,            2209,            1116,            2442,            1576,            1516,            1124,            1491,            2821,            5515,            1301,            876,            259,            614,            1978,            3662,            2851,            2400,            1273,            2549,            626,            1122,            922,            2169,            3069,            1655,            3305,            1896,            1626,            2488,            996,            1267,            784,            3189,            2607,            2674,            1629,            1942,            3972,            4466,            2156,            3050,            3419,            4795,            2527,            2630,            1333,            2092,            2691,            2571,            766,            2717,            4015,            5141,            2345,            1898,            2348,            2254,            3123,            5630,            1770,            4100,            1946,            816,            ],        [            908,            3357,            2603,            3449,            3632,            4147,            5083,            3462,            3787,            4278,            2825,            3927,            4211,            3208,            3105,            4523,            4285,            2674,            3863,            2533,            4107,            3679,            4168,            213,            4371,            3129,            2164,            4275,            3810,            4059,            4301,            3704,            921,            550,            0,            1976,            3182,            2395,            1302,            2628,            1687,            1627,            1310,            1677,            3007,            4916,            1412,            645,            370,            383,            2164,            3848,            3037,            2586,            975,            2735,            737,            1233,            1033,            1938,            3719,            1766,            3491,            2007,            1812,            2536,            1182,            1453,            486,            3375,            2793,            2860,            1815,            2128,            4083,            4577,            2342,            3236,            3605,            4196,            2638,            2816,            1519,            2203,            2802,            2682,            952,            2828,            4126,            4100,            2456,            2084,            2459,            2440,            3234,            5031,            1881,            4211,            2057,            927,            ],        [            2948,            3745,            3674,            4997,            4020,            3916,            6181,            3850,            4885,            4666,            3213,            5025,            3065,            3596,            3493,            5621,            3139,            3062,            4961,            2921,            2961,            3799,            4288,            1841,            4759,            2281,            3792,            4250,            4198,            4447,            4780,            4092,            2616,            1397,            1628,            0,            2125,            2430,            2513,            2509,            1656,            1848,            1712,            1833,            2439,            6014,            1800,            2273,            1656,            2011,            2400,            2835,            1964,            2803,            2670,            1775,            1490,            849,            1421,            3566,            3568,            386,            2923,            1269,            2628,            3530,            1782,            1609,            2181,            2302,            1720,            2277,            1850,            2866,            4471,            3431,            2377,            2644,            2568,            5294,            662,            2851,            2730,            2591,            1477,            706,            1552,            3216,            4514,            5640,            1393,            2047,            2847,            2403,            3622,            6129,            2269,            4599,            2445,            1201,            ],        [            4150,            4140,            4069,            5392,            4415,            4930,            6576,            4245,            5280,            5061,            3608,            5420,            4994,            3991,            3888,            6016,            5068,            3457,            5356,            3316,            4890,            4194,            4683,            3358,            5154,            3024,            4180,            4645,            4593,            4842,            5175,            4487,            2864,            2914,            3145,            2381,            0,            787,            2476,            554,            1391,            1583,            1956,            1589,            484,            6409,            1678,            4394,            3038,            3528,            1239,            1325,            529,            931,            3012,            997,            2514,            2602,            2480,            5574,            3963,            2497,            968,            1277,            1870,            1818,            2084,            1853,            2932,            773,            436,            322,            1700,            1998,            4866,            5360,            995,            689,            804,            5689,            1727,            1139,            2163,            2986,            1309,            1771,            2308,            3611,            4909,            4783,            963,            2115,            3242,            1331,            4017,            6524,            2664,            4994,            2840,            2704,            ],        [            2891,            3353,            3282,            4605,            3628,            4143,            5789,            3458,            4493,            4274,            2821,            4633,            4207,            3204,            3101,            5229,            4281,            2670,            4569,            2529,            4103,            3407,            3896,            2468,            4367,            3125,            3393,            3858,            3806,            4055,            4388,            3700,            2077,            2024,            2255,            2558,            787,            0,            1689,            233,            1640,            1619,            1169,            802,            776,            5622,            1404,            3135,            2251,            2638,            452,            1617,            1241,            254,            2225,            1204,            1727,            1815,            1693,            5020,            3176,            2348,            1260,            1481,            1083,            1264,            1297,            1066,            2145,            1579,            1025,            465,            913,            1444,            4079,            4573,            208,            896,            1374,            4902,            2274,            585,            1376,            2199,            1836,            2318,            1521,            2824,            4122,            3996,            1490,            1328,            2455,            654,            3230,            5737,            1877,            4207,            2053,            1814,            ],        [            1202,            3847,            3152,            3998,            4122,            4637,            6283,            3952,            4987,            4768,            3315,            5127,            4701,            3698,            3595,            5723,            4775,            3164,            5063,            3023,            4597,            3901,            4390,            1386,            4861,            3619,            2713,            4352,            4300,            4549,            4882,            4194,            388,            1627,            1168,            2466,            2200,            1413,            0,            1646,            2177,            2117,            867,            1234,            2025,            6116,            1902,            1446,            1168,            1287,            962,            2866,            2654,            1861,            536,            2549,            1227,            1723,            1523,            2487,            3670,            2256,            2509,            2497,            610,            1472,            739,            1010,            850,            2992,            2438,            1878,            1461,            926,            4573,            5067,            1206,            2254,            2623,            5396,            3128,            1909,            317,            2693,            3052,            3172,            924,            3318,            4616,            4649,            2706,            1898,            2949,            2261,            3724,            6231,            2371,            4701,            2547,            1417,            ],        [            3124,            3586,            3515,            4838,            3861,            4376,            6022,            3691,            4726,            4507,            3054,            4866,            4440,            3437,            3334,            5462,            4514,            2903,            4802,            2762,            4336,            3640,            4129,            2701,            4600,            3358,            3626,            4091,            4039,            4288,            4621,            3933,            2310,            2257,            2488,            2791,            554,            233,            1922,            0,            1873,            1676,            1402,            1035,            458,            5855,            1637,            3368,            2484,            2871,            685,            1299,            1008,            377,            2458,            971,            1960,            2048,            1926,            5253,            3409,            2581,            942,            1630,            1316,            1497,            1530,            1299,            2378,            1346,            792,            232,            1146,            1677,            4312,            4806,            441,            663,            1056,            5135,            2080,            818,            1609,            2432,            1662,            2124,            1754,            3057,            4355,            4229,            1316,            1561,            2688,            777,            3463,            5970,            2110,            4440,            2286,            2047,            ],        [            2666,            2862,            2791,            4114,            3137,            3652,            5298,            2967,            4002,            3783,            2330,            4142,            3716,            2713,            2610,            4738,            3790,            2179,            4078,            2038,            3612,            2916,            3405,            1983,            3876,            2634,            2902,            3367,            3315,            3564,            3897,            3209,            2334,            1539,            1770,            1773,            1507,            1695,            2198,            1928,            0,            333,            1371,            1004,            1821,            5131,            428,            2415,            1798,            2153,            1665,            2217,            1346,            1288,            2388,            1157,            1252,            1293,            1183,            3128,            2685,            1889,            2305,            326,            1893,            2795,            1499,            1268,            1899,            1684,            1102,            1659,            1115,            2131,            3588,            4082,            1642,            2026,            1950,            4411,            1119,            2116,            2186,            1708,            1121,            1163,            1314,            2333,            3631,            4757,            775,            532,            1964,            888,            2739,            5246,            1386,            3716,            1562,            1329,            ],        [            2781,            2897,            2826,            4149,            3172,            3687,            5333,            3002,            4037,            3818,            2365,            4177,            3751,            2748,            2645,            4773,            3825,            2214,            4113,            2073,            3647,            2951,            3440,            2018,            3911,            2669,            2937,            3402,            3350,            3599,            3932,            3244,            2449,            1574,            1805,            2071,            1422,            1362,            1865,            1595,            414,            0,            1038,            671,            1326,            5166,            463,            2450,            1833,            2188,            1332,            2167,            1152,            955,            2401,            850,            1287,            1328,            1562,            3163,            2720,            1861,            1810,            734,            1560,            2462,            1166,            935,            2014,            1490,            908,            1164,            782,            1798,            3623,            4117,            1309,            1531,            1924,            4446,            1527,            1783,            1853,            1743,            1353,            1571,            1390,            2368,            3666,            4792,            1007,            199,            1999,            555,            2774,            5281,            1421,            3751,            1597,            1364,            ],        [            1743,            3056,            2985,            4308,            3331,            3846,            5492,            3161,            4196,            3977,            2524,            4336,            3910,            2907,            2804,            4932,            3984,            2373,            4272,            2232,            3806,            3110,            3599,            1470,            4070,            2828,            3096,            3561,            3509,            3758,            4091,            3403,            1411,            1026,            1257,            1643,            1961,            1174,            827,            1407,            1343,            1322,            0,            367,            1786,            5325,            1107,            1987,            1082,            1640,            1144,            2627,            1984,            1365,            1363,            1682,            626,            900,            922,            3195,            2879,            1433,            2270,            1663,            916,            2274,            128,            143,            976,            2322,            1740,            1639,            594,            1610,            3782,            4276,            1121,            2015,            2384,            4605,            2305,            1595,            939,            1902,            2185,            2349,            352,            2527,            3825,            4951,            1839,            1031,            2158,            1387,            2933,            5440,            1580,            3910,            1756,            816,            ],        [            2110,            2689,            2618,            3941,            2964,            3479,            5125,            2794,            3829,            3610,            2157,            3969,            3543,            2540,            2437,            4565,            3617,            2006,            3905,            1865,            3439,            2743,            3232,            1769,            3703,            2461,            2729,            3194,            3142,            3391,            3724,            3036,            1778,            1325,            1556,            1756,            1594,            807,            1194,            1040,            976,            955,            367,            0,            1419,            4958,            740,            2354,            1449,            1939,            777,            2260,            1617,            998,            1730,            1315,            925,            1013,            891,            2955,            2512,            1546,            1903,            1296,            1005,            1907,            495,            264,            1343,            1955,            1373,            1272,            227,            1243,            3415,            3909,            754,            1648,            2017,            4238,            2089,            1228,            1298,            1535,            1818,            2133,            719,            2160,            3458,            4584,            1472,            664,            1791,            1020,            2566,            5073,            1213,            3543,            1389,            1115,            ],        [            4279,            4406,            4335,            5658,            4681,            5644,            6842,            4511,            5333,            5327,            3874,            5686,            4791,            4257,            4154,            6282,            4867,            3723,            5622,            3582,            4689,            4460,            4949,            3624,            5420,            3343,            4446,            4911,            4859,            5108,            5441,            4753,            3304,            3180,            3411,            3611,            907,            1197,            2742,            881,            1718,            1910,            2222,            1855,            0,            6675,            2457,            4523,            3304,            3794,            1505,            841,            1106,            1539,            3452,            1324,            2780,            2868,            2746,            5703,            4229,            3203,            484,            1983,            2136,            1947,            2350,            2119,            3198,            1244,            1145,            649,            1966,            2127,            5132,            5159,            1261,            818,            598,            5955,            2433,            1268,            2429,            3252,            2268,            2477,            2574,            3877,            5175,            7247,            1669,            2381,            3508,            1939,            4283,            6790,            2930,            5260,            3106,            2970,            ],        [            4429,            1723,            2344,            1567,            1888,            2403,            167,            2154,            1329,            3761,            3552,            989,            4022,            3019,            2916,            393,            4096,            2485,            1258,            3246,            3918,            1629,            2118,            4294,            2627,            3856,            2455,            1817,            847,            1649,            1034,            1077,            4760,            3850,            4081,            4031,            4974,            4187,            4637,            4420,            3662,            3602,            3863,            3496,            4799,            0,            3387,            4436,            4109,            4464,            4157,            5640,            4829,            4378,            4814,            4813,            3678,            3288,            3548,            2681,            1934,            3821,            5283,            3982,            4385,            5287,            3991,            3760,            4325,            5167,            4585,            4652,            3607,            4623,            2339,            2636,            4134,            5028,            5397,            720,            4693,            4608,            4678,            3475,            4777,            4737,            3740,            2020,            2382,            2218,            4431,            3876,            2364,            4232,            2349,            115,            3153,            2467,            2822,            3755,            ],        [            2477,            2673,            2602,            3925,            2948,            3463,            5109,            2778,            3813,            3594,            2141,            3953,            3527,            2524,            2421,            4549,            3601,            1990,            3889,            1849,            3423,            2727,            3216,            1794,            3687,            2445,            2713,            3178,            3126,            3375,            3708,            3020,            2145,            1350,            1581,            1847,            1637,            1577,            2080,            1810,            275,            215,            1253,            886,            1541,            4942,            0,            2226,            1609,            1964,            1547,            2382,            1367,            1170,            2616,            1426,            1063,            1104,            994,            2939,            2496,            2158,            2025,            595,            1775,            2677,            1381,            1150,            1710,            1705,            1123,            1379,            997,            2013,            3399,            3893,            1524,            1746,            2139,            4222,            1388,            1998,            2068,            1519,            1390,            1432,            1125,            2144,            3442,            4568,            1044,            414,            1775,            770,            2550,            5057,            1197,            3527,            1373,            1140,            ],        [            425,            3863,            2120,            2966,            4138,            4653,            4600,            3968,            3304,            4784,            3331,            3444,            4717,            3714,            3611,            4040,            4791,            3180,            3380,            3039,            4613,            3196,            3685,            433,            4877,            3635,            1681,            3792,            4316,            3624,            3818,            4210,            1177,            1056,            646,            2482,            3535,            2748,            1662,            2981,            2193,            2133,            1663,            2030,            3360,            4433,            1918,            0,            876,            334,            2359,            4201,            3285,            2939,            1231,            3344,            1243,            1739,            1539,            1455,            3236,            2272,            3844,            2513,            2007,            2792,            1535,            1806,            845,            3623,            3041,            3213,            2168,            2323,            4589,            5083,            2695,            3589,            3958,            3713,            3144,            3229,            1714,            2709,            3308,            3188,            1305,            3505,            4632,            3617,            2962,            2332,            2965,            2688,            3740,            4548,            2387,            4717,            2563,            1433,            ],        [            1781,            2987,            2916,            4239,            3262,            3777,            5423,            3092,            4127,            3908,            2455,            4267,            3841,            2838,            2735,            4863,            3915,            2304,            4203,            2163,            3737,            3041,            3530,            1211,            4001,            2759,            3027,            3492,            3440,            3689,            4022,            3334,            1449,            767,            998,            1606,            2659,            1872,            1326,            2105,            1317,            1257,            787,            1154,            2484,            5256,            1042,            1643,            0,            1381,            1842,            3325,            2409,            2063,            1503,            2468,            367,            863,            663,            2936,            2810,            1396,            2968,            1637,            1415,            2277,            659,            930,            1014,            2747,            2165,            2337,            1292,            1731,            3713,            4207,            1819,            2713,            3082,            4536,            2268,            2293,            1438,            1833,            2432,            2312,            429,            2458,            3756,            4882,            2086,            1456,            2089,            1812,            2864,            5371,            1511,            3841,            1687,            557,            ],        [            545,            4879,            2240,            3086,            5154,            5669,            4720,            4984,            3424,            5800,            3127,            3564,            5733,            4730,            4627,            4160,            5807,            4196,            3500,            2835,            5629,            3316,            3805,            229,            5893,            3431,            1801,            3912,            5332,            3744,            3938,            5226,            973,            852,            442,            2278,            3539,            2752,            1458,            2985,            1989,            1929,            1667,            2034,            3364,            4553,            1714,            282,            672,            0,            2155,            4205,            3394,            2943,            1027,            3092,            1039,            1535,            1335,            1575,            3356,            2068,            3848,            2309,            1803,            2588,            1539,            1810,            843,            3732,            3150,            3217,            2172,            2119,            5605,            6099,            2399,            3593,            3962,            3833,            2940,            3025,            1510,            2505,            3104,            2984,            1309,            3625,            5648,            3737,            3249,            2441,            3969,            2797,            4444,            4668,            2183,            5733,            2359,            1229,            ],        [            2160,            3366,            3295,            4618,            3641,            4156,            5802,            3471,            4506,            4287,            2834,            4646,            4220,            3217,            3114,            5242,            4294,            2683,            4582,            2542,            4116,            3420,            3909,            2225,            4380,            3138,            3406,            3871,            3819,            4068,            4401,            3713,            1346,            2012,            2012,            2571,            1238,            451,            958,            684,            1653,            1632,            1182,            815,            1063,            5635,            1417,            2404,            2012,            2245,            0,            1904,            1692,            698,            1494,            1655,            1612,            1828,            1706,            4780,            3189,            2361,            1547,            1973,            352,            1136,            1310,            1079,            1808,            2030,            1476,            916,            926,            590,            4092,            4586,            244,            1292,            1661,            4915,            2766,            872,            645,            2212,            2280,            2810,            1338,            2837,            4135,            4009,            1934,            1341,            2468,            1098,            3243,            5750,            1890,            4220,            2066,            1802,            ],        [            4977,            4515,            6540,            5763,            4790,            4803,            6947,            4620,            5651,            5432,            3716,            5791,            3950,            4955,            4263,            6387,            4026,            4421,            5727,            4001,            3848,            5825,            6314,            4322,            5529,            2502,            6651,            5020,            4968,            5217,            5550,            4862,            4002,            3878,            4109,            2976,            1605,            1895,            3440,            1579,            2416,            2608,            2920,            2553,            1311,            6780,            3155,            5221,            4002,            4492,            2203,            0,            1132,            2237,            4150,            2022,            3478,            3566,            3444,            6401,            6130,            3092,            1206,            2156,            2834,            2645,            3048,            2817,            3896,            794,            1376,            1347,            2664,            2825,            5241,            4318,            1959,            1516,            677,            6060,            2322,            1966,            3127,            3950,            1427,            2077,            3272,            4575,            5284,            6406,            1842,            3079,            4206,            2637,            4392,            6895,            3628,            5369,            3804,            3668,            ],        [            3615,            3811,            3740,            5063,            4086,            4601,            6247,            3916,            4738,            4732,            3279,            5091,            4665,            3662,            3559,            5687,            4739,            3128,            5027,            2987,            4561,            3865,            4354,            2932,            4825,            2832,            3851,            4316,            4264,            4513,            4846,            4158,            3283,            2488,            2719,            2189,            529,            1241,            2816,            1008,            1090,            1282,            1989,            1622,            938,            6080,            1377,            3364,            2747,            3102,            1693,            904,            0,            1385,            3352,            727,            2201,            2242,            2132,            4077,            3634,            2305,            1166,            1085,            2511,            2272,            2117,            1886,            2848,            338,            244,            776,            1733,            2452,            4537,            5031,            1449,            1143,            637,            5360,            1535,            1593,            2804,            2657,            1117,            1579,            2341,            3282,            4580,            5706,            771,            1216,            2913,            1572,            3688,            6195,            2335,            4665,            2511,            2278,            ],        [            3083,            3545,            3474,            4797,            3820,            4335,            5981,            3650,            4685,            4466,            3013,            4825,            4399,            3396,            3293,            5421,            4473,            2862,            4761,            2721,            4295,            3599,            4088,            2763,            4559,            3317,            3585,            4050,            3998,            4247,            4580,            3892,            2269,            2319,            2550,            2750,            931,            254,            1881,            377,            1107,            1299,            1361,            994,            1119,            5814,            1394,            3327,            2443,            2933,            698,            1960,            1381,            0,            2417,            1079,            1919,            2007,            1885,            3811,            3368,            2540,            1603,            1227,            1275,            1607,            1489,            1258,            2337,            1719,            1137,            609,            1105,            1513,            4271,            4765,            454,            1040,            1717,            5094,            2020,            928,            1568,            2391,            1582,            2064,            1713,            3016,            4314,            4188,            1236,            1520,            2647,            400,            3422,            5929,            2069,            4399,            2245,            2109,            ],        [            811,            3632,            2761,            3607,            3907,            4422,            5241,            3737,            3945,            4553,            3100,            4085,            4486,            3483,            3380,            4681,            4560,            2949,            4021,            2808,            4382,            3837,            4326,            850,            4646,            3404,            2322,            4137,            4085,            4334,            4667,            3979,            335,            1412,            640,            2251,            3010,            2223,            778,            2456,            1962,            1902,            1138,            1505,            2835,            5074,            1687,            1055,            870,            751,            1517,            3676,            2865,            2414,            0,            2563,            1012,            1508,            1308,            2096,            3877,            2041,            3319,            2282,            1165,            1950,            1010,            1281,            314,            3203,            2621,            2688,            1643,            1481,            4358,            4852,            2170,            3064,            3433,            4354,            2913,            2387,            872,            2478,            3077,            2957,            780,            3103,            4401,            4258,            2720,            1912,            2734,            2268,            3509,            5189,            2156,            4486,            2332,            1202,            ],        [            3005,            3173,            3102,            4425,            3448,            3963,            5609,            3278,            4313,            4094,            2641,            4453,            4027,            3024,            2921,            5049,            4101,            2490,            4389,            2349,            3923,            3227,            3716,            2294,            4187,            2945,            3213,            3678,            3626,            3875,            4208,            3520,            2673,            1850,            2081,            2347,            2373,            1586,            2089,            1819,            690,            693,            1262,            895,            2198,            5442,            739,            2726,            2109,            2464,            1556,            3039,            1442,            1245,            2625,            0,            1563,            1604,            1494,            3439,            2996,            2137,            2682,            1010,            1784,            2686,            1390,            1159,            2238,            1780,            1198,            2051,            1006,            2022,            3899,            4393,            1533,            1821,            2796,            4722,            1803,            2007,            2077,            2019,            1805,            1847,            1614,            2644,            3942,            5068,            1459,            489,            2275,            845,            3050,            5557,            1697,            4027,            1873,            1640,            ],        [            1822,            2620,            2549,            3872,            2895,            3410,            5056,            2725,            3760,            3541,            2088,            3900,            3474,            2471,            2368,            4496,            3548,            1937,            3836,            1796,            3370,            2674,            3163,            1108,            3634,            2392,            2660,            3125,            3073,            3322,            3655,            2967,            1490,            664,            895,            1161,            2257,            1470,            1367,            1703,            950,            890,            587,            779,            2082,            4889,            675,            1540,            923,            1278,            1440,            2923,            2042,            1661,            1544,            2101,            0,            418,            296,            2833,            2443,            951,            2566,            1270,            1668,            2570,            715,            484,            1055,            2380,            1798,            1935,            890,            1906,            3346,            3840,            1417,            2311,            2680,            4169,            1823,            1891,            1479,            1466,            2065,            1867,            470,            2091,            3389,            4515,            1719,            1089,            1722,            1445,            2497,            5004,            1144,            3474,            1320,            454,            ],        [            2238,            3035,            2964,            4287,            3310,            3825,            5471,            3140,            4175,            3956,            2503,            4315,            3889,            2886,            2783,            4911,            3963,            2352,            4251,            2211,            3785,            3089,            3578,            1131,            4049,            3024,            3082,            3540,            3488,            3737,            4070,            3382,            1906,            687,            918,            743,            2507,            1720,            1803,            1953,            1365,            1305,            1002,            1123,            2332,            5304,            1090,            1563,            946,            1301,            1690,            3173,            2707,            1911,            1960,            2518,            780,            0,            711,            2856,            2858,            533,            2816,            2012,            1918,            2820,            1072,            899,            1471,            3045,            2463,            2185,            1140,            2156,            3761,            4255,            1667,            2561,            2930,            4584,            1405,            2141,            2020,            1881,            2220,            1449,            842,            2506,            3804,            4930,            2136,            1504,            2137,            1860,            2912,            5419,            1559,            3889,            1735,            491,            ],        [            2080,            2324,            2253,            3576,            2599,            3114,            4760,            2429,            3464,            3245,            1792,            3604,            3178,            2175,            2072,            4200,            3252,            1641,            3540,            1500,            3074,            2378,            2867,            1757,            3338,            2096,            2364,            2829,            2777,            3026,            3359,            2671,            1748,            1313,            1544,            1492,            1961,            1174,            1677,            1407,            654,            594,            850,            483,            1786,            4593,            379,            2324,            1419,            1927,            1144,            2627,            1746,            1365,            1802,            1805,            913,            749,            0,            2590,            2147,            1282,            2270,            974,            1372,            2274,            978,            747,            1313,            2084,            1502,            1639,            594,            1610,            3050,            3544,            1121,            2015,            2384,            3873,            1767,            1595,            1665,            1170,            1769,            1811,            728,            1795,            3093,            4219,            1423,            793,            1426,            1149,            2201,            4708,            848,            3178,            1024,            1103,            ],        [            1598,            2966,            665,            2177,            3241,            3756,            2802,            2331,            1506,            3887,            2922,            1646,            3820,            2817,            2714,            2242,            3894,            2283,            1582,            3044,            3716,            1398,            1887,            2038,            3980,            3226,            226,            1994,            3419,            1826,            2020,            3313,            2589,            3648,            2251,            3829,            4772,            3985,            3063,            4218,            3460,            3400,            3661,            3294,            4597,            2635,            3185,            1605,            2481,            1939,            3955,            4977,            4627,            4176,            2643,            4611,            3476,            3086,            3346,            0,            747,            3619,            5081,            3780,            3408,            4381,            3789,            3558,            2520,            4965,            4383,            4450,            3405,            4561,            3692,            4186,            3932,            4826,            5195,            1915,            4491,            4406,            3115,            2300,            4575,            4535,            3538,            890,            3735,            2828,            4229,            3674,            1189,            4030,            2526,            2750,            1978,            3820,            1647,            3553,            ],        [            3451,            2219,            410,            589,            2494,            3009,            2055,            1584,            759,            3140,            2603,            899,            3073,            2070,            1967,            1495,            3147,            1536,            835,            2297,            2969,            651,            1140,            3345,            3233,            2907,            521,            1247,            2672,            1079,            1273,            2566,            3811,            2901,            3132,            3082,            4025,            3238,            3688,            3471,            2713,            2653,            2914,            2547,            3850,            1888,            2438,            3458,            3160,            3515,            3208,            4691,            3880,            3429,            3865,            3864,            2729,            2339,            2599,            747,            0,            2872,            4334,            3033,            3436,            4338,            3042,            2811,            3376,            4218,            3636,            3703,            2658,            3674,            2945,            3439,            3185,            4079,            4448,            1168,            3744,            3659,            3729,            1946,            3828,            3788,            2791,            453,            2988,            1240,            3482,            2927,            835,            3283,            1779,            2003,            1624,            3073,            1293,            2806,            ],        [            2562,            3359,            3288,            4611,            3634,            4149,            5795,            3464,            4499,            4280,            2827,            4639,            4213,            3210,            3107,            5235,            4287,            2676,            4575,            2535,            4109,            3413,            3902,            1455,            4373,            2637,            3406,            3864,            3812,            4061,            4394,            3706,            2230,            1011,            1242,            356,            2831,            2044,            2127,            2277,            1689,            1629,            1326,            1447,            2656,            5628,            1414,            1887,            1270,            1625,            2014,            3497,            2320,            2235,            2284,            2131,            1104,            463,            1035,            3180,            3182,            0,            3140,            1625,            2242,            3144,            1396,            1223,            1795,            2658,            2076,            2509,            1464,            2480,            4085,            4579,            1991,            2885,            3254,            4908,            1018,            2465,            2344,            2205,            1833,            1062,            1166,            2830,            4128,            5254,            1749,            1828,            2461,            2184,            3236,            5743,            1883,            4213,            2059,            815,            ],        [            4414,            4872,            4470,            6120,            5147,            5160,            7304,            4977,            6008,            5789,            4073,            6148,            4307,            4392,            4620,            6744,            4383,            3858,            6084,            4358,            4205,            6182,            6671,            3759,            5886,            2859,            4581,            5377,            5325,            5574,            5907,            5219,            3439,            3315,            3546,            2778,            1042,            1332,            2877,            1016,            1853,            2045,            2357,            1990,            748,            7137,            2592,            4658,            3439,            3929,            1640,            357,            622,            1674,            3587,            1459,            2915,            3003,            2881,            5838,            4364,            2894,            0,            1674,            2271,            2082,            2485,            2254,            3333,            760,            833,            784,            2101,            2262,            5598,            4675,            1396,            953,            114,            6417,            2124,            1403,            2564,            3387,            1784,            2434,            2709,            4012,            5641,            6763,            1360,            2516,            3643,            2074,            4749,            7252,            3065,            5726,            3241,            3105,            ],        [            2912,            3108,            3037,            4360,            3383,            3898,            5544,            3213,            4248,            4029,            2576,            4388,            3962,            2959,            2856,            4984,            4036,            2425,            4324,            2284,            3858,            3162,            3651,            2229,            4122,            2880,            3148,            3613,            3561,            3810,            4143,            3455,            2580,            1785,            2016,            2282,            2001,            1941,            2444,            2174,            387,            579,            1617,            1250,            1905,            5377,            674,            2661,            2044,            2399,            1911,            2746,            1610,            1534,            2634,            1421,            1498,            1539,            1429,            3374,            2931,            2153,            2389,            0,            2139,            3041,            1745,            1514,            2145,            1948,            1366,            1743,            1361,            2377,            3834,            4328,            1888,            2110,            2214,            4657,            1383,            2362,            2432,            1954,            1385,            1427,            1560,            2579,            3877,            5003,            1039,            778,            2210,            1134,            2985,            5492,            1632,            3962,            1808,            1575,            ],        [            1808,            3594,            3523,            4846,            3869,            4384,            6030,            3699,            4734,            4515,            3062,            4874,            4448,            3445,            3342,            5470,            4522,            2911,            4810,            2770,            4344,            3648,            4137,            1873,            4608,            3366,            3319,            4099,            4047,            4296,            4629,            3941,            994,            1660,            1660,            2499,            1590,            803,            606,            1036,            1881,            1860,            912,            1043,            1415,            5863,            1645,            2052,            1660,            1893,            352,            2256,            2044,            1251,            1142,            2220,            1260,            1756,            1556,            3093,            3417,            2289,            1899,            2201,            0,            862,            784,            1055,            1456,            2382,            1828,            1268,            1154,            316,            4320,            4814,            596,            1644,            2013,            5143,            2994,            1299,            293,            2440,            2723,            3205,            986,            3065,            4363,            4237,            2377,            1569,            2696,            1651,            3471,            5978,            2118,            4448,            2294,            1450,            ],        [            2237,            3945,            3874,            5197,            4220,            4735,            6381,            4050,            5085,            4866,            3413,            5225,            4799,            3796,            3693,            5821,            4873,            3262,            5161,            3121,            4695,            3999,            4488,            2302,            4959,            3717,            4378,            4450,            4398,            4647,            4980,            4292,            1423,            2089,            2089,            2928,            1961,            1411,            1035,            1644,            2232,            2211,            1341,            1394,            1643,            6214,            1996,            2481,            2089,            2322,            703,            2484,            2415,            1602,            1571,            2378,            1689,            2185,            1985,            4152,            3768,            2718,            2127,            2552,            429,            0,            1213,            1484,            1885,            2753,            2199,            1876,            1505,            180,            4671,            5165,            947,            1872,            2241,            5494,            3345,            945,            722,            2791,            3074,            3389,            1415,            3416,            4714,            4588,            2728,            1920,            3047,            2002,            3822,            6329,            2469,            4799,            2645,            1879,            ],        [            1615,            3118,            3047,            4370,            3393,            3908,            5554,            3223,            4258,            4039,            2586,            4398,            3972,            2969,            2866,            4994,            4046,            2435,            4334,            2294,            3868,            3172,            3661,            1342,            4132,            2890,            3158,            3623,            3571,            3820,            4153,            3465,            1283,            898,            1129,            1737,            2089,            1302,            699,            1535,            1471,            1388,            128,            495,            1914,            5387,            1173,            1859,            954,            1512,            1272,            2755,            2112,            1493,            1235,            1810,            498,            994,            794,            3067,            2941,            1527,            2398,            1791,            788,            1650,            0,            271,            848,            2450,            1868,            1767,            722,            1104,            3844,            4338,            1249,            2143,            2512,            4667,            2399,            1723,            811,            1964,            2313,            2443,            224,            2589,            3887,            5013,            1967,            1159,            2220,            1515,            2995,            5502,            1642,            3972,            1818,            688,            ],        [            3037,            2913,            2842,            4165,            3188,            3703,            5349,            3018,            4053,            3834,            2381,            4193,            3767,            2764,            2661,            4789,            3841,            2230,            4129,            2089,            3663,            2967,            3456,            2028,            3927,            2685,            2953,            3418,            3366,            3615,            3948,            3260,            2223,            1584,            1815,            2081,            1818,            1031,            1835,            1264,            1200,            1179,            1381,            224,            1643,            5182,            964,            2460,            1843,            2198,            1001,            2484,            1841,            1222,            2371,            1539,            1297,            1338,            1228,            3179,            2736,            1871,            2127,            1520,            1229,            2131,            1509,            0,            1944,            2179,            1597,            1496,            451,            1467,            3639,            4133,            978,            1872,            2241,            4462,            2313,            1452,            1522,            1759,            2042,            2357,            1733,            2384,            3682,            4808,            1696,            888,            2015,            1244,            2790,            5297,            1437,            3767,            1613,            1374,            ],        [            1026,            3318,            3247,            4570,            3593,            4108,            5754,            3423,            4458,            4239,            2786,            4598,            4172,            3169,            3066,            5194,            4246,            2635,            4534,            2494,            4068,            3372,            3861,            852,            4332,            3090,            2537,            3823,            3771,            4020,            4353,            3665,            694,            1098,            639,            1937,            2696,            1909,            816,            2142,            1648,            1588,            824,            1191,            2521,            5587,            1373,            1270,            639,            1022,            1678,            3362,            2551,            2100,            748,            2249,            698,            1194,            994,            2311,            3141,            1727,            3005,            1968,            1326,            2188,            696,            967,            0,            2889,            2307,            2374,            1329,            1642,            4044,            4538,            1856,            2750,            3119,            4867,            2599,            2330,            1033,            2164,            2763,            2643,            466,            2789,            4087,            5213,            2406,            1598,            2420,            1954,            3195,            5702,            1842,            4172,            2018,            888,            ],        [            3953,            4149,            4078,            5401,            4424,            5061,            6585,            4254,            5076,            5070,            3617,            5429,            4208,            4000,            3897,            6025,            4284,            3466,            5365,            3325,            4106,            4203,            4692,            3270,            5163,            1946,            4189,            4654,            4602,            4851,            5184,            4496,            3621,            2826,            3057,            2527,            898,            1579,            3154,            1346,            1428,            1620,            2327,            1960,            1276,            6418,            1715,            3702,            3085,            3440,            2031,            1042,            338,            1723,            4248,            1065,            2539,            2580,            2470,            4415,            3972,            2643,            1304,            1423,            2849,            2743,            2455,            2224,            3186,            0,            582,            1114,            2071,            2923,            4875,            4576,            1787,            1614,            775,            5698,            1873,            2064,            3142,            2995,            448,            1917,            2679,            3620,            4918,            6044,            1109,            1554,            3251,            1910,            4026,            6533,            2673,            5003,            2849,            2616,            ],        [            3371,            3567,            3496,            4819,            3842,            4357,            6003,            3672,            4494,            4488,            3035,            4847,            4421,            3418,            3315,            5443,            4495,            2884,            4783,            2743,            4317,            3621,            4110,            2688,            4581,            2588,            3607,            4072,            4020,            4269,            4602,            3914,            3039,            2244,            2475,            1945,            405,            1025,            2572,            792,            846,            1038,            1745,            1378,            722,            5836,            1133,            3120,            2503,            2858,            1477,            1115,            244,            1169,            3108,            483,            1957,            1998,            1888,            3833,            3390,            2061,            1206,            841,            2267,            2056,            1873,            1642,            2604,            582,            0,            560,            1489,            2236,            4293,            4787,            1233,            927,            848,            5116,            1291,            1377,            2560,            2413,            873,            1335,            2097,            3038,            4336,            5462,            527,            972,            2669,            1328,            3444,            5951,            2091,            4421,            2267,            2034,            ],        [            3892,            3818,            3747,            5070,            4093,            4608,            6254,            3923,            4958,            4739,            3286,            5098,            4672,            3669,            3566,            5694,            4746,            3135,            5034,            2994,            4568,            3872,            4361,            3036,            4832,            3590,            3858,            4323,            4271,            4520,            4853,            4165,            2542,            2592,            2823,            3023,            322,            465,            2154,            232,            1133,            1325,            1634,            1267,            226,            6087,            1420,            4136,            2716,            3206,            917,            1067,            776,            609,            2690,            739,            2192,            2280,            2158,            5316,            3641,            2618,            710,            1398,            1548,            1560,            1762,            1531,            2610,            1114,            560,            0,            1378,            1740,            4544,            5038,            673,            431,            824,            5367,            1848,            881,            1841,            2664,            1430,            1892,            1986,            3289,            4587,            4461,            1084,            1793,            2920,            1009,            3695,            6202,            2342,            4672,            2518,            2382,            ],        [            2586,            3114,            3043,            4366,            3389,            3904,            5550,            3219,            4254,            4035,            2582,            4394,            3968,            2965,            2862,            4990,            4042,            2431,            4330,            2290,            3864,            3168,            3657,            2332,            4128,            2886,            3154,            3619,            3567,            3816,            4149,            3461,            1772,            1888,            2119,            2319,            1367,            580,            1384,            813,            1401,            1380,            930,            563,            1192,            5383,            1165,            2830,            2012,            2502,            550,            2033,            1821,            771,            1920,            1740,            1488,            1576,            1454,            3380,            2937,            2109,            1676,            1721,            778,            1680,            1058,            827,            1906,            2159,            1605,            1045,            0,            1016,            3840,            4334,            527,            1421,            1790,            4663,            2514,            1001,            1071,            1960,            2243,            2558,            1282,            2585,            3883,            5009,            1897,            1089,            2216,            1171,            2991,            5498,            1638,            3968,            1814,            1678,            ],        [            2057,            3765,            3694,            5017,            4040,            4555,            6201,            3870,            4905,            4686,            3233,            5045,            4619,            3616,            3513,            5641,            4693,            3082,            4981,            2941,            4515,            3819,            4308,            2122,            4779,            3537,            4624,            4270,            4218,            4467,            4800,            4112,            1243,            1909,            1909,            2748,            2018,            1231,            855,            1464,            2052,            2031,            1161,            1214,            1843,            6034,            1816,            2301,            1909,            2142,            523,            2684,            2472,            1422,            1391,            2391,            1509,            2005,            1805,            4398,            3588,            2538,            2327,            2372,            249,            541,            1033,            1304,            1705,            2810,            2256,            1696,            1325,            0,            4491,            4985,            767,            2072,            2441,            5314,            3165,            1191,            542,            2611,            2894,            3209,            1235,            3236,            4534,            4408,            2548,            1740,            2867,            1822,            3642,            6149,            2289,            4619,            2465,            1699,            ],        [            5485,            765,            3408,            2631,            484,            188,            2492,            1318,            1909,            2300,            1833,            2659,            1386,            4083,            1219,            2006,            713,            3549,            2090,            1527,            656,            2693,            3182,            5358,            288,            2137,            3519,            1487,            1072,            1684,            1639,            966,            5824,            4914,            5145,            5095,            6038,            5251,            5701,            5484,            4726,            4666,            4927,            4560,            5863,            2325,            4451,            5492,            5173,            5528,            5221,            5877,            5893,            5442,            5878,            5877,            4742,            4352,            4612,            3745,            2998,            4885,            6347,            5046,            5449,            6351,            5055,            4824,            5389,            5483,            5649,            5716,            4671,            5687,            0,            421,            5198,            6449,            5610,            2928,            3082,            5672,            5742,            1901,            3065,            2837,            4804,            3084,            167,            3274,            3480,            4940,            3428,            5296,            1090,            2440,            2136,            128,            2601,            4819,            ],        [            5955,            1062,            3878,            3101,            781,            485,            2789,            1615,            2206,            2770,            1648,            3129,            1288,            4553,            1601,            2303,            292,            4019,            3065,            1824,            470,            3163,            3652,            5828,            709,            1792,            3989,            1784,            1369,            1981,            1936,            1263,            6294,            5384,            5615,            5565,            5602,            5721,            6171,            5954,            5196,            5136,            5397,            5030,            5308,            2622,            4921,            5962,            5643,            5998,            5691,            4941,            4885,            5912,            6348,            3796,            5212,            4822,            5082,            4215,            3468,            3754,            5203,            3696,            5919,            6821,            5525,            5294,            5859,            4547,            3741,            5344,            5141,            6157,            421,            0,            5668,            5513,            4674,            3398,            2984,            6142,            6212,            2198,            2967,            2739,            5274,            3554,            464,            3744,            3382,            5410,            3898,            5766,            1387,            2737,            2433,            549,            2898,            5289,            ],        [            2404,            3321,            3250,            4573,            3596,            4111,            5757,            3426,            4461,            4242,            2789,            4601,            4175,            3172,            3069,            5197,            4249,            2638,            4537,            2497,            4071,            3375,            3864,            2539,            4335,            3093,            3361,            3826,            3774,            4023,            4356,            3668,            1590,            2095,            2326,            2526,            994,            207,            1202,            440,            1608,            1587,            1137,            770,            819,            5590,            1372,            2648,            2219,            2709,            244,            1660,            1448,            454,            1738,            1411,            1695,            1783,            1661,            5063,            3144,            2316,            1303,            1928,            596,            1307,            1265,            1034,            2052,            1786,            1232,            672,            881,            834,            4047,            4541,            0,            1048,            1417,            4870,            2721,            628,            889,            2167,            2036,            2765,            1489,            2792,            4090,            3964,            1690,            1296,            2423,            854,            3198,            5705,            1845,            4175,            2021,            1885,            ],        [            3461,            3588,            3517,            4840,            3863,            4378,            6024,            3693,            4728,            4509,            3056,            4868,            4442,            3439,            3336,            5464,            4516,            2905,            4804,            2764,            4338,            3642,            4131,            2806,            4602,            3360,            3628,            4093,            4041,            4290,            4623,            3935,            2486,            2362,            2593,            2793,            1166,            379,            1924,            612,            1875,            1854,            1404,            1037,            641,            5857,            1639,            3705,            2486,            2976,            687,            1482,            1413,            721,            2634,            1376,            1962,            2050,            1928,            4885,            3411,            2583,            1125,            2195,            1318,            1129,            1532,            1301,            2380,            1751,            1197,            844,            1148,            1309,            4314,            4808,            443,            0,            1239,            5137,            2988,            450,            1611,            2434,            2303,            3032,            1756,            3059,            4357,            4231,            1957,            1563,            2690,            1121,            3465,            5972,            2112,            4442,            2288,            2152,            ],        [            4300,            4427,            4356,            5679,            4702,            5217,            6863,            4532,            5354,            5348,            3895,            5707,            4836,            4278,            4175,            6303,            5355,            3744,            5643,            3603,            5177,            4481,            4970,            3645,            5441,            4199,            4467,            4932,            4880,            5129,            5462,            4774,            3325,            3201,            3432,            3632,            928,            1218,            2763,            902,            1739,            1931,            2243,            1876,            634,            6696,            2478,            4544,            3325,            3815,            1526,            886,            1382,            1560,            3473,            1345,            2801,            2889,            2767,            5724,            4250,            3224,            529,            2004,            2157,            1968,            2371,            2140,            3219,            1289,            1166,            670,            1987,            2148,            5153,            5647,            1282,            839,            0,            5976,            2454,            1289,            2450,            3273,            2036,            2498,            2595,            3898,            5196,            5070,            1690,            2402,            3529,            1960,            4304,            6811,            2951,            5281,            3127,            2991,            ],        [            3709,            1972,            1624,            847,            2040,            2555,            887,            1434,            609,            3041,            2832,            269,            3302,            2299,            2196,            327,            3376,            1765,            538,            2526,            3198,            909,            1398,            3574,            2779,            3136,            1735,            1097,            1567,            929,            1042,            2319,            4040,            3130,            3361,            3311,            4254,            3467,            3917,            3700,            2942,            2882,            3143,            2776,            4079,            720,            2667,            3716,            3389,            3744,            3437,            4920,            4109,            3658,            4094,            4093,            2958,            2568,            2828,            1961,            1214,            3101,            4563,            3262,            3665,            4567,            3271,            3040,            3605,            4447,            3865,            3932,            2887,            3903,            2491,            2788,            3414,            4308,            4677,            0,            3973,            3888,            3958,            2755,            4057,            4017,            3020,            1300,            2534,            1498,            3711,            3156,            1644,            3512,            1629,            835,            2433,            2619,            2102,            3035,            ],        [            3332,            4129,            4058,            5381,            4404,            3488,            6565,            4234,            5056,            5050,            3597,            5409,            2637,            3980,            3877,            6005,            2711,            3446,            5345,            3305,            2533,            4183,            4672,            2225,            5143,            1853,            4176,            4634,            4582,            4831,            5164,            4476,            3000,            1781,            2012,            654,            1645,            2814,            2897,            2029,            1176,            1368,            2096,            2217,            1959,            6398,            2184,            2657,            2040,            2395,            2784,            2355,            1484,            2323,            3054,            1295,            1874,            1233,            1805,            3950,            3952,            770,            2443,            789,            3012,            3914,            2166,            1993,            2565,            1822,            1240,            1797,            2234,            3250,            4855,            3003,            2761,            2164,            2088,            5678,            0,            3235,            3114,            2975,            997,            278,            1936,            3600,            4898,            6024,            913,            1567,            3231,            1923,            4006,            6513,            2653,            4983,            2829,            1585,            ],        [            3011,            4077,            4006,            5329,            4352,            4867,            6513,            4182,            5217,            4998,            3545,            5357,            4931,            3928,            3825,            5953,            5005,            3394,            5293,            3253,            4827,            4131,            4620,            3034,            5091,            3849,            4661,            4582,            4530,            4779,            5112,            4424,            2036,            2768,            2869,            3282,            1418,            868,            1714,            1101,            2364,            2343,            1893,            1526,            1100,            6346,            2128,            3255,            2869,            2935,            1176,            1941,            1872,            1210,            2184,            1835,            2368,            2539,            2417,            4435,            3900,            3072,            1584,            2494,            1108,            679,            2021,            1790,            2498,            2210,            1656,            1160,            1637,            859,            4803,            5297,            932,            1329,            1698,            5626,            2944,            0,            1401,            2923,            2526,            2988,            2094,            3548,            4846,            4720,            2180,            2052,            3179,            1610,            3954,            6461,            2601,            4931,            2777,            2558,            ],        [            1515,            3887,            3816,            5139,            4162,            4677,            6323,            3992,            5027,            4808,            3355,            5167,            4741,            3738,            3635,            5763,            4815,            3204,            5103,            3063,            4637,            3941,            4430,            1580,            4901,            3659,            3026,            4392,            4340,            4589,            4922,            4234,            701,            1663,            1367,            2502,            1883,            1096,            313,            1329,            2174,            2153,            915,            1336,            1708,            6156,            1938,            1759,            1367,            1600,            645,            2549,            2337,            1544,            849,            2513,            1263,            1759,            1559,            2800,            3710,            2292,            2192,            2494,            293,            1155,            787,            1058,            1163,            2675,            2121,            1561,            1447,            609,            4613,            5107,            889,            1937,            2306,            5436,            3164,            1592,            0,            2733,            3016,            3208,            989,            3358,            4656,            4530,            2670,            1862,            2989,            1944,            3764,            6271,            2411,            4741,            2587,            1453,            ],        [            3229,            1064,            2108,            2316,            1339,            1854,            3500,            1169,            2204,            1985,            622,            2344,            1921,            3768,            812,            2940,            1995,            3234,            2280,            330,            1817,            2378,            2867,            2431,            2078,            926,            2219,            1569,            1517,            1766,            2099,            1411,            2897,            1987,            2218,            2168,            3111,            2324,            2774,            2557,            1799,            1739,            2000,            1633,            2936,            3333,            1524,            2863,            2246,            2601,            2294,            3777,            2966,            2515,            2951,            2950,            1815,            1425,            1685,            2445,            2683,            1958,            3420,            2119,            2522,            3424,            2128,            1897,            2462,            3304,            2722,            2789,            1744,            2760,            1790,            2087,            2271,            3165,            3534,            2613,            2830,            2745,            2815,            0,            2810,            2874,            1877,            1650,            1833,            2959,            2568,            2013,            1281,            2369,            941,            3448,            414,            1918,            879,            1892,            ],        [            3641,            3837,            3766,            5089,            4112,            4627,            6273,            3942,            4977,            4758,            3305,            5117,            4691,            3688,            3585,            5713,            4765,            3154,            5053,            3013,            4587,            3891,            4380,            2958,            4851,            1916,            3877,            4342,            4290,            4539,            4872,            4184,            3309,            2514,            2745,            1833,            1179,            1796,            2918,            1563,            1116,            1308,            2091,            1724,            1493,            6106,            1403,            3390,            2773,            3128,            2385,            1889,            1018,            2263,            3454,            829,            2227,            2268,            2158,            4103,            3660,            1949,            1977,            729,            2613,            2827,            2219,            1988,            2874,            1356,            774,            1331,            1835,            2851,            4563,            5057,            2362,            1698,            1622,            5386,            1179,            2148,            2906,            2683,            0,            1223,            2289,            3308,            4606,            5732,            415,            1507,            2939,            1863,            3714,            6221,            2361,            4691,            2537,            2304,            ],        [            3593,            2924,            4319,            4172,            3199,            3210,            5356,            3029,            4060,            3841,            2018,            4200,            2359,            4241,            2672,            4796,            2433,            3707,            4136,            2303,            2255,            4234,            4723,            2486,            3103,            1575,            4437,            3429,            3377,            3626,            3959,            3271,            3261,            2042,            2273,            915,            1740,            2357,            3158,            2124,            1271,            1463,            2357,            2478,            2054,            5189,            1558,            2918,            2301,            2656,            3045,            2889,            1579,            2418,            3315,            1390,            2135,            1494,            2066,            4211,            4539,            1031,            2538,            884,            3273,            3388,            2427,            2254,            2826,            1917,            1335,            1892,            2495,            3511,            2815,            2725,            3022,            2259,            2622,            4469,            261,            2709,            3375,            3236,            1378,            0,            2197,            3861,            2858,            4815,            1008,            1662,            3492,            2018,            2801,            5304,            2914,            2943,            3090,            1846,            ],        [            1441,            2894,            2823,            4146,            3169,            3684,            5330,            2999,            4034,            3815,            2362,            4174,            3748,            2745,            2642,            4770,            3822,            2211,            4110,            2070,            3644,            2948,            3437,            1118,            3908,            2666,            2934,            3399,            3347,            3596,            3929,            3241,            1109,            674,            905,            1513,            2272,            1485,            897,            1718,            1224,            1164,            358,            725,            2097,            5163,            949,            1685,            780,            1288,            1455,            2938,            2127,            1676,            1163,            1825,            274,            770,            570,            2843,            2717,            1303,            2581,            1544,            986,            1848,            230,            501,            674,            2465,            1883,            1950,            905,            1302,            3620,            4114,            1432,            2326,            2695,            4443,            2175,            1906,            1009,            1740,            2339,            2219,            0,            2365,            3663,            4789,            1982,            1174,            1996,            1530,            2771,            5278,            1418,            3748,            1594,            464,            ],        [            3803,            2571,            458,            941,            2846,            3361,            2407,            1936,            1111,            3492,            2254,            1251,            3425,            2422,            2319,            1847,            3499,            1888,            1187,            1962,            3321,            1003,            1492,            2963,            3585,            2558,            569,            1599,            3024,            1431,            1625,            2918,            3429,            2519,            2750,            2700,            3643,            2856,            3304,            3089,            2331,            2271,            2532,            2165,            3468,            2240,            2056,            3810,            2778,            3133,            2826,            4309,            3498,            3047,            3483,            3482,            2347,            1957,            2217,            795,            352,            2490,            3952,            2651,            3054,            3956,            2660,            2429,            2994,            3836,            3254,            3321,            2276,            3292,            3297,            3791,            2803,            3697,            4066,            1520,            3362,            3277,            3347,            1632,            3446,            3406,            2409,            0,            3340,            1592,            3100,            2545,            521,            2901,            2131,            2355,            1310,            3425,            979,            2424,            ],        [            5504,            784,            3427,            2650,            503,            207,            2511,            1337,            1928,            2319,            1852,            2678,            1405,            4102,            1238,            2025,            732,            3568,            2109,            1546,            675,            2712,            3201,            5377,            431,            2156,            3538,            1506,            1091,            1703,            1658,            985,            5843,            4933,            5164,            5114,            6057,            5270,            5720,            5503,            4745,            4685,            4946,            4579,            5882,            2344,            4470,            5511,            5192,            5547,            5240,            5896,            5912,            5461,            5897,            5896,            4761,            4371,            4631,            4788,            3017,            4904,            6366,            5065,            5468,            6370,            5074,            4843,            5408,            5502,            5668,            5735,            4690,            5706,            143,            440,            5217,            6468,            5629,            2947,            3101,            5691,            5761,            1920,            3084,            2856,            4823,            3103,            0,            3293,            3499,            4959,            3447,            5315,            1109,            2459,            2155,            271,            2620,            4838,            ],        [            2211,            3103,            1990,            1310,            3378,            3893,            2944,            3208,            1648,            4024,            3487,            1788,            3957,            2954,            2851,            2384,            4031,            2420,            1724,            3181,            3853,            1540,            2029,            2651,            4117,            3791,            1721,            2136,            3556,            1968,            2162,            3450,            3202,            3790,            4021,            3971,            4914,            4127,            4575,            4360,            3602,            3542,            3803,            3436,            4739,            2777,            3327,            2218,            4049,            2552,            4097,            5580,            4769,            4318,            3256,            4753,            3618,            3228,            3488,            1495,            1580,            3761,            5223,            3922,            4325,            5227,            3931,            3700,            3133,            5107,            4525,            4592,            3547,            4563,            3829,            4323,            4074,            4968,            5337,            2057,            4633,            4548,            4618,            3555,            4717,            4677,            3680,            1849,            3872,            0,            4371,            3816,            2193,            4172,            2668,            2892,            2982,            3957,            2651,            3695,            ],        [            3226,            3422,            3351,            4674,            3697,            4212,            5858,            3527,            4562,            4343,            2890,            4702,            4276,            3273,            3170,            5298,            4350,            2739,            4638,            2598,            4172,            3476,            3965,            2543,            4436,            2406,            3462,            3927,            3875,            4124,            4457,            3769,            2894,            2099,            2330,            1418,            1090,            1707,            2758,            1474,            701,            893,            1931,            1564,            1404,            5691,            988,            2975,            2358,            2713,            2225,            1800,            929,            1848,            2948,            740,            1812,            1853,            1743,            3688,            3245,            1534,            1888,            314,            2453,            2738,            2059,            1828,            2459,            1267,            685,            1242,            1675,            2691,            4148,            4642,            2202,            1609,            1533,            4971,            764,            2059,            2746,            2268,            717,            808,            1874,            2893,            4191,            5317,            0,            1092,            2524,            1448,            3299,            5806,            1946,            4276,            2122,            1889,            ],        [            3204,            3400,            3329,            4652,            3675,            4190,            5836,            3505,            4540,            4321,            2868,            4680,            4254,            3251,            3148,            5276,            4328,            2717,            4616,            2576,            4150,            3454,            3943,            2521,            4414,            3172,            3440,            3905,            3853,            4102,            4435,            3747,            2872,            2077,            2308,            2574,            1223,            1010,            2637,            1133,            679,            871,            2117,            1750,            1127,            5669,            966,            2953,            2336,            2691,            1454,            1968,            953,            756,            3173,            651,            1790,            1831,            1721,            3666,            3223,            2362,            1611,            799,            2031,            2363,            2245,            2014,            2437,            1291,            709,            965,            1653,            2269,            4126,            4620,            1210,            1332,            1725,            4949,            1592,            1684,            2324,            2246,            1154,            1636,            1852,            2871,            4169,            5295,            808,            0,            2502,            356,            3277,            5784,            1924,            4254,            2100,            1867,            ],        [            3240,            2175,            1005,            1068,            2450,            2965,            2534,            2280,            1238,            3096,            1733,            1378,            3552,            2549,            1923,            1974,            3626,            2015,            1314,            1441,            3448,            1130,            1619,            2442,            3189,            2037,            1116,            1726,            2628,            1558,            1752,            2522,            2908,            1998,            2229,            2179,            3122,            2335,            2785,            2568,            1810,            1750,            2011,            1644,            2947,            2367,            1535,            2874,            2257,            2612,            2305,            3788,            2977,            2526,            2962,            2961,            1826,            1436,            1696,            1342,            899,            1969,            3431,            2130,            2533,            3435,            2139,            1908,            2473,            3315,            2733,            2800,            1755,            2771,            2901,            3198,            2282,            3176,            3545,            1647,            2841,            2756,            2826,            1111,            2925,            2885,            1888,            547,            2944,            1719,            2579,            2024,            0,            2380,            2258,            2482,            789,            3029,            458,            1903,            ],        [            3232,            3428,            3357,            4680,            3703,            4218,            5864,            3533,            4568,            4349,            2896,            4708,            4282,            3279,            3176,            5304,            4356,            2745,            4644,            2604,            4178,            3482,            3971,            2549,            4442,            3200,            3468,            3933,            3881,            4130,            4463,            3775,            2669,            2105,            2336,            2602,            1331,            654,            2281,            777,            707,            899,            1761,            1394,            1155,            5697,            994,            2981,            2364,            2719,            1098,            1996,            981,            400,            2817,            679,            1818,            1859,            1749,            3694,            3251,            2390,            1639,            827,            1675,            2007,            1889,            1658,            2465,            1319,            737,            1009,            1505,            1913,            4154,            4648,            854,            1360,            1753,            4977,            1620,            1328,            1968,            2274,            1182,            1664,            2113,            2899,            4197,            5323,            836,            1098,            2530,            0,            3305,            5812,            1952,            4282,            2128,            1895,            ],        [            4597,            343,            2520,            1743,            618,            1133,            2102,            319,            1141,            1412,            945,            1771,            1418,            3195,            331,            2367,            1492,            2661,            1707,            639,            1314,            1805,            2294,            4470,            1357,            1249,            2631,            719,            796,            916,            1249,            690,            4936,            4026,            4257,            4207,            5150,            4363,            4813,            4596,            3838,            3778,            4039,            3672,            4975,            1935,            3563,            4604,            4285,            4640,            4333,            4989,            5005,            4554,            4990,            4989,            3854,            3464,            3724,            3881,            2110,            3997,            5459,            4158,            4561,            5463,            4167,            3936,            4501,            4595,            4761,            4828,            3783,            4799,            1069,            1366,            4310,            5561,            4722,            2040,            3037,            4784,            4854,            1013,            3020,            2792,            3916,            2196,            1112,            2386,            4607,            4052,            2540,            4408,            0,            2050,            1248,            1197,            1713,            3931,            ],        [            4544,            1171,            2459,            1682,            1336,            1851,            52,            2269,            1444,            2706,            3667,            1104,            4137,            3134,            3031,            508,            2185,            2600,            1373,            3361,            2007,            1744,            2233,            4409,            2075,            3971,            2570,            1932,            732,            1764,            1149,            962,            4875,            3965,            4196,            4146,            5089,            4302,            4752,            4535,            3777,            3717,            3978,            3611,            4914,            115,            3502,            4551,            4224,            4579,            4272,            5755,            4944,            4493,            4929,            4928,            3793,            3403,            3663,            2796,            2049,            3936,            5398,            4097,            4500,            5402,            4106,            3875,            4440,            5282,            4700,            4767,            3722,            4738,            1787,            2084,            4249,            5143,            5512,            835,            4808,            4723,            4793,            3590,            4892,            4852,            3855,            2135,            1830,            2333,            4546,            3991,            2479,            4347,            2464,            0,            3268,            1915,            2937,            3870,            ],        [            2815,            1750,            1694,            1757,            2025,            2540,            3223,            1855,            1927,            2671,            1308,            2067,            2607,            2906,            1498,            2663,            2681,            2372,            2003,            1016,            2503,            1819,            2308,            2017,            2764,            1612,            1805,            2415,            2203,            2247,            2441,            2097,            2483,            1573,            1804,            1754,            2697,            1910,            2360,            2143,            1385,            1325,            1586,            1219,            2522,            3056,            1110,            2449,            1832,            2187,            1880,            3363,            2552,            2101,            2537,            2536,            1401,            1011,            1271,            2031,            1588,            1544,            3006,            1705,            2108,            3010,            1714,            1483,            2048,            2890,            2308,            2375,            1330,            2346,            2476,            2773,            1857,            2751,            3120,            2336,            2416,            2331,            2401,            686,            2500,            2460,            1463,            1236,            2519,            2408,            2154,            1599,            867,            1955,            1627,            3171,            0,            2604,            465,            1478,            ],        [            5613,            893,            3536,            2759,            612,            316,            2620,            1446,            2037,            2428,            1961,            2787,            1514,            4211,            1347,            2134,            841,            3677,            2218,            1655,            784,            2821,            3310,            5486,            160,            2265,            3647,            1615,            1200,            1812,            1767,            1094,            5952,            5042,            5273,            5223,            6166,            5379,            5827,            5612,            4854,            4794,            5055,            4688,            5991,            2453,            4579,            5620,            5301,            5656,            5349,            6005,            6021,            5570,            6006,            6005,            4870,            4480,            4740,            4897,            3126,            5013,            6475,            5174,            5577,            6479,            5183,            4952,            5517,            5611,            5777,            5844,            4799,            5815,            128,            549,            5326,            6577,            5738,            3056,            3210,            5800,            5870,            2029,            3193,            2965,            4932,            3212,            295,            3402,            3608,            5068,            3556,            5424,            1218,            2568,            2264,            0,            2729,            4947,            ],        [            3007,            2922,            1229,            1292,            3197,            3712,            2758,            2287,            1462,            3843,            1500,            1602,            3776,            2773,            2670,            2198,            3850,            2239,            1538,            1208,            3672,            1354,            1843,            2209,            3936,            1804,            1340,            1950,            3375,            1782,            1976,            3269,            2675,            1765,            1996,            1946,            2889,            2102,            2550,            2335,            1577,            1517,            1778,            1411,            2714,            2591,            1302,            2641,            2024,            2379,            2072,            3555,            2744,            2293,            2729,            2728,            1593,            1203,            1463,            1566,            1123,            1736,            3198,            1897,            2300,            3202,            1906,            1675,            2240,            3082,            2500,            2567,            1522,            2538,            3648,            4142,            2049,            2943,            3312,            1871,            2608,            2523,            2593,            878,            2692,            2652,            1655,            771,            3691,            1943,            2346,            1791,            402,            2147,            2482,            2706,            556,            3776,            0,            1670,            ],        [            2145,            3156,            3085,            4408,            3431,            3946,            5592,            3261,            4296,            4077,            2624,            4436,            4010,            3007,            2904,            5032,            4084,            2473,            4372,            2332,            3906,            3210,            3699,            1038,            4170,            2928,            2989,            3661,            3609,            3858,            4191,            3503,            1813,            594,            825,            1049,            2628,            1841,            1710,            2074,            1486,            1426,            1123,            1244,            2453,            5425,            1211,            1470,            853,            1208,            1811,            3294,            3013,            2032,            1867,            2824,            901,            306,            832,            2763,            2979,            839,            2937,            2318,            2039,            2941,            1193,            1020,            1378,            3351,            2769,            2306,            1261,            2277,            3882,            4376,            1788,            2682,            3051,            4705,            1711,            2262,            1972,            2002,            2526,            1755,            963,            2627,            3925,            5051,            2442,            1530,            2258,            1886,            3033,            5540,            1680,            4010,            1856,            0,            ],        ], 'starts': [0, 19, 56, 79,91], 'ends': [0, 19, 56, 79,91]},    'NW-36': {'distance_matrix': [        [            0,            2537,            1223,            1699,            1109,            2310,            890,            1908,            1276,            1775,            2609,            2415,            781,            5684,            5054,            2150,            591,            1745,            2189,            519,            225,            2804,            1895,            1739,            5931,            6862,            4556,            6059,            1337,            2168,            4858,            1250,            1012,            664,            1061,            293,            934,            4513,            2814,            4035,            2934,            3294,            3177,            3054,            3727,            4213,            3665,            1956,            2536,            607,            2055,            2742,            1926,            1783,            2278,            1572,            3185,            6356,            5394,            1231,            807,            444,            5740,            5204,            2896,            1075,            3345,            1089,            1600,            948,            1783,            ],        [            2108,            0,            953,            2037,            1447,            1858,            2455,            3040,            2491,            1732,            1277,            3547,            1327,            5680,            3722,            1698,            2699,            2355,            3321,            2375,            1883,            1472,            2505,            1287,            5927,            9836,            3224,            5294,            1102,            2045,            3526,            1962,            2164,            1738,            2833,            2401,            2761,            3181,            876,            2097,            1666,            1356,            1909,            1738,            1123,            2881,            1389,            3088,            598,            1883,            1633,            1474,            1474,            560,            1856,            2787,            2021,            5130,            6290,            2663,            2112,            2300,            4408,            4040,            1580,            1111,            1597,            2805,            1486,            1454,            2393,            ],        [            1434,            1314,            0,            1254,            664,            1087,            1781,            2257,            1708,            931,            1386,            2764,            653,            5789,            3831,            927,            2025,            1554,            2538,            1701,            1209,            1581,            1704,            516,            6036,            7294,            3333,            5403,            319,            1244,            3635,            1179,            1381,            955,            2032,            1727,            1978,            3290,            1591,            2812,            2381,            2071,            2624,            2453,            2504,            2990,            2442,            2305,            1313,            1209,            832,            2189,            703,            560,            1055,            2004,            1962,            5239,            6399,            1862,            1438,            1626,            4517,            3981,            2295,            359,            2122,            2004,            377,            671,            1592,            ],        [            1582,            2372,            1058,            0,            651,            2145,            2472,            1693,            1144,            1733,            2444,            2200,            1249,            5552,            4889,            1985,            2173,            2356,            1974,            1376,            1357,            2639,            2506,            1574,            5799,            6730,            4391,            5927,            879,            2126,            4693,            615,            817,            924,            2643,            1875,            1414,            4348,            2649,            3870,            3439,            3129,            3682,            3511,            3562,            4048,            3500,            1741,            2371,            1679,            1890,            3247,            1761,            1618,            2113,            1440,            3020,            6224,            5262,            2664,            1908,            1301,            5575,            5039,            3353,            1033,            3180,            2671,            1435,            640,            2394,            ],        [            1589,            2272,            958,            590,            0,            2045,            2479,            1593,            1044,            1633,            2344,            2100,            1149,            5452,            4789,            1885,            2180,            2256,            1874,            1276,            1257,            2539,            2406,            1474,            5699,            6630,            4291,            5827,            779,            2026,            4593,            515,            717,            824,            2650,            1882,            1314,            4248,            2549,            5200,            3793,            3029,            4036,            3865,            3462,            3948,            3400,            1641,            2271,            1579,            1790,            3601,            1661,            1518,            2013,            1340,            2920,            6124,            5162,            2564,            1808,            1201,            5475,            4939,            3707,            933,            3080,            2678,            1335,            540,            2294,            ],        [            2310,            1291,            1155,            2239,            1649,            0,            4299,            3242,            2693,            1934,            299,            3749,            1529,            4702,            2744,            160,            2901,            2557,            3523,            2577,            2085,            494,            2707,            571,            4949,            8858,            2246,            4316,            1304,            2247,            2548,            2164,            2366,            1940,            4128,            2603,            2963,            2203,            1905,            3155,            2835,            3422,            3078,            2907,            1417,            1903,            1355,            3290,            2049,            2085,            1835,            2643,            384,            1851,            2058,            2989,            875,            4152,            5312,            4148,            2314,            2502,            3430,            2894,            2749,            1313,            1035,            4100,            828,            1656,            2595,            ],        [            890,            3017,            1570,            2589,            1999,            2657,            0,            1965,            2166,            1272,            2956,            2492,            1128,            6574,            5423,            2497,            299,            855,            2266,            1409,            1115,            3151,            1005,            2086,            6821,            7752,            4925,            6949,            2227,            879,            5227,            2140,            1902,            1554,            171,            597,            1824,            4882,            2456,            4960,            2044,            3197,            2287,            2164,            2964,            4582,            3230,            2033,            2783,            596,            1278,            1852,            2273,            2130,            1052,            2462,            3554,            6831,            6284,            341,            343,            1334,            6109,            5573,            2006,            1422,            3438,            199,            1947,            1838,            893,            ],        [            981,            3266,            1952,            1944,            1838,            3039,            1871,            0,            1508,            2504,            3338,            597,            1510,            5916,            5783,            2879,            1572,            3127,            371,            462,            954,            3533,            3277,            2468,            6163,            7094,            5285,            6291,            2066,            2897,            5587,            1399,            1161,            1393,            2042,            1274,            457,            5242,            3543,            6194,            3915,            4023,            4158,            4035,            4456,            4942,            4394,            138,            3265,            1336,            2784,            3723,            2655,            2512,            3007,            1804,            3914,            6588,            5626,            2212,            1565,            537,            6469,            5933,            3877,            1804,            4074,            2070,            2329,            1677,            3165,            ],        [            1740,            3399,            2085,            1733,            1678,            3172,            2630,            1085,            0,            2760,            3471,            1592,            2276,            4408,            5916,            3012,            2331,            3383,            1366,            1221,            1713,            3666,            3533,            2601,            4655,            5586,            5418,            4783,            1906,            3153,            5720,            1188,            950,            1951,            2801,            2033,            806,            5375,            3676,            6327,            4478,            4156,            4721,            4992,            4589,            5075,            4527,            1133,            3398,            2095,            2917,            4728,            2788,            2645,            3140,            296,            4047,            5080,            4118,            2971,            2324,            1296,            5752,            6130,            4834,            2060,            4207,            2829,            2462,            1667,            3421,            ],        [            1775,            2240,            926,            1836,            1246,            2013,            1272,            2839,            2290,            0,            2312,            3346,            994,            6715,            4757,            1853,            1571,            623,            3120,            2042,            1550,            2507,            773,            1442,            6962,            7876,            4259,            6329,            1258,            393,            4561,            1761,            1963,            1537,            1101,            2068,            2457,            4216,            1784,            5168,            2424,            2997,            2667,            2496,            3430,            3916,            3368,            2887,            2239,            1550,            1758,            2232,            1629,            1486,            820,            2586,            2888,            6165,            6408,            931,            1615,            1967,            5443,            4907,            2338,            700,            3048,            1073,            1303,            1253,            661,            ],        [            2609,            1030,            1454,            2538,            1948,            299,            3499,            3541,            2992,            2312,            0,            4048,            1828,            4403,            2445,            459,            3200,            2935,            3822,            2876,            2384,            195,            3085,            870,            4650,            8559,            1947,            4017,            1603,            2705,            2249,            2463,            2665,            2239,            3413,            2902,            3262,            1904,            1606,            2856,            2536,            3123,            2779,            2608,            1118,            1604,            1056,            3589,            1750,            2384,            2187,            2344,            683,            1590,            2410,            3288,            576,            3853,            5013,            3243,            2613,            2801,            3131,            2595,            2450,            1612,            736,            3385,            1127,            1955,            2973,            ],        [            1578,            3863,            2549,            2541,            2435,            3636,            2468,            597,            2105,            3101,            3935,            0,            2107,            6513,            6380,            3476,            2169,            3724,            226,            1059,            1551,            4130,            3874,            3065,            6760,            7691,            5882,            6888,            2663,            3494,            6184,            1996,            1758,            1990,            2639,            1871,            830,            5839,            4140,            5361,            4512,            4620,            4755,            4632,            5053,            5539,            4991,            459,            3862,            1933,            3381,            4320,            3252,            3109,            3604,            2401,            4511,            7185,            6223,            2809,            2162,            1134,            7066,            6530,            4474,            2401,            4671,            2667,            3001,            2274,            3762,            ],        [            781,            1756,            442,            1352,            762,            1529,            1128,            2355,            1806,            994,            1828,            2862,            0,            6231,            4273,            1369,            1372,            1617,            2636,            1048,            556,            2023,            1767,            958,            6478,            7392,            3775,            5845,            774,            1387,            4077,            1277,            1541,            899,            1299,            1074,            1463,            3732,            2033,            3254,            2835,            2513,            3078,            2907,            2946,            3432,            2884,            2403,            1755,            556,            1274,            2643,            1145,            1002,            1497,            2102,            2404,            5681,            5924,            1925,            785,            973,            4959,            4423,            2749,            294,            2564,            1327,            894,            769,            1655,            ],        [            5644,            4824,            5989,            5637,            5582,            4169,            6534,            4989,            4460,            6664,            3870,            5496,            6180,            0,            2678,            4329,            6235,            7287,            5270,            5125,            5617,            3675,            7437,            4740,            247,            5024,            2523,            525,            5810,            7057,            2482,            5092,            4854,            5855,            6705,            5937,            4710,            4560,            4446,            5696,            5376,            5963,            5619,            5448,            3958,            4260,            3896,            5037,            4590,            5999,            6821,            5184,            4553,            5384,            7044,            4220,            3558,            822,            610,            6875,            6228,            5200,            1494,            1872,            5290,            5964,            3576,            6733,            4997,            5571,            7325,            ],        [            7602,            2992,            3492,            7595,            3986,            2337,            5008,            6947,            6418,            4350,            2038,            7454,            3866,            1958,            0,            2497,            8193,            4973,            7228,            7083,            4422,            1843,            5123,            2908,            2205,            6982,            691,            1572,            3641,            3705,            650,            7050,            6812,            4277,            4837,            7895,            6668,            2728,            2614,            3864,            3544,            4131,            3787,            3616,            2126,            2428,            2064,            6995,            2758,            4422,            4225,            3352,            2721,            3552,            3692,            6178,            1726,            1408,            2568,            4857,            4651,            7158,            686,            150,            3458,            3650,            1744,            4809,            3165,            3993,            5011,            ],        [            2150,            1451,            995,            2079,            1489,            160,            3040,            3082,            2533,            1853,            459,            3589,            1369,            4862,            2904,            0,            2741,            2476,            3363,            2417,            1925,            654,            2626,            411,            5109,            9018,            2406,            4476,            1144,            2246,            2708,            2004,            2206,            1780,            2954,            2443,            2803,            2363,            2065,            3315,            2995,            3582,            3238,            3067,            1577,            2063,            1515,            3130,            2209,            1925,            1728,            2803,            224,            1608,            1951,            2829,            1035,            4312,            5472,            2784,            2154,            2342,            3590,            3054,            2909,            1153,            1195,            2926,            668,            1496,            2514,            ],        [            591,            3316,            1814,            2290,            1700,            2901,            299,            1666,            1867,            1571,            3200,            2193,            1372,            6275,            5645,            2741,            0,            1154,            1967,            1110,            816,            3395,            1304,            2330,            6522,            7453,            5147,            6650,            1928,            1178,            5449,            1841,            1603,            1255,            470,            298,            1525,            5104,            2755,            4237,            2343,            3496,            2586,            2463,            3263,            4804,            3529,            1734,            3082,            469,            1577,            2151,            2517,            2374,            1351,            2163,            3776,            7053,            5985,            640,            216,            1035,            6331,            5795,            2305,            1666,            3936,            498,            2266,            1539,            1192,            ],        [            2398,            1720,            1549,            2459,            1869,            2466,            855,            3462,            2913,            623,            2765,            3969,            1617,            7168,            5210,            2306,            1154,            0,            3743,            2665,            2173,            2960,            150,            1895,            7415,            8499,            4712,            6782,            1881,            230,            5014,            2384,            2586,            2160,            684,            1452,            3080,            4669,            1367,            2849,            2007,            2108,            2250,            2079,            1875,            4369,            2141,            3510,            1486,            1451,            629,            1815,            2082,            1939,            403,            3209,            3341,            6618,            7778,            514,            1198,            2590,            5896,            5360,            1921,            1323,            2349,            656,            1831,            1876,            38,            ],        [            1352,            3637,            2323,            2315,            2209,            3410,            2242,            371,            1879,            2875,            3709,            226,            1881,            6287,            6154,            3250,            1943,            3498,            0,            833,            1325,            3904,            3648,            2839,            6534,            7465,            5656,            6662,            2437,            3268,            5958,            1770,            1532,            1764,            2413,            1645,            604,            5613,            4518,            6565,            4286,            5901,            4529,            4406,            4827,            5313,            4765,            233,            3636,            1707,            3155,            4094,            3026,            2883,            3378,            2175,            4285,            6959,            5997,            2583,            1936,            908,            6840,            6304,            4248,            2175,            4445,            2441,            2775,            2048,            3536,            ],        [            519,            2804,            1490,            1482,            1376,            2577,            1409,            1595,            757,            2042,            2876,            2102,            1048,            5165,            5321,            2417,            1110,            2665,            1876,            0,            492,            3071,            2815,            2006,            5412,            6343,            4823,            5540,            1604,            2435,            5125,            937,            699,            931,            1580,            812,            415,            4780,            3081,            5732,            3453,            3561,            3696,            3573,            3994,            4480,            3932,            1643,            2803,            874,            2322,            3261,            2193,            2050,            2545,            1053,            3452,            5837,            4875,            1750,            1103,            75,            6007,            5471,            3415,            1342,            3612,            1608,            1942,            1215,            2703,            ],        [            225,            2312,            998,            1474,            884,            2085,            1115,            1881,            1249,            1550,            2384,            2388,            556,            5657,            4829,            1925,            816,            2173,            2162,            492,            0,            2579,            2323,            1514,            5904,            6835,            4331,            6032,            1112,            1943,            4633,            1223,            985,            439,            1286,            518,            907,            4288,            2589,            5240,            3159,            3069,            3402,            3463,            3502,            3988,            3440,            1929,            2311,            382,            1830,            2967,            1701,            1558,            2053,            1545,            2960,            6237,            5367,            1456,            611,            417,            5515,            4979,            3121,            850,            3120,            1314,            1450,            723,            2211,            ],        [            2804,            1225,            1649,            2733,            2143,            494,            3694,            3736,            3187,            2507,            195,            4243,            2023,            4208,            2250,            654,            3395,            3130,            4017,            3071,            2579,            0,            3280,            1065,            4455,            8364,            1752,            3822,            1798,            2502,            2054,            2658,            2860,            2434,            3634,            3097,            3457,            1709,            1411,            2661,            2341,            2928,            2584,            2413,            923,            1409,            861,            3784,            1555,            2579,            2382,            2149,            878,            1785,            2605,            3483,            381,            3658,            4818,            3438,            2808,            2996,            2936,            2400,            2255,            1807,            541,            3606,            1322,            2150,            3168,            ],        [            2548,            1870,            1699,            2609,            2019,            2616,            1005,            3612,            3063,            773,            2915,            4119,            1767,            7318,            5360,            2456,            1304,            150,            3893,            2815,            2323,            3110,            0,            2045,            7565,            8649,            4862,            6932,            2031,            380,            5164,            2534,            2736,            2310,            834,            1602,            3230,            4819,            1517,            2999,            2157,            2258,            2400,            2229,            2025,            4519,            2291,            3660,            1636,            1601,            779,            1965,            2232,            2089,            553,            3359,            3491,            6768,            7181,            664,            1348,            2740,            6046,            5510,            2071,            1473,            2499,            806,            1981,            2026,            112,            ],        [            1739,            1490,            584,            1668,            1078,            571,            2629,            2671,            2122,            1442,            870,            3178,            958,            5273,            3315,            411,            2330,            2065,            2952,            2006,            1514,            1065,            2215,            0,            5520,            9429,            2817,            4887,            733,            1835,            3119,            1593,            1795,            1369,            2543,            2032,            2392,            2774,            2476,            3726,            3406,            3993,            3649,            3478,            1988,            2474,            1926,            2719,            2620,            1514,            1317,            3214,            187,            1197,            1540,            2418,            1446,            4723,            5883,            2373,            1743,            1931,            4001,            3465,            3320,            742,            1606,            2515,            257,            1085,            2103,            ],        [            5397,            5341,            5742,            5390,            5335,            4686,            6287,            4742,            4213,            6417,            4387,            5249,            5933,            667,            3195,            4846,            5988,            7040,            5023,            4878,            5370,            4192,            7190,            5257,            0,            4777,            3040,            1042,            5563,            6810,            2999,            4845,            4607,            5608,            6458,            5690,            4463,            5077,            4963,            6213,            5893,            6480,            6136,            5965,            4475,            4777,            4413,            4790,            5107,            5752,            6574,            5701,            5070,            6302,            6797,            3973,            4075,            1339,            363,            6628,            5981,            4953,            2011,            2389,            5807,            5717,            4093,            6486,            5514,            5324,            7078,            ],        [            9721,            4314,            4814,            9714,            5308,            3659,            6330,            9066,            8537,            5672,            3360,            9573,            5188,            2732,            2691,            3819,            10312,            6295,            9347,            9202,            5744,            3165,            6445,            4230,            2979,            0,            2193,            2346,            4963,            5027,            2495,            9169,            8931,            5599,            6159,            10014,            8787,            4050,            3936,            5186,            4866,            5453,            5109,            4938,            3448,            3750,            3386,            9114,            4080,            5744,            5547,            4674,            4043,            4874,            5014,            8297,            3048,            2182,            3342,            6179,            5973,            9277,            1460,            1406,            4780,            4972,            3066,            6131,            4487,            5315,            6333,            ],        [            3956,            2301,            2801,            3885,            3295,            1646,            4317,            4888,            4339,            3659,            1347,            5395,            3175,            2456,            498,            1806,            4547,            4282,            5169,            4223,            3731,            1152,            4432,            2217,            2703,            8692,            0,            2070,            2950,            3014,            302,            3810,            4012,            3586,            4146,            4249,            4609,            2037,            1923,            3173,            2853,            3440,            3096,            2925,            1435,            1737,            1373,            4936,            2067,            3731,            3534,            2661,            2030,            2861,            3001,            4635,            1035,            1906,            3066,            4166,            3960,            4148,            1184,            648,            2767,            2959,            1053,            4118,            2474,            3302,            4320,            ],        [            6260,            4299,            4799,            6253,            5293,            3644,            6315,            5605,            5076,            5657,            3345,            6112,            5173,            616,            2153,            3804,            6851,            6280,            5886,            5741,            5729,            3150,            6430,            4215,            863,            5640,            1998,            0,            4948,            5012,            1957,            5708,            5470,            5584,            6144,            6553,            5326,            4035,            3921,            5171,            4851,            5438,            5094,            4923,            3433,            3735,            3371,            5653,            4065,            5729,            5532,            4659,            4028,            4859,            4999,            4836,            3033,            297,            1226,            6164,            5958,            5816,            969,            1347,            4765,            4957,            3051,            6116,            4472,            5300,            6318,            ],        [            1334,            1493,            179,            1283,            693,            1266,            1681,            2286,            1737,            1037,            1565,            2793,            553,            5968,            4010,            1106,            1925,            1660,            2567,            1601,            1109,            1760,            1810,            695,            6215,            7323,            3512,            5582,            0,            1423,            3814,            1208,            1410,            984,            2138,            1627,            2007,            3469,            1770,            4421,            3014,            4688,            3257,            3086,            2683,            3169,            2621,            2334,            1492,            1109,            1011,            2822,            882,            739,            1234,            2033,            2141,            5418,            5855,            1968,            1338,            1526,            4696,            4160,            2928,            337,            2301,            2110,            631,            700,            1698,            ],        [            2168,            2633,            1319,            2229,            1639,            2406,            879,            3232,            2683,            393,            2705,            3739,            1387,            7108,            5150,            2246,            1178,            230,            3513,            2435,            1943,            2900,            380,            1835,            7355,            8269,            4652,            6722,            1651,            0,            4954,            2154,            2356,            1930,            708,            1476,            2850,            4609,            1391,            5561,            2031,            2132,            2274,            2103,            3823,            4309,            3761,            3280,            1510,            1475,            653,            1839,            2022,            1879,            427,            2979,            3281,            6558,            6801,            538,            1222,            2360,            5836,            5300,            1945,            1093,            3441,            680,            1771,            1646,            268,            ],        [            4843,            3188,            3688,            4772,            4182,            2533,            5204,            7143,            6614,            4294,            2234,            7650,            4062,            2154,            196,            2693,            5434,            5169,            7424,            7279,            4618,            2039,            5319,            3104,            2401,            9607,            887,            1768,            3837,            3901,            0,            7246,            7008,            4473,            5033,            5136,            6864,            2924,            2810,            4060,            3740,            4327,            3983,            3812,            2322,            2624,            2260,            7191,            2954,            4618,            4114,            3548,            2917,            3748,            3888,            6374,            1922,            1604,            2764,            5053,            4847,            5035,            882,            346,            3654,            4299,            1940,            5005,            3361,            4642,            5207,            ],        [            1421,            2211,            897,            545,            490,            1984,            2311,            1310,            761,            1572,            2283,            1817,            1088,            5169,            4728,            1824,            2012,            2195,            1591,            993,            1196,            2478,            2345,            1413,            5416,            6347,            4230,            5544,            718,            1965,            4532,            0,            434,            763,            2482,            1714,            1031,            4187,            3092,            5139,            3732,            5406,            3521,            3350,            3401,            3887,            3339,            1358,            2210,            1518,            1729,            3086,            1600,            1457,            1952,            1057,            2859,            5841,            4879,            2503,            1747,            918,            5414,            4878,            3192,            872,            3019,            2510,            1349,            479,            2233,            ],        [            1659,            2449,            1135,            783,            728,            2222,            2549,            1081,            532,            1810,            2521,            1588,            1326,            4940,            4966,            2062,            2250,            2433,            1362,            764,            1434,            2716,            2583,            1651,            5187,            6118,            4468,            5315,            956,            2203,            4770,            238,            0,            1001,            2720,            1952,            802,            4425,            3330,            5377,            3516,            5644,            3759,            3588,            3639,            4125,            3577,            1129,            2448,            1756,            1967,            3324,            1838,            1695,            2190,            828,            3097,            5612,            4650,            2741,            1985,            689,            5652,            5116,            3430,            1110,            3257,            2748,            1587,            717,            2471,            ],        [            658,            2166,            852,            1035,            445,            1939,            1548,            2038,            1489,            1527,            2238,            2545,            567,            5897,            4683,            1779,            1249,            2150,            2319,            925,            433,            2433,            2300,            1368,            6144,            7075,            4185,            6272,            673,            1920,            4487,            960,            1162,            0,            1719,            951,            1340,            4142,            2443,            5094,            3233,            5361,            3476,            3305,            3356,            3842,            3294,            2086,            2165,            755,            1684,            3041,            1555,            1412,            1907,            1785,            2814,            6091,            5607,            1889,            984,            850,            5369,            4833,            3147,            827,            2974,            1747,            1304,            284,            2188,            ],        [            1061,            2846,            2027,            2760,            2170,            2944,            171,            2136,            2337,            1101,            3243,            3476,            1299,            7210,            5252,            2784,            470,            684,            3250,            1580,            1286,            3438,            834,            2373,            7457,            7923,            4754,            6824,            2398,            708,            5056,            2311,            2073,            1725,            0,            768,            1995,            4711,            2285,            4789,            1873,            3026,            2116,            1993,            2793,            4411,            3059,            3017,            2612,            767,            1107,            1681,            2560,            2417,            881,            2633,            3383,            6660,            6455,            170,            514,            1505,            5938,            5402,            1835,            1801,            3267,            28,            2309,            2009,            722,            ],        [            293,            2830,            1516,            1992,            1402,            2603,            597,            1368,            1569,            2068,            2902,            1895,            1074,            5977,            5347,            2443,            298,            1452,            1669,            812,            518,            3097,            1602,            2032,            6224,            7155,            4849,            6352,            1630,            1476,            5151,            1543,            1305,            957,            768,            0,            1227,            4806,            3053,            5758,            2641,            6025,            2884,            2761,            4020,            4506,            3958,            1436,            2829,            767,            2348,            2449,            2219,            2076,            1649,            1865,            3478,            6755,            5687,            938,            514,            737,            6033,            5497,            2603,            1368,            3638,            796,            1968,            1241,            1490,            ],        [            934,            3185,            1871,            1519,            1464,            2958,            1824,            279,            342,            2457,            3257,            876,            1463,            4750,            5702,            2798,            1525,            3080,            650,            415,            907,            3452,            3230,            2387,            4997,            5928,            5204,            5125,            1692,            2850,            5506,            974,            736,            1346,            1995,            1227,            0,            5161,            3462,            6113,            4252,            3942,            4495,            4324,            4375,            4861,            4313,            417,            3184,            1289,            2703,            4060,            2574,            2431,            2926,            638,            3833,            5422,            4460,            2165,            1518,            490,            6094,            5852,            4166,            1757,            3993,            2023,            2323,            1453,            3118,            ],        [            5259,            3151,            4104,            8444,            4598,            2496,            5167,            7796,            7267,            4257,            2197,            8303,            4478,            3486,            1528,            2656,            5850,            5585,            8077,            7932,            5034,            2002,            5735,            3067,            3733,            6655,            1030,            3100,            4253,            3864,            1332,            7899,            7661,            4889,            4996,            5552,            7517,            0,            2773,            4023,            3703,            4290,            3946,            3775,            2285,            2587,            2223,            7844,            2917,            5034,            4784,            3511,            2880,            3711,            3851,            7027,            1885,            2936,            4096,            5016,            5263,            8007,            2214,            1678,            3617,            4262,            1903,            4968,            3324,            4605,            5623,            ],        [            2669,            561,            1514,            2598,            2008,            1867,            2577,            3601,            3052,            2372,            1568,            4108,            1888,            4925,            2967,            2027,            3260,            2995,            3882,            2936,            2444,            1373,            3145,            1848,            5172,            9081,            2469,            4539,            1663,            1411,            2771,            2523,            2725,            2299,            2406,            2962,            3322,            2426,            0,            2400,            1113,            741,            1356,            1185,            508,            2126,            774,            3649,            327,            2444,            2194,            921,            2251,            1121,            1398,            3348,            1098,            4375,            5535,            2426,            2673,            2861,            3653,            3117,            1027,            1672,            982,            2378,            2047,            2015,            3033,            ],        [            3120,            1012,            1965,            3049,            2459,            2297,            3028,            4052,            3503,            2118,            1998,            4559,            2339,            5355,            3397,            2457,            3711,            3446,            4333,            3387,            2895,            1803,            3596,            2868,            5602,            9511,            2899,            4969,            2114,            1725,            3201,            2974,            3176,            2750,            2857,            3413,            3773,            2856,            634,            0,            1564,            267,            1807,            1636,            938,            2556,            1204,            4100,            778,            2895,            2645,            1372,            2681,            1572,            1712,            3799,            1528,            4805,            5965,            2877,            3124,            3312,            4083,            3547,            1478,            2123,            1412,            2829,            2498,            2466,            3484,            ],        [            3305,            1924,            2877,            3961,            3371,            3230,            2415,            4964,            4415,            2557,            2931,            5471,            3251,            6288,            4330,            3390,            2714,            2778,            5245,            4299,            3807,            2736,            2928,            3211,            6535,            10444,            3832,            5902,            3026,            2164,            4134,            3886,            4088,            3662,            2244,            3012,            4685,            3789,            1363,            2916,            0,            3183,            243,            1071,            1871,            3489,            2137,            5012,            1690,            3011,            2377,            563,            3614,            2484,            2151,            4711,            2461,            5738,            6898,            2264,            2758,            4224,            5016,            4480,            913,            3035,            2345,            2216,            3410,            3378,            2816,            ],        [            2853,            745,            1698,            2782,            2192,            2030,            2761,            3785,            3236,            1851,            1731,            4292,            2072,            5088,            3130,            2190,            3444,            3179,            4066,            3120,            2628,            1536,            3329,            2601,            5335,            9244,            2632,            4702,            1847,            1458,            2934,            2707,            2909,            2483,            2590,            3146,            3506,            2589,            367,            2563,            1297,            0,            1540,            1369,            671,            2289,            937,            3833,            511,            2628,            2378,            1105,            2414,            1305,            1445,            3532,            1261,            4538,            5698,            2610,            2857,            3045,            3816,            3280,            1211,            1856,            1145,            2562,            2231,            2199,            3217,            ],        [            3314,            1933,            2886,            3970,            3380,            3239,            2424,            4973,            4424,            2566,            2940,            5480,            3260,            6297,            4339,            3399,            2723,            2787,            5254,            4308,            3816,            2745,            2937,            3220,            6544,            10453,            3841,            5911,            3035,            2173,            4143,            3895,            4097,            3671,            2253,            3021,            4694,            3798,            1372,            2673,            1286,            2940,            0,            1080,            1880,            3498,            2146,            5021,            1699,            3020,            2386,            572,            3623,            2493,            2160,            4720,            2470,            5747,            6907,            2273,            2767,            4233,            5025,            4489,            922,            3044,            2354,            2225,            3419,            3387,            2825,            ],        [            3725,            1617,            2570,            3654,            3064,            2923,            2156,            4657,            4108,            2250,            2624,            5164,            2944,            5981,            4023,            3083,            2455,            2519,            4938,            3992,            3500,            2429,            2669,            2904,            6228,            10137,            3525,            5595,            2719,            1857,            3827,            3579,            3781,            3355,            1985,            2753,            4378,            3482,            1056,            3608,            692,            1797,            935,            0,            1564,            3182,            1830,            4705,            1383,            2752,            2070,            500,            3307,            2177,            1844,            4404,            2154,            5431,            6591,            2005,            2499,            3917,            4709,            4173,            150,            2728,            2038,            1957,            3103,            3071,            2557,            ],        [            3669,            2014,            2514,            3598,            3008,            1359,            4030,            4601,            4052,            3372,            1060,            5108,            2888,            4417,            2459,            1519,            4260,            3995,            4882,            3936,            3444,            865,            4145,            1930,            4664,            8573,            1961,            4031,            2663,            2727,            2263,            3523,            3725,            3299,            3859,            3962,            4322,            1918,            1636,            2594,            2566,            2861,            2809,            2638,            0,            1618,            266,            4649,            1780,            3444,            3247,            2374,            1743,            2574,            2714,            4348,            590,            3867,            5027,            3879,            3673,            3861,            3145,            2609,            2480,            2672,            474,            3831,            2187,            3015,            4033,            ],        [            5106,            3451,            3951,            5035,            4445,            2796,            5467,            8096,            7567,            4809,            2497,            8603,            4325,            3786,            1828,            2956,            5697,            5432,            8377,            8232,            4881,            2302,            5582,            3367,            4033,            6955,            1330,            3400,            4100,            4164,            1632,            8199,            7961,            4736,            5296,            5399,            7817,            300,            3073,            4323,            4003,            4590,            4246,            4075,            2585,            0,            2523,            8144,            3217,            4881,            4684,            3811,            3180,            4011,            4151,            7327,            2185,            3236,            4396,            5316,            5110,            8307,            2514,            1978,            3917,            4109,            2203,            5268,            3624,            4452,            5470,            ],        [            3403,            1748,            2248,            3332,            2742,            1093,            3764,            4335,            3786,            3106,            794,            4842,            2622,            4151,            2193,            1253,            3994,            3729,            4748,            3670,            3178,            599,            3879,            1664,            4398,            8307,            1695,            3765,            2886,            2461,            1997,            3257,            3591,            3165,            3593,            3696,            4085,            1652,            1370,            2328,            2300,            2595,            2543,            2372,            882,            1352,            0,            4515,            1514,            3178,            2981,            2108,            1477,            2308,            2448,            4082,            324,            3601,            4761,            3613,            3407,            3595,            2879,            2343,            2214,            2406,            208,            3565,            1921,            2749,            3767,            ],        [            1119,            3404,            2090,            2082,            1976,            3177,            2009,            138,            1646,            2642,            3476,            459,            1648,            6054,            5921,            3017,            1710,            3265,            233,            600,            1092,            3671,            3415,            2606,            6301,            7232,            5423,            6429,            2204,            3035,            5725,            1537,            1299,            1531,            2180,            1412,            371,            5380,            3681,            6332,            4053,            4161,            4296,            4543,            4594,            5080,            4532,            0,            3403,            1474,            2922,            3861,            2793,            2650,            3145,            1942,            4052,            6726,            5764,            2350,            1703,            675,            6607,            6071,            4015,            1942,            4212,            2208,            2542,            1815,            3303,            ],        [            2342,            234,            1187,            2271,            1681,            1173,            3232,            3274,            2725,            2045,            912,            3781,            1561,            5315,            3357,            1333,            2933,            2668,            3555,            2609,            2117,            1107,            2818,            1521,            5562,            9471,            2859,            4929,            1336,            2279,            3161,            2196,            2398,            1972,            3146,            2635,            2995,            2816,            531,            1752,            1321,            1011,            1564,            1393,            778,            2516,            1968,            3322,            0,            2117,            1867,            1129,            1708,            794,            2090,            3021,            1488,            4765,            5925,            2976,            2346,            2534,            4043,            3507,            1235,            1345,            1648,            3118,            1720,            1688,            2706,            ],        [            607,            2312,            998,            1760,            1170,            2085,            596,            2263,            1631,            1550,            2384,            2770,            556,            6039,            4829,            1925,            469,            1451,            2544,            874,            382,            2579,            1601,            1514,            6286,            7217,            4331,            6414,            1398,            1475,            4633,            1605,            1367,            725,            767,            767,            1289,            4288,            3052,            5240,            2640,            3069,            2883,            2760,            3502,            3988,            3440,            2311,            2311,            0,            1830,            2448,            1701,            1558,            2053,            1927,            2960,            6237,            5749,            937,            253,            799,            5515,            4979,            2602,            850,            3120,            795,            1450,            1009,            1489,            ],        [            2075,            2064,            920,            2004,            1414,            1837,            2965,            3007,            2458,            805,            2136,            3514,            1294,            6539,            4581,            1677,            2666,            844,            3288,            2342,            1850,            2331,            994,            1266,            6786,            8044,            4083,            6153,            1069,            412,            4385,            1929,            2131,            1705,            1322,            2368,            2728,            4040,            1363,            2845,            2003,            2104,            2246,            2075,            3254,            3740,            3192,            3055,            1482,            1850,            0,            1811,            1453,            1310,            223,            2754,            2712,            5989,            6576,            1152,            2079,            2267,            5267,            4731,            1917,            909,            2872,            1294,            1202,            1421,            882,            ],        [            2742,            1361,            2314,            3398,            2808,            2667,            1852,            4401,            3852,            1994,            2368,            4908,            2688,            5725,            3767,            2827,            2151,            2215,            4682,            3261,            2967,            2173,            2365,            2648,            5972,            9881,            3269,            5339,            2463,            1601,            3571,            3323,            3525,            3099,            1681,            2449,            4122,            3226,            800,            3108,            192,            1541,            435,            508,            1308,            2926,            1574,            4449,            1127,            2448,            1814,            0,            3051,            1921,            1588,            4148,            1898,            5175,            6335,            1701,            2195,            3186,            4453,            3917,            350,            2472,            1782,            1653,            2847,            2815,            2253,            ],        [            1926,            1675,            771,            1987,            1397,            384,            2816,            2990,            2441,            1629,            683,            3497,            1145,            5086,            3128,            224,            2517,            2252,            3271,            2193,            1701,            878,            2402,            187,            5333,            9242,            2630,            4700,            1409,            2022,            2932,            1912,            2114,            1688,            2730,            2219,            2608,            2587,            2289,            3539,            3219,            2875,            3462,            3291,            1801,            2287,            1739,            3038,            2433,            1701,            1504,            3027,            0,            1384,            1727,            2737,            1259,            4536,            5696,            2560,            1930,            2118,            3814,            3278,            3133,            929,            1419,            2702,            444,            1404,            2290,            ],        [            1548,            1037,            393,            1477,            887,            1310,            2438,            2480,            1931,            1251,            1609,            2987,            767,            6012,            4054,            1150,            2139,            1874,            2761,            1815,            1323,            1804,            2024,            739,            6259,            7517,            3556,            5626,            542,            1485,            3858,            1402,            1604,            1178,            2352,            1841,            2201,            3513,            1314,            2535,            2104,            1794,            2347,            2176,            1561,            3213,            1827,            2528,            1036,            1323,            1073,            1912,            926,            0,            1296,            2227,            2185,            5462,            6049,            2182,            1552,            1740,            4740,            4204,            2018,            551,            2345,            2324,            675,            894,            1912,            ],        [            2301,            2290,            1146,            2230,            1640,            2063,            1270,            3233,            2684,            582,            2362,            3740,            1520,            6765,            4807,            1903,            2892,            621,            3514,            2568,            2076,            2557,            771,            1492,            7012,            8270,            4309,            6379,            1295,            189,            4611,            2155,            2357,            1931,            1099,            2594,            2892,            4266,            1140,            2622,            1780,            1881,            2023,            1852,            1648,            3966,            1914,            3281,            1259,            1985,            226,            1588,            1679,            1536,            0,            2980,            2938,            6215,            7375,            929,            2214,            2402,            5493,            4957,            1694,            1135,            3098,            1071,            1428,            1647,            659,            ],        [            1870,            3529,            2215,            1863,            1808,            3302,            2760,            1215,            686,            2890,            3601,            1722,            2406,            4112,            6046,            3142,            2461,            3513,            1496,            1351,            1843,            3796,            3663,            2731,            4359,            5290,            5548,            4487,            2036,            3283,            5850,            1318,            1080,            2081,            2931,            2163,            936,            5505,            4410,            6457,            6137,            4286,            4839,            5122,            4719,            5205,            4657,            1263,            3528,            2225,            3047,            4858,            2918,            2775,            3270,            0,            4177,            4784,            3822,            3101,            2454,            1426,            6769,            6715,            4964,            2190,            4337,            2959,            2667,            1797,            3551,            ],        [            3718,            1610,            2563,            3647,            3057,            955,            3626,            4650,            4101,            3421,            656,            5157,            2937,            3827,            1869,            1115,            4309,            4044,            4931,            3985,            3493,            461,            4194,            1526,            4074,            7983,            1371,            3441,            2712,            2323,            1673,            3572,            3774,            3348,            3455,            4011,            4400,            1328,            1232,            2482,            2162,            1818,            2405,            2234,            744,            1028,            682,            4504,            1376,            3493,            2843,            1970,            1339,            2170,            2310,            4397,            0,            3277,            4437,            3475,            3396,            3910,            2555,            2019,            2076,            2721,            362,            3427,            1783,            3064,            4082,            ],        [            6194,            4251,            6539,            6187,            6132,            3596,            7084,            5539,            5010,            7214,            3297,            6046,            5252,            550,            2105,            3756,            6785,            7837,            5820,            5675,            6167,            3102,            7987,            4167,            797,            5574,            1950,            164,            6360,            4964,            1909,            5642,            5404,            6405,            7255,            6487,            5260,            3987,            3873,            5123,            4803,            4459,            5046,            4875,            3385,            3687,            3323,            5587,            4017,            6549,            5484,            4611,            3980,            4811,            5654,            4770,            2985,            0,            1160,            6116,            6778,            5750,            921,            1299,            4717,            6514,            3003,            6068,            4424,            6121,            7875,            ],        [            5034,            5823,            5379,            5027,            4972,            5168,            5924,            4379,            3850,            6054,            4869,            4886,            5570,            1149,            3677,            5328,            5625,            6677,            4660,            4515,            5007,            4674,            6827,            5895,            1229,            4414,            3522,            1524,            5200,            6447,            3481,            4482,            4244,            5245,            6095,            5327,            4100,            5559,            5445,            6695,            6375,            6031,            6618,            6447,            4957,            5259,            4895,            4427,            5589,            5389,            6211,            6183,            6082,            5939,            6434,            3610,            4557,            1821,            0,            6265,            5618,            4590,            2493,            2871,            6289,            5354,            4575,            6123,            5756,            4961,            6715,            ],        [            1231,            2028,            1857,            2767,            2177,            2774,            341,            2306,            2507,            931,            3073,            3646,            1925,            7476,            5518,            2614,            640,            514,            3420,            1750,            1456,            3268,            664,            2203,            7723,            8093,            5020,            7090,            2189,            538,            5322,            2481,            2243,            1895,            170,            938,            2165,            4977,            1675,            3157,            1893,            2416,            2136,            2387,            2183,            4677,            2449,            3187,            1794,            937,            937,            1701,            2390,            2247,            711,            2803,            3649,            6926,            6625,            0,            684,            1675,            6204,            5668,            2229,            1631,            3809,            142,            2064,            2179,            552,            ],        [            807,            2541,            1227,            1989,            1399,            2314,            343,            2492,            1860,            1779,            2613,            2999,            785,            6268,            5058,            2154,            216,            1198,            2773,            1103,            611,            2808,            1348,            1743,            6515,            7446,            4560,            6643,            1627,            1222,            4862,            1834,            1596,            954,            514,            514,            1518,            4517,            2799,            4281,            2387,            3540,            2630,            2507,            3731,            4217,            3669,            2540,            3126,            253,            2059,            2195,            1930,            1787,            1395,            2156,            3189,            6466,            5978,            684,            0,            1028,            5744,            5208,            2349,            1079,            3349,            542,            1604,            1238,            1236,            ],        [            444,            3337,            1415,            1407,            1301,            2502,            1334,            1520,            832,            1967,            2801,            2027,            973,            5240,            5246,            2342,            1035,            2590,            1801,            75,            417,            2996,            2740,            1931,            5487,            6418,            4748,            5615,            1529,            2360,            5050,            862,            624,            856,            1505,            737,            490,            4705,            3610,            5657,            3378,            4993,            3621,            3498,            3919,            4405,            3857,            1568,            4551,            799,            2247,            3186,            2118,            1975,            2470,            1128,            3377,            5912,            4950,            1675,            1028,            0,            5932,            5396,            3340,            1267,            3537,            1533,            1792,            1140,            2628,            ],        [            5334,            3552,            4052,            6909,            4805,            2897,            5568,            6261,            5732,            4831,            2598,            6768,            4553,            1272,            1406,            3057,            5925,            5454,            6542,            6397,            5109,            2403,            5604,            3468,            1519,            6296,            1251,            886,            4817,            5144,            1210,            6364,            6126,            5096,            5397,            5627,            5982,            3288,            3174,            4424,            4104,            3760,            4347,            4176,            2686,            2988,            2624,            6309,            3318,            5109,            4732,            3912,            3281,            4112,            4955,            5492,            2286,            722,            1882,            5417,            5338,            6472,            0,            600,            4018,            4259,            2304,            5369,            3725,            4812,            5492,            ],        [            4734,            2952,            3452,            4795,            4205,            2297,            4968,            5798,            5249,            4231,            1998,            6305,            3953,            1808,            806,            2457,            5325,            4854,            6079,            5001,            4509,            1803,            5004,            2868,            2055,            6832,            651,            1422,            4217,            4544,            610,            4720,            4922,            4496,            4797,            5027,            5416,            2688,            2574,            3824,            3504,            3160,            3747,            3576,            2086,            2388,            2024,            5846,            2718,            4509,            4132,            3312,            2681,            3512,            4355,            5545,            1686,            1258,            2418,            4817,            4738,            4926,            536,            0,            3418,            3659,            1704,            4769,            3125,            4212,            4892,            ],        [            3575,            1467,            2420,            3504,            2914,            2773,            2006,            4507,            3958,            2100,            2474,            5014,            2794,            5831,            3873,            2933,            2305,            2369,            4788,            3842,            3350,            2279,            2519,            2754,            6078,            9987,            3375,            5445,            2569,            1707,            3677,            3429,            3631,            3205,            1835,            2603,            4228,            3332,            906,            2388,            542,            1647,            785,            158,            1414,            3032,            1680,            4555,            1233,            3350,            1920,            350,            3157,            2027,            1694,            4254,            2004,            5281,            6441,            1855,            2349,            3767,            4559,            4023,            0,            2578,            1888,            1807,            2627,            2921,            2407,            ],        [            1075,            2148,            226,            1136,            546,            1313,            1965,            2139,            1590,            700,            1612,            2646,            294,            6015,            4057,            1153,            1666,            1323,            2420,            1342,            850,            1807,            1473,            742,            6262,            7176,            3559,            5629,            558,            1093,            3861,            1061,            1263,            837,            1801,            1368,            1757,            3516,            2421,            4468,            3061,            3804,            3304,            3133,            2730,            3216,            2668,            2187,            3362,            850,            1058,            2869,            929,            786,            1281,            1886,            2188,            5465,            5708,            1631,            1079,            1267,            4743,            4207,            2975,            0,            2348,            1773,            603,            553,            1361,            ],        [            3677,            1569,            2522,            3606,            3016,            1151,            3585,            4609,            4060,            3085,            852,            5116,            2896,            4209,            2251,            1311,            4268,            4003,            4890,            3944,            3452,            657,            4153,            1722,            4456,            8365,            1753,            3823,            2671,            2282,            2055,            3531,            3733,            3307,            3414,            3970,            4330,            1710,            1191,            2120,            2121,            2387,            2364,            2193,            995,            1410,            641,            4657,            1335,            3452,            2986,            1929,            1535,            2129,            3425,            4356,            382,            3659,            4819,            3434,            3681,            3869,            2937,            2401,            2035,            2680,            0,            3386,            1979,            3023,            4041,            ],        [            1089,            2818,            1999,            2788,            2198,            2916,            199,            2164,            2365,            1073,            3215,            3504,            1327,            6773,            5224,            2756,            498,            656,            3278,            1608,            1314,            3410,            806,            2345,            7020,            7951,            4726,            7148,            2426,            680,            5028,            2339,            2101,            1753,            28,            796,            2023,            4683,            2257,            3739,            1845,            2998,            2088,            1965,            2765,            4383,            3031,            3045,            2584,            795,            1079,            1653,            2532,            2389,            853,            2661,            3355,            6632,            6483,            142,            542,            1533,            5910,            5374,            1807,            1773,            3239,            0,            2206,            2037,            694,            ],        [            1609,            1806,            327,            1411,            821,            971,            2499,            2414,            1865,            1106,            1270,            2921,            828,            5673,            3715,            811,            2200,            1729,            2695,            1876,            1384,            1465,            1879,            400,            5920,            7451,            3217,            5287,            476,            1419,            3519,            1336,            1538,            1112,            2207,            1902,            2135,            3174,            2370,            4126,            3806,            3462,            4049,            3878,            2388,            2874,            2326,            2462,            3020,            1384,            1007,            3614,            587,            735,            1230,            2161,            1846,            5123,            6283,            2037,            1613,            1801,            4401,            3865,            3720,            534,            2006,            2179,            0,            828,            1767,            ],        [            942,            1882,            568,            751,            161,            1655,            1832,            1754,            1205,            1243,            1954,            2261,            759,            5613,            4399,            1495,            1533,            1866,            2035,            1209,            717,            2149,            2016,            1084,            5860,            6791,            3901,            5988,            389,            1636,            4203,            676,            878,            284,            2003,            1235,            1475,            3858,            2763,            4810,            3403,            4146,            3646,            3475,            3072,            3558,            3010,            1802,            3704,            1039,            1400,            3211,            1271,            1128,            1623,            1501,            2530,            5807,            5323,            2173,            1268,            1134,            5085,            4549,            3317,            543,            2690,            2031,            945,            0,            1904,            ],        [            2436,            1758,            1587,            2497,            1907,            2504,            893,            3500,            2951,            661,            2803,            4007,            1655,            7206,            5248,            2344,            1192,            38,            3781,            2703,            2211,            2998,            112,            1933,            7453,            8537,            4750,            6820,            1919,            268,            5052,            2422,            2624,            2198,            722,            1490,            3118,            4707,            1405,            2887,            2045,            2146,            2288,            2117,            1913,            4407,            2179,            3548,            1524,            1489,            667,            1853,            2120,            1977,            441,            3247,            3379,            6656,            7069,            552,            1236,            2628,            5934,            5398,            1959,            1361,            3539,            694,            1794,            1914,            0,            ],        ], 'starts': [0, 15, 58, 69], 'ends': [0, 15, 58, 69]},    'NW-37': {'distance_matrix': [        [            0,            3205,            2485,            3067,            5692,            3904,            2686,            7081,            3622,            6255,            735,            944,            160,            343,            7415,            6935,            3262,            4837,            1809,            7777,            4362,            4983,            4617,            2303,            4149,            5337,            ],        [            2342,            0,            720,            138,            2804,            1016,            519,            4193,            734,            3367,            2996,            2384,            2502,            2685,            4527,            4047,            374,            1949,            1396,            4889,            1474,            2095,            1729,            902,            1261,            2449,            ],        [            1622,            720,            0,            582,            3207,            1419,            201,            4596,            1137,            3770,            2276,            1664,            1782,            1965,            4930,            4450,            777,            2352,            676,            5292,            1877,            2498,            2132,            182,            1664,            2852,            ],        [            2204,            138,            582,            0,            2942,            1154,            381,            4331,            872,            3505,            2858,            2246,            2364,            2547,            4665,            4185,            512,            2087,            1258,            5027,            1612,            2233,            1867,            764,            1399,            2587,            ],        [            4829,            2804,            3207,            2942,            0,            1788,            3368,            1389,            2070,            563,            5483,            4871,            4989,            5172,            1723,            1243,            2430,            855,            3883,            2085,            1330,            709,            1075,            3389,            1543,            355,            ],        [            3041,            1016,            1419,            1154,            1788,            0,            1580,            3177,            282,            2351,            3695,            3083,            3201,            3384,            3511,            3031,            642,            933,            2095,            3873,            458,            1079,            713,            1601,            245,            1433,            ],        [            1823,            519,            201,            381,            3368,            1580,            0,            4757,            1298,            3931,            2477,            1865,            1983,            2166,            5091,            4611,            938,            2513,            877,            5453,            2038,            2659,            2293,            383,            1825,            3013,            ],        [            6218,            4193,            4596,            4331,            1389,            3177,            4757,            0,            3459,            826,            6872,            6260,            6378,            6561,            334,            146,            3819,            2244,            5272,            696,            2719,            2098,            2464,            4778,            2932,            1744,            ],        [            2759,            734,            1137,            872,            2070,            282,            1298,            3459,            0,            2633,            3413,            2801,            2919,            3102,            3793,            3313,            360,            1215,            1813,            4155,            740,            1361,            995,            1319,            527,            1715,            ],        [            5392,            3367,            3770,            3505,            563,            2351,            3931,            826,            2633,            0,            6046,            5434,            5552,            5735,            1160,            680,            2993,            1418,            4446,            1522,            1893,            1272,            1638,            3952,            2106,            918,            ],        [            673,            2941,            2221,            2803,            5428,            3640,            2422,            6817,            3358,            5991,            0,            680,            833,            344,            7151,            6671,            2998,            4573,            1545,            7513,            4098,            4719,            4353,            2039,            3885,            5073,            ],        [            531,            2799,            2079,            2661,            5286,            3498,            2280,            6675,            3216,            5849,            612,            0,            691,            874,            7009,            6529,            2856,            4431,            1403,            7371,            3956,            4577,            4211,            1897,            3743,            4931,            ],        [            1163,            3045,            2325,            2907,            5532,            3744,            2526,            6921,            3462,            6095,            575,            784,            0,            183,            7255,            6775,            3102,            4677,            1649,            7617,            4202,            4823,            4457,            2143,            3989,            5177,            ],        [            980,            2862,            2142,            2724,            5349,            3561,            2343,            6738,            3279,            5912,            392,            601,            1140,            0,            7072,            6592,            2919,            4494,            1466,            7434,            4019,            4640,            4274,            1960,            3806,            4994,            ],        [            6552,            4527,            4930,            4665,            1723,            3511,            5091,            334,            3793,            1160,            7206,            6594,            6712,            6895,            0,            480,            4153,            2578,            5606,            362,            3053,            2432,            2798,            5112,            3266,            2078,            ],        [            6072,            4047,            4450,            4185,            1243,            3031,            4611,            146,            3313,            680,            6726,            6114,            6232,            6415,            480,            0,            3673,            2098,            5126,            842,            2573,            1952,            2318,            4632,            2786,            1598,            ],        [            2399,            374,            777,            512,            2430,            642,            938,            3819,            360,            2993,            3053,            2441,            2559,            2742,            4153,            3673,            0,            1575,            1453,            4515,            1100,            1721,            1355,            959,            887,            2075,            ],        [            3974,            1949,            2352,            2087,            855,            933,            2513,            2244,            1215,            1418,            4628,            4016,            4134,            4317,            2578,            2098,            1575,            0,            3028,            2940,            475,            146,            220,            2534,            688,            500,            ],        [            946,            1396,            676,            1258,            3883,            2095,            877,            5272,            1813,            4446,            1600,            988,            1106,            1289,            5606,            5126,            1453,            3028,            0,            5968,            2553,            3174,            2808,            494,            2340,            3528,            ],        [            6914,            4889,            5292,            5027,            2085,            3873,            5453,            696,            4155,            1522,            7568,            6956,            7074,            7257,            362,            842,            4515,            2940,            5968,            0,            3415,            2794,            3160,            5474,            3628,            2440,            ],        [            3499,            1474,            1877,            1612,            1330,            458,            2038,            2719,            740,            1893,            4153,            3541,            3659,            3842,            3053,            2573,            1100,            475,            2553,            3415,            0,            621,            255,            2059,            213,            975,            ],        [            4120,            2095,            2498,            2233,            709,            1079,            2659,            2098,            1361,            1272,            4774,            4162,            4280,            4463,            2432,            1952,            1721,            146,            3174,            2794,            621,            0,            366,            2680,            834,            354,            ],        [            3754,            1729,            2132,            1867,            1075,            713,            2293,            2464,            995,            1638,            4408,            3796,            3914,            4097,            2798,            2318,            1355,            220,            2808,            3160,            255,            366,            0,            2314,            468,            720,            ],        [            1440,            902,            182,            764,            3389,            1601,            383,            4778,            1319,            3952,            2094,            1482,            1600,            1783,            5112,            4632,            959,            2534,            494,            5474,            2059,            2680,            2314,            0,            1846,            3034,            ],        [            3286,            1261,            1664,            1399,            1543,            245,            1825,            2932,            527,            2106,            3940,            3328,            3446,            3629,            3266,            2786,            887,            688,            2340,            3628,            213,            834,            468,            1846,            0,            1188,            ],        [            4474,            2449,            2852,            2587,            355,            1433,            3013,            1744,            1715,            918,            5128,            4516,            4634,            4817,            2078,            1598,            2075,            500,            3528,            2440,            975,            354,            720,            3034,            1188,            0,            ],        ], 'starts': [0, 12], 'ends': [0, 12]},    }

In [ ]:
filtered_df = df[(df['styleUrl'] == 0)&(df['ward']=='NW-7')] ##change here 
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def find_possible_paths(ward, ward_data):
    """Finds all possible paths for buses in a given ward."""
    # Extract data for the specified ward
    data = ward_data[ward]
    distance_matrix = data["distance_matrix"]
    num_vehicles = len(data["starts"])  # Assuming number of vehicles equals number of starts
    starts = data["starts"]
    ends = data["ends"]

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(distance_matrix), num_vehicles, starts, ends)

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        1000000,  # vehicle maximum travel distance, assuming a higher limit
        True,  # start cumul to zero
        dimension_name,
    )

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    # Collecting the paths
    paths = []
    if solution:
        for vehicle_id in range(num_vehicles):
            index = routing.Start(vehicle_id)
            vehicle_path = []
            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                vehicle_path.append(node_index)
                index = solution.Value(routing.NextVar(index))
            vehicle_path.append(manager.IndexToNode(index))  # Add the end node
            paths.append(vehicle_path)
    return paths


paths = find_possible_paths("NW-7", ward_data) # Change here

In [ ]:
print(paths)

In [ ]:
filtered_df = df[(df['styleUrl'] == 0)&(df['ward']=='NW-15')] ##change here only
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
import requests
import folium
import pandas as pd

# Assuming 'filtered_df' is defined earlier in the code
filtered_df.reset_index(drop=True, inplace=True)

# Initialize a map centered around Mumbai
mumbai_map = folium.Map(location=[19.0760, 72.8777], zoom_start=12)

# Define a single path as output from the VRP function
path = [item for sublist in paths for item in sublist]
# Add markers for each location
for index, row in filtered_df.iterrows():
    color = 'red'  # Default color
    if index in path:
        color = 'blue'  # Color for locations in the path
        
        # Create the marker
        marker = folium.Marker(
            location=[row['latitude'], row['longitude']],
            icon=folium.Icon(color=color),
            popup=row['name']
        )
        
        # Add the marker to the map
        marker.add_to(mumbai_map)

# Define the points for the OSRM API for the path
points = [filtered_df.loc[i, ['longitude', 'latitude']].tolist() for i in path]

# Convert the points to a string with the format required by the OSRM API
points_str = ';'.join([f'{point[0]},{point[1]}' for point in points])

# Send a request to the OSRM API
response = requests.get(f'http://router.project-osrm.org/route/v1/driving/{points_str}?overview=full&geometries=geojson')

# Parse the JSON response
data = response.json()

# Get the coordinates of the route for the path
route = data['routes'][0]['geometry']['coordinates']

# Convert the coordinates from (longitude, latitude) to (latitude, longitude)
route = [(lat, lon) for lon, lat in route]

# Draw the path with the specified color
folium.PolyLine(route, color='blue', weight=2.5, opacity=1).add_to(mumbai_map)

# Display the map
mumbai_map

In [ ]:
import requests
import folium
import pandas as pd

# Assuming 'filtered_df' is defined earlier in the code
filtered_df.reset_index(drop=True, inplace=True)

# Initialize a map centered around Mumbai
mumbai_map = folium.Map(location=[19.0760, 72.8777], zoom_start=12)

# Define a single path as output from the VRP function
path = [item for sublist in paths for item in sublist]
# Add markers for each location
for index, row in filtered_df.iterrows():
    color = 'red'  # Default color
    if index in path:
        color = 'blue'  # Color for locations in the path
        
        # Create the marker
        marker = folium.Marker(
            location=[row['latitude'], row['longitude']],
            icon=folium.Icon(color=color),
            popup=row['name']
        )
        
        # Add the marker to the map
        marker.add_to(mumbai_map)

# Define the points for the OSRM API for the path
points = [filtered_df.loc[i, ['longitude', 'latitude']].tolist() for i in path]

# Convert the points to a string with the format required by the OSRM API
points_str = ';'.join([f'{point[0]},{point[1]}' for point in points])

# Send a request to the OSRM API
response = requests.get(f'http://router.project-osrm.org/route/v1/driving/{points_str}?overview=full&geometries=geojson')

# Parse the JSON response
data = response.json()

# Get the coordinates of the route for the path
route = data['routes'][0]['geometry']['coordinates']

# Convert the coordinates from (longitude, latitude) to (latitude, longitude)
route = [(lat, lon) for lon, lat in route]

# Draw the path with the specified color
folium.PolyLine(route, color='blue', weight=2.5, opacity=1).add_to(mumbai_map)

# Display the map
mumbai_map

In [ ]:
import requests
import folium
import pandas as pd

# Assuming 'filtered_df' is defined earlier in the code
filtered_df.reset_index(drop=True, inplace=True)

# Initialize a map centered around Mumbai
mumbai_map = folium.Map(location=[19.0760, 72.8777], zoom_start=12)

# Define colors for different paths
path_colors = ['blue', 'green', 'purple', 'orange', 'darkred']
bus_stop_icon_color = 'lightgray'  # Color for bus stops

# Assuming 'paths' is a 2D list where each sublist represents a path
# Example: paths = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

for path_index, path in enumerate(paths):
    # Ensure we don't exceed the number of defined colors
    color_index = path_index % len(path_colors)
    path_color = path_colors[color_index]

    # Define the points for the OSRM API for the current path
    points = [filtered_df.loc[i, ['longitude', 'latitude']].tolist() for i in path]

    # Convert the points to a string with the format required by the OSRM API
    points_str = ';'.join([f'{point[0]},{point[1]}' for point in points])

    # Send a request to the OSRM API
    response = requests.get(f'http://router.project-osrm.org/route/v1/driving/{points_str}?overview=full&geometries=geojson')

    # Parse the JSON response
    data = response.json()

    # Get the coordinates of the route for the path
    route = data['routes'][0]['geometry']['coordinates']

    # Convert the coordinates from (longitude, latitude) to (latitude, longitude)
    route = [(lat, lon) for lon, lat in route]

    # Draw the path with the specified color
    folium.PolyLine(route, color=path_color, weight=2.5, opacity=1).add_to(mumbai_map)

# Add markers for each bus stop location
for index, row in filtered_df.iterrows():
    # Create the marker with a different icon color
    marker = folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=folium.Icon(color=bus_stop_icon_color),
        popup=row['name']
    )
    
    # Add the marker to the map
    marker.add_to(mumbai_map)

# Display the map
mumbai_map

# Based on Population

In [ ]:
df

In [ ]:
population = {
  "high": [14, 19, 5, 22, 6, 26, 23, 3, 20, 32, 11, 8, 12, 31, 33, 35],
  "mid" : [18, 28, 7, 16, 30, 13, 17, 1, 5, 25, 2, 10, 24, 4, 21, 27, 29, 34, 36],
  "low" : [15, 14, 24, 9, 37]
}

In [ ]:
df['ward'].value_counts()

In [ ]:
import random

# Population dictionary
population = {
  "high": [14, 19, 5, 22, 6, 26, 23, 3, 20, 32, 11, 8, 12, 31, 33, 35],
  "mid" : [18, 28, 7, 16, 30, 13, 17, 1, 5, 25, 2, 10, 24, 4, 21, 27, 29, 34, 36],
  "low" : [15, 14, 24, 9, 37]
}

# Ward counts
ward_counts = {
    4: 112, 8: 112, 13: 100, 35: 100, 30: 94, 10: 94, 6: 85, 20: 84, 2: 83, 28: 82,
    29: 80, 33: 79, 14: 78, 12: 75, 21: 75, 27: 75, 31: 74, 19: 73, 22: 73, 36: 71,
    26: 71, 32: 68, 11: 68, 7: 65, 16: 60, 23: 59, 15: 56, 1: 55, 18: 44, 25: 41,
    34: 40, 9: 35, 24: 32, 5: 31, 3: 29, 17: 26, 37: 26
}

# Function to assign random numbers
# Function to assign numbers using uniform distribution
def assign_uniform_numbers(population, ward_counts):
    uniform_assignments = {}
    for category in population:
        uniform_assignments[category] = {}
        num_assignments = 5 if category == "high" else 4 if category == "mid" else 3
        for ward in population[category]:
            count = ward_counts.get(ward, 0)
            if count > 0:
                # Calculate the step size for uniform distribution
                step = max(1, count // (num_assignments + 1))
                # Generate uniformly distributed numbers
                numbers = [step * i for i in range(1, num_assignments + 1)]
                # Adjust if the generated number exceeds the count
                numbers = [min(n, count - 1) for n in numbers]
            else:
                numbers = []
            uniform_assignments[category][ward] = numbers
    return uniform_assignments

# Assign uniform numbers
uniform_numbers = assign_uniform_numbers(population, ward_counts)

# Print the result
for category, wards in uniform_numbers.items():
    print(f"{category}:")
    for ward, numbers in wards.items():
        print(f"  Ward {ward}: {numbers}")

In [ ]:
distance_matrix=[
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
    ]


In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    data = {}
    data["distance_matrix"] = distance_matrix
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")

def get_routes(data, manager, routing, solution):
    routes = []
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

def main():
    data = create_data_model()
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )
    routing = pywrapcp.RoutingModel(manager)

    
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  
        10000,  
        True,  
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    
    solution = routing.SolveWithParameters(search_parameters)

    
    if solution:
        print_solution(data, manager, routing, solution)
        print("\n\n\n")
        routes = get_routes(data, manager, routing, solution)
        print(routes)
    else:
        print("No solution found !")


if __name__ == "__main__":
    main()

In [ ]:
#!/usr/bin/env python3
# Copyright 2010-2024 Google LLC
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [START program]
"""Simple Vehicles Routing Problem."""

# [START import]
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]


# [START data_model]
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
        # fmt: on
    ]
    data["num_vehicles"] = 1
    # [START starts_ends]
    data["starts"] = [1]
    data["ends"] = [1]
    # [END starts_ends]
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model()
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["starts"], data["ends"]
    )
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)
    # [END routing_model]

    # Create and register a transit callback.
    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Add Distance constraint.
    # [START distance_constraint]
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        1000000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    # [END distance_constraint]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        print_solution(data, manager, routing, solution)
    # [END print_solution]
    else:
      print("No Solution Found!")


if __name__ == "__main__":
    main()
    # [END program]